In [6]:
"""Practice notebook using https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial to learn Word2Vec and additional word processing
techniques""" 


import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing
from spacy.lang.en import English

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

#word2vec stuff here
import multiprocessing

from gensim.models import Word2Vec

from gensim.models.phrases import Phrases, Phraser  #for bigrams/common phrase ID


def cleaning(doc):
    #removes stopwords, easy to modifiy and lemmatize
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]      #easy enough to stop lemmatizing by removing .lemma_ from token.lemma_
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)



In [9]:

t = time()
rough_df = pd.read_csv(r"C:\Users\bowri\square1\square2\dialogue-lines-of-the-simpsons\simpsons_dataset.csv")
print("size of dataset: ", rough_df.shape)  #prints size of matrix
#rough_data.head()
rough_df.isnull().sum()  #sum of null values, from pandas 

rough_df = rough_df.dropna().reset_index(drop=True) #removes null values from dataframe
rough_df.isnull().sum()

nlp = spacy.load('en_core_web_sm', disable=["parser", "ner"])
#nlp = spacy.load("en_core_web_sm", disable=["parser", "tagger"]) r"C:\Users\bowri\Anaconda3\Lib\site-packages\en_core_web_sm
#nlp = spacy.load('en_core_web_sm')

brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in rough_df['spoken_words']) #removing non-alphabetical char w REGEX




txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

#everything in list at this point, txt

df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

#now in a clean dataframe with dropped duplibcations

sent = [row.split() for row in df_clean['clean']]              #to ID common phrases? idk about benefit here, 
phrases = Phrases(sent, min_count=30, progress_per=10000)   #also sent is a list here 

bigram = Phraser(phrases)
sentences = bigram[sent]
print("sentences is type ", type(sentences))


size of dataset:  (158314, 2)
Time to clean up everything: 1.19 mins


INFO - 10:51:33: collecting all words and their counts
INFO - 10:51:33: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 10:51:33: PROGRESS: at sentence #10000, processed 63561 words and 52816 word types
INFO - 10:51:34: PROGRESS: at sentence #20000, processed 130943 words and 99866 word types
INFO - 10:51:34: PROGRESS: at sentence #30000, processed 192972 words and 138532 word types
INFO - 10:51:34: PROGRESS: at sentence #40000, processed 249842 words and 172659 word types
INFO - 10:51:34: PROGRESS: at sentence #50000, processed 311265 words and 208566 word types
INFO - 10:51:34: PROGRESS: at sentence #60000, processed 373588 words and 243702 word types
INFO - 10:51:34: PROGRESS: at sentence #70000, processed 436441 words and 278740 word types
INFO - 10:51:34: PROGRESS: at sentence #80000, processed 497829 words and 311886 word types
INFO - 10:51:34: collected 330804 word types from a corpus of 537160 words (unigram + bigrams) and 85964 sentences
INFO - 10:51:34: us

sentences is type  <class 'gensim.interfaces.TransformedCorpus'>


In [13]:
#Word to Vec stuff begins here full
cores = multiprocessing.cpu_count() # Count the number of cores in a computer, it's 8

#1) INITALIZING THE MODEL
w2v_model = Word2Vec(min_count=20,         #Ignores all words with total absolute frequency lower than this - (2, 100)
                     window=2,           #The maximum distance between the current and predicted word within a sentence.
                     size = 300,           #size of feature vectors
                     sample=6e-5,   #threshold for configuring which higher-frequency words are randomly downsampled. influencial. (0, 1e-5)
                     alpha=0.03,        #learning rate initial (0.01 - 0.05)
                     min_alpha=0.0007,     #final learning rate, linearly drops to this as training goes
                     negative=20,  # If > 0,  how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
                     workers=cores-2)        #how many cores doing the work


#2) BUILDING VOCAB

t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

#3) TRAINING MODEL

#    total_examples = int, Count of sentences;
 #   epochs = int - Number of iterations (epochs) over the corpus - [10, 20, 30]
    
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))


w2v_model.init_sims(replace=True)  #do this when done training the model?

INFO - 11:22:22: collecting all words and their counts
INFO - 11:22:22: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 11:22:22: PROGRESS: at sentence #10000, processed 61718 words, keeping 9558 word types
INFO - 11:22:22: PROGRESS: at sentence #20000, processed 127351 words, keeping 14506 word types
INFO - 11:22:23: PROGRESS: at sentence #30000, processed 187829 words, keeping 17619 word types
INFO - 11:22:23: PROGRESS: at sentence #40000, processed 243332 words, keeping 20385 word types
INFO - 11:22:23: PROGRESS: at sentence #50000, processed 303182 words, keeping 22878 word types
INFO - 11:22:23: PROGRESS: at sentence #60000, processed 363940 words, keeping 25200 word types
INFO - 11:22:23: PROGRESS: at sentence #70000, processed 425408 words, keeping 27401 word types
INFO - 11:22:24: PROGRESS: at sentence #80000, processed 485464 words, keeping 29275 word types
INFO - 11:22:24: collected 30178 word types from a corpus of 523700 raw words and 85964 sentence

Time to build vocab: 0.03 mins


INFO - 11:22:24: training model with 6 workers on 3319 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 11:22:25: EPOCH 1 - PROGRESS: at 53.70% examples, 103594 words/s, in_qsize 0, out_qsize 0
INFO - 11:22:26: worker thread finished; awaiting finish of 5 more threads
INFO - 11:22:26: worker thread finished; awaiting finish of 4 more threads
INFO - 11:22:26: worker thread finished; awaiting finish of 3 more threads
INFO - 11:22:26: worker thread finished; awaiting finish of 2 more threads
INFO - 11:22:26: worker thread finished; awaiting finish of 1 more threads
INFO - 11:22:26: worker thread finished; awaiting finish of 0 more threads
INFO - 11:22:26: EPOCH - 1 : training on 523700 raw words (198820 effective words) took 1.9s, 102725 effective words/s
INFO - 11:22:27: EPOCH 2 - PROGRESS: at 43.67% examples, 84641 words/s, in_qsize 0, out_qsize 0
INFO - 11:22:28: EPOCH 2 - PROGRESS: at 97.45% examples, 95585 words/s, in_qsize 0, out_qsize 0
INFO - 1

Time to train the model: 1.14 mins


INFO - 11:23:32: precomputing L2-norms of word weight vectors


In [14]:
#testing
w2v_model.wv.most_similar(positive=["homer"])

[('bongo', 0.7788041830062866),
 ('marge', 0.7733016014099121),
 ('gee', 0.7590972185134888),
 ('crummy', 0.753426194190979),
 ('wife', 0.7501407861709595),
 ('rude', 0.7463955879211426),
 ('snuggle', 0.7443416714668274),
 ('sweetheart', 0.7426704168319702),
 ('sorry', 0.7412137985229492),
 ('worry', 0.7329186201095581)]

In [9]:
"""This program takes in a single list which contains all of the words in a book, it then 
creates a word2vec model, as well as its vocab, and trains it on the given book/list.
It also takes in a variety of default variables to initalize the word2vec model."""


import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import spacy  # For preprocessing
from spacy.lang.en import English  #not sure what this is for, check if needed
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

#word2vec stuff here
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser  #for bigrams/common phrase ID
#import json_read_clean as jRC 

#TO-DO:
#2) document
#3) Test various parameters for word2vec functions
#4) maybe write it such that it can take in a list of lists, each list being all the words in a book
#5) TF-IDF
#6) Maybe make it return a word2vec model that then can be probed? for instance to get cosine similarity?


def word2vec_book_model(list_of_book_words, min_count=2, window = 2, size = 300, sample = 6e-5, alpha=0.03
                       ,min_alpha = 0.0007, negative = 0, workers = 6):

    phrases = Phrases(list_of_book_words, min_count=30, progress_per=10000)   #also sent is a list here 
    bigram = Phraser(phrases)
    sentences = bigram[list_of_book_words]
    
    cores = multiprocessing.cpu_count() # Count the number of cores in a computer, it's 8

#1) INITALIZING THE MODEL
    w2v_model = Word2Vec(min_count = min_count,         #Ignores all words with total absolute frequency lower than this - (2, 100)
                     window = window,           #The maximum distance between the current and predicted word within a sentence.
                     size = size,           #size of feature vectors
                     sample = sample,   #threshold for configuring which higher-frequency words are randomly downsampled. influencial. (0, 1e-5)
                     alpha = alpha,        #learning rate initial (0.01 - 0.05)
                     min_alpha = min_alpha,     #final learning rate, linearly drops to this as training goes
                     negative = negative,  # If > 0,  how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
                     workers = workers)        #how many cores doing the work

#2) BUILDING VOCAB
    t = time()
    w2v_model.build_vocab(sentences, progress_per=10000)
    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
    
#3) TRAINING MODEL

    #    total_examples = int, Count of sentences;
     #   epochs = int - Number of iterations (epochs) over the corpus - [10, 20, 30]
        #total_examples=w2v_model.corpus_count
    
    t = time()
    w2v_model.train(sentences, total_examples = len(sentences), epochs=30, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))


    w2v_model.init_sims(replace=True)  #do this when done training the model? normalizes the length of the vectors before comparing
    
    print("the size of the vocabulary is: ", len(w2v_model.wv.vocab))
    print(w2v_model.wv.most_similar(positive=["polymer"]))
    print("cosine similarity between polymer and plastic: ", w2v_model.similarity("polymer", "plastic"))
    print("size of the corpus_count:", w2v_model.corpus_count)  #
    return w2v_model



In [24]:
test_schaller_list = ['thumbnail', 'space-filling', 'model', 'section', 'polyethylene', 'terephthalate', 'polymer', 'also', 'known', 'pet', 'pete', 'polyester', 'used', 'plastic', 'bottles', 'color', 'code', 'carbon', 'c', '(black)', 'hydrogen', 'h', '(white)', 'oxygen', 'o', '(red)', 'image', 'used', 'permission', '(public', 'domain', 'jynto)', 'the', 'first', 'synthetic', 'polymer', 'really', 'seize', 'publics', 'consciousness', '--', 'first', 'one', 'change', 'course', 'world', 'events', '--', 'nylon', 'not', 'long', 'discovery', '1930s', 'supplanted', 'silk', 'material', 'choice', 'stockings', 'transformed', 'luxury', 'item', 'widely', 'available', 'commodity', 'nylon', 'came', 'time', 'second', 'world', 'war', 'soon', 'parachutes', 'went', 'made', 'silk', 'fabricated', 'nylon', 'the', 'development', 'nylon', 'took', 'place', 'du', 'ponts', 'experimental', 'research', 'station', 'delaware', 'for', 'much', 'twentieth', 'century', 'du', 'pont', 'large', 'companies', 'relied', 'pursuit', 'basic', 'science', 'engine', 'future', 'development', 'knowledge', 'infrastructure', 'made', 'future', 'products', 'possible', 'scientists', 'asked', 'explore', 'unknown', 'applications', 'work', 'would', 'follow', 'naturally', 'when', 'wallace', 'carrothers', 'du', 'pont', 'developed', 'nylon', 'trying', 'explore', 'ideas', 'proposed', 'hermann', 'staudinger', 'eth', 'zurich', 'these', 'ideas', 'initially', 'controversial', 'held', 'many', 'materials', 'around', 'us', 'including', 'things', 'like', 'cotton', 'spider', 'silk', 'made', 'small', 'molecular', 'units', 'covalently', 'bonded', 'these', 'little', 'molecules', 'monomers', 'bind', 'together', 'form', 'immensely', 'long', 'chains', 'in', 'monomers', 'lose', 'individual', 'identities', 'simply', 'become', 'repeating', 'units', 'long', 'chain', 'they', 'come', 'together', 'form', 'polymers', 'nylon', 'particular', 'example', 'polyamide', 'it', 'composed', 'long', 'chains', 'contain', 'regularly', 'repeating', 'amide', 'bonds', 'n-co', 'thats', 'motif', 'found', 'proteins', 'biology', 'pattern', 'found', 'spiders', 'silk', 'composed', 'protein', 'conceptually', 'taught', 'biology', 'class', 'proteins', 'made', 'amino', 'acids', 'an', 'amide', 'bond', '(or', 'peptide', 'bond)', 'occurs', 'nitrogen', 'amine', 'brought', 'together', 'carbon', 'carboxylic', 'acid', 'a', 'molecule', 'water', 'lost', 'nitrogen', 'takes', 'place', 'oxygen', 'carboxylic', 'acid', 'forming', 'amide', 'instead', 'the', 'loss', 'water', 'leads', 'term', 'condensation', 'reaction', 'early', 'studies', 'reactions', 'led', 'water', 'forming', 'droplets', 'laboratory', 'glassware', 'bubbled', 'reaction', 'in', 'practice', 'industrial', 'nylon', 'production', 'really', 'complicated', 'of', 'course', 'old', 'amide', 'become', 'protein', 'what', 'makes', 'amino', 'acids', 'special', 'fact', 'difunctional', 'they', 'contain', 'one', 'functional', 'group', '(an', 'amine', 'say', 'carboxylic', 'acid)', 'two', 'so', 'amine', 'group', 'amino', 'acid', 'bonds', 'carbonyl', 'neighbour', 'carboxyl', 'group', 'bond', 'amine', 'different', 'neighbour', 'its', 'like', 'two', 'hands', 'hold', 'onto', 'friend', 'hand', 'friends', 'hold', 'onto', 'another', 'forming', 'chain', 'this', 'functionality', 'key', 'part', 'polymers', 'form', 'because', 'difunctional', 'molecules', 'form', 'bonds', 'two', 'directions', 'simple', 'coupling', 'reaction', '(an', 'amine', 'plus', 'carboxylic', 'acid', 'making', 'amide', 'two', 'small', 'molecules', 'making', 'another', 'small', 'molecule)', 'becomes', 'polymerization', '(many', 'small', 'molecules', 'forming', 'enormous', 'molecule)', 'well', 'nylon', 'isnt', 'made', 'amino', 'acid', 'although', 'coupling', 'reaction', 'similar', 'instead', 'made', 'two', 'different', 'molecules', 'difunctional', 'diamine', 'dicarboxylic', 'acid', 'the', 'diamine', 'bonds', 'neighbouring', 'carbonyl', 'end', 'dicarboxylic', 'acid', 'bonds', 'neighbouring', 'amine', 'end', 'for', 'common', 'material', 'two', 'reactants', 'six', 'carbons', 'long', 'giving', 'rise', 'term', 'nylon', '66.', 'nylon', '66', 'example', 'something', 'called', 'alternating', 'co-polymer', 'it', 'isnt', 'one', 'unit', 'repeating', 'along', 'chain', 'two', 'the', 'two', 'monomers', 'make', 'polymer', 'together', 'we', 'sometimes', 'say', 'monomers', 'enchained', 'become', 'part', 'polymer', 'and', 'course', 'alternate', 'along', 'chain', 'one', 'bind', 'complementary', 'in', 'drawings', 'dashed', 'lines', 'meant', 'suggest', 'continuity', 'pattern', 'shown', 'drawing', 'keeps', 'repeating', 'right', 'left', 'more', 'commonly', 'polymers', 'drawn', 'using', 'parenthetical', 'notation', 'below', 'part', 'shown', 'parentheses', 'keeps', 'repeating', 'if', 'could', 'make', 'stamp', 'picture', 'could', 'construct', 'polymer', 'chain', 'simply', 'stamping', 'image', 'across', 'sheet', 'paper', 'carrothers', 'development', 'nylon', 'du', 'pont', 'led', 'indirectly', 'similar', 'polymer', 'made', 'different', 'way', 'across', 'atlantic', 'ocean', 'paul', 'schlack', 'ig', 'farben', 'germany', 'looking', 'way', 'make', 'similar', 'material', 'would', 'subject', 'du', 'ponts', 'patent', 'his', 'efforts', 'led', 'material', 'called', 'perlon', 'sometimes', 'referred', 'nylon', '6', 'nylon', '6', 'alternating', 'co-polymer', 'like', 'nylon', '66', 'its', 'polymer', 'and', 'isnt', 'made', 'difunctional', 'monomer', 'like', 'nylon', '66', 'proteins', 'instead', 'made', 'cyclic', 'amide', 'sometimes', 'called', 'lactam', 'to', 'polymerize', 'lactam', 'break', 'open', 'linear', 'form', 'lactam', 'monomers', 'end', 'enchained', 'head-to-tail', 'this', 'process', 'called', 'ring-opening', 'polymerization', 'there', 'number', 'things', 'different', 'two', 'ways', 'making', 'nylon', 'one', 'turns', 'whole', 'concept', 'synthesis', 'synthesis', 'act', 'making', 'things', 'its', 'little', 'like', 'cooking', 'you', 'gather', 'ingredients', 'you', 'take', 'right', 'amount', 'right', 'amount', 'you', 'mix', 'together', 'get', 'something', 'wonderful', 'but', 'thats', 'difference', 'making', 'nylon', '66', 'carrothers', 'mixed', 'two', 'different', 'compounds', 'together', 'he', 'poured', 'one', 'compound', 'one', 'bottle', 'another', 'compound', 'another', 'bottle', 'he', 'mixed', 'together', 'got', 'polymer', 'in', 'contrast', 'schlack', 'didnt', 'mix', 'anything', 'together', 'least', 'far', 'seen', 'far', 'it', 'came', 'one', 'bottle', 'so', 'wasnt', 'already', 'nylon', '6', 'bottle', 'if', 'monomers', 'react', 'couldnt', 'gone', 'ahead', 'done', 'bottle', 'ring-opening', 'polymerization', 'least', 'context', 'example', 'chain', 'reaction', 'chain', 'reactions', 'dont', 'happen', 'need', 'jump', 'start', 'to', 'understand', 'appreciate', 'two', 'materials', 'come', 'two', 'different', 'classes', 'polymerization', 'reactions', 'nylon', '66', 'product', 'condensation', 'reaction', 'an', 'amine', 'mixed', 'carboxylic', 'acid', 'water', 'released', 'amide', 'bond', 'formed', 'even', 'pair', 'reactants', 'chosen', 'synthetically', 'efficient', '--', 'say', 'amine', 'acid', 'chloride', 'instead', 'carboxylic', 'acid', '--', 'condensation', 'reaction', 'still', 'results', 'case', 'releasing', 'hydrogen', 'chloride', 'nylon', '6', 'hand', 'product', 'condensation', 'reaction', '--', 'least', 'first', 'glance', 'look', 'carefully', 'monomer', 'polymer', 'count', 'atoms', 'theres', 'nothing', 'missing', 'no', 'molecule', 'water', 'hydrogen', 'chloride', 'anything', 'else', 'released', 'the', 'nitrogen', 'one', 'end', 'chain', 'simply', 'attaches', 'carbonyl', 'next', 'but', 'walk', 'end', 'chain', 'reach', 'last', 'carbonyl', 'find', 'theres', 'nitrogen', 'attached', 'that', 'nitrogen', 'nucleophile', 'bound', 'carbonyl', 'next', 'monomer', 'there', 'something', 'though', 'carbon', 'cant', 'sitting', 'three', 'bonds', 'so', 'must', 'nucleophile', 'added', 'carbonyl', 'springing', 'loose', 'amine', 'added', 'next', 'carbonyl', 'springing', 'next', 'amine', 'there', 'original', 'nucleophile', 'this', 'jump', 'start', 'chain', 'reaction', 'needed', 'a', 'nucleophile', 'added', 'get', 'things', 'going', 'once', 'reacted', 'first', 'monomer', 'amine', 'became', 'nucleophile', 'next', 'monomer', 'ring-opening', 'produced', 'another', 'amine', 'nucleophile', 'because', 'always', 'new', 'nucleophile', 'produced', 'next', 'ring', 'opens', 'reaction', 'keeps', 'going', 'that', 'first', 'nucleophile', 'one', 'got', 'everything', 'started', 'called', 'initiator', 'the', 'need', 'initiator', 'hallmark', 'chain', 'reactions', 'in', 'addition', 'initiator', 'ring-opening', 'polymerizations', 'frequently', 'employ', 'catalysts', 'accelerate', 'reaction', 'condensation', 'polymerizations', 'the', 'catalyst', 'may', 'lewis', 'acid', 'activates', 'carbonyl', 'organocatalyst', 'thing', 'via', 'hydrogen', 'bonding', 'both', 'catalytic', 'approaches', 'also', 'make', 'use', 'nucleophilic', 'catalysis', 'in', 'case', 'nucleophilicity', 'nucleophile', 'may', 'enhanced', 'else', 'temporary', 'nucleophile', 'may', 'add', 'carbonyl', 'replaced', 'nucleophilic', 'group', 'resulting', 'enchainment', 'other', 'polyamides', 'also', 'made', 'ring-opening', 'cyclic', 'amides', 'the', 'approach', 'also', 'used', 'prepare', 'polyesters', 'cyclic', 'esters', 'also', 'called', 'lactones', 'in', 'case', 'reaction', 'sometimes', 'called', 'ring-opening', 'trans-esterification', 'polymerization', 'rotep', 'short', 'polyolefins', 'made', 'olefins,', 'may', 'know', 'alkenes', 'olefin', 'older', 'term', 'alkene', 'still', 'commonly', 'used', 'industry', 'these', 'compounds', 'make', 'significant', 'fraction', 'commercially-used', 'polymers', 'today', 'if', 'think', 'common', 'recyclable', 'plastics', 'polyethylene', '(2', '4', 'depending', 'material', 'made)', 'poly(vinyl', 'chloride)', '(3)', 'polypropylene', '(5)', 'polystyrene', '(6)', 'examples', 'polyolefins', 'polyolefins', 'known', 'time', 'although', 'took', 'century', 'time', 'first', 'documented', 'recognized', 'polymers', 'german', 'chemists', 'mid-1800s', 'aware', 'certain', 'tree', 'resins', 'gave', 'rise', 'hard', 'materials', 'time', 'materials', 'included', 'polystyrene', 'if', 'compare', 'repeating', 'structure', 'polystyrene', 'structure', 'styrene', 'imagine', 'polystyrene', 'made', 'series', 'styrene', 'molecules', 'strung', 'together', 'essentially', 'double', 'bond', 'styrene', 'moved', 'attach', 'next', 'molecule', 'double', 'bond', 'done', 'thing', 'polyolefins', 'formed', 'way', 'wide', 'variety', 'alkenes', 'leading', 'array', 'different', 'materials', 'properties', 'suited', 'unique', 'applications', 'note', 'like', 'ring-opening', 'polymerization', 'olefin', 'polymerization', 'depends', 'molecules', 'reacting', 'molecules', 'like', 'that', 'isnt', 'likely', 'happen', 'need', 'something', 'come', 'along', 'induce', 'reaction', 'olefin', 'polymerization', 'thus', 'another', 'case', 'monomers', 'tied', 'together', 'chain', 'reaction', 'the', 'reaction', 'require', 'initiator', 'get', 'things', 'started', 'that', 'initiator', 'likely', 'become', 'end', 'group', 'hanging', 'one', 'end', 'polymer', 'chain', 'polyolefins', 'usually', 'made', 'olefins', 'tethering', 'one', 'alkene', 'unit', 'next', 'trading', 'pi', 'bond', 'within', 'monomer', 'single', 'bond', 'two', 'repeat', 'units', 'however', 'another', 'approach', 'converts', 'cyclic', 'alkenes', 'polymers', 'this', 'approach', 'reminiscent', 'ring-opening', 'cyclic', 'esters', 'amides', 'for', 'example', 'imagine', 'cyclopentene', 'ring', 'opening', 'double', 'bond', 'reaching', 'join', 'rings', 'either', 'side', 'a', 'series', 'cyclopentene', 'rings', 'joined', 'together', 'row', 'would', 'look', 'something', 'like', 'we', 'would', 'probably', 'draw', 'usual', 'zig-zag', 'conformation', 'this', 'act', 'taking', 'cyclic', 'alkene', 'splitting', 'open', 'double', 'bond', 'knitting', 'together', 'alkenes', 'long', 'chain', 'called', 'ring-opening', 'metathesis', 'polymerization', 'sometimes', 'called', 'romp', 'short', 'it', 'things', 'common', 'ring-opening', 'polymerizations', 'rotep', 'in', 'rotep', 'romp', 'like', 'molecules', 'react', 'together', 'form', 'polymer', 'in', 'rotep', 'would', 'two', 'cyclic', 'esters', 'in', 'romp', 'would', 'two', 'cyclic', 'alkenes', 'that', 'different', 'condensation', 'polymerization', 'required', 'two', 'complementary', 'molecules', 'difunctional', 'amine', 'difunctional', 'acid', 'chloride', 'partly', 'consequence', 'like', 'molecules', 'reacting', 'together', 'rotep', 'romp', 'chain', 'reactions', 'in', 'order', 'get', 'molecules', 'react', 'need', 'initiator', 'the', 'initiator', 'jump-starts', 'reaction', 'other', 'olefin', 'polymerizations', 'followed', 'pattern', 'well', 'alkenes', 'dont', 'normally', 'react', 'together', '(there', 'circumstances', 'neednt', 'get', 'events', 'dont', 'usually', 'anything', 'polymerization)', 'olefin', 'polymerizations', 'general', 'go', 'chain', 'reactions', 'require', 'initiator', 'get', 'started', 'despite', 'similarities', 'rotep', 'romp', 'reactions', 'actually', 'quite', 'different', 'occur', 'regular', 'olefin', 'polymerizations', 'conditions', 'required', 'initiate', 'polymerization', 'unique', 'case', 'romp', 'requires', 'something', 'called', 'olefin', 'metathesis', 'catalyst', 'an', 'olefin', 'metathesis', 'catalyst', 'transition', 'metal', 'compound', 'capable', 'splitting', 'double', 'bond', 'alkene', 'half', 'putting', 'two', 'pieces', 'together', 'alkenes', 'the', 'key', 'part', 'olefin', 'metathesis', 'catalyst', 'metal-carbon', 'double', 'bond', 'that', 'group', 'capable', 'switching', 'ends', 'alkenes', 'around', 'different', 'partners', 'polymers', 'long-chain', 'molecules', 'formed', 'individual', 'molecular', 'building', 'blocks', 'typically', 'building', 'blocks', 'organic', 'molecules', 'held', 'together', 'via', 'covalent', 'bonds', 'what', 'kinds', 'building', 'blocks', 'available', 'the', 'formation', 'coordination', 'compounds', 'one', 'many', 'important', 'aspects', 'inorganic', 'chemistry', 'in', 'coordination', 'compound', 'electron', 'pair', 'donor', 'called', 'ligand', 'shares', 'electron', 'pair', 'metal', 'atom', 'frequently', 'metal', 'atom', 'transition', 'metal', 'commonly', 'transition', 'metal', 'ion', 'examples', 'well', 'for', 'example', 'nitrogen', 'pyridine', 'ring', 'lone', 'pair', 'pyridine', 'act', 'ligand', 'lone', 'pair', 'shared', 'metal', 'center', 'vanadium', 'trichloro(oxo)vanadium', 'the', 'lone', 'pair', 'becomes', 'nitrogen-vanadium', 'bond', 'sometimes', 'bond', 'drawn', 'short', 'arrow', 'lone', 'pair', 'vanadium', 'emphasizing', 'origin', 'often', 'simply', 'drawn', 'line', 'like', 'bond', 'one', 'interesting', 'things', 'metal', 'atoms', 'capacity', 'form', 'variable', 'numbers', 'bonds', 'although', 'palladium', 'dichloride', 'could', 'form', 'coordination', 'complex', 'binding', 'one', 'pyridine', 'ligand', 'also', 'two', 'pyridines', 'in', 'former', 'case', 'would', 'form', 'three-coordinate', 'complex', 'latter', 'case', 'would', 'lead', 'four-coordinate', 'palladium', 'compound', 'remember', 'ability', 'bond', 'two', 'neighbouring', 'groups', 'rather', 'one', 'important', 'feature', 'allow', 'small', 'molecule', 'become', 'enchained', 'polymer', 'the', 'palladium', 'link', 'chain', 'three', 'units', 'long', 'what', 'ligand', 'also', 'capacity', 'bind', 'two', 'things', 'what', 'instead', 'pyridine', 'ligand', '44-bipyridine', 'that', 'molecule', 'could', 'bind', 'palladium', 'atom', 'either', 'end', 'once', 'bpy', '(thats', 'abbreviation', 'bipyridine', 'pronounced', 'bippy)', 'bound', 'palladium', 'atom', 'would', 'still', 'second', 'nitrogen', 'lone', 'pair', 'could', 'use', 'bind', 'another', 'the', 'palladium', 'would', 'free', 'bind', 'second', 'bpy', 'as', 'result', 'two', 'monomeric', 'units', 'able', 'form', 'alternating', 'chain', 'why', 'would', 'people', 'want', 'the', 'general', 'idea', 'exploit', 'properties', 'metal', 'atoms', 'new', 'ways', 'what', 'metals', 'good', 'they', 'good', 'conducting', 'electricity', 'sometimes', 'hold', 'useful', 'magnetic', 'properties', 'if', 'properties', 'incorporated', 'material', 'like', 'organic', 'polymers', 'tend', 'inexpensive', 'lightweight', 'maybe', 'used', 'make', 'materials', 'sorts', 'applications', 'everyday', 'life', 'metals', 'also', 'good', 'catalysts', 'wide', 'range', 'reactions', 'a', 'coordination', 'polymer', 'serves', 'space', 'metal', 'atoms', 'evenly', 'within', 'structure', 'lot', 'surface', 'area', 'could', 'promote', 'catalytic', 'efficiency', 'all', 'potential', 'uses', 'driven', 'great', 'deal', 'research', 'coordination', 'polymers', 'recent', 'years', '(as', 'well', 'related', 'metal-organic', 'frameworks)', 'now', 'bonds', 'along', 'metal-ligand', 'chain', 'really', 'hold', 'together', 'form', 'polymer', 'one', 'key', 'difference', 'covalent', 'bonds', 'seen', 'organic', 'polymers', 'dative', 'bonds', 'dative', 'bonds', 'reversible', 'there', 'always', 'equilibrium', 'metal-ligand', 'complex', 'free', 'metal', 'ligand', 'of', 'course', 'bond', 'might', 'strong', 'case', 'equilibrium', 'lies', 'toward', 'metal-ligand', 'complex', 'in', 'words', 'large', 'fraction', 'material', 'would', 'form', 'metal-ligand', 'complex', 'how', 'large', 'fraction', 'the', 'real', 'value', 'isnt', 'important', 'moment', 'for', 'purposes', 'lets', 'say', 'put', 'metal', 'ligand', 'together', '90', 'molecules', 'form', 'complex', 'and', 'suppose', 'thats', 'also', 'true', 'next', 'step', 'bringing', 'another', 'ligand', 'picture', 'bind', 'metal', 'step', 'bringing', 'another', 'metal', 'bind', 'end', 'one', 'ligands', 'so', 'suppose', 'events', 'leads', '90', 'product', 'formation', 'just', 'getting', 'four-unit', 'chain', 'relies', 'three', 'different', 'equilibria', 'if', 'say', 'step', 'proceeds', '90', 'way', '(which', 'sounds', 'pretty', 'good', 'would', 'happy', '90', 'yield', 'reaction', 'lab)', 'entire', 'three-step', 'process', 'would', 'yield', '090', 'x', '090', 'x', '090', '073', '73', 'product', 'not', 'bad', 'but', 'useful', 'polymer', 'chains', 'might', 'thousand', 'units', 'long', 'case', 'amount', 'polymer', 'actually', 'formed', 'proper', 'chain', 'length', 'would', '(090)999', '194', 'x', '10-46', '194', 'x', '10-44', ',', 'thats', 'ridiculously', 'small', 'amount', 'now', 'real', 'calculation', 'equilibrium', 'concentration', 'course', 'sophisticated', 'quick', 'exercise', 'underscores', 'important', 'point', 'order', 'form', 'stable', 'coordination', 'polymer', 'appreciable', 'length', 'metal', 'must', 'bind', 'ligand', 'tightly', 'for', 'reason', 'many', 'approaches', 'coordination', 'polymers', 'employed', 'multidentate', 'ligands', 'course', 'bind', 'tightly', 'monodentate', 'ligands', 'for', 'example', 'polymer', 'formed', 'using', 'bidentate', 'binding', 'shown', 'would', 'much', 'stable', 'respect', 'depolymerization', '(ie', 'falling', 'apart', 'monomers', 'again)', 'monodentate', 'example', 'shown', 'earlier', 'so', 'far', 'seen', 'covalent', 'bonds', 'used', 'bind', 'monomers', 'together', 'longer', 'chains', 'forming', 'polymers', 'there', 'also', 'great', 'deal', 'interest', 'recently', 'using', 'intermolecular', 'attractions', 'make', 'similar', 'structures', 'of', 'course', 'intermolecular', 'attractions', 'important', 'forming', 'large', 'organized', 'structures', 'biology', 'think', 'twin', 'helices', 'held', 'together', 'dna', 'secondary', 'higher-order', 'structures', 'proteins', 'clearly', 'intermolecular', 'attractions', 'used', 'form', 'large', 'stable', 'structures', 'if', 'goal', 'form', 'chain', 'analogous', 'chains', 'monomers', 'seen', 'polymers', 'would', 'useful', 'two', 'sites', 'within', 'monomer', 'interact', 'monomers', 'such', 'molecule', 'would', 'able', 'hold', 'hands', 'two', 'partners', 'speak', 'if', 'take', 'hydrogen', 'bonding', 'example', 'strong', 'intermolecular', 'force', 'look', 'molecule', 'ethylene', 'glycol', '(also', 'called', '12-dihydroxyethane)', 'ethylene', 'glycol', 'looks', 'kind', 'like', 'ethanol', '(ch3ch2oh)', 'extra', 'oh', 'group', 'a', 'physical', 'sample', 'ethylene', 'glycol', 'colorless', 'liquid', 'like', 'ethanol', 'noticeable', 'differences', 'two', 'the', 'boiling', 'point', 'ethylene', 'glycol', 'around', '198°c', 'hundred', 'degrees', 'higher', 'ethanol', 'furthermore', 'ethanol', 'isnt', 'viscous', 'pours', 'quite', 'easily', 'tried', 'stir', 'pour', 'ethylene', 'glycol', 'would', 'notice', 'seems', 'much', 'thicker', 'something', 'holding', 'ethylene', 'glycol', 'molecules', 'tightly', 'together', 'it', 'isnt', 'hard', 'imagine', 'formation', 'chain', 'structures', 'involving', 'groups', 'molecules', 'if', 'look', 'diols', 'containing', 'longer', 'carbon', 'chains', 'even', 'easier', 'imagine', 'something', 'looks', 'kind', 'like', 'polymer', 'monomers', 'enchained', 'together', 'via', 'hydrogen', 'bonding', 'interactions', 'hydrogen', 'bonds', 'strong', 'still', 'intermolecular', 'attractions', 'rather', 'true', 'covalent', 'bonds', 'they', 'readily', 'reversible', 'chains', 'formed', 'via', 'interaction', 'diols', 'could', 'depolymerize', 'easily', 'individual', 'diols', 'could', 'readily', 'fall', 'end', 'chain', 'diol', 'chains', 'could', 'easily', 'break', 'somewhere', 'middle', 'what', 'tetraol', 'if', 'two', 'hydroxy', 'groups', 'either', 'end', 'chain', 'chain', 'would', 'much', 'less', 'likely', 'come', 'apart', 'even', 'one', 'hydrogen', 'bond', 'broke', 'one', 'linkages', 'would', 'still', 'second', 'one', 'holding', 'two', 'neighbors', 'together', 'a', 'molecule', 'like', 'could', 'form', 'much', 'stable', 'supramolecular', 'assembly', 'the', 'dynamic', 'nature', 'supramolecular', 'assemblies', 'part', 'makes', 'interesting', 'researchers', 'stable', 'structures', 'easily', 'form', 'one', 'set', 'circumstances', 'gently', 'come', 'apart', 'another', 'could', 'lots', 'applications', 'for', 'example', 'researchers', 'interested', 'developing', 'novel', 'materials', 'liquid', 'crystals', 'might', 'converted', 'one', 'form', 'another', 'using', 'different', 'stimuli', 'others', 'trying', 'develop', 'materials', 'useful', 'medicinal', 'therapeutic', 'applications', 'the', 'principle', 'functionality', 'means', 'almost', 'type', 'organic', 'reaction', 'could', 'potentially', 'used', 'make', 'polymers', 'for', 'instance', 'compound', 'two', 'functional', 'groups', 'kind', 'could', 'undergo', 'reaction', 'two', 'different', 'sites', 'forming', 'new', 'bonds', 'two', 'neighbors', 'the', 'compound', 'thereby', 'becomes', 'enchained', 'trio', 'formerly', 'independent', 'molecules', 'if', 'neighboring', 'molecules', 'also', 'difunctional', 'pattern', 'repeat', 'forming', 'polymer', 'take', 'diels', 'alder', 'reaction', 'illustrative', 'example', 'on', 'paper', 'reaction', 'fairly', 'straightforward', 'even', 'isnt', 'common', 'a', 'molecule', 'pair', 'conjugated', 'alkenes', 'diene', 'cyclizes', 'another', 'alkene', 'dienophile', 'form', 'new', 'six-membered', 'ring', 'cyclohexene', 'the', 'diene', 'needs', 'two', 'double', 'bonds', 'dienophile', 'needs', 'one', 'nevertheless', 'happens', 'dienophile', 'extra', 'double', 'bond', 'the', 'diels', 'alder', 'adduct', 'forms', 'would', 'end', 'two', 'double', 'bonds', 'one', 'formed', 'original', 'diene', 'always', 'leftover', 'dienophile', 'the', 'resulting', 'compound', 'difunctional', 'alkene', 'each', 'end', 'molecule', 'could', 'potentially', 'undergo', 'another', 'diels', 'alder', 'reaction', 'additional', 'diene', 'each', 'time', 'reaction', 'occurs', 'new', 'alkene', 'left', 'behind', 'diene', 'used', 'preserving', 'difunctionality', 'another', 'step', 'that', 'difunctionality', 'forms', 'basis', 'potential', 'polymer', 'chemistry', 'in', 'fact', 'diels', 'alder', 'reactions', 'exploited', 'researchers', 'number', 'ways', 'make', 'materials', 'useful', 'properties', 'the', 'fact', 'six-membered', 'rings', 'introduced', 'along', 'backbone', 'rather', 'chain', 'single', 'bonds', 'means', 'materials', 'display', 'varying', 'levels', 'conformational', 'rigidity', 'resulting', 'unique', 'properties', 'polymers', 'large', 'molecules', 'made', 'smaller', 'ones', 'how', 'smaller', 'units', 'arranged', 'within', 'polymer', 'issue', 'havent', 'addressed', 'closely', 'yet', 'topology', 'study', 'three-dimensional', 'shapes', 'relationships', 'individual', 'parts', 'arranged', 'within', 'whole', 'lets', 'take', 'look', 'topology', 'polymers', 'at', 'simplest', 'level', 'thinking', 'polymers', 'chains', 'monomers', 'strung', 'together', 'like', 'beads', 'string', 'we', 'even', 'use', 'verb', 'enchain', 'describe', 'act', 'taking', 'monomer', 'tying', 'larger', 'polymer', 'the', 'chain', 'basic', 'polymer', 'structures', 'this', 'topology', 'often', 'referred', 'simply', 'chain', 'underscore', 'structure', 'linear', 'polymer', 'even', 'simple', 'chain', 'context', 'additional', 'structural', 'features', 'worth', 'considering', 'these', 'features', 'connected', 'polymer', 'chain', 'grows', 'individual', 'monomers', 'if', 'polymer', 'results', 'chain', 'reaction', 'typically', 'used', 'polymerization', 'alkenes', 'growing', 'chain', 'usually', 'two', 'distinct', 'ends', 'one', 'end', 'sometimes', 'called', 'tail', 'site', 'first', 'monomer', 'incorporated', 'polymer', 'well', 'remnant', 'initiator', 'begin', 'polymerization', 'process', 'the', 'end', 'sometimes', 'called', 'head', 'commonly', 'growing', 'end', 'active', 'site', 'place', 'new', 'monomers', 'enchained', 'polymer', 'thats', 'happens', 'cases', 'initiator', 'capable', 'reacting', 'one', 'monomer', 'starting', 'chain', 'reaction', 'however', 'sometimes', 'initiators', 'difunctional', 'difunctional', 'initiators', 'capable', 'getting', 'two', 'monomers', 'start', 'growing', 'polymer', 'chain', 'one', 'direction', 'in', 'case', 'initiator', 'fragment', 'left', 'behind', 'middle', 'new', 'growing', 'chain', 'grows', 'outward', 'middles', 'both', 'ends', 'chain', 'growing', 'ends', 'in', 'type', 'growth', 'chain', 'referred', 'telechelic.', 'sometimes', 'polymer', 'chains', 'simple', 'linear', 'structures', 'instead', 'chains', 'branch', 'this', 'topology', 'called', 'branched', 'instead', 'looking', 'like', 'snake', 'piece', 'spaghetti', 'structure', 'looks', 'like', 'length', 'seaweed', 'branching', 'sometimes', 'artifact', 'polymer', 'made', 'sometimes', 'monomers', 'may', 'lead', 'linear', 'polymer', 'branched', 'one', 'the', 'prime', 'example', 'polyethylene', 'form', 'high-density', 'polyethylene', '(hdpe)', 'low-density', 'polyethylene', '(ldpe)', 'depending', 'conditions', 'ethylene', 'enchained.', 'in', 'branched', 'polymer', 'smaller', 'chains', 'grow', 'like', 'limbs', 'along', 'tree', 'trunk', 'leaves', 'along', 'stem', 'we', 'take', 'arrangement', 'step', 'something', 'looks', 'like', 'net', 'in', 'call', 'crosslinked', 'structure', 'branches', 'connect', 'one', 'main', 'chain', 'next', 'tying', 'together', 'one', 'big', 'piece', 'sometimes', 'type', 'structure', 'called', 'thermoset', 'the', 'word', 'thermoset', 'really', 'description', 'physical', 'property', 'polymer', 'contrast', 'thermoplastic', 'a', 'thermoplastic', 'polymer', 'melted', 'reformed', 'new', 'shapes', 'polymerization', 'in', 'contrast', 'thermoset', 'polymerized', 'retains', 'shape', 'even', 'heated', 'doesnt', 'melt', 'these', 'terms', 'connotations', 'topology', 'material', 'nonetheless.', 'the', 'reason', 'thermoplastics', 'melted', 'formed', 'new', 'shapes', 'made', 'separate', 'molecules', 'the', 'molecules', 'may', 'long', 'may', 'even', 'branched', 'high', 'enough', 'temperatures', 'molecules', 'move', 'completely', 'independently', 'they', 'melt', 'material', 'comprised', 'separate', 'molecules', 'take', 'new', 'shapes', 'in', 'thermoset', 'crosslinks', 'connect', 'different', 'chains', 'material', 'forming', 'bridges', 'span', 'chain', 'chain', 'chain', 'essentially', 'uniting', 'material', 'one', 'big', 'molecule', 'if', 'one', 'big', 'molecule', 'chains', 'never', 'move', 'completely', 'independently', 'material', 'cant', 'form', 'new', 'shape', 'looked', 'another', 'way', 'crosslinks', 'tie', 'main', 'chains', 'place', 'they', 'may', 'able', 'move', 'around', 'never', 'get', 'far', 'if', 'amount', 'crosslinking', 'sufficient', 'always', 'hold', 'material', 'basic', 'shape', 'of', 'course', 'little', 'bit', 'crosslinking', 'may', 'effect', 'you', 'may', 'two', 'three', 'chains', 'tied', 'together', 'form', 'one', 'big', 'molecule', 'behave', 'like', 'highly', 'branched', 'chains', 'like', 'extended', 'nets', 'its', 'also', 'worth', 'noting', 'term', 'crosslink', 'actually', 'mean', 'different', 'things', 'used', 'different', 'ways', 'sometimes', 'crosslinks', 'refer', 'true', 'covalent', 'bonds', 'connecting', 'two', 'chains', 'together', 'these', 'connections', 'called', 'chemical', 'crosslinks;', 'word', 'chemical', 'refers', 'covalent', 'bonds', 'alternatively', 'chains', 'connected', 'strong', 'intermolecular', 'forces', 'thats', 'thing', 'course', 'intermolecular', 'forces', 'overcome', 'sufficient', 'energy', 'point', 'chains', 'may', 'longer', 'tied', 'together', 'these', 'connections', 'called', 'physical', 'crosslinks', 'distinguish', 'permanent', 'bonds', 'to', 'use', 'biology', 'analogy', 'sulfide', 'bonds', 'proteins', 'would', 'example', 'chemical', 'crosslinks', 'hold', 'protein', 'firmly', 'one', 'shape', 'chemical', 'reaction', 'required', 'break', 'connection', 'the', 'ordinary', 'hydrogen', 'bonds', 'prevalent', 'protein', 'physical', 'crosslinks', 'because', 'overcome', 'adding', 'heat', 'proteins', 'sensitive', 'temperature', 'dendrimers', 'another', 'type', 'branched', 'structure', 'term', 'comes', 'greek', 'dendron', 'tree', 'dendrimers', 'differ', 'regular', 'branched', 'polymers', 'much', 'regular', 'branching', 'pattern', 'a', 'dendrimer', 'actually', 'grow', 'outward', 'center', 'branching', 'regular', 'intervals', 'many', 'dendrimers', 'polyamides', 'although', 'types', 'well', 'generally', 'least', 'one', 'monomers', 'trifunctional', 'introduces', 'branching', 'predictable', 'way', 'as', 'dendrimers', 'grow', 'outward', 'layer', 'layer', 'often', 'described', 'terms', 'generations', 'suppose', 'start', 'trifunctional', 'monomer', 'middle', 'thats', 'generation', '0', 'if', 'monomer', 'polymerized', 'outward', 'another', 'set', 'trifunctional', 'monomers', 'attached', 'edge', 'first', 'generation', 'dendrimer', 'if', 'keep', 'going', 'add', 'another', 'layer', 'trifunctional', 'monomers', 'second', 'generation', 'dendrimer', 'these', 'dendrimers', 'start', 'look', 'circular', 'paper', 'like', 'pancakes', 'steric', 'interactions', 'groups', 'forces', 'things', 'three', 'dimensions', 'as', 'result', 'dendrimers', 'roughly', 'spherical', 'shape', 'in', 'examples', 'polymers', 'seen', 'chain', 'actually', 'composed', 'two', 'different', 'monomers', 'thats', 'true', 'case', 'polyamides', 'nylon-66', 'in', 'example', 'chain', 'composed', 'difunctional', 'amines', 'alternating', 'difunctional', 'carboxyloids', '(such', 'carboxylic', 'acids', 'acid', 'chlorides)', 'we', 'think', 'polymer', 'like', 'composed', 'two', 'different', 'monomers', 'of', 'course', 'complementary', 'reactivity', 'alternate', 'amine', 'carboxyloid', 'form', 'amide', 'we', 'think', 'polymers', 'co-polymers,', 'meaning', 'formed', 'one', 'kind', 'monomer', 'we', 'go', 'say', 'alternating', 'co-polymers', 'two', 'different', 'monomers', 'alternate', 'along', 'chain', 'in', 'cases', 'need', 'two', 'different', 'monomers', 'alternate', 'way', 'nylon-66', 'if', 'take', 'mixture', 'alkenes', 'capable', 'forming', 'polymers', 'polymerize', 'together', 'may', 'well', 'get', 'randomly', 'enchained', 'growing', 'polymer', 'this', 'arrangement', 'called', 'random', 'co-polymer', 'sometimes', 'statistical', 'co-polymer', 'for', 'example', 'maybe', 'manage', 'get', 'random', 'sequence', 'propene', 'vinyl', 'chloride', 'units', 'along', 'polymer.', 'notice', 'example', 'still', 'zig-zag', 'main', 'chain', 'polymer', 'changes', 'one', 'monomer', 'group', 'attached', 'main', 'chain', 'it', 'sometimes', 'useful', 'think', 'main', 'chain', 'backbone', 'separately', 'attached', 'pendant', 'groups', 'in', 'case', 'looks', 'like', 'polyethylene', 'backbone', 'pendant', 'chlorides', 'methyls', 'randomly', 'attached.', 'a', 'random', 'arrangement', 'isnt', 'possibility', 'maybe', 'vinyl', 'chlorides', 'polymerized', 'row', 'propenes', 'incorporated', 'the', 'result', 'would', 'diblock', 'co-polymer', 'solid', 'block', 'poly(vinyl', 'chloride)', 'one', 'end', 'chain', 'solid', 'block', 'polypropylene', 'end', 'there', 'couple', 'ways', 'could', 'happen', 'maybe', 'waited', 'vinyl', 'chlorides', 'enchained', 'adding', 'propene', 'vinyl', 'chlorides', 'enchained', 'tail', 'end', 'propenes', 'added', 'growing', 'end', 'later', 'alternatively', 'maybe', 'added', 'vinyl', 'chlorides', 'underwent', 'polymerization', 'whole', 'lot', 'faster', 'propenes', 'vinyl', 'chlorides', 'became', 'enchained', 'propenes', 'chance', 'those', 'possibilities', 'assume', 'monomers', 'enchained', 'linearly', 'there', 'possibilities', 'maybe', 'one', 'set', 'monomers', 'form', 'polymer', 'backbone', 'set', 'form', 'pendant', 'branches', 'along', 'chain', 'that', 'arrangement', 'called', 'graft', 'co-polymer,', 'though', 'little', 'apple', 'trees', 'grafted', 'onto', 'trunk', 'another', 'breed', 'problem', 'mp11', 'problem', 'mp12', 'problem', 'mp13', 'problem', 'mp21', 'problem', 'mp22', 'problem', 'mp23', 'problem', 'mp31', 'problem', 'mp32', 'problem', 'mp41', 'problem', 'mp42', 'problem', 'mp43', 'problem', 'mp51', 'problem', 'mp52', 'problem', 'mp53', 'problem', 'mp61', 'problem', 'mp71', 'problem', 'mp72', 'molecular', 'weight', 'one', 'central', 'aspects', 'polymer', 'properties', 'of', 'course', 'molecules', 'molecular', 'weights', 'it', 'might', 'seem', 'obvious', 'molecular', 'weight', 'essential', 'property', 'molecular', 'compound', 'in', 'polymers', 'molecular', 'weight', 'takes', 'added', 'significance', 'thats', 'polymer', 'large', 'molecule', 'made', 'repeating', 'units', 'many', 'repeating', 'units', 'thirty', 'a', 'thousand', 'a', 'million', 'any', 'possibilities', 'might', 'still', 'considered', 'representative', 'material', 'molecular', 'weights', 'would', 'different', 'would', 'properties', 'that', 'variation', 'introduces', 'unique', 'aspects', 'polymer', 'molecular', 'weight', 'because', 'polymers', 'assembled', 'smaller', 'molecules', 'length', '(and', 'consequently', 'molecular', 'weight)', 'polymer', 'chain', 'depends', 'number', 'monomers', 'enchained', 'polymer', 'the', 'number', 'enchained', 'monomers', 'average', 'polymer', 'chain', 'material', 'called', 'degree', 'polymerization', '(dp)', 'notice', 'key', 'point', 'average', 'in', 'given', 'material', 'chains', 'added', 'monomers', 'chains', 'added', 'fewer', 'why', 'difference', 'first', 'polymer', 'growth', 'dynamic', 'process', 'it', 'requires', 'monomers', 'come', 'together', 'react', 'what', 'one', 'monomer', 'starts', 'reacting', 'forming', 'growing', 'chain', 'others', 'get', 'started', 'with', 'head', 'start', 'chain', 'become', 'longer', 'rest', 'what', 'something', 'goes', 'wrong', 'one', 'growing', 'chains', 'longer', 'add', 'new', 'monomers', 'that', 'chain', 'experienced', 'early', 'death', 'never', 'grow', 'long', 'others', 'as', 'result', 'speaking', 'molecular', 'weight', 'polymer', 'always', 'talking', 'average', 'value', 'some', 'chains', 'material', 'longer', '(and', 'heavier)', 'chains', 'material', 'shorter', '(and', 'lighter)', 'as', 'group', 'measurements', 'helpful', 'know', 'widely', 'distributed', 'individual', 'values', 'really', 'in', 'polymer', 'chemistry', 'width', 'distribution', 'molecular', 'weights', 'described', 'dispersity', '(ð', 'also', 'called', 'older', 'texts', 'polydispersity', 'polydispersity', 'index', 'pdi)', 'the', 'dispersity', 'polymer', 'sample', 'often', '1', '2', '(although', 'even', 'higher', '2)', 'the', 'closer', '1', 'narrower', 'distribution', 'that', 'dispersity', '10', 'would', 'mean', 'chains', 'sample', 'exactly', 'length', 'molecular', 'weight', 'the', 'original', 'idea', 'dispersity', 'based', 'alternative', 'methods', 'measuring', 'molecular', 'weight', '(or', 'chain', 'length)', 'polymer', 'sample', 'one', 'set', 'methods', 'gave', 'something', 'called', 'number', 'average', 'molecular', 'weight', '(symbol', 'mn)', 'these', 'methods', 'essentially', 'took', 'weight', 'sample', 'counted', 'molecules', 'sample', 'therefore', 'found', 'average', 'weight', 'molecule', 'sample', 'a', 'classic', 'example', 'approach', 'colligative', 'properties', 'experiment', 'freezing', 'point', 'depression', 'you', 'know', 'impurities', 'liquid', 'tend', 'disrupt', 'intermolecular', 'interactions', 'lower', 'freezing', 'point', 'liquid', 'you', 'may', 'also', 'know', 'amount', 'freezing', 'gets', 'lowered', 'depends', 'number', 'molecules', 'ions', 'get', 'dissolved', 'hence', 'weigh', 'sample', 'polymer', 'dissolve', 'solvent', 'measure', 'freezing', 'point', 'could', 'figure', 'number', 'molecules', 'dissolved', 'consequently', 'arrive', 'mn', 'thats', 'easy', 'practice', 'freezing', 'point', 'depressions', 'small', 'theyre', 'used', 'often', 'anymore', 'a', 'common', 'example', 'kind', 'measurement', 'widely', 'used', 'determine', 'mn', 'today', 'end', 'group', 'analysis', 'in', 'end', 'group', 'analysis', 'use', '1h', 'nmr', 'measurements', 'determine', 'ratio', 'specific', 'proton', 'repeat', 'units', 'specific', 'proton', 'end', 'group', 'remember', 'end', 'group', 'might', 'something', 'like', 'initiator', 'added', 'onto', 'first', 'monomer', 'get', 'polymerization', 'going', 'by', 'end', 'polymerization', 'still', 'found', 'end', 'polymer', 'chain', 'end', 'group', 'there', 'one', 'per', 'chain', 'whereas', 'lots', 'monomers', 'enchained', 'polymer', 'ratio', 'enchained', 'monomers', 'end', 'group', 'tells', 'us', 'long', 'chain', 'the', 'set', 'methods', 'upon', 'dispersity', 'based', 'gave', 'something', 'called', 'weight', 'average', 'molecular', 'weight', '(symbol', 'mw)', 'the', 'classic', 'example', 'light-scattering', 'experiment', 'in', 'experiment', 'solution', 'polymer', 'exposed', 'beam', 'light', 'resulting', 'scattered', 'light', '--', 'coming', 'sample', 'different', 'directions', '--', 'analyzed', 'determine', 'size', 'polymer', 'chains', 'solution', 'the', 'results', 'heavily', 'influenced', 'larger', 'molecules', 'solution', 'as', 'result', 'measurement', 'molecular', 'weight', 'always', 'higher', 'measurements', 'based', 'counting', 'every', 'single', 'molecule', 'the', 'resulting', 'ratio', 'ð', 'mw', 'mn', 'became', 'known', 'polydispersity', 'index', 'recently', 'dispersity', 'because', 'mw', 'always', 'strongly', 'influenced', 'longer', 'chains', 'little', 'bigger', 'mn', 'therefore', 'dispersity', 'always', 'bigger', '10', 'nowadays', 'molecular', 'weight', 'dispersity', 'commonly', 'measured', 'using', 'gel', 'permeation', 'chromatography', '(gpc)', 'synonymous', 'size-exclusion', 'chromatography', '(sec)', 'this', 'method', 'high-performance', 'liquid', 'chromatography', '(hplc)', 'technique', 'solvent', 'containing', 'sample', 'polymer', 'pumped', 'specialized', 'chromatography', 'column', 'capable', 'separating', 'molecules', 'based', 'size', 'differences', 'as', 'sample', 'emerges', 'column', 'detected', 'recorded', 'most', 'commonly', 'presence', 'sample', 'solvent', 'emerging', 'column', 'causes', 'slight', 'change', 'refractive', 'index', 'a', 'graph', 'refractive', 'index', 'versus', 'time', 'presents', 'record', 'amount', 'sample', 'emerging', 'column', 'given', 'time', 'because', 'column', 'separated', 'molecules', 'based', 'size', 'time', 'axis', 'corresponds', 'indirectly', 'chain', 'length', 'molecular', 'weight', 'how', 'column', 'separate', 'molecules', 'based', 'size', 'the', 'column', 'packed', 'porous', 'material', 'usually', 'insoluble', 'polymer', 'beads', 'the', 'pore', 'sizes', 'vary', 'these', 'pores', 'crucial', 'separation', 'molecules', 'flowing', 'column', 'may', 'tarry', 'pores', 'smaller', 'molecules', 'could', 'become', 'delayed', 'pores', 'material', 'whereas', 'larger', 'molecules', 'delayed', 'largest', 'pores', 'consequently', 'longer', 'elution', 'time', 'corresponds', 'lower', 'molecular', 'weight', 'if', 'injected', 'series', 'different', 'polymers', 'gpc', 'different', 'molecular', 'weight', 'distribution', 'would', 'observe', 'one', 'eluting', 'different', 'time', 'whats', 'peak', 'may', 'broader', 'narrower', 'depending', 'dispersity', 'particular', 'sample', 'the', 'wider', 'peak', 'gpc', 'broader', 'distribution', 'molecular', 'weights', 'narrower', 'peaks', 'uniform', 'chains', 'normally', 'software', 'package', 'analyzes', 'curve', 'determine', 'dispersity', 'note', 'x', 'axis', 'gpc', 'trace', 'commonly', 'labeled', 'elution', 'time', 'normally', 'runs', 'left', 'right', 'however', 'often', 'x', 'axis', 'labeled', 'molecular', 'wright', 'really', 'quantity', 'interested', 'in', 'fact', 'sometimes', 'axis', 'reversed', 'peaks', 'higher', 'molecular', 'weights', 'appear', 'right', 'feel', 'natural', 'look', 'way', 'you', 'need', 'look', 'carefully', 'data', 'see', 'displayed', 'there', 'problems', 'relying', 'gpc', 'molecular', 'weight', 'measurements', 'the', 'main', 'difficulty', 'polymers', 'solution', 'tend', 'coil', 'balls', 'coils', 'contain', 'greater', 'lesser', 'amounts', 'solvent', 'depending', 'strongly', 'polymer', 'solvent', 'interact', 'if', 'interacts', 'strongly', 'solvent', 'pull', 'lots', 'solvent', 'molecules', 'inside', 'coils', 'the', 'coil', 'get', 'bigger', 'make', 'room', 'internal', 'solvent', 'molecules', 'if', 'doesnt', 'interact', 'strongly', 'solvent', 'mostly', 'stick', 'blocking', 'solvent', 'molecules', 'there', 'broad', 'range', 'behaviors', 'as', 'result', 'different', 'polymers', 'may', 'swell', 'different', 'extents', 'different', 'solvents', 'that', 'matters', 'gpc', 'really', 'using', 'size', 'polymer', 'coil', 'index', 'molecular', 'weight', 'comparing', 'gpc', 'traces', 'two', 'different', 'kinds', 'polymers', 'done', 'caution', 'problem', 'cp11', 'in', 'following', 'cases', 'state', 'polymer', 'higher', 'molecular', 'weight', 'one', 'narrower', 'dispersity', 'problem', 'cp12', 'calculate', 'molecular', 'weight', 'following', 'samples', 'problem', 'cp13', 'use', 'nmr', 'end', 'group', 'analysis', 'determine', 'degrees', 'polymerization', 'following', 'samples', 'polymers', 'come', 'occupy', 'important', 'niche', 'materials', 'use', 'every', 'day', 'what', 'makes', 'unique', 'why', 'properties', 'arent', 'easily', 'replicated', 'materials', 'lets', 'start', 'property', 'probably', 'familiar', 'viscosity', 'term', 'use', 'describe', 'thickness', 'different', 'liquids', 'for', 'example', 'say', 'honey', 'viscous', 'water', 'motor', 'oil', 'viscous', 'gasoline', 'when', 'say', 'mean', 'water', 'much', 'easier', 'stir', 'pour', 'honey', 'the', 'honey', 'moves', 'slowly', 'it', 'resists', 'movement', 'spoon', 'stir', 'viscosity', 'often', 'described', 'general', 'terms', 'resistance', 'flow', 'the', 'honey', 'doesnt', 'flow', 'easily', 'especially', 'compared', 'something', 'like', 'water.', 'now', 'honey', 'isnt', 'polymer', 'its', 'concentrated', 'solution', 'it', 'contains', 'little', 'bit', 'water', 'whole', 'lot', 'sugars', 'plus', 'small', 'molecules', 'produced', 'plants', 'bees', 'gathered', 'nectar', 'make', 'honey', 'the', 'sugars', 'arent', 'polymers', 'either', 'simple', 'monosaccharides', 'glucose', 'fructose', 'nevertheless', 'high', 'viscosity', 'something', 'honey', 'common', 'polymer', 'solutions', 'well', 'oligomers', 'short-chain', 'polymers', 'liquids', 'instead', 'solids', 'why', 'honey', 'viscous', 'partly', 'sugar', 'molecules', 'water', 'much', 'larger', 'water', 'molecules', 'experience', 'lot', 'drag', 'move', 'solution', 'compared', 'plain', 'water', 'the', 'strong', 'intermolecular', 'attractions', 'individual', 'sugar', 'molecules', 'also', 'major', 'factor', 'sugars', 'covered', 'oh', 'hydroxyl', 'groups', 'capable', 'strong', 'hydrogen', 'bonds', 'as', 'molecules', 'move', 'past', 'concentrated', 'solution', 'cling', 'slowing', 'flow', 'liquid', 'the', 'motor', 'oil', 'example', 'works', 'similar', 'way', 'motor', 'oil', 'gasoline', 'composed', 'class', 'compounds', 'hydrocarbons', 'composed', 'carbon', 'chains', 'covered', 'hydrogen', 'atoms', 'theyre', 'similar', 'the', 'intermolecular', 'attractions', 'molecules', 'much', 'weaker', 'sugar', 'molecules', 'theyre', 'london', 'dispersion', 'forces', 'the', 'major', 'difference', 'motor', 'oil', 'gasoline', 'motor', 'oil', 'contains', 'much', 'longer', 'hydrocarbon', 'chains', 'the', 'molecules', 'bigger', 'experience', 'drag', 'move', 'liquid', 'the', 'amount', 'intermolecular', 'interactions', 'also', 'crucial', 'determining', 'tightly', 'two', 'molecules', 'hold', 'onto', 'factor', 'especially', 'important', 'weak', 'london', 'dispersion', 'forces', 'small', 'advantage', 'goes', 'long', 'way', 'those', 'longer', 'hydrocarbon', 'chains', 'motor', 'oil', 'cling', 'much', 'strongly', 'shorter', 'ones', 'gasoline', 'gasoline', 'flows', 'much', 'easily', 'motor', 'oil', 'for', 'similar', 'reasons', 'oligomers', 'polymers', 'liquids', 'also', 'display', 'high', 'viscosity', 'the', 'extended', 'contact', 'long', 'molecular', 'chains', 'leads', 'enhanced', 'intermolecular', 'attractions', 'contribute', 'resistance', 'flow', 'increased', 'drag', 'also', 'factor', 'even', 'solution', 'large', 'molecules', 'encounter', 'resistance', 'move', 'past', 'solvent', 'molecules', 'compared', 'resistance', 'smaller', 'molecules', 'would', 'experience', 'in', 'fact', 'viscosity', 'measurements', 'polymer', 'solutions', 'another', 'way', 'determine', 'size', 'polymer', '--', 'leading', 'chain', 'length', 'molecular', 'weight', 'the', 'larger', 'polymer', 'drag', 'also', 'intermolecular', 'attraction', 'higher', 'viscosity', 'so', 'viscosity', 'increases', 'molecular', 'weight', 'necessarily', 'linear', 'way', 'that', 'make', 'difficult', 'use', 'graph', 'like', 'predict', 'molecular', 'weight', 'viscosity', 'measurement', 'one', 'common', 'thing', 'situation', 'take', 'log', 'values', 'that', 'approach', 'often', 'give', 'straight', 'line', 'two', 'things', 'related', 'often', 'nature', 'one', 'physical', 'property', 'depends', 'another', 'follow', 'power', 'law', 'that', 'means', 'one', 'quantity', 'depends', 'raised', 'exponent.', 'in', 'case', 'relationship', 'described', 'mark-houwink', 'equation', '[η', 'kmα]', 'in', 'equation', 'η', 'intrinsic', 'viscosity', 'thats', 'viscosity', 'attributed', 'solute', 'rather', 'viscosity', 'solvent', 'm', 'molecular', 'weight', 'k', 'α', 'constants', 'particular', 'polymer', 'if', 'take', 'log', 'sides', 'equation', '--', 'thats', 'ok', 'mathematically', 'like', 'could', 'multiply', 'sides', 'equation', 'four', 'subtract', 'three', 'sides', 'still', 'equivalent', 'expression', '--', 'get', 'log[η', 'log(kma)]', 'if', 'take', 'log', 'two', 'things', 'multiplied', 'together', 'adding', 'individual', 'logs', 'things', 'log[η', 'log', 'k', 'log(ma)]', 'and', 'take', 'log', 'something', 'raised', 'exponent', 'exponent', 'comes', 'become', 'coefficient', 'log[η', 'log', 'k', 'αlog', 'm]', 'log[η', 'αlogm', 'log', 'k]', 'that', 'looks', 'like', 'equation', 'straight', 'line', 'mx', 'b', 'here', 'plot', 'logη', 'axis', 'logm', 'x', 'axis', 'get', 'straight', 'line', 'constants', 'α', 'k', 'slope', 'y-intercept', 'respectively', 'this', 'approach', 'yield', 'absolute', 'measurement', 'you', 'couldnt', 'take', 'two', 'completely', 'different', 'polymers', 'take', 'viscosity', 'measurements', 'make', 'conclusions', 'one', 'higher', 'average', 'molecular', 'weight', 'however', 'took', 'series', 'polymers', 'kind', 'measured', 'viscosities', 'solution', 'would', 'able', 'deduce', 'ones', 'longer', 'ones', 'shorter', 'in', 'fact', 'dependence', 'viscosity', 'molecular', 'weight', 'polymers', 'complicated', 'graph', 'suggests', 'if', 'deal', 'large', 'enough', 'polymers', 'graph', 'starts', 'look', 'like', 'one', 'why', 'bend', 'remember', 'polymers', 'solution', 'coiled', 'balls', 'a', 'longer', 'chain', 'corresponds', 'bigger', 'coil', 'at', 'point', 'chains', 'become', 'big', 'enough', 'arent', 'likely', 'stay', 'coiled', 'independently', 'they', 'become', 'likely', 'interact', 'they', 'become', 'entangled', 'there', 'plenty', 'factors', 'would', 'take', 'account', 'using', 'viscosity', 'assess', 'molecular', 'weight', 'one', 'concentration', 'example', 'the', 'concentrated', 'polymer', 'solution', 'large', 'molecules', 'exerting', 'drag', 'interacting', 'higher', 'concentration', 'leads', 'higher', 'viscosity', 'measurement', 'consequently', 'would', 'make', 'sure', 'accounted', 'polymer', 'concentrations', 'used', 'method', 'problem', 'pp21', 'estimate', 'values', 'α', 'k', 'polymer', 'provided', 'plot', 'problem', 'pp22', 'suppose', 'polymer', 'described', 'plot', 'if', 'sample', 'molecular', 'weight', '1', 'million', 'gmol', 'would', 'expect', 'intrinsic', 'viscosity', 'conditions', 'experiment', 'problem', 'pp23', 'suppose', 'calibration', 'plot', 'particular', 'polymer', 'if', 'measure', 'intrinsic', 'viscosity', 'sample', 'find', 'value', '800', 'ml', 'g', 'molecular', 'weight', 'sample', 'problem', 'pp24', 'ethanol', '(ch3ch2oh)', 'absolute', 'viscosity', '1095', 'centipoise', 'whereas', 'ethylene', 'glycol', '(hoch2ch2oh)', 'absolute', 'viscosity', '162', 'centipoise', 'explain', 'two', 'values', 'different', 'viscosity', 'measurements', 'realm', 'field', 'science', 'called', 'rheology', 'rheology', 'literally', 'study', 'flow', 'another', 'simple', 'definition', 'attributed', 'chemical', 'engineer', 'chris', 'macosko', 'university', 'minnesota', 'study', 'happens', 'squish', 'stuff', 'theres', 'element', 'force', 'pressure', 'comes', 'play', 'too.', 'one', 'common', 'ways', 'assessing', 'properties', 'rheology', 'place', 'sample', 'two', 'parallel', 'plates', 'move', 'one', 'plate', 'respect', 'one', 'plate', 'says', 'still', 'one', 'moves', 'what', 'happens', 'liquid', 'plates', 'there', 'ought', 'friction', 'stationary', 'plate', 'liquid', 'keep', 'liquid', 'still', 'there', 'also', 'ought', 'friction', 'moving', 'plate', 'liquid', 'make', 'liquid', 'move', 'along', 'speed', 'plate', 'so', 'one', 'extreme', 'liquid', 'moving', 'along', 'sliding', 'plate', 'extreme', 'liquid', 'perfectly', 'still', 'if', 'imagine', 'liquid', 'two', 'extremes', 'divided', 'thin', 'layers', 'layer', 'must', 'moving', 'slightly', 'different', 'speed', 'next', 'in', 'diagram', 'symbol', 'u', 'stands', 'speed', 'layer', 'liquid', 'the', 'arrow', 'beside', 'layer', 'meant', 'convey', 'relative', 'speed', 'top', 'layer', 'moving', 'fastest', 'next', 'layer', 'little', 'slower', 'bottom', 'layer', 'isnt', 'moving', 'there', 'important', 'quantity', 'called', 'strain', 'rate', 'shear', 'rate', '(given', 'symbol', 'greek', 'letter', 'gamma', 'γ', 'dot', 'top)', 'describes', 'speed', 'liquid', 'moving', 'along', 'x-axis', '(left', 'right)', 'changes', 'layer', 'layer', 'y-direction', 'what', 'picture', 'viscosity', 'well', 'order', 'get', 'top', 'plate', 'move', 'apply', 'sort', 'force', 'in', 'rheology', 'force', 'given', 'per', 'unit', 'area', '(like', 'pressure)', 'force', 'per', 'unit', 'area', 'described', 'stress', '(given', 'symbol', 'greek', 'letter', 'sigma', 'σ)', 'what', 'wanted', 'slide', 'plate', 'even', 'faster', 'what', 'would', 'happen', 'liquid', 'well', 'shear', 'rate', 'would', 'go', 'the', 'top', 'layer', 'would', 'moving', 'even', 'faster', 'bottom', 'layer', 'would', 'still', 'stationary', 'and', 'would', 'get', 'faster', 'shear', 'rate', 'we', 'would', 'push', 'top', 'plate', 'little', 'harder', 'the', 'result', 'graph', 'looks', 'like', 'the', 'graph', 'says', 'shear', 'stress', 'shear', 'rate', 'directly', 'related', 'the', 'stress', 'called', 'shear', 'stress', 'direction', 'force', 'parallel', 'liquid', 'causing', 'shear', 'strain', 'when', 'increase', 'shear', 'rate', 'shear', 'stress', 'also', 'increases', 'proportionally', 'so', 'experiment', 'using', 'water', 'liquid', 'plates', 'would', 'get', 'linear', 'relationship', 'shear', 'stress', 'shear', 'rate', 'thats', 'sir', 'isaac', 'newton', 'described', 'behavior', 'liquids', 'see', 'linear', 'relationship', 'stress', 'strain', 'describe', 'liquid', 'newtonian', 'liquid', 'what', 'happens', 'experiment', 'honey', 'instead', 'water', 'what', 'would', 'happen', 'example', 'exerted', 'certain', 'stress', 'get', 'certain', 'shear', 'rate', 'water', 'wanted', 'get', 'shear', 'rate', 'using', 'honey', 'well', 'honey', 'thicker', 'water', 'its', 'going', 'move', 'easily', 'we', 'apply', 'greater', 'stress', 'order', 'achieve', 'shear', 'rate', 'measured', 'water', 'that', 'would', 'true', 'every', 'possible', 'shear', 'rate', 'if', 'graph', 'shear', 'stress', 'vs', 'shear', 'strain', 'honey', 'would', 'get', 'linear', 'relationship', 'like', 'water', 'value', 'shear', 'stress', 'would', 'always', 'higher', 'water', 'the', 'slope', 'line', 'honey', 'would', 'greater', 'slope', 'line', 'water', 'viscosity', 'formally', 'defined', 'slope', 'line', 'honey', 'thicker', 'water', 'requires', 'higher', 'higher', 'stresses', 'achieve', 'strain', 'rates', 'would', 'get', 'using', 'lower', 'stresses', 'water', 'in', 'simple', 'kitchen', 'analog', 'experiment', 'push', 'spoon', 'harder', 'stir', 'honey', 'stirring', 'water', 'honey', 'water', 'newtonian', 'liquids', 'the', 'relationship', 'simple', 'increase', 'shear', 'stress', 'increase', 'shear', 'rate', 'not', 'liquids', 'behave', 'way', 'polymer', 'solutions', 'dont', 'example', 'they', 'display', 'non-linear', 'relationship', 'shear', 'stresses', 'needed', 'get', 'higher', 'shear', 'rates', 'high', 'would', 'expect', 'that', 'behavior', 'described', 'shear', 'thinning', 'what', 'makes', 'polymer', 'solutions', 'remember', 'solution', 'individual', 'polymer', 'molecule', 'tends', 'coil', 'ball', 'normally', 'balls', 'spherical', 'at', 'high', 'shear', 'rates', 'distort', 'ellipses', 'that', 'reduces', 'amount', 'drag', 'making', 'solution', 'little', 'less', 'viscous', 'would', 'expect', 'the', 'shear', 'stress', 'still', 'increases', 'higher', 'shear', 'rates', 'much', 'newtonian', 'fluid', 'thats', 'remember', 'polymers', 'display', 'chain', 'entanglement', 'especially', 'large', 'chain', 'entanglement', 'increases', 'viscosity', 'solution', 'these', 'chains', 'mobile', 'polymer', 'constantly', 'undergoing', 'conformational', 'changes', 'as', 'solution', 'gets', 'sheared', 'polymer', 'chains', 'may', 'wiggle', 'loose', 'neighbors', 'random', 'conformational', 'changes', 'always', 'result', 'new', 'entanglements', 'right', 'away', 'but', 'happens', 'high', 'rate', 'shear', 'strain', 'the', 'polymer', 'chains', 'still', 'become', 'untangled', 'dont', 'time', 'form', 'new', 'entanglements', 'everything', 'whizzing', 'quickly', 'without', 'entanglements', 'viscosity', 'doesnt', 'increase', 'sharply', 'would', 'expect', 'some', 'liquids', 'behave', 'like', 'water', 'polymer', 'solutions', 'thats', 'consequence', 'enormously', 'long', 'chain', 'structures', 'polymers', 'its', 'worth', 'pointing', 'things', 'dont', 'behave', 'normal', 'way', 'colloids', 'often', 'display', 'shear', 'thickening', 'you', 'see', 'examples', 'videos', 'people', 'running', 'across', 'wading', 'pools', 'filled', 'corn', 'starch', 'water', 'colloidal', 'mixture', 'solid', 'cornstarch', 'particles', 'suspended', 'water', 'dissolved', 'if', 'walk', 'across', 'pool', 'sink', 'if', 'run', 'stay', 'top', 'thats', 'shear', 'thickening', 'solid', 'particles', 'move', 'shearing', 'when', 'walk', 'slide', 'past', 'run', 'collide', 'come', 'stop', 'a', 'bingham', 'plastic', 'hand', 'actually', 'appears', 'solid', 'give', 'good', 'shove', 'flows', 'like', 'liquid', 'peanut', 'butter', 'behaves', 'sort', 'like', 'problem', 'pp31', 'honey', 'newtonian', 'fluid', 'molasses', 'undergoes', 'shear', 'thinning', 'what', 'might', 'deduce', 'composition', 'molasses', 'honey', 'the', 'glass', 'transition', 'probably', 'commonly-cited', 'characteristic', 'polymeric', 'material', 'at', 'glass', 'transition', 'temperature', 'physical', 'nature', 'material', 'changes', 'subtly', 'it', 'goes', 'rubbery', 'flexible', 'material', 'higher', 'temperature', 'glass', 'transition', 'glassy', 'harder', 'material', 'lower', 'temperature', 'the', 'material', 'still', 'solid', 'either', 'way', 'definite', 'change', 'responds', 'stimuli', 'a', 'material', 'could', 'certainly', 'useful', 'either', 'state', 'we', 'might', 'want', 'plastic', 'rigid', 'like', 'water', 'bottle', 'rod', 'shower', 'curtain', 'alternatively', 'might', 'want', 'softer', 'like', 'seat', 'cushion', 'either', 'way', 'might', 'helpful', 'know', 'temperature', 'material', 'change', 'one', 'type', 'the', 'glass', 'transition', 'little', 'like', 'happens', 'gummy', 'bears', 'put', 'ice', 'cream', 'straight', 'bag', 'gummy', 'bears', 'chewy', 'dont', 'feel', 'like', 'break', 'teeth', 'put', 'ice', 'cream', 'changes', 'they', 'become', 'much', 'harder', 'chew', 'the', 'classic', 'explanation', 'glass', 'transition', 'based', 'idea', 'chain', 'flow', 'polymers', 'long-chain', 'molecules', 'given', 'little', 'energy', 'chains', 'move', 'around', 'they', 'wiggle', 'they', 'undergo', 'bond', 'rotations', 'switching', 'one', 'conformation', 'another', 'a', 'block', 'material', 'contains', 'piles', 'chains', 'like', 'nest', 'snakes', 'chain', 'ends', 'loops', 'squirm', 'past', 'continuously', 'chain', 'flow', 'allows', 'material', 'adapt', 'forces', 'exerted', 'we', 'bend', 'eraser', 'chains', 'rubber', 'slide', 'adopt', 'new', 'shape', 'at', 'least', 'thats', 'happens', 'room', 'temperature', 'it', 'might', 'work', 'eraser', 'accidentally', 'dropped', 'liquid', 'nitrogen', 'cold', 'as', 'block', 'material', 'cools', 'contracts', 'the', 'molecules', 'become', 'packed', 'closely', 'together', 'at', 'point', 'free', 'volume', '--', 'thats', 'amount', 'volume', 'material', 'actually', 'taken', 'molecules', '--', 'becomes', 'small', 'allow', 'chains', 'move', 'past', 'all', 'chains', 'need', 'little', 'room', 'around', 'going', 'undergo', 'conformational', 'changes', 'conformational', 'changes', 'polymer', 'chains', 'move', 'without', 'extra', 'room', 'material', 'suddenly', 'becomes', 'less', 'flexible', 'if', 'start', 'glassy', 'state', 'increase', 'temperature', 'volume', 'material', 'expanding', 'free', 'volume', 'increasing', 'at', 'point', 'free', 'volume', 'becomes', 'great', 'enough', 'chains', 'slip', 'past', 'the', 'material', 'becomes', 'flexible', 'it', 'becomes', 'rubbery', 'now', 'transition', 'thing', 'melting', 'the', 'chains', 'completely', 'overcoming', 'interactions', 'gaining', 'freedom', 'movement', 'direction', 'the', 'chains', 'still', 'highly', 'entangled', 'portions', 'chain', 'sliding', 'past', 'overall', 'things', 'changed', 'drastically', 'the', 'block', 'polymer', 'turn', 'gooey', 'puddle', 'liquid', 'well', 'material', 'change', 'glassy', 'rubbery', 'certain', 'temperature', 'consequences', 'material', 'behaves', 'for', 'example', 'tires', 'car', 'meant', 'rubbery', 'flexible', 'factor', 'helps', 'grip', 'road', 'if', 'weather', 'gets', 'cold', 'tires', 'become', 'glassy', 'tires', 'longer', 'amount', 'flexibility', 'dont', 'much', 'traction', 'all-weather', 'tires', 'composed', 'rubber', 'low', 'glass', 'transition', 'temperature', 'helping', 'avoid', 'problem', '(snow', 'tires', 'different', 'thing', 'patterns', 'treads', 'help', 'channel', 'away', 'snow', 'order', 'improve', 'traction)', 'knowing', 'change', 'occur', 'would', 'useful', 'so', 'determine', 'glass', 'transition', 'phase', 'changes', 'melting', 'points', 'measured', 'using', 'calorimetry', 'lets', 'start', 'when', 'solid', 'heated', 'temperature', 'increases', 'that', 'seems', 'simple', 'enough', 'temperature', 'basically', 'measure', 'heat', 'content', 'heat', 'flows', 'measured', 'temperature', 'rises', 'this', 'simple', 'relationship', 'breaks', 'melting', 'point', 'at', 'point', 'heat', 'flowing', 'material', 'consumed', 'breaking', 'intermolecular', 'forces', 'overcoming', 'attractions', 'costs', 'extra', 'energy', 'consequently', 'point', 'temperature', 'rise', 'material', 'stalls', 'temporarily', 'melts', 'this', 'extra', 'heat', 'needed', 'melt', 'material', 'called', 'heat', 'melting', 'commonly', 'heat', 'fusion', 'heat', 'fusion', 'actually', 'refers', 'opposite', 'process', 'material', 'cooled', 'frozen', 'as', 'heat', 'carried', 'away', 'cooling', 'material', 'cools', 'gradually', 'comes', 'point', 'strong', 'intermolecular', 'interactions', 'form', 'giving', 'extra', 'heat', 'the', 'fusion', 'temperature', 'melting', 'temperature', 'heat', 'fusion', 'heat', 'melting', 'one', 'case', 'heat', 'added', 'another', 'case', 'heat', 'given', 'in', 'principle', 'heat', 'something', 'look', 'stalling', 'point', 'temperature', 'find', 'temperature', 'phase', 'transition', 'differential', 'scanning', 'calorimetry', '(dsc)', 'technique', 'commonly', 'used', 'measure', 'phase', 'transitions', 'including', 'glass', 'transition', 'temperature', 'it', 'based', 'ideas', 'experiment', 'run', 'slightly', 'different', 'way', 'in', 'dsc', 'two', 'tiny', 'sample', 'chambers', 'side', 'side', 'one', 'contains', 'material', 'interested', '(empty)', 'one', 'used', 'reference', 'the', 'instrument', 'heats', 'samples', 'constant', 'rate', 'maintaining', 'temperature', 'consequently', 'may', 'actually', 'add', 'heat', 'one', 'chamber', 'reach', '305', '°c', 'time', '306', '°c', 'once', 'reach', 'melting', 'point', 'sample', 'interest', 'heat', 'flow', 'sample', 'must', 'increased', 'keep', 'reference', 'past', 'point', 'heat', 'flow', 'drops', 'back', 'normal', 'level', 'if', 'look', 'dsc', 'scan', 'see', 'graph', 'heat', 'flow', 'y-axis', 'temperature', 'x-axis', 'the', 'heat', 'flow', 'usually', 'stays', 'pretty', 'constant', 'temperature', 'increases', 'at', 'melting', 'point', 'heat', 'flow', 'increases', 'drops', 'back', 'melting', 'point', 'overcome', 'it', 'may', 'drop', 'back', 'level', 'however', 'heat', 'flow', 'recorded', 'related', 'heat', 'capacity', 'material', 'the', 'solid', 'liquid', 'usually', 'respond', 'differently', 'heat', 'in', 'general', 'molecules', 'liquid', 'freely', 'rotate', 'something', 'else', 'added', 'heat', 'liquids', 'thus', 'slightly', 'higher', 'heat', 'capacity', 'corresponding', 'solids', 'all', 'explanation', 'hopefully', 'prepares', 'us', 'dsc', 'study', 'glass', 'transition', 'temperature', 'it', 'works', 'way', 'point', 'if', 'look', 'dsc', 'scan', 'still', 'see', 'heat', 'flow', 'y-axis', 'temperature', 'x-axis', 'at', 'glass', 'transition', 'heat', 'flow', 'bumps', 'slightly', '--', 'remains', 'it', 'looks', 'different', 'melting', 'point', 'a', 'dsc', 'scan', 'melting', 'point', 'shows', 'peak', 'transition', 'temperature', 'a', 'dsc', 'scan', 'glass', 'transition', 'point', 'shows', 'little', 'step', 'like', 'riverbank', 'why', 'different', 'well', 'glass', 'transition', 'really', 'phase', 'change', 'like', 'melting', 'point', 'boiling', 'point', 'it', 'involve', 'physical', 'change', 'state', 'it', 'solid', 'solid', 'afterward', 'no', 'intermolecular', 'interactions', 'must', 'suddenly', 'overcome', 'free', 'molecules', 'instead', 'volume', 'change', 'the', 'free', 'volume', 'became', 'great', 'enough', 'chains', 'slip', 'past', 'chains', 'still', 'clinging', 'together', 'solid-state', 'now', 'additional', 'movement', 'consequences', 'the', 'material', 'becomes', 'flexible', 'if', 'heat', 'flows', 'freedom', 'motion', 'heat', 'distributed', 'in', 'words', 'slight', 'increase', 'heat', 'capacity', 'thats', 'observe', 'dsc', 'one', 'practical', 'note', 'dsc', 'scans', 'actually', 'displayed', 'two', 'different', 'ways', 'the', 'y-axis', 'either', 'display', 'heat', 'flowing', 'heat', 'flowing', 'that', 'means', 'melting', 'points', 'look', 'like', 'peaks', 'like', 'valleys', 'depending', 'data', 'displayed', 'often', 'data', 'labeled', 'arrow', 'says', 'endo', 'tell', 'direction', 'along', 'y-axis', 'means', 'heat', 'flowing', '(or', 'possibly', 'exo,', 'meaning', 'direction', 'means', 'heat', 'flowing', 'out)', 'you', 'need', 'look', 'carefully', 'things', 'seem', 'backward', 'problem', 'cp41', 'for', 'dsc', 'trace', 'state', 'sort', 'transition', 'os', 'occurring', 'temperature', 'why', 'glass', 'transition', 'temperature', 'vary', 'one', 'kind', 'polymer', 'another', 'what', 'structural', 'factors', 'influence', 'glass', 'transition', 'temperature', 'this', 'structure-property', 'relationship', 'isnt', 'straightforward', 'one', 'seem', 'number', 'different', 'variables', 'involved', 'however', 'simplest', 'factors', 'molecular', 'weight', 'the', 'higher', 'molecular', 'weight', 'polymer', 'higher', 'glass', 'transition', 'temperature', 'this', 'relationship', 'true', 'certain', 'point', 'however', 'the', 'non-linear', 'dependence', 'glass', 'transition', 'temperature', 'molecular', 'weight', 'described', 'flory-fox', 'equation', 'tg', 'tg(∞)', '-', 'k', 'mn', 'here', 'tg(∞)', 'refers', 'glass', 'transition', 'temperature', 'infinitely', 'long', 'chain', 'polymer', 'k', 'constant', 'particular', 'polymer', 'polystyrene', 'polyethylene', 'a', 'plot', 'tg', 'vs', 'mn', 'resembles', 'saturation', 'curve', 'line', 'rises', 'sharply', 'gradually', 'stalling', 'continuing', 'parallel', 'x-axis', 'in', 'words', 'although', 'relationship', 'increasing', 'glass', 'transition', 'temperature', 'increasing', 'molecular', 'weight', 'holds', 'true', 'relatively', 'low', 'molecular', 'weights', 'glass', 'transition', 'temperature', 'remains', 'constant', 'threshold', 'molecular', 'weight', 'reached', 'we', 'often', 'think', 'polymers', 'form', 'plastics', 'solid', 'materials', 'serve', 'structural', 'function', 'like', 'water', 'bottle', 'venetian', 'blinds', 'polymers', 'different', 'many', 'solids', 'metals', 'instance', 'crystalline', 'structures', 'atoms', 'form', 'regularly', 'repeating', 'pattern', 'row', 'upon', 'row', 'polymers', 'contrast', 'generally', 'somewhat', 'amorphous', 'think', 'strainer', 'filled', 'cooked', 'spaghetti', 'long', 'chains', 'pasta', 'looping', 'randomly', 'thats', 'molecules', 'amorphous', 'shapeless', 'polymers', 'tend', 'distribute', 'polymers', 'capable', 'forming', 'ordered', 'structures', 'if', 'aligned', 'instead', 'long', 'chains', 'parallel', 'would', 'able', 'get', 'much', 'closer', 'together', 'intermolecular', 'attractions', 'chains', 'would', 'much', 'stronger', 'the', 'energy', 'system', 'would', 'decrease', 'crystalline', 'structure', 'inherently', 'favored', 'so', 'doesnt', 'always', 'happen', 'think', 'polymer', 'cooling', 'completely', 'melted', 'state', 'case', 'structure', 'certainly', 'amorphous', 'as', 'cools', 'material', 'becomes', 'less', 'like', 'liquid', 'less', 'soft', 'with', 'small', 'molecules', 'transition', 'relatively', 'abrupt', 'molecules', 'slide', 'place', 'guided', 'strengthening', 'intermolecular', 'attractions', 'the', 'molecules', 'rotate', 'little', 'face', 'correct', 'direction', 'maybe', 'budge', 'little', 'side', 'with', 'polymers', 'transition', 'much', 'gradual', 'great', 'long', 'chains', 'slide', 'uncoil', 'order', 'lie', 'parallel', 'at', 'point', 'isnt', 'enough', 'energy', 'keep', 'working', 'way', 'optimal', 'alignment', 'as', 'result', 'many', 'polymers', 'semi-crystalline', 'regions', 'called', 'lamellae', 'portions', 'chains', 'aligned', 'parallel', 'also', 'large', 'amorphous', 'areas', 'much', 'randomly', 'oriented', 'as', 'result', 'polymer', 'sample', 'might', '80', 'amorphous', '20', 'chain', 'lengths', 'aligned', 'crystalline', 'lamellae', 'even', 'crystalline', 'domains', 'exert', 'strong', 'influence', 'properties', 'polymer', 'because', 'stronger', 'intermolecular', 'attractions', 'closer', 'chains', 'chains', 'much', 'less', 'able', 'slide', 'past', 'the', 'result', 'material', 'stronger', 'rigid', 'important', 'reliable', 'structural', 'materials', 'how', 'know', 'whether', 'polymer', 'sample', 'contains', 'crystalline', 'domains', 'we', 'use', 'differential', 'scanning', 'calorimetry', 'observe', 'transition', 'ordered', 'crystalline', 'phase', 'disordered', 'melt', 'phase', 'added', 'heat', 'as', 'saw', 'looking', 'glass', 'transition', 'melting', 'point', 'shows', 'peak', 'dsc', 'trace', 'you', 'look', 'temperature', 'tg', 'recall', 'looks', 'like', 'step', 'baseline', 'below', 'tg', 'chains', 'mobile', 'enough', 'move', 'alignment', 'in', 'example', 'see', 'tg', '128', '°c', 'well', 'melting', 'point', 'tm', 'attractions', 'chains', 'lamellae', 'overcome', '155-160', '°c', 'if', 'regular', 'molecular', 'elemental', 'solid', 'melting', 'point', 'would', 'fusion', 'point', 'material', 'would', 'melt', 'temperature', 'warming', 'froze', 'cooling', 'in', 'polymers', 'usually', 'doesnt', 'work', 'way', 'hysteresis', 'observed', 'in', 'hysteresis', 'sample', 'changed', 'previous', 'events', 'leading', 'results', 'dont', 'seem', 'reproducible', 'way', 'would', 'normally', 'expect', 'in', 'general', 'crystallization', 'temperature', 'tc', 'lower', 'melting', 'temperature', 'tm', 'thats', 'material', 'melts', 'chains', 'move', 'alignment', 'chain', 'entanglement', 'hindered', 'getting', 'back', 'alignment', 'leading', 'delay', 'crystallization', 'example', '(pageindex1\\)', 'describe', 'observed', 'following', 'dsc', 'traces', 'crystallinity', 'also', 'probed', 'ways', 'the', 'classic', 'approach', 'x-ray', 'diffraction', 'when', 'x-rays', 'pass', 'ordered', 'materials', 'give', 'rise', 'diffraction', 'patterns', 'bright', 'spots', 'space', 'ricocheting', 'x-rays', 'constructively', 'interfered', 'shining', 'starkly', 'amid', 'blackness', 'x-rays', 'undergone', 'destructive', 'interference', 'the', 'drawing', 'cartoon', 'x-ray', 'diffraction', 'pattern', 'it', 'might', 'captured', 'digital', 'camera', 'earlier', 'days', 'sheet', 'photographic', 'film', 'the', 'bright', 'spots', 'result', 'x-rays', 'scattering', 'middle', 'encountered', 'sample', 'in', 'middle', 'black', 'circle', 'blocks', 'original', 'x-ray', 'beam', 'would', 'otherwise', 'bright', 'allow', 'observation', 'spots', 'around', 'the', 'data', 'x-ray', 'diffraction', 'experiment', 'fed', 'program', 'mathematically', 'deconvolute', 'spots', 'showed', 'the', 'process', 'compared', 'throwing', 'stone', 'shoulder', 'pond', 'watching', 'ripples', 'decide', 'stone', 'must', 'with', 'one', 'stone', 'pretty', 'easy', 'you', 'would', 'find', 'stone', 'center', 'circle', 'ripples', 'with', 'two', 'stones', 'interference', 'patterns', 'make', 'ripples', 'complex', 'may', 'take', 'work', 'determine', 'stone', 'lies', 'pond', 'the', 'picture', 'illustrates', 'results', 'relatively', 'simple', 'x-ray', 'diffraction', 'experiment', 'commonly', 'used', 'crystalline', 'polymer', 'samples', 'the', 'technique', 'called', 'wide-angle', 'x-ray', 'scattering', 'waxs', 'it', 'simply', 'reports', 'angle', 'scattering', 'intensity', 'observed', 'relative', 'initial', 'x-ray', 'beam', 'that', 'angle', 'corresponds', 'distance', 'atoms', 'sample', 'case', 'usually', 'corresponds', 'distance', 'parallel', 'polymer', 'chains', 'sometimes', 'one', 'peak', 'example', 'simple', 'one', 'start', 'the', 'relationship', 'scattering', 'angle', 'interatomic', 'distance', 'given', 'braggs', 'law', 'fundamental', 'starting', 'point', 'x-ray', 'crystallography', 'first', 'expressed', 'sir', 'william', 'bragg', 'son', 'sir', 'lawrence', 'bragg', 'the', 'law', 'states', '[nλ', '2dsin', 'θ', 'labelbragg1\\', 'n', 'integer', '(typically', 'assumed', '1)', 'lambda', 'wavelength', 'x-ray', 'used', '(commonly', '1541', 'å', 'although', 'others', 'possible)', 'distance', 'regularly', 'repeating', 'atoms', 'θ', 'scattering', 'angle', 'rearranging', 'equation', 'refbragg1,', 'find', 'dfrac1541{2\\sin', 'θ\\', 'in', 'words', 'distance', 'atoms', 'inversely', 'proportional', 'scattering', 'angle', 'the', 'greater', 'angle', 'smaller', 'distance', 'example', '(pageindex1\\)', 'in', 'following', 'cases', 'determine', 'sample', '(a', 'b)', 'higher', 'crystalline', 'content', 'crystalline', 'domains', 'provide', 'additional', 'strength', 'polymer', 'materials', 'the', 'strong', 'attraction', 'possible', 'closely-aligned', 'chains', 'results', 'long', 'segments', 'polymer', 'held', 'firmly', 'position', 'consequently', 'chain', 'flow', 'limited', 'material', 'becomes', 'rigid', 'sometimes', 'rigid', 'segments', 'polymer', 'deliberately', 'built', 'structure', 'for', 'example', 'block', 'co-polymers', 'softer', 'flexible', 'blocks', 'often', 'paired', 'harder', 'rigid', 'blocks', 'the', 'soft', 'segments', 'may', 'greater', 'conformational', 'flexibility', 'weaker', 'intermolecular', 'attractions', 'the', 'hard', 'segments', 'may', 'conformationally', 'rigid', 'may', 'stronger', 'intermolecular', 'attractions', 'strong', 'dipoles', 'hydrogen', 'bonds', 'if', 'block', 'lengths', 'right', 'size', 'two', 'segments', 'able', 'separate', 'two', 'phases', 'as', 'result', 'stronger', 'intermolecular', 'attractions', 'lengths', 'chains', 'containing', 'hard', 'segments', 'cluster', 'together', 'pushing', 'soft', 'segments', 'would', 'otherwise', 'get', 'way', 'intermolecular', 'attractions', 'this', 'phenomenon', 'called', 'microphase', 'separation', 'the', 'result', 'material', 'contains', 'islands', 'strength', 'rigidity', 'matrix', 'flexible', 'polymer', 'chains', 'that', 'useful', 'combination', 'the', 'flexible', 'chains', 'soft', 'segments', 'allow', 'polymer', 'distorted', 'bent', 'compressed', 'hard', 'segments', 'put', 'limits', 'flexibility', 'keeping', 'material', 'firmly', 'together', 'because', 'usually', 'dealing', 'large', 'numbers', 'enchained', 'monomers', 'difference', 'two', 'kinds', 'segments', 'need', 'even', 'dramatic', 'a', 'copolymer', 'butadiene', 'styrene', 'hydrocarbons', 'form', 'microphase', 'separated', 'materials', 'in', 'case', 'intermolecular', 'attractions', 'dominated', 'weak', 'london', 'dispersion', 'forces', 'aromatic', 'groups', 'styrene', 'delocalized', 'pi', 'systems', 'london', 'dispersion', 'forces', 'slightly', 'stronger', 'as', 'result', 'polystyrene', 'blocks', 'cluster', 'together', 'surrounded', 'softer', 'polybutadiene', 'blocks', 'problem', 'pp61', 'identify', 'hard', 'segment', 'soft', 'segment', 'following', 'block-co-polymers', 'sometimes', 'separation', 'phases', 'directly', 'observed', 'via', 'microscopy', 'tunneling', 'electron', 'microscopy', '(tem)', 'technique', 'generate', 'images', 'cross-sectional', 'slice', 'material', 'the', 'material', 'generally', 'stained', 'heavy', 'metal', 'osmium', 'binds', 'preferentially', 'one', 'phase', 'the', 'stained', 'phase', 'shows', 'darker', 'tem', 'phase', 'isnt', 'stained', 'x-ray', 'diffraction', 'techniques', 'often', 'used', 'measure', 'distances', 'hard', 'segments', 'small-angle', 'x-ray', 'scattering', '(saxs)', 'similar', 'wide-angle', 'x-ray', 'scattering', '(waxs)', 'because', 'inverse', 'relationship', 'scattering', 'angle', 'distance', 'saxs', 'used', 'probe', 'regularly', 'repeating', 'structures', 'greater', 'distances', 'seen', 'waxs', 'that', 'makes', 'possible', 'see', 'peaks', 'hard', 'segments', 'distributed', 'regularly', 'enough', 'within', 'soft', 'matrix', 'note', 'saxs', 'x-axis', 'usually', 'labeled', 'q', 'scattering', 'vector', 'q', '4πsinθ', 'λ', 'but', 'since', '2sinθλ', 'q', '2πd', '2πq', 'that', 'gives', 'us', 'pretty', 'straightforward', 'way', 'calculating', 'distances', 'regularly-spaced', 'hard', 'segments', '(or', 'regularly-spaced', 'objects)', 'once', 'waxs', 'inverse', 'relationship', 'quantity', 'shown', 'x-axis', 'distances', 'space', 'problem', 'pp62', 'calculate', 'approximate', 'distances', 'revealed', 'following', 'saxs', 'results', 'if', 'think', 'materials', 'made', 'polymers', 'couple', 'things', 'might', 'readily', 'come', 'mind', 'you', 'might', 'think', 'ubiquitous', 'use', 'plastics', 'structures', 'around', 'us', 'automobile', 'bumpers', 'you', 'might', 'think', 'rubber', 'band', 'use', 'fasten', 'something', 'firmly', 'either', 'home', 'lab', 'in', 'either', 'case', 'function', 'material', 'relies', 'response', 'stimuli', 'do', 'want', 'flexible', 'change', 'shape', 'snap', 'back', 'came', 'exactly', 'much', 'flexibility', 'need', 'where', 'trade-off', 'flexibility', 'strength', 'these', 'questions', 'important', 'polymer', 'chemistry', 'consequently', 'often', 'need', 'probe', 'materials', 'behave', 'different', 'conditions', 'know', 'employed', 'effectively', 'tensile', 'testing', 'one', 'simplest', 'ways', 'probe', 'material', 'responds', 'stress', 'remember', 'stress', 'context', 'means', 'force', 'exerted', 'material', 'per', 'unit', 'cross-sectional', 'area', 'as', 'units', 'pressure', 'pascals', '(pa)', 'to', 'experiment', 'two', 'ends', 'sample', 'attached', 'two', 'clamps', 'one', 'movable', 'the', 'movable', 'clamp', 'pulled', 'sample', 'becomes', 'stretched', 'the', 'force', 'required', 'pull', 'sample', 'recorded', 'given', 'cross-sectional', 'area', 'sample', 'perpendicular', 'force', 'force', 'converted', 'units', 'strain', 'the', 'experiment', 'also', 'measures', 'distance', 'sample', 'stretched', '(the', 'strain)', 'that', 'distance', 'usually', 'expressed', 'ratio', 'comparing', 'change', 'length', 'original', 'length', 'sample', 'it', 'written', 'either', 'fraction', 'percent', 'in', 'rough', 'sketch', 'see', 'typically', 'happens', 'experiment', 'the', 'sample', 'stretches', '(the', 'strain', 'increases)', 'gets', 'harder', 'stretch', 'seen', 'increasing', 'force', '(and', 'therefore', 'increasing', 'stress)', 'needed', 'keep', 'stretching', 'eventually', 'stress', 'plummets', 'sample', 'breaks', 'already', 'graph', 'see', 'couple', 'important', 'pieces', 'information', 'learn', 'tensile', 'testing', 'number', 'one', 'far', 'stretch', 'material', 'breaks', 'that', 'quantity', 'called', 'strain', 'break', 'in', 'sample', 'strain', 'break', 'looks', 'like', '11', '110;', 'means', 'sample', 'stretched', 'twice', 'original', 'length', 'number', 'two', 'much', 'stress', 'sample', 'support', 'without', 'breaking', 'that', 'quantity', 'called', 'ultimate', 'tensile', 'strength.', 'in', 'sample', 'ultimate', 'tensile', 'strength', '750', 'pa.', 'if', 'look', 'closely', 'graph', 'another', 'sample', 'get', 'third', 'important', 'quantity', 'see', 'additional', 'features', 'the', 'important', 'quantity', 'derived', 'first', 'part', 'curve', '(a)', 'stress', 'increases', 'linearly', 'strain', 'in', 'linear', 'region', 'material', 'behaving', 'hookean', 'solid', 'meaning', 'obeys', 'hookes', 'law', 'hookes', 'law', 'says', 'stress', 'strain', 'directly', 'proportional', 'in', 'original', 'words,', 'extension', 'proportional', 'force', 'f', 'kx', 'f', 'force', 'x', 'extension', 'solid', 'k', 'proportionality', 'constant', 'hookes', 'law', 'commonly', 'applied', 'behavior', 'mechanical', 'springs', 'also', 'holds', 'solid', 'materials', 'the', 'slope', 'linear', 'region', 'graph', '(a)', 'would', 'equal', 'proportionality', 'constant', 'k', 'graph', 'shows', 'ratio', 'fx', 'in', 'materials', 'science', 'slope', 'commonly', 'called', 'youngs', 'modulus', 'it', 'measure', 'inherent', 'stiffness', 'material.', 'youngs', 'modulus', 'e', 'σε', 'σ', 'stress', 'ε', 'extensional', 'strain', 'the', 'extensional', 'strain', 'strain', 'observed', 'stretching', 'material', 'in', 'graph', 'shown', 'youngs', 'modulus', 'around', '250', 'pa', '01', '2500', 'pa', 'this', 'initial', 'region', 'curve', 'hookes', 'law', 'obeyed', 'sometimes', 'called', 'linear', 'elastic', 'region', 'the', 'word', 'elastic', 'immediate', 'connotation', 'everyday', 'english', 'bringing', 'mind', 'rubber', 'band', 'stretched', 'think', 'stretching', 'part', 'hear', 'word', 'elastic', 'however', 'returning', 'motion', 'essential', 'feature', 'elastic', 'behavior', 'the', 'rubber', 'band', 'always', 'comes', 'back', 'original', 'shape.', 'likewise', 'within', 'linear', 'elastic', 'region', 'solid', 'material', 'returns', 'original', 'shape', 'deformed', 'stress.', 'with', 'solids', 'aluminum', 'concrete', 'linear', 'elastic', 'region', 'spans', 'narrow', 'range', 'strains', 'just', 'looking', 'wouldnt', 'notice', 'materials', 'deformed', 'however', 'linear', 'stress-strain', 'relationship', 'typical', 'solids', 'in', 'rheology', 'behavior', 'described', 'elastic', 'of', 'course', 'many', 'polymers', 'sample', 'stretched', 'far', 'see', 'change', 'naked', 'eye', 'still', 'returns', 'original', 'shape', 'what', 'makes', 'polymers', 'different', 'the', 'long-chain', 'structure', 'polymers', 'make', 'behave', 'differently', 'materials', 'these', 'chains', 'undergo', 'conformational', 'change', 'bond', 'along', 'chain', 'rotate', 'converting', 'polymer', 'chain', 'slightly', 'different', 'shape', 'that', 'ability', 'gives', 'soft', 'material', 'great', 'deal', 'flexibility', 'the', 'conformations', 'chains', 'adapt', 'accommodate', 'stress', 'moving', 'chains', 'new', 'shapes', 'offer', 'lower-energy', 'packing', 'when', 'stress', 'removed', 'chains', 'eventually', 'slide', 'back', 'original', 'conformations', 'they', 'return', 'equilibrium', 'shapes', 'the', 'presence', 'physical', 'chemical', 'crosslinks', 'help', 'material', 'return', 'original', 'shape', 'functioning', 'anchor', 'points', 'chains', 'dont', 'stray', 'far', 'those', 'interactions', 'might', 'found', 'hard-phase', 'interactions', 'microphase', 'separated', 'material', 'illustrated', 'although', 'could', 'also', 'found', 'homogeneous', 'material', 'the', 'linear', 'elastic', 'region', 'isnt', 'see', 'stress-strain', 'curve', 'at', 'point', '(b)', 'linear', 'relationship', 'suddenly', 'lost', 'the', 'stress', 'might', 'even', 'drop', 'seen', 'particular', 'case', 'this', 'feature', 'graph', 'called', 'yield', 'point', 'the', 'stress', 'experienced', 'material', 'resulting', 'strain', 'become', 'sufficient', 'overcome', 'natural', 'elastic', 'behavior', 'solid', 'as', 'noted', 'previously', 'physical', 'crosslinks', 'hydrogen', 'bonds', 'help', 'reinforce', 'elastic', 'behavior', 'polymer', 'sample', 'if', 'point', 'interactions', 'overcome', 'chains', 'start', 'slide', 'readily', 'past', 'other.', 'as', 'result', 'material', 'loses', 'elasticity', 'when', 'stress', 'released', 'material', 'still', 'spring', 'back', 'chains', 'settle', 'new', 'conformational', 'equilibrium', 'however', 'equilibrium', 'one', 'new', 'physical', 'crosslinks', 'form', 'groups', 'form', 'intermolecular', 'interactions', 'nearest', 'neighbors', 'encounter', 'it', 'isnt', 'likely', 'always', 'groups', 'interacting', 'as', 'result', 'material', 'settle', 'slightly', 'different', 'shape', 'you', 'probably', 'seen', 'happen', 'rubber', 'band', 'stretched', 'far', 'often', 'long', 'chains', 'dropped', 'old', 'interactions', 'picked', 'new', 'ones', 'formed', 'easily', 'extended', 'shape', 'at', 'point', '(c)', 'example', 'graph', 'slope', 'curve', 'starts', 'increase', 'the', 'change', 'stress', 'results', 'smaller', 'smaller', 'changes', 'strain', 'material', 'getting', 'stiffer', 'this', 'phenomenon', 'called', 'strain', 'hardening', 'this', 'feature', 'would', 'always', 'observed', 'happen', 'would', 'explain', 'in', 'polymers', 'one', 'explanation', 'may', 'lay', 'fact', 'volume', 'material', 'remain', 'constant', 'stretched', 'if', 'sample', 'getting', 'longer', 'means', 'also', 'getting', 'narrower.', 'as', 'consequence', 'narrowing', 'cross-section', 'chains', 'become', 'compressed', 'together', 'at', 'point', 'physical', 'crosslinks', 'begin', 'occurring', 'neighboring', 'chains', 'these', 'crosslinks', 'dont', 'occur', 'equilibrium', 'positions', 'polymer', 'chains', 'coiled', 'around', 'like', 'original', 'sample', 'these', 'crosslinks', 'occur', 'chains', 'extended', 'lying', 'parallel', 'closer', 'contact', 'distances', 'normal', 'we', 'see', 'opposite', 'sort', 'thing', 'happening', '(d)', 'slope', 'curve', 'decreasing', 'instead', 'increasing', 'this', 'phenomenon', 'called', 'strain-softening', 'again', 'might', 'observed', 'samples', 'when', 'happen', 'causes', 'in', 'case', 'answer', 'simpler', 'having', 'overcome', 'interactions', 'held', 'chains', 'together', 'nothing', 'left', 'resist', 'deformation', 'as', 'chains', 'begin', 'disentangle', 'becomes', 'even', 'easier', 'pull', 'apart', 'facilitating', 'extension', 'stretching', 'sample.', 'eventually', 'breaking', 'point', '(e)', 'chains', 'start', 'lose', 'contact', 'location', 'sample', 'resulting', 'catastrophic', 'rupture', 'sample', 'problem', 'pp71', 'in', 'following', 'curves', 'estimate', 'ultimate', 'tensile', 'strength', 'strain', 'break', 'problem', 'pp72', 'in', 'following', 'curves', 'calculate', 'youngs', 'modulus', 'problem', 'pp73', 'in', 'following', 'curves', 'identify', 'diagnostic', 'features', 'we', 'saw', 'earlier', 'inherent', 'stiffness', 'material', 'assessed', 'youngs', 'modulus', 'the', 'youngs', 'modulus', 'ratio', 'stress-induced', 'material', 'applied', 'strain', 'the', 'stress', 'amount', 'deformation', 'material', 'change', 'length', 'extensional', 'experiment', 'expressed', 'fraction', 'beginning', 'length', 'the', 'strain', 'force', 'exerted', 'sample', 'divided', 'cross-sectional', 'area', 'sample', 'if', 'strain', 'limited', 'small', 'deformation', 'varies', 'linearly', 'stress', 'if', 'graph', 'relationship', 'slope', 'line', 'gives', 'us', 'youngs', 'modulus', 'e', 'thats', 'proportionality', 'constant', 'stress', 'strain', 'hookes', 'law', 'e', 'σ', 'ε', 'hookes', 'law', 'sometimes', 'used', 'describe', 'behavior', 'mechanical', 'springs', 'the', 'modulus', 'thought', 'resistance', 'stretching', 'spring', 'resistance', 'spring', 'offers', 'greater', 'force', 'needed', 'stretch', 'the', 'force', 'snaps', 'spring', 'back', 'place', 'let', 'go', 'in', 'experiments', 'saw', 'earlier', 'didnt', 'let', 'go', 'we', 'continued', 'stretch', 'material', 'farther', 'farther', 'applying', 'generally', 'increasing', 'stress', 'material', 'finally', 'broke', 'now', 'look', 'much', 'limited', 'approach', 'instead', 'stretching', 'material', 'far', 'stretch', 'tiny', 'bit', 'release', 'stress', 'snaps', 'back', 'original', 'length', 'we', 'stress', 'release', 'we', 'keep', 'repeating', 'instead', 'continuously', 'increasing', 'strain', 'sample', 'subjected', 'oscillatory', 'strain', 'one', 'repeats', 'cycle', 'this', 'approach', 'called', 'dynamic', 'mechanical', 'analysis', 'we', 'use', 'dynamic', 'mechanical', 'analysis', 'measure', 'modulus', 'material', 'instead', 'continuously', 'moving', 'way', 'linear', 'elastic', 'region', 'beyond', 'hookes', 'law', 'breaks', 'carefully', 'keep', 'sample', 'hookean', 'region', 'entire', 'experiment', 'now', 'one', 'experiment', 'good', 'enough', 'extract', 'modulus', 'letting', 'go', 'why', 'the', 'principle', 'reason', 'running', 'experiment', 'way', 'get', 'additional', 'information', 'we', 'get', 'information', 'polymers', 'dont', 'quite', 'follow', 'hookes', 'law', 'perfectly', 'in', 'reality', 'even', 'within', 'linear', 'elastic', 'region', 'stress-strain', 'curve', 'quite', 'linear', 'in', 'picture', 'curvature', 'exaggerated', 'quite', 'bit', 'illustrative', 'purposes', 'even', 'relationship', 'quite', 'linear', 'release', 'strain', 'stress', 'material', 'simply', 'follow', 'curve', 'back', 'zero', 'it', 'instead', 'phenomenon', 'called', 'hysteresis', 'work', 'hysteresis', 'means', 'property', 'material', 'depends', 'material', 'came', 'current', 'situation', 'in', 'case', 'hookes', 'law', 'seems', 'imply', 'specific', 'sample', 'subjected', 'specific', 'strain', 'would', 'experience', 'specific', 'stress', '(or', 'vice', 'versa)', 'however', 'depends', 'whether', 'stretching', 'sample', 'letting', 'relax', 'as', 'let', 'sample', 'relax', 'back', 'initial', 'length', 'strain', 'different', 'saw', 'stretching', 'typically', 'lower', 'again', 'see', 'curve', 'curvature', 'exaggerated', 'the', 'difference', 'loading', 'curve', '(when', 'stress', 'first', 'applied)', 'unloading', 'curve', '(when', 'stress', 'removed)', 'represents', 'energy', 'loss', 'a', 'force', 'applied', 'move', 'sample', 'portion', 'sample', 'distance', 'when', 'sample', 'snapped', 'back', 'distance', 'force', 'unequal', 'one', 'initially', 'applied', 'some', 'energy', 'therefore', 'lost', 'the', 'slope', 'loading', 'curve', 'analogous', 'youngs', 'modulus', 'tensile', 'testing', 'experiment', 'called', 'storage', 'modulus', 'e', 'the', 'storage', 'modulus', 'measure', 'much', 'energy', 'must', 'put', 'sample', 'order', 'distort', 'the', 'difference', 'loading', 'unloading', 'curves', 'called', 'loss', 'modulus', 'e.', 'it', 'measures', 'energy', 'lost', 'cycling', 'strain.', 'why', 'would', 'energy', 'lost', 'experiment', 'in', 'polymer', 'chiefly', 'chain', 'flow', 'the', 'resistance', 'deformation', 'polymer', 'comes', 'entanglement', 'including', 'physical', 'crosslinks', 'general', 'occlusions', 'chains', 'encounter', 'undergoing', 'conformational', 'changes', 'accommodate', 'new', 'shape', 'material', 'once', 'stress', 'removed', 'material', 'springs', 'back', 'equilibrium', 'shape', 'reason', 'chains', 'would', 'follow', 'exact', 'conformational', 'pathway', 'return', 'equilibrium', 'conformations', 'because', 'moved', 'original', 'positions', 'able', 'follow', 'lower-energy', 'pathway', 'back', 'starting', 'point', 'pathway', 'less', 'resistance', 'neighboring', 'chains', 'for', 'reason', 'stretching', 'polymer', 'quite', 'stretching', 'mechanical', 'spring', 'a', 'spring-and-dashpot', 'analogy', 'often', 'invoked', 'describe', 'soft', 'materials', 'whereas', 'spring', 'simply', 'bounces', 'back', 'original', 'shape', 'pulled', 'dashpot', 'if', 'dont', 'know', 'dashpot', 'picture', 'hydraulic', 'arms', 'support', 'hatchback', 'car', 'open', 'upward', 'there', 'resistance', 'opening', 'hatchback', 'piston', 'pulled', 'hydraulic', 'fluid', 'arm', 'stretches', 'when', 'stop', 'lifting', 'arms', 'stay', 'length', 'hydraulic', 'fluid', 'also', 'resists', 'movement', 'piston', 'back', 'original', 'position', 'the', 'dashpot', 'tendency', 'stay', 'put', 'rather', 'spring', 'back', 'polymers', 'display', 'little', 'properties', 'they', 'elastic', 'element', 'rooted', 'entanglement', 'makes', 'resist', 'deformation', 'return', 'original', 'shapes', 'they', 'also', 'viscous', 'element', 'rooted', 'chain', 'flow', 'that', 'viscous', 'element', 'means', 'distort', 'polymeric', 'materials', 'might', 'return', 'exactly', 'form', 'started', 'taken', 'together', 'behaviors', 'described', 'viscoelastic', 'properties', 'many', 'materials', 'viscoelastic', 'properties', 'meaning', 'display', 'aspects', 'elastic', 'solids', 'aspects', 'viscous', 'liquids', 'so', 'far', 'concentrated', 'extensional', 'deformations', 'materials', 'looking', 'happens', 'stretch', 'its', 'worth', 'looking', 'another', 'type', 'deformation', 'commonly', 'used', 'materials', 'testing', 'this', 'second', 'approach', 'uses', 'shear', 'instead', 'extension', 'probe', 'material', 'respond', 'a', 'shear', 'force', 'applied', 'unevenly', 'material', 'tilts', 'twists', 'rather', 'stretching', 'one', 'reasons', 'approach', 'used', 'often', 'easy', 'a', 'sample', 'sandwiched', 'two', 'plates', 'the', 'bottom', 'plate', 'held', 'place', 'top', 'plate', 'twisted', 'shearing', 'material', 'held', 'if', 'take', 'closer', 'look', 'layer', 'sample', 'maybe', 'surface', 'along', 'edge', 'sandwich', 'imagine', 'breaking', 'individual', 'layers', 'under', 'shear', 'strain', 'layers', 'move', 'different', 'amounts', 'the', 'top', 'layer', 'right', 'beneath', 'top', 'plate', 'moves', 'the', 'bottom', 'layer', 'sitting', 'stationary', 'lower', 'plate', 'doesnt', 'move', 'in', 'layer', 'moves', 'little', 'one', 'beneath', 'this', 'gradation', 'deformation', 'across', 'sample', 'much', 'like', 'saw', 'analysis', 'viscosity', 'liquids', 'the', 'difference', 'viscosity', 'looks', 'variation', 'strain', 'time', 'nevertheless', 'modulus', 'solids', 'roughly', 'analogous', 'viscosity', 'liquids', 'we', 'use', 'parallel', 'plate', 'geometry', 'obtain', 'values', 'storage', 'modulus', 'loss', 'modulus', 'like', 'via', 'extensional', 'geometry', 'the', 'values', 'get', 'quite', 'for', 'reason', 'modulus', 'obtained', 'shear', 'experiments', 'given', 'different', 'symbol', 'modulus', 'obtained', 'extensional', 'experiments', 'in', 'shear', 'experiment', 'g', 'σ', 'ε', 'that', 'means', 'storage', 'modulus', 'given', 'symbol', 'g', 'loss', 'modulus', 'given', 'symbol', 'g', 'apart', 'providing', 'little', 'information', 'experiment', 'actually', 'conducted', 'distinction', 'shear', 'modulus', 'extension', 'modulus', 'important', 'resulting', 'values', 'quite', 'different', 'in', 'general', 'value', 'storage', 'modulus', 'obtained', 'extensional', 'experiment', 'three', 'times', 'larger', 'value', 'storage', 'modulus', 'obtained', 'shear', 'experiment.', 'e', '3', 'g', 'the', 'reason', 'difference', 'extension', 'actually', 'involves', 'deformation', 'material', 'three', 'directions', 'as', 'material', 'stretched', 'one', 'direction', '(lets', 'say', 'y-direction)', 'order', 'preserve', 'constant', 'volume', 'material', '(there', 'still', 'amount', 'stuff', 'stretching)', 'material', 'compresses', 'two', 'directions', '(x', 'z)', 'problem', 'pp81', 'metric', 'prefixes', 'frequently', 'encountered', 'reading', 'modulus', 'rank', 'following', 'units', 'stress', 'smallest', 'largest', 'case', 'provide', 'conversion', 'factor', 'pa', 'gpa', 'kpa', 'mpa', 'pa', 'the', 'storage', 'modulus', 'measures', 'resistance', 'deformation', 'elastic', 'solid', 'its', 'related', 'proportionality', 'constant', 'stress', 'strain', 'hookes', 'law', 'states', 'extension', 'increases', 'force', 'in', 'dynamic', 'mechanical', 'analysis', 'look', 'stress', '(σ)', 'force', 'per', 'cross-sectional', 'unit', 'area', 'needed', 'cause', 'extension', 'sample', 'strain', '(ε)', 'e', 'σ', 'ε', 'alternatively', 'shear', 'experiment', 'g', 'σ', 'ε', 'the', 'dynamic', 'mechanical', 'analysis', 'differs', 'simple', 'tensile', 'testing', 'performing', 'experiment', 'cyclically', 'the', 'sample', 'stretched', 'released', '(or', 'sheared', 'released)', 'it', 'subjected', 'additional', 'stress', 'released', 'there', 'element', 'time', 'involved', 'specifically', 'experiment', 'cyclic', 'carried', 'different', 'frequencies', 'when', 'plot', 'resulting', 'modulus', 'frequency', 'get', 'additional', 'information', 'sample', 'the', 'results', 'would', 'typically', 'presented', 'graph', 'like', 'one', 'what', 'graph', 'tells', 'us', 'frequency', 'clearly', 'matters', 'when', 'experiment', 'run', 'higher', 'frequencies', 'storage', 'modulus', 'higher', 'the', 'material', 'appears', 'stiffer', 'in', 'contrast', 'loss', 'modulus', 'lower', 'high', 'frequencies', 'material', 'behaves', 'much', 'less', 'like', 'viscous', 'liquid', 'in', 'particular', 'sharp', 'drop', 'loss', 'modulus', 'related', 'relaxation', 'time', 'material', 'in', 'context', 'thats', 'time', 'takes', 'chains', 'flow', 'new', 'conformations', 'response', 'applied', 'stress', 'if', 'dont', 'time', 'flow', 'viscous', 'response', 'material', 'lost', 'the', 'material', 'behaves', 'much', 'like', 'elastic', 'solid', 'subjected', 'high-frequency', 'cyclic', 'deformation', 'thats', 'important', 'know', 'material', 'might', 'subjected', 'vibrations', 'stimuli', 'everyday', 'use', 'properties', 'might', 'change', 'accordingly', 'another', 'variation', 'kind', 'experiment', 'called', 'dynamic', 'mechanical', 'thermal', 'analysis', 'instead', 'changing', 'frequency', 'stimulus', 'throughout', 'experiment', 'frequency', 'held', 'constant', 'temperature', 'changed', 'instead', 'as', 'result', 'see', 'material', 'responds', 'different', 'conditions', 'might', 'tell', 'us', 'behave', 'everyday', 'applications', 'the', 'result', 'experiment', 'might', 'graph', 'like', 'one', 'at', 'point', 'already', 'familiar', 'glass', 'transition', 'it', 'shouldnt', 'surprising', 'properties', 'dependent', 'temperature', 'at', 'glass', 'transition', 'temperature', 'expanding', 'volume', 'material', 'increasing', 'temperature', 'becomes', 'sufficient', 'allow', 'chain', 'flow', 'as', 'result', 'material', 'suddenly', 'behaves', 'much', 'like', 'viscous', 'liquid', 'loss', 'modulus', 'increases', 'the', 'stiffness', 'material', 'drops', 'entangled', 'chains', 'longer', 'resist', 'deformation', 'strongly', 'storage', 'modulus', 'decreases', 'the', 'dynamic', 'mechanical', 'thermal', 'analysis', 'thus', 'provides', 'alternative', 'way', 'determine', 'glass', 'transition', 'temperature', 'because', 'actually', 'measuring', 'different', 'physical', 'phenomenon', 'differential', 'scanning', 'calorimetry', 'tg', 'obtained', 'dmta', 'experiment', 'may', 'agree', 'exactly', 'one', 'obtained', 'dsc', 'experiment', 'nevertheless', 'often', 'useful', 'different', 'ways', 'assessing', 'properties', 'in', 'order', 'facilitate', 'analysis', 'tg', 'experiment', 'different', 'quantity', 'usually', 'displayed', 'tan', 'delta', 'ratio', 'loss', 'modulus', 'storage', 'modulus', 'it', 'peaks', 'glass', 'transition', 'temperature', 'the', 'term', 'tan', 'delta', 'refers', 'mathematical', 'treatment', 'storage', 'modulus', 'happens', 'in-phase', '(or', 'time', 'as)', 'application', 'stress', 'whereas', 'loss', 'modulus', 'happens', 'out-of-phase', 'application', 'stress', 'because', 'would', 'take', 'time', 'chains', 'move', 'new', 'confirmation', 'subjected', 'stress', 'strain', 'actually', 'lags', 'behind', 'stress', 'experiments', 'delta', 'refers', 'phase', 'lag', 'amount', 'time', 'application', 'stress', 'observation', 'maximum', 'strain', 'you', 'may', 'remember', 'sine', 'curve', 'cosine', 'curve', 'phase', 'storage', 'modulus', 'described', 'proportional', 'cosδ', 'whereas', 'loss', 'modulus', 'proportional', 'sinδ', 'the', 'ratio', 'cosδ', 'sinδ', 'tanδ', 'why', 'tanδ', 'peak', 'glass', 'transition', 'temperature', 'clearly', 'chains', 'begin', 'move', 'freely', 'loss', 'modulus', 'increases', 'consequently', 'material', 'also', 'becomes', 'less', 'stiff', 'rubbery', 'the', 'storage', 'modulus', 'drops', 'if', 'tan', 'delta', 'ratio', 'loss', 'modulus', 'storage', 'modulus', 'increase', 'point', '--', 'why', 'drop', 'thats', 'loss', 'modulus', 'refers', 'energy', 'loss', 'material', 'gotten', 'softer', 'less', 'stress', '(and', 'less', 'energy)', 'put', 'sample', 'first', 'place', 'energy', 'loss', 'also', 'gets', 'smaller', 'as', 'result', 'tan', 'delta', 'goes', 'glass', 'transition', 'drops', 'shortly', 'beyond', 'point', 'problem', 'pp91', 'estimate', 'storage', 'loss', 'modulus', 'glassy', 'phase', 'rubbery', 'phase', 'following', 'cases', 'problem', 'pp92', 'estimate', 'tg', 'following', 'cases', 'problem', 'pp11', 'a)', 'with', 'shorter', 'elution', 'time', 'i', 'higher', 'molecular', 'weight', 'ii', 'the', 'narrower', 'peak', 'means', 'i', 'narrower', 'dispersity', 'ii', 'b)', 'with', 'shorter', 'elution', 'time', 'iii', 'higher', 'molecular', 'weight', 'iv', 'the', 'wider', 'peak', 'means', 'iii', 'broader', 'dispersity', 'iv', 'c)', 'with', 'higher', 'elution', 'time', 'v', 'lower', 'molecular', 'weight', 'vi', 'the', 'wider', 'peak', 'means', 'v', 'broader', 'dispersity', 'vi', 'problem', 'pp12', 'a)', 'repeat', 'unit', '109', 'x', '(c4h6o2)', '109', 'x', '8609', 'gmol', '938381', 'gmol', 'end', 'groups', 'c9h11', 'br', '11919', '799', '19909', 'gmol', 'total', '938381', '19909', '958290', 'gmol', 'b)', 'repeat', 'unit', '763', 'x', '(c8h8)', '763', 'x', '10415', 'gmol', '7946645', 'gmol', 'end', 'groups', 'c4h9', 'h', '5712', '1008', '5813', 'gmol', 'total', '7946645', '5813', '7952458', 'gmol', 'c)', 'repeat', 'unit', '48', 'x', '(c4h6o2)', '48', 'x', '8609', 'gmol', '413232', 'gmol', 'end', 'groups', 'c4h9', 'c7h5s2', '5712', '15326', '21038', 'gmol', 'total', '413232', '21038', '434270', 'gmol', 'problem', 'pp13', 'the', 'ratio', 'repeat', 'unit', 'integral', 'per', 'proton', 'end', 'group', 'proton', 'per', 'integral', 'gives', 'degree', 'polymerization', 'we', 'could', 'take', 'entire', 'integration', 'end', 'group', 'divide', 'entire', 'number', 'protons', 'group', 'select', 'one', 'position', 'represent', 'end', 'group', 'similarly', 'select', 'one', 'position', 'represent', 'repeat', 'unit', 'a)', 'repeat', 'unit', 'integral', 'per', 'proton', '360', '2h', '18', 'end', 'group', 'integral', 'per', 'proton', '032', '2h', '016', 'degree', 'polymerization', '18016', '112', 'b)', 'repeat', 'unit', 'integral', 'per', 'proton', '260', '1h', '26', 'end', 'group', 'integral', 'per', 'proton', '132', '9h', '015', 'degree', 'polymerization', '26015', '173', 'c)', 'repeat', 'unit', 'integral', 'per', 'proton', '540', '4h', '135', 'end', 'group', 'integral', 'per', 'proton', '049', '1h', '049', 'degree', 'polymerization', '135049', '28', 'problem', 'pp21', 'α', 'slope', 'riserun', 'thats', 'approximately', '40-24/66-44', '16/22', '073.', 'k', 'y-intercept', 'the', 'equation', 'straight', 'line', 'mx', 'b', 'case', '073x', 'b', 'if', 'choose', 'point', 'line', '(xy)', '(49', '20)', 'substitute', 'values', 'x', 'get', 'b', 'so', '20', '073(49)', 'b', 'b', '20', '-', '356', '-156', 'problem', 'pp22', 'if', 'molecular', 'weight', 'million', 'gmol', 'log(mw)', '6', 'interpolating', 'log(η)', '4', 'η', '10000', 'mlg', 'problem', 'pp23', 'if', 'intrinsic', 'viscosity', 'η', '800', 'mlg', 'log(η)', '29', 'interpolating', 'log(mw)', '51', 'mw', '126000', 'gmol', 'problem', 'pp24', 'ethylene', 'glycol', 'form', 'hydrogen', 'bonds', 'either', 'end', 'molecule', 'forming', 'supramolecular', 'assembly', 'much', 'like', 'polymer', 'as', 'result', 'much', 'greater', 'drag', 'solution', 'higher', 'viscosity', 'problem', 'pp31', 'honey', 'concentrated', 'solution', 'simple', 'sugars', 'small', 'molecules', 'molasses', 'although', 'similar', 'honey', 'ways', 'also', 'contain', 'starches', 'polymers', 'this', 'polymeric', 'content', 'leads', 'shear-thinning', 'behavior', 'problem', 'pp41', 'a)', 'there', 'glass', 'transition', 'around', '-18°c', 'b)', 'there', 'melting', 'point', 'around', '125°c', 'c)', 'there', 'glass', 'transition', 'around', '-4°c', 'd)', 'there', 'glass', 'transition', 'around', '117°c', 'melting', 'point', 'around', '146°c', 'problem', 'pp51', 'a)', 'tg', 'observed', 'around', '78°c', 'tm', 'observed', 'around', '117°c', 'tc', 'observed', 'around', '104°c', 'b)', 'tg', 'observed', 'around', '134°c', 'tm', 'observed', 'around', '167°c', 'tc', 'observed', 'sample', 'failed', 'crystallize', 'remained', 'amorphous', 'solid', 'c)', 'tm', 'observed', 'around', '194°c', 'tc', 'observed', 'around', '187°c', 'tg', 'observed', 'probably', 'occurs', '150°c', 'd)', 'tg', 'observed', 'around', '123°c', 'tm', 'observed', 'the', 'experiment', 'checked', 'much', 'higher', 'tg', '(over', 'hundred', 'degrees)', 'material', 'may', 'amorphous', 'solid', 'problem', 'pp61', 'problem', 'pp62', 'a)', '(2', 'x', '314)', '040', '16', 'å', '(2', 'x', '314)', '070', '90', 'å', 'b)', '(2', 'x', '314)', '025', '25', 'å', '(2', 'x', '314)', '085', '74', 'å', 'c)', '(2', 'x', '314)', '025', '25', 'å', '(2', 'x', '314)', '052', '12', 'å', '(2', 'x', '314)', '066', '10', 'å', 'problem', 'pp71', 'a)', 'ultimate', 'tensile', 'strength', '800', 'pa', 'strain', 'break', '55', 'b)', 'ultimate', 'tensile', 'strength', '750', 'pa', 'strain', 'break', '215', 'c)', 'ultimate', 'tensile', 'strength', '220', 'pa', 'strain', 'break', '120', 'problem', 'pp72', 'a)', 'e', 'σ', 'ε', '180', 'pa', '030', '600', 'pa', 'b)', 'e', 'σ', 'ε', '450', 'pa', '015', '3000', 'pa', 'c)', 'e', 'σ', 'ε', '50', 'pa', '025', '200', 'pa', 'd)', 'e', 'σ', 'ε', '75', 'pa', '030', '250', 'pa', 'problem', 'pp73', 'problem', 'pp81', '1', 'pa', '1', 'pa', '1', 'kpa', '1000', 'pa', '1', 'mpa', '1000000', 'pa', '1gpa', '1000000000', 'pa', 'problem', 'pp91', 'a)', 'glassy', 'storage', 'modulus', '15', 'mpa', 'loss', 'modulus', '70', 'kpa', 'rubbery', 'storage', 'modulus', '7', 'mpa', 'loss', 'modulus', '80', 'kpa', 'b)', 'glassy', 'storage', 'modulus', '600', 'kpa', 'loss', 'modulus', '140', 'kpa', 'rubbery', 'storage', 'modulus', '130', 'kpa', 'loss', 'modulus', '150', 'kpa', 'c)', 'glassy', 'storage', 'modulus', '320', 'kpa', 'loss', 'modulus', '80', 'kpa', 'rubbery', 'storage', 'modulus', '70', 'kpa', 'loss', 'modulus', '70', 'kpa', 'problem', 'pp92', 'a)', '89', '°c', 'b)', '170', '°c', 'c)', '124', '°c']
pract_list = []
test_ravve_list = ['principles', 'polymer', 'chemistry', 'a', 'ravve', 'third', 'edition', 'principles', 'polymer', 'chemistry', 'a', 'ravve', 'niles', 'il', 'usa', 'isbn', '978-1-4614-2211-2', 'doi', '101007978-1-4614-2212-9', 'springer', 'new', 'york', 'heidelberg', 'dordrecht', 'london', 'isbn', '978-1-4614-2212-9', '(ebook)', 'library', 'congress', 'control', 'number', '2012934695', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', 'this', 'work', 'subject', 'copyright', 'all', 'rights', 'reserved', 'publisher', 'whether', 'whole', 'part', 'material', 'concerned', 'speciﬁcally', 'rights', 'translation', 'reprinting', 'reuse', 'illustrations', 'recitation', 'broadcasting', 'reproduction', 'microﬁlms', 'physical', 'way', 'transmission', 'information', 'storage', 'retrieval', 'electronic', 'adaptation', 'computer', 'software', 'similar', 'dissimilar', 'methodology', 'known', 'hereafter', 'developed', 'exempted', 'legal', 'reservation', 'brief', 'excerpts', 'connection', 'reviews', 'scholarly', 'analysis', 'material', 'supplied', 'speciﬁcally', 'purpose', 'entered', 'executed', 'computer', 'system', 'exclusive', 'use', 'purchaser', 'work', 'duplication', 'publication', 'parts', 'thereof', 'permitted', 'provisions', 'copyright', 'law', 'publisher’s', 'location', 'current', 'version', 'permission', 'use', 'must', 'always', 'obtained', 'springer', 'permissions', 'use', 'may', 'obtained', 'rightslink', 'copyright', 'clearance', 'center', 'violations', 'liable', 'prosecution', 'respective', 'copyright', 'law', 'the', 'use', 'general', 'descriptive', 'names', 'registered', 'names', 'trademarks', 'service', 'marks', 'etc', 'publication', 'imply', 'even', 'absence', 'speciﬁc', 'statement', 'names', 'exempt', 'relevant', 'protective', 'laws', 'regulations', 'therefore', 'free', 'general', 'use', 'while', 'advice', 'information', 'book', 'believed', 'true', 'accurate', 'date', 'publication', 'neither', 'authors', 'editors', 'publisher', 'accept', 'legal', 'responsibility', 'errors', 'omissions', 'may', 'made', 'the', 'publisher', 'makes', 'warranty', 'express', 'implied', 'respect', 'material', 'contained', 'herein', 'printed', 'acid-free', 'paper', 'springer', 'part', 'springer', 'sciencebusiness', 'media', '(wwwspringercom)', 'preface', 'this', 'book', 'unlike', 'ﬁrst', 'second', 'editions', 'primarily', 'aimed', 'textbook', 'graduate', 'course', 'polymer', 'chemistry', 'reference', 'book', 'practicing', 'polymer', 'chemists', 'the', 'ﬁrst', 'second', 'editions', 'hand', 'aimed', 'graduate', 'undergraduate', 'students', 'comments', 'reviewers', 'ﬁrst', 'two', 'editions', 'detailed', 'use', 'undergraduates', 'prompted', 'change', 'the', 'book', 'describes', 'organic', 'physical', 'chemistry', 'polymers', 'this', 'includes', 'physical', 'properties', 'polymers', 'syntheses', 'subsequent', 'use', 'plastics', 'elastomers', 'reagents', 'functional', 'materials', 'the', 'syntheses', 'characterized', 'according', 'chemical', 'mechanism', 'reactions', 'kinetics', 'scope', 'utility', 'whenever', 'possible', 'descriptions', 'industrial-', 'scale', 'preparations', 'included', 'emphasis', 'placed', 'reaction', 'parameters', 'preparation', 'polymeric', 'materials', 'utilization', 'reagents', 'also', 'possible', 'industrial', 'trade', 'names', 'polymeric', 'materials', 'included', 'familiarize', 'students', 'this', 'book', 'also', 'describes', 'chemical', 'modiﬁcations', 'polymers', 'a', 'separate', 'chapter', 'dedicated', 'utilization', 'polymers', 'reagents', 'supports', 'catalyst', 'drug', 'release', 'electricity', 'conductors', 'photonic', 'materials', 'use', 'book', 'requires', 'proﬁciency', 'organic', 'physical', 'chemistries', 'while', 'prior', 'knowledge', 'polymer', 'chemistry', 'elementary', 'level', 'required', 'exposure', 'subject', 'undergraduate', 'level', 'would', 'probably', 'helpful', 'each', 'topic', 'however', 'presented', 'assump-', 'tion', 'reader', 'prior', 'knowledge', 'subject', 'this', 'book', 'consists', 'ten', 'chapters', 'a', 'separate', 'chapter', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'added', 'in', 'previous', 'editions', 'subject', 'part', 'introduction', 'handled', 'limited', 'scale', 'this', 'book', 'aimed', 'graduate', 'students', 'however', 'rigorous', 'treatment', 'required', 'the', 'kinetic', 'treatment', 'expanded', 'chapters', 'deal', 'polymer', 'syntheses', 'in', 'addition', 'discussions', 'thermodynamics', 'reactions', 'added', 'chapters', 'in', 'earlier', 'two', 'editions', '5¼', 'diskette', 'included', 'end', 'books', 'computer', 'programs', 'pascal', 'these', 'programs', 'offer', 'students', 'experience', 'calculating', 'results', 'size', 'exclusion', 'chromatograph', 'determine', 'sequence', 'distribution', 'polymers', 'nmr', 'spectra', 'others', 'these', 'programs', 'omitted', 'however', 'considerably', 'better', 'programs', 'written', 'professional', 'computer', 'scientists', 'commercially', 'available', 'this', 'book', 'like', 'earlier', 'editions', 'dedicated', 'scientists', 'whose', 'names', 'appear', 'references', 'niles', 'il', 'usa', 'a', 'ravve', 'v', 'contents', '1', 'introduction', 'nomenclature', '11', 'brief', 'historical', 'introduction', '12', 'deﬁnitions', '13', 'nomenclature', 'polymers', '131', 'nomenclature', 'chain-growth', 'polymers', '132', 'nomenclature', 'step-growth', 'polymers', '14', 'steric', 'arrangement', 'macromolecules', 'appendix', 'review', 'questions', 'references', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', '21', 'structure', 'property', 'relationship', 'organic', 'polymers', '211', 'effects', 'dipole', 'interactions', 'induction', 'forces', 'polymers', '212', '22', 'the', 'amorphous', 'state', '221', 'the', 'glass', 'transition', 'glassy', 'state', '222', 'elasticity', '223', 'rheology', 'viscoelasticity', 'polymeric', 'materials', '23', 'the', 'crystalline', 'state', '231', 'crystallization', 'melt', '232', 'crystallization', 'solution', '233', 'spherulitic', 'growth', '24', 'the', 'mesomorphic', 'state', 'liquid', 'crystal', 'polymers', '25', 'orientation', 'polymers', '26', 'solutions', 'polymers', '261', 'radius', 'gyration', '262', 'the', 'thermodynamics', 'polymer', 'solutions', '27', 'molecular', 'weights', 'molecular', 'weight', 'determinations', '271', 'molecular', 'weight', 'averages', '272', 'methods', 'measuring', 'molecular', 'weights', 'polymers', '28', 'optical', 'activity', 'polymers', 'review', 'questions', 'references', '1', '1', '2', '7', '7', '11', '11', '13', '13', '15', '17', '17', '17', '18', '21', '21', '24', '27', '34', '34', '36', '38', '43', '47', '48', '48', '50', '51', '51', '53', '60', '61', '66', 'vii', 'viii', 'contents', '32', '31', '35', '36', '33', '34', '3', 'free-radical', 'chain-growth', 'polymerization', 'free-radical', 'chain-growth', 'polymerization', 'process', '311', 'kinetic', 'relationships', 'free-radical', 'polymerizations', 'reactions', 'leading', 'formation', 'initiating', 'free', 'radicals', 'thermal', 'decomposition', 'azo', 'compound', 'peroxides', '321', 'bimolecular', 'initiating', 'systems', '322', 'boron', 'metal', 'alkyl', 'initiators', 'free-radical', 'polymerizations', '323', 'photochemical', 'initiators', '324', 'initiation', 'polymerization', 'radioactive', 'sources', '325', 'electron', 'beams', 'capture', 'free', 'radicals', 'monomers', 'propagation', 'steric', 'polar', 'resonance', 'effects', 'propagation', 'reaction', '341', 'effect', 'reaction', 'medium', '342', 'ceiling', 'temperature', '343', 'autoacceleration', '344', 'polymerization', 'monomers', 'multiple', 'double', 'bonds', '345', 'the', 'termination', 'reaction', 'copolymerization', 'reactivity', 'ratios', '361', 'q', 'e', 'scheme', '362', '363', 'solvent', 'effect', 'copolymerization', 'terpolymerization', '37', 'allylic', 'polymerization', '38', 'inhibition', 'retardation', '39', '310', 'thermal', 'polymerization', '311', 'donor–acceptor', 'complexes', 'copolymerization', '312', 'polymerization', 'complexes', 'lewis', 'acids', '313', 'steric', 'control', 'free-radical', 'polymerization', '314', 'controlled“living”', 'free-radical', 'polymerization', '3141', 'cobalt', 'mediated', 'polymerizations', '3142', 'atom', 'transfer', 'radical', 'polymerizations', '3143', 'nitroxide-mediated', 'radical', 'polymerizations', '3144', 'reversible', 'addition-fragmentation', 'chain', 'transfer', 'polymerization', '3145', 'special', 'types', 'controlled“living”', 'polymerizations', '3146', 'kinetics', 'controlledliving', 'free-radical', 'polymerizations', '315', 'thermodynamics', 'free-radical', 'polymerization', 'reaction', '3151', 'effects', 'monomer', 'structure', 'thermodynamics', 'polymerization', '131', '3152', 'thermodynamics', 'constrains', 'free-radical', 'polymerization', 'reaction', '316', 'polymer', 'preparation', 'techniques', 'review', 'questions', 'references', '4', 'ionic', 'chain-growth', 'polymerization', 'the', 'chemistry', 'ionic', 'chain-growth', 'polymerization', '41', 'kinetics', 'ionic', 'chain-growth', 'polymerization', '42', '69', '69', '69', '72', '72', '76', '79', '79', '80', '80', '84', '84', '87', '88', '89', '90', '92', '96', '97', '99', '100', '101', '102', '103', '106', '107', '111', '113', '114', '116', '117', '121', '126', '129', '130', '131', '132', '132', '139', '143', '151', '151', '152', 'contents', '45', '44', '43', 'cationic', 'polymerization', '431', 'two', 'electron', 'transposition', 'initiation', 'reactions', '432', 'one', 'electron', 'transposition', 'initiation', 'reactions', '433', 'propagation', 'cationic', 'polymerization', '434', 'termination', 'reactions', 'cationic', 'polymerizations', '435', 'living', 'cationic', 'polymerizations', '436', 'thermodynamics', 'cationic', 'polymerization', 'anionic', 'polymerization', 'oleﬁns', 'initiation', 'anionic', 'chain-growth', 'polymerization', '441', '442', 'propagation', 'anionic', 'chain-growth', 'polymerization', '443', 'termination', 'anionic', 'polymerization', '444', 'thermodynamics', 'anionic', 'polymerization', 'coordination', 'polymerization', 'oleﬁns', '451', 'heterogeneous', 'ziegler–natta', 'catalysts', '452', 'homogeneous', 'ziegler–natta', 'catalysts', '453', 'steric', 'control', 'polymerization', 'conjugated', 'dienes', '454', 'post', 'ziegler', 'natta', 'coordination', 'polymerization', 'oleﬁns', '455', 'effect', 'lewis', 'bases', '456', 'terminations', 'coordination', 'polymerizations', '457', 'reduced', 'transition', 'metal', 'catalysts', 'support', 'isomerization', 'polymerizations', 'coordination', 'catalysts', '458', 'polymerization', 'aldehydes', '461', 'cationic', 'polymerization', 'aldehydes', '462', 'anionic', 'polymerization', 'aldehydes', '463', 'polymerization', 'unsaturated', 'aldehydes', '464', 'polymerizations', 'di', 'aldehydes', 'polymerization', 'ketones', 'isocyanates', 'copolymerizations', 'ionic', 'mechanism', 'group', 'transfer', 'polymerization', '47', '48', '49', '410', 'conﬁgurational', 'statistics', 'propagation', 'mechanism', 'chain-growth', '46', 'polymerization', '411', 'thermodynamics', 'equilibrium', 'polymerization', 'review', 'questions', 'references', '51', '52', '53', '5', 'ring-opening', 'polymerizations', 'chemistry', 'ring-opening', 'polymerizations', 'kinetics', 'ring-opening', 'polymerization', 'polymerization', 'oxiranes', '531', 'cationic', 'polymerization', '532', 'anionic', 'polymerization', '533', 'polymerization', 'coordination', 'mechanism', '534', 'steric', 'control', 'polymerizations', 'oxiranes', 'polymerization', 'oxetanes', '541', 'the', 'initiation', 'reaction', '542', 'the', 'propagation', 'reaction', 'polymerization', 'tetrahydrofurans', '551', 'the', 'initiation', 'reaction', '552', 'the', 'propagation', 'reaction', '553', 'the', 'termination', 'reaction', '54', '55', 'ix', '154', '155', '163', '167', '177', '178', '181', '182', '182', '191', '198', '201', '201', '202', '207', '209', '211', '219', '219', '219', '220', '221', '221', '223', '226', '227', '228', '228', '231', '234', '240', '241', '243', '253', '253', '253', '255', '255', '259', '261', '264', '266', '267', '268', '269', '270', '271', '272', 'x', 'contents', '58', '59', '56', '57', 'polymerization', 'oxepanes', 'ring-opening', 'polymerizations', 'cyclic', 'acetals', '571', 'polymerization', 'trioxane', '572', 'polymerization', 'dioxolane', '573', 'polymerization', 'dioxopane', 'other', 'cyclic', 'acetals', 'polymerization', 'lactones', '581', 'cationic', 'polymerization', '582', 'anionic', 'polymerization', 'lactones', '583', 'polymerization', 'lactones', 'coordination', 'mechanism', '584', 'special', 'catalysts', 'polymerizations', 'lactones', 'polymerizations', 'lactams', '591', 'cationic', 'polymerization', 'lactams', '592', 'anionic', 'polymerization', 'lactams', '593', 'hydrolytic', 'polymerization', 'lactams', '510', 'polymerization', 'n-carboxy-a-amino', 'acid', 'anhydrides', '511', 'metathesis', 'polymerization', 'alicyclics', '512', 'polymerization', 'cyclic', 'amines', '513', 'ring-opening', 'polymerizations', 'cyclic', 'sulﬁdes', '514', 'copolymerization', 'cyclic', 'monomers', '515', 'spontaneous', 'alternating', 'zwitterion', 'copolymerizations', '516', 'ring-opening', 'polymerizations', 'free', 'radical', 'mechanism', '517', 'thermodynamics', 'ring-opening', 'polymerization', 'review', 'questions', 'references', '61', '6', 'common', 'chain-growth', 'polymers', 'polyethylene', 'related', 'polymers', '611', 'preparation', 'polyethylene', 'free-radical', 'mechanism', '612', 'preparation', 'polyethylene', 'coordination', 'mechanism', '613', 'commercial', 'high-density', 'polyethylene', 'properties', '62', '63', '64', '65', '66', 'manufacture', '614', 'materials', 'similar', 'polyethylene', 'polypropylene', '621', 'manufacturing', 'techniques', '622', 'syndiotactic', 'polypropylene', 'polyisobutylene', 'poly(a-oleﬁn)s', '641', 'properties', 'poly(a-oleﬁn)s', '642', 'poly(butene-1)', '643', 'poly(4-methyl', 'pentene-1)', 'copolymers', 'ethylene', 'propylene', '651', 'ethylene', 'propylene', 'elastomers', '652', 'copolymers', 'ethylene', 'a-oleﬁns', 'ethylene', 'carbon', 'monoxide', '653', 'copolymers', 'propylene', 'dienes', '654', 'copolymers', 'ethylene', 'vinyl', 'acetate', 'ionomers', '655', 'homopolymers', 'conjugated', 'dienes', '661', 'polybutadiene', '662', 'polyisoprene', '273', '273', '274', '276', '277', '278', '278', '280', '281', '283', '284', '285', '290', '296', '297', '301', '307', '309', '311', '312', '316', '318', '319', '322', '329', '329', '329', '332', '335', '338', '339', '342', '342', '343', '345', '345', '345', '345', '347', '347', '348', '351', '351', '351', '352', '352', '356', 'contents', '67', 'methyl', 'rubber', 'poly(23-dimethylbutadiene)', 'chloroprene', 'rubber', 'poly(2-chloro-13-butadiene)', '68', '69', 'special', 'polymers', 'dienes', '610', 'cyclopolymerization', 'conjugated', 'dienes', '611', 'copolymers', 'dienes', '6111', 'gr-s', 'rubber', '6112', 'gr-n', 'rubber', '612', 'polystyrene', 'polystyrene-like', 'polymers', '6121', 'preparation', 'polystyrene', 'free-radical', 'mechanism', '6122', 'polystyrene', 'prepared', 'ionic', 'chain-growth', 'polymerization', '6123', 'polymers', 'substituted', 'styrenes', '613', 'copolymers', 'styrene', '6131', 'high-impact', 'polystyrene', '6132', 'abs', 'resins', '6133', 'copolymers', 'styrene', 'maleic', 'anhydride', '614', 'polymers', 'acrylic', 'methacrylic', 'esters', '6141', 'polymerizations', 'acrylic', 'methacrylic', 'esters', '6142', 'acrylic', 'elastomers', '6143', 'thermoplastic', 'thermoset', 'acrylic', 'resins', '615', 'acrylonitrile', 'methacrylonitrile', 'polymers', '616', 'polyacrylamide', 'poly(acrylic', 'acid)', 'poly(methacrylic', 'acid)', '617', 'halogen-bearing', 'polymers', '6171', 'polytetraﬂuoroethylene', '6172', 'polychlorotriﬂuoroethylene', '6173', 'poly(vinylidine', 'ﬂuoride)', '6174', 'poly(vinyl', 'ﬂuoride)', '6175', 'copolymers', 'fluorooleﬁns', '6176', 'miscellaneous', 'fluorine', 'containing', 'chain-growth', 'polymers', '6177', 'poly(vinyl', 'chloride)', '6178', 'poly(vinylidine', 'chloride)', '618', 'poly(vinyl', 'acetate)', '619', 'poly(vinyl', 'alcohol)', 'poly(vinyl', 'acetal)s', 'review', 'questions', 'references', '72', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '71', 'mechanism', 'kinetics', 'step-growth', 'polymerization', '711', 'reactions', 'functional', 'groups', '712', 'kinetic', 'considerations', '713', 'ring', 'formation', 'step-growth', 'polymerizations', '714', 'techniques', 'polymer', 'preparation', 'polyesters', '721', 'linear', 'saturated', 'polyesters', '722', 'linear', 'unsaturated', 'polyesters', '723', 'network', 'polyesters', 'surface', 'coatings', '724', 'polycarbonates', '725', 'polyesters', 'lactones', 'polyamides', '731', 'nylons', '732', 'fatty', 'polyamides', '73', 'xi', '358', '358', '360', '360', '361', '361', '363', '364', '364', '365', '367', '370', '370', '371', '372', '372', '373', '375', '376', '379', '381', '382', '382', '383', '383', '383', '384', '385', '386', '389', '390', '391', '393', '396', '403', '403', '403', '405', '410', '412', '412', '412', '424', '425', '427', '428', '430', '430', '441', 'xii', 'contents', '77', '78', '79', '74', '75', '76', 'special', 'reactions', 'formation', 'polyamides', '733', '734', 'aromatic', 'polyamides', 'aromatic', 'polyamide-imides', 'aromatic', 'polyester-imides', 'polyimides', 'polyethers', 'poly(phenylene', 'oxide)s', '761', '762', 'phenoxy', 'polymers', 'polyacetals', 'polyketals', 'poly(p-xylylene)s', 'sulfur-containing', 'polymers', 'polysulfones', '791', 'polythiols', 'polymercaptans', '792', '710', 'polyurethanes', '7101', 'preparations', 'polyfunctional', 'isocyanates', '7102', 'commercial', 'polyisocyanates', '7103', 'chemical', 'reactions', 'isocyanates', '7104', 'the', 'effect', 'catalysts', '7105', 'polyurethane', 'fibers', '7106', 'polyurethane', 'elastomers', '7107', 'polyurethane', 'foams', '711', 'epoxy', 'resins', '7111', 'preparation', 'commercial', 'epoxy', 'resins', '7112', 'the', 'cross-linking', 'reactions', '7113', 'cycloaliphatic', 'epoxides', '712', 'phenol-formaldehyde', 'resins', '7121', 'resols', '7122', 'novolacs', '7123', 'ammonia-catalyzed', 'phenolic', 'resins', '7124', 'typical', 'commercial', 'preparations', '713', 'amino', 'polymers', '7131', 'urea-formaldehyde', 'resins', '7132', 'melamine-formaldehyde', 'resins', '714', 'silicone', 'polymers', '7141', 'polysiloxanes', '7142', 'silicone', 'elastomers', '7143', 'polysiloxane', 'coating', 'resins', '7144', 'fluorosilicone', 'elastomers', '7145', 'polyarylsiloxanes', '(also', 'see', 'sect', '7174)', '715', 'polysilanes', '716', 'phosphonitrile', 'polymers', '717', 'high-performance', 'polymers', '7171', 'fluorine', 'containing', 'aromatic', 'polymers', '7172', 'polyphenylene', '7173', 'diels–alder', 'polymers', '7174', 'silicon-containing', 'aromatic', 'polymers', '7175', 'direct', 'condensation', 'polymers', '7176', 'oligomers', 'terminal', 'functional', 'groups', '7177', 'cardo', 'polymers', '7178', 'double-stranded', 'polymers', '7179', 'poly(arylene', 'ether)s', 'poly(arylene', 'ether', 'ketone)s', '441', '443', '447', '450', '456', '456', '459', '459', '461', '463', '463', '465', '468', '469', '469', '470', '471', '472', '473', '474', '474', '475', '476', '482', '483', '483', '487', '490', '491', '492', '492', '493', '494', '494', '496', '498', '498', '499', '499', '500', '502', '502', '504', '505', '511', '512', '514', '517', '517', '520', 'contents', '718', 'dendrimers', 'polyrotaxanes', '7181', 'dendrimers', 'hyperbranched', 'polymers', '7182', 'polyrotaxanes', '719', 'thermodynamics', 'step-growth', 'polymerization', 'review', 'questions', 'references', '81', '82', '83', '84', '85', '8', 'naturally', 'occurring', 'polymers', 'naturally', 'occurring', 'polymers', 'polysaccharides', 'hemicelluloses', '821', 'starch', '822', 'cellulose', '823', '824', 'miscellaneous', 'polysaccharides', 'lignin', 'polyisoprene', 'proteins', 'a-amino', 'acids', '851', 'structures', 'chemistry', 'proteins', '852', 'synthetic', 'methods', 'preparation', 'polypeptides', '853', '854', 'chemical', 'modiﬁcation', 'proteins', 'nucleic', 'acids', 'dna', 'rna', '861', '862', 'synthetic', 'methods', 'preparation', 'nucleic', 'acids', '87', 'polyalkanoates', 'review', 'questions', 'references', '86', '91', '92', '9', 'organic', 'reactions', 'polymers', 'reactivity', 'macromolecules', 'diffusion-controlled', 'reactions', '911', 'paired', 'group', 'neighboring', 'group', 'effects', '912', 'effect', 'molecular', 'size', '913', 'effects', 'changes', 'solubility', '914', 'effects', 'crystallinity', '915', '916', 'reactions', 'that', 'favor', 'large', 'molecules', 'addition', 'reactions', 'halogenation', '921', 'hydrogenation', '922', 'addition', 'carbenes', '923', 'electrophilic', 'additions', 'aldehydes', '924', '925', 'polar', 'additions', 'rearrangement', 'reactions', 'isomerization', 'reactions', '931', '932', 'cyclizations', 'intramolecular', 'rearrangements', 'substitution', 'reactions', 'substitution', 'reactions', 'saturated', 'polymeric', 'hydrocarbons', '941', 'substitution', 'reactions', 'halogen-bearing', 'polymers', '942', 'substitution', 'reactions', 'polymers', 'aromatic', 'rings', '943', 'reactions', 'acrylic', 'methacrylic', 'related', 'polymers', '944', '93', '94', 'xiii', '521', '522', '523', '524', '525', '529', '537', '537', '537', '537', '538', '539', '545', '546', '547', '547', '548', '548', '554', '556', '557', '559', '560', '561', '562', '563', '567', '567', '569', '569', '570', '570', '571', '571', '572', '572', '574', '575', '576', '577', '584', '584', '586', '590', '590', '592', '597', '606', 'xiv', 'contents', '961', '962', '963', '964', '965', '966', '967', '971', '972', '973', '974', '975', '976', '977', '945', 'substitution', 'reactions', 'poly(vinyl', 'alcohol)', '946', 'miscellaneous', 'exchange', 'reactions', '95', 'cross-linking', 'reactions', 'polymers', '951', 'vulcanization', 'elastomers', 'cross-linking', 'polymers', 'aid', 'peroxides', '952', '953', 'miscellaneous', 'cross-linking', 'reactions', 'polymers', '96', 'graft', 'copolymers', 'free-radical', 'grafting', 'chain-transferring', 'process', 'free-radical', 'grafting', 'reactions', 'polymers', 'double', 'bonds', 'preparation', 'graft', 'copolymers', 'aid', 'macromonomers', 'initiations', 'polymerizations', 'backbone', 'polymers', 'photochemical', 'syntheses', 'graft', 'copolymers', 'graft', 'copolymer', 'formation', 'aid', 'high-energy', 'radiation', 'preparation', 'graft', 'copolymers', 'ionic', 'chain-growth', 'step-growth', 'polymerization', 'reactions', '968', 'miscellaneous', 'graft', 'copolymerizations', '97', 'block', 'copolymers', 'block', 'copolyesters', 'block', 'copolyamides', 'polyurethane-polyamide', 'block', 'copolymers', 'polyamide-polyester', 'block', 'copolymers', 'polyurethane', 'ionomers', 'block', 'copolymers', 'poly(a-oleﬁn)s', 'simultaneous', 'use', 'free', 'radical', 'ionic', 'chain-growth', 'polymerizations', 'preparation', 'block', 'copolymers', 'homogeneous', 'ionic', 'copolymerization', '979', 'special', 'reactions', 'preparation', 'block', 'copolymers', '9710', 'miscellaneous', 'block', 'copolymers', '9711', 'mechanochemical', 'techniques', 'formation', 'block', 'copolymers', '98', 'processes', 'polymer', 'degradation', 'thermal', 'degradation', 'common', 'chain-growth', 'polymers', 'thermal', 'degradation', 'polyoleﬁns', 'polymers', 'conjugated', 'dienes', 'thermal', 'degradation', 'polystyrene', 'polystyrene-like', 'polymers', 'thermal', 'degradation', 'methacrylic', 'acrylic', 'polymers', 'thermal', 'degradation', 'chlorocarbon', 'fluorocarbon', 'polymers', 'thermal', 'degradation', 'poly(vinyl', 'acetate)', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', 'thermal', 'degradation', 'polyoxides', 'thermal', 'degradation', 'polyesters', 'thermal', 'degradation', 'polyamides', 'thermal', 'degradation', 'epoxy', 'resins', 'thermal', 'degradation', 'polyimides', 'polyoxidiazoles', 'polyquinoxalines', 'thermal', 'degradation', 'aromatic', 'polysulfones', '996', 'thermal', 'degradation', 'polyethers', '997', 'thermal', 'degradation', 'cellulosic', 'materials', '998', '999', 'hydrolytic', 'degradation', 'polymers', 'elevated', 'temperatures', '9910', 'oxidative', 'degradation', 'polymers', '991', '992', '993', '994', '995', '983', '984', '985', '986', '981', '982', '978', '610', '612', '614', '614', '616', '617', '617', '617', '619', '620', '622', '625', '626', '627', '630', '631', '631', '632', '633', '633', '633', '634', '635', '637', '639', '642', '643', '643', '643', '644', '646', '647', '649', '652', '652', '652', '653', '656', '658', '659', '661', '661', '661', '662', '663', 'contents', '9911', 'oxidation', 'chain-growth', 'polymers', '9912', 'oxidation', 'step-growth', 'polymers', '9913', 'photo-degradation', 'macromolecules', '9914', 'photo-oxidative', 'degradations', 'polymers', '9915', 'degradation', 'polymeric', 'materials', 'ionizing', 'radiation', 'review', 'questions', 'references', '10', 'polymeric', 'materials', 'special', 'applications', '101', 'polymer', 'supports', 'reagents', 'catalysts', 'drug', 'release', '1011', 'support', 'materials', '1012', 'special', 'gels', 'drug', 'release', '1013', 'utilization', 'support', 'materials', '102', 'electricity-conducting', 'polymers', 'photonic', 'polymers', '103', '1031', 'the', 'nature', 'light', '1032', 'quantum-mechanical', 'description', 'light', 'interaction', 'light', 'organic', 'molecules', '1033', '1034', 'energy', 'transfer', 'process', '1035', 'the', 'electron', 'transfer', 'process', '1036', 'the', 'charge', 'transfer', 'processes', 'polymers', '1037', 'the', 'antenna', 'effect', 'polymers', 'photosensitizers', 'photocross-linkable', 'polymers', '1051', 'polymers', 'that', 'photocross-link', 'formation', 'cyclobutane', 'rings', '1052', 'polymers', 'functional', 'chalcone', 'groups', '1053', 'polymers', 'functional', 'groups', 'similar', 'cinnamates', '1054', 'polymers', 'pendant', 'furan', 'groups', '1055', 'polymers', 'pendant', 'maleimide', 'groups', '1056', 'polymers', 'pendant', 'abietate', 'dibenzazepine', 'groups', '1057', 'polymers', 'that', 'cross-link', 'dimerization', 'nitrenes', 'other', 'combinations', 'free-radicals', 'form', 'covalent', 'bonds', '1058', 'polymers', 'pendant', 'azide', 'groups', '1059', 'polymers', 'designed', 'cross-link', 'upon', 'irradiation', '104', '105', '106', 'laser', 'beams', 'photo-responsive', 'polymers', '1061', 'polymers', 'harvesting', 'sun’s', 'energy', '1062', 'photo-isomerization', 'polymeric', 'materials', '1063', 'changes', 'viscosity', 'solubility', 'polymeric', 'solutions', '1064', 'application', 'optical', 'data', 'storage', '1065', 'liquid', 'crystalline', 'alignment', 'photo-conducting', 'polymers', '1071', 'photoconductive', 'polymers', 'based', 'carbazole', '1072', 'photo-conducting', 'polymers', 'that', 'are', 'not', 'based', 'carbazole', '108', 'polymer-based', 'solar', 'cells', 'review', 'questions', 'references', '107', 'xv', '663', '666', '668', '674', '677', '677', '682', '695', '695', '696', '704', '705', '710', '717', '717', '719', '719', '726', '729', '729', '732', '732', '735', '736', '743', '744', '745', '746', '746', '748', '748', '750', '750', '751', '755', '759', '760', '762', '767', '768', '771', '775', '782', '784', 'index', '791', 'chapter', '1', 'introduction', 'nomenclature', '11', 'brief', 'historical', 'introduction', 'the', 'initial', 'proof', 'existence', 'large', 'organic', 'molecules', 'supplied', 'raoult', '1', 'van’t', 'hoff', '2,', 'carried', 'cryoscopic', 'molecular', 'weight', 'determinations', 'rubber', 'starch', 'cellulose', 'nitrate', 'by', 'methods', 'developed', 'raoult', 'van’t', 'hoff', 'formulation', 'solution', 'laws', 'molecular', 'weights', '10000–40000', 'demonstrated', 'unfortunately', 'chemists', 'day', 'failed', 'appreciate', 'evidence', 'refused', 'accept', 'the', 'main', 'reason', 'response', 'inability', 'distinguish', 'macromolecules', 'colloidal', 'substances', 'could', 'obtained', 'low', 'molecular', 'weights', 'the', 'opinion', 'majority', 'day', '“raoul’s', 'solution', 'apply', 'materials', 'colloidal', 'state”', 'during', 'period', '1890–1910', 'idea', 'molecular', 'complexes', 'generally', 'accepted', '3.', 'it', 'used', 'explain', 'polymeric', 'structures', 'terms', 'physical', 'aggregates', 'small', 'molecules', 'in', 'fact', 'molecular', 'association', 'considered', 'polymerization', 'thus', 'rubber', 'example', 'assumed', 'composed', 'short', 'sequences', 'isoprene', 'units', 'either', 'chains', 'cyclic', 'structures', 'the', 'structure', 'isoprene', 'known', 'isolated', 'natural', 'rubber', '1860', 'what', 'added', 'general', 'confusion', 'fact', 'one', 'able', 'show', 'existence', 'end', 'groups', 'macromolecules', 'studied', 'this', 'enhanced', 'idea', 'rubber', 'ring-like', 'structure', 'dimethyl', 'cycloocatadiene', 'large', 'numbers', 'rings', 'assumed', 'held', 'together', 'associations', 'giving', 'rise', 'colloidal', 'materials', 'this', 'illustrated', 'follows', 'early', 'synthetic', 'polymeric', 'products', 'usually', 'discarded', 'oils', 'tars', '(“gooks”)', 'considered', 'useless', 'to', 'fair', 'chemists', 'period', 'however', 'one', 'forget', 'spite', 'general', 'attitude', 'time', 'structures', 'polymers', 'like', 'polyethylene', 'glycol', '(n', '¼', '6)', 'instance', 'ho', 'o', 'oh', 'x', 'o', 'n', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-91', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '1', '2', '1', 'introduction', 'nomenclature', 'correctly', 'assigned', '1860', 'concept', 'extending', 'structure', 'large', 'molecular', 'weights', 'continued', 'condensation', 'understood', '4–6.', 'at', 'approximately', 'time', 'poly(methacrylic', 'acid)', 'know', 'linear', 'molecule', 'cooh', 'cooh', 'cooh', 'prepared', '1880', '7.', 'but', 'cyclic', 'structure', 'assigned', 'believed', 'attached', 'cyclic', 'structures', '“partial', 'valences”', 'thereby', 'forming', 'gels', 'what', 'noteworthy', 'emil', 'fischer', 'coworkers', 'studied', 'many', 'natural', 'polymers', 'rubber', 'starch', 'polypeptides', 'cellulose', 'lignin', 'his', 'work', 'probably', 'entitles', 'called', 'spiritual', 'father', 'polymer', 'chemistry', 'during', 'period', 'willst€atter', 'worked', 'synthesis', 'polysaccharides', 'studied', 'lignin', 'enzymes', '8.', 'one', 'also', 'acknowledge', 'fact', 'spite', 'ignorance', 'structure', 'many', 'inventors', 'developed', 'ways', 'convert', 'cellulose', 'cellulose', 'acetate', 'use', 'products', 'form', 'ﬁbers', 'ﬁlms', 'coatings', 'cellulose', 'also', 'converted', 'cellulose', 'nitrate', 'used', 'prepare', 'explosives', 'products', 'at', 'turn', 'century', 'baekeland', 'formed', 'hard', 'resin', 'condensing', 'phenol', 'formaldehyde', '9,', 'the', 'evolvement', 'present-day', 'understanding', 'polymeric', 'structures', 'occurred', 'early', '1920s', 'thus', 'staudinger', 'et', 'al', 'ﬁrmly', 'established', 'existence', 'macromolecules', '10–15.', 'others', 'x-ray', 'analyses', 'careful', 'use', 'molecular', 'weight', 'determinations', 'conﬁrmed', 'ﬁndings', '16.', 'in', '1929', 'series', 'outstanding', 'investigations', 'carried', 'carothers', 'polymeric', 'materials', 'this', 'resulted', 'much', 'today’s', 'knowledge', 'understanding', '17.', 'long', '1', 'now', 'know', 'typical', 'molecule', 'polyethylene', 'contour', 'length', '25000', 'a˚', 'diameter', '49', 'a˚', 'such', 'molecule', 'compared', 'dimensions', 'long', 'snarled', 'clothesline', '75', 'ft', 'diameter', 'furthermore', 'work', 'naturally', 'occurring', 'macromolecules', 'nucleic', 'acids', 'instance', 'revealed', 'even', 'startling', 'dimensions', 'when', 'molecules', 'virus', 'dinucleic', 'acids', 'tritium-labeled', '(whose', 'nuclear', 'emission', 'less', '1', 'mm)', 'autoradiographs', 'prepared', 'showed', 'molecules', '50', 'mm', 'long', '18.', 'such', 'length', 'would', 'signify', 'molecular', 'weight', '100', 'million', 'similar', 'work', 'carried', 'dinucleic', 'acids', 'bacteria', 'revealed', 'molecular', 'weights', 'approximately', '200', 'million', 'the', 'ﬁgures', 'course', 'extremes', 'molecular', 'dimensions', 'typical', 'synthetic', 'polymers', 'range', 'molecular', 'weights', 'anywhere', 'ten', 'several', 'hundred', 'thousand', 'although', 'synthetic', 'polymers', 'molecular', 'weight', 'ranges', 'several', 'million', 'well', 'known', 'used', 'commer-', 'cially', 'interestingly', 'enough', 'many', 'polymers', 'prepared', 'use', 'organic', 'reactions', 'known', 'long', 'time', 'also', 'new', 'reactions', 'catalysts', 'still', 'discovered', 'applied', 'polymer', 'syntheses', 'it', 'probably', 'safe', 'predict', 'situation', 'undoubtedly', 'continue', 'distant', 'future', '12', 'deﬁnitions', 'the', 'word', 'polymer', 'commonly', 'understood', 'mean', 'large', 'molecule', 'composed', 'repeating', 'units', 'mers', '(from', 'greek', 'word', 'meros—part)', 'connected', 'covalent', 'bonds', 'such', 'units', 'may', 'connected', 'variety', 'ways', 'the', 'simplest', 'linear', 'polymer', 'polymer', 'units', 'connected', '12', 'deﬁnitions', '3', 'linear', 'sequence', 'like', 'beads', 'string', 'many', 'examples', 'linear', 'polymers', 'possible', 'instance', 'linear', 'polyethylene', 'n', 'repeat', 'unit', 'the', 'terminal', 'units', 'molecules', 'must', 'different', 'internal', 'ones', 'satisfy', 'valence', 'requirements', 'polyethylene', 'like', 'polymers', 'written', 'show', 'number', 'repeat', 'units', '––ch2–ch2–n(cid2)', 'using', 'number', 'letter', 'like', 'case', 'n', 'it', 'represents', 'average', 'quantity', 'mers', 'present', 'polymer', 'called', 'degree', 'polymerization', 'dp', 'average', 'number', 'repeat', 'units', 'polymeric', 'chain', 'thus', 'average', 'molecular', 'weight', 'polystyrene', 'dp', '100', '104', '(cid3)', '100', '10400', '(there', 'actually', 'several', 'ways', 'expressing', 'average', 'molecular', 'weights', 'polymers', 'this', 'discussed', 'chapter)', 'an', 'alternative', 'linear', 'polymer', 'branched', 'one', 'the', 'branches', 'long', 'short', 'low-', 'density', 'polyethylene', 'instance', 'short', 'long', 'branches', 'linear', 'branched', 'molecules', 'shown', 'fig', '11a', 'b', 'branched', 'polymers', 'also', 'star-', 'comb-shaped', '(fig', '11c', 'd)', 'in', 'addition', 'polymer', 'molecules', 'also', 'double-stranded', 'such', 'polymers', 'called', 'ladder', 'polymers', '(fig', '11e)', 'it', 'also', 'possible', 'polymers', 'semi-ladder', 'structures', '(fig', '11f)', 'when', 'branches', 'different', 'polymers', 'become', 'interconnected', 'network', 'structures', 'form', 'planar', 'networks', 'resemble', 'structure', 'graphite', 'three-dimensional', 'networks', 'space', 'networks', 'how-', 'ever', 'compared', 'diamonds', 'a', 'network', 'polymer', 'shown', 'fig', '11g', 'the', 'term', 'polymer', 'canbe', 'applied', 'molecules', 'made', 'either', 'single', 'repeating', 'structural', 'units', 'like', 'shown', 'polyethylene', 'different', 'ones', 'if', 'two', 'structural', 'units', 'term', 'copolymer', 'used', 'an', 'example', 'would', 'copolymer', 'ethyl', 'methacrylate', 'styrene', 'g', 'fig', '11', 'shapes', 'polymer', 'molecules', '(a)', 'linear', 'polymer', '(b)', 'branched', 'polymer', '(c)', 'star-shaped', 'polymer', '(d)', 'comb', 'shaped', 'polymer', '(e)', 'ladder', 'polymer', '(f)', 'semiladder', 'polymer', '(g)', 'network', 'structure', 'b', 'c', 'e', 'f', '4', '1', 'introduction', 'nomenclature', 'x', 'o', 'o', 'n', 'methyl', 'methacrylate', 'styrene', 'unit', 'unit', 'n', 'o', 'o', 'o', 'n', 'a', 'copolymer', 'also', 'linear', 'branched', 'should', 'regularity', 'repetition', 'structural', 'units', 'repetition', 'alternate', 'copolymer', 'called', 'alternating', 'copolymer', 'an', 'absence', 'regularity', 'would', 'make', 'random', 'copolymer', 'an', 'example', 'alternating', 'copolymer', 'copolymer', 'styrene', 'maleic', 'anhydride', 'in', 'addition', 'random-sequence', 'alternating', 'one', 'sometimes', 'called', 'ordered-sequence', 'also', 'block', 'copolymers', 'these', 'copolymers', 'made', 'blocks', 'individual', 'polymers', 'joined', 'covalent', 'bonds', 'an', 'example', 'block', 'copolymer', 'styrene', 'isoprene', 'polystyrene', 'block-polyisoprene', 'block', 'still', 'another', 'type', 'copolymer', 'one', 'possesses', 'backbones', 'composed', 'one', 'individual', 'polymer', 'branches', 'another', 'one', 'it', 'called', 'graft', 'copolymer', 'many', 'materials', 'formed', 'grafting', 'branch', 'polymers', 'polymer', 'backbone', 'this', 'however', 'always', 'case', 'many', 'graft', 'copolymers', 'formed', 'polymerizing', 'branch', 'copolymer', 'different', 'polymer', 'backbone', '(the', 'subject', 'block', 'graft', 'copolymers', 'discussed', 'chap', '9)', 'a', 'graft', 'copolymer', 'polyacrylonitrile', 'polyethylene', 'serve', 'example', 'cn', 'cn', 'in', 'block', 'graft', 'copolymers', 'length', 'uninterrupted', 'sequences', 'may', 'vary', 'cn', 'cn', '12', 'deﬁnitions', 'in', '1929', 'carothers', '19', 'suggested', 'separation', 'polymers', 'two', 'classes', 'condensation', 'addition', 'polymers', 'by', 'condensation', 'polymers', 'deﬁned', 'polymers', 'lack', 'certain', 'atoms', 'monomer', 'units', 'formed', 'may', 'degraded', 'chemical', 'means', 'an', 'example', 'would', 'polyester', 'o', 'ho', 'oh', 'o', 'ho', 'oh', 'o', 'o', 'o', 'o', 'he', 'also', 'deﬁned', 'addition', 'polymers', 'polymers', 'identical', 'structures', 'repeat', 'units', 'monomers', 'derived', 'according', 'deﬁnition', 'example', 'addition', 'polymer', 'polystyrene', 'formed', 'addition', 'styrene', 'monomers', '5', 'n', 'n', 'n', 'monomer', 'polymer', 'note', 'the', 'deﬁnition', 'ignores', 'loss', 'double', 'bonds', 'the', 'carothers', 'deﬁnition', 'fails', 'describe', 'polymers', 'ﬁt', 'category', 'condensation', 'polymers', 'yet', 'form', 'without', 'evolution', 'byproduct', 'an', 'example', 'polyurethane', 'form', 'reaction', 'glycol', 'diisocyanate', 'n', 'co', 'n', 'r', 'n', 'c', 'o', 'n', 'ho', 'r', 'oh', 'c', 'n', 'r', 'n', 'c', 'o', 'r', 'o', 'o', 'o', 'h', 'h', 'n', 'flory', 'proposed', 'superior', 'deﬁnition', '20.', 'it', 'based', 'reaction', 'mechanism', 'involved', 'formation', 'two', 'classes', 'polymers', 'into', 'ﬁrst', 'category', '(it', 'includes', 'condensation', 'polymers)', 'falls', 'macromolecules', 'form', 'reactions', 'occur', 'discreet', 'steps', 'they', 'therefore', 'called', 'step-growth', 'polymers', 'such', 'polymerizations', 'require', 'long', 'periods', 'time', 'macromolecule', 'form', 'usually', 'measured', 'hours', 'into', 'second', 'category', 'belong', 'polymers', 'form', 'chain', 'propagating', 'reactions', 'they', 'therefore', 'called', 'chain-growth', 'polymers', 'one', 'might', 'expect', 'such', 'reactions', 'depend', 'upon', 'presence', 'active', 'centers', 'ends', 'growing', 'chains', 'the', 'chains', 'grow', 'propagating', 'reactive', 'sites', 'inclusion', 'monomers', 'sites', 'these', 'inclusions', 'rapid', 'chain-growth', 'take', 'place', 'fraction', 'second', 'chains', 'successively', 'add', 'monomers', 'the', 'important', 'features', 'step-growth', 'polymerizations', '1', 'the', 'monomer', 'consumed', 'early', 'beginning', 'reaction', 'increase', 'molecular', '2', 'the', 'growth', 'polymeric', 'chains', 'takes', 'place', 'reactions', 'monomers', 'oligomers', 'weight', 'occurs', 'slowly', 'polymers', '6', '1', 'introduction', 'nomenclature', '3', 'there', 'termination', 'step', 'end', 'groups', 'polymers', 'reactive', 'throughout', 'process', 'polymerization', '4', 'the', 'reaction', 'mechanism', 'functions', 'throughout', 'process', 'polymerization', 'the', 'important', 'features', 'chain-growth', 'polymerizations', '1', 'chain-growth', 'takes', 'place', 'repeated', 'additions', 'monomers', 'growing', 'chains', 'reactive', 'sites', '2', 'the', 'monomer', 'consumed', 'slowly', 'present', 'throughout', 'process', 'polymerization', '3', 'there', 'two', 'distinct', 'mechanisms', 'polymer', 'formations', 'these', 'initiation', 'propagation', '4', 'in', 'majority', 'cases', 'also', 'termination', 'step', 'when', 'polymerization', 'reaction', 'takes', 'place', 'three', 'dimensions', 'progressed', 'certain', 'point', 'gelation', 'occurs', 'this', 'well-deﬁned', 'change', 'polymerization', 'known', 'gel', 'point', 'at', 'point', 'reaction', 'mixture', 'changes', 'viscous', 'liquid', 'elastic', 'gel', 'before', 'gelation', 'polymer', 'soluble', 'fusible', 'after', 'however', 'neither', 'soluble', 'fusible', 'this', 'result', 'restraining', 'effects', 'three-dimensional', 'space', 'networks', 'another', 'classiﬁ-', 'cation', 'polymers', 'also', 'possible', 'it', 'based', 'whether', 'material', 'form', 'crosslinked', 'gelled', 'networks', 'the', 'polymers', 'eventually', 'reach', 'gelation', 'called', 'thermosetting', 'such', 'polymers', 'also', 'called', 'crosslinkable', 'polymers', 'once', 'past', 'gelation', 'raising', 'temperature', 'longer', 'attain', 'plasticity', 'molecules', 'longer', 'move', 'past', 'for', 'reason', 'longer', 'dissolved', 'solvent', 'polymers', 'never', 'gel', 'become', 'crosslinked', 'called', 'thermoplastic', 'such', 'polymers', 'always', 'reﬂowed', 'upon', 'application', 'heat', 'they', 'also', 'dissolved', 'appropriate', 'solvents', 'the', 'wiggly', 'lines', 'illustration', 'imply', 'polymer', 'extends', 'directions', 'the', 'illustration', 'one', 'thermoset', 'polymer', 'formed', 'step-growth', 'mechanism', 'it', 'also', 'possible', 'form', 'crosslinked', 'polymers', 'chain-growth', 'mechanism', 'this', 'requires', 'presence', 'polymerization', 'mixture', 'comonomer', 'possesses', 'multiple', 'function-', 'ality', 'copolymerization', 'styrene', 'comonomer', 'like', 'divinyl', 'benzene', 'serve', 'example', 'k', 'an', 'oligomer', 'low', 'molecular', 'weight', 'polymer', 'it', 'consists', 'small', 'number', 'mers', 'the', 'deﬁnition', 'telomer', 'chain-growth', 'polymer', 'composed', 'molecules', 'end', 'groups', 'consisting', 'different', 'species', 'monomer', 'units', 'telomers', 'form', 'either', 'free', 'radical', 'ionic', 'chain-growth', 'polymerization', 'mechanism', '13', 'nomenclature', 'polymers', '7', 'telechelic', 'polymers', 'macromolecules', 'reactive', 'functional', 'groups', 'terminal', 'ends', 'chains', 'an', 'example', 'telechelic', 'polymer', 'polybutadiene', 'carboxylic', 'acid', 'end', 'groups', 'hooc', 'cooh', 'n', 'into', 'special', 'category', 'placed', 'starburst', 'dendrimer', 'polymers', 'these', 'molecules', 'formed', 'growing', 'three', 'dimensions', 'these', 'materials', 'often', 'possess', 'radially', 'symmetrical', 'star-shaped', 'structures', 'successive', 'cascades', 'branched', 'polymer', 'structures', 'for', 'additional', 'discussions', 'see', 'chap', '6', 'another', 'group', 'polymers', 'rotaxanes', 'they', 'discussed', 'chap', '7', 'the', 'materials', 'consist', 'polymeric', 'chains', 'threaded', 'macrocycles', 'z', 'n', 'tables', '11', '12', 'illustrate', 'common', 'chain-growth', 'step-growth', 'polymers', 'well', 'monomers', 'used', 'preparations', '13', 'nomenclature', 'polymers', 'the', 'names', 'many', 'polymers', 'based', 'monomers', 'prepared', 'there', 'however', 'frequent', 'variation', 'format', 'a', 'nomenclature', 'polymers', 'recommended', 'iupac', '21–23', 'used', 'publications', 'strict', 'adherence', 'recommendation', 'however', 'mainly', 'found', 'reference', 'works', 'also', 'problems', 'often', 'encountered', 'complex', 'polymeric', 'structures', 'crosslinked', 'branches', 'in', 'addition', 'polymers', 'derive', 'names', 'trade', 'names', 'for', 'instance', 'large', 'family', 'polyamides', 'known', 'nylons', 'also', 'one', 'functional', 'group', 'present', 'structure', 'material', 'may', 'called', 'according', 'functional', 'groups', 'structure', 'an', 'example', 'polyesteramide', 'a', 'thermoset', 'polymer', 'prepared', 'two', 'different', 'materials', 'may', 'called', 'names', 'for', 'instance', 'condensation', 'product', 'melamine', 'formaldehyde', 'called', 'melamine–formaldehyde', 'polymer', '131', 'nomenclature', 'chain-growth', 'polymers', '1', 'a', 'polymer', 'unspeciﬁed', 'chain', 'length', 'named', 'preﬁx', 'poly', 'the', 'preﬁx', 'followed', 'name', 'monomer', 'also', 'customary', 'use', 'common', 'names', 'monomers', 'polymers', 'for', 'instance', 'common', 'names', 'phenylethene', 'polyphenylethene', 'styrene', 'polystyrene', 'this', 'however', 'inﬂexible', 'rule', 'when', 'monomer', 'named', 'single', 'word', 'preﬁx', 'poly', 'simply', 'added', 'like', 'polyethylene', 'polymer', 'ethylene', 'polystyrene', 'polymer', 'styrene', 'if', 'however', 'monomer', 'named', 'two', 'words', 'preceded', 'number', 'like', 'methyl', 'methacrylate', 'parentheses', 'used', 'examples', 'poly(methyl', 'methacrylate)', 'poly(1-hexene)', '2', 'end', 'groups', 'usually', 'speciﬁed', 'high', 'polymers', 'end', 'groups', 'however', 'known', 'parts', 'structure', 'this', 'case', 'telomers', 'here', 'end', 'groups', 'named', 'radicals', '8', '1', 'introduction', 'nomenclature', 'table', '11', 'illustration', 'common', 'chain-growth', 'polymers', 'name', 'polyethylene', 'monomer', 'polymer', 'polyisobutylene', 'polystyrene', 'poly(vinyl', 'chloride)', 'poly(vinyl', 'acetate)', 'poly(methyl', 'methacrylate)', 'polyisoprene', 'cl', 'o', 'o', 'o', 'o', 'n', 'n', 'n', 'cl', 'o', 'n', 'n', 'o', 'n', 'o', 'o', 'n', 'preﬁxed', 'greek', 'letters', 'v', 'they', 'appear', 'name', 'polymer', 'the', 'structure', 'telomer', 'like', 'cl–(–ch2–)n–ccl3', 'therefore', 'called', 'a-chloro-o-trichloromethyl', 'poly(methylene)', '3', 'in', 'naming', 'polymer', 'following', 'steps', 'recommended', 'iupac', '(1)', 'identify', 'constitutional', 'repeating', 'unit', '(2)', 'orient', 'constitutional', 'repeating', 'unit', '(3)', 'name', 'constitutional', 'repeating', 'unit', '4', 'random', 'copolymers', 'designated', 'preﬁx', 'co', 'poly(butadiene-co', 'styrene)', 'poly', '(vinyl', 'chloride-co', 'vinyl', 'acetate)', 'alternating', 'copolymers', 'differentiated', 'substituting', 'alt', 'co', 'poly(ethylene-alt-carbon', 'monoxide)', '5', 'the', 'preﬁx', 'g', 'describes', 'graft', 'copolymers', 'preﬁx', 'b', 'describes', 'block', 'copolymers', 'in', 'system', 'nomenclature', 'ﬁrst', 'polymer', 'segment', 'corresponds', 'homopolymer', 'copolymer', 'formed', 'ﬁrst', 'stage', 'synthesis', 'should', 'graft', 'copolymer', 'represent', 'backbone', 'polymer', 'for', 'instance', 'polystyrene', 'graft', 'copolymerized', 'polyethyl-', 'ene', 'product', 'called', 'poly(ethylene-g-styrene)', 'a', 'complex', 'example', 'poly(butadi-', 'ene-co-styrene-g-acrylonitrile-co-vinylidine', 'chloride)', 'similarly', 'examples', 'block', 'copolymers', 'would', 'poly(acrylonitrile-b-methyl', 'methacrylate)', 'poly(methyl', 'methacrylate-b-acrylonitrile)', '6', 'conventional', 'preﬁxes', 'indicating', 'cis', 'trans', 'isomers', 'placed', 'front', 'polymer', 'name', 'an', 'example', 'cis-14-polybutadiene', 'trans-14-polyisoprene', '13', 'nomenclature', 'polymers', 'table', '12', 'illustration', 'step-growth', 'polymers', 'monomers', 'used', 'preparation', 'poly(ethylene', 'terephthalate)', 'oh', 'o', 'ho', 'ho', 'hooc', 'h2n', 'oh', 'o', 'o', 'o', 'cooh', 'nh2', 'o', 'c', 'h', 'n', 'o', 'c', 'poly(hexamethylene', 'adipate)', 'nylon', '66', 'polycaprolactam', 'nylon', '6', 'poly(ethylene', 'oxide)', 'poly(lactic', 'acid)', 'poly(benzimidazole)', 'poly(p-xylylene)', 'poly(butyrolactone)', 'o', 'o', 'o', 'n', 'h', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'n', 'h', 'n', 'h', 'n', 'o', 'n', 'o', 'o', '9', 'n', 'n', 'h2n', 'h2n', 'nh2', 'n', 'nh2', 'hn', 'n', 'hn', 'o', 'o', 'o', 'n', 'n', 'c', 'o', '7', 'the', 'nomenclature', 'adopted', 'iupac', 'rests', 'upon', 'selection', 'preferred', 'constitutional', 'repeating', 'units', '5', 'polymer', 'multiple', 'the', 'unit', 'named', 'wherever', 'possible', 'according', 'deﬁnitive', 'rules', 'nomenclature', 'organic', 'chemistry', '24.', 'for', 'single-stranded', 'polymers', 'unit', 'bivalent', 'group', 'an', 'example', 'polymer', 'oxy(1-ﬂuoroethylene)', 'constitutional', 'repeat', 'unit', 'o', 'n', 'f', 'polyoxy(1-fluoroethylene)', '10', '1', 'introduction', 'nomenclature', 'the', 'following', 'examples', 'simple', 'constitutional', 'repeat', 'units', 'o', 'n', 'n', 'n', 'polymethylene', 'polyoxyethylene', 'poly(1-butenylene)', '8', 'polymers', 'repeating', 'units', 'consisting', 'one', 'simple', 'bivalent', 'radical', 'named', 'according', 'order', 'seniority', 'among', 'types', 'bivalent', 'radicals', '(a)', 'heterocyclic', 'rings', '(b)', 'chains', 'containing', 'hetero', 'atoms', '(c)', 'carbocyclic', 'rings', '(d)', 'chains', 'containing', 'carbons', 'this', 'illustrated', 'o', 'n', 'n', 'n', 'poly(35-pyridinediylmethyleneoxy', '-14-phenylene)', 'poly(26-biphenyleneethylene)', '9', 'double-stranded', '“ladder”', 'polymers', 'tetravalent', 'repeat', 'units', 'named', 'similarly', 'bivalent', 'units', 'the', 'relation', 'four', 'free', 'valences', 'denoted', 'pairs', 'locants', 'separated', 'chain', 'n', 'poly(1212-ethane', 'diylidene)', 'poly(2367-naphthalenetetrayl-67-dimethylene)', 'n', '10', 'for', 'polymers', 'contain', 'heteroatoms', 'acyclic', 'subunits', 'containing', 'heteroatoms', 'decreasing', 'seniority', 'naming', 'it', 'following', 'order', 'o', 's', 'se', 'te', 'n', 'sb', 'bi', 'si', 'ge', 'sn', 'pb', 'b', 'similarly', 'polymers', 'containing', 'ring', 'structures', 'seniority', 'heterocyclic', 'ring', 'greater', 'seniority', 'heteroatoms', 'acyclic', 'subunits', 'similarly', 'heterocyclic', 'subunits', 'greater', 'seniority', 'carbocyclic', 'ring', 'turn', 'greater', 'seniority', 'acyclic', 'substructures', 'an', 'example', 'would', 'n', 'n', 'poly(24-pyridinediiyl-14-phenylene)', '14', 'steric', 'arrangement', 'macromolecules', '11', '132', 'nomenclature', 'step-growth', 'polymers', 'the', 'nomenclature', 'step-growth', 'polymers', 'complicated', 'due', 'possibility', 'many', 'different', 'repeat', 'units', 'usually', 'polymers', 'referred', 'according', 'functional', 'units', 'a', 'polyester', 'ethylene', 'glycol', 'terephthalic', 'acid', 'called', 'poly(ethylene', 'terephthalate)', 'a', 'product', 'ring', 'opening', 'polymerization', 'like', 'instance', 'polymer', 'caprolactam', 'might', 'called', 'polycaprolactam', 'here', 'based', 'source', 'the', 'repeat', 'unit', 'actually', 'lactam', 'rather', 'open', 'chain', 'polyamide', 'because', 'derived', 'lactam', 'may', 'still', 'carry', 'name', 'in', 'particular', 'instance', 'however', 'common', 'call', 'polymer', 'generic', 'name', 'namely', 'nylon', '6', 'the', 'would', 'true', 'polymer', 'lactone', 'like', 'poly(b-propiolactone)', 'the', 'iupac', 'name', 'polymer', 'however', 'o', 'o', 'n', 'polyoxy(1-oxotrimethylene)', 'the', 'name', 'based', 'presence', 'two', 'subunits', 'note', 'carbonyl', 'oxygen', 'called', 'oxo', 'substituent', 'in', 'addition', 'presence', '1-oxo', 'substituent', 'requires', 'parentheses', 'enclose', 'subunit', '14', 'steric', 'arrangement', 'macromolecules', 'in', 'linear', 'polymers', 'due', 'polymerization', 'process', 'pendant', 'groups', 'arranged', 'orderly', 'conﬁgurations', 'lack', 'orderliness', 'propylene', 'instance', 'polymerized', 'two', 'types', 'orderly', 'steric', 'arrangement', 'it', 'also', 'polymerized', 'one', 'lacking', 'steric', 'order', 'the', 'true', 'monosubstituted', 'vinyl', 'monomers', 'the', 'steric', 'arrangement', 'macromolecules', 'called', 'tacticity', 'polymers', 'isotactic', 'chiral', 'centers', 'conﬁguration', '(see', 'fig', '12)', 'by', 'picturing', 'chain', 'backbone', 'drawn', 'plane', 'paper', 'picturing', 'phenyl', 'groups', 'oriented', 'plane', '(fig', '12a)', 'isotactic', 'polystyrene', 'visualized', 'the', 'orderliness', 'also', 'type', 'every', 'chiral', 'center', 'conﬁguration', 'such', 'arrangement', 'called', 'syndiotactic', '(fig', '12b)', 'a', 'lack', 'orderliness', 'randomness', 'steric', 'arrangement', 'called', 'atactic', 'heterotactic', 'stereospeciﬁc', 'polymers', 'also', 'prepared', '12', 'disubstitued', 'oleﬁns', 'these', 'macromolecules', 'distereoisomers', 'ditactic', 'polymers', 'to', 'describe', 'arrangement', 'polymers', 'threo–erythro', 'terminology', 'used', 'an', 'erythrodiisotactic', 'polymer', 'one', 'possessing', 'alternating', 'substituents—(–chr0chr–)–n', 'if', 'draw', 'carbon', 'chain', 'backbone', 'plane', 'paper', 'r', 'groups', 'would', 'ﬁnd', 'one', 'side', 'chain', 'r0', 'groups', 'they', 'would', 'however', 'side', 'chain', 'fischer', 'newman', 'projection', '(fig', '12c)', 'a', 'threo', 'isomer', 'threodiisotactic', 'polymer', 'illustrated', 'fig', '12d', 'polymerization', '14-disubstituted', 'butadienes', 'lead', 'products', 'possess', 'two', 'asymmetric', 'carbon', 'atoms', 'one', 'double', 'bond', 'repeat', 'unit', 'such', 'tritactic', 'polymers', 'named', 'preﬁxes', 'cis', 'trans', 'together', 'erythro', 'threo', '(see', 'fig', '13)', 'in', 'polymers', 'single', 'carbon', 'carbon', 'bonds', 'free', 'rotation', 'provided', 'steric', 'hindrance', 'interfere', 'this', 'allows', 'molecules', 'assume', 'different', 'spatial', 'arrangements', 'conformations', 'most', 'possible', 'isomers', 'however', 'represent', 'prohibitively', 'high-energy', 'states', 'the', 'three', 'lowest', 'energy', 'states', 'probable', '8', 'one', 'trans', 'two', 'gauche', '12', '1', 'introduction', 'nomenclature', 'fig', '12', 'steric', 'arrangement', 'macromolecules', '(r', '¼', 'phenyl)', '(a)', 'isotactic', 'polymer', '(b)', 'syndiotactic', 'polymer', '(c)', 'erythrodiisotactic', 'polymer', '(d)', 'threodiisotactic', 'polymer', 'c', 'b', 'h', 'h', 'h', 'h', 'r', 'h', 'h', 'r', 'h', 'r', 'r', 'h', 'r', 'h', 'r', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'r', 'h', 'r', 'r', 'h', 'h', 'r', 'rh', 'h', 'h', 'h', 'h', 'h', 'r', 'r', 'h', 'h', 'h', 'h', 'h', 'h', 'r', 'h', 'h', 'r', 'h', 'r', 'h', 'r', 'h', 'h', 'h', 'h', 'r', 'r', 'h', 'h', 'r', 'h', 'h', 'r', 'h', 'r', 'h', 'r', 'fig', '13', 'tritactic', 'polymers', '(a)', 'trans-erythro', 'tritactic', 'polymer', '(b)', 'trans-threo', 'tritactic', 'polymer', '(c)', 'cis-erythro', 'tritactic', 'polymer', '(d)', 'cis-threo', 'tritactic', 'polymer', 'several', 'rules', 'govern', 'conﬁguration', 'repeat', 'units', 'along', 'polymeric', 'chains', 'elucidated', 'natta', 'carradini', '24.', 'a', 'basic', 'assumption', 'used', 'predict', 'lowest', 'energy', 'conformation', 'this', 'assumption', 'known', 'equivalence', 'postulate', 'it', 'says', 'structural', 'units', 'along', 'chain', 'geometrically', 'equivalent', 'regard', 'axis', 'chain', 'all', 'stereoreg-', 'ular', 'polymers', 'appear', 'meet', 'condition', 'exceptions', '25.', 'review', 'questions', '13', 'it', 'possible', 'calculate', '(in', 'principle)', 'lowest', 'internal', 'conformational', 'energy', 'given', 'polymer', 'molecule', 'to', 'necessary', 'know', '(1)', 'energy', 'vs', 'bond', 'rotation', 'curve', 'relates', 'interactions', 'within', 'structural', 'units', '(2)', 'interaction', 'energy', 'vs', 'distance', 'curve', 'neighboring', 'pendant', 'groups', 'adjacent', 'structural', 'units', 'it', 'noted', 'neighboring', 'units', 'approach', 'close', 'enough', 'substantial', 'interaction', 'as', 'result', 'interaction', 'energies', 'groups', 'may', 'distort', 'basic', 'energy–rotation', 'curves', 'the', 'core', 'electron', 'repulsion', 'units', 'contributes', 'signiﬁcantly', 'deviations', 'simple', 'trans', 'gauche', 'conformation', 'the', 'speciﬁc', 'chain', 'conformation', 'therefore', 'dependent', 'upon', 'exact', 'nature', 'repulsive', 'potential', 'a', 'contour', 'plot', 'represents', 'internal', 'energy', 'per', 'mole', 'polymer', 'prepared', 'the', 'variables', 'plot', 'relative', 'bond', 'rotations', 'successive', 'bonds', 'appendix', 'additional', 'deﬁnitions', 'term', 'tensile', 'strength', 'flexural', 'strength', 'tensile', 'stress', '(s)', 'tensile', 'strain', '(e)', 'tensile', 'modulus', '(e)', 'deﬁnition', 'ability', 'resist', 'stretching', 'resistance', 'breaking', 'snapping', '¼', 'forcecross-section', 'area', '¼', 'fa', 'change', 'sample', 'length', '(when', 'stretched)', 'divided', 'original', 'length', 'e', '¼', 'dll', 'ratio', 'stress', 'strain', 'e', '¼', 'se', 'recommended', 'reading', 'rb', 'seymour', 'history', 'polymer', 'science', 'technology', 'dekker', 'new', 'york', '1982', 'h', 'morawetz', 'the', 'origin', 'growth', 'science', 'wiley-interscience', 'new', 'york', '1982', 'mp', 'stevens', 'polymer', 'chemistry', '3rd', 'ed,', 'oxford', 'university', 'press', '1998', '(chapter', '1)', 'g', 'odian', 'principles', 'polymerization', '3rd', 'ed', 'wiley', 'new', 'york', '1991', '(introduction)', 'review', 'questions', 'section', '12', '1', 'deﬁne', 'degree', 'polymerization', '2', 'polystyrene', 'dp', '300', 'what', 'molecular', 'weight', 'polymer', '3', 'what', 'linear', 'branched', 'star-shaped', 'comb-shaped', 'ladder', 'polymer', 'explain', '4', 'what', 'network', 'structure', '5', 'how', 'network', 'structure', 'formed', 'step-growth', 'polymerization', 'also', 'chain-growth', '6', 'what', 'difference', 'random', 'alternating', 'copolymers', '7', 'what', 'meant', 'graft', 'block', 'copolymers', 'illustrate', 'copolymers', 'styrene', 'methyl', 'polymerization', 'methacrylate', '14', '1', 'introduction', 'nomenclature', '8', 'what', 'important', 'features', 'chain-growth', 'step-growth', 'polymerizations', 'explain', 'difference', 'two', 'can', 'suggest', 'analytical', 'procedure', 'determine', 'mechanism', 'particular', 'polymerization', 'reaction', 'takes', 'place', '9', 'what', 'dp', 'polystyrene', 'molecular', 'weight', '104000', 'poly(vinyl', 'chloride)', 'molecular', 'weight', '63000', '10', 'explain', 'differences', 'thermosetting', 'thermoplastic', 'polymers', 'deﬁne', 'gel', 'point', 'how', 'would', 'determine', 'gel', 'point', 'thermoset', 'polymer', '11', 'give', 'deﬁnitions', 'oligomer', 'telomer', 'telechelic', 'polymers', '12', 'what', 'dendrimer', 'polymer', 'rotaxane', 'polymer', '13', 'illustrate', 'three', 'chain-growth', 'polymers', 'three', 'step-growth', 'polymers', 'section', '13', 'names', '1', 'name', 'following', 'chain-growth', 'polymers', 'iupac', 'system', 'giving', 'trivial', '15', '1', 'what', 'meant', 'tacticity', '2', 'give', 'deﬁnition', 'illustrate', 'examples', 'polystyrene', 'isotactic', 'syndiotactic', 'atactic', 'arrangement', 'macromolecules', 'this', 'include', 'fischer', 'newman', 'projections', '3', 'explain', 'meant', 'erythrodiisotactic', 'threodiisotaactic', 'polymers', 'illustrate', 'do', 'erythrodisyndiotactic', 'threodisyndiotactic', '4', 'what', 'tritactic', 'polymers', 'draw', 'cis', 'trans', 'tritactic', 'polymers', 'references', 'section', '14', 'references', '1', 'fm', 'raoult', 'compi', 'rend,', '98', '1030(1882)', 'ann', 'chim', 'phys', '2(6)', '66', '(1884)', 'coinpi', 'rend,', '101', '1056', '(1885)', '2', 'j', 'h', 'van’t', 'hoff', 'z', 'physik', 'chem,', '1481(1887)', 'phil', 'mag,', '26(5)', '81(1888)', '3', 'p', 'j', 'flory', 'principles', 'polymer', 'chemistry', 'cornell', 'univ', 'press', 'ithaca', 'n', 'y,', '1953', '4', 'm', 'berthelot', 'bul', 'soc', 'chim', 'france', '6(2)', '294', '(1866)', '5', 'g', 'bouchardat', 'compi', 'rend,', '89', '1117', '(1879)', '6', 'g', 'wagner', 'ber,', '11', '1260', '(1878)', '7', 'r', 'fittig', 'f', 'engelhorn', 'ann,', '200', '65', '(1880)', '8', 'r', 'willst€atter', 'l', 'zechmeister', 'ber', '46', '2401', '(1913)', '9', 'l', 'h', 'baekeland', 'md', 'eng', 'chem,', '5506(1913)', '10', 'h', 'staudinger', 'ber,', '53', '1073', '(1920)', '11', 'h', 'staudinger', 'j', 'fritschi', 'helv', 'chim', 'acta', '5', '785', '(1922)', '12', 'h', 'staudinger', 'ber,', '59', '3019', '(1926)', '13', 'h', 'staudinger', 'k', 'frey', 'w', 'starck', 'ber,', '60', '1782', '(1927)', '14', 'h', 'staudinger', 'm', 'luthy', 'heiv', 'chim', 'ada', '841(1925)', 'h', 'staudinger', 'ibid', '867(1925)', '15', 'h', 'staudinger', 'h', 'johner', 'r', 'signer', 'g', 'mie', 'j', 'hengstenberg', 'z', 'physik', 'chem,', '126434', '(1927)', '16', 'o', 'l', 'sponsler', 'j', 'gen', 'phvsiol,', '9', '677(1926)', 'o', 'l', 'sponsler', 'w', 'k', 'dore', 'colloid', 'sumposium', 'monograph', 'vol', 'iv', 'chemical', 'catalog', 'co,', 'new', 'york', '1926', 'p', '174', '17', 'w', 'n', 'carothers', 'j', 'am', 'chem', 'soc,', '51', '2548', '(1929)', 'chem', 'rev,', '8', '353', '(1931)', 'collected', 'papers', 'wallace', 'hume', 'carothers', 'high', 'polymeric', 'suhstances', '(h', 'mark', 'g', 's', 'whitby', 'eds)', 'wiley', '(lnterscience)', 'new', 'york', '1940', '18', 'j', 'cairns', 'j', 'mol', 'biol,', '4407', '(1962)', '19', 'w', 'h', 'carothers', 'j', 'am', 'chem', 'soc,', '51', '2548', '(1929)', 'chem', 'rev,', '8', '353', '(1931)', 'collected', 'papers', 'wallace', 'hume', 'carothers', 'hiqh', 'polymeric', 'substances', 'h', 'mark', 'gswhitby', '(eds)', 'wiley-interscience', 'new', 'york', '1926', '20', 'p', 'j', 'flory', 'principles', 'polymer', 'chemistry', 'cornell', 'univ', 'press', 'ithaca', 'ny,', '1951', 'm', 'l', 'huggins', 'pcarradini', 'v', 'desreux', 'o', 'kratsky', 'h', 'mark', 'j', 'polymer', 'sci,', 'b6', '257', '(1968)', '21', 'ad', 'jenkins', 'jl', 'koening', '“nomenclature”', 'chap2', 'comprehensive', 'polymer', 'science', 'voli', 'c', 'booth', 'c', 'price', '(eds)', 'pergamon', 'press', 'oxford', '1989', '22', 'international', 'union', 'pure', 'applied', 'chemistry', 'macromolecular', 'nomenclature', 'commission', 'j', 'polymer', 'sci,', 'polymer', 'letters', '11', '389', '(1973)', 'macromolecules', '6', '149', '(1973)', 'pure', 'applied', 'chemistry', '40', '479', '(1974)', '23', 'rs', 'cahn', 'an', 'introduction', 'chemical', 'nomenclature', '3rd', 'ed,', 'butterworth', 'london', 'england', '1968', '24', 'g', 'natta', 'p', 'corradini', 'j', 'polymer', 'sci,', '39', '29', '(1959)', '25', 'f', 'p', 'redding', 'j', 'polymer', 'sci,', '21', '547', '(1956)', 'chapter', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', '21', 'structure', 'property', 'relationship', 'organic', 'polymers', 'for', 'large', 'proportion', 'polymeric', 'materials', 'commercial', 'use', 'mechanical', 'properties', 'paramount', 'importance', 'used', 'structural', 'materials', 'ﬁbers', 'coatings', 'properties', 'determine', 'usefulness', 'properties', 'also', 'determine', 'utilization', 'compressive', 'tensile', 'ﬂexural', 'strength', 'impact', 'resistance', 'hardness', 'tear', 'abrasion', 'resistance', 'also', 'concern', 'in', 'addition', 'polymers', 'may', 'shaped', 'extrusion', 'molten', 'state', 'molds', 'deposition', 'solutions', 'various', 'surfaces', 'this', 'makes', 'ﬂow', 'behaviors', 'molten', 'state', 'solution', 'melting', 'temperatures', 'amount', 'crystallization', 'well', 'solubility', 'parameters', 'important', 'the', 'physical', 'properties', 'polymer', 'molecules', 'inﬂuenced', 'composition', 'also', 'size', 'nature', 'primary', 'secondary', 'bond', 'forces', 'they', 'also', 'affected', 'amount', 'symmetry', 'uniformity', 'molecular', 'structures', 'arrangements', 'macromolecules', 'amorphous', 'crystalline', 'domains', 'this', 'turn', 'affects', 'melting', 'softening', 'temperatures', 'solubilities', 'melt', 'solution', 'viscosities', 'physical', 'properties', '1.', 'due', 'large', 'sizes', 'polymeric', 'molecules', 'secondary', 'bond', 'forces', 'assume', 'much', 'greater', 'roles', 'inﬂuencing', 'physical', 'properties', 'small', 'organic', 'molecules', 'these', 'secondary', 'bond', 'forces', 'van', 'der', 'waal', 'forces', 'hydrogen', 'bonding', 'the', 'van', 'der', 'waal', 'forces', 'subdivided', 'three', 'types', 'dipole–dipole', 'interactions', 'induced', 'dipoles', 'time', 'varying', 'dipoles', '211', 'effects', 'dipole', 'interactions', 'dipole', 'interactions', 'result', 'molecules', 'carrying', 'equal', 'opposite', 'electrical', 'charges', 'the', 'amounts', 'interactions', 'depend', 'upon', 'abilities', 'dipoles', 'align', 'one', 'another', 'molecular', 'orientations', 'subject', 'thermal', 'agitation', 'tends', 'interfere', 'electrical', 'ﬁelds', 'as', 'result', 'dipole', 'forces', 'strongly', 'temperature', 'dependent', 'an', 'example', 'dipole', 'interaction', 'illustration', 'two', 'segments', 'molecular', 'chains', 'linear', 'polyester', 'each', 'carbonyl', 'group', 'ester', 'linkages', 'sets', 'weak', 'ﬁeld', 'polarization', 'the', 'ﬁeld', 'though', 'weak', 'interacts', 'another', 'ﬁeld', 'type', 'another', 'chain', 'this', 'results', 'formation', 'forces', 'cohesion', 'because', 'polymeric', 'molecules', 'large', 'many', 'ﬁelds', 'polyesters', 'while', 'ﬁeld', 'weak', 'net', 'effect', 'strong', 'cohesion', 'chains', 'the', 'interactions', 'illustrated', 'fig', '21', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-92', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '17', '18', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'fig', '21', 'intramolecular', 'forces', 'o', 'o', 'o', 'o', 'o', 'r', 'r', 'r', 'o', 'o', 'r', 'o', 'o', 'r', 'o', 'o', 'o', 'o', 'o', 'r', 'r', 'r', 'o', 'o', 'r', 'o', 'o', 'r', '212', 'induction', 'forces', 'polymers', 'electrostatic', 'forces', 'also', 'result', 'slight', 'displacement', 'electrons', 'nuclei', 'covalent', 'molecules', 'proximity', 'electrostatic', 'ﬁelds', 'associated', 'dipoles', 'molecules', 'these', 'induced', 'dipoles', 'the', 'displacements', 'cause', 'interactions', 'induced', 'dipoles', 'perma-', 'nent', 'dipoles', 'creating', 'forces', 'attraction', 'the', 'energy', 'induction', 'forces', 'however', 'small', 'temperature-dependent', 'there', 'additional', 'attraction', 'forces', 'result', 'different', 'instantaneous', 'conﬁgurations', 'electrons', 'nuclei', 'bonds', 'polymeric', 'chains', 'these', 'time', 'varying', 'dipoles', 'average', 'zero', 'they', 'polarizations', 'arising', 'molecular', 'motions', 'the', 'total', 'bond', 'energy', 'secondary', 'bond', 'forces', 'combined', 'including', 'hydrogen', 'bonding', 'ranges', '2', '10', 'kcalmole', 'of', 'however', 'hydrogen', 'bonding', 'takes', 'greatest', 'share', 'bond', 'strength', 'in', 'table', '21', 'listed', 'intramolecular', 'forces', 'linear', 'polymers', '2', '3.', 'as', 'seen', 'table', '21,', 'polyethylene', 'possesses', 'much', 'less', 'cohesive', 'energy', 'polyamide', 'this', 'difference', 'primarily', 'due', 'hydrogen', 'bonding', 'a', 'good', 'illustration', 'comparison', 'molecules', 'polyamide', 'like', 'nylon', '11', 'linear', 'polyethylene', 'both', 'similar', 'chemical', 'structures', 'difference', 'nylon', '11', 'structure', 'periodic', 'amide', 'linkages', 'every', 'tenth', 'carbon', 'linkages', 'absent', 'polyethylene', 'the', 'amide', 'linkages', 'participate', 'hydrogen', 'bonding', 'neighboring', 'chains', 'this', 'illustrated', 'fig', '22', 'due', 'hydrogen', 'bonding', 'nylon', '11', 'melts', '184–187(cid1)c', 'soluble', 'strong', 'solvents', 'linear', 'polyethylene', 'hand', 'melts', '130–134(cid1)c', 'soluble', 'hot', 'aromatic', 'solvents', 'the', 'energy', 'dipole', 'interactions', '(єk)', 'calculated', 'equation', '1:', 'c(cid3)', 'k', '¼', '(cid3)ð2m43rtþr(cid3)6', 'represents', 'dipole', 'moment', 'polarized', 'section', 'molecule', 'r', 'distance', 'dipoles', 't', 'temperature', 'kelvin', 'r', 'ideal', 'gas', 'constant', 'intermolecular', 'forces', 'affect', 'rigidity', 'polymers', 'should', 'forces', 'weak', 'cohesive', 'energy', 'low', '(1–2', 'kcalmole)', 'polymeric', 'chains', 'tend', 'ﬂexible', 'such', 'chains', 'respond', 'readily', 'applied', 'stresses', 'exhibit', 'typical', 'properties', 'elastomers', 'high', 'cohesive', 'energy', 'hand', '(5', 'kcalmole', 'higher)', 'causes', 'materials', 'strong', 'tough', 'these', 'polymers', 'exhibit', 'resistance', 'applied', 'stresses', 'usually', 'possess', 'good', 'mechanical', 'properties', 'the', 'temperatures', 'ﬂexibility', 'polymeric', 'molecules', 'govern', 'sizes', 'molecular', 'segments', '21', 'structure', 'property', 'relationship', 'organic', 'polymers', '19', 'table', '21', 'molecular', 'cohesion', 'linear', 'polymersa', 'polymer', 'polyethylene', 'cohesion5', 'a˚', 'chain', 'length', 'polymer', 'structure', '10', 'polybutadiene', '11', 'polyisobutylene', '12', 'polyisoprene', '(cis)', '13', 'poly(vinyl', 'chloride)', '26', 'poly(vinyl', 'acetate)', '32', 'polystyrene', '40', 'poly(vinyl', 'alcohol)', '42', 'polyamides', '58', 'cellulose', '62', 'n', 'n', 'n', 'n', 'n', 'n', 'o', 'n', 'cl', 'o', 'n', 'oh', 'o', 'r', 'n', 'h', 'r', 'n', 'oh', 'oh', 'o', 'o', 'oh', 'from', 'refs', '2', '3,', 'literature', 'sources', 'motion', 'frequency', 'occurs', 'this', 'turn', 'determines', 'rate', 'polymer', 'molecules', 'respond', 'molecular', 'stresses', 'in', 'ﬂexible', 'polymers', 'thermal', 'energy', 'sufﬁciently', 'high', 'large', 'segments', 'disengage', 'slip', 'past', 'quite', 'readily', 'response', 'applied', 'stress', 'all', 'elastomers', 'possess', 'properties', 'properties', 'polymers', 'depend', 'also', 'upon', 'morphology', 'upon', 'arrangement', 'polymeric', 'chains', 'this', 'arrangement', 'amorphous', 'crystalline', 'the', 'term', 'amorphous', 'designates', 'lack', 'orderly', 'arrangement', 'crystalline', '20', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'fig', '22', 'hydrogen', 'bonding', '(a)', 'nylon', '11', 'absent', '(b)', 'polyethylene', 'o', 'h', 'n', 'h', 'n', 'o', 'o', 'h', 'n', 'nylon', '11', 'polyethylene', 'morphology', 'hand', 'means', 'chains', 'aligned', 'orderly', 'fashion', 'generally', 'freedom', 'molecular', 'motion', 'along', 'backbones', 'polymeric', 'chains', 'contributes', 'lowering', 'melting', 'temperature', 'substituents', 'interfere', 'motion', 'tend', 'raise', 'melting', 'point', 'for', 'instance', 'isotactic', 'polypropylene', 'melts', 'higher', 'temperature', 'linear', 'polyethylene', 'if', 'substituent', 'bulky', 'rigid', 'raises', 'melting', 'point', 'interferes', 'molecular', 'motion', 'dipole', 'interactions', 'discussed', 'similar', 'effect', '4.', 'a', 'good', 'illustration', 'comparison', 'poly(ethylene', 'terephthalate)', 'melts', '265(cid1)c', 'poly(ethylene', 'adipate)', 'melts', '50(cid1)c', 'in', 'ﬁrst', 'polyester', 'rigid', 'benzene', 'ring', 'ester', 'groups', 'second', 'one', 'ﬂexible', 'chain', 'four', 'carbons', 'o', 'o', 'o', 'o', 'o', '2', 'o', 'n', '4', 'o', 'o', '2', 'n', 'poly(ethylene', 'terephthalate)', 'poly(ethylene', 'adipate)', 'this', 'ﬂexibility', 'four', 'carbon', 'segment', 'poly(ethylene', 'adipate)', 'contributes', 'signiﬁcantly', 'lowering', 'melting', 'point', 'linear', 'polymers', 'possess', 'single', 'bonds', 'atoms', 'backbones', 'c–c', 'c–o', 'c–n', 'undergo', 'rapid', 'conformational', 'changes', '5.', 'also', 'ether', 'imine', 'cis-double', 'bonds', 'reduce', 'energy', 'barriers', 'result', '“soften”', 'chains', 'causing', 'polymer', 'become', 'rubbery', 'soluble', 'various', 'solvents', 'the', 'opposite', 'true', 'cyclic', 'structures', 'backbones', 'shown', 'poly(ethylene', 'terephthalate)', 'actually', 'cyclic', 'structures', 'inhibit', 'conformational', 'changes', 'also', 'make', 'crystallization', 'difﬁcult', 'among', 'polymers', 'a-oleﬁns', 'structures', 'pendant', 'groups', 'inﬂuence', 'melting', 'point', '6.', 'all', 'linear', 'polyethylene', 'melts', '132', '136(cid1)c', '7.', 'isotactic', 'polypropylene', 'hand', 'melts', '168(cid1)c', '8.', 'n', 'n', 'mp', '132-136', '°c', 'mp', '168', '°c', '22', 'the', 'amorphous', 'state', '21', 'as', 'length', 'side', 'chain', 'increase', 'however', 'melting', 'points', 'decrease', 'accompanied', 'increases', 'ﬂexibility', '7', 'length', 'side', 'chains', 'reached', 'six', 'carbons', 'at', 'point', 'minimum', 'takes', 'upturn', 'increase', 'melting', 'points', 'decrease', 'ﬂexibility', 'this', 'phenomenon', 'believed', 'due', 'crystallization', 'side-chains', '9.', 'alkyl', 'substituents', 'polymers', 'a-oleﬁns', 'a-carbon', 'yield', 'polymers', 'highest', 'melting', 'points', 'isomers', 'substituted', 'b-carbon', 'however', 'symmetrical', 'yield', 'polymers', 'lower', 'melting', 'points', 'unsymmetrical', 'substitutions', 'b-carbon', 'hand', 'tend', 'lower', 'melting', 'points', 'additional', 'drop', 'melting', 'points', 'result', 'substitutions', 'g-carbon', 'side-chains', 'terminal', 'branching', 'yields', 'rubbery', 'polymers', '10.', 'copolymers', 'melt', 'lower', 'temperatures', 'homopolymers', 'individual', 'monomers', 'by', 'increasing', 'amount', 'comonomer', 'melting', 'point', 'decreases', 'minimum', '(this', 'could', 'perhaps', 'compared', 'eutectic)', 'rises', 'the', 'tightest', 'internal', 'arrangement', 'macromolecules', 'achieved', 'crystallinity', 'as', 'result', 'density', 'polymer', 'directly', 'proportional', 'degree', 'crystallinity', 'leads', 'high', 'tensile', 'strength', 'stiff', 'hard', 'materials', 'poorly', 'soluble', 'common', 'solvents', '11.', 'the', 'solubility', 'polymer', 'however', 'function', 'crystallinity', 'alone', 'also', 'internal', 'structure', 'molecular', 'weight', 'the', 'solubility', 'generally', 'decreases', 'increases', 'molecular', 'weight', 'the', 'fact', 'crystalline', 'polymers', 'less', 'soluble', 'amorphous', 'ones', 'attributed', 'binding', 'forces', 'crystals', 'these', 'binding', 'forces', 'must', 'overcome', 'achieve', 'dissolution', 'once', 'solution', 'however', 'crystalline', 'polymers', 'exhibit', 'different', 'properties', 'amorphous', 'ones', 'one', 'also', 'keep', 'mind', 'fact', 'crosslinked', 'polymers', 'melt', 'dissolve', 'solvent', 'this', 'due', 'fact', 'crosslinks', 'prevent', 'chains', 'separating', 'slipping', 'past', '22', 'the', 'amorphous', 'state', 'a', 'number', 'macromolecules', 'show', 'little', 'tendency', 'crystallize', 'align', 'chains', 'form', 'order', 'remain', 'disordered', 'solid', 'form', 'this', 'course', 'also', 'condition', 'molten', 'polymers', 'some', 'however', 'due', 'structural', 'arrangement', 'remain', 'completely', 'amorphous', 'upon', 'cooling', 'the', 'crystalline', 'polymers', 'crystallized', 'melt', 'hand', 'containing', 'areas', 'crystallinity', 'also', 'contain', 'amorphous', 'material', 'all', 'crystalline', 'polymers', 'also', 'amorphous', 'melting', 'temperature', 'when', 'sufﬁciently', 'cooled', 'amorphous', 'polymers', 'resemble', 'glass', 'above', 'glassy', 'state', 'long-range', 'segmented', 'motions', 'possible', 'molecular', 'chains', 'free', 'move', 'past', 'on', 'hand', 'rigid', 'glass', 'like', 'state', 'short-', 'range', 'vibrational', 'rotational', 'motions', 'segments', 'possible', 'at', 'temperatures', 'glassy', 'state', 'amorphous', 'polymers', 'resemble', 'rubbers', 'crosslinked', 'if', 'crosslinked', 'amorphous', 'polymers', 'resemble', 'viscous', 'liquids', 'properties', 'molecular', 'disorder', 'tobolsky', 'suggested', '12', 'polymer', 'molecules', 'amorphous', 'state', 'might', 'compared', 'bowl', 'long', 'strands', 'cooked', 'spaghetti', 'when', 'molten', 'molecules', 'state', 'wriggling', 'motion', 'though', 'amplitude', 'speed', 'depend', 'much', 'temperature', '13.', 'the', 'important', 'thing', 'know', 'chains', 'possess', 'random', 'conformations', 'these', 'conformations', 'characterized', 'aid', 'statistical', 'analysis', '14.', '221', 'the', 'glass', 'transition', 'glassy', 'state', 'when', 'polymer', 'cools', 'temperature', 'lowers', 'mobility', 'amorphous', 'regions', 'polymer', 'decreases', 'the', 'lower', 'temperature', 'stiffer', 'polymer', 'becomes', 'point', '22', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'transition', 'reached', 'this', 'transition', 'called', 'glass', 'transition', 'second-order', 'transition', '15.', 'the', 'temperature', 'transition', 'called', 'glass', 'transition', 'temperature', 'designated', 'tg', 'beyond', 'stiffness', 'change', 'manifested', 'speciﬁc', 'volume', 'heat', 'content', 'thermal', 'conductivity', 'refractive', 'index', 'dielectric', 'loss', 'bueche', 'illustrated', 'glass', 'transition', 'follows', '16.', 'in', 'measuring', 'force', 'necessary', 'force', 'needle', 'polymer', 'like', 'polystyrene', 'various', 'temperatures', 'relationship', 'force', 'required', 'insert', 'needle', 'temperature', '16.', 'as', 'temperature', 'lowered', 'maximum', 'resistance', 'penetration', 'reached', 'tg', 'as', 'stated', 'tg', 'chains', 'undergo', 'cooperative', 'localized', 'motion', 'it', 'actually', 'estimated', 'tg', 'segmental', 'motion', 'anywhere', '20', '50', 'chain', 'atoms', 'possible', 'below', 'second-order', 'transition', 'temperature', 'however', 'insufﬁcient', 'energy', 'available', 'enable', 'whole', 'segments', 'polymeric', 'chains', 'move', 'the', 'structure', 'stiff', 'brittle', 'resists', 'deformation', 'when', 'however', 'sufﬁcient', 'amount', 'heat', 'energy', 'enters', 'material', 'temperature', 'rises', 'tg', 'larger', 'molecule', 'motion', 'involving', 'coordinated', 'movement', 'returns', 'this', 'requires', 'space', 'speciﬁc', 'volume', 'also', 'increases', 'polymer', 'rubbery', 'plastic', 'state', 'above', 'tg', 'large', 'elastic', 'deformations', 'possible', 'polymer', 'actually', 'tougher', 'pliable', 'chemical', 'structures', 'polymers', 'important', 'factors', 'affect', 'glass', 'transition', 'temperatures', 'molecular', 'weights', 'also', 'inﬂuence', 'tg', 'increases', 'molecular', 'weight', 'in', 'addition', 'tg', 'also', 'varies', 'rate', 'cooling', 'table', '22', 'shows', 'structures', 'lists', 'relative', 'tg', 'values', 'common', 'polymers', 'one', 'way', 'obtain', 'tg', 'studying', 'thermal', 'expansion', 'polymers', 'it', 'generally', 'observed', 'thermal', 'coefﬁcient', 'expansion', 'greater', 'glass', 'transition', 'temperature', 'though', 'magnitude', 'change', 'differs', 'one', 'polymer', 'another', 'by', 'plotting', 'volume', 'vs', 'temperature', 'polymer', 'one', 'obtain', 'tg', 'shown', 'fig', '23', 'illustrates', 'obtaining', 'tg', 'speciﬁc', 'volume-cooling', 'temperature', 'curves', '17.', 'polymers', 'bulky', 'tightly', 'held', 'side', 'groups', 'stiff', 'bulky', 'components', 'backbone', 'high', 'tg', 'values', 'this', 'due', 'fact', 'side', 'groups', 'bulky', 'components', 'interfere', 'segmental', 'motion', 'such', 'polymers', 'require', 'higher', 'temperatures', 'acquire', 'sufﬁcient', 'free', 'volume', 'segmental', 'motion', 'this', 'observed', 'table', '22', 'shows', 'glass', 'temperature', 'polystyrene', 'stiff', 'benzene', 'ring', 'side', 'groups', 'much', 'higher', 'polyethylene', 'also', 'polymers', 'high', 'attractive', 'forces', 'chains', 'require', 'heat', 'energy', 'go', 'glassy', 'rubbery', 'plastic', 'state', 'on', 'hand', 'polymeric', 'chains', 'loose', 'hanging', 'side', 'chains', 'tend', 'loosen', 'polymer', 'structure', 'increase', 'free', 'volume', 'segmental', 'movement', 'lower', 'tg', 'for', 'instance', 'glass', 'transition', 'temperature', 'poly(methyl', 'methacrylate)', 'higher', 'poly(n-butyl', 'methacrylate)', 'seen', 'table', '22', 'the', 'transition', 'glassy', 'state', 'equilibrium', 'liquid', 'results', 'changes', 'enthalpy', 'h', 'volume', 'v', 'the', 'speciﬁc', 'heat', 'related', 'enthalpy', 'deﬁnition', 'cp', '¼', 'ðh@tþp', 'the', 'glassy', 'state', 'nonequilibrium', 'nature', 'exhibits', 'tendency', 'undergo', 'structural', 'relaxation', 'toward', 'equilibrium', 'this', 'tendency', 'glassy', 'state', 'relax', 'structurally', 'toward', 'equilib-', 'rium', 'often', 'referred', 'structural', 'recovery', 'it', 'observed', 'however', 'progress', 'towards', 'structural', 'recovery', 'time', 'varies', 'signiﬁcantly', 'down-quench', 'up-quench', 'this', 'referred', 'asymmetry', 'structural', 'recovery', 'the', 'nonlinearity', 'process', 'described', 'following', 'equation', '16:', 'dðu', '(cid3)', 'u0þdt', '¼', '(cid3)ðu', '(cid3)', 'u0þt', '22', 'the', 'amorphous', 'state', '23', 'table', '22', 'glass', 'transition', 'temperatures', 'tg', 'common', 'polymersa', 'tg', '((cid1)c)', 'polymer', 'repeat', 'unit', '81', '90', 'polystyrene', 'no2', 'o', 'cellulose', 'nitrate', 'o', 'o', 'nylon', '66', 'n', 'h', '4', 'n', '6', 'h', 'o', 'ch3oc', 'poly(vinyl', 'chloride)', '75', 'poly(methyl', 'methacrylate)', '57–68', '53', '47', '30', '22', '(cid3)22', '(cid3)39', '(cid3)56', '(cid3)70', 'poly(vinyl', 'acetate)', 'poly(butyl', 'methacrylate)', 'poly(ethyl', 'acrylate)', 'poly(vinyl', 'ﬂuoride)', 'poly(butyl', 'acrylate)', 'polyisoprene', 'cl', 'cooch3', 'o', 'no2', 'o', 'ono2', 'cooc4h9', 'cooc2h5', 'cooc4h9', 'f', 'n', 'o', 'si', 'polyethylene', '(cid3)80', '(cid3)90', 'silicone', 'rubber', '(cid3)123', 'from', 'ref', '15', 'sources', 'literature', 'relaxation', 'time', 'u', 'speciﬁc', 'volume', 'the', 'value', 'tg', 'also', 'affected', 'parameters', 'molecular', 'weight', 'diluent', 'concentration', 'tacticity', 'external', 'pressure', 'crosslinking', 'chain', 'stiffness', 'mechanical', 'deformation', 'for', 'linear', 'polymer', 'fox–flory', 'equation', '18', 'described', 'relationship', 'tg', 'molecular', 'weight', 'tgðmþ', '¼', 'tgð1þ', '(cid3)', 'km', '24', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'fig', '23', 'the', 'relationship', 'speciﬁc', 'volume', 'polymer', 'temperature', 'this', 'equation', 'however', 'limited', 'scope', 'it', 'actually', 'neither', 'valid', 'high', 'molecular', 'weight', 'polymers', 'applicable', 'low', 'molecular', 'weight', 'polymers', 'the', 'transition', 'polymer', 'molten', 'state', 'glassy', 'one', 'actually', 'occurs', 'temperature', 'range', 'this', 'range', 'also', 'includes', 'tg', 'at', 'glass', 'transition', 'temperature', 'however', 'change', 'viscosity', 'rapid', 'viscous', 'glassy', 'one', 'polymeric', 'materials', 'undergo', 'rearrangements', 'response', 'outside', 'stimulus', 'like', 'light', 'becoming', 'increasingly', 'important', 'various', 'industrial', 'application', '(see', 'chap', '10)', 'urban', 'coworkers', '19', 'studied', 'stimuli-responsive', '(tsr)', 'transitions', 'correlated', 'glass', 'transition', 'temperatures', '(tg)', 'based', 'empirical', 'data', 'obtained', 'copolymer', 'concluded', 'relationship', 'tg', 'tsr', 'logðv1v2þ', '¼', 'ðp1ðtsr', '(cid3)', 'tgþþðp2ðtsr', '(cid3)', 'tgþþ', 'v1', 'v2', 'copolymer’s', 'total', 'volumes', 'tsr', 'respectively', 'tg', 'glass', 'transition', 'temperature', 'copolymer', 'p1', 'p2', 'fraction', 'free', 'volume', '(', 'ffree)', 'tg', '(p1)', '(tg', 'midpoint', '(cid3)', 'tsr)5050', 'random', 'copolymer', '(p2)', 'respectively', 'they', 'feel', 'relationship', 'utilized', 'predict', 'total', 'volume', 'changes', 'function', 'tsr', '(cid3)', 'tg', 'different', 'copolymer', 'compositions', '222', 'elasticity', 'the', 'phenomenon', 'elasticity', 'rubber', 'elastomers', 'result', 'tendency', 'large', 'ﬂexible', 'amorphous', 'polymeric', 'chains', 'form', 'random', 'thermodynamically', 'favorable', 'conformations', '18.', 'if', 'certain', 'amount', 'crosslinking', 'also', 'present', 'random', 'conformations', 'occur', 'crosslinks', 'in', 'vulcanized', '(crosslinked)', 'rubber', 'elastomer', 'crosslinks', 'may', 'occur', 'every', 'ﬁve', 'hundred', 'thousand', 'carbon', 'atoms', 'the', 'distance', 'ends', 'polymers', 'much', 'shorter', 'elastomers', 'fully', 'stretched', 'deformation', 'stretching', 'rubber', 'straightens', 'various', 'conformations', 'molecules', 'they', 'tend', 'return', 'original', 'state', 'however', 'forces', 'deformation', 'removed', 'so', 'segment', 'behaves', 'manner', 'resembles', 'spring', 'some', 'elastomeric', 'materials', 'capable', 'high', 'elongation', 'yet', 'still', 'capable', 'returning', 'original', 'conformation', 'some', 'soft', 'rubbers', 'instance', 'extended', 'high', '800', 'even', 'higher', 'full', 'recovery', 'there', 'preference', 'trans', 'conformation', 'planar', 'zigzag', 'high', 'extension', 'rigidity', 'chains', 'however', 'crystallinity', 'would', 'hinder', 'extension', 'particularly', 'recovery', 'high', 'viscosity', 'glassy', 'state', 'would', 'the', 'high', 'degree', 'elasticity', 'rubbers', 'due', 'part', 'effects', 'thermal', 'motions', 'upon', 'long', 'polymeric', 'chains', 'these', 'motions', 'tend', 'restrict', 'vibrational', 'rotational', 'motions', 'single', 'bonds', 'main', 'chain', 'such', 'restrictive', 'forces', 'lateral', 'direction', 'however', 'much', '22', 'the', 'amorphous', 'state', '25', 'weaker', 'primary', 'valence', 'forces', 'longitudinal', 'direction', 'greater', 'amplitudes', 'motion', 'also', 'take', 'place', 'perpendicular', 'chains', 'rather', 'direction', 'chains', 'these', 'increased', 'motions', 'perpendicular', 'direction', 'result', 'repulsive', 'forces', 'extended', 'parallel', 'chains', 'such', 'forces', 'cause', 'draw', 'together', 'stretching', 'so', 'stretched', 'rubber', 'molecules', 'retract', 'due', 'longitudinal', 'tension', 'irregular', 'arrangement', 'molecules', 'achieved', 'this', 'random', 'conformation', 'actually', 'higher', 'entropy', 'state', 'when', 'unstretched', 'rubber', 'heated', 'increases', 'dimension', 'increase', 'temperature', 'one', 'might', 'expect', 'at', 'higher', 'temperatures', 'however', 'rubbers', 'upon', 'elongation', 'higher', 'tendency', 'contract', 'this', 'written', 'follows', 'retractive', 'force', 'f', '¼', '(cid3)tðs@lþtv', 'ðf', '@tþadiabatic', '¼', '(cid3)ðl@tþtcp', 'in', 'summary', 'polymeric', 'materials', 'exhibit', 'rubber', 'elasticity', 'satisfy', 'three', 'requirements', '(a)', 'polymer', 'must', 'composed', 'long-chain', 'molecules', '(b)', 'secondary', 'bond', 'forces', 'molecules', 'must', 'weak', '(c)', 'must', 'occasional', 'interlocking', 'molecules', 'along', 'chain', 'lengths', 'form', 'three-dimensional', 'networks', 'should', 'interlocking', 'arrangements', 'absent', 'elastomers', 'lack', 'memory', 'fading', 'memory', 'capable', 'completely', 'reversible', 'elastic', 'deformations', '2221', 'thermodynamics', 'elasticity', 'stretching', 'elastomer', 'reduces', 'entropy', 'changes', 'free', 'energy', 'the', 'retractive', 'force', 'elastomer', 'primarily', 'result', 'tendency', 'increase', 'entropy', 'towards', 'maximum', 'value', 'original', 'deformed', 'state', 'current', 'explanations', 'rubber-like', 'elasticity', 'based', 'several', 'assumption', '20.', 'the', 'ﬁrst', 'one', 'rubber-like', 'elasticity', 'entirely', 'intramolecular', 'origin', 'the', 'second', 'one', 'free', 'energy', 'network', 'separable', 'two', 'parts', 'elastic', 'one', 'liquid', 'one', 'the', 'liquid', 'one', 'presumably', 'dependent', 'deformation', 'when', 'elastomer', 'stretched', 'free', 'energy', 'changed', 'subjected', 'work', 'if', 'consider', 'stretching', 'one', 'direction', 'work', 'done', 'wel', 'equal', 'f∂l', 'f', 'retractive', 'force', '∂l', 'change', 'length', 'the', 'retractive', 'force', '19–21.', 'f', '¼', 'ðf@lþtp', '¼', 'ðh@lþtp', '(cid3)', 'tðs@lþtp', 'f', 'free', 'energy', 'h', 'enthalpy', 's', 'entropy', 'system', 'an', 'ideal', 'elastomer', 'deﬁned', 'one', '(∂h∂l)tp', '¼', '0', 'f', '¼', '(cid3)t(∂s∂l)tp', 'the', 'negative', 'sign', 'due', 'fact', 'work', 'done', 'stretch', 'increase', 'length', 'elastomer', 'this', 'description', 'ideal', 'elastomer', 'based', 'therefore', 'understanding', 'retractive', 'force', 'due', 'decrease', 'entropy', 'upon', 'extension', 'in', 'words', 'entropy', 'elasticity', 'distortion', 'polymer', 'chains', 'probable', 'random', 'conformations', 'unstretched', 'condition', 'the', 'probability', 'one', 'chain', 'end', 'unit', 'volume', 'space', 'coordinates', 'x', 'z', 'distance', 'r', 'end', '21', '22:', 'b2', '¼', '32xl2', 'the', 'number', 'links', 'x', 'length', 'l', 'the', 'entropy', 'system', 'proportional', 'logarithm', 'number', 'conﬁgurations', 'billmeyer', 'expresses', 'follows', '7:', 'wðx', 'zþ', '¼', 'ðbp05þ3e(cid3)b2r2', 's', '¼', 'ðconstantþ', '(cid3)', 'kb2r2', '26', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'k', 'boltzmann’s', 'constant', 'the', 'retractive', 'force', 'single', 'polymer', 'chain', 'f', '0', 'stretched', 'length', 'dr', 'temperature', 't', 'therefore', 'f', '0', '¼', '(cid3)tðdsdrþ', '¼', '2ktb2r', 'it', 'generally', 'assumed', 'total', 'retractive', 'force', 'given', 'sample', 'elastomer', 'sum', 'f', '0', 'forces', 'polymeric', 'chains', 'consists', 'this', 'claimed', 'justiﬁed', 'cases', 'though', 'inaccurate', 'detail', '22.', 'tobolsky', 'wrote', 'equation', 'entropy', 'change', 'unstretched', 'stretched', 'elastomer', 'depending', 'upon', 'number', 'conﬁgurations', 'two', 'states', '12:', 's', '(cid3)', 'su', '¼', 'k', 'ln', 'oou', 'o', 'ou', 'represent', 'number', 'conﬁgurations', 'the', 'evaluation', 'conﬁgurations', 'numerous', 'methods', 'allows', 'one', 'write', 'equations', 'change', 'entropy', 'n0', 'l', 'lu', 'relative', 'lengths', 'unstretched', 'stretched', 'elastomer', 'tobolsky', 'derived', 'tensile', 'strength', '12:', 's', '(cid3)', 'su', '¼', '(cid3)', 'n0k½ðlluþ2', 'þ', '2llu', '(cid3)', '3(cid4)', '1', '2', 'x', '¼', 'ðn0ktluþ½ðlluþ', '(cid3)', 'ðlluþ2(cid4)', 'by', 'dividing', 'sides', 'equation', 'cross-sectional', 'area', 'sample', 'one', 'obtain', 'stress–strain', 'curve', 'ideal', 'rubber', 'the', 'retractive', 'force', 'elastomer', 'explained', 'increases', 'temperature', 'in', 'words', 'temperature', 'elastomers', 'increases', 'adiabatic', 'stretching', '21', '22.', 'the', 'equation', 'relationship', 'written', 'kelvin', 'back', '1857', '22:', 'ðf', '@tþpl', '¼', 'ðcptþðt@lþpadiabatic', 'cp', 'speciﬁc', 'heat', '∂l', 'change', 'length', 'elastomer', 'experimental', 'evidence', 'supports', 'temperature', 'elastomers', 'like', 'rubber', 'rises', 'upon', 'stretching', 'this', 'equation', 'also', 'written', 'another', 'form', 'ðt@f', '0þpadiabatic', '¼', 'tcpðl@tþp', 'f', '0', 'in', 'actual', 'dealing', 'polymers', 'stretching', 'rubber', 'elastomers', 'requires', 'overcoming', 'energy', 'barriers', 'polymeric', 'chains', 'internal', 'energy', 'material', 'depending', 'slightly', 'elongation', 'ðh@lþtp', '¼', 'ðe@lþtp', 'þ', 'pðv@lþtp', '∂v', 'change', 'volume', 'at', 'normal', 'pressures', 'second', 'term', 'right', 'becomes', 'negligible', 'it', 'represents', 'deviation', 'ideality', 'the', 'contribution', 'internal', 'energy', 'e', 'force', 'retraction', 'f', '¼', 'ðe@lþtp', '(cid3)', 'tðs@lþt', 'bueche', '16', 'expressed', 'differently', 'work', 'done', 'stretching', 'elastic', 'polymeric', 'body', 'it', 'describes', 'deforming', 'elastomer', 'x', 'length', 'stretched', 'increase', 'length', 'polymeric', '22', 'the', 'amorphous', 'state', '27', 'chain', 'composed', 'n', 'freely', 'oriented', 'segments', 'the', 'dimensions', 'polymeric', 'chain', 'z', 'these', 'coordinates', 'become', 'reduced', 'result', 'stretching', '1(a)05', 'work', 'done', 'per', 'chain', '¼', 'ð3kt2na2þ½ða2', '(cid3)', 'a(cid3)1þx0', '2', 'þ', 'ða(cid3)1', '(cid3)', '1þr0', '2(cid4)', 'chain', 'ends', 'initially', 'r0', 'distance', 'apart', '16,', 'r0', '2', '¼', 'x2', 'þ', 'y2', 'þ', 'z2', 'bueche', '16', 'also', 'described', 'average', 'energy', 'per', 'chain', 'ave', 'energychain', '¼', 'ð3kt2þð13þ½ða2', '(cid3)', 'a(cid3)1þ', 'þ', 'ða(cid3)1', '(cid3)', '1þ(cid4)', 'the', 'modulus', 'elasticity', 'g', 'shown', 'related', 'strain', 'elongation', 'polymeric', 'elastomers', 'for', '300', 'elongation', 'following', 'relationship', '22', 'applies', 'where', 'k', 'boltzmann’s', 'constant', 'represents', 'number', 'polymeric', 'chains', 'sample', 'g', 'strain', 'the', 'relationship', 'stress', 'strain', 'g', '¼', '2mktb2ð1', 'þ', '2g3þ', 'stress', '¼', '2mktb2ðg', 'þ', '2g3þ', 'there', 'several', 'molecular', 'theories', 'rubber-like', 'elasticity', 'the', 'simplest', 'one', 'based', 'gaussian', 'distribution', 'function', 'end', 'end', 'separation', 'network', 'chains', '23', '(the', 'dimensions', 'free', 'chains', 'unperturbed', 'excluded', 'volume', 'effect', 'represented', '(r2)0)', 'oðrþ', '¼', 'ð32pðr2þ0þ32', 'expð3r22ðr2þ0þ', 'within', 'gaussian', 'distribution', 'function', '(r2)0', 'applies', 'network', 'chains', 'unstretched', 'stretched', 'state', 'the', 'free', 'energy', 'chain', 'described', 'boltzmann', 'relationship', '23:', 'fðtþ', '¼', '(cid3)kt', 'ln', 'oðrþ', '¼', 'cðtþ', 'þ', 'ð3kt2ðr2þ0þr2', 'c(t)', 'constant', 'speciﬁed', 'absolute', 'temperature', 't', 'the', 'change', 'free', 'energy', 'stretched', 'elastomer', 'expressed', 'follows', 'df', '¼', '½3kt2ðr2þ0(cid4)½ða2', 'xx2', 'þ', 'a2', 'yy2', 'þ', 'a2', 'z', 'z2þ', '(cid3)', 'ðx2', 'þ', 'y2', 'þ', 'z2þ(cid4)', 'molecular', 'deformation', 'ratio', 'r', 'components', 'xyz', 'directions', 'unstretched', 'elastomer', 'one', 'stretched', 'deformed', 'additional', 'discussions', 'theory', 'theories', 'elasticity', 'presented', 'thorough', 'discussions', 'subject', 'belong', 'books', 'dedicated', 'physical', 'properties', 'polymers', '223', 'rheology', 'viscoelasticity', 'polymeric', 'materials', 'when', 'amorphous', 'polymer', 'possesses', 'certain', 'amount', 'rotational', 'freedom', 'deformed', 'application', 'force', 'application', 'force', 'cause', 'polymer', 'ﬂow', 'molecules', '28', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'move', 'past', 'the', 'science', 'deformation', 'ﬂow', 'called', 'rheology', 'in', 'event', 'force', 'applied', 'quickly', 'withdrawn', 'rapidly', 'polymer', 'molecules', 'tend', 'revert', 'back', 'previous', 'undisturbed', 'conﬁguration', 'this', 'called', 'relaxation', 'thus', 'amorphous', 'polymers', 'exhibit', 'elastic', 'behavior', 'due', 'disruption', 'thermodynamically', 'favorable', 'arrangements', 'if', 'however', 'force', 'applied', 'gradually', 'consistently', 'molecules', 'ﬂow', 'irreversibly', 'due', 'chain', 'entanglement', 'increases', 'molecular', 'weight', 'due', 'frictional', 'effects', 'viscosity', 'ﬂowing', 'liquid', 'high', 'thus', 'molecular', 'weight', 'control', 'important', 'polymer', 'processing', 'in', 'way', 'paradoxical', 'situation', 'higher', 'molecular', 'weights', 'usually', 'yield', 'better', 'mechanical', 'properties', 'on', 'hand', 'higher', 'molecular', 'weight', 'materials', 'harder', 'process', 'the', 'molecular', 'weight', 'control', 'therefore', 'quite', 'critical', 'the', 'combination', 'properties', 'polymeric', 'liquids', 'elasticity', 'viscous', 'ﬂow', 'referred', 'viscoelasticity', 'it', 'means', 'reversible', 'uncoiling', 'polymeric', 'chains', 'by', 'terminology', 'viscous', 'ﬂow', 'means', 'irreversible', 'slipping', 'chains', 'past', 'thus', 'viscoelastic', 'materials', 'exhibit', 'simultaneously', 'combination', 'elastic', 'viscous', 'behavior', 'this', 'type', 'behavior', 'particularly', 'prominent', 'polymeric', 'materials', 'the', 'ﬂow', 'behavior', 'polymeric', 'liquids', 'also', 'inﬂuenced', 'molecular', 'weight', 'distribution', 'at', 'proper', 'temperatures', 'mechanical', 'properties', 'many', 'amorphous', 'polymers', 'may', 'approach', 'physical', 'properties', 'three', 'idealized', 'materials', 'individually', 'these', '20', '22', '24:', '1', 'a', 'hookian', 'ideal', 'elastic', 'solid', 'whose', 'small', 'reversible', 'deformations', 'directly', 'proportional', 'applied', 'force', '2', 'a', 'newtonian', 'liquid', 'ﬂows', 'viscosity', 'independent', 'rate', 'shear', '3', 'an', 'ideal', 'elastomer', 'capable', 'reversible', 'extension', 'several', 'hundred', 'percent', 'much', 'smaller', 'stiffness', 'hookian', 'solid', 'in', 'elastic', 'response', 'hookian', 'liquid', 'stress–strain', 'relationship', 's(t)', '¼', 'gg(t)', 'for', 'newtonian', 'liquid', 's(t)', '¼', '(cid2)g0(t)', 'in', 'equations', 'g(t)', 'g0(t)', 'represent', 'shear', 'strain', 'shear', 'rate', 'time', 's(t)', 'shear', 'stress', 'many', 'forces', 'applied', 'polymer', 'deformation', 'the', 'force', 'rheologists', 'particularly', 'concerned', 'tangential', 'stress', 'shear', 'this', 'due', 'fact', 'many', 'polymers', 'extruded', 'forced', 'ﬂow', 'dies', 'shaping', 'commercial', 'use', 'if', 'shearing', 'force', 'f', 'applied', 'cube', 'molten', 'polymer', 'per', 'unit', 'area', 'causes', 'top', 'layer', 'liquid', 'move', 'distance', 'x', 'ﬁxed', 'point', 'bottom', 'material', 'velocity', 'v', 'shear', 'causes', 'molecules', 'move', 'past', 'this', 'illustrated', 'fig', '24', 'the', 'assumes', 'viscosity', 'material', 'sufﬁciently', 'small', 'cube', 'distorted', 'process', 'the', 'viscosity', 'material', '(cid2)', 'deﬁned', 'ratio', 'applied', 'force', 'velocity', 'gradient', '∂v∂x', 'rate', 'shear', 'g', 'free', 'energy', 'f', '¼', '(cid2)ðvxþ', 'the', 'equation', 'rearranged', 'another', 'form', 'fv', 'energy', 'used', 'per', 'second', 'cube', 'the', 'shear', 'stress', 'deﬁned', '(cid2)', '¼', 'fvv2', '¼', '(cid2)v@x', '¼', '(cid2)g', '22', 'the', 'amorphous', 'state', 'fig', '24', 'illustration', 'movement', 'upper', 'layer', 'due', 'applied', 'force', '29', 'g', 'shear', 'rate', 'in', 'instance', 'equal', 'v', 'if', 'viscosity', '(cid2)', 'independent', 'shear', 'rate', 'liquid', 'exhibits', 'ideal', 'ﬂow', 'newtonian', 'liquid', 'tobolsky', 'commented', '12', 'probably', 'newtonian', 'liquids', 'even', 'like', 'water', 'benzene', 'ﬂuid', 'possess', 'elastic', 'well', 'viscous', 'behavior', 'however', 'ﬂow', 'polymer', 'liquids', 'deviates', 'strongly', 'ideal', 'behavior', 'either', 'viscosity', 'decreases', 'rate', 'shear', 'ﬂow', 'occurs', 'certain', 'minimum', 'force', 'applied', 'the', 'decrease', 'viscosity', 'rate', 'shear', 'called', 'shear', 'thinning', 'it', 'believed', 'result', 'tendency', 'applied', 'forces', 'disturb', 'long', 'chains', 'favored', 'equilibrium', 'conformations', 'in', 'case', 'polymer', 'yield', 'point', 'yield', 'value', 'a', 'liquid', 'yield', 'point', 'called', 'bingham', 'newtonian', 'ﬂuid', 'provided', 'exhibits', 'newtonian', 'behavior', 'starts', 'ﬂowing', 'it', 'deﬁned', '¼', 'tc', 'þ', '(cid2)g', 'tc', 'critical', 'shear', 'stress', 'threshold', 'stress', 'needed', 'initiate', 'ﬂow', 'the', 'bingham', 'ﬂuid', 'behavior', 'might', 'attributed', 'structural', 'arrangements', 'molecules', 'give', 'rise', 'conformational', 'secondary', 'bonding', 'forces', 'the', 'non-newtonian', 'behavior', 'occurs', 'shear', 'stress', 'increase', 'proportion', 'shear', 'rate', 'in', 'addition', 'thixotropic', 'liquids', 'exhibit', 'high', 'viscosity', 'even', 'resemble', 'gelation', 'low', 'shear', 'rate', 'ﬂow', 'readily', 'exhibit', 'low', 'viscosity', 'upon', 'application', 'high', 'shear', 'high', 'shear', 'rates', 'cause', 'chain', 'rupture', 'result', 'loss', 'molecular', 'weight', 'in', 'cases', 'shear', 'rate', 'may', 'increase', 'due', 'increase', 'molecular', 'entanglement', 'in', 'case', 'ﬂexible', 'chain', 'polymers', 'critical', 'molecular', 'weight', 'chain', 'entanglement', 'may', 'show', 'increase', 'for', 'common', 'polymers', 'may', 'molecular', 'weight', 'range', '4000–15000', 'flow', 'also', 'inﬂuenced', 'chain', 'branching', 'the', 'higher', 'degree', 'branching', 'polymer', 'lower', 'degree', 'entanglement', 'given', 'molecular', 'weight', 'in', 'general', 'viscosity', 'higher', 'linear', 'polymers', 'branched', 'ones', 'given', 'shear', 'rate', 'molecular', 'weight', 'flow', 'behavior', 'also', 'inﬂuenced', 'molecular', 'weight', 'distribution', '(see', 'sect', '271)', 'usually', 'broader', 'molecular', 'weight', 'distribution', 'polymers', 'lower', 'shear', 'rate', 'needed', 'cause', 'shear', 'thinning', 'on', 'hand', 'polymers', 'narrow', 'molecular', 'weight', 'distribution', 'shear', 'thinning', 'starts', 'pronounced', 'this', 'due', 'absence', 'chain', 'entanglement', 'higher', 'molecular', 'weight', 'polymeric', 'chains', 'the', 'viscosity', 'low', 'molecular', 'weight', 'polymers', 'related', 'temperature', 'arrhenius-', 'type', 'relationship', '19', '22', '25:', 'm(cid2)', '¼', 'ae(cid3)ert', 'e', 'activation', 'energy', 'viscous', 'ﬂow', 'a', 'constant', 'related', 'molecular', 'motion', 'm(cid2)', 'viscosity', 'average', 'molecular', 'weight', 'for', 'branched', 'polymers', 'larger', 'bulkier', 'side', 'chains', 'greater', 'activation', 'energy', 'e', 'the', 'true', 'polymers', 'large', 'pendant', 'groups', 'the', 'activation', 'energy', 'ﬂow', 'e', 'increase', 'proportionately', 'heat', 'vaporization', 'polymers', 'rather', 'levels', 'value', 'independent', 'molecular', 'weight', 'this', 'implies', 'long', 'chains', 'unit', 'ﬂow', 'much', 'smaller', 'whole', 'molecule', 'instead', 'segments', 'molecules', 'larger', '50', 'carbon', 'atoms', 'move', 'successive', 'jumps', '30', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'fig', '25', 'illustration', 'physical', 'crosslink', 'molten', 'polymers', 'this', 'accomplished', 'degree', 'coordination', 'results', 'whole', 'chain', 'shifting', 'based', 'experimental', 'evidence', 'viscosity', 'deﬁned', 'ln', '(cid2)', '¼', '34', 'ln', 'mw', 'þ', 'c', 'þ', 'ert', 'c', 'constant', 'mw', 'weight', 'average', 'molecular', 'weight', '(see', 'sect', '27)', 'chain', 'length', 'z', 'molecular', 'weight', 'polymers', 'important', 'variable', 'inﬂuences', 'ﬂow', 'properties', 'polymers', 'the', 'relationship', 'newtonian', 'viscosity', 'amorphous', 'polymer', 'chain', 'length', 'shear', 'stress', 'low', 'expressed', '19', '22', '25,', 'log', '(cid2)', '¼', '34', 'log', 'zw', 'þ', 'k', 'constant', 'k', 'temperature-dependent', 'by', 'token', 'based', 'experimental', 'evidence', 'relationship', 'viscosity', 'temperature', 'chain', 'length', 'low', 'shear', 'rates', 'many', 'polymers', 'expressed', 'follows', 'log', '(cid2)', '¼', '34', 'log', 'zw', '(cid3)', '1744ðt', '(cid3)', 'tgþð516', 'þ', 't', '(cid3)', 'tgþ', 'þ', 'k0', 'constant', 'k0', 'speciﬁc', 'polymer', 'tg', 'glass', 'transition', 'temperature', '(tg', 'discussed', 'sect', '223)', 'although', 'linear', 'molten', 'polymers', 'exhibit', 'well-deﬁned', 'viscosities', 'also', 'exhibit', 'elastic', 'effects', 'these', 'effects', 'present', 'even', 'absence', 'crosslinks', 'rubber', 'network', 'it', 'referred', 'creep', 'this', 'creep', 'attributed', 'entanglement', 'polymeric', 'chain', 'form', 'temporary', 'physical', 'crosslinks', 'this', 'illustrated', 'fig', '25', 'deviations', 'newtonian', 'ﬂow', 'occur', 'shear', 'stress', 'increase', 'direct', 'proportion', 'shear', 'rate', 'such', 'deviation', 'may', 'direction', 'thickening', '(called', 'dilatent', 'ﬂow)', 'direction', 'thinning', '(called', 'pseudo', 'plastic)', 'related', 'non-newtonian', 'ﬂow', 'behavior', 'thixotropic', 'liquids', 'subjected', 'shear', 'explained', 'flow', 'behavior', 'represented', 'following', 'equation', '¼', 'agb', 'a', 'constant', 'b', 'index', 'ﬂow', 'for', 'newtonian', 'liquids', 'a', '¼', '(cid2)', 'b', '¼', '1', 'all', 'polymers', 'tend', 'exhibit', 'shear', 'thinning', 'high', 'shear', 'rates', 'this', 'commonly', 'explained', 'molecular', 'entanglement', 'mentioned', 'certainly', 'amorphous', 'state', 'considerable', 'entangle-', 'ment', 'polymeric', 'chains', 'low', 'shear', 'rates', 'may', 'disrupt', 'certain', 'degree', 'chains', 'still', 'remain', 'entangled', 'as', 'shear', 'rates', 'increases', 'disruption', 'occur', 'faster', 'rate', 'chain', '22', 'the', 'amorphous', 'state', 'fig', '26', 'illustration', 'plot', 'modulus', 'elasticity', 'time', '31', 'reentangle', 'the', 'decreased', 'amount', 'entanglement', 'results', 'lower', 'viscosity', 'liquid', 'allowing', 'molecules', 'ﬂow', 'less', 'resistance', 'actually', 'two', 'factors', 'contribute', 'chain', 'entanglement', 'these', 'high', 'length', 'chains', 'large', 'molecules', 'andor', 'bulky', 'substituents', 'stress', 'applied', 'newtonian', 'liquid', 'outside', 'initial', 'spike', 'zero', 'stress', 'however', 'applied', 'viscoelastic', 'ﬂuid', 'starts', 'initial', 'value', 'this', 'value', 'decreases', 'time', 'reaches', 'equilibrium', 'value', 'due', 'viscoelastic', 'property', 'material', 'figure', '26', 'illustrates', 'plot', 'modulus', 'elasticity', 'g(t)', 'depends', 'temperature', 'plotted', 'time', 'looks', 'like', 'the', 'equation', 'shear–stress', 'relaxation', 'modulus', 'varies', 'temperature', 'written', 'follows', 'gðtþ', '¼', 'sðtþg0', 'with', 'constant', 'stress', 's(t)', '¼', 'gg0', 'creep', 'strain', 'g(t)', 'constant', 'g(t)', '¼', 's0g', 'hookean', 'solid', 'it', 'would', 'directly', 'proportional', 'time', 'newtonian', 'liquid', '(g(t)', '¼', 's0(cid2))t.', 'here', 'initial', 'time', 'recovery', 'viscoelastic', 'material', 'begins', 'for', 'viscoelastic', 'ﬂuid', 'stress', 'applied', 'period', 'creep', 'followed', 'period', 'recovery', 'for', 'liquids', 'strains', 'return', 'back', 'toward', 'zero', 'ﬁnally', 'reach', 'equilibrium', 'smaller', 'total', 'strain', 'for', 'viscoelastic', 'liquid', 'creep', 'phase', 'strain', 'starts', 'small', 'value', 'builds', 'rapidly', 'decreasing', 'rate', 'steady', 'state', 'reached', 'after', 'strain', 'simply', 'increases', 'linearly', 'time', 'during', 'linear', 'range', 'ratio', 'shear', 'strain', 'shear', 'stress', 'function', 'time', 'alone', 'this', 'shear', 'creep', 'compliance', 'j(t)', 'the', 'equation', 'shear', 'creep', 'compliance', 'written', 'follows', 'the', 'stress', 'relaxation', 'modulus', 'creep', 'compliance', 'manifestations', 'dynamic', 'process', 'molecular', 'level', 'closely', 'related', 'this', 'relationship', 'however', 'simple', 'reciprocal', 'relations', 'would', 'expressed', 'g(t)', '¼', '1j(t)', 'rather', 'integral', 'equation', 'derived', 'boltzman', 'superposition', 'principle', 'it', 'relates', 'recoverable', 'compliance', 'j0', '(cid2)0', 'zero', 'shear', 'viscosity', '22.', 'jðtþ', '¼', 'gðtþs0', 'j0', '¼', 'tgðtþdt', '1', '1', '(cid2)2', '0', '0', 'z', '1', '0', 'z', '(cid2)0', '¼', 'gðtþdt', 'in', 'relaxation', 'back', 'equilibrium', 'polymer', 'assumes', 'new', 'conformation', 'at', 'ﬁrst', 'response', 'glassy', 'the', 'modulus', 'organic', 'glass', 'large', 'gg', '109', 'pa', 'this', 'modulus', 'decreases', '32', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'fig', '27', 'illustration', 'cone', 'plate', 'rheometer', 'time', 'polymer', 'begins', 'relax', 'continues', 'along', 'whole', 'length', 'chain', 'for', 'short', 'chains', 'relaxation', 'zero', 'takes', 'place', 'fairly', 'constant', 'rate', 'for', 'long', 'chains', 'however', 'relaxation', 'rate', 'tends', 'three', 'stages', 'it', 'starts', 'certain', 'rate', 'stage', 'one', 'noticeably', 'slow', 'point', 'stage', 'two', 'modulus', 'remains', 'relatively', 'constant', 'period', 'time', 'after', 'stage', 'three', 'relaxation', 'resumed', 'rapid', 'rate', 'full', 'equilibrium', 'reached', 'at', 'stage', 'two', 'period', 'relatively', 'constant', 'modulus', 'affected', 'chain', 'architecture', 'material', 'resembles', 'rubber', 'network', 'the', 'length', 'third', 'stage', 'however', 'profoundly', 'affected', 'molecular', 'weight', 'molecular', 'weight', 'distribution', 'long-chain', 'branching', 'polymer', 'koga', 'tanaka', '23', 'studied', 'behavior', 'normal', 'stresses', 'associated', 'networks', 'composed', 'telechelic', 'polymers', 'steady', 'shear', 'ﬂow', 'they', 'showed', 'numerically', 'ﬁrst', 'second', 'normal', 'stress', 'coefﬁcients', 'reveal', 'thickening', 'function', 'shear', 'rate', 'sign', 'second', 'normal', 'stress', 'coefﬁcient', 'changes', 'depending', 'nonlinearity', 'chain', 'tension', 'dissociation', 'rate', 'associative', 'groups', 'junctions', 'shear', 'rate', 'analytic', 'calculation', 'showed', 'limit', 'small', 'shear-rate', 'sign', 'inversion', 'occurs', 'competition', 'nonlinear', 'stretching', 'dissociation', 'associative', 'groups', 'thus', 'molecular', 'mecha-', 'nism', 'sign', 'inversion', 'shown', 'similar', 'thickening', 'shear', 'viscosity', 'in', 'behavior', 'polymeric', 'liquids', 'two', 'quantities', 'important', 'these', 'steady-state', 'recover-', '(as', 'shown', 'above)', 'steady-state', 'viscosity', 'zero', 'shear', 'rate', '(cid2)0', 'these', 'able', 'shear', 'compliance', 'j0', 'quantities', 'related', 'recoverable', 'shear', 'compliance', 'j0', '¼', 'grs0', 'zero', 'shear', 'viscosity', '(cid2)0', '¼', 's0g0', 'ss', 'g0', 'ss', 'shear', 'rate', 'gr', 'total', 'recoil', 'strain', 'both', 'shear', 'compliance', 'shear', 'viscosity', 'obtained', 'creep', 'studies', 'the', 'product', 'two', 'zero', 'shear', 'compliance', 'zero', 'shear', 'viscosity', 'characteristic', 'relaxation', 'time', 'polymer', 'there', 'various', 'techniques', 'determining', 'viscosity', 'molten', 'polymers', 'one', 'commonly', 'used', 'piece', 'equipment', 'cone', 'plate', 'rotational', 'viscometer', 'the', 'equipment', 'illustrated', 'fig', '27', 'the', 'molten', 'polymer', 'placed', 'bottom', 'plate', 'cone', 'cone', 'rotated', 'constant', 'speed', 'shear', 'stress', 'obtained', 'following', 'relationship', 'm', 'torque', 'dynes', 'per', 'centimeter', 'r', 'cone', 'radius', 'centimeters', '(or', 'meters)', 'the', 'shear', 'rate', 'obtained', 'following', 'equation', 't0', '¼', 'j0', '(cid2)0', '¼', '3m2pr', '22', 'the', 'amorphous', 'state', 'fig', '28', 'illustration', 'capillary', 'viscometer', '33', 'cone', 'angle', 'radians', 'degrees', 'o', 'angular', 'velocity', 'radians', 'per', 'second', 'degrees', 'per', 'second', 'the', 'viscosity', 'obtained', 'following', 'relationship', '15', '16:', '(cid2)', '¼', 'tg0', '¼', '3am2pr3o', '¼', 'kmo', 'k', 'constant', 'speciﬁc', 'viscometer', 'used', 'it', 'obtained', 'relationship', '7:', 'the', 'cone', 'plate', 'rheometers', 'useful', 'relatively', 'low', 'shear', 'rates', 'for', 'higher', 'shear', 'rates', 'capillary', 'rheometers', 'employed', 'they', 'usually', 'constructed', 'metals', 'the', 'molten', 'polymer', 'forced', 'capillary', 'constant', 'displacement', 'rate', 'also', 'may', 'constructed', 'suit', 'various', 'speciﬁc', 'shear', 'stresses', 'encountered', 'commercial', 'operation', 'their', 'big', 'disadvantage', 'shear', 'stress', 'capillary', 'tubes', 'varies', 'maximum', 'walls', 'zero', 'center', 'on', 'hand', 'stable', 'operation', 'much', 'higher', 'shear', 'rates', 'possible', 'determination', 'however', 'z0', 'usually', 'possible', 'due', 'limitations', 'instruments', 'at', 'low', 'shear', 'rates', 'one', 'determine', 'steady-', 'state', 'viscosity', 'measurements', 'volumetric', 'ﬂow', 'rates', 'q', 'pressure', 'drop', 'p0', 'ambient', 'pressure', 'a', 'capillary', 'viscometer', 'illustrated', 'fig', '28', 'diameter', 'capillary', 'designated', 'd', 'for', 'newtonian', 'liquids', 'viscosity', 'determined', 'following', 'equation', 'l', 'represents', 'length', 'capillary', 'the', 'shear', 'stress', 'capillary', 'wall', 'calculated', 'pressure', 'drop', 'g0', '¼', 'oa', 'k', '¼', '3a2pr3', 'dp', '¼', 'p', '(cid3)', 'p0', '(cid2)', '¼', 'pd4dp128lq', 'swall', '¼', 'ddp4l', 'also', 'shear', 'rate', 'walls', 'capillary', 'calculated', 'ﬂow', 'rate', '22:', 'g0', 'wall', '¼', '8q', '3', 'þ', '(cid2)', 'log', 'q', 'log', 'dp', 'pd3', '(cid3)', '(cid4)', 'wang', 'et', 'al', 'studied', 'homogeneous', 'shear', 'wall', 'slip', 'shear', 'banding', 'entangled', 'polymeric', 'liquids', 'simple-shear', 'rheometry', 'capillary', 'viscometry', 'shown', '20.', 'they', 'observed', 'recent', 'particle-tracking', 'velocimetric', 'observations', 'revealed', 'well-entangled', 'polymer', 'solutions', 'melts', 'tend', 'either', 'exhibit', 'wall', 'slip', 'assume', 'inhomogeneous', 'state', 'deformation', 'ﬂow', 'nonlinear', 'rheological', 'measurements', 'simple-shear', 'rheometric', 'setups', 'it', 'important', 'control', 'viscoelastic', 'properties', 'conﬁned', 'polymers', 'many', 'applications', 'these', 'applications', 'microelectronics', 'optics', 'the', 'rheological', 'properties', '34', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'ﬁlms', 'however', 'hard', 'measure', 'recently', 'chan', 'et', 'al', '27', 'reported', 'viscoelasticity', 'measured', 'thermal', 'wrinkling', 'thermally', 'induced', 'instability', 'develops', 'polymer', 'ﬁlms', 'compressed', 'rigid', 'stiffer', 'layers', 'this', 'due', 'differences', 'coefﬁcients', 'thermal', 'expansion', 'polymer', 'inorganic', 'layers', 'a', 'net', 'compressive', 'stress', 'develops', 'polymer–substrate', 'interface', 'composite', 'layers', 'heated', 'temperatures', 'promote', 'mobility', 'polymer', 'layer', 'such', 'wrinkling', 'substrate', 'surface', 'characterized', 'isotropic', 'morphology', 'approximated', 'sinusoidal', 'proﬁle', 'chan', 'et', 'al', 'utilized', 'thermal', 'wrinkling', 'measure', 'rubbery', 'modulus', 'shear', 'viscosity', 'polystyrene', 'thin', 'ﬁlms', 'function', 'temperature', 'they', 'used', 'surface', 'laser-light', 'scattering', 'characterize', 'wrinkled', 'surface', 'real-', 'time', 'order', 'monitor', 'changes', 'morphology', 'function', 'annealing', 'time', 'ﬁxed', 'annealing', 'temperatures', 'the', 'results', 'compared', 'theoretical', 'model', 'viscoelastic', 'properties', 'ps', 'thin', 'ﬁlm', 'extracted', '23', 'the', 'crystalline', 'state', 'mendelkern', '5', 'pointed', 'essentially', 'properties', 'polymers', 'controlled', 'molecular', 'morphology', 'in', 'contrast', 'amorphous', 'liquid', 'state', 'crystalline', 'state', 'relatively', 'inelastic', 'rigid', 'in', 'crystalline', 'state', 'bonds', 'adopt', 'set', 'successive', 'preferred', 'orientations', 'liquid', 'state', 'bond', 'orientation', 'chains', 'adopt', 'statistical', 'conformations', 'in', 'crystalline', 'state', 'properties', 'polymers', 'differ', 'considerably', 'if', 'polymer', 'molecule', 'coiled', 'randomly', 'cannot', 'ﬁt', 'readily', 'geometrically', 'arranged', 'regular', 'crystalline', 'lattice', 'so', 'molecules', 'must', 'change', 'uniform', 'shape', 'ﬁt', 'crystal', 'pattern', 'in', 'many', 'cases', 'assume', 'either', 'helix', 'zigzag', 'conformation', 'such', 'arrangements', 'regular', 'random', 'coil', '29–40.', 'this', 'even', 'detected', 'spectral', 'thermodynamic', 'studies', 'during', 'crystallization', 'polymers', 'bulky', 'substituents', 'spaced', 'close', 'polymeric', 'chains', 'tend', 'form', 'helical', 'conformations', 'remain', 'crystalline', 'phase', 'the', 'arrangement', 'allows', 'close', 'packing', 'substituents', 'without', 'much', 'distortion', 'chain', 'bonds', 'that', 'particularly', 'true', 'many', 'isotactic', 'polymers', 'crystallize', 'helical', 'conformations', 'taking', 'gauche', 'trans', 'positions', 'for', 'gauche', 'position', 'steric', 'hindrance', 'always', 'forces', 'rotation', 'place', 'substituents', 'juxtaposition', 'generating', 'either', 'right-hand', 'left-had', 'helix', 'the', 'helical', 'conformations', 'isotactic', 'vinyl', 'polymers', 'illustrated', 'gaylord', 'mark', '41', 'shown', 'fig', '29', 'when', 'macromolecules', 'possess', 'certain', 'amount', 'symmetry', 'strong', 'accompanying', 'tendency', 'form', 'ordered', 'domains', 'crystalline', 'regions', 'crystallinity', 'however', 'polymers', 'differs', 'nature', 'small', 'molecules', 'when', 'small', 'molecules', 'crystallize', 'crystal', 'forms', 'made', 'molecules', 'totally', 'participate', 'makeup', 'but', 'polymers', 'crystallize', 'melt', 'means', 'certain', 'elements', 'polymeric', 'system', 'segments', 'polymeric', 'chains', 'attained', 'form', 'three-dimensional', 'order', 'complete', 'crystallization', 'melt', 'however', 'seldom', 'ever', 'achieved', 'the', 'ordered', 'conformations', 'may', 'fully', 'extended', 'may', 'one', 'helical', 'forms', 'shown', 'this', 'resembles', 'orderly', 'arrangement', 'small', 'molecules', 'crystals', 'the', 'crystalline', 'domains', 'however', 'much', 'smaller', 'crystals', 'small', 'molecules', 'possess', 'many', 'imperfections', '231', 'crystallization', 'melt', 'certain', 'basic', 'information', 'established', 'crystallization', 'melt', '5:', 'the', 'process', 'ﬁrst-order', 'phase', 'transition', 'follows', 'general', 'mathematical', 'formulation', 'kinetics', '23', 'the', 'crystalline', 'state', 'fig', '29', 'illustration', 'helical', 'conformation', 'isotactic', 'vinyl', 'polymers', '35', 'phase', 'change', 'equilibrium', 'conditions', 'however', 'seldom', 'ever', 'attained', 'result', 'complete', 'crystallinity', 'stated', 'reached', 'the', 'tendency', 'crystallize', 'depends', 'upon', 'following', '1', 'structural', 'regularity', 'chains', 'leads', 'establishment', 'identity', 'periods', '2', 'free', 'rotational', 'vibrational', 'motions', 'chains', 'allow', 'different', 'conformations', 'assumed', '3', 'presence', 'structural', 'groups', 'capable', 'producing', 'lateral', 'intermolecular', 'bonds', '(van', 'der', 'wall', 'forces)', 'regular', 'periodic', 'arrangement', 'bonds', '4', 'absence', 'bulky', 'substituents', 'space', 'irregularity', 'would', 'prevent', 'segments', 'chains', 'ﬁtting', 'crystal', 'lattices', 'prevent', 'laterally', 'bonding', 'groups', 'approaching', 'close', 'enough', 'best', 'interaction', 'natta', 'carradini', '30', 'postulated', 'three', 'principles', 'determining', 'crystal', 'structures', 'polymers', 'these', '1', 'it', 'possible', 'assume', 'mer', 'units', 'crystal', 'occupy', 'geometrically', 'equivalent', 'positions', 'respect', 'chain', 'axis', 'this', 'known', 'equivalence', 'postulate', '2', 'the', 'conformation', 'chain', 'crystal', 'assumed', 'approach', 'conformation', 'minimum', 'potential', 'energy', 'isolated', 'chain', 'oriented', 'along', 'axis', 'this', 'minimum', 'energy', 'postulate', '3', 'as', 'many', 'elements', 'symmetry', 'isolated', 'chain', 'possible', 'maintained', 'lattice', 'equivalent', 'atoms', 'different', 'mer', 'units', 'along', 'axis', 'tend', 'assume', 'equivalent', 'positions', 'respect', 'atoms', 'neighboring', 'chains', 'this', 'packing', 'postulate', 'x-ray', 'diffraction', 'studies', 'polymers', 'crystallized', 'melt', 'show', 'recognizable', 'features', 'the', 'bragg', 'reﬂections', 'however', 'appear', 'broad', 'diffuse', 'obtained', 'well-developed', 'single', 'crystals', 'such', 'broadening', 'could', 'result', 'crystals', 'small', 'it', 'could', 'also', 'result', 'lattice', 'defects', 'because', 'diffraction', 'patterns', 'weak', 'impossible', 'reach', 'conclusion', 'the', 'majority', 'opinion', 'however', 'leans', 'toward', 'small', 'crystal', 'size', '36', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'fig', '210', 'illustration', 'fringed', 'micelle', 'cause', 'broadening', '5.', 'the', 'crystals', 'melt', 'approximately', '100', '(cid5)', '200', '(cid5)', '200', 'a˚', 'size', 'rough', 'estimates', 'diffraction', 'studies', 'indicate', 'size', 'crystals', 'crystallites', 'rarely', 'exceeds', 'hundred', 'angstroms', 'the', 'fact', 'substantial', 'background', 'diffuse', 'scattering', 'suggests', 'considerable', 'amorphous', 'areas', 'also', 'present', 'because', 'chains', 'long', 'often', 'suggested', 'individual', 'chains', 'contribute', 'several', 'different', 'crystalline', 'amorphous', 'domains', 'this', 'resulted', 'proposal', 'composite', 'single-phase', 'structure', '5,', 'fringed', 'micelle', 'fringed', 'crystallite', 'model', 'this', 'illustrated', 'fig', '210', 'the', 'fringes', 'transition', 'phases', 'crystalline', 'amorphous', 'regions', 'some', 'analytical', 'studies', 'however', 'failed', 'support', 'concept', 'the', 'proportion', 'crystalline', 'matter', 'polymer', 'deﬁned', 'degree', 'crystallinity', 'it', 'expressed', 'volume', 'mass', 'by', 'expressing', 'volume', 'would', 'degree', 'crystallinity', 'xv', '¼', 'vcðvc', 'þ', 'vaþ', '¼', 'vcv', 'vc', 'va', 'v', 'represent', 'respective', 'volumes', 'crystalline', 'amorphous', 'regions', 'total', 'volume', 'when', 'expressed', 'terms', 'mass', 'equation', 'would', 'xm', '¼', 'mcðmc', '¼', 'maþ', '¼', 'mcm', 'mc', 'masses', 'crystalline', 'amorphous', 'phases', 'total', 'mass', '232', 'crystallization', 'solution', 'for', 'many', 'polymers', 'crystal', 'growth', 'also', 'take', 'place', 'dilute', 'solutions', 'solutions', 'yield', 'single', 'crystals', 'crystal', 'formations', 'polymers', 'studied', 'intensively', 'almost', 'time', 'recognition', 'existence', 'macromolecules', 'single', 'crystals', 'organic', 'polymers', 'recognized', 'early', '1927', '26.', 'it', 'became', 'subject', 'intensive', 'investigations', 'observations', 'linear', 'polyethylene', 'crystallize', 'single', 'crystals', 'the', 'observations', 'made', 'polyethylene', 'followed', 'observations', 'possible', 'also', 'grow', 'single', 'crystals', 'polymers', 'some', 'polyoxymethylene', '30,', 'polyamides', '31,', 'polypropylene', '32,', 'polyoxyethylene', '33,', '37', '23', 'the', 'crystalline', 'state', 'fig', '211', 'electron', 'diffraction', 'pattern', 'single', 'crystal', 'polyacrylonitrile', '(from', 'ref', '50)', 'fig', '212', 'models', 'chain', 'folding', '(a)', 'adjacent', 'reentry', 'model', '(b)', 'nonadjacent', 'reentry', 'model', 'cellulose', '34,', 'others', '35.', 'the', 'electron', 'diffraction', 'pattern', 'single', 'crystal', 'polyacryloni-', 'trile', 'shown', 'fig', '211', 'there', 'good', 'correlation', 'structural', 'regularity', 'speciﬁc', 'polymer', 'appear-', 'ance', 'single', 'crystals', 'relatively', 'larger', 'single', 'crystals', 'smooth', 'sharp', 'edges', 'little', 'random', 'growth', 'formed', 'polymers', 'known', 'high', 'regularity', 'structures', 'to', 'explain', 'arrangement', 'chains', 'single', 'crystals', 'theory', 'folded', 'chain', 'lamella', 'proposed', '1947', '36.', 'it', 'states', 'basic', 'units', 'single', 'crystals', 'lamellae', '100', 'a˚', 'thick', 'the', 'evidence', 'existence', 'lamellae-like', 'crystallites', 'comes', 'microscopic', 'observations', 'direct', 'microscopic', 'observations', 'however', 'yield', 'information', 'chain', 'structure', 'molecular', 'level', 'the', 'thickness', 'lamellae', 'relative', 'chain', 'length', 'led', 'postulate', 'molecules', 'arranged', 'perpendicular', 'nearly', 'layers', 'also', 'sharp', 'folded', 'conﬁgurations', 'form', 'fold', 'length', 'corresponding', 'layer', 'thickness', 'in', 'view', 'one', 'polymer', 'molecule', 'essentially', 'constrained', 'one', 'lamellae', 'interface', 'smooth', 'regularly', 'structured', 'chain', 'folding', 'back', 'forth', 'like', 'accordion', 'crystallization', 'two', 'models', 'chain', 'folding', 'visualized', 'in', 'one', 'model', 'chain', 'folding', 'regular', 'sharp', 'uniform', 'fold', 'period', 'this', 'called', 'adjacent-reentry', 'model', 'in', 'one', 'molecules', 'wander', 'irregular', 'surface', 'lamella', 'reentering', 'neighboring', 'lamella', 'this', 'mode', 'called', 'switchboard', 'nonadjacent-reentry', 'model', 'the', 'two', 'illustrated', 'fig', '212', 'some', 'early', 'experimental', 'evidence', 'failed', 'support', 'lamellae', 'concept', '37.', 'since', 'much', 'evidence', 'presented', 'support', 'today', 'accepted', 'established', 'fact', '5.', 'recently', 'hosoda', 'coworkers', '37', 'reported', 'studies', 'morphology', 'specially', 'synthesized', 'polyethylene', 'n-butyl', 'branches', 'precisely', 'spaced', 'every', '39th', 'carbon', 'this', 'compared', 'commercial', 'ethylene1-hexene', 'copolymer', 'possessing', 'branching', 'probability', 'the', 'goal', 'work', 'elucidate', 'effect', 'intramolecular', 'sequence', 'length', 'heterogeneity', 'lamella', 'crystal', 'thickness', 'distribution', 'the', 'commercial', 'copolymer', 'found', 'orthorhombic', 'crystalline', 'polymor-', 'phism', 'normal', 'polyethylenes', 'different', 'specially', 'synthesized', 'material', 'the', 'synthesized', 'one', 'exhibits', 'narrow', 'lamella', 'thickness', 'distribution', 'average', 'thickness', 'corresponding', 'exactly', 'space', 'length', 'two', 'consecutive', 'branches', 'this', 'suggests', '38', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'complete', 'exclusion', 'n-butyl', 'branches', 'crystal', 'stem', 'on', 'hand', '1-hexene', 'copolymer', 'forms', 'much', 'thicker', 'lamellae', 'broader', 'thickness', 'distribution', 'synthesized', 'polyethylene', 'butyl', 'branches', 'here', 'average', 'thickness', '15', 'times', 'larger', 'calculated', 'probable', 'ethylene', 'sequence', 'length', 'obtained', 'nmr', 'theoretical', 'ethylene', 'sequence', 'length', 'distribution', 'indicating', 'lamellae', 'composed', 'predom-', 'inantly', 'sparsely', 'branched', 'longer', 'ethylene', 'sequences', 'linear', 'polyethylene', 'single', 'crystals', 'often', 'exhibit', 'secondary', 'structural', 'features', 'include', 'corrugations', 'pleats', 'it', 'suggested', 'crystals', 'actually', 'grow', 'form', 'pyramids', 'pyramidal', 'structures', 'collapse', 'solvent', 'removed', 'preparation', 'micros-', 'copy', '42.', 'various', 'investigators', 'described', 'complex', 'structures', 'besides', 'pyramidal', 'typical', 'among', 'descriptions', 'sheaf-like', 'arrays', 'would', 'correspond', 'nuclei', 'also', 'observations', 'dendridic', 'growths', 'clusters', 'hollow', 'pyramids', 'spiral', 'growths', 'epitaxial', 'growths', 'moire´', 'patterns', 'etc', '42', '43.', '233', 'spherulitic', 'growth', 'for', 'polymers', 'crystallize', 'melt', 'important', 'parameter', 'characterization', 'two-phase', 'systems', 'weight', 'fraction', 'crystalline', 'regions', 'the', 'degree', 'crystallinity', 'reached', 'dependent', 'temperature', 'crystallization', 'takes', 'place', 'at', 'low', 'temperatures', 'one', 'attains', 'much', 'lower', 'degree', 'crystallization', 'higher', 'temperatures', 'this', 'implies', 'crystallization', 'remains', 'incomplete', 'kinetic', 'reasons', '7.', 'normal', 'conditions', 'cooling', 'molten', 'polymer', 'establish', 'crystalline', 'texture', 'polymer', 'usually', 'result', 'formation', 'tiny', 'crystals', 'these', 'crystals', 'part', 'closely', 'spaced', 'cluster', 'called', 'spherulite', 'the', 'formation', 'single', 'nucleus', 'polymer', 'cooled', 'melting', 'point', 'favors', 'formation', 'another', 'nucleus', 'vicinity', 'due', 'creation', 'local', 'stresses', 'microscopic', 'examinations', 'polarized', 'light', 'many', 'polymeric', 'materials', 'crystallized', 'melt', 'show', 'specimen', 'packed', 'spherulites', 'often', 'appear', 'symmetrical', 'structures', 'black', 'crosses', 'center', '38.', 'it', 'believed', '39', 'spherulites', 'grow', 'individual', 'nuclei', 'ribbons', 'crystallites', 'grow', 'one', 'spherulitic', 'center', 'fan', 'directions', 'initially', 'spherical', 'mutual', 'interference', 'irregular', 'shapes', 'develop', 'the', 'diameters', 'spherulites', 'range', '0005', '0100', 'mm', 'this', 'means', 'spherulite', 'consists', 'many', 'crystalline', 'noncrystalline', 'regions', 'the', 'black', 'crosses', 'seen', 'spherulites', 'explained', '39', 'assuming', 'crystallites', 'arranged', 'chains', 'preferentially', 'normal', 'radii', 'spherulites', 'spherulitic', 'morphology', 'universal', 'mode', 'polymer', 'crystallization', 'spherulitic', 'morphology', 'occurs', 'usually', 'nucleation', 'started', 'molten', 'polymer', 'concentrated', 'solution', 'polymer', 'spherulitic', 'growth', 'illustrated', 'fig', '213', 'the', 'size', 'number', 'spherulites', 'polymer', 'tends', 'affects', 'physical', 'properties', 'thus', 'impact', 'strength', 'polymer', 'ﬁlms', 'ﬂex', 'life', 'usually', 'increases', 'spherulite', 'size', 'decreases', 'on', 'hand', 'appear', 'correlation', 'yield', 'stress', 'ultimate', 'elongation', 'size', 'spherulites', 'rhythmic', 'crystal', 'growth', 'generally', 'encountered', 'thin', 'ﬁlms', 'semicrystalline', 'polymers', 'this', 'believed', 'due', 'formation', 'ring-banded', 'spherulites', 'attributed', 'periodical', 'lamellae', 'twisting', 'along', 'radial', 'growth', 'direction', 'spherulites', '42.', 'recently', 'gu', 'coworkers', '43', 'reported', 'using', 'mild', 'methylamine', 'vapor', 'etching', 'method', 'periodical', 'cooperative', 'twisting', 'lamellar', 'crystals', 'ring-banded', 'spherulites', 'clearly', 'observed', 'when', 'melt', 'solutions', 'stirred', 'epitaxial', 'crystallinity', 'usually', 'observed', 'one', 'crystalline', 'growth', 'occurs', 'right', 'top', 'another', 'this', 'arrangement', 'often', 'called', 'shish-kebab', '23', 'the', 'crystalline', 'state', '39', 'fig', '213', 'spherulitic', 'growth', '(from', 'ref', '50)', 'morphology', 'it', 'contains', 'lamella', 'growth', 'long', 'ﬁbrils', 'drawing', 'crystalline', 'polymer', 'forces', 'spherulites', 'rearrange', 'parallel', 'arrangements', 'known', 'drawn', 'ﬁbrilar', 'morphology', 'in', 'order', 'ordered', 'phase', 'crystallize', 'amorphous', 'melt', 'nucleation', 'barrier', 'must', 'overcome', 'this', 'barrier', 'result', 'interfacial', 'energy', 'ordered', 'phase', 'melt', 'causes', 'super', 'cooling', 'sirota', '44', 'suggested', 'order', 'nucleation', 'barrier', 'stable', 'phase', 'sufﬁciently', 'high', 'form', 'melt', 'another', 'phase', 'lower', 'nucleation', 'barrier', 'free', 'energy', 'intermediate', 'stable', 'phase', 'melt', 'must', 'form', 'this', 'points', 'implied', 'oswald’s', 'rule', '45', 'evidence', 'presented', 'keller', '35', 'crystallization', 'semicrys-', 'talline', 'polymer', 'systems', 'mediated', 'transient', 'metastable', 'phase', '47', '48.', 'stroble', 'coworkers', 'demonstrated', 'lamellar', 'thickness', 'determined', 'transition', 'metastable', 'phase', 'stable', 'crystal', 'phase', '46–49.', 'in', 'addition', 'relating', 'crystallization', 'temperature', 'melting', 'temperature', 'crystalline', 'lamellar', 'thickness', 'suggested', 'lamellar', 'growth', 'fronts', 'thin', 'layers', 'mesomorphic', 'phases', 'he', 'feels', 'phases', 'thicken', 'thickness', 'reached', 'stable', 'crystal', 'phases', 'favored', 'the', 'conversion', 'occurs', 'block', 'wise', 'fashion', 'results', 'granular', 'structures', 'observed', 'many', 'polymers', '46–49.', 'this', 'conver-', 'sion', 'stabilization', 'process', 'lowers', 'free', 'energy', 'newly', 'formed', 'crystallites', 'prevents', 'returning', 'mesomorphic', 'phase', 'upon', 'subsequent', 'elevation', 'temperature', '46–49.', 'such', 'concept', 'crystallization', 'however', 'universally', 'accepted', 'sirota', '49,', 'pointed', 'however', 'picture', 'thermodynamic', 'framework', 'generally', 'correct', 'many', 'cases', 'sirota', '49', 'believes', 'origin', 'granular', 'structures', 'mentioned', 'understood', 'following', 'way', 'the', 'initial', 'nucleation', 'growth', 'take', 'place', 'stem', 'addition', 'mesophases', 'lamellae', 'thickness', 'occurs', 'chains', 'mobile', 'mesophase', 'when', 'thicknesses', 'grow', 'large', 'enough', 'allow', 'conversion', 'mesophases', 'crystals', 'average', 'densities', 'lamellae', 'set', 'crystals', 'break', 'blocks', 'the', 'transitions', 'mesophases', 'crystals', 'also', 'involves', 'increases', 'lateral', 'packing', 'densities', 'in', 'semicrystalline', 'polymers', 'entanglements', 'amorphous', 'regions', 'well', 'folding', 'chains', 'lamellar', 'spanning', 'chains', 'also', 'effect', 'these', 'effects', 'limit', 'allowable', 'lateral', 'displacements', '40', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'crystalline', 'stems', 'rapid', 'solid-state', 'transformation', 'mesophases', 'crystal', 'forms', 'as', 'result', 'long', 'distances', 'molecular', 'scale', 'short', 'times', 'stems', 'kinetically', 'constrained', 'close', 'lateral', 'position', 'maintained', 'grew', 'mesophase', 'at', 'time', 'local', 'packing', 'favors', 'stable', 'crystal', 'phase', 'higher', 'lateral', 'density', 'the', 'constraints', 'chain', 'ends', 'favoring', 'lateral', 'density', 'mesophases', 'forces', 'competition', 'density', 'thermodynamically', 'favored', 'crystal', 'structures', 'this', 'causes', 'crystal', 'break', 'domains', 'sirota', 'suggests', 'granular', 'structures', 'observed', 'many', 'polymer', 'crystals', 'ﬁngerprints', 'transitions', 'transient', 'intermediates', 'crystals', 'formed', '49.', 'it', 'reported', 'recently', 'polymeric', 'also', 'form', 'quasicrystals', 'hayashida', 'et', 'al', '50', 'demonstrated', 'certain', 'blends', 'polyisoprene', 'polystyrene', 'poly(2-vinylpyridine)', 'form', 'star-', 'shaped', 'copolymers', 'assemble', 'quasicrystals', 'by', 'probing', 'samples', 'transmission', 'electron', 'microscopy', 'x-ray', 'diffraction', 'methods', 'conclude', 'ﬁlms', 'composed', 'periodic', 'patterns', 'triangles', 'squares', 'exhibit', '12-fold', 'symmetry', 'these', 'signs', 'quasi-', 'crystalline', 'ordering', 'such', 'ordering', 'differ', 'conventional', 'crystals', 'lack', 'periodic', 'structures', 'yet', 'well-ordered', 'indicated', 'sharp', 'diffraction', 'patterns', 'generate', 'quasi-crystals', 'also', 'differ', 'ordinary', 'crystals', 'another', 'fundamental', 'way', 'they', 'exhibit', 'rotational', 'symmetries', '(often', 'ﬁve', 'tenfold)', 'there', 'still', 'basic', 'questions', 'structure', '2331', 'thermodynamics', 'crystallization', 'the', 'free', 'energy', 'change', 'polymer', 'liquid', 'crystal', 'at', 'melting', 'point', 'tm', 'crystals', 'liquid', 'polymer', 'equilibrium', 'the', 'change', 'free', 'energy', 'therefore', 'df', '¼', '0', 'that', 'means', 'melting', 'point', 'df', '¼', 'dh', '(cid3)', 'tds', 'tm', '¼', 'dhds', 'if', 'heat', 'fusion', 'large', 'melting', 'temperature', 'polymer', 'crystals', 'high', '(due', 'high', 'intermolecular', 'attraction)', 'entropy', 'fusion', 'small', 'mendelkern', '42', 'noted', 'three', 'different', 'interfacial', 'free', 'energies', 'characteristic', 'crystallites', 'one', 'se', 'equilibrium', 'extended', 'chain', 'crystallite', 'second', 'one', 'sec', 'represents', 'mature', 'non-equilibrium', 'crystallite', 'third', 'one', 'sen', 'interfacial', 'free', 'energy', 'involved', 'forming', 'nucleus', 'these', 'quantities', 'cannot', 'identiﬁed', 'one', 'another', 'because', 'portions', 'polymeric', 'chains', 'participate', 'formations', 'crystallites', 'section', 'sections', 'chains', 'x', 'length', 'participate', 'crystallite', 'formation', 'designated', 'ze0', 'sections', 'chains', 'remain', 'disorder', 'amorphous', 'x', '(cid3)', 'ze0', 'the', 'dependence', 'melting', 'temperature', 'system', 'upon', 'chain', 'length', '42:', '1tm', '(cid3)', '1t0', '¼', 'ðrdhuþf½ð1xþ', 'þ', 'ð1x', '(cid3)', 'ze0', 'þ', '1þ(cid4)g', 'dhu', 'enthalpy', 'fusion', 'per', 'repeat', 'unit', 'ze', 'equilibrium', 'crystalline', 'length', 'the', 'effective', 'interfacial', 'free', 'energy', 'associated', 'basal', 'plane', 'equilibrium', 'crystalline', 'length', 'ze0', '0', 'designated', 'se', '42:', '2se', '¼', 'rtmef½zeðx', '(cid3)', 'ze', 'þ', '1þ(cid4)', 'þ', 'ln½ðx', '(cid3)', 'ze', 'þ', '1þx(cid4)g', '23', 'the', 'crystalline', 'state', '41', 't0', 'equilibrium', 'melting', 'temperature', 'inﬁnite', 'molecular', 'weight', 'chain', 'tme', 'corresponding', 'melting', 'temperature', 'fraction', 'contains', 'x', 'repeat', 'units', 'from', 'stand', 'point', 'thermodynamics', 'tme', 'ﬁrst-order', 'transition', 'temperature', '15.', 'the', 'melting', 'points', 'measured', 'carefully', 'many', 'cases', 'sharp', 'on', 'hand', 'melting', 'points', 'ordinary', 'crystals', 'may', 'melt', 'range', '(table', '23)', 'table', '12', 'shows', 'ﬁrst-order', 'transition', 'temperatures', 'commonly', 'designated', 'tm', 'for', 'systems', 'polydisperse', 'probable', 'chain-length', 'distribution', 'melting', 'temperature', 'molecular', 'weight', 'relation', 'expressed', '42:', '1t(cid6)', '(cid3)', '1t0', '¼', 'ðrdhuþð2xnþ', 'xn', 'number', 'average', 'degree', 'polymerization', 'in', 'equation', 'quantity', '2', '⁄', 'xn', 'represents', 'mole', 'fraction', 'non-crystallizing', 'units', 'this', 'equation', 'based', 'stipulation', 'conditions', 'phase', 'equilibrium', 'it', 'speciﬁc', 'valid', 'polymers', 'probable', 'molecular', 'weight', 'distribution', 'this', 'relationship', 'melting', 'temperature', 'polydisperse', 'system', 'treated', 'individually', '42.', 'by', 'applying', 'clapeyron', 'equation', 'measurements', 'applied', 'hydrostatic', 'pressure', 'value', 'dhu', 'determined', 'this', 'equation', 'tm', '¼', 'dhmdsm', 'applies', 'high', 'molecular', 'weight', 'polymers', 'for', 'polymers', 'medium', 'low', 'molecular', 'weight', 'degree', 'polymerization', '(x)', 'included', 'tm', '¼', 'ðdh0', 'þ', 'xdh1þðds0', 'þ', 'xds1þ', 'mandelkern', 'points', '38', 'experimental', 'data', 'shows', 'correlation', 'melting', 'temperature', 'polymeric', 'crystal', 'enthalpy', 'fusion', 'found', 'many', 'small', 'molecules', 'the', 'dh', 'values', 'polymers', 'generally', 'fall', 'two', 'classes', 'they', 'ether', 'within', 'thousand', 'calories', 'per', '10000', 'calmole', 'polymers', 'fall', 'category', 'elastomers', 'low', 'melting', 'temperatures', 'high', 'entropies', 'fusion', 'this', 'reﬂects', 'compacted', 'highly', 'ﬂexible', 'nature', 'chains', 'at', 'extreme', 'rigid', 'engineering', 'plastics', 'these', 'materials', 'possess', 'high', 'melting', 'points', 'correspondingly', 'lower', 'entropies', 'fusion', '42.', '2332', 'kinetics', 'crystallization', 'the', 'rate', 'crystallization', 'polymeric', 'materials', 'paramount', 'importance', 'for', 'polymers', 'like', 'atactic', 'polystyrene', 'rubbers', 'rapid', 'cooling', 'lead', 'glassy', 'state', 'without', 'formation', 'crystallites', 'the', 'amount', 'crystallinity', 'actually', 'depends', 'much', 'upon', 'thermal', 'history', 'material', 'the', 'amount', 'crystallinity', 'turn', 'inﬂuences', 'mechanical', 'properties', 'material', 'microscopic', 'observation', 'growth', 'spherulites', 'function', 'time', 'yield', 'information', 'crystallization', 'rate', 'the', 'rate', 'function', 'temperature', 'as', 'temperature', 'lowered', 'rate', 'increases', 'this', 'growth', 'usually', 'observed', 'linear', 'time', 'presence', 'impurities', 'slow', 'growth', 'rate', 'when', 'growth', 'rate', 'plotted', 'crystallization', 'temperature', 'maximum', 'observed', 'this', 'due', 'fact', 'temperature', 'lowered', 'mobilities', 'molecules', 'decrease', 'process', 'eventually', 'becomes', 'diffusion-controlled', 'according', 'avrami', 'equation', 'fraction', 'crystallizes', 'time', 'deﬁned', '1', '(cid3)', 'l(t)', 'written', '42:', '1', '(cid3)', 'lðtþ', '¼', '1', '(cid3)', 'exp', '(cid3)', 'vðt', 'tþnðtþdt', 'rc', 'rl', 'z', '0', '(cid2)', '(cid3)', '42', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'table', '23', 'melting', 'points', 'tm', 'crystalline', 'polymersa', 'repeat', 'unit', 'o', 'o', 'polymer', 'poly(tetramethylene', 'suberate)', 'melting', 'point', '((cid1)c)', '45', 'o', 'o', '4', '6', 'o', 'o', 'cn', 'cl', 'high-density', 'polyethylene', '132–138', 'poly(ethylene', 'oxide)', 'poly(propylene', 'oxide)', 'polyacrylonitrile', 'poly(vinyl', 'chloride)', '66', '70', '317', '212', '210', '168', 'poly(vinylidene', 'chloride)', 'cl', 'cl', 'polypropylene', 'f', 'f', 'polychlorotriﬂuoroethylene', '210', 'cl', 'polystyrene', '230–248', 'o', 'o', 'poly(hexamethylene', 'adipamide)', '250', 'n', 'h', 'n', '6', 'h', '4', 'f', 'f', 'polytetraﬂuoroethylene', '327', 'f', 'f', 'from', 'ref', '15', 'sources', 'literature', '24', 'the', 'mesomorphic', 'state', 'liquid', 'crystal', 'polymers', '43', 'n(t)', 'nucleation', 'frequency', 'per', 'untransformed', 'volume', 'v(tt)', 'corresponding', 'volume', 'growing', 'center', 'rc', 'rl', 'represent', 'densities', 'crystalline', 'liquid', 'phases', 'based', 'rate', 'constant', 'crystallization', 'kinetics', 'described', '42:', 'ln½ðv1', '(cid3)', 'vtþðv1', '(cid3)', 'v0þ(cid4)', '¼', '(cid3)ð1wcþktn', 'v1', 'vt', 'v0', 'speciﬁc', 'volumes', 'times', 'shown', 'subscripts', 'wc', 'weight', 'fraction', 'polymer', 'crystallized', 'k', 'rate', 'constant', 'crystallization', 'it', 'found', 'however', 'crystallization', 'continues', 'polymeric', 'materials', 'much', 'longer', 'periods', 'time', 'avrami', 'equation', 'predicts', 'for', 'homopolymers', 'rate', 'crystal', 'growth', 'increases', 'linearly', 'time', 'g', '¼', 'drdt', 'mandelkern', 'deﬁnes', 'steady-state', 'nucleation', 'rate', 'n', 'follows', '42:', 'n', '¼', 'n0', 'expð(cid3)edtrt', '(cid3)', 'dg(cid6)rtþ', 'ed', 'energy', 'activation', 'transporting', 'chain', 'segments', 'across', 'crystal–liquid', 'interface', 'if', 'crystallization', 'takes', 'place', 'extended', 'temperature', 'range', 'homopolymers', 'display', 'maxima', 'rates', 'spherulitic', 'growth', 'overall', 'crystallization', 'the', 'equation', 'spherulitic', 'growth', 'written', 'follows', '42:', 'g', '¼', 'g0', 'expð(cid3)u(cid6)ðt', '(cid3)', 't1þ', 'expðkt0', 'mtcdgutþ', 't1', 'temperature', 'molecular', 'segmental', 'motion', 'stops', '24', 'the', 'mesomorphic', 'state', 'liquid', 'crystal', 'polymers', 'the', 'state', 'mesomorphism', 'spontaneously', 'ordered', 'liquid–ﬂuid', 'crystalline', 'state', 'liquid', 'crystals', 'discovered', 'early', '1888', 'they', 'materials', 'exhibit', 'order', 'one', 'two', 'dimensions', 'three', 'by', 'comparison', 'amorphous', 'materials', 'lack', 'order', 'crystalline', 'ones', 'exhibit', 'order', 'three', 'dimensions', 'all', 'liquid', 'crystalline', 'polymers', 'exhibit', 'degree', 'ﬂuidity', 'they', 'investigated', 'extensively', '1900s', 'became', 'commercially', 'important', '1960s', 'these', 'macromolecules', 'align', 'crystalline', 'arrays', 'solution', '(lyotropic)', 'molten', 'state', '(thermotropic)', 'such', 'liquids', 'exhibit', 'anisotropic', 'behavior', '51', '52.', 'the', 'regions', 'orderliness', 'liquids', 'called', 'mesophases', 'molecular', 'rigidity', 'found', 'rigid', 'rod-shaped', 'polymers', 'instance', 'chief', 'cause', 'liquid', 'crystalline', 'behavior', 'it', 'excludes', 'one', 'molecule', 'occupying', 'speciﬁc', 'volume', 'result', 'intermolecular', 'attractive', 'forces', 'some', 'aromatic', 'polyesters', 'polyamides', 'good', 'examples', 'like', 'polyphenylene', 'terephthalate', 'o', 'o', 'o', 'n', 'o', 'because', 'molecules', 'posses', 'anisotropy', 'aligned', 'still', 'ﬂuid', 'form', 'this', 'differs', 'ordinary', 'liquids', 'isotropic', 'molecules', 'lack', 'kind', 'arrangement', '44', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'fig', '214', 'illustration', 'arrangement', 'liquid', 'crystal', 'nematic', 'smectic', 'orders', 'nematic', 'smectic', 'anisotropy', 'affected', 'conformational', 'changes', 'generally', 'molecules', 'rigid', 'rod', 'like', 'elongated', 'disc', 'like', 'shape', 'type', 'form', 'liquid', 'crystal', 'arrangements', 'some', 'biological', 'polymers', 'exhibit', 'liquid', 'crystalline', 'behavior', 'due', 'rigid', 'helical', 'conformations', 'among', 'synthetic', 'polymers', 'hand', 'rigid', 'rod', 'structures', 'mentioned', 'ones', 'exhibit', 'liquid', 'crystalline', 'behavior', 'polymers', 'form', 'liquid', 'crystals', 'may', 'exhibit', 'multiple', 'mesophases', 'different', 'temperatures', 'based', 'arrangement', 'liquid', 'crystals', 'mesophases', 'classiﬁed', 'nematic', 'smectic', 'cholesteric', '51', '52.', 'both', 'smectic', 'nematic', 'parallel', 'arrangements', 'along', 'molecular', 'axes', 'the', 'smectic', 'liquid', 'crystals', 'ordered', 'however', 'nematic', 'ones', 'this', 'result', 'differences', 'orientations', 'chain', 'ends', 'in', 'smectic', 'liquid', 'crystals', 'chain', 'ends', 'lined', 'next', 'in', 'nematic', 'ones', 'however', 'lack', 'particular', 'orientation', 'also', 'smectic', 'liquid', 'crystals', 'layered', 'nematic', 'ones', 'microscopic', 'observations', '51', 'help', 'distinguish', 'two', 'forms', 'smectic', 'elastomers', 'due', 'layered', 'structure', 'exhibit', 'distinct', 'anisotropic', 'mechanical', 'properties', 'mechanical', 'deformation', 'processes', 'parallel', 'perpendicular', 'normal', 'orientation', 'smectic', 'layer', 'such', 'elastomers', 'important', 'due', 'optical', 'ferroelectric', 'properties', 'networks', 'macroscopic', 'uniformly', 'ordered', 'direction', 'conical', 'distribution', 'smectic', 'layer', 'normal', 'respect', 'normal', 'smetic', 'direction', 'mechanically', 'deformed', 'uniaxial', 'shear', 'deformations', 'under', 'uniaxial', 'deformations', 'two', 'processes', 'observed', '53:', 'parallel', 'direction', 'mechanical', 'ﬁeld', 'directly', 'couples', 'smectic', 'tilt', 'angle', 'perpendicular', 'director', 'reorientation', 'process', 'takes', 'place', 'this', 'process', 'reversible', 'shear', 'deformation', 'perpendicular', 'irreversible', 'applying', 'shear', 'force', 'parallel', 'smetic', 'direction', 'this', 'illustrated', 'fig', '214', 'if', 'mesogens', 'chiral', 'twisted', 'nematic', 'suprarmolecular', 'cholesteric', '(twisted)', 'phase', 'form', '51', '52.', 'the', 'achiral', 'nonlinear', 'mesogens', 'also', 'form', 'chiral', 'supramolecular', 'arrangements', 'tilted', 'smectic', 'phases', 'recently', 'tokita', 'coworkers', '54', 'reported', 'direct', 'transition', 'isotropic', 'smectic', 'arrangement', 'liquid', 'crystalline', 'polymer', 'determined', 'experimentally', 'existence', 'metasta-', 'ble', 'nematic', 'orientational', 'ordering', 'preceded', 'formation', 'translational', 'smectic', 'ordering', 'a', 'polymeric', 'material', 'used', 'exhibits', 'slow', 'liquid', 'crystalline', 'transition', 'dynamics', '55.', 'this', 'enabled', 'use', 'conventional', 'methods', 'study', 'transitions', 'polarized', 'light', 'scattering', 'synchrotron', 'wide-angle', 'x-ray', 'diffraction', 'analyses', 'it', 'observed', 'high', 'quench', 'rates', 'super', 'cooling', 'metastable', 'nematic', '(orientational)', 'ordering', 'occurs', 'preceding', 'full', 'smectic', '(orienta-', 'tional', 'translational)', 'order', 'also', 'occurrence', 'nematic', 'preordering', '(high', 'super', 'cooling)', 'resulted', 'morphological', 'changes', 'growing', 'liquid', 'crystalline', 'domains', 'compared', 'solely', 'smectic', 'growth', 'speciﬁcally', 'samples', 'cooled', 'rates', 'high', 'enough', 'exhibit', 'nematic', 'preordering', 'formed', 'well-', 'oriented', '“neat”', 'tactoidal', 'smectic', 'domains', 'samples', 'cooled', 'lower', 'rates', 'smectic', 'ordering', 'observed', 'formed', 'radially', 'oriented', 'textured', 'spherulitic', 'domains', '55.', 'in', 'commenting', 'observation', 'abukhdeir', 'rey', '56', 'point', 'simulation', 'model', 'isotropic', 'smectic', 'liquid', 'crystalline', 'transition', 'experimental', 'observations', 'preordering', 'smectic', 'liquid', 'crystalline', 'transitions', 'studied', 'phase', 'transition', 'kinetics', 'results', 'presented', 'show', 'nematic', 'preordering', 'results', 'thermodynamic', 'potential', 'dynamic', 'differences', 'phase-', 'ordering', 'time', 'scales', '24', 'the', 'mesomorphic', 'state', 'liquid', 'crystal', 'polymers', '45', 'fig', '215', 'illustration', 'arrangements', 'liquid', 'crystal', 'structure', 'side', 'chain', 'liquid', 'crystals', 'main', 'chain', 'liquid', 'crystalls', 'the', 'chemical', 'structure', 'polymers', 'determines', 'whether', 'molecules', 'form', 'rigid', 'rods', 'if', 'backbone', 'polymer', 'composed', 'rigid', 'structures', 'tends', 'form', 'main', 'chain', 'liquid', 'crystals', 'if', 'however', 'side', 'chains', 'rigid', 'polymer', 'tend', 'form', 'side', 'chain', 'liquid', 'crystals', 'from', 'practical', 'considerations', 'two', 'properties', 'prime', 'interest', 'the', 'structures', 'illustrated', 'fig', '215', 'liquid', 'crystalline', 'behavior', 'affects', 'melt', 'viscosity', 'polymer', 'ability', 'polymer', 'retain', 'ordered', 'arrangement', 'solid', 'state', 'thus', 'liquid', 'crystalline', 'behavior', 'melt', 'results', 'lower', 'viscosity', 'rigid', 'polymeric', 'mesophases', 'align', 'direction', 'ﬂow', 'as', 'result', 'polymer', 'easier', 'process', 'also', 'retention', 'arrangement', 'upon', 'cooling', 'yields', 'material', 'greatly', 'improved', 'mechanical', 'properties', 'several', 'thermotropic', 'liquid', 'crystal-', 'line', 'copolyesters', 'polyamides', 'available', 'commercially', 'samulski', '57', 'gives', 'examples', 'molecules', 'typically', 'form', 'liquid', 'crystals', 'these', '1', 'a', 'discotic', 'liquid', 'crystal', 'o', 'n-1', 'o', 'n-1', 'the', 'shown', 'structure', 'mesogenic', 'core', '(hard', 'central', 'segment)', 'correlated', 'dynamic', 'packing', 'anisometric', 'shapes', 'the', 'ﬂexible', 'tales', 'often', 'hydrocarbon', 'chains', 'extend', 'mesogenic', 'core', 'facilitate', 'transformation', 'solid', 'state', 'liquid', 'crystalline', 'phase', '2', 'a', 'calamitic', 'liquid', 'crystal', 'o', 'n-1', 'o', 'n-1', 'o', 'n-1', 'o', 'n-1', 'o', 'n', 'o', 'n', 'n', 'o', 'n', 'in', 'case', 'prolate', 'mesogen', 'axis', 'ﬂexible', 'hydrocarbon', 'chains', 'extend', '46', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', '3', 'a', 'nonlinear', 'liquid', 'crystal', 'o', 'o', 'o', 'o', 'n', 'n', 'n', 'o', 'n', 'many', 'liquid', 'crystal', 'polymers', 'tend', 'exhibit', 'multiple', 'mesophases', 'different', 'pressures', 'temperatures', '57.', 'when', 'heated', 'polymers', 'go', 'multiple', 'ﬁrst-order', 'transitions', 'such', 'transitions', 'ordered', 'less', 'ordered', 'arrangements', 'this', 'referred', 'clearing', 'temperature', 'last', 'one', 'resulting', 'isotropic', 'melts', 'a', 'lyotropic', 'liquid', 'crystalline', 'aromatic', 'polyamide', 'sold', 'trade', 'name', 'kevlar', 'example', 'polymer', 'available', 'commercially', 'h', 'n', 'o', 'n', 'h', 'kevlar', 'o', 'n', 'the', 'polyamide', 'forms', 'liquid', 'crystals', 'sulfuric', 'acid', 'solution', 'extruded', 'ﬁber', 'after', 'solvent', 'removed', 'remaining', 'ﬁber', 'possesses', 'greater', 'uniform', 'alignment', 'would', 'obtained', 'mere', 'drawing', 'this', 'results', 'superior', 'mechanical', 'properties', 'there', 'polymers', 'however', 'exhibit', 'liquid', 'crystalline', 'behavior', 'high', 'melting', 'insoluble', 'common', 'organic', 'solvents', 'this', 'drawback', 'materials', 'hard', 'process', 'a', 'preparation', 'new', 'liquid', 'crystal', 'polymers', 'bilaterally', 'linked', 'mesogens', 'main-chain', 'reported', '57.', 'such', 'materials', 'exhibit', 'biaxial', 'ﬂuctuation', 'nematic', 'phase', 'this', 'interesting', 'commonly', 'encountered', 'polymeric', 'liquid', 'crystals', 'mesogens', 'linked', 'ends', 'polymer', 'backbone', 'ﬂexible', 'spacers', 'on', 'hand', 'liquid', 'crystal', 'polymers', 'mesogens', 'linked', 'bilaterally', 'ﬂexible', 'spacers', 'common', 'examples', 'reported', '58.', 'one', 'material', 'illustrated', 'follows', 'c6h13o', 'o', 'o', 'oc6h13', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'n', '25', 'orientation', 'polymers', '47', 'it', 'also', 'shown', 'possible', 'synthesize', 'polymethacrylate', 'liquid', 'crystal', 'polymers', 'mesomorphic', 'properties', 'contain', 'ferrocenes', 'two', 'ﬂexible', 'chains', 'll0-positions', '59.', 'based', 'dilatometric', 'measurements', 'head-to-tail', 'molecular', 'arrangement', 'monomeric', 'units', 'occurs', 'within', 'smectic', 'a', 'phase', 'because', 'special', 'electrochemical', 'properties', 'ferrocene', 'materials', 'interest', 'developing', 'electroactive', 'mesomorphic', 'polymers', 'the', 'structure', 'polymer', 'shown', 'follows', '59:', 'x', 'o', 'o', '6', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'fe', 'finkelmann', 'reported', 'synthesis', 'novel', 'cross-linked', 'smectic-c', 'main-chain', 'liquid-crystalline', 'elastomer', 'formed', 'polycondensation', 'vinyloxy-terminated', 'mesogens', 'tetramethyldi-', 'siloxane', 'pentamethyl-pentaoxapentasilicane', 'the', 'introduction', 'functional', 'vinyloxy', 'group', 'allows', 'synthesis', 'well-deﬁned', 'networks', 'good', 'mechanical', 'properties', 'due', 'elimination', 'side', 'reactions', 'case', 'vinyl', 'groups', '60.', 'large', 'amplitude', 'oscillatory', 'shear', 'frequently', 'capable', 'generating', 'macroscopic', 'alignment', 'initially', 'random', 'orientation', 'distribution', 'ordered', 'polymer', 'ﬂuids', 'berghardt', 'coworkers', '61', 'reported', 'combined', 'rheological', 'situ', 'synchrotron', 'x-ray', 'scattering', 'investigate', 'induced', 'alignment', 'smectic', 'side-groups', 'pf', 'liquid', 'crystalline', 'polymers', 'in', 'cases', 'found', 'shear', 'promotes', 'anisotropic', 'orientation', 'states', 'lamellar', 'normal', 'tends', 'align', 'along', 'vorticity', 'direction', 'shear', 'ﬂow', '(“perpendicular”', 'alignment)', 'rheologi-', 'cal', 'measurements', 'dynamic', 'moduli', 'revealed', 'large', 'amplitude', 'shearing', 'smectic', 'phase', 'causes', 'notable', 'decrease', 'modulus', 'they', 'also', 'observed', 'increasing', 'strain', 'promotes', 'higher', 'degrees', 'orientation', 'increasing', 'molecular', 'weight', 'impedes', 'development', 'smectic', 'alignment', 'ahn', 'et', 'al', 'reported', '62', 'preparation', 'smetic', 'liquid', 'crystalline', 'elastomer', 'shape', 'memory', 'properties', 'shape', 'memory', 'polymeric', 'materials', 'recover', 'equilibrium', 'permanent', 'shapes', 'nonequilibrium', 'temporary', 'shapes', 'result', 'external', 'stimuli', 'like', 'heat', 'light', 'such', 'materials', 'application', 'medical', 'practice', 'main-chain', 'polynorbornenes', 'linked', 'three', 'different', 'side-chains', 'cholesterol', 'poly(ethylene', 'glycol)', 'butylacrylate', '25', 'orientation', 'polymers', 'there', 'preferred', 'direction', 'arrangement', 'manner', 'macromolecules', 'align', 'polymeric', 'mass', 'crystallization', 'if', 'however', 'crystallization', 'external', 'stress', 'applied', 'crystalline', 'material', 'undergoes', 'rearrangement', 'from', 'x-ray', 'diffraction', 'patter', 'surmised', 'chains', 'realign', 'direction', 'applied', 'stress', 'polymeric', 'ﬁlms', 'ﬁbers', 'usually', 'show', 'considerable', 'increase', 'strength', 'direction', 'stress', 'oriented', 'ﬁbers', 'considerably', 'stronger', 'along', 'length', 'perpendicularly', 'generally', 'degree', '48', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'crystallinity', 'crystalline', 'polymers', 'well', 'developed', 'prior', 'drawing', 'process', 'change', 'amount', 'crystallinity', 'appreciably', 'the', 'orientation', 'direction', 'applied', 'stress', 'occurs', 'also', 'amorphous', 'materials', 'the', 'amorphous', 'polymers', 'like', 'crystalline', 'ones', 'also', 'exhibit', 'increased', 'strength', 'direction', 'orientation', 'if', 'small', 'amount', 'crystallinity', 'polymers', 'crystallinity', 'often', 'increases', 'result', 'cold', 'drawing', 'orientation', 'ﬁbers', 'ﬁlms', 'generally', 'carried', 'glass', 'transition', 'temperature', 'at', 'time', 'usually', 'desired', 'limit', 'tm', 'thus', 'preferred', 'crystalline', 'melting', 'temperature', 'exceed', '300(cid1)c', '26', 'solutions', 'polymers', 'polymers', 'usually', 'dissolve', 'two', 'stages', 'first', 'materials', 'tend', 'swell', 'form', 'gels', 'the', 'gels', 'tend', 'disintegrate', 'true', 'solutions', 'agitation', 'speeds', 'process', 'second', 'stage', 'chains', 'held', 'together', 'crosslinks', 'swell', 'crystallinity', 'strong', 'hydrogen', 'bonding', 'might', 'also', 'keep', 'polymeric', 'materials', 'dissolution', 'materials', 'swell', 'some', 'general', 'rules', 'polymer', 'solubility', '(a)', 'solubility', 'favored', 'chemical', 'structural', 'similarity', 'solvent', 'polymer', '(b)', 'solubility', 'polymer', 'tends', 'decrease', 'increase', 'molecular', 'weight', 'when', 'polymer', 'dissolves', 'free', 'energy', 'solution', 'df', 'negative', 'entropy', 'change', 'ds', 'positive', 'increased', 'conformational', 'mobility', 'polymer', 'the', 'magnitude', 'enthalpy', 'solution', 'determines', 'whether', 'df', 'positive', 'negative', 'the', 'heat', 'mixing', 'dhmix', 'binary', 'systems', 'suggested', 'related', 'concentration', 'energy', 'parameters', 'expression', '7:', 'dhmix', '¼', 'vmix½ðde1v1þ05', '(cid3)', 'ðde2v2þ05(cid4)2f1f2', 'vmix', 'total', 'volume', 'mixture', 'molar', 'volumes', 'two', 'components', 'v1', 'v2', 'respectively', 'f1', 'f2', 'volume', 'fractions', 'the', 'energies', 'vaporization', 'de1', 'de2', 'cohesive', 'densities', '(energy', 'required', 'remove', 'molecule', 'nearest', 'neighbor)', 'de1v1', 'de2v2', '261', 'radius', 'gyration', 'in', 'solution', 'polymeric', 'chains', 'form', 'different', 'conformations', 'depending', 'upon', 'solvent', 'when', 'solvent', 'chains', 'fully', 'solvated', 'relatively', 'extended', 'molecules', 'randomly', 'coiled', 'the', 'polymer–solvent', 'interaction', 'forces', 'determine', 'amount', 'space', 'molecular', 'coil', 'polymer', 'occupies', 'solution', 'while', 'quite', 'extended', 'good', 'solvent', 'solvent', '“poor”', 'one', 'chains', 'curled', 'a', 'measure', 'size', 'polymer', 'molecule', 'solution', 'amount', 'space', 'polymer', 'molecule', 'occupies', 'solution', 'determined', 'radius', 'gyration,', 'root', 'mean', 'square', 'radius', 'gyration', 's', 'qualitatively', 'average', 'distance', 'mass', 'molecule', 'center', 'mass', '(from', 'center', 'gravity)', 'the', 'following', 'equation', '26', 'solutions', 'polymers', 'fig', '216', 'illustration', 'molecular', 'coil', '49', 'deﬁnes', 'relationship', '7.', 'to', 'put', 'words', 'square', 'distances', 'various', 'masses', 'center', 'mass', 's2', '¼', '2', 'misi', 'mi', 'x', 'x', 's2', '¼', 'ms2', 'ð1nmþ', 'n', 'x', 'av', 'mass', 'associated', 'n', 'chain', 'bonds', 's', 'vector', 'distance', 'center', 'mass', 'terminal', 'chain', 'bond', 'the', 'size', 'randomly', 'coiled', 'polymer', 'molecules', 'commonly', 'designated', 'root-mean', 'square', 'distance', 'ends', 'r2', 'a', 'molecular', 'coil', 'illustrated', 'fig', '216', 'the', 'distance', 'chain', 'ends', 'often', 'expressed', 'terms', 'unperturbed', 'dimensions', '(s0', 'r0)', 'expansion', 'factor', '(a)', 'result', 'interaction', 'solvent', 'polymer', 's2', '¼', 's0', '2a2', 'r2', '¼', 'r0', '2a2', 'the', 'unperturbed', 'dimensions', 'refers', 'molecular', 'size', 'exclusive', 'solvent', 'effects', 'it', 'arises', 'intramolecular', 'polar', 'steric', 'interactions', 'free', 'rotation', 'the', 'expansion', 'factor', 'result', 'solvent', 'polymer', 'molecule', 'interaction', 'for', 'linear', 'polymers', 'square', 'radius', 'gyration', 'related', 'mean-square', 'end-to-end', 'distance', 'following', 'relationship', '2', 's', '2', '¼', 'r', '6', 'this', 'follows', 'expansion', 'factor', 'greater', 'unity', 'good', 'solvent', 'actual', '“perturbed', 'dimensions”', 'exceed', 'unperturbed', 'ones', 'the', 'greater', 'value', 'unperturbed', 'dimensions', 'better', 'solvent', 'the', 'relationship', 'average', 'derived', 'experimentally', 'numerous', 'computations', 'because', 'branched', 'chains', 'multiple', 'ends', 'simpler', 'describe', 'terms', 'radius', 'gyration', 'the', 'volume', 'branched', 'polymer', 'molecule', 'occupies', 'solution', 'smaller', 'linear', 'one', 'equals', 'molecular', 'weight', 'number', 'segments', 'the', 'volume', 'molecules', 'occupy', 'solution', 'important', 'determinations', 'molecular', 'weights', 'it', 'referred', 'hydrodynamic', 'volume', 'this', 'volume', 'depends', 'upon', 'variety', 'factors', '50', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'these', 'interactions', 'polymer', 'molecule', 'solvent', 'chain', 'branching', 'conformational', 'factors', 'arising', 'polarity', 'restricted', 'rotation', 'due', 'resonance', 'bulk', 'substituents', 'the', 'course', 'assumes', 'polymer', 'molecules', 'fully', 'separated', '262', 'the', 'thermodynamics', 'polymer', 'solutions', 'solutions', 'polymers', 'deviate', 'great', 'extent', 'raoult’s', 'law', 'except', 'extreme', 'dilution', 'in', 'extremely', 'dilute', 'solutions', 'ideal', 'behavior', 'approached', 'asymptotic', 'limit', 'these', 'deviations', 'arise', 'largely', 'small', 'entropies', 'mixing', 'that', 'mostly', 'due', 'large', 'difference', 'size', 'solute', 'solvent', 'the', 'change', 'entropy', 'mixing', 'according', 'flory–huggins', 'theory', 'polymer', 'solutions', '63', 'ds', '¼', '(cid3)kðn1', 'ln', 'v1', 'þ', 'n2', 'ln', 'v2þ', 'subscript', '1', 'denotes', 'solvent', 'subscript', '2', 'solute', 'v1', 'v2', 'volume', 'fractions', 'they', 'deﬁned', 'follows', 'v1', '¼', 'n1ðn1', 'þ', 'xn2þ', 'v2', '¼', 'xn2ðn1', 'þ', 'xn2þ', 'dh', '¼', 'x1ktn1v2', 'x', 'heat', 'mixing', 'the', 'change', 'heat', 'content', 'mixing', 'polymer', 'solutions', 'similar', 'solutions', 'x1', 'characterizes', 'internal', 'energy', 'per', 'solvent', 'molecule', 'the', 'change', 'free', 'energy', 'mixing', 'according', 'flory–huggins', '63,', 'df', '¼', 'ktðn1', 'ln', 'v1', 'þ', 'n2', 'ln', 'v2', 'þ', 'x1n1v2þ', 'the', 'flory–huggins', 'treatment', 'overlooks', 'fact', 'dilute', 'solutions', 'polymers', 'consist', 'domains', 'clusters', 'polymeric', 'chains', 'separated', 'regions', 'pure', 'solvent', 'free', 'solute', 'flory–krigbaum', 'treatment', 'assumes', 'model', 'dilute', 'polymeric', 'solutions', 'polymeric', 'clusters', 'approximately', 'spherical', 'density', 'reaches', 'maximum', 'centers', 'decreases', 'approximately', 'gaussian', 'function', 'away', 'center', 'the', 'volume', 'occupied', 'segments', 'molecule', 'excludes', 'volumes', 'molecules', 'long', 'range', 'intramole-', 'cular', 'interactions', 'take', 'place', 'within', 'excluded', 'volumes', 'the', 'thermodynamic', 'functions', 'interactions', 'derived', 'free', 'energy', 'change', 'enthalpy', 'change', 'entropy', 'change', 'c', 'entropy', 'parameter', 'expressed', 'df1', '¼', 'ktðk1', '(cid3)', 'c1þv2', '2', 'dh1', '¼', 'ktk1v2', '2', 'ds1', '¼', 'kc1v2', '2', 'k1', '(cid3)', 'c1', '¼', 'x1', '(cid3)', '05', '27', 'molecular', 'weights', 'molecular', 'weight', 'determinations', '51', 'in', 'addition', 'flory', 'temperature', 'y', 'treated', 'parameter', 'y', '¼', 'k1tc1', 'at', 'y', 'temperature', 'partial', 'molar', 'free', 'energy', 'due', 'solute–solvent', 'interaction', 'assumed', 'zero', 'deviations', 'ideality', '(ideal', 'solution)', 'become', 'zero', 'what', 'means', 'temperature', 'solution', 'polymer', 'approaches', 'y', 'solvent', 'becomes', 'increasingly', 'poorer', 'excluded', 'volume', 'effect', 'becomes', 'smaller', 'approaches', 'zero', 'molecules', 'interpenetrating', 'one', 'another', 'zero', 'net', 'interaction', 'the', 'solvent', 'referred', 'y', 'solvent', 'below', 'y', 'temperature', 'polymer', 'molecules', 'attract', 'excluded', 'volume', 'negative', 'polymer', 'precipitates', 'this', 'expressed', '63:', 'c1', '(cid3)', 'k1', '¼', 'c1ð1', '(cid3)', 'ytþ', '¼', '05', '(cid3)', 'x1', 'when', 'chains', 'extended', 'conformations', 'may', 'considered', 'determined', 'equilibrium', 'forces', 'expansion', 'due', 'excluded', 'volume', 'forces', 'contraction', 'due', 'chain', 'segments', 'expanding', 'less', 'probable', 'conformations', 'based', 'random', 'ﬂight', 'statistics', 'chains', 'extended', 'linearly', 'factor', 'dimensions', 'the', 'actual', 'root-mean-square', 'end-', 'to-end', 'distance', 'equal', 'a(r0', '2)05', 'the', 'change', 'elastic', 'part', 'free', 'energy', 'parameter', 'expressed', 'terms', 'thermodynamic', 'quantities', 'dfel', '¼', 'kt½15ða2', '(cid3)', '1þ', '(cid3)', 'ln', 'a2(cid4)', 'a5', '(cid3)', 'a3', '¼', '2cmc1ð1', '(cid3)', 'ytþm05', 'in', 'equation', 'cm', 'represents', 'combination', 'molecular', 'numerical', 'constants', 'based', 'equation', 'y', 'temperature', '¼', '1', 'it', 'stated', 'flory–krigbaum', 'treatment', 'must', 'treated', 'reservations', 'predicts', 'increases', 'without', 'limit', 'increasing', 'molecular', 'weight', '63.', '27', 'molecular', 'weights', 'molecular', 'weight', 'determinations', 'the', 'physical', 'properties', 'polymers', 'also', 'related', 'molecular', 'weights', 'melt', 'viscosity', 'hot', 'strength', 'solvent', 'resistance', 'overall', 'toughness', 'increase', 'molecular', 'size', 'table', '24', 'illustrates', 'effect', 'molecular', 'weights', '(size)', 'upon', 'physical', 'properties', 'polyethylene', '64.', '271', 'molecular', 'weight', 'averages', 'random', 'events', 'govern', 'process', 'synthetic', 'polymer', 'formation', 'whether', 'chain', 'propagating', 'process', 'step-growth', 'reaction', 'the', 'result', 'chains', 'vary', 'lengths', '(there', 'special', 'methods', 'available', 'however', 'chain-growth', 'polymerizations', 'lead', 'formation', 'polymer', 'molecules', 'almost', 'equal', 'length', 'this', 'discussed', 'subsequent', 'chapters)', 'as', 'result', 'polymeric', 'materials', 'cannot', 'characterized', 'single', 'molecular', 'weight', 'instead', 'must', 'represented', 'statistical', 'averages', '64.', 'these', 'averages', 'expressed', 'several', 'ways', 'one', 'way', 'present', 'average', 'number', 'average', 'molecular', 'weight', 'it', 'sum', 'molecular', 'weights', 'individual', 'molecules', 'present', 'divided', 'total', 'number', 'each', 'molecule', 'contributes', 'equally', 'average', 'obtained', 'averaging', 'measurements', 'colligative', 'properties', 'if', 'total', 'number', 'moles', 'ni', 'sum', 'molecules', 'present', 'expressed', 'sni', 'the', 'total', 'weight', 'sample', 'similarly', 'sum', 'weights', 'molecular', 'species', 'present', '52', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'table', '24', 'properties', 'low', 'density', 'polyethylene', '48', '49', 'dp', 'molecular', 'weight', '1', '6', '35', '140', '250', '430', '750', '1350', '28', '170', '1000', '4000', '7000', '12000', '21000', '38000', 'softening', 'temperature', '((cid1)c)', '(cid3)169', '(mp)', '(cid3)12', '(mp)', '37', '93', '98', '104', '110', '112', 'physical', 'state', '25(cid1)c', 'gas', 'liquid', 'grease', 'wax', 'hard', 'wax', 'plastic', 'plastic', 'plastic', 'by', 'dividing', 'total', 'weight', 'molecules', 'total', 'number', 'number', 'average', 'molecular', 'weight', '¼', 'oi', '¼', 'mini', 'x', 'x', 'mn', '¼', 'ni', '¼', 'mini', 'ni', 'x', 'x', 'x', 'mw', '¼', 'mi', '2ni', 'ni', 'x', 'x', 'another', 'way', 'express', 'molecular', 'weight', 'average', 'weight', 'average', 'each', 'molecule', 'average', 'contributes', 'according', 'ratio', 'particular', 'weight', 'total', 'the', 'illustrated', 'quite', 'readily', 'imagining', 'sample', 'polymer', 'consists', 'ﬁve', 'molecules', 'molecular', 'weights', '2468', '10', 'respectively', 'to', 'calculate', 'number', 'average', 'molecular', 'weight', 'weights', 'individual', 'molecules', 'added', 'the', 'sum', 'divided', 'total', 'number', 'molecules', 'sample', '(in', 'case', '5)', 'mn', '¼', '25', 'þ', '45', 'þ', '65', 'þ', '85', 'þ', '105', '¼', '6', 'to', 'calculate', 'weight', 'average', 'molecular', 'weight', 'sample', 'squares', 'individual', 'weight', 'divided', 'total', 'sum', 'molecular', 'weights', 'case', '30', 'mw', '¼', '2230', 'þ', '4230', 'þ', '6230', 'þ', '8230', 'þ', '10230', '¼', '733', 'mw', 'sensitive', 'higher', 'molecular', 'weight', 'species', 'mn', 'sensitive', 'lower', 'ones', 'this', 'seen', 'imagining', 'equal', 'weights', 'two', 'different', 'sizes', 'molecules', 'combined', 'm1', '¼', '10000', 'm2', '¼', '100000', 'the', 'combination', 'would', 'consist', 'ten', 'molecules', 'm1', 'one', 'molecule', 'm2', 'the', 'weight', 'average', 'molecular', 'weight', 'mixture', '(1082', '(cid5)', '105', '10102', '(cid5)', '105)', '¼', '55000', 'number', 'average', 'molecular', 'weight', '18182', 'if', 'however', 'mixture', 'consists', 'equal', 'number', 'molecules', 'weight', 'average', 'molecular', 'weight', '92000', 'number', 'average', 'molecular', 'weight', '55000', 'the', 'ratio', 'weight', 'average', 'molecular', 'weight', 'number', 'average', 'molecular', 'weight', 'important', 'affects', 'properties', 'polymers', 'this', 'ratio', 'called', 'molecular', 'weight', 'distribution', 'molecular', 'weight', 'distribution', '¼', 'mwmn', 'when', 'macromolecular', 'species', 'size', 'number', 'average', 'molecular', 'weight', 'equal', 'weight', 'average', 'molecular', 'weight', 'on', 'hand', 'greater', 'distribution', '27', 'molecular', 'weights', 'molecular', 'weight', 'determinations', '53', 'molecular', 'sizes', 'greater', 'disparity', 'averages', 'the', 'ratio', 'disparity', 'mwmn', 'measure', 'polymeric', 'dispersity', 'a', 'monodisperse', 'polymer', 'ratio', 'mwmn', '¼', '1', 'in', 'synthetic', 'polymers', 'many', 'naturally', 'occurring', 'ones', 'weight', 'average', 'molecular', 'weight', 'greater', 'number', 'average', 'molecular', 'weight', 'such', 'polymers', 'polydisperse', 'two', 'samples', 'polymer', 'equal', 'weight', 'average', 'molecular', 'weight', 'may', 'exhibit', 'different', 'physical', 'properties', 'differ', 'molecular', 'weight', 'distributions', 'molecular', 'weight', 'distribution', 'affect', 'elongation', 'relaxation', 'modulus', 'tensile', 'strength', 'tenacity', 'materials', '65.', 'an', 'additional', 'average', 'molecular', 'weight', 'obtained', 'aid', 'ultracentrifugation', 'referred', 'sedimenta-', 'tion', 'average', 'molecular', 'weight', 'z-average', 'molecular', 'weight', 'mz', 'this', 'often', 'used', 'past', 'particularly', 'naturally', 'occurring', 'polymeric', 'materials', 'it', 'used', 'however', 'much', 'today', 'in', 'solutions', 'polymers', 'viscosities', 'affected', 'long', 'chains', 'short', 'ones', 'a', 'correlation', 'viscosity', 'solution', 'size', 'chains', 'molecular', 'weight', 'solute', 'allows', 'expression', 'viscosity', 'average', 'molecular', 'weight', 'mz', '¼', 'mi', '3ni', 'mi', '2ni', 'x', 'x', 'm(cid2)', '¼', 'mi', 'bþ1ni', '1b', 'mini', '(cid5)', 'x', 'x', '(cid6)', 'b', 'constant,', 'usually', 'less', 'unity', 'when', 'b', '¼', '1', 'mz', 'becomes', 'equal', 'mw', 'actually', 'though', 'values', 'm(cid2)', 'usually', 'within', '20', 'weight', 'average', 'molecular', 'weights', 'for', 'polydisperse', 'polymers', 'mw', 'larger', 'm(cid2)', 'm(cid2)', 'turn', 'larger', 'mn', 'most', 'methods', 'determining', 'molecular', 'weights', 'sizes', 'polymers', '(with', 'exception', 'small-angle', 'neutron', 'scattering)', 'require', 'dissolving', 'polymers', 'proper', 'solvents', 'ﬁrst', 'the', 'measurements', 'carried', 'dilute', 'solutions', 'solution', 'viscosities', 'linear', 'polymers', 'relate', 'empirically', 'molecular', 'weights', 'this', 'used', 'various', 'ways', 'industry', 'designate', 'size', 'polymers', 'the', 'values', 'obtained', 'measuring', 'efﬂux', 'time', 'polymer', 'solution', 'capillary', 'it', 'related', 'efﬂux', 'time', 'pure', 'solvent', 'typical', 'viscometers', 'like', 'designed', 'ubbelohde', 'cannon–fenske', '(shown', 'fig', '13)', 'similar', 'ones', 'utilized', 'measurements', 'carried', 'constant', 'temperature', 'baths', 'the', 'viscosity', 'expressed', 'following', 'ways', 'common', 'name', '1', 'relative', 'viscosity', '2', 'speciﬁc', 'viscosity', '3', 'reduced', 'viscosity', '4', 'inherent', 'viscosity', '5', 'intrinsic', 'viscosity', 'symbol', '(cid2)rel', '(cid2)sp', '(cid2)red', '(cid2)i', '(cid2)c0', 'deﬁnitions', '(cid2)(cid2)o', '¼', 'tto', '((cid2)', '(cid3)', '(cid2)o)(cid2)o', '¼', '(cid2)rel', '(cid3)', '1', '(t', '(cid3)', 'to)to', '(cid2)spc', '¼', '(cid2)rel(cid3)1', 'cz', 'ln', '(cid2)relc', '((cid2)spc)c', '¼', '0', '¼', '((cid2)i)c', '¼', '0', '272', 'methods', 'measuring', 'molecular', 'weights', 'polymers', 'to', 'determine', 'intrinsic', 'viscosity', 'inherent', 'reduced', 'viscosities', 'plotted', 'concen-', 'tration', '(c)', 'graph', 'paper', 'extrapolated', 'zero', 'if', 'intercepts', 'coincide', 'taken', '54', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'fig', '217', 'cannon–fenske', 'capillary', 'viscometer', 'intrinsic', 'viscosity', 'if', 'two', 'intercepts', 'averaged', 'the', 'relationship', 'intrinsic', 'viscosity', 'molecular', 'weight', 'expressed', 'mark–houwink–sakurada', 'equation', '66:', '½(cid2)(cid4)c¼0', '¼', 'kma', '(cid2)', 'k', 'constants', 'various', 'capillary', 'viscometers', 'available', 'commercially', 'figure', '217', 'illustrates', 'typical', 'capillary', 'viscometer', 'the', 'logarithms', 'intrinsic', 'viscosities', 'fractionated', 'samples', 'plotted', 'log', 'mw', 'log', 'mn', 'the', 'constants', 'k', 'mark–houwink–sakurada', 'equation', 'intercept', 'slope', 'respectively', 'plot', 'except', 'lower', 'molecular', 'weight', 'samples', 'plots', 'linear', 'linear', 'polymers', 'many', 'values', 'k', 'different', 'linear', 'polymers', 'found', 'literature', '66.', 'actually', 'mark–houwink–sakurada', 'equation', 'applies', 'narrow', 'molecular', 'weight', 'distri-', 'bution', 'polymers', 'for', 'low', 'molecular', 'weight', 'polydisperse', 'polymers', 'equation', 'useful', 'deviations', 'due', 'chain', 'entanglement', 'still', 'negligible', 'on', 'hand', 'chain', 'entanglement', 'high', 'molecular', 'weight', 'polydisperse', 'polymers', 'affects', 'viscosity', 'equation', 'really', 'apply', 'the', 'determinations', 'molecular', 'weights', 'polymers', 'rely', 'cases', 'upon', 'physical', 'methods', 'in', 'special', 'ones', 'however', 'molecular', 'weights', 'fairly', 'low', 'chemical', 'techniques', 'used', 'such', 'determinations', 'limited', 'macromolecules', 'possess', 'one', 'functional', 'group', 'located', 'end', 'chain', 'ends', 'in', 'place', 'functional', 'group', 'may', 'heteroatom', 'in', 'case', 'elemental', 'analysis', 'might', 'sufﬁcient', 'determine', 'molecular', 'weight', 'if', 'functional', 'group', 'however', 'reaction', 'group', 'allows', 'calculating', 'molecular', 'weight', 'molecular', 'weights', '25000', 'make', 'chemical', 'approaches', 'impractical', 'in', 'chemical', 'determination', 'molecule', 'contributes', 'equally', 'total', 'this', 'determination', 'therefore', 'yields', 'number', 'average', 'molecular', 'weight', 'with', 'development', 'gel', 'permeation', 'chromatography', '(discussed', 'below)', 'method', 'hardly', 'ever', 'used', 'today', 'there', 'various', 'physical', 'methods', 'available', 'the', 'prominent', 'ones', 'ebullioscopy', 'cryoscopy', 'osmotic', 'pressure', 'measurements', 'light', 'scattering', 'ultracentrifugation', 'gel', 'permeation', 'chromatography', '(also', 'called', 'size', 'exclusion', 'chromatography)', 'all', 'determinations', 'carried', 'solutions', 'polymers', 'also', 'except', 'gel', 'permeation', 'chromatography', 'require', 'results', '27', 'molecular', 'weights', 'molecular', 'weight', 'determinations', '55', 'fig', '218', 'membrane', 'osmometer', 'capillary', 'solution', 'solvent', 'membrane', 'measurements', 'extrapolated', 'zero', 'concentrations', 'fulﬁll', 'requirements', 'theory', 'the', 'laws', 'govern', 'various', 'relationships', 'determinations', 'apply', 'ideal', 'solutions', 'only', 'complete', 'absence', 'chain', 'entanglement', 'interaction', 'solute', 'solvent', 'ideality', 'solutions', 'approached', 'a', 'brief', 'discussion', 'techniques', 'used', 'molecular', 'weight', 'determination', 'follows', 'ebullioscopy', 'boiling', 'point', 'elevation', 'well', 'cryoscopy', 'freezing', 'point', 'depression', 'well-', 'known', 'methods', 'organic', 'chemistry', 'they', 'used', 'determining', 'molecular', 'weights', 'small', 'molecules', 'the', 'limitation', 'using', 'methods', 'determine', 'molecular', 'weight', 'macromolecules', 'dtb', 'dtf', 'become', 'increasingly', 'smaller', 'molecular', 'sizes', 'increase', 'the', 'methods', 'limited', 'therefore', 'capabilities', 'temperature', 'sensing', 'devices', 'detect', 'small', 'differences', 'temperature', 'this', 'places', 'upper', 'limits', 'determinations', 'somewhere', '40000', '50000', 'the', 'thermodynamic', 'relationships', 'determinations', '½dtbc(cid4)c0', '¼', 'rt2rdhbm', '½dtfc(cid4)c0', '¼', 'rt2rdhfm', 'boiling', 'point', 'rise', 'elevation', 'freezing', 'point', 'depression', 'the', 'two', 'determinations', 'molecule', 'contributes', 'equally', 'properties', 'solutions', 'yield', 'number', 'average', 'molecular', 'weights', 'how', 'much', 'technique', 'used', 'today', 'hard', 'tell', 'a', 'method', 'used', 'higher', 'molecular', 'weight', 'polymers', 'based', 'osmotic', 'pressure', 'measurements', 'it', 'applied', 'polymers', 'range', 'molecular', 'weights', '20000', '500000', '(some', 'claim', '1000000', 'higher)', 'the', 'method', 'based', 'van’t', 'hoff’s', 'law', 'when', 'pure', 'solvent', 'placed', 'one', 'side', 'semi-permeable', 'membrane', 'solution', 'pressure', 'develops', 'pure', 'solvent', 'side', 'this', 'pressure', 'due', 'tendency', 'liquids', 'equilibrate', 'concentrations', 'it', 'inversely', 'proportional', 'size', 'solute', 'molecules', 'the', 'relationship', 'follow', 'mn', '¼', 'rtðpcþc¼0', 'þ', 'a2c', 'p', 'osmotic', 'pressure', 'c', 'concentration', 't', 'temperature', 'r', 'gas', 'constant', 'a2', 'measure', 'interaction', 'solvent', 'polymer', '(second', 'viral', 'coefﬁcient)', 'a', 'static', 'capillary', 'osmometer', 'illustrated', 'fig', '218', 'rather', 'rely', 'liquid', 'rise', 'capillary', 'side', 'solution', 'response', 'osmotic', 'pressure', 'done', 'static', 'method', 'dynamic', 'equilibrium', 'method', 'used', 'here', 'counter', 'pressure', 'applied', 'maintain', 'equal', 'levels', 'liquid', 'capillaries', 'prevents', 'ﬂow', 'solvent', 'different', 'types', 'dynamic', 'membrane', 'osmometers', 'available', 'commercially', 'the', 'principle', 'illustrated', 'fig', '219', '56', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'fig', '219', 'schematic', 'high', 'speed', 'osmometer', '(from', 'ref', '68)', 'fig', '220', 'extrapolation', 'zero', 'concentration', 'πc', '0', 'c', 'gl', 'the', 'results', 'obtained', 'either', 'method', 'must', 'still', 'extrapolated', 'zero', 'concentration', 'van’t', 'hoff’s', 'law', 'apply', 'such', 'extrapolation', 'illustrated', 'fig', '220', 'because', 'molecules', 'contribute', 'equally', 'total', 'pressure', 'osmotic', 'pressure', 'measurements', 'yield', 'number', 'average', 'molecular', 'weight', 'light', 'scattering', 'measurement', 'technique', 'determining', 'weight', 'average', 'molecular', 'weight', 'when', 'light', 'passes', 'solvent', 'part', 'energy', 'light', 'lost', 'due', 'absorption', 'conversion', 'heat', 'scattering', 'the', 'scattering', 'pure', 'liquids', 'attributable', 'differences', 'densities', 'result', 'ﬁnite', 'nonhomogenuities', 'distribution', 'molecules', 'within', 'adjacent', 'areas', 'additional', 'scattering', 'results', 'presence', 'solute', 'liquid', 'the', 'intensity', 'amplitude', 'additional', 'scattering', 'depends', 'upon', 'concentration', 'size', 'polarizability', 'solute', 'plus', 'factors', 'the', 'refractive', 'index', 'pure', 'solvent', 'solution', 'also', 'dependent', 'upon', 'amplitude', 'vibration', 'the', 'turbidity', 'arises', 'scattering', 'related', 'concentration', 'turbidity', '¼', 't1', '(cid3)', 'h', 'cimi', '¼', 'hcmw', 'x', 'h', '¼', '32p3no', 'x', '2ðdndcþ23l4no', 'refractive', 'index', 'solvent', 'n', 'refractive', 'index', 'solution', 'l', 'wavelength', 'incident', 'light', 'no', 'avogadro’s', 'number', 'c', 'concentration', 'the', 'dndc', 'relationship', 'obtained', 'measuring', 'slope', 'refractive', 'index', 'function', 'concentration', 'it', 'constant', 'given', 'polymer', 'solvent', 'temperature', 'called', 'speciﬁc', 'refractive', 'increment', '27', 'molecular', 'weights', 'molecular', 'weight', 'determinations', '57', 'fig', '221', 'a', 'typical', 'zimm', 'plot', 'θ0', 'kcr', 'c0', 'sin2', 'θ2', 'kc', 'because', 'scattering', 'varies', 'different', 'angles', 'main', 'beam', 'light', 'results', 'must', 'extrapolated', 'zero', 'concentration', 'zero', 'angle', 'scattering', 'this', 'done', 'simultaneously', 'method', 'developed', 'zimm', 'a', 'typical', 'zimm', 'plot', 'illustrated', 'fig', '221', 'a', 'popular', 'technique', 'determining', 'molecular', 'weights', 'molecular', 'weight', 'distributions', 'gel', 'permeation', 'chromatography', 'it', 'also', 'called', 'size', 'exclusion', 'chromatography', '69', '70.', 'the', 'proce-', 'dure', 'allows', 'one', 'determine', 'mw', 'mn', 'molecular', 'weight', 'distribution', 'one', 'operation', 'the', 'procedure', 'resembles', 'hplc', 'it', 'separates', 'molecules', 'according', 'hydrodynamic', 'volumes', 'effective', 'sizes', 'solutions', 'the', 'separation', 'takes', 'place', 'one', 'columns', 'packed', 'small', 'porous', 'particles', 'as', 'solution', 'travels', 'columns', 'retention', 'polymer', 'molecules', 'pores', 'packing', 'it', 'postulated', 'past', 'separation', 'takes', 'place', 'molecular', 'sizes', 'due', 'smaller', 'molecules', 'diffusing', 'pores', 'larger', 'ones', 'pores', 'the', 'largest', 'molecules', 'thought', 'diffuse', 'none', 'pores', 'pass', 'interstitial', 'volumes', 'as', 'result', 'polymer', 'molecules', 'different', 'sizes', 'travel', 'different', 'distances', 'column', 'this', 'means', 'molecules', 'largest', 'size', '(highest', 'molecular', 'weight)', 'eluted', 'ﬁrst', 'ﬁt', 'least', 'number', 'pores', 'the', 'smallest', 'molecules', 'hand', 'eluted', 'last', 'enter', 'greatest', 'number', 'pores', 'travel', 'longest', 'path', 'the', 'rest', 'fall', 'the', 'process', 'however', 'complex', 'postulated', 'picture', 'it', 'yet', 'fully', 'explained', 'it', 'found', 'instance', 'different', 'gels', 'display', 'almost', 'identical', 'course', 'relation', 'dependence', 'vr', '(retention', 'volume)', 'molecular', 'weight', 'yet', 'study', 'pores', 'different', 'gels', 'show', 'varying', 'cumulative', 'distributions', 'inner', 'volumes', 'this', 'means', 'simple', 'function', 'correlating', 'volume', 'andor', 'size', 'separated', 'molecules', 'size', 'distribution', 'pores', '69.', 'also', 'shape', 'pores', 'inferred', 'ratio', 'area', 'volume', 'inner', 'pores', 'important', '70.', 'different', 'models', 'proposed', 'explain', 'separation', 'phenomenon', 'these', 'reviewed', 'thoroughly', 'literature', 'they', 'beyond', 'scope', 'book', 'as', 'indicated', 'volume', 'liquid', 'corresponds', 'solute', 'eluting', 'columns', 'called', 'retention', 'volume', 'elution', 'volume', '(vr)', 'it', 'related', 'physical', 'parameters', 'column', 'follows', 'vr', '¼', 'vo', 'þ', 'kv1', 'vo', '¼', 'interstitial', 'volume', 'column(s)', 'k', '¼', 'distribution', 'coefﬁcient', 'v1', '¼', 'internal', 'solvent', 'volume', 'inside', 'pores', 'the', 'total', 'volume', 'columns', 'vt', 'equal', 'sum', 'vo', 'v1', 'the', 'retention', 'volume', 'expressed', 'follows', 'vr', '¼', 'voð1', 'þ', 'kþ', 'þ', 'kvt', '58', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'fig', '222', 'molecular', 'weight', 'calibration', 'curve', 'gel', 'permeation', 'chromatography', 'from', 'earlier', 'statement', 'clear', 'polymer', 'fractionation', 'gel', 'permeation', 'chroma-', 'tography', 'depends', 'upon', 'spaces', 'polymer', 'molecules', 'occupy', 'solution', 'by', 'measuring', 'experi-', 'mentally', 'molecular', 'weights', 'polymer', 'molecules', 'eluted', 'one', 'obtains', 'molecular', 'weight', 'distribution', 'to', 'accomplish', 'however', 'one', 'must', 'chromatograph', 'equipped', 'dual', 'detectors', 'one', 'must', 'detect', 'presence', 'polymer', 'molecules', 'efﬂuent', 'the', 'one', 'must', 'measure', 'molecular', 'weights', 'such', 'detectors', 'might', 'instance', 'refractive', 'index', 'monitor', 'low', 'angle', 'laser', 'light', 'scattering', 'photogoniometer', 'ﬁnd', 'absolute', 'value', 'm', 'many', 'molecular', 'weight', 'measurements', 'however', 'done', 'chromatographs', 'equipped', 'one', 'detector', 'monitors', 'presence', 'solute', 'efﬂuent', 'the', 'equipment', 'must', 'therefore', 'calibrated', 'prior', 'use', 'the', 'relationship', 'ordinate', 'chromatogram', 'commonly', 'represented', 'f(v)', 'must', 'related', 'molecular', 'weight', 'this', 'relationship', 'varies', 'polymer', 'type', 'structure', 'there', 'three', 'methods', 'calibrating', 'chromatograph', 'the', 'ﬁrst', 'popular', 'one', 'makes', 'use', 'narrow', 'molecular', 'weight', 'distribution', 'reference', 'standards', 'the', 'second', 'one', 'based', 'upon', 'polydisperse', 'reference', 'material', 'the', 'third', 'one', 'assumes', 'separation', 'determined', 'molecular', 'size', 'all', 'three', 'methods', 'require', 'experimentally', 'established', 'calibration', 'curve', 'relationship', 'molecular', 'size', 'polymer', 'solution', 'molecular', 'weight', 'developed', 'a', 'chromatogram', 'obtained', 'ﬁrst', 'every', 'standard', 'sample', 'a', 'plot', 'prepared', 'logarithms', 'average', 'weights', 'peak', 'retention', 'volumes', '(vr)', 'the', 'values', 'vr', 'measured', 'points', 'injection', 'appearances', 'maximum', 'values', 'chromatograms', 'above', 'm1', 'm4', 'effective', 'fractionation', 'total', 'exclusion', 'ﬁrst', 'place', 'total', 'permeation', 'second', 'case', 'these', 'limits', 'separation', 'packing', 'material', 'to', 'date', 'standard', 'samples', 'narrow', 'molecular', 'weight', 'distribution', 'polymers', 'available', 'commercially', 'mainly', 'polystyrenes', 'these', 'samples', 'polydispersity', 'indexes', 'close', 'unity', 'available', 'wide', 'range', 'molecular', 'weights', 'for', 'determining', 'molecular', 'weights', 'polymers', 'polystyrene', 'however', 'molecular', 'weights', 'obtained', 'samples', 'would', 'approximations', 'sometimes', 'could', 'error', 'to', 'overcome', 'difﬁculty', 'universal', 'calibration', 'method', 'used', 'the', 'basis', 'universal', 'calibration', 'observation', '51', 'multiplication', 'products', 'intrinsic', 'viscosities', 'molecular', 'weights', 'independent', 'polymer', 'types', 'thus', '(cid2)m', 'universal', 'calibration', 'parameter', 'as', 'result', 'plot', 'log', '((cid2)m)', 'vs', 'elution', 'volume', 'yields', 'curve', 'applicable', 'many', 'polymers', 'the', 'log', '((cid2)m)', 'given', 'column', '(or', 'columns)', 'temperature', 'elution', 'volume', 'assumed', 'constant', 'polymers', 'this', 'illustrated', 'fig', '222', '27', 'molecular', 'weights', 'molecular', 'weight', 'determinations', '59', 'fig', '223', 'a', 'typical', 'digitized', 'gel', 'permeation', 'chromatogram', 'heated', 'sample', 'inlet', 'heated', 'oven', 'glass', 'gage', 'sample', 'valve', '2', '2', '2', '2', 'reference', 'column', 'sample', 'column', 'solvent', 'reservoir', 'control', 'valve', 'degasser', 'pump', 'filter', 'control', 'valves', 'refractometer', 'syphon', 'photo', 'cell', 'light', 'source', 'waste', 'fraction', 'collector', 'fig', '224', 'schematic', 'illustration', 'gel', 'permeation', 'equipment', '(the', 'illustration', 'shows', 'one', 'sample', 'column', 'several', 'sample', 'columns', 'often', 'used)', '(from', 'ref', '68)', 'numerous', 'materials', 'used', 'packing', 'columns', 'semi', 'rigid', 'crosslinked', 'polystyrene', 'beads', 'available', 'commercially', 'they', 'used', 'quite', 'frequently', 'porous', 'beads', 'glass', 'silica', 'also', 'available', 'in', 'addition', 'commercial', 'gel', 'permeation', 'equipment', 'usually', 'provided', 'automatic', 'sample', 'injection', 'fraction', 'collection', 'features', 'the', 'favorite', 'detectors', 'refractive', 'index', 'ultraviolet', 'light', 'spectroscopic', 'detectors', 'some', 'infrared', 'spectroscopic', 'detectors', 'also', 'use', 'commercially', 'available', 'instruments', 'also', 'contain', 'pumps', 'high-pressure', 'rapid', 'ﬂow', 'usually', 'equipped', 'microcomputer', 'assist', 'data', 'treatment', 'also', 'come', 'plotter', 'equipment', 'plot', 'detector', 'response', 'samples', 'eluted', 'column', 'columns', 'a', 'typical', 'chromatogram', 'illustrated', 'fig', '223', 'schematic', 'basic', 'equipment', 'shown', 'fig', '224', 'when', 'polydisperse', 'samples', 'analyzed', 'quantitative', 'procedures', 'consist', 'digitized', '60', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', 'chromatograms', 'indication', 'equally', 'spaced', 'retention', 'volumes', 'these', 'every', '25', '50', 'ml', 'volumes', 'the', 'resultant', 'artiﬁcial', 'fractions', 'characterized', 'heights', 'hi', 'solute', 'concentrations', 'ci', 'area', 'occupy', 'within', 'curve', 'ai', 'the', 'cumulative', 'polymer', 'weight', 'values', 'calculated', 'according', 'iðvþ', '¼', '1', 'at', 'ai', 'after', 'conversion', 'retention', 'volumes', 'vi', 'molecular', 'weights', '(using', 'calibration', 'curve)', 'x', 'molecular', 'weights', 'mw', 'mn', 'mz', 'calculated', 'mn', '¼', 'hi', 'himi', 'mw', '¼', 'himi', 'hi', 'mz', '¼', '2', 'himi', 'himi', 'x', '(cid5)', 'x', '(cid6)', 'x', 'x', 'x', 'x', 'if', 'chromatogram', 'equipped', 'microcomputer', 'data', 'treatment', 'one', 'easily', 'determine', 'results', 'available', 'pc', 'programs', 'data', 'treatment', 'written', 'various', 'computer', 'languages', 'they', 'available', 'many', 'sources', 'recently', 'several', 'reports', 'literature', 'combining', 'size', 'exclusion', 'high', 'pressure', 'liquid', 'chromatography', 'comprehensive', 'characterization', 'polymers', 'thus', 'gray', 'et', 'al', 'reported', 'combination', 'high', 'pressure', 'liquid', 'chromatography', 'size', 'exclusion', 'chromatography', 'allows', 'comprehensive', 'structural', 'characterization', 'macromolecules', '71.', 'on', 'hand', 'chang', 'et', 'al', 'reported', 'using', 'modiﬁed', 'form', 'high', 'pressure', 'liquid', 'chromatography', 'analysis', 'referred', 'interaction', 'chromatography', 'polymer', 'characterization', 'the', 'process', 'utilizes', 'enthalpic', 'interaction', 'polymeric', 'solutes', 'stationary', 'phase', 'such', 'interaction', 'depends', 'chemical', 'composition', 'molecular', 'weight', 'it', 'claimed', 'less', 'sensitive', 'chain', 'architecture', 'offer', 'superior', 'resolution', 'sec', 'the', 'typical', 'hplc', 'instrument', 'modiﬁed', 'precisely', 'control', 'temperature', 'column', 'the', 'temperature', 'column', 'mobile', 'phase', 'controlled', 'circulating', 'ﬂuid', 'column', 'jacket', 'programmable', 'bathcirculator', 'the', 'mobile', 'stationary', 'phases', 'require', 'careful', 'choices', 'adjust', 'interaction', 'strength', 'polymer', 'solutes', 'stationary', 'phase', 'polymer', 'solutes', 'elute', 'reasonable', 'elution', 'time', 'the', 'process', 'depends', 'upon', 'variations', 'column', 'temperature', 'precise', 'control', 'solute', 'retention', 'isocratic', 'elution', 'mode', 'mixed', 'solvent', 'system', 'polar', 'less', 'polar', 'solvent', 'often', 'employed', 'adjust', 'interaction', 'strength', '72.', '28', 'optical', 'activity', 'polymers', 'optical', 'activity', 'biopolymers', 'known', 'studied', 'well', 'phenomenon', 'observed', 'synthetic', 'polymers', 'homopolymerization', 'vinyl', 'monomers', 'result', 'structures', 'asymmetric', 'centers', '(the', 'role', 'end', 'groups', 'generally', 'negligible)', 'polymers', 'formed', 'exhibit', 'optical', 'activity', 'however', 'contain', 'centers', 'asymmetry', 'backbones', '73.', 'this', 'result', 'optical', 'activity', 'monomers', 'this', 'activity', 'becomes', 'incorporated', 'polymer', 'backbone', 'process', 'chain', 'growth', 'it', 'also', 'result', 'polymerization', 'involves', 'asymmetric', 'induction', '74', '75.', 'these', 'processes', 'polymer', 'formation', 'explained', 'subsequent', 'chapters', 'an', 'example', 'inclusion', 'optically', 'active', 'monomer', 'polymer', 'chain', 'polymerization', 'optically', 'active', 'propylene', 'oxide', '(see', 'chap', '5', 'additional', 'discussion)', 'the', 'process', 'chain', 'growth', 'monomer', 'addition', 'sterically', 'controlled', 'asymmetric', 'portion', 'monomer', 'several', 'factors', 'appear', 'important', 'order', 'produce', 'measurable', 'optical', 'activity', 'copolymers', '76.', 'these', '(1)', 'selection', 'comonomer', 'must', 'induced', 'asymmetric', 'center', 'polymer', 'backbone', 'remains', 'center', 'asymmetry', '(2)', 'the', 'four', 'substituents', 'originally', 'inducing', 'center', 'center', 'portion', 'must', 'differ', 'considerably', 'size', '(3)', 'the', 'location', 'review', 'questions', '61', 'inducing', 'center', 'must', 'close', 'polymer', 'backbone', '(4)', 'the', 'polymerization', 'reaction', 'must', 'conducted', 'sufﬁciently', 'low', 'temperature', 'insure', 'stereo', 'chemical', 'selectivity', 'an', 'example', 'copolymerization', 'maleic', 'anhydride', 'optically', 'active', 'l-a-methylbenzyl', 'vinyl', 'ether', 'the', 'copol-', 'ymer', 'exhibits', 'optical', 'activity', 'removal', 'original', 'center', 'asymmetry', '77.', 'an', 'example', 'asymmetric', 'induction', 'optically', 'inactive', 'monomers', 'anionic', 'polymeri-', 'zation', 'esters', 'butadiene', 'carboxylic', 'acids', '(r)-2-methylbutyllithium', 'butyllithium', 'complexed', '((cid3))methyl', 'ethyl', 'ether', 'catalyst', '(this', 'type', 'polymerization', 'reaction', 'described', 'chap', '4)', 'the', 'products', 'tritactic', 'polymers', 'exhibit', 'small', 'measurable', 'optical', 'rotations', '78.', 'also', 'benzofuran', 'exhibits', 'optical', 'activity', 'polymerized', 'cationic', 'catalysts', 'like', 'aluminum', 'chloride', 'complexed', 'optically', 'active', 'co', 'catalyst', 'like', 'phenylalanine', 'optically', 'active', 'polymer', 'obtained', '77.', 'by', 'contrast', 'example', 'formation', 'enantioselective', 'polymer', 'achiral', 'monomers', 'chirality', 'inherent', 'main', 'chain', 'polymerization', '15-hexadiene', 'optically', 'active', 'catalyst', '77.', 'the', 'catalyst', 'precursors', '(rr)', '(ss)-ethylene-12-bis(z5-4567-tetrahydro-1-', 'indenyl)zirconium', '(110-by-1-naphtholate)', 'the', 'product', 'optically', 'active', 'version', 'poly(methy-', 'lene-13-cyclopentane)', 'n', 'this', 'polymer', 'highly', 'isotactic', 'contains', '72', 'trans', 'rings', 'it', 'also', 'observed', 'conjugated', 'polymers', 'also', 'electrical', 'conductors', '(see', 'chap', '10)', 'exhibit', 'optical', 'activity', 'depends', 'critically', 'structural', 'organization', '78.', 'thus', 'strong', 'chiroptical', 'properties', 'obtained', 'substituted', 'polythiophene', '79', '(chap', '10)', 'optically', 'active', 'side', 'chains', 'especially', 'monomers', 'coupled', 'within', 'polymer', 'regioregular', 'head-to-tail', 'fashion', 'actually', 'optical', 'activity', 'materials', 'found', 'polymers', 'aggregated', 'low', 'temperature', 'poor', 'solvent', 'solution', 'cast', 'ﬁlms', 'this', 'contrasts', 'optically', 'active', 'polymers', 'like', 'polypeptides', 'poly(l-alkynes)', 'polyisocyanates', 'show', 'optically', 'active', 'conformation', 'main', 'chain', 'absence', 'supramolecular', 'association', 'in', 'addition', 'shown', 'repetitive', 'inversion', 'optical', 'activity', 'ﬁlms', 'obtained', 'warming', 'cooling', 'cycles', 'cooling', 'rate', 'determines', 'handedness', 'associates', '78.', 'a', 'similar', 'result', 'concerning', 'inversion', 'chirality', 'found', 'solution', 'depending', 'composition', 'binary', 'solvent', 'mixture', 'review', 'questions', 'section', '21', 'section', '211', '1', 'what', 'secondary', 'bond', 'forces', 'inﬂuence', 'physical', 'properties', 'macromolecules', '1', 'explain', 'illustrate', 'dipole–dipole', 'interactions', 'polymers', 'affect', 'properties', 'polymeric', 'materials', 'can', 'give', 'examples', '62', 'section', '212', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', '1', 'what', 'induction', 'forces', '2', 'explain', 'type', 'chemical', 'structures', 'chemical', 'bonds', 'backbones', 'polymeric', 'chains', 'stiffen', 'type', 'ﬂexibilizes', '(or', '“softens”)', '3', 'how', 'pendant', 'groups', 'affect', 'melting', 'points', 'polymers', '4', 'does', 'copolymerization', 'raise', 'lower', 'melting', 'point', 'polymer', '5', 'which', 'polymer', 'a', 'b', 'would', 'higher', 'melting', 'point', 'explain', 'a', 'f', 'f', 'n', 'f', 'n', 'o', 'n', 'b', 'cl', 'n', 'o', 'n', 'ch3', 'f', 'cl', 'f', 'f', 'n', 'f', 'f', '6', 'explain', 'induction', 'forces', 'polymers', 'affect', 'physical', 'properties', '7', 'illustrate', 'explain', 'nylon', '11', 'melts', 'approximately', '50(cid1)c', 'higher', 'linear', 'polyethylene', '8', 'why', 'poly(ethylene', 'adipate)', 'melt', 'considerably', 'lower', 'temperature', 'poly', 'materials', '(ethylene', 'terephthalate)', '9', 'what', 'type', 'bonds', 'polymeric', 'chain', 'tend', 'soften', '10', 'does', 'copolymerization', 'lower', 'raise', 'melting', 'temperature', 'polymers', 'explain', '1', 'describe', 'amorphous', 'state', 'polymers', 'section', '22', 'section', '221', '1', 'describe', 'glassy', 'state', 'polymers', '2', 'what', 'second', 'order', 'transition', 'temperature', 'are', 'send', 'order', 'transition', 'temperatures', 'polymers', 'absolute', 'values', 'vary', 'depending', 'upon', 'various', 'conditions', 'explain', '3', 'how', 'transition', 'glassy', 'state', 'observed', '4', 'explain', 'structural', 'recovery', 'asymmetry', 'structural', 'recovery', '5', 'what', 'equation', 'describes', 'asymmetry', 'structural', 'recovery', '6', 'what', 'flory–fox', 'equation', 'limited', 'review', 'questions', 'section', '222', '63', '1', 'explain', 'phenomenon', 'elasticity', '2', 'what', 'natural', 'structural', 'arrangement', 'polymeric', 'chains', 'leads', 'rubber', 'elasticity', '3', 'what', 'meant', 'negative', 'coefﬁcient', 'expansion', 'section', '2221', 'deﬁne', 'ideal', 'elastomer', '1', 'write', 'equation', 'reactive', 'force', 'elastomer', 'explain', 'term', 'represents', '2', 'what', 'entropy', 'elasticity', 'what', 'proportional', '3', 'what', 'relationship', 'retractive', 'force', 'elastomer', 'temperature', '4', 'write', 'equation', 'work', 'done', 'stretching', 'chain', 'ideal', 'elastomer', 'explain', 'term', 'represents', '5', 'write', 'equation', 'free', 'energy', 'change', 'stretched', 'elastomer', '6', 'write', 'equation', 'average', 'energy', 'per', '(stretched)', 'chain', '7', 'what', 'free', 'energy', 'change', 'stretched', 'elastomer', 'explain', 'section', '2222', '1', 'deﬁne', 'newtonian', 'liquid', 'hookian', 'ideal', 'elastic', 'solid', 'ideal', 'elastomer', '2', 'what', 'meant', 'viscoelasticity', '3', 'explain', 'meant', 'yield', 'point', 'bingham', 'newtonian', 'ﬂuid', '4', 'how', 'viscosity', 'molten', 'polymer', 'related', 'temperature', '5', 'how', 'shear', 'stress', 'deﬁned', 'mathematically', '6', 'what', 'thixotropic', 'liquid', '7', 'what', 'relationship', 'newtonian', 'viscosity', 'amorphous', 'polymers', 'chain', 'length', '8', 'what', 'shear', 'creep', 'compliance', 'equal', '9', 'why', 'shear', 'thinning', 'thixotropy', 'two', 'different', 'phenomena', 'explain', '10', 'what', 'relationship', 'describes', 'ﬂow', 'behavior', 'liquids', 'deviate', 'newtonian', 'ﬂow', '11', 'write', 'equation', 'shear-stress-relaxation', 'modulus', 'viscous', 'ﬂuids', 'explain', '12', 'what', 'two', 'important', 'quantities', 'behavior', 'polymeric', 'liquids', '13', 'describe', 'two', 'techniques', 'measuring', 'viscosity', 'molten', 'polymers', 'section', '23', 'molecules', '1', 'how', 'crystallinity', 'polymers', 'formed', 'melt', 'differ', 'small', '2', 'what', 'two', 'ways', 'crystal', 'growth', 'take', 'place', 'polymeric', 'materials', '3', 'what', 'type', 'polymers', 'tend', 'crystallize', '4', 'what', 'typical', 'size', 'polymeric', 'crystals', 'formed', 'melt', '5', 'what', 'fringed', 'micelle', 'fringed', 'crystallite', 'model', '6', 'what', 'folded', 'chain', 'lamella', 'an', 'adjacent-reentry', 'model', 'a', 'switchboard', 'nonadjacent-', 'reentry', 'model', '7', 'what', 'drawn', 'ﬁbrilar', 'morphology', 'what', 'vitriﬁcation', '64', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', '8', 'what', 'spherulites', 'how', 'observed', '9', 'explain', 'difference', 'crystallite', 'formed', 'melt', 'polymer', 'crystal', 'formed', 'dilute', 'solution', '1', 'what', 'three', 'different', 'interfacial', 'free', 'energies', 'characteristics', 'crystals', '2', 'what', 'equation', 'described', 'dependence', 'melting', 'temperature', 'polymer', 'crystal', 'upon', 'chain', 'length', 'explain', 'write', 'equation', '3', 'what', 'ﬁrst', 'order', 'transition', 'temperature', '1', 'how', 'one', 'obtain', 'information', 'kinetics', 'crystallization', '2', 'what', 'avrami', 'equation', 'drawbacks', 'section', '231', 'section', '232', 'section', '24', '1', 'what', 'mesophases', '2', 'what', 'anisotropic', 'behavior', '3', 'what', 'lyotropic', 'thermotropic', 'liquid', 'crystals', '4', 'explain', 'nematic', 'smectic', 'cholesteric', 'liquid', 'crystal', 'arrangements', '5', 'which', 'two', 'polymers', 'would', 'expect', 'exhibit', 'liquid', 'crystalline', 'behavior', 'explain', 'o', 'n', 'n', 'b', 'o', 'o', 'a', '6', 'what', 'difference', 'main', 'chain', 'side', 'chain', 'liquid', 'crystals', 'section', '25', 'external', 'stress', '1', 'what', 'happens', 'arrangement', 'polymeric', 'chains', 'ﬁlms', 'ﬁbers', 'upon', 'application', '2', 'what', 'orientation', 'beneﬁt', 'properties', 'polymeric', 'materials', '65', 'review', 'questions', 'section', '26', 'section', '261', 'write', 'equation', 'equation', 'temperature', 'section', '27', '1', 'what', 'two', 'stages', 'dissolution', 'polymer', 'dissolves', 'solvent', '2', 'what', 'heat', 'mixing', 'binary', 'systems', 'write', 'equation', '3', 'what', 'radius', 'gyration', 'write', 'equation', '4', 'deﬁne', 'unperturbed', 'dimensions', 'expansion', 'factor', '1', 'what', 'change', 'entropy', 'mixing', 'according', 'flory–huggins', 'theory', 'explain', '2', 'what', 'change', 'free', 'energy', 'mixing', 'according', 'flory–huggings', 'theory', 'write', '3', 'what', 'flory', 'temperature', 'write', 'equation', 'explain', 'happens', 'solution', 'polymer', '1', 'deﬁne', 'degree', 'polymerization', '2', 'what', 'dp', 'polystyrene', 'molecular', 'weight', '104000', 'poly(vinyl', 'chloride)', 'molecular', 'weight', '63000', '3', 'what', 'important', 'features', 'chain-growth', 'step-growth', 'polymerizations', 'can', 'explain', 'difference', 'two', 'can', 'suggest', 'analytical', 'procedure', 'determine', 'mechanism', 'particular', 'polymerization', 'reaction', 'takes', 'place', '4', 'what', 'dp', 'polystyrene', 'molecular', 'weight', '104000', 'poly(vinyl', 'chloride)', 'molecular', 'weight', '630000', '5', 'explain', 'differences', 'thermosetting', 'thermoplastic', 'polymers', 'deﬁne', 'gel', 'point', '6', 'give', 'deﬁnitions', 'oligomer', 'telomer', 'telechelic', 'polymers', '7', 'why', 'must', 'statistical', 'averages', 'used', 'express', 'molecular', 'weights', 'polymers', '8', 'what', 'number', 'average', 'molecular', 'weight', 'what', 'equation', 'number', 'average', 'molecular', '9', 'what', 'weight', 'average', 'molecular', 'weight', 'what', 'equation', 'weight', 'average', 'weight', 'molecular', 'weight', 'molecular', 'weight', 'polymer', '10', 'in', 'mixture', 'two', 'kinds', 'molecules', 'ten', 'kind', 'the', 'molecular', 'weight', 'molecules', 'a', '10000', 'molecular', 'weight', 'molecules', 'b', '100000', 'what', 'number', 'average', 'molecular', 'weight', 'mixture', 'weight', 'average', 'molecular', 'weight', '11', 'what', 'viscosity', 'average', 'molecular', 'weight', 'differ', 'weight', 'average', '12', 'what', 'molecular', 'weight', 'distribution', 'what', 'monodisperse', 'polymer', 'polydisperse', '13', 'what', 'mark–houwink–sakurada', 'equation', 'can', 'suggest', 'way', 'determine', 'k', '14', 'give', 'deﬁnitions', 'formulas', 'relative', 'viscosity', 'speciﬁc', 'viscosity', 'reduced', 'viscosity', 'constants', 'experimentally', 'given', 'polymer', 'inherent', 'viscosity', 'intrinsic', 'viscosity', '15', 'why', 'necessary', 'extrapolate', 'zero', '(explain', 'done)', 'order', 'obtain', 'intrinsic', 'viscosity', '16', 'discuss', 'various', 'methods', 'molecular', 'weight', 'determination', 'explain', 'particular', 'method', 'yields', 'number', 'weight', 'average', 'molecular', 'weight', 'case', 'gpc', '2', 'physical', 'properties', 'physical', 'chemistry', 'polymers', '66', 'section', '28', '1', 'discuss', 'optical', 'activity', 'polymers', 'recommended', 'reading', 'h', 'sperling', 'introduction', 'physical', 'polymer', 'science', '2nd', 'ed,', 'wiley', 'new', 'york', '2006', 'references', '1', 's', 'mizushima', 't', 'shimanouchi', 'j', 'am', 'chem', 'soc,', '1964', '85', '3521', 'lh', 'sperling', 'introduction', 'physical', 'polymer', 'science', '3rd', 'ed,', 'j', 'wiley', 'new', 'york', '2006', 'g', 'strobel', 'the', 'physics', 'polymers', 'ii', 'edition', 'springer', 'berlin', '1997', '2', 'j', 'w', 'le', 'noble', 'highlights', 'organic', 'chemistry', 'dekker', 'new', 'york', 'ny,', '1974', '3', 'm', 'avrami', 'j', 'chem', 'phys,', '1939', '7', '103', 'ibid', '1940', '8', '212', '4', 'f', 'p', 'redding', 'j', 'polymer', 'sci,', '1956', '21', '547', '5', 'l', 'mandelkern', 'am', 'chem', 'soc', 'polymer', 'preprints', '1979', '20', '(1)', '267', 'dc', 'basset', 'principles', 'polymer', 'morphology', 'cambridge', 'university', 'press', 'cambridge', '1981', 'a', 'sharples', '“crystallinity”', 'chapt', '4', 'polymer', 'science', 'voli', 'a', 'jenkins', 'ed,', 'north', 'holland', 'amsterdam', '1972', 'l', 'mandelkern', '“the', 'crystalline', 'state”', 'chapter', '4', 'jmark', 'kngal', 'w', 'graessley', 'l', 'mandelkern', 'e', 'samulsky', 'j', 'koenig', 'g', 'wignall', 'physical', 'properties', 'polymers', '3rd', 'edcambridge,', '2004', '6', 'j', 'brandrup', 'eh', 'immergut,', 'eds,polymer', 'handbook', 'wiley', 'sons', '2nd', '3rd', 'ed;', 'ea', 'turi', 'ed,thermal', 'characterization', 'polymeric', 'materials,academic', 'press', 'new', 'york', '1981', 'ww', 'wedlandt', 'thermal', 'analysis', '3-', 'rd', 'ed,', 'wiley', 'new', 'york', '1986', 't', 'nakaoki', 'r', 'kitamaru', 'rg', 'alamo', 'wt', 'huang', 'l', 'mandelkern', 'poymer', 'j,', '2000', '32', '876', '7', 'f', 'w', 'billmeyer', 'jr,', 'textbook', 'polymer', 'science', 'wiley-interscience', '3rd', 'ed,', 'new', 'york', '1984', '8', 'l', 'marker', 'r', 'early', 's', 'l', 'agrarwall', 'j', 'polymer', 'sci,', '1959', '38', '369', '9', 't', 'w', 'campbell', 'a', 'c', 'haven', 'jr,', 'j', 'appl', 'polymer', 'sci,', '1959', '1', '73', '10', 'g', 'natta', 'f', 'danusso', 'g', 'moraglis', 'j', 'polymer', 'sci,', '1957', '25', '119', '11', 'f', 'p', 'redding', 'j', 'polymer', 'sci,', '1956', '21', '547', '12', 'av', 'tobolsky', 'properties', 'structure', 'polymers', 'j', 'wiley', 'new', 'york', '1960', '13', 'gi', 'wilks', 'j', 'chem', 'ed,', '1981', '58', '880', '14', 'dj', 'williams', 'polymer', 'science', 'engineering', 'prentice-hall', 'englewood', 'cliffs', 'nj,', '(1971)', '15', 'm', 'g', 'zachman', 'angew', 'chem,', 'intern', 'edit,', '1974', '13', '(4)', '244', '16', 'f', 'bueche', 'physical', 'properties', 'polymers', 'interscience', 'new', 'york', '1962', '17', 'dj', 'plazec', 'kl', 'ngai', 'physical', 'properties', 'polymers', 'handbook', 'je', 'mark', 'ed,', 'aip', 'press', 'woodbury', 'new', 'york', '1996', 'also', 'see', 'klngai', 'chapter', '2', 'jmark', 'knegai', 'w', 'grassley', 'l', 'mandelkern', 'e', 'samulski', 'j', 'koenig', 'g', 'wignall', 'physical', 'properties', 'polymers', '3rd', 'ed,', 'cambridge', '2004', '18', 'tgfox', 'jr', 'pj', 'floty', 'j', 'appl', 'phys,', '1964', '21', '581', '19', 'fang', 'liu', 'william', 'l', 'jarrett', 'marek', 'w', 'urban', 'shelby', 'f', 'thames', 'macromolecules', '2010', '43', '5330', '20', 'r', 'd', 'deanin', 'polymer', 'structure', 'properties', 'application', 'cahmers', 'publishing', 'co,', 'inc,', 'boston', 'mass,', '1972', 's-q', 'wang', 's', 'ravindranath', 'pe', 'boukany', 'macromolecules', '2011', '44', '183', '21', 'p', 'j', 'flory', 'principles', 'polymer', 'chemistry', 'cornell', 'univ', 'press', 'ithaca', 'ny,', '1951', '22', 'lrg', 'treloar', 'the', 'physics', 'rubber', 'elasticity', '3rd', 'ed,', 'clarendon', 'press', 'oxford', '1975', 'angent', 'rubber', 'elasticity', 'basic', 'concepts', 'behavior', 'chapter', '1', 'fr', 'eirich', '(ed)', 'science', 'technology', 'rubber', 'academic', 'press', 'new', 'york', '1978', '23', 't', 'koga', 'f', 'tanaka', 'macromolecules', '2010', '43', '3052', '24', 'ww', 'graessley', 'chapter', '3', 'jmark', 'knegai', 'w', 'grassley', 'l', 'mandelkern', 'e', 'samulski', 'j', 'koenig', 'gwignall', 'physical', 'properties', 'polymers', '3rd', 'ed,', 'cambridge', '2004', '25', 'j', 'mark', 'chapter', '1', 'jmark', 'knegai', 'w', 'grassley', 'l', 'mandelkern', 'e', 'samulski', 'j', 'koenig', 'g', 'wignall', 'physical', 'properties', 'polymers', '3rd', 'ed,', 'cambridge', '2004', '26', 'hf', 'mark', 'j', 'polymer', 'sci,', '1965', 'c9', '1', 'a', 'peterlin', 'j', 'polymer', 'sci,', '1965', 'c9', '61', '27', 'e', 'p', 'chan', 'ka', 'page', 'ca', 'stafford', 'am', 'chem', 'soc', 'polymer', 'preprints', '2010', '51', '(1)109', 'references', '67', '28', 'huang', 'r', 'j', 'mech', 'phys', 'solids', '2005', '53', '63–89', '29', 'j', 'r', 'collier', 'ind', 'eng', 'chem,', '1969', '61', '(10)', '72', '30', 'gnatta', 'pcarradini', 'j', 'polymer', 'sci', '1959', '3929', '(1959)', '31', 'ph', 'geil', 'polymer', 'single', 'crystals', 'wiley-interscience', 'new', 'york', '1963', '32', 'ph', 'geil', 'nkj', 'symons', 'rg', 'scott', 'j', 'appl', 'phys,', '30', '1516', '33', 'rh', 'geil', 'j', 'polymer', 'sci,', '1960', '44', '449', '34', 'bh', 'ranby', 'ff', 'morehead', 'nm', 'walter', 'j', 'polymer', 'sci,', '1960', '44', '349', '35', 'a', 'keller', '“growth', 'perfection', 'crystals”', 'wiley', 'new', 'york', '1958', 'a', 'keller', 'a', 'o’connor', 'polymer', '36', 'wmd', 'bryant', 'j', 'polymer', 'sci,', '1947', '2', '547', '37', 'rstj', 'mauley', 'nature', '1961', '189', '390', 's', 'hosoda', 'y', 'nozue', 'ykawashima', 'k', 'suita', 's', 'seno', 'tnagamatsu', '1960', '1', '163', 'macromolecules', '2011', '44', '313', '38', 'm', 'farina', 'm', 'peraldo', 'gnatta', 'angew', 'chem,', 'intern', 'ed,', '1965', '4', '107', '39', 'dc', 'basselt', 'fc', 'frank', 'a', 'keller', 'nature', '1959', '184', '810', 'wd', 'niegisch', 'pr', 'swan', 'j', 'appl', 'phys,', '1960', '31', '1906', 'dh', 'reneker', 'p', 'h', 'geil', 'j', 'appl', 'phys,', '1960', '31', '1916', '40', 'm', 'goodman', 'a', 'abe', 'yl', 'fan', 'macromol', 'rev,', '1967', '1', '8', '41', 'ng', 'gaylord', 'hmark', 'linear', 'stereoregular', 'addition', 'polymers', 'interscience', 'new', 'york', '1959', '42', 'l', 'mandelkern', 'chapter', '4', 'jmark', 'knegai', 'w', 'grassley', 'l', 'mandelkern', 'e', 'samulski', 'j', 'koenig', 'g', 'wignall', 'physical', 'properties', 'polymers', '3rd', 'ed,', 'cambridge', '2004', '43', 'z', 'wang', 'y', 'li', 'j', 'yang', 'q', 'gou', 'y', 'wu', 'x', 'wu', 'p', 'liu', 'q', 'gu', 'macromolecules', '2010', '43', '4441', '44', 'eb', 'sirota', 'macromolecules', '2007', '40', '1045', '45', 'wz', 'oswald', 'z', 'phys', 'chem,', '1897', '22', '286', '46', 'g', 'stroble', 'eur', 'phys', 'j,', '2000', '3', '165', '47', 'g', 'stroble', 'eur', 'phys', 'j', '2005', '18', '295', '48', 'g', 'stroble', 'prog', 'polym', 'sci,', '2006', '31', '398', '49', 'b', 'heckt', 'hugel', 'm', 'iijima', 'g', 'strobl', 'polymer', '2000', '41', '8839', '50', 'k', 'hayashida', 'a', 'takano', 'y', 'matsushita', 'phys', 'rev', 'lett,', '2007', '98', '195502', '51', 's', 'chandrasekhar', 'liquid', 'crystals', 'cambridge', 'university', 'press', 'london', '1977', '52', 'a', 'ciferri', 'wr', 'krigbaum', 'rb', 'meyers', 'eds,', 'polymer', 'liquid', 'crystals', 'academic', 'press', 'new', 'york', '1982', '53', 'tokita', 'm;', 'kim', 'kw;', 'kang', 's;', 'watanabe', 'j', 'macromolecules', '2006', '39', '2021', 'tokita', 'm;', 'funaoka', 's,', 'watanabe', 'j', 'macromolecules', '2004', '37', '9916', '54', 'keith', 'h', 'o', 'chen', 'w', 'y,', 'polymer', '2002', '43', '6263-', '6272;', 'lots', 'band', 'cheng', 's', '2', 'd', 'polymer', '2005', '46', '577-', '610', '55', 'y', 'gnanou', 'm', 'fontanille', 'organic', 'physical', 'chemistry', 'polymers', 'wileynew', 'york', '2008', '56', 'nm', 'abukhdeir', 'ad', 'rey', 'macromolecules', '2009', '42', '3841', '57', 'et', 'samulski', 'chapter', '5', 'jmark', 'knegai', 'w', 'grassley', 'l', 'mandelkern', 'e', 'samulski', 'j', 'koenig', 'g', 'wignall', '“physical', 'properties', 'polymers”', '3rd', 'ed,', 'cambridge', '2004', '58', 't-h', 'tong', 'a', 'primak', 's', 'kumar', 'l-c', 'chien', 'macromolecules', '1998', '31', '3537', '59', 'r', 'deschenaux', 'f', 'turpin', 'd', 'guillon', 'macromolecules', '1997', '30', '3759', '60', 'a', 'sanchez-ferrer', 'h', 'finkelmann', 'macromolecules', '2008', 'web', 'article101021ma7025644s0024-', '9207(70)0264-0', '61', 's', 'rendon', 'wr', 'burghardt', 'ml', 'auad', 'ja', 'kornﬁeld', 'macromolecules', '2007', 'asap', 'article', '101021', 'ma062912c', 'web', 'release', 'date', 'august', '14', '2007', '62', 'ahn', 'deshmukh', 'kosi', 'am', 'chem', 'soc', 'polymer', 'preprints', '2010', '51', '(2)', '430', '63', 'ml', 'huggins', 'physical', 'chemistry', 'high', 'polymers', 'wiley', 'new', 'york', '1958', '64', 'nc', 'billingham', 'molar', 'mass', 'measurements', 'polymer', 'science', 'halsted', 'press', 'new', 'york', '1977', 'jf', 'rabeck', 'experimental', 'methods', 'polymer', 'chemistry', 'wiley', 'new', 'york', '1980', '65', 'j', 'r', 'collier', 'ind', 'eng', 'chem,', '1969', '61', '(10)', '72', '66', 'j', 'brandrup', 'eh', 'immergut,', 'eds,polymer', 'handbook', 'wiley', 'sons', '2nd', '3rd', 'ed;', 'ea', 'turi', 'ed,', '“thermal', 'characterization', 'polymeric', 'materials”', 'academic', 'press', 'new', 'york', '1981', '67', 'f', 'p', 'redding', 'j', 'polymer', 'sci,', '21', '547', '(1956)', '68', 'a', 'ravve', 'organic', 'chemistry', 'macromolecules', 'delkker', 'new', 'york', '1967', '69', 'j', 'janca', 'ed,', '“steric', 'exclusion', 'liquid', 'chromatography', 'polymers”', 'dekker', 'new', 'york', '1984', '70', 't', 'provder', 'ed,', '“detection', 'data', 'analysis', 'size', 'exclusion', 'chromatography”', 'am', 'chem,', 'socwashington,', '1987', '71', 'm', 'gray', 'c', 'silverman', 'p', 'kile', 'am', 'chem', 'soc', 'polymer', 'preprints', '2010', '51(2)', '162', '72', 't', 'chang', 's', 'ahn', 'h', 'lee', 'am', 'chem', 'soc', 'polymer', 'preprints', '2010', '51', '(2)', '160', '73', 'n', 'beredjick', 'c', 'schuerch', 'j', 'am', 'chem', 'soc,', '195678', '2646', 'ibid,', '1958', '80', '1933', '74', 'g', 'natta', 'm', 'farina', 'm', 'peraldo', 'g', 'bressan', 'makromol', 'chem,', '1961', '43', '68', '75', 'm', 'farina', 'g', 'bressan', 'makromol', 'chem,', '1963', '61', '79', '76', 'p', 'pino', 'f', 'ciardelli', 'g', 'p', 'lorenzi', 'j', 'am', 'chem', 'soc,', '1963', '85', '3883', '77', 'p', 'pino', 'f', 'ciardelli', 'g', 'p', 'lorenzi', 'makromol', 'chem,', '1964', '70', '182', '78', 'm', 'goodman', 'a', 'abe', 'yl', 'fan', 'macromol', 'rev,', '1967', '1', '8', '79', 'b', 'goderis', 'h', 'reynaers', 'h', 'scharrengerg', 'v', 'mathot', 'm', 'koch', 'macromolecules', '2001', '34', '1770', 'chapter', '3', 'free-radical', 'chain-growth', 'polymerization', '31', 'free-radical', 'chain-growth', 'polymerization', 'process', 'polymerizations', 'free-radical', 'mechanism', 'typical', 'free-radical', 'reactions', 'that', 'say', 'initiation', 'radicals', 'formed', 'propagation', 'products', 'developed', 'termination', 'free-radical', 'chain', 'reactions', 'end', 'in', 'polymerizations', 'propagations', 'usually', 'chain', 'reactions', 'a', 'series', 'rapid', 'repetitive', 'steps', 'follow', 'single', 'act', 'initiation', 'leading', 'addition', 'thousands', 'monomers', 'this', 'process', 'polymerization', 'vinyl', 'monomers', 'takes', 'place', 'expense', 'double', 'bonds', '(cid1)c¼c–', '–c–c–', 'table', '31', 'illustrates', 'steps', 'process', 'formation', 'initiating', 'radicals', 'rate-determining', 'step', 'initiation', 'reaction', 'the', 'formation', 'initiating', 'radicals', 'result', 'shown', 'table', '31', 'cleavages', 'compounds', 'peroxides', 'sources', 'actually', 'many', 'reactions', 'lead', 'formations', 'free', 'radicals', 'the', 'initiating', 'radicals', 'however', 'must', 'energetic', 'enough', 'react', 'vinyl', 'compounds', 'a', 'linear', 'correlation', 'exists', 'afﬁnities', 'radicals', 'vinyl', 'monomers', 'energy', '(calculated)', 'required', 'localize', 'p', 'electron', 'b-carbon', 'monomer', '1.', 'by', 'comparison', 'steps', 'polymerization', 'process', 'initiation', 'slow', 'step', 'requires', 'high', 'energy', 'activation', '311', 'kinetic', 'relationships', 'free-radical', 'polymerizations', 'a', 'kinetic', 'scheme', 'typical', 'free-radical', 'polymerizations', 'pictured', 'follows', '2;', 'rate', 'propagation', 'rp', 'equal', 'rate', 'polymerization', 'rpol', 'monomer', 'molecules', '(except', 'one)', 'consumed', 'step', 'initiation', 'kd', '2r', 'i', 'propagation', 'termination', 'transfer', 'r(cid3)', 'þ', 'm', 'ki', 'rm(cid3)', 'kp', 'rm(cid3)', 'þ', 'nm', '(cid5)pnþ1(cid3)', '(cid5)', 'pnþ1(cid3)þ', '(cid5)', 'pmþ1(cid3)', 'ktr', 'pn', 'þ', 's(cid3)', '(cid5)', 'pn(cid3)', 'þ', 's', 's(cid3)', 'þ', 'nm', 'smn(cid3)', 'd½r(cid3)(cid4)dt', '¼', 'ri', '¼', '2kd½i(cid4)', '(cid1)', 'dr(cid3)/dt', '¼', 'ki½r(cid3)(cid4)½m(cid4)', 'rp', '¼', 'kp', '½', 'rmn(cid3)', '(cid4)½m(cid4)', 'kt', 'polymer', '(cid1)', 'dp(cid3)/dt', '¼', '2kt½p(cid3)(cid4)2', 'p', 'in', 'shown', 'kinetic', 'scheme', 'm', 'stands', 'monomer', 'concentration', 'i', 'concentration', 'initiator', 'r•', 'p•', 'mean', 'concentration', 'primary', 'polymer', 'radicals', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-93', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '69', '70', '3', 'free-radical', 'chain-growth', 'polymerization', 'table', '31', 'illustration', 'free', 'radical', 'polymerization', '1', 'initiation', '2', 'propagation', '3', 'termination', 'by', 'combination', 'b', 'by', 'disproportionation', 'c', 'by', 'transfer', 'respectively', 's', 'stands', 'chain', 'transferring', 'agent', 'ri', 'denotes', 'decomposition', 'rate', 'initiator', 'rp', 'rate', 'polymerization', 'the', 'rate', 'constants', 'initiator', 'decomposition', 'kd', 'initiation', 'reaction', 'ki', 'propagation', 'kp', 'termination', 'kt', 'the', 'based', 'assumption', 'kp', 'ki', 'independent', 'sizes', 'radicals', 'this', 'supported', 'experimental', 'evidence', 'shows', 'radical', 'reactivity', 'affected', 'size', 'chain', 'length', 'exceeds', 'dimer', 'trimer', 'dimensions', '3.', 'the', 'reactions', 'involved', 'typical', 'free-radical', 'polymerization', 'process', 'stated', 'illustrated', 'table', '31', 'the', 'equation', 'rate', 'propagation', 'shown', 'kinetic', 'scheme', 'contains', 'term', 'm•.', 'it', 'designates', 'radical', 'concentration', 'this', 'quantity', 'hard', 'determine', 'quantitatively', 'concen-', 'tration', 'usually', 'low', 'a', 'steady', 'state', 'assumption', 'therefore', 'made', 'simplify', 'calculations', 'it', 'assumed', 'radical', 'concentration', 'increases', 'start', 'reaction', 'reaches', 'constant', 'value', 'almost', 'instantly', 'this', 'value', 'maintained', 'rate', 'change', 'free-', 'radical', 'concentration', 'assumed', 'quickly', 'become', 'remain', 'zero', 'polymerization', 'at', 'steady', 'state', 'rates', 'initiation', 'termination', 'equal', 'ri', '¼', 'rt', '¼', '2kim•', '4.', 'this', 'assumption', 'makes', 'possible', 'solve', 'm•', 'expressed', 'the', 'rate', 'propagation', 'the', 'rate', 'propagation', 'approximately', 'equal', 'total', 'rate', 'polymerization', 'the', 'total', 'rate', 'designated', 'rpol', 'because', 'one', 'molecule', 'converted', 'step', 'propagation', 'write', 'this', 'rate', 'propagation', 'applies', 'kinetic', 'chain', 'length', 'large', 'transfer', 'monomer', 'efﬁcient', 'the', 'rate', 'monomer', 'disappearance', 'expressed', '½m(cid3)(cid4)', '¼', 'ðkd½i(cid4)ktþ12', 'rp', '¼', 'kp½m(cid4)ðkd½i(cid4)ktþ12', 'rp', '¼', 'kp', 'rmn', 'ðmþ', 'n', 'h', 'x', '(cid1)', 'd½m(cid4)dt', '¼', 'ri', 'þ', 'rp', '31', 'free-radical', 'chain-growth', 'polymerization', 'process', '71', 'because', 'many', 'molecules', 'monomer', 'involved', 'propagation', 'initiation', 'step', 'close', 'approximation', 'the', 'average', 'lifetime', 'growing', 'radical', 'steady', 'state', 'conditions', 'written', 'follows', '3', '5,', '(cid1)', 'd½m(cid4)dt', '¼', 'rp', '¼', 'kp½m(cid4)2ktðrpþ', 'not', 'primary', 'radicals', 'form', 'attack', 'monomer', 'some', 'lost', 'side', 'reactions', 'an', 'initiator', 'efﬁciency', 'factor', 'f', 'therefore', 'needed', 'it', 'fraction', 'radicals', 'form', 'expressed', 'f', '¼', 'initiating', 'radicalstotal', 'number', 'radicals', 'form', 'the', 'rate', 'initiator', 'decomposition', 'rate', 'equation', 'expressed', 'ri', '¼', '2', 'fkd½i(cid4)', '¼', 'ki', 'rmn(cid3)', '2', 'h', 'x', 'rp', '¼', 'kp½m(cid4)ðkd½i(cid4)f', 'ktþ12', '¼', '(cid1)d½m(cid4)dt', 'according', 'kinetic', 'scheme', 'chain', 'transfer', 'affect', 'rate', 'polymerization', 'alters', 'molecular', 'weight', 'product', 'also', 'important', 'deﬁne', 'average', 'number', 'monomer', 'units', 'consumed', 'per', 'initiation', 'this', 'kinetic', 'chain', 'length', 'equal', 'rate', 'polymerization', 'per', 'rate', 'initiation', 'kinetic', 'chain', 'length', '¼', 'n', '¼', 'kpki', 'at', 'steady', 'state', 'conditions', 'n', 'also', 'equal', 'kpkt', 'the', 'kinetic', 'chain', 'length', 'also', 'expressed', 'by', 'substituting', 'expression', 'm•', 'equation', 'becomes', 'n', '¼', 'kp½m(cid4)2kt½m(cid3)(cid4)', 'n', '¼', 'kpm(cid4)2ð', 'fkdkt½i(cid4)þ12z', 'the', 'number', 'average', 'degree', 'polymerization', 'dp', 'equal', '2', 'n', 'termination', 'takes', 'place', 'coupling', 'it', 'equal', 'n', 'takes', 'place', 'disproportionation', '(terminations', 'coupling', 'disproportionation', 'discussed', 'section', 'termination', 'reactions)', 'above', 'kinetic', 'relationships', 'apply', 'many', 'cases', 'they', 'fail', 'however', 'apply', 'cases', '2.', 'to', 'account', 'several', 'mechanisms', 'advanced', 'they', 'involve', 'modiﬁcations', 'initiation', 'termination', 'propagation', 'steps', 'these', 'beyond', 'discussions', 'book', 'at', 'steady', 'state', 'conditions', 'ri', '¼', 'f', 'i½m(cid4)', '¼', 'rt', '¼', 'kt', 'rmn(cid3)', 'n', 'x', 'h', '2', '72', '3', 'free-radical', 'chain-growth', 'polymerization', '32', 'reactions', 'leading', 'formation', 'initiating', 'free', 'radicals', 'initiating', 'free', 'radical', 'come', 'many', 'sources', 'thermal', 'decompositions', 'compounds', 'azo', 'peroxy', 'groups', 'common', 'sources', 'radicals', 'the', 'radicals', 'also', 'come', '“redox”', 'reactions', 'various', 'light', 'induced', 'decompositions', 'various', 'compounds', 'ionizing', 'radiation', 'also', 'used', 'form', 'initiating', 'radicals', '321', 'thermal', 'decomposition', 'azo', 'compound', 'peroxides', 'the', 'azo', 'compound', 'peroxides', 'contain', 'weak', 'valence', 'bonds', 'structures', 'heating', 'causes', 'weak', 'bonds', 'compounds', 'cleave', 'dissociate', 'free', 'radicals', 'follows', 'for', 'many', 'azo', 'compounds', 'dissociations', 'occur', 'convenient', 'elevated', 'temperatures', 'one', 'commonly', 'used', 'azo', 'compound', 'aa0-azobisisobutyronitrile', 'an', 'original', 'synthesis', 'compound', 'reported', 'follows', '3', '4:', 'r-nn-r', '(cid1)d', '2r(cid3)', 'þ', 'n2', 'o', 'hcn', 'n', 'oh', 'n', '2', 'oh', 'h2n', 'nh2', 'n', 'nh', 'nh', 'n', 'br2', 'n', 'n', 'n', 'n', 'the', 'ﬁnal', 'products', 'decomposition', 'compound', 'two', 'cyanopropyl', 'radicals', 'molecule', 'nitrogen', 'n', 'n', 'n', 'δ', 'n', 'n', 'n', 'n', 'as', 'stated', 'earlier', 'free', 'radicals', 'form', 'however', 'initiate', 'polymerizations', 'some', 'lost', 'side', 'reactions', 'thus', 'instance', 'free', 'radicals', 'form', 'recombine', 'inside', 'outside', 'solvent', 'cage', 'decompositions', 'take', 'place', 'yield', 'either', 'tetramethylsuccinonitrile', 'ketenimine', '4', '5:', 'n', 'n', 'n', '2', 'n', 'n', 'n', 'n2', 'n', '32', 'reactions', 'leading', 'formation', 'initiating', 'free', 'radicals', '73', 'table', '32', 'decomposition', 'rates', 'azonitrile', 'initiatorsa', 'compound', '220-azobisisobutyronitirle', '220-azobis-2-ethylpropionitrile', '220-azobis-2-cyclopropylpropionitrile', '110-azobiscyclohexanenitrile', '220-azobis-2-cyclohexylpropionitrile', '110-azobiscyclooctanenitrile', 'from', 'ref', '13,', 'sources', 'literature', 'b', 'kc', '¼', 'aeð(cid1)eartþ', 'solvent', 'benzene', 'nitrobenzene', 'toluene', 'toluene', 'toluene', 'toluene', 't((cid6)c)', '780', '1000', '500', '800', '800', '450', 'kd', 'b(s(cid1)1)', '8', '(cid7)', '10(cid1)5', '11', '(cid7)', '10(cid1)3', '82', '(cid7)', '10(cid1)5', '65', '(cid7)', '10(cid1)6', '83', '(cid7)', '10(cid1)6', '15', '(cid7)', '10(cid1)4', 'examples', 'fairly', 'efﬁcient', 'azo', 'initiators', 'include', 'following', '1', 'n-nitrosoacylanilides', 'n', 'n', 'o', 'o', 'r', 'δ', 'n', 'o', 'r', 'δ', 'o', 'n', 'o', 'n2', 'o', 'r', '2', 'bromobenzenediazohydroxide', 'br', 'n', 'n', 'oh', 'δ', 'br', 'n2', 'no', '3', 'triphenylazobenzene', 'δ', 'n', '3', 'n', 'n2', '3', 'the', 'triphenylmethyl', 'radical', 'shown', 'resonance', 'stabilized', 'unable', 'initiate', 'polymerizations', 'the', 'phenyl', 'radical', 'hand', 'hot', 'radical', 'it', 'initiates', 'polymerizations', 'readily', 'decomposition', 'rates', 'azonitrile', 'initiators', 'listed', 'table', '32', 'there', 'also', 'many', 'peroxides', 'available', 'initiating', 'free-radical', 'polymerizations', 'these', 'organic', 'inorganic', 'compounds', 'there', 'however', 'many', 'organic', 'peroxides', 'available', 'commercially', 'inorganic', 'ones', 'the', 'organic', 'ones', 'include', 'dialkyl', 'diaryl', 'peroxides', 'alkyl', 'aryl', 'hydroperoxides', 'diacyl', 'peroxides', 'peroxy', 'esters', 'peracids', 'hydrogen', 'peroxide', 'simplest', 'inorganic', 'peroxide', 'syntheses', 'structures', 'chemistry', 'various', 'peroxides', 'described', 'thoroughly', 'literature', '5.', 'here', 'mentioned', 'properties', 'peroxides', 'performance', 'pertain', 'initiations', 'polymerizations', 'decompositions', 'peroxides', 'azo', 'compounds', 'also', 'temperature', 'dependent', '6.', 'this', 'means', 'rates', 'increase', 'temperature', 'the', 'rates', 'also', 'inﬂuenced', 'surrounding', 'medium', 'solvents', 'imprison', '“cage”', 'produced', 'pairs', 'free', 'radicals', 'before', 'undergoing', 'net', 'translational', 'diffusion', 'cage', 'one', '74', '3', 'free-radical', 'chain-growth', 'polymerization', 'table', '33', 'effect', 'substituents', 'decomposition', 'dibenzoylperoxide', 'benzene', '10', '11', 'substituent', 'pp0-dimethoxy', 'p-methoxyy', 'pp0-dimethyl', 'pp0-di-t-butyl', 'parent', 'compound', 'pp0-dichloro', 'mm0-dichloro', 'mm0-dibromo', 'pp0-dicyano', 'ki', '(cid7)', '103', '706', '454', '368', '365', '252', '217', '158', '154', '122', 'log', 'k1k', '0447', '0255', '0164', '0161', '0000', '(cid1)0065', '(cid1)0203', '(cid1)0215', '(cid1)0314', 'si', 's0', '0536', '0268', '0340', '0394', '0000', '0454', '0746', '0782', '1300', 'radicals', 'may', 'may', 'expel', 'small', 'molecule', 'for', 'instance', 'benzoyl', 'peroxide', 'often', 'decompose', 'phenyl', 'radical', 'carbon', 'dioxide', 'follows', 'o', 'o', 'o', 'o', '∆', '2', 'o', 'o', '2', '2co2', 'the', 'resultant', 'phenyl', 'radicals', 'combine', 'yield', 'new', 'completely', 'inactive', 'species', '2', 'the', 'above-described', 'recombination', 'reactions', 'free', 'radicals', 'causes', 'inefﬁciency', 'among', 'initiators', 'the', 'average', 'time', 'recombination', 'free', 'radicals', 'inside', 'solvent', 'cage', 'also', 'time', 'diffusion', 'cage', '10(cid1)10', '7.', 'in', 'addition', 'efﬁciency', 'initiator', 'affected', 'monomer', 'solvent', 'it', 'shown', 'viscosity', 'medium', 'inversely', 'proportional', 'initiator', 'efﬁciency', 'viscous', 'solution', 'greater', 'cage-', 'effect', '8', '9.', 'numerous', 'lists', 'available', 'literature', 'give', 'decomposition', 'temperatures', 'half-', 'lives', 'certain', 'elevated', 'temperatures', 'many', 'initiators', '6.', 'decompositions', 'peroxides', 'may', 'proceed', 'via', 'concerted', 'mechanisms', '10', '11', 'rates', 'structure', 'dependent', 'this', 'illustrated', 'benzoyl', 'peroxide', 'the', 'benzoyl', 'groups', 'two', 'halves', 'molecule', 'dipoles', 'they', 'attached', 'yet', 'repel', 'rupture', 'peroxide', 'link', 'releases', 'electrostatic', 'repulsion', 'two', 'dipoles', 'presence', 'electron', 'donating', 'groups', 'para', 'position', 'increases', 'repulsion', 'lowers', 'decomposition', 'temperature', 'increases', 'decomposition', 'rate', 'the', 'opposite', 'expected', 'electron', 'attracting', 'groups', 'position', '6.', 'the', 'effect', 'substituents', 'rate', 'spontaneous', 'cleavage', 'dibenzoyl', 'peroxide', 'expressed', '11', 'terms', 'hammett', 'equation', 'log', '(kko)', '¼', 'r', 'this', 'shown', 'table', '33', 'in', 'addition', 'peroxides', 'cleave', 'two', 'ways', 'heterolytically', 'homolytically', 'heterolytic', 'cleavage', 'peroxides', 'results', 'formation', 'ions', 'r(cid8)', 'þ', '(cid9)o-o-r0', 'ð', 'r-o-o-r0', 'ð', 'r-o-o(cid9)', 'þ', 'r0(cid8)', 'homolytic', 'cleavage', 'results', 'formation', 'radicals', 'r-o-o-r0', 'ð', 'r-o(cid3)', 'þ', '(cid3)o-r0', '32', 'reactions', 'leading', 'formation', 'initiating', 'free', 'radicals', '75', 'table', '34', 'decomposition', 'benzoyl', 'peroxide', 'various', 'solvents', '798(cid6)c', '15', 'solvent', 'anisole', 'benzene', 'carbon', 'tetrachloride', 'chlorobenzene', 'chloroform', 'cyclohexane', 'cyclohexene', 'ethyl', 'acetate', 'ethylbenzene', 'methyl', 'benzoate', 'methylene', 'chloride', 'nitrobenzene', 'tetrachloroethylene', 'toluene', 'approximate', 'decomposition', '4', 'h', '430', '500', '400', '490', '440', '840', '400', '850', '460', '410', '620', '490', '350', '500', 'in', 'gaseous', 'phase', 'cleavage', 'usually', 'homolytic', 'requires', 'least', 'amount', 'energy', '12.', 'in', 'solution', 'however', 'dissociation', 'may', 'either', 'one', 'two', 'depending', 'upon', 'nature', 'r', 'groups', 'heterolytic', 'cleavage', 'may', 'favored', 'cases', 'two', 'groups', 'r', 'r0', 'differ', 'electron', 'attraction', 'the', 'true', 'reaction', 'solvent', 'high', 'dielectric', 'constant', 'solvation', 'ions', 'would', 'form', 'due', 'heterolytic', 'cleavage', 'also', 'promoting', 'inﬂuence', 'cleavage', 'ab', '2s', 'sa(cid9)', 'bs(cid8)', 's', 'represents', 'solvent', 'in', 'sum', 'total', 'types', 'amounts', 'side', 'reactions', 'take', 'place', 'function', 'structures', 'peroxides', 'stability', 'formed', 'radicals', 'solvent', 'monomer', 'polymerized', 'the', 'stability', 'radicals', 'form', 'also', 'affect', 'amount', 'radicals', 'captured', 'monomers', 'also', 'reported', 'generally', 'character', 'free', 'radicals', 'neutral', 'electrophilic', '(such', 'chloro)', 'others', 'nucleophilic', '(such', 't-butyl)', 'this', 'tendency', 'however', 'relatively', 'slight', 'compared', 'positive', 'negative', 'ions', '15.', 'there', 'much', 'information', 'literature', 'rates', 'manner', 'decomposition', 'many', 'peroxides', 'various', 'media', 'beyond', 'diagnostic', 'tests', 'exist', 'aid', 'determining', 'decomposition', 'rates', 'particular', 'peroxide', 'particular', 'media', '13.', 'table', '34', 'presented', 'show', 'different', 'solvents', 'affect', 'rate', 'decomposition', 'benzoyl', 'peroxide', 'radicals', 'some', 'initiators', 'function', 'thermal', 'photoinitiators', 'such', 'initiator', 'instance', '220-azobisisobutyronitrile', 'also', 'engel', 'coworkers', '16', 'reported', 'synthesis', 'initiator', 'function', 'thermal', 'free', 'radical', 'initiator', 'photoinitiator', '(see', 'sect', '324)', 'it', 'illustrated', 'follows', 'n', 'n', 'o', 'o', 'o', 'the', 'claimed', 'advantage', 'initiator', 'used', 'form', 'block', 'copolymers', '76', '3', 'free-radical', 'chain-growth', 'polymerization', '322', 'bimolecular', 'initiating', 'systems', 'decompositions', 'peroxides', 'initiating', 'radicals', 'also', 'possible', 'bimolecular', 'reactions', 'involving', 'electron', 'transfer', 'mechanisms', 'such', 'reactions', 'often', 'called', 'redox', 'initiations', 'illustrated', 'follows', 'a', 'electron', 'donor', 'þ', 'r-o-o-r0', 'electron', 'acceptor', '(cid1)', 'a(cid8)', 'þ', 'ro(cid3)', 'þ(cid9)or0', 'a', 'reducing', 'agent', 'roor0', 'peroxide', 'the', 'illustrated', 'decomposition', 'persulfate', '(an', 'inorganic', 'peroxide)', 'ferrous', 'ion', 'so', 'o', 'o', 's', 'o', 'fe', 'so', 'o', 'so', 'o', 'fe', 'o', 'o', 'o', 'o', 'electron', 'acceptor', 'electron', 'donor', 'initiating', 'ion-radical', 'side', 'reactions', 'possible', 'presence', 'sufﬁcient', 'quantities', 'reducing', 'ions', 'o', 'o', 'fe', 'so4', 'fe', 'o', 'o', 'o', 'o', 'o', 's', 'o', 'a', 'redox', 'reaction', 'also', 'take', 'place', 'peroxide', 'electron', 'acceptor', 'r-o-o-h', 'ce', 'r-o-o', 'ce', 'h', 'side', 'reactions', 'excess', 'ceric', 'ion', 'occur', 'well', 'r-o-o', 'ce', 'r', 'o2', 'ce', 'another', 'example', 'redox', 'reaction', 't-butyl', 'hydroperoxide', 'cobaltous', 'ion', '17:', 'the', 'cobaltic', 'ion', 'forms', 'act', 'electron', 'acceptor', 'co', 'o', 'h', 'o', 'co', 'oh', 'the', 'cobaltic', 'ion', 'forms', 'act', 'electron', 'acceptor', 'co', 'o', 'h', 'o', 'co', 'o', 'h', 'o', 'o', '32', 'reactions', 'leading', 'formation', 'initiating', 'free', 'radicals', '77', 'side', 'reactions', 'occur', 'o', 'o', 'o', 'o', 'h', 'o', 'o2', 'oh', 'o', 'co', 'o', 'co', 'nevertheless', 'cobaltous', 'ions', 'form', 'efﬁcient', 'redox', 'initiating', 'systems', 'peroxydisulfate', 'ions', '18.', 'tertiary', 'aromatic', 'amines', 'also', 'participate', 'bimolecular', 'reactions', 'organic', 'peroxides', 'one', 'unpaired', 'electrons', 'nitrogen', 'atom', 'transfers', 'peroxide', 'link', 'inducing', 'decomposition', 'no', 'nitrogen', 'however', 'found', 'polymer', 'it', 'therefore', 'true', 'redox', 'type', 'initiation', 'amine', 'acts', 'like', 'promoter', 'decomposition', '19.', 'two', 'mechanisms', 'proposed', 'explain', 'reaction', 'the', 'ﬁrst', 'one', 'offered', 'horner', 'et', 'al', '19:', 'ho', 'o', 'ho', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', '78', '3', 'free-radical', 'chain-growth', 'polymerization', 'a', 'second', 'mechanism', 'proposed', 'imoto', 'choe', '20', 'shows', 'complex', 'intermediate', 'step', 'n', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'the', 'dimethyl', 'aniline', 'radical-cation', 'shown', 'undergoes', 'reactions', 'addition', 'monomer', 'the', 'benzoyl', 'radical', 'one', 'initiates', 'polymerizations', 'presence', 'electron-releasing', 'substituents', 'diethyl', 'aniline', 'increases', 'rate', 'reaction', 'benzoyl', 'peroxide', '21.', 'this', 'suggests', 'lone', 'pair', 'electrons', 'nitrogen', 'attack', 'positively', 'charged', 'oxygen', 'peroxide', 'link', '22.', 'by', 'comparison', 'peroxides', 'azo', 'compounds', 'generally', 'susceptible', 'chemically', 'induced', 'decompositions', 'it', 'shown', '23,', 'however', 'possible', 'accelerate', 'decomposi-', 'tion', 'aa0-azobisisobutyronitrile', 'reacting', 'bis(-)-ephedrine-copper', '(ii)', 'chelate', 'the', 'mechanism', 'postulated', 'involve', 'reductive', 'decyanation', 'azobisisobutyronitrile', 'coor-', 'dination', 'chelate', '23.', 'initiations', 'polymerizations', 'vinyl', 'chloride', 'styrene', 'aa0-', 'azobisisobutyronitrile', 'coupled', 'aluminum', 'alkyls', 'investigated', '24.', 'gas', 'evolution', 'measurements', 'indicated', 'accelerated', 'decomposition', 'also', 'additions', 'large', 'amounts', 'tin', 'tetrachloride', 'either', 'aa0-azobisisobutyronitrile', 'dimethyl-aa0-azobisisobutyrate', 'increase', 'decomposition', 'rates', '25.', 'molar', 'ratios', 'sncl4/aibn', '¼', '2165', 'sncl4/maib', '¼', '1953', 'increase', 'rates', 'factors', '45', '17', 'respectively', 'decomposition', 'rates', 'also', 'enhanced', 'donor', 'solvents', 'ethyl', 'acetate', 'propionitrile', 'presence', 'tin', 'tetrachloride', '25.', '32', 'reactions', 'leading', 'formation', 'initiating', 'free', 'radicals', '79', 'a', 'bimolecular', 'initiating', 'system', 'based', '220-azobisisobutyronitrile', 'reported', 'michl', 'coworkers', '26.', 'it', 'consists', 'weakly', 'solvated', 'lithium', 'combination', 'cyanopropyl', 'radical', '(from', 'aibn)', 'the', 'combination', 'initiate', 'polymerizations', 'oleﬁns', 'the', 'reaction', 'illustrated', 'follows', 'r', 'li', 'r', 'cn', 'r', 'li', 'r', 'cn', 'n', '323', 'boron', 'metal', 'alkyl', 'initiators', 'free-radical', 'polymerizations', 'these', 'initiators', 'originally', 'reported', 'long', 'time', 'ago', '27–29.', 'oxygen', 'plays', 'important', 'role', 'reactions', '30', '31.', 'it', 'reacts', 'alkyl', 'boride', 'mild', 'conditions', 'form', 'peroxides', '32', '33:', 'br', 'oo', 'br', 'o', 'o', 'r', 'r', 'r', 'r', 'initiating', 'radicals', 'apparently', 'come', 'reactions', 'peroxides', 'molecules', 'boron', 'alkyls', '34', '35.', 'one', 'postulated', 'reaction', 'mechanism', 'illustrated', 'follows', '35:', 'rb', 'ob', 'r', 'r', 'ob', 'or', 'o', 'b', 'another', 'suggested', 'reaction', 'path', '36:', 'r2b-o-o-r', 'þ', '2r0', '3b', '2r(cid3)', 'þ', 'r0', '2b-o-br0', '2', 'þ', 'r0', '2b-o-r', 'catalytic', 'action', 'oxygen', 'observed', 'various', 'organometallic', 'compounds', '35.', 'one', 'example', 'dialkylzinc', '37', 'probably', 'forms', 'active', 'peroxide', '38.', 'the', 'also', 'true', 'dialkylcadmium', 'triethylaluminum', '38.', 'peroxide', 'formation', 'believed', 'important', 'step', 'initiations', 'initiating', 'radicals', 'however', 'appear', 'produced', 'mere', 'decompositions', 'peroxides', '35.', '324', 'photochemical', 'initiators', 'this', 'subject', 'discussed', 'greater', 'detail', 'chap', '10', 'section', 'photo-cross-linking', 'reactions', 'coatings', 'ﬁlms', 'a', 'brief', 'explanation', 'also', 'offered', 'initiations', 'used', 'limited', 'scale', 'conventional', 'preparation', 'polymers', '80', '3', 'free-radical', 'chain-growth', 'polymerization', 'many', 'organic', 'compounds', 'decompose', 'cleave', 'radicals', 'upon', 'irradiation', 'light', 'appropriate', 'wavelength', '38', '39.', 'because', 'reactions', 'strictly', 'light', 'heat', 'induced', 'possible', 'carry', 'polymerizations', 'low', 'temperatures', 'in', 'addition', 'employing', 'narrow', 'wavelength', 'bands', 'excite', 'photoinitiators', 'possible', 'stop', 'reaction', 'merely', 'blocking', 'light', 'among', 'compounds', 'decompose', 'readily', 'peroxides', 'azo', 'compounds', 'disulﬁde', 'ketones', 'aldehydes', 'a', 'photodecomposition', 'disulﬁde', 'illustrated', 'follows', 's', 's', 'hν', '2', 's', 'today', 'many', 'commercially', 'prepared', 'photoinitiators', 'available', 'some', 'consists', 'aromatic', 'ketones', 'cleave', 'norrish', 'reaction', 'photoreduced', 'form', 'free', 'radicals', 'there', 'also', 'numerous', 'two', 'three', 'component', 'photoinitiating', 'systems', 'there', 'also', 'decom-', 'pose', 'irradiation', 'visible', 'light', 'make', 'possible', 'initiate', 'reactions', 'longer', 'wavelength', 'light', '(see', 'chap', '10)', 'some', 'examples', 'various', 'photoinitiators', 'given', 'chap', '10', 'many', 'others', 'found', 'literature', 'as', 'example', 'sited', 'work', 'barner-kowollik', 'coworkers', 'studied', 'photoinitiation', 'process', 'methyl', 'methacrylate', 'polymerization', 'using', 'high-resolution', 'electro', 'spray-', 'mass', 'spectrometry', '40.', 'the', 'polymerization', 'conducted', 'using', 'pulsed', 'laser', 'temperatures', '(cid10)0(cid6)c', 'presence', 'photoinitiators', '22-dimethoxy-2-phenylacetophenone', 'benzoin', 'benzil', 'benzoin', 'ethyl', 'ether', '22-azobisisobutylnitrile', 'they', 'identiﬁed', 'termination', 'products', 'combination', 'disproportionation', 'high', 'accuracy', 'both', 'benzoyl', 'acetal', 'fragments', 'generated', 'result', '22-dimethoxy-2-phenylacetophenone', 'photocleavage', 'found', 'initiate', 'highly', 'likely', 'terminate', 'polymerization', 'both', 'benzoyl', 'ether', 'fragments', 'produced', 'result', 'benzoin', 'photocleavage', 'found', 'act', 'initiating', 'probable', 'terminating', 'species', 'indicating', 'ether', 'radical', 'fragment', 'act', 'exclusively', 'terminating', 'species', '325', 'initiation', 'polymerization', 'radioactive', 'sources', 'electron', 'beams', 'different', 'radioactive', 'sources', 'initiate', 'free-radical', 'polymerizations', 'vinyl', 'monomers', 'they', 'emitters', 'gamma', 'rays', 'beta', 'rays', 'alpha', 'particles', 'most', 'useful', 'strong', 'gamma', 'emitters', '60co', '90sr', 'electron', 'beams', 'electrostatic', 'accelerators', 'also', 'efﬁcient', 'initiators', 'the', 'products', 'irradiation', 'radioactive', 'sources', 'electron', 'beams', 'similar', 'identical', 'products', 'irradiation', 'ultraviolet', 'light', 'irradiation', 'ionizing', 'radiation', 'causes', 'excited', 'monomer', 'molecules', 'decompose', 'free', 'radicals', 'ionic', 'species', 'also', 'form', 'initial', 'electron', 'captures', 'no', 'sensitizers', 'extraneous', 'initiating', 'materials', 'required', 'it', 'commonly', 'accepted', 'free', 'radicals', 'ions', 'initial', 'products', 'act', 'intermediate', 'species', 'reactions', 'there', 'still', 'insufﬁcient', 'information', 'however', 'exact', 'nature', 'species', '38', '39.', 'the', 'polymerizations', 'predominantly', 'free-radical', 'mechanism', 'monomers', 'ionic', 'one', 'others', '38', '39.', '33', 'capture', 'free', 'radicals', 'monomers', 'once', 'initiating', 'radical', 'formed', 'competition', 'addition', 'monomer', 'possible', 'secondary', 'reactions', 'a', 'secondary', 'reaction', 'recombination', 'fragments', 'shown', 'caused', 'cage', 'effect', 'solvent', 'molecules', '41.', 'other', 'reactions', 'take', '33', 'capture', 'free', 'radicals', 'monomers', '81', 'place', 'radical', 'parent', 'initiator', 'molecule', 'this', 'lead', 'formation', 'different', 'initiating', 'species', 'it', 'however', 'also', 'dead', 'end', 'far', 'polymerization', 'reaction', 'concerned', 'after', 'initiating', 'radical', 'diffused', 'proximity', 'monomer', 'capture', 'free', 'radical', 'monomer', 'completes', 'step', 'initiation', 'this', 'straightforward', 'addition', 'reaction', 'subject', 'steric', 'effects', 'r', 'r1', 'r2', 'r3', 'r3', 'r3', 'r3', 'r3', 'r3', 'r1', 'r', 'r3', 'r3', 'r3', 'the', 'unpaired', 'electron', 'radical', 'believed', 'pure', 'p-orbital', 'planar', 'sp2', 'carbon', 'atom', 'occasionally', 'however', 'radicals', 'sp3', 'conﬁguration', 'appear', 'form', '42–44.', 'using', 'quantum', 'chemical', 'calculations', 'demonstrated', 'nucleophilic', 'electrophilic', 'alkyl', 'aryl', 'radicals', 'attack', 'alkenes', 'following', 'tetrahedral', 'trajectory', '45', '46:', 'rx', 'h', 'y', 'z', 'h', 'this', 'means', 'substituents', 'y', 'attacked', 'oleﬁnic', 'carbon', 'exert', 'large', 'steric', 'effects', '47.', 'in', 'addition', 'steric', 'effects', 'rates', 'addition', 'strongly', 'nucleophilic', 'electrophilic', 'radicals', 'governed', 'mainly', 'polar', 'effects', 'substituents', 'rx', 'y', 'z', '48.', 'in', 'borderline', 'cases', 'however', 'stabilities', 'adducts', 'products', 'tend', 'dominate', '47.', 'also', 'demonstrated', 'acyclic', 'radicals', 'react', 'high', 'stereoselectivity', '45.', 'in', 'order', 'reactions', 'stereoselective', 'radicals', 'adopt', 'preferred', 'conformations', 'two', 'faces', 'prochiral', 'radical', 'centers', 'shielded', 'different', 'extents', 'stereogenic', 'centers', 'giese', 'coworkers', '49', 'demonstrated', 'help', 'electron', 'spin', 'resonance', 'studies', 'ester-', 'substituted', 'radicals', 'stereogenic', 'centers', 'b-positions', 'adopt', 'preferred', 'conformations', 'mini-', 'mize', 'allylic', 'strain', '49', '(shown', 'below)', 'in', 'conformations', 'large', '(l)', 'medium', 'sized', 'substituents', '(m)', 'shield', 'two', 'faces', 'the', 'attacks', 'come', 'preferentially', 'less', 'shielded', 'sides', 'radicals', 'stereoselectivity', 'a-strain', 'conformation', 'limited', 'ester-substituted', 'radicals', '50.', 'the', 'strains', 'steric', 'control', 'reactions', 'radicals', 'alkenes', 'illustrated', 'follows', '50:', 'r', 'n', 'r', 'c', 'h', 'c', 'o', 'c', 'or', 'o', 'c', 'c', 'h', 'l', 'm', 'auxilliary', 'control', 'a-strain', 'substrate', 'control', 'c', 'm', 'or', 'h', 'l', 'cram', 'felkin-anh', 'the', 'considerations', 'illustrated', 'initiation', 'benzoyl', 'peroxide', 'commonly', 'used', 'initiating', 'compound', 'the', 'half-life', 'initial', 'benzoyloxy', 'radicals', 'decompositions', 'benzoyl', '82', '3', 'free-radical', 'chain-growth', 'polymerization', 'peroxide', 'estimated', '10(cid1)4', '10(cid1)5', 'past', 'time', 'decompose', 'phenyl', 'radicals', 'carbon', 'dioxide', '49.', 'this', 'sufﬁcient', 'time', 'benzoyloxy', 'radicals', 'trapped', 'fast-reacting', 'monomers', 'slow-reacting', 'monomers', 'however', 'likely', 'react', 'phenyl', 'radicals', 'form', 'elimination', 'reaction', 'in', 'effect', 'two', 'competing', 'reactions', '50:', '1', 'decomposition', 'free', 'radical', 'o', 'k', 'o', 'co2', '2', 'two', 'types', 'radicals', 'add', 'monomer', '(where', 'x', 'represents', 'typical', 'substituent', 'vinyl', 'monomers', 'halogens', 'esters', 'aromatic', 'groups', 'nitriles', 'etc)', 'o', 'o', 'x', 'k', 'o', 'o', 'x', 'x', 'k"', 'x', 'the', 'ratio', 'rates', 'two', 'reactions', 'k00k0', '(or', 'k000k0)', 'depends', 'upon', 'reactivity', 'monomers', 'it', 'shown', 'table', '35', '51', '52.', 'the', 'benzoyloxy', 'radical', 'used', 'table', 'illustration', 'a', 'similar', 'comparison', 'possible', 'redox', 'initiating', 'system', 'an', 'initiating', 'sulfate', 'radical', 'ion', 'persulfate', 'initiator', 'react', 'another', 'reducing', 'ion', 'add', 'monomer', 'so4', 'fe', 'so4', 'fe', 'k', 'so4', 'x', 'k', 'x', 'so4', 'in', 'table', '36', 'shown', 'relative', 'reaction', 'rates', 'so4•', 'monomers', '25(cid6)c', '53–57.', 'as', 'explained', 'rate', 'addition', 'radical', 'double', 'bond', 'affected', 'steric', 'hindrance', 'bulky', 'substituents', 'polar', 'effect', 'dipole', 'interactions', 'also', 'inﬂuence', 'rate', 'addition', 'forbes', 'yashiro', 'studied', 'addition', 'initiating', 'radials', 'methyl', 'methacrylate', 'liquid', 'supercritical', 'carbon', 'dioxide', '58.', 'they', 'demonstrated', 'rate', 'addition', 'initiating', 'radicals', 'monomers', 'kadd', 'values', 'measured', 'liquid', 'co2', 'phenyl', 'methyl', 'groups', 'located', 'carbon', 'atom', 'direct', 'attack', 'free', 'radical', 'expected', 'interfere', 'sterically', 'approach', 'for', 'instance', 'due', 'steric', 'hindrance', 'trans-', 'b-methylstilbene', 'reactive', 'toward', 'radical', 'attack', 'cis', 'isomer', '58.', 'yet', 'trans', 'isomer', 'stable', 'two', 'while', '11-disubstituted', 'oleﬁns', 'homopolymerize', 'readily', '12-', 'disubstituted', 'oleﬁns', 'hard', 'homopolymerize', '59.', 'some', 'exceptions', 'vinyl', 'carbonate', '61', 'maleimide', 'derivatives', '62.', 'also', 'perﬂuoroethylene', 'chlorotriﬂuoroethylene', 'polymerize', 'readily', 'table', '36', 'shows', 'relative', 'reaction', 'rates', 'so4•', 'monomers', '25(cid6)c', '50.', '33', 'capture', 'free', 'radicals', 'monomers', '83', 'table', '35', 'relative', 'reactivities', 'benzoyloxy', 'radical', '60(cid6)c', 'monomer', 'structure', 'acrylonitrile', 'k0k00', '(moll)', '012', 'methyl', 'methacrylate', 'vinyl', 'acetate', 'styrene', '25-dimethylstyrene', 'n', 'och3', 'o', 'o', 'o', '030', '091', '250', '50', 'addition', 'monomerdecomposition', 'rates', '51', '52', 'table', '36', 'relative', 'rates', 'reactions', 'sulfate', 'ion-radical', 'monomers', '51', '52', 'monomer', 'methyl', 'methacrylate', 'relative', 'rates', 'k00k0', '77', '(cid7)', '10(cid1)3', 'structure', 'och3', 'methyl', 'acrylate', 'och3', '11', '(cid7)', '10(cid1)3', 'acrylonitrile', '39', '(cid7)', '10(cid1)4', 'o', 'o', 'n', 'homopolymerizations', 'diethyl', 'fumarate', 'free-radical', 'mechanism', 'reported', '63.', 'the', 'mn', 'found', '15000', 'the', 'true', 'homopolymerizations', 'several', 'dialkyl', 'fumarates', 'also', 'dialkyl', 'maleates', '64–66.', 'the', 'polymerization', 'rates', 'sizes', 'polymers', 'form', 'decrease', 'increases', 'lengths', 'linear', 'alkyl', 'ester', 'groups', 'there', 'however', 'opposite', 'correlation', 'ester', 'groups', 'branched', 'also', 'maleate', 'esters', 'appear', 'isomerize', 'fumarates', 'prior', 'polymerization', '66.', '84', '3', 'free-radical', 'chain-growth', 'polymerization', '34', 'propagation', 'the', 'transition', 'state', 'propagation', 'reaction', 'illustrated', 'follows', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'in', 'transition', 'state', 'macroradical', 'electron', 'localized', 'terminal', 'carbon', 'also', 'two', 'p', 'electrons', 'double', 'bond', 'localized', 'oleﬁnic', 'carbon', 'interaction', 'takes', 'place', 'p-orbital', 'terminal', 'atom', 'active', 'polymer', 'chain', 'associated', 'carbon', 'monomer', 'this', 'results', 'formation', 's-bonds', '67.', 'the', 'rate', 'propagation', 'reaction', 'depends', 'upon', 'reactivity', 'monomer', 'growing', 'radical', 'chain', 'steric', 'factors', 'polar', 'effects', 'resonance', 'also', 'important', 'factors', 'reaction', 'another', 'factor', 'affect', 'rate', 'propagation', 'interaction', 'propagating', 'radicals', 'siegmann', 'beuermann', '60', 'studied', 'rate', 'propagation', '1h1h2h2h-tridecaﬂuorooctyl', 'methacrylate', 'compared', 'rate', 'propagation', 'methyl', 'methacrylate', 'they', 'observed', 'kp', '1h1h2h2h-tridecaﬂuorooctyl', 'methacrylate', 'polymerization', '19', 'times', 'kp', 'methyl', 'methacrylate', 'they', 'concluded', 'higher', 'rate', 'due', 'less', 'interactions', 'occurring', 'propagating', 'macroradicals', 'bowman', 'coworkers', 'studied', 'impact', 'intermolecular', 'intramolecular', 'interactions', 'polymerization', 'kinetics', 'monoacrylates', '69.', 'they', 'carried', 'polymerization', 'studies', 'presence', 'extensive', 'amounts', 'solvent', 'this', 'attempt', 'elucidate', 'effects', 'intermolec-', 'ular', 'interactions', 'bulk', 'medium', 'polarity', 'p–p', 'stacking', 'hydrogen', 'bonding', 'characterize', 'contribution', 'intramolecular', 'conformational', 'effects', 'monomer', 'reactivity', 'solution', 'polymeri-', 'zation', 'kinetics', 'various', 'monomers', 'measured', 'presence', '95', 'wt', '14-dioxane', 'the', 'results', 'compared', 'bulk', 'polymerization', 'kinetics', 'the', 'studies', 'revealed', 'aliphatic', 'acrylates', 'hexyl', 'acrylate', 'exhibit', 'approximately', 'two', 'threefold', 'reduction', 'reactivity', 'upon', 'dilution', 'monomers', 'characterized', 'hydrogen-bonding', 'features', 'hydroxyethyl', 'acrylate', 'exhibit', '8-', '12-fold', 'reduction', 'upon', 'dilution', 'monomers', 'possessing', 'aromatic', 'ring', 'stacking', 'interactions', 'phenyl', 'acrylate', 'exhibit', 'approximately', 'ﬁve', 'tenfold', 'reduction', 'upon', 'dilution', 'similar', 'conditions', 'even', 'concentration', '5', 'wt', 'monomer', '14-dioxane', 'approximately', 'two', 'ﬁvefold', 'differences', 'reactivity', 'observed', 'various', 'acrylates', 'bowman', 'coworkers', 'attributed', 'reactivity', 'differences', 'various', 'acrylates', 'upon', 'extensive', 'dilution', 'solely', 'intramolecular', 'interactions', '69.', '341', 'steric', 'polar', 'resonance', 'effects', 'propagation', 'reaction', 'the', 'steric', 'effects', 'depend', 'upon', 'sizes', 'substituents', 'the', 'resonance', 'stabilization', 'substituents', 'shown', 'following', 'order', '70:', 'o', 'o', 'cl', 'o', 'och3', 'n', '34', 'propagation', '85', 'table', '37', 'afﬁnity', 'methyl', 'radical', 'oleﬁnsa', '56', '57', '67', 'monomer', 'structure', 'styrene', 'methyl', 'afﬁnity', '792', 'a-methyl', 'styrene', 'cis-b-methyl', 'styrene', 'trans-b-methyl', 'styrene', 'ab-dimethyl', 'styrene', 'abb0-trimethyl', 'styrene', '926', '40', '925', '66', '20', 'from', 'carrick', 'szwarc', 'leavitt', 'levy', 'stannett', 'permission', 'american', 'chemical', 'society', 'the', 'reactivities', 'propagating', 'polymer-radicals', 'however', 'exert', 'greater', 'inﬂuence', 'rates', 'propagation', 'reactivities', 'monomers', 'resonance', 'stabilization', 'polymer-', 'radicals', 'predominant', 'factor', 'this', 'fairly', 'common', 'view', 'comes', 'observations', 'methyl', 'radical', 'reacts', 'temperature', '60(cid6)c', 'approximately', '25', 'times', 'faster', 'styrene', 'vinyl', 'acetate', '72.', 'in', 'homopolymerizations', 'two', 'monomers', 'however', 'rates', 'propagation', 'fall', 'opposite', 'order', 'also', 'poly(vinyl', 'acetate)-radicals', 'react', '46', 'times', 'faster', 'n-butyl', 'mercaptan', 'hydrogen', 'abstraction', 'reactions', 'polystyrene-radicals', '71.', 'the', 'conclusion', 'polystyrene', 'radicals', 'much', 'resonance', 'stabilized', 'poly', '(vinyl', 'acetate)-radicals', 'several', 'structures', 'polystyrene-radicals', 'possible', 'due', 'conju-', 'gation', 'unpaired', 'electrons', 'terminal', 'carbons', 'adjacent', 'unsaturated', 'groups', 'these', 'resonance', 'hybrids', 'illustrated', 'follows', 'there', 'opportunity', 'however', 'resonance', 'stabilization', 'poly(vinyl', 'acetate)', 'radicals', 'oxygen', 'accommodate', 'eight', 'electrons', 'the', 'effect', 'steric', 'hindrance', 'afﬁnity', 'methyl', 'radical', 'illustrated', 'table', '37', '56', '57.', '86', '3', 'free-radical', 'chain-growth', 'polymerization', 'in', 'vinyl', 'monomers', 'oleﬁnic', 'carbons', 'potentially', 'subject', 'free-radical', 'attack', 'each', 'would', 'give', 'rise', 'different', 'terminal', 'unit', 'r', 'x', 'r', 'r', 'x', 'x', 'the', 'newly', 'formed', 'radicals', 'potentially', 'react', 'next', 'monomer', 'two', 'ways', 'this', 'means', 'four', 'propagation', 'reactions', 'occur', '1', '2', '3', '4', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'contrary', 'shown', 'four', 'propagation', 'modes', '“head', 'tail”', 'placement', 'shown', '(31)', 'strongly', 'predominates', 'this', 'true', 'free', 'radical', 'vinyl', 'polymerizations', 'it', 'consistent', 'localized', 'energy', 'a-carbon', 'monomer', 'also', 'calculations', 'resonance', 'stabilization', 'tend', 'predict', 'head', 'tail', 'additions', '68.', 'the', 'free-radical', 'propagation', 'reactions', 'correspond', 'conversions', 'double', 'bonds', 'single', 'bonds', 'strongly', 'exothermic', 'in', 'addition', 'rates', 'increase', 'temperature', 'it', 'often', 'assumed', 'viscosity', 'medium', 'change', 'viscosity', 'polymerization', 'reaction', 'affect', 'propagation', 'rate', 'polymer', 'growth', 'reaction', 'this', 'involves', 'diffusion', 'small', 'monomer', 'molecules', 'reactive', 'sites', 'small', 'molecules', 'however', 'also', 'impeded', 'process', 'diffusion', 'this', 'impede', 'growth', 'rate', '50.', 'during', 'chain', 'growth', 'radical', 'great', 'deal', 'freedom', 'little', 'steric', 'control', 'manner', 'monomer', 'placement', 'decrease', 'reaction', 'temperature', 'however', 'lowers', 'mobility', 'species', 'increases', 'steric', 'control', 'placement', 'this', 'accompanied', 'increase', 'stereoregularity', 'product', '70', '71.', 'the', 'preferred', 'placement', 'trans–trans', 'lower', 'energy', 'required', 'placement', 'as', 'result', 'certain', 'amount', 'syndiotactic', 'arrangement', 'observed', 'polymerizations', 'lower', 'temperatures', '72.', 'trans–trans', 'conﬁgurations', '(with', 'respect', 'carbon', 'atoms', 'chains)', 'yield', 'zigzag', 'backbones', 'this', 'predicted', 'observations', 'steric', 'effects', 'small', 'molecules', '74', '75.', 'it', 'conﬁrmed', 'experimentally', 'many', 'polymers', 'instance', 'formation', 'poly(12-polybutadiene)', '74', 'poly(vinyl', 'chloride)', '72.', 'also', 'free-radical', 'polymerizations', 'methyl', 'methacrylate', 'syndiotactic', 'placement', 'becomes', 'increas-', 'ingly', 'dominant', 'lower', 'temperatures', 'conversely', 'randomness', 'increases', 'higher', 'temperatures', '74.', 'the', 'true', 'free-radical', 'polymerization', 'halogenated', 'vinyl', 'acetate', '75.', 'one', 'proposed', 'mechanism', 'follows', 'the', 'least', 'amounts', 'steric', 'compression', 'within', 'macromolecules', 'occur', 'growth', 'reactions', 'ultimate', 'penultimate', 'units', '34', 'propagation', '87', 'trans', 'also', 'lone', 'electrons', 'face', 'oncoming', 'monomers', 'transition', 'states', '80', '81,', 'shown', 'syndiotactic', 'placement', 'favored', 'ch2', 'cooch3', 'h', 'h', 'cooch3', 'cooch3', 'transition', 'state', 'cooch3', 'h', 'h', 'h', 'h', 'cooch3', 'cooch3', 'while', 'model', 'explains', 'formation', 'syndiotactic', 'poly(methyl', 'methacrylate)', 'possible', 'interactions', 'free', 'radicals', 'chain', 'ends', 'monomers', 'considered', 'such', 'interactions', 'however', 'dominant', 'factor', 'syndiotactic', 'placement', 'terminal', 'carbons', 'sp2', 'planar', 'structures', '75.', '342', 'effect', 'reaction', 'medium', 'there', 'early', 'reports', 'reaction', 'media', 'inﬂuences', 'polymerizations', 'vinyl', 'chloride', 'aliphatic', 'aldehydes', '50(cid6)c', '80', '81.', 'this', 'conﬁrmed', 'subsequent', 'studies', '82–84.', 'subsequently', 'rate', 'polymerization', 'shown', 'inﬂuenced', 'ph', 'reaction', 'medium', 'polymerizations', 'monomers', 'methacrylic', 'acid', '(maa)', '85', '88.', 'also', 'rate', 'polymerization', 'solution', 'viscosities', 'increase', 'polymerizations', 'acrylamide', 'acrylic', 'acid', 'increase', 'water', 'concentration', '83.', 'it', 'quite', 'clear', 'whether', 'due', 'increases', 'speeds', 'propagations', 'due', 'decreases', 'termination', 'rates', 'in', 'free-radical', 'polymerization', 'vinyl', 'benzoate', 'rate', 'propagation', 'varies', 'different', 'solvents', 'following', 'order', '88', '89:', 'n', 'o', 'c2h5', 'o', 'cl', 'o', 'o', 'similarly', 'rate', 'photopolymerization', 'vinyl', 'acetate', 'affected', 'solvents', '88.', 'in', 'cases', 'however', 'rate', 'polymerization', 'proportional', 'square', 'root', 'initiator', 'concen-', 'tration', 'concentration', 'monomer', '5.', 'de', 'sterck', 'coworkers', '90', 'studied', 'solvent', 'effect', 'tacticity', 'methyl', 'methacrylate', 'free-', 'radical', 'polymerization', 'they', 'observed', 'solvents', 'ch3oh', '(cf3)3coh', 'h-bonded', 'carbonyl', 'oxygens', 'located', 'side', 'backbone', 'growing', 'polymer', 'radical', 'hinder', 'formation', 'isotactic', 'poly(methyl', 'methacrylate)', 'extent', 'methanol', 'less', 'effective', 'reducing', 'isotacticity', 'small', 'size', 'also', 'relatively', 'loose', 'hydrogen', 'bonds', 'carbonyl', 'oxygens', 'there', 'controversial', 'suggestion', 'solvent', 'affects', 'propagation', 'step', 'reactions', 'forming', '“hot”', 'radicals', '91.', 'these', 'radicals', 'supposed', 'possess', 'higher', 'amounts', 'energy', 'at', 'moment', 'formation', 'obtain', 'surplus', 'energy', 'heat', 'reaction', 'activation', 'energy', 'propagation', 'reaction', 'this', 'claimed', 'provide', 'extra', 'energy', 'needed', 'activate', 'next', 'chain', 'propagation', 'step', 'the', 'surplus', 'energy', 'may', 'affect', 'polymerization', 'kinetics', 'average', 'lifetime', 'hot', 'radicals', 'sufﬁcient', 'react', 'monomer', 'molecules', 'this', 'surplus', 'energy', 'lost', 'hot', 'radicals', 'collisions', 'monomer', 'solvent', 'molecules', '88', '3', 'free-radical', 'chain-growth', 'polymerization', 'there', 'difference', 'rate', 'constants', 'propagation', 'hot', 'ordinary', 'radicals', 'two', 'different', 'reaction', 'schemes', 'written', '91:', 'propagation', 'ordinary', 'radicals', 'propagation', 'hot', 'radicals', 'energy', 'transfer', 'processes', 'r(cid3)', 'þ', 'm', 'rm(cid3)', 'ðk2þ', 'r(cid11)', 'þ', 'm', 'rm', '(cid11)', 'ðk2', '(cid11)þ', 'r', '(cid11)', 'þ', 'solvent', 'r(cid3)', 'þ', 'solvent', 'ðk0', '3', '(cid11)þ', 'r', 'symbol', 'hot', 'radicals', 'the', 'rate', 'expression', 'polymerization', 'written', 'follows', '92:', '(cid1)', 'd½m(cid4)dt', '¼', 'kxx05½m(cid4)', '(cid7)', '½i', 'þ', '1ðg', 'þ', 'g0', 'þ', 's½m(cid4)þ(cid4)', 'kx', '¼', 'k2(2k1fk4)12', 'g', '¼', 'k1/k2', 'g0', '¼', 'k3/k2,', 's', '¼', 'solvent', 'm', '¼', 'monomer', 'in', 'comparing', 'free-radical', 'polymerizations', 'ethyl', 'acrylates', 'benzene', 'dimethyl', 'formam-', 'ide', '50(cid6)c', '218', 'rates', 'found', 'proportional', 'square', 'roots', 'initiator', 'concentration', 'they', 'proportional', 'however', 'concentrations', 'monomer', 'this', 'interpreted', 'terms', 'hot', 'radicals', '93.', 'similar', 'results', 'however', 'interpreted', 'others', 'differently', 'for', 'instance', 'butyl', 'acrylate', 'butyl', 'propionate', 'polymerizations', 'benzene', 'also', 'fail', 'meet', 'ideal', 'kinetic', 'models', 'the', 'results', 'however', 'explained', 'terms', 'termination', 'primary', 'radicals', 'chain', 'transferring', '(see', 'sect', '35', 'explanation', 'chain', 'transferring)', '343', 'ceiling', 'temperature', 'for', 'free-radical', 'polymerization', 'reactions', 'elevated', 'temperatures', 'chain-growth', 'process', 'becomes', 'reversible', 'depropagation', 'takes', 'place', 'kd·p', 'rate', 'constant', 'depropagation', 'depolymerization', 'the', 'equilibrium', 'polymerization–depolymerization', 'reaction', 'temperature', 'dependent', 'the', 'reaction', 'isotherm', 'written', 'kdp', 'mnþ1(cid1)', 'mn(cid3)', 'þ', 'm', 'df', '¼', 'df0', 'þ', 'rt', 'lnk', 'in', 'equation', 'df0', 'free', 'energy', 'polymerization', 'monomer', 'polymer', 'appropriate', 'standard', 'states', '88.', 'the', 'standard', 'state', 'polymer', 'usually', 'solid', '(amorphous', 'partly', 'crystalline)', 'it', 'also', 'one', 'molar', 'solution', 'the', 'monomer', 'pure', 'liquid', 'one', 'molar', 'solution', 'the', 'relationships', 'monomer', 'concentration', 'heat', 'content', 'entropy', 'free', 'energy', 'shown', 'following', 'expression', 'this', 'applies', 'wide', 'range', 'temperatures', '5.', '34', 'propagation', '89', 'ln½m(cid4)', '¼', 'dh0', 'prtc', '(cid1)', 'ds0', 'pr', 'in', 'equation', 'tc', 'ceiling', 'temperature', 'equilibrium', 'monomer', 'concentration', 'it', 'function', 'temperature', 'reaction', 'because', 'heat', 'content', 'negative', 'quantity', 'concentration', 'monomer', '(in', 'equilibrium', 'polymer)', 'increases', 'increasing', 'temperatures', 'there', 'series', 'ceiling', 'temperatures', 'correspond', 'different', 'equilibrium', 'monomer', 'concentrations', 'for', 'given', 'concentration', 'monomer', 'solution', 'also', 'upper', 'temperature', 'polymerization', 'proceed', 'this', 'however', 'thermodynamic', 'approach', 'when', 'active', 'centers', 'present', 'polymer', 'structure', 'material', 'appear', 'stable', 'even', 'ceiling', 'temperature', 'state', 'metastable', 'equilibrium', 'the', 'magnitude', 'heat', 'polymerization', 'vinyl', 'monomers', 'related', 'two', 'effects', '(1)', 'steric', 'strains', 'form', 'single', 'bonds', 'interactions', 'substituents', 'these', 'substituents', 'located', 'alternate', 'carbon', 'atoms', 'polymeric', 'backbones', 'interfere', 'monomers', 'entering', 'chains', '(2)', 'differences', 'resonance', 'stabilization', 'monomer', 'double', 'bonds', 'conjugated', 'substituents', '70.', 'most', '12', 'disubstituted', 'monomers', 'stated', 'earlier', 'difﬁcult', 'polymerize', 'it', 'attributed', 'steric', 'interactions', 'one', 'two', 'substituents', 'vinyl', 'monomer', 'b-substituent', 'ultimate', 'unit', 'polymeric', 'chain', '94.', 'a', 'strain', 'also', 'imposed', 'bond', 'formed', 'transition', 'state', 'the', 'propagation', 'reaction', 'usually', 'requires', 'activation', 'energy', '5', 'kcalmol', 'as', 'result', 'rate', 'vary', 'rapidly', 'temperature', 'on', 'hand', 'transfer', 'reaction', 'requires', 'higher', 'activation', 'energies', 'chain-growth', 'reaction', 'this', 'means', 'average', 'molecular', 'weight', 'affected', 'transfer', 'reaction', 'higher', 'temperature', 'when', 'allowances', 'made', 'chain', 'transferring', 'molecular', 'weight', 'passes', 'maximum', 'temperature', 'raised', 'at', 'temperatures', 'maximum', 'product', 'molecular', 'weight', 'lower', 'kinetic', 'chain', 'length', 'decreases', 'temperature', 'above', 'maximum', 'however', 'product', 'molecular', 'weight', 'also', 'lower', 'increases', 'temperature', 'this', 'due', 'increase', 'transfer', 'reactions', 'the', 'assumes', 'rate', 'initiation', 'independent', 'temperature', 'the', 'relationship', 'kinetic', 'chain', 'length', 'temperature', 'expressed', 'follows', '5:', 'ln', 'ndt', '¼', 'ðep', '(cid1)', '12et', '(cid1)', '12eiþrt2', 'ep', 'et', 'ei', 'energies', 'propagation', 'termination', 'initiation', 'respectively', 'a', 'large', 'ei', 'means', 'temperature', 'polymerization', 'raised', 'kinetic', 'chain', 'length', 'decreases', 'this', 'affected', 'greater', 'frequency', 'chain', 'transferring', 'higher', 'temperatures', 'in', 'addition', 'possibility', 'disproportionation', 'may', 'become', 'signiﬁcant', '344', 'autoacceleration', 'when', 'concentrations', 'monomers', 'high', 'solution', 'bulk', 'polymerizations', 'typical', 'auto-', 'accelerations', 'rates', 'observed', 'this', 'known', 'gel', 'effect', 'trammsdorff', 'effect', 'also', 'norrish–smith', 'effect', '66.', 'the', 'effect', 'explained', 'caused', 'decrease', 'rate', 'termination', 'due', 'increased', 'viscosity', 'medium', 'termination', 'reaction', 'requires', 'two', 'large', 'polymer-radicals', 'come', 'together', 'impeded', 'viscosity', 'at', 'time', 'propagation', 'small', 'molecules', 'monomer', 'still', 'diffuse', 'time', 'radical', 'sites', 'feed', 'chain', 'growth', '90', '3', 'free-radical', 'chain-growth', 'polymerization', 'one', 'mistake', 'acceleration', 'polymerization', 'reaction', 'due', 'rise', 'temperature', 'nonisothermal', 'conditions', 'true', 'gel', 'effect', 'rise', 'viscosity', 'the', 'gel', 'effect', 'occur', 'temperature', 'reaction', 'kept', 'constant', 'a', 'critical', 'analysis', 'gel', 'effect', 'suggests', 'situation', 'complicated', 'in', 'polymerizations', 'three', 'different', 'stages', 'appear', 'present', 'rp[m[i12', 'plotted', 'conversion', 'time', '95.', 'the', 'plot', 'indicates', 'ﬁrst', 'stage', 'either', 'constant', 'declining', 'rate', 'second', 'stage', 'autoacceleration', 'during', 'third', 'stage', 'constant', 'declining', 'rate', '95.', 'numerous', 'publication', 'made', 'substantial', 'case', 'associating', 'andor', 'attributing', 'gel', 'effect', 'entanglement', 'polymerizing', 'chain', 'radicals', 'resulting', 'marked', 'reduction', 'termination', 'rate', 'parameter', 'kt', 'this', 'often', 'done', 'using', 'assumption', 'neighborhood', 'gel', 'effect', 'kt', 'controlled', 'polymer', 'self-diffusion', 'turn', 'exhibits', 'entangled', 'polymer', 'dynam-', 'ics', 'o’neil', 'et', 'al', '96,', 'however', 'argued', 'opinion', 'they', 'carried', 'series', 'experiments', 'involving', 'bulk', 'polymerizations', 'methyl', 'methacrylate', 'styrene', 'feel', 'data', 'contradicts', 'widely', 'held', 'belief', 'gel', 'effect', 'onset', 'related', 'formation', 'chain', 'entanglements', 'the', 'experimental', 'conditions', 'used', 'tended', 'delay', 'eliminate', 'formation', 'chain', 'entanglement', 'these', 'conditions', 'high', 'initiator', 'andor', 'chain', 'transferring', 'agent', 'concentrations', 'additions', 'low', 'molecular', 'weight', 'polymers', 'prior', 'reactions', 'the', 'results', 'indicated', 'gel', 'effect', 'occurs', 'readily', 'absence', 'entanglement', 'delaying', 'onset', 'entanglements', 'necessarily', 'delay', 'onset', 'gel', 'effect', 'also', 'critical', 'examination', 'molecular', 'weights', 'produced', 'experiments', 'indicated', 'values', 'low', 'entanglement', 'formation', 'solution', '(polymer', 'plus', 'monomer)', 'sometimes', 'even', 'bulk', 'polymer', 'onset', 'also', 'throughout', 'gel', 'effect', '96.', 'o’neil', 'et', 'al', '96,', 'found', 'even', 'conditions', 'entanglements', 'likely', 'exist', 'gel', 'effect', 'onset', 'correlate', 'polymer', 'molecular', 'weight', 'chains', 'produced', 'manner', 'consistent', 'entanglement', 'arguments', 'whether', 'kinetics', 'gel', 'effect', 'may', 'affected', 'entanglements', 'left', 'uncertain', '345', 'polymerization', 'monomers', 'multiple', 'double', 'bonds', 'polymerizations', 'monomers', 'multiple', 'double', 'bonds', 'yield', 'products', 'vary', 'according', 'locations', 'bonds', 'respect', 'monomers', 'conjugated', 'double', 'bonds', '13-butadiene', 'derivatives', 'polymerize', 'two', 'different', 'ways', 'one', 'way', 'one', 'double', 'bonds', 'another', 'way', 'double', 'bonds', 'simultaneously', 'such', '14', 'propagation', 'attributable', 'effect', 'conjugation', 'hybridization', 'c2–c3', 'bond', 'involves', 'sp2', 'hybrid', 'orbitals', '97.', 'all', 'three', 'modes', 'propagation', 'possible', 'one', 'polymerization', 'reaction', 'product', 'effect', 'copolymer', 'the', '14', '12', '34', 'placement', 'propagations', 'illustration', 'follows', 'r2', 'r1', 'r', 'r4', 'r5', 'r2', 'r4', 'n', 'r1', 'r5', 'r', 'r3', 'r3', '14-placement', '34', 'propagation', '91', 'r', 'n', 'r1', 'r2', 'r3', 'r4', 'r5', 'r4', 'r5', 'r', 'r1', 'r', 'r2', 'n', 'r3', 'r3', 'r4', 'n', 'r5', 'r1', 'r2', 'the', 'polymerizations', 'copolymerizations', 'various', 'conjugated', 'dienes', 'discussed', 'chap', '6', '12', '34', 'placements', '3451', 'ring', 'forming', 'polymerization', 'propagation', 'reactions', 'unconjugated', 'dienes', 'proceed', 'intra-intermolecular', 'process', 'this', 'usually', 'results', 'ring', 'formation', 'cyclopolymerization', 'it', 'illustrated', 'follows', 'r', 'x', 'r', 'x', 'r', 'x', 'r', 'x', 'x', 'designate', 'either', 'carbon', 'heteroatom', 'an', 'example', 'polymerization', 'free-radical', 'polymerization', 'quaternary', 'diethyldiallylamine', '98:', 'another', 'example', 'polymerization', '26-disubstituted', '16-heptadiene', '103:', 'r', 'r', 'r', 'r', 'i', '3', 'i', '3', 'r', 'r', 'r', 'r', 'n', 'br', 'n', 'n', 'br', 'i', 'r¼cooc2h5', 'cooch3', 'cooh', '92', '3', 'free-radical', 'chain-growth', 'polymerization', 'the', 'intra-intermolecular', 'propagations', 'result', 'ring', 'structures', 'various', 'sizes', 'for', 'instance', 'three-membered', 'rings', 'form', 'transanular', 'polymerizations', 'bicycloheptadiene', '100', '101:', 'four-membered', 'rings', 'form', 'free-radical', 'polymerization', 'perﬂuoro-14-pentadiene', '103.', 'the', 'size', 'ring', 'forms', 'depends', 'mainly', 'number', 'atoms', 'double', 'bonds', 'f', 'f', 'f', 'f', 'gamma', 'rays', 'f', 'f', 'f', 'f', 'f', 'f', 'f', '11000', 'atm', '110', 'oc', 'f', 'f', 'f', 'f', 'cf3', 'f', 'f', 'formation', 'many', 'ﬁve-membered', 'rings', 'also', 'known', 'one', 'example', 'polymerization', '23-', 'dicarboxymethyl-16', 'hexadiene', '98:', 'n', 'f', 'f', 'n', 'f', 'f', 'r', 'r', 'r', 'n', 'r', 'r¼cooch3', 'the', 'polymer', 'forms', 'shown', 'cross-linked', 'spectroscopic', 'analysis', 'shows', '90', 'monomer', 'placement', 'ring', 'formation', '104.', 'formations', 'six-membered', 'rings', 'also', 'well', 'documented', 'two', 'examples', 'shown', 'polymerization', 'quaternary', 'diethyldiallylamine', 'polymerization', '26', 'disubstitued', '16-heptadiene', 'many', '16-', 'heptadienes', 'yield', 'linear', 'polymers', 'containing', 'six-membered', 'rings', '105.', 'this', 'tendency', 'propagate', 'intra-intermolecularly', 'unconjugated', 'dienes', 'greater', 'expected', 'purely', 'statistical', 'predictions', '106.', 'butler', 'suggested', 'results', 'interactions', 'oleﬁnic', 'bonds', '107–109.', 'ultraviolet', 'absorption', 'spectra', 'several', 'unconju-', 'gated', 'dioleﬁns', 'show', 'bathochromic', 'shifts', 'absorption', 'maxima', 'relative', 'values', 'calculated', 'woodward’s', 'rule', '102', '104.', 'this', 'supports', 'butler’s', 'explanation', '107', '109.', 'a', 'raft', 'type', 'controlledliving', 'polymerization', '(see', 'sect', '3144)', 'applied', 'cyclopoly-', 'merization', 't-butyl', '(hydroxyethyl)', 'acrylate', 'ether', 'dimer', '108.', 'the', 'polymerizations', 'carried', 'xylene', '70(cid6)c', 'yielded', 'polymers', 'six-membered', 'tetrahydropyran', 'repeat', 'units', 'o', 'o', 'n', 'ro', 'o', 'or', 'n', 'coor', 'rooc', 'o', '35', 'the', 'termination', 'reaction', 'the', 'termination', 'process', 'free-radical', 'polymerization', 'caused', 'shown', 'early', 'chapter', 'one', 'three', 'types', 'reactions', '(1)', 'second', 'order', 'radical-radical', 'reaction', '(2)', 'second', 'order', 'radical-molecule', 'reaction', '(3)', 'ﬁrst', 'order', 'loss', 'radical', 'activity', '35', 'the', 'termination', 'reaction', '93', 'the', 'ﬁrst', 'reaction', 'either', 'one', 'combination', 'disproportionation', 'in', 'combination', 'reaction', 'two', 'unpaired', 'spin', 'electrons', 'terminal', 'end', 'different', 'polymer-radical', 'unite', 'form', 'covalent', 'bond', 'large', 'polymer', 'molecule', 'in', 'disproportionation', 'hand', 'two', 'polymer-radicals', 'react', 'one', 'abstracts', 'atom', 'one', 'this', 'results', 'formation', 'two', 'inactive', 'polymer', 'molecules', 'the', 'two', 'differ', 'one', 'terminal', 'saturated', 'structure', 'one', 'terminal', 'double', 'bond', 'usually', 'atom', 'transferred', 'hydrogen', 'it', 'suggested', '111', 'basic', 'rule', 'thumb', 'applied', 'determine', 'termination', 'reaction', 'predominates', 'typical', 'homopolymerization', 'thus', 'polymerizations', '11-disubstituted', 'oleﬁns', 'likely', 'terminate', 'disproportionation', 'steric', 'effects', 'polymerizations', 'vinyl', 'mono-', 'mer', 'however', 'favor', 'terminations', 'combination', 'unless', 'contain', 'particularly', 'labile', 'atoms', 'transferring', 'higher', 'activation', 'energies', 'usually', 'required', 'termination', 'reactions', 'disproportion-', 'ation', 'this', 'means', 'terminations', 'combination', 'predominate', 'lower', 'temperatures', 'for', 'polymer', 'radical', 'simply', 'grows', 'adding', 'monomeric', 'units', 'still', 'possesses', 'active', 'center', 'growth', 'number', 'monomeric', 'units', '(r)', 'added', 'radical', 'center', 'time', 'interval', 'according', 'tobita', '112,', 'conforms', 'following', 'poisson', 'distribution', 'pðrþ', '¼', 'e(cid1)yyrr', '¼', 'kp½m(cid4)t', 'ptðytþ', '¼', 'x', 'exp', 'ð(cid1)xytþ', 'expected', 'number', 'monomeric', 'units', 'added', 'radical', 'center', 'given', 'kp', 'propagation', 'rate', 'coefﬁcient', 'm', 'monomer', 'concentration', 'if', 'number', 'added', 'monomeric', 'units', 'r', 'large', 'enough', 'one', 'approximate', 'r', 'ﬃ', 'for', 'bimolecular', 'termina-', 'tion', 'reactions', 'independent', 'chain', 'length', 'required', 'time', 'bimolecular', 'termination', 'particular', 'radical', 'pair', 'also', 'given', 'following', 'probable', 'distribution', '112:', 'x', '¼', 'kt(kpmnna)', 'kt', 'bimolecular', 'termination', 'rate', 'coefﬁcient', 'the', 'imaginary', 'time', 'chain', 'stoppage', 'bimolecular', 'termination', 'must', 'considered', 'radical', 'pairs', 'exist', 'reaction', 'medium', '112.', 'the', 'third', 'type', 'termination', 'reaction', 'chain', 'transferring', 'premature', 'termination', 'transferring', 'results', 'lower', 'molecular', 'weight', 'polymer', 'expected', 'termination', 'reactions', 'the', 'product', 'chain', 'transferring', 'inert', 'polymer', 'molecule', 'often', 'new', 'free', 'radical', 'capable', 'new', 'initiation', 'if', 'however', 'new', 'radical', 'capable', 'starting', 'growth', 'new', 'chain', 'degenerative', 'chain', 'transferring', 'it', 'also', 'referred', 'ﬁrst-order', 'termination', 'reaction', 'the', 'molecules', 'accept', 'new', 'radical', 'sites', '(participate', 'chain', 'transferring)', 'present', 'reaction', 'medium', 'this', 'includes', 'solvents', 'monomer', 'molecules', 'inactive', 'polymeric', 'chains', 'initiators', 'the', 'ease', 'chain', 'transferring', 'takes', 'place', 'depends', 'upon', 'bond', 'strength', 'labile', 'atoms', 'abstracted', 'rest', 'molecule', 'attached', 'for', 'instance', 'chain', 'transferring', 'methyl', 'methacrylate', 'polymerization', 'solvent', 'occurs', 'following', 'order', '115:', 'benzene', 'toluene', 'ethyl', 'benzene', 'cumene', 'the', 'rate', 'chain', 'transferring', 'reaction', 'rtr', '¼', 'ktr½m(cid3)(cid4)½xa(cid4)', '94', '3', 'free-radical', 'chain-growth', 'polymerization', 'ktr', 'chain', 'transferring', 'constant', 'reaction', 'm(cid3)', 'þ', 'x', '(cid1)', 'a', 'ktr', 'm', '(cid1)', 'x', 'þ', 'a(cid3)', 'examples', 'molecules', 'particularly', 'labile', 'atoms', 'contribute', 'readily', 'chain-', 'transferring', 'mercaptans', 'halogen', 'compounds', 'chloroform', 'carbon', 'tetrachloride', 'etc', 'a', 'polymer', 'prematurely', 'terminated', 'growth', 'chain', 'transferring', 'may', 'telomer', 'in', 'cases', 'telomer', 'formation', 'newly', 'formed', 'radical', 'monomer', 'radical', 'active', 'enough', 'initiate', 'new', 'chain', 'growth', 'thus', 'life', 'kinetic', 'chain', 'maintained', 'an', 'illustration', 'telomerization', 'reaction', 'free-radical', 'polymerization', 'ethylene', 'presence', 'chloroacetyl', 'chloride', 'initiation', 'roor', '2ro', 'o', 'cl', 'ch2cl', '2ro', 'o', 'o', 'cl', 'ch2', 'rocl', 'o', 'propagation', 'cl', 'ch2', 'cl', 'n', 'o', 'n', 'o', 'o', 'cl', 'n', 'termination', 'cl', 'cl', 'ch2cl', 'o', 'cl', 'o', 'cl', 'cl', 'n', 'ch2', 'chain', 'transferring', 'affected', 'temperature', 'changes', 'viscosity', 'reaction', 'medium', '115.', 'when', 'transfer', 'takes', 'place', 'monomer', 'independent', 'polymerization', 'rate', '116', '117.', 'when', 'however', 'transfer', 'takes', 'place', 'initiator', 'rate', 'increases', 'rapidly', '118.', 'a', 'chain', 'transferring', 'reaction', 'monomer', 'illustrated', 'follows', 'x', 'x', '1', '2', 'x', 'hydrogen', 'abstraction', 'x', 'hydrogen', 'donation', 'x', 'x', 'x', 'x', 'a', 'transfer', 'reaction', 'also', 'occur', 'terminal', 'group', 'polymer-radical', 'location', 'polymeric', 'backbone', 'this', 'known', 'backbiting', '35', 'the', 'termination', 'reaction', '95', 'the', 'new', 'free-radical', 'site', 'polymer', 'backbone', 'starts', 'chain', 'growth', 'results', 'formation', 'branch', 'the', 'reaction', 'take', 'place', 'polymer-radical', 'location', 'another', 'polymer', 'chain', 'in', 'either', 'case', 'fresh', 'chain', 'growth', 'results', 'formation', 'branch', 'whether', 'chain', 'transferring', 'take', 'place', 'initiator', 'depends', 'upon', 'initiator’s', 'chemical', 'structure', 'it', 'believed', 'past', 'chain', 'transferring', 'aa0-azobisisobutyronitrile', 'occur', 'later', 'shown', 'chain', 'transferring', 'initiator', 'occur', 'well', 'least', 'polymerizations', 'methyl', 'methacrylate', '118', '119.', 'the', 'amount', 'chain', 'transferring', 'takes', 'place', 'monomers', 'usually', 'low', 'reaction', 'requires', 'breaking', 'strong', 'carbon-hydrogen', 'bonds', 'monomers', 'however', 'vinyl', 'chloride', 'vinyl', 'acetate', 'fairly', 'large', 'chain', 'transferring', 'constants', 'in', 'case', 'vinyl', 'acetate', 'attributed', 'presence', 'acetoxy', 'methyl', 'group', 'this', 'explanation', 'however', 'cannot', 'used', 'vinyl', 'chloride', 'the', 'chain', 'transferring', 'constants', 'usually', 'deﬁned', 'cm', '¼', 'ktrmkp', 'monomers', 'cs', '¼', 'ktrskp', 'solvents', 'ci', '¼', 'ktrikp', 'initiators', 'the', 'values', 'found', 'handbooks', 'places', 'literature', 'presence', 'chain', 'transferring', 'agents', 'polymerization', 'reaction', 'requires', 'redeﬁning', 'degree', 'polymerization', 'include', 'chain', 'termination', 'terms', 'the', 'number', 'average', 'degree', 'polymerization', 'written', 'follows', 'dp', '¼', 'rp', 'ðrt2þ', 'þ', 'ktrm½m(cid3)(cid4)½m(cid4)', 'þ', 'ktrs½m(cid3)(cid4)½s(cid4)', 'þ', 'ktri½m(cid3)(cid4)½i(cid4)', 'it', 'also', 'expressed', 'terms', 'chain', 'transferring', 'constants', 'follows', '1dp', '¼', '2rpri', 'þ', 'cm', 'þ', 'cs½s(cid4)½m(cid4)', 'þ', 'ci½i(cid4)½m(cid4)', 'this', 'also', 'written', 'still', 'another', 'form', '1dp', '¼', 'ktrpkp', '2½m(cid4)2', 'þ', 'cm', 'þ', 'cs½s(cid4)½m(cid4)', 'þ', 'ciktrp', '2kp', '2f', 'kd½m(cid4)3', 'when', 'polymerization', 'reaction', 'conducted', 'concentrated', 'solution', 'complete', 'absence', 'solvent', 'viscosity', 'medium', 'increases', 'time', '(unless', 'polymer', 'precipitates', 'out)', 'this', 'impedes', 'steps', 'polymerization', 'process', 'particularly', 'diffusions', 'large', 'polymer-radicals', '54.', 'the', 'decreased', 'mobility', 'polymer-radicals', 'affects', 'termination', 'process', 'it', 'appears', 'common', 'many', 'though', 'free-radical', 'polymerizations', 'all', 'molecular', 'processes', 'termination', 'reactions', 'fully', 'understood', 'particularly', 'high', 'conversions', '119', 'this', 'complex', 'process', 'consists', 'three', 'deﬁnable', 'steps', 'these', 'pictured', 'follows', 'first', 'two', 'polymer', 'radicals', 'migrate', 'together', 'means', 'translational', 'diffusion', 'second', 'radical', 'sites', 'reorient', 'toward', 'segmental', 'diffusion', 'third', 'radicals', 'overcome', 'small', 'chemical', 'activation', 'barriers', 'react', 'the', 'termination', 'reaction', 'therefore', 'diffusion', 'controlled', 'at', 'low', 'concentrations', 'segmental', 'diffusion', 'medium', 'high', 'concentrations', 'translational', 'diffusion', 'present', 'theories', 'terminations', 'suggest', 'intermediate', 'conversions', 'terminations', 'domi-', 'nated', 'interactions', 'short', 'chains', 'formed', 'transfer', 'entangled', 'long', 'chains', '121.', '96', '3', 'free-radical', 'chain-growth', 'polymerization', 'when', 'terminations', 'diffusion', 'controlled', 'termination', 'events', 'involve', 'two', 'highly', 'entangled', 'chains', 'whose', 'ends', 'move', '“reaction-diffusion”', 'process', '119.', 'in', 'process', 'terminations', 'occur', 'propagation-induced', 'diffusion', 'chain', 'ends', 'growing', 'macroradicals', 'this', 'means', 'rates', 'terminations', 'depend', 'upon', 'chain', 'lengths', '113.', 'diffusion', 'theories', 'proposed', 'relate', 'rate', 'constant', 'termination', 'initial', 'viscosity', 'polymerization', 'medium', 'the', 'rate-determining', 'step', 'termination', 'segmental', 'diffusion', 'chain', 'ends', 'inversely', 'proportional', 'microviscosity', 'solution', '123.', 'yokota', 'itoh', '124', 'modiﬁed', 'rate', 'equation', 'include', 'viscosity', 'medium', 'according', 'equation', 'overall', 'polymerization', 'rate', 'constant', 'proportional', 'square', 'root', 'initial', 'viscosity', 'system', 'the', 'number', 'average', 'termination', 'rate', 'constants', 'methyl', 'methacrylate', 'polymerization', 'measured', 'in-line', 'esr', 'spectrometer', 'this', 'done', 'observing', 'radical', 'decay', 'rates', '120.', 'the', 'results', 'disagreement', 'concept', 'termination', 'propagation-diffusion', 'expected', 'dominant', 'high', 'conversion', 'rates', 'instead', 'termination', 'rate', 'constants', 'decrease', 'dramatically', 'posteffect', 'period', 'high', 'conversions', 'actually', 'fraction', 'radicals', 'found', 'trapped', 'polymerization', 'thus', 'two', 'types', 'radicals', 'reaction', 'mixture', 'trapped', 'free', 'radicals', 'in', 'propagations', 'termination', 'reactions', 'two', 'types', 'radical', 'populations', 'different', 'reactivities', '120.', 'shipp', 'coworkers', '120', 'described', 'method', 'analyzing', 'chain', 'length', 'dependence', 'termination', 'rate', 'coefﬁcients', 'reacting', 'radicals', 'low', 'conversion', 'free', 'radical', 'polymerizations', 'their', 'method', 'involves', 'comparing', 'experimental', 'molecular', 'weight', 'distributions', 'polymers', 'formed', 'pulsed', 'laser', 'photolysis', 'experiments', 'predicted', 'kinetic', 'simulation', 'the', 'method', 'enabled', 'direct', 'measurements', 'concentration', 'radicals', 'generated', 'per', 'laser', 'pulse', 'knowledge', 'radical', 'concentrations', 'mean', 'unknowns', 'simulations', 'termination', 'rate', 'coefﬁcients', 'they', 'concluded', 'analysis', 'demonstrates', 'need', 'chain', 'length', 'dependent', 'termination', 'rate', 'constants', 'describing', 'polymerization', 'kinetics', 'free-radical', 'photopolymerizations', '(see', 'chap', '10)', 'multifunctional', 'acrylic', 'monomers', 'result', 'cross-linked', 'polymeric', 'networks', 'the', 'kinetic', 'picture', 'polymerizations', 'varies', 'ordinary', 'linear', 'polymerization', 'diffusion', 'free', 'radicals', 'functional', 'groups', 'becomes', 'severely', 'restricted', 'this', 'causes', 'growing', 'polymer', 'chains', 'rapidly', 'cyclize', 'cross-link', 'clusters', '(microgels)', 'the', 'clusters', 'become', 'linked', 'networks', 'many', 'free', 'radicals', 'become', 'trapped', 'terminations', 'take', 'place', 'combinations', 'chain', 'transferring', 'the', 'cumulative', 'chain', 'length', 'polymerizations', 'calculated', 'following', 'equation', '125:', 'n', '¼', 'xnm0', 'nrg', 'w', 'conversion', 'functional', 'groups', 'nm0', 'initial', 'number', 'functional', 'groups', 'nrg', 'total', 'number', 'radicals', 'generated', '36', 'copolymerization', 'if', 'one', 'monomer', 'species', 'present', 'reaction', 'medium', 'copolymer', 'interpolymer', 'result', 'polymerization', 'reaction', 'whether', 'reaction', 'products', 'consist', 'copolymers', 'mixture', 'homopolymers', 'however', 'depends', 'largely', 'upon', 'reactivity', 'monomers', 'a', 'useful', 'simplifying', 'assumption', 'kinetic', 'analyses', 'free-radical', 'copolymer-', 'izations', 'reactivity', 'polymer', 'radicals', 'governed', 'entirely', 'terminal', 'monomer', 'units', '52.', 'for', 'instance', 'growing', 'polymer', 'radical', 'contains', 'methyl', 'methacrylate', 'terminal', 'unit', '36', 'copolymerization', '97', 'table', '38', 'some', 'reactivity', 'ratios', '132', '136', 'monomer', '1', 'styrene', 'styrene', 'styrene', 'vinyl', 'acetate', 'methyl', 'acrylate', 'monomer', '2', 'butadiene', 'methyl', 'methacrylate', 'vinyl', 'acetate', 'vinyl', 'chloride', 'vinyl', 'chloride', 'r1', '078', '052', '550', '023', '900', 'r2', '139', '046', '001', '168', '0083', 'considered', 'terms', 'reactivity', 'poly(methyl', 'methacrylate)', 'radical', 'this', 'assumption', 'although', 'always', 'adequate', '52', 'used', 'predict', 'satisfactorily', 'behavior', 'many', 'mixtures', 'monomers', 'based', 'assumption', 'copolymerization', 'pair', 'monomers', 'involves', 'four', 'distinct', 'growth', 'reactions', 'two', 'types', 'polymer', 'radicals', '361', 'reactivity', 'ratios', 'in', 'reaction', 'two', 'monomers', 'designated', 'm1', 'm2', 'four', 'distinct', 'reactions', 'written', 'follows', '1', '(cid5)', 'm1(cid3)', 'þ', 'm1', '(cid1)', 'k11', '(cid5)m1m1(cid3)', '2', '(cid5)m1(cid3)þ', 'm2', '(cid1)', 'k12', '(cid5)m1m2(cid3)', '3', '(cid5)m1(cid3)þ', 'm2', '(cid1)', 'k21', '(cid5)m1m2(cid3)', '4', '(cid5)m2(cid3)', 'þ', 'm2', '(cid1)', 'k22', '(cid5)m2m2(cid3)', 'r1', '¼', 'k11k12', 'r2', '¼', 'k22k21', 'the', 'ratios', 'k11k12', 'k22k21', 'called', 'monomer', 'reactivity', 'ratios', 'they', 'written', 'follows', 'the', 'relationship', 'express', 'terms', 'ratio', 'monomers', 'm1/m2', 'end', 'formed', 'polymer', 'rp', 'rp', '¼', 'rmðr12rm', 'þ', '1þðr21', 'þ', 'rmþ', 'rm', 'equal', 'm1/m2.', 'table', '38', 'illustrates', 'typical', 'reactivity', 'ratios', 'taken', 'literature', 'many', 'found', '128.', 'these', 'reactivity', 'ratios', 'represent', 'relative', 'rates', 'reactions', 'polymer', 'radicals', 'monomers', 'vs', 'comonomers', 'when', 'r1', '1', 'radical', 'm1•', 'reacting', 'monomer', 'm1', 'faster', 'comonomer', 'm2', 'on', 'hand', 'r1', '1', 'opposite', 'true', 'based', 'r', 'values', 'composition', 'copolymers', 'calculated', 'copolymerization', 'equation', '52', 'shown', 'd½m1(cid4)', 'd½m2(cid4)', '¼', '½m1(cid4)', '½m2(cid4)', 'r1½m1(cid4)', 'þ', '½m2(cid4)', 'r2½m2(cid4)', 'þ', '½m1(cid4)', '(cid3)', '(cid2)', '98', '3', 'free-radical', 'chain-growth', 'polymerization', 'in', 'ideal', 'copolymerization', 'reaction', 'k11k12', '¼', 'k22k21', 'r1r2', '¼', '1', 'if', 'r1', 'r2', 'values', 'equal', 'approach', 'zero', 'polymer', 'radical', 'reacts', 'preferentially', 'monomer', 'this', 'results', 'alternating', 'copolymer', 'regardless', 'composition', 'monomer', 'mixture', 'that', 'however', 'limiting', 'case', 'in', 'majority', 'instances', 'r1', '(cid7)', 'r2', 'greater', 'zero', 'less', 'one', 'when', 'polymer', 'radicals', 'react', 'preferentially', 'monomer', 'r1', '(cid7)', 'r2', '1', 'mainly', 'mixture', 'homopolymers', 'forms', 'copolymerization', 'takes', 'place', 'reactivity', 'vinyl', 'monomers', 'often', 'determined', 'experimentally', 'studying', 'copolymer-', 'izations', 'values', 'many', 'free-radical', 'reactivity', 'ratios', 'tabulated', 'many', 'different', 'monomer', 'pairs', '126.', 'also', 'qualitative', 'correlations', 'copolymerization', 'data', 'molecular', 'orbital', 'calculations', 'found', 'literature', '136.', 'some', 'general', 'conclusion', 'monomer', 'reactivity', 'toward', 'attacking', 'radicals', 'drawn', 'mayo', 'walling', '130:', '1', 'the', 'alpha', 'substituents', 'monomer', 'effect', 'increasing', 'reactivity', 'following', 'order', 'n', 'o', 'o', 'o', 'or', 'cl', 'ch2x', 'o', 'or', '2', 'the', 'effect', 'second', 'alpha', 'substituent', 'roughly', 'additive', 'giese', 'coworkers', '131–136', 'developed', 'special', 'technique', 'studying', 'effects', 'substituents', 'upon', 'relative', 'reactivity', 'vinyl', 'monomers', 'toward', 'free', 'radicals', 'brieﬂy', 'follows', 'free', 'radicals', 'produced', 'reducing', 'organomercury', 'halides', 'sodium', 'borohydride', 'the', 'radicals', 'undergo', 'competitive', 'additions', 'pairs', 'various', 'substituted', 'oleﬁns', 'the', 'adducts', 'turn', 'trapped', 'hydrogen', 'transfers', 'formed', 'organomercury', 'hydrides', 'relative', 'quantities', 'product', 'determined', 'rmgx', 'nabh4', 'r', 'x', 'y', 'r', 'r', 'kx', 'ky', 'rhgh', 'rhgh', 'r', 'r', 'x', 'y', 'x', 'y', 'this', 'method', 'applied', 'copolymerization', 'acrylonitrile', 'methyl', 'acrylate', '129.', 'it', 'showed', 'ratios', 'rate', 'constants', 'two', 'monomers', 'independent', 'concentrations', 'copolymerization', 'reactions', 'affected', 'solvents', 'one', 'example', 'cited', 'effect', 'addition', 'water', 'glacial', 'acetic', 'acid', 'copolymerization', 'mixture', 'methyl', 'methacrylate', 'acrylamide', 'dimethyl', 'sulfoxide', 'chloroform', 'this', 'causes', 'changes', 'reactivity', 'ratios', '131.', 'changes', 'r', 'values', 'result', 'changes', 'solvents', 'copolymerizations', 'styrene', 'methyl', 'methacrylate', 'another', 'example', '133', '134.', 'the', 'true', 'styrene', 'acrylonitrile', 'copolymeri-', 'zation', '132.', 'there', 'also', 'indications', 'temperature', 'may', 'effect', 'reactivity', 'ratios', '135,', 'least', 'cases', '36', 'copolymerization', '362', 'q', 'e', 'scheme', '99', 'though', 'molecular', 'orbital', 'calculations', 'allow', 'accurate', 'predictions', 'reactivity', 'ratios', '133,', 'many', 'chemists', 'also', 'rely', 'upon', 'price–alfrey', 'q–e', 'equations', '140.', 'these', 'based', '(1)', 'polarity', 'double', 'bonds', 'monomers', 'measures', 'propagating', 'chain', 'ends', '(2)', 'mesomerism', 'substituents', 'double', 'bonds', 'chain', 'ends', '(3)', 'steric', 'hindrance', 'substituents', 'this', 'relationship', 'expressed', 'following', 'equation', '149:', 'also', 'written', 'follows', 'k12', '¼', 'p1q(cid1)e1e2', '2', 'log', 'k12', '¼', 'log', 'q1', 'þ', 'log', 'q2', '(cid1)', '04343e1e2', 'k12', 'represents', 'rate', 'constant', 'reaction', 'propagating', 'radical', 'm1•', 'monomer', 'm2', 'p1', 'represents', 'general', 'reactivity', 'polymer', 'radical', 'terminal', 'unit', 'monomer', 'm1', 'q1', 'q2', 'reactivities', 'monomer', 'm1', 'm2', 'e1', 'e2', 'measures', 'polar', 'characters', 'monomers', 'it', 'possible', 'calculate', 'q', 'e', 'values', 'r1', 'r2', 'conversely', 'r', 'values', 'obtained', 'q', 'e', 'values', 'the', 'relationship', 'follows', '136:', 'the', 'q', 'e', 'scheme', 'based', 'semiempirical', 'approach', 'nevertheless', 'attempts', 'made', 'develop', 'theoretical', 'interpretations', 'thus', 'schwann', 'price', '141', 'developed', 'following', 'relationship', 'r1', '¼', 'k11k12', '¼', 'q1q2', 'exp½(cid1)e1ðe1', '(cid1)', 'e2þ(cid4)', 'r2', '¼', 'k22k21', '¼', 'q2q1', 'exp½(cid1)e2ðe2', '(cid1)', 'e1þ(cid4)', 'q', '¼', 'expð(cid1)qrtþ', 'e', '¼', 'eðgdrtþ12', 'in', 'equation', 'q', 'represents', 'resonance', 'stabilization', '(kcalmol)', 'e', 'electrical', 'charge', 'transition', 'state', 'g', 'distance', 'centers', 'charge', 'radical', 'monomer', 'd', 'stands', 'effective', 'dielectric', 'constant', 'reaction', 'ﬁeld', 'the', 'values', 'q', 'e', 'derived', 'calculation', 'in', 'addition', 'rigorous', 'molecular', 'orbital', 'calculations', '138', 'show', 'relationship', 'q', 'localized', 'energy', 'monomer', 'e', 'electron', 'afﬁnity', 'also', 'scale', 'q', 'e', 'values', 'deduced', 'essentially', 'molecular', 'orbital', 'considerations', '143.', 'in', 'addition', 'huckel', 'treatment', 'transition', 'state', 'monomer–radical', 'reaction', 'free-radical', 'copolymerization', 'developed', '142.', 'the', 'resulting', 'reactivity', 'ratios', 'compared', 'well', 'derived', 'q', 'e', 'scheme', 'this', 'scheme', 'regarded', 'version', 'molecular', 'orbital', 'approach', '145.', 'nevertheless', 'scheme', 'considered', 'empirical', 'one', 'the', 'precision', 'calculating', 'q', 'e', 'values', 'poor', 'steric', 'factors', 'taken', 'account', 'it', 'good', 'however', 'qualitative', 'semiquantitative', 'results', 'a', 'revised', 'reactivity', 'scheme', 'proposed', 'jenkins', 'called', 'uv', 'scheme', '148.', 'it', 'claimed', 'accurate', 'also', 'capable', 'application', 'copolymerizations', 'transfer', 'reactions', 'the', 'scheme', 'retains', 'much', 'format', 'q', 'e', 'scheme', 'in', 'one', 'intrinsic', 'radical', 'reactivity', 'quantiﬁed', 'reference', 'rate', 'reaction', 'radical', 'styrene', 'monomer', '100', '3', 'free-radical', 'chain-growth', 'polymerization', 'the', 'original', 'approach', 'scheme', 'based', 'copolymerizations', 'styrene', 'acrylonitrile', 'acrylic', 'monomers', '149.', 'in', 'developing', 'general', 'approach', 'however', 'jenkins', 'concluded', 'principle', 'general', 'procedure', 'involves', 'fewer', 'assumptions', 'practice', 'much', 'utility', 'lost', 'he', 'proposed', 'convenient', 'practice', 'employ', 'kis', 'styrene', 'copolymerizations', 'in', 'writing', 'original', 'scheme', 'styrene', 'copolymerization', 'acrylic', 'monomers', 'benzene', 'jenkins', 'introduced', 'term', '“s”', 'denote', 'styrene', 'sp', 'represents', 'hammett', 'sigma', 'constant', 'substituent', 'para', 'position', 'in', 'revised', 'scheme', 'still', 'uses', 'kis', 'writes', 'follows', 'the', 'term', 'k11', 'substituted', 'sides', 'equation', 'yield', 'log', 'r12', '¼', 'log', 'k1s', 'þ', 'u2', 'þ', 'n2', 'log', 'r12', '¼', 'log', 'k1s', '(cid1)', 'u2s1', '(cid1)', 'n2', 'the', 'equation', 'represents', 'postulate', 'bases', 'q', 'e', 'scheme', 'this', 'scheme', 'contains', 'assumption', 'intrinsic', 'reactivity', 'radical', 'measured', 'value', 'k1s', 'polarity', 's1', '(or', 'p1)', 'thus', 'it', 'follows', 'rs2', '¼', '(cid1)n2', 's1', '¼', 'ps', '¼', 'us', '¼', 'ns', '¼', '0', 'the', 'ﬁnal', 'equation', 'becomes', 'log', 'r12', '(cid1)', 'log', 'k1s', '¼', '(cid1)u2s1', '(cid1)', 'n2', 'log', 'r12', '¼', 'p1ðu1', '(cid1)', 'u2þ', 'þ', 'ðn1', 'þ', 'n2þ', 'kis', 'represents', 'intrinsic', 'reactivity', 'styrene', 'polymer', 'radical', '(assuming', 'monomer', '1)', 'n2', 'represents', 'intrinsic', 'reactivity', 'monomer', '2', 's1', 'represents', 'polarity', 'polymer', 'radical', 'derived', 'monomer', '1', 'u2', 'represent', 'polarity', 'monomer', '2', 'whether', 'scheme', 'applicable', 'general', 'copolymerizations', 'clear', 'point', '363', 'solvent', 'effect', 'copolymerization', 'it', 'reported', 'barb', '1953', 'solvents', 'affect', 'rates', 'copolymerization', 'composition', 'copolymer', 'copolymerizations', 'styrene', 'maleic', 'anhydride', '145.', 'later', 'klumperman', 'also', 'observed', 'similar', 'solvent', 'effects', '145.', 'this', 'reviewed', 'coote', 'coworkers', '145.', 'a', 'number', 'complexation', 'models', 'proposed', 'describe', 'copolymerizations', 'styrene', 'maleic', 'anhydride', 'styrene', 'acrylonitrile', 'there', 'explanations', 'offered', 'deviation', 'terminal', 'model', 'assumes', 'radical', 'reactivity', 'depends', 'terminal', 'unit', 'growing', 'chain', 'thus', 'harwood', 'proposed', '“bootstrap', 'model”', 'based', 'upon', 'study', 'styrene', 'copolymerized', 'maa', 'acrylic', 'acid', 'acrylamide', '146.', 'it', 'hypothesized', 'solvent', 'modify', 'inherent', 'reactivity', 'growing', 'radical', 'affects', 'monomer', 'partitioning', 'concentrations', 'two', 'monomers', 'reactive', 'site', '(and', 'thus', 'ratio)', 'differ', 'bulk', 'hutchinson', 'coworkers', 'investigated', '147', 'effects', 'solvent', 'free-radical', 'copolymer', 'composition', 'propagation', 'kinetics', 'copolymerizations', 'styrene', 'three', 'methacrylates', '37', 'terpolymerization', '101', '2-hydroxyethyl', 'methacrylate', 'glycidyl', 'methacrylate', 'n-butyl', 'methacrylate', 'three', 'different', 'solvents', 'n-butanol', 'toluene', 'nn-dimethylformamide', 'used', 'they', 'found', 'three', 'solvents', 'effected', 'composition', 'styrene-2-hydroxyethyl', 'methacrylate', 'copolymer', 'liang', 'hutchinson', '147', 'also', 'observed', 'variations', 'monomer', 'reactivity', 'ratios', 'solvent', 'polarity', 'butanol', 'one', 'affected', 'styrene', 'butyl', 'methacrylate', 'copolymer', 'composition', 'none', 'solvents', 'appeared', 'effect', 'composition', 'styrene–glycidyl', 'methacrylate', 'copolymer', '37', 'terpolymerization', 'a', 'quantitative', 'treatment', 'terpolymerization', 'three', 'different', 'monomers', 'interpolymerized', 'becomes', 'complex', 'nine', 'growth', 'reactions', 'take', 'place', '155:', 'reaction', '(cid5)m1(cid3)', 'þ', 'm1(cid1)', '(cid5)m1(cid3)', 'þ', 'm2(cid1)', '(cid5)m1(cid3)', 'þ', 'm3(cid1)', '(cid5)m2(cid3)', 'þ', 'm1(cid1)', '(cid5)m2(cid3)', 'þ', 'm2(cid1)', '(cid5)m2(cid3)', 'þ', 'm3(cid1)', '(cid5)m3(cid3)', 'þ', 'm1(cid1)', '(cid5)m3(cid3)', 'þ', 'm2(cid1)', '(cid5)m3(cid3)', 'þ', 'm3(cid1)', 'k11', '(cid5)m1m1(cid3)', 'k12', '(cid5)m1m2(cid3)', 'k13', '(cid5)', 'm1m3(cid3)', 'k22', '(cid5)m2m1(cid3)', 'k22', '(cid5)m2m2(cid3)', 'k23', '(cid5)m2m3(cid3)', 'k31', '(cid5)m3m1(cid3)', 'k32', '(cid5)m3m2(cid3)', 'k33', '(cid5)m3m3(cid3)', 'rate', 'r11', '¼', 'k11½m1(cid3)(cid4)½m1(cid4)', 'r12', '¼', 'k12½m1(cid3)(cid4)½m2(cid4)', 'r13', '¼', 'k13½m1(cid3)(cid4)½m3(cid4)', 'r21', '¼', 'k21½m2(cid3)(cid4)½m1(cid4)', 'r22', '¼', 'k22½m2(cid3)(cid4)½m2(cid4)', 'r23', '¼', 'k23½m2(cid3)(cid4)½m3(cid4)', 'r31', '¼', 'k31½m3(cid3)(cid4)', 'm1', '½', '(cid4)', 'r32', '¼', 'k32½m3(cid3)(cid4)½m2(cid4)', 'r33', '¼', 'k33½m3(cid3)(cid4)', 'm3', '½', '(cid4)', 'there', 'six', 'reactivity', 'ratios', 'copolymerizations', 'three', 'monomers', '148:', 'r12', '¼', 'k11k12', 'r13', '¼', 'k11k13', 'r21', '¼', 'k22k21', 'r23', '¼', 'k22k23', 'r31', '¼', 'k33k31', 'r32', '¼', 'k33k32', 'the', 'rates', 'disappearance', 'three', 'monomers', 'given', '(cid1)', 'd½m1(cid4)dt', '¼', 'r11', 'þ', 'r21', 'þ', 'r31', '(cid1)', 'd½m2(cid4)dt', '¼', 'r12', 'þ', 'r22', 'þ', 'r32', '(cid1)', 'dm3(cid4)dt', '¼', 'r13', 'þ', 'r23', 'þ', 'r33', 'by', 'assuming', 'steady', 'state', 'conditions', 'three', 'radicals', 'm1•', 'm2•', 'm3•', 'possible', 'write', 'r12', 'þ', 'r13', '¼', 'r21', 'þ', 'r31', 'r21', 'þ', 'r23', '¼', 'r12', 'þ', 'r32', 'r31', 'þ', 'r32', '¼', 'r13', 'þ', 'r23', 'an', 'equation', 'terpolymer', 'composition', 'developed', 'rate', 'expressions', 'expressing', 'steady', 'state', 'relationships', '94:', 'r12', '¼', 'r21', 'r23', '¼', 'r32', 'r31', '¼', 'r13', '102', '3', 'free-radical', 'chain-growth', 'polymerization', 'd½m1(cid4)d½m2(cid4)d½m3(cid4)', '¼', '½m1(cid4)f½m1(cid4)', 'þ', '½m2(cid4)r12', 'þ', '½m3(cid4)r13g', '½m2(cid4)r21r12f½m1(cid4)r21', 'þ', '½m2(cid4)', 'þ', '½m3(cid4)r23g', '½m3(cid4)r31r13f½m1(cid4)r31', 'þ', '½m2(cid4)r32', 'þ', '½m3(cid4)g', 'it', 'claimed', 'terpolymerization', 'composition', 'equation', 'often', 'good', 'agreement', 'experimental', 'results', 'other', 'complicated', 'equations', 'also', 'exist', 'apparently', 'yield', 'results', 'similar', 'obtained', 'shown', 'expression', '94', '149.', 'one', 'example', 'equations', 'early', 'theoretical', 'relationship', 'terpolymerization', 'written', 'alfrey', 'goldﬁnger', '150.', 'b', '¼', 'a½aðr31r21þ', 'þ', 'bðr21r32þ', 'þ', 'cðr31r23þ(cid4)½a', 'þ', 'br12', 'þ', 'cr13(cid4)', 'b½ðaðr12r31þ', 'þ', 'bðr12r32þ', 'þ', 'cðr32r13þ(cid4)½b', 'þ', 'ar21', 'þ', 'cr23(cid4)', 'c', '¼', 'a½aðr31r21þ', 'þ', 'bðr21r32þ', 'þ', 'cðr31r23þ(cid4)½a', 'þ', 'br12', 'þ', 'cr13(cid4)', 'c½aðr13r21þ', 'þ', 'bðr23r12þ', 'þ', 'cðr13r23þ(cid4)½c', 'þ', 'ar13', 'þ', 'br32(cid4)', 'b', 'c', 'quantities', 'monomer', 'found', 'resultant', 'terpolymer', 'a', 'b', 'c', 'quantities', 'monomer', 'feed', 'system', 'needless', 'say', 'copolymerization', 'four', 'even', 'monomers', 'becomes', 'progressively', 'difﬁcult', 'treat', 'rigorously', '38', 'allylic', 'polymerization', 'compounds', 'possessing', 'allylic', 'structures', 'polymerize', 'free-radical', 'mechanism', 'low', 'molecularweight', 'oligomers', 'in', 'cases', 'products', 'consist', 'mostly', 'dimers', 'trimers', 'the', 'dp', 'poly(allyl', 'acetate)', 'instance', '14', 'this', 'due', 'fact', 'allylic', 'monomer', 'radicals', 'resonance-stabilized', 'extent', 'extensive', 'chain', 'propagations', 'occur', 'instead', 'large', 'amount', 'chain', 'transferring', 'such', 'chain', 'transferring', 'essentially', 'terminates', 'reactions', '151.', 'the', 'resonance', 'stabilization', 'illustrated', 'allyl', 'alcohol', 'radical', 'the', 'hydrogen', 'transfer', 'takes', 'place', 'allylic', 'hydrogen', 'shown', 'allyl', 'acetate', 'oh', 'oh', 'r', 'o', 'o', 'rh', 'o', 'o', 'o', 'o', '39', 'inhibition', 'retardation', '103', 'hydrogen', 'transfer', 'also', 'take', 'place', 'acetate', 'moiety', 'r', 'rh', 'o', 'o', 'o', 'o', 'o', 'o', 'the', 'described', 'chain', 'transferring', 'called', 'degradative', 'chain', 'transferring', 'other', 'monomers', 'methyl', 'methacrylate', 'methacrylonitrile', 'also', 'contain', 'allylic', 'carbon–hydrogen', 'bonds', 'they', 'fail', 'undergo', 'extensive', 'degradative', 'chain', 'transferring', 'however', 'form', 'high-molecular-', 'weight', 'polymers', 'this', 'believed', 'due', 'lower', 'reactivity', 'propagating', 'radicals', 'form', 'monomers', '5.', 'yamasaki', 'et', 'al', '152,', 'reported', 'successfully', 'performed', 'radical', 'polymerization', 'allylbiguanide', 'hydrochloride', 'concentrated', 'acid', 'solution', 'using', 'either', 'hydrochloric', 'acid', 'phosphoric', 'acid', 'presence', 'radical', 'initiator', '50(cid6)c', 'the', 'polymer', 'precipitated', 'reaction', 'solution', 'addition', 'excess', 'amount', 'acetone', 'the', 'molecular', 'weight', 'average', 'product', '10340–113200', 'low', 'polydispersity', '104–168', 'in', 'spite', 'degradative', 'chain', 'transferring', 'polyallyl', 'compounds', 'readily', 'polymerized', 'free-radical', 'mechanism', 'three-dimensional', 'lattices', 'high', 'dp', 'necessary', 'achieve', 'growth', 'three', 'dimensions', 'an', 'example', 'polyallyl', 'compounds', 'triallyl', 'phosphate', 'o', 'o', 'p', '3', 'many', 'polyallyl', 'derivatives', 'offered', 'commercially', 'use', 'cross-linked', 'ﬁlms', 'described', 'trade', 'literature', '39', 'inhibition', 'retardation', 'free-radical', 'polymerizations', 'subject', 'inhibition', 'retardation', 'side', 'reactions', 'various', 'molecules', '54.', 'such', 'polymerization', 'suppressors', 'classiﬁed', 'according', 'effect', 'exert', 'upon', 'reaction', 'inhibitors', 'compounds', 'react', 'rapidly', 'every', 'initiating', 'free', 'radical', 'forms', 'this', 'prevents', 'polymerization', 'reaction', 'taking', 'place', 'inhibitor', 'completely', 'consumed', 'process', 'the', 'reactions', 'inhibitors', 'initiating', 'radicals', 'result', 'formations', 'new', 'free', 'radicals', 'the', 'newly', 'formed', 'free', 'radicals', 'however', 'stable', 'initiate', 'chain', 'growths', 'as', 'result', 'well-deﬁned', 'induction', 'periods', 'exist', 'after', 'inhibitors', 'used', 'polymerizations', 'proceed', 'normal', 'rates', 'retarders', 'compounds', 'also', 'react', 'initiating', 'radicals', 'they', 'react', 'however', 'energetically', 'inhibitors', 'initiating', 'radicals', 'escape', 'start', 'chain', 'growth', 'this', 'affects', 'general', 'rate', 'reaction', 'slows', 'there', 'induction', 'period', 'retarders', 'active', 'throughout', 'course', 'polymerization', '104', '3', 'free-radical', 'chain-growth', 'polymerization', 'the', 'efﬁciency', 'inhibitor', 'depends', 'upon', 'three', 'factors', '(1)', 'chain', 'transfer', 'constant', 'inhibitor', 'respect', 'particular', 'monomer', '(2)', 'reactivity', 'inhibitor', 'radical', 'forms', '(3)', 'reactivity', 'particular', 'monomer', 'phenols', 'arylamines', 'common', 'chain', 'transfer', 'inhibitors', 'the', 'reaction', 'phenols', 'though', 'fully', 'elucidated', 'believed', 'follows', '153', 'x', 'ho', 'r', 'x', 'o', 'r', 'r', 'r', 'r', 'r', 'x', 'r', 'r', 'r', 'o', 'o', 'r', 'r', 'r', 'r', 'o', 'r', 'r', 'r', 'r', 'r', 'o', 'x', 'r', 'x', 'x', 'o', 'r', 'r', '1', '2', '3', 'quinones', 'effective', 'inhibitors', 'many', 'polymerization', 'reactions', 'the', 'reaction', 'occurs', 'either', 'oxygen', 'ring', 'carbon', '153–156:', 'h', 'x', 'o', 'o', 'r', 'o', 'o', 'o', 'x', 'r', 'r', 'x', 'the', 'reaction', 'however', 'always', 'strict', 'inhibition', 'thus', 'instance', 'hydroquinone', 'acts', 'efﬁcient', 'inhibitor', 'methyl', 'methacrylate', 'radical', 'retarder', 'styrene', '39', 'inhibition', 'retardation', '105', 'radical', '155.', 'hydroquinone', 'often', 'employed', 'inhibitor', 'requires', 'however', 'oxygen', 'activity', '156', '157:', '02', 'h', 'x', 'h', 'o', 'x', 'o', 'ho', 'oh', 'h', 'o', 'x', 'oh', 'ho', 'o', 'oxygen', 'however', 'also', 'act', 'comonomer', 'styrene', 'polymerization', 'o2', 'o', 'o', 'h', 'o', 'x', 'o', 'o', 'o', 'it', 'causes', 'marked', 'retardation', 'however', 'polymerizations', 'methyl', 'methacrylate', '158.', 'the', 'true', 'many', 'free-radical', 'polymerizations', 'the', 'ability', 'phenols', 'inhibit', 'free-radical', 'polymerizations', 'appears', 'increase', 'number', 'hydroxyl', 'groups', 'molecules', '157.', 'the', 'locations', 'hydroxyl', 'groups', 'benzene', 'rings', 'relationships', 'important', 'for', 'instance', 'catechol', 'efﬁcient', 'inhibitor', 'resorcinol', '158.', 'aromatic', 'nitro', 'compounds', 'act', 'strong', 'retarders', 'their', 'effect', 'proportional', 'quantity', 'nitro', 'groups', 'per', 'molecule', '160', '161.', 'x', 'o2n', 'o', 'n', 'x', 'o', 'x', 'o', 'n', 'o', 'figure', '31', 'illustrates', 'effect', 'inhibitors', 'retarders', 'free-radical', 'polymerization', '162.', 'the', 'equation', 'relates', 'rate', 'data', 'inhibited', 'polymerizations', '2rp', '2ktkp', '2½m(cid4)2', 'þ', 'rp½z(cid4)kzkp½m(cid4)', '(cid1)', 'ri', '¼', '0', '106', '3', 'free-radical', 'chain-growth', 'polymerization', 'fig', '31', 'illustration', 'effects', 'inhibitors', 'retarders', '(a)', 'normal', 'polymerization', 'rate', '(b)', 'effect', 'retarder', '(c)', 'effect', 'ideal', 'inhibitor', '(d)', 'effect', 'non', 'ideal', 'inhibitor', 'the', 'time', 'a', 'c', 'induction', 'period', 'caused', 'ideal', 'inhibitor', 'conversion', 'a', 'c', 'b', 'd', 'time', 'induct', 'period', 'z', 'inhibitor', 'retarder', 'chain-growth', 'termination', '(cid5)', 'mn(cid3)', 'þ', 'z(cid1)', 'kz', '(cid5)', 'mn', 'þ', 'z(cid3)ðandor', '(cid5)', 'mnz(cid3)þ', 'to', 'simplify', 'kinetics', 'assumed', 'z•', 'mnz•', 'initiate', 'new', 'chain', 'growth', 'regenerate', 'z', 'upon', 'termination', '310', 'thermal', 'polymerization', 'a', 'monomers', 'styrene', 'methyl', 'methacrylate', 'careful', 'puriﬁcation', 'presumably', 'free', 'impurities', 'polymerize', 'elevated', 'temperatures', 'it', 'supposed', 'ring-substituted', 'styrenes', 'act', 'similarly', 'the', 'rates', 'thermal', 'self-initiated', 'polymerizations', 'slower', 'carried', 'aid', 'initiators', 'styrene', 'instance', 'polymerizes', 'rate', '01', 'per', 'hour', '60(cid6)c', '14', '127(cid6)c', 'the', 'rate', 'thermal', 'polymerization', 'methyl', 'methacrylate', '1', 'rate', 'styrene', '163', '164.', 'several', 'mechanisms', 'initiation', 'proposed', 'earlier', 'the', 'subject', 'reviewed', 'critically', '165.', 'more', 'recently', 'initiation', 'mechanism', 'styrene', 'polymerization', 'shown', 'ultraviolet', 'spectroscopy', 'consist', 'initial', 'formation', 'diels–alder', 'dimer', 'the', 'dimer', 'believed', 'subsequently', 'transfer', 'hydrogen', 'styrene', 'molecule', 'form', 'free', 'radical', '166:', 'ch2', 'h', 'styrene', '2', 'h', 'dimer', 'thermal', 'polymerization', 'methyl', 'methacrylate', 'hand', 'appears', 'proceed', 'initial', 'dimerization', 'diradical', '167.', 'this', 'followed', 'hydrogen', 'abstraction', 'available', 'source', 'reaction', 'mixture', '311', 'donor–acceptor', 'complexes', 'copolymerization', '107', 'o', 'o', 'rh', 'o', 'o', 'o', 'o', 'hydrogen', 'abstraction', 'o', 'o', 'o', 'o', '311', 'donor–acceptor', 'complexes', 'copolymerization', 'polar', 'interactions', 'electron', 'donor', 'monomers', 'electron', 'acceptor', 'monomers', 'lead', 'strong', 'tendencies', 'toward', 'formations', 'alternating', 'copolymers', 'also', 'alternating', 'copolymerizations', 'might', 'even', 'result', 'compounds', 'capable', 'conventional', 'polymerization', 'an', 'example', 'copolymerization', 'dioxene', 'maleic', 'anhydride', 'two', 'reaction', 'mechanisms', 'proposed', 'one', 'suggests', 'interactions', 'donor', 'monomers', 'acceptor', 'radicals', 'acceptor', 'monomers', 'donor', 'radicals', 'lead', 'decreased', 'energies', 'activation', 'cross-propagations', '168.', 'the', 'transition', 'state', 'stabilized', 'partial', 'electron', 'transfer', 'donor', 'acceptor', 'species', '169.', 'the', 'second', 'mechanism', 'suggests', 'interactions', 'result', 'formations', 'charge-transfer', 'complexes', '170.', 'an', 'electron', 'completely', 'transferred', 'donor', 'monomer', 'acceptor', 'monomer', 'after', 'transfer', 'complex', 'converts', 'diradical', 'subsequently', 'polymerizes', 'intermolecular', 'coupling', 'for', 'instance', 'many', 'believe', 'diels–alder', 'reaction', 'takes', 'place', 'concerted', 'mechanism', 'intermediate', 'postulated', 'charge-transfer', 'complex', 'an', 'electron', 'transferred', 'donor', 'acceptor', 'charge-transfer', 'complex', 'forms', '171.', 'this', 'illustrated', 'diels–alder', 'reaction', 'butadiene', 'maleic', 'anhydride', 'o', 'o', 'o', 'donor', 'acceptor', 'o', 'o', 'o', 'charge', 'transfer', 'complex', 'δ', 'diels', 'alder', 'adduct', 'o', 'o', 'o', 'if', 'reaction', 'mixture', 'irradiated', 'high', 'energy', 'radiation', 'gamma', 'rays', 'instead', 'heated', 'alternating', 'copolymer', 'forms', 'the', 'complex', 'converts', 'diradical', '1', '171', 'homopolymerizes', 'o', 'o', 'o', 'γ-rays', 'polymerization', 'o', 'o', 'o', 'o', 'o', 'o', 'n', '108', '3', 'free-radical', 'chain-growth', 'polymerization', 'alternating', 'copolymerization', 'styrene', 'maleic', 'anhydride', 'also', 'explained', 'donor', 'acceptor', 'interactions', '171.', 'a', 'charge-transfer', 'complex', 'seen', 'new', 'monomer', 'diradical', 'polymerizes', 'coupling', '171–174.', 'donor', 'acceptor', 'charge', 'transfer', 'complex', 'o', 'o', 'o', 'o', 'o', 'o', 'diradical', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'charge-transfer', 'complexes', 'also', 'claimed', 'intermediates', 'free-radical', 'alternating', 'copolymerization', 'dioxene', 'vinyl', 'ethers', 'maleic', 'anhydride', '176–179:', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'r', 'donor', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'r•', 'polymerization-initiating', 'radical', 'here', 'third', 'monomer', 'included', 'interpolymerize', 'complex', 'acts', 'unit', 'the', 'product', 'terpolymer', '176', '177.', 'a', 'diradical', 'intermediate', 'also', 'postulated', 'sulfur', 'dioxide', 'copolymerizations', 'terpolymer-', 'izations', 'bicycloheptene', 'third', 'monomers', '173.', 'these', 'third', 'monomers', 'enter', 'copolymer', 'chain', 'block', 'segments', 'donor–acceptor', 'pairs', 'enter', 'chains', 'one-to-', 'one', 'molar', 'ratio', 'this', 'one-to-one', 'molar', 'ratio', 'pairs', 'maintained', 'regardless', 'overall', 'nature', 'monomer', 'mixtures', 'the', 'propagation', 'termination', 'steps', 'reactions', 'claimed', '175', '181–183', 'related', 'as', 'stated', 'interaction', 'coupling', 'two', 'diradicals', 'propagation', 'step', 'when', 'interactions', 'result', 'disproportionations', 'however', 'termination', 'steps', 'this', 'means', 'charge-', 'transfer', 'mechanisms', 'different', 'conventional', 'free-radical', 'polymerizations', 'they', 'involve', 'interactions', 'growing', 'polymer-radicals', 'monomers', 'also', 'polymer-', 'radicals', 'complexes', 'in', 'addition', 'polymer', 'radicals', 'react', '175', '181–184.', '311', 'donor–acceptor', 'complexes', 'copolymerization', '109', 'li', 'turner', '185,', 'reported', 'copolymerization', 'maleic', 'anhydride', 'trans-stilbene', 'r1', 'thf', 'aibn', 'r1', 'o', 'o', 'o', 'r2', 'o', 'o', 'o', 'r2', 'n', 'r3', 'r3', 'r1', 'r2', 'r3', 'either', 'methyl', 'groups', 'hydrogens', 'the', 'stability', 'charge-transfer', 'complexes', 'depends', 'upon', 'internal', 'resonance', 'stabilization', 'this', 'degree', 'stabilization', 'determines', 'easily', 'diradicals', 'open', '183.', 'consequently', 'stability', 'also', 'determines', 'copolymerization', 'occurs', 'it', 'occur', 'spontaneously', 'inﬂuence', 'light', 'heat', 'attack', 'initiating', 'free', 'radical', 'there', 'many', 'examples', 'spontaneous', 'reactions', 'when', 'instance', 'isobutylene', 'added', 'methyl', 'a-cyanoacrylate', 'spontaneous', 'copolymerization', '11', 'ratio', 'takes', 'place', 'room', 'tempera-', 'ture', 'this', 'explained', 'following', 'scheme', '175', '181–183:', 'n', 'o', 'och3', 'och3', 'o', 'n', 'n', 'o', 'och3', 'och3', 'o', 'n', 'n', 'the', 'happens', 'sulfur', 'dioxide', 'added', 'bicycloheptene', '(cid1)40(cid6)c', '184.', 'donor', 'acceptor', 'so2', 'so2', 'so2', 'n', 'o', 's', 'o', 'another', 'example', 'room', 'temperature', '11', 'copolymerization', 'vinylidene', 'cyanide', 'sty-', '3', 'free-radical', 'chain-growth', 'polymerization', '110', 'rene', '185:', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'still', 'another', 'example', 'reaction', '13', 'dioxalene', 'maleic', 'anhydride', '183.', 'examples', 'stable', 'complexes', 'reactions', 'sulfur', 'dioxide', 'styrene', '175,', 'vinyl', 'ethers', 'maleic', 'anhydride', '184,', 'also', 'a-oleﬁns', 'maleic', 'anhydride', '179–181.', 'also', 'reaction', 'trans-stilbene', 'maleic', 'anhydride', '182,', 'in', 'reactions', 'charge-transfer', 'complexes', 'form', 'they', 'stable', 'existence', 'detected', 'spectroscopic', 'means', 'additional', 'energy', 'heat', 'free-radical', 'attack', 'converts', 'diradicals', 'polymerizes', 'alternating', 'copolymers', '175', '185–191.', 'examples', 'intermediates', 'two', 'extremes', 'stability', 'reaction', 'products', 'sulfur', 'dioxide', 'conjugated', 'dienes', '192.', 'in', 'case', 'reaction', 'results', 'formation', 'mixtures', 'alternating', 'copolymers', 'cyclic', 'adducts', 'o', 's', 'o', 'o', 'o', 's', 'o', 'o', 's', 's', 'n', 'o', 'o', 'the', 'yield', 'polymer', 'increases', 'expense', 'cyclic', 'structure', 'heat', 'radiation', 'applied', 'a', 'free-radical', 'attack', 'effect', '192.', '312', 'polymerization', 'complexes', 'lewis', 'acids', '111', 'if', 'donor–acceptor', 'interactions', 'subsequent', 'polymerizations', 'occur', 'upon', 'irradiation', 'ultra-', 'violet', 'light', 'reactions', 'selective', 'an', 'example', 'triphenylphosphine', 'interaction', 'acrylic', 'monomers', '193:', 'o', 'o', 'p', 'hν', '3', '3', 'o', 'o', 'p', 'o', 'o', 'p', '3', 'this', 'reaction', 'occur', 'however', 'triphenylphosphine', 'styrene', 'vinyl', 'acetate', '193.', 'the', 'nature', 'amount', 'solvent', 'inﬂuence', 'yield', 'composition', 'copolymers', 'copolymerizations', 'thus', 'copolymerization', 'phenanthrene', 'maleic', 'anhy-', 'dride', 'benzene', 'yields', '12', 'adduct', 'in', 'dioxane', 'however', '11', 'adduct', 'obtained', 'in', 'dimethyl', 'formamide', 'copolymer', 'forms', '193.', 'another', 'example', 'terpolymerization', 'acrylonitrile', '2-chloroethyl', 'vinyl', 'ether', 'maleic', 'anhydride', 'p-dioxene-maleic', 'anhydride', 'the', 'amount', 'acrylonitrile', 'terpolymer', 'increases', 'increase', 'p-electron', 'density', 'solvent', 'following', 'order', '194:', 'xylene', '(cid13)', 'toluene', '(cid13)', 'benzene', 'chlorobenzene', '(cid13)', 'chloroform', 'the', 'ratio', 'maleic', 'anhydride', 'vinyl', 'ether', 'product', 'remains', 'however', 'equimolar', 'whether', 'concept', 'charge-transfer', 'complexes', 'copolymerizations', 'fully', 'accepted', 'certain', 'much', 'accumulated', 'evidence', 'date', 'uv', 'nmr', 'spectroscopy', 'support', 'many', 'systems', '195.', 'further', 'support', 'comes', 'strong', 'tendencies', 'form', 'alternating', 'copolymers', 'wide', 'range', 'feed', 'compositions', 'also', 'high', 'reaction', 'rates', 'equimolar', 'feed', 'compositions', '171.', 'on', 'hand', 'shown', 'claimed', 'past', 'copolymerization', 'styrene', 'maleic', 'anhydride', 'involves', 'charge-transfer', 'complexes', '171', '181–183.', 'this', 'however', 'contradicted', 'publication', 'study', 'radical', 'copolymerization', 'maleic', 'acid', 'styrene', 'the', 'reaction', 'carried', 'dioxane', 'solution', '70(cid6)c', 'the', 'authors', 'reported', 'uv', 'spectroscopy', 'fails', 'show', 'presence', 'charge', 'transfer', 'formation', 'complex', 'two', 'monomers', 'copolymerization', 'system', '196.', '312', 'polymerization', 'complexes', 'lewis', 'acids', 'some', 'polar', 'vinyl', 'monomers', 'methyl', 'methacrylate', 'acrylonitrile', 'interact', 'complex', 'lewis', 'acids', 'they', 'subsequently', 'polymerize', 'faster', 'rate', 'higher', 'molecular', 'weight', 'expected', 'otherwise', 'the', 'effective', 'lewis', 'acids', 'zncl2', 'alcl3', 'al(c2h5)2cl', 'alcl2(c2h5)', 'sncl4', 'others', '197–199.', 'complexes', 'form', 'equimolar', 'basis', 'undergo', '112', '3', 'free-radical', 'chain-growth', 'polymerization', 'homopolymerizations', 'application', 'heat', 'a', 'free-radical', 'attack', 'irradiation', 'ultraviolet', 'light', 'gamma', 'rays', 'also', 'effective', '198.', 'it', 'proposed', '198–201', 'greater', 'reactivity', 'due', 'delocalization', 'electrons', 'complexed', 'molecules', 'n', 'zncl2', 'n', 'zncl2', 'molecular', 'orbital', 'calculations', 'support', '200', '201.', 'polymerizations', 'also', 'carried', 'less', 'stoichiometric', 'ratios', 'lewis', 'acids', 'monomers', 'as', 'illustration', 'coupling', 'acrylic', 'methacrylic', 'monomer', 'lewis', 'acid', 'shown', 'follows', '203', '204', 'am', 'þ', 'mexn', 'ð', 'ðm)a', '(cid1)', '(cid1)', '(cid1)', '(cid1)', '(cid1)', '(cid1)mexn', 'mexn', 'represents', 'lewis', 'acid', 'vinyl', 'pyridine', 'vinyl', 'imidazole', 'also', 'form', 'complexes', 'zinc', 'chloride', 'here', 'conjugation', 'metal', 'salt', 'results', 'super', 'delocalizability', 'leads', 'spontaneous', 'thermal', 'polymerization', '203', '204.', 'the', 'increased', 'reactivity', 'toward', 'polymerization', 'monomers', 'even', 'presence', 'less', 'equimolar', 'amounts', 'lewis', 'acids', 'suggested', 'due', 'one', 'two', 'possibilities', 'it', 'may', 'due', 'greater', 'reactivity', 'portion', 'monomer', 'complexes', 'toward', 'growing', 'chain', 'end', 'it', 'may', 'also', 'due', 'formation', 'new', 'complexes', 'uncomplexed', 'monomer', 'complexed', 'ones', '171:', 'n', 'n', 'n', 'zncl2', 'n', 'zncl2', 'the', 'picture', 'complicated', 'lewis', 'acids', 'used', 'combinations', 'donor–acceptor', 'monomers', 'the', 'donor–acceptor', 'complexes', 'believed', 'form', 'ﬁrst', 'react', 'lewis', 'acids', 'according', 'following', 'scheme', '203', '204:', 'am1', 'bm2', 'mexn', 'm1', 'mexn', 'bm2', 'mexn', 'b', 'm2', 'am1', 'a', 'structure', 'methyl', 'methacrylate', 'styrene', 'complexed', 'sncl4', 'shown', 'follows', '204:', 'o', 'sn', 'o', 'o', 'o', '313', 'steric', 'control', 'free-radical', 'polymerization', '113', 'in', 'formations', 'ternary', 'complexes', 'acceptor', 'vinyl', 'compound', 'must', 'double', 'bond', 'conjugated', 'cyano', 'carbonyl', 'group', 'such', 'acceptors', 'acrylonitrile', 'methacrylonitrile', 'acrylic', 'methacrylic', 'esters', 'acids', 'methyl', 'vinyl', 'ketone', 'acrylamide', 'etc', 'donor', 'monomers', 'styrene', 'a-methyl', 'styrene', 'butadiene', '2–3-dimethyl', 'butadiene', 'isoprene', 'chloroprene', 'etc', 'one', 'proposed', 'mechanism', '171', '204', 'charge-transfer', 'polymerizations', 'effect', 'a.', 'mexn.', 'in', 'words', 'metal', 'homopolymerizations', 'charge-transfer', 'complexes', 'd', 'halide', 'complexed', 'electron', 'acceptor', 'monomer', 'acts', 'acceptor', 'component', 'l', 'the', 'opinion', 'however', 'universal', 'others', 'hold', 'increased', 'susceptibility', 'ultraviolet', 'radiation', 'initiating', 'radicals', '205', 'due', 'increased', 'reactivity', 'propagating', 'radicals', 'complexed', 'monomers', 'toward', 'incoming', 'uncomplexed', 'ones', 'arguments', 'ternary', 'complex', 'mechanism', 'follows', '(1)', 'physical', 'evidence', 'proves', 'existence', 'ternary', 'molecular', 'complexes', 'weak', '(2)', 'ternary', 'molecular', 'complexes', 'bearing', 'copolymerizations', 'equilibrium', 'concentration', 'complexed', 'monomers', 'low', 'compared', 'uncomplexed', 'ones', '206.', 'a', 'third', 'opinion', 'complex', 'acceptor', 'monomer', 'lewis', 'acid', 'copolymerizes', 'alternately', 'donor', 'monomer', 'uncomplexed', 'acceptor', 'monomer', '207', '208.', 'this', 'presumably', 'takes', 'place', 'according', 'conventional', 'chain-growth', 'polymerization', 'scheme', 'radical', 'copolymerization', 'the', 'alternate', 'placement', 'monomers', 'due', 'highly', 'enhanced', 'values', 'cross-', 'propagation', 'constants', 'it', 'results', 'complexing', 'acceptor', 'monomers', 'lewis', 'acids', 'such', 'mechanism', 'fails', 'explain', 'satisfactorily', 'completely', 'alternating', 'incorporation', 'monomers', 'inefﬁciency', 'chain-transfer', 'reagents', 'it', 'also', 'fails', 'explain', 'spontaneous', 'initiation', 'alternating', 'copolymerization', 'kabanov', 'suggested', '209', 'lewis', 'acid–monomer', 'complex', 'uncomplexed', 'complexed', 'monomers', 'may', 'participate', 'chain', 'propagation', 'this', 'would', 'result', 'appearance', 'complexed', 'propagating', 'radicals', 'besides', 'usual', 'ones', 'in', 'complexed', 'ones', 'last', 'unit', 'carrying', 'valence', 'ligand', 'coordination', 'complex', 'primary', 'free-radical', 'formation', '(cid5)m(cid3)', 'mexn', 'it', 'excludes', 'however', 'electron', 'transfer', 'reactions', 'may', 'take', 'place', 'due', 'ultraviolet', 'light', 'irradiation', '313', 'steric', 'control', 'free-radical', 'polymerization', 'in', 'free-radical', 'polymerization', 'reactions', 'propagating', 'radical', 'chain', 'great', 'amount', 'freedom', 'atactic', 'polymers', 'therefore', 'usually', 'formed', 'some', 'control', 'reaction', 'conditions', 'exercise', 'propagating', 'species', 'increases', 'lower', 'temperatures', 'due', 'lower', 'mobilities', 'this', 'leads', 'increased', 'syndiotactic', 'placement', 'discussed', 'section', 'propagation', 'special', 'techniques', 'however', 'use', 'canal', 'complexes', 'employed', 'form', 'stereoregular', 'polymers', 'free-', 'radical', 'mechanism', 'urea', 'thiourea', 'used', 'originally', 'purposes', '210', '211.', 'monomers', 'butadiene', 'others', 'form', 'complexes', 'within', 'voids', 'canals', 'crystal', 'lattices', 'compounds', 'brief', 'exposure', 'high', 'energy', 'radiation', 'initiates', 'chain', 'growth', 'in', 'canals', 'monomer', 'molecules', 'held', 'ﬁxed', 'positions', 'chain', 'growth', 'restricted', 'one', 'direction', 'steric', 'control', 'exercised', 'ﬁxed', 'positions', 'monomer', 'molecules', 'tend', 'align', 'uniformly', 'it', 'suggested', 'canal', 'complexes', 'monomers', 'lined', 'end', 'end', 'packed', 'overlapping', 'arrangement', 'for', 'molecules', 'isobutylene', 'vinylidene', 'chloride', '114', '3', 'free-radical', 'chain-growth', 'polymerization', 'may', 'possible', 'monomers', 'lie', 'directly', 'top', 'resembling', 'stack', 'coins', 'such', 'stacking', 'greatly', 'facilitates', 'reactions', 'guest', 'molecules', '210', '211.', 'polymerizations', 'thiourea', 'canal', 'complexes', 'yields', 'high', 'melting', 'crystalline', 'trans-14', 'polybutadi-', 'ene', '23-dimethylbutadiene', '23-dichlorobutadiene', '13-cyclohexadiene', 'cyclohexadiene', 'mon-', 'oxide', 'vinyl', 'chloride', 'acrylonitrile', 'also', 'form', 'stereoregular', 'polymers', 'on', 'hand', 'polymerizations', 'isobutylene', 'vinylidene', 'chloride', 'fail', 'yield', 'stereospeciﬁc', 'polymers', 'sodium', 'montmorillonite', 'also', 'used', 'polymerize', 'polar', 'monomers', 'lamellae', 'here', 'organization', 'monomer', 'molecules', 'within', 'monolayers', 'inﬂuences', 'structure', 'resultant', 'polymers', '212', '213.', 'poly(methyl', 'methacrylate)', 'formed', 'sodium', 'montmorillonite', 'composed', 'short', 'predominantly', 'isotactic', 'stereosequences', '211.', 'the', 'percentage', 'isotactic', 'component', 'increases', 'increase', 'ion', 'exchanging', 'population', 'surface', 'mineral', 'independent', 'temperature', '20', '160(cid6)c', 'in', 'way', 'possible', 'vary', 'population', 'isotactic', 'triads', '50', 'composition', '205.', 'perhydrotriphenylene', 'also', 'forms', 'channel-like', 'inclusions', 'conjugated', 'dienes', 'polymerization', 'dienes', 'yields', 'steric', 'control', '216', '218.', 'uemura', 'coworkers', '217', 'carried', 'radical', 'polymerizations', 'vinyl', 'monomers', '(styrene', 'methyl', 'methacrylate', 'vinyl', 'acetate)', 'within', 'various', 'nanochannels', 'porous', 'coordination', 'polymers', 'they', 'studied', 'relationships', 'channel', 'size', 'polymerization', 'behaviors', 'monomer', 'reactivity', 'molecular', 'weight', 'stereostructures', 'they', 'reported', 'polymerization', 'systems', 'polymer-growing', 'radicals', 'remarkably', 'stabilized', 'efﬁcient', 'suppression', 'termination', 'reactions', 'within', 'channels', 'resulting', 'relatively', 'narrow', 'molecular', 'weight', 'distributions', 'a', 'signiﬁcant', 'nanochannel', 'effect', 'polymer', 'stereoregularity', 'also', 'seen', 'leading', 'clear', 'increase', 'isotactic', 'placement', 'resulting', 'polymers', 'there', 'attempts', 'controlling', 'steric', 'placement', 'technique', 'called', 'template', 'polymeriza-', 'tion', 'an', 'example', 'methyl', 'methacrylate', 'polymerization', 'presence', 'isotactic', 'poly(methyl', 'methacrylate)', '208', '209.', 'thus', 'template', 'polymerization', 'process', 'polymerizing', 'monomer', 'presence', 'polymer', 'usually', 'different', 'monomer', 'the', 'presence', 'template', 'polymers', 'however', 'results', 'accelerating', 'rates', 'polymerizations', '219.', '314', 'controlled“living”', 'free-radical', 'polymerization', 'living', 'polymerizations', 'chain-growth', 'reactions', 'propagating', 'centers', 'growing', 'chains', 'terminate', 'undergo', 'chain', 'transfer', 'such', 'polymerizations', 'noted', 'preparations', 'polymers', 'controlled', 'molecular', 'weights', 'desired', 'end', 'groups', 'low', 'polydispersities', 'in', 'addition', 'preparations', 'polymers', 'predetermined', 'molecular', 'weights', 'narrow', 'molecular', 'weight', 'distributions', 'require', 'fast', 'initiations', 'fast', 'exchanges', 'sites', 'variable', 'activities', 'variable', 'lifetimes', 'such', 'chain-growth', 'reactions', 'ionic', 'nature', 'discussed', 'chap', '4', 'in', 'typical', 'homogeneous', 'free', 'radical', 'polymerizations', 'however', 'bimolecular', 'terminations', 'two', 'growing', 'radicals', 'cannot', 'avoided', 'therefore', 'typical', 'living', 'free', 'radical', 'polymeri-', 'zation', 'cannot', 'fully', 'realized', 'also', 'conventional', 'free', 'radical', 'polymerizations', 'initiations', 'slow', 'high-molecular-weight', 'polymers', 'form', 'shortly', 'start', 'reactions', 'as', 'reactions', 'progress', 'polydispersities', 'increase', 'molecular', 'weights', 'actually', 'decrease', 'it', 'possible', 'however', 'adjust', 'conditions', 'radical', 'polymerizations', 'way', 'polymers', 'controlled', 'molecular', 'weights', 'relatively', 'low', 'polydispersities', 'form', '220', '221.', 'these', 'true', 'living', 'polymerization', 'termination', 'reactions', 'occur', 'they', 'possess', 'however', 'characteristics', 'similar', 'living', 'polymerizations', 'referred', 'many', 'controlled“living”', 'polymerizations', 'such', 'reactions', 'yield', 'polymers', 'controlled', 'molecular', 'weights', 'exhibit', 'increase', 'molecular', 'weight', 'conversion', 'yield', 'narrow', 'molecular', 'weight', '314', 'controlled“living”', 'free-radical', 'polymerization', '115', 'distributions', 'used', 'form', 'copolymers', 'some', 'examples', 'polymerizations', 'include', 'nitroxyl', 'radical-mediated', 'polymerizations', 'styrene', '222–225,', 'atom', 'transfer', 'polymerizations', 'controlled', 'ruthenium-(ii)aluminum', '226', '227', 'copperbipyridine', 'complexes', '228,', 'co', '(ii)-mediated', 'polymerizations', 'methacrylates', 'acrylates', '229,', 'polymerization', 'styrene', 'using', 'degenerative', 'transfer', 'method', '230,', 'well', 'others', 'some', 'features', 'unusual', 'radical', 'processes', 'radical', 'nature', 'reactions', 'might', 'questioned', 'instance', 'polymerizations', 'catalyzed', 'transition', 'metals', 'evidence', 'presented', 'however', 'strongly', 'indicates', 'radical', 'nature', 'least', 'atom', 'transfer', 'polymerizations', '231.', 'the', 'evidence', 'however', 'unambiguous', 'some', 'initial', 'attempts', 'producing', '“living”', 'polymerizations', 'made', 'use', 'iniferters', 'this', 'term', 'appears', 'come', 'word', 'inifer', 'bifunctional', 'compound', 'brings', 'initiation', 'chain', 'transfer', '“living”', 'cationic', 'polymerizations', 'make', 'use', 'inifers', 'form', 'block', 'copolymers', 'this', 'discussed', 'chap', '9', 'the', 'term', 'iniferter', 'proposed', 'otsu', 'yoshida', '1982', '232.', 'iniferters', 'used', 'controlledliving', 'free-radical', 'polymerizations', 'sulfur-centered', 'free', 'radicals', 'generated', 'sulfur-containing', 'molecules', 'dithiocarbamates', 'the', 'radicals', 'react', 'reversibly', 'growing', 'polymeric', 'chain', 'ends', 'thereby', 'controlling', 'concentration', 'radical', 'species', 'many', 'sulfur', 'centered', 'radicals', 'however', 'also', 'initiate', 'new', 'polymer', 'chains', 'this', 'lead', 'uncontrolled', 'growth', 'to', 'overcome', 'difﬁculties', 'approaches', 'also', 'developed', '233.', 'deactivation', 'growing', 'radicals', 'stable', 'radicals', 'carried', 'aid', 'various', 'nitroxyl', 'radicals', 'protected', 'phenoxy', 'radicals', 'dithiocarbamate', 'trityl', 'benzhydryl', 'derivatives', 'growing', 'radicals', 'also', 'deactivated', 'nonradicals', 'presence', 'organometallic', 'compounds', 'form', 'stabilized', 'hyper-coordinated', 'radicals', 'the', 'polymerizations', 'aid', 'reversible', 'degradative', 'chain', 'transferring', 'unique', 'requires', 'rapid', '“clean”', 'chain', 'transfers', 'without', 'side', 'reactions', 'the', 'enhanced', 'control', 'polymerization', 'process', 'relies', 'reduction', 'ratio', 'rate', 'termination', 'propagation', 'due', 'low', 'instantaneous', 'concentration', 'growing', 'radicals', 'this', 'means', 'initiation', 'propagation', 'reactions', 'must', 'proceed', 'similar', 'rates', 'due', 'application', 'initiators', 'resembling', 'polymer', 'end', 'groups', 'dormant', 'state', 'also', 'polymerization', 'reactions', 'must', 'low', 'proportion', 'chains', 'marked', 'uncontrolled', 'termination', 'andor', 'transfer', 'due', 'relatively', 'low', 'molecular', 'weights', 'homogenous', 'controlled“living”', 'free', 'radical', 'polymerizations', 'based', 'therefore', 'revers-', 'ible', 'deactivations', 'growing', 'radicals', 'early', 'matyjaszewski', 'divided', 'polymerizations', 'three', 'classes', '240', '241.', 'these', '1', 'deactivations', 'growing', 'radicals', 'stable', 'radicals', 'reversible', 'formations', 'dormant', 'covalent', 'species', 'followed', 'homolytic', 'cleavages', '2', 'reversible', 'deactivations', 'growing', 'radicals', '“nonradical”', 'species', 'formation', 'dormant', 'persistent', 'radicals', '3', 'reversible', 'degenerative', 'transfers', 'based', 'thermodynamically', 'neutral', 'exchange', 'reactions', 'growing', 'radicals', 'transfer', 'agents', 'p(cid3)', 'þ', 'r(cid3)(cid1)', 'p----------r', 'deact', '(cid1)', 'act', 'p(cid3)þx(cid11)(cid1)', 'ðp---------x)', 'deact', '(cid1)', 'act', 'ktr', 'pn(cid3)', 'þ', 'p1--------r', '(cid1)', '(cid1)', 'p1(cid3)', 'þ', 'pn--------r', '116', '3', 'free-radical', 'chain-growth', 'polymerization', 'f', 'o', 'f', 'o', 'b', 'l', 'n', 'n', 'co', 'n', 'n', 'l', 'o', 'f', 'o', 'b', 'f', '(co(ii))', 'co2r', 'co2r', 'propagating', 'polymer', 'chain', 'co(iii)', 'co2r', 'co2r', 'f', 'o', 'b', 'l', 'f', 'o', 'n', 'n', 'co', 'n', 'n', 'l', 'o', 'f', 'b', 'o', 'f', 'co(iii)', 'h', 'co(iii)', 'co2r', 'co2r', 're-initiation', 'fig', '32', 'cobalt', 'mediated', 'controlled“living”', 'polymerization', '3141', 'cobalt', 'mediated', 'polymerizations', 'catalytic', 'chain', 'transfer', 'polymerizations', 'utilize', 'metals', 'low', 'spin', 'cobalt(ii)', 'compounds', 'chain', 'transferring', 'agents', 'the', 'mechanism', 'believed', 'involve', 'repeated', 'disturbing', 'propagating', 'step', 'abstraction', 'hydrogen', 'atoms', 'propagating', 'polymers', 'this', 'yields', 'chains', 'unsatu-', 'rated', 'terminal', 'units', 'hydrogen', 'transfer', 'agent', 'adducts', 'co(iii)–h', 'subsequent', 'transfers', 'hydrogens', 'growing', 'chains', 'result', 'reinitiating', 'processes', '241.', 'the', 'b-hydrogen', 'abstractions', 'growing', 'radicals', 'formations', 'metal', 'hydrides', '229', 'illustrated', 'follows', 'r', 'mtligand', 'r', 'ligandmt', 'h', 'mt', '¼', 'co', 'this', 'catalytic', 'cycle', 'cobalt', 'mediated', 'polymerization', 'using', 'bis-dimethyl-glyoximate', 'cobalt', 'boron', 'ﬂuoride', 'catalyst', 'illustrated', 'haddleton', 'et', 'al', '233', '234.', 'a', 'similar', 'illustration', 'process', 'shown', 'fig', '32', 'the', 'afﬁnities', 'metals', 'hydrogen', 'abstractions', 'andor', 'abilities', 'lose', 'electrons', 'depend', 'oxidation', 'states', 'nature', 'ligands', 'such', 'reactions', 'therefore', 'suppressed', 'this', 'done', 'choosing', 'high', 'oxidation', 'state', 'metals', 'ligands', 'protect', 'metal', 'abstraction', 'p–h', 'atoms', 'controlling', 'position', 'oxidation–reduction', 'equilibrium', 'r(cid1)(cid1)', 'co', 'ln', 'ð', 'r(cid3)þ(cid3)co', 'ln', 'rmx', 'þ', 'co', 'ln', 'ð', 'rmx', '(cid1)(cid1)co', 'ln', 'the', 'metal', 'cobalt(ii)', 'usually', 'chelated', 'this', 'cobalt', 'porphyrin', 'cobalt', 'phthalocyanin', 'cobalt', 'oxime', 'well', 'others', 'the', 'polymer', 'molecules', 'form', 'stated', 'earlier', 'terminal', 'double', 'bonds', 'illustrated', 'follows', '314', 'controlled“living”', 'free-radical', 'polymerization', '117', 'the', 'cobalt', 'hydride', 'turn', 'reacts', 'new', 'monomer', 'molecules', 'regenerate', 'co(ii)', '235.', 'one', 'publication', 'describes', 'chain', 'transferring', 'agents', 'used', 'controlled', 'polymerization', 'methacrylate', 'monomers', 'reductive', 'elimination', 'cobalt', 'hydride', 'neighboring', 'methyl', 'group', 'deﬂects', 'chain', 'growth', '236.', 'the', 'agent', 'illustrated', 'follows', 'n-1', 'o', 'o', 'o', 'o', 'con', 'n', 'n', 'n', 'a', 'described', 'example', 'reaction', 'conducted', '60(cid6)c', 'deoxygenated', 'benzene', 'using', 'neopentylcobalt', 'tetramesityl-porphyrin', 'ligand', 'methyl', 'acrylate', 'monomer', '236.', 'a', 'slow', 'polymerization', 'yields', '66', 'conversion', '38', 'h', 'the', 'product', 'narrow', 'molecular', 'weight', 'distribution', 'polymer', 'mn', '¼', '144000', 'the', 'polymerization', 'even', 'slower', 'less', 'hindered', 'phenyl', 'substituents', 'porphyrin', 'ligand', 'both', 'homopolymers', 'block', 'copolymers', 'formed', 'catalytic', 'chain', 'transfer', 'cobalt(ll)', 'porphyrin', 'radical', 'polymerization', 'maa', 'water', 'studied', 'wayland', 'coworkers', '237.', 'cobalt', 'tetrasulfonatophenylporphyrin', 'found', 'exceptionally', 'effective', 'catalytic', 'chain', 'transfer', 'radical', 'polymerization', 'maa', 'water', 'a', 'remarkable', 'feature', 'process', 'increase', 'degree', 'polymerization', 'conversion', 'requires', 'existing', 'macromonomers', 'initiation', 'propagation', 'new', 'chains', 'chain', 'transfer', 'monomer', 'reinitiation', 'oligomer', 'oleﬁns', 'chain', 'growth', 'signiﬁcant', 'inherent', 'reactivity', 'features', 'cobalt+-porphyrin', 'catalyzed', 'chain', 'transfer', 'process', 'monomer', 'consumed', 'chain', 'growth', '3142', 'atom', 'transfer', 'radical', 'polymerizations', 'atom', 'transfer', 'radical', 'polymerizations', '(atrp)', 'reported', 'simultaneously', 'two', 'groups', '(1)', 'matyjaszewski', 'et', 'al', '218', '(2)', 'sawamoto', 'coworkers', '226.', 'matyjaszewski', 'et', 'al', 'utilized', 'cubipyridine', 'complex', 'halogen', 'transfer', 'agent', 'functions', 'dormant', 'active', 'polymer', 'chains', 'formation', 'polymers', 'predetermined', 'molecular', 'weight', 'mn', '(cid14)', '10', '5', 'polydispersity', 'narrow', '105', 'reported', '238', '239.', 'this', 'type', 'polymerization', 'appears', 'offer', 'possibility', 'preparing', 'broad', 'range', 'polymeric', 'materials', '240–242.', 'the', 'reactions', 'proceed', 'conditions', 'could', 'make', 'process', 'commercially', 'attractive', 'thus', 'instance', 'using', 'nonionic', 'surfactants', 'poly(oxyethylene', 'oleyl', 'ethers)', 'possible', 'prepare', 'polymers', 'butyl', 'methacrylate', 'methyl', 'methacrylate', 'styrene', 'butyl', 'acrylate', 'aqueous', 'emulsions', 'in', 'addition', 'using', 'multidentate', 'ligand', 'tris(2-dimethyl-amino)ethylamine', 'atom', 'transfer', '118', '3', 'free-radical', 'chain-growth', 'polymerization', 'polymerizations', 'made', 'proceed', 'rapidly', 'room', 'temperature', '242', '243.', 'the', 'atom', 'transfer', 'polymerization', 'reaction', 'illustrated', 'follows', 'polymerizations', 'styrene', 'using', '220-dipyridyl', 'ligand', 'indicated', 'proceed', 'ﬁrst', 'order', 'respect', 'concentration', 'initiator', '04', '06', 'orders', 'respect', 'concentration', 'cu(l)', 'halide', 'ligand', '218', '229.', 'the', 'copper', 'bipyridyl', 'complexes', 'mentioned', 'pictured', 'haddleton', 'et', 'al', '233', '234', 'follows', 'n', 'n', 'cu', 'cu', 'cl', 'cl', 'n', 'n', 'cu', 'n', 'n', 'n', 'n', 'cucl2', 'cu', 'n', 'n', 'n', 'n', 'cl', '314', 'controlled“living”', 'free-radical', 'polymerization', '119', 'recently', 'matyjaszewski', 'summarized', 'mechanism', 'polymerizations', '245.', 'matyjaszewski', 'coworkers', '246–250', 'reported', 'small', 'amounts', 'air', 'present', 'reaction', 'mixture', 'consumed', 'addition', 'sufﬁcient', 'amounts', 'appropriate', 'reducing', 'agent', 'tin(ii)', '2-ethylhexanoate', 'ascorbic', 'acid', 'in', 'process', 'cuprous', 'ions', 'initially', 'oxidized', 'oxygen', 'cupric', 'ions', 'turn', 'reduced', 'reducing', 'agent', 'the', 'cuprous', 'ions', 'activate', 'reaction', 'there', 'induction', 'period', 'oxygen', 'consumed', 'this', 'referred', '(argent)', 'atrp', 'also', 'subsequently', 'reported', 'polymerizations', '2-(dimethylamino)ethyl', 'methacrylate', 'require', 'addition', 'reducing', 'agent', 'tertiary', 'amine', 'group', 'presumably', 'serves', 'internal', 'one', '251.', 'in', 'addition', 'percec', 'coworkers', '253', 'reported', 'polymerizations', 'polar', 'solvents', 'conjunction', 'copper', 'appropriate', 'ligands', 'allow', 'ultrafast', 'syntheses', 'high-molecular-weight', 'polymers', 'ambient', 'temperature', 'the', 'process', 'referred', 'single', 'electron', 'transfer-living', 'radical', 'polymerization', '(set-lrp)', 'the', 'mechanism', 'proposed', 'based', 'disproportionation', 'cuprous', 'ions', 'cupric', 'ions', 'metallic', 'copper', 'this', 'catalyzed', 'polar', 'solvents', 'appropriate', 'ligands', 'the', 'proposed', 'mechanism', 'illustrated', 'follows', 'pnx', 'cu', 'cu', 'xl', 'pn', 'polar', 'solvent', 'ligand', 'pnx', 'nm', 'kp', 'kt', 'r', 'k', 'disp', 'pn', 'pn', 'cu', 'cu', 'xl', 'the', 'work', 'percec', 'coworkers', 'included', 'investigation', 'various', 'solvents', 'ligands', 'catalyst', 'activity', 'ability', 'disproportionate', 'cuprous', 'ion', 'they', 'demonstrated', 'addition', '10', 'mol', 'phenol', 'ligand', 'leads', 'spontaneous', 'disproportionation', 'metallic', 'copper', 'cupric', 'ions', '253.', 'an', 'alternative', 'proposed', 'percec’s', 'mechanism', 'proposed', 'matyjaszewski', '254.', 'according', 'mechanism', 'metallic', 'copper', 'acts', 'reducing', 'agent', 'cupric', 'ions', 'yields', 'active', 'cuprous', 'ions', 'catalyze', 'polymerization', 'this', 'mechanism', 'similar', 'one', 'proposed', 'reactions', 'utilize', 'ascorbic', 'acid', 'tin', 'based', 'reagents', 'reduce', 'cupric', 'ions', 'cuprous', 'ones', '255', '256.', 'the', 'mechanism', 'illustrated', 'follows', 'p', 'n', 'x', 'cu', 'x', 'ligand', 'cu', 'x', 'ligand', 'p', 'n', 'p', 'n', 'p', 'cu0', 'kactiv', 'kdeactiv', 'kt', 'p', 'm', 'k', 'p', 'p', 'nm', 'haddleton', 'coworkers', '256', 'investigated', 'use', 'toluene', 'solvent', 'phenol', 'additive', 'use', 'livingcontrolled', 'polymerizations', 'they', 'demonstrated', 'direct', 'relationship', 'reaction', 'time', 'amount', 'phenol', 'added', 'the', 'optimum', 'amount', 'found', '20', 'equivalents', 'phenol', 'respect', 'initiator', 'their', 'products', 'narrow', 'molecular', 'weight', 'polymers', 'mwd', 'ranging', '105', '125', 'removal', 'copper', 'atp', 'products', 'sometimes', 'problem', '257.', 'honigfort', 'coworkers', 'reported', 'found', 'ligands', 'supported', 'janda', 'jel', '(see', 'chap', '10)', 'resins', 'easy', '120', '3', 'free-radical', 'chain-growth', 'polymerization', 'removal', 'catalyst', 'complex', 'possible', 'reaction', 'mixture', 'the', 'janda', 'jel', 'ligands', 'used', 'atrp', 'methyl', 'methacrylate', 'styrene', '2-(dimethylamino)ethyl', 'methacrylate', 'the', 'methyl', 'methacrylate', '2-(dimethylamino)ethyl', 'methacrylate', 'polymerizations', 'proceeded', 'quickly', 'high', 'conversion', '(90)', 'well', 'controlled', 'the', 'styrene', 'polymerization', 'however', 'found', 'sluggish', 'proceeded', '63', 'conversion', 'after', 'polymerizations', 'complete', 'catalyst', 'ligand', 'complex', 'easily', 'removed', 'ﬁltration', 'zhu', 'coworkers', '257', 'claim', 'simple', 'effective', 'method', 'puriﬁcation', 'atp', 'product', 'using', 'catalyst', 'precipitation', 'microﬁltration', 'the', 'method', 'relies', 'precipitation', 'cu', 'br', 'ligand', 'catalyst', 'complex', 'additions', 'cu+br2', 'the', 'precipitate', 'thus', 'formed', 'effectively', 'retained', '014-mm', 'ptfe', 'ﬁlter', 'resulting', '999', 'catalyst', 'removed', 'polymer', 'the', 'resulting', 'clear', 'polymer', 'ﬁltrate', 'contains', 'little', 'residual', 'copper', '10', 'ppm', 'matyjaszewski', 'coworkers', 'developed', 'process', '255', 'electrochemically', 'mediated', 'atrp', 'they', 'use', 'applied', 'voltage', 'drive', 'production', 'cu', 'ions', 'catalyze', 'polymer', 'formation', 'because', 'rate', 'reaction', 'controlled', 'redox', 'equilibrium', 'cuprous', 'cupric', 'ions', 'electrochemistry', 'permits', 'regulation', 'concentration', 'species', '-e-', 'rbr', 'cu(i)brligand', 'cu(ii)br2', 'ligand', 'r', 'e-', 'polymer', 'a', 'similar', 'atp', 'process', 'one', 'uses', 'iron(ii)', 'bis(triphenylphosphine)-dichloridefecl2(pph3)2.', 'it', 'induces', '“living”', 'polymerization', 'monomers', 'methyl', 'methacrylate', 'conjunction', 'organic', 'halides', 'initiators', 'presence', 'absence', 'al(oipr)3', 'toluene', '80(cid6)c', 'the', 'molecular', 'weight', 'distributions', 'products', '11–13', '269.', 'the', 'following', 'mechanism', 'visualized', '269:', 'fe(ii)', 'r', 'x', 'xr', 'fe(iii)', 'mma', '-fe(ii)', 'r', 'x', 'mma', 'co2ch3', 'fe(ii)', 'fe(ii)', 'x', 'r', 'r', 'x', 'fe(iii)', 'reversible', 'co2ch3', 'co2ch3', 'dormant', 'species', 'active', 'radical', 'species', 'the', 'atp', 'process', 'developed', 'sawamoto', 'coworkers', '226,', 'uses', 'initiating', 'system', 'consisting', 'methylaluminum', 'bis(26-di-tert-butylphenoxide)', 'polymerize', 'methyl', 'methacrylate', '226.', 'the', 'polymerization', 'involves', 'reversible', 'homolytic', 'cleavages', 'carbon-halogen', 'terminal', 'groups', 'assisted', 'transition', 'metal', 'complexes', '226.', 'dichlorotri(triphenyl-phosphine)-ruthenium', '(ii)', 'tetrachloride', 'carbon', 'rucl2', '(pph3)', 'meal', '(odbp)', 'cl', 'r', 'dormant', 'revesible', 'homolitic', 'cl', 'r', 'activated', '314', 'controlled“living”', 'free-radical', 'polymerization', '121', 'the', 'ruthenium(ii)', 'complexes', 'interact', 'ccl4', 'oxidized', 'process', 'become', 'ru(iii)', 'radicals', 'ccl3•', 'add', 'molecules', 'methyl', 'methacrylate', 'the', 'polymerization', 'proceeds', 'via', 'repetitive', 'additions', 'methyl', 'methacrylate', 'molecules', 'radical', 'species', 'repeatedly', 'generated', 'covalent', 'species', 'carbon-halogen', 'terminal', 'groups', '226.', 'suwamoto', 'also', 'reported', '226', 'addition', 'halogen', 'donor', 'ph3c–cl', 'aids', 'shift', 'equilibrium', 'balance', 'dormant', 'species', 'the', 'reaction', 'polymerization', 'illustrated', 'follows', 'pph3', 'ii', 'ru', 'cl', 'ph3', 'pph3', 'cl', 'cl', 'cl', 'cl', 'cl', 'br', 'cl', 'cl', 'cl', 'cl', 'cl', 'monomer', 'pph3', 'ii', 'ru', 'cl', 'pph3', 'cl', 'ph3', 'pph3', 'iii', 'ru', 'pph3', 'iii', 'ru', 'n', 'r', 'r', 'br', 'cl', 'pph3', 'ph3', 'cl', 'br', 'cl', 'pph3', 'cl', 'ph3', 'r', 'cl', 'cl', 'cl', 'br', 'r', 'klumperman', 'coworkers', '259', 'observed', 'lately', 'quite', 'common', 'treat', 'living', 'radical', 'copolymerization', 'completely', 'analogous', 'radical', 'counterpart', 'small', 'deviations', 'copolymerization', 'behavior', 'occur', 'they', 'interpret', 'deviations', 'basis', 'reactions', 'speciﬁc', 'controlledliving', 'radical', 'polymerization', 'activation—deactivation', 'equilib-', 'rium', 'atrp', 'they', 'observed', 'reactivity', 'ratios', 'obtained', 'atom', 'transfer', 'radical', 'copolymeri-', 'zation', 'data', 'interpreted', 'according', 'conventional', 'terminal', 'model', 'deviate', 'true', 'reactivity', 'ratios', 'propagating', 'radicals', 'velazquez', 'coworkers', '260,', 'developed', 'kinetic', 'model', 'incorporating', 'effects', 'diffusion-', 'controlled', 'reactions', 'atom-transfer', 'radical', 'polymerization', 'the', 'reactions', 'considered', 'diffusion-controlled', 'monomer', 'propagation', 'bimolecular', 'radical', 'termination', 'chain', 'transfer', 'propagating', 'radicals', 'catalyst', 'transfer', 'small', 'molecules', 'model', 'predictions', 'indicate', 'diffusion-controlled', 'propagation', 'reduces', '“living”', 'behavior', 'system', 'diffusion-', 'controlled', 'termination', 'enhances', 'livingness', 'also', 'diffusion-controlled', 'transfer', 'chains', 'catalyst', 'forward', 'reverse', 'directions', 'the', '“livingness”', 'system', 'enhanced', 'one', 'kept', 'unchanged', 'increased', '“livingness”', 'system', 'reduced', 'when', 'diffusion-controlled', 'termination', 'important', 'simulations', 'show', 'overall', 'effect', 'diffusion-controlled', 'phenomena', 'atrp', 'enhance', 'livingness', 'system', 'preparation', 'gradient', 'copolymer', 'styrene', 'n-butyl', 'acrylate', 'reported', 'use', 'atrp', '261.', 'gradient', 'copolymers', 'copolymers', 'sequence', 'distributions', 'varying', 'well-', 'deﬁned', 'order', 'functions', 'chain', 'lengths', 'it', 'suggested', 'gradient', 'copolymers', 'potential', 'outperforming', 'block', 'alternating', 'copolymers', 'instances', '261.', '3143', 'nitroxide-mediated', 'radical', 'polymerizations', 'a', 'nitroxide', 'mediated', 'polymerization', 'styrene', 'ﬁrst', 'reported', '1985', '262.', 'this', 'reaction', 'however', 'studied', 'extensively', 'since', '1993', 'the', 'monomer', 'conversion', 'rates', 'vs', 'temperatures', 'much', 'slower', 'conventional', 'styrene', 'polymerization', 'also', 'polydispersities', '122', '3', 'free-radical', 'chain-growth', 'polymerization', 'products', 'narrow', 'obtained', 'anionic', 'polymerization', 'generally', 'polydispersities', 'produced', 'process', 'proportional', 'molecular', 'weights', 'polymers', 'produced', 'in', 'fact', 'linear', 'relationship', 'polydispersity', 'molecular', 'weight', 'polystyrene', 'product', 'demonstrated', '263.', 'after', 'initial', 'nonstationary', 'period', 'typical', 'alkene', 'polymerizations', 'presence', 'alkoxyamines', 'proceed', 'according', 'ﬁrst', 'order', 'kinetics', 'molecular', 'weights', 'increasing', 'conversion', 'the', 'dispersity', 'products', 'contribution', 'nonstationary', 'periods', 'depend', 'upon', 'temperature', 'particular', 'initiating', 'system', 'nature', 'monomers', 'styrene', 'polymerizations', 'carried', 'presence', 'stable', 'nitroxyl', 'radicals', '2266-tetramethylpiperydinyl-1-oxy', 'radical', 'commonly', 'referred', 'tempo', '264', 'ditertiary', 'butyl', 'nitroxide', 'referred', 'dtb', 'n', 'n', 'o', 'n', 'o', 'tempo', 'dtbn', 'such', 'radicals', 'incapable', 'initiating', 'polymerizations', 'additions', 'double', 'bonds', 'react', 'selectively', 'growing', 'radicals', 'form', 'reversibly', 'covalent', 'species', '265.', 'in', 'addition', 'reactions', 'growing', 'radicals', 'dormant', 'species', 'occur', 'via', 'degenerative', 'transfer', 'pn', 'r2n', 'o', 'r', 'pn', 'o', 'nr2', 'r', 'the', 'position', 'equilibrium', 'constant', 'reactions', 'tempo', 'depends', 'nature', 'radical', 'solvent', 'temperature', 'these', 'polymerizations', 'initiated', 'either', 'bimolecular', 'initiators', 'unimolecular', 'ones', 'the', 'bimolecular', 'initiators', 'utilize', 'common', 'free', 'radical', 'sources', 'benzoyl', 'peroxide', 'azobisisobutyronitrile', 'start', 'reaction', 'the', 'carbon-centered', 'initiating', 'radicals', 'form', 'turn', 'react', 'tempo', 'this', 'illustrated', 'follows', 'r', 'r', 'r', 'n', 'o', 'r', 'r', 'n', 'o', 'etc', 'n', 'o', 'n', 'o', '314', 'controlled“living”', 'free-radical', 'polymerization', '123', 'various', 'descriptions', 'different', 'unimolecular', 'initiators', 'found', 'literature', 'a', 'presence', 'a-methyl', 'groups', 'alkoxyamines', 'appears', 'essential', '266.', 'these', 'compounds', 'yield', 'upon', 'dissociation', 'stable', 'radicals', 'initiating', 'ones', 'shown', 'follows', '267.', 'o', 'o', 'n', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'n', 'o', 'etc', 'the', 'optimal', 'amount', 'radical', 'initiator', 'depends', 'efﬁciency', 'initiation', 'ideally', 'concentration', 'radicals', 'generated', 'initiator', 'slightly', 'higher', 'concen-', 'tration', 'scavenger', 'at', 'higher', 'temperatures', '120(cid6)c', 'polymerizations', 'styrene', 'tend', 'exhibit', 'ideal', 'behavior', 'also', 'higher', 'temperatures', 'narrower', 'molecular', 'weight', 'distributions', 'obtained', 'indicating', 'sufﬁciently', 'high', 'exchange', 'rates', 'a', 'low-temperature', 'method', 'preparation', 'unimolecular', 'initiators', 'reported', '268.', 'in', 'method', 'oxidation', 'used', 'generate', 'carbon', 'radicals', 'presence', 'nitroxide', 'traps', 'tempo', 'buo2c', '1', 'pbo2', 'buo2c', 'o', 'n', '2', 'cucl2', 'tempo', '-78', 'oc', '124', '3', 'free-radical', 'chain-growth', 'polymerization', 'a', 'variation', 'controlled“living”', 'polymerization', 'vinyl', 'acetate', 'use', 'bidentate', 'ligand', '220-bipyridyl', 'tempo', 'composition', '212', 'ratio', 'reported', 'mardare', 'matyjaszewski', '267.', 'the', 'following', 'mechanism', 'proposed', '(1)', 'pentacoordinated', 'complexes', '(i)', 'formed', 'molar', 'ratio', '11', 'r', 'al', 'n', 'n', 'radicals', 'r', 'r', 'r', 'al', 'n', 'n', 'n', 'n', 'r', 'r', 'r', 'al', 'r', 'r', 'i', 'o', 'n', 'n', 'n', 'al', 'o', 'n', 'ii', 'n', 'n', 'n', 'o', 'o', 'n', 'iva', 'r', 'r', 'al', 'r', 'r', 'n', 'n', 'n', 'o', 'al', 'o', 'n', 'iia', '(2)', 'the', 'irreversible', 'attacks', 'tempo', 'pentacoordinated', 'complexes', '(al(ibu)3)bpy', '(i)', 'lead', 'relatively', 'stable', 'delocalized', 'radicals(ii)', 'tempo', 'also', 'reacts', 'short-lived', 'radicals', 'present', 'stage', '(1)', 'form', 'alkoxyamines', 'pentacoordinated', 'complexes', 'type', 'iv', 'the', 'radicals', 'ii', 'could', 'equilibrium', 'tiny', 'amounts', 'reactive', 'radicals', 'r•', 'capable', 'initiations', 'subsequent', 'propagations', 'aldabbagh', 'coworkers', '269', 'reported', 'carrying', 'nitroxide-mediated', 'polymerization', 'supercritical', 'carbon', 'dioxide', 'allows', 'improved', 'control', 'reaction', 'nesvadba', 'coworkers', '270', 'used', 'nitrones', 'controlled', 'radical', 'polymerization', 'vinyl', 'monomers', 'this', 'beginning', 'situ', 'nmp', 'concept', 'the', 'alkoxyamines', 'prepared', '314', 'controlled“living”', 'free-radical', 'polymerization', '125', 'reaction', 'free', 'radicals', 'obtained', 'decomposition', 'azo-initiators', 'azobisisobutyr-', 'onitrile', 'l10-azobis(cyclohexanecarbonitrile)', 'selected', 'nitrones', 'cn', 'cn', 'cn', 'cn', 'n', 'o', 'cn', 'n', 'o', 'cn', 'cn', 'n', 'o', 'n', 'o', 'cn', 'cn', 'n', 'o', 'cn', 'the', 'alkoxyamines', 'utilized', 'radical', 'polymerization', 'acrylates', 'styrene', 'bulk', 'solution', '100', '145(cid6)c', 'low', 'molecular', 'weight', 'polymers', '3000–14000', 'gmol', 'formed', 'rapidly', 'polydispersity', 'mwmn', '12', '34', 'high', 'styrene', 'conversion', 'observed', 'together', 'low', 'polydispersity', 'subsequently', 'nitroxides', 'parent', 'alkoxyamines', 'formed', 'directly', 'polymerization', 'medium', '(in', 'situ', 'nmp)', 'reaction', 'nitrone', 'free', 'radical', 'initiators', '270.', 'two', 'types', 'reactions', 'carried', 'one', 'reaction', 'monomer', 'addition', 'one', 'addition', 'in', 'either', 'case', 'prereaction', 'systematically', 'carried', 'temperatures', 'ranging', '60', '80(cid6)c', 'this', 'followed', 'polymerizations', '130(cid6)c', 'the', 'situ-formed', 'nitroxides', 'alkoxyamines', 'controlled', 'radical', 'polymerizations', 'n-butyl', 'acrylate', 'yielding', 'however', 'low', 'molecular', 'weight', 'polymers', 'mn', '10000', 'mwmn', 'equal', '165–20', 'a', 'patent', 'issued', 'wertmer', 'coworkers', '271', 'controlled', 'radical', '(co)polymerization', 'vinyl', 'monomers', 'mediated', 'nitrones', 'substituted', 'longer', 'alkyl', 'groups', 'contained', 'much', '18', 'carbon', 'atoms', 'the', 'nitrone', 'simply', 'heated', 'presence', 'peroxide', 'monomer', 'styrene', '130(cid6)c', '24', 'h', 'high-molecular-weight', 'polystyrene', 'mn', '¼', '98000–146000', 'formed', 'the', 'ratio', 'mwmn', 'however', 'disclosed', 'recently', 'grubbs', 'coworkers', '272', 'synthesized', 'active', 'alkoxyamine', 'reaction', '2-', 'methyl-2-nitrosopropane', '1-bromoethylbenzene', 'catalyzed', 'ligated', 'cubr', 'presence', 'metallic', 'copper', 'a', 'puriﬁed', 'alkoxyamine', 'used', 'initiate', 'radical', 'polymerization', 'styrene', 'isoprene', 'well-deﬁned', 'low', 'polydispersity', 'polymers', 'formed', 'mwmn', '¼', '114', 'polystyrene', '128', 'polyisoprene', 'subsequently', 'grubbs', 'coworkers', '273', 'used', 'alkoxyamine', 'successfully', 'controlled', 'radical', 'polymerization', 'n-butyl', 'acylate', '125(cid6)c', 'lower', 'ratio', 'mwmn', 'observed', 'alkoxyamine', 'preheated', 'temperatures', '125', '30', 'mm', 'prior', 'adding', 'monomer', 'this', 'prereaction', 'needed', 'excess', 'free', 'nitroxide', 'formed', 'situ', 'polymerization', 'controlled', '126', '3', 'free-radical', 'chain-growth', 'polymerization', '3144', 'reversible', 'addition-fragmentation', 'chain', 'transfer', 'polymerization', 'another', 'type', '“living”controlled', 'radical', 'polymerization', 'involves', 'reversible', 'addition-fragmentation', 'chain', 'transfer', 'it', 'named', 'therefore', 'raft', 'polymerization', 'great', 'versatility', 'effectiveness', 'shown', 'process', '274.', 'the', 'process', 'said', 'compatible', 'wide', 'range', 'monomers', 'including', 'functional', 'monomers', 'containing', 'functional', 'groups', 'acids', 'acid', 'salts', 'hydroxyl', 'tertiary', 'amine', 'groups', 'the', 'conditions', 'polymerization', 'used', 'conventional', 'free-radical', 'polymerizations', 'they', 'carried', 'bulk', 'solution', 'emulsion', 'suspension', '(see', 'sect', '316)', 'the', 'usual', 'azo', 'peroxide', 'initiators', 'employed', '274.', 'the', 'reaction', 'originally', 'illustrated', 'follows', '274:', 'm-1', 's', 's', 'r', 'k', 'add', 'z', 'k', '-add', 's', 'r', 'z', 'polymerization', 'kp', 'j', 'j', 'y', 'x', 'y', 'x', 'y', 'x', 's', 'z', 's', 'kp', 'r', 's', 'y', 'x', 'y', 'x', 'j', 'y', 'x', 'kp', 'j', 'p', 'the', 'raft', 'process', 'depends', 'upon', 'rapid', 'addition—fragmentation', 'equilibrium', 'reaction', 'propagating', '(pn•)', 'well', 'intermediate', 'radicals', 'chain', 'activity', 'dormancy', 'shown', 'reaction', 'scheme', 'the', 'concentrations', 'species', 'within', 'equilibrium', 'dependent', 'relative', 'rate', 'coefﬁcients', 'addition', 'propagating', 'radical', 'raft', 'agent', '(kadd)', 'fragmentation', 'formed', 'intermediate', 'radical', '(kfrag)', 'this', 'equilibrium', 'applies', 'correctly', 'polymeric', 'chains', 'present', 'signiﬁcant', 'concentrations', 'initialization', 'period', 'during', 'initiation', 'period', 'mainly', 'shorter', 'chains', 'present', 'the', 'important', 'part', 'equilibrium', 'relatively', 'stable', 'radical', 'intermediates', 'it', 'reported', 'raft-mediated', 'polymerization', 'reactions', 'typically', 'contain', 'anomalies', '“inhibition”', 'period', 'rate', 'retardation', 'the', 'rate', 'retardations', 'reductions', 'polymerization', 'rates', 'apparently', 'occur', 'presence', 'raft', 'agents', 'observed', 'raft', 'agents', 'absent', 'examples', 'dithiobenzoate-mediated', 'polymerization', 'reactions', '275.', 'tonge', 'coworkers', '276', 'investigated', 'reactions', 'short-chain', 'species', 'initial', 'period', 'cumyl', 'dithiobenzoate', 'mediated', 'polymerization', 'styrene', '84(cid6)c', 'using', 'electron', 'spin', 'resonance', 'hydrogen', 'carbon', 'nmr', 'spectroscopies', 'able', 'demonstrate', 'reactions', 'speciﬁc', 'initial', 'stages', 'there', 'strong', 'preference', 'add', 'single', 'monomer', 'species', 'this', 'followed', 'fragmentation', 'release', 'shorter', 'radicals', 'prior', 'formation', 'longer', 'chains', 'the', 'effectiveness', 'raft', 'agents', 'investigated', 'moad', 'coworkers', '277.', 'these', 'raft', 'agents', 'thiocarbonylthio', 'compounds', 'depend', 'effectiveness', 'nature', 'group', 'z', 'r', '(shown', 'below)', 'modify', 'reactivity', 'thiocarbonyl', 'group', 'toward', 'free', 'radical', 'addition', 'r', 'free', 'radical', 'leaving', 'group', '277:', 'z', 's', 's', 'r', '314', 'controlled“living”', 'free-radical', 'polymerization', '127', 'these', 'raft', 'agents', 'based', 'structure', 's¼c(ph)s–r', '277', 'they', 'found', 'effec-', 'tiveness', 'agents', 'also', 'depends', 'upon', 'nature', 'monomer', 'polymerization', 'conditions', 'for', 'polymerization', 'styrene', 'methyl', 'methacrylate', 'butyl', 'acrylate', 'methyl', 'acrylate', '60(cid6)c', 'effectiveness', 'r', 'decreases', 'following', 'order', '277:', 'r', 'c(alkylþ2cn', '(cid5)', 'c(ch3þ2arc(ch3þ2c(', 'o)o(alkyl)c(ch3þ2c(', 'o)nh(alkyl)', 'rc(ch3)hphc(ch3þ3', '(cid5)', 'ch2ph', 'c(ch3þ2ch2c(ch3þ3', 'in', 'addition', 'among', 'compounds', 'r¼c(ch3)2cn', 'c(ch3)2ph', 'thiocarbonylthio', 'compounds', 'yield', 'polymers', 'narrower', 'polydispersities', 'batch', 'polymerizations', 'also', 'compounds', 'allowed', 'molecular', 'weight', 'control', 'may', 'expected', 'living', 'polymerization', 'the', 'reaction', 'mechanism', 'proposed', 'moad', 'coworkers', 'follows', '277:', 'initiation', 'initiator', 'i', 'm', 'm', 'pn', 'chain', 'transfer', 'pn', 's', 's', 'r', 'kaddit', 'k-addit', 'p', 'n', 's', 's', 'r', 'p', 'n', 's', 's', 'r', 'k', 'β', 'k', 'β', '−', '2', '3', '4', 'reinitiation', 'r', 'm', 'ki', 'mr', 'pm', 'm', 'kp', 'chain', 'equilibration', 'pm', 's', 's', 'p', 'n', 'm', 'termination', 'pn', 'pm', 'k', 'dead', 'polymer', 'p', 's', 's', 'p', 'n', 'pm', 's', 's', 'pn', 'm', 'moad', 'coworkers', 'concluded', '277', 'major', 'factor', 'determines', 'transfer', 'coefﬁcient', 'dithiobenzoate', 'derivatives', 'way', 'intermediate', '3', '(see', 'equation)', 'partitions', 'starting', 'materials', 'products', 'this', 'turn', 'determined', 'relative', 'ability', 'leaving', 'group', 'r•', 'propagating', 'radical', 'steric', 'factors', 'radical', 'stability', 'r•', 'also', 'important', 'they', 'conclude', 'stable', 'electrophilic', 'bulkier', 'radicals', 'better', 'leaving', 'groups', 'the', 'partitioning', 'r•', 'monomers', '(to', 'reinitiate)', 'adding', 'polymeric', 'raft', 'also', 'signiﬁcant', 'effect', 'rate', 'consumption', 'raft', 'agent', '277.', 'vana', 'coworkers', '278', 'studied', 'conversion', 'vs', 'time', 'molecular', 'weight', 'distributions', 'conjunction', 'kinetic', 'scheme', 'rapt', 'process', 'in', 'particular', 'conditions', 'leading', 'inhibition', 'rate', 'retardation', 'examined', 'act', 'guide', 'optimizing', 'reaction', 'they', 'demonstrated', '128', '3', 'free-radical', 'chain-growth', 'polymerization', 'inhibition', 'period', 'considerable', 'length', 'it', 'induced', 'either', 'slow', 'fragmentation', 'intermediate', 'raft', 'radicals', 'appearing', 'preequilibrium', 'due', 'slow', 'reinitiation', 'leaving', 'group', 'radicals', 'initial', 'raft', 'agent', 'the', 'absolute', 'values', 'rate', 'coefﬁcients', 'governing', 'core', 'equilibrium', 'raft', 'process', '(at', 'ﬁxed', 'value', 'equilibrium', 'constant)', 'found', 'crucial', 'controlling', 'polydispersity', 'resulting', 'mwmn', 'values', 'higher', 'interchange', 'frequency', 'effects', 'narrower', 'distributions', 'they', 'also', 'demonstrated', 'size', 'rate', 'coefﬁcient', 'controlling', 'addition', 'reaction', 'propagating', 'radicals', 'polymer-raft', 'agent', 'kb', 'mainly', 'responsible', 'optimizing', 'control', 'polymerization', 'the', 'fragmentation', 'rate', 'coefﬁcient', 'k(cid1)b', 'macro', 'raft', 'intermediate', 'radical', 'hand', 'may', 'varied', 'orders', 'magnitude', 'without', 'affecting', 'amount', 'control', 'exerted', 'polymerization', 'based', 'basic', 'raft', 'mechanism', 'shown', 'value', 'mainly', 'governs', 'extent', 'rate', 'retardation', 'raft', 'polymerizations', '278.', 'calitz', 'tonge', 'sanderson', 'reported', 'results', 'study', 'raft', 'polymerization', 'means', 'electron', 'spin', 'resonance', 'spectroscopy', '276.', 'they', 'observed', 'intermediate', 'radical', 'signals', 'consistent', 'current', 'raft', 'theory', '276.', 'sawamoto', 'coworkers', 'reported', 'obtaining', 'simultaneous', 'control', 'molecular', 'weight', 'steric', 'structure', 'raft', 'polymerization', 'n-isopropylacrylamide', 'addition', 'rare', 'earth', 'metal', 'y(o-', 'tetraﬂuoromethanesulfonate)3', 'lewis', 'acid', 'the', 'mwmn', 'ratio', 'products', 'ranged', '14–19', 'isotactic', 'content', '80–84', '277.', 'goto', 'et', 'al', '279', 'developed', 'process', 'describe', 'reversible', 'living', 'chain', 'transfer', 'radical', 'polymerization', '278,', 'us', 'ge', 'sn', 'p', 'n', 'compounds', 'iodides', 'iodide', 'mediated', 'polymerizationsref', 'in', 'process', 'compound', 'gei4', 'chain', 'transferring', 'agent', 'polymer-iodide', 'catalytically', 'activated', 'via', 'rft', 'process', 'they', 'proposed', 'new', 'reversible', 'activation', 'process', 'referred', 'rtcp', '279.', 'the', 'process', 'illustrated', 'follows', '279:', '1', 'reversible', 'activation', 'polymer', 'x', 'polymer', 'kact', 'kdeact', 'monomers', '2', 'exchange', 'degenerative', 'chain', 'transfer', 'x1', 'polymer', 'i', 'polymer', 'polymer', 'i', 'polymer', 'kex', 'kex', '3', 'rt', 'reversible', 'chain', 'transfer', 'x1', 'polymer', 'i', 'a', 'polymer', 'i', 'a', 'ka', 'kda', 'i', 'a', 'gei4', 'sni4', 'pi3', 'nis', 'etc', '31441', 'combinations', 'click', 'chemistry', 'atp', 'well', 'atp', 'raft', 'polymerizations', 'in', 'last', 'years', '“click', 'reactions”', 'termed', 'sharpless', 'et', 'al', '280', 'received', 'attention', 'due', 'high', 'speciﬁcity', 'quantitative', 'yields', 'good', 'ﬁdelity', 'presence', 'functional', 'groups', 'the', '“click', 'chemistry”', 'reaction', 'includes', 'copper-catalyzed', 'huisgen', 'dipolar', 'cycloaddition', 'reaction', 'azide', 'alkyne', 'leading', '123-triazole', 'recent', 'publications', '“click', 'reaction”', 'indicate', 'useful', 'method', 'preparation', 'functional', 'polymers', '281.', '314', 'controlled“living”', 'free-radical', 'polymerization', 'matyjaszewski', 'gao', 'synthesized', 'functional', 'polymers', 'combining', 'atrp', '“click', 'reactions”', 'they', 'also', 'prepared', 'telechelic', 'polymers', 'star', 'polymers', 'brush', 'polymers', '282.', 'forma-', 'tion', 'telechelic', 'polymers', 'illustrated', 'follows', 'br', 'br', 'o', 'o', 'o', 'o', 'br', 'br', 'n3', 'n', 'cubr', 'toluene80', 'oc', 'nan3', 'dmf', '129', 'n3', 'n3', 'n', 'n', 'cubr', 'dmf', 'oh', 'n', 'n', 'n', 'ho', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'oh', 'ho', 'oh', 'cubrdmf', 'also', 'synthesis', 'iniferter', 'consists', 'trithiocarbonate', 'moiety', 'two', 'bromine', 'chains', 'ends', 'reported', '282.', 'this', 'iniferter', 'used', 'conduct', 'either', 'independently', 'concurrently', 'atrp', 'reversible', 'addition-fragmentation', 'chain', 'transfer', 'polymerizations', 'the', 'iniferter', 'illustrated', 'follows', 's', 'r', 'r', 'r', 'r', 'br', 's', 's', 'br', 'raft', 'polymerizations', 'iniferter', 'n-butyl', 'acrylate', 'styrene', 'yielded', 'polymers', 'mw', 'mn', 'equal', '115', '116', 'respectively', 'polymerization', 'methyl', 'methacrylate', 'however', 'yielded', 'polymer', 'broad', 'mwmn', 'ratio', 'on', 'hand', 'polymerization', 'presence', 'cubrtmpa', 'atrp', 'exclusively', 'bromine', 'chain', 'ends', 'yielded', 'polymer', 'narrow', 'mwmn', 'ratio', '282.', '3145', 'special', 'types', 'controlled“living”', 'polymerizations', 'it', 'reported', 'possible', 'employ', 'persistent', 'phosphorus-based', 'radicals', 'controlledliving', 'free-radical', 'polymerization', '283', '284.', 'also', 'cases', 'low', 'stability', 'hyper', 'coordinated', 'radicals', 'ligand', 'exchanges', 'become', 'facile', 'organoaluminum', 'organoboron', 'compounds', 'used', 'successfully', 'transfer', 'agents', 'polymerization', 'styrene', 'acrylics', 'vinyl', 'acetate', '283', '284.', 'chung', 'coworkers', '286', 'described', '“living”', 'radical', 'initiator', 'based', 'oxidation', 'adducts', 'alkyl-9-borabicyclononanes', 'hexyl-9-borabicyclononane', 'the', '“living”', 'radical', 'polymerizations', 'take', 'place', 'room', 'temperature', 'the', 'initiators', 'form', 'situ', 'reactions', 'oxygen', 'rch2', 'b', 'o2', 'rch2', 'o-o', 'b', 'r-ch2-o', 'bo', '130', '3', 'free-radical', 'chain-growth', 'polymerization', 'the', 'alkoxy', 'radicals', 'reactive', 'initiate', 'radical', 'polymerizations', 'readily', 'on', 'hand', 'borinate', 'radicals', 'stabilized', 'empty', 'p-orbitals', 'boron', 'back-donating', 'electron', 'density', 'stable', 'initiate', 'polymerizations', 'during', 'polymerization', 'borinate', 'radicals', 'may', 'form', 'weak', 'reversible', 'bonds', 'growing', 'chains', 'boroxyl', 'mediated', 'living', 'radical', 'polymerizations', 'subsequently', 'described', 'chung', '287', 'review', 'article', 'the', 'chemistry', 'centered', 'living', 'radical', 'initiator', 'ie,', 'alkylperoxydialkylborane', '(c—o—o—br)', 'species', 'similar', 'one', 'shown', 'formed', 'mono', 'oxidation', 'asymmetric', 'trialkylborane', 'oxygen', 'in', 'presence', 'polar', 'monomers', '(including', 'acrylates', 'methacrylates)', 'c—o—o—br,', 'undergoes', 'spontaneous', 'hemolytic', 'cleavage', 'ambient', 'tem-', 'perature', 'form', 'active', 'alkoxy', 'radical', 'stable', 'boroxyl', 'radical', 'the', 'alkoxyl', 'radical', 'active', 'initiating', 'polymerization', 'vinyl', 'monomers', 'stable', 'boroxyl', 'radical', 'forms', 'reversible', 'bond', 'propagating', 'radical', 'site', 'prevent', 'undesirable', 'termination', 'reactions', 'the', 'living', 'polymerizations', 'characterized', 'predictable', 'polymer', 'molecular', 'weight', 'narrow', 'molecular', 'weight', 'distributions', 'formation', 'telechelic', 'polymers', 'block', 'copolymers', 'sequen-', 'tial', 'monomer', 'addition', 'furthermore', 'living', 'radical', 'initiator', 'system', 'beneﬁts', 'two', 'unique', 'features', 'trialkylborane', 'these', '(a)', 'easy', 'incorporation', 'polymer', 'chains', '(chain', 'ends', 'side', 'chains)', '(b)', 'situ', 'auto-transformation', 'living', 'radical', 'initiators', 'lacroix', 'coworkers', 'reported', 'reverse', 'iodine', 'transfer', 'polymerization', '(ritp)', 'elemental', 'iodine', 'used', 'control', 'agent', 'living', 'radical', 'polymerization', '288.', 'styrene', 'butyl', 'acrylate', 'methyl', 'acrylate', 'butyl', 'a-ﬂuoroacrylate', 'homopolymerized', 'using', 'radical', 'catalyst', 'i2', 'chain', 'transfer', 'agent', 'methyl', 'acrylate', 'also', 'copolymerized', 'vinylidene', 'chloride', 'using', 'process', '3146', 'kinetics', 'controlledliving', 'free-radical', 'polymerizations', 'several', 'papers', 'published', 'describe', 'kinetics', 'controlled', 'free', 'radical', 'polymerization', 'goto', 'fukuda', '289', 'postulate', 'two', 'activation', 'processes', 'nitroxystyrene', 'polymerization', 'systems', 'the', 'equilibrium', 'constant', 'k', '¼', 'kdkc', 'stationary-state', 'concentrations', 'p•', 'x•', 'kd', 'process', 'i', 'p(cid1)(cid1)x', ')', 'kc', 'p(cid3)', 'x(cid3)', 'process', 'ii', 'p(cid1)(cid1)x', 'þ', 'p0(cid3)', ')', 'kex', 'p(cid3)', 'þ', 'p0', '(cid1)', 'x', '½p(cid3)(cid4)', '¼', 'ðrikiþ12', '½x(cid3)(cid4)', '¼', 'k½p', '(cid1)', 'x(cid4)½p(cid3)(cid4)', 'the', 'stationary', 'concentration', 'p•', 'x•', 'determined', 'different', 'mechanisms', 'p•', 'determined', 'balance', 'initiation', 'rate', 'ri', 'termination', 'rate', 'kt', 'p•', '2.', 'this', 'conventional', 'free', 'radical', 'polymerization', 'systems', 'x•', 'determined', 'however', 'equilibrium', 'equation', 'shown', 'process', 'i', 'it', 'depends', 'therefore', 'upon', 'equilibrium', 'constant', 'k', 'concentration', 'adduct', 'p–x', 'p•', '289.', 'the', 'rate', 'polymerization', 'stationary', 'state', 'rp', '¼', 'kp½p(cid3)(cid4)½m(cid4)', '¼', 'ðkpktþ12r12', '½m(cid4)', '315', 'thermodynamics', 'free-radical', 'polymerization', 'reaction', '131', 'the', 'polymer-nitroxyl', 'adduct', 'p–x', 'reversibly', 'dissociates', 'thermally', 'process', 'i', 'polymer', 'radical', 'p•', 'nitroxyl', 'radical', 'x•', 'the', 'rate', 'constants', 'dissociation', 'combination', 'kd', 'kc', 'respectively', 'the', 'so-called', '“degenerative', 'transfer”', 'takes', 'place', 'process', 'ii', 'the', 'second-order', 'rate', 'constant', 'active', 'species', 'either', 'direction', 'kex', 'here', 'rate', 'constants', 'assumed', 'independent', 'chain', 'length', 'since', 'frequency', 'cleavage', 'p–x', 'bond', 'proportional', 'p–x', 'process', 'i', 'p•1p0–x]', 'process', 'ii', 'overall', 'frequency', 'fa', 'per', 'unit', 'time', 'per', 'unit', 'volume', 'bond-cleaving', 'activation', 'reactions', 'may', 'expressed', '277:', 'fa', '¼', 'ka½p', '(cid1)', 'x(cid4)', 'ka', '¼', 'kd', 'þ', 'kd½p(cid3)(cid4)', 'ka', 'overall', 'activation', 'rate', 'constant', 'viewed', 'ﬁrst-order', 'reaction', 'goto', 'fukuda', 'concluded', 'may', 'convenient', 'represent', 'equation', 'form', '289:', 'show', 'general', 'expression', 'time-averaged', 'ka', 'batch', 'system', 'ka', '¼', 'kd', 'þ', 'ðkexkpþðrp½m(cid4)þ', 'ka', '¼', 'kd', 'þ', 'ðkexkptþ', 'lnð½m0(cid4)½m(cid4)þ', 'matyjaszewski', 'et', 'al', 'wrote', 'kinetic', 'equation', 'atom', 'transfer', 'polymreization', '290', '291.', 'it', 'based', 'atp', 'reaction', 'mechanism', 'described', 'by', 'assuming', 'fast', 'initiations', 'insigniﬁcant', 'termination', 'reactions', 'steady', 'concentrations', 'propagating', 'radicals', 'follow-', 'ing', 'relationship', 'derived', '290', '291:', 'rp', '¼', 'kpkeq½m(cid4)½rx(cid4)0', '½mtzlm(cid4)', '½xmtzþ1lm(cid4)', '¼', 'kapp½m(cid4)', '315', 'thermodynamics', 'free-radical', 'polymerization', 'reaction', '3151', 'effects', 'monomer', 'structure', 'thermodynamics', 'polymerization', 'there', 'close', 'relationship', 'monomer', 'structure', 'changes', 'free', 'energy', 'enthalpy', 'entropy', 'thus', 'instance', 'knowledge', 'changes', 'enthalpy', 'allow', 'appropriate', 'thermal', 'control', 'reaction', 'yield', 'proper', 'rate', 'propagation', 'molecular', 'weight', 'distribution', 'the', 'quantities', 'df', 'dh', 'ds', 'relate', 'rate', 'propagation', 'initiation', 'termination', 'single', 'steps', 'propagation', 'consists', 'multiple', 'steps', 'free', 'radical', 'polymerization', 'generally', 'exothermic', 'involves', 'conversion', 'p', 'bonds', 'bonds', 'thus', 'change', 'enthalpy', 'dh', 'negative', 'also', 'decrease', 'randomness', 'conversion', 'monomers', 'polymer', 'change', 'entropy', 'ds', 'also', 'negative', 'the', 'overall', 'change', 'free', 'energy', 'free', 'radical', 'polymerization', 'process', 'df', '¼', 'dh', '(cid1)', 'tds', 'the', 'free', 'energy', 'generally', 'negative', 'free-radical', 'polymerization', 'process', 'variations', 'monomer', 'structures', 'signiﬁcant', 'effect', 'values', 'dh', 'following', 'reasons', 'these', '132', '3', 'free-radical', 'chain-growth', 'polymerization', 'differences', 'resonance', 'stabilizations', 'due', 'differences', 'conjugation', 'hyperconjugation', 'also', 'due', 'steric', 'strains', 'arise', 'bond', 'angle', 'deformation', 'bond', 'stretching', 'well', 'variations', 'secondary', 'bond', 'forces', 'hydrogen', 'bonding', 'dipole', 'interactions', '3152', 'thermodynamics', 'constrains', 'free-radical', 'polymerization', 'reaction', 'free-radical', 'polymerization', 'reactions', 'equilibrium', 'reactions', 'the', 'equilibrium', 'mono-', 'mer', 'growing', 'polymer', 'subject', 'thermodynamic', 'conditions', 'at', 'equilibrium', 'therefore', 'change', 'free', 'energy', 'zero', 'df', '¼', '0', 'the', 'change', 'free', 'energy', 'reaction', 'therefore', 'written', 'df', '¼', 'df0', 'þ', 'rt', 'ln', 'k', '¼', 'dh0', '(cid1)', 'tds0', 'þ', 'rt', 'ln', 'k', '¼', '0', 'in', 'equation', 'df0', 'dh0', 'ds0', 'represent', 'statistical', 'variations', 'changes', 'free', 'energy', 'enthalpy', 'entropy', 'representing', 'transition', 'monomer', 'undergoes', 'placed', 'polymeric', 'chain', 'the', 'equilibrium', 'constant', 'written', 'pmn1', 'pmn', 'concentrations', 'species', 'assuming', 'practically', 'equal', 'one', 'write', 'would', 'mean', 'k', '¼', '½pm', '(cid11)', 'nþ1(cid4)f½pm(cid11)', 'n(cid4)½m(cid4)g', 'k', '¼', '1m', 'rt', 'ln', 'm', '¼', 'dh0', '(cid1)', 'tds0', 'rt', 'ln', 'm', '¼', 'ðdh0tþ', '(cid1)', 'ds0', 'the', 'ceiling', 'temperature', 'written', 'tc', '¼', 'dh0f', 'ds0', 'þ', 'r', 'lnm(cid4)equg', '316', 'polymer', 'preparation', 'techniques', 'four', 'general', 'techniques', 'used', 'preparation', 'polymers', 'free-radical', 'mechanism', 'polymeri-', 'zation', 'bulk', 'solution', 'suspension', 'emulsion', 'the', 'bulk', 'mass', 'polymerization', 'probably', 'simplest', 'four', 'methods', 'only', 'monomer', 'initiator', 'present', 'reaction', 'mixture', 'it', 'makes', 'reaction', 'simple', 'carry', 'though', 'exotherm', 'reaction', 'might', '316', 'polymer', 'preparation', 'techniques', '133', 'hard', 'control', 'particularly', 'done', 'large', 'scale', 'also', 'chance', 'local', 'hot', 'spots', 'might', 'develop', 'once', 'bulk', 'polymerization', 'vinyl', 'monomers', 'initiated', 'two', 'types', 'results', 'depending', 'upon', 'solubility', 'polymer', 'if', 'soluble', 'monomer', 'reaction', 'may', 'go', 'completions', 'polymer', 'remaining', 'soluble', 'throughout', 'stages', 'conversion', 'as', 'polymerization', 'progresses', 'viscosity', 'reaction', 'mixture', 'increases', 'markedly', 'the', 'propagation', 'proceeds', 'medium', 'associated', 'polymeric', 'chains', 'dissolved', 'swollen', 'monomer', 'monomer', 'consumed', 'if', 'polymer', 'insoluble', 'precipitates', 'without', 'noticeable', 'increase', 'solution', 'viscosity', 'examples', 'type', 'reaction', 'polymerizations', 'acrylonitrile', 'vinylidene', 'chloride', 'the', 'activation', 'energy', 'still', 'similar', 'polymerizations', 'soluble', 'polymers', 'initial', 'rates', 'proportional', 'square', 'root', 'initiator', 'concentration', 'also', 'molecular', 'weights', 'polymerization', 'products', 'inversely', 'proportional', 'polymerization', 'temperatures', 'initiator', 'concentrations', 'furthermore', 'molecular', 'weights', 'resultant', 'polymers', 'far', 'exceed', 'solubility', 'limits', 'polymers', 'monomers', 'the', 'limit', 'acrylonitrile', 'solubility', 'monomer', 'molecular', 'weight', '10000', 'yet', 'polymers', 'molecular', 'weights', 'high', '1000000', 'obtained', 'process', 'this', 'means', 'polymerizations', 'must', 'proceed', 'precipitated', 'polymer', 'particles', 'swollen', 'surrounded', 'monomer', 'molecules', 'the', 'kinetic', 'picture', 'free-radical', 'polymerization', 'applies', 'best', 'bulk', 'polymerizations', 'low', 'points', 'conversion', 'as', 'conversion', 'progresses', 'however', 'reaction', 'becomes', 'complicated', 'chain', 'transferring', 'polymer', 'gel', 'effect', 'the', 'amount', 'chain', 'transferring', 'varies', 'course', 'reactivity', 'polymer', 'radical', 'bulk', 'polymerization', 'employed', 'special', 'properties', 'required', 'high', 'molecular', 'weight', 'maximum', 'clarity', 'convenience', 'handling', 'industrially', 'bulk', 'polymerization', 'special', 'equipment', 'economic', 'advantages', 'bulk', 'polymerization', 'styrene', 'this', 'discussed', 'chap', '6', 'solution', 'polymerization', 'differs', 'bulk', 'polymerization', 'solvent', 'present', 'reaction', 'mixture', 'the', 'monomer', 'may', 'fully', 'partially', 'soluble', 'solvent', 'this', 'polymer', 'may', '(1)', 'completely', 'soluble', 'solvent', '(2)', 'partially', 'soluble', 'solvent', '(3)', 'insoluble', 'solvent', 'when', 'monomer', 'polymer', 'soluble', 'solvent', 'initiation', 'propagation', 'occur', 'homogeneous', 'environment', 'solvent', 'the', 'rate', 'polymerization', 'lower', 'however', 'bulk', 'in', 'addition', 'higher', 'dilution', 'reactants', 'lower', 'rate', 'lower', 'molecular', 'weight', 'product', 'this', 'due', 'chain', 'transferring', 'solvent', 'in', 'addition', 'solvent', 'react', 'form', 'telomers', 'also', 'combine', 'growing', 'chains', 'if', 'monomer', 'soluble', 'solvent', 'polymer', 'partially', 'soluble', 'insoluble', 'initiation', 'still', 'takes', 'place', 'homogeneous', 'medium', 'as', 'chains', 'grow', 'increase', 'viscosity', 'followed', 'precipitation', 'the', 'polymer', 'precipitates', 'swollen', 'state', 'remains', 'swollen', 'diffused', 'adsorbed', 'monomer', 'further', 'propagation', 'takes', 'place', 'swollen', 'particles', 'because', 'propagation', 'continues', 'precipitated', 'swollen', 'polymer', 'precipitation', 'exert', 'strong', 'effect', 'molecular', 'weight', 'product', 'this', 'demonstrated', 'polymeriza-', 'tion', 'styrene', 'benzene', '(where', 'polymer', 'soluble)', 'ethyl', 'alcohol', '(where', 'polymer', 'insoluble)', 'the', 'average', 'molecular', 'weight', 'obtained', 'benzene', '100(cid6)c', '53000', 'ethyl', 'alcohol', 'temperature', '51000', '280.', 'when', 'monomers', 'partially', 'soluble', 'polymers', 'insoluble', 'solvents', 'products', 'might', 'still', 'close', 'molecular', 'weights', 'obtained', 'soluble', 'monomers', 'polymers', 'polymerization', 'acrylonitrile', 'water', 'serve', 'example', 'the', 'monomer', 'soluble', 'extent', '5–7', 'polymer', 'effectively', 'insoluble', 'when', 'aqueous', 'saturated', 'solutions', 'acrylonitrile', 'polymerized', 'water-soluble', 'initiators', 'systems', 'behave', 'initially', 'typical', 'solution', 'polymerizations', 'the', 'polymers', 'however', 'precipitate', 'rather', 'quickly', 'form', 'yet', 'molecular', 'weights', '50000', 'readily', 'obtainable', 'conditions', '134', '3', 'free-radical', 'chain-growth', 'polymerization', 'there', 'different', 'techniques', 'carrying', 'solution', 'polymerization', 'reactions', 'some', 'simple', 'combining', 'monomer', 'initiator', 'solvent', 'applying', 'agitation', 'heat', 'inert', 'atmosphere', '292.', 'others', 'may', 'consist', 'feeding', 'stirred', 'heated', 'solvent', 'monomer', 'initiator', 'continuously', 'given', 'intervals', 'it', 'done', 'throughout', 'course', 'reaction', 'part', '293.', 'such', 'set', 'applied', 'laboratory', 'preparations', 'large-scale', 'commercial', 'preparations', 'it', 'allows', 'somewhat', 'better', 'control', 'exotherm', 'reaction', 'in', 'techniques', 'initiator', 'concentration', 'changes', 'percent', 'early', 'stages', 'reaction', 'reaction', 'temperature', 'high', 'the', 'polymerization', 'may', 'therefore', 'approach', 'steady', 'state', 'character', 'early', 'stages', 'after', 'initial', 'stages', 'however', 'higher', 'temperatures', 'square', 'root', 'dependence', 'rates', 'upon', 'initiator', 'concentration', 'longer', 'holds', 'this', 'result', 'initiator', 'depleted', 'rapidly', 'the', 'second', 'technique', 'initiator', 'monomer', 'initiator', 'added', 'continuously', 'investigated', 'various', 'temperatures', 'rates', 'addition', '294–299.', 'if', 'initiator', 'monomer', 'replenished', 'rates', 'ratios', 'remains', 'constant', 'steady', 'state', 'conditions', 'might', 'extended', 'beyond', 'early', 'stages', 'reactions', 'how', 'long', 'maintained', 'however', 'uncertain', 'suspension', 'polymerization', '298', 'considered', 'form', 'mass', 'polymerization', 'it', 'carried', 'small', 'droplets', 'liquid', 'monomer', 'dispersed', 'water', 'media', 'caused', 'polymerize', 'solid', 'spherical', 'particles', 'the', 'process', 'generally', 'involves', 'dispersing', 'monomer', 'non', 'solvent', 'liquid', 'small', 'droplets', 'the', 'agitated', 'stabilized', 'medium', 'usually', 'consists', 'nonsolvent', '(often', 'water)', 'containing', 'small', 'amounts', 'suspending', 'dispersing', 'agent', 'the', 'initiator', 'dissolved', 'monomer', 'liquid', 'included', 'reaction', 'medium', 'monomer', 'gas', 'to', 'form', 'dispersion', 'monomer', 'must', 'quite', 'insoluble', 'suspension', 'system', 'to', 'decrease', 'solubility', 'sometimes', 'also', 'increase', 'particle', 'size', 'resultant', 'polymer', 'bead', 'partially', 'polymerized', 'monomers', 'prepolymers', 'may', 'used', 'optimum', 'results', 'obtained', 'initiators', 'soluble', 'monomer', 'often', 'differences', 'rates', 'observed', 'polymerization', 'bulk', 'suspension', 'kinetic', 'studies', 'styrene', 'suspension', 'polymerization', 'shown', 'reaction', 'steps', 'initiation', 'propagation', 'termination', 'occur', 'inside', 'particles', '299.', 'the', 'main', 'difﬁculty', 'suspension', 'polymerization', 'forming', 'maintaining', 'uniform', 'suspensions', 'this', 'monomer', 'droplets', 'slowly', 'converted', 'thin', 'immiscible', 'liquids', 'sticky', 'viscous', 'materials', 'subsequently', 'become', 'rigid', 'granules', 'the', 'tendency', 'sticky', 'particles', 'attach', 'form', 'one', 'big', 'mass', 'the', 'suspending', 'agent’s', 'sole', 'function', 'prevent', 'coalescing', 'sticky', 'particles', 'such', 'agents', 'used', 'small', 'quantities', '(001–05', 'weight', 'monomer)', 'there', 'many', 'different', 'suspending', 'agents', 'organic', 'inorganic', 'the', 'organic', 'ones', 'include', 'methylcellulose', 'ethyl', 'cellulose', 'poly(acrylic', 'acid)', 'poly(methacrylic', 'acid)', 'salts', 'acids', 'poly(vinyl', 'alcohol)', 'gelatins', 'starches', 'gums', 'alginates', 'proteins', 'casein', 'zein', 'among', 'inorganic', 'suspending', 'agents', 'listed', 'talc', 'magnesium', 'carbonate', 'calcium', 'carbonate', 'calcium', 'phosphate', 'titanium', 'aluminum', 'oxides', 'silicates', 'clays', 'bentonite', 'others', 'the', 'diameter', 'resultant', 'beads', 'varies', '01', '5', 'mm', 'often', 'depends', 'upon', 'rate', 'agitation', 'it', 'usually', 'inversely', 'proportional', 'particle', 'size', 'suspension', 'polymerization', 'used', 'many', 'commercial', 'preparations', 'polymers', 'zhang', 'fu', 'jiang', 'reported', 'study', 'factors', 'inﬂuencing', 'size', 'polystyrene', 'microspheres', 'dispersion', 'polymerization', '300.', 'the', 'found', 'size', 'polystyrene', 'microspheres', 'decreased', 'increasing', 'amount', 'stabilizer', 'also', 'increased', 'increasing', 'amount', 'monomer', 'initiator', 'the', 'amount', 'stabilizer', 'monomer', 'concentration', 'major', 'factors', 'inﬂuencing', 'size', 'distribution', 'polystyrene', 'microspheres', 'the', 'size', 'microspheres', 'decreased', 'increase', 'solvency', 'reaction', 'media', 'the', 'size', 'distribution', 'however', 'hardly', 'changed', 'the', 'size', 'polystyrene', 'microspheres', 'increased', 'increase', 'reaction', 'temperature', 'size', 'distribution', 'hardly', 'changed', '316', 'polymer', 'preparation', 'techniques', '135', 'fig', '33', 'early', 'stages', 'emulsion', 'polymerization', '(from', 'ref', '306)', 'emulsion', 'polymerization', 'used', 'widely', 'commercial', 'processes', '300', '301.', 'the', 'success', 'technique', 'due', 'part', 'fact', 'method', 'yields', 'high-molecular-weight', 'polymers', 'in', 'addition', 'polymerization', 'rates', 'usually', 'high', 'water', 'continuous', 'phase', 'allows', 'efﬁcient', 'removal', 'heat', 'polymerization', 'also', 'product', 'reaction', 'latex', 'relatively', 'low', 'viscosity', 'spite', 'high', 'molecular', 'weight', 'polymer', 'a', 'disadvantage', 'process', 'water-soluble', 'emulsiﬁers', 'used', 'these', 'hard', 'remove', 'completely', 'polymers', 'may', 'leave', 'degree', 'water', 'sensitivity', 'the', 'reaction', 'commonly', 'carried', 'water', 'containing', 'monomer', 'emulsiﬁer', 'surface-', 'active', 'agent', 'water-soluble', 'initiator', 'initiation', 'may', 'accomplished', 'thermal', 'decom-', 'position', 'initiator', 'redox', 'reaction', 'the', 'polymer', 'forms', 'colloidal', 'dispersion', 'ﬁne', 'particles', 'polymer', 'recovery', 'requires', 'breaking', 'emulsion', 'the', 'full', 'mechanism', 'emulsion', 'polymerization', 'still', 'completely', 'worked', 'it', 'still', 'clear', 'simultaneous', 'increase', 'polymerization', 'rate', 'molecular', 'weight', 'product', 'often', 'observed', 'also', 'emulsion', 'polymerization', 'outset', 'reaction', 'monomer', 'form', 'ﬁnely', 'dispersed', 'droplets', 'these', 'droplets', '1', 'diameter', 'yet', 'process', 'typical', 'polymerization', 'converted', 'polymer', 'particles', 'submicro-', 'scopic', 'eg,', '1000', 'a˚', 'diameter', 'at', 'start', 'reaction', 'emulsiﬁer', 'exists', 'simultaneously', 'three', 'loci', '(a)', 'solute', 'water', '(b)', 'micelles', '(c)', 'stabilizing', 'emulsiﬁer', 'interface', 'monomer', 'droplets', 'water', 'the', 'bulk', 'emulsiﬁer', 'however', 'micelles', 'the', 'monomer', 'also', 'present', 'three', 'loci', '(a)', 'monomer', 'droplets', 'emulsiﬁed', 'perhaps', '1–10', 'diameter', '(b)', 'solubilized', 'micelles', 'perhaps', '50–100', 'a˚', 'diameter', '(c)', 'present', 'individual', 'molecules', 'dissolved', 'water', 'the', 'bulk', 'monomer', 'droplets', 'there', 'average', '1018ml', 'monomer-swollen', 'micelles', 'reaction', 'mixture', 'outset', 'reaction', '302.', 'at', 'start', 'reaction', 'also', 'average', '1012ml', 'monomer', 'droplets', 'act', 'reservoirs', 'the', 'monomer', 'supplied', 'droplets', 'radical-containing', 'micelles', 'reaction', 'progresses', 'process', 'diffusion', 'aqueous', 'phase', '(fig', '33)', 'the', 'ﬁrst', 'hypothesis', 'mechanism', 'emulsion', 'polymerization', 'formulated', 'harkins', '305.', 'according', 'hypothesis', 'water-soluble', 'initiator', 'decomposes', 'aqueous', 'phase', '136', '3', 'free-radical', 'chain-growth', 'polymerization', 'this', 'results', 'formation', 'primary', 'radicals', 'the', 'primary', 'radicals', 'turn', 'react', 'monomer', 'molecules', 'dissolved', 'water', '(though', 'number', 'may', 'quite', 'small)', 'additional', 'monomer', 'molecules', 'may', 'add', 'growing', 'radicals', 'water', 'growing', 'propagating', 'chains', 'free', 'radicals', 'acquire', 'surface-active', 'properties', 'at', 'stage', 'growing', 'radicals', 'consist', 'inorganic', 'organic', 'portions', 'these', 'growing', 'radical-ions', 'tend', 'diffuse', 'monomer-water', 'interfaces', 'the', 'probability', 'diffusion', 'takes', 'place', 'monomer-swollen', 'micelles', 'rather', 'monomer', 'droplets', 'backed', 'considerations', 'relative', 'surface', 'areas', 'two', 'there', 'average', '1018', 'micelles', 'milliliter', 'water', 'these', 'approximately', '75', 'a¯', 'diameter', 'swollen', 'micelle', 'contains', 'average', '30', 'molecules', 'monomer', 'at', 'time', 'diameters', 'monomer', 'droplets', 'approximately', '1', 'estimated', 'approximately', '1012', 'droplets', 'per', 'milliliter', 'water', 'thus', 'micelles', 'offer', '60', 'times', 'surfaces', 'penetration', 'droplets', 'the', 'initiating', 'radicals', 'almost', 'always', 'generated', 'water', 'phase', 'after', 'formation', 'water', 'phase', 'number', 'free', 'radicals', 'may', 'lost', 'due', 'recombination', 'termination', 'also', 'possible', 'reaction', 'free', 'radicals', 'monomers', 'dissolved', 'water', 'several', 'theories', 'tried', 'explain', 'entry', 'process', 'thus', '“diffusion', 'control”', 'model', '307', '308', 'supposes', 'diffusion', 'aqueous-phase', 'radicals', 'particle', 'surface', 'rate-controlling', 'step', 'entry', 'another', 'theory', 'suggests', 'displacement', 'surfactant', 'particle', 'surface', 'rate-', 'determining', 'step', '309.', 'a', 'third', 'one', 'assumes', 'entry', 'thought', 'colloidal', 'interaction', 'latex', 'particle', 'primary', 'phase', 'oligomeric', 'aqueous-phase', 'radical', 'these', 'radicals', 'formed', 'reactions', 'initiating', 'radicals', 'monomer', 'molecules', 'dissolved', 'water', '310.', 'the', 'accepted', 'entry', 'model', 'appears', '“control', 'aqueous-phase', 'growth”', 'model', 'maxwells', 'et', 'al', '311.', 'this', 'theory', 'postulates', 'free', 'radicals', 'generated', 'aqueous', 'phase', 'propagate', 'reach', 'critical', 'degree', 'polymerization', '(let', 'us', 'call', 'z)', 'point', 'become', 'surface-', 'active', 'fate', 'irreversible', 'entry', 'latex', 'particle', 'rate', 'entry', 'z-mers', 'particle', 'assumed', 'fast', 'rate-determining', 'an', 'efﬁciency', 'less', '100', 'arises', 'signiﬁcant', 'aqueous-phase', 'termination', 'propagating', 'radicals', 'the', 'entry', 'model', 'maxwells', 'et', 'al', 'derived', 'andor', 'supported', 'data', 'inﬂuence', 'particle', 'surface', 'characteristics', '(charge', 'size)', 'entry', 'rate', 'coefﬁcient', '312.', 'it', 'assumed', 'aqueous', 'radicals', 'became', 'surface', 'active', 'degree', 'polymerization', 'reached', '2–3', 'this', 'based', 'thermodynamic', 'considerations', 'entering', 'species', 'further', 'data', 'maxwell', 'et', 'al', 'entry', 'model', 'obtained', 'gilbert', 'coworkers', '313', 'studied', 'effects', 'initiator', 'particle', 'surface', 'charges', 'they', 'obtained', 'kinetic', 'data', 'radical', 'entry', 'emulsion', 'polymerization', 'styrene', 'concluded', 'data', 'supports', 'maxwell', 'et', 'al', 'entry', 'model', 'refutes', 'alternative', 'models', 'mentioned', 'once', 'radicals', 'penetrate', 'micelles', 'polymerization', 'continues', 'adding', 'monomers', 'inside', 'the', 'equilibrium', 'disturbed', 'propagation', 'process', 'proceeds', 'high', 'rate', 'due', 'concentration', 'crowding', 'stabilized', 'monomers', 'this', 'rapidly', 'transforms', 'monomer-', 'swollen', 'micelles', 'polymer', 'particles', 'the', 'changes', 'result', 'disruptions', 'micelles', 'growths', 'within', 'the', 'amount', 'emulsiﬁer', 'present', 'changing', 'micelles', 'insufﬁcient', 'stabilize', 'polymer', 'particles', 'in', 'trying', 'restore', 'equilibrium', 'micelles', 'polymer', 'growth', 'disintegrate', 'supply', 'growing', 'polymer', 'particles', 'emulsiﬁer', 'in', 'process', 'many', 'micelles', 'disappear', 'per', 'polymer', 'particle', 'forms', 'the', 'ﬁnal', 'latex', 'usually', 'ends', 'containing', '1015', 'polymer', 'particles', 'per', 'milliliter', 'water', 'by', 'time', 'conversions', 'reach', '10–20', 'micelles', 'present', 'reaction', 'mixtures', 'all', 'emulsiﬁer', 'adsorbed', 'surface', 'polymer', 'particles', 'this', 'means', 'new', 'polymer', 'particles', 'formed', 'all', 'reactions', 'sustained', 'diffusion', 'monomer', 'molecules', 'monomer', 'droplets', 'growing', 'polymer', 'particles', 'the', 'amount', 'monomer', 'diffusing', 'particles', 'always', 'excess', 'amount', 'consumed', 'polymerization', 'reaction', 'due', 'osmotic', 'forces', '297.', '316', 'polymer', 'preparation', 'techniques', '137', 'this', 'extra', 'monomer', 'supplied', 'sufﬁcient', 'equilibrium', 'swelling', 'particles', '298.', 'as', 'result', 'rate', 'polymerization', 'becomes', 'zero', 'order', 'respect', 'time', 'when', 'conversion', 'reaches', '70,', 'remaining', 'monomer', 'absorbed', 'polymer', 'particles', 'monomer', 'droplets', 'left', 'at', 'point', 'reaction', 'rate', 'becomes', 'ﬁrst', 'order', 'respect', 'time', 'the', 'qualitative', 'approach', 'harkins', 'put', 'quantitative', 'basis', 'smith', 'ewart', '314–316.', 'because', '1013', 'radicals', 'produced', 'per', 'second', 'enter', '1014', '1015', 'particles', 'smith', 'felt', 'free', 'radical', 'enter', 'particle', 'every', '10–100', 'it', 'cause', 'polymerization', 'occur', '10–100', 'another', 'free', 'radical', 'would', 'enter', 'terminate', 'chain', 'growth', '317.', 'a', 'period', 'inactivity', 'would', 'follow', 'would', 'last', '10–100', 'process', 'would', 'repeat', 'such', '“stop', 'go”', 'mechanism', 'implies', 'particle', 'contains', 'free', 'radical', 'approximately', 'half', 'time', 'it', 'also', 'said', 'average', 'number', 'radicals', 'per', 'particle', '05', 'this', 'predicted', 'conditions', '(a)', 'rate', 'chain', 'transfer', 'particle', 'negligible', '(b)', 'rate', 'termination', 'rapid', 'compared', 'rate', 'radical', 'entry', 'particle', 'the', 'kinetic', 'relationships', 'derived', 'smith', 'ewart', 'system', 'follows', 'the', 'rate', 'primary', 'radical', 'entering', 'particle', '¼', 'ri', '¼', 'rin', 'rate', 'polymerization', '¼', 'rp', '¼', 'kp½m(cid4)n2', 'average', 'degree', 'polymerization', '¼', 'dp', '¼', 'nkp½m(cid4)ri', 'kp', 'constant', 'propagation', 'm', 'concentration', 'monomer', 'n', 'number', 'particles', 'containing', 'n', 'radicals', '(05)', 'expression', 'number', 'particles', 'formed', 'n', '¼', 'kðrmþ04ðassþ06', 'volume', 'increase', 'particles', 'as', 'area', 'occupied', 'one', 'emulsiﬁer', 'molecule', 's', 'amount', 'emulsiﬁer', 'present', 'k', 'constant', '¼', '037', '(based', 'assumption', 'micelles', 'polymer', 'particles', 'compete', 'free', 'radicals', 'proportion', 'respective', 'total', 'surface', 'areas)', 'k', 'also', 'equal', '053', '(based', 'assumption', 'primary', 'radicals', 'enter', 'micelles', 'long', 'remain', 'micelles', 'reaction', 'mixture)', 'r', 'rate', 'entry', 'particles', 'the', 'kinetic', 'chain', 'length', 'written', 'u', '¼', 'kpnp½m(cid4)2d½rm', '(cid11)(cid4)dt', 'the', 'smith-ewart', 'mechanism', 'take', 'account', 'polymerization', 'aqueous', 'phase', 'this', 'may', 'true', 'monomers', 'quite', 'insoluble', 'water', 'styrene', 'appears', 'unlikely', 'hydrophilic', 'ones', 'methyl', 'methacrylate', 'vinyl', 'acetate', 'in', 'addition', 'calculated', 'flory', 'insufﬁcient', 'time', 'typical', 'cation-radical', '(such', 'sulfate', 'ion', 'radical)', 'add', 'dissolved', 'molecule', 'monomer', 'styrene', 'becomes', 'captured', 'micelle', '317.', 'this', 'argued', 'however', 'ground', 'flory’s', 'calculations', 'fail', 'consider', 'potential', 'energy', 'barrier', 'micelle', 'surfaces', 'electrical', 'double', 'layer', 'this', 'barrier', 'would', 'reduce', 'rate', 'diffusion', 'radical-ions', 'micelles', '316.', 'considerably', 'different', 'mechanisms', 'proposed', 'several', 'groups', '317', '318.', 'they', 'based', 'concept', 'polymerizations', 'must', 'take', 'place', 'surface', 'particles', 'outer', '“shell”', 'within', 'particles', 'it', 'claimed', 'interiors', 'particles', 'viscous', 'free', 'radicals', 'diffuse', 'inside', 'sufﬁciently', 'fast', 'rate', 'two', 'different', 'mechanisms', 'proposed', 'explain', 'polymerization', 'takes', 'place', 'preferentially', 'shell', 'layer', 'one', 'suggests', '138', '3', 'free-radical', 'chain-growth', 'polymerization', 'monomer', 'distributed', 'nonuniformly', 'polymer', 'particles', 'the', 'outer', 'shell', 'rich', 'monomer', 'molecules', 'inside', 'rich', 'polymer', 'molecules', '319.', 'the', 'explanation', 'radical', 'ions', 'form', 'water-soluble', 'initiator', 'hydrophilic', 'able', 'penetrate', 'polymer', 'particles', '320.', 'surfactant-free', 'emulsion', 'polymerization', 'carried', 'absence', 'surfactant', '321.', 'the', 'technique', 'requires', 'use', 'initiators', 'yields', 'initiating', 'species', 'surface-active', 'properties', 'imparts', 'polymer', 'particles', 'examples', 'initiators', 'persulfates', 'the', 'lattices', 'form', 'stabilized', 'chemically', 'bound', 'sulfate', 'groups', 'derived', 'persulfate', 'ions', 'because', 'surface-active', 'groups', 'chemically', 'bound', 'lattices', 'easier', 'purify', 'free', 'product', 'unreacted', 'monomer', 'initiator', 'generally', 'particle', 'number', 'per', 'milliliter', 'surfactant-free', 'emulsion', 'polymerization', 'smaller', 'particle', 'number', 'typical', 'emulsion', 'polymerization', 'in', 'inverse', 'emulsion', 'polymerization', 'aqueous', 'solution', 'hydrophilic', 'monomer', 'emulsiﬁed', 'organic', 'solvent', 'polymerization', 'initiated', 'solvent', 'soluble', 'initiator', 'this', 'type', 'emulsion', 'polymerizations', 'referred', 'water', 'oil', 'polymerization', 'inverse', 'emulsion', 'polymerization', 'used', 'various', 'commercial', 'polymerizations', 'copolymerization', 'water-soluble', 'monomers', 'often', 'nonionic', 'emulsiﬁers', 'utilized', 'the', 'product', 'emulsions', 'often', 'less', 'stable', 'oil', 'water', 'emulsions', 'a', 'special', 'approach', 'emulsion', 'polymerization', 'called', 'miniemulsion', 'polymerization', '322.', 'these', 'reactions', 'contain', 'micelles', 'monomer', 'droplets', 'monomer', 'droplets', 'smaller', 'macrosystems', 'usually', 'water-soluble', 'surfactant', 'used', 'emulsiﬁcation', 'an', 'example', 'surfactant', 'sodium', 'dodecyl', 'sulfate', 'in', 'addition', 'highly', 'water-insoluble', 'costabilizer', 'added', 'hexadecanol', 'thus', 'miniemulsions', 'dispersions', 'critically', 'stabilized', 'oil', 'droplets', 'size', '50', '500', 'nm', 'prepared', 'shearing', 'system', 'containing', 'oil', 'water', 'surfactant', 'hydrophobic', 'material', 'polymerizations', 'miniemulsions', 'carefully', 'prepared', 'result', 'latex', 'particles', 'size', 'initial', 'droplets', 'an', 'appropriate', 'formulation', 'miniemulsion', 'suppresses', 'coalescence', 'droplets', 'the', 'polymerization', 'miniemulsions', 'extends', 'possibilities', 'widely', 'applied', 'emulsion', 'polymerization', 'provides', 'advantages', 'respect', 'copolymerization', 'reactions', 'monomers', 'different', 'polarity', 'incorporation', 'hydrophobic', 'materials', 'respect', 'stability', 'formed', 'latexes', 'although', 'labeled', '“emulsion”', 'appears', 'may', 'involve', 'combination', 'emulsion', 'suspension', 'polymerizations', 'it', 'reported', '323', 'using', 'difunctional', 'alkoxyamine', 'initiator', 'homopolymerization', 'butyl', 'acrylate', 'miniemulsion', 'increase', 'achievable', 'molar', 'mass', 'use', 'polymer', 'difunctional', 'macroinitiator', 'synthesis', 'triblock', 'copolymers', 'aqueous', 'dispersed', 'systems', 'well-deﬁned', 'polymers', 'one', 'alkoxyamine', 'functionality', 'end', 'obtained', 'providing', 'monomer', 'conversion', 'kept', '70.', 'beyond', 'conversion', 'extensive', 'broadening', 'molar', 'mass', 'distribution', 'evidenced', 'consequence', 'termination', 'transfer', 'polymer', 'tsavalas', 'et', 'al', '324', 'reported', 'phenomenon', 'seemingly', 'unique', 'hybrid', 'miniemulsion', 'polymerization', 'observed', 'monomer', 'conversion', 'would', 'either', 'plateau', 'limiting', 'value', 'quickly', 'switch', 'dramatically', 'lesser', 'rate', 'they', 'attributed', 'phenomenon', 'combina-', 'tion', 'three', 'factors', 'the', 'ﬁrst', 'one', 'degree', 'monomer', 'resinous', 'component', 'compatible', 'the', 'second', 'resultant', 'particle', 'morphology', 'approximately', '80', 'monomer', 'conversion', 'roughly', 'corresponds', 'portion', 'reaction', 'morphology', 'established', 'the', 'third', 'factor', 'degree', 'interaction', 'growing', 'polymer', 'resin', '(a', 'grafting', 'reaction)', 'of', 'three', 'ﬁrst', 'two', 'factors', 'found', 'much', 'signiﬁcant', 'contributing', 'limiting', 'conversion', 'raft', 'emulsion', 'polymerization', 'new', 'development', 'attracted', 'considerable', 'attention', 'it', 'carried', 'regular', 'emulsion', 'polymerization', '325', 'reverse', 'emulsion', 'polymeriza-', 'tion', '326.', 'also', 'literature', 'described', 'miniemulsion', 'raft', 'polymerizations', 'in', 'instances', 'use', 'made', 'water-soluble', 'raft', 'agents', 'control', 'polymer', 'recently', 'several', 'reports', 'review', 'questions', '139', 'molecular', 'weight', '327.', 'also', 'hawkett', 'coworkers', 'reported', 'using', 'surface', 'active', 'raft', 'agents', 'emulsify', 'dispersed', 'phase', 'stabilize', 'particles', 'also', 'control', 'molecular', 'weight', 'this', 'yielded', 'polymer', 'latexes', 'free', 'surfactant', 'costablilizer', '328.', 'one', 'special', 'raft', 'agents', 'illustrated', 'follows', 'the', 'two', 'raft', 'agents', 'used', 'similar', 'structures', 'a', 'surface', 'active', 'iniferter', 'also', 'reported', 'choe', 'coworkers', '330', '331:', 's', 'cooh', 'ch3', 'c4h9s', 's', 'x', 'oh', 'o', 's', 's', 'coona', 'this', 'raft', 'agent', 'allowed', 'polymerization', 'methyl', 'methacrylate', 'initiated', 'ultraviolet', 'light', 'irradiation', 'absence', 'added', 'surfactant', 'initiator', 'rieger', 'coworkers', '332', 'reported', 'surfactant', 'free', 'raft', 'emulsion', 'polymerization', 'butyl', 'acrylate', 'styrene', 'using', 'poly(nn-dimethylacrylamide)', 'trithiocarbonate', 'macromolecular', 'transfer', 'agent', 'they', 'observed', 'polymerizations', 'fast', 'controlled', 'molar', 'masses', 'matched', 'well', 'theoretical', 'values', 'low', 'polydispersity', 'indexes', 'monomer', 'conversions', 'close', '100', 'reached', 'polymerizations', 'behaved', 'controlled', 'systems', 'even', '40', 'solids', 'contents', 'the', 'products', 'poly(nn-dimethyl', 'acrylamide)-b-poly(n-butyl', 'acrylate)', 'poly(nn-', 'dimethylacrylamide)-b-polystyrene', 'amphiphilic', 'diblock', 'copolymers', 'formed', 'situ', 'review', 'questions', 'section', '31', 'polymerization', '1', 'what', 'three', 'steps', 'free-radical', 'polymerization', 'illustrate', 'step', 'free-radical', 'chain', '2', 'what', 'rate-determining', 'step', 'free-radical', 'polymerization', '3', 'write', 'kinetic', 'expressions', 'initiation', 'propagation', 'termination', 'transfer', '4', 'what', 'steady', 'state', 'assumption', 'how', 'expressed', 'why', 'necessary', '5', 'what', 'expression', 'rate', 'propagation', 'rate', 'monomer', 'disappearance', 'the', 'average', 'lifetime', 'growing', 'radical', 'steady', 'state', 'conditions', '6', 'why', 'initiator', 'efﬁciency', 'factor', 'needed', 'rate', 'equation', 'what', 'kinetic', 'chain', 'length', '7', 'in', 'polymerization', 'styrene', 'benzene', '60(cid6)c', 'using', '01', 'mol', 'benzoyl', 'peroxide', 'initiator', '1', 'mol', 'monomer', 'kd', '138', '(cid7)', '10─5', 'assume', 'steady', 'state', 'calculate', 'free-radical', 'concentration', 'reaction', 'if', 'kp', '176mol', 'rate', 'propagation', 'what', 'lifetime', 'growing', 'radical', 'kt', '72', '(cid7)', '10─5', 'what', 'rate', 'propagation', 'initiator', 'efﬁciency', '72?', '140', 'section', '32', '3', 'free-radical', 'chain-growth', 'polymerization', '1', 'what', 'sources', 'initiating', 'free', 'radicals', 'know', 'illustrate', 'decomposition', 'reaction', 'aa0-', 'azobisisobutyronitrile', 'illustrate', 'free', 'radicals', 'recombine', 'inside', 'outside', 'solvent', 'cage', 'lost', 'initiation', 'process', '2', 'illustrate', 'one', 'inorganic', 'also', 'one', 'organic', 'peroxides', '3', 'how', 'solvent', '“cage”', 'affect', 'initiating', 'free', 'radicals', 'explain', 'illustrate', '4', 'explain', 'homolytic', 'heterolytic', 'cleavage', 'peroxides', '5', 'explain', 'give', 'chemical', 'equations', 'redox', 'initiations', 'fe+', 'co+', 'ce++', 'ions', 'reaction', 'mixture', '6', 'how', 'peroxides', 'benzoyl', 'peroxide', 'decomposed', 'aromatic', 'tertiary', 'amines', 'show', 'two', 'postulated', 'mechanisms', 'reaction', 'benzoyl', 'peroxide', 'dimethyl', 'aniline', '1', 'describe', 'reaction', 'initiating', 'free', 'radical', 'monomer', 'show', 'reaction', 'equations', 'using', 'phenyl', 'initiating', 'radical', 'styrene', 'monomer', 'example', '2', 'do', 'question', '1', 'redox', 'mechanism', 'showing', 'sulfate', 'ion-radical', 'adding', 'section', '33', 'vinyl', 'acetate', 'section', '34', '1', 'illustrate', 'transition', 'state', 'propagation', 'reaction', '2', 'explain', 'steric', 'polar', 'resonance', 'effects', 'propagation', 'reaction', '3', 'explain', 'tendency', 'trans–trans', 'placement', 'propagation', 'reactions', 'carried', 'low', 'temperatures', '4', 'how', 'reaction', 'medium', 'affect', 'propagation', 'reaction', '5', 'what', 'ceiling', 'temperature', 'kinetic', 'expression', 'phenomenon', '6', 'explain', 'meant', 'autoacceleration', 'manifest', '7', 'what', 'cyclopolymerization', 'explain', 'give', 'several', 'examples', 'section', '35', 'reaction', '1', 'what', 'three', 'termination', 'processes', 'free-radical', 'polymerization', '2', 'what', 'meant', 'degenerative', 'chain', 'transferring', 'illustrate', 'back-biting', 'the', 'telomerization', '3', 'what', 'meant', 'chain', 'transferring', 'constants', '4', 'write', 'equation', 'degree', 'polymerization', 'including', 'chain', 'transferring', 'constants', 'in', 'benzoyl', 'peroxide', 'initiated', 'polymerization', '2', 'moles', 'styrene', 'benzene', '85(cid6)c', '(kd', '¼', '894', '(cid7)', '10(cid1)5', 'lmol-s', '85(cid6)c)', 'how', 'much', 'benzoyl', 'peroxide', 'required', 'polymeri-', 'zation', 'solution', 'attain', 'average', 'molecular', 'weight', '250000', 'assume', 'termination', 'occurs', 'recombination', 'chain-transferring', 'takes', 'place', 'review', 'questions', '141', '5', 'in', 'polymerization', 'transferring', 'constant', 'monomer', 'cm', '(cid7)', '10─4', '¼', '374', 'transferring', 'constant', 'solvent', 'cs', '¼', '56', '(cid7)', '10─4', 'transferring', 'constant', 'initiator', 'ci', '¼', '075', 'assuming', 'f', '¼', '072', 'kt', '72', '(cid7)', '10─5', 'kp', '176mol', '(kd', 'shown', 'above)', 'calculate', 'rp', 'average', 'dp', '1', 'explain', 'meant', 'reactivity', 'ratios', 'derived', '2', 'write', 'copolymerization', 'equation', 'in', 'copolymerization', '1', 'mol', 'styrene', '1', 'mol', 'butadiene', 'r1', '¼', '078', 'r2', '¼', '139', 'expected', 'composition', 'copolymer', 'early', 'stages', 'polymerization', '3', 'how', 'substituents', 'monomer', 'molecules', 'affect', 'reactivity', 'monomers', 'toward', 'attacking', 'radicals', '4', 'explain', 'q', 'e', 'scheme', 'write', 'price–alfrey', 'equation', '5', 'how', 'r1', 'r2', 'derived', 'q', 'e', 'values', 'show', 'relationship', '6', 'from', 'chemical', 'structures', 'alone', 'predict', 'products', 'free-radical', 'copolymerizations', 'pairs', '(1)', 'styrene', 'methyl', 'methacrylate', '(2)', 'styrene', 'vinyl', 'acetate', '(3)', 'methyl', 'methacrylate', 'vinyl', 'chloride', 'consult', 'table', '38', 'reactivity', 'ratios', '1', 'how', 'many', 'reactivity', 'ratios', 'terpolymerization', '2', 'write', 'equation', 'terpolymerization', 'reaction', '1', 'what', 'allylic', 'polymerization', 'if', 'allyl', 'alcohol', 'polymerize', 'high-molecular-weight', 'polymer', 'free-radical', 'polymerization', 'triallyl', 'cyanurate', 'form', 'high-molecular-weight', 'network', 'structure', 'mechanism', '1', 'what', 'inhibition', 'retardation', 'explain', '2', 'give', 'example', 'good', 'inhibitor', 'good', 'retarder', 'show', 'chemical', 'equations', 'reaction', 'free', 'radicals', '3', 'show', 'reaction', 'quinone', 'free', 'radicals', '4', 'write', 'equation', 'relates', 'rate', 'data', 'inhibited', 'polymerizations', '5', 'calculate', 'rp', 'polymerization', '1', 'mol', 'styrene', 'containing', '001', 'mol', 'hydroquinone', 'inhibitor', 'kp', '¼', '176mol', 'kt', '¼', '72', '(cid7)', '10─5', 'section', '36', 'section', '37', 'section', '38', 'section', '39', '3', 'free-radical', 'chain-growth', 'polymerization', '1', 'what', 'thermal', 'polymerization', 'show', 'chemical', 'equations', 'postulated', 'mechanism', 'formation', 'initiating', 'radicals', 'styrene', 'thermal', 'polymerization', '1', 'show', 'proposed', 'charge-transfer', 'mechanism', 'copolymerization', 'styrene', 'maleic', 'anhydride', 'dioxene', 'maleic', 'anhydride', '2', 'what', 'determines', 'stability', 'charge-transfer', 'complexes', 'explain', 'give', 'examples', '1', 'how', 'polar', 'monomers', 'complex', 'lewis', 'acids', 'how', 'affect', 'polymerization', 'monomers', 'copolymerization', '1', 'how', 'canal', 'complexes', 'used', 'steric', 'control', 'free-radical', 'polymerization', 'give', '142', 'section', '310', 'section', '311', 'section', '312', 'section', '313', 'examples', 'section', '314', '1', 'how', 'controlled“living”', 'polymerizations', 'differ', 'typical', 'living', 'polymerizations', '2', 'list', 'different', 'types', 'controlled“living”', 'free-radical', 'polymerizations', 'think', 'what', 'three', 'classes', 'homogeneous', 'controlled“living”', 'polymerizations', 'described', 'matyjaszewski', 'illustrate', '3', 'describe', 'cobalt', 'mediated', 'polymerizations', 'illustrate', '4', 'describe', 'illustrate', 'atom', 'transfer', 'polymerizations', 'controlled', 'copperbipyridine', 'complex', 'carbon', 'tetrachloride', 'dichloro(triphenyl-phosphine)-ruthenium(ii)', 'methylaluminum', 'bis(26-di-tert-butyl-phenoxide)', 'explain', '(argent)atrp', '(set-lrp)', 'mean', 'illustrate', 'proposed', 'percec', 'mechanism', 'matyjaszewski', 'mechanisms', '5', 'describe', 'nitroxyl', 'radical', 'mediated', 'polymerizations', 'illustrate', 'tempo', 'controlled', 'polymerization', 'styrene', 'what', 'meant', 'situ', 'nmp', '6', 'explain', 'illustrate', 'reversible', 'addition-fragmentation', 'chain', 'transfer', 'polymerization', '(raft)', '7', 'write', 'matyjaszewski', 'proposed', 'kinetic', 'equation', 'atp', 'polymerization', 'references', 'section', '315', '143', '1', 'what', 'meant', 'bulk', 'mass', 'polymerization', 'explain', 'discuss', '2', 'what', 'techniques', 'carrying', 'solution', 'polymerizations', '3', 'give', 'qualitative', 'picture', 'emulsion', 'polymerization', 'described', 'harkins', '4', 'how', 'smith', 'ewart', 'put', 'harkins', 'picture', 'emulsion', 'polymerization', 'quantitative', 'basis', 'what', 'equation', 'rate', 'emulsion', 'polymerization', '5', 'describe', 'surfactant-free', 'emulsion', 'polymerization', 'inverse', 'emulsion', 'polymerization', 'miniemulsion', 'polymerization', '1', 'discuss', 'effect', 'monomer', 'structure', 'thermodynamics', 'free-radical', 'polymerization', 'section', '316', 'process', 'references', 'recommended', 'reading', 'g', 'moad', 'dh', 'solomon', 'the', 'chemistry', 'radical', 'polymerization', 'pergamon', 'press', 'oxford', '1995', '1', 't', 'fueno', 't', 'tsuruta', 'j', 'furukawa', 'j', 'polymer', 'sci,', '1959', '40', '487', '2', 'ge', 'scott', 'e', 'serrogles', 'j', 'macromol', 'sci', '-rev', 'macromol', 'chem,', '1973', 'c9(1)', '49', 'ch', 'bamford', '“radical', 'polymerization”', 'encyclopedia', 'polymer', 'science', 'engineering', 'vol13', 'hfmark', 'nm', 'bikales', 'cg', 'overberger', 'g', 'menges', '(eds)', 'wiley-interscience', 'new', 'york', '1988', '3', 's', 'inoue', 'chapter', '5', 'structure', 'mechanism', 'vinyl', 'polymerization', 't', 'tsuruta', 'k', 'f', 'o’driscoll', '(eds)', 'dekker', 'new', 'york', '1969', '4', 'j', 'thiele', 'k', 'hauser', 'ann,', '1896', '290', '5', 'd', 'swern', 'ed', 'organic', 'peroxides', 'wiley-interscience', 'new', 'york', '1970', 's', 'patai', '(ed),', 'the', 'chemistry', 'peroxides', 'wiley-interscience', 'new', 'york', '1983', '6', 'ia', 'saad', 'f', 'r', 'eirich', 'am', 'chem', 'soc,', 'polymer', 'preprints', '1960', '1', '276', '7', 'gt', 'russell', 'dh', 'napper', 'rg', 'gilbert', 'macromolecules', '1988', '21', '2133', '8', 'fh', 'solomon', 'g', 'moad', 'makromol', 'chem,', 'macromol', 'symp1987,', '1011', '109', '9', 'pd', 'bartlett', 'r', 'r', 'hiatt', 'j', 'am', 'chem', 'soc1958,', '80', '1398', '10', 'cg', 'szwarc', 'l', 'herk', 'j', 'chem', 'phys,', '1958', '9', '438', '(1958)', '11', 'avtobolsy', 'rbmesrobian', '“organic', 'peeroxides”wiley', 'new', 'york', '1954', '12', 'mg', 'evans', 'trans', 'faraday', 'soc1946,', '42', '101', '13', 'cg', 'swain', 'wt', 'stockmayer', 'jt', 'clarke', 'j', 'am', 'chem', 'soc,', '1950', '72', '5726', '14', 'pb', 'bartlett', 'k', 'nozaki', 'j', 'am', 'chem', 'soc,', '1946', '68', '1686', 'ibid,', '1947', '69', '2299', '15', 't', 'koenig', 'decomposition', 'peroxides', 'azoalkanes', 'ch3', 'free-radicals', 'voli', 'jk', 'kochi', '(ed)', 'wiley', 'new', 'york', '1973', '16', 'psengel', 'y', 'ying', 's', 'he', 'macromolecules', '2003', '36', '3821', '17', 'at', 'blomquist', 'aj', 'buselli', 'j', 'am', 'chem', 'soc,', '1951', '73', '3883', 'cs', 'sheppard', '“peroxy', 'compounds”', 'encyclopedia', 'polymer', 'science', 'engineering', 'vol11', 'hf', 'mark', 'nm', 'bikales', 'cg', 'overberger', 'g', 'menges', 'eds,', 'wiley-interscience', 'new', 'york', '1988', '18', 'mg', 'evans', 'trans', 'faraday', 'soc1946,', '42', '101', '19', 'l', 'horner', 'e', 'schlenk', 'angew', 'chem', '1949', '61', '711', '20', 'm', 'imoro', 's', 'choe', 'j', 'polymer', 'sci,', '1955', '15', '482', '144', '3', 'free-radical', 'chain-growth', 'polymerization', '21', 'ss', 'harihan', 'm', 'maruthamuthu', 'makromol', 'chem,', '1979', '180', '2031', '22', 'wa', 'pryor', 'wh', 'hendrickson', 'jr,', 'tetrahedron', 'lett,', '1983', '24', '1459', '23', 'v', 'horanska', 'j', 'barton', 'z', 'manasek', 'j', 'polymer', 'sci,', '1972', 'a-110', '2701', '24', 't', 'hirano', 't', 'miki', 't', 'tsuruta', 'makromol', 'chem,', '1967', '104', '230', '25', 'b', 'yamada', 'h', 'kamei', 't', 'otsu', 'j', 'polymer', 'sci,chem', 'ed,', '1980', '18', '1917', '26', 'l', 'michl', 's', 'korbek', 'vyakaranam', 'j', 'b', 'barbour', 'j', 'am', 'chem', 'soc,', '2006', '128', '5610', '5680', '27', 'gs', 'kolesnikov', 'ls', 'fedorova', 'izv', 'akad', 'nauk', 'sssr', 'otd', 'khim', 'nauk', '1957', '236', '28', 'j', 'furukawa', 't', 'tsuruta', 'sinoue', 'j', 'polymer', 'sci1957,', '26', '234', '29', 'n', 'ashikari', 'n', 'nishimura', 'j', 'polymer', 'sci,', '1958', '28', '250', '30', 'rc', 'petry', 'fh', 'verhoek', 'j', 'am', 'chem', 'soc,', '1956', '78', '6416', '31', 'j', 'furukawa', 't', 'tsuruta', 'j', 'polymer', 'sci,', '1958', '28', '227', '32', 'ceh', 'bawn', 'd', 'margerison', 'nm', 'richardson', 'proc', 'chem', 'soc', '(london)', '1959', '397', '33', 'td', 'parsons', 'mb', 'silverman', 'dm', 'ritter', 'j', 'am', 'chem', 'soc,', '1957', '79', '5091', '34', 'rl', 'hansen', 'rr', 'hammann', 'j', 'phys', 'chem,', '1963', '67', '2868', '35', 's', 'inoue', 'chapter', '5', 'structure', 'mechanism', 'vinyl', 'polymerization', 't', 'tsuruta', 'kf', 'o’driscoll', 'eds,', 'dekker', 'new', 'york', '1969', '36', 'al', 'barney', 'jm', 'bruce', 'jr,', 'j', 'n', 'coker', 'hw', 'jackobson', 'w', 'h', 'sharkey', 'j', 'polymer', 'sci,', '1966', 'a-14', '2617', '37', 'y', 'nakoyama', 't', 'tsuruta', 'j', 'furukawa', 'makromol', 'chem1960,', '40', '79', '38', 'a', 'charlseby', 'chap1,“irradiation', 'polymers”', 'n', 'platzer', '(ed)', 'am', 'chem', 'soc,', 'advancement', 'chem', 'series', '66', 'washington', '1967', 'jg', 'calvin', 'jn', 'pitts', 'jr,', 'photochemistry', 'wiley', 'new', 'york', '1967', 'n', 'j', 'turo', 'molecular', 'photochemistry', 'benjamin', 'new', 'york', '1967', '39', 'je', 'wilson', 'radiation', 'chemistry', 'monomers', 'polymers', 'plastics', 'dekker', 'new', 'york', '1974', '40', 'z', 'szablan', 't', 'm', 'lovestead', 't', 'p', 'davis', 'm', 'h', 'stenzel', 'c', 'barnerkowollik', 'macromolecules', '2007', '40', '26', '41', 'j', 'frank', 'e', 'rabinowitch', 'trans', 'faraday', 'soc193430,', '120', '42', 'wa', 'pryor', 'free', 'radicals', 'mcgraw-hill', 'new', 'york', 'ny,', '1966', '43', 'jc', 'martin', '“solvation', 'association”', 'chapt20', 'free-radicals', 'volii', 'jk', 'kochi', '(ed)', 'wiley', 'new', 'york', '1973', '44', 'rj', 'glitter', 'rj', 'albers', 'j', 'org', 'chem,', '1964', '29', '728', '45', 'b', 'giese', 'am', 'chem', 'soc', 'polymer', 'preprints', '1997', '38', '(1)', '639', '46', 'h', 'zipse', '1', 'he', 'k', 'n', 'houk', 'b', 'giese', '113', 'j', 'am', 'chem', 'sec', '1991', '4324', '47', 'b', 'giesce', 'j', 'he', 'w', 'mehl', 'chem', 'ber,', '1988', '121', '2063', '48', 'b', 'giese', 'angew', 'chem', 'int', 'ed', 'engl', '1983', '22', '753(1983)', '49', 'b', 'giese', 'w', 'damm', 'r', 'batra', 'chemtracts', '1994', '7', '355', '50', 'b', 'giese', 'm', 'bulliard', 'j', 'dickhaut', 'r', 'halbach', 'c', 'hassler', 'u', 'hoffmann', 'b', 'hinzen', 'msenn', 'synlen', '1995', '116', '51', 'jc', 'bevington', 'j', 'toole', 'j', 'polymer', 'sci1958,', '28', '413', '52', 'jc', 'bevington', '“radical', 'polymerization”', 'academic', 'press', 'london', '1961', '53', 'ca', 'barson', 'jc', 'bevington', 'de', 'eaves', 'trans', 'faraday', 'soc1958', '54', '1678', '54', 'jm', 'tedder', '“reactivity', 'free-radicals”', 'chapt2', '“reactivity', 'mechanism', 'strucure', 'polymer', 'chemistry”', 'ad', 'jenkins', 'a', 'ledwith', '(eds)', 'wiley-interscience', 'new', 'york', '1974', '55', 'rj', 'orr', 'h', 'l', 'williams', 'j', 'am', 'chem', 'soc1955,', '77', '3715', '56', 'f', 'carrick', 'm', 'szwarc', 'j', 'am', 'chem', 'soc,', '1959', '81', '4138', '57', 'f', 'leavitt', 'm', 'levy', 'm', 'szwarc', 'v', 'stannett', 'j', 'am', 'chem', 'soc1955,', '77', '5493', '58', 'mde', 'forbes', 'h', 'yashiro', 'macromolecules', '2007', '40', '1460', '59', 'a', 'r', 'boder', 'r', 'p', 'buckley', 'f', 'leavitt', 'm', 'szwarc', 'j', 'am', 'chem', 'soc', '1957', '79', '5621', '60', 'r', 'siegmann', 's', 'beuermann', 'macromolecules', '2010', '43', '3699', '61', 'm', 'hagiwara', 'h', 'okamoto', 't', 'kagiya', 't', 'kagiya', 'j', 'polymer', 'sci1970,', 'a-18', '3295', '62', 'g', 'smets', 'k', 'hayashi', 'j', 'polymer', 'sci,', '1958', '27', '626', '63', 'nd', 'field', 'jr', 'schaefgen', 'j', 'polymer', 'sci1962,', '58', '533', '64', 'm', 'yamada', 'i', 'takase', 'kobunshi', 'kagaku', '22', '626', '(1965)', 'chem', 'abstr,', '1966', '64', '19803g', '65', 'wi', 'bengough', 'gb', 'park', 'r', 'young', 'eur', 'polymer', 'j,', '1975', '11', '305', '66', 'bm', 'cubertson', '“maleic', 'fumaric', 'polymers”', '“encyclopedia', 'polymer', 'science', 'technology”', 'vol9', 'hf', 'mark', 'nm', 'bikalis', 'cg', 'overberger', 'gmenges', '(eds)', 'wiley-', 'interscience', 'new', 'york', '1987', '67', 't', 'otsu', 'o', 'ito', 'n', 'toyoda', 's', 'mori', 'macromol', 'chem,', 'rapid', 'commun1981,', '2', '725', '68', 't', 'otsu', 'o', 'ito', 'n', 'toyoda', 'makromol', 'chem,', 'rapid', 'commun,', '1981', '2', '729', '69', 'h', 'kilambi', 'jw', 'stansbury', 'cnbowman', 'macromolecules', '2007', '40', '47', '70', 'c', 'walling', 'e', 'h', 'huyser', 'vol', '13', 'organic', 'reactions', 'ac', 'cope', '(ed)', 'wiley', 'new', 'york', '1963', '71', 'ch', 'bamford', 'wg', 'barb', 'ad', 'jenkins', 'pf', 'onyon', 'kinetics', 'vinyl', 'polymerization', 'radical', 'mecha-', 'nism', 'butterworth', 'london', '1958', '72', 'gs', 'hammond', 'c', 'wu', 'od', 'trapp', 'j', 'warkentin', 'rt', 'keys', 'am', 'chem', 'soc', 'polymer', 'preprints', '1960', '1', '(1)', '168', '73', 'k', 'hayashi', 't', 'yonezawa', 'c', 'nagata', 's', 'okamura', 'f', 'fukii', 'j', 'polymer', 'sci1956,', '20', '537', 'references', '145', '74', 'kf', 'o’driscoll', 't', 'yonezawa', 'reviews', 'macromolecular', 'chemistry1966', '1', '(1)', '1', '75', 'ml', 'huggins', 'j', 'am', 'chem', 'soc1961,', '66', '1991', '76', 'cg', 'fox', 'bs', 'garret', 'we', 'goode', 's', 'gratch', 'jf', 'kinkaid', 'a', 'spell', 'jd', 'stroupe', 'j', 'am', 'chem', 'soc1958,', '80', '1968', '77', 'jwl', 'fordham', 'j', 'polymer', 'sci,', '1959', '39', '321', '78', 'dj', 'cram', 'j', 'chem', 'ed1960,', '37', '317', '79', 'am', 'north', 'd', 'postlethwaite', 'chap', '4', 'structure', 'mechanism', 'vinyl', 'polymerization', 't', 'tsuruta', 'kf', 'o’driscoll', '(eds)', 'dekker', 'new', 'york', '1969', '80', 'che', 'bawn', 'wh', 'janes', 'am', 'north', 'j', 'polymer', 'sci,', '1963', 'c4', '427', '81', 'ph', 'burleigh', 'j', 'am', 'chem', 'soc1960,', '82', '749', '82', 'i', 'rosen', 'ph', 'burleigh', 'jf', 'gillespie', 'j', 'polymer', 'sci,', '1961', '54', '31', '83', 'gm', 'burnett', 'fl', 'ross', 'j', 'polymer', 'sci,', '1967', 'a-15', '84', 'gm', 'burnett', 'fl', 'ross', 'jn', 'hay', 'makromol', 'chem,', '1967', '105', '1', '85', 't', 'uryu', 'h', 'shiroki', 'm', 'okada', 'k', 'hosonuma', 'k', 'matsuzaki', 'j', 'polymer', 'sci,', '1971', 'a-29', '2335', '86', 'g', 'blauerj', 'polymer', 'sci,', '1953', '11', '189', '87', 'fc', 'de', 'schryver', 'g', 'smets', 'van', 'thielen', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1968', '6', '54', '88', 'm', 'kamachi', 'j', 'satoh', 's', 'nozakura', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1978', '16', '(8)', '1789', '89', 't', 'yamamoto', 't', 'yamamoto', 'a', 'moto', 'm', 'hiroto', 'nippon', 'kagaku', 'kaishi', '1979', '(3)', '408', 'chem', 'abstr,', '(june', '1979)', '90', '(12)', 'macromol', 'section', '5', '90', 'b', 'de', 'sterk', 'khamelsoet', 'v', 'van', 'spaybroeck', 'm', 'waroquier', 'macromolecules', '2010', '43', '5602', '91', 'f', 'tudos', 'acta', 'chim', 'budapest', '1965', '43', '397', '44', '403', '92', 'a', 'fehervari', 't', 'foldes-berzsnich', 'ftudos', 'eur', 'polymer', 'j,', '1980', '16', '185', '93', 'w', 'wunderlich', 'makromol', 'chem,', '1976', '177', '973', '94', 'g', 'odian', 'principles', 'polymerization', '3-rd', 'edit', 'j', 'wiley', 'new', 'york', '1991', '95', 'jm', 'dionisio', 'hk', 'mahabadi', 'jh', 'o’driscoll', 'e', 'abuin', 'ea', 'lissi', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1979', '17', '1891', 'jm', 'dionisio', 'jh', 'o’driscoll', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1980', '18', '241', 'r', 'sack', 'gv', 'schulz', 'g', 'meyerhoff', 'macromolecules1988', '21', '3345', '96', 'ga', 'o’neil', 'mb', 'wisnudel', 'jm', 'turkelson', 'macromolecules1996', '29', '7477', '97', 'nf', 'phelan', 'm', 'orchin', 'j', 'chem', 'educ196845,', '633', '98', 'gb', 'butler', 'rj', 'angelo', 'j', 'am', 'chem', 'soc,', '1957', '79', '3128', '99', 'cs', 'marvel', 'rd', 'vest', 'j', 'am', 'chem', 'soc,', '1957', '79', '5771', '100', 'pj', 'graham', 'el', 'buhle', 'nj', 'pappas', 'j', 'org', 'chem1961,', '26', '4658', '101', 'rh', 'wiley', 'wh', 'rivera', 'th', 'crawford', 'nf', 'bray', 'j', 'polymer', 'sci,', '1962', '61', '538', 'polymer', 'sci,', '1964', 'a-12', '102', 'nl', 'zutty', 'j', 'polymer', 'sci,', '1963', 'a-11', '2231', '103', 'j', 'e', 'fearn', 'dw', 'brown', 'la', 'wall', 'j', 'polymer', 'sci,', '1966', 'a-14', '131', '104', 'cs', 'marvel', 'rd', 'vest', 'j', 'am', 'chem', 'soc,', '1959', '81', '984', '105', 'rj', 'cotter', 'm', 'matzner', 'ring-forming', 'polymerizations', 'academic', 'press', 'new', 'york', '1969', '106', 'gb', 'butler', 'ma', 'raymond', 'j', 'polymer', 'sci,', '1965', 'a-13', '3413', '107', 'gb', 'butler', 'j', 'polymer', 'sci,', '1960', '48', '279', '108', 'gb', 'butler', 'am', 'chem', 'soc,', 'polymer', 'preprints', '1967', '8', '(1)', '35', '109', 'gb', 'butler', 'ma', 'raynolds', 'j', 'org', 'chem,', '1965', '30', '2410', 'gb', 'butler', 'tw', 'brooks', 'j', 'org', 'chem,', '1963', '5025', '28', '2699', '110', 's', 'erkoc', 'a', 'ersin', 'acar', 'macromoleculles', '2008', 'asap', 'article101021ma801492a', 'web', 'release', 'nov', '6', '111', 'g', 'oster', 'nl', 'yang', 'chem', 'reviews', '1968', '68', '(2)', '125', '112', 'h', 'tobita', 'macromolecules', '1996', '29', '3073', '113', 'sr', 'chatterjee', 'sn', 'kanna', 'sr', 'palit', 'j', 'indian', 'chem', 'soc,', '1964', '41', '622', '114', 'pd', 'bartlett', 'r', 'altshul', 'j', 'am', 'chem', 'soc,', '1945', '67', '812', '115', 'ch', 'bamford', 'eft', 'white', 'trans', 'faraday', 'soc,', '1956', '52', '716', '116', 'c', 'walling', 'yw', 'chang', 'j', 'am', 'chem', 'soc,', '1954', '76', '4878', '117', 'dh', 'johnson', 'av', 'tobolsky', 'j', 'am', 'chem', 'soc,', '1952', '74', '938', '118', 'g', 'h', 'schulz', 'g', 'henrici-olive', 'z', 'electrochem,', 'ber', 'bunsenges', 'physik', 'chem,', '1956', '60', '296', '119', 's', 'zhu', 'y', 'tian', 'ae', 'hamielec', 'er', 'eaton', 'macromolecules', '1990', '23', '1144', '120', 'da', 'shipp', 'dh', 'solomon', 'ta', 'smith', 'g', 'moad', 'macromolecules', '2003', '36', '2032', '121', 'gt', 'russell', 'dh', 'napper', 'rg', 'gilbert', 'macromolecules', '1988', '21', '2133', '122', 'gt', 'russell', 'rg', 'gilbert', 'nh', 'napper', 'macromolecules', '1992', '25', '2459', '123', 'g', 'odian', '“principles', 'polymerization”', 'wiley', 'new', 'york', 'iv', 'ed,', '2003', '124', 'k', 'yukota', 'amd', 'm', 'itoh', 'jpolymer', 'sci,', 'polymer', 'letters', '1968', '6', '825', '125', 'a', 'ravve', '“light', 'associated', 'reactions', 'synthetic', 'polymers”', 'springer', 'new', 'york', '2006', '126', 'jb', 'brandrup', 'eh', 'immergut', 'eds,', 'polymer', 'handbook', '3', 'rd', 'edit,', 'j', 'wiley', 'new', 'york', '1989', '146', '3', 'free-radical', 'chain-growth', 'polymerization', '127', 'j', 'furukawa', 't', 'tsurutat', 'fueno', 'r', 'sakata', 'k', 'ito', 'macromol', 'chem1959,', '30', '109', '128', 'lj', 'young', 'j', 'polymer', 'sci,', '1961', '54', '411', 'ibid,', '1962', '62', '515', '129', 'kf', 'o’driscoll', 't', 'yonezawa', 'reviews', 'macromol', 'chem,', '1966', '1', '(1)', '1', '130', 'fr', 'mayo', 'c', 'walling', 'chem', 'rev,', '1950', '46', '191', '131', 'b', 'giese', 'g', 'kretzschmar', 'j', 'meixner', 'chem', 'ber,', '1980113', '2787', '132', 'b', 'giese', 'j', 'meixner', 'angew', 'chem,', 'int', 'ed,', '1979', '18', '154', '133', 'b', 'giese', 'k', 'heuck', 'chem', 'ber,', '1979', '112', '3759', '134', 'b', 'giese', 'k', 'heuck', 'tetr', 'lett,', '1980', '21', '1829', '135', 'b', 'giese', 'j', 'meixner', 'angew', 'chem,', 'int', 'ed,', '1977', '16', '178', '136', 'b', 'giese', 'w', 'zwick', 'angew', 'chem,', 'int', 'ed,', '1978', '17', '6130;b', 'giese', 'j', 'meixner', 'chem', 'ber,', '1977', '110', '2588', '137', 'm', 'jacob', 'g', 'smets', 'f', 'de', 'schryver', 'j', 'polymer', 'sci,', '1972', 'a-110', '669', '138', 'gg', 'cameron', 'gf', 'esslemont', 'polymer', '1972', '13', '435', '139', 'g', 'bonta', 'bm', 'gallo', 's', 'russo', 'polymer', '1975', '16', '429', '140', 't', 'alfrey', 'jr', 'cc', 'price', 'j', 'polymer', 'sci,', '1947', '2', '101', '141', 'tc', 'schwann', 'c', 'c', 'price', 'j', 'polymer', 'sci,', '1959', '40', '457', '142', 'gs', 'levinson', 'j', 'polymer', 'sci,', '1962', '60', '43', '143', 'k', 'hayashi', 't', 'yonezawa', 's', 'okamura', 'k', 'fukui', 'j', 'polymer', 'sci,', '1963', 'a-11', '1405', '144', 'gg', 'cameron', 'd', 'a', 'russell', 'j', 'macromol', 'sci,-chem,', '1971', 'a5', '(7)', '1229', '145', 'ad', 'jenkins', 'j', 'macromol', 'sci,-', 'pure', 'applied', 'chem,', '2000', 'a37', '1547', 'awg', 'barb', 'j', 'polymer', 'sci1953,', '11117', 'b', 'klumperman', 'ir', 'kraeger', 'macromolecules', '1994', '271559', 'c', 'coote', 'm', 'l;', 'davis', 't', 'p;', 'klumperman', 'b;', 'monteiro', 'm', 'j', 'rev', 'mocromol', 'chem', 'phys1998', '38', '567-593', '146', 'harwood', 'h', 'j', 'mokromol', 'chem,', 'mokromol', 'symp', '1987', '10111', '331-354', '147', 'k', 'liang', 'm', 'doasit', 'd', 'moscatelti', 'ra', 'hutchinson', 'macromolecules', '2009', '42', '(20)', '7736', 'kliang', 'ra', 'hutchinsone', 'macromolecules', '2010', '43', '6311', '148', 'ad', 'jenkins', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1996', '34', '3495', '149', 'ge', 'ham', 'general', 'aspects', 'free-radical', 'polymerization', 'vinyl', 'polymerization', 'geham', 'ed,', 'dekker', 'new', 'york', '1971', '150', 't', 'alfrey', 'jr,', 'g', 'goldﬁnger', 'j', 'polymer', 'sci,', '1944', '12', '322', '151', 'rc', 'laible', 'chem', 'rev,', '1958', '58', '807', '152', 's', 'yamasaki', 'stasaki', 'h', 'kudoh', 'm', 'matsunaga', 'mutsuo', 'journal', 'polymer', 'science', 'part', 'a', 'polymer', 'chemistry', '2004', '42(7)', '1707', '153', 'tl', 'simandi', 'a', 'rockenbauer', 'ftudos', 'eur', 'polymer', 'j,', '1982', '18', '67', '154', 'bl', 'funt', 'fd', 'williams', 'j', 'polymer', 'sci,', '1960', '46', '139', '155', 'c', 'bevington', 'na', 'ghanem', 'hw', 'melville', 'j', 'chem', 'soc,', '1955', '2822', '156', 'jw', 'breitenbach', 'a', 'springer', 'k', 'hoseichy', 'ber,', '1938', '71', '1438', '157', 'rg', 'caldwell', 'jl', 'ihrig', 'j', 'polymer', 'sci,', '1960', '46', '407', 'j', 'am', 'chem', 'soc,', '1962', '84', '2878', '158', 'gv', 'schulz', 'g', 'henrice', 'makromol', 'chem,', '1956', '18', '437', '159', 'rg', 'caldwell', 'jl', 'ihrig', 'j', 'am', 'chem', 'soc,', '1962', '84', '2878', '160', 'sc', 'foord', 'j', 'chem', 'soc,', '1950', '68', '48', '161', 'et', 'burrows', 'j', 'appl', 'chem,', '1955', '5', '379', '162', 'im', 'kolthoff', 'f', 'a', 'bovey', 'j', 'am', 'chem', 'soc,', '1947', '69', '2143', '163', 'fr', 'mayo', 'j', 'am', 'chem', 'soc,', '1953', '75', '6133', '164', 'gb', 'burnett', 'ld', 'loan', 'trans', 'faraday', 'soc,', '1955', '51', '219', '165', 'jr', 'ebdon', 'br', 'polymer', 'j,', '1971', '3', '9', '166', 'cc', 'price', 'j', 'polymer', 'sci,', '1978', '3', '772', 'ce', 'walling', 'erbriggs', 'kb', 'wolfstern', 'fr', 'mayo', 'j', 'am', 'chem', 'soc,', '1948', '70', '1537', '1544', '167', 'j', 'lingman', 'g', 'meyerhoff', 'macromolecules', '1984', '17', '941', 'makromol', 'chem,', '1984', '587', '168', 'h', 'hirai', 'j', 'polymer', 'sci,', 'macromol', 'rev,', '1976', '11', '47', 'i', 'kunz', 'nf', 'chamberlain', 'fj', 'stelling', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1978', '16', '1747', '169', 'gd', 'russell', 'rg', 'gilbert', 'nh', 'napper', 'macromolecules', '1992', '25', '2459', '()', '170', 'ng', 'gaylord', 'am', 'chem', 'soc', 'polymer', 'preprints', '1969', '10', '(1)', '277', '171', 'ng', 'gaylord', 'a', 'takahashi', 'chapt6', 'addition', 'condensation', 'polymerization', 'processes', 'advances', 'chemistry', 'series', '91', 'am', 'chem', 'soc,', 'washington', '1969', '172', 'y', 'tsuda', 'j', 'sakai', 'y', 'shinohara', 'iupac', 'int', 'symp', 'macromol', 'chem,', 'tokyo-kyoto', '1966', 'preprints', 'iii', '44', 'comprehensive', 'polymer', 'science', 'vol4', 'g', 'eastman', '173', 'jmg', 'cowie', '“alternating', 'copolymerization”', 'a', 'ledwith', 's', 'russo', 'p', 'sigwaldt', '(eds)', 'pergamon', 'press', 'oxford', '1989', '174', 'j', 'furukawa', '“alternating', 'copolymers”', '“encyclopedia', 'polymer', 'science', 'engineering”', 'vol4', '2nded,', 'hf', 'marknm', 'bikales', 'cg', 'overberger', 'g', 'menges', 'eds,', 'wiley-interscience', 'new', 'york', '1986', '175', 'hg', 'heine', 'hj', 'rosenkranz', 'h', 'rudolph', 'angew', 'chem,intern', 'ed,', '1972', '11', '(11)', '974', 'references', '147', '176', 's', 'iwatsuki', 'k', 'nishio', 'y', 'yamashito', 'kagyo', 'kagaku', 'zashi', '1967', '70', '384', '(from', 'ref', '107)', '177', 's', 'iwatsuki', 'y', 'yamashito', 'j', 'polymer', 'sci,', '1967', 'a-15', '1753', '178', 's', 'iwatsuki', 'y', 'yamashito', 'makromol', 'chem,', '1965', '89', '205', '179', 's', 'iwatsuki', 'm', 'shin', 'y', 'yamashita', 'makromol', 'chem,', '102', '232', '(1967)', '180', 'nl', 'zutty', 'cw', 'wilson', 'gh', 'potter', 'dc', 'priest', 'cj', 'whitworth', 'j', 'polymer', 'sci,', '1965', 'a-', '13', '2781', '181', 'jmg', 'cowie', '“alternating', 'copolymerization”', 'comprehensive', 'polymer', 'science', 'vol4', 'g', 'eastman', 'a', 'ledwith', 's', 'russo', 'p', 'sigwaldt', '(eds)', 'pergamon', 'press', 'oxford', '1989', '182', 'j', 'furukawa', '“alternating', 'copolymers”', '“encyclopedia', 'polymer', 'science', 'engineering”', 'vol4', '2nded,', 'hf', 'mark', 'nm', 'bikales', 'cg', 'overberger', 'g', 'menges', 'eds,', 'wiley-interscience', 'new', 'york', '1986', '183', 'i', 'kuntz', 'nf', 'chamberlain', 'fj', 'steling', 'j', 'polymer', 'sci,polchem', 'ed', '1978', '16', '1747', '184', 'h', 'gilbert', 'ff', 'miller', 'sj', 'averill', 'ej', 'carlson', 'vl', 'folt', 'hj', 'heller', 'fj', 'stewart', 'rt', 'schmidt', 'hl', 'trumbull', 'j', 'am', 'chem', 'soc,', '1956', '70', '1669', '185', 'yli', 'sr', 'turner', 'am', 'chem', 'soc', 'polymer', 'preprints', '2009', '50(2)4830', '186', 'lk', 'montgomery', 'k', 'schueller', 'pd', 'bartlett', 'j', 'am', 'chem', 'soc,', '1964', '86', '622', '187', 'm', 'imoto', 'totsu', 'm', 'nakabayashi', 'makromol', 'chemie', '1963', '65', '194', '188', 'mm', 'martin', 'n', 'p', 'jensen', 'j', 'org', 'chem,', '1962', '27', '1201', '189', 'rm', 'thomas', 'wj', 'sparks', 'us', 'patent', '2373067', '(april', '3', '1945)', '190', 'rf', 'mayo', 'c', 'walling', 'chem', 'rev,', '1950', '46101', '191', 'em', 'fetters', 'fo', 'davis', 'chapt', '15', 'polyethers', 'part', 'iii', 'polyalkylene', 'sulﬁdes', 'other', 'polythioethers', 'ng', 'gaylord', '(ed)', 'interscience', 'new', 'york', '1962', '192', 's', 'tazuke', 'adv', 'polymer', 'sci,', '1969', '6', '321', '193', 'y', 'nakayama', 'k', 'hayashi', 's', 'okamura', 'j', 'macromol', 'sci,-chem,', '1968', 'a2', '701', '194', 's', 'iwatsuki', 'y', 'yamashita', 'j', 'polymer', 'sci,', '1967', 'a-15', '1753', '195', 'dr', 'bassett', 'ae', 'hamielec', '(eds)', 'emulsion', 'polymers', 'emulsion', 'polymerization', 'am', 'chem', 'soc', 'symp', '165', 'washingotn', '1981', '196', 'm', 'switala-zeiiazkow', 'eur', 'polymer', 'j,', '1999', '35', '83', '197', 'ch', 'bamford', 's', 'brumby', 'rp', 'wayne', 'nature', '1966', '209', '292', '198', 'm', 'imoto', 't', 'otsu', 'y', 'harada', 'makromol', 'chem,', '1963', '65', '180', '199', 'm', 'imoto', 't', 'otsu', 's', 'shimizu', 'makromol', 'chem', '1963', '65', '114', '200', 'm', 'imoto', 't', 'otsu', 'm', 'nakabayashi', 'makromol', 'chem,', '1963', '65', '194', '()', '201', 'ng', 'gaylord', 'b', 'patnaik', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1971', '8', '411', '202', 'ng', 'gaylord', 'j', 'macromol', 'sci,', 'chem,', '1972', 'a6', '(2)', '259', '203', 's', 'tazuke', 's', 'okamura', 'j', 'polymer', 'sci,', '1967', 'b5', '95', '204', 's', 'tazuke', 'k', 'tsuji', 't', 'yonezawa', 's', 'okamura', 'j', 'phys', 'chem,', '1967', '71', '2957', '205', 'vp', 'zubov', 'mb', 'lachinov', 'vb', 'golubov', 'vp', 'kulikova', 'va', 'kabanov', 'ls', 'polak', 'va', 'kargin', 'iuapc', 'intern', 'symp', 'macromol', 'chem,', 'tokyo-kyoto', 'japan', '1966', '2', '56', '206', 't', 'ikegami', 'h', 'hirai', 'j', 'polymer', 'sci,', '1970', 'a-18', '195', '207', 'reushold', 'macromolecules', '1971', '4', '552', '208', 's', 'yabumoto', 'k', 'ishii', 'k', 'arita', 'j', 'polymer', 'sci,', '1969', 'a-17', '1577', '209', 'va', 'kabanov', 'j', 'polymer', 'sci,', 'polymer', 'symposia', '1980', '67', '17', '210', 'dm', 'white', 'j', 'am', 'chem', 'soc,', '1960', '82', '5678', '211', 'jf', 'brown', 'jr', 'dm', 'white', 'j', 'am', 'chem', 'soc,', '1960', '82', '5671', '212', 'a', 'blumstein', 'r', 'blumstein', 't', 'h', 'vanderspurt', 'j', 'colloid', 'sci,', '1969', '31', '237', '213', 'a', 'blumstein', 'sl', 'malhotra', 'ac', 'wattson', 'j', 'polymer', 'sci,', '1970', 'a-18', '1599', '214', 'a', 'blumstein', 'kk', 'parikh', 'sl', 'malhotra', '23rd', 'intern', 'conf', 'pure', 'appl', 'chem,', 'macromol', 'preprints', '1971', '1', '345', '215', 'm', 'farina', 'u', 'pedretti', 'mt', 'gramegna', 'g', 'audisio', 'macromolecules', '1970', '3', '(5)', '475', '()', '216', 'a', 'colombo', 'g', 'allegra', 'macromolecules', '1971', '4', '(5)', '579', '217', 't', 'uemura', 'yonok', 'kitagawa', 'kitagawa', 'macromolecules', '2007', 'asap', 'article', '101021', '218', 'r', 'buter', 'y', 'y', 'tan', 'g', 'challa', 'j', 'polymer', 'sci,', '1972', 'a-1', '10', '1031', '()', 'ibid,', '1973', 'a-111', '1013', '2975', '219', 'j', 'gons', 'e', 'j', 'vorenkamp', 'g', 'challa', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1975', '13', '1699', '1977', '15', '3031', '220', 'k', 'matyjaszewski', 'macromolecules', '1993', '26', '1787', '221', 'd', 'greszta', 'd', 'mardare', 'k', 'matyjaszewski', 'macromolecules', '1994', '27638', '222', 'sg', 'gaynor', 'd', 'greszta', 't', 'shigemoto', 'k', 'matyjaszewski', 'am', 'chem', 'soc', 'polymer', 'preprints', '1994', '35(2)', '585', '223', 'cj', 'hawker', 'j', 'am', 'chem', 'soc,', '1994', '116', '11185', '224', 'i', 'li', 'ba', 'howell', 'r', 'koster', 'db', 'priddy', 'am', 'chem', 'soc', 'polymer', 'preprints', '1994', '35', '(2)', '517', '225', 'j-l', 'wang', 't', 'grimaud', 'k', 'matyjaszeski', 'macromolecules', '1997', '30', '6507', '226', 't', 'ando', 'm', 'kato', 'mkamigaito', 'm', 'sawamoto', 'macromolecules', '1996', '291070', 'm', 'ouchi', 's', 'tokuoka', 'm', 'sawamoto', 'macromolecules', '2008', 'web', 'article', '101021ma70245d', 'm', 'sawamoto', 'am', 'chem', 'soc', 'polymer', '148', '3', 'free-radical', 'chain-growth', 'polymerization', 'preprints', '2010', '51(1)', '5', 'y', 'fukuzaki', 'y', 'tomita', 't', 'terashima', 'm', 'ouchi', 'm', 'sawamoto', 'maromolecules', '2010', '43', '5989', '227', 'bb', 'wayland', 'g', 'pszmik', 'sl', 'mukerjee', 'm', 'fryd', 'j', 'am', 'chem', 'soc,', '1994', '116', '7943', '228', 'jxia', 'k', 'matyjaszewski', 'am', 'chem', 'soc', 'polymer', 'preprints', '1996', '37', '(2)', '513', '229', 'sg', 'gaynor', 'js', 'wang', 'k', 'matyjaszewski', 'macromolecules', '1995', '8', '8051', '230', 'd', 'greszta', 'k', 'matyjaszewski', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1997', '351857', '231', 'j', 'chiefari', 'yk', 'chong', 'f', 'ercole', 'j', 'krstina', 'j', 'jeffery', 'tpt', 'lee', 'rta', 'mayadunne', 'gf', 'meijs', 'cl', 'moad', 'g', 'moad', 'e', 'rizzardo', 'sh', 'thang', 'macromolecules', '1998', '31', '5559', '232', 't', 'otsu', 'm', 'yoshida', 'makromol', 'chem,', 'rapid', 'comun', '1982', '3', '133', '233', 'dm', 'haddleton', 'cb', 'jasieczek', 'mj', 'hannon', 'aj', 'shooter', 'macromolecules', '1997', '30', '2190', '234', 'kg', 'suddaby', 'dr', 'maloney', 'dm', 'haddleton', 'macromolecules', '30', '702', '(1997)', 'ej', 'kelly', 'dm', 'haddleton', 'e', 'khoshdel', 'am', 'chem', 'soc', 'polymer', 'preprints', '1998', '39', '(2)', '453', '235', 'a', 'goto', 't', 'fukuda', 'macromolecules', '1997', '30', '4272', '5183', '236', 'tc', 'chung', 'hl', 'lu', 'w', 'janvikul', 'am', 'chem', 'soc', 'polymer', 'preprints', '1995', '36', '(1)', '241', '237', 'li', 'yunying', 'lu', 'zhi', 'wayland', 'bradford', 'b,', 'am', 'chem', 'soc,', 'polymer', 'preprints', '2003', '44(2)', '780', '238', 'qiujcharleux', 'b', 'matyjaszewski', 'k', 'prog', 'polymer', 'sci200126,2083', '239', 'aa', 'gridnev', 'sd', 'ittel', 'macromolecules', '1996', '29', '5864', '240', 'k', 'matyjaszewski', 'be', 'woodworth', 'x', 'zhang', 'sg', 'gaynor', 'z', 'metzner', 'macromolecules', '31', '595', '(1998)', '241', 'j', 'xia', 'sg', 'gaynor', 'k', 'matyjaszewski', 'macromolecules', '31', '5958', '(1998)', '242', 'k', 'matyjaszewski', 'macromolecules', '31', '4710', '(1998)', '243', 'd', 'mardare', 'k', 'matyjaszewski', 'macromolecules', '27', '645', '(1994)', '244', 'rpn', 'veregin', 'pg', 'odell', 'lm', 'michalak', 'and', 'mk', 'georges', 'macromolecules', '29', '2746', '(1996)', '245', 'braunecker', 'w', 'a;', 'matyjaszewski', 'k', 'prog', 'polym', 'sd', '2007', '32', '(1)', '93-146', '246', 'min', 'k;', 'gao', 'h;', 'matyjaszewski', 'k', 'i', 'am', 'chem', 'soc', '2005', '127', '3825—3830:', '247', 'jakubowski', 'w;', 'matyjaszewski', 'k', 'macromolecules', '2005', '38', '4139-4146', '248', 'jakubowski', 'w;', 'min', 'k;', 'matyjaszewski', 'k', 'macromolecules', '2006', '39', '39—45;', '249', 'k', 'matyjaszewski', 'tp', 'davis', '‘handbook', 'radical', 'polymerization’', 'wiley', 'new', 'york', '2002', '250', 'min', 'k;', 'jakubowski', 'w;', 'matyjaszewski', 'k', 'macromol', 'rapid', 'commun', '2006', '27', '594—598', '251', 'matyjaszewski', 'k;', 'dong', 'h;', 'jakubowski', 'w;pietrasik', 'j;', 'kusumo', 'a', 'langmuir', '2007', '23', '4528—4531', '252', 'pietrasik', 'j;', 'kusumo', 'a', 'langmuir', '2007', '23', '4528—4531', '253', 'percec', 'v', 'guliashvili', 't', 'ladislaw', 'j', 's;', 'wistrand', 'a', 'stjemdahl', 'a;sienkowska', 'm', 'j;', 'monteiro', 'm', 'j', 'sahoo', 's', 'jam', 'chem', 'soc', '2006128', '(43)', '14156-14165', '254', 'jakubowski', 'w', 'mm', 'k;', 'matyjaszewski', 'k', 'macromolecules', '2006', '3939-45', '255', 'matyjaszewski', 'k', 'macromolecules', '2007', '40', '(6)', '1789-1791', 'k', 'matyjaszewski', 'et', 'al', 'science', '2011', 'doi101126', '256', 'pm', 'wright', 'g', 'mantovani', 'dm', 'haddleton', 'polymer', 'preprints', '2007', '48', '132', '257', 'honigfort', 'mical', 'e;', 'bnttain', 'william', 'j', 'oh', '44325', 'usa)', 'macromolecules', '2003', '36(9)', '3111', 'b', 's', 'faucher', 'p', 'okrutny', 's', 'zhu', 'macromolecules', '2006', '39', '3', '258', 'bb', 'wayland', 'g', 'pszmik', 'sl', 'mukerjee', 'm', 'fryd', 'j', 'am', 'chem', 'soc,', '116', '7943', '(1994)', '259', 'klumperman', 'bert', 'chambard', 'gregory', 'brinkhuis', 'acs', 'symposium', 'series', '2003', '854', '260', 'velazquez', 'o;', 'vivtaldo', 'e;', 'ortega', 'i', 'a;', 'zhus', 'aiche', 'journal', '200248', '2597-2608', '261', 'j', 'chiefari', 'yk', 'chong', 'f', 'ercole', 'j', 'krstina', 'j', 'jeffery', 'tpt', 'lee', 'rta', 'mayadunne', 'gf', 'meijs', 'cl', 'moad', 'g', 'moad', 'e', 'rizzardo', 'sh', 'thang', 'macromolecules', '31', '5559', '(1998)', '262', 'cj', 'hawker', 'j', 'am', 'chem', 'soc,', '116', '11185', '(1994)', '263', 'i', 'li', 'ba', 'howell', 'r', 'koster', 'db', 'priddy', 'am', 'chem', 'soc', 'polymer', 'preprints', '35', '(2)', '517', '(1994)', '264', 'sg', 'gaynor', 'jqiu', 'k', 'matyjaszewski', 'macromolecules', '31', '5951', '(1998)', '265', 'pg', 'grifﬁths', 'rizzardo', 'e,', 'solomon', 'dh,', 'tetrahedron', 'lett,', '23', '1309', '(1982)', '266', 'cj', 'hawker', 'gg', 'barclay', 'rb', 'grubbs', 'jmj', 'frechet', 'am', 'chem', 'soc', 'polymer', 'preprints', '37', '(1)', '513', '(1996)', '267', 'd', 'mardare', 'k', 'matyjaszewski', 'macromolecules', '27', '645', '(1994)', '268', 'mfryd', 'et', 'al', 'us', 'patent', '5', '708', '102', '(jan', '13', '1998)', '269', 'f', 'aldabbagh', 'pbzetterland', 'm', 'okubo', 'macromolecules', '2008', 'asap', 'article', '101021ma702645b', '270', 'pnesvadba', 'a', 'kramer', 'a', 'steinmann', 'w', 'stauffer', 'us', 'patent', '6262206', '(2001)', '271', 'wertmer', 'h;', 'simon', 'd;', 'pfaender', 'r', 'pct', 'mt', 'appl', 'wo', '2005021630', 'al', '2005', '272', 'grubbs', 'r', 'b;', 'wegrzyn', 'j', 'k;', 'xia', 'q', 'chem', 'commun', '2005', '80', '273', 'xia', 'q;', 'grubbs', 'r', 'b', 'j', 'polym', 'sci,', 'part', 'a', 'polym', 'chem', '2006', '44', '5128', '274', 'sv', 'arehart', 'd', 'grezta', 'k', 'matyjaszewski', 'am', 'chem', 'soc', 'polymer', 'preprints', '38', '(1)', '705', '(19777)', '275', 'kwak', 'y;', 'goto', 'a;', 'tsujii', 'y;', 'murata', 'y;', 'komatsu', 'k;', 'fukuda', 't', 'macromolecules', '2002', '35', '3026—3029', 'references', '149', '276', 'f', 'm', 'calitz', 'j', 'b', 'mcleary', 'j', 'm', 'mckenzie', 'm', 'p', 'tonge', 'b', 'iuumperman', 'r', 'd', 'sanderson', 'macromolecules', '2003', '36', '9687', 'f', 'm', 'calitz', 'm', 'p', 'tonge', 'r', 'd', 'sanderson', 'macromolecular', 'symposia', '2003', '193', '277', '(bill)', 'yk', 'chong', 'j', 'kratina', 'tpt', 'lee', 'g', 'moad', 'a', 'postma', 'e', 'rizzardo', 'sh', 'thang', 'macromolecules', '278', 'vana', 'pbilipp', 'davis', 'thomas', 'p;', 'barner—kowollik', 'christopher', 'macromolecular', 'theory', 'simulations', '2002', '279', 'goto', 'a;', 'zushi', 'h;', 'kwak', 'y;', 'fukuda', 't', 'acs', 'symp', 'ser', '2006', '944', '595-603', 'goto', 'a;', 'zushi', 'h;', 'hirai', 'n;', 'walcada', 't;', 'kwak', 'y;', 'fukuda', 't', 'macromol', 'symp', '2007248', '126-131', 'goto', 'a;', 'zushi', 'h;', 'hirai', 'n;', 'wakada', 't;', 'tsujii', 'y;', 'fukuda', 't', 'j', 'am', 'chem', 'soc', '2007', '129', '13347-13354', '280', 'koib', 'h', 'c;', 'finn', 'm', 'g;', 'sharpless', 'k', 'b', 'angew', 'chem,', 'int', 'ed', '200140', '2004', '281', 'lutz', 'j-f', 'angew', 'chem', 'tnt', 'ed', '200746', '1018;', 'binder', 'w', 'h;', 'sachsenhofer', 'r', 'macromol', 'rapid', 'commun', '2003', '35', '2256', '11(8)', '823—835', '2007', '28', '15', '282', 'h', 'gao', 'k', 'matyjaszewski', 'amchem', 'soc', 'polymer', 'preprints', '2007', '48130', '283', 'n', 'renaud', 'k', 'yungwan', 'k', 'matyjaszewski', 'macromolecules', '2008', 'asap', 'article', '101021ma800539v', '284', 'eb', 'milovskaya', 'russ', 'chem', 'rev42(5),', '1979', '(1973)', '285', 'dh', 'solomon', 'e', 'riz', 'zardo', 'p', 'cacioli', 'eur', 'pat', 'appl', '1985', 'ep', '135280', '286', 'tc', 'chung', 'hl', 'lu', 'w', 'janvikul', 'am', 'chem', 'soc', 'polymer', 'preprints', '36', '(1)', '241', '(1995)', '287', 'tcchung', 'polymer', 'news', '200328(8)', '238—244', '288', 'lacroix-d,', 'patrick', 's,', 'romain', 'ob;', 'boutevin', 'b', '(acs', 'polymerpreprints', '2003', '44(2)683', '289', 'a', 'goto', 't', 'fukuda', 'macromolecules', '30', '4272', '5183', '(1997)', '290', 'j-l', 'wang', 't', 'grimaud', 'k', 'matyjaszeski', 'macromolecules', '30', '6507', '(1997)', '291', 'kadavis', 'h-j', 'paik', 'k', 'matyjaszewski', 'macromolecules', '32', '1767', '(1999)', '292', 'w', 'r', 'sorenson', 't', 'w', 'campbell', 'preparative', 'methods', 'polymer', 'chemistry', '2nd', 'edit,', 'wiley-interscience', 'new', 'york', '1968', '293', 'e', 'jenckel', 's', 'suss', 'naturwiss', '29', '339', '(1931)', '294', 'e', 'h', 'riddle', 'monomeric', 'acrylic', 'esters', 'reinhold', 'new', 'york', '1961', '295', 'j', 'coupek', 'm', 'kolinsky', 'd', 'lim', 'j', 'polymer', 'sci,', 'c4', '126', '(1964)', '296', 'rf', 'hoffmann', 's', 'schreiber', 'g', 'rosen', 'ind', 'eng', 'chem', '56', '51', '(1964)', '297', 'aravve', 'jt', 'khamis', 'lx', 'mallavarapu', 'j', 'polymer', 'sci,', 'a-13', '1775', '(1965)', '298', 'm', 'munzer', 'e', 'trammsdorff', '“polymerization', 'suspension”', 'chapt', '5', 'polymerization', 'processes', 'ce', 'schieldknecht', '(with', 'i', 'skeist)', '(ed)', 'wiley-interscience', 'new', 'york', '1977', '299', 'nn', 'semenov', 'j', 'polymer', 'sci,', '55', '563', '(1961)', '300', 'kzhang', 'qfu', 'l', 'jiang', 'cailiao', 'yanjiu', 'xuebao', '2003', '17(1)', '107—112', 'chem', 'abstr', 'synthetic', 'polymers', '2004', '140', '1000', '301', 'farcet', 'cb;', 'pirri', 'r;', 'guerret', 'o,', 'am', 'chem', 'soc', 'polymer', 'preprints', '200243(2)', '98—99', '302', 'jg', 'tsavalas', 'y', 'luo', 'lhudda', 'fj', 'schork', 'polymer', 'reaction', 'engineering', '2003', '11(3)', '277', '303', 'dc', 'blakely', 'emulsion', 'polymerization', 'theory', 'practice', 'applied', 'science', 'london', '1965', '304', 'dr', 'bassett', 'ae', 'hamielec', '(eds)', 'emulsion', 'polymers', 'emulsion', 'polymerization', 'am', 'chem', 'soc', 'symp', '165', 'washingotn', '1981', '305', 'wd', 'harkins', 'j', 'am', 'chem', 'soc,', '69', '1428', '(1947)', '306', 'a', 'ravve', 'organic', 'chemistry', 'macromolecules', 'dekker', 'new', 'york', '1967', '307', 'vanderhoff', 'j', 'w', 'in', 'vinyl', 'polymerization', 'ham', 'g,', 'ed;', 'marcel', 'dekker', 'new', 'york', '1969', 'vol', '7', 'part', '2fitch', 'r', 'm;', 'tsai', 'c', 'h', 'in', 'polymer', 'colloids', 'fitch', 'r', 'm,', 'ed;', 'plenum', 'new', 'york', '1971', 'p', '73', '308', 'hansen', 'f', 'k;', 'ugelstad', 'j', 'in', 'emulsion', 'polymerization', 'piirma', 'i,', 'ed;', 'academic', 'new', 'york', '1982', '309', 'yeliseyeva', 'v', 'i', 'in', 'emulsion', 'polymerization', 'piirma', 'i,', 'ed;academic', 'new', 'york', '1982', '310', 'ottewill', 'r', 'o', 'in', 'emulsion', 'polymerization', 'piirma', 'i,', 'ed;', 'academic', 'new', 'york', '1982', 'p', '1', '311', 'maxwell', 'i', 'a;', 'morrison', 'b', 'r;', 'napper', 'd', 'h;', 'gilbert', 'r', 'g', 'macromolecules', '1991', '24', '1629', 'jpa', 'heuts', 'rg', 'gilbert', 'ia', 'maxwells', 'macromolecules', '30', '726', '(1997)', '312', 'coen', 'e;', 'lyons', 'r', 'a;', 'gilbert', 'r', 'g', 'macromolecules', '1996', '29', '5128', '313', 'kimyberkel,g,', 'russel', 't', 'gilbert', 'rg,', 'macromolecules', '2003', '36', '3921', '314', 'rd', 'athley', 'jr,', 'emulsion', 'polymer', 'technology’', 'dekker', 'new', 'york', '1991', '315', 'wv', 'smith', 'r', 'h', 'ewart', 'j', 'chem', 'phys,', '1948', '16', '592', '316', 'bme', 'van', 'der', 'hoff', 'am', 'chem', 'soc,', 'advances', 'chemistry', 'series', '34', 'washington', '1962', '317', 'pj', 'flory', 'principles', 'polymer', 'chemistry', 'cornell', 'univ', 'press', 'ny', '1953', '318', 'wv', 'smith', 'j', 'am', 'chem', 'soc,', '1948', '70', '3695', '319', 'h', 'j', 'van', 'den', 'hul', 'j', 'w', 'vanderhoff', 'symposium', 'preprints', 'university', 'manchester', 'sept', '1969', '320', 'ss', 'medvedev', 'intern', 'symposium', 'macromolecular', 'chemistry', 'pergamon', 'press', 'new', 'york', '1959', 'ss', 'medvedev', 'ric', 'sci,', 'suppl,', '1955', '25', '897', '150', '3', 'free-radical', 'chain-growth', 'polymerization', '321', 'mr', 'grancio', 'dj', 'williams', 'j', 'polymer', 'sci,', '1970', 'a-18', '2617', 'cs', 'chern', 'gw', 'poehlein', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1987', '25', '617', '322', 'm', 'chainey', 'j', 'hearn', 'mc', 'wilkinson', 'j', 'polym', 'sci,', 'chem', 'ed,', '1987', '25', '505', '323', 'j', 'delgado', 'ms', 'el-aaser', 'ca', 'silebi', 'jw', 'vanderhoff', 'j', 'polym', 'sci,', 'chem', 'ed,', '1989', '27', '193', '324', 'b', 'liu', 'am', 'chem', 'soc', 'polymer', 'preprints', '2008', '49', '(1)', '521', '325', 'de', 'brouwer', 'h;', 'tsavalas', 'j', 'g;', 'schork', 'f', 'j;', 'monteiro', 'm', 'j', 'macromolecules', '2000', '33', '9239—', '9246', 'tsavalas', 'j', 'g;', 'schork', 'f', 'j;', 'de', 'brouwer', 'h;', 'monteiro', 'm', 'j', 'macromolecules', '2001', '34', '3938—3946', 'butte', 'a;', 'storti', 'g;', 'morbidelli', 'm', 'macromolecules', '2001', '34', '5885—5896', 'lansalot', '326', 'm;', 'davis', 't', 'p;', 'heuts', 'j', 'p', 'a', 'macromolecules', '2002', '35', '7582—7591', '327', 'y', 'luo', 'x', 'wang', 'b-gli', 's', 'zhu', 'macromolecules', '2011', '44', '221', '328', 'a', 'sugabe', 'jd', 'florestand', 'cl', 'mccormick', 'macromolecules', '2010', '43', '6599', '329', 'binh', 't', 't', 'pham', 'duc', 'nguyen', 'christopher', 'j', 'ferguson', 'brian', 's', 'hawkett', 'algirdas', 'k', 'serelis', 'christopher', 'h', 'such', 'macromolecules', '2003', '36', '8907', '330', 'sang', 'eun', 'shim', 'yoda', 'shin', 'jong', 'won', 'jun', 'kangseok', 'lee', 'hyejun', 'jung', 'soonja', 'choe', 'macromolecules', '331', 'b', 'ray', 'y', 'isobe', 'k', 'matsumotos', 'hibaue', 'y', 'okamoto', 'm', 'kaigaito', 'm', 'sawamoto', 'mamolecules', '2004', '37', '2003', '36', '7994', '1702', '332', 'j', 'rieger', 'w', 'zhang', 'f', 'stoffelbach', 'b', 'charleux', 'macromolecules', '2010', '43', '6302', 'chapter', '4', 'ionic', 'chain-growth', 'polymerization', '41', 'the', 'chemistry', 'ionic', 'chain-growth', 'polymerization', 'ionic', 'polymerization', 'either', 'cationic', 'anionic', 'this', 'difference', 'stems', 'nature', 'carrier', 'ions', 'growing', 'polymeric', 'chains', 'if', 'process', 'growth', 'chains', 'carry', 'positive', 'centers', 'carbon', 'cations', 'mechanism', 'chain', 'growth', 'designated', 'cationic', 'on', 'hand', 'growing', 'chains', 'carry', 'negative', 'ions', 'carbanions', 'polymerization', 'designated', 'anionic', 'the', 'two', 'different', 'modes', 'polymerization', 'compared', 'examining', 'two', 'types', 'initiation', 'the', 'cationic', 'ones', 'occur', 'electrophilic', 'attacks', 'initiators', 'monomers', 'conversely', 'anionic-ones', 'take', 'place', 'nucleophilic', 'attacks', 'the', 'two', 'reactions', 'shown', 'follows', 'type', 'reaction', 'reagent', 'monomer', 'transition', 'state', 'electrophilic', 'addition', 'nucleophilic', 'addition', 'r', 'r', 'x', 'x', 'r', 'r', 'h', 'x', 'h', 'x', 'generally', 'ionic', 'polymerizations', 'depend', 'upon', 'initial', 'formations', 'positive', 'negative', 'ions', 'organic', 'environment', 'propagations', 'chain', 'reactions', 'take', 'place', 'successive', 'additions', 'monomeric', 'units', 'charged', '“reactive”', 'terminal', 'groups', 'propagating', 'chains', '1.', 'the', 'transition', 'states', 'steps', 'propagation', 'formed', 'repeatedly', 'liquid', 'medium', 'systems', 'containing', 'monomer', 'initiator', 'formed', 'polymer', 'frequently', 'solvent', 'there', 'many', 'different', 'types', 'initiating', 'reactions', 'these', 'polymerization', 'however', 'never', 'terminate', 'combina-', 'tion', 'disproportionation', 'free-radical', 'chain-growth', 'polymerizations', 'instead', 'terminations', 'chain', 'growths', 'results', 'unimolecular', 'reactions', 'transfers', 'molecules', 'like', 'monomers', 'solvents', 'impurities', 'like', 'moisture', 'they', 'also', 'result', 'quenching', 'deliberate', 'additions', 'reactive', 'terminating', 'species', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-94', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '151', '152', '4', 'ionic', 'chain-growth', 'polymerization', '42', 'kinetics', 'ionic', 'chain-growth', 'polymerization', 'the', 'kinetic', 'picture', 'cationic', 'chain', 'polymerization', 'varies', 'considerably', 'much', 'depends', 'upon', 'mode', 'termination', 'particular', 'system', 'a', 'general', 'scheme', 'initiation', 'propagation', 'termination', 'presented', '1', '2.', 'by', 'representing', 'coinitiator', 'a', 'initiator', 'rh', 'monomer', 'm', 'write', 'hmn', '(ar)', 'm', 'hmnm', '(ar)', 'rp', 'kphm', '(ar)', '[m', 'cationic', 'initiation', 'process', 'a', 'rh', 'h', '(ar)', 'k', '(ar)', 'm', 'h', 'cationic', 'propagation', 'process', 'k', 'kp', 'cationic', 'termination', 'by', 'transfer', 'hmnm', '(ar)', 'm', 'ktrm', 'by', 'spontaneous', 'termination', 'hm', '(ar)', 'rate', 'initiation', 'ri', 'k', 'k', 'a[rh[m', 'rate', 'propagation', 'rate', 'termination', 'k', 'hm', 'r', '(ar)', 'mn1', 'h', 'm', '(ar)', 'hmn', 'm', 'mn1', 'h', 'by', 'rearrangement', 'kinetic', 'chain', '(ar)', 'kts', '(ar)', 'hmnm', '(ar)', 'hmnmar', 'in', 'expressions', 'hm——(ar)(cid1)', 'represents', 'total', 'concentration', 'propagating', 'ion-pairs', 'k', 'equilibrium', 'constant', 'formation', 'initiating', 'cations', 'when', 'steady', 'state', 'exists', 'rates', 'initiation', 'termination', 'equal', '½hmþ', 'ðar)(cid1)(cid3)', '¼', 'kki½a[rh[m(cid3)2kt', 'the', 'rate', 'propagation', 'written', 'the', 'number', 'average', 'degree', 'polymerization', 'equal', 'rp', '¼', 'kkikt½a(cid3)½rh(cid3)½m(cid3)2kt', 'dp', '¼', '¼', 'rp', 'rt', 'kp½m(cid3)', 'kt', 'during', 'reaction', 'terminations', 'occur', 'transferring', 'monomer', 'chain', 'transferring', 'agent', 's', 'termination', 'also', 'occur', 'spontaneous', 'termination', 'combination', 'counterion', 'if', 's', 'relatively', 'small', 'number', 'propagating', 'chains', 'remain', 'unchanged', 'the', 'degree', 'polymerization', 'decreased', 'chain-breaking', 'reactions', 'the', 'number', 'average', 'degree', 'polymerization', 'becomes', 'dp', '¼', 'rp', 'rt', 'þ', 'rts', 'þ', 'rtrm', 'þ', 'rtrs', 'the', 'rates', 'two', 'transfer', 'reactions', 'monomer', 'transferring', 'agent', '(s)', 'well', 'spontaneous', 'terminations', 'written', '42', 'kinetics', 'ionic', 'chain-growth', 'polymerization', '153', 'rtrm', '¼', 'ktrm½hmþ', '-', '-', '-', '-', '-', '(arþ(cid1)(cid3)½m(cid3)', 'rtrs', '¼', 'ktrs½hmþ', '-', '-', '-', '-', '-', 'ðarþ(cid1)(cid3)½s(cid3)', 'rts', '¼', 'kts½hmþ', '-', '-', '-', '-', '-', 'ðarþ(cid1)(cid3)', 'the', 'number', 'average', 'degree', 'polymerization', 'dp', '¼', 'kp½m(cid3)ðkt', 'þ', 'ktrm', 'þ', 'ktrs½s(cid3)', 'þ', 'ktsþ', 'the', 'rearranged', 'written', 'follows', '1dp', '¼', 'ktkp½m(cid3)', 'þ', 'ktskp½m(cid3)', 'þ', 'cm', 'þ', 'cs½s(cid3)½m(cid3)', 'cm', 'cs', 'chain', 'transferring', 'constants', 'monomer', 's', 'chain', 'transferring', 'agent', 'here', 'cm', '¼', 'ktrmkp', 'cs', '¼', 'ktrskp', 'when', 'chain', 'transferring', 'agent', 'present', 'chain', 'growth', 'terminated', 'rp', '¼', 'kkikkp½a(cid3)½rh(cid3)½m(cid3)2ðkt', 'þ', 'ktrs½s(cid3)þ', 'the', 'validity', 'steady-state', 'assumption', 'many', 'cationic', 'polymerizations', 'may', 'questioned', 'many', 'reactions', 'occur', 'high', 'rates', 'steady', 'state', 'achieved', 'nevertheless', 'equations', 'shown', 'generally', 'followed', '3', '4.', 'the', 'kinetic', 'picture', 'anionic', 'chain', 'polymerization', 'also', 'depends', 'mostly', 'upon', 'speciﬁc', 'reaction', 'for', 'initiated', 'metal', 'amides', 'liquid', 'ammonia', 'rate', 'initiation', 'shown', 'follows', 'anionic', 'initiation', 'process', 'knh2', 'k', 'nh2', 'k', 'ki', 'nh2', 'm', 'h2n-m', 'anionic', 'propagation', 'process', 'h', 'n-mn', 'm', 'h2', 'n-m', 'n', '-m', 'ri', 'ri', 'rate', 'initiation', 'ki', 'h2', 'n', '[m', 'kki', 'm[knh2', 'k]', 'rate', 'proagation', 'rp', 'kp', 'm', '[m', 'm(cid1)', 'represents', 'total', 'concentration', 'propagating', 'anions', 'if', 'termination', 'reaction', 'takes', 'place', 'transfer', 'solvent', 'rate', 'transfer', 'written', 'when', 'steady', 'state', 'conditions', 'exist', 'if', 'quantities', 'potassium', 'amide', 'ions', 'equal', 'normal', 'situation', 'somewhat', 'different', 'expression', 'written', 'rtr', '¼', 'ktrs½m(cid1)(cid3)½nh3(cid3)', 'rp', '¼', 'kkikp½m(cid3)2½knh2(cid3)', 'ktrs½kþ(cid3)½nh3(cid3)', 'ri', '¼', 'kik12½m(cid3)½knh2(cid3)12', 'rp', '¼', 'kik12kp½m(cid3)2½knh2(cid3)12', 'ktrs½nh3(cid3)', 'rp', '¼', 'kp½m(cid1)(cid3)½m(cid3)', 'in', 'non-terminating', '“living”', 'polymerizations', 'rate', 'propagation', 'written', '154', '4', 'ionic', 'chain-growth', 'polymerization', 'rate', 'living', 'polymerization', '(cid1)', 'd½m(cid3)dt', '¼', 'kp½pn', '(cid4)(cid3)½m(cid3)', 'when', 'effect', 'medium', 'propagation', 'takes', 'place', 'ion', 'pairs', 'free-propagating', 'ions', 'rate', 'propagation', 'written', 'follows', '4', '5:', 'rp', '¼', 'kp', '(cid1)', 'cþ½p(cid1)', 'cþ(cid3)½m', 'kp½p(cid1)(cid3)½m(cid3)', '(cid1)', 'c', 'kp', '(cid1)', 'rate', 'constants', 'propagation', 'ion', 'pairs', 'free-ions', 'kp', 'respectively', 'c', 'positive', 'counterion', 'p(cid1)', '.c', 'p(cid1)', 'concentrations', 'two', 'propagating', 'species', 'm', 'monomer', 'concentration', 'the', 'number', 'average', 'degree', 'polymerization', 'living', 'polymer', 'shown', 'follows', 'ave', 'dp', '¼', '½m(cid3)½m(cid1)(cid3)', 'it', 'simply', 'ratio', 'concentration', 'monomer', 'number', 'living', 'ends', 'a', 'model', 'developed', 'uniﬁed', 'treatment', 'kinetics', 'cationic', 'anionic', 'polymerizations', '6.', 'it', 'based', 'system', 'kinetic', 'models', 'cover', 'various', 'initiation', 'propagation', 'mechanisms', 'pseudosteady-state', 'assumption', 'the', 'treatment', 'beyond', 'scope', 'book', 'the', 'principal', 'kinetics', 'propylene', 'polymerization', 'magnesium', 'chloride', 'supported', 'ziegler–natta', 'catalyst', 'also', 'developed', '7.', 'the', 'polymerization', 'rate', 'described', 'langmuir–-', 'hinshelwood', 'equation', 'showing', 'dependence', 'rate', 'concentration', 'aluminum', 'alkyl', 'a', '¼', 'alr3.', 'because', 'polymerization', 'rate', 'ﬁrst', 'order', 'respect', 'concentration', 'monomer', 'rate', 'equation', 'written', 'follows', '7:', 'rp', '¼', 'kka½a(cid3)ð1', 'þ', 'ka½a(cid3)þ2', 'rp', '¼', 'kp½m(cid3)ka½a(cid3)ð1', 'þ', 'ka½a(cid3)þ2', '43', 'cationic', 'polymerization', 'as', 'mentioned', 'cationic', 'polymerizations', 'reactive', 'portions', 'chain', 'ends', 'carry', 'positive', 'charges', 'process', 'chain', 'growth', 'these', 'active', 'centers', 'either', 'unpaired', 'cations', 'cations', 'paired', 'associated', 'closely', 'anions', '(counterions)', 'the', 'initiations', 'result', 'transpositions', 'electrons', 'either', 'pair', 'single', 'one', 'a', 'two-electron', 'transposition', 'takes', 'place', 'initiating', 'species', 'either', 'protons', 'carbon', 'cations', 'ion', 'generations', 'take', 'place', 'heterolytic', 'bond', 'cleavages', 'dissociations', 'cationic', 'precursors', 'such', 'initiating', 'systems', 'include', 'lewis', 'acidbronsted', 'acid', 'combinations', 'bronsted', 'acids', 'stable', 'cationic', 'salts', 'organometallic', 'compounds', 'cation', 'forming', 'substances', 'when', 'however', 'initiations', 'take', 'place', 'one', 'electron', 'transposition', 'occur', 'direct', 'result', 'oxidation', 'free', 'radicals', 'they', 'also', 'take', 'place', 'electron', 'transfer', 'interactions', 'involving', 'electron', 'donor', 'monomers', 'the', 'carbon', 'cations', 'form', 'oleﬁns', 'variety', 'ways', 'one', 'way', 'direct', 'additions', 'cations', 'like', 'protons', 'positively', 'charged', 'species', 'oleﬁns', 'the', 'products', 'secondary', 'tertiary', 'carbon', 'cations', 'r', 'x', 'r', 'x', 'h', '43', 'cationic', 'polymerization', '155', 'when', 'cations', 'add', 'conjugated', 'dienes', 'charges', 'distributed', 'several', 'centers', 'products', 'r', 'r', 'ch', 'ch', 'r', 'ch', 'the', 'charge', 'may', 'also', 'distributed', 'polar', 'monomers', 'like', 'instance', 'vinyl', 'ethers', 'r', 'or', 'r', 'ch', 'r', 'o', 'chemical', 'considerations', 'indicate', 'diffuse', 'charges', 'stable', 'ions', 'cationic', 'polymerizations', 'affected', 'common', 'inhibitors', 'free-radical', 'polymerizations', 'they', 'however', 'greatly', 'inﬂuenced', 'impurities', 'act', 'ion', 'scavengers', 'these', 'water', 'ammonia', 'amines', 'compounds', 'basic', 'character', 'affecting', 'rates', 'molecular', 'weights', 'products', 'typical', 'cationic', 'polymerizations', 'proceed', 'high', 'rates', 'even', 'low', 'temperatures', 'low', '(cid1)100(cid5)c', '8.', 'in', 'literature', 'one', 'ﬁnd', 'many', 'reports', 'cationic', 'polymerizations', 'many', 'different', 'monomers', 'many', 'different', 'initiators', 'often', 'however', 'initiators', 'quite', 'speciﬁc', 'individual', 'monomers', 'activities', 'strongly', 'inﬂuenced', 'solvents', 'the', 'initiation', 'processes', 'summarized', 'follows', 'i', 'initiation', 'chemical', 'methods', 'a', 'two', 'electron', 'transposition', '(heterolytic)', 'initiation', 'reactions', '1', 'protonic', 'acids', '2', 'lewis', 'acids', '3', 'stable', 'carbon', 'cations', '4', 'certain', 'metal', 'alkyls', '5', 'cation', 'forming', 'substances', 'b', 'one', 'electron', 'transposition', '(homolytic)', 'initiations', 'reactions', '1', 'direct', 'oxidation', 'radicals', '2', 'cation-radicals', 'formed', 'charge-transfers', 'ii', 'initiation', 'physical', 'methods', 'a', 'photochemically', 'generated', 'cations', 'b', 'high-energy', 'irradiation', 'c', 'electro', 'initiation', '431', 'two', 'electron', 'transposition', 'initiation', 'reactions', 'the', 'step', 'cationic', 'initiation', 'subdivided', 'two', 'separate', 'reactions', '9.', 'the', 'ﬁrst', 'one', 'consists', 'formation', 'ionic', 'species', 'second', 'one', 'reactions', 'ionic', 'species', 'oleﬁns', '156', '4', 'ionic', 'chain-growth', 'polymerization', 'cationization', 'process', 'this', 'reaction', 'termed', '“priming”', 'kennedy', 'marechal', '9', 'process', 'ion', 'formation', 'non-nucleophilic', 'media', '(1)', 'dissociation', 'protonic', 'acids', 'form', 'protons', 'counterions', '(2)', 'reactions', 'lewis', 'acids', 'bronsted', 'acids', '(3)', 'dissociation', 'dimeric', 'lewis', 'acids', '(4)', 'complexation', 'lewis', 'acids', 'water', 'alkyl', 'halides', 'ethers', 'these', 'reactions', 'may', 'take', 'place', 'series', 'complicated', 'steps', 'the', 'second', 'reaction', 'cationization', 'oleﬁns', 'may', 'also', 'include', 'several', 'intermediate', 'steps', 'eventually', 'lead', 'propagating', 'species', '4311', 'initiation', 'protonic', 'acids', 'the', 'ability', 'proton', 'protonic', 'acid', 'initiate', 'polymerization', 'depends', 'upon', 'nucleophi-', 'licity', 'conjugate', 'base', 'a(cid1)', '10', '11.', 'if', 'low', 'protonation', 'cationization', 'oleﬁn', 'step', 'initiation', '12:', 'ha', 'h', 'a', 'h', 'a', 'r', 'r', 'r', 'r', 'a', 'in', 'protonic', 'acids', 'complex', 'anion', 'assists', 'ion', 'generation', 'acid', 'may', 'considered', 'providing', 'counterion', 'a(cid1)', 'when', 'counterions', 'highly', 'nucleophilic', 'complexing', 'agents', 'metal', 'salts', 'metal', 'oxides', 'used', 'immobilize', 'anions', 'such', 'complexing', 'agents', 'effect', 'coinitiators', 'thus', 'differences', 'activities', 'protonic', 'acids', 'due', 'quality', 'corresponding', 'anion', 'tendency', 'form', 'chemical', 'bonds', 'carbon', 'cation', 'if', 'anion', 'unable', 'form', 'bonds', 'without', 'extensive', 'regrouping', 'decomposition', 'addition', 'proton', 'followed', 'polymerization', 'should', 'reactivity', 'anion', 'suppressed', 'solvation', 'tendency', 'polymerize', 'enhanced', 'consequently', 'efﬁciencies', 'protonic', 'acids', 'depend', 'much', 'upon', 'polarities', 'media', 'upon', 'reaction', 'conditions', '12', '13.', 'also', 'stronger', 'protonic', 'acid', 'higher', 'reaction', 'rate', 'resultant', 'degree', 'polymerization', '14.', 'generally', 'hydrogen', 'halide', 'acids', 'initiate', 'polymerizations', 'alkyl-substituted', 'oleﬁns', 'they', 'may', 'however', 'initiate', 'polymerizations', 'aryl-substituted', 'oleﬁns', 'vinyl', 'ethers', 'polar', 'solvents', 'the', 'true', 'sulfuric', 'acid', '15.', '4312', 'lewis', 'acids', 'cationic', 'initiations', 'complexation', 'lewis', 'acids', 'water', 'another', 'case', 'formation', 'electrophiles', 'initiate', 'chain', 'growth', 'mexn', 'represents', 'lewis', 'acid', 'mexn', 'h2o', 'h', 'mexn', 'oh', 'h', 'mexn', 'oh', 'r', 'r', 'mexn', 'oh', 'n', 'polymer', 'r', 'r', 'r', 'r', 'r', 'r', 'mexn', 'oh', '43', 'cationic', 'polymerization', '157', 'the', 'shown', 'boron', 'triﬂuoride', 'initiation', 'polymerization', 'isobutylene', 'bf3', 'h2o', 'bf3oh', 'h', 'bf3oh', 'h', 'bf3oh', 'in', 'manner', 'reaction', 'lewis', 'acid', 'alkyl', 'halide', 'subsequent', 'initiation', 'illustrated', 'follows', 'mexn', 'rx', 'mexn1', 'r', 'mexn1', 'r', 'r', 'mexn1', 'in', 'shown', 'examples', 'lewis', 'acid', 'actually', 'co-initiator', 'water', 'alkyl', 'halide', 'initiators', 'numerous', 'studies', 'conﬁrm', 'need', 'many', 'lewis', 'acids', 'molecules', 'like', 'water', 'alkyl', 'halides', 'form', 'initiating', 'ions', 'for', 'instance', 'pure', 'ticl4', 'initiate', 'polymerization', 'isobutylene', '16', '17.', 'this', 'led', 'many', 'believe', 'none', 'lewis', 'acids', 'capable', 'initiating', 'cationic', 'polymerizations', 'oleﬁns', '18.', 'subsequent', 'investigations', 'however', 'demonstrated', '28', '30', 'strong', 'lewis', 'acids', 'capable', 'initiating', 'polymerizations', 'whether', 'lewis', 'acid', 'capable', 'initiating', 'polymerizations', 'tested', 'aid', 'early', 'discovery', 'it', 'known', 'time', 'hindered', 'bases', 'like', 'crowded', 'pyridine', 'derivatives', 'exhibit', 'speciﬁcity', 'toward', 'reactions', 'protons', '19.', 'such', 'bases', 'might', 'used', 'discriminate', 'two', 'types', 'initiating', 'mechanisms', 'encountered', 'lewis', 'acids', '20.', 'the', 'base', 'interfere', 'direct', 'electrophilic', 'additions', 'lewis', 'acids', 'monomer', 'on', 'hand', 'prevent', 'initiation', 'process', 'protons', 'taking', 'place', 'if', 'pathways', 'operative', 'pyridine', 'derivatives', 'quench', 'protonic', 'initiation', 'offer', 'means', 'assessing', 'relative', 'importance', 'process', 'kemedy', 'et', 'al', 'investigated', 'polymerizations', 'initiated', 'lewis', 'acids–water', 'presence', 'one', '“proton', 'trap”', '26-di-t-butylpyridine', '21.', 'the', 'presence', 'base', 'markedly', 'decreased', 'conversion', 'almost', '100,', 'cases', 'percent', 'at', 'time', 'marked', 'increase', 'molecular', 'weight', 'narrowing', 'molecular', 'weight', 'distribution', 'products', 'this', 'shows', 'protonic', 'reaction', 'far', 'important', 'mode', 'initiation', 'it', 'also', 'suggested', 'kemedy', 'et', 'al,', 'protonic', 'initiations', 'occur', 'exclusively', 'free', 'protons', 'they', 'might', 'also', 'proceed', 'concerted', 'protonations', 'these', 'would', 'blocked', '“proton', 'traps”', '21:', 'h', 'h', 'h', 'h', 'o', 'x', 'x', 'me', 'x', 'r', 'o', 'x', 'x', 'me', 'x', 'r', 'x', 'me', 'x', 'r', 'h', 'o', 'x', 'mex3', 'represents', 'lewis', 'acid', 'the', 'di-t-butyl', 'pyridine', 'block', 'reaction', 'due', 'steric', 'compression', 'it', 'also', 'suggested', 'protonic', 'initiations', 'water', 'presence', 'lewis', 'acids', 'probably', 'unlikely', 'sufﬁcient', 'quantities', 'materials', 'act', '“proton', 'traps”', 'present', '158', '4', 'ionic', 'chain-growth', 'polymerization', 'this', 'felt', 'true', 'even', 'polar', 'media', '21.', 'sigwalt', 'coworkers', 'however', 'disputed', '28.', 'they', 'based', 'opinions', 'cationic', 'dimerizations', '11-diphenylethylene', 'alcl3(cid1)h2o', 'presence', '26-di-t-butyl-4-methylpyridine', 'the', 'results', 'suggested', 'sterically', 'hindered', '(cid1)h2o', 'also', 'free', 'hindered', 'amine', 'amines', 'inhibit', 'co', 'catalytic', 'initiation', 'alcl3', 'powerful', 'terminating', 'agent', 'sterically', 'hindered', 'pyridine', 'forms', 'strong', 'complex', 'alcl3', 'when', 'engaged', 'complex', 'alcl3', 'reactivity', 'hindered', 'amine', 'termination', 'much', 'reduced', '28.', 'alkyl', 'halide', 'solvents', 'polymerization', 'reaction', 'mixture', 'may', 'cause', 'initiation', 'result', 'transfer', 'alx2l', '(rx)alx2l', 'basic', 'monomer', '23:', '2ax3', 'þ', '2rx', 'ð', 'alx2(cid6)ðrxþ2', '(cid7)', 'þ', 'a1x4', '(cid8)', 'ðrxþ2a1x2', 'ðmonomerþ', '(cid7)', 'þ', 'm', '(cid1)', 'rx(cid6)x2a1m(cid7)', 'þ', 'rx', 'x', '¼', 'halogen', 'r', '¼', 'alkyl', 'in', 'summarizing', 'initiation', 'mechanism', 'complexed', 'lewis', 'acids', '24,', 'catalysts', 'must', 'present', 'reaction', 'mixture', 'three', 'forms', '(1)', 'ionized', 'molecules', '(2)', 'ion', 'pairs', '(3)', 'free', 'ions', 'all', 'three', 'forms', 'equilibrium', 'when', 'monomer', 'introduced', 'complexes', 'ionized', 'molecules', 'new', 'equilibrium', 'established', 'additional', 'ion', 'pairs', 'form', 'slow', 'process', 'this', 'depends', 'upon', 'change', 'monomer', 'concentration', '25.', 'some', 'strong', 'lewis', 'acids', 'actually', 'capable', 'initiating', 'carbon', 'cationic', 'polymerizations', 'without', 'co-catalyst', 'as', 'result', 'many', 'mechanisms', 'offered', 'explain', 'reaction', 'paths', 'initiations', 'lewis', 'like', 'aluminum', 'halides', 'others', 'without', 'aid', 'co-reactant', 'following', 'prominent', 'ones', 'one', 'mechanism', '26', 'based', 'observation', 'oleﬁns', 'allylic', 'hydrogens', 'like', 'isoprene', 'methyl', 'styrene', 'indene', 'cyclopentadiene', 'polymerized', 'super', 'dry', 'pure', 'lewis', 'acids', 'alone', 'this', 'led', 'suggestion', 'process', 'may', 'involve', 'allylic', 'self-initiation', '26:', 'mnxn', 'mnxnh', 'the', 'suggested', 'mechanism', 'apply', 'polymerizations', 'monomers', 'allylic', 'hydrogens', 'also', 'contradicted', 'ability', '11-diphenylethylene', 'dimerize', 'super', 'dry', 'system', 'presence', 'aluminum', 'halides', 'yet', 'compound', 'lacks', 'allylic', 'hydrogens', '27.', 'two', 'similar', 'mechanisms', '28', 'based', 'concept', 'initiation', 'takes', 'place', 'process', 'halometalation', 'mexn', 'lewis', 'acid', 'mexn-1', 'depending', 'upon', 'substituents', 'oleﬁns', 'metal', 'organic', 'compound', 'may', 'ionize', 'presence', 'second', 'molecule', 'lewis', 'acid', 'may', 'eliminate', 'hx', 'mexn-1', '-hx', 'mexn-1', 'mexn', 'mexn-1', 'mexn1', 'r', 'r', 'r', 'x', 'r', 'r', 'r', 'r', 'r', 'r', 'x', 'r', '43', 'cationic', 'polymerization', '159', 'the', 'mechanism', '29', 'based', 'observation', 'strong', 'lewis', 'acids', 'form', 'stable', 'molecularly', 'bound', 'complexes', 'hindered', 'alkenes', 'instance', 'adamantyladamantane', 'sbf6', 'sbf6', 'the', 'carbon', 'cation', 'forms', 'shown', 'complex', 'actually', 'cannot', 'initiate', 'cationic', 'polymerization', 'excess', 'alkene', 'molecules', 'present', 'reaction', 'mixture', 'due', 'steric', 'hindrance', '29', '31.', 'the', 'less', 'hindered', 'alkenes', 'however', 'like', 'isobutylene', 'polymerize', 'rapidly', 'conditions', 'antimony', 'pentaﬂuoride', 'it', 'therefore', 'suggested', '29', '31', 'initiation', 'involves', 'opening', 'previously', 'formed', 'p', 'complex', 'sbf5', 'double', 'bond', 'halide', 'participation', 'the', 'ab-haloalkylantimony', 'tetraﬂuoride', 'ionizes', 'excess', 'lewis', 'acid', 'form', 'related', 'carbon', 'cation', 'capable', 'initiating', 'cationic', 'chain', 'growth', 'sbf5', 'f', 'sbf4', 'sbf4', 'f', 'sbf4', 'sbf5', 'n', 'sbf6', 'polymer', 'alternately', 'intermediate', 'could', 'shown', 'splitting', 'hf', 'react', 'another', 'molecule', 'lewis', 'acid', 'metalated', 'alkene', 'tertiary', 'halide', 'eliminate', 'hydrogen', 'halide', 'sbf4', 'f', 'sbf4', 'sbf5', 'sbf4', 'hf', 'sbf6', 'the', 'initiation', 'mechanism', 'oleﬁns', 'alb3', 'explained', 'similarly', '29.', 'the', 'intermediate', 'compound', 'aluminum', 'bromide', 'oleﬁn', 'expected', 'lose', 'hbr', 'although', 'initially', 'reaction', 'mixture', 'free', 'protonic', 'acid', 'could', 'form', 'conditions', 'initiation', 'takes', 'place', 'conjugate', 'acid', 'catalyzed', 'system', 'in', 'addition', 'cationic', 'polymerizations', 'oleﬁns', 'considered', 'typical', 'examples', 'general', 'carbocationic', 'reactivity', 'electrophilic', 'reactions', '29.', 'the', 'separate', 'mechanisms', 'looked', 'upon', 'various', 'examples', 'differ', 'nature', 'initial', 'electrophile', 'they', 'always', 'lead', 'related', 'trivalent', 'alkyl', 'cation', 'polymeric', 'chain', 'growth', 'initiated', '29:', 'e', '(electrophile)', 'r', 'r', 'r', 'r', 'n', 'r', 'r', 'e', 'polymer', 'much', 'evidence', 'gathered', 'date', 'however', 'supports', 'concept', 'initiation', 'due', 'auto', 'ionization', 'lewis', 'acids', 'this', 'originally', 'proposed', '1948', '30.', 'lewis', 'acids', 'aggregate', 'generally', 'dimers', 'auto', 'ionize', 'the', 'electrophilic', 'portion', 'adds', 'oleﬁn', 'albr2', 'albr4', '2', 'albr3', 'albr2', 'albr4', 'r', 'r', 'albr2', 'albr4', 'r', 'r', 'afterwards', '1965', 'separate', 'investigation', '32', 'also', 'led', 'conclusion', 'evidence', 'support', 'mechanism', 'the', 'additional', 'data', '33', 'shows', 'isobutylene', 'polymerized', '160', '4', 'ionic', 'chain-growth', 'polymerization', 'super', 'dry', 'pure', 'albr3', 'at', 'time', 'however', 'bf3', 'ticl4', 'require', 'addition', 'water', 'initiate', 'polymerizations', 'monomer', 'furthermore', 'rate', 'albr3', 'initiated', 'polymerization', 'isobu-', 'tylene', 'enhanced', 'considerably', 'addition', 'lewis', 'acids', 'like', 'sncl4', 'sbcl5', 'ticl4', 'vcl4', '34', '35.', 'signiﬁcantly', 'lewis', 'acids', 'incapable', 'initiating', 'polymerizations', 'isobutylene', '16', '17.', 'it', 'believed', 'therefore', 'mixed', 'lewis', 'acids', 'interact', 'generate', 'ions', 'lead', 'rapid', 'polymerizations', 'there', 'many', 'investigations', 'isobutylene', '36', 'polymerization', 'the', 'earlier', 'ones', 'yielded', 'additional', 'supporting', 'evidence', 'in', 'study', 'aluminum', 'chloride', 'bromide', 'catalysts', 'used', 'alkyl', 'halide', 'solutions', 'the', 'reaction', 'monitored', 'conductivity', 'measurements', 'tritium', 'radiotracer', 'techniques', 'conductivity', 'changes', 'polymerization', 'unexchanged', 'tritium', 'polymer', 'support', 'theory', 'initiation', 'take', 'place', 'addition', 'alx2l', 'double', 'bonds', 'monomers', 'this', 'results', 'formation', 'carbon', 'cations', '36.', 'it', 'also', 'observed', '36', '(1)', 'only', 'small', 'fraction', 'aluminum', 'halide', 'ionized', '(2)', '“active', 'species”', 'formed', 'reactions', 'alx2l', 'double', 'bonds', 'covalent', 'bonds', 'formed', 'aluminum', 'carbon', 'atoms', 'the', 'cations', 'form', 'start', 'polymerizations', 'other', 'cations', 'may', 'form', 'impurities', 'may', 'also', 'act', 'initiators', 'the', 'polymerizations', 'follow', 'terminated', 'fairly', 'rapidly', 'concentration', 'initiating', 'cations', 'much', 'smaller', 'amount', 'aluminum', 'halide', 'this', 'explains', 'low', 'efﬁciency', 'particular', 'initiators', 'only', 'little', 'polymer', 'forms', 'per', 'mole', 'aluminum', 'halide', 'used', '(3)', 'most', 'unionized', 'aluminum', 'halide', 'complexes', 'unreacted', 'monomer', 'molecules', 'these', 'complexes', 'act', 'initiators', 'the', 'result', 'concentration', 'aluminum', 'halide', 'formation', 'alx2l', 'becomes', 'small', 'initiation', 'such', 'conclusions', 'explain', 'yields', 'low', 'reactions', 'started', 'additions', 'initiator', 'solutions', 'monomers', 'high', 'however', 'monomers', 'added', 'initiator', 'solutions', '36.', 'faust', 'et', 'al', 'carried', 'detailed', 'kinetic', 'investigation', 'compare', 'polymerization', 'rates', 'isobutylene', 'et2alcl', 'etalcl2', 'tici4', 'they', 'demonstrated', 'etalcl2', 'induced', 'faster', 'polymerizations', 'tici4', 'hexanesch3cl', '(cid1)80(cid5)c', 'polymerization', 'etalcl2', 'also', 'examined', 'dichloromethane', 'chloroform', 'low', 'temperatures', 'the', 'feasibility', 'living', 'cationic', 'polymeriza-', 'tion', 'aluminum-based', 'lewis', 'acids', 'examined', 'hexanesch3cl', 'also', '(cid1)80(cid5)c', 'mealcl2', 'induced', 'rapid', 'less', 'controlled', 'reactions', 'completed', 'within', '30', 'on', 'hand', 'slow', 'well-controlled', 'polymerizations', 'took', 'place', 'me2alcl', 'despite', 'rapid', 'reactions', 'product', 'polymers', 'narrow', 'molecular', 'weight', 'distributions', '36.', 'it', 'clear', 'lewis', 'acids', 'like', 'bcl3', 'require', 'water', 'compounds', 'must', 'added', 'monomer', 'obtain', 'high', 'conversions', '37.', 'boron', 'triﬂuoride', 'initiate', 'polymerizations', 'styrene', 'presumably', 'methylene', 'chloride', 'solvent', '38.', 'one', 'explanation', 'offered', 'different', 'behaviors', 'various', 'lewis', 'acids', 'based', 'differences', 'electronic', 'conﬁgurations', 'around', 'metal', 'atoms', '12.', '4313', 'initiation', 'stable', 'cations', 'these', 'initiations', 'originally', 'reported', 'bawn', 'et', 'al', '39.', 'the', 'cations', 'must', 'used', 'low', 'concentrations', 'insure', 'complete', 'dissociation', 'respective', 'counterions', 'stable', 'organic', 'cations', 'formed', 'oleﬁns', 'aromatic', 'structures', 'compounds', 'heteroatoms', 'possessing', 'unshared', 'electrons', 'like', 'oxygen', 'nitrogen', 'sulfur', 'some', 'examples', 'stable', 'cations', 'initiate', 'cationic', 'polymerizations', '40:', 'c', '3', 't', 'riphenylmethyl', 'ion', 'cyclopentatrienyl', '(tropylium)', 'ion', 'o', 'xanthylium', 'ion', '43', 'cationic', 'polymerization', '161', 'bf4', 'the', 'shown', 'cations', 'form', 'crystalline', 'salts', 'anions', 'these', 'anions', 'clo4', '(cid1)', 'pf6', '(cid1)', 'asf6', 'the', 'initiation', 'reactions', 'stable', 'cations', 'designated', '(x', '(cid1)', 'fecl4', '(cid1)', 'sbf6', '(cid1)', '40.', ')', 'may', 'proceed', 'several', 'possible', '(cid1)', 'sbcl6', '(cid1)', 'mechanisms', '41:', '(a)', 'by', 'direct', 'additions', 'unsaturated', 'systems', 'x', 'r', '(b)', 'by', 'hydride', 'abstractions', 'l', 'r', 'x', 'r', 'r', 'x', 'r', 'xh', '(cid129)', 'x', 'r', 'x', '(c)', 'by', 'formations', 'cation-radicals', 'electron', 'transfer', 'mechanisms', 'in', '(a)', 'reaction', 'predominate', 'left-hand', 'side', 'exceptions', 'appear', 'oleﬁns', 'strong', 'electron', 'releasing', 'substituents', 'confer', 'thermodynamic', 'stability', 'newly', 'formed', 'cation', '40.', 'this', 'either', 'results', 'suitable', 'charge', 'delocalization', 'p-electron', 'system', 'ch2–chr', 'presence', 'heteroatom', 'accordingly', 'oleﬁns', 'possess', 'relatively', 'strong', 'nucleophilic', 'characteristics', 'polymerized', 'stable', 'carbon', 'cations', 'such', 'oleﬁns', 'alkyl', 'vinyl', 'ethers', 'n-vinyl', 'carbazole', 'p-methoxystyrene', 'indene', 'vinylnaphthalenes', 'styrene', 'a-methyl', 'styrene', 'however', 'polymerize', 'less', 'reactive', 'l', 'the', 'initiation', 'mechanism', 'suggested', 'ledwith', '40,', 'follows', 'path', 'reaction', '(a)', 'result', 'direct', 'addition', 'cation', 'oleﬁn', 'this', 'based', 'observations', 'reaction', 'xanthylium', 'cation', '11-diphenylethylene', 'it', 'also', 'based', 'reaction', 'tropilium', 'cation', 'n-vinyl', 'carbazole', 'the', 'last', 'may', 'perhaps', 'inﬂuenced', 'steric', 'factors', 'the', 'high', 'initiating', 'efﬁciency', 'tropilium', 'ion', 'stable', 'six', 'p-electrons', 'system', 'may', 'result', 'formation', 'charge', 'transfer', 'complex', '39:', 'c7h7', 'c7h7', 'c7h7', 'r', 'r', 'r', 'based', 'polymerization', 'n-vinylcarbazole', 'shown', 'follows', '40:', 'a', 'n', 'charge', 'transfer', 'complex', 'a', 'n', 'n', 'n', 'a', 'n', '162', '4', 'ionic', 'chain-growth', 'polymerization', '4314', 'metal', 'alkyls', 'initiations', 'cationic', 'polymerizations', 'initiations', 'polymerizations', 'vinyl', 'monomers', 'metal', 'alkyls', 'generally', 'take', 'place', 'anionic', 'mechanisms', 'this', 'discussed', 'chapter', 'there', 'however', 'reports', 'literature', '41', 'cationic', 'polymerizations', 'initiated', 'metal', 'alkyls', 'these', 'polymerizations', 'monomers', 'like', 'vinyl', 'ethers', 'o-', 'p-methoxystyrene', '42,', 'isobutylene', '43', '44', 'initiated', 'compounds', 'like', 'dialkyl', 'aluminum', 'chloride', 'one', 'explanation', 'follows', '45.', 'these', 'metals', 'strong', 'electron', 'acceptors', 'their', 'valence', 'shells', 'unﬁlled', 'orbitals', 'accommodate', 'electrons', 'donor', 'molecules', 'as', 'result', 'organometallic', 'compounds', 'behave', 'like', 'lewis', 'acids', 'this', 'observed', 'polymerization', 'isobutyl', 'vinyl', 'ether', '46.', 'while', 'triethylaluminum', 'initiator', 'polymerizations', 'reactions', 'take', 'place', 'presence', 'typical', 'electron', 'acceptors', '45:', 'al(c2h3)3', 'clch2och3', '(al(c2h3)3', 'cl', 'ch2och3', ')', '(', 'al(c2h3)3', 'cl', 'ch2och3', ')', 'oc4h9', 'ch3och2', 'al(c2h3)3', 'cl', 'oc4h9', 'h', 'the', 'catalyst', 'components', 'must', 'combined', 'presence', 'monomer', 'this', 'due', '45', 'high', 'instability', 'carbon', 'cation', 'complexes', 'al(c2h5)3cl(cid1)', 'ch3och2+', 'it', 'interesting', '(cid1)h2o', 'also', 'show', 'rate', 'decreases', 'reaction', 'progresses', '47.', 'when', 'polymerizations', 'c2h5alcl2', 'however', 'additional', 'water', 'added', 'rapid', 'polymerizations', 'start', '47.', 'it', 'possible', 'modify', 'catalytic', 'activity', 'metal', 'alkyls', 'controlled', 'additions', 'modiﬁers', 'these', 'water', 'alcohol', '45,', 'oxygen', '48,', 'carbon', 'dioxide', '49,', 'aldehydes', '50,', 'organic', 'peroxides', '45,', 'metal', 'oxides', 'like', 'v2o5', 'nio', 'hgo', '45.', 'the', 'exact', 'action', 'modiﬁers', 'clear', 'certain', 'considered', 'co-initiators', 'initiators', 'their', 'addition', 'however', 'affect', 'catalytic', 'activity', 'yield', 'stereospeciﬁcity', 'molecular', 'weight', 'products', 'oxygen', 'instance', 'act', 'modiﬁer', 'grignard', 'reagent', 'initiate', 'cationic', 'polymerizations', 'vinyl', 'ethers', 'yet', 'introduction', 'oxygen', 'vinyl', 'ether-grignard', 'reagent', 'system', 'initiate', 'polymerization', 'yield', 'high', 'molecular', 'weight', 'products', '50.', 'it', 'suggested', '45', 'oxygen', 'may', 'cause', 'transformation', 'alkyl', 'magnesium', 'groups', 'n', 'alkoxy', 'magnesium', 'groups', 'this', 'results', 'greater', 'concentrations', 'magnesium', 'dihalides', 'induce', 'polymerizations', 'substances', 'generate', 'cations', 'vary', 'widely', 'they', 'molecules', 'dissociate', 'ions', 'react', 'compounds', 'like', 'solvent', 'monomer', 'form', 'cations', 'iodine', 'example', 'substance', 'in', 'system', 'n-butyl', 'vinyl', 'ether-iodine-diethyl', 'ether', 'iodine', 'apparently', 'ﬁrst', 'forms', 'inactive', 'p-complex', 'solvent', 'it', 'subsequently', 'dissociates', 'rearranges', 'isomeric', 'active', 'ion', '12:', 'i2', 'ro', 'ro', 'i2', 'i', 'i2', 'i3', 'or', 'another', 'group', 'compounds', 'recently', 'reported', 'capable', 'initiating', 'cationic', 'polymerizations', 'metalloceneborate', 'complexes', 'such', 'material', 'instance', 'generated', 'zirconocene', 'dimethyl', 'compounds', '(cp2zrme2)', 'anilinium', 'borate', 'thus', 'hnme2ph', 'b', '(c6h5)4', 'polymerize', 'amine-functionalized', 'a-oleﬁns', '51', 'well', 'isobutylene', 'homo', 'copolymerizations', '52.', 'also', 'compounds', 'like', 'cpmme3', '(m', '¼', 'ti', 'zr', 'hf', '43', 'cationic', 'polymerization', '163', 'cp', '¼', 'z5-pentamethylcyclopentadienyl)', 'reacted', 'b(c6f5)3', 'methyl', 'cation', 'abstracted', 'complex', 'forms', 'shown', 'cp(cid4)time3', 'þ', 'b(c6f5þ3', '½cp(cid4)time2', 'þ(cid3)½meb(c6f5þ3', '(cid1)(cid3)', 'indicates', 'evidence', 'presented', 'compound', 'behaves', 'carbocationic', 'polymerization', 'initiator', 'styrene', 'n-vinylcarbazole', 'vinyl', 'ethers', 'isobutylene', 'the', 'mechanism', 'initiation', 'polymerization', 'monomers', 'metallocene', 'complexes', 'still', 'investigated', 'it', 'suggested', 'wang', 'et', 'al', '53,', 'mechanism', 'carbocationic', 'polymerization', 'oleﬁns', 'complex', 'would', 'involve', 'coordination', 'oleﬁns', 'shown', 'nonclassical', 'z1-fashion', 'metal-oleﬁn', 'this', 'interaction', 'stabilized', 'complementary', 'borate–oleﬁn', 'interaction', 'the', 'next', 'step', 'polymerization', 'process', 'mechanism', 'involves', 'attack', 'carbocationic', 'centers', 'metal', 'ions-activated', 'oleﬁn', 'molecules', 'secondary', 'oleﬁn', 'monomers', 'followed', 'chain', 'growth', '53:', 'ti', 'h', 'h', 'h', 'monomer', 'b', 'c6f5', 'c6f5', 'c6f5', 'h', 'h', 'ti', 'h', 'h', 'polymerization', 'initiator', 'monomer', 'coordination', 'chain', 'growth', 'ti', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'b', 'c6f5', 'c6f5', 'c6f5', '432', 'one', 'electron', 'transposition', 'initiation', 'reactions', 'some', 'radical', 'sources', 'presence', 'oxidizing', 'agents', 'light', 'heat', 'energy', 'initiate', 'cationic', 'polymerizations', 'monomers', 'like', 'n-butyl', 'vinyl', 'ether', 'those', 'readily', 'oxidized', 'carbon', 'atom', 'centered', 'radicals', 'substituents', 'like', 'benzyl', 'allyl', 'alkoxy', 'structures', 'nitrogen', 'sulfur', 'also', 'radicals', 'formed', 'addition', 'radicals', 'alkyl', 'vinyl', 'ethers', 'particularly', 'reactive', 'oxidants', 'used', 'reactions', 'salts', 'like', '(c6h5)2ipf6', '(cid1)', 'such', 'salts', 'oxidize', 'radical', 'also', 'supply', 'counter-ions', 'shown', '54:', 'ho', 'oh', 'ho', '2', 'i', 'pf6', '2', '2', '2', '2', '2', '∆', '80', 'oc', 'o', '2', '2h', 'pf6', '2', 'i', '2', '164', '4', 'ionic', 'chain-growth', 'polymerization', 'what', 'happens', 'ﬁnal', 'radical', 'uncertain', 'high', 'conversions', 'reported', 'polymerizations', 'n-butyl', 'vinyl', 'ether', 'presence', 'azobisisobutyronitrile', '(as', 'source', 'radicals)', '(cid8)', '50(cid5)c', '55.', 'the', 'illustration', 'thermal', 'process', '12.', 'free', 'radicals', 'ðp', '-', 'tolylþ2(cid1)i(cid7)pf6', 'also', 'generated', 'aid', 'uv', 'light', 'this', 'instance', 'done', 'uv', 'light', 'decomposition', 'benzoin', 'methyl', 'ether', 'presence', 'oxidizing', 'salt', '56:', 'o', 'och3', 'hν', 'o', 'och3', 'i', '2', 'pf6', 'och3', 'pf6', 'och3', 'i', '4321', 'charge', 'transfer', 'complexes', 'ionic', 'initiations', 'formations', 'copolymers', 'charge', 'transfer', 'mechanisms', 'free-radical', 'polymerizations', 'discussed', 'chap', '3', 'reactions', 'donor', 'acceptor', 'molecules', 'however', 'also', 'result', 'charge', 'transfers', 'yield', 'ion-radicals', 'subsequent', 'polymerization', 'ionic', 'mechanism', 'd', 'a', 'da', 'd', 'a', 'ground', 'state', 'd', 'a', 'da', 'excited', 'state', 'the', 'nonbonding', 'form', 'predominates', 'ground', 'state', 'charge', 'transfer', 'predominates', 'excited', 'state', 'the', 'energy', 'separation', 'ground', 'state', 'ﬁrst', 'excited', 'state', 'small', 'if', 'ionization', 'potential', 'donor', 'molecule', 'low', 'acceptor', 'molecule', 'strong', 'electron', 'afﬁnity', 'transfer', 'electron', 'occur', 'signiﬁcant', 'extent', 'even', 'ground', 'state', 'mutual', 'oxidation–reduction', 'takes', 'place', 'd', 'þ', 'a', 'ð', 'complex', 'ð', '½d(cid6)(cid7)þa(cid6)(cid8)(cid3)', 'some', 'vinyl', 'compounds', 'ie,', 'vinyl', 'ethers', 'function', 'donor', 'molecules', 'possess', 'low', 'ionization', 'potential', 'the', 'acceptors', 'neutral', 'molecules', 'like', 'quinones', 'anhydrides', 'nitrile', 'compounds', 'etc', 'they', 'also', 'ionic', 'intermediates', 'metal', 'ions', 'ionized', 'acids', 'carbon', 'cations', 'an', 'interaction', 'acceptor', 'donor', 'followed', 'subsequent', 'collapse', 'charge', 'transfer', 'complex', 'this', 'results', 'formation', 'cation-radicals', 'capable', 'initiating', 'cationic', 'polymerizations', '57.', 'the', 'exact', 'mechanism', 'reaction', 'cation-radicals', 'oleﬁns', 'still', 'completely', 'determined', 'one', 'example', 'combination', 'alkyl', 'vinyl', 'ether', '(donor)', 'vinylidene', 'cyanide', '(acceptor)', 'results', 'ionic', 'polymerizations', '58.', 'the', 'reaction', 'actually', 'contains', 'ingredients', 'cationic', 'anionic', 'type', 'polymerizations', '59:', '43', 'cationic', 'polymerization', 'ro', 'n', 'n', 'donor', 'acceptor', '165', 'n', 'n', 'ct', 'ro', 'cationic', 'polymerization', 'anionic', 'polymerization', 'ct', 'represents', 'charge', 'transfer', 'complex', 'solvation', 'charged', 'species', 'accelerates', 'transfer', 'electrons', 'ionizations', 'enhanced', 'polar', 'solvents', '57.', 'charge', 'transfer', 'reactions', 'studies', 'tetracyanoethylene', 'acceptor', 'n-vinylcarbazole', 'donor', 'benzene', 'solution', 'demonstrated', 'cation-radicals', 'anion-radicals', 'form', 'this', 'used', 'subsequent', 'cationic', 'polymerizations', '60:', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'monomer', 'polymer', 'a', 'similar', 'reaction', 'takes', 'place', 'chloranil', 'n-vinyl', 'carbazole', '61:', 'o', 'o', 'cl', 'cl', 'n', 'ct', 'cl', 'monomer', 'polymer', 'o', 'cl', 'cl', 'cl', 'o', 'n', 'alkyl', 'vinyl', 'ethers', 'also', 'polymerize', 'presence', 'strong', 'acceptors', 'like', 'tetracyanoben-', 'zoquinone', '23-dichloro-56-dicyano-p-benzoquinone', 'tetracyanoethylene', '60.', 'a', 'similar', 'reac-', 'tion', 'mechanism', 'proposed', '60.', 'solutions', 'maleic', 'anhydride', 'ether', 'initiate', 'cationic', 'polymerizations', 'isobutyl', 'vinyl', 'ether', 'n-vinyl', 'carbazole', 'subjected', 'attacks', 'free', 'radicals', 'the', 'true', 'solutions', 'irradiated', 'ultraviolet', 'light', 'gamma', 'rays', '62', '63.', 'also', 'active', 'species', 'generated', 'reactions', 'aldehydes', 'ketones', 'maleic', 'anhydride', 'attacked', 'free', 'radicals', 'irradiated', 'uv-light', 'gamma', 'rays', '60co', '63.', 'these', 'active', 'species', 'presumed', 'formed', 'charge-transfer', 'reactions', 'occur', 'electron', 'acceptors', 'p-acids', 'electron', 'donors', 'p-', 'bases', 'form', 'cations', '64.', 'n', 'n', 'cl', 'cl', '166', '4', 'ionic', 'chain-growth', 'polymerization', 'some', 'lewis', 'acids', 'form', 'charge', 'transfer', 'complexes', 'monomers', 'yield', 'cation', 'radicals', 'irradiated', 'ultraviolet', 'light', '65', '66.', 'this', 'shown', 'lewis', 'acids', 'vcl4', 'ticl4', 'tibr4', 'polymerizations', 'isobutylene', 'the', 'charge', 'transfer', 'complexes', 'collapse', 'irradiation', '66:', 'vcl4', 'þ', '½m(cid3)', 'monomer', 'ð', '½vcl4m(cid3)', '½vcl4m(cid3)hv½vcl4m(cid3)(cid6)', 'vcl4', '(cid7)(cid1)', '(cid1)', '(cid1)', '(cid1)', '(cid1)(cid1)m(cid6)(cid8)', '2m(cid6)(cid8)(cid7)m--m(cid8)(cid1)', 'polymer', 'monomer', '4322', 'radiation', 'initiated', 'polymerizations', 'photochemical', 'initiations', 'cationic', 'polymerizations', '67–71', 'used', 'commercially', 'irradiation', 'ionizing', 'radiation', 'oleﬁns', 'forms', 'several', 'kinds', 'active', 'initiating', 'species', 'free-radical', 'cationic', 'anionic', 'polymerizations', 'potentially', 'initiated', 'active', 'species', 'generally', 'characteristics', 'ionizing', 'radiation', 'induced', 'polymerizations', 'free-radical', 'ionic', 'polymerizations', 'coexist', '72.', 'under', 'dry', 'conditions', 'polymerizations', 'proceed', 'predominantly', 'cationic', 'intermediates', 'cyclopentadiene', '73', 'styrene', '74', '75.', 'also', 'radiation-induced', 'polymerizations', 'monomers', 'liquid', 'state', 'subject', 'retardation', '76', 'low', 'concentrations', 'ion', 'scavengers', 'ammonia', 'water', '72.', 'the', 'efﬁciency', 'retardation', 'depends', 'upon', 'proton', 'afﬁnity', 'scavengers', '72.', 'in', 'addition', 'radiation-induced', 'cationic', 'polymerizations', 'isobutylene', 'anhydrous', 'conditions', 'occur', 'propagation', 'free', 'ions', '77.', '4323', 'electro', 'initiation', 'polymerization', 'these', 'polymerizations', 'sometime', 'also', 'called', 'electrolytic', 'polymerizations', 'carried', 'electrical', 'ﬁeld', 'the', 'ﬁeld', 'applied', 'initiate', 'chain', 'growth', 'passage', 'electrical', 'current', 'solutions', 'monomers', 'suitable', 'solvents', 'produce', 'initiating', 'species', 'majority', 'species', 'however', 'free-radical', 'nature', '78–80.', 'early', 'studies', 'initiation', 'cationic', 'polymerizations', 'styrene', 'isobutyl', 'vinyl', 'ether', 'n-', 'vinylcarbazole', 'carried', 'dissolving', 'agclo4', 'pure', 'monomers', 'nitrobenzene', 'electric', 'current', 'passed', 'room', 'temperature', '81', '82.', 'rubbery', 'polymers', 'formed', 'well', 'copolymers', 'suggesting', 'cationic', 'path', 'polymerization', 'a', 'mechanism', 'suggested', 'based', 'anodic', 'oxidation', 'monomer', 'clo4', '(cid1)e', '(cid8)(cid1)', 'clo4(cid6)', 'þ', 'e', 'clo4(cid6)', 'þ', 'm', '(cid1)', 'clo4', '(cid8)', 'þ', 'm(cid7)', 'm', 'monomer', 'later', 'however', 'concluded', 'polymerizations', 'due', 'formation', 'cation-radicals', 'initiating', 'species', '83.', 'further', 'studies', 'phenomenon', 'led', 'proposed', 'mechanism', 'initiation', '84', 'styrene', 'polymerization', 'acetonitrile', 'solution', 'controlled', 'potential', 'electrolysis', 'anode', 'aid', 'salt', 'like', 'tetrabutylammonium', 'ﬂuoroborate', 'result', 'direct', 'anodic', 'oxidation', 'electrons', 'transferred', 'anode', 'this', 'accompanied', 'formations', 'radical-', 'cations', '43', 'cationic', 'polymerization', '167', 'in', 'initiation', 'monomer', 'initiator', 'anode', 'regarded', '“co-initiator”', 'if', 'supporting', 'electrolyte', 'ﬁrst', 'oxidized', 'radical', 'indirect', 'cationic', 'polymerization', 'result', 'the', 'radical', 'subsequent', 'step', 'oxidizes', 'monomer', 'ﬁrst', 'initiating', 'entity', 'cation-radical', '85.', 'such', 'indirect', 'initiation', 'also', 'suggested', 'electropolymerization', 'isobutyl', 'vinyl', 'ether', 'presence', 'bf4', 'supporting', 'electrolyte', '85:', 'e', 'bf4', 'bf4', 'anode', 'or', 'bf4', 'bf4', 'or', '433', 'propagation', 'cationic', 'polymerization', 'in', 'propagation', 'step', 'polarity', 'medium', 'affects', 'strongly', 'reaction', 'intimacy', 'ion', 'pair', 'depends', 'upon', 'solvent', 'polarity', 'the', 'bond', 'two', 'ions', 'vary', 'high', 'degree', 'covalence', 'pair', 'free', 'solvated', 'ions', '86:', 'c', 'a', 'c', 'a', 'c', 'a', 'covalent', 'intimate', 'ion', 'pair', 'solvent', 'separated', 'ion', 'pair', 'c', 'a', 'free', 'solvated', 'ion', 'pair', 'the', 'chemical', 'structures', 'monomers', 'also', 'determine', 'reactivity', 'toward', 'cationic', 'polymerizations', 'electron', 'donating', 'groups', 'enhance', 'electron', 'densities', 'double', 'bonds', 'because', 'monomers', 'must', 'act', 'nucleophiles', 'electron', 'donors', 'course', 'propagation', 'increased', 'electron', 'densities', 'double', 'bonds', 'increase', 'reaction', 'rates', 'it', 'follows', 'therefore', 'electron', 'withdrawing', 'substituents', 'oleﬁns', 'hinder', 'cationic', 'polymerizations', 'they', 'instead', 'enhance', 'ability', 'anionic', 'polymerization', 'the', 'polarity', 'substituents', 'however', 'determining', 'factor', 'monomer', 'reactivity', 'steric', 'effects', 'also', 'exert', 'considerable', 'controls', 'rates', 'propagation', 'modes', 'addition', 'active', 'centers', 'investigation', '•o(c2h5)2', 'toluene', 'methylene', 'polymerizations', 'alkyl', 'vinyl', 'ethers', 'instance', 'bf3', 'chloride', '(cid1)79(cid5)c', 'showed', 'rate', 'monomer', 'consumption', 'falls', 'following', 'order', '87:', 'alkyl', 'group', '¼', '-', 'butyli', '-', 'propylethyln', '-', 'butylmethyl', '168', 'table', '41', 'relative', 'rates', 'reactions', 'alkenes', '12-dichloroethane', '24(cid5)c', 'phch2', 'cationa', '(ph)2c', 'l', 'l', 'oleﬁn', '4', 'ionic', 'chain-growth', 'polymerization', 'phch2', '19', '(cid9)', '106', 'l', '19', '(cid9)', '107', '27', '(cid9)', '107', '87', '(cid9)', '107', '(ph)2ch', '105', 'l', '95', '(cid9)', '106', '15', '(cid9)', '107', '105', 'afrom', 'wang', 'dorfmann', '88,', 'permission', 'american', 'chemical', 'society', 'another', 'example', 'study', 'differences', 'rates', 'reactions', 'various', 'alkenes', 'two', 'generated', 'electron', 'pulses', '88.', 'carbon', 'cations', 'free', 'cations', 'phch2l', 'complexities', 'ion', 'pairing', 'cation', 'aggregation', 'encountered', 'typical', 'polymerizations', 'used', 'table', '41', 'shows', 'data', 'reported', '88.', '(ph)2ch', 'l', '89', 'reported', 'investigations', 'recently', 'kolishetti', 'faust', 'polymerizations', 'p-methylstyrene', 'presence', 'isobutylene', 'styrene', 'p-chlorostyrene', '13', 'butadiene', '(cid1)40(cid5)c', 'the', 'polymerizations', 'carried', '5050', 'mixture', 'ch2cl2', 'methyl', 'cyclohexane', 'solvent', 'weak', 'lewis', 'acid', 'snbr2cl2', 'the', 'reactions', 'conducted', 'mono', 'additions', 'monomer', 'followed', 'instantaneous', 'terminations', 'the', 'results', 'showed', 'p-methylstyrene', 'roughly', '38', 'times', 'reactive', 'isobutylene', '48', 'times', 'reactive', 'styrene', '72', 'times', 'reactive', 'p-chlorostyrene', '100', 'times', 'reactive', 'butadiene', 'chain', 'growth', 'reactions', 'fairly', 'tight', 'ion', 'pairs', 'occur', 'medium', 'low', 'polarity', 'require', 'monomers', 'inserted', 'repeatedly', 'two', 'ions', 'these', 'consist', 'carbon', 'cations', 'terminal', 'units', 'paired', 'counterions', 'the', 'ion', 'pairs', 'ﬁrst', 'loosened', '“relaxed”', 'complexations', 'monomers', 'follow', 'insertions', 'complete', 'process', 'all', 'insertions', 'course', 'results', 'formations', 'new', 'carbon', 'cations', 'they', 'upon', 'formation', 'pair', 'immediately', 'counterions', 'process', 'continues', 'x', 'y', 'a', 'x', 'y', 'x', 'y', 'x', 'y', 'a', 'the', 'mechanisms', 'insertions', 'consist', 'repeated', 'push-pull', 'attacks', 'ion', 'pairs', 'double', 'bonds', 'incoming', 'monomers', '90:', 'c', 'a', 'a', 'the', 'degree', 'association', 'ion', 'pairs', 'depends', 'also', 'upon', 'nature', 'counterion', 'temperature', 'reaction', 'medium', 'completely', 'dissociated', 'ion', 'pairs', 'allow', 'chain', 'growth', 'take', 'place', 'free', 'inﬂuence', 'counterions', 'the', 'carbon', 'cations', 'simply', 'add', 'directly', 'double', 'bonds', 'incoming', 'monomers', 'propagation', 'rates', 'reactions', 'greater', 'tight', 'ion', 'pairs', '91.', 'the', 'efﬁciency', 'counterion', 'related', 'acid', 'strength', '43', 'cationic', 'polymerization', '169', 'in', 'isobutylene', 'polymerization', '(cid1)78(cid5)c', 'following', 'lewis', 'acids', 'rated', 'order', 'efﬁciencies', '92:', 'bf3', 'albr3', 'tibr4', 'bbr3', 'sncl4', 'the', 'polymerization', 'reactions', 'complicated', 'sometimes', 'two', 'different', 'types', 'propaga-', 'tion', 'paths', 'some', 'chains', 'may', 'grow', 'without', 'terminal', 'counter-ion', 'free', 'propagating', 'species', 'other', 'polymeric', 'chains', 'however', 'may', 'paired', 'counterions', 'it', 'noted', 'references', 'made', 'free', 'propagating', 'ions', 'ions', 'free', 'electrostatic', 'inﬂuences', 'anions', 'they', 'however', 'still', 'associated', 'interact', 'polar', 'polarizable', 'solvent', 'molecules', 'monomers', '4331', 'steric', 'control', 'cationic', 'polymerization', 'ionic', 'polymerizations', 'yield', 'highly', 'stereoregular', 'polymer', 'control', 'exercised', 'monomer', 'placement', 'much', 'steric', 'control', 'cationic', 'polymerization', 'occurs', 'low', 'temperatures', 'the', 'earliest', 'stereospeciﬁc', 'vinyl', 'polymerizations', 'observed', 'preparation', 'poly(isobutyl', 'vinyl', 'ether)', 'bf3-ether', 'complex', 'catalyst', '(cid1)70(cid5)c', 'an', 'isotactic', 'polymer', 'formed', 'reaction', '93.', 'the', 'catalyst', 'employed', 'later', 'yield', 'stereospeciﬁc', 'poly(vinyl', 'ether)s', '93–96.', 'the', 'amount', 'steric', 'placement', 'increases', 'decrease', 'reaction', 'temperature', 'conversely', 'decreases', 'increase', 'temperature', '96', '97.', 'various', 'mechanisms', 'proposed', 'explain', 'steric', 'placement', 'cationic', 'polymerization', 'most', 'pertain', 'vinyl', 'ethers', 'there', 'general', 'agreement', 'some', 'suggested', 'mechanisms', 'discussed', 'section', 'most', 'offered', 'homogeneous', 'conditions', 'soluble', 'initiators', 'like', 'bf3–o(c2h5)2', 'there', 'however', 'also', 'explanations', 'steric', 'control', 'insoluble', 'catalysts', 'like', 'al(so4)2', '•', 'h2so4', 'control', 'homogeneous', 'polymerizations', 'not', 'explanations', 'steric', 'control', 'homogeneous', 'conditions', 'give', 'equal', 'weight', 'inﬂuence', 'counterion', 'a', 'bawn', 'ledwith', 'mechanism', '99', 'polymerization', 'vinyl', 'ethers', 'based', 'data', 'suggests', 'one', 'mesomeric', 'form', 'ethers', 'exists', 'presumably', 'trans', '98', 'ch2', 'o', 'r', 'or', 'ch2', 'r', 'o', 'an', 'alkyl', 'substituent', 'composed', 'three-carbon', 'chain', 'causes', 'steric', 'blocking', 'one', 'side', 'bond', 'h', 'h', 'o', 'a', 'ﬁve-carbon', 'substituent', 'exhibit', 'highest', 'degree', 'steric', 'hindrance', 'hand', 'decrease', 'decrease', 'size', 'group', 'there', 'blocking', 'ethyl', 'group', 'isopropyl', 'one', '98.', 'this', 'demonstrated', 'experimentally', '99.', 'it', 'suggested', 'therefore', 'homogeneous', 'polymerizations', 'vinyl', 'ethers', 'growing', 'cations', 'stabilized', '170', '4', 'ionic', 'chain-growth', 'polymerization', 'form', 'neighboring', 'group', 'interaction', 'this', 'interaction', '(or', 'intramolecular', 'solvation)', 'would', 'oxygen', 'atoms', 'penultimate', 'monomer', 'units', '98.', 'these', 'forms', '“backside”', 'stabilization', 'growing', 'chains', 'force', 'reactions', 'occur', 'opposite', 'sides', 'locations', 'counterions', 'the', 'mechanisms', 'forms', 'sn2', 'attacks', 'retention', 'conﬁgurations', 'these', 'conﬁguration', 'formed', 'existing', 'newly', 'formed', 'carbon', 'cations', 'transition', 'states', 'r', 'a', '76', '5', '4', 'o', '3', 'ro', 'or', 'ro', '1', 'h', 'h', '2', 'h', 'ro', '76', '5', '4', 'o', '3', 'r', 'a', 'or', '1', 'or', 'ch2', '2', 'h', 'ro', '7', '6', 'r', '54', '3', '2', 'o', '1', 'a', 'ro', 'or', 'solvations', 'new', 'cations', 'might', 'even', 'occur', 'completely', 'formed', 'maintaining', 'steric', 'arrangement', 'throughout', 'provided', 'monomers', 'enter', 'shown', '98.', 'one', 'weakness', 'mechanism', 'fails', 'consider', 'nature', 'counterions', 'another', 'mechanism', 'proposed', 'cram', 'kopecky', '100,', 'places', 'emphasis', 'formation', 'six-', 'membered', 'rings', 'the', 'growing', 'polymeric', 'chains', 'vinyl', 'ethers', 'occupy', 'equatorial', 'arrangements', '–or', 'groups', 'attached', 'growing', 'ends', 'virtue', 'size', 'larger', 'in', 'reactions', 'monomers', 'six-membered', 'ring', 'oxonium', 'ions', 'relative', 'conﬁgurations', 'two', 'asymmetric', 'centers', 'form', 'determine', 'total', 'chain', 'conﬁgurations', 'if', 'conﬁgurations', 'similar', 'chains', 'become', 'isotactic', 'different', 'become', 'syndiotactic', 'molecular', 'models', 'suggest', 'isotactic', 'placement', 'likely', '100.', 'ro', 'ro', 'h', 'ro', 'h', 'h', 'o', 'r', 'or', 'or', 'or', 'o', 'r', 'or', 'ro', 'ro', 'ro', 'ro', 'h', 'h', 'h', 'h', 'the', 'cram', 'kopecky', 'mechanism', '43', 'fails', 'explain', 'inﬂuence', 'various', 'r', 'groups', 'upon', 'stereospeciﬁcity', 'ﬁnal', 'product', 'in', 'mechanism', 'proposed', 'kunitake', 'aso', '101', 'two', 'factors', 'given', 'primary', 'importance', 'these', '(1)', 'steric', 'repulsions', 'determine', 'conformations', 'propagating', 'chains', 'special', 'arrangement', 'counterions', 'incoming', 'monomers', '(2)', 'the', 'directions', 'monomer', 'attacks', 'determined', 'tightness', 'growing', 'ion', 'pairs', 'it', 'assumed', 'growing', 'carbon', 'cations', 'essentially', 'sp2', 'hybridized', 'conformations', 'least', 'steric', 'repulsion', 'therefore', '101', 'shown', 'fig', '41', 'the', 'position', 'counterion', 'assumed', 'side', 'carbon', 'cation', 'away', 'penultimate', 'unit', 'the', 'stability', 'conformations', 'dependent', 'temperature', 'polymerization', 'size', 'substituents', 'experimental', 'evidence', 'conﬁrms', 'thus', 'low', 'known', 'stereoregular', 'polymers', 'whether', 'isotactic', 'syndiotactic', 'form', '43', 'cationic', 'polymerization', '171', 'a', 'l', 'l', 's', 's', 'fig', '41', 'steric', 'arrangement', 'l', 'large', 'substituent', 's', 'small', 'substituent', 'l', 's', '(l)', 's', 'front-side', 'attack', 'l(s)', 'a', 'back-side', 'attack', 'front-side', 'l', 's', 's', 'l', 'l', 'l', 's', 'l', 'l', 's', 'l', 's', 'a', 's', 'l', 'l', 's', 'a', 's', 's', 'a', 'fig', '42', 'propagation', 'mechanism', 'temperatures', 'homogeneous', 'polymerizations', '(as', 'stated', 'earlier)', 'this', 'suggests', 'ﬁxation', 'conformations', 'growing', 'chain', 'ends', 'important', 'enhancing', 'polymer', 'stereoregularity', 'in', 'polar', 'solvents', 'counterions', 'interact', 'weakly', 'growing', 'cations', 'the', 'steric', 'effects', 'become', 'major', 'factors', 'deciding', 'courses', 'propagation', 'in', 'situations', 'carbon', 'cations', '101', 'attack', 'least', 'hindered', 'side', '(frontal', 'side', 'attacks)', 'these', 'give', 'rise', 'syndiotactic', 'structures', 'the', 'terminal', 'carbon', 'cations', 'probably', 'rotate', 'freely', 'vinyl', 'monomers', 'capable', 'approaching', 'direction', 'in', 'non-polar', 'solvents', 'hand', 'ion', 'pairs', 'tight', 'enough', 'incoming', 'monomers', 'may', 'approach', 'cations', 'back', 'sides', 'giving', 'rise', 'isotactic', 'placements', 'this', 'illustrated', 'fig', '42', 'if', 'steric', 'hindrance', 'backside', 'approaches', 'due', 'large', 'sizes', 'penultimate', 'substituents', 'front-side', 'attacks', 'take', 'place', 'this', 'occurs', 'even', 'non-polar', 'medium', '101.', 'the', 'incom-', 'ing', 'monomers', 'therefore', 'attack', 'cation', 'either', 'frontside', 'backside', 'all', 'depends', 'upon', 'tightness', 'coulombic', 'interaction', 'ion', 'pair', 'difference', 'steric', 'hindrance', 'two', 'modes', 'attack', 'in', 'reaction', 'mechanism', 'possible', 'interactions', 'counterions', 'monomers', 'ignored', 'this', 'justiﬁed', 'weak', 'interactions', 'electron-rich', 'monomers', 'like', 'a-methyl', 'styrene', 'vinyl', 'ethers', 'weak', 'anions', '101.', 'the', 'nature', 'counterions', 'however', 'ignored', 'mechanism', 'tightness', 'ion', 'pairs', 'considered', 'later', 'work', 'hirokawa', 'et', 'al', '102,', 'suggested', 'sizes', 'r', 'groups', 'alkenyl', 'ethers', 'play', 'important', 'role', 'determining', 'steric', 'structures', 'resultant', 'polymers', 'for', 'instance', 'allyl', 'vinyl', 'ethers', 'polymerized', 'highly', 'isotactic', 'polymers', 'aid', 'sbcl5', '1h', '13c', 'nmr', 'data', 'shows', 'evidence', 'however', 'steric', 'control', 'though', 'show', 'relationship', 'active', 'chain', 'ends', 'incoming', 'monomers', 'in', 'addition', 'amounts', 'isotactic', 'placement', 'differ', 'signiﬁcantly', '(cid1)10(cid5)c', '(cid1)75(cid5)c', '103.', 'this', 'suggests', 'isotactic', 'selection', 'generated', 'orienting', 'substituents', 'monomer', 'chain', 'away', 'a', 'coulombic', 'attraction', 'visualized', 'counterion', 'positively', 'polarized', 'oxygen', 'monomer', 'also', 'studies', 'optically', 'active', 'vinyl', 'ethers', 'observed', '104', 'trimethyl', 'vinyl', 'silane', 'bulky', 'non-chiral', 'forms', 'highly', 'syndiotactic', 'polymers', 'equally', 'bulky', 'chiral', '((cid1))-', 'menthyl', 'vinyl', 'ether', 'however', 'produces', 'isotactic', 'polymers', 'polar', 'solvents', 'this', 'suggests', 'isotactic', 'propagation', 'preferred', 'polar', 'medium', 'helical', 'conformation', 'polymer', '172', '4', 'ionic', 'chain-growth', 'polymerization', 'chain', 'forced', 'bulky', 'chiral', 'substituent', 'kunitake', 'takarabe', '105,', 'therefore', 'modiﬁed', 'original', 'kunitake', 'aso', 'mechanism', 'the', 'growing', 'chain', 'ends', 'crowded', 'bulky', 'substituents', 'this', 'may', 'result', 'steric', 'interference', 'bulky', 'side', 'group', 'counterions', 'the', 'interactions', 'propagating', 'ion', 'pairs', 'decrease', 'sizes', 'counterions', 'increase', 'a', 'frontal', 'attack', 'syndiotactic', 'placement', 'monomers', 'results', 'when', 'however', 'monomer', 'side', 'groups', 'less', 'bulky', 'steric', 'repulsion', 'becomes', 'insigniﬁcant', 'larger', 'counterion', 'becomes', 'responsible', 'retarding', 'frontside', 'attack', 'gives', 'isotactic', 'placement', 'studies', 'model', 'reactions', 'cationic', 'polymerization', 'alkyl', 'propenyl', 'ethers', 'showed', 'mode', 'double', 'bond', 'opening', 'independent', 'geometric', 'structure', 'ether', 'mainly', 'threo', 'opening', 'takes', 'place', 'mode', 'monomer', 'addition', 'dependent', 'geometric', 'structure', 'monomer', 'bulkiness', 'substituent', '106.', 'finally', 'still', 'another', 'investigation', 'model', 'systems', 'uv', 'visible', 'spectroscopy', 'used', 'together', 'conductivity', 'measurements', 'results', 'showed', 'charge-transfer', 'complexes', 'form', 'counterions', 'p-acceptors', 'lewis', 'acids', 'acceptor', 'solvents', '196.', 'this', 'led', 'heublein', 'suggest', 'interactions', 'monomers', 'lead', 'alterations', 'solvation', 'spheres', 'ionpairs', 'direction', 'counterions', 'the', 'temporary', 'dissymmetry', 'sphere', 'solvation', 'affects', 'stereoregularities', 'structures', 'polymers', 'form', 'as', 'result', 'propagation', 'reactions', 'seen', 'hueblein', 'competing', 'interaction', 'chain', 'carriers', 'monomers', 'counterions', 'solvents', '106.', 'control', 'heterogeneous', 'polymerizations', 'several', 'reaction', 'mechanisms', 'also', 'proposed', 'explain', 'stereospeciﬁc', 'placement', 'insoluble', 'catalysts', 'furukawa', '46', 'suggested', 'mechanism', 'cationic', 'polymerization', 'vinyl', 'ethers', 'depends', 'upon', 'multicentered', 'coordinations', 'he', 'felt', 'coordinations', 'polymeric', 'chains', 'monomers', 'catalysts', 'possible', 'complexed', 'counteranions', 'electrically', 'positive', 'centers', 'this', 'take', 'place', 'case', 'aluminum', 'alkyl', 'boron', 'ﬂuoride', 'further', 'coordination', 'aluminum', 'alkyl', 'anions', 'possible', 'coordination', 'number', 'central', 'atom', 'sufﬁciently', 'large', '46:', 'r3al', 'þ', 'bf3', '½r2al(cid3)(cid7)', 'þ', '½bf3r(cid3)', '(cid8)', '½bf3r(cid3)', 'þ', 'ar3', '½r3al', 'bf3r(cid3)', 'the', 'products', 'complex', 'counterions', 'enable', 'multicentered', 'coordination', 'polymerization', 'thus', 'mechanism', 'vinyl', 'ether', 'polymerization', 'proposed', 'furukawa', '46', 'follows', 'two', 'neighboring', 'ether', 'oxygens', 'linked', 'polymer', 'chain', 'close', 'terminal', 'cation', 'become', 'coordinated', 'metal', 'center', 'complexed', 'counterion', 'the', 'molecules', 'monomer', 'approach', 'growing', 'chain', 'opposite', 'side', 'isotactic', 'placement', 'results', '(cid8)', '(cid8)', 'or', 'h', 'r', 'o', 'o', 'h', 'r', 'b', 'z', 'me', '43', 'cationic', 'polymerization', '173', 'b', 'represents', 'boron', 'me', 'aluminum', 'a', 'different', 'mechanism', 'however', 'offered', 'nakano', 'coworkers', '47.', 'they', 'felt', 'must', 'relationship', 'crystal', 'structures', 'heterogeneous', 'catalysts', 'resultant', 'stereoregularity', 'polymers', 'if', 'crystal', 'structures', 'catalysts', 'tetrahedral', 'crystals', 'active', 'edges', 'stereoregular', 'polymers', 'form', 'even', 'room', 'temperature', 'in', 'addition', 'shorter', 'active', 'edges', 'make', 'catalysts', 'suitable', 'stereospeciﬁc', 'polymerization', 'the', 'follow-', 'ing', 'mechanism', 'therefore', 'proposed', '47.', 'if', 'terminal', 'end', 'growing', 'chain', 'ends', 'o', 'r', 'sp2', 'type', 'conﬁgurations', 'vacant', 'orbitals', 'terminal', 'carbon', 'atoms', 'growing', 'polymeric', 'chains', 'state', 'resonance', 'lone', 'pair', 'electrons', 'adjacent', 'oxygen', 'atoms', 'this', 'means', 'positive', 'charges', 'distributed', 'adjacent', 'oxygens', 'localized', 'carbons', 'δ', 'δ', 'o', 'r', 'counterion', 'the', 'monomer', 'potentially', 'add', 'four', 'different', 'ways', '1', 'δ', 'o', 'δ', 'r', '2', 'or', 'δ', 'o', 'δ', 'or', 'r', '3', '4', 'δ', 'o', 'δ', 'r', 'ro', 'δ', 'o', 'δ', 'r', 'or', 'reaction', '3', 'yields', 'isotactic', 'polymers', 'mode', 'addition', '47', 'isotactic', 'polymer', 'forms', '4332', 'pseudo-cationic', 'polymerization', 'most', 'cationic', 'polymerizations', 'oleﬁns', 'proceed', 'carbon', 'cation', 'carriers', 'there', 'however', 'instances', 'cationic', 'polymerizations', 'evidence', 'interpreted', 'suggesting', 'propagating', 'species', 'carbon', 'cations', 'instead', 'reactions', 'said', 'proceed', 'covalently', 'propagating', 'species', 'such', 'reactions', 'termed', 'pseudo', 'cationic', '109', '110.', 'in', 'polymerizations', 'propagating', 'species', 'may', 'combinations', 'ionic', '(free', 'ions', 'ion', 'pairs)', 'covalently', 'bonded', 'species', 'reaction', 'conditions', 'claimed', 'determine', 'relative', 'amounts', 'examples', 'polymerizations', 'cationic', 'polymerizations', 'styrene', 'acenaphthene', 'protonic', 'acids', 'like', 'hclo4', 'iodine', 'initiators', 'the', 'propagations', 'suggested', '110', 'take', 'place', 'three', 'successive', 'stages', 'reactions', 'carried', 'methylene', 'chloride', '(cid1)20(cid5)c', 'in', 'ﬁrst', 'one', 'rapid', 'short-lived', 'ionic', 'reactions', 'take', 'place', 'in', 'second', 'stage', 'ions', 'longer', 'detected', 'spectroscopy', 'conductivity', 'measurements', 'in', 'third', 'one', 'rapid', 'increase', 'presence', 'ionic', 'species', 'shown', '(detected', 'conductivity', 'measurements', 'spectroscopy)', 'at', 'temperatures', '(cid1)20', '30(cid5)c', 'effectively', 'stage', 'one', 'stage', 'three', 'shorter', 'on', 'hand', 'temperatures', 'low', '(cid1)80(cid5)c', 'stage', 'one', 'in', 'stages', 'one', 'three', 'propagation', 'takes', 'places', 'combinations', 'free', 'ions', 'ion', 'pairs', 'these', 'combinations', 'ions', '174', '4', 'ionic', 'chain-growth', 'polymerization', 'claimed', 'result', 'formations', 'covalent', 'perchlorate', 'esters', 'solvated', 'stabilized', 'monomer', 'the', 'propagation', 'stage', 'two', 'therefore', 'pseudo', 'cationic', 'covalent', 'consists', 'monomer', 'insertion', 'c–o', 'bond', 'perchlorate', 'ester', '109', '110:', 'o', 'o2', 'cl', 'o', 'clo2', 'o', 'when', 'insufﬁcient', 'amounts', 'monomer', 'present', 'stabilize', 'covalent', 'esters', 'high', 'conversions', 'ionizations', 'take', 'place', 'this', 'leads', 'rapid', 'ionic', 'stage', 'three', 'polymerizations', 'the', 'propagation', 'free', 'ions', 'ion', 'pairs', 'stage', 'three', '104', '105', 'lmole', 'low', 'temperatures', '(cid1)60', '(cid1)80(cid5)c', 'by', 'comparison', 'polymerizations', 'stage', 'two', 'much', 'slower', 'somewhere', '01', '20', 'lmole', 'the', 'rates', 'depend', 'course', 'upon', 'solvent', 'upon', 'temperature', 'while', 'ion–ion', 'pair', 'type', 'propagations', 'yield', 'polymers', 'molecular', 'weights', 'equal', '104', 'covalent', 'propagations', 'yield', 'oligomers', 'differences', 'molecular', 'weights', 'molecular', 'weight', 'distributions', 'products', 'three', 'stages', 'polymerization', 'detected', 'size', 'exclusion', 'chromatography', 'the', 'chromatogram', 'shows', 'bimodal', 'distributions', 'this', 'taken', 'supporting', 'evidence', 'two', 'modes', 'propagation', 'ionic', 'covalent', '110.', 'similar', 'results', 'reported', 'polymerizations', 'styrene', 'ch3coclo4', 'cf3so3h', 'cf3cooh', 'clso3h', 'fso3h', '109.', 'pseudo-cationic', 'mechanism', 'also', 'claimed', 'polymerizations', 'styrene', 'derivatives', 'like', 'p-methyl', 'styrene', 'p-methoxystyrene', 'p-chlorostyrene', 'protonic', 'acids', '109.', 'although', 'concept', 'pseudo-cationic', 'mechanism', 'may', 'accepted', 'accepted', 'many', 'an', 'alternative', 'mechanisms', 'based', 'ion', 'pairs', 'offered', 'instead', 'past', '111–113.', 'a', 'publication', 'scwarc', 'claims', 'detailed', 're-examinations', 'evidence', 'pseudo-cationic', 'polymerizations', 'shows', 'features', 'styrene', 'polymerization', 'initiated', 'acids', 'accounted', 'satisfactorily', 'convincingly', 'orthodox', 'ionic', 'mechanism', 'to', 'support', 'claim', 'reported', 'optical', 'absorption', 'electric', 'conductance', 'styrene', 'solutions', 'undergoing', 'poly-', 'merization', 'induced', 'perchloric', 'acid', 'stopped-ﬂow', 'studies', 'demonstrated', 'protonations', 'monomer', 'acid', 'formation', 'ionic', 'species', '114.', 'further', 'proof', 'pseudo-cationic', 'polymerization', 'mechanism', 'came', 'earlier', 'work', 'matyjaszewski', 'successfully', 'synthesized', 'styryl', 'perchlorate', 'demonstrated', 'hydrolyzes', 'rapidly', 'reaction', 'water', '115:', '2ch3chph', '(cid1)', 'oclo3(cid1)', '(ch2chph)2o', 'þ', '2hclo4', 'h2o', 'this', 'contradicts', 'basic', 'claim', 'pseudo-cationic', 'polymerization', 'mechanism', 'originally', 'proposed', 'gandini', 'plesch', 'hydrolysis', 'ester', 'negligible', '109.', '4333', 'isomerization', 'polymerization', 'in', 'cationic', 'polymerizations', 'monomers', 'may', 'rearrange', 'process', 'placements', 'polymeric', 'chains', 'they', 'isomerize', 'energetically', 'preferred', 'conﬁgurations', 'the', 'result', '43', 'cationic', 'polymerization', '175', 'units', 'ﬁnal', 'polymers', 'structurally', 'different', 'original', 'monomers', 'such', 'rearrangements', 'limited', 'cationic', 'polymerizations', 'in', 'section', 'however', 'discussed', 'isomerizations', 'cationic', 'polymerizations', 'the', 'isomerization', 'polymerizations', 'classiﬁed', 'kennedy', 'according', 'type', 'rearrange-', 'ment', 'accompanies', 'propagation', 'particular', 'processes', '116:', '1', 'propagation', 'reactions', 'accompanied', 'bond', 'electron', 'rearrangement', 'a', 'intra-intermolecular', 'polymerization', 'b', 'transanular', 'polymerization', 'c', 'polymerization', 'strain', 'relief', '2', 'propagation', 'reactions', 'accompanied', 'migration', 'one', 'atoms', 'examples', 'intra', 'intermolecular', 'polymerizations', 'cyclic', 'polymerizations', 'nonconjugated', 'dienes', 'this', 'resemble', 'cyclopolymerization', 'free-radical', 'mechanism', 'r', 'r', 'r', 'in', 'group', 'monomers', 'aliphatic', 'dienes', 'various', 'types', 'also', 'found', 'the', 'propagation', 'proceeds', 'internal', 'attack', 'electrophile', 'another', 'part', 'molecule', 'yielding', 'new', 'carbon', 'cation', 'the', 'following', 'example', 'reaction', '58', '59:', 'r', 'r', 'polynortricyclene', '116–118', 'forms', 'cationic', 'polymerization', 'norbornadiene', 'transanular', 'the', 'reaction', 'best', 'carried', '(cid1)100(cid5)c', 'prevent', 'crosslinking', 'two', 'propagation', 'paths', 'r', 'r', 'n', 'n', 'nmr', 'ir', 'spectra', 'demonstrated', 'propagations', 'shown', 'take', 'place', 'polymerization', 'resultant', 'polymer', 'effect', 'copolymer', 'structures', '119.', 'similarly', 'cationic', 'polymerization', '2-methylene', '5-norbornene', 'involves', 'transanular', 'addition', 'initially', 'formed', 'carbon', 'cation', 'strained', 'endocyclic', 'double', 'bond', 'the', 'product', 'polymeric', 'nortricyclane', '119.', 'polymerization', 'strain', 'relief', 'occurs', 'opening', 'strained', 'rings', 'rearrangements', 'internal', 'double', 'bonds', 'propagation', 'an', 'example', 'polymerization', 'a-pinene', 'the', 'propagation', 'accompanied', 'rearrangement', 'initially', 'formed', 'carbon', 'cation', 'tertiary', 'cation', 'relieves', 'ring', 'strain', '119:', 'h', 'r', 'polymerization', 'possible', 'r', 'r', '176', '4', 'ionic', 'chain-growth', 'polymerization', 'as', 'result', 'formed', 'polymer', 'following', 'structure', 'isomerization', 'polymerizations', 'material', 'transport', 'include', 'propagation', 'reactions', 'accompanied', 'hydride', 'shifts', 'one', 'example', '12-hydride', 'shift', 'polymerization', '3-methylbutene-1', '116:', 'r', 'r', 'hydrogen', 'migration', 'r', 'during', 'propagation', 'tertiary', 'hydrogen', 'atom', 'migrates', 'hydride', 'ion', 'second', 'carbon', 'it', 'results', 'energetically', 'favored', 'tertiary', 'carbon', 'cation', 'the', 'ﬁnal', 'product', '13-polymer', '116:', 'n', 'n', 'another', 'example', 'hydride', 'shift', 'propagation', 'polymerization', 'vinyl', 'cyclohexane', 'aid', 'alcl3', 'temperatures', 'ranging', '(cid1)144', '70(cid5)c', '120', '121', 'r', 'r', 'hydrude', 'r', 'shift', 'n', 'successive', 'hydride', 'shifts', 'possible', 'propagation', 'reactions', 'structures', 'monomers', 'favor', '121:', 'h', 'hyddride', 'shift', 'another', 'hydride', 'shift', 'the', 'temperature', 'polymerization', 'effect', 'this', 'observed', 'polymerization', '3-methylbutene-1', '122.', 'at', 'lower', 'temperatures', 'amount', '13', 'vs,', '12', 'placement', 'apparently', 'increases', 'rapidly', 'at', '(cid1)130(cid5)c', 'essentially', 'pure', 'crystalline', '13', 'poly(methylbutene-1)', 'forms', 'the', 'nature', 'counterion', 'also', 'effect', 'slope', 'product', 'composition', 'curve', 'the', 'polarity', 'solvent', 'however', 'apparently', 'monomer', 'concentration', '122.', 'simultaneous', 'migrations', 'hydrogen', 'atoms', 'methyl', 'groups', 'take', 'place', 'cationic', 'polymeriza-', 'tion', '44-dimethyl-1-pentene', '(neopentylethylene)', '123.', 'in', 'presence', 'alcl3', '(cid1)78', '(cid1)130(cid5)c', 'polymerization', 'proceeds', 'mainly', 'follows', 'n', '14-placement', 'n', '43', 'cationic', 'polymerization', '177', 'the', 'propagation', 'also', 'involve', 'intramolecular', 'halide', 'migration', 'the', 'polymerization', '3-chloro-3-methylbutene-1', 'proceeds', 'low', 'temperatures', 'chloride', 'shift', '(about', '50)', '124:', 'cl', 'cl', 'n', 'cl', 'recently', 'faust', 'coworkers', '125', 'reported', 'study', 'mechanism', 'isomerization', 'polymerization', 'isobutylene', 'cationic', 'polymerization', 'they', 'concluded', 'b-proton', 'elimination', 'tertiary', 'polyisobutylene', 'cation', 'results', 'exo-', 'endo-oleﬁns', 'tetra-substituted', 'oleﬁns', 'form', 'initial', 'cationic', 'rearrangements', 'leading', 'tertiary', 'cations', 'second', 'third', 'positions', 'followed', 'deprotonation', 'all', 'three', 'oleﬁns', 'possess', 'regular', 'carbon', 'numbers', 'the', 'formation', 'tri-', 'substituted', 'oleﬁns', 'follows', 'different', 'mechanism', 'first', 'hydride', 'shift', 'distant', 'carbons', '(backbiting)', 'leads', 'secondary', 'cation', 'undergoes', 'methyde', 'shift', 'minimize', 'energy', 'the', 'resulting', 'hindered', 'tertiary', 'cation', 'undergoes', 'chain', 'scission', 'yielding', 'n-substituted', 'polyisobutylene', 'oleﬁn', 'irregular', 'carbon', 'number', 'this', 'also', 'results', 'oligomeric', 'byproduct', 'known', '“light', 'polymer”', '434', 'termination', 'reactions', 'cationic', 'polymerizations', 'the', 'termination', 'reaction', 'occur', 'transfer', 'monomer', 'proper', 'reaction', 'conditions', 'a', 'a', 'the', 'termination', 'reactions', 'cationic', 'polymerizations', 'often', 'lead', 'low', 'molecular', 'weight', 'products', 'this', 'result', 'various', 'effects', 'also', 'counterions', 'may', 'involved', 'terminations', '126.', 'thus', 'lewis', 'acids', 'polymer', 'cation', 'may', 'react', 'counterion', 'abstracting', 'halogen', 'an', 'example', 'polymerization', 'isobutylene', 'bcl3', 'in', 'reaction', 'termination', 'chain', 'transferring', 'absent', '126.', 'instead', 'following', 'mechanism', 'takes', 'place', '127:', 'bcl3oh', 'cl', 'bcl2oh', 'alkyl', 'aluminum', 'initiated', 'reactions', 'terminate', 'transfer', 'alkyl', 'group', '127:', '(ch3)3alx', '(ch3)2alx', 'x', 'halogen', 'when', 'hydrogen', 'atom', 'located', 'b', 'aluminum', 'alkyl', 'aluminum', 'moiety', 'termination', 'hydrogen', 'transfer', 'appears', 'occur', 'preferentially', '125:', '178', '4', 'ionic', 'chain-growth', 'polymerization', 'x', 'al', 'c2h5', 'c2h5', '(c2h5)2alx', 'termination', 'rearrangement', 'propagating', 'ion-pair', 'also', 'possible', 'such', 'terminations', 'sometimes', 'called', 'spontaneous', 'terminations', 'chain', 'transfers', 'counterions', 'the', 'ion', 'couple', 'rearranged', 'expulsion', 'hydrogen', 'leaving', 'terminal', 'unsaturated', '120:', 'bf3oh', 'h', 'bf3oh', 'many', 'substances', 'act', 'catalysts', '(in', 'combination', 'lewis', 'acids)', 'like', 'water', 'also', 'good', 'chain-transferring', 'agents', 'a', 'solvent', 'impurity', 'act', 'way', '435', 'living', 'cationic', 'polymerizations', 'in', 'cationic', 'polymerizations', 'conditions', 'carefully', 'controlled', 'quasi', 'termination', 'less', 'termination', 'less', 'systems', 'achieved', '130–136.', 'the', '“living”', 'polymers', '“quasiliving”', 'form', 'chain', 'transfer', 'monomer', 'forms', 'termination', 'greatly', 'decreased', 'made', 'reversible', 'throughout', 'reaction', 'the', 'important', 'aspect', 'living', 'cationic', 'polymerization', 'propagating', 'centers', 'sufﬁciently', 'low', 'reactivity', 'transfer', 'termination', 'reactions', 'suppressed', 'the', 'propagation', 'however', 'maintained', 'the', 'molecular', 'weights', 'must', 'increase', 'proportion', 'cumula-', 'tive', 'amount', 'monomer', 'added', 'the', 'lifetimes', 'propagating', 'species', 'extended', 'polymerizations', 'carrying', 'continuous', 'slow', 'additions', 'monomer', 'because', 'chain-transferring', 'reactions', 'completely', 'eliminated', 'systems', 'term', '“quasi”', 'sometimes', 'used', 'living', 'cationic', 'polymerizations', 'carried', 'number', 'monomers', 'isobutylene', 'styrene', 'p-methyl', 'styrene', 'p-methoxystyrene', 'n-vinylcarbazole', 'others', '137', '138.', 'to', 'achieve', 'living', 'conditions', 'necessary', 'match', 'propagating', 'carbon', 'cation', 'counterion', 'solvent', 'polarity', 'reaction', 'temperature', 'some', 'examples', 'offered', 'table', '42', 'by', 'stabilizing', 'inherently', 'unstable', 'carbocationic', 'growing', 'species', 'preventing', 'chain', 'transfer', 'termination', 'living', 'cationic', 'polymerizations', 'achieved', 'this', 'done', 'two', 'methods', '(1)', 'use', 'suitable', 'nucleophilic', 'counterions', '(2)', 'external', 'additions', 'weak', 'lewis', 'bases', '139.', 'the', 'ion', 'pairs', 'tight', 'reactions', 'may', 'border', 'covalent', 'the', 'propagation', 'illustrated', 'follows', 'r', 'i', 'r', 'zni2', 'r', 'δ', 'δ', 'i', 'r', 'r', 'r', 'δ', 'i', 'δ', 'r', 'δ', 'δ', 'i', 'zni2', 'r', 'r', 'r', 'δ', 'δ', 'i', 'zni2', '43', 'cationic', 'polymerization', '179', 'table', '42', 'conditions', 'preparations', 'living', 'polymersab', 'monomer', 'vinyl', 'ethers', 'isobutyl', 'vinyl', 'ether', 'p-methyl', 'styrene', 'isobutylene', 'n-vinylcarbazole', 'styrene', 'indene', 'indene', 'b-pinene', 'from', 'various', 'sources', 'literature', 'b', 'protonic', 'acids', 'used', 'ch3so3h', 'r2p(o)oh', 'r0co2h', 'r', '¼', 'oc6h5', 'c6h5', 'n-c4h9', 'r0', '¼', 'cf3', 'ccl3', 'chcl2', 'ch2cl', 'initiating', 'system', 'hi', 'zni2', 'protonic', 'acidsb', 'zncl2', 'ch3coclo4', 'bcl3', 'cumyl', 'acetate', 'hi', 'ch3ch(c6h5)cl', 'sncl4', 'n-c4h9ncl', 'cumyl', 'methyl', 'ether', 'ticl4', '2-chloro-244-trimethylpentane', 'ticl4', 'ch3ch(och2–ch2cl)cl', 'ticl2(o-i-pr)', 'n-bu4ncl', 'ch2cl2', 'solvent', 'toluene', 'toluene', 'ch2cl2toluene', '(14)', '(cid1)78', '(cid1)30', 'ch2cl2', '(cid1)40', 'toluene', '(cid1)15', 'ch2cl2', '(cid1)40', '(cid1)75', 'ch2cl2', '(cid1)80', 'ch3clch3c6h11', '(cid1)78', '40', 'temp', '((cid5)c)', '(cid1)40', '(cid1)25', '(cid1)40', '(cid1)0', 'as', 'shown', 'carbon–iodine', 'bond', 'stretched', 'without', 'help', 'lewis', 'acid', 'the', 'lewis', 'acid', 'assists', 'stretching', 'carbon–iodine', 'bond', 'whether', 'needed', 'depends', 'upon', 'strength', 'bond', 'this', 'strength', 'turn', 'varies', 'ability', 'solvent', 'temperature', 'substituent', 'r', 'stabilize', 'center', 'depending', 'upon', 'conditions', 'lewis', 'acid', 'convert', 'counterion', 'like', 'i(cid1)', 'stable', 'less', 'nucleophilic', 'species', 'lewis', 'bases', 'like', 'dioxane', 'ethyl', 'acetate', 'may', 'function', 'reacting', 'directly', 'propagating', 'centers', 'living', 'cationic', 'polymerizations', 'also', 'carried', 'heterogeneous', 'catalysts', 'thus', 'aoshima', 'coworkers', '139', 'reported', 'heterogeneous', 'living', 'cationic', 'polymerization', 'isobutyl', 'vinyl', 'ether', 'using', 'fe2o3', 'conjunction', 'isobutyl', 'vinyl', 'ether-hcl', 'adduct', 'toluene', 'this', 'done', 'presence', 'added', 'base', '0(cid5)c', 'such', 'bases', 'ethyl', 'acetate', '14-dioxane', 'these', 'bases', 'effective', 'homogeneous', 'living', 'cationic', 'polymerization', 'vinyl', 'ethers', 'presence', 'various', 'metal', 'halides', 'the', 'living', 'cationic', 'polymerization', 'isobutyl', 'vinyl', 'ether', 'produced', 'polymers', 'narrow', 'molecular', 'weight', 'distribution', 'product', '139.', 'the', 'number', 'average', 'molecular', 'weight', 'increased', 'direct', 'proportion', 'monomer', 'conversion', 'reported', 'molecular', 'weight', 'distributions', 'narrow', 'throughout', 'polymerization', 'mwmn', '¼', '11.', 'stereoselectivity', 'products', 'similar', 'polymers', 'obtained', 'soluble', 'catalysts', 'controlled', 'polymerization', 'also', 'occurred', 'even', 'higher', 'temperature', '(30(cid5)c)', 'what', 'particularly', 'interesting', 'report', 'separated', 'catalyst', 'mixture', 'centrifugation', 'used', 'catalyst', 'catalyze', 'second', 'living', 'polymerization', 'conditions', 'yielding', 'polymer', 'narrow', 'molecular', 'weight', 'distribution', 'the', 'ease', 'catalyst', 'separation', 'permitted', 'repeated', 'reuse', 'catalyst', 'up', 'ﬁfth', 'use', 'catalyst', 'maintained', 'activity', 'give', 'well-deﬁned', 'polymers', 'narrow', 'mwd', '139.', 'heterogeneous', 'conditions', 'due', 'poor', 'solubility', 'heteropoly', 'acid', 'polymerization', 'isobutyl', 'vinyl', 'ether', 'h3pw12o40', 'ch2cl2', 'also', 'studied', 'when', 'bases', 'like', '14-dioxane', 'tetrahy-', 'drofuran', 'present', 'molecular', 'weight', 'distributions', 'broad', 'by', 'contrast', 'polymerizations', 'presence', 'dimethyl', 'sulﬁde', '(cid1)30(cid5)c', 'yielded', 'living', 'polymerizations', 'ether', 'here', 'product', 'narrow', 'molecular', 'weight', 'distribution', '139.', 'in', 'summary', 'typical', 'features', 'living', 'cationic', 'polymerizations', '1', 'the', 'quasiliving', 'livingcontrolled', 'carbocationic', 'polymerizations', 'characterized', 'ioniza-', 'tion', 'equilibrium', 'active', 'reversibly', 'deactivated', 'chains', 'dormant', '2', 'the', 'number', 'average', 'molecular', 'weight', 'polymers', 'form', 'proportional', 'amount', 'monomer', 'introduced', 'reaction', 'mixture', 'in', 'cases', 'reactions', 'rapid', 'often', 'point', 'impossible', 'stop', 'monomer', 'consumed', '3', 'the', 'concentration', 'polymers', 'formed', 'constant', 'independent', 'conversion', 'this', 'concentration', 'often', 'equal', 'concentration', 'initiator', '180', '4', 'ionic', 'chain-growth', 'polymerization', '4', 'addition', 'monomer', 'completed', 'polymerization', 'reaction', 'results', 'polymeriza-', 'tion', 'proportional', 'increase', 'molecular', 'weight', 'storey', 'et', 'al', '140', 'point', 'living', 'polymerizations', 'monomers', 'like', 'isobutylene', 'co-initiated', 'ticl4', 'temperatures', 'low', '(cid1)80(cid5)c', 'livingness', 'limited', 'chain', 'transfer', 'monomer', 'rather', 'unimolecular', 'termination', 'process', 'unimolecular', 'terminations', 'often', 'involve', 'b-proton', 'expulsions', 'produce', 'polymers', 'terminal', 'unsaturation', 'they', 'claim', 'however', 'happen', 'rather', 'normal', 'tert-chloride', 'chain', 'ends', 'polyisobutylene', 'formed', 'type', 'polymerization', 'gradually', 'become', 'depleted', 'they', 'propose', 'therefore', 'isomeri-', 'zation', 'mechanism', 'takes', 'place', 'instead', 'presence', 'active', 'lewis', 'acid', 'monomer', 'starvation', 'conditions', 'it', 'illustrated', 'follows', '140:', '12-methide', 'shift', 'ch2', 'storey', 'et', 'al', '140', 'reported', 'rate', 'depletion', 'tert-chloride', 'end', 'groups', 'follows', 'ﬁrst-order', 'kinetics', 'apparent', 'rate', 'constant', '8', '(cid9)', '10(cid1)5', 's(cid1)1', 'the', 'ratio', 'rate', 'constants', 'propagation', 'rearrangement', 'kpkr', 'found', '3', '(cid9)', '10(cid1)4', 'm(cid1)1', 'fedor', 'et', 'al', '141', 'also', 'studied', 'living', 'cationic', 'polymerization', 'isobutylene', 'using', '2-chloro-', '444-trimethylpentaneticl426-di-tert-butyl-pyridine', 'system', 'hexanemethyl', 'chloride', '(6040', '4060)', 'solvent', 'mixtures', 'various', 'temperatures', 'ranging', '(cid1)25', '(cid1)80(cid5)c', 'temperatures', 'their', 'structural', 'analysis', 'products', 'obtained', '(cid1)40(cid5)c', 'using', '1h', 'nmr', 'spectroscopy', 'shows', 'presence', 'oleﬁnic', 'end', 'groups', 'increasing', 'content', 'increased', 'conversion', 'this', 'led', 'conclude', 'termination', 'polymerizations', 'carried', 'higher', 'temperature', 'involve', 'terminative', 'chain', 'transfer', 'b-proton', 'elimination', 'living', 'chain', 'ends', 'they', 'found', 'eliminated', 'proton', 'trapped', 'instantly', 'proton', 'trap', '26-di-tert-butylpyridine', '141.', 'aoshima', 'coworkers', '142', 'investigated', 'living', 'polymerization', 'isobutyl', 'vinyl', 'ether', 'using', 'pyrrole', 'metal', 'halides', 'weak', 'lewis', 'base', 'in', 'conjunction', 'oxophilic', 'acids', 'zrc14', 'long-lived', 'species', 'produced', 'yield', 'polymers', 'narrow', 'molecular', 'weight', 'distributions', 'number', 'average', 'molecular', 'weight', 'values', 'based', 'used', 'amounts', 'pyrrole', 'acid-trapping', 'experiments', 'using', 'silyl', 'ketene', 'acetal', 'indicated', 'pyrrole', 'worked', 'initiator', 'transfer', 'agent', 'the', 'polymerization', 'started', 'moisture', 'reacting', 'zirconium', 'chloride', 'followed', 'reactions', 'propagating', 'species', '2-', '5-positions', 'pyrrole', 'accompanied', 'generation', 'hcl', 'in', 'addition', 'propagation', 'generated', 'hcl', 'produced', 'pyrrole-bonded', 'chain-end', 'structures', 'also', 'activated', 'oxophilic', 'chlorides', 'generate', 'propagating', 'carbocationic', 'via', 'abstraction', 'isobutoxy', 'group', 'as', 'result', 'number', 'growing', 'chains', 'increased', 'such', 'transfer', 'reactions', 'occur', 'predominantly', 'early', 'stage', 'polymerization', 'stemming', 'highly', 'nucleophilic', 'nature', 'pyrrole', 'thus', 'resulting', 'polymers', 'expected', 'molecular', 'weight', 'values', 'narrow', 'molecular', 'weight', 'distribution', '142.', '43', 'cationic', 'polymerization', '181', '436', 'thermodynamics', 'cationic', 'polymerization', 'sawada', '406', 'described', 'thermodynamics’', 'cationic', 'polymerization', 'follows', 'the', 'process', 'protonic', 'acid', 'initiation', 'cationic', 'polymerization', 'starts', 'heterolytic', 'bond', 'dissociation', 'acid', 'the', 'energy', 'process', 'designated', 'quantity', 'e2', 'afﬁnity', 'proton', 'oleﬁn', 'the', 'potential', 'energy', 'ion', 'pairs', 'separated', 'distance', 'r', 'e', 'unit', 'electronic', 'charge', 'd', 'dielectric', 'constant', 'the', 'total', 'entropy', 'change', 'system', 'e1', 'acid', 'strength', 'initiator', 'e2', 'proton', 'afﬁnity', 'monomer', 'e3', 'solvating', 'power', 'solvent', 'the', 'change', 'free', 'energy', 'initiation', 'e3', '¼', 'e2rd', 'dh', '¼', 'e1', '(cid1)', 'e2', '(cid1)', 'e3', 'dfi', '¼', 'dh', '(cid1)', 'tds', 'by', 'substituting', 'values', 'free', 'energy', 'change', 'initiation', 'written', 'dfi', '¼', 'e1', '(cid1)', 'e2', '(cid1)', 'e2rd', '(cid1)', 'tds', 'the', 'entropy', 'change', 'enthalpy', 'change', 'negative', 'free', 'energy', 'change', 'negative', 'in', 'propagation', 'process', 'change', 'activation', 'entropy', 'based', 'two', 'phenomena', '(a)', 'entropy', 'change', 'immobilization', 'free-moving', 'monomer', 'polymeric', 'chain', '(b)', 'difference', 'entropy', 'solvation', 'reactants', 'transition', 'state', 'written', 'ds(cid4)', '¼', 'ds(cid4)ðimmobilization', 'monomerþ', 'þ', 'ds(cid4)ðsolvationþ', 'ds', 'solvation', 'positive', 'term', 'transition', 'state', 'electric', 'charge', 'diffuse', 'degree', 'solvation', 'lower', 'compact', 'dipolar', 'species', 'corresponding', 'reactants', 'the', 'propagation', 'constant', 'based', 'eyring', 'theory', 'kp', '¼', 'ap', 'expð(cid1)dep', '(cid4)rtþ', '¼', 'ðkthþ', 'expðds(cid4)rþ', 'expð(cid1)dep', '(cid4)rtþ', 'activation', 'entropy', 'dsp', 'function', 'ft', 'fi', 'partition', 'functions', 'transition', 'initial', 'states', 'propagation', 'step', 'respectively', 'dsp', '(cid4)', '¼', 'r', 'lnðftfiþ', 'þ', 'rt', 'lnðftfiþdt', 'ft', 'fi', 'assigned', 'according', 'sawada', '406', 'partition', 'functions', 'monomer', 'transition', 'sate', '(', 'ft(m))', 'monomer', 'initial', 'state', '(', 'fi(m))', 'parts', 'growing', 'chain', 'common', 'transition', 'initial', 'states', 'propagation', 'reaction', '182', '4', 'ionic', 'chain-growth', 'polymerization', 'the', 'term', 'rt', 'ln', '(', 'ftfi)dt', 'negligible', 'compared', 'r', 'ln', '(', 'ftfi)', 'the', 'activation', 'entropy', 'therefore', 'written', 'dsp', '(cid4)', '¼', 'r', 'lnð', 'ftfiþ', '44', 'anionic', 'polymerization', 'oleﬁns', 'as', 'stated', 'beginning', 'chapter', 'distinctive', 'features', 'anionic', 'chain-growth', 'polymerizations', 'negative', 'charges', 'active', 'centers', 'many', 'different', 'bases', 'initiate', 'polymerizations', 'depending', 'monomer', 'the', 'common', 'ones', 'organometallic', 'compounds', 'alkali', 'metals', 'metal', 'amides', 'grignard', 'reagents', 'monomers', 'carbanion', 'stabilizing', 'substituents', 'either', 'resonance', 'induction', 'polymerize', 'readily', 'such', 'substituents', 'fact', 'determine', 'base', 'strength', 'initiators', 'needed', 'carry', 'polymerizations', 'electron', 'withdrawing', 'capacity', 'thus', 'instance', 'weak', 'bases', 'water', 'initiate', 'vinylidene', 'cyanide', 'polymerizations', 'h2o', 'n', 'n', 'no2', 'khco3', 'n', 'n', 'n', 'n', 'n', 'n', 'no2', 'potassium', 'bicarbonate', 'initiates', 'polymerization', '2-nitropropene', 'on', 'hand', 'monomers', 'like', 'acrylic', 'methacrylic', 'esters', 'conjugated', 'dienes', 'require', 'strong', 'bases', 'like', 'alkali', 'metals', 'organometallic', 'compounds', '441', 'initiation', 'anionic', 'chain-growth', 'polymerization', 'anionic', 'initiations', 'take', 'different', 'paths', 'that', 'depends', 'upon', 'initiator', 'monomer', 'solvent', '143.', 'as', 'rough', 'generalization', 'however', 'possible', 'separate', 'initiation', 'reactions', 'two', 'types', '(1)', 'take', 'place', 'addition', 'negative', 'ion', 'oleﬁn', '(2)', 'result', 'electron', 'transfer', 'the', 'polymerization', 'reactions', 'furthermore', 'occur', 'two', 'ways', 'one', 'way', 'homogeneous', 'environment', 'monomer', 'initiator', 'soluble', 'solvent', 'the', 'way', 'heterogeneous', 'one', 'monomer', 'soluble', 'organolithium', 'compounds', 'examples', 'soluble', 'initiators', 'metal', 'dispersions', 'examples', 'insoluble', 'ones', 'the', 'homogeneous', 'polymerizations', 'separated', 'carried', 'non-polar', 'solvents', 'polar', 'ones', 'in', 'non-polar', 'solvents', 'conﬁned', 'mostly', 'organolithium', 'initiators', 'other', 'organometallic', 'compounds', 'require', 'polar', 'solvents', 'solubility', '44', 'anionic', 'polymerization', 'oleﬁns', '183', '4411', 'initiation', 'addition', 'anion', 'oleﬁn', 'when', 'organolithium', 'compounds', 'dissolved', 'non-polar', 'solvents', 'strong', 'tendency', 'solute', 'molecules', 'associate', 'aggregates', 'for', 'instance', 'butyllithium', 'hexameric', 'hexane', 'solution', 'this', 'true', 'ethyllithium', '144', '145', 'well', 'addition', 'lewis', 'bases', 'solutions', 'causes', 'formation', 'strong', 'complexes', 'bases', 'organometallic', 'compounds', '146', '147.', 'this', 'causes', 'clusters', 'breakup', 'succession', 'equilibrium', 'tetrameric', 'dimeric', 'intermediates', 'becoming', 'complexes', 'lewis', 'bases', '146.', 'particularly', 'effective', 'bases', 'allow', 'close', 'approach', 'lithium', 'ion', 'heteroatom', 'often', 'carbon–lithium', 'bond', 'normally', 'ranges', '20', '40', 'ionic', 'character', 'becomes', 'much', 'ionic', '148', '149.', 'the', 'breakdown', 'aggregates', 'shown', 'dramatically', 'enhance', 'reactivity', 'organometallic', 'compounds', '150', '151.', 'for', 'instance', 'polymerizations', 'styrene', 'benzene', 'butyllithium', 'slow', 'reactions', 'when', 'however', 'polymerizations', 'carried', 'tetrahydro-', 'furan', 'extremely', 'rapid', 'tetrahydrofuran', 'course', 'lewis', 'base', 'nevertheless', 'breakdown', 'aggregates', 'even', 'lewis', 'bases', 'tetrahydrofuran', 'diethyl', 'ether', 'complete', 'though', 'clusters', 'smaller', 'solvated', 'differences', 'reactivity', 'however', 'observed', 'even', 'different', 'non-polar', 'solvents', '152.', 'the', 'reaction', 'rates', 'depend', 'great', 'extent', 'nature', 'organometallic', 'compounds', 'polarity', 'bonds', 'degree', 'solvation', 'in', 'polar', 'solvents', 'free', 'solvated', 'ion-pairs', 'predominate', 'mechanism', 'initiation', 'may', 'simply', 'consist', 'direct', 'addition', 'anion', 'monomer', 'if', 'solvents', 'non-polar', 'hand', 'initiation', 'complex', 'in', 'solvents', 'metal', 'cation', 'coordinates', 'monomer', 'ﬁrst', 'this', 'followed', 'rearrangement', '153:', 'mr', 'x', 'r', 'm', 'x', 'r', 'm', 'x', 'in', 'initial', 'coordination', 'p-electron', 'cloud', 'oleﬁn', 'overlaps', 'outer', 'bond', 'orbital', 'metal', 'cation', 'this', 'causes', 'stretching', 'eventual', 'rupture', 'r–m', '(metal)', 'bond', 'an', 'intramo-', 'lecular', 'rearrangement', 'follows', 'migration', 'carbanion', '(r(cid8))', 'electron-deﬁcient', 'carbon', 'atom', 'double', 'bond', 'a', 'new', 'covalent', 'bond', 'new', 'carbanion', 'formed', 'simultaneously', 'h', 'c', 'c', 'xm', 'h', 'h', 'r', 'r', 'c', 'c', 'm', 'h', 'h', 'h', 'x', 'm', 'represents', 'metal', 'the', 'basicity', 'anion', 'portion', 'initiating', 'species', 'also', 'important', 'for', 'instance', 'ﬂuorenelithium', 'initiates', 'polymerizations', 'methyl', 'methacrylate', 'fails', 'initiate', 'polymerizations', 'styrene', 'a', 'electronegative', 'butyl', 'anion', 'butyl', 'lithium', 'hand', 'initiates', 'polymerizations', 'monomers', 'yet', 'also', 'shown', 'order', 'reactivity', 'often', 'contrary', 'inherent', 'basicity103–146', 'this', 'may', 'due', 'size', 'aggregates', 'particular', 'organometallic', 'compound', 'forms', 'in', 'addition', 'higher', 'reactivity', 'may', 'also', 'due', 'favor-', 'able', 'energy', 'requirements', 'breaking', 'aggregates', '146.', '184', '4', 'ionic', 'chain-growth', 'polymerization', 'presence', 'heteroatoms', 'like', 'oxygen', 'nitrogen', 'monomer', 'causes', 'rapid', 'complexation', 'organometallic', 'compounds', 'for', 'instance', 'butyllithium', 'initiates', 'polymerization', 'styrene', 'hydrocarbon', 'solvents', 'induction', 'period', 'overall', 'reaction', 'slow', 'sluggish', 'when', 'however', 'initiates', 'polymerization', 'o-methoxystyrene', 'conditions', 'induction', 'period', 'reaction', 'rapid', 'this', 'due', 'initiator', 'coordinating', 'oxygen', 'atom', '153–155:', 'c4h9', 'li', 'o', 'c4h9', 'c4h9', 'li', 'o', 'li', 'o', 'a', 'similar', 'coordination', 'heteroatom', 'takes', 'place', 'polymerizations', 'acrylic', 'methacrylic', 'esters', '156.', 'here', 'metal', 'coordinates', 'electron', 'cloud', 'whole', 'conjugated', 'structure', 'the', 'overlap', 'stretches', 'eventually', 'ruptures', 'metal-alkyl', 'bond', 'a', 'new', 'carbon', 'carbon', 'covalent', 'bond', 'forms', 'together', 'new', 'metal', 'carbon', 'linkage', 'o', 'o', 'r', 'm', 'o', 'o', 'h', 'm', 'h', 'r', 'r', 'o', 'm', 'o', 'm', 'represents', 'metal', 'the', 'product', 'might', 'resonance', 'hybrid', 'two', 'structures', 'metal', 'associated', 'carbanion', 'one', 'oxygen', '157.', 'some', 'abnormalities', 'reported', 'initiations', 'methyl', 'methacrylate', 'polymerizations', 'toluene', 'butyllithium', 'their', 'nature', 'suggest', 'possibility', 'one', 'reaction', 'taking', 'place', 'simultaneously', '158.', 'one', 'must', 'major', 'one', 'organome-', 'tallic', 'compound', 'reacting', 'carbon', 'carbon', 'double', 'bond', 'shown', 'the', 'minor', 'one', 'may', 'carbon', 'oxygen', 'double', 'bond', 'the', 'major', 'reaction', 'produces', 'methyl', 'methacry-', 'late', 'anions', 'the', 'minor', 'reaction', 'however', 'yields', 'butyl', 'isopropenyl', 'ketone', 'accompanying', 'formation', 'lithium', 'methoxide', '158:', 'c4h9li', 'o', 'o', 'c4h9', 'lioch3', 'o', 'lithium', 'methoxide', 'initiate', 'polymerizations', 'methyl', 'methacrylate', 'the', 'methoxide', 'molecules', 'however', 'react', 'carbanions', 'growing', 'chain', 'the', 'resultant', 'anions', 'less', 'reactive', 'methyl', 'methacrylate', 'anions', 'add', 'new', 'methyl', 'methacrylate', 'monomers', 'slowly', '44', 'anionic', 'polymerization', 'oleﬁns', '185', 'once', 'added', 'however', 'reaction', 'proceeds', 'normal', 'rate', '158.', 'polymerizations', 'methyl', 'methacrylate', 'polar', 'solvents', 'hand', 'proceed', 'might', 'described', 'ideal', 'manner', 'formations', 'one', 'kind', 'ion', 'pairs', '159–162.', 'other', 'monomers', 'also', 'exhibit', 'abnormal', 'behavior', 'anionic', 'polymerizations', 'thus', 'instance', 'organomagnesium', 'initiation', 'methacrylonitrile', 'polymerization', 'results', 'formation', 'two', 'types', 'active', 'centers', '163', '164:', 'rmgx', 'n', 'r', 'n', 'mgx', 'r', 'mgx', 'n', 'methyl', 'methacrylate', 'polymerizations', 'initiated', 'organomagnesium', 'compounds', 'also', 'yield', 'abnormal', 'products', 'here', 'active', 'centers', 'unusually', 'persistent', 'stable', 'in', 'addition', 'a-carbon', 'atoms', 'monomers', 'found', 'assume', 'tetrahedral', 'conﬁgurations', '165–167.', 'this', 'suggests', 'active', 'centers', 'contain', 'covalent', 'magnesium', 'carbon', 'bonds', 'also', 'gel', 'permeation', 'chromatography', 'curves', 'products', 'show', 'one', 'active', 'center', 'operates', 'independently', '166', '167.', 'a', '“pseudo', 'anionic”', 'mechanism', 'therefore', 'postulated', 'polymerizations', 'acrylic', 'methacrylic', 'esters', '111', '112', 'grignard', 'reagents', 'three', 'different', 'simultaneous', 'reactions', 'appear', 'taking', 'place', 'butyllithium-initiated', 'polymerizations', 'vinyl', 'ketones', 'benzene', '168:', '(c4h9li)', '6c4h9li', 'c4h9', 'o', 'li', 'c4h9li', '2', '3', '1', '4', 'o', '34', 'o', 'li', 'c4h9', 'r', 'li', '14', '12', 'c4h9', 'li', 'o', 'h', 'h', 'h', 'li', 'r', 'h', 'h', 'the', 'initiations', 'polymerizations', 'conjugated', 'dienes', 'inert', 'hydrocarbons', 'also', 'believed', 'preceded', 'coordination', 'organometallic', 'compounds', 'p-electron', 'clouds', 'monomers', '186', '4', 'ionic', 'chain-growth', 'polymerization', 'the', 'rearrangement', 'follows', 'result', 'either', '12', '14', '34', 'additions', 'double', 'bond', 'c4h9li', '1', '3', '2', '4', '34', '12', '14', 'c4h9', 'li', 'c4h9', 'ch2', 'li', 'c4h9', 'ch2', 'li', '4412', 'one', 'electron', 'transfer', 'initiation', 'alkali', 'metals', 'initiate', 'anionic', 'polymerizations', 'oleﬁns', 'either', 'homogeneous', 'heterogeneous', 'conditions', 'this', 'depends', 'upon', 'metal', 'upon', 'solvent', 'for', 'instance', 'potassium', 'soluble', 'ethers', 'like', 'dimethoxyethane', 'tetrahydrofuran', 'initiation', 'conditions', 'homogeneous', 'on', 'hand', 'sodium', 'dispersions', 'insoluble', 'hydrocarbons', 'initiations', 'heteroge-', 'neous', 'liquid', 'ammonia', 'solvent', 'many', 'alkali', 'metals', 'though', 'like', 'sodium', 'reactant', 'form', 'metal', 'amides', 'provided', 'traces', 'iron', 'present', 'initiation', 'reactions', 'many', 'metal', 'solutions', 'take', 'place', 'electron', 'transfer', 'metal', 'monomer', 'form', 'anion', 'radicals', 'the', 'resultant', 'anion', 'radicals', 'may', 'undergo', 'propagation', 'reactions', 'these', 'propagations', 'proceed', 'anionically', 'free-radical', 'mechanism', 'simultaneously', 'if', 'radicals', 'unstable', 'anion-radicals', 'dimerize', 'propagation', 'proceeds', 'anionic', 'mechanism', 'ends', 'chain', 'when', 'liquid', 'ammonia', 'employed', 'solvent', 'stated', 'earlier', 'particular', 'mechanism', 'initiation', 'depend', 'upon', 'metal', 'used', 'lithium', 'metal', 'forms', 'solutions', 'liquid', 'ammonia', 'initiates', 'polymerization', 'monomers', 'like', 'methacrylonitrile', 'electron', 'transfer', '169:', 'li', 'ch2', 'li', 'n', 'these', 'radical-anions', 'couple', 'chain', 'growth', 'takes', 'places', 'ends', 'chains', '2', 'ch2', 'li', 'n', 'li', 'n', 'li', 'n', 'dimerization', 'radical-ions', 'depends', 'upon', 'radical’s', 'stability', 'also', 'upon', 'p-energy', 'changes', 'accompany', 'reaction', '170', '171.', '44', 'anionic', 'polymerization', 'oleﬁns', '187', 'potassium', 'metal', 'ammonia', 'however', 'initiates', 'polymerizations', 'monomers', 'like', 'methacry-', 'lonitrile', 'styrene', 'differently', 'these', 'reactions', 'include', 'additions', 'amide', 'ions', 'oleﬁns', 'formations', 'amine', 'groups', 'end', 'chains', '172:', '2k', '2nh3', '2k', '2', 'nh2', 'solutions', 'potassium', 'metal', 'ethers', 'however', 'form', 'ion', 'radicals', 'additions', 'electrons', 'monomers', 'it', 'noted', 'fresh', 'solutions', 'potassium', 'metal', 'various', 'ethers', 'like', 'tetrahydrofuran', 'dimethoxyethane', 'blue', 'color', 'this', 'blue', 'color', 'attributed', 'presence', 'spin-coupled', 'electron', 'pairs', '(e2)', 'the', 'initiation', 'styrene', 'polymerization', 'takes', 'place', '0', '(cid1)78(cid5)c', 'therefore', 'pictured', 'reactions', 'spin-coupled', 'pair', 'monomers', 'also', 'additions', 'individual', 'electrons', '169:', 'k', 'nh2', 'knh2', 'nh2', 'h2n', 'e2', 'e', 'ch2', 'e', 'ch2', 'when', 'conditions', 'heterogeneous', 'sodium', 'metal', 'dispersions', 'used', 'polymerizations', 'dienes', 'initiation', 'mechanism', 'also', 'electron', 'transfer', 'it', 'believed', '135', 'initially', '12', 'anion-radical', 'forms', 'this', 'followed', 'coupling', 'reaction', 'na', 'coupling', 'process', 'na', 'ch2', 'initial', 'ion-radical', 'na', 'ch2', 'rearrangement', 'ch2', '2', 'na', 'ch2', 'na', 'ch2', 'ch2', 'ch2', 'na', '188', '4', 'ionic', 'chain-growth', 'polymerization', 'different', 'kind', 'coupling', 'also', 'possible', 'na', 'ch2', 'ch2', 'ch2', 'na', 'na', 'ch2', 'ch2', 'na', 'the', 'following', 'initiation', 'mechanism', 'postulated', '173', 'heterogeneous', 'conditions', 'at', 'ﬁrst', 'highly', 'polarized', 'monomers', 'adsorb', 'strongly', 'metal', 'surfaces', 'electron', 'transfer', 'takes', 'place', 'the', 'adsorbed', 'molecules', 'assumed', 'still', 'sufﬁciently', 'mobile', 'able', 'rotate', 'adsorption', 'the', 'rotation', 'allows', 'radical-anions', 'form', 'couple', 'the', 'concentration', 'anionic', 'charges', 'develop', 'methine', 'carbons', 'creates', 'strong', 'enough', 'attractive', 'forces', 'remove', 'metal', 'cations', 'surface', 'solvents', 'strongly', 'solvate', 'cations', 'like', 'tetrahy-', 'drofuran', 'enhance', 'process', 'to', 'ﬁt', 'mechanism', '173,', 'monomers', 'must', 'capable', 'adsorbing', 'strongly', 'metal', 'surfaces', 'solvents', 'must', 'also', 'strongly', 'solvating', 'when', 'reactions', 'carried', 'without', 'solvents', 'inert', 'solvents', 'like', 'heptane', 'free-energy', 'gain', 'due', 'solvation', 'largely', 'lost', 'this', 'increases', 'probability', 'instances', 'propagations', 'occur', 'surface', 'metals', 'oligomers', 'grow', 'certain', 'length', 'when', 'sufﬁcient', 'length', 'reached', 'create', 'favorable', 'free-energy', 'change', 'desorption', 'occurs', 'molecules', 'pass', 'solution', '173.', 'electron', 'transfer', 'initiations', 'also', 'result', 'reactions', 'alkali', 'metals', 'aromatic', 'hydrocarbons', 'aromatic', 'ketones', 'result', 'formations', 'radical-ions', 'na', 'k', 'li', 'na', 'k', 'o', 'o', 'li', 'the', 'ion-radicals', 'transfer', 'electron', 'ion', 'charge', 'monomers', 'process', 'initiation', 'shown', 'szwarc', 'coworkers', '174–180:', 'na', 'ch2', 'na', '44', 'anionic', 'polymerization', 'oleﬁns', '189', 'the', 'styrene', 'monomer-radical-ion', 'unstable', 'tends', 'dimerize', 'ch2', '2', 'na', 'na', 'na', 'metal', 'ketyls', 'form', 'reactions', 'alkali', 'metals', 'aromatic', 'ketones', 'polar', 'solvents', 'like', 'dioxane', 'tetrahydrofuran', 'these', 'ketyls', 'exist', 'equilibrium', 'mixtures', 'monomeric', 'anion-radicals', 'dimeric', 'dianions', '136.', 'originally', 'controversy', 'mechanism', 'initiation', 'monomers', 'like', 'acrylonitrile', 'methyl', 'methacrylate', 'sodium', 'benzophenone', 'the', 'following', 'mechanism', 'derived', 'spectral', 'evidence', 'the', 'initiation', 'transfer', 'electron', 'ion', 'charge', '181–183:', 'o', 'o', 'o', 'o', 'o', 'o', '4413', 'initiations', 'alﬁn', 'catalysts', 'these', 'catalysts', 'formed', 'combining', 'alkyl', 'sodium', 'sodium', 'alkoxide', 'alkali', 'metal', 'halide', '184–189.', 'a', 'typical', 'effective', 'catalyst', 'polymerization', 'dienes', 'consists', 'allyl', 'sodium', 'sodium', 'isopropoxide', 'sodium', 'chloride', 'the', 'preparation', 'catalyst', 'carried', 'combining', 'amyl', 'chloride', 'sodium', 'reacting', 'product', 'isopropyl', 'alcohol', 'after', 'propylene', 'bubbled', 'reaction', 'mixture', 'convert', 'amyl', 'sodium', 'allyl', 'sodium', 'the', 'reactions', 'summarized', 'follows', '15c5h11cl', 'þ', '3na', '15c5h11na', 'þ', '15nacl', 'c5h11na', 'þ', 'ðch3þ2choh', 'ðch3þ2chona', 'þ', 'c5h12', 'c5h11na', 'þ', 'ch3', '(cid1)', 'ch', 'ch2', 'c5h12', 'þ', 'ch2', 'ch', '(cid1)', 'ch2na', 'diisopropyl', 'ether', 'may', 'used', 'place', 'isopropyl', 'alcohol', 'in', 'case', 'reaction', 'require', 'addition', 'propylene', 'oleﬁn', 'forms', 'situ', '189.', 'these', 'catalysts', 'particularly', 'effective', 'polymerizations', 'conjugated', 'dienes', 'high', 'molecular', 'weight', 'products', 'styrene', 'also', 'polymerized', 'catalysts', 'polymers', 'dienes', 'form', 'high', 'trans-', '14', 'repeat', 'units', 'butadiene', 'polymerized', 'catalysts', 'much', 'rapidly', 'isoprene', 'on', 'hand', '23-dimethylbutadiene', 'fails', 'polymerize', '190', '4', 'ionic', 'chain-growth', 'polymerization', 'there', 'general', 'agreement', 'mechanism', 'polymerizations', 'both', 'anionic', 'free-radical', 'mechanisms', 'proposed', '184–189', 'probable', 'reaction', 'paths', 'the', 'role', 'sodium', 'chloride', 'clear', 'mixture', 'though', 'shown', 'essential', '184–189.', 'it', 'may', 'act', 'perhaps', 'support', 'catalyst', 'may', 'part', 'sort', 'lattice', 'involving', 'sodium', 'alkoxide', 'allyl', 'sodium', 'the', 'anionic', 'mechanism', 'pictured', 'follows', '184–189:', 'r', 'na', 'monomer', 'r', 'na', 'ch2', 'r', 'na', 'na', 'cl', 'na', 'cl', 'catalytic', 'bed', 'na', 'cl', 'na', 'cl', 'initiation', 'na', 'cl', 'na', 'cl', 'the', 'free-radical', 'mechanism', 'suggested', 'due', 'high', 'predominance', 'trans-14', 'placement', 'polymerization', 'butadiene', '189.', 'according', 'mechanism', 'complex', 'sodium', 'isopropoxide', 'allyl', 'sodium', 'forms', 'ﬁrst', 'o', 'na', 'ch2na', 'o', 'na', 'na', 'ch2', 'the', 'monomer', 'adsorbs', 'surface', 'visualized', 'displacing', 'allyl', 'anion', 'complex', 'form', 'ion', 'pair', 'ﬁrst', 'electron', 'rearrangement', 'radical', '189:', 'ch2', 'ch2na', 'ch2', 'ch2', 'ch2na', 'the', 'polymerization', 'assumed', 'go', 'point', 'via', 'free-radical', 'mechanism', 'combination', 'allyl', 'radical', 'takes', 'place', 'because', 'allyl', 'radical', 'bound', 'catalyst', 'surface', 'combination', 'take', 'place', 'readily', 'high', 'molecular', 'weights', 'attained', '4414', 'electroinitiation', 'anionic', 'polymerizations', 'electrolytic', 'polymerizations', 'described', 'section', 'dealing', 'cationic', 'polymerizations', 'anionic', 'polymerizations', 'also', 'initiated', 'electric', 'ﬁeld', 'when', 'lialh4', 'naal(c2h5)4', 'used', 'electrolytes', 'tetrahydrofuran', 'solvent', '“living”', 'polymers', 'formed', 'a-methylstyrene', '190.', 'the', 'deep', 'red', 'color', 'carbanions', 'develops', 'ﬁrst', 'cathode', 'compartments', 'divided', 'cells', '44', 'anionic', 'polymerization', 'oleﬁns', '191', 'by', 'cooling', 'solutions', '(cid1)80(cid5)c', 'almost', 'quantitative', 'conversions', 'high', 'molecular', 'weight', 'polymers', 'take', 'place', '190.', 'similar', 'conditions', 'yield', 'polystyrene', '191.', '4-vinyl', 'pyridine', 'polymerizes', 'liquid', 'ammonia', 'manner', '191.', 'initiation', 'results', 'reduction', 'monomer', 'direct', 'electron', 'transfer', 'cathode', 'form', 'red–orange', 'vinyl', 'pyridyl', 'radical-anion', 'e', 'n', 'ch2', 'ch', 'n', 'another', 'example', 'polymerization', 'isoprene', 'electric', 'ﬁeld', 'tetrahydrofuran', 'solution', 'here', '“living”', 'polymer', 'forms', '192.', '442', 'propagation', 'anionic', 'chain-growth', 'polymerization', 'the', 'propagation', 'reaction', 'consists', 'successive', 'additions', 'monomer', 'molecules', 'active', 'centers', 'growing', 'chains', 'm', 'x', 'x', 'x', 'x', 'm', 'no', 'matter', 'mechanism', 'initiation', 'propagation', 'reaction', 'takes', 'place', 'strictly', 'monomer', 'growing', 'polymeric', 'chain', 'without', 'counterion', 'when', 'reaction', 'occurs', 'non-polar', 'solvents', 'propagation', 'step', 'hampered', 'much', 'tendency', 'ion', 'pairs', 'cluster', 'aggregates', 'encountered', 'initiation', 'for', 'instance', 'butyllithium', 'initiated', 'polymerizations', 'styrene', 'benzene', 'propagation', 'step', 'much', 'faster', 'initiation', '193', '194.', 'this', 'probably', 'due', 'absence', 'aggregates', 'some', 'association', 'growing', 'polymeric', 'chains', 'however', 'occur', '193.', 'it', 'may', 'shown', 'follows', '2', 'li', 'r', 'r', 'li', 'li', 'r', 'these', 'association', 'equilibriums', 'however', 'mobile', 'character', '193.', 'the', 'driving', 'force', 'propagation', 'reaction', 'similar', 'initiation', 'in', 'non-polar', 'solvents', 'reaction', 'incoming', 'monomers', 'similar', 'initiation', 'step', 'the', 'monomers', 'coordinate', 'cations', 'end', 'chains', 'ﬁrst', 'this', 'followed', 'intramolecular', 'rearrangements', 'lead', 'regenerations', 'new', 'metal', 'carbon', 'linkages', 'li', 'li', 'h', 'h', 'c', 'c', 'h', 'li', '192', '4', 'ionic', 'chain-growth', 'polymerization', 'in', 'polar', 'solvents', 'hand', 'reactions', 'go', 'extreme', 'the', 'propagation', 'simply', 'consist', 'successive', 'additions', 'monomers', 'growing', 'anions', 'in', 'homogeneous', 'anionic', 'polymerizations', 'simple', 'vinyl', 'monomers', 'steric', 'placement', 'also', 'temperature', 'dependent', 'cationic', 'polymerizations', 'syndiotactic', 'placement', 'favored', 'isotactic', 'placement', 'polar', 'solvents', 'predominates', 'temperatures', 'here', 'results', 'mainly', 'degree', 'association', 'counterion', '181.', 'in', 'non-polar', 'solvents', 'however', 'low', 'temperature', 'much', 'current', 'knowledge', 'propagation', 'reaction', 'based', 'studies', 'carried', 'highly', 'solvating', 'ether', 'solvents', 'less', 'information', 'available', 'homogeneous', 'reactions', 'non-', 'polar', 'medium', 'generally', 'though', 'rate', 'propagation', 'increases', 'solvent', 'polarity', 'degree', 'ion', 'pair', 'dissociation', '195.', 'organolithium', 'compounds', 'undergo', 'greatest', 'degree', 'solvation', 'changed', 'hydrocarbons', 'polar', 'ether', 'solvents', 'cesium', 'compounds', 'hand', 'least', 'affected', 'changes', 'solvent', 'polarity', 'in', 'addition', 'nmr', 'studies', 'polystyryl', 'carbanion', 'structures', 'associated', 'lithium', 'potassium', 'cesium', 'counter', 'cations', 'studied', 'different', 'solvents', 'different', 'temperatures', '196.', 'the', 'results', 'show', 'interaction', 'larger', 'radius', 'cations', 'phenyl', 'rings', 'ultimate', 'monomer', 'units', 'chains', 'the', 'structures', 'potassium', 'cesium', 'counterions', 'judging', 'model', 'compounds', 'found', 'planar', 'sp2-hybridized', 'a-carbon', 'it', 'suggests', 'presence', 'larger', 'counterions', 'rotation', 'terminal', 'phenyl', 'ring', 'styrene', 'polymerization', 'strongly', 'hindered', 'propagation', 'rates', 'also', 'depend', 'upon', 'structures', 'monomers', 'for', 'polymerizations', 'initiated', 'alkali', 'amides', 'following', 'order', 'reactivity', 'observed', '197:', 'n', 'o', 'o', 'also', 'methyl', 'substitution', 'a-carbon', 'tends', 'decrease', 'reaction', 'rate', 'due', 'electron', 'releasing', 'effect', 'alkyl', 'group', 'this', 'tends', 'destabilize', 'carbanion', 'also', 'cause', 'steric', 'interference', 'solvation', 'chain', 'end', 'addition', 'monomer', '197.', '4421', 'steric', 'control', 'anionic', 'polymerization', 'use', 'hydrocarbon', 'solvents', 'advantage', 'polymerizations', 'conjugated', 'dienes', 'yield', 'steric', 'control', 'monomer', 'placement', 'this', 'true', 'tacticity', 'geometric', 'isomerism', 'as', 'stated', 'earlier', 'insertions', '12', '34', '14', 'furthermore', '14-placements', 'cis', 'trans', 'lithium', 'organolithium', 'initiators', 'hydrocarbon', 'solvents', 'yield', 'polyiso-', 'prene', 'instance', '90', 'cis-14', 'structure', '99.', 'the', 'reaction', 'polar', 'solvents', 'however', 'yields', 'polymers', 'mostly', '12', '34', 'trans-14', 'structure', 'there', 'still', 'mechanism', 'fully', 'explains', 'steric', 'control', 'polymerization', 'dienes', 'high', 'cis-14', 'polyisoprene', 'forms', 'lithium', 'alkyl', 'lithium', 'initiators', 'non-polar', 'solvents', 'propagation', 'takes', 'place', 'essentially', 'covalent', 'intimate', 'ion-pair', 'lithium', 'carbon', 'bonds', '99.', 'an', 'intermediate', 'pseudo-six-membered', 'ring', 'believed', 'form', 'process', 'addition', 'diene', '99:', 'li', 'ch2', 'ch2', 'li', '44', 'anionic', 'polymerization', 'oleﬁns', '193', 'formation', 'intermediates', 'favorable', 'lithium', 'small', 'ionic', 'radius', 'high', 'proportion', 'p-character', 'organometallic', 'compounds', 'alkali', 'metals', '(sodium', 'potassium', 'rubidium', 'cesium)', 'polar', 'dissociated', 'they', 'react', 'essentially', 'solvated', 'ions', 'even', 'hydrocarbon', 'medium', 'yielding', 'high', '34', 'placement', 'o’driscoll', 'et', 'al', '198', 'proposed', 'mechanism', 'steric', 'control', 'in', 'isoprene', 'polymerization', 'terminal', 'charges', 'complexed', 'metal', 'cations', 'these', 'cations', 'close', 'active', 'centers', 'occupied', 'p-orbitals', 'chain', 'ends', 'unoccupied', 'p-orbitals', 'lithium', 'ions', 'in', 'transition', 'state', 'monomers', 'complexed', 'cations', 'way', '198.', 'the', 'lithium', 'cations', 'assumed', 'hybridized', 'tetrahedral', 'sp3', 'conﬁgurations', 'four', 'vacant', 'orbitals', 'the', 'chain', 'ends', 'presumed', 'allylic', 'diene', 'monomers', 'bidentate', '198.', 'during', 'propagation', 'steps', 'monomers', 'chain', 'ends', 'complex', 'counterions', 'δ', 'li', 'ch2', 'δ', 'in', 'hydrocarbon', 'solvents', 'complexes', 'tight', 'rotations', 'c2–c3', 'bonds', 'sterically', 'hindered', 'methyl', 'groups', 'this', 'constrains', '14-additions', 'cis-conﬁgurations', 'in', 'polar', 'solvents', 'however', 'like', 'tetrahydrofuran', 'complexes', 'loose', 'thermodynamically', 'favored', 'trans', 'additions', 'take', 'place', '198.', 'it', 'observed', 'however', 'polymerizations', '23-dimethylbutadiene', 'organolithium', 'initiators', 'non-polar', 'solvents', 'result', 'high', 'trans-14', 'structures', '199.', 'this', 'appears', 'contradict', 'above-proposed', 'mechanism', 'proton', 'nmr', 'spectra', 'show', 'solvation', 'shifts', 'structures', 'carbanionic', 'chain', 'ends', 'localized', '14-species', 'delocalized', '“p-allylic”', 'type', 'structures', '200:', 'ch2', 'li', 'cis', 'ch2', 'li', 'trans', 'li', 'ch2', 'anti', 'li', 'ch2', 'syn', 'the', 's-bonded', 'lithium', 'chain', 'expected', 'predominate', 'in', 'highly', 'solvating', 'solvents', 'ethers', 'p-allyl', 'structure', 'dominant', 'leading', 'high', '12', 'placements', 'because', '23-bond', 'maintained', 'shown', 'equilibrium', 'expected', 'lead', 'cis–trans', 'isomerization', '200.', 'in', 'fact', 'isomerizations', 'take', 'place', 'butadiene', 'isoprene', 'polymerized', 'hydrocarbon', 'solvents', 'they', 'occur', 'however', 'polar', 'solvents', 'high', 'temperatures', 'this', 'suggests', 'additional', 'equilibrium', 'exist', 'p-allylic', 'structures', 'covalent', '12', 'chain', 'ends', '200.', 'table', '43', 'shows', 'manner', 'different', 'polymerization', 'initiators', 'solvents', 'affect', 'microstructures', 'polyisoprene', '194', '4', 'ionic', 'chain-growth', 'polymerization', 'table', '43', 'effect', 'counterion', 'solvent', 'microstructure', 'polyisoprenea', 'approximate', 'adduct', 'initiator', 'lithium', 'dispersion', 'ethyllithium', 'butyllithium', 'sodium', 'dispersion', 'ethylsodium', 'butylsodium', 'potassium', 'dispersion', 'ethylpotassium', 'butylpotassium', 'rubidium', 'dispersion', 'cesium', 'dispersion', 'ethyllithium', 'ethylsodium', 'lithium', 'dispersion', 'afrom', 'various', 'sources', 'literature', 'the', 'alkane', 'low', 'boiling', 'aliphatic', 'hydrocarbon', 'trans-14', '1', '–', '–', '43', '42', '35', '52', '39', '41', '47', '51', '30', '14', '27', 'solventa', 'alkane', 'alkane', 'alkane', 'alkane', 'alkane', 'alkane', 'alkane', 'alkane', 'alkane', 'alkene', 'ether', 'ether', 'ether', 'cis-14', '93', '94', '93', '–', '6', '4', '–', '24', '20', '5', '4', '6', '–', '3', '12', '–', '–', '–', '6', '7', '7', '8', '6', '6', '9', '8', '5', '10', '6', '34', '5–6', '6', '7', '51', '45', '54', '40', '31', '33', '39', '37', '59', '76', '64', 'polymerizations', 'polar', 'monomers', 'like', 'acrylic', 'methacrylic', 'esters', 'alkyllithium', 'initiators', 'yield', 'greatest', 'amount', 'steric', 'control', '151.', 'an', 'almost', 'isotactic', 'poly(methyl', 'methacrylate)', 'forms', 'low', 'temperatures', 'addition', 'lewis', 'bases', 'ethers', 'amines', 'reduces', 'degree', 'isotactic', 'placement', 'depending', 'upon', 'temperature', 'atactic', 'syndiotactic', 'polymers', 'form', '151.', 'also', 'butyllithium', 'heptane', 'yields', 'isotactic', 'poly(nn0-dibutyla-', 'crylamide)', 'room', 'temperature', '201.', 'the', 'propagation', 'rates', 'methyl', 'methacrylate', 'polymerization', 'polar', 'solvents', 'like', 'tetrahydrofu-', 'ran', 'dimethylformamide', 'lower', 'rates', 'initiation', '203.', 'there', 'evidence', 'however', 'one', 'kind', 'ion', 'pairs', 'exist', '204–206.', 'the', 'ion', 'pairs', 'form', 'apparently', 'contact-', 'ion', 'pairs', '203.', 'furthermore', 'based', 'evidence', 'counterions', 'coordinated', 'enolate', 'oxygen', 'atoms', 'carbonyl', 'groups', 'a-carbons', 'as', 'result', 'exert', 'less', 'inﬂuence', 'reactivity', 'carbanions', '203.', 'the', 'amount', 'solvation', 'solvents', 'affects', 'reaction', 'rates', 'in', 'addition', '“intramolecular', 'solvation”', 'neighboring', 'ester', 'groups', 'polymer', 'chains', 'also', 'affects', 'rates', 'in', 'solvents', 'like', 'dimethylformamide', 'tetrahydrofuran', 'similar', 'ones', '203,', 'propagating', 'chain', 'ends–ion', 'pairs', 'pictured', 'hybrid', 'intermediates', 'two', 'extreme', 'structures', 'this', 'depends', 'upon', 'counterion', 'solvent', 'temperature', '203:', 'o', 's', 'me', 'o', 's', 'o', 's', 'me', 'o', 's', 's', 's', 'means', 'solvent', 'molecule', 'me', 'represents', 'metal', 'several', 'mechanisms', 'offered', 'explain', 'steric', 'control', 'polymerizations', 'polar', 'monomers', 'furukawa', 'coworkers', '157', 'based', 'mechanism', 'infra-red', 'spectroscopy', 'data', 'interactions', 'cations', 'growing', 'polymeric', 'chains', 'polymerizations', 'methyl', 'methacrylate', 'methacrylonitrile', 'they', 'observed', 'correlation', 'tacticities', 'growing', 'molecules', 'carbonyl', 'stretching', 'frequencies', 'the', 'higher', 'frequency', 'higher', 'amount', 'isotactic', '44', 'anionic', 'polymerization', 'oleﬁns', '195', 'placement', 'resultant', 'chains', 'the', 'adducts', 'initiation', 'reactions', 'resonance', 'hybrids', 'two', 'structures', 'a', 'b', 'me', 'o', 'o', 'b', 'me', 'o', 'o', 'a', 'li', 'o', 'me', 'represents', 'metal', 'furukawa', 'coworkers', 'concluded', 'due', 'extremely', 'low', 'tendency', 'adducts', 'dissociate', '157,', 'carbonyl', 'absorption', 'ascribed', 'undissociated', 'ion', 'pairs', 'the', 'magnitude', 'carbonyl', 'absorption', 'shifts', 'higher', 'frequencies', 'show', 'degree', 'contribution', 'structure', 'b', 'shown', 'the', 'absorption', 'shifts', 'also', 'explained', 'conﬁgurations', 'electrons', 'anti-bonding', 'orbitals', 'carbonyl', 'groups', 'the', 'higher', 'stretching', 'frequencies', 'nearer', 'positions', 'counterions', 'carbonyl', 'groups', 'terminal', 'units', '207.', 'this', 'accompanied', 'higher', 'tacticity', '157.', 'the', 'carbanions', 'terminal', 'units', 'transition', 'states', 'located', 'near', 'b-carbons', 'incoming', 'monomers', 'at', 'time', 'rotations', 'around', 'axis', 'two', 'carbons', 'may', 'quite', 'restricted', 'cations', 'vicinity', 'carbonyl', 'groups', 'terminal', 'unit', 'near', 'incoming', 'monomers', 'in', 'manner', 'isotactic', 'placement', 'enhanced', '157:', 'li', 'ro', 'or', 'o', 'o', 'the', 'mechanism', 'proposed', 'polymerization', 'methacrylonitrile', '157.', 'cram', 'kopecky', '219', 'offered', 'different', 'mechanism', 'steric', 'control', 'according', 'mechanism', 'methyl', 'methacrylate', 'polymerization', 'growing', 'ether', 'enolate', 'possesses', 'complete', 'alkoxide', 'character', 'o', 'o', 'o', 'o', 'o', 'attacks', 'alkoxide', 'ion', 'carbonyl', 'groups', 'penultimate', 'units', 'lead', 'formations', 'six-membered', 'rings', 'li', 'o', 'o', 'li', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'li', '196', '4', 'ionic', 'chain-growth', 'polymerization', 'the', 'six-membered', 'ring', 'destroyed', 'process', 'propagation', 'o', 'o', 'o', 'o', 'li', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'li', 'the', 'transition', 'state', 'formed', '14-dipolar', 'addition', 'polarized', 'double', 'bond', 'coordination', 'lithium', 'atom', 'two', 'oxygen', 'atoms', 'determines', 'stereoregulation', 'each', 'new', 'incoming', 'monomer', 'must', 'approach', 'plane', 'side', 'blocked', 'axial', 'methyl', 'group', 'this', 'favors', 'isotactic', 'placement', 'there', 'doubt', 'however', 'whether', 'correct', 'assume', 'rigid', 'six-', 'membered', 'cyclic', 'alkoxide', 'structure', 'propagating', 'lithium', 'enolate', '209.', 'a', 'slightly', 'similar', 'model', 'suggested', 'bawn', 'ledwith', '209.', 'it', 'based', 'probability', 'growing', 'polymeric', 'alkyl', 'lithium', 'enolic', 'character', 'lithium', 'coordinating', 'carbonyl', 'oxygen', 'penultimate', 'unit', 'o', 'o', 'o', 'o', 'o', 'o', 'li', 'o', 'o', 'o', 'o', 'o', 'o', 'li', 'the', 'cyclic', 'intermediate', 'forms', 'due', 'intramolecular', 'solvation', 'lithium', 'due', 'intramo-', 'lecular', 'shielding', 'one', 'side', 'lithium', 'ion', 'the', 'nucleophilic', 'attack', 'monomer', 'therefore', 'occur', 'opposite', 'side', 'the', 'transition', 'state', 'similar', 'sn2', 'reaction', 'when', 'bond', 'lithium', 'incoming', 'monomer', 'forms', 'oxygen', 'lithium', 'bond', 'ruptures', 'simulta-', 'neously', 'charge', 'migrates', 'methylene', 'group', 'newly', 'added', 'monomer', 'the', 'resultant', 'new', 'molecule', 'stabilized', 'immediately', 'intramolecular', 'solvation', 'in', 'manner', 'retention', 'conﬁguration', 'assured', 'incoming', 'monomer', 'always', 'assumes', 'conﬁgura-', 'tion', 'toward', 'lithium', 'ion', 'nmr', 'spectra', 'poly', '(nn-dimethylacrylamide)', 'formed', 'sec-butyllithium', 'polar', 'non-polar', 'solvents', 'show', 'penultimate', 'unit', 'affect', 'monomer', 'placement', '210.', 'also', 'coordination', 'observed', 'heteroatoms', '152', '153,', 'one', 'ultimate', 'one', 'penultimate', 'unit', 'many', 'reﬁnements', 'introduced', 'various', 'proposed', 'explanations', 'steric', 'control', 'anionic', 'polymerizations', '211', 'last', '30', 'years', 'two', 'important', 'features', 'mechanisms', '(1)', 'coordinations', 'chain', 'ends', 'counterions', '(2)', 'counterion', 'solvation', 'use', 'complex', 'lanthanide', 'catalysts', 'allows', 'high', 'cis-14', 'placement', 'isoprene', 'monomer', 'preparation', 'polymers', 'close', 'natural', 'rubber', '212.', 'thus', 'complex', 'neodymium', 'catalysts', 'yield', 'polymers', 'greater', '98', 'cis-14', 'polyisoprenes', 'the', 'preparation', 'catalyst', 'however', 'difﬁcult', 'evans', 'et', 'al', 'reported', 'however', 'simple', 'tmi2', 'ndi2', 'dyi2', 'initiate', 'polymerization', 'isoprene', 'without', 'additives', 'also', 'yield', 'high', 'cis-14', 'placement', '213:', '44', 'anionic', 'polymerization', 'oleﬁns', '197', 'lai2', '14-polyisoprene', 'n', '4422', 'hydrogen', 'transfer', 'polymerization', 'anionic', 'polymerization', 'unsubstituted', 'acrylamide', 'catalyzed', 'strong', 'bases', 'yield', 'typical', 'vinyl', 'polymers', 'instead', 'product', '13-adduct', 'poly(b-alanine)', '214.', 'two', 'alternate', 'reaction', 'paths', 'originally', 'proposed', '214.', 'these', 'shown', 'follows', '1', 'b', 'o', 'nh2', 'bh', 'o', 'nh', '2', 'o', 'nh2', 'b', 'b', 'ch', 'b', 'o', 'nh2', 'o', 'nh', 'subsequently', 'studies', 'rate', 'disappearance', 'acrylamide', 'dry', 'sulfolane', 'pyridine', 'potassium-t-butoxide', 'initiator', 'led', 'following', 'proposed', 'mechanism', '215:', 'b', 'ch', 'o', 'nh2', 'o', 'nh2', 'b', 'nh2', 'o', 'o', 'nh', 'propagation', 'proceeds', 'manner', '215:', 'o', 'nh', 'o', 'nh2', 'o', 'nh2', 'o', 'n', 'h', 'o', 'nh', 'o', 'n', 'h', 'o', 'nh2', 'o', 'ch', 'nh2', 'o', 'n', 'h', 'c', 'h', 'o', 'n', 'h', 'o', 'nh2', 'etc', 'the', 'propagating', 'center', 'neither', 'ion', 'radical', 'carbon', 'carbon', 'double', 'bond', 'end', 'chain', 'the', 'monomer', 'anion', 'adds', 'double', 'bond', 'this', 'process', 'step-growth', 'polymerization', 'monomer', 'anion', 'called', 'activated', 'monomer', 'not', 'acrylamide', 'polymerizations', 'initiated', 'strong', 'bases', 'however', 'proceed', 'hydrogen', 'transfer', 'process', 'depending', 'upon', 'reaction', 'conditions', 'solvent', 'monomer', 'concentration', 'temperature', 'polymerizations', 'take', 'place', 'carbon', 'carbon', 'double', 'bonds', '216.', '198', '4', 'ionic', 'chain-growth', 'polymerization', 'cis', 'trans', 'crotonamides', 'also', 'polymerize', 'hydrogen', 'transfer', 'polymerization', 'sodium', 't-butoxide', 'pyridine', 'yields', 'identical', 'polymers', 'isomers', '217.', 'also', 'hydrogen', 'transfer', 'polymerization', 'acrylamide', 'optically', 'active', 'basic', 'catalysts', 'yields', 'optically', 'active', 'polymers', '218.', 'the', 'reactions', 'carried', 'toluene', 'using', 'optically', 'active', 'alcoholates', 'amyl', 'alcohol', 'the', 'initiating', 'ability', 'metal', 'ions', 'following', 'order', 'na', 'ba', 'ca', 'mg', 'al', '218.', 'optically', 'active', 'polymethacrylamide', 'forms', 'optically', 'active', 'barium', 'calcium', 'alcoholates', 'cations', '218.', 'in', 'reaction', 'however', 'asymmetric', 'synthesis', 'takes', 'place', 'intermo-', 'lecular', 'hydrogen', 'transfer', 'rather', 'intramolecular', 'hydrogen', 'migration', '218:', 'me', 'or', 'h2n', 'o', 'o', 'nh2', 'o', 'h', 'n', 'h', 'me', 'or', 'h2n', 'o', 'o', 'h2n', 'o', 'me', 'or', 'n', 'h', '443', 'termination', 'anionic', 'polymerization', 'termination', 'reactions', 'anionic', 'polymerization', 'particularly', 'non-polar', 'monomers', 'non-', 'polar', 'solvent', 'common', 'if', 'carbanion', 'quenching', 'impurities', 'absent', 'many', 'polymerization', 'reactions', 'may', 'terminate', 'complete', 'disappearance', 'monomer', 'styryl', 'anion', '(one', 'stable', 'ones)', 'instance', 'persist', 'long', 'time', 'weeks', 'monomer', 'consumed', 'addition', 'monomer', 'results', 'continuation', 'reaction', 'increase', 'molecular', 'weight', 'the', 'anionic', '“living”', 'polymers', 'retain', 'activities', 'considerably', 'longer', 'periods', 'time', 'cationic', '“living”', 'ones', '180.', 'the', 'termination', 'steps', 'anionic', 'polymerizations', 'result', 'deliberate', 'introductions', 'carbanion', 'quenchers', 'water', 'acids', 'impurities', 'terminations', 'however', 'take', 'place', 'instances', 'chain', 'transferring', 'proton', 'another', 'molecule', 'like', 'solvent', 'monomer', 'even', 'molecule', 'another', 'polymer', 'in', 'solvents', 'like', 'liquid', 'ammonia', 'transfer', 'solvent', 'extensive', 'styrene', 'polymerization', 'amide', 'ions', '219.', 'in', 'addition', 'polymerizations', 'termination', 'might', 'occur', 'following', 'reactions', '1', 'elimination', 'hydride-ion', 'form', 'unsaturated', 'end', '2', 'isomerization', 'inactive', 'anion', '3', 'some', 'irreversible', 'reaction', 'active', 'center', 'molecule', 'monomer', 'solvent', 'it', 'observed', 'instance', 'hydrogen', 'transfer', 'monomer', 'growing', 'chain', 'way', 'termination', 'polymerizations', 'polar', 'monomers', 'like', 'acrylonitrile', '220:', 'n', 'me', 'n', 'n', 'n', 'me', 'me', 'represents', 'metal', 'cation', '44', 'anionic', 'polymerization', 'oleﬁns', '199', 'as', 'mentioned', 'polystyryl', 'carbanions', 'particularly', 'stable', 'persist', 'weeks', 'non-', 'polar', 'solvents', 'yet', 'even', 'absence', 'terminating', 'agents', 'concentration', 'carbanion', 'active', 'centers', 'decreases', 'time', '159.', 'the', 'mechanism', 'decay', 'fully', 'understood', 'based', 'spectral', 'evidence', 'believed', 'consist', 'hydride-ion', 'elimination', '219:', 'na', 'nah', 'the', 'reaction', 'followed', 'abstraction', 'allylic', 'hydrogen', 'product', 'elimination', 'another', 'active', 'center', 'polystyryl', 'carbanions', 'much', 'less', 'stable', 'polar', 'solvents', 'they', 'decay', 'within', 'days', 'room', 'temperature', 'at', 'lower', 'temperatures', 'however', 'stability', 'considerably', 'better', 'the', 'termination', 'polar', 'solvent', 'occurs', 'mechanism', 'abstracting', 'a-hydrogens', 'andor', 'nucleophilic', 'attack', 'carbon–oxygen', 'bonds', 'polar', 'monomers', 'like', 'methyl', 'methacrylate', 'acrylonitrile', 'methyl', 'vinyl', 'ketone', 'contain', 'substituents', 'react', 'nucleophiles', 'this', 'lead', 'terminations', 'side', 'reactions', 'compete', 'initiation', 'propagation', '219.', 'an', 'example', 'nucleophilic', 'substitution', 'intramolecular', 'backbiting', 'attack', 'propagating', 'carbanion', 'o', 'o', 'cooch3', 'cooch3', 'o', 'cooch3', 'cooch3', 'ch3o', 'side', 'reactions', 'like', 'one', 'shown', 'minimized', 'using', 'less', 'nucleophilic', 'initiators', 'low', 'temperatures', 'this', 'yield', '“living”', 'polymerizations', 'acrylic', 'methacrylic', 'monomers', 'in', 'addition', 'possible', 'add', 'common', 'ions', 'like', 'licl', 'alkyl', 'lithium', 'tighten', 'ion', 'pairs', 'propagating', 'anion-counterion', 'species', 'that', 'also', 'increases', 'tendency', 'form', '“living”', 'polymers', '221.', 'this', 'approach', 'however', 'offers', 'limited', 'success', 'in', 'addition', 'found', 'lewis', 'acid', 'assisted', 'polymerizations', 'methyl', 'methacrylate', 'aluminum', 'porphyrin', 'initiators', 'yield', '“living”', 'polymers', '222.', 'the', 'polymerizations', 'methacrylate', 'esters', 'alkylaluminum', 'porphyrin', 'initiators', 'occur', 'formations', 'enolate', 'aluminum', 'porphyrin', 'intermediate', 'growing', 'species', '222.', 'for', 'sake', 'illustration', 'methylaluminum', 'porphyrin', 'molecules', '(see', 'fig', '43)', 'designated', 'al', 'ch3', '200', '4', 'ionic', 'chain-growth', 'polymerization', 'fig', '43', 'methylaluminum', 'porphyrin', 'molecule', 'al', 'n', 'n', 'n', 'n', 'the', 'reaction', 'illustrated', 'follows', '222:', 'al', 'ch3o', 'n', 'o', 'ch3o', 'al', 'o', 'n-1', 'co2ch3', 'monomer', 'polymer', 'the', 'preparatory', 'procedure', 'improved', '222', 'addition', 'sterically', 'crowded', 'organoaluminum', 'phenolates', 'reaction', 'mixture', 'r', 'r', 'c4h9', 'c4h9', 'o', 'o', 'al', 'r', 'r', 'r0', 'h', 'r00', 'butyl', 'r0', 'butyl', 'r00', 'methyl', 'this', 'yields', '“living”', 'high', 'molecular', 'weight', 'monodisperse', 'polymers', 'methyl', 'methacrylate', '222.', 'monomer', 'insertion', 'might', 'taking', 'place', 'al–c', 'bond', 'the', 'mechanism', 'however', 'yet', 'fully', 'understood', '222.', 'ishizone', 'coworkers', '223', 'studied', 'living', 'anionic', 'polymerization', 'series', 'nn-dialkyl', 'methacrylamides', 'carried', 'diphenyl', 'methyllithium', 'triphenylmethylpotassium', 'presence', 'licl', 'diethyl', 'zinc', 'tetrahydrofuran', 'polymers', 'methacroyl', 'azetidine', 'possessed', 'predicted', 'molecular', 'weights', 'narrow', 'molecular', 'weight', 'distributions', '11', 'they', 'obtained', 'quantitatively', 'catalysts', '(cid1)40', '0(cid5)c', 'within', '24', 'h', 'the', 'acryloyl', 'counterpart', 'n-acrytoyilazetidine', 'also', 'underwent', 'anionic', 'polymerization', 'afford', 'well-deﬁned', 'polymer', 'quantitatively', 'by', 'contrast', 'polymer', 'obtained', 'anionic', 'polymerization', 'system', 'n-methacroyl', 'piperidine', 'nn-dimethyl', 'methacrytamide', 'from', 'experimental', 'results', 'demonstrated', 'polymerizability', 'series', 'nn-dialkylmethacrytamides', 'cyclic', 'substituents', 'decreased', 'drastically', 'increasing', 'ring', 'size', '3', '6', '45', 'coordination', 'polymerization', 'oleﬁns', '201', '444', 'thermodynamics', 'anionic', 'polymerization', 'the', 'change', 'equilibrium', 'constant', 'dissociation', 'ion', 'pairs', 'free', 'ions', 'expressed', 'k', 'increases', 'decreasing', 'temperatures', 'dh', 'negative', 'the', 'rate', 'constant', 'composite', 'rate', 'constant', 'ion', 'pairs', 'rate', 'constant', 'solvent', 'separated', 'ion', 'pairs', 'it', 'expressed', 'kc', 'rate', 'constant', 'contact', 'ion', 'pair', 'ks', 'constant', 'solvent', 'separated', 'ion', 'pair', 'equilibrium', 'constant', 'interconversions', 'unsolvated', 'solvated', 'ion', 'pairs', 'the', 'relationship', 'three', 'rate', 'constants', 'temperature', 'expressed', 'follows', 'ln', 'k', '¼', '(cid1)dhrt', 'þ', 'dsr', 'k(cid10)', 'p', '¼', 'ðkc', 'þ', 'kskcsþð1', 'þ', 'kcsþ', 'ln', 'kc', '¼', 'ln', 'ec', '(cid1)', 'kcrt', 'ln', 'ks', '¼', 'ln', 'es', '(cid1)', 'ksrt', 'ln', 'kcs', '¼', 'ln', 'ecs', '(cid1)', 'kcsrt', 'ec', 'es', 'ecs', 'activation', 'energies', 'sawada', '406', 'writes', 'apparent', 'activation', 'energy', 'deapp', 'follows', 'deapp', '¼', 'deð(cid10)þs', 'þ', 'dh0', 'csð1', 'þ', 'kcsþ', 'de((cid10))s', 'activation', 'energy', 'propagation', 'solvent-separated', 'ion', 'pair', 'dh0', 'enthalpy', 'change', 'formation', 'solvent-separated', 'ion', 'pair', 'contact', 'ion', 'pair', 'cs', '45', 'coordination', 'polymerization', 'oleﬁns', 'the', 'catalysts', 'polymerizations', 'separated', 'two', 'groups', 'to', 'ﬁrst', 'one', 'belong', 'so-called', 'ziegler–natta', 'catalysts', 'to', 'second', 'one', 'transition', 'metal', 'oxides', 'special', 'supports', 'like', 'carbon', 'black', 'silica-alumina', 'etc', 'besides', 'two', 'related', 'catalysts', 'like', 'transition', 'metal', 'alkyls', 'metal', 'halides', 'also', 'catalyze', 'coordinated', 'anionic', 'polymerization', 'this', 'group', 'also', 'includes', 'transition', 'metal-p-allylic', 'compounds', 'transition', 'metal', 'hydrides', 'the', 'mechanism', 'polymerization', 'generally', 'coordinated', 'anionic', 'based', 'evidence', 'date', 'the', 'ziegler–natta', 'catalysts', 'received', 'initial', 'attention', 'ziegler', 'showed', 'transition', 'metal', 'halides', 'upon', 'reaction', 'aluminum', 'alkyls', 'initiate', 'polymerizations', 'ethylene', '224.', 'polymers', 'form', 'linear', 'high', 'molecular', 'weight', 'the', 'reactions', 'require', 'much', 'lower', 'pressures', 'free-radical', 'polymerizations', 'ethylene', 'simultaneously', 'natta', 'demonstrated', '225', 'similar', 'catalysts', 'polymerize', 'various', 'oleﬁns', 'like', 'propylene', 'butylene', 'higher', 'a-oleﬁns', 'high', 'molecular', 'weight', 'linear', 'polymers', 'form', 'well', 'important', 'highly', 'stereospeciﬁc', 'ones', 'the', 'two', 'disclosures', 'stimulated', 'intensive', 'research', 'mechanism', 'catalysis', 'much', 'knowl-', 'edge', 'gained', 'date', 'some', 'uncertainties', 'exact', 'mechanisms', 'reactions', 'still', 'persist', '202', '4', 'ionic', 'chain-growth', 'polymerization', 'ziegler–natta', 'catalysts', '226', 'products', 'reactions', 'metal', 'alkyls', 'hydrides', 'groups', 'i', 'iii', 'periodic', 'table', 'metals', 'salts', 'complexes', 'groups', 'iv', 'viii', 'not', 'compounds', 'however', 'ﬁt', 'broad', 'deﬁnitions', 'actually', 'useful', 'catalysts', 'in', 'fact', 'range', 'active', 'useless', 'ones', 'also', 'among', 'found', 'materials', 'initiate', 'polymeriza-', 'tion', 'monomers', 'free-radical', 'cationic', 'anionic', 'mechanisms', 'coordinated', 'anionic', 'nevertheless', 'number', 'active', 'ziegler–natta', 'catalysts', 'still', 'large', 'the', 'catalysts', 'based', 'metals', 'large', 'numbers', 'd-electrons', '(mostly', 'group', 'viii)', 'effective', 'polymerizations', 'conjugated', 'dienes', 'they', 'don’t', 'appear', 'work', 'well', 'however', 'a-oleﬁns', 'on', 'hand', 'metals', 'groups', 'iv', 'v', 'vi', 'fewer', 'd-electrons', 'useful', 'polymerizing', 'oleﬁns', 'conjugated', 'dienes', 'also', 'catalysts', '163', 'polymerize', 'propylene', 'also', 'poly-', 'merize', 'ethylene', 'yet', 'converse', 'always', 'true', 'the', 'ziegler–natta', 'catalysts', 'sub-divided', 'two', 'groups', '(1)', 'heterogeneous', 'insoluble', 'catalysts', '(2)', 'homogeneous', 'soluble', 'ones', 'at', 'times', 'however', 'difﬁcult', 'distinguish', 'two', 'for', 'instance', 'may', 'hard', 'determine', 'whether', 'particular', 'catalyst', 'truly', 'solution', 'merely', 'form', 'ﬁne', 'colloidal', 'suspension', '(and', 'fact', 'heterogeneous)', '451', 'heterogeneous', 'ziegler–natta', 'catalysts', 'these', 'catalysts', 'form', 'soluble', 'metal', 'alkyl', 'like', 'triethylaluminum', 'diethyl', 'aluminum', 'chloride', 'combined', 'metal', 'salt', 'like', 'titanium', 'chloride', 'medium', 'inert', 'hydrocarbon', 'diluent', 'the', 'transition', 'metal', 'reduced', 'formation', 'catalyst', 'the', 'following', 'chemical', 'scheme', 'illustrates', 'reactions', 'believed', 'take', 'place', 'aluminum', 'alkyls', 'transition', 'metal', 'halides', '227', '228.', 'titanium', 'chloride', 'used', 'example', 'ticl4', 'þ', 'alr3', 'ticl3r', 'þ', 'alr2cl', 'ticl4', 'þ', 'alr2cl', 'ticl3', 'þ', 'alrcl2', 'þ', 'r(cid6)', 'ticl4', 'þ', '2alr3', 'ticl2r2', 'þ', '2alr2cl', 'ticl2r2', 'ticl2r', 'þ', 'r(cid6)', 'ticl3', 'þ', 'alr3', 'ticl2r', 'þ', 'alr2cl', 'ticl2r', 'ticl2', 'þ', 'r(cid6)', 'ticl2r', 'þ', 'alr3', 'ticlr2', 'þ', 'alr2cl', 'the', 'radicals', 'form', 'shown', 'reactions', 'probably', 'undergo', 'combinations', 'reactions', 'radicals', 'perhaps', 'react', 'solvents', 'decay', 'the', 'reduction', 'tetravalent', 'titanium', 'unlikely', 'complete', 'due', 'heterogeneous', 'nature', 'catalyst', 'better', 'catalytic', 'activity', 'results', 'ticl3', 'used', 'directly', 'place', 'ticl4', 'many', 'catalysts', 'however', 'prepared', 'ticl4', 'in', 'addition', 'ticl3', 'exists', 'four', 'different', 'crystalline', 'forms', 'referred', 'b', 'g', 'of', 'b', 'g', 'forms', 'yield', 'highly', 'stereospeciﬁc', 'polymers', 'a-oleﬁns', 'the', 'a-form', 'however', 'yields', 'polymers', 'high', 'atactic', 'material', 'the', 'ratio', 'transition', 'metal', 'compounds', 'compounds', 'metals', 'group', 'i', 'iii', 'affect', 'polymerization', 'rates', 'they', 'also', 'affect', 'molecular', 'weights', 'steric', 'arrangement', 'products', 'also', 'additives', 'like', 'lewis', 'bases', 'amines', 'electron', 'donors', '45', 'coordination', 'polymerization', 'oleﬁns', '203', 'help', 'increase', 'stereoregularity', 'product', 'thus', 'instance', 'dialkylzinc', 'plus', 'titanium', 'trichloride', 'catalyst', 'yields', 'polypropylene', '65', 'stereoregular', 'addition', 'amine', 'however', 'catalytic', 'system', 'raises', 'stereoregularity', '93', '229.', 'heterogeneous', 'catalysts', 'typically', 'form', 'polymers', 'wide', 'molecular', 'weight', 'distributions', 'many', 'mechanisms', 'proposed', 'explain', 'action', 'heterogeneous', 'ziegler–natta', 'catalysts', 'all', 'agree', 'polymerizations', 'take', 'place', 'localized', 'active', 'sites', 'catalyst', 'surfaces', 'also', 'generally', 'accepted', 'reactions', 'take', 'place', 'coordinated', 'anionic', 'mechanisms', 'the', 'organometallic', 'component', 'generally', 'believed', 'activate', 'site', 'surface', 'alkylating', 'transition', 'metal', 'some', 'controversy', 'however', 'still', 'exists', 'exact', 'mechanism', 'catalytic', 'action', 'whether', 'monometallic', 'bimetallic', 'most', 'opinion', 'leans', 'former', 'also', 'well', 'accepted', 'monomer', 'insertion', 'polymer', 'chain', 'takes', 'place', 'transition', 'metal', 'atom', 'terminal', 'carbon', 'growing', 'polymeric', 'chain', '230.', 'only', 'one', 'bimetallic', 'mechanism', 'presented', 'example', 'one', 'originally', 'proposed', 'natta', '230.', 'he', 'felt', 'chemisorptions', 'organometallic', 'compounds', 'transition', 'metal', 'halides', 'take', 'place', 'reactions', 'partially', 'reduced', 'forms', 'di-', 'tri-chlorides', 'strongly', 'electropositive', 'metals', 'small', 'ionic', 'radius', '(aluminum', 'beryllium', 'magnesium)', 'facilitate', 'these', 'chemisorptions', 'result', 'formations', 'electron-deﬁcient', 'complexes', 'two', 'metals', 'such', 'complexes', 'contain', 'alkyl', 'bridges', 'similar', 'present', 'dimeric', 'aluminum', 'beryllium', 'alkyls', '231.', 'the', 'polymeric', 'growth', 'takes', 'place', 'aluminum–carbon', 'bond', 'bimetallic', 'electron-deﬁcient', 'complexes', '232', '234:', 'ti', 'al', 'r', 'ti', 'al', 'r', 'ch2', 'ti', 'al', 'δ', 'ch2', 'ch2', 'δ', 'ch2', 'ti', 'ch2', 'ch2', 'r', 'al', 'ti', 'al', 'r', 'r', 'represents', 'ethyl', 'group', 'the', 'bulk', 'evidence', 'however', 'indicates', 'chain', 'growth', 'occurs', 'repeated', 'four-center', 'insertion', 'reactions', 'monomer', 'transition', 'metal-', 'carbon', 's-bond', '230.', 'it', 'still', 'established', 'whether', 'base', 'metal', 'alkyls', 'serve', 'produce', 'active', 'centers', 'additional', 'function', 'this', 'would', 'make', 'mechanism', 'monometallic', 'if', 'however', 'active', 'centers', 'must', 'stabilized', 'coordination', 'base', 'metals', 'mechanism', 'bimetallic', '234.', 'the', 'two', 'active', 'centers', 'depicted', 'follows', 'r', 'x', 'x', 'mt', 'x', 'x', 'r', 'mn', 'x', 'x', 'r', 'mt', 'x', 'x', 'x', 'monometallic', 'site', 'bimetallic', 'site', 'mt', 'represents', 'transition', 'metal', 'mn', 'base', 'metal', 'vacant', 'site', 'both', 'types', 'active', 'centers', 'might', 'conceivably', 'present', 'heterogeneous', 'ziegler–natta', 'catalysts', '234.', 'the', 'exact', 'locations', 'sites', 'solid', 'catalyst', 'crystals', 'still', 'debated', 'some', 'speculations', 'center', 'whether', 'located', 'whole', 'crystal', 'surface', 'edges', 'crystals', '235.', 'most', 'evidence', 'points', 'location', 'edges', 'an', 'example', 'monometallic', 'mechanisms', 'one', 'originally', 'proposed', 'cossee', 'arlman', '236', '237.', 'this', 'mechanism', 'assumes', 'reaction', '204', '4', 'ionic', 'chain-growth', 'polymerization', 'occurs', 'transition', 'metal', 'ion', 'surface', 'layer', 'metal', 'trichloride', '(or', 'perhaps', 'dichloride)', 'lattice', 'here', 'halide', 'replaced', 'alkyl', 'group', '(r)', 'the', 'adjacent', 'chloride', 'site', 'vacant', 'accommodates', 'incoming', 'monomer', 'molecule', 'using', 'titanium', 'chloride', 'illustration', 'cl', 'r', 'ti', 'cl', 'cl', 'cl', 'cl', 'r', 'ti', 'cl', 'cl', 'cl', 'cl', 'ch2', 'r', 'ti', 'cl', 'cl', 'cl', 'r', 'cl', 'cl', 'cl', 'ti', 'cl', 'four', 'center', 'intermediate', 'represents', 'vacant', 'site', 'd-orbital', 'the', 'newly', 'formed', 'transition', 'metal–alkyl', 'bond', 'becomes', 'active', 'center', 'new', 'vacant', 'site', 'forms', 'place', 'previous', 'transition', 'metal–alkyl', 'bond', 'the', 'driving', 'force', 'reactions', '236', '237', 'depends', 'p-type', 'oleﬁn', 'complexes', 'in', 'complexes', 'p-electrons', 'oleﬁns', 'overlap', 'vacant', 'd-orbitals', 'transition', 'metals', 'this', 'results', 'p-bonds', 'transitory', 'also', 'd-orbitals', 'metals', 'simultaneously', 'overlap', 'vacant', 'anti-bonding', 'orbitals', 'oleﬁns', 'this', 'decreases', 'distances', 'highest', 'ﬁlled', 'bonding', 'orbitals', 'empty', '(or', 'nearly', 'empty)', 'd-orbitals', 'in', 'situations', 'carbon–metal', 'bonds', 'transition', 'metals', 'weaken', 'alkyl', 'groups', 'migrate', 'one', 'end', 'incoming', 'oleﬁn', '238.', 'the', 'insertion', 'process', 'results', 'cis-opening', 'oleﬁnic', 'double', 'bond', '237', '239', '240.', 'the', 'scheme', 'propagation', 'might', 'also', 'pictured', 'bimetallic', 'active', 'centers', 'complexations', 'precede', 'monomer', 'insertions', 'vacant', 'octahedral', 'sites', 'followed', 'metal–carbon', 'bonds', '237.', 'when', 'transition', 'metals', 'insertion', 'reactions', 'immobilized', 'crystal', 'lattices', 'active', 'centers', 'ligands', 'expected', 'interchange', 'propagation', 'step', 'the', 'model', 'monometallic', 'mechanism', 'though', 'widely', 'accepted', 'still', 'occasionally', 'questioned', 'some', 'evidence', 'instance', 'presented', 'years', 'support', 'bimetallic', 'mechanism', '242.', 'it', 'shown', 'elimination', 'organometallic', 'portion', 'complex', 'catalyst', 'polymerization', 'propylene', 'results', 'deactivation', 'catalyst', 'by', 'contrast', 'replacement', 'initial', 'organometallic', 'compound', 'another', 'one', 'results', 'change', 'polymerization', 'rate', 'deactivation', 'catalyst', 'in', 'addition', 'monometallic', 'mechanisms', 'based', 'different', 'mode', 'monomer', 'insertion', 'also', 'proposed', 'an', 'example', 'reaction', 'mechanism', 'proposed', 'ivin', 'et', 'al', '241.', 'this', 'mechanism', 'based', 'insertion', 'mechanism', 'involving', 'a-hydrogen', 'reversible', 'shift', 'carbene', 'metallocyclobutane', 'intermediate', 'mt', 'h', 'mt', 'h', 'h', 'mt', 'mt', 'h', 'mt', 'mt', 'means', 'metal', 'the', 'stereospeciﬁcity', 'dependent', 'upon', 'relative', 'conﬁguration', 'substituted', 'carbons', 'metallocyclobutane', 'ring', 'hydrogen', 'transfer', 'metal', 'substituted', 'carbon', 'exclude', 'branching', '241.', 'the', 'following', 'evidence', 'supports', 'mechanism', '45', 'coordination', 'polymerization', 'oleﬁns', '205', '1', 'there', 'unambiguous', 'examples', 'characterized', 'metaloalkyl-oleﬁn', 'compound', 'may', 'induced', 'react', '2', 'there', 'close', 'identity', 'catalysts', 'cause', 'ziegler–natta', 'type', 'polymerizations', 'cause', 'metathesis', 'type', 'polymerization', 'via', 'carbene', 'mechanism', '(see', 'chap', '5)', 'this', 'mechanism', 'argued', 'however', 'simpliﬁcation', 'ignores', 'experimentally', 'observed', 'regiospeciﬁcity', 'may', 'propylene', 'polymerizations', '243.', 'on', 'hand', 'argued', 'correct', 'regiospeciﬁcity', 'monomer', 'accounted', 'isotactic', 'syndiotactic', 'propagation', 'different', 'energies', 'steric', 'control', 'qualitatively', 'explained', 'ivin', 'mecha-', 'nism', 'this', 'done', 'simple', 'considerations', 'different', 'distances', 'substituted', 'carbons', 'four-membered', 'ring', '243.', 'in', 'addition', 'cavallo', 'et', 'al', '244', 'reported', 'study', 'ethylene', 'polymerization', 'using', 'model', 'heterogeneous', 'ziegler–natta', 'catalyst', '(mg2ci6ti)', 'propagation', 'well', 'termination', 'reactions', 'considered', 'from', 'study', 'concluded', 'absence', 'coordinated', 'oleﬁn', 'ti–c', '(chain)', 's-bond', 'occupy', 'octahedral', 'coordination', 'position', 'oriented', 'along', 'axis', 'intermediate', 'two', 'octahedral', 'coordination', 'positions', 'available', 'the', 'propagation', 'reactions', 'occur', 'stepwise', 'fashions', 'favored', 'mechanism', 'requires', 'rearrangements', 'growing', 'chains', 'four-center', 'transition-state', 'planes', 'the', 'insertion', 'reactions', 'facilitated', 'a-agostic', 'interactions', 'the', 'favored', 'termination', 'reactions', 'b-hydrogen', 'transfers', 'monomers', 'this', 'type', 'reaction', 'favored', 'relative', 'c–h', 's-bond', 'activation', 'coordinated', 'monomer', 'well', 'b-hydrogen', 'transfer', 'metal', '244.', '4511', 'steric', 'control', 'heterogeneous', 'catalysts', 'different', 'independent', 'approaches', 'used', 'investigate', 'mechanism', 'polymerization', 'a-oleﬁns', 'heterogeneous', 'catalysts', 'as', 'result', 'shown', 'isotactic', 'polymerization', 'racemic', 'mixtures', 'a-oleﬁns', 'stereoselective', '245', '246.', 'also', 'optically', 'active', 'polymers', 'form', 'optically', 'active', 'catalysts', '247.', 'stereoelectivity', 'stereoselectivity', 'due', 'intrinsic', 'asymmetry', 'catalytic', 'centers', '248.', 'this', 'conclusion', 'comes', 'part', 'knowledge', 'propylene', 'coordinates', 'asymmetrically', 'platinum', 'complexes', '248.', 'a', 'study', 'chemical', 'conﬁgurational', 'sequences', 'supports', '249.', 'in', 'addition', 'polymerization', 'racemic', 'mixture', '(r)(s)-3-methyl-1-pentene', 'ordinary', 'ziegler–natta', 'catalysts', '(eg,', 'al(c2h5)3', 'ticl4', 'al', '(c2h5)2cl', 'ticl3)', 'yields', 'racemic', 'mixture', 'isotactic', 'polymers', 'the', 'mixture', 'separated', 'column', 'chromatography', 'pure', 'optically', 'active', 'components', 'these', 'polymeric', 'chains', 'exclusively', 'either', 'poly(s)-3-methyl-1-pentene', 'poly(r)-3-methyl-1-pentene', '248', '250', '251.', 'this', 'means', 'typical', 'ziegler–natta', 'catalysts', 'essentially', 'two', 'types', 'active', 'sites', 'differ', 'chiral', 'sense', 'such', 'sites', 'polymerize', 'monomers', 'stereoselectively', 'ie,', 'either', '(r)', '(s)', 'exclusion', 'enantiomeric', 'form', 'r', '(s)', 'r', '(s)', 'r', '(s)', 'r', '(r)', 'r', '(r)', 'r', '(r)', 'ticl4', 'alr3', 'r', '(s)', 'r', '(r)', 'r', 'c2h5', 'h', '206', '4', 'ionic', 'chain-growth', 'polymerization', 'fig', '44', 'two', 'enantiomorphs', 'form', 'catalyst', 'r', 'indicates', 'ligands', 'replaced', 'alkyl', 'groups', 'alkylation', 'organometallic', 'compounds', 'groups', 'i', 'iii', 'cl', 'ti', 'ti', 'cl', 'cl', 'ti', 'cl', 'cl', 'ti', 'cl', 'r', 'ti', 'cl', 'r', 'cl', 'ti', 'in', 'heterogeneous', 'catalyst', 'like', 'a-ticl3', 'crystals', 'made', 'elementary', 'sheets', 'alternating', 'titanium', 'chlorine', 'atoms', 'these', 'atoms', 'aligned', 'along', 'principal', 'crystal', 'axis', 'the', 'chlorines', 'hexagonally', 'packed', 'titanium', 'atoms', 'octahedral', 'interstices', 'chlorine', 'lattice', 'every', 'third', 'titanium', 'lattice', 'missing', 'there', 'vacancy', 'pairs', 'titanium', 'atoms', 'many', 'ligand', 'vacancies', 'present', 'crystals', 'order', 'accommodate', 'electrical', 'neutrality', 'titanium', 'atoms', 'surface', 'bonded', 'ﬁve', 'chlorines', 'instead', 'six', 'neighboring', 'transition', 'metal', 'atoms', 'bridged', 'chlorines', 'opposite', 'chiralities', '57.', 'this', 'means', 'two', 'enantiomorphic', 'forms', 'exist', '(see', 'fig', '44)', 'the', 'monomers', 'coordinate', 'either', 'one', 'two', 'faces', '(at', 'vacant', 'sites)', 'coordination', 'results', 'formation', 'one', 'two', 'diastereoisomeric', 'intermediate', 'transition', 'states', 'both', 'result', 'isotactic', 'placements', 'products', 'either', 'meso', 'racemic', '240.', 'in', 'addition', 'enantioselectivity', 'two', 'faces', 'crystals', 'requires', 'minimum', 'amount', 'steric', 'bulk', 'active', 'site', 'this', 'enhanced', 'larger-sized', 'monomers', '240.', 'a', 'schematic', 'representation', 'catalytic', 'center', 'showing', 'chirality', 'asterisk', 'follows', '252:', 'h', 'h', 'mt', 'h', 'the', 'means', 'active', 'sites', 'act', 'templates', 'molds', 'successive', 'orientations', 'monomers', 'the', 'monomers', 'forced', 'approach', 'site', 'face', 'this', 'sort', 'monomer', 'placement', 'called', 'enantiomorphic', 'site', 'control', 'catalyst', 'site', 'control', 'so', 'far', 'evidence', 'initial', 'complex', 'formation', 'catalyst', 'oleﬁn', 'strong', 'the', 'working', 'hypothesis', 'commonly', 'accepted', 'today', 'follows', 'the', 'position', 'p-complex', 'diastereoisomeric', 'rotameric', 'equilibria', 'andor', 'activation', 'energy', 'insertion', 'reaction', 'cause', 'large', 'regioselectivity', 'that', 'also', 'account', 'enantioface', 'discrimination', 'necessary', 'synthesis', 'stereoregular', 'poly-a-oleﬁns', '252.', '13c', 'nmr', 'analyses', 'isotactic', 'polypropylenes', 'formed', 'd-ticl3–al(13ch3)2i', 'catalyzed', 'reactions', 'show', 'enriched', '13c', 'located', 'isopropyl', 'end', 'group', 'also', 'located', 'predominantly', 'threo', 'position', 'relative', 'methyl', 'group', 'penultimate', 'unit', '253.', 'this', 'supports', 'concept', 'steric', 'control', 'must', 'come', 'chirality', 'catalytic', 'centers', '253.', 'it', 'might', 'interesting', 'note', 'proponents', 'carbene', 'mechanism', '(mentioned', 'earlier)', 'point', 'also', 'consistent', 'mechanism', '254', '255.', 'the', 'reaction', 'consist', '(a)', 'insertion', 'metal', 'a-ch', 'bond', 'metal', 'alkyl', 'form', 'metal–carbene', 'hydride', 'complex', 'this', 'followed', '(b)', 'reaction', 'metal-carbene', 'unit', 'alkene', 'form', 'metal-cyclobutane-', 'hydride', 'intermediate', 'the', 'ﬁnal', 'step', '(c)', 'reductive', 'elimination', 'hydride', 'alkyl', 'groups', 'produce', 'chain-lengthened', 'metal', 'alkyls', 'this', 'assures', 'chiral', 'metal', 'environment', 'maintained', '254.', 'it', 'generally', 'believed', '258,', 'however', 'stereospeciﬁc', 'propagation', 'comes', 'concerted', 'multicentered', 'reactions', 'shown', 'cossee–arlman', 'mechanism', 'the', 'initiator', 'coordinated', '45', 'coordination', 'polymerization', 'oleﬁns', '207', 'propagating', 'chain', 'end', 'incoming', 'monomer', 'coordination', 'holds', 'monomer', 'place', 'process', 'addition', 'chain', 'this', 'coordination', 'broken', 'simultaneously', 'formation', 'new', 'coordination', 'new', 'monomer', '258:', 'h', 'h', 'h', 'h', 'mt', 'h', 'h', 'h', 'h', 'h', 'mt', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'mt', 'mt', 'means', 'metal', 'this', 'capability', 'initiator', 'control', 'placement', 'monomer', 'overrides', 'common', 'tendency', 'syndiotactic', 'placement', 'while', 'syndiotactic', 'polypropylene', 'prepared', 'heterogeneous', 'catalysts', 'yield', 'syndiotactic', 'placement', 'low', 'soluble', 'ziegler–natta', 'catalysts', 'hand', 'yield', 'high', 'amounts', 'syndiotactic', 'placement', 'this', 'discussed', 'next', 'section', 'when', '12-disubstituted', 'oleﬁns', 'polymerized', 'ziegler–natta', 'catalysts', 'di', 'tacticity', 'products', 'depends', 'mode', 'addition', 'it', 'also', 'depends', 'structure', 'monomer', 'whether', 'cis', 'trans', 'a', 'threodiisotactic', 'structure', 'results', 'syn', 'addition', 'trans', 'monomer', 'a', 'syn', 'addition', 'cis', 'monomer', 'results', 'formation', 'erythrodiisotactic', 'polymer', 'for', 'instance', 'cis', 'trans-1-d-propylenes', 'give', 'erythro', 'threo', 'diisotactic', 'polymers', 'respectively', '255.', 'to', 'avoid', '12-interactions', 'fully', 'eclipsed', 'conformation', 'carbon', 'bonds', 'monomer', 'units', 'rotate', 'addition', 'monomer', 'polymeric', 'chain', '240.', 'also', 'systematic', 'investigation', 'chain', 'propagation', 'ethylene', 'insertion', 'metal–c6h5', 'bond', 'showed', 'backside', 'insertion', 'barriers', 'depend', 'little', 'identity', 'metal', 'backside', 'insertion', 'requires', 'little', 'deformation', 'metal–ligand', 'framework', '256.', 'for', 'systems', 'sterically', 'unencumbered', 'insertion', 'reaction', 'proceeds', 'frontside', 'backside', 'channels', 'equal', 'parts', 'this', 'due', 'fact', 'transition', 'states', 'close', 'energy', 'the', 'inﬂuence', 'ligand', 'upon', 'insertion', 'good', 'p', 'donor', 'ligands', 'lower', 'front-side', 'insertion', 'barrier', 'favor', 'trigonal', 'planar', 'geometry', 'trigonal', 'pyramidal', 'coordination', 'the', 'activity', 'various', 'metal', 'centers', 'strongly', 'inﬂuenced', 'however', 'sterically', 'bulky', 'ligands', 'insertion', 'barriers', 'generally', 'lowered', 'steric', 'bulk', 'compression', 'active', 'site', 'favors', 'transition', 'state', 'geometry', 'p-complex', 'geometry', '256.', '452', 'homogeneous', 'ziegler–natta', 'catalysts', 'the', 'catalysts', 'ﬁt', 'category', 'quite', 'diverse', 'chemically', 'they', 'instance', 'reaction', 'products', 'vanadium', 'tetrachloride', 'dialkyl', 'aluminum', 'chloride', 'reaction', 'products', 'bis-cyclopentadienyl', 'titanium', 'dichloride', 'aluminum', 'alkyls', 'organic', 'titanates', 'also', 'form', 'soluble', 'catalysts', 'upon', 'reactions', 'certain', 'aluminum', 'alkyls', 'the', 'homogeneous', 'catalysts', 'range', 'efﬁciency', 'high', 'low', 'usually', 'much', 'narrower', 'molecular', 'weight', 'distribution', 'obtained', 'polymers', 'formed', 'catalysts', 'heterogeneous', 'ones', 'in', 'addition', 'syndiotactic', 'poly(a-oleﬁns)', 'form', 'low', 'temperatures', '((cid1)78(cid5)c)', '208', '4', 'ionic', 'chain-growth', 'polymerization', 'the', 'amount', 'stereospeciﬁc', 'placement', 'however', 'usually', 'great', 'isotactic', 'placement', 'heterogeneous', 'catalysts', 'group', 'i–iii', 'metal', 'alkyl', 'components', 'essential', 'formation', 'active', 'centers', '239', '240.', 'blue', 'complexes', 'di', 'cyclopentadienyl', 'titanium', 'dichloride', 'aluminum', 'diethyl', 'chloride', 'aluminum', 'ethyl', 'dichloride', 'early', 'known', 'soluble', 'catalysts', '257', '258:', 'cl', 'r', 'ti', 'al', 'cl', 'r', 'many', 'soluble', 'catalysts', 'developed', 'last', '30', 'years', 'most', 'materials', 'must', 'prepared', 'used', 'low', 'temperatures', '259.', 'the', 'polymerization', 'mechanism', 'also', 'believed', 'take', 'place', 'initial', 'coordination', 'monomers', 'transition', 'metals', 'this', 'followed', 'four-center', 'insertion', 'reaction', 'monomer', 'transition', 'metal–carbon', 'bond', '4521', 'steric', 'control', 'homogeneous', 'catalysts', 'stereospeciﬁc', 'placement', 'appears', 'result', 'steric', 'interactions', 'substituents', 'ultimate', 'units', 'growing', 'chains', 'incoming', 'monomers', '239.', 'nmr', 'spectral', 'evidence', 'shows', 'double', 'bond', 'opening', 'also', 'cis', 'heterogeneous', 'catalysts', '240', '258–262.', 'the', 'monomer–polymer', 'interaction', 'model', 'described', 'requires', 'sufﬁcient', 'steric', 'hindrance', 'active', 'centers', 'prevent', 'rotation', 'penultimate', 'units', 'such', 'rotation', 'would', 'cause', 'substituent', 'interfere', 'incoming', 'monomer', 'the', 'fact', 'syndiotactic', 'placement', 'takes', 'place', 'low', 'temperatures', 'atactic', 'placement', 'elevated', 'ones', 'supports', 'concept', 'following', 'illustration', 'structure', 'catalyst', 'active', 'center', '258:', 'r', 'al', 'v', 'al', 'r', 'cl', 'cl', 'cl', 'cl', 'denotes', 'vacant', 'site', 'a', 'proposed', 'model', 'syndiotactic', 'placement', 'shown', 'follows', '234:', 'h', 'v', 'h', 'v', 'h', 'h', 'v', 'h', 'v', 'h', 'h', 'h', 'the', 'model', 'shows', 'insertion', 'monomers', 's-metal', 'bonds', '239', '240.', 'nmr', 'spectroscopy', 'supports', 'hypothesis', '243.', 'also', 'model', 'shows', 'vanadium', 'penta-', 'coordinated', 'following', 'zambelli', 'allegra', 'suggestion', '243.', 'hexacoordinated', 'vanadium', '45', 'coordination', 'polymerization', 'oleﬁns', '209', 'pictured', 'others', '259', '260.', 'in', 'model', 'chain', 'ends', 'control', 'monomer', 'placement', 'this', 'differs', 'signiﬁcantly', 'mechanism', 'isotactic', 'placement', 'catalytic', 'site', 'control', 'heterogeneous', 'catalysts', 'the', 'active', 'sites', 'homogeneous', 'catalyst', 'discriminate', 'faces', 'incoming', 'monomers', 'step', 'coordination', 'instead', 'steric', 'hindrance', 'substituents', 'terminal', 'units', 'propagating', 'chains', 'ligands', 'attached', 'transition', 'metals', 'prevent', 'rotations', 'metal–carbon', 'bonds', 'in', 'addition', 'steric', 'repulsion', 'forces', 'monomers', 'coordinate', 'opposite', 'faces', 'successive', 'step', 'propagation', 'resulting', 'syndiotactic', 'placement', '453', 'steric', 'control', 'polymerization', 'conjugated', 'dienes', 'the', 'subject', 'also', 'received', 'considerable', 'attention', 'nevertheless', 'mechanism', 'still', 'fully', 'understood', 'it', 'reasonable', 'assume', 'form', 'structure', 'catalyst', 'valence', 'transition', 'metals', 'must', 'play', 'role', 'the', 'conformation', 'monomer', '(s-cis', 's-trans)', 'probably', 'also', 'important', 'it', 'known', 'cocl2al(c2h5)cl', 'polymerize', 'trans-13-pentadiene', 'cis', 'isomer', '263.', 'this', 'suggests', 'two-point', 'coordination', 'required', 'several', 'reaction', 'schemes', 'provide', 'attack', 'either', 'c1', 'c4', 'positions', 'proposed', 'years', '263.', 'one', 'mechanism', 'polymerization', 'butadiene', 'suggests', 'complexes', 'catalysts', 'solvents', 'low', 'dielectric', 'constant', 'either', 'act', 'ion', 'pairs', 'independent', 'solvated', 'entities', 'also', 'growing', 'chain', 'may', 'bound', 'either', 'p', 'linkage', 'suspected', 'continuous', 'p', 'isomerism', 'possible', '264:', 'r', 'mt', 'x', 'c4h6', 'mt', 'x', 'c4h6', 'mt', 'x', 'r', 'mt', 'x', 'polymer', 'mt', 'means', 'metal', 'soluble', 'catalysts', 'transition', 'metal', 'acetoneacetonates', '(from', 'nickel', 'titanium)', 'combined', 'triethylaluminum', 'triethylaluminum', 'chloride', 'yield', 'cis', 'polymers', 'dienes', 'the', 'cis', 'placement', 'decrease', 'bases', 'added', '265.', 'this', 'decrease', 'proportional', 'base', 'strength', 'the', 'addition', 'believed', 'decrease', 'electron', 'densities', 'orbitals', 'transition', 'metals', '266.', 'this', 'suggests', 'electrostatic', 'interactions', 'nearly', 'non-bonding', 'electrons', 'transition', 'metals', 'dienes', 'growing', 'chain', 'ends', 'must', 'play', 'important', 'role', 'such', 'interactions', 'must', 'affect', 'placements', 'incoming', 'monomers', 'hirai', 'et', 'al', 'studied', 'esr', 'signals', 'obtained', 'polymerizations', 'butadiene', '13-pentadiene', 'isoprene', '267', 'catalyst', 'based', 'n-butyltitanatetriethylaluminum', 'they', 'concluded', 'catalyst', 'must', 'possess', 'two', 'substituted', 'p-allyl', 'groups', 'one', 'alkoxy', 'group', 'chain', 'growing', 'process', '267:', '210', '4', 'ionic', 'chain-growth', 'polymerization', 'c2h5', 'n', 'c4h6', 'o', 'ti', 'c2h5', 'n', 'the', 'insertions', 'monomers', 'believed', 'occur', 'two', 'steps', '268', '269.', 'in', 'ﬁrst', 'one', 'incoming', 'monomer', 'coordinates', 'transition', 'metal', 'this', 'results', 'formation', 'short-lived', 's-allylic', 'species', 'in', 'second', 'one', 'metal–carbon', 'bond', 'transferred', 'coordinated', 'monomer', 'formation', 'p-butenyl', 'bond', 'coordination', 'diene', 'take', 'place', 'double', 'bonds', 'depending', 'upon', 'transition', 'metal', '270', 'structure', 'diene', 'when', 'monomer', 'coordinates', 'monodentate', 'ligand', 'syn', 'complex', 'forms', 'if', 'however', 'coordinates', 'bidentate', 'ligand', 'anti', 'complex', 'results', '271.', 'in', 'syn', 'complex', 'carbons', 'one', 'four', 'chirality', 'anti', 'complex', 'opposite', 'chiralities', '268.', 'due', 'lower', 'thermodynamic', 'stability', 'anti', 'complex', 'isomerizes', 'syn', 'complex', '268.', 'if', 'allylic', 'system', 'substituent', 'second', 'carbon', 'isomerization', 'anti', 'syn', 'usually', 'occurs', 'spontaneously', 'even', 'room', 'temperature', '268.', 'transition', 'metal', 'alkyls', 'probably', 'cannot', 'classiﬁed', 'typical', 'ziegler–natta', 'catalysts', 'some', 'however', 'exhibit', 'strong', 'catalytic', 'activity', 'therefore', 'investigated', '272–275.', 'this', 'also', 'led', 'polymerization', 'mechanisms', 'involve', 'formations', 'p-allylic', 'complexes', 'conclusion', 'intermediates', 'it', 'similar', 'mechanism', 'visualized', 'ziegler–natta', 'catalysis', '276–283.', 'the', 'initial', 'formation', 'p-allylic', 'ligands', 'solvents', 'used', 'catalyst', 'preparations', 'strongly', 'inﬂuence', 'catalytic', 'activity', 'stereospeciﬁcity', 'product', '280.', 'nmr', 'studies', 'polymerizations', 'conjugated', 'dienes', 'p-crotyl-nickel', 'iodide', '281', 'showed', 'monomers', 'incorporated', 'metal-carbon', 'bonds', 'formations', 'syn-p-crotyl', 'ligands', 'the', 'syn-ligands', 'transform', 'trans-14-segments', 'next', 'crotyl', 'group', 'trans-14-segments', 'become', 'trans-14-units', 'polymers', '282.', 'in', 'summary', 'general', 'mechanisms', 'cis', 'trans', 'placement', 'coordinated', 'catalysts', 'pictured', 'follows', '280:', 'mt', 'x', 'r', 'l', 'mt', 'x', 'r', 'mt', 'x', 'r', 'r', 'mt', 'x', 'cis-12', 'placement', 'r', 'mt', 'l', 'x', 'r', '12-placement', 'trans-14-placement', 'mt', 'l', 'x', '45', 'coordination', 'polymerization', 'oleﬁns', '211', 'l', 'means', 'ligand', 'mt', 'means', 'transition', 'metal', 'it', 'possible', 'apply', 'homogeneous', 'rate', 'equation', 'ziegler–natta', 'catalysts', 'this', 'expressed', 'rp', '¼', 'kp½c(cid4)(cid3)½m(cid3)', 'c', 'represents', 'concentration', 'active', 'sites', 'moles', 'per', 'liter', 'heterogeneous', 'kinetics', 'however', 'needed', 'adsorption', 'phenomena', 'important', 'coordinated', 'anionic', 'polymerizations', 'in', 'addition', 'likely', 'excess', 'components', 'groups', 'i–iii', 'metals', 'solution', 'assumed', 'compete', 'monomer', 'active', 'sites', 'the', 'fractions', 'therefore', 'active', 'sites', 'fractions', 'sites', 'covered', 'metal', 'compounds', 'expressed', 'follows', '283:', 'fraction', 'active', 'site', 'coordinated', 'monomer', 'fraction', 'active', 'sites', 'coordinated', 'components', 'metal', 'groups', 'i–iii', 'y1', '¼', 'k1½m(cid3)ð1', 'þ', 'k2½mt(cid3)', 'þ', 'k1½m(cid3)þ', 'y2', '¼', 'k2½mt(cid3)ð1', 'þ', 'k2½mt(cid3)', 'þ', 'k1½m(cid3)þ', 'm', 'represents', 'amount', 'monomer', 'solution', 'mt', 'concentration', 'metal', 'components', 'group', 'i–iii', 'k1', 'k2', 'equilibrium', 'constants', 'adsorption', 'the', 'rate', 'polymerization', 'takes', 'place', 'trough', 'reaction', 'adsorbed', 'monomer', 'active', 'sites', 'expressed', 'rp', '¼', 'kp½c(cid4)(cid3)y1', '¼', 'kp½c(cid4)(cid3)½k1½m(cid3)ð1', 'þ', 'k2½mt(cid3)', 'þ', 'k1½m(cid3)þ(cid3)', 'if', 'competition', 'components', 'metal', 'group', 'i–iii', 'rp', '¼', 'kp½c(cid4)(cid3)y1', '¼', 'kp½c(cid4)(cid3)½k1½m(cid3)ð1', 'þ', 'k1½m(cid3)þ(cid3)', 'the', 'degree', 'polymerization', 'derived', 'dividing', 'propagation', 'rate', 'sum', 'rates', 'terminations', '(by', 'transfer)', 'follows', '1dpn', '¼', 'ktrmkp', 'þ', 'kskpkm½m(cid3)', 'þ', 'ktraka½a(cid3)kpkm½m(cid3)', 'þ', 'ktrhh(cid3)kpkm½m(cid3)', 'the', 'equation', 'written', 'assumption', 'adsorption', 'hydrogen', 'active', 'sites', '454', 'post', 'ziegler', 'natta', 'coordination', 'polymerization', 'oleﬁns', 'in', 'current', 'industrial', 'practice', 'coordinated', 'anionic', 'catalysts', 'differ', 'considerably', 'original', 'ones', 'developments', 'ziegler', 'natta', 'others', 'using', 'basic', 'chemistry', 'new', 'compounds', 'developed', 'years', 'yield', 'large', 'quantities', 'polyoleﬁns', 'small', 'amounts', 'catalysts', 'in', 'addition', 'catalysts', 'designed', 'yield', 'products', 'either', 'wide', 'narrow', 'molecular', 'weight', 'distribution', 'needed', '312.', 'the', 'multinuclear', 'oleﬁn', 'polymerization', 'catalysts', '212', '4', 'ionic', 'chain-growth', 'polymerization', 'reviewed', 'recently', '313.', 'the', 'new', 'catalysts', 'ethylene', 'polymerization', 'divided', 'three', 'groups', '312:', '1', 'products', 'reactions', 'trivalent', 'alkoxy', 'chlorides', 'transition', 'metals', 'certain', 'halogen-free', 'organoaluminum', 'compounds', 'eg,', 'triisobutylaluminum', 'such', 'catalysts', 'used', 'without', 'support', '2', 'products', 'reactions', 'magnesium', 'compounds', 'titanium', 'compounds', 'in', 'catalysts', 'transition', 'metals', 'attached', 'chemically', 'surfaces', 'solid', 'magnesium', 'compounds', 'the', 'reactions', 'take', 'place', 'halogen', 'atoms', 'titanium', 'compounds', 'hydroxyl', 'groups', 'surfaces', 'magnesium', 'hydroxide', 'mg', 'oh', 'tixn', 'mg', 'o', 'tixn1', 'hx', 'titanium', 'compounds', 'bonded', 'surface', 'oh', 'groups', 'mg(oh)2', 'mainly', 'inactive', 'the', 'active', 'sites', 'ones', 'associated', 'coordinated', 'unsaturated', 'negative', 'oxygen', 'ions', '317.', 'reactions', 'aluminum', 'alkyls', 'activate', 'catalysts', '312.', 'for', 'high', 'efﬁciency', 'special', 'carriers', 'must', 'used', 'together', 'correct', 'balance', 'reactants', 'proper', 'reaction', 'conditions', 'some', 'choice', 'combinations', '312:', 'mg(oh)cl', 'ticl4', 'mgcl2', '(cid11)', '3mg(oh)2', 'ti(or)xcly', 'mgso4', '•', '3mg(oh)2', 'ti(or)xcly', 'mg(oh)2', 'ticl4', 'highly', 'active', 'unsupported', 'catalysts', 'form', 'reactions', 'magnesium', 'alkoxides', 'tetra-', 'valent', 'titanium', 'chlorides', 'the', 'true', 'reaction', 'products', 'mgcl2', 'mgcl2-electron', 'donor', 'adducts', 'like', 'mgcl2', '(cid11)', '6c2h5oh', 'tetravalent', 'titanium', 'compounds', '3', 'products', 'reactions', 'organisilanols', 'chromium', 'trioxide', 'also', 'active', 'catalysts', 'the', 'silylchromate', 'forms', 'deposited', 'silica', 'support', 'activated', 'alkyl', 'aluminum', 'compounds', '312:', '2', 'si', 'oh', '3', 'cro3', 'o', 'cr', 'o', 'si', 'o', 'o', 'si', '3', 'silica', '3', 'suported', 'sillyl', 'chromate', 'caralyst', 'alr3', 'the', 'activity', 'performance', 'coordination', 'catalysts', 'polymerization', 'propylene', 'also', 'much', 'improved', 'titanium', 'compounds', 'also', 'supported', 'carriers', 'activated', 'reactions', 'aluminum', 'alkyls', '341.', 'the', 'patent', 'literature', 'describes', 'variety', 'inorganic', 'supports', '342.', 'the', 'common', 'ones', 'however', 'based', 'mgcl2', 'treated', 'various', 'lewis', 'bases', 'like', 'ethyl', 'benzoate', 'one', 'catalytic', 'system', 'described', 'prepared', 'treating', 'complex', 'ticl3', '•', '3c5h5n', 'diethyl', 'aluminum', 'chloride', 'presence', 'highly', 'dispersed', 'mgcl2', 'the', 'product', 'reaction', 'combined', 'triethylaluminum', '355.', 'the', 'reaction', 'electron', 'donor', 'lewis', 'base', 'mgcl2', 'two', 'step', 'exothermic', 'reaction', '343.', 'the', 'ﬁrst', 'one', 'rapid', 'adsorption', 'inorganic', 'surface', 'second', 'one', 'slower', 'one', 'formation', 'complex', 'the', 'commonly', 'used', 'lewis', 'bases', 'ethyl', 'benzoate', 'di-n-butyl', 'phthalate', 'methyl-', 'p-toluenate', 'amines', 'like', '2266-tetramethylpiperidine', 'phenols', 'also', 'used', 'a', 'common', 'practice', 'ball', 'mill', 'lewis', 'base', 'support', 'material', 'ﬁrst', 'the', 'transition', 'metal', 'component', 'added', 'mixture', 'milled', 'thoroughly', 'mixed', 'this', 'lewis', 'base', 'called', 'internal', 'lewis', 'base', 'this', 'followed', 'addition', 'group', 'i–iii', 'metal-alkyl', 'component', 'additional', 'lewis', 'base', 'the', 'base', 'added', 'second', 'time', 'may', 'different', 'one', 'one', 'used', 'original', 'milling', 'in', 'either', 'case', 'called', 'external', 'lewis', 'base', 'it', 'uncommon', 'use', 'ester', 'internal', 'base', 'organosilane', 'compound', 'like', 'phenyltriethoxysilane', 'external', 'one', '45', 'coordination', 'polymerization', 'oleﬁns', '213', 'initially', 'series', 'soluble', 'highly', 'isospeciﬁc', 'catalysts', 'developed', 'propylene', 'polymeri-', 'zation', '168.', 'these', 'materials', 'zirconium', 'titanium', 'hafnium', 'based', 'metallocenes', 'racemic', '11-ethylene-di-z5-indenylzirconium', 'dichloride', 'the', 'term', 'metallocene', 'applies', 'complexes', 'transition', 'metals', 'sandwiched', 'two', 'aromatic', 'rings', 'usually', 'two', 'cyclopentadienyl', 'they', 'rigid', 'structures', 'due', 'ethylene', 'bridges', 'two', 'ﬁve-membered', 'rings', 'syntheses', 'compounds', 'yield', 'racemic', 'mixtures', 'two', 'enantiomers', 'both', 'produce', 'isotactic', 'polypropylene', 'cl', 'zr', 'cl', 'cl', 'cl', 'zr', 'methylaluminoxane', 'co', 'catalyst', 'required', 'metallocenes', 'become', 'highly', 'isospeciﬁc', 'catalysts', 'capable', 'high', 'isotactic', 'placement', 'they', 'also', 'active', 'yielding', 'large', 'quantities', 'polypropylene', 'per', 'gram', 'zirconia', 'catalysts', 'yield', 'highly', 'syndiotactic', 'polypropylene', '(86', 'racemic', 'pentads)', 'also', 'developed', 'one', 'i-propylene(z5-cyclopentadienyl-z3ﬂuorenyl)zirconium', 'dichloride', '291.', 'initial', 'disclosures', 'metallocene', 'catalysts', 'followed', 'numerous', 'publications', 'literature', 'described', 'similar', 'materials', 'polymerizations', 'either', 'ethylene', 'propylene', 'formation', 'various', 'copolymers', 'thus', 'instance', 'kaminsky', 'et', 'al', '292,', 'reported', 'preparation', 'zirconium', 'dichloride-type', 'catalyst', 'copolymerization', 'cyclic', 'oleﬁns', 'ethylene', 'these', 'cyclic', 'oleﬁns', 'cyclopentene', 'norbornene', 'hindered', 'cyclopentadiene', 'adducts', 'norbornene', '292.', 'the', 'catalytic', 'system', 'consists', 'bridged', 'indene', 'derivative', 'combined', 'methylaluminoxane', 'zr', 'cl', 'cl', 'it', 'interesting', 'copolymers', 'cyclooleﬁns', 'ethylene', 'reported', 'transparent', 'amorphous', 'oxidatively', 'stable', 'materials', 'in', 'addition', 'claimed', 'glass', 'transition', 'temperatures', 'excess', '200(cid5)c', 'melt', 'processable', 'solvent', 'stable', 'possess', 'high', 'mechanical', 'strength', '292.', 'another', 'example', 'work', 'han', 'et', 'al', '293,', 'synthesized', 'ansa-dichloroo-phenylenedi-', 'methylenebis(z5-1-indenyl)zirconium', 'both', 'racemic', 'meso', 'forms', 'obtained', 'separated', 'zr', 'cl', 'cl', 'zr', 'cl', 'cl', 'racemic', '-', '1', 'meso', '-', '1', '214', '4', 'ionic', 'chain-growth', 'polymerization', 'these', 'catalysts', 'also', 'polymerize', 'ethylene', 'propylene', 'presence', 'methyl-aluminoxane', '293.', 'the', 'meso-1mao', 'analog', 'found', 'active', 'rac-1mao', 'ethylene', 'polymerization', 'in', 'propylene', 'polymerization', 'however', 'racemic', 'one', 'active', 'meso', 'inactive', 'ethylene', 'polymerization', 'activity', 'rac-1mao', 'increases', 'temperature', 'increases', 'an', 'inverse', 'temperature', 'effect', 'observed', 'propylene', 'polymerization', 'rac-1mao', 'catalyst', 'system', 'the', 'stereoregularity', 'polypropylene', 'formed', 'racemic', 'catalyst', 'low', '293.', 'this', 'led', 'han', 'et', 'al', 'conclude', '293', 'a-oleﬁn', 'polymerization', 'inﬂuenced', 'changes', 'structure', 'ansa', 'chiral', 'metallocene', 'catalysts', 'variation', 'transition', 'metal', 'steric', 'electronic', 'properties', 'p-', 's-ligands', 'length', 'rigidity', 'bridging', 'groups', 'the', 'activity', 'stereoselectivity', 'a-oleﬁn', 'polymerization', 'reactions', 'signiﬁcantly', 'affected', 'slight', 'structural', 'variations', 'bridging', 'group', 'metallocene', 'catalysts', 'the', 'length', 'bridging', 'chain', 'affects', 'angle', 'cyclopentadienes', 'metal', 'atom', 'the', 'chiral', 'metallocenes', 'p-ligands', 'bridged', 'one', 'atom', '(carbon', 'silicon)', 'two', 'carbon', 'atoms', 'either', 'c2', '(racemic)', 'c8', '(meso)', 'symmetry', 'ansa-ligand', 'enantiotropic', '293.', 'the', 'mechanism', 'monomer', 'insertion', 'steric', 'control', 'polymerizations', 'a-oleﬁns', 'metallocene', 'catalysts', 'received', 'considerable', 'attention', '293–297.', 'there', 'consensus', 'mechanism', 'polymerization', 'many', 'studies', 'chain', 'propagation', 'tend', 'support', 'cossee-arleman', 'mechanism', '293–297.', 'an', 'example', 'work', 'miyake', 'et', 'al', '294', 'synthesized', 'unsymmetrical', 'ansa-metallocenes', 'separated', 'threo', 'erythro', 'isomers', 'both', 'isomers', 'coupled', 'methylaluminoxane', 'polymerize', 'propylene', 'toluene', 'highly', 'isotactic', 'polymers', 'mw', '¼', '105000', 'the', 'isotactic', 'placement', 'greater', '996', 'polymer', 'melting', 'point', '161(cid5)c', 't-bu', 't-bu', 'cl', 'm', 'cl', 'cl', 'm', 'cl', 'y', 'y', 'r', 'threo-', 'form', 'r', 'erythro-form', 'based', 'experimental', 'evidence', 'obtained', 'catalysts', 'concluded', 'miyake', 'et', 'al,', 'isotactic', 'propylene', 'polymerization', 'zirconium', 'catalysts', 'takes', 'place', 'regiose-', 'lective', '12-insertion', 'propylene', 'monomer', 'metal-polymer', 'bond', '294.', 'monomer', 'insertion', 'believed', 'take', 'place', 'two', 'active', 'sites', 'metal', 'center', 'alternating', 'manner', 'in', 'addition', 'shown', '295', 'substituents', 'cyclopentadiene', 'rings', 'determine', 'conformation', 'polymer', 'chain', 'end', 'ﬁxed', 'polymer', 'chain', 'end', 'conformation', 'turn', 'determines', 'stereochemistry', 'oleﬁn', 'insertion', 'transition', 'state', 'form', 'indirect', 'steric', 'control', 'with', 'catalysts', 'however', 'stereochemistry', 'two', 'sites', 'different', 'miyake', 'et', 'al', '294,', 'suggest', 'monomer', 'insertion', 'takes', 'place', 'active', 'site', 'metal', 'center', 'r', 'zr', 'cp', 'ind', 'r', 'zr', 'cp', 'ind', 'r', 'zr', 'cp', 'ind', 'r', 'zr', 'cp', 'ind', '45', 'coordination', 'polymerization', 'oleﬁns', '215', 'lohrenz', 'et', 'al', 'reported', '296', 'quantum', 'mechanical', 'calculations', 'model', 'systems', '297', 'support', 'cossee-arleman', 'mechanism', 'insertion', 'ethylene', 'cp2zr-me', 'preceded', 'initial', 'oleﬁn', 'complexation', 'vacant', 'coordination', 'site', 'formation', 'p-complex', '1', 'shown', 'complex', '1', 'complex', '2', 'complex', '3', 'h', 'c', 'h', 'zr', 'c', 'c', 'h', 'h', 'h', 'h', 'h', '-96', 'kjmol', 'h', 'h', 'h', 'c', 'zr', 'c', 'h', 'h', 'c', 'h', 'h', '00', 'kjmol', 'h', 'c', 'c', 'h', 'h', 'h', 'zr', 'h', 'c', 'hh', '-93', 'kjmol', 'h', 'h', 'h', 'h', 'h', 'c', 'zr', 'c', 'c', 'h', 'h', '-128', 'kjmol', 'h', 'h', 'h', 'c', 'h', 'h', 'c', 'h', 'zr', 'c', 'h', '-22', 'kjmol', 'complex', '3a', 'complex', '3b', 'after', 'formation', 'complex', '1', 'insertion', 'follows', 'low', 'activation', 'barrier', '(eact', '¼', '3', 'klmol)', 'yielding', 'g-agostic', 'product', '3a', 'these', 'calculations', 'also', 'led', 'conclusion', 'importance', 'agostic', 'interactions', 'present', 'product', '296.', 'the', 'stable', 'conformation', 'b-agostic', 'structure', '(3b)', '22', 'kjmole', 'stable', 'primary', 'insertion', 'product', '(3a)', 'it', 'assumed', 'conformation', 'serves', 'model', 'resting', 'state', 'growing', 'chain', 'attached', 'cationic', 'group-4', 'metallocenes', 'insertions', '297.', 'to', 'shed', 'additional', 'light', 'complex', '3b', 'shown', 'calculations', 'carried', '296', 'reaction', 'ethylene', 'model', 'complex', 'cl2zr', 'et', 'two', 'reaction', 'paths', '(a)', '(b)', 'possible', 'illustrated', 'front', 'side', 'attack', 'h', 'zr', 'h2c', 'ch2', 'ch2', 'a', 'b', 'a', 'b', 'back', 'side', 'attack', 'ch2', 'h', 'c', 'zr', 'zr', 'ch2', 'insetion', 'inversion', 'ch3', 'ch2', 'h-exchange', 'chain', 'termination', 'ch2', 'h', 'h', 'ch2', 'zr', 'ch2', 'ch2', 'ch2', 'inserion', 'inversion', '216', '4', 'ionic', 'chain-growth', 'polymerization', 'front-side', 'attack', 'leads', 'formation', 'p-complex', 'turn', 'undergo', 'two', 'possible', 'reactions', 'insertion', 'oleﬁn', 'zr–ca', 'bond', 'may', 'take', 'place', 'rotation', 'around', 'zr–ca', 'moves', 'cb', 'plane', 'the', 'b-hydrogen', 'transferred', 'polymer', 'chain', 'oleﬁn', 'this', 'leads', 'chain', 'termination', 'formation', 'vinyl-terminated', 'polymer', 'ethyl-zirconocene', 'start', 'new', 'polymer', 'chain', '296.', 'the', 'backside', 'attack', 'hand', 'allows', 'insertion', 'oleﬁn', 'without', 'rotation', 'around', 'zr–ca', 'bond', 'the', 'front-side', 'insertion', 'accompanied', 'chain', 'migration', 'one', 'side', 'whereas', 'backside', 'attack', 'involve', 'inversion', 'metal', 'center', 'lohrenz', 'et', 'al', '296', 'concluded', 'insertion', 'metal-polymer', 'bond', 'takes', 'place', 'exclusively', 'backside', 'that', 'means', 'inversion', 'takes', 'place', 'they', 'also', 'feel', 'propylene', 'polymerization', 'two', 'orientations', 'occur', 'in', 'ﬁrst', 'step', 'polymer', 'chain', 'points', 'larger', 'ligand', 'side', 'propylene', 'methyl', 'group', 'points', 'away', 'large', 'ligand', 'polymer', 'chain', 'the', 'next', 'step', 'governed', 'stronger', 'interaction', 'propylene', 'polymer', 'chain', 'ligand', '296.', 'an', 'analysis', 'molecular', 'mechanics', 'using', 'model', 'metallocene', 'complexes', 'possible', 'intermediates', 'propylene', 'polymerization', 'also', 'reported', 'guerra', 'et', 'al', '298.', 'the', 'two', 'coordination', 'positions', 'available', 'monomer', 'growing', 'chain', 'diastereotopic', 'the', 'conclusion', 'energy', 'difference', 'corresponding', 'diastereoisomeric', 'pre-insertion', 'intermediates', 'appear', 'relevant', 'model', 'complexes', '298.', 'it', 'also', 'concluded', '391', 'energy', 'differences', 'related', 'increased', 'probability', 'back-skip', 'growing', 'chain', 'toward', 'outward', 'coordina-', 'tion', 'position', 'monomer', 'insertion', 'prior', 'coordination', 'new', 'oleﬁn', 'molecule', 'busico', 'et', 'al,', 'hand', 'came', 'conclusion', '299', 'stereoregularity', 'polypro-', 'pylene', 'produced', 'c2-symmetric', 'group', '4', 'ansa-metallocene', 'catalysts', 'result', 'interplay', 'two', 'competing', 'reactions', 'these', 'isotactic', 'monomer', 'polyinsertion', 'side', 'process', 'epimerization', 'polymer', 'chain', 'active', 'end', 'that', 'makes', 'class', 'homogenous', 'catalysts', 'different', 'typical', 'ziegler–natta', 'catalyst', 'catalysts', 'enantioselectivity', 'stereoselectivity', 'necessarily', 'coincidental', '96.', 'zang', 'et', 'al', 'reported', '300', 'achieved', 'highly', 'efﬁcient', 'rapid', 'reversible', 'chain', 'transfer', 'reactions', 'active', 'transition-metal', 'based', 'propagating', 'centers', 'catalysts', 'derived', 'cphf', '(me)n(et)c(me)n(et)}', 'b(c6f5)4', '(cp', '¼', 'z5-c5me5)', 'cphf', '(me)n(et)c(me)n', '(et)}', 'b(c6f5)3me', 'n', 'hf', 'n', 'b(c6f5)4', 'multiple', 'equivalents', 'dialkylzinc', '(znr2)', 'acting', '“surrogate”', 'chain-growth', 'sites', 'this', 'done', 'achieve', 'living', 'coordinative', 'chain-transfer', 'polymerization', 'ethylene', 'a-oleﬁns', 'ao-', 'non', 'conjugated', 'dienes', 'it', 'claimed', 'investigators', 'living', 'coordinated', 'chain', 'transfer', 'processes', 'provide', 'work-around', 'solution', '“one', 'chain', 'per', 'metal”', 'cap', 'product', 'yield', 'in', 'addition', 'claimed', 'provide', 'access', 'practical', 'volumes', 'variety', 'unique', 'new', 'classes', 'precision', 'polyoleﬁns', 'tunable', 'molecular', 'weights', 'narrow', 'polydispersity', '(mwmn', '(cid12)', '11)', 'a', 'japanese', 'patent', 'issued', 'watanabe', 'okamoto', '301', 'describes', 'preparation', 'illustrates', 'iron', 'containing', 'catalyst', 'polyethylene', 'preparation', 'it', 'shown', 'illustration', '45', 'coordination', 'polymerization', 'oleﬁns', '217', 'n', 'n', 'ni', 'br', 'br', 'n', 'fe', 'this', 'catalyst', 'reported', 'yield', '266', 'kg', 'polyethylene', 'per', 'one', 'minimol', 'nickel', 'subsequent', 'research', 'efforts', 'focused', 'discovering', 'efﬁcient', 'catalytic', 'processes', 'beneﬁting', 'cooperative', 'effects', 'active', 'centers', 'multinuclear', 'complexes', 'the', 'idea', 'ultimately', 'mimic', 'advantageous', 'enzyme', 'characteristics', 'thus', 'instance', 'li', 'stern', 'marks', '303', 'reported', 'earlier', 'work', 'demonstrated', 'constrained', 'geometry', 'binuclear', 'catalyst', 'binuclear', 'co', 'catalyst', 'combination', 'n', 'zr', 'si', 'f', 'f', 'ph3c', 'b(c6h5)3', 'f', 'f', '(c6h5)3', 'b', 'ph3c', 'si', 'zr', 'n', 'affords', 'via', 'modiﬁcation', 'chain', 'transfer', 'pathway', 'kinetics', 'signiﬁcantly', 'enhanced', 'branching', 'ethylene', 'homopolymerization', 'enhanced', 'comonomer', 'incorporation', 'well', 'ethylene', 'plus', '1-hexene', 'copolymerization', 'such', 'catalysts', 'however', 'typically', 'produce', 'unacceptably', 'low', 'molecular', 'weight', 'polyoleﬁns', 'earlier', 'reported', 'preliminary', 'results', 'showed', 'metal–metal', 'proximity', 'cocatalyst', 'structure', 'signiﬁcant', 'impact', 'terms', 'product', 'molecular', 'weight', 'comonomer', 'enchainment', 'selectivity', 'following', 'reported', 'subject', 'publication', 'synthesis', 'single', 'methylene-bridged', '(m-ch2-330)(z5-indenyl)1-me2si(tbun)(zrme2)2zr', 'based', 'catalyst', 'n', 'sime2', 'me2zr', 'me2si', 'zrme2', 'n', '218', '4', 'ionic', 'chain-growth', 'polymerization', 'examine', 'achievable', 'metal-metal', 'proximity', 'effects', 'they', 'also', 'reported', 'preparing', 'sterically', 'encumbered', 'naphthyl-derivatized', 'catalyst', '1-me2si', '3-(1)-naphthyl', 'indenyl-(tbun)zrme2', '(n-zn)', 'additionally', 'probe', 'catalytic', 'center', 'steric', 'effects', 'placing', 'two', 'metals', 'closer', 'proximity', 'shown', 'signiﬁcantly', 'increase', 'polyethylene', 'molecular', 'weight', 'ethylene', 'homopolymer-', 'izations', 'a-oleﬁn', 'enchainment', 'ethylene', 'copolymerizations', '1-hexene', 'additionally', 'found', 'methyl', 'alumoxane', 'used', 'co', 'catalyst', 'catalyst', 'shown', 'yields', 'even', 'greater', 'enhancement', 'weight', 'average', 'molecular', 'weight', 'also', 'used', 'polar', 'solvent', 'c6h5cl', 'reaction', 'caused', 'weakening', 'catalyst-cocatalyst', 'ion', 'pairing', 'signiﬁcant', 'alteration', 'catalyst', 'response', 'polymer', 'product', 'properties', 'observed', 'their', 'results', 'showed', 'zr', '(cid11)', '(cid11)', '(cid11)', 'zr', 'spatial', 'proximity', 'modulated', 'ion', 'pairing', 'signiﬁcantly', 'inﬂuences', 'chain', 'transfer', 'rate', 'selectivity', 'comonomer', 'enhancement', 'these', 'proximity', 'effects', 'highly', 'cocatalyst', 'solvent', 'sensitive', 'in', 'subsequent', 'publication', '304', 'reported', 'synthesis', 'activity', 'binuclear', 'catalysts', '27-di(26-isopropylphenyl)', 'imino-18-naphthalenediolato', 'group', '4', 'metal', 'complexes', 'copoly-', 'merization', 'ethylene', 'various', 'mono', 'oleﬁns', 'the', 'bimolecular', 'catalysts', 'exhibited', 'enhanced', 'activity', 'compared', 'mononuclear', 'analogs', 'the', 'polymerizations', 'illustrated', 'follows', 'thf', 'cl', 'thf', 'cl', 'cl', 'o', 'cl', 'o', 'cl', 'cl', 'thf', 'cl', 'cl', 'cl', 'n', 'ar', 'c4h9', 'n', 'c4h9', '94', 'branching', 'n', 'c4h9', 'n', 'n', 'c4h9', 'n', 'n', 'c4h9', '43', 'branching', 'n', 'n', '!6', '34', 'rings', 'at', 'present', 'time', 'spite', 'research', 'effort', 'still', 'need', 'catalyst', 'systems', 'enable', 'preparation', 'low', 'density', 'polyethylene', 'controlled', 'levels', 'short', 'long', 'branches', 'homura', 'coworkers', '305', 'investigated', 'series', 'half-titanocenes', 'containing', 'pyrazole', 'ligands', 'employed', 'catalyst', 'precursors', 'ethylene', 'polymerization', 'syndiospeciﬁc', 'styrene', 'polymerization', 'copolymerization', 'ethylene', '1-hexene', 'styrene', 'norbornene', 'presence', 'methyl', 'aluminoxane', 'cocatalyst', 'the', 'catalyst', 'ticl2', '(35r2c3hn2)', 'r', 'hydrogen', 'methyl', 'propyl', 'phenyl', 'the', 'ethylenestyrene', 'copolymerization', 'proceeds', 'living', 'manner', 'irrespective', 'styrene', 'concentration', 'toluene', '25(cid5)c', 'system', 'exhibits', 'relatively', 'high', 'catalytic', 'activity', 'ethylenenorbornene', 'copolymerization', 'highly', 'efﬁcient', 'norbornene', 'incorporation', '305.', 'the', 'catalyst', 'system', 'illustrated', 'follows', 't-c4h9', 'i-c3h7', 'n', 'ti', 'n', 'cl', 'cl', 'al', 'o', 'n', 'i-c3h7', '45', 'coordination', 'polymerization', 'oleﬁns', '219', '455', 'effect', 'lewis', 'bases', 'many', 'investigations', 'carried', 'understand', 'role', 'lewis', 'bases', 'play', 'affecting', 'stereospeciﬁcity', 'ziegler–natta', 'catalysts', 'one', 'instance', 'included', 'studies', 'optically', 'active', 'bases', 'determine', 'reactions', 'active', 'sites', '252.', 'nevertheless', 'effect', 'lewis', 'bases', 'like', 'tertiary', 'amines', 'fully', 'understood', 'evidence', 'gathered', 'date', 'suggests', 'react', 'aluminum', 'alkyl', 'halides', '306', 'increase', 'number', 'stereospeciﬁc', 'sites', 'assisting', 'stabilization', 'andor', 'dispersal', 'catalyst', 'support', 'they', 'appear', 'alter', 'identities', 'attached', 'ligands', 'this', 'might', 'due', 'increasing', 'steric', 'bulk', 'sites', 'the', 'lewis', 'bases', 'also', 'given', 'credit', 'reducing', 'reactivity', 'less', 'stereospeciﬁc', 'sites', 'because', 'internal', 'external', 'lewis', 'bases', 'used', 'enhancement', 'stereospeciﬁcity', 'external', 'bases', 'attributed', 'displacing', 'internal', 'base', 'andor', 'augmenting', 'effect', '307.', '456', 'terminations', 'coordination', 'polymerizations', 'the', 'terminations', 'usually', 'occur', 'chain', 'transferring', 'either', 'internal', 'hydride', 'transfer', '261:', 'x', 'x', 'rn', 'x', 'x', 'h', 'catalyst', 'site', 'catalyst', 'site', 'r', 'n', 'transfer', 'monomer', 'x', 'x', 'x', 'r', 'n', 'x', 'x', 'x', 'terminal', 'unsaturation', 'products', 'conﬁrmed', 'infrared', 'spectroscopy', 'hydrogen', 'terminates', 'reactions', 'often', 'added', 'control', 'molecular', 'weights', 'products', 'protonic', 'acids', 'thing', 'x', 'x', 'r', 'h2(ha)', 'x', 'x', 'h', '(a)', 'r', 'n', 'r', 'n', '457', 'reduced', 'transition', 'metal', 'catalysts', 'support', 'the', 'catalysts', 'belong', 'group', 'efﬁcient', 'homopolymerizations', 'ethylene', 'they', 'also', 'used', 'copolymerizations', 'ethylene', 'a-oleﬁns', 'such', 'catalysts', 'prepared', 'tio2', 'v2o5', 'nb2o5', 'ta2o5', 'cro3', 'cr2o3', 'moo3', 'wo3', 'nio', 'coo', 'the', 'supports', 'charcoal', 'silica', 'alumina', 'zinc', 'oxide', 'alumina-silica', 'these', 'catalysts', 'require', 'higher', 'temperatures', 'pressures', 'activity', 'typical', 'ziegler–natta', 'catalysts', 'the', 'temperatures', 'range', '140', '230(cid5)c', 'pressures', '420', '1000', 'psi', 'one', 'original', 'disclosures', 'catalyst', 'preparation', 'follows', '308.', 'a', 'transition', 'metal', 'nitrate', 'solution', 'used', 'saturate', 'charcoal', 'bed', 'previously', 'leached', 'nitric', 'acid', 'the', 'charcoal', 'bed', 'heated', 'temperatures', 'high', 'enough', 'decompose', 'nitrate', 'oxide', '220', '4', 'ionic', 'chain-growth', 'polymerization', 'following', 'heating', 'oxide', 'reduced', 'h2', 'nabh4', 'sometimes', 'powdered', 'al2o3', 'moo3-al2o3', 'added', 'prior', 'reduction', 'for', 'al2o3-sio2', 'support', '3', 'metal', 'nitrate', 'solution', 'used', 'the', 'saturated', 'bed', 'heated', '500(cid5)c', 'decompose', 'nitrate', 'this', 'followed', 'reduction', 'many', 'modiﬁcations', 'procedure', 'developed', 'since', 'preparations', 'based', 'molybdenum', 'oxide', 'called', 'standard', 'oil', 'co', 'catalysts', 'based', 'chromium', 'oxide', 'known', 'phillips', 'petroleum', 'co', 'catalysts', 'following', 'description', 'preparation', 'molybdenum', 'based', 'catalyst', 'alumina', 'saturated', 'solution', 'ammonium', 'molybdate', 'subjected', 'heating', 'air', '500–600(cid5)c', 'the', 'oxide', 'forms', 'reduced', 'hydrogen', '430–480(cid5)c', 'reducing', 'agents', 'like', 'co', 'so', 'hydrocarbons', 'also', 'used', 'hydrogen', 'however', 'preferred', 'pressures', 'approximately', '75', 'psi', 'the', 'catalyst', 'may', 'contain', '5', '25', 'molybdenum', 'compound', 'dispersed', 'surface', '309.', 'a', 'typical', 'chromium', 'based', 'catalyst', 'prepared', 'similarly', 'it', 'usually', 'supported', '91', 'sio2', 'al2o3', '347', 'carrier', 'either', 'cr(no3)3', '(cid6)', '9h2o', 'cro3', 'solutions', 'nitric', 'acid', 'used', 'impregnate', 'support', 'the', 'nitrates', 'decomposed', 'air', '400–1000(cid5)c', '310.', 'optimum', 'chromium', 'content', 'catalysts', '2–3.', 'the', 'mechanism', 'polymerization', 'initially', 'believed', 'follows', '212.', 'cr–o–si', 'bonds', 'present', 'catalyst', 'surface', 'contact', 'ethylene', 'results', 'oxidation-reduction', 'reaction', 'formation', 'ethylene-chromium', 'complex', 'the', 'initiations', 'accompanied', 'gains', 'hydride', 'ions', 'formations', 'terminal', 'methyl', 'groups', 'polymer', 'growth', 'pictured', 'follows', 'cr', 'catalyst', 'surface', 'cr', 'cr', 'the', 'termination', 'reactions', 'believed', 'result', 'hydride-ion', 'transfers', '262.', 'later', 'however', 'polyethylene', 'formed', 'catalysts', 'found', 'contain', 'approximately', 'equal', 'numbers', 'saturated', 'unsaturated', 'chain', 'ends', 'this', 'contradicts', 'concept', 'it', 'may', 'also', 'mean', 'initiation', 'involves', 'p-complexes', 'disproportionate', 'yield', 'coordination', 'metal', '–ch¼ch2', '–ch2–ch3', 'groups', '262.', 'polymerization', 'propylene', 'catalysts', 'yields', 'polymers', 'little', 'crystallinity', 'higher', 'a-oleﬁns', 'yield', 'completely', 'amorphous', 'polymers', 'formation', 'partially', 'crystalline', 'polymers', 'reported', 'vinylcyclohexane', 'allylcyclohexane', '4-phenyl-1-butene', 'styrene', 'polymerize', 'isoprene', 'forms', 'trans-14-polymer', '262.', '458', 'isomerization', 'polymerizations', 'coordination', 'catalysts', 'polymerizations', 'many', 'internal', 'oleﬁns', 'like', '2-butene', '2-pentene', '3-heptene', '4-methyl-2-pentene', '4-', 'phenyl-2-butene', 'others', 'ziegler–natta', 'catalysts', 'accompanied', 'monomer', 'rearrangements', 'the', 'isomerizations', 'take', 'place', 'insertions', 'chains', '311–318:', 'r', 'n', 'ch2r', 'n', 'ch2r', 'the', 'double', 'bonds', 'migrate', 'internal', 'a-positions', '46', 'polymerization', 'aldehydes', '221', 'the', 'products', 'consist', 'exclusively', 'poly(1-oleﬁn)', 'units', 'polymerizations', '2', '3', '4', 'octenes', 'ticl3al(c2h5)3', '80(cid5)c', 'instance', 'result', 'high', 'molecular', 'weight', 'homopolymer', 'poly', '(1-octene)', '319.', 'also', 'reported', 'addition', 'transition', 'metal', 'compounds', 'group', 'viii', 'ziegler–natta', 'catalysts', 'enhances', 'isomerization', '2-oleﬁns', '321', '329.', 'when', 'nickel', 'compounds', 'added', 'ticl3–(c2h5)3al', 'react', 'form', 'ticl3–nix2–(c2h5)3al', 'x', 'chloride', 'acetylacetonate', 'dimethyl', 'glyoximate', '328.', 'the', 'product', 'efﬁcient', 'isomerization', 'catalyst', '46', 'polymerization', 'aldehydes', 'aldehyde', 'polymers', 'probably', 'known', 'well', '100', 'years', 'ago', '322–324.', 'in', 'spite', 'polyoxymethylene', 'product', 'aldehyde', 'polymerization', 'produced', 'large', 'commercial', 'quantities', 'formaldehyde', 'polymerizes', 'cationic', 'anionic', 'mechanisms', 'an', 'oxonium', 'ion', 'acts', 'propagating', 'species', 'cationic', 'polymerizations', '322', '323.', 'in', 'anionic', 'ones', 'propagation', 'via', 'alkoxide', 'ion', 'most', 'polymerizations', 'aldehydes', 'conducted', 'aprotic', 'anhydrous', 'solvents', 'proton-', 'yielding', 'impurities', 'efﬁcient', 'chain', 'transferring', 'agents', 'polymerization', 'formaldehyde', 'looked', 'upon', 'exception', '323.', 'when', 'protonic', 'solvents', 'used', 'however', 'resultant', 'polymer', 'wide', 'molecular', 'weight', 'distribution', 'usually', 'therefore', 'solvents', 'low', 'dielectric', 'constants', 'preferred', 'in', 'cases', 'aldehyde', 'polymerizations', 'polymers', 'precipitate', 'solution', 'form', 'choice', 'low', 'dielectric', 'solvents', 'contributes', 'in', 'spite', 'polymerizations', 'often', 'keep', 'going', 'rapidly', 'monomers', 'added', 'reaction', 'mixtures', 'in', 'addition', 'low', 'molecular', 'weight', 'polyoxymethylene', 'forms', 'high', 'dielectric', 'solvents', 'like', 'dimethylformamide', 'reaction', 'quite', 'sluggish', 'higher', 'aldehydes', 'fail', 'polymerize', 'dimethylformamide', 'as', 'stated', 'propagation', 'reaction', 'continues', 'polymer', 'precipitation', 'that', 'due', 'fact', 'precipitated', 'macromolecules', 'highly', 'swollen', 'monomers', 'this', 'shown', 'polymerizations', 'n-butyraldehyde', 'heptane', '326.', 'the', 'physical', 'state', 'polymers', 'surface', 'areas', 'crystalline', 'domains', 'therefore', 'inﬂuence', 'paths', 'polymerizations', 'the', 'solubility', 'initiators', 'solvents', 'affect', 'strongly', 'molecular', 'weights', 'resultant', 'products', 'bulk', 'densities', '323.', 'the', 'whole', 'reaction', 'even', 'affected', 'small', 'changes', 'chemical', 'structures', 'initiators', 'changes', 'counterions', '323.', 'chain', 'transferring', 'important', 'termination', 'step', 'aldehyde', 'polymerizations', '461', 'cationic', 'polymerization', 'aldehydes', 'the', 'cationic', 'polymerizations', 'formaldehyde', 'carried', 'anhydrous', 'media', 'typical', 'cationic', 'initiators', 'initiation', 'takes', 'place', 'electrophilic', 'addition', 'initiating', 'species', 'carbonyl', 'oxygens', 'this', 'results', 'formations', 'oxonium', 'ions', '323:', 'or', 'r', 'o', 'r', 'ch2', 'o', 'the', 'oxonium', 'ions', 'may', 'react', 'oxygen-carbon', 'cations', 'the', 'propagation', 'steps', 'consist', 'attacks', 'electrophilic', 'carbon', 'atoms', 'upon', 'carbonyl', 'oxygens', 'highly', 'polar', 'formaldehyde', 'molecules', 'r', 'o', 'o', 'r', 'o', 'o', '222', '4', 'ionic', 'chain-growth', 'polymerization', 'in', 'polymerizations', 'propagating', 'oxonium', 'ions', 'probably', 'solvated', '226:', 'o', 'ch2', 'o', 'ch2', 'o', 'ch2', 'o', 'terminations', 'chain', 'growths', 'occur', 'recombinations', 'reactions', 'impurities', 'chain', 'transfers', 'o', 'o', 'o', 'mexn', 'rx', 'h2o', 'o', 'ch2x', 'mxn-1', 'o', 'ch2x', 'r', 'o', 'ch2oh', 'o', 'h', 'mnxn', 'lewis', 'acid', 'formaldehyde', 'polymerize', 'anhydrous', 'form', 'three', 'physical', 'states', 'matter', 'gas', 'liquid', 'solid', '322.', 'it', 'also', 'polymerizes', 'water', 'acid', 'catalysts', 'many', 'lewis', 'acids', 'efﬁcient', 'catalysts', 'reaction', 'in', 'addition', 'protonic', 'acids', 'also', 'effective', 'among', 'perchloric', 'sulfuric', 'acids', 'monoesters', 'sulfuric', 'acid', '322.', 'the', 'activity', 'initiators', 'independent', 'solvents', 'cationic', 'polymerizations', 'formaldehyde', '322.', 'there', 'however', 'differences', 'performance', 'initiators', 'with', 'bf3', 'reaction', 'reaches', 'equilibrium', '30', 'conversion', 'stops', 'much', 'higher', 'conversions', 'achieved', 'sncl4', 'snbr4', '322.', 'both', 'lewis', 'acids', 'yield', 'polyoxymethylenes', 'similar', 'formed', 'anionic', 'mechanism', '324–327.', 'higher', 'aldehydes', 'polymerized', 'lewis', 'acids', 'bronsted', 'acids', 'acidic', 'metal', 'salts', '322.', 'initiation', 'propagation', 'reactions', 'consist', 'repeated', 'nucleophilic', 'attacks', 'carbonyl', 'oxygens', 'electrophiles', 'formations', 'carbon-oxonium', 'ions', '328:', 'r', 'o', 'r', 'o', 'r', 'o', 'r', 'r', 'o', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'o', 'o', 'n', 'o', 'r', 'o', 'o', 'o', 'n', 'the', 'counterions', 'shown', 'equations', 'when', 'initiators', 'protonic', 'acids', 'reactions', 'illustrated', 'follows', '(additional', 'monomers', 'shown)', 'h', 'a', 'o', 'h', 'o', 'r', 'a', 'r', 'r', 'h', 'a', 'o', 'o', 'n', 'r', 'r', 'r', 'many', 'aldehydes', 'trimerize', 'acidic', 'conditions', 'paraldehyde', 'many', 'trimers', 'fail', 'undergo', 'polymerizations', 'high', 'molecular', 'weight', 'linear', 'polymers', 'the', 'trimer', 'formalde-', 'hyde', 'trioxane', 'however', 'unique', 'it', 'polymerizes', 'mechanism', 'ring', 'opening', 'polymerization', 'this', 'discussed', 'chap', '5', 'to', 'avoid', 'trimer', 'formations', 'low', 'dielectric', 'solvents', 'must', 'used', 'low', 'temperatures', 'in', 'addition', 'initiator', 'must', 'carefully', 'selected', '46', 'polymerization', 'aldehydes', '223', 'potentially', 'atactic', 'isotactic', 'syndiotactic', 'polymers', 'form', 'higher', 'aldehydes', 'catalysts', 'like', 'al2o3', 'cro3', 'moo3', 'sio2', 'form', 'high', 'molecular', 'weight', 'atactic', 'polyacetaldehyde', '(cid1)78(cid5)c', '329.', 'these', 'solid', 'catalysts', 'yield', 'polymers', 'heterogeneous', 'conditions', '330.', 'some', 'alumina', 'converted', 'efﬁcient', 'catalysts', 'treating', 'surface', 'hcl', 'f3ccooh', 'bf3', '331.', 'acetaldehyde', 'polymerized', 'variety', 'soluble', 'initiators', 'h3po4', 'hcl', 'hbo3', 'f3ccooh', 'asf3', 'ascl3', 'zncl2', 'zrcl4', 'cl3cooh', '332.', 'polymerization', 'reactions', 'require', 'good', 'solvent', 'weak', 'cationic', 'initiators', 'used', 'when', 'strong', 'initiators', 'employed', 'reactions', 'difﬁcult', 'control', 'low', 'temperatures', 'maintained', 'techniques', 'boiling', 'low', 'boiling', 'solvents', 'formation', 'pure', 'isotactic', 'polyacetaldehyde', 'reported', 'it', 'forms', 'small', 'quantities', 'presence', 'atactic', 'polymer', 'obtained', 'crystalline', 'bf3-etherate', '332.', '462', 'anionic', 'polymerization', 'aldehydes', 'many', 'patents', 'describe', 'polymerizations', 'anhydrous', 'formaldehyde', 'anionic', 'mechanism', 'the', 'initiators', 'included', 'amines', 'phosphines', 'metal', 'alcoholates', 'kern', 'pictured', 'initiations', 'formaldehyde', 'polymerizations', 'tertiary', 'amines', 'direct', 'addition', 'reactions', '332', '334:', 'r3n', 'þ', 'ch2o', 'r3n(cid7)', '(cid1)(cid1)', 'ch2', '(cid1)(cid1)', 'o', '(cid8)', 'earlier', 'however', 'machacek', 'suggested', '335–337', 'initiations', 'take', 'place', 'help', 'protonic', 'impurities', 'r3n', 'h2o', 'r3n', 'h', 'oh', 'r3n', 'h', 'oh', 'o', 'nr3', 'ho', 'o', 'h', 'much', 'evidence', 'presented', 'since', 'favors', 'machacek', 'mechanism', 'initiation', '335.', 'by', 'contrast', 'tertiary', 'phosphines', 'apparently', 'initiate', 'polymerizations', 'zwitter', 'ion', 'mechanism', '338.', 'this', 'may', 'perhaps', 'due', 'higher', 'nucleophilicity', 'lower', 'basicity', 'tertiary', 'amines', 'phosphorus', 'incorporates', 'polymer', '338', 'process', 'the', 'propagation', 'reactions', 'tertiary', 'amine', 'initiated', 'polymerizations', 'pictured', 'follows', '339:', 'ho', 'o', 'o', 'n', 'n3r', 'h', 'o', 'ho', 'o', 'o', 'n', '1', 'n3r', 'h', 'the', 'terminations', 'probably', 'result', 'chain', 'transferring', '241:', 'ho', 'o', 'o', 'n1', 'n3r', 'h', 'roh', 'ho', 'o', 'ch2oh', 'n1', 'r3n', 'or', 'h', 'the', 'newly', 'formed', 'active', 'species', 'initiate', 'new', 'polymerizations', 'metal', 'alkyls', 'metal', 'alcoholates', 'effective', 'anionic', 'initiators', 'higher', 'aldehydes', 'aldol', 'condensation', 'occur', 'however', 'presence', 'strong', 'bases', 'thus', 'initiators', 'yield', 'high', 'molecular', 'weight', 'polymers', 'formaldehyde', 'yield', 'low', 'molecular', 'weight', 'polymers', 'higher', 'aldehydes', 'initiations', 'metal', 'alkyls', 'result', 'additions', 'carbonyl', 'group', '224', '4', 'ionic', 'chain-growth', 'polymerization', 'r', 'o', 'rme', 'r', 'r', 'me', 'o', 'where', 'me', 'represents', 'metal', 'group', 'the', 'propagation', 'reaction', 'series', 'successive', 'nucleo-', 'philic', 'additions', '340,', 'alkoxide', 'ion', 'propagating', 'species', 'r', 'r', 'me', 'o', 'r', 'n', 'o', 'r', 'r', 'r', 'o', 'n', 'o', 'me', 'there', 'indications', 'many', 'aldehyde', 'polymerizations', 'result', 'formations', '“living”', 'polymers', 'similarly', 'anionic', 'polymerizations', 'vinyl', 'compounds', 'termination', 'occur', 'hydride', 'transfer', 'via', 'form', 'crossed', 'cannizzaro', 'reaction', 'r', 'o', 'o', 'me', 'rcho', 'r', 'r', 'o', 'h', 'o', 'o', 'me', 'r', 'r', 'o', 'o', 'me', 'o', 'the', 'alkyl', 'substituent', 'tendency', 'destabilize', 'propagating', 'anion', 'increasing', 'charge', 'density', 'oxygen', 'c', 'h', 'o', 'r', 'as', 'result', 'weak', 'bases', 'like', 'amines', 'fail', 'initiate', 'polymerizations', 'higher', 'aldehydes', 'a', 'stereospeciﬁc', 'anionic', 'polymerization', 'acetaldehyde', 'originally', 'reported', '1960', '343', '344.', 'two', 'alkali', 'metal', 'compounds', '341', 'organozinc', '342', 'one', 'used', 'initiators', 'trialkylaluminum', 'triarylaluminum', 'heptane', 'also', 'yield', 'crystalline', 'isotactic', 'polymers', 'acetaldehyde', 'heptaldehyde', 'propionaldehyde', '(cid1)80(cid5)c', '343.', 'aluminum', 'oxide', 'activated', 'diethylzinc', 'yields', 'stereoblock', 'crystalline', 'polymers', 'various', 'aldehydes', '342', '344.', 'lithium', 'alkoxide', 'formed', 'polyacetaldehyde', 'insoluble', 'common', 'solvents', 'it', 'melts', '165(cid5)c', '341.', 'the', 'mechanism', 'stereoregulation', 'still', 'debated', 'some', 'concepts', 'presented', 'rest', 'section', 'natta', '343', 'believed', 'polymerizations', 'initiated', 'organoaluminum', 'compounds', 'proceed', 'coordinated', 'anionic', 'mechanism', 'the', 'aluminum', 'atoms', 'seen', 'forming', 'complexes', 'oxygen', 'atoms', 'penultimate', 'units', 'o', 'al', 'o', 'o', 'al', 'o', 'o', 'o', 'al', 'the', 'activated', 'complexes', 'form', 'steric', 'conﬁgurations', 'allow', 'minimum', 'amounts', 'non-', 'bonded', 'interactions', 'other', 'mechanisms', 'proposed', 'since', 'for', 'instance', 'furukawa', 'et', 'al', '344', 'concluded', 'metal', 'alkyl', 'compounds', 'must', 'become', 'metal', 'alkoxides', 'reactions', 'aldehydes', 'r', 'o', 'mer', 'r', 'r', 'o', 'me', 'me', 'means', 'metal', '46', 'polymerization', 'aldehydes', '225', 'his', 'polymerization', 'mechanism', 'therefore', 'based', 'known', 'reactions', 'metal', 'alkoxides', 'carbonyl', 'compounds', 'side', 'reactions', 'like', 'meerwein-ponndorf', 'tischenko', 'expected', 'included', 'reaction', 'scheme', 'main', 'reaction', 'side', 'reaction', 'mer', 'o', 'r', 'r', 'me', 'o', 'o', 'o', 'me', 'o', 'r', 'o', 'o', 'me', 'o', 'me', 'o', 'meerwin-ponndorf', 'oppenauer', 'r', 'o', 'o', 'me', 'r', 'o', 'o', 'r', 'o', 'o', 'me', 'o', 'r', 'o', 'o', 'o', 'me', 'etc', '226', '4', 'ionic', 'chain-growth', 'polymerization', 'each', 'monomer', 'addition', 'growing', 'chain', 'requires', 'transfer', 'alkoxide', 'anion', 'carbonyl', 'group', 'this', 'results', 'formation', 'new', 'alkoxide', 'anion', '(a', 'hydride', 'transfer', 'alkoxide', 'group', 'carbon', 'atom', 'aldehyde', 'take', 'place', 'meerwein-ponndorf', 'reduction)', 'chain', 'growth', 'takes', 'place', 'repetition', 'coordination', 'aldehyde', 'subsequent', 'transfer', 'alkoxide', 'anion', 'in', 'vogl', 'bryant', '345', 'mechanism', 'four', 'oxygens', 'coordinated', 'metal', 'atom', 'these', 'oxygens', 'penultimate', 'ultimate', 'units', 'growing', 'chains', 'two', 'monomers', 'r', 'me', 'o', 'o', '2', 'rcho', 'r', 'r', 'o', 'o', 'o', 'me', 'o', 'r', 'r', 'o', 'o', 'r', 'me', 'r', 'o', 'o', 'r', 'r', 'o', 'o', 'o', 'me', 'o', 'r', 'r', 'r', 'o', 'me', 'o', 'o', 'o', 'r', 'a', 'simultaneous', 'coordination', 'two', 'aldehydes', 'prior', 'addition', 'may', 'explain', 'observed', 'sequence', 'isotactic', 'dyads', 'yashida', 'tani', 'derived', 'mechanism', 'investigations', 'isotactic', 'aldehyde', 'polymerizations', 'r2alocr0nc6h5', 'catalysts', '346–348.', 'the', 'bulkiness', 'substituent', 'group', 'considered', 'important', 'factor', 'steric', 'control', 'the', 'catalyst', 'enhances', 'degree', 'stereoregulation', 'controlling', 'mode', 'approach', 'coordination', 'also', 'lewis', 'acidity', 'catalyst', 'must', 'conﬁned', 'narrow', 'range', 'particular', 'aldehyde', 'yield', 'isotactic', 'polymers', '347', '348.', 'if', 'acidity', 'strong', 'amorphous', 'polymer', 'forms', 'if', 'weak', 'polymerization', 'takes', 'place', '348.', 'in', 'addition', 'special', 'techniques', 'like', 'high', 'pressures', 'instance', 'result', 'formations', 'isotactic', 'butyraldehyde', 'heptaldehyde', '349.', '463', 'polymerization', 'unsaturated', 'aldehydes', 'unsaturated', 'aldehydes', 'unique', 'polymerize', 'special', 'ways', 'much', 'effort', 'acrolein', 'polymerizations', 'form', 'products', 'aldehyde', 'groups', 'tact', 'this', 'means', 'reactions', 'must', 'conﬁned', 'carbon', 'carbon', 'double', 'bond', 'portions', 'molecules', '350–356.', 'organometallic', 'catalysts', 'like', 'n-butylmagnesium', 'bromide', 'however', 'also', 'initiators', 'polymerizations', 'carbonyl', 'groups', 'in', 'fact', 'unsaturated', 'aldehyde', 'monomers', 'polymerized', 'four', 'ways', 'similar', 'dienes', 'the', 'placement', 'carbon', 'carbon', 'double', 'bond', 'carbonyl', 'group', '14', '1', '2', '3', '4', 'o', '14', 'ladder', 'structure', '12', '34', 'o', 'n', 'o', 'o', 'o', '12', 'n', 'o', '34', 'o', 'n', '46', 'polymerization', 'aldehydes', '227', 'because', 'acrolein', 'polymerizes', 'free-radical', 'ionic', 'mechanisms', 'reactions', 'possible', 'products', 'quite', 'complex', 'the', 'structures', 'materials', 'include', 'linkages', 'vinyl', 'carbonyl', 'groups', 'in', 'addition', 'tetrahydropyran', 'rings', 'shown', 'also', 'form', '357.', 'coordination', 'complexes', 'like', 'cdi2(pyridine)2', 'also', 'initiate', 'polymerizations', 'acrolein', 'propaga-', 'tion', 'reactions', 'precede', 'vinyl', 'carbonyl', 'groups', '358:', 'o', 'o', 'n', 'o', 'the', 'ratio', 'vinyl', 'carbonyl', 'placement', 'depends', 'upon', 'nature', 'complex', 'polymers', 'formed', 'complexes', 'metallic', 'salts', 'triphenylphosphine', 'contain', 'considerably', 'less', 'aldehyde', 'groups', 'formed', 'triphenylphosphine', 'alone', '358.', 'polymerization', 'propionaldehyde', '(ch(cid13)c–ch¼o)', 'also', 'unique', 'in', 'dimethylformamide', '0(cid5)c', 'sodium', 'cyanide', 'tri-n-butyl', 'phosphine', 'catalysts', 'reactions', 'yield', 'polymers', 'composed', 'two', 'different', 'structural', 'units', 'one', 'polyaldehyde', 'one', 'polyacetylene', '355.', 'the', 'reaction', 'tetrahydrofuran', 'however', '(cid1)78(cid5)c', 'sodium', 'cyanide', 'catalyst', 'results', 'crystalline', 'poly(ethynyl', 'oxymethylene)', '359.', 'radical', 'initiated', 'polymerizations', 'monomer', '60(cid5)c', 'hand', 'proceed', 'acetylenic', 'group', 'crotonaldehyde', 'like', 'acrolein', 'expected', 'yield', 'polymers', 'structures', 'derived', '12', '34', '14', 'additions', 'anionic', 'catalysts', 'however', 'yield', 'predominantly', 'polyacetal', 'structures', '360.', '464', 'polymerizations', 'di', 'aldehydes', 'as', 'one', 'might', 'expect', 'di', 'aldehydes', 'polymerized', 'yield', 'polymers', 'cyclic', 'ether', 'linkages', 'this', 'resembles', 'polymerizations', 'non-conjugated', 'dienes', '361:', 'r', 'o', 'o', 'o', 'r', 'o', 'o', 'o', 'r', 'etc', 'cationic', 'polymerizations', 'take', 'place', 'either', 'one', 'three', 'paths', 'stepwise', 'additions', 'intermediate-type', 'additions', 'concerted', 'additions', 'when', 'phthalaldehyde', 'polymerized', 'cationic', 'mechanism', 'g-irradiation', '(cid1)78(cid5)c', 'products', 'consist', 'dioxyphthalan', 'units', '362:', 'r', 'n', 'phthalaldehyde', 'h', 'c', 'c', 'h', 'c', 'c', 'h', 'h', 'or', 'o', 'o', 'o', 'h', 'o', 'r', 'c', 'c', 'o', 'h', 'n', 'aromatic', 'aldehydes', 'usually', 'polymerize', 'difﬁculty', 'the', 'enhanced', 'polymerizability', 'phthalaldehyde', 'due', 'formation', 'intermediates', 'concerted', 'propagation', 'reactions', '362.', 'in', 'illustration', 'anhydride', 'rings', 'either', 'cis', 'trans', 'conﬁguration', 'cationic', 'polymerizations', 'yield', 'polymers', 'high', 'cis', 'content', 'anionic', 'catalysts', 'particularly', 'coordination', 'catalysts', 'like', 'triethylaluminum-transition', 'metal', 'halides', 'yield', 'high', 'trans', 'polymers', '363.', '228', '4', 'ionic', 'chain-growth', 'polymerization', '47', 'polymerization', 'ketones', 'isocyanates', 'in', 'spite', 'many', 'attempts', 'prepare', 'useful', 'polymers', 'ketones', 'far', 'successful', 'acetone', 'polymerizes', 'aid', 'magnesium', '364.', 'the', 'reaction', 'requires', 'high', 'vacuum', 'vapors', 'magnesium', 'metal', 'dry', 'acetone', 'condense', 'simultaneously', 'vacuum', 'surface', 'cooled', 'liquid', 'nitrogen', 'the', 'white', 'elastic', 'polymer', 'forms', 'possesses', 'polyketal', 'structure', 'n', 'o', 'o', 'n', 'like', 'low', 'molecular', 'weight', 'ketals', 'polymer', 'unstable', 'decomposes', 'even', 'room', 'temperature', 'acetone', 'also', 'polymerizes', 'upon', 'irradiation', 'frozen', 'state', 'high', 'vacuum', 'the', 'product', 'yellow', 'rubbery', 'material', '364.', 'infrared', 'spectra', 'show', 'presence', 'c–o–c', 'linkages', 'carbonyl', 'groups', 'monobromoacetone', 'polymerizes', 'manner', 'irradiation', 'frozen', 'state', 'the', 'hard', 'resinous', 'polymer', 'forms', 'stable', 'polyacetone', '364.', 'polymerizations', 'ketenes', 'yield', 'varieties', 'structures', 'monomer', 'placements', 'possible', 'either', 'carbon', 'carbon', 'double', 'bond', 'carbon', 'oxygen', 'double', 'bond', 'dimethylketene', 'polymerizes', 'anionic', 'mechanism', 'polymer', 'following', 'structures', '46:', 'n', 'o', 'o', 'o', 'o', 'o', 'p', 'polar', 'solvents', 'increase', 'formations', 'ether', 'groups', 'non-polar', 'solvents', 'used', 'lithium', 'magne-', 'sium', 'aluminum', 'counterions', 'yield', 'products', 'high', 'ketones', '46.', 'the', 'solvents', 'used', 'sodium', 'potassium', 'counterions', 'form', 'polymers', 'predominately', 'polyester', 'units', '46.', 'isocyanates', 'polymerize', 'carbon', 'nitrogen', 'double', 'bonds', 'anionic', 'mechanism', 'reactions', 'catalyzed', 'sodium', 'potassium', 'cyanide', '(cid1)58(cid5)c', '365.', 'nn0-dimethyl-', 'formamide', 'good', 'solvent', 'reaction', 'other', 'anionic', 'catalysts', 'ranging', 'alkali', 'salts', 'various', 'carboxylic', 'acids', '344', 'sodium-naphthalene', 'also', 'effective', '367.', 'in', 'addition', 'polymerizations', 'carried', 'cationic', '368,', 'thermal', '369,', 'radiation-induced', '369', 'methods', 'although', 'anionic', 'polymerizations', 'yield', 'high', 'molecular', 'weight', 'polymers', 'tendency', 'depolymerize', 'high', 'temperatures', 'the', 'products', 'anionic', 'polymerization', 'substituted', 'polyamides', '365', '(for', 'information', 'see', 'chap', '7)', '48', 'copolymerizations', 'ionic', 'mechanism', 'ionic', 'copolymerizations', 'complicated', 'free-radical', 'ones', 'various', 'complicating', 'factors', 'arise', 'effects', 'counterions', 'inﬂuences', 'solvents', 'these', 'affect', 'reactivity', 'ratios', 'in', 'addition', 'monomer', 'reactivity', 'affected', 'substituents', 'they', 'inﬂuence', 'electron', 'densities', 'double', 'bonds', 'cationic', 'polymerizations', 'resonance', 'stabilization', 'resultant', 'carbon', 'cations', 'also', 'effects', 'counter-ions', 'solvents', 'even', 'reaction', 'temperatures', 'even', 'greater', 'substituents', 'cationic', 'polymerizations', 'there', 'studies', 'reported', 'literature', 'reactivity', 'ratios', 'determined', 'different', '48', 'copolymerizations', 'ionic', 'mechanism', '229', 'monomers', 'using', 'temperature', 'solvent', 'counterion', 'one', 'study', 'carried', 'cationic', 'copolymerizations', 'styrene', 'two', 'substituted', 'styrenes', 'these', 'a-methylstyrene', 'chlorostyrene', '370–372.', 'the', 'relative', 'reactivity', 'ratios', 'substituted', 'styrenes', 'correlated', 'hammett', 'rs', 'values', 'the', 'effect', 'substituents', 'reactivity', 'styrene', 'falls', 'following', 'order', 'p', '-', 'och3p', '-', 'ch3p', '-', 'hp', '-', 'clm', '-', 'clm', '-', 'oh', 'this', 'information', 'however', 'useful', 'copolymerizations', 'substituted', 'styrene', 'monomers', 'cationic', 'copolymerization', 'studies', 'demonstrated', 'steric', 'factors', 'important', 'copolymerizations', '373–375.', 'for', 'instance', 'b-methyl', 'styrene', 'less', 'reactive', 'styrene', 'also', 'trans-b-methylstyrene', 'reactive', 'cis', 'isomer', 'the', 'effect', 'solvents', 'reactivity', 'ratios', 'cationic', 'copolymerizations', 'seen', 'copolymerizations', 'isobutylene', 'p-chlorostyrene', 'using', 'aluminum', 'bromide', 'initiator', '376.', 'the', 'r1', 'r2', 'values', 'hexane', 'isobutylene', 'chlorostyrene', 'copolymerization', 'equal', '10', 'in', 'nitrobenzene', 'however', 'r1', 'equal', '147', 'r2', 'equal', '015', 'in', 'copolymerization', 'styrene', 'p-methylstyrene', 'catalyzed', 'sbcl5', 'alx3', 'ticl4', 'sncl4', 'bf3', '•', 'oet2', 'sbcl3', 'cl3cooh', 'iodine', 'copolymer', 'composition', 'depends', 'upon', 'solvent', 'acid', 'strength', 'catalysts', 'there', 'difference', 'copolymer', 'composition', 'highly', 'polar', 'solvents', 'except', 'sbcl5', 'used', '377.', 'generally', 'amount', 'styrene', 'copolymer', 'decreases', 'solvent', 'polarity', 'increases', 'in', 'solvents', 'low', 'polarity', 'hand', 'amount', 'styrene', 'copolymer', 'decreases', 'decrease', 'strength', 'lewis', 'acids', 'this', 'depends', 'upon', 'amount', 'solvation', 'ion', 'pair', 'complexation', 'solvent', 'monomer', 'ion', 'pair', 'with', 'increase', 'tightness', 'ion', 'pairs', 'less', 'polar', 'solvents', 'selectivity', 'increases', 'because', 'p-methylstyrene', 'polar', 'complexes', 'greater', 'extent', 'styrene', 'polymerization', 'temperature', 'cationic', 'polymerizations', 'affects', 'monomer', 'reactivity', 'strongly', 'this', 'effect', 'considerably', 'greater', 'cationic', 'copolymerizations', 'free-radical', 'ones', 'no', 'general', 'trend', 'however', 'appears', 'established', 'far', 'electron', 'withdrawing', 'substituents', 'anionic', 'polymerizations', 'enhance', 'electron', 'density', 'double', 'bonds', 'stabilize', 'carbanions', 'resonance', 'anionic', 'copolymerizations', 'many', 'respects', 'behave', 'similarly', 'cationic', 'ones', 'for', 'comonomer', 'pairs', 'steric', 'effects', 'give', 'rise', 'tendency', 'alternate', '378.', 'the', 'reactivities', 'monomers', 'copolymerizations', 'compositions', 'resultant', 'copolymers', 'subject', 'solvent', 'polarity', 'effects', 'counterions', 'the', 'two', 'like', 'cationic', 'polymerizations', 'cannot', 'considered', 'independently', 'this', 'due', 'tightness', 'ion', 'pairs', 'amount', 'solvation', 'furthermore', 'monomers', 'possess', 'similar', 'polarity', 'copolymerized', 'anionic', 'mechanism', 'thus', 'instance', 'styrene', 'derivatives', 'copolymer-', 'ize', 'styrene', 'however', 'unable', 'add', 'methyl', 'methacrylate', 'anion', '379–381,', 'though', 'copolymerizes', 'butadiene', 'isoprene', 'in', 'copolymerizations', 'initiated', 'n-butyllithium', 'toluene', 'tetrahydrofuran', '(cid1)78(cid5)c', 'following', 'order', 'reactivity', 'methyl', 'methacrylate', 'anions', 'observed', '382.', 'in', 'toluene', 'order', 'diphenylmethyl', 'methacrylate', 'benzyl', 'methacrylate', 'methyl', 'methacrylate', 'ethyl', 'methacrylate', 'a-methylbenzyl', 'methacrylate', 'isopropyl', 'methacrylate', 't-', 'butyl', 'methacrylate', 'trityl', 'methacrylate', 'aa0-di-methylbenzyl', 'methacrylate', 'in', 'tetrahydrofuran', 'order', 'changes', 'trityl', 'methacrylate', 'benzyl', 'methacrylate', 'methyl', 'methacrylate', 'diphenylmethyl', 'methacrylate', 'ethyl', 'methacrylate', 'a-methylbenzyl', 'methacrylate', 'isopropyl', 'methacrylate', 'aa0-', 'dimethylbenzyl', 'methacrylate', 't-butyl', 'methacrylate', 'copolymerizations', 'styrene', 'butadiene', 'hydrocarbon', 'solvents', 'using', 'lithium', 'alkyls', 'initiators', 'initially', 'yield', 'copolymers', 'containing', 'mainly', 'butadiene', 'the', 'amount', 'styrene', 'copolymer', 'increases', 'considerably', 'however', 'tetrahydrofuran', 'solvent', 'hashimoto', 'coworkers', 'studied', 'anionic', 'living', 'copolymerization', 'ionic', 'mechanism', 'two', 'monomers', 'styrene', 'isoprene', 'dilute', 'benzene', 'solution', 'aid', 'combined', 'time-resolved', '230', '4', 'ionic', 'chain-growth', 'polymerization', 'measurements', 'small', 'angle', 'neutron', 'scattering', 'nmr', 'spectroscopy', 'size', 'exclusion', 'chromatography', 'ultraviolet–visible', 'spectroscopy', '383.', 'they', 'reported', 'observing', 'structural', 'changes', 'living', 'chains', 'polymerization', 'process', 'they', 'also', 'found', 'copolymerization', 'process', 'divided', 'two', 'time', 'regions', 'deﬁned', 'regions', 'i', 'ii', 'in', 'region', 'i', 'copolymerization', 'styrene', 'isoprene', 'monomers', 'occurred', 'i', 'monomers', 'consumed', 'end', 'region', 'i', 'region', 'ii', 'pure', 'polystyrene', 'block', 'chains', 'formed', 'in', 'beginning', 'region', 'ii', 'living', 'polymers', 'terminal', 'isoprenyl', 'anion', 'started', 'add', 'styrene', 'monomer', 'rapidly', 'changed', 'chains', 'terminal', 'styryl', 'anions', 'this', 'followed', 'slow', 'change', 'polyisoprene', 'polystyrene', 'latter', 'part', 'region', 'ii', 'as', 'consequence', 'polymerization', 'styrene', 'monomers', 'presence', 'polyisoprene', 'polystyrene', 'found', 'increase', 'mwmn', 'in', 'addition', 'also', 'found', 'conversion', 'rate', 'styrene', 'monomers', 'conditions', 'slow', 'compared', 'one', 'homopolymerization', 'anionic', 'copolymerizations', 'useful', 'forming', 'block', 'copolymers', '(see', 'chap', '9', 'discus-', 'sion)', 'ziegler–natta', 'catalysts', 'also', 'form', 'block', 'copolymers', 'similarly', 'anionic', 'initiators', 'much', 'work', 'copolymerization', 'coordinated', 'anionic', 'initiators', 'done', 'develop', 'ethylene', 'propylene', 'copolymers', 'ethylene', 'considerably', 'reactive', 'copolymerizations', 'to', 'form', 'random', 'copolymers', 'soluble', 'ziegler–natta', 'catalysts', 'used', 'this', 'aided', 'carefully', 'controlling', 'monomer', 'feed', '384.', 'the', '12-disubstituted', 'oleﬁnic', 'monomers', 'usually', 'homopolymerize', 'ziegler–natta', 'catalysts', 'they', 'however', 'copolymerized', 'ethylene', 'a-oleﬁns', '384.', 'due', 'poorer', 'reactivity', 'monomer', 'feed', 'must', 'consist', 'higher', 'ratios', '12-disubstituted', 'oleﬁns', 'comonomers', 'copolymers', 'cis-2', 'butene', 'ethylene', 'portions', 'macromolecules', 'crystalline', 'form', 'vanadium', 'based', 'catalysts', 'the', 'products', 'alternating', 'structures', 'pendant', 'methyl', 'groups', 'erythrodiisotactic', 'arrangements', '197.', 'similarly', 'vanadium', 'based', 'catalysts', 'yield', 'alternating', 'copolymers', 'ethylene', 'butadiene', 'butadi-', 'ene', 'placement', 'predominantly', 'trans-14', '197.', 'giuronett', 'mecking', 'carried', 'copolymerizations', 'ethylene', '1-oleﬁns', 'supercritical', 'carbon', 'dioxide', 'electron', 'poor', 'nickel', 'complexes', '385.', 'the', 'catalyst', 'illustrated', 'follows', 'f3c', 'n', 'o', 'ni', 'n', 'o', 'cf3', 'f3c', 'f3c', 'cf3', 'cf3', 'they', 'reported', 'catalyst', 'showed', 'high', 'activity', 'supercritical', 'co2', 'yielding', 'high', 'molecular', 'weight', 'copolymers', 'ethylene', '1-hexene', 'norbornene', '385.', 'copolymerizations', 'aldehydes', 'take', 'place', 'anionic', 'cationic', 'mechanisms', 'an', 'elastic', 'copolymer', 'formaldehyde', 'acetaldehyde', 'forms', 'triisobutylaluminum', 'the', 'rate', 'copoly-', 'merization', 'rapid', '(cid1)78(cid5)c', 'the', 'reaction', 'complete', 'within', '30', 'min', '364.', 'the', 'product', 'however', 'crosslinked', 'aldehydes', 'also', 'copolymerize', 'vinyl', 'monomers', '386.', 'an', 'acetone', 'block', 'copolymer', 'forms', '387', 'propylene', 'ziegler–natta', 'catalysts', 'used', '(cid1)78(cid5)c', 'copolymers', 'acetone', 'oleﬁns', 'formaldehyde', 'also', 'prepared', '388', '389.', 'many', 'initiators', 'effective', 'copolymerizations', 'aldehydes', 'ketones', 'epoxies', '387', '390.', '49', 'group', 'transfer', 'polymerization', '231', 'o’driscoll', '407', 'developed', 'kinetics', 'ionic', 'copolymerization', 'dissimilar', 'monomers', 'it', 'based', 'assumption', 'ratio', 'monomers', 'copolymer', 'dm1/dm2', 'directly', 'proportional', 'square', 'initial', 'monomer', 'rate', '(m1/m2)2', 'copolymerization', 'reaction', 'd½m1(cid3)d½m2(cid3)', '¼', 'ðk1k2þðk11k22þð½m1(cid3)½m2(cid3)þ2', 'the', 'shown', 'equation', 'derived', 'assumption', 'initial', 'homopropagation', 'fast', 'relative', 'propagation', 'also', 'crossover', 'reaction', 'negligible', 'beginning', 'copolymerization', '49', 'group', 'transfer', 'polymerization', 'this', 'technology', 'offers', 'considerable', 'promise', 'commercial', 'preparations', 'living', 'polymers', 'methyl', 'methacrylate', 'without', 'resorting', 'low', 'temperature', 'anionic', 'polymerizations', 'although', 'mechanism', 'polymerization', 'completely', 'explained', 'propagation', 'generally', 'believed', 'covalent', 'character', 'a', 'silyl', 'ketene', 'acetal', 'initiator', 'it', 'forms', 'ester', 'enolate', '391:', 'o', 'och3', 'si', 'cl', 'n', 'li', '2', 'o', 'si', 'och3', 'o', 'li', 'och3', 'the', 'initiation', 'catalyzed', 'either', 'nucleophilic', 'lewis', 'acid', 'catalyst', 'explained', 'consisting', 'concerted', 'attack', 'ketene', 'acetal', 'monomer', '392:', 'o', 'o', 'och3', 'och3', 'o', 'si', 'o', 'o', 'si', 'och3', 'this', 'results', 'transfer', 'silyl', 'ketene', 'acetal', 'center', 'monomer', 'the', 'process', 'repeated', 'step', 'propagation', 'the', 'ketone', 'double', 'bond', 'acts', 'propagating', 'center', '391.', 'o', 'och3', 'o', 'n', 'o', 'och3', 'o', 'si', 'o', 'o', 'och3', 'si', 'o', 'n', '1', 'o', 'och3', '232', '4', 'ionic', 'chain-growth', 'polymerization', 'the', 'mechanism', 'shows', 'step', 'chain-growth', 'involving', 'transfer', 'trialkylsilyl', 'group', 'silyl', 'ketene', 'acetal', 'chain', 'end', 'carbonyl', 'group', 'incoming', 'monomer', 'this', 'disputed', 'excludes', 'silyl', 'exchange', 'growing', 'chains', '393.', 'such', 'exchange', 'however', 'observed', '392', 'led', 'suggestion', 'mechanism', 'involve', 'ester', 'enolate', 'anion', 'intermediates', 'these', 'reversibly', 'complexed', 'silyl', 'ketene', 'acetal', 'chain', 'ends', 'propagating', 'species', '394.', 'it', 'noted', 'however', 'results', 'exclude', 'possibility', 'transfer', 'reactions', 'occurring', 'well', 'difunctional', 'initiators', 'cause', 'chain', 'growth', 'proceed', 'end', '395.', 'because', 'group-transfer', 'polymerizations', '“living”', 'polymerization', 'monomer', 'consumed', 'different', 'monomer', 'added', 'block', 'copolymers', 'formed', 'the', 'effective', 'nucleophilic', 'catalysts', 'reaction', 'biﬂuoride', '(hf2', '(cid1))', 'ﬂuoride', 'ions', 'they', 'generated', 'soluble', 'reagents', 'like', 'tris(dimethylamino)sulfonium', 'biﬂuoride', 'other', 'nucleophiles', 'like', 'cn(cid1)', 'nitrophenolate', 'also', 'used', 'these', 'nucleophilic', 'catalysts', 'function', 'assisting', 'displacement', 'trialkylsilyl', 'group', 'they', 'effective', 'concentrations', '01', 'mol', 'initiator', 'among', 'electrophilic', 'catalysts', 'lewis', 'acids', 'like', 'zinc', 'chloride', 'zinc', 'bromide', 'zinc', 'iodide', 'dialkylaluminum', 'chloride', 'such', 'catalysts', 'probably', 'function', 'coordinating', 'carbonyl', 'oxygens', 'monomers', 'increasing', 'electrophilicity', 'double', 'bonds', 'this', 'makes', 'reactive', 'nucleophilic', 'reagents', 'they', 'must', 'used', 'however', 'much', 'higher', 'concentrations', 'water', 'compounds', 'active', 'hydrogen', 'must', 'excluded', 'reaction', 'medium', 'oxygen', 'hand', 'interfere', 'reaction', 'tetrahydrofuran', 'acetonitrile', 'aromatic', 'solvents', 'commonly', 'used', 'polymerizations', 'catalyzed', 'nucleophiles', 'chlorinated', 'solvents', 'dimethylformamide', 'utilized', 'many', 'reactions', 'catalyzed', 'electrophiles', 'living', 'polymerizations', 'methacrylate', 'esters', 'carried', '0–50(cid5)c', 'the', 'acrylate', 'esters', 'however', 'require', 'temperatures', '0(cid5)c', 'living', 'group-transfer', 'polymerizations', 'reactive', 'undergo', 'side', 'reactions', 'weakly', 'acidic', 'compounds', 'methyl', 'a-phenylacetate', 'a-phenylpropionitrile', 'added', 'terminate', 'reaction', 'they', 'effective', 'anionic', 'catalysts', '396.', 'the', 'trialkylsilyl', 'group', 'transferred', 'chain', 'end', 'transfer', 'agent', 'o', 'si', 'och3', 'hf2', 'ah', '(acid)', 'o', 'a', 'si', 'och3', 'group-transfer', 'polymerizations', 'yield', 'narrow', 'molecular', 'weight', 'distribution', 'polymers', 'when', 'mixtures', 'monomers', 'used', 'random', 'copolymers', 'form', 'the', 'polymerization', 'reaction', 'tolerant', 'functional', 'groups', 'monomer', 'thus', 'instance', 'p-vinylbenzyl', 'methacrylate', 'converted', 'poly(p-vinylbenzyl', 'methacrylate)', 'without', 'polymerization', 'vinyl', 'group', '397.', 'in', 'addition', 'possible', 'form', 'polymers', 'high', 'syndiotactic', 'content', 'an', 'example', 'copolymerization', 'methyl', 'methacrylate', 'methyl', 'a-phenylacrylate', '398.', 'while', 'homopolymer', 'formed', 'methyl', 'a-phenylacrylate', 'alternating', 'copolymers', 'obtained', 'mixtures', 'two', 'equimolar', 'incorporation', 'monomers', 'reported', 'even', 'methyl', 'methacrylate', 'excess', 'long', 'methyl', 'a-phenylacrylate', 'present', 'reaction', 'mixture', 'the', 'propagation', 'constants', 'found', 'decrease', 'order', 'k12', '(cid14)', 'k11k21', '(cid14)', 'k22', 'block', 'copolymers', 'alternating', 'block', 'poly(methyl', 'methacrylate)', 'block', 'formed', 'monomer', 'feed', 'mma', 'excess', '398.', 'a', 'variation', 'group', 'transfer', 'polymerization', 'n-butyl', 'acrylate', 'reported', '399.', 'here', '1-methoxy-1(trimethylsiloxy)-2-methyl-1-propene', 'used', 'initiator', 'hgi2', 'catalyst', 'this', 'differs', 'nucleophilic-catalyzed', 'group', 'transfer', 'polymerizations', 'described', '49', 'group', 'transfer', 'polymerization', '233', 'the', 'half-lives', 'reported', 'range', 'minutes', 'hours', 'induction', 'periods', 'observed', 'formation', 'trimethylsilyl', 'iodide', 'proposed', 'cause', 'induction', 'period', '399.', 'the', 'polymerization', 'follows', 'ﬁrst-order', 'kinetics', 'respect', 'concentrations', 'initiator', 'catalyst', 'with', 'respect', 'concentration', 'monomer', 'reaction', 'described', 'ﬁrst-order', 'internally', 'it', 'follows', 'however', 'external', 'reaction', 'order', '152', 'due', 'higher', 'polarity', 'reaction', 'medium', 'higher', 'monomer', 'concentrations', 'the', 'authors', 'tentatively', 'propose', 'active', 'species', 'formed', 'initiator', 'catalyst', 'trimethylsilyl', 'iodide', '399:', 'osi(ch3)3', 'hgi2', 'och3', 'i', 'i', 'hg', 'osi(ch3)3', 'och3', 'hgi', 'o', 'och3', 'ohgi', 'och3', '(ch3)3sih', 'part', 'proposed', 'mechanism', 'tmsi', 'forms', 'mts', 'iodine', 'it', 'assumed', 'therefore', 'tmsi', 'forms', 'complex', 'hgi2', 'turn', 'activates', 'mts', 'form', 'active', 'center', 'capable', 'initiating', 'polymerization', '399:', '(ch3)3sii', 'hgi2', '(ch3)3si', 'i', 'hgi2', 'k1', 'osi(ch3)3', 'och3', '(ch3)3si', 'i', 'hgi2', 'k2', '(ch3)3si', 'i', 'hg', 'i', 'i', 'o', 'si(ch3)3', 'och3', 'because', 'group', 'transfer', 'polymerization', 'another', 'form', 'living', 'polymerization', 'attempts', 'made', 'write', 'kinetic', 'equations', 'include', 'forms', 'polymerizations', 'one', 'uniﬁed', 'scheme', 'livinenko', 'muller', '400', 'carried', 'general', 'kinetic', 'analysis', 'compared', 'molecular', 'weight', 'distributions', 'various', 'mechanisms', 'activity', 'exchange', 'living', 'polymerizations', 'they', 'concluded', 'molecular', 'weight', 'distributions', 'many', 'living', 'eg', 'anionic', 'group', 'transfer', 'cationic', 'radical', 'polymerizations', 'strongly', 'depend', 'dynamics', 'various', 'equilibria', 'chain', 'ends', 'different', 'reactivity', '400.', 'they', 'also', 'concluded', 'important', 'special', 'case', 'equilibria', 'active', 'dormant', 'centers', '400.', 'mechanisms', 'include', 'uni-', 'bimolecular', 'isomerizations', '(or', 'activationsdeactivations)', 'aggregation', 'direct', 'bimolecular', 'activity', 'exchange', 'fast', 'slow', 'monomer', 'addition', 'uniﬁed', 'the', 'averages', 'molecular', 'weight', 'distributions', 'polydispersity', 'indexes', 'pwpn', 'derived', 'dependencies', 'averages', 'three', 'universal', 'parameters', 'analyzed', '(1)', 'reactivity', 'ratio', 'two', 'species', 'l', '¼', 'k0', 'p', '(2)', 'fraction', 'active', 'species', '¼', 'p/i0', 'determined', 'initial', 'concentrations', 'reagents', '(3)', 'generalized', 'exchange', 'rate', 'parameter', 'b', 'quantiﬁes', 'rate', 'exchange', 'relative', 'propagation', 'the', 'dependence', 'b', 'initial', 'concentrations', 'reagents', 'deﬁned', 'mechanism', 'exchange', 'used', 'mechanistic', 'criterion', 'distinguish', 'various', 'possible', 'mechanisms', 'livinenko', 'miller', 'concluded', 'cases', 'b', '1', 'polydispersity', 'indexes', 'decrease', 'monomer', 'conversions', 'this', 'common', 'observation', 'many', 'living', 'polymerizations', '10', 'b', '100', 'at', 'full', 'conversions', 'simple', 'relation', 'pwpn', '(cid15)', '1', 'þ', 'yb', 'valid', 'y', 'depends', 'l', 'for', 'common', 'cases', 'one', 'pk(cid4)', '234', '4', 'ionic', 'chain-growth', 'polymerization', 'type', 'species', 'dormant', 'simpliﬁed', 'pwpn', '(cid15)', '1', 'þ', '1b', 'generally', 'molecular', 'weight', 'distributions', 'narrower', 'monomer', 'added', 'slowly', '400.', 'y', '¼', 'a2ð1', '(cid1)', 'aþð1', '(cid1)', 'lþ½a', 'þ', 'lð1', '(cid1)', 'aþ(cid3)2', 'y1', 'scholten', 'et', 'al', '401', 'reported', 'n-heterocyclic', 'carbenes', 'potent', 'catalysts', 'group', 'transfer', 'polymerization', 'acrylic', 'esters', 'the', 'reaction', 'illustrated', 'follows', 'r', 'o', 'o', 'o', 'si', 'o', 'r', 'r', 'r', 'n', 'o', 'o', 'h', 'o', 'o', 'o', 'o', 'n', 'n', 'n', 'n', '01–05', 'moles', 'catalyst', 'used', 'tetrahydrofuran', 'the', 'reaction', 'quenched', 'methyl', 'alcohol', 'the', 'product', 'narrow', 'molecular', 'weight', 'distribution', '410', 'conﬁgurational', 'statistics', 'propagation', 'mechanism', 'chain-growth', 'polymerization', 'analyses', 'polymers', 'determine', 'stereosequence', 'distributions', 'understand', 'propagation', 'mechanism', 'carried', 'nmr', 'spectroscopy', 'aided', 'statistical', 'propagation', 'models', '222', '402', '403.', 'a', 'detailed', 'discussion', 'subject', 'beyond', 'book', 'the', 'following', 'brief', 'explanation', 'concepts', 'the', 'bernoulli', 'markov', 'colman-fox', 'models', 'describe', 'propagation', 'reactions', 'chain', 'end', 'control', 'monomer', 'placement', 'the', 'bernoulli', 'model', 'assumes', 'last', 'monomer', 'unit', 'propagating', 'chain', 'end', 'determines', 'stereochemistry', 'polymer', 'no', 'consideration', 'given', 'penultimate', 'unit', 'units', 'back', 'in', 'event', 'two', 'modes', 'propagation', 'possible', 'meso', 'racemic', 'the', 'statistical', 'probabilities', 'pm', 'pr', 'called', 'conditional', 'probabilities', 'transitions', 'forming', 'meso', 'racemic', 'dyads', 'deﬁned', 'following', 'equations', 'pm', 'pr', 'pm', '¼', 'rmðrm', 'þ', 'rrþ', 'pr', '¼', 'rrðrm', 'þ', 'rrþ', 'pm', 'þ', 'pr', '¼', '1', '410', 'conﬁgurational', 'statistics', 'propagation', 'mechanism', 'chain-growth', 'polymerization', '235', 'rm', 'rr', 'rates', 'meso', 'racemic', 'dyad', 'placement', 'these', 'dyads', 'isotactic', 'syndiotactic', 'one', 'another', 'such', 'dyads', 'frequently', 'called', 'meso', 'racemic', 'isotactic', '(meso)', 'syndiotactic', '(racemic)', 'the', 'probabilities', 'represent', 'dyad', 'tactic', 'fractions', 'r', 'the', 'triad', 'tacticity', 'represent', 'isotactic', 'syndiotactic', 'heterotactic', '(or', 'atactic)', 'arrangements', 'they', 'designated', 'mm', 'rr', 'mr', 'respectively', 'one', 'way', 'understand', 'deﬁnitions', 'examining', 'representation', 'section', 'polymeric', 'chain', 'total', '9', 'repeating', 'units', '8', 'dyads', '7', 'triads', 'shown', 'in', 'segment', '2', 'racemic', 'dyads', '6', 'meso', 'dyads', 'in', 'addition', 'also', '4', 'isotactic', '1', 'syndiotactic', '2', 'atactic', 'dyads', 'the', 'triads', 'deﬁned', 'follows', 'ðmmþ', '¼', 'p2', 'ðmrþ', '¼', '2pmð1', '(cid1)', 'pmþ', 'ðrrþ', '¼', 'ð1', '(cid1)', 'pmþ2', 'note', 'atactic', 'triad', 'produced', 'two', 'ways', 'mr', 'rm', 'the', 'tetrad', 'probabilities', 'deﬁned', 'ðmmmþ', '¼', 'p3', 'ðmrmþ', '¼', 'p2', 'mð1', '(cid1)', 'pmþ', 'ðmmrþ', '¼', '2p2', 'mð1', '(cid1)', 'pmþ', 'ðrrmþ', '¼', '2pmð1', '(cid1)', 'pmþ2', 'ðrmrþ', '¼', 'pmð1', '(cid1)', 'pmþ2', 'ðrrrþ', '¼', 'ð1', '(cid1)', 'pmþ3', 'the', 'pentad', 'probabilities', 'given', 'ðmmmmþ', '¼', 'p4', 'ðrmrmþ', '¼', '2p2', 'mð1', '(cid1)', 'pmþ2', 'ðmmmrþ', '¼', '2p3', 'mð1', '(cid1)', 'pmþ', 'ðrmrrþ', '¼', '2pmð1', '(cid1)', 'pmþ3', 'ðrmmrþ', '¼', 'p2', 'mð1', '(cid1)', 'pmþ2', 'ðmrrmþ', '¼', 'p2', 'mð1', '(cid1)', 'pmþ2', 'ðmmrmþ', '¼', '2p3', 'mð1', '(cid1)', 'pmþ', 'ðrrrmþ', '¼', '2pmð1', '(cid1)', 'pmþ3', 'ðmmrmþ', '¼', '2p2', 'mð1', '(cid1)', 'pmþ', 'ðrrrrþ', '¼', 'ð1', '(cid1)', 'pmþ4', '236', '4', 'ionic', 'chain-growth', 'polymerization', 'in', 'bernoulli', 'model', 'inﬂuence', 'stereochemistry', 'growing', 'polymeric', 'chain', 'incoming', 'monomer', 'the', 'ﬁrst', 'order', 'markov', 'model', 'hand', 'based', 'concept', 'adding', 'monomer', 'inﬂuenced', 'stereochemistry', 'growing', 'end', 'this', 'end', 'may', 'r', 'this', 'means', 'four', 'probabilities', 'characterizing', 'addition', 'process', 'the', 'monomer', 'add', 'fashion', 'chain', 'end', 'r', 'chain', 'end', 'these', 'designated', 'pmm', 'prm', 'pmr', 'prr', 'these', 'modes', 'propagation', 'illustrated', 'follows', 'pmm', 'pmr', 'prm', 'prr', 'pmm', 'þ', 'pmr', '¼', '1', 'prm', 'þ', 'prr', '¼', '1', 'ðmþ', '¼', 'prmðpmr', 'þ', 'prmþ', 'ðrþ', '¼', 'pmrðpmr', 'þ', 'prmþ', 'there', 'four', 'probabilities', 'propagation', 'pmm', 'pmr', 'prm', 'prr', 'with', 'conservation', 'relationship', 'write', 'we', 'recast', 'dyads', 'triads', 'tetrads', 'pentads', 'terms', 'pmr', 'prm', 'the', 'dyads', 'functions', 'the', 'triad', 'fractions', 'there', 'six', 'tetrad', 'functions', 'ðmmþ', '¼', 'ð1', '(cid1)', 'pmrþprmðpmr', 'þ', 'prmþ', 'ðmrþ', '¼', '2pmrprmðpmr', 'þ', 'prmþ', 'ðrrþ', '¼', 'ð1', '(cid1)', 'prmþpmrðpmr', '¼', 'prmþ', 'ðmmmþ', '¼', 'prmð1', '(cid1)', 'pmrþ2ðpmr', 'þ', 'prmþ', 'ðmmrþ', '¼', '2pmrprmð1', '(cid1)', 'pmrþðpmr', 'þ', 'prmþ', 'ðmrmþ', '¼', 'pmrp2', 'rmðpmr', 'þ', 'prmþ', '410', 'conﬁgurational', 'statistics', 'propagation', 'mechanism', 'chain-growth', 'polymerization', '237', 'ten', 'pentad', 'functions', 'ðmrrþ', '¼', '2pmrprmð1', '(cid1)', 'prmþðpmr', 'þ', 'prmþ', 'ðrmrþ', '¼', 'p2', 'mrprmðpmr', 'þ', 'prmþ', 'ðrrrþ', '¼', 'pmrð1', '(cid1)', 'prmþ2', 'pmr', 'þ', 'prm', '(cid2)', '(cid3)', 'ðmmmmþ', '¼', 'prmð1', '(cid1)', 'pmrþ3ðpmr', 'þ', 'prmþ', 'ðmmmrþ', '¼', '2pmrprmð1', '(cid1)', 'pmrþ2ðpmr', 'þ', 'prmþ', 'ðrmrmþ', '¼', '2p2', 'mrp2', 'rmðpmr', 'þ', 'prmþ', 'ðrmrrþ', '¼', '2p2', 'mrprmð1', '(cid1)', 'prmþðpmr', 'þ', 'prmþ', 'ðrmmrþ', '¼', 'p2', 'mrprmð1', '(cid1)', 'pmrþðpmr', 'þ', 'prmþ', 'ðmrrmþ', '¼', 'pmrp2', 'rmð1', '(cid1)', 'prmþðpmr', 'þ', 'prmþ', 'ðmmrmþ', '¼', '2pmrp2', 'rmð1', '(cid1)', 'pmrþðpmr', 'þ', 'prmþ', 'ðmrrrþ', '¼', '2pmrprmð1', '(cid1)', 'prmþ2ðpmr', 'þ', 'prmþ', 'ðmmrrþ', '¼', '2pmrprmð1', '(cid1)', 'pmrþð1', '(cid1)', 'prmþðpmr', 'þ', 'prmþ', 'ðrrrrþ', '¼', 'pmrð1', '(cid1)', 'prmþ3ðpmr', 'þ', 'prmþ', 'the', 'second', 'order', 'markov', 'model', 'requires', 'speciﬁcation', 'eight', 'conditional', 'probabilities', 'this', 'due', 'inﬂuence', 'last', 'three', 'pseudo', 'asymmetric', 'centers', 'growing', 'chain', 'the', 'details', 'second', 'markov', 'described', 'bovey', '403.', 'for', 'details', 'reader', 'advised', 'consult', 'reference', 'for', 'convenience', 'eight', 'conditional', 'probabilities', 'designated', 'greek', 'letters', '403:', 'pðmmmþ', '¼', 'pðmmrþ', '¼', 'a0', 'pðmrmþ', '¼', 'b', 'pðmrrþ', '¼', 'b0', 'pðrmmþ', '¼', 'g', 'pðrmrþ', '¼', 'g0', 'pðrrmþ', '¼', 'pðrrmþ', '¼', 'd0', 'in', 'designation', 'p(mmm)', 'probability', 'monomer', 'adding', 'chain', 'end', 'manner', 'p(mmr)', 'probability', 'adding', 'chain', 'r', 'manner', 'because', 'þ', 'a0', '¼', '1', 'b', 'þ', 'b0', '¼', '1', 'g', 'þ', 'g0', '¼', '1', 'þ', 'd0', '¼', '1', '238', '4', 'ionic', 'chain-growth', 'polymerization', 'four', 'independent', 'probabilities', 'when', '¼', 'g', 'b', '¼', 'model', 'reduces', 'ﬁrst', 'order', 'markov', 'model', 'the', 'coleman-fox', 'model', 'attempts', 'explain', '“block', 'like”', 'conﬁgurations', 'exhibited', 'varying', 'degrees', 'propagating', 'species', 'deviate', 'bernoullian', 'statistics', 'they', 'proposed', '“block”', 'conﬁgurations', 'generated', 'propagating', 'chain', 'ends', 'might', 'exist', 'two', 'states', 'these', 'states', 'would', 'correspond', 'chelation', 'counterion', 'interruption', 'chelation', 'solvation', 'here', 'details', 'reader', 'advised', 'go', 'original', 'literature', '403.', 'the', 'enantiomorphic', 'site', 'control', 'based', 'probability', 'monomer', 'adding', 'either', 'r', 's', 'site', 'catalyst', 'the', 'propagation', 'occurs', 'faces', 'r', 's', 'monomers', 'the', 'model', 'described', 'terms', 'single', 'parameter', 'commonly', 'designated', '404.', 'it', 'probability', 'r', 'monomer', 'adding', 'r', 'site', 's', 'monomer', 'adding', 's', 'site', 'the', 'dyads', 'described', 'ðmþ', '¼', 's2', 'þ', 'ð1', 'þ', 'sþ2', 'ðrþ', '¼', '2sð1', '(cid1)', 'sþ', 'the', 'triads', 'functions', 'given', 'the', 'tetrad', 'fractions', 'ðmmþ', '¼', '1', '(cid1)', '3sð1', '(cid1)', 'sþ', 'ðmrþ', '¼', '2sð1', '(cid1)', 'sþ', 'ðrrþ', '¼', 'sð1', '(cid1)', 'sþ', 'pentad', 'fractions', 'ðmmmþ', '¼', '2s4', '(cid1)', '4s3', 'þ', '6s2', '(cid1)', '4s', 'þ', '1', 'ðmmrþ', '¼', 'ðmrrþ', '¼', '(cid1)4s4', 'þ', '8s3', '(cid1)', '6s2', 'þ', '2s', 'ðmrmþ', '¼', 'ðrmrþ', '¼', 'ðrrrþ', '¼', '2s4', '(cid1)', '4s3', 'þ', '2s2', 'ðmmmmþ', '¼', '5s4', '(cid1)', '10s3', 'þ', '10s2', '(cid1)', '5s', 'þ', '1', 'ðmrrmþ', '¼', '(cid1)3s4', 'þ', '6s3', '(cid1)', '4s2', 'þ', 'ðmmmrþ', '¼', 'ðmmrrþ', '¼', '(cid1)6s4', 'þ', '12s3', '(cid1)', '8s2', 'þ', '2s', 'ðrmmrþ', '¼', 'ðrrrrþ', '¼', 's4', '(cid1)', '2s3', 'þ', 's2', 'ðmmrmþ', '¼', 'ðrmrrþ', '¼', 'ðrmrmþ', '¼', 'ðmrrrþ', '¼', '2s4', '(cid1)', '4s3', 'þ', '2s2', 'nmr', 'spectroscopy', 'allows', 'testing', 'whether', 'particular', 'polymerization', 'propagation', 'follows', 'bernoulli', 'markov', 'enantiomorphic', 'statistical', 'form', 'attempts', 'usually', 'made', 'ﬁt', 'data', 'dyads', 'triads', 'tetrads', 'higher', 'sequence', 'fractions', 'equation', 'different', 'models', 'spectral', 'intensities', 'associated', 'theoretical', 'expression', 'involving', 'reaction', 'probability', 'parameters', 'theoretical', 'intensities', 'compared', 'observed', 'ones', 'this', 'optimized', 'obtain', 'best-ﬁt', 'values', 'reaction', 'probability', 'parameters', 'fully', 'characterize', 'structure', 'macromolecule', 'the', 'ﬁtting', 'data', 'carried', 'aid', 'computers', '410', 'conﬁgurational', 'statistics', 'propagation', 'mechanism', 'chain-growth', 'polymerization', '239', 'by', 'determining', 'statistical', 'model', 'followed', 'polymerization', 'bernoullian', 'markov', 'others', 'possible', 'understand', 'better', 'mechanism', 'steric', 'control', 'thus', 'bernoulli', 'model', 'describes', 'reactions', 'chain', 'ends', 'determine', 'steric', 'arrangement', 'these', 'polymerizations', 'carried', 'conditions', 'yield', 'mostly', 'atactic', 'polymers', 'the', 'high', 'isotactic', 'sequences', 'follow', 'enantiomorphic', 'site', 'model', 'high', 'syndiotactic', 'ones', 'usually', 'follow', 'markov', 'models', 'the', 'dyads', 'described', 'ðmþ', '¼', 's2', 'þ', 'ð1', 'þ', 'sþ2', 'ðrþ', '¼', '2sð1', '(cid1)', 'sþ', 'the', 'triads', 'functions', 'given', 'the', 'tetrad', 'fractions', 'ðmmþ', '¼', '1', '(cid1)', '3sð1', '(cid1)', 'sþ', '(mr)', '¼', '2sð1', '(cid1)', 'sþ', '(rr)', '¼', 'sð1', '(cid1)', 'sþ', 'pentad', 'fractions', 'ðmmmþ', '¼', '2s4', '(cid1)', '4s3', 'þ', '6s2', '(cid1)', '4s', 'þ', '1', 'ðmmrþ', '¼', 'ðmrrþ', '¼', '(cid1)4s4', 'þ', '8s3', '(cid1)', '6s2', 'þ', '2s', 'ðmrmþ', '¼', 'ðrmrþ', '¼', 'ðrrrþ', '¼', '2s4', '(cid1)', '4s3', 'þ', '2s2', 'ðmmmmþ', '¼', '5s4', '(cid1)', '10s3', 'þ', '10s2', '(cid1)', '5s', 'þ', '1', 'ðmrrmþ', '¼', '(cid1)3s4', 'þ', '6s3', '(cid1)', '4s2', 'þ', 'ðmmmrþ', '¼', 'ðmmrrþ', '¼', '(cid1)6s4', 'þ', '12s3', '(cid1)', '8s2', 'þ', '2s', 'ðrmmrþ', '¼', 'ðrrrrþ', '¼', 's4', '(cid1)', '2s3', 'þ', 's2', 'ðmmrmþ', '¼', 'ðrmrrþ', '¼', 'ðrmrmþ', '¼', 'ðmrrrþ', '¼', '2s4', '(cid1)', '4s3', 'þ', '2s2', 'nmr', 'spectroscopy', 'allows', 'testing', 'whether', 'particular', 'polymerization', 'propagation', 'follows', 'bernoulli', 'markov', 'enantiomorphic', 'statistical', 'form', 'attempts', 'usually', 'made', 'ﬁt', 'data', 'dyads', 'triads', 'tetrads', 'higher', 'sequence', 'fractions', 'equation', 'different', 'models', 'spectral', 'intensities', 'associated', 'theoretical', 'expression', 'involving', 'reaction', 'probability', 'parameters', 'theoretical', 'intensities', 'compared', 'observed', 'ones', 'this', 'optimized', 'obtain', 'best-ﬁt', 'values', 'reaction', 'probability', 'parameters', 'fully', 'characterize', 'structure', 'macromolecule', 'the', 'ﬁtting', 'data', 'carried', 'aid', 'computers', 'cheng', 'published', 'useful', 'program', 'basic', 'language', 'applies', 'model-ﬁtting', 'approach', 'various', 'polymeric', 'systems', '405.', 'many', 'programs', 'various', 'computer', 'languages', 'available', 'today', 'by', 'determining', 'statistical', 'model', 'followed', 'polymerization', 'bernoullian', 'markov', 'others', 'possible', 'understand', 'better', 'mechanism', 'steric', 'control', 'thus', 'bernoulli', 'model', 'describes', 'reactions', 'chain', 'ends', 'determine', 'steric', 'arrangement', 'these', 'polymerizations', 'carried', 'conditions', 'yield', 'mostly', 'atactic', 'polymers', 'the', 'high', 'isotactic', 'sequences', 'follow', 'enantiomorphic', 'site', 'model', 'high', 'syndiotactic', 'ones', 'usually', 'follow', 'markov', 'models', '240', '4', 'ionic', 'chain-growth', 'polymerization', '411', 'thermodynamics', 'equilibrium', 'polymerization', 'the', 'effect', 'solvents', 'equilibrium', 'constants', 'anionic', 'polymerization', 'solution', 'considered', 'thermodynamic', 'terms', '407.', 'sawada', '407', 'expressed', 'change', 'free', 'energy', 'converting', 'one', 'mole', 'liquid', 'monomer', 'anionic', 'polymerization', 'one', 'base-mole', 'amorphous', 'polymer', 'df1c', 'follows', 'df1c', '¼', 'dfm', '(cid1)', 'dfp', 'dfm', 'partial', 'molar', 'free', 'energy', 'monomer', '(per', 'mole)', 'equilibrium', 'mixture', 'relative', 'pure', 'liquid', 'monomer', 'dfp', 'partial', 'molar', 'free', 'energy', 'polymer', '(per', 'base', 'mole)', 'equilibrium', 'mixture', 'relative', 'amorphous', 'polymer', 'flory', '409', 'described', 'dfm', 'dfp', 'three', 'component', 'system', 'polymer', 'monomer', 'solvent', 'follow', 'dfmrt', '¼', 'ln', '’m', 'þ', 'ð1', '(cid1)', '’mþ', '(cid1)', '’sðxmxpþ', '(cid1)', '’pðxmxpþ', 'þ', 'ðwms’s', 'þ', 'wms’pþð’s', 'þ', '’pþ', 'dfprt', '¼', '1n½ln', '’p', 'þ', 'ð1', '(cid1)', '’pþ', '(cid1)', '’mðxpxmþ', '(cid1)', '’sðxpxmþ', 'þ', 'ðwpm’m', 'þ', 'wps’sþð’m', 'þ', '’sþ', '(cid1)', 'wspðxmxpþ’s’p', '(cid1)', 'wmsðxpxmþ’m’s', '’i', 'volume', 'fraction', 'component', 'xi', 'number', 'segments', 'per', 'molecule', 'component', 'n', 'degree', 'polymerization', 'r', 'gas', 'constant', 't', 'temperature', 'w', 'free', 'energy', 'parameter', 'two', 'components', 'the', 'subscripts', 'p', 'indicate', 'monomer', 'solvent', 'polymer', 'respectively', 'flory', 'puts', 'ration', 'molar', 'volume', 'assuming', 'vpvm', '¼', 'n', 'expressing', 'wpm', 'wps', 'terms', 'molecular', 'weight-independent', 'quantities', 'wmp', 'wsp', 'relationship', 'vivj', '¼', 'xixj', 'wpm', '¼', 'wmpðvpvmþ', '¼', 'vmpn', 'wps', '¼', 'wspnðvmvsþ', 'the', 'free', 'energy', 'polymerization', 'written,', 'df1crt', '¼', 'ln', '’m', 'þ', '1', 'þ', 'ðwms’s', 'þ', 'wms’pþð’s', 'þ', '’pþ', '(cid1)', 'wspðvmvsþ’s’p', '(cid1)', 'ð1nþ', 'ln', '’p', '(cid1)', '1n', '(cid1)', 'ðwmp’m', 'þ', 'wsp’svmvsþð’m', 'þ', '’sþ', 'þ', 'wm’m’s', '’m', 'equilibrium', 'volume', 'fraction', 'monomer', 'volume', 'fraction', 'polymer', '’p', 'sawata', 'points', '407', 'neglecting', 'terms', '1n', 'n', 'large', 'replacing', '(’s', '’p)', '(1', '(cid1)', '’m)', '(’m', '’s)', '(1', '(cid1)', '’p)', 'one', 'gets', 'general', 'expression', 'df1crt', '¼', 'ln', '’m', 'þ', '1', 'þ', '’sðwms', '(cid1)', 'wspvmvsþ', 'þ', 'wmsð’p', '(cid1)', '’mþ', '241', 'review', 'questions', 'review', 'questions', 'section', '41', 'section', '42', 'section', '43', '1', 'explain', 'differences', 'similarities', 'anionic', 'cationic', 'chain', 'growth', 'polymerizations', '1', 'write', 'chemical', 'equations', 'cationic', 'initiation', 'propagation', 'termination', 'steps', 'show', 'kinetic', 'expressions', '2', 'write', 'cationic', 'kinetic', 'expression', 'degree', 'polymerization', 'rate', 'propagation', '3', 'repeat', 'questions', '1', '2', 'anionic', 'polymerizations', '1', 'explain', 'meant', 'expression', 'initiation', 'cationic', 'polymerization', 'results', 'transposition', 'either', 'one', 'two', 'electrons', 'what', 'type', 'initiating', 'species', 'involved', '2', 'summarize', 'initiation', 'process', 'cationic', 'chain-growth', 'polymerization', '3', 'show', 'chemical', 'equations', 'initiation', 'process', 'protonic', 'acids', 'lewis', 'acids', 'complexed', 'water', 'alkyl', 'halides', '4', 'what', 'proposed', 'mechanism', 'initiation', 'polymerization', 'lewis', 'arid', 'like', 'aluminum', 'chloride', 'bromide', 'auto', 'ionization', 'process', '5', 'what', 'several', 'proposed', 'mechanisms', 'initiations', 'stable', 'cations', '6', 'explain', 'cationic', 'polymerizations', 'initiated', 'metal', 'alkyls', 'substances', 'generate', 'cations', '7', 'show', 'mechanism', 'one', 'electron', 'transposition', 'initiation', 'chemical', 'equations', 'based', 'free-radical', 'oxidation', 'diphenyliodonium', 'hexaﬂuorophosphate', '(let', 'source', 'free-', 'radicals', 'photo', 'decomposition', 'methyl', 'benzoin', 'ether)', '8', 'discuss', 'brieﬂy', 'role', 'charge', 'transfer', 'complexes', 'initiations', 'cationic', 'polymerizations', '9', 'what', 'mechanism', 'electro', 'initiation', 'cationic', 'polymerization', '10', 'how', 'tightness', 'ion', 'pairs', 'affect', 'propagation', 'cationic', 'polymerization', '11', 'what', 'mechanism', 'chain', 'growth', 'tight', 'ion', 'pairs', 'cationic', 'polymerization', '12', 'discuss', 'kunitake', 'takanabe', 'mechanism', 'steric', 'control', 'homogeneous', 'cationic', '13', 'what', 'nakano', 'et', 'al', 'mechanism', 'steric', 'control', 'heterogeneous', 'cationic', '14', 'what', 'pseudo-cationic', 'polymerization', 'explain', 'illustrate', 'polymerization', 'styrene', 'polymerizations', 'polymerizations', 'initiated', 'perchloric', 'acid', '15', 'describe', 'give', 'examples', 'chemical', 'reactions', 'isomerization', 'polymerization', 'propagation', 'accompanied', 'bond', 'electron', 'rearrangement', '16', 'give', 'two', 'examples', 'chemical', 'reactions', 'propagation', 'isomerization', 'polymerization', 'accompanied', 'migration', 'one', 'atoms', '17', 'illustrate', 'termination', 'reactions', 'cationic', 'polymerizations', '18', 'what', 'important', 'aspects', 'living', 'cationic', 'polymerization', 'can', 'give', 'examples', '242', 'section', '44', '4', 'ionic', 'chain-growth', 'polymerization', '1', 'what', 'two', 'types', 'initiations', 'anionic', 'polymerization', '2', 'describe', 'illustrate', 'chemical', 'equations', 'initiation', 'mechanism', 'homogeneous', 'polymerization', 'addition', 'butyllithium', 'oleﬁn', 'polar', 'solvent', 'what', 'effect', 'heteroatoms', 'oxygen', 'present', 'monomers', 'illustrate', '3', 'describe', 'illustrate', 'one', 'electron', 'transfer', 'initiations', 'anionic', 'polymerization', 'give', 'several', '4', 'describe', 'initiations', 'propagation', 'alﬁn', 'catalysts', '5', 'discuss', 'electoinitiation', 'anionic', 'polymerization', '6', 'how', 'propagation', 'reaction', 'anionic', 'polymerization', 'differ', 'presence', 'polar', '7', 'how', 'structures', 'monomers', 'inﬂuence', 'propagation', 'rates', 'anionic', 'polymerizations', '8', 'discuss', 'steric', 'control', 'homogeneous', 'anionic', 'polymerization', 'methyl', 'methacrylate', 'non-', 'examples', 'non-polar', 'solvents', 'illustrate', 'polar', 'solvents', '9', 'discuss', 'steric', 'control', 'homogeneous', 'anionic', 'polymerization', 'isoprene', 'none-polar', 'solvents', '10', 'discuss', 'steric', 'control', 'homogeneous', 'anionic', 'polymerizations', 'polar', 'solvents', '11', 'discuss', 'illustrate', 'hydrogen', 'transfer', 'polymerization', 'acrylamide', '12', 'how', 'termination', 'reactions', 'take', 'place', 'anionic', 'chain-growth', 'polymerization', '13', 'discuss', 'living', 'anionic', 'polymerizations', '14', 'how', 'living', 'monodisperse', 'methyl', 'methacrylate', 'polymers', 'prepared', 'aid', 'enolate', 'aluminum', 'porphyrin', 'intermediates', 'explain', 'illustrate', 'section', '45', '1', 'what', 'metals', 'form', 'catalysts', 'ziegler–natta', 'coordinated', 'anionic', 'polymerization', '2', 'illustrate', 'reactions', 'takes', 'place', 'aluminum', 'alkyls', 'transition', 'metal', 'compounds', 'like', 'titanium', 'chloride', 'formations', 'ziegler–natta', 'catalysis', '3', 'describe', 'bimetallic', 'polymerization', 'mechanism', 'g', 'natta', '4', 'describe', 'cossee', 'arlman', 'mono', 'metallic', 'mechanism', '5', 'describe', 'steric', 'control', 'heterogeneous', 'coordinated', 'anionic', 'polymerizations', '6', 'what', 'homogeneous', 'ziegler–natta', 'catalysts', '7', 'discuss', 'steric', 'control', 'homogenous', 'catalysts', '8', 'describe', 'currently', 'used', 'industrial', 'ziegler–natta', 'catalysts', 'support', 'soluble', 'ones', '9', 'discuss', 'effects', 'lewis', 'bases', 'ziegler–natta', 'typo', 'coordinated', 'anionic', 'polymerizations', '10', 'what', 'termination', 'reaction', 'coordinated', 'anionic', 'polymerizations', 'illustrate', '11', 'describe', 'transition', 'metal', 'oxide', 'support', 'catalysts', 'prepared', 'used', '12', 'describe', 'metallocene', 'catalysts', '13', 'how', 'isomerization', 'polymerization', 'take', 'place', 'coordinated', 'anionic', 'mechanism', 'section', '46', '1', 'describe', 'cationic', 'polymerization', 'aldehydes', '2', 'describe', 'anionic', 'polymerizations', 'aldehydes', 'section', '47', 'section', '48', 'encountered', 'section', '49', 'section', '410', 'references', '243', '3', 'what', 'mechanism', 'control', 'anionic', 'polymerization', 'aldehydes', 'polymerizations', '4', 'describe', 'polymerization', 'unsaturated', 'aldehydes', 'crotonaldehyde', 'acrolein', '5', 'describe', 'polymerization', 'dialdehydes', '1', 'discuss', 'polymerization', 'ketones', 'isocyanates', '1', 'discuss', 'copolymerization', 'ionic', 'mechanism', 'what', 'problems', '1', 'discuss', 'proposed', 'mechanisms', 'group', 'transfer', 'polymerization', '1', 'what', 'bernoulli', 'markov', 'colman-fox', 'statistical', 'models', 'propagation', 'reactions', 'can', 'describe', 'hl', 'hsieh', 'rp', 'quick', 'anionic', 'polymerization', '-', 'principles', 'practical', 'applications', 'm', 'dekker', 'new', 'york', 'k', 'matyjaszewski', 'ed,', 'cationic', 'polymerization', '-', 'mechanism', 'synthesis', 'application', 'm', 'dekker', 'new', 'york', '1996', 'recommended', 'reading', '1996', 'references', '1', 'jp', 'kennedy', 'e', 'marechal', 'cationic', 'polymerization', 'wiley-interscience', 'new', 'york', '1982', 'mmofton', 'anionic', 'polymerization', 'principles', 'practice', 'academic', 'press', 'new', 'york', '1983', '2', 'te', 'hogenesch', 'j', 'smid', 'eds', 'recent', 'advances', 'anionic', 'polymerization', 'elsevier', 'new', 'york', '1987', '3', 'ml', 'burstall', 'fe', 'treloar', 'chapt', '1', 'the', 'chemistry', 'cationic', 'polymerization', 'pr', 'plesch', 'ed', 'macmillan', 'co,', 'new', 'york', '1963', '4', 'dn', 'bhattacharyya', 'cl', 'lee', 'i', 'smid', 'm', 'szwarc', 'j', 'phys', 'chem1965', '69', '612', '5', 'dn', 'bhattacharyya', 'i', 'smid', 'm', 'szwarcj', 'phys', 'chem1965,', '69', '624', '6', 'gt', 'chen', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed', '1982,', '20', '2915', '7', 't', 'keii', 'e', 'suzuki', 'm', 'tamura', 'm', 'murata', 'y', 'doi', 'makromol', 'chem,', '1982', '183', '2285', '244', '4', 'ionic', 'chain-growth', 'polymerization', '8', 'j', 'p', 'kennedy', 'rm', 'thornas', 'advan', 'chem', 'ser,', '1967', '34', '111', 'aoshima', 's,', 'yoshida', 't,', 'kanazawa', 'a', 'kanaoka', 's', 'j', 'polym', 'sc.', 'part', 'a', 'polym', 'chem,', '2007', '45', '1801yoshida', 't,', 'kanaoka', 's', 'aoshima', 's,', 'j', 'polym', 'sc.', 'port', 'a', 'polymchem', '2005', '43', '5138', '9', 'jp', 'kennedy', 'b', 'marechal', 'j', 'polymer', 'sci,', 'macromol', 'rev,', '1981', '16', '123', '10', 'g', 'a', 'olah', 'makromol', 'chemie', '1974', '175', '1039', '11', 'm', 'chmelir', 'j', 'polymer', 'sci', 'symposium', '1976', '56', '311', '12', 'z', 'zlamal', 'chapt', '6', 'vinyl', 'polymerization', 'part', 'il', 'geham', 'ed', 'dekker', 'new', 'york', '1969', '13', 'dd', 'eley', 'the', 'chemiatiy', 'cationic', 'polymerization”', 'ph', 'plesch', 'ed,', 'macmillan', 'new', 'york', '1963', '14', 'o', 'wichterle', 'm', 'kolinsky', 'm', 'marek', 'chem', 'listy', '1958', '52', '1049', '15', 'rg.', 'heiligman', 'j', 'polymer', 'sci1951,', '6', '155', '16', 'na', 'ghanem', 'h', 'mark', 'eur', 'polymer', 'j', '1972', '8', '999', '17', 'ag', 'evans', 'gw', 'meadows', 'm', 'polanyi', 'nature', '1946', '158', '94', 'ibid', '1947', '160', '869', '18', 'ph', 'pleat', 'the', 'chemistry', 'cationic', 'polymerization', 'pr', 'plesch', 'ed,', 'macmillan', 'ny,', '1963', '19', 'hc', 'brown', 'bw', 'kanner', 'j', 'am', 'chem.', 'soc', '1963', '75', '3865', '20', 'mj', 'moulis', 'j', 'collomb', 'a', 'gandini', 'h', 'cheradame', 'polymer', 'bull', '(berlin)', '1980', '3', '(4)', '197', 'from', 'chemabstr', '1981', '94', '31143u', '21', 'jp', 'kemedy', 't', 'kelen', 'sc', 'guhaniyogi', 'rt', 'chou,', 'j', 'macromol', 'sci-chem,', '1982', '418', '129', '22', 'm', 'masure', 'p', 'sigwalt', 'makromol', 'chem,', 'rapid', 'commun', '1983', '4269', '23', 'ph', 'plesch', 'makromol', 'chemie', '1974', '75', '1065', '24', 'm', 'chmelir', 'm', 'marek', 'o', 'wichterle', 'j', 'polymer', 'sci,', '1967', 'c-16', '833', '25', 'm', 'chmelir', 'j', 'polymer', 'sci,', 'symposium', '1970', '56', '311', '26', 'jp', 'kennedy', 'jmacromol', 'sci', '1972', '6-a', '329', '27', 'g', 'sauvet', 'jpvairon', 'p', 'sigwalt', 'j', 'polymer', 'sci,', 'chem.', '1978', '16', '3047', '28', 'p', 'sigwalt', 'makromol', 'chemie', '1974', '175', '1017', '29', 'ga', 'olah', 'makromol', 'chemie', '1974', '1751039', '30', 'v', 'korshakand', 'nn', 'lebedev', 'j', 'gen', 'chem', 'ussr', '1948', '18', '1766', '31', 'g', 'sauvet', 'jpvairon', 'p', 'sigwalt', 'bull', 'soc', 'chim', 'france', '1970', '4031', '32', 'm', 'marek', 'm', 'chmelir', 'intent', 'symp', 'macromol', 'chem,', 'prague', '1965', 'p4', '33', 'fam', 'abdul-rasoul', 'a', 'ledwith', 'y', 'yagci', 'polymer', '1978', '19', '1219', '34', 'm', 'chmelir', 'm', 'marek', 'j', 'polymer', 'sci', '1968', 'c22', '177', '35', 'p', 'lopour', 'm', 'marek', 'makromol', 'chem,', '1970', '134', '23', '36', 'dwgrattan', 'ph', 'plesch', 'makromol', 'chem,', '1980', '181', '751', 'sipos', 'l,', 'de', 'p', 'faust', 'r', 'macromolecules', '2003', '36', '8282', 'dimitrov', 'i', 'faust', 'r', 'macromolecules', '2004', '37', '9753', 'hadjikyriacou', 's,', 'acar', 'm', 'faust', 'r', 'macromolecules', '2004', '37', '75', 'dc', 'p', 'faust', 'r', 'macromolecules', '200639', '7527', '37', 'jp', 'kenedy', 'sy', 'huang', 'sc', 'feinberg,', 'j', 'polymer', 'sci,', '1977', '15', '2801', '38', 'p', 'giusti', 'f', 'andrizzi', 'p', 'cerrai', 'gl', 'possanzini', 'makromol', 'chem,', '1970', '136', '97', '39', 'ceh', 'bawn', 'c', 'fitzimmons', 'a', 'ledwith', 'proc', 'chem', 'soc,', '1964', '391', '40', 'a', 'ledwith', 'makromol', 'chem,', '1974', '175', '1117', '41', 'ceh', 'bawn', 'pure', 'applied', 'chem', '1968', '16(2-3)', '385', '42', 'g', 'natta', 'g', 'dall', 'asta', 'g', 'mazzxanti', 'u', 'giannini', 's', 'cesea', 'angew', 'chem1959,', '71', '205', '43', 'g', 'natta', 'g', 'dall', 'asta', 'g', 'mazzxanti', 's', 'cesea', 'makromol', 'chem,', '1962', '58217', '44', 'jp', 'kennedy', 'am', 'chem', 'soc', 'polymer', 'preprints', '1966', '7', '485', 'j', 'polymer', 'sci,', 'symposium', '1976', '56', '1', '45', 't', 'saegusa', 'structure', 'mechanism', 'vinyl', 'polymerization', 't', 'tsuruta', 'hs', 'o’driscoll', 'edsdekker,', 'new', 'york', '1969', '46', 'jc', 'bevington', 'polyethers', '1', 'n', 'gaylord', 'ed,', '(high', 'polymers', 'vol', '13)', 'wiley-interscience', 'new', 'york', '1963', '47', 'h', 'sinn', 'h', 'winter', 'wv', 'tirptiz', 'makromol', 'chem,', '1961', '48', '59', '48', 't', 'saegusa.', 'h', 'imai', 'j', 'furukawa', 'makromol', 'chem', '1963', '65', '60', '49', 'k', 'iwasaki', 'h', 'imai', 'y', 'tsuchita', 's', 'nakano', 'j', 'polymer', 'sci,', '1963', 'a-11', '2371', '50', 'jm', 'bruce', 'dw', 'farren', 'polymer', '1963', '4', '407', '51', 'm', 'udo', 'km', 'stehling', 'mr', 'stein', 'itw', 'wamouthmacromolecules', '1998', '31', '2019', '52', 'ag', 'can', 'dm', 'dawsom', 'andm', 'bothmann', 'mocromolecules', '1998', '31', '2035', '53', 'q', 'wang', 'd', 'jeremic', 'f', 'barsan', 'it', 'quyoum', 'dj', 'gills', 'mc', 'baird', 'am', 'chem', 'soc', 'polymer', 'preprints', '1996', '37(1)', '251', '54', 'k', 'matsuzaki', 'r', 'mciii', 'n', 'inoue', 't', 'kanai', 'makromol', 'chem,', '1981', '182', '2421', '55', 'fam', 'abdul-rasoul', 'a', 'ledwith', 'y', 'yagci', 'polymer', 'bull,', '1978', '1', '1', '56', 'a', 'ledwith', 'polymer', '1978', '19', '1217', '57', 'lp', 'ellinger', 'advan', 'macromol', 'chem,', '1968', '1', '169', '58', 'h', 'gilbert', 'ff', 'miller', 'sj', 'averill', 'bh', 'carlson', 'jl', 'folt', 'hj', 'heller', 'fd', 'stewart', 'rf', 'schmidt', 'il', 'l', 'trumbull', 'j', 'am', 'chem', 'soc,', '1956', '78', '1669', 'references', '245', '59', 'a', 'ledwith', 'j', 'polymer', 'sci,,', 'symposium', '56', '1976', '483', '60', 'rf', 'tarvin', 's', 'aoki', 'jk', 'stille', 'macromolecules', '1972', '5', '663', '61', 't', 'natsuume', 'm', 'naahiniura', 'm', 'fujirnatsu', 'm', 'shimizu', 'y', 'shirota', 'h', 'hirata', 's', 'kikabayaahi', 'mikawa', 'polymer', 'j', 'japan', '1971', '1', '181', '(from', 'private', 'translation)', '62', 'k', 'takakura', 'k', 'hayashi', 's', 'okamura', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1965', '3', '565', '63', 'k', 'takakura', 'k', 'hayashi', 's', 'okamura', 'j', 'polymer', 'sci,', 'polymerletters', '1965', '4', '5', '09', '64', 'm', 'irie', 's', 'tominto', 'r', 'hsyashi', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1970', '8', '809', '65', 'm', 'marek', 'l', 'toman', 'j.', 'polymer', 'sci', 'polymer', 'symposium', '42', '1973', '339', '66', 'm', 'marek', 'ltoman', 'makromol', 'chem,', 'rapid', 'commun,', '1980', '1161', '67', 'jv', 'criveilo', 'jh', 'lam', 'j', 'polymer', 'sci,', 'symposium', '56', '1976', '383', '68', 'jv', 'crivello', 'jh', 'lam', 'j', 'polymer', 'sci', 'polymer', 'letters', '1978', '16', '563', '69', 'jv', 'crivello', 'jh', 'lam,j', 'polymer', 'sci,', 'chem', 'ed,', '1980', '17', '1047', 'ibid', '1980', '18', '2677', 'ibid', '1980.', '18', '2697', '70', 'jv', 'crivello', 'j', 'h', 'lam', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1979', '17', '759', '71', 'jv', 'crivelloand', 'jh', 'lam', 'j', 'polymer', 'sci,', 'chem', 'ed', '1980', '18', '1021', '72', 'k', 'hayashi', 's', 'okamura', 'j', 'polymer', 'sci,', 'symposium', '22', '1968', '15', '73', 'f', 'williams', 'et', 'al', 'discussion', 'faraday', 'soc,', '1963', '36', '254', '74', 'rk', 'ueno', 'k', 'hayashi', 's', 'okamura', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1965', '3', '363', 'polymer', '1966', '7451', '75', 'rc', 'potter', 'rm', 'bretton', 'dj', 'metz', 'j', 'polymer', 'sci', '1966', 'a-14', '419', '2259', '76', 'f', 'williants', 'j', 'am', 'chem', 'soc.1964', '86', '3954', '77', 'rb', 'taylor', 'f', 'williams', 'j', 'am', 'chem', 'soc,', '1969', '91', '3728', '78', 'jw', 'breitenback', 'ch', 'srna', 'pure', 'appl', 'chem,', '1962', '4', '245', '79', 'bi', 'funt', 'sw', 'laurent', 'j', 'polymer', 'sci,', '1964', 'a-12', '865', '80', 'bi', 'funt', 'kc', 'yu', 'j', 'polymer', 'sci,', '1962', '62', '359', '81', 'jw', 'breitenbath', 'ch', 'sma', '0', 'f', 'olaj', 'macromol', 'chem,', '1960', '42', '171', '82', 'jw', 'breitenbach', 'h', 'gabler', 'monatsch', 'chem,', '1961', '91', '202', '83', 'f', 'sommer', 'jw', 'breitenbach', 'iupac', 'int', 'symp', 'macromol', 'chem,', '1069', '1', '257', '84', 'u', 'akbulut', 'je', 'fernandez', 'rl', 'birke', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1975', '13', '133', '85', 'bl', 'funt', 'tj', 'blainjpolymer', 'sci,', '1971', 'a-29', '115', '86', 'jp', 'kennedy', 'rm', 'thomas', 'j', 'polymer', 'sci,', '1960', '46', '233', '87', 't', 'higashisnura', '5', 'okamura', 'j', 'masamoto', 'kibunski', 'kagaku', '1968', '25', '(282)', '702', '(private', 'translation)', '88', 'y', 'wang', 'l', 'm', 'dorfmann', 'macromolecules', '1980', '13', '63', '89', 'n', 'kolishetti', 'r', 'faust', 'macromolecules', '2008', 'web', 'edition', '101021ma801669t', '90', 'j', 'p', 'kennedy', 'copolymerization', 'gb', 'ham', 'ed,', 'wiley-interscience', 'new', 'york', '1964', '91', 'dc', 'pepper', 'pj', 'reilly', 'j', 'polymer', 'sal,', '1961', '58', '639', '92', 'ag', 'evans', 'm', 'polanyi', 'j', 'chem', 'soc,', '1947', '252', '93', 'ce', 'schiklmecht', 'ao.', 'zoss', 'e', 'mckinley', 'ind', 'eng', 'chem,', '1948', '39', '180', '94', 'g', 'natta', 'iw', 'bossi', 'p', 'carradini', 'makromol', 'chem,', '1955', '18', '455', '95', 'c', 'e', 'schildknecht', 'ind', 'eng', 'chem,', '1958', '50', '107', '96', 'sokamura', 't', 'higashimura', 'sakurada', 'j', 'polymer', 'chem', '1959', '39', '507', '97', 't', 'saegusa', 'h', 'lmai', 'j', 'furukawa', 'macromol', 'chem,', '1964', '79', '207', '98', 'j', 'freenley', 'a', 'ledwith', 'rh', 'sutcliffe', 'j', 'chem', 'soc,', '1962', '2021', '99', 'ceh', 'bawn', 'a', 'ledwith', 'quartrev', '(london)', '1962', '16', '361', '100', 'dj', 'cram', 'k', 'r', 'kopecky', 'j', 'am', 'chem', 'soc', '1959', '81', '2748', '101', 't', 'kunitake', 'c', 'aso', 'j', 'polymer', 'sci.', '1970', 'a-18', '665', '102', 'y', 'hirokawa', 'tr', 'higashimura', 'k', 'matsuzaki', 't', 'kawamura', 't', 'uryu', 'j', 'polymer', 'sci,', 'chem', 'ed', '1979', '17', '3923', '103', 'd1', 'sikkerna', 'h', 'angad-gaur', 'makromol', 'chem,', '1980', '181', '2259', '104', 'a', 'ledwith', 'b', 'chiellini', 'k', 'solano', 'macromolecules', '1979', '12', '240', '105', 't', 'kunitake', 'k', 'takarabe', 'mokromol', 'chem,', '1981', '182', '817', '106', 'g', 'heublin', 'j', 'macromol', 'sci-chem,', '1981', 'a16', '563', '107', 'j', 'furukawa', 'polymer', '1962', '3487', '108', 's', 'nakano', 'k', 'iwasaki', 'h', 'fukutani', 'j', 'polymer', 'sat', '1963', 'a-11', '3277', '109', 'ph', 'plesch', 'b', 'polymer', 'j.', '1973', '1', 'ph', 'plesch', 'mokromol', 'chem,', 'macromol', 'symp,', '1988', '1314', '375393', 'k', 'matyjaszewski', 'mokromol', 'chem', 'macromol', 'symp,', '1988', '1314389', '110', 'a', 'gandini', 'p', 'h', 'plesch', 'eur', 'polymer', 'j,', '1968', '455', 'dc', 'pepper', 'makromol', 'chem,', '1974', '175', '1077', '111', 's', 'bywater', 'dj.', 'woisford', 'canj.', 'chem,', '1966', '44', '1671', '112', 't', 'masuda', 't', 'higashimura', 'jpolymer', 'sci,', 'polym', 'letters', '1972', '9', '783', '113', 'sd', 'hamann', 'aj', 'murphy', 'dm', 'solomon', 'r', 'willing', 'j', 'macromol', 'sci-chem,', '1972', 'a6', '771', '114', 'm', 'szwarc', 'macromolecules', '1995', '287309', '7312', '246', '4', 'ionic', 'chain-growth', 'polymerization', '115', 'k', 'matyjaszewski', 'mokromol', 'chem', 'symp,', '1988', '1314', '389', '116', 'jp', 'kennedy', 'trans', 'ny', 'acad', 'sci,', '1966', '1080', '117', 'jp', 'kennedy', 'ja', 'hinlicky', 'polymer', '1965', '6', '133', '118', 'g', 'santori', 'a', 'valvassori', 'v', 'turba', 'mp', 'lachi', 'chim', 'ind', '(mllano)', '1963', '45', '1529', '119', 'jp', 'kennedy', 'hs', 'makowskij', 'polymer', 'sci,', '1968', 'c(22)', '247', '120', 'jp', 'kennedy', 'jj', 'elliott', 'w', 'naegele,j', 'polymer', 'sci,', '1964', 'a-12', '5029', '121', 'cg', 'wanless', 'jp', 'kenedy,polymer', '1965', '6', '111', '122', 'jp', 'kennedy', 'ww', 'schulz', 'rg', 'squires', 'rm', 'thomas', 'polymer', '1965', '6', '289', '123', 'ga', 'sartori', 'hi', 'lammens', 'j', 'siffert', 'a', 'bernard', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1971', '9', '599', '124', 'jp', 'kennedy', 'p', 'berzel', 'w', 'naegele', 'rg', 'squires', 'j', 'macromol', 'sci-chem,', '1966', '93', '191', '125', 'p', 'dimitrov', 'j', 'emert', 'j', 'hua', 's', 'kekis', 'r', 'faust', 'macromolecules', '2011', '44', '1831', '126', 'jp', 'kennedy', 'a', 'schinakawa', 'f', 'williams', 'j', 'polymer', 'sci1971,', 'a-19', '1551', '127', 'jp', 'kannedy', 'sy', 'huang', 'sc', 'feinberg', 'j', 'pajymer', 'sci,', '1977', '2801', '128', 'jp', 'kennedy', 'jpolymer', 'sci,', 'symposia', '56', '1976', '1', '129', 'jp', 'kennedy', 'emarechal', 'cationic', 'polymerization', 'wiley-interscience', 'new', 'york', '1982', '130', 'm', 'karnigaito', 'k', 'yarnaoka', 'm', 'sawarnoto', 'th', 'higashirnura', 'macromolecules', '1992', '25', '6400', 'jp', 'kennedy', 's', 'midha', 'b', 'keszler', 'macromolecules', '1993', '26', '424', 'j', 'lu', 'm', 'karnigaito', 'm', 'sawarnoto', 'th', 'higashirnura', 'y-x', 'deng', 'macromolecules', '1997', '30', '22', '131', 'jp', 'kemedy', 't', 'kelen', 'f', 'tudoa', 'j', 'macromolsci-chem,', '1982-1983', 'a18', '1189', '132', 'k', 'faust', 'a', 'fehervari', 'jp', 'kennedy', 'j', 'macromol', 'sci-chem,', '1982-1983', 'a18', '1209', '133', 'jpuskas', 'g', 'kaszas', 'jp', 'kennedy', 't', 'kelen', 'f', 'tudoa', 'j', 'macroml', 'sci-chem,', '1982-1983', 'a18', '1229', '134', 'jpuskas', 'g', 'kaszas', 'jp', 'kennedy', 't', 'kelen', 'f', 'tudoa', 'j', 'macroml', 'sci-chem,', '1982-1983', 'a18', '1245', '135', 'jpuskas', 'g', 'kaszas', 'jp', 'kennedy', 't', 'kelen', 'f', 'tudoa', 'j', 'macroml', 'sci-chem,', '1982-1983', 'a18', '1263', '136', 'm.', 'sawamoto', 'jp', 'kennedyj', 'macromol', 'sci-chem,', '1982-1983', 'a18', '1275', 's', 'aoshima', 's', 'kanaoka', 'chem', 'rev2009,', '1095245', '137', 'k', 'faust', 'jp', 'kerinedy', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed', '1987', '25', '1847', 't', 'higashumura', 's', 'aoshima', 'm', 'sawamoto', 'makromol', 'chem,macromol', 'symp', '1988', '1314457(1988)', 'y', 'lshihara', 'm', 'sawamoto', 't', 'higashimura', 'polym', 'bull', '1990', '24', '201', '138', 'g', 'kaszaa', 'je', 'puskas', 'cc', 'chen', 'jp', 'kennedy', 'macromolecules', '1990', '23', '3909', 'jp', 'kennedy', 'makromol', 'chem,', 'macromol', 'symp,', '1992', '32119', 'o.', 'nuyken', 'h', 'kroner', 'makromol', 'chem,', '1990', '191', '1', 'l', 'thomas', 'a', 'poton', 'm', 'tardi', 'p', 'sigwalt', 'macromolecules', '1992', '25', '5886', '139', 'm', 'sawamoto', 't', 'higashimura', 'am', 'chem', 'soc,', 'polymer', 'preprints', '1991', '32', '312', 'yoshida', 't,', 'kanazawa', 'a,', 'kanaoka', 's', 'aoshima', 's', 'j', 'polym.', 'sci,', 'part', 'a', 'polym', 'chem', '2005', '43', '4288', 'kanazawa', 'a,', 'kanaoka', 's', 'aoshima', 's', 'macromolecules', '2009', '42', '3965', 'kanazawa', 'a,', 'kanaoka', 's', 'aoshtma', 's', 'j', 'am', 'chem', 'soc', '2007', '129', '2420', 'sugihara', 's,', 'tanabe', 'y,', 'kitagawa', 'm', 'lkeda', 'i', 'j', 'polym', 'sci,', 'part', 'a', 'polym', 'chem', '2008', '46', '1913', '140', 'rf', 'storey', 'cl', 'curry', 'lb', 'blister', 'macromolecules', '1998', '31', '1058', '141', 'z', 'fedor', 'yc', 'bae', 'r', 'faust', 'macromolecules', '1998', '314439', '142', 'a', 'kanazawa', 's', 'kanaoka', 's', 'i', 'aoshlma', 'macromoleculee', '2010', '43', '3682', '143', 'dh', 'richards', 'polymer', '1978', '19', '109', '144', 'd', 'margerison', 'jp', 'newport', 'trans', 'faraday', 'soc,', '1963', '59', '2058', '145', 'tl', 'brown', 'rl', 'gerteis', 'da', 'bafus', 'ja', 'ladd', 'j', 'am', 'chem', 'soc,', '1964', '86', '2134', '146', 'j', 'smid', 'chapt', '11', 'stnidure', 'mechanism', 'vinyl', 'polymerization', 't', 'tsunita', 'kf', 'o’driscoll', 'eds', 'dekker', 'new', 'york', '1969', '147', 'tl', 'brown', 'advan', 'organometallic', 'chem,', '1965', '3', '365', '148', 'hf', 'ebel', 'tetrahedron', '1965', '21', '699', '149', 'g', 'fraenkel', 'dg', 'adams', 'j', 'williams', 'tetrahedron', 'letters', '1963', '767', '150', 'dj', 'worsford', 's', 'bywater', 'can', 's', 'chem,', '1962', '40', '1564', '151', 'cg', 'screttas', 'jf', 'eastham', 'j', 'am', 'chem', 'soc,', '1965', '87', '3276', '152', 'a', 'zilkha', 's', 'barzakay', 'a', 'ottolenghi', 'j', 'polymer', 'sci.', '1963', 'a-11', '1813', '153', 'jp', 'kennedy', 'aw', 'langer', 'jr,', 'fortschr', 'hochpolymer', 'forsch,', '1964', '3', '544', '154', 'j', 'geerts', 'mvan', 'beylen', 'g', 'smets', 'j', 'polymer', 'sci,', '1969', 'a-17', '2859', '155', 'chb', 'tsvetanov', 'eur', 'polymer', 'j', '1979', '15', '503', '156', 'aa', 'korotkov', 'sp', 'mitzengendler', 'vn', 'krasulina', 'j', 'polmer', 'sci,', '196153', '217', '157', 't', 'fujimoto', 'n', 'kawabata', 'j', 'furukawa', 'j', 'polymer', 'sci,', '1968', 'a-16', '1209', '158', 'k', 'hatada', 't', 'kitayama', 'k', 'fujikawa', 'k', 'ohta', 'yuki', 'am', 'chem', 'soc', 'polymer', 'preprints', '1980', '21(1)', '59', '159', 'g', 'lohr', 'gvschulz', 'eur', 'polymer', 'j,', '1974', '10', '121', '160', 'v', 'warzelhan', 'h', 'hocker', 'gv', 'schulz', 'makromol', 'chem,', '1978', '179', '2221', '161', 'r', 'kraft', 'ahe', 'muller', 'v', 'warzelhan', 'h', 'hocker', 'gv', 'schulz', 'macromolecules', '1978', '11', '1093', '162', 'r', 'kraft', 'ahe', 'muller', 'h', 'hocker', 'gv', 'schulz', 'makromol', 'chem,', 'rapid', 'commun,', '1980', '1', '363', 'references', '247', '163', 'y', 'joh', 'y', 'kotake', 't', 'yoshihara', 'f', 'ide', 'k', 'nakatsuka', 'j', 'polymer', 'sci,', '1967', 'a-la', '593', '605', '164', 'chb', 'tsvetanov', 'eur', 'polymer', 'j.', '1979', '15', '503', '165', 'pem.', 'alien', 'j', 'macromol', 'sci', '1980', 'a14', '11', '166', 'bl', 'erussaliniski', 'iv', 'kulevskaya', 'vv', 'mazurek,j', 'polymer', 'sci,', '1967', '16', '1355', '167', 'bl', 'erussalinski', 'vysokomol', 'soed', '1971', 'a13', '1293', '168', 'ar', 'lyons', 'catterall', 'eur', 'polymer', 'j,', '1970', '7', '839', '169', 'ng', 'gaylord', 'ss', 'dixit', 'macromol', 'rev,', '1974', '8', '51', '170', 'cg.', 'overberger', 'y', 'yamamoto', 'j', 'polymer', 'sci,', '1959', 'b', '3569', '(1959)', 'j', 'polymer', 'sci', '1966', 'a-14', '3101', '171', 'k', 'higashi', 'h', 'baba', 'a', 'reanbaum', 'quantum', 'organic', 'chemistry', 'wiley-interscience', 'new', 'york', '1965', '172', 'dc', 'pepper', 'makromol', 'cliem', '1974', '175', '1077', '173', 'dh', 'richards', 'polymer', '1972', '19', '109', '174', 'm', 'szwarc', 'nature', '1956', '1781168', '175', 'm', 'brady', 'm', 'ladaki', 'r', 'miilcovich', 'm', 'szwarc', 'j', 'polymer', 'sci1957,', '221', '176', 'm', 'szwarc', 'm', 'levy', 'r', 'milkovich', 'j', 'am', 'chem', 'soc,', '1956', '78', '2656', '177', 'm', 'szwarc', 'makromol', 'chem,', '1960', '35', '132', '178', 'm', 'szwarc', 'van', 'beylen', '‘ionic', 'polymerization', 'living', 'polymers’', 'chapman', 'hall', 'new', 'york', '1993', '179', 'm', 'levy', 'm', 'szwarc', 'j', 'am', 'chem', 'soc1961,', '82', '521', '180', 'm', 'szwarc', 'carbanions', 'living', 'polymers', 'electron', 'transfer', 'processes', 'interscience', 'new', 'york', '1968', '181', 'a', 'zilkha', 'p', 'neta', 'm', 'frankel', 'j', 'chem', 'soc,', '1960', '3357', '182', 's', 'inoue', 't', 'tsunita', 'j', 'furukawa', 'makromol', 'chem,', '1960', '42', '12', '183', 'im', 'panagotov', 'ib', 'rashkov', 'i', 'n', 'yukhnovski', 'eur', 'polymer', 'j.', '1971', '7', '749', '184', 'aa', 'morton', 'bb', 'magat', 'rl', 'letsinger', 'j', 'am', 'chem', 'soc', '1947', '69', '950', '185', 'aa', 'morton', 'fm', 'bolton', 'fw', 'co1lins', 'bf', 'cluf', 'ind', 'eng', 'chem,', '1952', '44', '2876', '186', 'aa', 'morton', 'rubberage', '1953', '72', '473', '187', 'aa', 'morton', 'ind', 'eng', 'chem,', '1950', '42', '1488', '188', 'aa', 'morton', 'bj', 'laupher', 'j', 'polymer', 'sci,', '1960', '44', '233', '189', 'l', 'reich', 'a', 'schindler', 'polymerization', 'organometallic', 'compounds', 'wiley-interscience', 'new', 'york', '1966', '190', 'n', 'yamazaki', 's', 'nakarnura', 's', 'karnbara', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1965', '3', '57', '191', 'd', 'laurin', 'g', 'parravane', 'j', 'polymer', 'sci.', 'polymer', 'letters', '1966', '4', '797', '192', 'bl', 'funt', 'sn', 'bhadanij', 'polymer', 'sci,', '1968', 'c17', '1', '193', 'fi', 'welch', 'j', 'am', 'chem', 'soc,', '1959', '81', '1345', '194', 'kf', 'o’driscoll', 'av', 'tobolsky', 'j', 'polymer', 'sci1959,', '35259', '195', 's', 'bywater', 'dj', 'woraford', 'can', 'j', 'chem,', '1966', '44', '1671', '196', 'k', 'matsuzaki', 'y', 'shinobara', 't', 'kanaimakromol', 'chem,', '1980', '18', '1923', '(1980)', '197', 'j', 'boor', 'jr,', 'ziegler-natta', 'catalysts', 'polymerization', 'academic', 'press', 'new', 'york', '1979', '180', '198', 'kf', 'o’driscoll', 't', 'yonezawa', 't', 'higashimura', 'jmacromol', 'sci-chem,', '1966', '1', '1', '199', 'm', 'morton', 'rd', 'sanderson', 'r', 'sakata', 'j', 'polymer', 'sci', 'polymer', 'letters', '1971', '9', '61', '200', 'm', 'morton', 'indeng', 'chem,', 'prod', 'res', 'dev,', '1972', '11', '106', '201', 'dl', 'glusker', 'e', 'stiles', 'b', 'yancoskie', 'j', 'polymer', 'sci,', '1961', '49', '297', '202', 'k', 'butler', 'pr', 'thomas', 'j', 'tyler', 'j', 'polymer', 'sci.', '1960', '48', '357', '203', 'r', 'kraft', 'ahe', 'muller', 'h', 'hoker', 'gv', 'schulz', 'macromol', 'chem', 'rapid', 'commun,', '1980', '1', '363', '204', 'g', 'lohr', 'gv', 'schulz', 'makromol', 'chem,', '1973', '172', '137', '205', 'v', 'warzelhan', 'h', 'hocker', 'gv', 'schulz', 'makromol', 'chem,', '1978', '179', '2221', '206', 'r', 'kraft', 'are', 'muller', 'v', 'warzelhan', 'h', 'hocker', 'gv', 'schulz', 'macromolecules', '1978', '11', '1093', '207', 'w', 'fowells', 'c', 'schuerch', 'fa', 'bovey', 'fp', 'hood', 'j', 'am', 'chem', 'soc,', '1967', '89', '1396', '208', 'dl', 'gluaker', 'e', 'stiles', 'b', 'yonkpskie', 'j', 'polymer', 'sci,', '1961', '49', '297', '209', 'ceh', 'bawn', 'a', 'ledwith', 'quart', 'rev', '(london)', '1962', '16', '361', '210', 'huynh', 'ba', 'gia', 'je', 'mcgrath', 'am', 'chem', 'soc', 'polymer', 'preprints', '1980', '21(1)', '74', '211', 'te', 'hogen-esch', 'j', 'smid', 'eds', 'recent', 'advances', 'anionic', 'polymerization', 'elsevier', 'newyork', '1987', '212', 'tj', 'clark', 'j', 'tien', 'dc', 'duffy', 'pke', 'whitesides', 'j', 'am', 'chem', 'soc,', '2001', '123', '7677', '213', 'wj', 'evans', 'dg', 'giarikos', 'nt', 'allen', 'macromolecules', '2003', '36', '4256', '214', 'dj', 'breslow', 'ge', 'hulse', 'as.', 'matlack', 'jam.', 'chem', 'soc,', '1957', '79', '3760', '215', 'lw', 'bush', 'ds', 'breslow', 'macromolecules', '1968', '1', '189', '216', 'y', 'kohuke', 'k', 'hanji', 'j', 'furukawa', 't', 'fueno', 'j', 'polymer', 'sci,', '1971', 'a-19', '431', 't', 'otsu', 'b', 'yamada', 'm', 'itahashi', 't', 'mori', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1976', '14', '1347', '217', 'wm', 'saltman', 'we', 'gildes', 'j', 'lal', 'jam', 'chem', 'soc,', '1958', '80', '5615', '218', 'k', 'yamaguchi', 'm', 'mirioura', 'j', 'polymer', 'sci,', '1972', 'a-110', '1217', '219', 'dj', 'cram', 'kr', 'kopecky', 'j', 'am', 'chem', 'soc,', '1959', '81', '2748', 'te', 'hogen-esch', 'j', 'smid', 'eds,', 'recent', 'advances', 'anionic', 'polymerization', 'elsevier', 'new', 'york', '1987', 'ahe', 'muller', 'carbonionic', 'polymerization', '248', '4', 'ionic', 'chain-growth', 'polymerization', 'kinetics', 'thermodynamics', 'comprehensive', 'polymer', 'science', 'vol', '3', 'gc', 'eaatmond', 'a', 'ledwith', 's', 'russo', 'p', 'sigwalt', 'eds', 'pergamon', 'press', 'london.', '1989', '220', 'a', 'ottolenghi', 'a', 'zilkha', 'j', 'polymer', 'sci,', '1963', 'a-1l', '687', 'y', 'imanishi', 'carbanionic', 'polymerization', 'hydrogen', 'migration', 'polymerization', 'comprehensive', 'polymer', 'science', 'vol', '3', 'gc', 'eaatmond', 'a', 'ledwith', 'srusso', 'p', 'sigwalt', 'eds,', 'pergamon', 'press', 'iondon', '1989', '221', 'ph', 'teyssie', 'k', 'fayt', 'jp', 'hautekeer', 'c', 'jacobs', 'r', 'jerome', 'l', 'leemans', 'sk', 'varshney', 'macromol', 'chem,', 'macromol', 'symp,', '1990', '32', '61', 'sk', 'varshney', 'jp', 'hautekeer', 'k', 'fayt', 'r', 'jerome', 'ph', 'teysaie', 'macromolecules1990', '23', '2618', '222', 'm', 'kuroki', 't', 'watanabe', 't', 'aida', 's', 'inoue', 'j', 'am', 'chem', 'soc,', '1991', '113', '5903', 't', 'adachi', 'h', 'sugirnoto', 't', 'aida', 's', 'inoue', 'macromolecules', '1992', '25', '2280', '223', 't', 'suzuki', 'j', 'kusakabe', 'k', 'kitazawat', 'nakagawa', 's', 'kawauchi', 't', 'ishizone', 'macromolecules', '2010', '43', '107', '224', 'k', 'ziegler', 'e', 'holzkamp', 'h', 'breil', 'h', 'martin', 'angew', 'chem,', '1955', '67', '541', '225', 'g', 'natta', 'j', 'polymer', 'sci,', '1955', '16', '143', '226', 'w', 'cooper', 'chaptl', 'vinyl', 'allied', 'polymers', 'vol1', 'pd', 'ritchie', 'ed,', 'crc', 'press', '1968', '227', 'h', 'bestian', 'k', 'clauss', 'h', 'jensen', 'e', 'prinz', 'angew', 'chem,', 'intern', 'ed', '1963', '2', '32', '228', 'g', 'natta', 'j', 'inorg', 'nuc', 'chem,', '1958', '8', '589', '229', 'do', 'jordan', 'chaptl', 'stereochemistry', 'macromolecules', 'vol1', 'a', 'v', 'ketley', 'ed,', 'dekker', 'new', 'york', '1967', '230', 'r', 'rieger', 'x', 'mu', 'dt', 'mallin', 'md', 'raush', 'jcw', 'chen', 'macromolecules', '1990', '23', '383', 'nh', 'cheng', 'ja', 'ewen', 'makromol', 'chem,', '1989', '190', '1931', 'a', 'grassi', 'azambelli', 'l', 'resconi', 'e', 'albizzati', 'r', 'mazzocchi', 'macromolecules', '1988', '21', '617', 'yvkissin', 'te', 'nowlin', 'ri', 'mink', 'macromolecules', '1993', '26', '2151', '231', 'lewis', 'rundle', 'j', 'chem', 'phys,', '1953', '1986', '232', 'g', 'natta', 'p', 'pino', 'g', 'mazzanti', 'v', 'gianini', 'e', 'mantica', 'm', 'peraldo', 'j', 'polymer', 'sci,', '1957', '26', '20', '233', 'g', 'natta', 'g', 'mazzanti', 'tetrahedron', '1960', '6', '86', '234', 'n', 'c', 'billinghani', 'br', 'polymerj.', '1974', '6', '299', '235', 'p', 'corradini', 'g', 'guerra', 'r', 'rusco', 'v', 'barone', 'eur', 'polymerj,', '1980', '16', '835', '236', 'p', 'cossee', 'tetrahedron', 'letters', '1960', '17', '12', '17', 'trans', 'faraday', 'soc,', '1962', '58', '1226', '237', 'p', 'cossee', 'e', 'j', 'arlman', 'j', 'catalysis', '1964', '3', '99', 'p', 'cosaeej', 'catalysis', '1964', '3', '80', '238', 'j', 'chatt', 'bi', 'shaw', 'j', 'chem', 'soc,', '1959', '705', '239', 'a', 'zambelli', 'i', 'pasquon', 'r', 'signorini', 'g', 'natta', 'makromol', 'chem,', '1968', '112', '160', '240', 'a', 'zambelli', 'mg', 'giongo', 'g', 'natta', 'makromol', 'chem,', '1968', '112', '183', 'mgoodman', 'concepts', 'polymer', 'stereochemistry', 'topics', 'stereochemistry', 'ng', 'allinger', 'el', 'eliel', 'eds,', 'wiley-interscience', 'new', 'york', '1967', '241', 'k', 'j.', 'ivin', 'jj.', 'rooney', 'cd', 'stewaft', 'mlh', 'green', 'r', 'mahtab', 'j', 'chem', 'soc,', 'chem', 'commun,', '1978', '604', 'kj.', 'ivin', 'proc', 'eur', 'symp', 'polym', 'spectrosc,', '1978', '11', '267', '242', 't', 'kahara', 'm', 'shinoyama', 'y', 'doi', 't', 'keii', 'makromol', 'chem,', '1979', '180', '2199', '243', 'a', 'zambelli', 'mcsacchi', 'p', 'locatelli', 'macromolecules', '1979', '12', '1051', 'a', 'zanibelli', 'g', 'allegra', 'macromolecules', '1980', '13', '42', '244', 'l', 'cavallo', 'g', 'guerra', 'p', 'caradini', 'j', 'am', 'chem', 'soc,', '1998', '120', '2428', '245', 'p', 'pino', 'gp', 'lorenzi', 'l', 'lardicci', 'chim', 'ind', '(milan)', '1960', '42', '712', '246', 'p', 'pino', 'f', 'ciardelli', 'gp', 'lorenzi', 'j', 'am', 'chem', 'soc,', '1961', '84', '1487', '247', 'p', 'pino', 'f', 'ciardelli', 'gp', 'lonenzi', 'j', 'polymer', 'sci,', '1963', 'c4', '21', '248', 'p', 'carradini', 'g', 'pajaro', 'a', 'pannunzi', 'j', 'polymer', 'sci,', '1967', 'cl6', '2905', '249', 'a', 'zambelli', 'p', 'locatelli', 'm', 'c', 'sacchi', 'e', 'rigamoti', 'macromolecules', '1980', '13', '798', '250', 'p', 'pino', 'fortschr', 'hochpolym', 'forsch1965,', '4', '393', '251', 'g', 'allegra', 'macromol', 'chem,', '1971', '145', '235', 'f', 'ciardelli', 'macrornolecules', '1970', '3527', '252', 'p', 'pino', 'r', 'melhaupt', 'angew', 'chem,', 'intern', 'ed.', '1980', '19', '857', '253', 'cp', 'casey', 'macromolecules', '1981', '14', '64', '254', 'mlh', 'green', 'pure', 'appl', 'chem,', '1978', '50', '27', '255', 'fa', 'bovey', 'high', 'resolution', 'nmr', 'macromolecules', 'acad', 'press', 'new', 'york', '1972', '256', 'p', 'margl', 'l', 'deng', 't', 'ziegler', 'j', 'am', 'chem', 'soc,', '1998', '120', '5517', '257', 'jwl', 'fordham', 'ph', 'burleigh', 'cl', 'sturm', 'j', 'polymer', 'sci,', '1959', '42', '73', '258', 'fa', 'bovey', 'j', 'polymer', 'sci,', '1960', '46', '59', '259', 'g', 'natta', 'ipasquon', 'a', 'zambelli', 'j', 'am', 'chem', 'soc,', '1962', '84', '1488', '260', 'a', 'zambelli', 'g', 'natta', 'i', 'pasquon', 'j', 'polymer', 'sci,', '1963', 'c4', '411', '261', 'a', 'zambelli', '23rd', 'int', 'cong', 'iupac', 'macromoll', 'preprints', '1971', '1', '124', 'a', 'zambelli', 'g', 'gatti', 'macromolecules', '1978', '11485', 'a', 'zambelli', 'p', 'locatelli', 'mc', 'sacchi', 'e', 'rigamonte', 'macromolecules', '1980', '13', '798', 'p', 'amandolla', 't', 'tancredi', 'a', 'zambelli', 'macromolecules', '1986', '19', '307', 'i', 'tnitto', 'mc', 'zacchi', 'p', 'locatelli', 'makromol', 'chem', '1986', '187', '214', '262', 'j', 'boor', 'jr', 'ziegler-nalta', 'catalysts', 'polymerizations', 'academic', 'press', 'new', 'york', '1979', 'references', '249', '263', 'bg', 'dolgoploak', 'pa', 'vinogradov', 'op', 'parenago', 'ei', 'tinyatova', 'bs', 'turov', 'iuapc', 'int', 'symp', 'macromol', 'chem', 'prague', 'preprints', 'p1314', '1965', '264', 'a', 'ledwith', 'dc', 'sherrington', 'reactivity', 'mechanism', 'in', 'polymerization', 'complex', 'organometallic', 'derivatives', 'reactivity', 'mechanism', 'structure', 'polymer', 'chemistry', 'a', 'd', 'jenkins', 'a', 'ledwith', 'eds,', 'wiley-interscience', 'london', '1974', '265', 'kmatsuzaki', 'tyasukawa', 'jpolymer', 'sci', '1967', 'a-l5', '521', '266', 'k', 'matsuzaki', 't', 'yasukawa', 'j', 'polymer', 'sci,', '1967', 'a-25', '511', '267', 'h', 'hirai', 'k', 'hiraki', 'i', 'noguchi', 't', 'lnoue', 's', 'makishima', 'j', 'polymer', 'sci,', '1970', 'a-l8', '2395', '268', 's', 'destri', 'mc', 'gallazzi', 'a', 'giarrusso', 'l', 'pori', 'mokromol', 'chem,', 'rapid', 'commun,', '1980', '1193', '269', 'rp', 'hughes', 'j', 'powell', 'jam', 'chem', 'soc,', '1972', '94', '7723', '270', 'ho', 'murdoch', 'e', 'weiss', 'halv', 'chim', 'acta,', '1962', '45', '1156', '271', 'ca', 'tolman', 'j', 'am', 'chem', 'soc,', '1970', '92', '6785', '272', 'a', 'yammoto', 't', 'yamamoto', 'jpolymer', 'sci,', 'macromol', 'reviews', '1978', '13', '161', '273', 'g', 'wilke', 'b', 'bogdanovic', 'angew', 'chem,', '1961', '73', '756', '274', 'g', 'wilke', 'angew', 'chem,', 'intern', 'ed', '1963', '2', '105', '275', 'mlh', 'green', 'organometallic', 'compounds', 'vol', '2', 'menthen', 'london', 'p39', '(1968)', '276', 'g', 'wiike', 'angew', 'chem,', 'intern', 'ed,', '1966', '5', '151', '277', 'dgb', 'ballard', 'e', 'jones', 't', 'medinger', 'ajp', 'pioli', 'mokromol', 'chem', '1971', '148', '175', '278', 'dgh', 'ballard', 'adv', 'catal,', '1973', '23', '263', '279', 'f', 'dawans', 'p', 'teyssie', 'ind', 'eng', 'chem,', 'prod', 'res', 'develop,', '1971', '10(3)', '261', '280', 'p', 'bourdauducq', 'f', 'dawans', 'j', 'polymer', 'sci,', '1971', 'a-110', '2527', '281', 'v1', 'klepkova', 'gp', 'kondratenkov', 'va', 'kormer', 'mi', 'lobach', 'la', 'churlyaeva', 'j', 'polymer', 'sci,polymer', 'letters', '1973', '11', '193', '282', 'jp', 'durand', 'f', 'dawans', 'phteyssie', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1967', '5', '785', '283', 'p', 'j', 'ttait', 'n', 'd', 'watkins', 'monoalkene', 'polymerization', 'mechanisms', 'chap', '2', 'comprehensive', 'polymer', 'science', 'vol', '4', 'gc', 'eastman', 'a', 'ledwith', 's', 'russo', 'p', 'sigwalt', 'eds,', 'pergamon', 'press', 'oxford', '1989', '284', 'r', 'wessermel', 'h', 'cbedron', 'j', 'berthod', 'b', 'diedrich', 'kd', 'keil', 'k', 'rust', 'h', 'strarnelz', 't', 'toth', 'j', 'polymer', 'sci', 'symposia', '1975', '51', '187', '285', 'a', 'simon', 'a', 'grobler', 'jpolymer', 'sci,polymer', 'chem', 'ed', '1980', '18', '3111', '286', 'y', 'doi', 'e', 'suzuki', 't', 'keii', 'makromol', 'chem,', 'rapid', 'comm,', '1981', '2', '293', '287', 's', 'sivaram', 'ind', 'eng', 'chem,', 'prod', 'resdev,', '1977', '16', '121', '288', 'k', 'soga', 'm', 'terano', 'mokromol', 'chem,', '1981', '182', '2439', '289', 'b', 'kezler', 'a', 'grobler', 'e', 'takacs', 'a', 'simon', 'polymer', '1981', '22', '818', '290', 'b', 'rieger', 'x', 'mu', 'dt', 'mallin', 'md', 'raush', 'jcw', 'chen', 'macromolecules', '1990', '23', '383', 'nh', 'cheng', 'ja', 'ewen', 'makromol', 'chem1989,', '190', '1931', 'a', 'grassi', 'a', 'zambelli', 'i', 'resconi', 'e', 'albizzati', 'r', 'mazzocchi', 'macromolecules', '1988', '21', '617', 'yvkiasin', 'te', 'nowlin', 'ri', 'mink', 'macromolecules', '199326', '2151', '291', 't', 'asanurna', 'y', 'nishirnori', 'm', 'ito', 'n', 'uchicawa', 't', 'shiornura', 'polym', 'bull', '1991', '25', '567', '292', 'w', 'kaminsky', 'r', 'engehausen', 'j', 'kopf', 'angew', 'chem', 'int', 'ed', '1995', '34', '2273', '293', 'tk', 'han', 'bw', 'woo', 'jt', 'park', 'y', 'do', 'ys', 'ko', 'si', 'woo', 'macromolecules', '1995', '8', '4801', '294', 's', 'miyake', 'y', 'okumura', 's', 'inazawa', 'macromolecules', '1995', '28', '3074', '295', 'h', 'kuribayashi-kawamura', 'n', 'koga', 'k', 'morokuma', 'j', 'amchem', 'soc,', '1992', '114', '8687', '296', 'jcw', 'lohrenz', 'rk', 'woo', 'l', 'fan', 't', 'ziegler', 'am', 'chem', 'soc', 'polymer', 'preprints', '1995', '36(1)', '231', '297', 'tk', 'woo', 'l', 'fan', 't', 'ziegler', 'organometallics', '1994', '13', '2252', '298', 'g', 'guerra', 'l', 'cavallo', 'g', 'moscardi', 'm', 'vacatello', 'p', 'corradini', 'macromolecules', '1996', '29', '4834', '299', 'v', 'busico', 'd', 'brita', 'l', 'caporaso', 'r', 'cipullo', 'm', 'vacalello', 'macromolecules', '1997', '30', '3971', '300', 'w', 'zang', 'k', 'wei,', 'lr', 'sita', 'macromolecules', '2008', 'web', 'publication', '101021801962v', '301', 'm', 'watanabe', 'k', 'okamoto', 'jpn', 'kokai', 'tokkyo', 'kobe', 'jp', '2004', '107390', 'chem', 'abstr', '302', 'hli', 'clstern', 'tj', 'marks', 'macromolecules', '2005', '389028', '303', 'h', 'li', 'l', 'li', 'tj', 'marks', 'ang', 'chem', 'int', 'ed', '2004', '37', '4937', '304', 'mr', 'salata', 'tj', 'marks', 'macromolecules', '2009', 'web', 'publication', '101021ma8020745', '305', 'k', 'homura', 'h', 'fukuda,', 's', 'katao', 'm', 'fujiki', 'h', 'j', 'kim', 'd-h', 'kim', 'i', 'saeed', 'macromolecules', '201144', '1986', '306', 'ad', 'caunt', 'j.polymer', 'sci', '1964', 'c4', '49', 'ad', 'caunt', 'brit', 'polymer', 'j,', '1981', '13', '22', '307', 'icw', 'chien', 'jc', 'wu', 'ci', 'kuo', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed', '1982', '20', '2019', 'ksoga', 't', 'shiono', 'y', 'doi', 'makromol', 'chem,', '1988', '1891531', 'mc', 'sacchi', 'c', 'shan', 'p', 'locatelli', 's', 'tritto', 'macromolecules', '1990', '23383', '308', 'n', '0', 'gaylord', 'h', 'f', 'mark', 'linear', 'stereoregular', 'addition', 'polymers', 'wiley-lnterscience', 'new', 'york', '1959', '309', 'ef', 'peters', 'a', 'zletz', 'bl', 'evering', 'ind', 'eng', 'chem,', '1957', '48', '1879', '310', 'a', 'clark', 'jp', 'hogan', 'bl', 'banks', 'wc', 'lanning', 'ind', 'eng', 'chem,', '1956', '48', '1152', '311', 'k', 'endo', 't', 'otsu', 'j', 'polymer', 'sci', 'polymer', 'chem', 'ed', '1979', '17', '1453', '250', '4', 'ionic', 'chain-growth', 'polymerization', '312', 'a', 'shimizu', 't', 'otsu', 'm', 'imoto', 'j', 'polymer', 'sci.', 'polymer', 'letters', '1965', '3449', '1031', 'a', 'shimizu', 't', 'otsu', 'm', 'ito', 'j', 'polymer', 'sci1966,', 'a-14', '1579', '313', 'mdelferro', 'tj', 'marks', 'chem', 'rev,', '2011', '111', '2450', '314', 'jp', 'kennedy', 't', 'otsu', 'adv', 'polymer', 'sci,', '1970', '7', '369', '315', 'a', 'shirnizu', 'k', 'itakura', 't', 'otsu', 'm', 'imoto', 'j', 'polymer', 'sci,', '1969', 'a-17', '3119', '316', 't', 'otsu', 'h', 'nagahama', 'k', 'fndo', 'j', 'polymer', 'sci', 'polymer', 'letters', '1972', '10', '601', '317', 't', 'otsu', 'h', 'nagahama', 'k', 'endo', 'j', 'macromol', 'sci-chem,', '1975', '9', '1249', '318', 't', 'otsu', 'k', 'endo', 'j', 'macromol', 'sci-chem,', '1975', '9', '899', '319', 't', 'otsu', 'a', 'shimizu', 'k', 'itakura', 'kendo', 'j', 'polymer', 'sci,,', 'polymer', 'chem', 'ed1975', '13', '1589', '320', 'kendo', 't', 'otsu', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1991', '29', '847', '321', 'k', 'endo', 'it', 'ueda', 't', 'otsu', 'macromolecules', '1991', '24', '6849', '322', 'o', 'vogl', 'makromol', 'chem,', '1975', '175', '1281', '323', 'o', 'vogl', 'jmacromol', 'sci,', 'revmacromol', 'chem,', '1975', 'c12', '(1)', '109', '324', 'o', 'vogl', 'polyaldehydes', 'dekker', 'new', 'york', '1967', '325', 'w', 'kern', 'e', 'eberius', 'v', 'jaacks', 'mokromol', 'chem,', '1971', '141', '63', '326', 'v', 'jaacks', 'k', 'boehlke', 'e', 'eberius', 'makromol', 'chem,', '1968', '118', '354', '327', 'k', 'boehlke', 'v', 'jaacks', 'makromol', 'chem,', '1971', '142', '189', '328', 'p', 'kubisa', 'k', 'neeld', 'j', 'starr', 'o', 'vogi', 'polymers', '1980,', '1', '1433', '329', 'j.', 'furukawa', 't', 'saegusa', 't', 'tsuruta', 'h', 'fujii', 'a', 'kawasaki', 't', 'tatano', 'makromol', 'chem,', '1960', '33', '32', '330', 'j', 'funikawa', 't', 'saeguaa', 'h', 'fujii', 'a', 'kawasaki', 't', 'tatano', 'j', 'polymer', 'sci,', '1969', '6', '546', '331', 'k', 'weissermel', 'w', 'schneider', 'makromol', 'chem,', '1962', '1', '39', '332', 'o', 'vogl', 'j', 'polymer', 'sci,', '1960', '46', '261', '333', 'w', 'kern', 'chem', 'z', '1964', '88', '623', '334', 'e', 'kunzel', 'a', 'gieffer', 'w', 'kern', 'mokromol', 'chem,', '1966', '96', '17', '335', 'z', 'machacek', 'j', 'mejzlick', 'j', 'pac', 'vysokomol', 'soed,', '1961', '3(9)', '1421', '336', 'j', 'mejzlick', 'j', 'mencikova', 'z', 'machacek', 'vysokomol', 'soed.', '1962', '4', '769776', '337', 'k', 'vesely', 'j', 'mejzlick', 'vysokomol', 'soed', '1963', '5(9)', '1415', '338', 'n', 'mathes', 'k', 'jaacks', 'makromol', 'chem,', '1970', '135', '49', '339', 'ce', 'schweitzer', 'rn', 'macdonald', 'jo.', 'punderson', 'j', 'appl', 'polymer', 'sci,', '1959', '1', '158', '340', 'a', 'henglein', 'w', 'sshnabel', 'rc', 'schulz', 'mokromol', 'chem,', '1959', '31', '131', '341', 'o', 'vogl', 'jpolymer', 'sci,', '1960', '46', '161', '342', 'j', 'furukawa', 't', 'saegusa', 'h', 'fujii', 'a', 'kawasaki', 'h', 'imai', 'y', 'fujii', 'makromol', 'chem,', '1959', '37', '149', '343', 'g', 'natta', 'g.', 'mazzanti', 'p', 'corradini', 'jw', 'bassi', 'makromol', 'chem,', '1959', '37', '156', '344', 'j', 'furukawa', 't', 'saeguaa', 'h', 'fujii', 'makromol', 'chem,', '1961', '44-46', '398', '345', 'o', 'vogi', 'wmd', 'bryant', 'j', 'polymer', 'sc.', '1964', 'a-12', '921', '346', 'h', 'tani', 'fortschr', 'hochpolymer', 'forsch,', '1973', '11', '57', '347', 'h', 'tani', 't', 'araki', 'y', 'yasuda', 'j', 'polymer', 'sci,polymer', 'letters', '1966', '4', '727', '348', 'h', 'yasuda', 'h', 'tani', 'macromolecules', '1973', '6', '17', '349', 'a', 'novak', 'e', 'whaley', 'can', 'j', 'chem,', '1959', '37', '1710', '1718', '350', 'rc', 'schulz', 'h', 'fauth', 'w', 'kern', 'makromol', 'chem,', '1956', '21', '227', '351', 'rc', 'schulz', 'h', 'cherdron', 'w', 'kern', 'makromol', 'chem,', '195724141', '(1957)', '352', 'a', 'henglein', 'w', 'schnabel', 'rc', 'schultz', 'makromol', 'chem', '1959', '31', '131', '353', 'g', 'odian', '‘principles', 'polymerization’', 'iv', 'ed', 'wiley', 'new', 'york', '2004', '354', 'rc', 'schulz', 'w', 'passrnann', 'makromol', 'chem,', '1963', '60', '139', '355', 'rc', 'schulz', 'gwagner', 'w', 'ken', 'j', 'polymer', 'sci', '1967', 'c16', '989', '356', 'rc', 'schulz', 's', 'suzuki', 'h', 'cherdron', 'w', 'kern', 'makromol', 'chem,', '1962', '53', '145', '357', 'c', 'aso', 's', 'tagami', 'macromolecules', '1969', '2', '414', '358', 'y', 'katahama', 's', 'lshida', 'makromol', 'chem,', '1968', '119', '64', '359', 'k', 'kobayashi', 'h', 'sumimoto', 'j', 'polymer', 'chem,', 'polymer', 'letters', '1972', '10', '703', '360', 'vv', 'amerik', 'ba', 'krentsel', 'mv', 'shiskina', 'vysokomol', 'soyed', '1965', '7(10)', '1713', '361', 'gp', 'pregaglia', 'm', 'benaghi', 'stereochemistry', 'macromolecules', 'ad', 'ketiey', 'eddekker,', 'new', 'york', '1967', '362', 'c', 'aso', 's', 'tagarni', 't', 'kunitake', 'j', 'polymer', 'sci,', '1969', 'a-17', '497', '363', 'c', 'aso', 's', 'tagami', 'macromolecules', '1969', '2', '414', '364', 'j', 'furukawa', 't', 'saegusa', 'polymerization', 'aldehydes', 'oxides', 'wiley-', 'interscience', 'new', 'york', '1963', '365', 've', 'shashoua', 'w', 'sweeny', 'rf', 'tietz', 'j', 'am', 'chem', 'soc,', '1960,', '82', '866', '366', 'y', 'iwakura', 'k', 'uno', 'n', 'kobayashi', 'j', 'polymer', 'sci,', '1968', 'a-16', '793', '367', 'gc', 'east', 'j', 'furukawa', 'polymer', '1979', '20', '659', '368', 'g', 'natta', 'j', 'di', 'pietro', 'm', 'cambini', 'makromol', 'chem,', '1962', '56', '200', '369', 'hc', 'beachell', 'cp', 'ngocson', 'j', 'polymer', 'sci,', 'polymerletters', '1963', '1', '25', 'references', '251', '370', 'dc', 'pepper', 'friedel-craft', 'related', 'reactions', 'volii', 'ga', 'olah', 'ed,', 'interscience', 'new', 'york', '1964', '371', 'jf', 'dunphy', 'cs', 'marvel', 'j', 'polymer', 'sci,', '1960', '47', '1', '372', 'rb', 'cundall', 'the', 'chemistry', 'cationic', 'polymerization.', 'ph', 'plesch', 'ed', 'macmillan', 'co,', 'new', 'york', '1963', '373', 'cg', 'overberger', 'lh', 'arnold', 'jj', 'taylor', 'j', 'am', 'chem', 'soc,', '1951', '73', '5541', '374', 'cg', 'overberger', 'rj', 'ehrig', 'dh', 'tanner.', 'am', 'chem', 'soc,', '1954', '76', '772', '375', 'cg.', 'overberger', 'dh', 'tanner', 'em', 'pierce', 'j', 'am', 'chem', 'soc,', '1958', '80', '4566', '376', 'cg.', 'overberger', 'vg', 'kamath', 'j', 'am', 'chem', 'soc,', '1959', '81', '2910', '377', 'kf', 'o’driscoll', 't', 'yonezawa', 'ahhigashirnura', 'j', 'macromol', 'sci-chem,', '1966', '1', '17', '378', 'h', 'yuki', 'k', 'kosai', 's', 'murahashi', 'j', 'hotta', 'j', 'polymer', 'sci,', 'polymerletters', '1964', '3', '1121', '379', 'kf', 'o’driscoll', 'rj', 'boudreau', 'av', 'tobolsky', 'j', 'polymer', 'sci,', '1958', '115', '380', 'kf', 'o’driscoll', 'av', 'tobolsky', 'j', 'polymer', 'sci,', '1958', '31', '123', 'ibid', '1959', '32', '363', '381', 'av', 'tobolsky', 'db', 'hartley', 'j', 'polymer', 'sci,', '1963', 'a-11', '15', '382', 'h', 'yuki', 'y', 'okarnoto', 'k', 'ohta', 'k', 'hatada', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1975', '13', '116', '383', 'a', 'zhao', 'h', 'tanaka', 'nmiyamoto', 's', 'kaizumi', 't', 'hashimoto', 'macromolecules', '2009', '(acs', 'web', 'publication', '101021ma8017519)', '384', 'ca', 'lukach', 'hm', 'sperlin', 'copolymerization', 'ge', 'kain', 'ed,', 'wiley-interscience', 'new', 'york', '1964', 'hf', 'mark', 'n', 'ogata', 'j', 'polymer', 'sci,', '1963', 'a-11', '3439', '385', 'd', 'guironnet', 's', 'mecking', 'am', 'chem.', 'soc', 'polymer', 'preprints', '2008', '49', '(1)', '450', '386', 'j', 'furukawa', 'o', 'vogi', 'ionic', 'polymerization', 'dekker', 'new', 'york', '1976', '387', 'yp', 'castille', 'v', 'stannett', 'j', 'polymer', 'sci,', '1966', 'a-14', '2063', '388', 'j', 'furukawa', 't', 'saegusa', 't', 'tsunita', 's', 'ohta', 'g.', 'wasai', 'makromol', 'chem,', '1962', '52', '230', '389', 'ho', 'colornb', 'fe', 'bailey', 'jr', 'rd', 'lunterg', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1978', '16', '507', '390', 't', 'saeguaa', 'jmacromol', 'sci,', '1972', 'a6,', '997', '391', 'ahe', 'muller', 'makromol', 'chem,', 'macromol', 'symp,', '1990', '32', '87', 'w', 'schubert', 'fbandermann', 'makromol', 'chem1989,', '190', '2721', 'w', 'schubert', 'hd', 'sitz', 'f', 'bandermann', 'makromol', 'chem,', '1989', '190', '2193', 'dy', 'sogab', 'wr', 'hertler', 'lb', 'dicker', 'pa', 'depra', 'jr', 'butera', 'malcromol', 'chem,', 'macromol', 'symp,', '1990', '32', '75', '392', 'rp', 'quirk', 'j', 'ren', 'macromolecules', '1992', '25', '6612', '393', 'wv', 'farman', 'dy', 'sogah', 'am', 'chem', 'soc', 'polymer', 'preprints', '1986', '27', '167', '394', 'rp', 'quirk', 'gp', 'bidinger', 'polym', 'bull,', '1989', '22', '6', '395', 'mt', 'reetz', 'r', 'osterek', 'ke', 'piejko', 'd', 'arlt', 'd', 'bomer', 'ang', 'chem,', 'int', 'ed,', '1986', '25', '1108', '396', 'wt', 'herter', 'macromolecules', '1987', '20', '297', '397', 'c', 'pugh', 'v', 'percec', 'polym', 'bull,', '1985', '14', '109', '398', 'w', 'mormann', 'w', 'juergen', 'h', 'pasch', 'k', 'rode', 'macromolecules', '1998', '1', '249', '399', 'dy', 'sogah', 'ow', 'webster', 'macromolecules', '1986', '19', '1775', '400', 'glivinenko', 'ahe', 'muller', 'macromolecules', '1997', '30', '1253', '401', 'mb', 'scholten', 'ji', 'hedrick', 'rw', 'waymouth', 'am', 'chem', 'soc', 'polymer', 'preprints', '2007', '48(2)167', '402', 't', 'saegusa', 's', 'kobayshi', 'andl', 'jfurukawa', 'macromolecules', '1977', '10', '73', '403', 'fa', 'bovey', 'high', 'resolution', 'nmr', 'macromolecules', 'acad', 'press', 'new', 'york', '1972', '404', 'y', 'doi', 'makromol', 'chem,', 'rapid', 'commun', '1982', '3', '635', '405', 'hn', 'cheng', 'j', 'cham', 'infcomput', 'sci', '1987', '27', '8', '406', 'h', 'sawada', 'thermodynamics', 'polymerization', 'dekker', 'new', 'york', '1976', '407', 'kf', 'o’driscoll', 'j', 'polymer', 'sci,', '1962', '57', '721', '408', 'h', 'sawada', 'thermodynamics', 'polymerization', 'dekker', 'new', 'york', '1976', 'kj', 'ivin', 'j', 'leonard', 'eu', 'polymer', 'j,', '1970', '6', '331', 'a', 'vrancken', 'j', 'smid', 'm', 'swarc', 'trans', 'faraday', 'soc,', '1962', '56', '2036', '409', 'pj', 'flory', 'principles', 'polymer', 'chemistry,', 'cornell', 'univ', 'press', 'ithaca', 'new', 'york', '1953', 'chapter', '5', 'ring-opening', 'polymerizations', '51', 'chemistry', 'ring-opening', 'polymerizations', 'formation', 'polymers', 'ring-opening', 'reactions', 'cyclic', 'compounds', 'important', 'process', 'polymer', 'chemistry', 'in', 'polymerizations', 'chain-growth', 'takes', 'place', 'successive', 'additions', 'opened', 'structures', 'polymer', 'chain', 'an', 'example', 'ring-opening', 'polymerization', 'ethylene', 'oxide', 'results', 'formation', 'poly(ethylene', 'oxide)', 'polyether', 'n', 'r', 'x', 'r', 'x', 'n', 'o', 'n', 'o', 'n', 'the', 'cyclic', 'monomers', 'undergo', 'ring-opening', 'polymerizations', 'quite', 'diverse', 'among', 'cyclic', 'alkenes', 'lactones', 'lactams', 'many', 'heterocyclics', 'one', 'heteroatom', 'ring', 'such', 'polymerizations', 'ionic', 'character', 'may', 'exhibit', 'characteristics', 'typical', 'ionic', 'chain-growth', 'polymerizations', '(eg,', 'effect', 'counterion', 'solvent)', 'it', 'would', 'however', 'wrong', 'assume', 'polymerizations', 'necessarily', 'take', 'place', 'chain-propagating', 'mechanisms', 'actually', 'many', 'reactions', 'step-growth', 'nature', 'polymer', 'size', 'increasing', 'slowly', 'throughout', 'whole', 'course', 'process', 'there', 'hand', 'cyclic', 'monomers', 'polymerize', 'typical', 'chain-growth', 'manner', '52', 'kinetics', 'ring-opening', 'polymerization', 'there', 'general', 'similarity', 'kinetics', 'many', 'ring-opening', 'polymerization', 'step-growth', 'polymerizations', 'discussed', 'chap', '7', 'some', 'kinetic', 'expressions', 'ring-opening', 'polymerizations', 'hand', 'resemble', 'ionic', 'chain-growth', 'reactions', 'there', 'several', 'forms', 'rate', 'law', 'describe', 'cationic', 'ring-opening', 'polymerization', 'for', 'living', 'polymerizations', 'without', 'termination', 'one', 'write', 'rp', '¼', 'kp½m(cid2)(cid3)½m(cid3)', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-95', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '253', '254', '5', 'ring-opening', 'polymerizations', 'm]', 'concentration', 'propagating', 'oxonium', 'ions', 'such', 'ions', 'could', 'oxonium', 'sulfonium', 'others', 'when', 'however', 'propagation–depropagation', 'equilibrium', 'expressed', 'follows', 'm(cid2)', 'kp', 'n', 'þ', 'm', 'ð', 'kdp', 'm(cid2)', 'nþ1', 'the', 'rate', 'expression', 'written', 'propagation–depropagation', 'rp', '¼', '(cid4)d½m(cid3)dt', '¼', 'kp½m(cid2)(cid3)½m(cid3)', '(cid4)', 'kdp½m(cid2)(cid3)', 'at', 'condition', 'equilibrium', 'designate', 'monomer', 'concentration', 'mc', 'polymeri-', 'zation', 'rate', 'zero', 'write', 'hirota', 'fukuda', '1', 'described', 'quantitative', 'dependence', 'degree', 'polymerization', 'various', 'reaction', 'parameters', 'equilibrium', 'polymerization', 'the', 'equilibrium', 'described', 'kp½m(cid3)c', '¼', 'kdp', 'ki', 'i', 'þ', 'm', 'ð', 'm(cid2)', 'i', 'initiating', 'species', 'it', 'assumed', 'equilibrium', 'constants', 'initiation', 'propagation', 'independent', 'size', 'propagating', 'species', 'the', 'concentration', 'propagating', 'chains', 'm]', 'size', 'n', 'equilibrium', 'c', 'written', '½m(cid2)(cid3)', '¼', 'ki½i(cid3)c½m(cid3)cðkp½m(cid3)cþn(cid4)1', 'the', 'total', 'concentration', 'molecules', 'size', 'n', 'expressed', 'follows', '½n(cid3)', '¼', '½m(cid2)', 'n(cid3)ki½i(cid3)c½m(cid3)cð1', '(cid4)', 'kp½m(cid3)cþ', 'the', 'total', 'concentration', 'monomer', 'segments', 'incorporated', 'polymer', 'also', 'expressed', 'follows', '½w(cid3)', '¼', '½m(cid2)(cid3)', '¼', 'ki½i(cid3)c½m(cid3)cð1', '(cid4)', 'kp½m(cid3)cþ2', 'x', 'xn', 'this', 'allows', 'us', 'express', 'average', 'degree', 'polymerization', 'w/n', 'follows', 'dp', '¼', '1', '1', '(cid4)', 'kp½m(cid3)c', 'we', 'describe', 'rate', 'polymerization', 'terms', '(cid4)dm/dt', '53', 'polymerization', 'oxiranes', '255', 'rp', '¼', '(cid4)d½m(cid3)dt', '¼', 'kpð½m(cid2)(cid3)mþ', '(cid4)', '½m(cid3)þ', 'the', 'expression', 'integrated', 'yield', '½m(cid3)0', '(cid4)', '½m(cid3)c', '½m(cid3)', '(cid4)', '½m(cid3)c', 'in', '(cid2)', '(cid3)', '¼', 'kp½m(cid2)(cid3)t', 'm0', 'initial', 'monomer', 'concentration', '53', 'polymerization', 'oxiranes', 'polymerizations', 'oxiranes', 'epoxides', 'occur', 'one', 'three', 'different', 'mechanisms', '(1)', 'cationic', '(2)', 'anionic', '(3)', 'coordination', 'in', 'respect', 'oxiranes', 'differ', 'rest', 'cyclic', 'ethers', 'polymerized', 'help', 'strong', 'cationic', 'initiators', 'it', 'appears', 'though', 'sometimes', 'coordination', 'catalysis', 'might', 'also', 'effective', 'polymerizations', 'oxetanes', 'the', 'susceptibility', 'oxirane', 'compounds', 'anionic', 'initiation', 'explained', 'fact', 'strained', 'ring', 'compounds', 'because', 'rings', 'consist', 'three', 'atoms', 'electrons', 'oxygen', 'crowded', 'vulnerable', 'attack', '2.', '531', 'cationic', 'polymerization', 'various', 'lewis', 'protonic', 'acids', 'capable', 'initiating', 'cationic', 'polymerization', 'epoxies', 'among', 'following', 'metal', 'salts', 'effective', 'polymerizations', 'ethylene', 'propylene', 'oxides', '3', '4:', 'zncl2', 'alcl3', 'sbcl5', 'bf3', 'bcl3', 'becl2', 'fecl3', 'sncl4', 'ticl4', 'often', 'polymerizations', 'carried', 'bulk', 'without', 'solvent', 'particularly', 'laboratory', 'the', 'mechanism', 'reactions', 'complex', 'however', 'depending', 'upon', 'particular', 'lewis', 'acid', 'used', 'in', 'fact', 'polymerizations', 'even', 'treated', 'general', 'terms', 'cationic', 'for', 'instance', 'ferric', 'chloride', 'initiated', 'polymerizations', 'epoxides', 'initially', 'proceed', 'mechanism', 'superﬁcial', 'features', 'cationic', 'polymerization', 'after', 'initial', 'stages', 'however', 'polymerizations', 'proceed', 'coordination', 'mechanism', 'this', 'discussed', 'section', 'stannic', 'chloride', 'yields', 'low', 'molecular', 'weight', 'poly(ethylene', 'oxide)', 'ethylene', 'oxide', '(molecular', 'weight', '5000)', 'reaction', 'carried', 'ethylene', 'chloride', 'room', 'temperature', 'some', 'dioxane', 'dioxolane', 'also', 'form', 'process', 'following', 'reaction', 'scheme', 'proposed', '2–6:', 'initiation', 'o', 'sncl4', 'sncl4', 'o', 'o', 'o', 'o', '256', 'propagation', '5', 'ring-opening', 'polymerizations', 'sncl4', 'o', 'o', 'o', 'o', 'slow', 'sncl4', 'o', 'o', 'o', 'o', 'ch2', 'ch2', 'sncl4', 'o', 'o', 'o', '2', '2', 'o', 'ch2', '2', 'o', 'fast', 'o', 'o', 'o', 'sbcl4', 'ch2', 'o', 'o', 'o', 'ch2', 'sncl4', 'o', 'o', 'termination', 'sncl4', 'o', 'o', 'o', 'o', 'ch2', 'o', 'ch2', 'o', 'sncl4', 'o', 'o', 'ch2', 'o', 'o', 'ch2', 'o', 'o', 'n', 'o', 'ch2', 'sncl3', 'o', 'o', 'ch2', 'o', 'o', 'o', 'n', 'ch2cl', 'the', 'initiation', 'step', 'depends', 'upon', 'formation', 'oxonium', 'ions', 'because', 'carbon', 'cation', 'intermediate', 'indicated', 'suggested', '4', 'propagation', 'probably', 'occurs', 'ether', 'exchange', 'results', 'nucleophilic', 'attack', 'monomer', 'oxonium', 'ion', 'boron', 'triﬂuoride', 'forms', 'complexes', 'oxygen-containing', 'compounds', 'like', 'water', 'alcohols', 'ethers', 'when', 'initiates', 'polymerization', 'epoxides', 'associate', 'simultaneously', 'several', 'different', 'moieties', 'these', 'monomeric', 'cyclic', 'ethers', 'well', 'open-chain', 'polymeric', 'ether', 'groups', 'hydroxy', 'groups', 'chain', 'ends', 'in', 'addition', 'also', 'associate', 'hydroxy', 'groups', 'water', 'the', 'following', 'illustration', 'shows', 'type', 'equilibrium', 'take', 'place', '2:', 'bf3', '(cid129)', '(cid129)', 'o', 'h', 'o', 'r', 'o', 'bf3', '(cid129)', '(cid129)', 'o', 'r', 'r', 'h', 'r', 'r', 'r', 'h', 'o', 'r', 'roh', '53', 'polymerization', 'oxiranes', '257', 'the', 'alcohols', 'open-chain', 'ethers', 'comparable', 'basicities', 'toward', 'coordinated', 'acid', 'roh', 'bf3', 'ethylene', 'oxide', 'hand', 'much', 'less', 'basic', 'open-chain', 'ethers', '6.', 'in', 'initiation', 'step', 'therefore', 'monomer', 'reacts', 'coordinated', 'acid', '1:', 'bf3', '(cid129)', '(cid129)', 'o', 'r', 'h', 'o', 'bf3', '(cid129)', '(cid129)', 'o', 'o', 'r', 'h', 'bf3', '(cid129)', '(cid129)', 'o', 'r', 'oh', 'during', 'propagation', 'three', 'different', 'reactions', 'occur', '2:', '1', 'r', 'h', 'o', 'bf3', '(cid129)', '(cid129)', 'o', 'r', 'h', 'o', '2', 'r', 'h', 'ho', 'r', 'h', 'ho', 'oh', 'oh', 'bf3', '(cid129)', '(cid129)', 'o', 'bf3', '(cid129)', '(cid129)', 'o', 'bf3', '(cid129)', '(cid129)', 'o', 'bf3', '(cid129)', '(cid129)', 'o', 'oh', 'ro', 'o', 'o', 'oh', 'r', 'h', 'o', 'o', 'o', 'this', 'reaction', 'also', 'accompanied', 'formation', 'dioxane', 'it', 'actually', 'step', 'o', 'ro', 'o', 'o', 'o', 'o', '3', 'r', 'h', 'o', 'ho', 'o', 'ro', 'bf3', 'depolymerization', 'oh', 'robf3', 'o', 'o', 'o', 'o', 'o', 'bf3', '(cid129)', '(cid129)', 'o', 'oh', 'o', 'o', 'robf3', 'o', 'o', 'o', 'the', 'ring-opening', 'reaction', 'nucleophilic', 'substitution', 'usually', 'takes', 'place', 'inversion', 'conﬁguration', 'carbon', 'atom', 'undergoes', 'nucleophilic', 'attack', '8', '9', '11.', 'this', 'illustrated', 'follows', 'h', 'o', '(s)', '(s)', 'h', '(s)', 'o', '(s)', 'h', 'h', 'h', '(r)', 'o', '(s)', '(s)', 'h', '258', '5', 'ring-opening', 'polymerizations', 'alkyl', 'substituents', 'ethylene', 'oxide', 'ring', 'enhance', 'process', 'cationic', 'polymerization', 'for', 'instance', 'ethylene', 'oxide', 'yields', 'low', 'molecular', 'weight', 'oils', 'strong', 'lewis', 'acids', 'tetramethy-', 'lethylene', 'oxide', 'hand', 'converted', 'readily', 'bf3', 'high', 'molecular', 'weight', 'polymers', 'insoluble', 'common', 'solvents', '10.', 'when', 'proton', 'donors', 'initiate', 'polymerizations', 'epoxides', 'low', 'molecular', 'weight', 'products', 'result', 'the', 'reaction', 'quite', 'straightforward', 'oxonium', 'ions', 'form', 'initiation', 'step', 'follow', 'o', 'ha', 'h', 'o', 'a', 'propagation', 'result', 'ring-opening', 'attack', 'monomer', 'h', 'o', 'a', 'o', 'ho', 'a', 'o', 'chain-growth', 'terminate', 'reaction', 'water', 'a', 'o', 'n', 'h', 'o', 'h2o', 'h', 'o', 'o', 'n', 'oh', 'ha', 'in', 'cationic', 'polymerizations', 'propylene', 'oxide', 'ring-opening', 'step', 'involves', 'direct', 'attack', 'oxonium', 'ion', 'carbon', 'bears', 'labile', 'bond', 'oxygen', 'h', 'o', 'a', 'o', 'o', 'h', 'a', 'o', 'cationic', 'ring-opening', 'polymerization', 'oxiranes', 'also', 'carried', 'photochemically', '(photo-', 'chemical', 'reactions', 'discussed', 'chap', '10)', 'yagci', 'coworkers', 'reported', 'polymerizations', 'cyclohexene', 'oxide', 'aid', 'highly', 'conjugated', 'thiophene', 'derivatives', '12.', 'the', 'reaction', 'illustrated', 'follows', 'o', 'n', 'visible', 'light', 'ph2i', 'pf6', 's', 'n', 'n', 'o', 'n', 'o', 'o', 'o', 'o', '53', 'polymerization', 'oxiranes', '259', 'the', 'cationic', 'polymerization', 'initiated', 'room', 'temperature', 'upon', 'irradiation', 'light', 'visible', 'region', 'ch2cl2', 'solutions', 'presence', 'diphenyliodonium', 'hexaﬂuorophosphate', 'thiophene', 'derivative', 'according', 'suggested', 'mechanism', 'ayodgan', 'et', 'al', '12', 'discuss', 'formation', 'exciplex', '(see', 'chap', '10)', 'absorption', 'light', 'subsequent', 'electron', 'transfer', 'excited', 'photosensitizer', 'iodonium', 'ion', 'yields', 'radical', 'cations', 'thiophene', 'derivatives', 'the', 'resulting', 'strong', 'bronsted', 'acid', 'derived', 'process', 'catalyzes', 'cationic', 'polymerization', '532', 'anionic', 'polymerization', 'anionic', 'polymerizations', 'ethylene', 'oxide', 'originally', 'observed', 'early', '1894', '13.', 'a', 'step-growth', 'mechanism', 'polymerizations', 'proposed', 'later', '14.', 'this', 'mechanism', 'well', 'established', '14.', 'the', 'conversion', 'increases', 'linearly', 'time', 'molecular', 'weight', 'also', 'increases', 'conver-', 'sion', 'reactions', 'bases', 'like', 'sodium', 'potassium', 'hydroxides', 'alkoxides', 'yield', 'low', 'molecular', 'weight', 'polymers', 'the', 'initiation', 'sn2', 'displacement', 'results', 'formation', 'alkoxide', 'ion', 'subsequent', 'propagation', 'may', 'occur', 'nucleophilic', 'displacement', 'involving', 'new', 'alkoxide', 'ion', 'or', 'formation', 'addition', 'r', 'o', 'o', 'r', 'o', 'r', 'o', 'o', 'o', 'o', 'termination', 'takes', 'place', 'transfer', 'hydroxy', 'group', 'another', 'molecule', 'it', 'also', 'take', 'place', 'terminal', 'hydroxyl', 'group', 'formed', 'polymer', 'starts', 'new', 'chain-growth', 'second', 'unit', 'o', 'o', 'o', 'r', 'o', 'o', 'ho', 'o', 'r', 'o', 'o', 'oh', 'o', 'o', 'the', 'chain', 'ends', 'remain', 'active', 'higher', 'molecular', 'weights', 'obtained', 'additions', 'monomer', 'the', 'reaction', 'often', 'complex', 'shown', 'many', 'polymerizations', 'catalyzed', 'alkali', 'hydroxides', 'alkoxides', 'carried', 'presence', 'alcohols', 'this', 'done', 'achieve', 'homogeneous', 'system', 'such', 'conditions', 'however', 'lead', 'exchange', 'reactions', 'r', 'o', 'o', 'n', 'r', 'o', 'o', 'na', 'o', 'n', 'roh', 'oh', 'ro', 'na', 'of', 'course', 'exchanges', 'shown', 'affect', 'kinetics', 'process', 'the', 'extent', 'reactions', 'subject', 'acid', 'strength', 'alcohols', 'present', 'including', 'terminal', 'hydroxyl', 'groups', 'formed', 'polymers', 'if', 'acidities', 'approximately', 'equal', 'exchange', 'reactions', 'take', 'place', 'throughout', 'whole', 'course', 'polymerization', '2.', '260', '5', 'ring-opening', 'polymerizations', 'one', 'reasons', 'relatively', 'low', 'molecular', 'weights', 'products', 'low', 'reactivity', 'epoxide', 'ring', 'toward', 'anionic', 'propagation', 'another', 'reason', 'tendency', 'chain', 'transfer', 'monomers', 'particularly', 'polymerizations', 'substituted', 'ring', 'structures', 'like', 'instance', 'propylene', 'oxide', 'o', 'na', 'o', 'oh', 'o', 'ch2', 'na', 'the', 'newly', 'formed', 'species', 'rearranges', 'rapidly', 'o', 'fast', 'ch2', 'na', 'ch2o', 'na', 'such', 'transfer', 'reactions', 'e-2', 'type', 'eliminations', 'this', 'shown', 'tetramethylethylene', 'oxide', 'undergoes', 'reaction', 'treated', 'catalytic', 'amounts', 'potassium', 't-butoxide', '5:', 'koc4h9', 'o', 'c4h9oh', 'o', 'k', 'oh', 'in', 'propylene', 'oxide', 'polymerization', 'therefore', 'e-2', 'type', 'elimination', 'reaction', 'competition', 'propagation', 'o', 'o', 'propagation', 'transfer', 'o', 'o', 'oh', 'o', 'there', 'allyl', 'propenyl', 'ether', 'end', 'groups', 'products', 'according', 'infra-red', 'spectra', '5.', 'this', 'suggests', 'addition', 'e-2', 'type', 'elimination', 'intramolecular', 'transfer', 'takes', 'place', 'allylic', 'hydrogen', 'raynaud', 'et', 'al', 'reported', 'carrying', 'ring-opening', 'polymerizations', 'ethylene', 'oxide', 'initiated', 'heterocyclic', 'carbene', '15.', 'the', 'reaction', 'yields', 'high', 'molecular', 'weight', 'polymers', 'it', 'illustrated', 'follows', 'n', 'n', 'r', 'r', 'o', 'dmso', '50', 'oc', 'n', 'r', 'n', 'r', 'on-1', 'o', 'n', 'r', 'n', 'r', 'o', 'o', 'n-1', 'r', 'propyl', 'tertiary', 'butyl', 'group', '53', 'polymerization', 'oxiranes', '261', 'fig', '51', 'metalloporphyrin', 'catalysts', 'x', '¼', 'methyl', 'methoxy', 'groups', 'x', 'n', 'n', 'n', 'al', 'n', 'x', 'n', 'n', 'zn', 'n', 'n', '533', 'polymerization', 'coordination', 'mechanism', 'the', 'coordination', 'catalysts', 'reactions', 'diverse', 'they', 'compounds', 'alkaline', 'earth', 'metals', 'like', 'calcium', 'amide', 'calcium', 'amide-alkoxide', 'they', 'also', 'ziegler–natta', 'type', 'catalysts', 'these', 'alkoxides', 'aluminum', 'magnesium', 'zinc', 'combined', 'ferric', 'chloride', 'others', 'reaction', 'products', 'dialkylzinc', 'water', 'alcohol', 'they', 'also', 'bimetallic', 'm-oxoalkoxides', '(ro)2alo2zn', 'other', 'catalysts', 'aluminum', 'zinc', 'metalloporphyrin', 'derivatives', '(see', 'fig', '51)', 'isotactic', 'polymers', '16.', 'living', 'polymerizations', 'metalloporphyrin', 'derivatives', 'difﬁcult', 'terminate', 'therefore', 'called', 'immortal', '18.', 'catalysts', 'like', '(c6h5)3–sbbr2–(c2h5)3n', 'combination', 'lewis', 'acids', 'also', 'yield', 'crystalline', 'poly(propylene', 'oxide)', 'others', 'like', 'pentavalent', 'organoantimony', 'halides', 'useful', 'polymerizations', 'ethylene', 'oxide', '19.', 'from', 'propylene', 'oxide', 'catalysts', 'yield', 'crystalline', 'polymerizations', 'epoxides', 'coordination', 'mechanism', 'result', 'high', 'molecular', 'weight', 'products', 'the', 'details', 'reaction', 'mechanism', 'fully', 'resolved', 'yet', 'commonly', 'believed', 'involve', 'coordination', 'monomers', 'electrophilic', 'centers', 'catalyst', 'this', 'followed', 'activation', 'attack', 'anion', '2.', 'such', 'mechanism', '1', 'illustrated', 'following', 'reactions', 'y', 'me', 'o', 'y', 'me', 'o', 'y', 'me', 'o', 'δ', 'δ', 'me', 'means', 'metal', 'y', 'o', 'me', 'o', 'y', 'δ', 'o', 'o', 'me', 'δ', 'y', 'o', 'me', 'o', 'etc', 'me', 'represents', 'metal', 'catalyst', 'ferric', 'chloride', 'polymerizes', 'propylene', 'oxide', 'monomer', 'asymmetric', 'carbon', 'atom', 'retention', 'asymmetry', 'backbone', '3.', 'the', 'products', 'polymerization', 'contain', 'either', 'optically', 'active', 'polymers', 'racemic', 'mixtures', 'depending', 'upon', 'monomers', 'used', 'when', 'pure', 'optical', 'isomer', 'monomer', 'used', 'products', 'crystalline', 'polymers', 'composed', 'optically', 'active', 'units', '262', '5', 'ring-opening', 'polymerizations', 'h', 'o', '(r)', 'ho', '(s)', 'h', 'o', 'h', 'o', 'h', 'section', 'polymer', 'the', 'polymers', 'fairly', 'high', 'molecular', 'weight', 'approximately', '100', 'times', 'greater', 'products', 'koh', 'initiations', 'propylene', 'oxide', 'initially', 'reacts', 'ferric', 'chloride', 'form', 'oligomer', 'chloropolyalkoxide', 'the', 'material', 'contains', 'approximately', 'four', 'ﬁve', 'propylene', 'oxide', 'repeat', 'units', 'this', 'forms', 'two', 'different', 'halogen', 'sites', 'it', 'illustrated', 'follows', 'cl', 'fe', 'o', 'o', 'x', 'o', 'o', 'cl', 'cl', 'the', 'compound', 'may', 'catalyst', 'one', 'closely', 'related', 'forming', 'stereoregular', 'polymers', 'water', 'appears', 'play', 'role', 'proportion', 'crystallinity', 'increases', 'addition', 'water', 'when', 'water', 'added', 'molar', 'ratio', '1810', 'water', 'iron', 'proportion', 'crystalline', 'amorphous', 'fraction', 'increases', '013', '086', 'price', 'osgen', '17', 'suggested', 'polymeri-', 'zation', 'proceeds', 'step-growth', 'mechanism', 'follows', 'possible', 'catalyst', 'surface', 'o', 'h', 'fe', 'o', 'hh', 'h', 'surface', 'fe', 'o', 'h', 'h', 'h', 'o', 'h', 'h', 'h', 'the', 'solid', 'surface', 'catalyst', 'causes', 'transition', 'state', 'compressed', 'steric', 'repulsions', 'incoming', 'monomer', 'ultimate', 'unit', 'minimized', 'incoming', 'monomer', 'molecule', 'forced', 'trans', 'methyl', 'group', 'previous', 'unit', 'such', 'conforma-', 'tional', 'approach', 'also', 'results', 'minimum', 'repulsion', 'incoming', 'monomer', 'bulky', 'growing', 'polymer', 'chain', '18–20.', 'also', 'ferric', 'alkoxides', 'associated', 'nonpolar', 'solvents', 'a', 'dimer', 'may', 'following', 'structure', 'r', 'o', 'r', 'o', 'fe', 'fe', 'o', 'r', 'o', 'r', 'r', 'o', 'o', 'r', 'by', 'comparison', 'intramolecular', 'chelation', 'expected', 'reduce', 'degree', 'association', 'catalyst', 'addition', 'water', 'results', 'increased', 'association', 'hydrolysis', 'ferric', 'alkoxide', 'this', 'may', 'explain', 'effect', 'promoting', 'stereoregularity', 'addition', 'water', '20.', 'the', 'ferric', 'alkoxide', 'catalyst', 'also', 'made', 'highly', 'stereospeciﬁc', 'partial', 'hydrolysis', 'still', 'remain', 'soluble', 'ether', 'polymerization', 'medium', '21.', 'this', 'led', 'suggestion', '22', 'catalyst', 'may', 'contain', 'active', 'fe–', 'o–fe', 'bonds', 'such', 'bonds', 'would', 'formed', 'condensation', 'partially', 'hydrolyzed', 'alkoxide', 'derivative', 'the', 'monomer', 'insertion', 'iron–oxygen', 'bonds', 'illustrated', 'follows', '53', 'polymerization', 'oxiranes', '263', 'o', 'o', 'o', 'fe', 'o', 'fe', 'r', 'fe', 'o', 'o', 'fe', 'r', 'o', 'o', 'fe', 'o', 'fe', 'r', 'o', 'fe', 'fe', 'o', 'o', 'r', 'the', 'forces', 'interaction', 'iron', 'atoms', 'various', 'oxygen', 'atoms', 'shown', 'assure', 'cis', 'opening', 'epoxide', 'ring', 'the', 'mechanism', 'reaction', 'ferric', 'alkoxide', 'sn2', 'type', 'there', 'therefore', 'increased', 'restriction', 'conformation', 'monomer', 'unit', 'approaches', 'reaction', 'center', '22.', 'many', 'coordinated', 'anionic', 'catalysts', 'metal', 'alkoxides', 'metal', 'alkyls', 'also', 'much', 'reactive', 'presence', 'water', 'alcohols', 'the', 'function', 'co', 'reactants', 'modify', 'catalyst', 'for', 'instance', 'diethylzinc', 'combined', 'water', 'ratio', '11', 'yields', 'reactive', 'species', 'the', 'exact', 'nature', 'catalyst', 'still', 'fully', 'established', 'however', 'reaction', 'product', 'pictured', 'follows', '23', '24:', 'c2h5', '(cid4)', 'zn', '(cid4)', 'o', '(cid4)', 'zn', '(cid4)', 'c2h5', 'several', 'reaction', 'mechanisms', 'proposed', 'one', 'suggested', 'pathway', 'propylene', 'oxide', 'poly-', 'merization', 'pictures', 'initial', 'coordination', 'monomer', 'cationically', 'active', 'center', '25:', 'c2h5', 'o', 'zn', 'zn', 'c2h5', 'o', 'c2h5', 'δ', 'o', 'c2h5', 'zn', 'zn', 'δ', 'o', 'the', 'propagation', 'preceded', 'intramolecular', 'rearrangement', 'δ', 'o', 'c2h5', 'c2h5', 'zn', 'zn', 'δ', 'o', 'c2h5', 'c2h5', 'zn', 'zn', 'o', 'δ', 'o', 'δ', 'c2h5', 'o', 'zn', 'zn', 'o', 'c2h5', '264', '5', 'ring-opening', 'polymerizations', 'another', 'mechanism', 'derived', 'structure', 'diethylzinc–water', 'catalyst', '25', 'visualized', 'dimer', 'c2h5', 'zn', 'zn', 'c2h5', 'h', 'o', 'o', 'h', 'a', 'similar', 'structure', 'pictured', 'shown', 'diethylzinc–alcohol', 'the', 'asymmetric', 'induction', 'suggested', 'takes', 'place', 'coordination', 'monomer', 'catalyst', 'site', 'this', 'result', 'indirect', 'regulation', 'results', 'form', 'interactions', 'monomer', 'penultimate', 'unit', '25.', 'in', 'yet', 'another', 'mechanism', 'initial', 'coordination', 'subsequent', 'propagation', 'steps', 'pictured', 'follows', '26', 'r', 'zn', 'zn', 'r', 'o', 'o', 'o', 'r', 'o', 'o', 'r', 'o', 'r', 'zn', 'zn', 'r', 'o', 'o', 'r', 'o', 'o', 'r', 'r', 'o', 'r', 'zn', 'zn', 'r', 'o', 'o', 'o', 'o', 'r', 'while', 'detailed', 'structures', 'catalyst', 'sites', 'still', 'unknown', 'established', 'stereoselectivity', 'come', 'chirality', 'growing', 'chain', 'end', 'rather', 'built', 'catalyst', 'site', '27', '28.', 'normal', 'preparations', 'catalysts', 'give', 'equal', 'numbers', '(r)', '(s)', 'chiral', 'catalyst', 'sites', 'these', 'coordinate', 'selectively', '(r)', '(s)', 'monomers', 'respectively', 'process', 'catalytic-site', 'control', '23.', '534', 'steric', 'control', 'polymerizations', 'oxiranes', 'cationic', 'polymerizations', 'oxiranes', 'much', 'less', 'isospeciﬁc', 'regiospeciﬁc', 'anionic', 'polymerizations', 'in', 'anionic', 'coordinated', 'anionic', 'polymerizations', 'chiral', 'epoxides', 'like', 'propylene', 'oxide', 'yield', 'stereoregular', 'polymers', 'both', 'pure', 'enantiomers', 'yield', 'isotactic', 'polymers', 'reaction', 'proceeds', 'regiospeciﬁc', 'manner', 'bond', 'cleavage', 'taking', 'place', 'primary', 'carbon', 'in', 'polymerizations', 'oxiranes', 'cationic', 'anionic', 'coordinated', 'anionic', 'mechanisms', 'ring-opening', 'generally', 'accompanied', 'inversion', 'conﬁguration', 'carbon', 'cleavage', 'takes', 'place', 'a', 'linear', 'transition', 'state', 'mechanism', 'involving', 'dissociated', 'nucleophilic', 'species', 'proposed', '15.', 'yet', 'known', 'instances', 'ring-opening', 'reactions', 'epoxies', 'stereochemically', 'retentive', 'for', 'instance', 'ring', 'opening', '23-epoxybutane', 'alcl3', 'results', '53', 'polymerization', 'oxiranes', '265', 'formation', '3-chloro-2-butanol', 'cis', 'trans', 'epoxides', 'converted', 'erythro', 'threo-chlorohydrins', 'inoue', 'coworkers', '19', 'found', 'however', 'polymerizations', 'cis', 'trans', '23-epoxybutanes', 'take', 'place', 'inversion', 'conﬁguration', 'aluminum', '5101520-', 'tetraphenylporphine', 'zinc', '5101520-tetraphenyl-21-methylporphine', 'catalysts', 'used', 'to', 'explain', 'inversion', 'inoue', 'coworkers', 'proposed', 'linear', 'transition', 'state', 'mechanism', 'involves', 'simultaneous', 'participation', '19.', 'one', 'porphyrin', 'molecule', 'two', 'porphyrin', 'molecules', 'accommodates', 'coordinative', 'activation', 'epoxide', 'one', 'serves', 'nucleophile', 'attack', 'coordinated', 'epoxide', 'back', 'side', 'potassium', 'hydroxide', 'alkoxide', 'polymerizes', 'racemic', 'propylene', 'oxide', 'better', '95', 'regioselectivity', 'cleavage', 'bond', 'oxygen', 'carbon', 'substituted', 'two', 'hydrogens', 'the', 'product', 'however', 'atactic', 'both', '(r)', '(s)', 'propylene', 'oxides', 'react', 'rate', 'this', 'shows', 'initiator', 'unable', 'distinguish', 'two', 'enantiomers', 'propylene', 'oxide', 'when', 't-butyl', 'ethylene', 'oxide', 'polymerized', 'koh', 'yields', 'crystalline', 'product', 'this', 'product', 'different', 'melting', 'point', 'x-ray', 'diffraction', 'pattern', 'solution-nmr', 'spectra', 'typical', 'isotactic', 'polymers', 'it', 'contains', 'alternating', 'isotactic', 'syndiotactic', 'sequences', '31.', 'it', 'suggested', '34', 'may', 'result', 'conﬁguration', 'incoming', 'monomer', 'opposite', ')', 'last', 'next', 'last', 'oxygen', 'penultimate', 'unit', 'chelation', 'paired', 'cation', '(k', 'visualized', 'geometry', 'chelate', 'dictated', 'requirement', 'penultimate', 't-butyl', 'group', 'equatorial', 'conformation', 'this', 'makes', 'reasonable', 'postulate', 'necessary', 'preference', 'incoming', 'monomer', 'opposite', 'penultimate', 'unit', '31:', 'l', 'h', 'r', 'o', 'h', 'o', 'r', 'k', 'h', 'o', 'r', 'o', 'h', 'o', 'r', 'k', 'h', 'o', 'r', 'o', 'o', 'h', 'r', 'when', 'phenyl', 'glycidyl', 'ethers', 'polymerized', 'conditions', 'steric', 'arrangement', 'isotactic', 'rather', 'isotactic–syndiotactic', '31.', 'price', 'explained', 'basis', 'oxygen', 'c6h6–o–ch2', 'seeking', 'coordinate', 'potassium', 'ions', 'transition', 'state', '31.', 'in', 'case', 't-butylethylene', 'oxide', 'hand', 'tertiary', 'butyl', 'group', 'tends', 'far', 'possible', 'away', 'potassium', 'ion', '34.', 'this', 'supported', 'observation', 'p-methoxy', 'p-methyl', 'groups', 'phenyl', 'glycidyl', 'ether', 'increase', 'crystalline', 'portion', 'polymer', 'p-chloro', 'substituent', 'decreases', '31.', 'most', 'stereoselective', 'coordination', 'catalysts', 'polymerize', 'propylene', 'oxide', 'yield', 'polymers', 'contain', 'high', 'ratios', 'isotactic', 'syndiotactic', 'sequences', 'large', 'portions', 'amorphous', 'materials', 'however', 'also', 'present', 'materials', 'these', 'amorphous', 'portions', 'contain', 'head', 'head', 'units', 'imperfections', 'structures', '29', '30.', 'for', 'every', 'head', 'head', 'placement', 'one', '(r)', 'monomer', 'converted', '(s)', 'unit', 'polymer', '23.', 'this', 'shows', 'coordination', 'sites', 'abnormal', 'ring', 'openings', 'occur', 'secondary', 'carbon', 'inversion', 'conﬁguration', 'results', 'head', 'head', 'placements', '23', '31.', 'also', 'erythro', 'threo', 'isomers', 'units', 'present', 'the', 'isotactic', 'portion', 'consists', 'almost', 'exclusively', 'erythro', 'isomer', 'amorphous', 'fraction', 'contains', '40–45', 'erythro', '55–60', 'threo', '31', 'all', 'information', 'indirect', 'evidence', 'typical', 'catalyst', '(c2h5)2zn–h2o', 'contains', 'isotactic', 'amorphous', 'sites', 'the', 'isotactic', 'sites', 'selective', 'coordinate', 'either', '(r)', '(s)', 'monomers', 'the', 'amorphous', 'sites', 'hand', 'coordinate', 'equally', 'well', '(r)', '(s)', 'monomers', 'in', 'addition', 'little', 'preference', 'attack', 'either', 'primary', 'secondary', 'carbons', 'ring-opening', 'reactions', '23.', '266', '5', 'ring-opening', 'polymerizations', 'according', 'tsuruta', 'mechanism', '36', 'ﬁrst', 'step', 'propylene', 'oxide', 'polymerization', 'catalysts', 'like', 'zinc', 'alcoholates', 'coordination', 'ether', 'oxygen', 'onto', 'zinc', 'atom', 'the', 'second', 'step', 'nucleophilic', 'attack', 'oxirane', 'ring', 'alkoxy', 'ion', 'almost', 'bond', 'cleavage', 'takes', 'place', 'ch2–o', 'bond', 'this', 'results', 'retention', 'steric', 'conﬁguration', 'carbon', 'atom', 'c–h', 'group', 'the', 'next', 'oxirane', 'molecule', 'repeats', 'process', 'coordinates', 'zinc', 'atom', 'undergoes', 'ring-opening', 'reaction', 'form', 'dimer', 'repetition', 'process', 'many', 'times', 'yields', 'high', 'molecular', 'weight', 'polymer', '36:', 'zn(or)2', 'o', '(s)', '(r)', '(s)', '(s)', 'o', 'o', '(s)', '(r)', 'o', 'o', 'o', 'o', '(r)', '(r)', 'the', 'catalyst', 'also', 'znr2–ch3–oh', 'special', 'catalyst', 'complexes', 'like', 'zn(och3)2(cid5)(c2h5och3)6,', 'form', 'carefully', 'control', 'reaction', 'conditions', 'adding', '16', 'moles', 'methyl', 'alcohol', '14', 'moles', 'diethylzinc', 'heptane', 'argon', 'atmosphere', 'x-ray', 'analysis', 'shows', 'two', 'different', 'structures', '36.', 'one', 'centrosym-', 'metric', 'complex', 'two', 'enantiomorphic', 'distorted', 'cubes', 'share', 'corner', 'zn', 'atom', 'the', 'two', 'would', 'equivalent', 'distorted', 'another', 'structure', 'also', 'centrosymmetric', 'consists', 'two', 'enantiomorphic', 'distorted', 'structures', 'resemble', '“chairs', 'without', 'legs”', 'surfaces', 'share', 'common', 'seat', 'both', 'types', 'complexes', 'active', 'initiators', 'polymerization', 'propylene', 'oxide', 'each', 'two', 'enantiomorphic', 'sites', 'polymerization', 'based', 'knowledge', 'nmr', 'spectra', 'gpc', 'curves', 'tsuruta', 'suggested', 'following', 'mechanism', 'monomer', 'coordinating', 'catalyst', '36', '(see', 'fig', '52)', 'the', 'bonds', 'central', 'zinc', 'atom', 'loosened', 'coordination', 'takes', 'place', 'methyl-oxirane', 'molecule', 'central', 'atom', 'cleavage', 'o–ch2', 'bond', 'oxirane', 'takes', 'place', 'concerted', 'mechanism', 'if', 'bond', 'loosening', 'takes', 'place', 'cube', 'nucleophilic', 'attack', 'takes', 'place', 'one', 'methoxy', 'groups', 'cube', 'chirality', 'around', 'central', 'zinc', 'favor', 'l', 'monomer', 'd', 'monomer', 'this', 'origin', 'l', 'catalyst', 'site', 'if', 'bond', 'loosening', 'takes', 'place', 'l', 'cube', 'catalyst', 'site', 'chirality', 'because', 'probability', 'bond', 'loosening', 'cube', 'exactly', 'l', 'cube', 'equal', 'number', 'l', 'sites', 'expected', 'form', 'these', 'two', 'cubes', 'become', 'source', 'l', 'chiral', 'nature', '35.', '54', 'polymerization', 'oxetanes', 'oxetanes', '(or', 'oxacyclobutanes)', 'preferably', 'polymerized', 'solution', 'maintain', 'temperature', 'stirring', 'control', 'it', 'necessary', 'purify', 'monomer', 'solvent', 'impurities', 'interfere', 'attainment', 'high', 'molecular', 'weight', '54', 'polymerization', 'oxetanes', '267', 'cube', 'zn', 'o', 'ch3', 'o', 'ch2', 'ch', 'ch3', 'ch3', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'ch3', 'zn', 'o', 'zn', 'o', 'o', 'zn', 'l', 'cube', 'catalyst', 'complex', 'ch3', 'cube', 'ch3', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'o', 'ch3', 'o', 'zn', 'l', 'cube', 'transition', 'state', 'ch3', 'fig', '52', 'tsuruta', 'mechanism', '541', 'the', 'initiation', 'reaction', 'ch3', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'o', 'zn', 'l', 'cube', 'catalyst', 'complex', 'cube', 'zn', 'o', 'ch3', 'ch3', 'o', 'ch3', 'cube', 'ch3', 'zn', 'o', 'ch3', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'o', 'zn', 'l', 'cube', 'o', 'ch3', 'product', 'ch3', 'o', 'theoretically', 'lewis', 'acid', 'catalyze', 'oxetane', 'polymerizations', 'however', 'acids', 'differ', 'considerably', 'effectiveness', 'boron', 'triﬂuoride', 'etherates', 'widely', 'reported', 'catalysts', 'moisture', 'must', 'excluded', 'tends', 'detrimental', 'reaction', '35.', 'chlorinated', 'hydrocarbon', 'solvents', 'like', 'methylene', 'chloride', 'chloroform', 'carbon', 'tetrachloride', 'common', 'choices', 'the', 'reactions', 'usually', 'conducted', 'low', 'temperatures', 'indications', 'lower', 'reaction', 'temperature', 'higher', 'molecular', 'weight', 'product', 'it', 'reported', 'oxetane', 'polymerizations', 'carried', 'boron', 'triﬂuoride', 'catalyst', 'methylene', 'chloride', 'temperatures', '0', '(cid4)278(cid6)c', 'cocatalyst', 'required', '32.', 'the', 'product', 'however', 'mixture', 'linear', 'polymer', 'small', 'amount', 'cyclic', 'tetramer', 'this', 'agreement', 'earlier', 'observation', 'polymerizations', 'oxetane', 'complicated', 'formations', 'small', 'amounts', 'cyclic', 'tetramers', '33.', 'other', 'catalysts', 'like', 'protonic', 'acids', 'capable', 'generating', 'oxonium', 'ions', 'also', 'polymerize', 'oxetane', 'such', 'acids', 'sulfuric', 'triﬂuoroacetic', 'ﬂuorosulfuric', 'the', 'initiation', 'reaction', 'illustrated', 'follows', 'h', 'a', 'o', 'oh', 'a', '268', '5', 'ring-opening', 'polymerizations', 'the', 'adduct', 'reacts', 'cyclic', 'ether', 'o', 'ho', 'oh', 'a', 'o', 'a', 'when', 'complexes', 'lewis', 'acids', 'active', 'hydrogen', 'compounds', 'initiate', 'polymerizations', 'complexes', 'acts', 'protonic', 'acids', 'on', 'hand', 'etherates', 'initiate', 'forming', 'oxonium', 'ions', 'may', 'involve', 'alkyl', 'exchange', 'reactions', 'monomer', '(c2h5)3', 'o(bf4)', 'o', 'c2h5', 'o', 'bf4', '(c2h5)2o', '542', 'the', 'propagation', 'reaction', 'the', 'propagation', 'takes', 'place', 'via', 'tertiary', 'oxonium', 'ions', '37', '38:', 'a', 'o', 'o', 'o', 'o', 'o', 'a', 'o', 'a', 'cyclic', 'oligomer', 'forms', 'instances', 'addition', 'polymer', '40.', 'for', 'instance', 'polymerizations', 'bf3', 'methylene', 'chloride', 'low', 'temperatures', 'cyclic', 'tetramer', 'forms', 'probably', 'backbiting', 'process', '40.', 'the', 'oxonium', 'exchange', 'reactions', 'may', 'occur', 'polymer', 'ether', 'linkages', 'well', 'cyclic', 'tetramers', 'form', 'shown', 'the', 'concentrations', 'oxonium', 'ions', 'ether', 'group', 'polymer', 'cyclic', 'tetramers', 'however', 'small', '42.', 'polymerizations', 'pf5', 'hand', '(c2h5)3opf6', 'either', 'bulk', 'methylene', 'chloride', 'solutions', 'yield', 'signiﬁcant', 'amounts', 'cyclic', 'oligomers', '43.', 'the', 'activation', 'energy', 'polymerizations', 'oxetane', 'monomers', 'higher', 'tetrahydro-', 'furan', '(see', 'next', 'section)', 'this', 'indicates', 'orientation', 'cyclic', 'oxonium', 'ion', 'monomer', 'looser', 'sn2', 'transition', 'state', '42:', 'h', 'h', 'h', 'h', 'h', 'h', 'δ', 'o', 'o', 'h', 'h', 'h', 'h', 'h', 'h', 'in', 'principle', 'stereospeciﬁcity', 'possible', 'substitute', 'polyoxycyclobutanes', '2-methyl', '3-methyl', 'others', 'the', '2-methyl', 'derivative', 'however', 'yields', 'amorphous', 'polymers', 'this', 'due', 'monomer’s', 'unsymmetrical', 'structure', '33', 'nmr', 'studies', 'microstructure', 'polymers', '33-dimethyloxetane', '44', '2-methyloxetane', '43', 'led', 'conclusions', 'manner', 'ring', 'opening', 'the', 'predominant', 'head', 'tail', 'structures', 'may', 'result', 'attacks', 'either', 'methylene', 'methine', 'carbons', 'next', 'oxonium', 'ions', 'propagating', 'species', '55', 'polymerization', 'tetrahydrofurans', '269', 'oxetane', 'compounds', 'also', 'polymerize', 'aid', 'aluminum', 'trialkyl–water', 'acetylacetone', 'catalysts', '45', '46.', 'the', 'reactions', 'take', 'place', '65(cid6)c', 'heptane', 'yield', 'high', 'molecular', 'weight', 'polymers', 'these', 'polymerizations', 'however', 'ten', 'times', 'slower', 'similar', 'ones', 'carrier', 'propylene', 'oxide', 'using', 'catalyst', 'the', 'reaction', 'conditions', 'high', 'molecular', 'weights', 'products', 'led', 'assumption', 'coordinated', 'mechanisms', 'polymerizations', 'take', 'place', '46.', 'also', 'reported', '218', 'quaternary', 'onium', 'salts', 'coupled', 'wit', 'bulky', 'organoaluminum', 'diphenolates', 'initiate', 'controlled', '(living)', 'coordinate', 'anion', 'polymerizations', 'oxetane', 'give', 'narrow', 'molecular', 'weight', 'distribution', 'polyethers', 'the', 'catalyst', 'system', 'consists', 'onium', 'salts', 'combined', 'sterically', 'hindered', 'quaternal', 'methylaluminum', 'diphenolates', '47', 'ammonium', 'phosphonium', 'halides', 'r2', 'r2', 'r1', 'al', 'r3', 'r4', 'r4', 'r3', 'crivello', 'reported', 'frontal', 'photopolymerization', 'oxetane', '48.', 'he', 'proposed', 'mechanism', 'polymerization', 'shown', 'following', 'scheme', 'ar2i', 'mtxn', 'hν', 'ari', 'ari', 'mtxn', 'ar', 'ar', 'mtxn', 'hmtxn', 'hmtxn', 'o', 'h', 'o', 'mtxn', 'h', 'o', 'mtxn', 'o', 'ho', 'o', 'mtxn', 'h', 'o', 'mtxn', 'o', 'ho', 'o', 'o', 'mtxn', 'crivello', 'reported', 'diaryliodonium', 'salts', 'used', 'cationic', 'photoinitiators', 'study', 'photo', 'activation', 'carried', 'uv', 'irradiation', 'prior', 'thermal', 'initiation', 'initiation', 'carried', 'using', 'electrically', 'heated', 'wire', 'immersed', 'monomer', 'the', 'velocity', 'resulting', 'propagating', 'front', 'quite', 'high', 'temperature', 'front', 'reaching', '110(cid6)c', '55', 'polymerization', 'tetrahydrofurans', 'lewis', 'acids', 'carbon', 'cations', 'salts', 'oxonium', 'ions', 'strong', 'protonic', 'acids', 'initiate', 'polymerizations', 'tetrahydrofuran', 'the', 'reactions', 'conducted', 'solution', 'without', 'solvent', 'it', 'originally', 'polymerized', '49', '50', 'trialkyloxomum', 'salt', 'r3o•bf4', '270', '5', 'ring-opening', 'polymerizations', '551', 'the', 'initiation', 'reaction', 'the', 'initiations', 'result', 'coordination', 'cation', 'catalysts', 'oxygen', 'monomers', 'form', 'oxonium', 'ions', '48', '49.', 'this', 'weakens', 'oxygen–carbon', 'bonds', 'leads', 'ring', 'openings', 'reactions', 'second', 'molecule', 'monomer', 'new', 'oxonium', 'ions', 'generated', 'process', 'r3o', 'bf4', 'o', 'or', 'r2o', 'or', 'o', 'bf4', 'bf4', 'o', 'r', 'o', 'bf4', 'some', 'active', 'oxonium', 'salts', '48–50:', '(cid4)', '(c2h5o)3o', 'alcl4', '(cid4)', '(c2h5o)3o+', 'fecl4', '(c2h5o)3o+', 'bf4(cid4)', '(c2h5o)3o+', 'sbcl6', '(cid4)', 'examples', 'carbon', 'cations', 'initiate', 'polymerizations', 'tetrahydrofuran', 'well', 'cyclic', 'ethers', 'hclo4', 'o', 'agclo4', 'sncl4', 'o', 'o', 'cl', 'o', 'cl', 'o', '(clo4)', 'oh', 'o', '(clo4)', 'agcl', 'o', 'sncl5', 'cl', 'fecl3', 'ch2', 'fecl4', 'r', 'ro', 's', 'o', 'o', 'bf3', 'o', 's', 'o', 'r', 'ro', 'bf3', 'ccl', 'sbcl5', '3', 'c', '3', 'sbcl6', 'the', 'initiation', 'mechanisms', 'however', 'many', 'carbon', 'cations', 'instance', 'triphenylmethyl', 'cations', 'straightforward', 'initially', 'hydride', 'ions', 'abstracted', 'monomers', 'form', 'triphenylmethanes', '51–53.', 'simultaneously', 'acids', 'released', 'counterions', 'the', 'acids', '55', 'polymerization', 'tetrahydrofurans', '271', 'become', 'stabilized', 'complexing', 'monomers', 'after', 'complexes', 'react', 'slowly', 'additional', 'monomers', 'form', 'propagating', 'oxonium', 'ions', 'this', 'makes', 'acids', 'real', 'initiators:', 'o', 'o', 'oh', 'hx', '3', 'x', 'c', '3', 'o', 'hx', '2', 'o', 'h', 'o', 'x', 'o', 'h', 'o', 'x', 'ho', 'o', 'x', 'other', 'initiators', 'tetrahydrofuran', 'polymerizations', 'also', 'include', 'lewis', 'acids', 'combinations', '“promoters”', 'these', 'complexes', 'lewis', 'acids', 'like', 'bf3', 'sncl1', 'c2h5alcl2', 'epirane', 'compounds', 'like', 'epichlorohydrin', '42.', 'the', 'small', 'ring', 'compounds', 'mote', 'reactive', 'toward', 'many', 'lewis', 'acids', 'protonic', 'acids', 'tetrahydrofuran', 'act', 'promoters', 'initiation', 'reactions', 'the', 'initiations', 'presence', 'small', 'quantities', 'oxirane', 'compounds', 'instance', 'illustrated', 'follows', 'cat', 'o', 'o', 'ch2cl', 'cat', 'o', 'o', 'ch2cl', 'cat-x', 'ch2cl', 'oh', 'o', 'ho', 'ch2cl', 'o', 'cat-x', 'strong', 'bronsted', 'acids', 'form', 'diaryliodonium', 'salts', 'like', 'bf4', '(cid4)', 'reduced', 'compounds', 'like', 'ascorbic', 'acid', 'presence', 'copper', 'salts', 'acids', 'also', 'initiate', 'polymerizations', 'tetrahydrofuran', 'cyclohexene', 's-trioxane', '54.', '(cid4)', 'sbf6', '(cid4)', 'asf6', '(cid4)', 'pf6', '552', 'the', 'propagation', 'reaction', 'the', 'propagation', 'process', 'succession', 'nucleophilic', 'attacks', 'fn', 'electrons', 'oxygens', 'monomers', 'upon', 'a-carbons', 'heteroatoms', 'ultimate', 'polymerizing', 'species', '1:', 'o', 'o', 'o', 'o', '272', '5', 'ring-opening', 'polymerizations', 'the', 'products', 'reactions', 'linear', 'actually', 'common', 'polymerizations', 'many', 'heterocyclics', 'the', 'propagation', 'reactions', 'proceed', 'stepwise', 'additions', 'monomer', 'sn2', 'mecha-', 'nism', 'growing', 'ends', 'propagating', 'chains', 'the', 'nmr', 'spectra', 'growing', 'chains', 'shows', 'presence', 'oxonium', 'ions', '55', '56:', 'ch2', 'o', 'the', 'oxonium', 'ions', 'could', 'principle', 'equilibrium', 'minute', 'quantities', 'carbon', 'cations', '–', 'active', 'all', 'evidence', 'ch2', 'tetrahydrofuran', 'polymerizations', 'presence', 'carbon', 'cations', 'negligible', 'propagation', 'process', '57.', 'also', 'rate', 'constant', 'propagation', 'free', 'macroions', 'counterions', 'equal', 'within', 'experimental', 'error', 'rate', 'constant', 'macroions–counterion', 'pairs', 'this', 'appear', 'depend', 'upon', 'stricture', 'anion', 'studied', 'the', 'information', 'however', 'obtained', 'large', 'anions', 'with', 'smaller', 'anions', 'differences', 'rates', 'propagation', 'macrocations', 'macroion–', 'counterion', 'pairs', 'ruled', 'date', 'however', 'shows', 'an', 'sn2', 'attack', 'requires', 'reaction', 'occur', 'oxygen', 'carbon', 'bond', 'in', 'attack', 'steric', 'requirements', 'less', 'restricted', 'anionic', 'polymerization', 'in', 'addition', 'positive', 'negative', 'charges', 'macroion-pairs', 'contain', 'oxonium', 'ions', 'dispersed', 'anions', 'large', 'this', 'means', 'electrostatic', 'interactions', 'less', 'important', 'cationic', 'polymerizations', 'type', 'anionic', 'ones', 'when', 'polymerization', 'tetrahydrofuran', 'carried', 'aid', 'cf3so3h', 'covalent', 'ionic', 'species', 'present', 'they', 'detected', 'propagation', 'means', 'nmr', 'spectros-', 'copy', 'both', 'species', 'exist', 'mobile', 'equilibrium', 'solvent', 'polarity', 'apparently', 'inﬂuences', 'position', 'equilibria', 'in', 'nitromethane', '95', 'growing', 'chains', 'macroions', 'in', 'carbon', 'tetrachloride', '95', 'macroesters', 'in', 'methylene', 'chloride', 'species', 'present', 'reaction', 'mixture', 'approximately', 'equal', 'amounts', '58–62.', 'the', 'propagation', 'rate', 'macroions', 'however', '102', 'times', 'faster', 'macroesters', 'as', 'result', 'chain', 'growth', 'even', 'carbon', 'tetrachloride', 'still', 'way', 'ions', 'the', 'macroesters', 'therefore', 'considered', 'dormant', 'species', '59,', 'suggest', 'even', 'cases', 'temporary', 'termination', '59.', 'the', 'much', 'higher', 'reactivity', 'macroions', 'attributed', 'contribution', 'partially', 'released', 'strain', 'ionic', 'species', '49.', 'macroions', 'macroesters', 'illustrated', 'follows', 'cf6', 'so2', 'o', 'δ', 'δ', 'c', 'h', 'covalent', 'o', 'h', 'h', 'o', 'h', 'o', 'ionic', '553', 'the', 'termination', 'reaction', 'the', 'termination', 'reactions', 'tetrahydrofuran', 'polymerizations', 'depend', 'upon', 'choice', 'counterion', 'particularly', 'reaction', 'conducted', 'room', 'temperature', '60.', 'in', 'many', 'reactions', 'chains', 'continue', 'grow', 'without', 'considerable', 'termination', 'transfer', '63', '64.', 'some', 'refer', '(cid4)', 'process', '“living”', 'polymerization', 'thus', 'polymerizations', 'tetrahydmfuran', '65', 'pf6', '57', 'ring-opening', 'polymerizations', 'cyclic', 'acetals', '273', '(cid4)', 'counterions', 'molecular', 'weights', 'products', 'calculated', 'directly', 'ratios', 'sbf6', 'initiators', 'monomers', 'the', 'molecular', 'weight', 'distributions', 'polymers', '(cid4)', 'however', 'start', 'narrow', 'broaden', 'polymerization', 'reactions', 'pf6', 'this', 'believed', 'due', 'transfer', 'reactions', 'ether', 'oxygen', 'it', 'supported', 'evidence', '(cid4)', 'initiation', 'termination', 'transfer', 'reactions', 'take', 'place', '65.', 'in', 'addition', 'polymerizations', 'sbf6', 'tetrahydrofuran', 'like', 'epoxides', 'accompanied', 'formations', 'macrocy-', 'clic', 'oligomers', 'this', 'often', 'case', '66', '67', 'strong', 'acids', 'used', 'initiators', 'the', 'proposed', 'mechanism', 'involves', 'backbiting', 'chain', 'coupling', 'results', 'linear', 'polymers', 'hydroxyl', 'groups', 'oxonium', 'ions', 'opposite', 'chain', 'ends', 'well', 'macrocycles', '(cid4)', 'sbf6', 'the', 'absence', 'linear', 'oligomers', 'due', 'rapid', 'reactions', 'hydroxyl', 'oxonium', 'ion', 'end', 'groups', 'this', 'mechanism', 'quite', 'general', '67', '68', 'ring-opening', 'polymerizations', 'cyclic', 'ethers', 'initiated', 'string', 'protonic', 'acids', 'substituted', 'tetrahydrofurans', 'generally', 'resist', 'polymerizations', '56', 'polymerization', 'oxepanes', 'oxepanes', 'polymerized', 'various', 'cationic', 'initiators', 'like', '(c2h5)3cbf4', '(cid4)', 'bf3-epichlorohydrin', 'sbcl6-epichlorohydrin', '42.the', 'reactions', 'take', 'place', 'chlorinated', 'solvents', 'like', 'methylene', 'chloride', 'the', 'rates', 'reactions', 'however', 'quite', 'slow', '42.', 'in', 'addition', 'polymerizations', 'reversible', 'the', 'rates', 'propagation', 'three', 'cyclic', 'ether', 'oxetane', 'tetrahydrofuran', 'oxepane', '0(cid6)c', 'fall', 'following', 'order', '42:', '(cid4)', '(c2h5)3csbci6', 'oxetane', 'tetrahydrofuran', 'oxepane', 'at', 'temperature', 'oxetane', '35', 'times', 'reactive', 'tetrahydrofuran', 'turn', '270', 'times', 'reactive', 'oxepane', 'this', 'cannot', 'explained', 'basis', 'ring', 'strain', 'explained', 'considerations', 'basic', 'strength', 'saegusa', 'suggested', '42', 'differences', 'propagation', 'rates', 'governed', 'nucleophilic', 'reactivities', 'monomers', 'they', 'also', 'affected', 'reactivities', 'cyclic', 'oxonium', 'ions', 'propagating', 'species', 'also', 'steric', 'hindrances', 'transition', 'states', 'propagation', 'higher', 'activation', 'energy', 'oxepane', 'explained', 'increased', 'stability', 'seven-membered', 'oxonium', 'ion', 'the', 'oxepane', 'molecule', 'puckered', 'structure', 'strain', 'comes', 'trivalent', 'oxygen', 'relieved', 'small', 'deformations', 'angles', 'bonds', '42.', '57', 'ring-opening', 'polymerizations', 'cyclic', 'acetals', 'the', 'cationic', 'polymerizations', 'cyclic', 'acetals', 'different', 'polymerizations', 'rest', 'cyclic', 'ethers', 'the', 'differences', 'arise', 'great', 'nucleophilicity', 'cyclic', 'ethers', 'compared', 'acetals', 'in', 'addition', 'cyclic', 'ether', 'monomers', 'like', 'epirane', 'tetrahydrofuran', 'oxepane', 'stronger', 'bases', 'corresponding', 'polymers', 'the', 'opposite', 'tree', 'acetals', 'as', 'result', 'acetal', 'polymerizations', 'active', 'species', 'like', '13-dioxolane', 'may', 'exist', 'equilibrium', 'macroalkoxy', 'carbon', 'cations', 'tertiary', 'oxonium', 'ions', '69.', 'by', 'comparison', 'active', 'propagating', 'species', 'polymerizations', 'cyclic', 'ethers', 'like', 'tetrahydrofuran', 'tertiary', 'oxonium', 'ions', 'the', 'properties', 'equilibrium', 'active', 'species', 'acetal', 'polymerizations', 'depend', 'much', 'upon', 'polymerization', 'conditions', 'upon', 'structures', 'individual', 'monomers', '274', '5', 'ring-opening', 'polymerizations', '571', 'polymerization', 'trioxane', 'trioxane', 'unique', 'among', 'cyclic', 'acetals', 'used', 'commercially', 'form', 'polyoxymethylene', 'polymer', 'much', 'like', 'one', 'obtained', 'cationic', 'polymerization', 'formaldehyde', 'some', 'questions', 'still', 'exist', 'exact', 'mechanism', 'initiation', 'trioxane', 'polymerizations', 'it', 'uncertain', 'instance', 'whether', 'cocatalyst', 'required', 'strong', 'lewis', 'acids', 'like', 'bf3', 'ticl4', 'the', 'cationic', 'polymerization', 'trioxane', 'initiated', 'protonic', 'acids', 'complexes', 'organic', 'acids', 'inorganic', 'salts', 'compounds', 'form', 'cations', '70.', 'these', 'initiators', 'differ', 'activity', 'inﬂuence', 'terminations', 'side', 'reactions', 'trioxane', 'also', 'polymerized', 'high-energy', 'radiation', '70.', 'in', 'addition', 'polymerizations', 'trioxane', 'carried', 'solid', 'phase', 'melt', 'gas', 'phase', 'suspension', 'solution', 'some', 'procedures', 'lead', 'different', 'products', 'however', 'variations', 'polymerization', 'conditions', 'cause', 'different', 'side', 'reactions', 'polymerizations', 'melt', '62(cid6)c', 'rapid', 'they', 'come', 'within', 'minutes', 'completion', '70(cid6)c', 'catalyzed', 'ten', 'moles', 'boron', 'triﬂuoride', 'this', 'procedure', 'however', 'useful', 'preparation', 'small', 'quantities', 'polymer', 'exothermic', 'heat', 'reaction', 'hard', 'control', 'typical', 'cationic', 'polymerizations', 'trioxane', 'characterized', 'induction', 'period', 'during', 'period', 'oligomers', 'monomeric', 'formaldehyde', 'form', 'this', 'formaldehyde', 'apparently', 'results', 'splitting', 'carbon', 'cations', 'form', 'primary', 'steps', 'polymerization', 'the', 'reaction', 'starts', 'temperature', 'dependent', 'equilibrium', 'concentration', 'formaldehyde', 'reached', '70.', 'ro', 'o', 'o', 'ch2', 'ro', 'o', 'o', 'ro', 'o', 'ch2', 'o', 'several', 'reaction', 'mechanisms', 'proposed', 'one', 'based', 'concept', 'lewis', 'acids', 'like', 'bf3', 'coordinate', 'directly', 'oxygen', 'acetal', 'this', 'results', 'ring', 'opening', 'induced', 'form', 'resonance', 'stabilized', 'zwiter', 'ion', '71:', 'bf3', 'o', 'o', 'o', 'bf3', 'o', 'o', 'o', 'o', 'o', 'o', 'bf3', 'o', 'o', 'o', 'bf3', 'ch2', 'resonance', 'stabilizations', 'adjacent', 'oxonium', 'ions', 'lead', 'formations', 'carbon', 'cations', 'believed', 'propagating', 'species', 'propagations', 'consist', 'repetitions', 'sequences', 'addition', 'carbon', 'cations', 'monomer', 'molecules', 'followed', 'ring', 'opening', 'the', 'mechanism', 'questioned', 'however', 'rigorously', 'dried', 'trioxane', 'solutions', 'cyclohexane', 'fail', 'polymerize', 'bf3(cid5)o(c4h9)3', 'catalyst', '72.', 'the', 'true', 'molten', 'trioxane', '73.', 'it', 'appears', '57', 'ring-opening', 'polymerizations', 'cyclic', 'acetals', '275', 'therefore', 'bf3-trioxane', 'complexes', 'don’t', 'form', 'suggested', 'result', 'initiations', 'polymerizations', 'additions', 'small', 'quantities', 'water', 'however', 'result', 'initiations', 'polymerizations', 'another', 'mechanism', 'based', 'concept', 'two', 'molecules', 'bf3', 'involved', 'initiation', 'process', '69.', 'this', 'also', 'appears', 'improbable', 'since', 'without', 'water', 'bf3', 'fails', 'initiate', 'reaction', 'following', 'mechanism', 'based', 'water', 'cocatalyst', 'developed', '73:', 'bf3', 'o', '(c4h9)2', 'h2o', 'h', 'bf3oh', 'o', '(c4h9)2', 'h', 'bf3oh', 'o', 'o', 'o', 'h', 'o', 'o', 'o', 'bf3oh', 'chain', 'growth', 'reaction', 'accompanied', 'formations', 'tetraoxane', '13-dioxalane', 'backbiting', '71', '74.', 'complex', 'molybdenyl', 'acetylacetones', 'also', 'act', 'catalysts', 'trioxane', 'polymerization', 'the', 'mech-', 'anism', 'visualized', 'involves', 'formation', 'coordinated', 'intermediate', '75:', 'the', 'termination', 'mechanism', 'catalyst', 'requirement', 'yet', 'fully', 'explored', 'some', 'transfer', 'water', 'takes', 'place', 'reaction', 'as', 'result', 'polymer', 'contains', 'least', 'one', 'terminal', 'hydroxyl', 'group', '76.', 'besides', 'water', 'methyl', 'alcohol', 'low', 'molecular', 'weight', 'ethers', 'also', 'act', 'transfer', 'agents', '77.', '276', '5', 'ring-opening', 'polymerizations', 'o', 'ch2', 'o', 'n', 'r', 'r', 'o', 'n', 'or', 'r', 'the', 'new', 'cation', 'initiate', 'chain', 'growth', 'r', 'o', 'o', 'o', 'o', 'ch2', 'o', 'o', 'o', 'o', 'r', '572', 'polymerization', 'dioxolane', 'polymerization', 'cyclic', 'monomer', 'yields', 'polymers', 'consists', 'strictly', 'alternating', 'oxymethylene', 'oxyethylene', 'units', '76.', 'polymerization', 'reaction', 'induced', 'acidic', 'catalysts', 'like', 'sulfuric', 'acid', 'boron', 'triﬂuoride', 'p-toluenesulfonic', 'acid', 'phosphorous', 'pentaﬂuoride', '76:', 'o', 'o', 'n', 'o', 'o', 'n', 'the', 'polymers', 'molecular', 'weight', '10000', 'higher', 'tough', 'solids', 'cold', 'drawn', 'the', 'following', 'mechanism', 'proposed', 'polymerizations', 'initiated', 'reaction', 'products', 'acetic', 'anhydride', 'perchloric', 'acid', '78:', 'o', 'clo4', 'o', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'hclo4', 'o', 'o', 'o', 'initiation', 'propagation', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', '57', 'ring-opening', 'polymerizations', 'cyclic', 'acetals', '277', 'termination', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n1', 'o', 'acetate', 'groups', 'present', 'end', 'polymer', 'molecules', 'shown', '78.', 'this', 'conﬁrmed', 'analytical', 'evidence', 'the', 'initiation', 'dioxolane', 'polymerization', 'pictured', 'differently', '79', '80:', '(c2h5)o', 'bf3', 'o', 'o', 'bf3', 'o', 'o', '(c2h5)o', 'bf3', 'o', 'o', 'o', 'o', 'o', 'bf3', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'chain', 'cleavage', 'occur', 'result', 'bf3', 'complexation', 'oxygen', 'chain', '80:', 'o', 'o', 'bf3', 'o', 'o', 'bf3', 'there', 'disagreement', 'nature', 'end', 'groups', 'speculations', 'polymers', 'might', 'possess', 'large', 'cyclic', 'structures', 'nevertheless', 'polymerizations', 'initiated', 'benzoyllium', 'hexaﬂuoroantimonate', '(c6h5cosbf6', '(cid4))', 'conducted', '(cid4)15(cid6)c', 'nitromethane', 'methylene', 'chloride', 'result', 'mostly', 'linear', 'polymers', 'the', 'terminal', 'end', 'groups', 'come', 'terminating', 'agents', 'deliberately', 'added', '81.', 'these', 'polymerizations', 'proceed', 'without', 'appreciable', 'amounts', 'transfer', 'reactions', 'affecting', 'dp', '573', 'polymerization', 'dioxopane', 'other', 'cyclic', 'acetals', 'polymerization', 'six', 'membered', 'cyclic', 'formals', 'apparently', 'explored', '1.', 'polymerization', '13-dioxopane', 'initiated', 'camphor', 'sulfonic', 'acid', '82', '83:', 'o', 'o', 'o', 'o', 'n', '278', '5', 'ring-opening', 'polymerizations', '13-trioxopane', 'product', 'condensation', 'trioxane', 'ethylene', 'oxide', 'polymerized', 'cationic', 'mechanism', 'solution', 'bulk', '84:', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'polymerizations', 'carried', 'boron', 'triﬂuoride', 'catalyst', 'dichloroethane', 'solvent', 'result', 'several', 'reactions', 'occur', 'simultaneously', '84:', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'h', 'h', 'copolymer', 'at', 'low', 'temperatures', 'amount', 'dioxolane', 'forms', 'reaction', 'decreases', 'consider-', 'ably', 'become', 'zero', 'ring-opening', 'polymerizations', 'trioxocane', 'result', 'following', 'polymer', '84', '85:', 'so', 'far', 'nature', 'end', 'groups', 'established', 'nor', 'shown', 'macrocyclic', 'structure', 'form', '1369-tetraoxacycloundecane', '(triethylene', 'glycol', 'formal)', 'polymerized', 'polymerized', 'several', 'cationic', 'initiators', 'solution', 'bulk', 'varying', 'temperatures', '(cid4)30', '150(cid6)c', '84:', 'o', 'o', 'o', 'oo', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', '3', 'n', '58', 'polymerization', 'lactones', 'polymerization', 'lactones', 'carried', 'three', 'mechanisms', 'namely', 'cationic', 'anionic', 'coordinated', 'one', 'often', 'mechanism', 'speciﬁc', 'lactone', 'polymerizes', 'depends', 'upon', 'size', 'ring', '581', 'cationic', 'polymerization', 'cationic', 'polymerizations', 'lactones', 'carried', 'help', 'alkylating', 'agents', 'acylating', 'agents', 'lewis', 'acids', 'protonic', 'acids', 'various', 'reaction', 'schemes', 'proposed', 'explain', '58', 'polymerization', 'lactones', '279', 'cationic', 'mechanism', 'they', 'tend', 'resemble', 'schemes', 'suggested', 'polymerization', 'cyclic', 'ethers', '86', '87.', 'the', 'initiation', 'step', 'involves', 'equilibrium', 'followed', 'ring-opening', 'reaction', 'o', 'r', '(ch2)x', 'o', 'o', 'r', 'o', '(ch2)x', 'the', 'propagation', 'consists', 'many', 'repetitions', 'step', '(ch2)5', 'o', 'ro', 'o', '5', 'ro', 'o', 'n', '(ch2)5', 'o', 'o', 'o', 'o', '5', 'o', '5', 'n1', 'o', 'ro', 'o', 'o', '5', 'o', 'x', '5', 'o', 'the', 'polymerization', 'propiolactone', 'methylene', 'chloride', 'antimony', 'pentachloride-', 'dietherate', 'catalyst', 'investigated', '88.', 'the', 'results', 'show', 'concentration', 'active', 'centers', 'dependent', 'upon', 'catalyst', 'concentration', 'upon', 'initial', 'concentration', 'monomer', 'they', 'also', 'support', 'concept', 'opening', 'lactone', 'rings', 'includes', 'initial', 'formation', 'oxonium', 'ions', '88:', 'o', 'o', 'slow', 'o', 'o', 'o', '2', '2', 'o', 'o', 'fast', 'o', '2', 'o', 'c', 'o', '2', 'o', 'o', 'o', 'o', 'o', 'c', 'because', 'carbonyl', 'oxygen', 'basic', 'oxygens', 'lactone', 'molecule', 'reverse', 'reaction', 'o', 'oc', 'o', 'oc', 'conductivity', 'measurements', 'polymerizations', 'b-propiolactone', 'antimony', 'pentaﬂuoride-', 'etherate', 'p-toluenesulfonic', 'acid', 'show', '89', 'ion', 'triplets', 'form', 'reaction', 'these', 'o', 'c', 'a', 'o', 'c', 'the', 'triplets', 'appear', 'active', 'centers', 'throughout', 'course', 'polymerizations', 'in', 'addition', 'growing', 'chain', 'ends', 'exist', 'ion', 'pairs', 'depending', 'upon', 'concentration', 'monomer', '89.', 'bourissou', 'et', 'al', 'reported', 'recently', 'controlled', 'cationic', 'polymerization', 'lactones', 'using', 'combina-', 'tion', 'triﬂic', 'acid', 'protonic', 'reagent', 'initiators', '90.', 'the', 'reaction', 'carried', 'ch2cl2', 'results', 'indicated', 'process', 'controlled', 'linear', 'relationship', 'molecular', '280', '5', 'ring-opening', 'polymerizations', 'weight', 'product', 'monomer', 'initiator', 'ratio', 'well', 'monomer', 'conversion', 'the', 'process', 'believed', 'proceed', 'protonation', 'lactone', 'triﬂic', 'acid', 'followed', 'nucleophilic', 'attack', 'initiating', 'alcohol', 'o', 'o', 'roh', 'cf3so2', 'h', 'o', 'h', 'o', 'ro', 'oh', 'it', 'believed', 'controlled', 'cationic', 'ring-opening', 'polymerization', 'proceeds', '“activated', 'anionic', 'mechanism”', 'suggested', 'penczek', '91.', 'according', 'suggestion', 'acid', 'activates', 'cyclic', 'ester', 'alcohol', 'subsequently', 'initiates', 'polymerization', 'kakuchi', 'coworkers', 'reported', 'controlledliving', 'cationic', 'ring-opening', 'polymerizations', 'd-valerolactone', 'e-caprolactone', 'aid', 'diphenyl', 'phosphate', '89.', 'the', 'reaction', 'illustrated', 'follows', 'ho', 'p', 'o', 'o', 'o', 'o', 'o', 'roh', 'o', 'r', 'o', 'o', 'h', 'n', 'the', 'ring-opening', 'polymerization', 'd-valerolactone', 'e-caprolactone', 'carried', 'using', '3-', 'phenyl-1-propanol', 'initiator', 'diphenyl', 'phosphate', 'catalyst', 'toluene', 'room', 'temperature', 'they', 'reported', 'reaction', 'proceeded', 'homogeneously', 'yield', 'poly(d-valerolactone)', 'poly(e-', 'caprolactone)', 'narrow', 'polydispersity', 'indices', 'analyses', 'indicated', 'presence', 'residues', 'initiator', '582', 'anionic', 'polymerization', 'lactones', 'in', 'anionic', 'polymerizations', 'initiations', 'result', 'attacks', 'bases', 'upon', 'carbonyl', 'groups', 'o', 'r', '(ch2)5', 'o', 'r', 'o', 'o', '5', 'common', 'initiators', 'li', 'k', 'alkoxides', 'in', 'addition', 'reported', 'phosphazene', 'bases', 'used', 'carry', 'polymerizations', 'cyclic', 'esters', '92.', 'also', 'commercially', 'available', 'materials', 'like', 'tert-butoxybis(dimethylamino)methane', 'tris(dimethylamino)methane', 'yield', 'high', 'molecular', 'weight', 'polylactic', 'acid', 'ring-opening', 'polymerization', 'narrow', 'molecular', 'weight', 'distribution', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'o', '58', 'polymerization', 'lactones', '281', 'the', 'propagations', 'take', 'place', 'similar', 'process', 'o', 'r', 'o', '5', 'o', '(ch2)5', 'o', 'o', 'r', 'o', '5', 'o', 'o', '5', 'these', 'steps', 'repeat', 'chains', 'built', 'anionic', 'polymerizations', 'yield', 'optically', 'active', 'polymers', 'this', 'observed', 'formations', 'poly(a-methyl', 'a-ethyl-b-', 'propiolactone)', '193', 'contains', 'asymmetric', 'carbon', 'atoms', '583', 'polymerization', 'lactones', 'coordination', 'mechanism', 'the', 'mechanism', 'coordination', 'polymerization', 'pictured', 'yong', 'malzner', 'pilato', '90', 'intermediate', 'two', 'modes', 'polymerization', '(a', 'cationic', 'anionic', 'one)', 'initiation', 'r', 'mt', '(ch2)5', 'o', 'o', 'o', '(ch2)5', 'mt', 'o', 'r', 'o', 'r', 'omt', '5', 'mt', 'means', 'metal', 'propagation', 'o', 'r', '(ch2)5', 'o', 'omt', '5', 'o', '(ch2)5', 'o', 'o', 'o', 'r', 'o', 'mt', '5', 'the', 'shown', 'mechanism', 'however', 'incorrect', 'caprolactone', 'polymerized', 'tin', 'compounds', '95.', 'yet', 'appears', 'correct', 'polymerizations', 'propiolactones', 'ethylzinc', 'monoxide', 'catalyst', '95.', 'the', 'bimetallic', 'oxoalkoxides', 'useful', 'catalysts', 'polymerizations', 'e-caprolactone', 'the', 'general', 'course', 'reaction', 'quite', 'similar', 'one', 'oxiranes', 'a', 'typical', 'coordination', 'mechanism', 'indicated', 'kinetic', 'structural', 'data', '97.', 'the', 'molecular', 'weight', 'increases', 'conversion', 'reaction', 'exhibits', '“living”', 'character', 'linear', 'relationship', 'dp', 'conversion', 'when', 'monomer', 'used', 'addition', 'fresh', 'monomer', 'reaction', 'mixture', 'results', 'increases', 'dp', 'by', 'avoiding', 'side', 'reactions', 'possible', 'achieve', 'high', 'molecular', 'weights', '(up', '200000)', 'narrow', 'molecular', 'weight', 'distribution', '(mwmn', '(cid7)', '105)', '97.', 'the', 'reaction', 'proceeds', 'insertion', 'lactone', 'units', 'al–or', 'bonds', 'the', 'acyl–oxygen', 'bond', 'cleaves', 'chain', 'binds', 'oxygen', 'catalyst', 'forming', 'alkoxide', 'link', 'rather', 'carboxylate', 'one', 'o', 'al', 'or', '(ch2)5', 'o', 'o', 'al', 'o', '5', 'or', 'n', '282', '5', 'ring-opening', 'polymerizations', 'there', 'potentially', 'four', 'active', 'sites', 'per', 'trinuclear', 'catalytic', 'molecule', 'the', 'number', 'actual', 'sites', 'however', 'depends', 'upon', 'aggregation', 'oxoalkoxides', 'two', 'different', 'types', 'or', 'groups', 'exist', 'depending', 'upon', 'bridging', 'aggregates', 'only', 'one', 'active', 'polymerization', 'this', 'results', 'catalytic', 'star-shaped', 'entity', 'the', 'fact', 'dissociated', 'catalysts', 'generate', 'four', 'growing', 'chains', 'per', 'al2(ch2)5co2(or)4', 'molecule', '97', 'tends', 'conﬁrm', 'the', 'commercially', 'available', 'aluminum', 'triisopropoxide', 'reported', 'effective', 'initiator', '“living”', 'ring-opening', 'polymerizations', 'e-caprolactone', 'lactides', 'glacolide', 'cyclic', 'anhydrides', '98.', 'based', 'kinetic', 'structural', 'data', 'ring-opening', 'polymerization', 'believed', 'take', 'place', 'coordination-insertion', 'mechanism', 'while', 'molecules', 'aluminum', 'triisopropoxide', 'coordinatively', 'associated', 'toluene', 'presence', 'lactones', 'single', 'isolated', 'monomeric', 'species', 'form', 'believed', 'remain', 'unassociated', 'propagation', 'reaction', '98.', 'actually', 'ring-opening', 'polymerizations', 'e-caprolactone', 'achieved', 'various', 'catalysts', 'only', 'however', 'initiate', '“living”', 'polymerizations', 'among', 'aluminum', 'alkoxides', 'described', 'bimetallic', 'm-alkoxides', '99,', 'porphynatoaluminum', '100,', 'mono(cyclopentadienyl)', 'titanium', 'complexes', '101,', 'rare', 'earth', 'alkoxides', '102', '103.', 'examples', 'rare', 'earth', 'alkoxides', 'ln', 'nd', 'y', 'nd', 'isopropoxy', 'diethyl', 'acetoacetates', '(c5h5)2lnor', 'c5(ch3)52lnch3', '(donor)', 'complexes', 'it', 'suggested', 'steric', 'effect', 'bulky', 'groups', 'catalysts', 'suppress', 'interfering', 'transesteriﬁcation', 'reaction', 'screening', 'linear', 'polymeric', 'chains', 'active', 'centers', 'reactions', 'yield', '“living”', 'polymerizations', '104.', 'these', 'catalysts', 'also', 'useful', 'formation', 'various', 'block', 'copolymers', 'lactones', 'monomers', '104', '105.', 'among', 'lactones', 'polymerized', 'help', 'rare', 'earth', 'catalysts', 'lactide', '106–108,', 'd-valerolactone', '109,', 'b-propiolactone', '109,', 'b-butyrolactone', '107.', 'polymerization', 'e-caprolactone', 'catalyst', 'system', 'consisting', 'tris(26-di-tert-butylphenoxy)', 'yttrium', '2-propanol', 'ﬁrst', 'order', 'respect', 'monomer', 'initiator', '105.', 'this', 'led', 'conclusion', 'reaction', 'proceeds', 'via', 'three-step', 'mechanism', 'illustrated', 'follows', '105:', '3', 'roh', 'o', 'y', 'o', 'o', 'o', 'o', 'y(or)3', '3', 'y(o', '3x', 'or)3', 'o', '3h', 'o', 'or', 'y', 'ro', 'or', '3ho', 'y(o', 'o', 'o', 'or)3', 'o', 'h', 'o', 'x', 'o', 'or', 'o', '58', 'polymerization', 'lactones', '283', 'it', 'also', 'reported', '“living”', 'e-caprolactone', 'polymerization', 'carried', 'bis(acryloxy-)', 'lanthanide', '(ii)', 'complexes', 'based', 'samarium', '110.', 'thus', '(aro)2sm(thf)4', '(where', 'aro', '¼', '26-di-', 'tert-butyl-4-methyl-phenoxy)', 'yielded', '98', 'conversion', 'toluene', '60(cid6)c', '1', 'h', 'the', 'central', 'ions', 'ligands', 'appear', 'effect', 'activity', 'catalyst', '110.', '584', 'special', 'catalysts', 'polymerizations', 'lactones', 'some', 'lactones', 'polymerize', 'presence', 'compounds', 'like', 'alcohols', 'amines', 'carboxylic', 'acids', 'without', 'additional', 'catalysts', 'the', 'reactions', 'however', 'slow', 'yield', 'low', 'molecular', 'weight', 'polymers', '95.', 'exception', 'polymerizations', 'pivalolactone', 'presence', 'cyclic', 'amines', 'yield', 'high', 'molecular', 'weight', 'polyesters', 'high', 'conversion', '111.', 'the', 'initiating', 'steps', 'result', 'formations', 'adducts', 'amine-pivalate', 'betaines', 'o', 'slow', 'o', 'n', 'coo', 'n', 'n', 'n', 'coo', 'o', 'fast', 'o', 'coo', 'slow', 'o', 'o', 'n', 'o', 'o', 'n', 'the', 'reaction', 'appears', 'restricted', 'highly', 'strained', 'lactones', 'may', 'work', 'larger', 'lactones', '95,', 'for', 'instance', 'polymerization', 'd-valerolactone', 'initiated', 'ethanolamine', 'temperatures', '200(cid6)c', 'initially', 'rapid', 'reaction', 'amine', 'group', 'monomer', 'o', '(ch2)4', 'o', 'h2n', 'oh', 'ho', 'oh', 'the', 'subsequent', 'reactions', 'however', 'slow', 'ho', 'o', '4', 'n', 'h', 'o', 'slow', 'oh', '(ch2)4', 'o', 'ho', 'o', 'oh', '4', 'o', 'o', '4', 'n', 'h', 'o', '4', 'n', 'h', '284', '5', 'ring-opening', 'polymerizations', 'it', 'suggested', 'initiators', 'like', 'dibutylzinc', 'lack', 'active', 'hydrogens', 'placed', 'special', 'category', '96.', 'they', 'initiate', 'polymerizations', 'lactones', 'one', 'e-', 'caprolactone', 'polymers', 'form', 'inversely', 'proportional', 'molecular', 'weights', 'catalyst', 'concentrations', '112.', 'the', 'true', 'stannic', 'tetraacrylate', 'high', 'molecular', 'weight', 'poly(e-', 'caprolactone)', 'high', '100000', 'forms', 'addition', 'compounds', 'may', 'serve', 'source', 'active', 'hydrogens', 'necessary', '95.', 'this', 'group', 'initiators', 'also', 'includes', 'dimethylcadmium', 'methylmagnesium', 'bromide', 'others', 'effective', 'polymerizations', 'd-valerolactone', 'e-caprolactone', 'alkyl', 'substituted', 'derivatives', 'the', 'polymers', 'form', 'high', 'molecular', 'weight', 'high', '250000', '113.', 'another', 'group', 'consists', 'zinc', 'lead', 'salts', 'stannous', 'esters', 'phosphines', 'alkyl', 'titanates', 'this', 'group', 'require', 'additions', 'compounds', 'active', 'hydrogens', 'such', 'additives', 'polyols', 'polyamines', 'carboxylic', 'acid', 'compounds', '95.', 'molecular', 'weight', 'control', 'difﬁcult', 'catalysts', 'belonging', 'ﬁrst', 'group', 'this', 'second', 'group', 'hand', 'allows', 'control', 'molecular', 'weights', 'also', 'nature', 'end', 'groups', '95.', 'weymouth', 'coworkers', 'carried', 'kinetic', 'mechanistic', 'studies', 'heterocyclic', 'carbene', 'mediated', 'zwitterionic', 'polymerization', 'cyclic', 'esters', '96.', 'based', 'results', 'proposed', 'following', 'ring-opening', 'mechanism', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'r', 'n', 'n', 'r', 'r', 'n', 'n', 'r', 'o', 'o', 'o', 'o', 'r', 'n', 'n', 'o', 'o', 'o', 'n', 'o', 'r', 'o', 'from', 'kinetic', 'studies', 'able', 'conclude', 'heterocyclic', 'carbene', 'initiated', 'polymerization', 'lactide', 'rate', 'initiation', 'slower', 'rate', 'propagation', 'also', 'rate', 'propagation', 'much', 'faster', 'chain', 'termination', 'via', 'cyclization', '59', 'polymerizations', 'lactams', 'polymerizations', 'lactams', 'produce', 'important', 'commercial', 'polymers', 'the', 'polymerization', 'reactions', 'therefore', 'received', 'considerable', 'attention', 'lactam', 'molecules', 'polymerize', 'three', 'different', 'mechanisms', 'cationic', 'anionic', 'hydrolytic', 'one', '(by', 'water', 'water', 'releasing', 'substances)', 'the', 'lactam', 'ring', 'strongly', 'resonance', 'stabilized', 'carbonyl', 'activity', 'low', 'nevertheless', 'ring-opening', 'polymerizations', 'start', 'small', 'amounts', 'initiators', 'trans-acylation', 'reactions', 'fairly', 'high', 'temperatures', 'however', 'needed', 'often', '200(cid6)c', 'in', 'reactions', 'one', 'molecule', 'acts', 'acylating', 'agent', 'electrophile', 'one', 'acts', 'nucleophile', 'undergoes', 'acylation', 'generally', 'initiators', 'activate', 'inactive', 'amide', 'groups', 'causing', 'react', 'lactams', 'successive', 'transamidations', 'result', 'formations', 'polyamides', 'both', 'acids', 'bases', 'catalyze', 'transamidation', 'reactions', 'the', 'additions', 'electrophiles', 'affect', 'increases', 'electrophilicity', 'carbonyl', 'carbon', 'acylating', 'lactam', 'the', 'nucleophiles', 'hand', 'increase', 'nucleophilic', 'character', 'lactam', 'substrate', '(if', 'bases)', '59', 'polymerizations', 'lactams', '285', 'all', 'initiators', 'divided', 'two', 'groups', 'to', 'ﬁrst', 'one', 'belong', 'strong', 'bases', 'capable', 'forming', 'lactam', 'anions', 'removing', 'amide', 'proton', 'this', 'starts', 'anionic', 'polymerization', 'reaction', 'to', 'second', 'one', 'belong', 'active', 'hydrogen', 'compounds', 'capable', 'protonating', 'amide', 'bond', 'thereby', 'affecting', 'cationic', 'polymerization', '114.', 'side', 'reactions', 'common', 'lactam', 'polymerizations', 'their', 'nature', 'extent', 'depends', 'upon', 'concentration', 'character', 'initiators', 'temperatures', 'reactions', 'structures', 'lactams', 'when', 'cationic', 'polymerizations', 'lactams', 'initiated', 'strong', 'acids', 'strongly', 'basic', 'amidine', 'groups', 'produced', 'these', 'groups', 'bind', 'strong', 'acids', 'inactive', 'growth', 'centers', 'decrease', 'rate', 'polymerization', 'use', 'strong', 'bases', 'initiate', 'polymerizations', 'lactams', 'possessing', 'least', 'one', 'a-hydrogen', 'also', 'result', 'side', 'reaction', 'compounds', 'form', 'decrease', 'basicity', 'lactams', 'polyamides', 'slow', 'polymerizations', 'also', 'side', 'reactions', 'give', 'rise', 'irregular', 'structures', 'namely', 'branching', 'the', 'ring-opening', 'polymerization', 'reactions', 'depend', 'upon', 'thermodynamic', 'kinetic', 'factors', 'total', 'molecular', 'strain', 'energies', 'particular', 'ring', 'structures', 'six-membered', 'd-valerolactam', 'stable', 'ring', 'structure', 'difﬁcult', 'polymerize', 'also', 'presence', 'substituents', 'increases', 'stability', 'rings', 'decreases', 'ability', 'polymerize', '591', 'cationic', 'polymerization', 'lactams', 'the', 'catalysts', 'cationic', 'polymerization', 'strong', 'anhydrous', 'acids', 'lewis', 'acids', '115,', 'salts', 'primary', 'secondary', 'amines', 'carboxylic', 'acids', 'salts', 'amines', 'carboxylic', 'acids', 'split', 'water', 'elevated', 'temperatures', '114.', 'the', 'initiators', 'react', 'coordinating', 'forming', 'rapid', 'pre', 'equilibrium', 'lactam', 'cations', 'these', 'cations', 'reactive', 'species', 'polymerizations', 'initiations', 'type', 'also', 'possible', 'weakly', 'acidic', 'compound', 'compounds', 'able', 'transfer', 'protons', 'lactam', 'they', 'capable', 'however', 'forming', 'hydrogen', 'bonds', 'lactams', 'the', 'high', 'reactivity', 'lactam', 'cations', 'may', 'attributed', 'decreased', 'electron', 'density', 'carbonyl', 'carbon', 'atoms', 'this', 'makes', 'subject', 'nucleophilic', 'attacks', '114.', 'protonations', 'amides', 'occur', 'oxygens', '116,', 'small', 'fractions', 'n-protonated', 'amides', 'also', 'presumed', 'exist', 'tautomeric', 'equilibrium', 'to', 'simplify', 'illustrations', 'lactams', 'shown', 'section', 'so', 'structure', 'commonly', 'represents', 'propiolactam', 'section', 'mean', 'lactam', 'like', 'caprolactam', 'valerolactam', 'etc', 'thus', 'equilibrium', 'shown', 'follows', 'c', 'n', 'c', 'n', 'o', 'h', 'oh', 'h', 'h', 'o', 'c', 'n', 'h', 'h', 'n', 'h', 'c', 'h', 'c', 'o', 'nh2', '286', '5', 'ring-opening', 'polymerizations', 'in', 'reaction', 'mixture', 'initiators', 'strong', 'acids', 'strongest', 'nucleophiles', 'formations', 'monomers', 'acylations', 'monomers', 'amidinium', 'cations', 'result', 'aminoacyllactams', '113:', 'o', 'c', 'n', 'h', 'o', 'c', 'n', 'h2', 'o', 'c', 'n', 'c', 'o', 'nh3', 'acylation', 'amine', 'groups', 'molecules', 'protonated', 'lactams', 'results', 'monomers', 'becoming', 'incorporated', 'polymers', '117.', 'the', 'growth', 'centers', 'preserved', 'molecule', 'lactam', 'protonated', 'this', 'occurs', 'two', 'steps', '117:', '1', 'o', 'c', 'n', 'c', 'o', 'nh3', 'o', 'c', 'n', 'h', 'o', 'c', 'n', 'c', 'o', 'nh2', 'o', 'c', 'n', 'h2', '2', 'nh2', 'o', 'c', 'n', 'h2', 'c', 'o', 'nh3', 'n', 'h', 'these', 'reactions', 'attain', 'equilibrium', 'quickly', 'aminolyses', 'acyllactams', 'reverse', 'initiation', 'reactions', 'precede', 'rapidly', '117–119.', 'aminolyses', 'aminoacyllactams', 'actually', 'contribute', 'propagation', 'process', '120', '121:', '1', 'o', 'c', 'n', 'c', 'o', 'nh3', 'o', 'c', 'n', 'h', 'o', 'c', 'n', 'c', 'o', 'nh2', 'o', 'c', 'n', 'h2', '2', 'nh2', 'o', 'c', 'n', 'h2', 'c', 'o', 'nh3', 'n', 'h', 'these', 'reactions', 'attain', 'equilibrium', 'quickly', 'aminolyses', 'acyllactams', 'reverse', 'initiation', 'reactions', 'precede', 'rapidly', '117–119.', 'the', 'reaction', 'therefore', 'proceeds', 'follows', '120', '121:', '59', 'polymerizations', 'lactams', '287', 'o', 'h', 'o', 'o', 'c', 'h3ncl', 'c', 'n', 'c', 'n', 'c', 'o', 'h2n', 'c', 'o', 'h3ncl', 'n', 'c', 'o', 'h3ncl', 'n', 'c', 'o', 'cl', 'o', 'c', 'o', 'o', 'o', 'h3ncl', 'c', 'n', 'h', 'c', 'n', 'h2cl', 'c', 'n', 'c', 'o', 'the', 'reaction', 'results', 'destruction', 'equilibrium', 'regeneration', 'strongly', 'acidic', 'amide', 'salt', 'total', 'lactam', 'consumption', 'results', 'repetitions', 'sequences', 'formations', 'new', 'aminoacyllactam', 'molecules', '113–121.', 'initiations', 'polymerizations', 'acid', 'salts', 'primary', 'secondary', 'amines', 'result', 'chain', 'growths', 'proceeds', 'predominantly', 'additions', 'protonated', 'lactams', 'amines', '113:', 'o', 'c', 'nh2', 'nh2', 'h', 'n', 'o', 'c', 'nh3', 'the', 'rate', 'initiating', 'amines', 'incorporated', 'proportional', 'basicity', 'as', 'conversion', 'progresses', 'concentration', 'protonated', 'lactams', 'reaction', 'mixture', 'decreases', 'protonated', 'polymer', 'amide', 'groups', 'increases', 'the', 'latter', 'takes', 'part', 'initiation', 'reactions', 'lactam', 'molecules', 'exchange', 'reactions', 'polymer', 'molecules', '113:', 'o', 'c', 'n', 'h', 'o', 'c', 'nh2', 'o', 'c', 'n', 'h', 'o', 'c', 'n', 'h2', 'in', 'initiation', 'steps', 'strongest', 'nucleophile', 'present', 'reacts', 'lactam', 'cation', 'when', 'strong', 'anhydrous', 'bronsted', 'acids', 'initiate', 'polymerizations', 'free', 'lactams', 'acylated', 'ﬁrst', 'formation', 'aminoacyllactams', 'when', 'polymerizations', 'initiated', 'amine', 'salts', 'initial', 'steps', 'conversions', 'amines', 'amino', 'acid', 'amides', 'on', 'hand', 'hydrolytic', 'polymerizations', 'start', 'formations', 'unsubstituted', 'amino', 'acids', '122:', 'h2n', 'c', 'o', 'h2o', 'h3n', 'cooh', '288', '5', 'ring-opening', 'polymerizations', 'when', 'weak', 'carboxylic', 'acids', 'acids', 'medium', 'strength', 'initiate', 'lactam', 'polymerizations', 'anhydrous', 'conditions', 'induction', 'period', '123.', 'in', 'addition', 'rates', 'reactions', 'proportional', 'pka', 'acids', '105.', 'it', 'appears', 'different', 'reaction', 'mechanisms', 'involved', 'depending', 'upon', 'acid', 'strengths', '113.', 'the', 'nucleophiles', 'present', 'equilibrium', 'rcooh', 'o', 'c', 'nh', 'cr', 'o', 'o', 'c', 'nh2', 'o', 'the', 'acylation', 'carboxylate', 'anions', 'assumed', 'lead', 'formations', 'mixed', 'anhydrides', 'acids', 'amino', 'acids', '124', 'subsequent', 'rearrangements', 'o', 'c', 'o', 'o', 'c', 'nh2', 'o', 'c', 'o', 'nh2', 'o', 'c', 'o', 'c', 'o', 'c', 'nh', 'oh', 'when', 'strong', 'acids', 'however', 'initiate', 'polymerizations', 'strongest', 'nucleophiles', 'present', 'lactam', 'amide', 'groups', 'undergo', 'acylations', 'as', 'result', 'acids', 'incorporated', 'polymers', 'the', 'propagation', 'steps', 'cationic', 'polymerizations', 'lactams', 'occur', 'transamidation', 'reactions', 'lactam', 'rings', 'ammonium', 'groups', 'formed', 'steps', 'initiation', 'it', 'believed', 'reaction', 'proton', 'transfers', 'take', 'place', 'ﬁrst', 'amine', 'salts', 'lactams', 'acyllactams', 'form', 'cations', 'these', 'turn', 'acylate', 'free', 'amines', 'form', 'regeneration', 'ammonium', 'groups', 'nh3', 'o', 'c', 'nh', 'nh2', 'h2n', 'c', 'o', 'o', 'c', 'o', 'h', 'nh3', 'n', 'c', 'o', 'nh2', 'c', 'n', 'c', 'o', 'nh3', 'c', 'n', 'c', 'o', 'nh2', 'c', 'h', 'o', 'h', 'o', 'h', 'o', 'n', 'c', 'o', 'nh', 'c', 'h2', 'n', 'c', 'o', 'h', 'nh3', 'h2', 'n', 'c', 'o', 'h', 'o', 'n', 'c', 'nh3', 'h', 'the', 'propagation', 'step', 'rapid', 'aminolysis', 'takes', 'place', 'carbonyl', 'group', 'activated', 'acid', 'derivative', '(like', 'acyllactam', 'acid', 'chloride)', 'it', 'slower', 'however', 'involves', 'amide', 'group', 'monomer', '114.', 'as', 'typical', 'many', 'carbonyl', 'reactions', 'acylations', 'followed', 'eliminations', '125:', '59', 'polymerizations', 'lactams', '289', 'nh2', 'o', 'c', 'nh2', 'oh', 'nh', 'c', 'nh2', '-h2o', 'h2o', 'nh', 'c', 'n', 'h', 'c', 'n', 'h', 'h', 'n', 'the', 'water', 'elimination', 'reaction', 'results', 'formations', 'amidines', 'acylamidinium', 'ions', 'also', 'result', 'dehydration', 'tetrahedral', 'intermediates', 'reactions', 'amino', 'groups', 'acyllactams', 'such', 'groups', 'could', 'also', 'present', 'within', 'polymer', 'molecules', 'the', 'water', 'released', 'reactions', 'hydrolyzes', 'acyllactams', 'acylamidine', 'salts', 'lactam', 'salts', 'yield', 'carboxylic', 'acids', '114.', 'in', 'cationic', 'polymerization', 'lactams', 'ammonium', 'amidinium', 'groups', 'form', 'n-terminal', 'chain', 'ends', 'the', 'c-terminal', 'chain', 'ends', 'form', 'carboxylic', 'acid', 'groups', 'alkylamide', 'residues', 'this', 'important', 'nature', 'end', 'groups', 'reactivity', 'determine', 'types', 'cationic', 'steps', 'polymerizations', 'lactams', 'results', 'different', 'end', 'groups', 'form', 'initiation', 'steps', 'formation', 'amidines', 'increases', 'increasing', 'acidity', 'concentration', 'initiator', 'increase', 'temperature', 'follow', 'polymerizations', 'this', 'means', 'different', 'r', 'nh', 'c', 'o', 'cl', 'r', 'nh', '(ch2)', 'x', 'c', 'o', 'cl', 'when', 'strong', 'acids', 'amine', 'salts', 'initiate', 'polymerizations', 'almost', 'amine', 'salt', 'groups', 'become', 'converted', 'amidine', 'salts', 'shortly', 'start', 'initiation', 'reaction', '108.', 'formation', 'amidinium', 'salts', 'leads', 'decrease', 'reaction', 'rate', 'initiate', 'polymerizations', 'lactams', 'less', 'effectively', 'ammonium', 'salts', '125', '126.', 'lewis', 'acids', 'act', 'similar', 'manner', 'unless', 'co-reactant', 'present', 'like', 'water', 'in', 'case', 'lewis', 'acids', 'transformed', 'protonic', 'acids', 'polymerizations', 'proceed', 'initiated', 'protonic', 'acids', '114.', 'n-substituted', 'lactams', 'generally', 'polymerized', 'some', 'exceptions', 'however', 'known', 'cationic', 'mechanisms', 'employed', '122', 'strong', 'carboxylic', 'inorganic', 'acids', 'used', 'initiators', 'in', 'cases', 'anions', 'initiating', 'acids', 'like', 'cl(cid4)', 'react', 'lactam', 'cations', 'yield', 'amino', 'acid', 'chlorides', '114:', 'h', 'r', 'n', 'c', 'o', 'cl', 'h', 'cl', 'r', 'n', 'c', 'o', 'hcl', 'r', 'n', 'o', 'c', 'cl', 'only', 'strained', 'four-', 'eight-', 'nine-membered', 'n-substituted', 'lactams', 'far', 'shown', 'capable', 'polymerizations', '113.', 'the', '22-dimethylquinuclidone', 'highly', 'strained', 'undergoes', 'polymerizations', 'room', 'temperature', '127.', 'the', 'propagation', 'reaction', 'substituted', 'lactams', 'illustrated', 'follows', '122:', '290', '5', 'ring-opening', 'polymerizations', 'cl', 'cl', 'h', 'n', 'r', 'c', 'o', 'h', 'n', 'cr', 'o', 'r', 'c', 'n', 'cr', 'o', 'h', 'cl', 'h', 'cl', 'hcl', 'o', 'hcl', 'n', 'r', 'c', 'o', 'hcl', 'n', 'cr', 'o', 'hn', 'cr', 'n', 'r', 'c', 'o', 'o', 'h', 'n', 'hcl', 'cl', 'cl', '2', 'hcl', '592', 'anionic', 'polymerization', 'lactams', 'the', 'anionic', 'polymerizations', 'lactams', 'initiated', 'strong', 'bases', '128', 'capable', 'forming', 'lactam', 'anions', 'o', 'c', 'nh', 'b', 'o', 'c', 'n', 'bh', 'such', 'bases', 'alkali', 'metals', 'metal', 'hydrides', 'organometallic', 'compounds', 'metal', 'amides', 'the', 'initiation', 'step', 'ring-opening', 'amidation', 'shown', 'follows', '2', 'o', 'c', 'nh', '2', 'na', '2', 'o', 'c', 'n', '2', 'na', 'h2', 'o', 'c', 'n', 'o', 'c', 'nh', 'o', 'c', 'n', 'c', 'nho', 'the', 'primary', 'amine', 'anions', 'abstract', 'protons', 'rapidly', 'molecules', 'lactams', 'form', 'amino-acyllactams', '130:', 'o', 'c', 'o', 'c', 'n', 'nh', 'hn', 'o', 'c', 'o', 'c', 'n', 'o', 'c', 'nh2', 'n', 'in', 'reactions', 'propagation', 'centers', 'cyclic', 'amide', 'linkages', 'n-acylated', 'terminal', 'lactam', 'rings', 'acylation', 'amide', 'nitrogens', 'effect', 'increasing', 'electron', 'deﬁciencies', 'groups', 'this', 'turn', 'increases', 'reactivities', 'cyclic', 'amide', 'carbonyls', 'toward', 'attacks', 'nucleophilic', 'lactam', 'anions', '115:', 'o', 'c', 'n', 'o', 'c', 'n', 'o', 'c', 'o', 'c', 'n', 'n', 'o', 'c', 'o', 'c', 'o', 'c', '59', 'polymerizations', 'lactams', '291', 'very', 'rapid', 'proton', 'exchanges', 'follows', 'this', 'results', 'equilibrium', 'lactam', 'polymeric', 'amide', 'anions', '129:', 'o', 'c', 'o', 'c', 'n', 'n', 'o', 'c', 'o', 'c', 'hn', 'o', 'c', 'o', 'c', 'nh', 'n', 'o', 'c', 'o', 'c', 'n', 'the', 'polymer', 'amide', 'anions', 'undergo', 'acylation', 'acyllactam', 'groups', 'accompanying', 'ring', 'opening', 'formation', 'lactam', 'anions', 'in', 'ﬁrst', 'instance', 'alternate', 'path', 'propagation', 'formation', 'imide', 'groups', 'o', 'c', 'n', 'o', 'c', 'o', 'c', 'n', 'o', 'c', 'n', 'o', 'c', 'o', 'c', 'n', 'c', 'o', 'o', 'c', 'n', 'o', 'c', 'n', 'o', 'c', 'o', 'c', 'n', 'o', 'c', 'n', 'h', 'n', 'c', 'o', 'n', 'h', 'o', 'c', 'h', 'n', 'n', 'h', 'o', 'c', 'o', 'c', 'o', 'c', 'n', 'n', 'o', 'c', 'o', 'n', 'c', 'h', 'n', 'h', 'n', 'o', 'c', 'o', 'n', 'c', 'the', 'acylation', 'reactions', 'shown', 'much', 'faster', 'initiation', 'reactions', '129', '131', 'as', 'result', 'induction', 'periods', 'anionic', 'polymerizations', 'lactams', '113.', 'in', 'addition', 'steep', 'increases', 'molecular', 'weights', 'take', 'place', 'beginning', 'polymerizations', 'bimolecu-', 'lar', 'aminolyses', 'may', 'contribute', 'though', 'contributions', 'total', 'conversions', 'negligible', '113.', '292', '5', 'ring-opening', 'polymerizations', 'the', 'overall', 'rates', 'polymerizations', 'depend', 'concentrations', 'acyllactams', 'diacylamine', 'groups', 'well', 'lactam', 'anions', 'the', 'latter', 'result', 'form', 'dissociations', 'lactam', 'salts', 'depending', 'upon', 'nature', 'metal', 'me', 'n', 'c', 'o', 'me', 'n', 'c', 'o', 'me', 'means', 'metal', 'the', 'alkali', 'metals', 'rated', 'following', 'order', 'respect', 'rates', 'initiations', 'propagations', '113', '132:', 'linakca', 'additions', 'activators', 'cocatalysts', 'acyl', 'halides', 'anhydrides', 'isocyanates', 'result', 'elimination', 'induction', 'period', 'these', 'additives', 'insure', 'formations', 'stabilized', 'adducts', 'o', 'cl', 'o', 'c', 'o', 'c', 'n', 'cr', 'o', 'c', 'n', 'r', 'cl', 'o', 'c', 'n', 'r', 'n', 'c', 'o', 'o', 'c', 'n', 'n', 'o', 'c', 'n', 'h', 'nr', 'c', 'o', 'o', 'c', 'n', 'o', 'c', 'o', 'c', 'o', 'c', 'n', 'h', 'r', 'r', 'o', 'c', 'the', 'structures', 'activators', 'determine', 'rates', 'addition', 'ﬁrst', 'lactam', 'anion', '113.', 'if', 'instance', 'acyl', 'group', 'large', 'pivaloylcaprolactam', 'decrease', 'rate', 'merely', 'due', 'steric', 'hindrance', '113', '129.', 'on', 'hand', 'substituents', 'like', 'benzoyl', 'group', 'increase', 'rates', 'additions', 'ﬁrst', 'lactam', 'anions', '113', '129.', 'in', 'addition', 'structures', 'activators', 'also', 'affect', 'course', 'polymerization', 'this', 'become', 'incorporated', 'end', 'polymeric', 'molecules', 'may', 'inﬂuence', 'basicity', 'polymerization', 'reactions', 'polymerizations', 'presence', 'acylating', 'agents', 'often', 'called', 'activated', 'polymerization', 'if', 'acylating', 'agents', 'absent', 'reaction', 'mixture', 'reactions', 'may', 'called', 'nonactivated', 'sometimes', 'terms', 'assisted', 'nonassisted', 'used', 'instead', 'several', 'reaction', 'mechanisms', 'offered', 'explain', 'mechanism', 'anionic', 'ring-opening', 'polymerizations', 'lactams', 'one', 'mechanism', 'based', 'nucleophilic', 'attacks', 'lactam', 'anions', 'cyclic', 'carbonyl', 'groups', 'n-acylated', 'lactams', 'this', 'leads', 'formations', 'intermediate', 'symmet-', 'rical', 'mesomeric', 'anions', 'rearrange', 'openings', 'rings', '133', '134:', 'o', 'o', 'r', 'c', 'n', 'c', 'n', 'o', 'c', 'o', 'o', 'c', 'r', 'c', 'n', 'n', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'r', 'n', 'n', '59', 'polymerizations', 'lactams', '293', 'champetier', 'sekiguchi', 'concluded', 'intermediate', 'anions', 'neutralized', 'ﬁrst', 'protons', 'lactams', 'polymer', 'amide', 'groups', 'the', 'neutral', 'molecules', 'subsequently', 'rearrange', 'openings', 'penultimate', 'units', '135', '136.', 'o', 'c', 'o', 'c', 'r', 'n', 'n', 'o', 'c', 'o', 'c', 'oh', 'n', 'c', 'n', 'o', 'c', 'h', 'r', 'r', 'h', 'n', 'o', 'c', 'o', 'c', 'n', 'o', 'c', 'they', 'also', 'felt', 'acylating', 'strength', 'acyllactams', 'enhanced', 'coordination', 'cations', 'imide', 'carbonyl', 'groups', '135', '136.', 'this', 'based', 'assumption', 'incorporations', 'lactam', 'units', 'proceed', 'additions', 'lactam', 'anions', 'protonations', 'subsequent', 'rearrangements', 'follow', '135', '136.', 'this', 'type', 'chain', 'growth', 'termed', 'lactomolytic', 'propagation', '137:', 'c', 'o', 'n', 'me', 'c', 'o', 'n', 'c', 'o', 'h', 'n', 'c', 'o', 'c', 'o', 'c', 'n', 'oh', 'me', 'n', 'c', 'o', 'c', 'o', 'c', 'n', 'o', 'me', 'n', 'c', 'n', 'h', 'o', 'c', 'o', 'c', 'o', 'me', 'n', 'c', 'o', 'the', 'mechanism', 'implies', 'alkaline', 'cation', 'ﬁxed', 'imide', 'group', 'nucleophilic', 'attack', '(that', 'rate-determining', 'step)', 'lactam', 'anion', 'takes', 'place', 'endocyclic', 'carbonyl', 'group', 'imide', 'give', '“carbinolate”', 'anion', 'proton', 'exchange', 'takes', 'place', 'interme-', 'diate', 'lactam', 'monomer', 'intramolecular', 'rearrangement', 'results', 'ring', 'opening', 'unit', 'penultimate', 'position', 'in', 'anionic', 'activated', 'polymerization', 'e-caprolactam', 'chain', 'growth', 'involves', 'free', 'anions', 'ion', 'pairs', '138.', 'quantum-chemical', 'calculations', 'suggest', 'alkali', 'metal', 'lactamate', 'molecule', 'negative', 'charge', 'delocalized', 'oxygen', 'nitrogen', 'heteroatoms', 'this', 'led', 'suggestion', 'frunze', 'et', 'al', '138.', 'acts', 'initiation', 'formations', 'activated', 'intermediate', 'chelate', 'type', 'complexes', 'activators', 'catalyst', 'molecules', '138:', 'r', 'o', 'c', 'n', 'c', 'o', 'me', 'o', 'c', 'n', 'r', 'o', 'c', 'n', 'me', 'c', 'o', 'n', 'c', 'o', 'complex', 'me', 'metal', 'like', 'li', 'k', 'cs', 'etc', 'the', 'carbinol', 'fragment', 'resulting', 'complex', 'shown', 'undergoes', 'intramolecular', 'rear-', 'rangement', 'it', 'leads', 'opening', 'heterocyclic', 'ring', 'growth', 'polymer', 'chain', 'one', 'unit', '294', '5', 'ring-opening', 'polymerizations', 'o', 'r', 'c', 'o', 'c', 'n', 'n', 'me', 'h', 'o', 'o', 'c', 'n', 'c', 'o', 'n', 'c', 'me', 'r', 'o', 'o', 'oh', 'c', 'n', 'n', 'c', 'c', 'c', 'me', 'o', 'n', 'o', 'c', 'n', 'r', 'h', 'o', 'c', 'n', 'o', 'c', 'the', 'frunze', 'et', 'al', 'mechanism', '138', 'much', 'common', '“alkali', 'lactomolytic”', 'mechanism', 'champetier', 'sekiguchi', '135,', 'except', 'formation', 'shown', 'complex', 'frunze', 'et', 'al', 'also', 'believe', 'probably', 'single', 'mechanism', 'exists', 'anionic', 'polymerization', 'lactams', 'describe', 'ion-coordinative', '138.', 'the', 'contributions', 'various', 'mechanisms', 'via', 'ion', 'pairs', 'via', 'free', 'ions', 'depend', 'upon', 'nature', 'alkali', 'metal', 'counterion', 'upon', 'capacity', 'coordinate', 'electron', 'donating', 'compounds', '(activator', 'monomer)', 'the', 'growth', 'ion', 'pairs', 'may', 'mainly', 'expected', 'lithium', 'counterion', 'growth', 'free', 'anions', 'may', 'expected', 'potassium', 'cesium', 'the', 'products', 'anionic', 'lactam', 'polymerizations', 'vary', 'depending', 'upon', 'reaction', 'conditions', 'temperatures', 'upon', 'structures', 'lactams', 'thus', 'ﬁve-', 'six-', 'seven-membered', 'lactams', 'polymerize', 'different', 'temperatures', 'products', 'differ', 'molecular', 'weights', 'for', 'instance', 'a-pyrrolidone', 'polymerizes', 'readily', '30(cid6)c', 'polymer', 'molecular', 'weight', '15000', 'e-caprolactam', 'requires', '178(cid6)c', 'form', 'polymers', 'size', 'larger', '137.', 'a', 'third', 'lactam', 'a-piperidone', 'hard', 'polymerize', 'high', 'molecular', 'weight', 'polymer', 'good', 'yields', '137.', 'the', 'regular', 'molecules', 'form', 'activated', 'anionic', 'polymerizations', 'already', 'shown', '139:', 'activated', 'lactam', 'anions', 'o', 'o', 'c', 'cr', 'n', 'n', 'h2n', 'n', 'n', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'at', 'higher', 'polymerization', 'temperatures', 'however', 'side', 'reactions', 'occur', 'among', 'claisen', 'type', 'condensations', 'they', 'lead', 'two', 'type', 'n-acylated', 'b-keto', 'imide', 'structures', 'take', 'place', 'readily', '200(cid6)c', 'formation', 'imides', 'decreases', 'concentration', 'lactam', 'anions', 'o', 'c', 'o', 'n', 'c', 'ch', 'h', 'h', 'o', 'c', 'n', 'o', 'c', 'o', 'n', 'c', 'h', 'c', 'h', 'o', 'c', 'o', 'c', 'o', 'n', 'c', 'h', 'c', 'h', 'o', 'c', 'o', 'c', 'nh', 'o', 'c', 'o', 'n', 'c', 'h', 'o', 'c', 'c', 'h', '59', 'polymerizations', 'lactams', '295', 'cyclic', 'keto', 'imides', 'well', 'linear', 'ones', 'yield', 'active', 'species', 'acylation', 'lactam', 'anions', 'this', 'results', 'formations', 'growth', 'centers', 'keto', 'amides', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'h', 'chc', 'n', 'ch2', 'n', 'o', 'chc', 'o', 'c', 'h', 'n', 'ch2', 'n', 'o', 'c', 'h', 'o', 'c', 'o', 'o', 'c', 'ch', 'c', 'n', 'ch2', 'n', 'o', 'chc', 'o', 'c', 'h', 'o', 'c', 'n', 'h', 'ch2', 'n', 'o', 'o', 'c', 'ch2', 'n', 'c', 'o', 'c', 'o', 'c', 'cn', 'ch', 'c', 'ch2', 'o', 'c', 'o', 'c', 'o', 'n', 'o', 'c', 'o', 'c', 'o', 'keto', 'amide', 'o', 'c', 'n', 'h', 'ch', 'h', 'keto', 'amide', 'the', 'acidity', 'keto', 'amides', 'a-hydrogen', 'atoms', 'much', 'greater', 'monomers', 'polymer', 'amide', 'groups', 'any', 'formation', 'structures', 'therefore', 'decreases', 'concentration', 'lactam', 'anions', 'side', 'reactions', 'give', 'rise', 'variety', 'irregular', 'structures', 'may', 'present', 'either', 'backbones', 'ends', 'polymer', 'molecules', 'formation', 'branches', 'anionic', 'polymerizations', 'occurs', 'polymerizations', 'e-caprolactam', '140', '141.', 'this', 'lactam', 'higher', 'ones', 'polymerize', 'temperatures', 'greater', '120(cid6)c', 'above', '120(cid6)c', 'b-keto-amide', 'units', 'possibly', 'n-acyl-keto-amide', 'structures', 'preserved', 'they', 'may', 'however', 'potential', 'sites', 'chain', 'splitting', 'later', 'polymer', 'processing', 'takes', 'place', 'much', 'higher', 'temperatures', '142.', 'a', 'new', 'group', 'catalysts', 'metal', 'dialkoxyaluminum', 'hydrides', 'anionic', 'polymerizations', 'lactams', 'reported', 'recently', '143.', 'a', 'different', 'anionic', 'mechanism', 'polymerization', 'apparently', 'takes', 'place', 'when', 'e-caprolactam', 'treated', 'sodium', 'dialkoxyaluminum', 'hydride', 'sodium', 'salt', '2(dialkoxyaluminoxy)-1-azacycloheptane', 'forms', 'h', 'c', 'na', 'n', 'o', 'al', 'or', 'or', 'such', 'compound', 'differs', 'nucleophilicity', 'activated', 'monomers', 'these', 'salts', 'products', 'deprotonation', 'lactam', 'monomers', 'amides', 'followed', 'reduction', 'carbonyl', 'functions', 'it', 'postulated', 'lactam', 'polymerizations', 'monomer', 'addition', 'active', 'species', 'form', 'two', 'steps', '143.', 'in', 'ﬁrst', 'one', 'proton', 'exchanges', 'take', 'place', '296', '5', 'ring-opening', 'polymerizations', 'o', 'c', 'o', 'c', 'na', 'n', 'h', 'n', 'r', 'r', 'o', 'c', 'o', 'c', 'n', 'n', 'h', 'c', 'h', 'c', 'or', 'or', 'or', 'or', 'o', 'al', 'h', 'n', 'c', 'o', 'o', 'al', 'na', 'n', 'c', 'o', 'second', 'step', 'hydrogen', 'dialkoxyaluminum', 'group', 'transferred', 'o', 'c', 'h', 'n', 'r', 'o', 'c', 'o', 'c', 'h', 'n', 'r', 'o', 'c', 'n', 'c', 'na', 'h', 'c', 'n', 'o', 'or', 'o', 'al', 'or', 'ro', 'or', 'al', 'o', 'n', 'c', 'o', 'na', 'n', 'c', 'h', 'the', 'third', 'step', 'propagation', 'it', 'consists', 'addition', 'unit', 'chain', 'end', 'takes', 'place', 'upon', 'reaction', 'terminal', 'acyl', 'lactam', 'sodium', 'salt', '2(dialkoxyaluminoxy)-1-azacycloheptane', 'shown', '143.', '593', 'hydrolytic', 'polymerization', 'lactams', 'this', 'polymerization', 'looked', 'upon', 'special', 'case', 'cationic', 'polymerization', 'it', 'particularly', 'true', 'weak', 'acids', 'added', 'reaction', 'mixture', 'often', 'case', 'industrial', 'practice', 'in', 'hydrolytic', 'polymerizations', 'lactams', 'initiated', 'water', 'hydrolysis–condensation', 'equilibria', 'determine', 'concentrations', 'amine', 'carboxylic', 'acid', 'groups', 'both', 'functional', 'groups', 'participate', 'propagation', 'reactions', 'h', 'n', 'c', 'o', 'h2o', 'nh2', 'c', 'h', 'o', 'n', 'h', 'o', 'c', 'o', 'c', 'h2o', 'oh', 'h2n', 'o', 'c', 'oh', 'oh', 'h2n', 'oc', 'h2o', 'oh', 'c', 'o', 'o', 'c', 'n', 'h', '510', 'polymerization', 'n-carboxy-a-amino', 'acid', 'anhydrides', '297', 'the', 'concentration', 'groups', 'also', 'determines', 'molecular', 'weights', 'ﬁnal', 'products', '118–128.', 'this', 'type', 'equilibria', 'also', 'occurs', 'polymerizations', 'initiated', 'amino', 'acids', 'salts', 'carboxylic', 'acids', 'formed', 'primary', 'secondary', 'amines', 'in', 'hydrolytic', 'polymerizations', 'caprolactam', 'reactions', 'involve', 'percent', 'total', 'lactam', 'molecules', 'present', '144', '145.', 'the', 'predominant', 'propagation', 'reaction', 'step-growth', 'addition', 'lactam', 'molecules', 'end', 'groups', 'it', 'acid', 'catalyzed', '144', '145:', 'nh2', 'o', 'c', 'n', 'h', 'o', 'c', 'nh2', 'n', 'h', 'the', 'exact', 'mechanism', 'addition', 'uncertain', 'it', 'postulated', 'addition', 'steps', 'reactions', 'neutral', 'lactam', 'molecules', 'ammonium', 'cations', '146–148.', 'others', 'felt', 'however', 'lactam', 'molecules', 'add', 'undissociated', 'salts', '149.', 'hydrolytic', 'polymerizations', 'smoothest', 'three', 'types', 'polymerization', 'reactions', 'growing', 'species', 'less', 'activated', 'either', 'cationic', 'anionic', 'polymerizations', 'many', 'commercial', 'processes', 'utilize', 'e-caprolactam', 'polymerizations', 'formation', 'irregular', 'structures', 'however', 'even', 'crosslinked', 'material', 'detected', 'in', 'addition', 'elevated', 'temperatures', 'deamination', 'decarboxylation', 'polycaprolactam', 'take', 'place', '150', 'such', 'reactions', 'result', 'formations', 'ketones', 'secondary', 'amine', 'groups', 'the', 'ketones', 'turn', 'react', 'amines', 'form', 'schiff', 'bases', 'this', 'leads', 'branching', 'crosslinking', 'in', 'industrial', 'preparations', 'water', 'used', 'initiate', 'polymerizations', 'removed', 'conversions', 'reach', '80–90', 'order', 'attain', 'high', 'molecular', 'weights', 'the', 'ﬁnal', 'products', 'contain', '8', 'caprolactam', '2', 'cyclic', 'oligomer', '150.', 'these', 'removed', 'vacuum', 'hot', 'water', 'extraction', 'the', 'material', 'dried', 'vacuum', '100–200(cid6)c', 'reduce', 'moisture', '01.', '510', 'polymerization', 'n-carboxy-a-amino', 'acid', 'anhydrides', 'the', 'polymerizations', 'anhydrides', '(or', 'substituted', 'oxazolidine-25-diones)', 'carried', 'basic', 'catalysts', 'yield', 'polyamides', 'h', 'n', 'n', 'r', 'o', 'o', 'o', 'r', 'o', 'n', 'h', 'n', 'these', 'polymerization', 'reactions', 'important', 'biochemists', 'products', 'poly(a-', 'amino', 'acid)s', 'resemble', 'building', 'blocks', 'naturally', 'occurring', 'polyamides', 'when', 'polymerization', 'initiated', 'strong', 'bases', 'initiating', 'step', 'hydrogen', 'abstraction', 'anhydride', 'base', 'this', 'results', 'formation', 'activated', 'species', 'h', 'n', 'r', 'o', 'o', 'o', 'b', 'bh', 'n', 'r', 'o', 'o', 'o', '298', '5', 'ring-opening', 'polymerizations', 'the', 'reaction', 'proceeds', 'activated', 'mechanism', 'the', 'initiation', 'reaction', 'pictured', 'ballard', 'bamford', '151', 'follows', 'h', 'n', 'r', 'o', 'o', 'o', 'n', 'r', 'o', 'o', 'o', 'r', 'h', 'n', 'r', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'r', 'n', 'o', 'r', 'o', 'o', 'o', 'o', 'o', 'r', 'r', 'n', 'n', 'h', 'h', 'o', 'co2', 'b', 'each', 'propagation', 'step', 'consists', 'addition', 'one', 'unit', 'anhydride', 'accompanying', 'n', 'h', 'r', 'loss', 'carbon', 'dioxide', 'o', 'o', 'o', 'r', 'n', 'h', 'n', 'r', 'o', 'o', 'o', 'nh2', 'o', 'o', 'o', 'o', 'r', 'r', 'n', 'h', '2', 'n', 'h', 'o', 'co2', 'when', 'reaction', 'initiated', 'primary', 'amines', 'ﬁrst', 'step', 'nucleophilic', 'attack', 'amine', 'c3', 'anhydride', '151–153.', 'the', 'carbon', 'dioxide', 'released', 'comes', 'c2', 'carbonyl', 'group', 'the', 'propagation', 'proceeds', 'addition', 'terminal', 'amine', 'groups', 'c3', 'carbonyl', 'groups', 'monomers', '151–153:', 'h', 'n', 'r', 'o', 'o', 'o', 'h2nr', 'rnh', 'cooh', 'o', 'h', 'n', 'r', 'o', 'nh2', 'r', 'rnh', 'co2', '510', 'polymerization', 'n-carboxy-a-amino', 'acid', 'anhydrides', '299', 'the', 'polymerization', 'rate', 'depends', 'upon', 'concentration', 'amine', 'monomer', 'the', 'degree', 'polymerization', 'often', 'always', 'equal', 'ratio', 'monomer', 'amine', '154.', 'it', 'means', 'reaction', 'may', 'similar', 'identical', 'living', 'type', 'polymeriza-', 'tion', 'in', 'addition', 'molecular', 'weight', 'distribution', 'curve', 'may', 'broadened', 'bimodal', 'this', 'may', 'due', 'chemical', 'termination', 'reactions', 'these', 'intramolecular', 'reactions', 'terminal', 'amine', 'group', 'functional', 'group', 'side', 'chain', 'lead', 'formation', 'hydantoic', 'acid', 'end', 'groups', '154.', 'it', 'may', 'also', 'due', 'physical', 'termination', 'precipitation', 'product', 'dialkylzinc', 'initiated', 'polymerizations', 'apparently', 'take', 'place', 'different', 'mechanism', 'the', 'ﬁrst', 'step', 'pictured', 'makino', 'inoue', 'tsuruta', 'hydrogen', 'abstraction', 'dialkylzinc', 'nh', '155.', 'this', 'similar', 'reaction', 'base', 'shown', 'earlier', 'the', 'second', 'stage', 'initiation', 'however', 'reaction', 'two', 'molecules', 'activated', 'carboxyanhydrides', 'formation', 'zinc', 'carbamate', '155:', 'n', '2', 'r', 'o', 'o', 'o', 'r2zn', '-rh', 'zn', 'r', 'n', 'n', 'r', 'o', 'o', 'o', 'o', 'o', 'o', 'r', 'n', 'n', 'r', 'o', 'o', 'zn', 'o', 'o', 'o', 'o', 'the', 'propagation', 'carbonyl', 'addition', 'zinc', 'carbamate', 'activated', 'n-carboxyanhydride', 'form', 'mixed', 'anhydride', 'the', 'mixed', 'anhydride', 'changes', 'amide', 'group', 'elimination', 'carbon', 'dioxide', '155:', 'r', 'n', 'n', 'r', 'zn', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'r', 'n', 'n', 'o', 'o', 'o', 'o', 'zn', 'activated', 'n-carbpxyanhydride', 'o', 'zn', 'activated', 'n-caroxyanhydride', 'activated', 'n-carboxyanhydride', 'portion', '155:', 'r', 'c', 'h', 'n', 'o', 'c', 'c', 'o', 'o', 'r', 'c', 'h', 'n', 'o', 'c', 'c', 'o', 'o', '300', '5', 'ring-opening', 'polymerizations', 'the', 'complete', 'reaction', 'illustrated', 'follows', 'o', 'zn', 'n', 'o', 'zn', 'o', 'o', 'r', 'n', 'o', 'o', 'o', 'r', 'o', 'o', 'r', 'o', 'o', 'o', 'o', 'n', 'n', 'o', 'n', 'r', 'r', 'r', 'o', 'o', 'o', 'o', 'n', 'n', 'o', 'o', 'o', 'r', 'o', 'h', 'n', 'n', 'n', 'h', 'o', 'n', 'o', 'zn', 'r', 'o', 'o', 'r', 'n', 'o', 'o', 'o', 'h20', 'r', 'n', 'o', 'o', 'o', 'o', 'r', 'o', 'h', 'n', 'ho', 'n', 'h', 'o', 'nh2', 'n', 'r', 'organotin', 'compounds', 'also', 'active', 'catalysts', 'polymerizations', 'n-carboxyanhydrides', '156.', 'the', 'mechanism', 'reaction', 'postulated', 'freireich', 'gertner', 'zilkha', '156', 'consist', 'addition', 'organotin', 'compound', 'anhydride', 'formation', 'organotin', 'carbamate', 'it', 'subsequently', 'decarboxylates', 'leaves', 'active', '–n–sn–', 'group', 'adds', 'another', 'molecule', 'n-', 'carboxyanhydride', 'this', 'process', 'repeated', 'every', 'step', 'propagation', '156:', 'h', 'n', 'o', 'o', 'r', 'o', '(c4h9)3', 'och3', 'sn', 'ch3o', 'o', '(c4h9)3', 'sn', 'o', 'h', 'n', 'r', 'o', 'o', 'h', 'n', 'r', '-co2', 'ch3o', '(c4h9)3', 'sn', '511', 'metathesis', 'polymerization', 'alicyclics', '301', 'propagation', 'o', 'h', 'n', 'ch3o', 'r', 'o', 'h', 'n', 'sn', '(c4h9)3', 'r', 'o', 'o', 'o', 'r', 'o', 'r', 'o', 'h', 'n', 'ch3o', 'r', 'o', 'n', 'h', 'sn', 'o', '(c4h9)3', '-co2', 'ch3o', 'o', 'h', 'n', 'r', 'o', 'sn', '(c4h9)3', 'n', 'h', 'when', 'n-carboxyanhydride', 'polymerizations', 'initiated', 'secondary', 'amines', 'small', 'substituents', 'amines', 'act', 'nucleophiles', 'similarly', 'primary', 'amines', '157.', 'secondary', 'amines', 'bulky', 'substituents', 'however', 'produce', 'n-carboxyanhydride', 'anions', 'the', 'true', 'tertiary', 'amines', 'these', 'anions', 'turn', 'initiate', 'polymerizations', 'proceed', '“active', 'monomer', 'mechanism”', 'messman', 'coworkers', 'mechanistic', 'study', 'a-amino', 'acid', 'carboxy', 'anhydride', 'polymeriza-', 'tion', '158', 'they', 'polymerization', 'high', 'vacuum', 'polymerization', 'atmospheric', 'the', 'conclusion', 'work', 'poly(o-benzyl-l-tyrosine)', 'prepared', 'vacuum', 'yields', 'polymer', 'normal', 'amine', 'mechanism', 'minimum', 'termination', 'by', 'contrast', 'reaction', 'carried', 'high', 'vacuum', 'several', 'termination', 'products', '511', 'metathesis', 'polymerization', 'alicyclics', 'ring-opening', 'polymerizations', 'alicyclics', 'ziegler–natta', 'type', 'catalysts', 'resulted', 'general', 'studies', 'oleﬁn', 'metathesis', '158–160.', 'these', 'interesting', 'reactions', 'accomplished', 'aid', 'many', 'catalysts', 'the', 'best', 'results', 'however', 'obtained', 'complex', 'catalysts', 'based', 'tungsten', 'molybdenum', 'halides', 'one', 'good', 'catalyst', 'forms', 'tungsten', 'hexachloride', 'combined', 'right', 'proportions', 'ethylaluminum', 'dichloride', 'ethanol', 'several', 'reaction', 'mechanisms', 'proposed', 'explain', 'course', 'oleﬁn', 'metathesis', 'most', 'evidence', 'supports', 'carbene', 'mechanism', 'involving', 'metal', 'complexes', 'originally', 'suggested', 'harrison', 'chauvin', '160–163.', 'a', 'typical', 'metathesis', 'reaction', 'oleﬁns', 'illustrated', 'follows', 'wcl6', 'c2h5oh', 'c2h5alcl2', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'w', 'w', 'r', 'r', 'r', 'r', 'r', 'r', '302', '5', 'ring-opening', 'polymerizations', 'when', 'reaction', 'applied', 'cyclopentene', 'high', 'molecular', 'weight', 'polymer', 'forms', '164:', 'n', 'tungsten', 'hexachloride', 'apparently', 'also', 'act', 'catalyst', 'without', 'aluminum', 'alkyl', 'in', 'case', 'believed', 'activated', 'oxygen', '166.the', 'propagation', 'reaction', 'based', 'tungsten', 'carbene', 'mechanism', 'shown', 'follows', '162:', 'o2', 'wcl6', 'w', 'o', 'w', 'o', 'c', 'w', 'c', 'o', '2', 'initiation', 'propagation', 'n', 'h', 'c', 'o', '(ch2)3', 'ch', 'w', 'c', 'c', 'w', 'ch', 'chw', 'w', 'ch', 'it', 'signiﬁcant', 'metal', 'carbenes', 'act', 'catalysts', 'reaction', 'thus', 'carbene', '(c6h5)2–', 'c¼w(co)5', 'polymerize', '1-methylcyclobutene', 'yield', 'polymer', 'similar', 'structure', 'cis-polyisoprene', '162:', 'c', 'w(co)5', '2', 'this', 'carbene', 'also', 'yields', 'high', 'molecular', 'weight', 'linear', 'polymers', 'bicyclo420octa-7-ene', 'monomer', '167:', 'h', 'cc', 'h', 'ch', 'ch', 'n', 'n', 'the', 'product', 'also', 'obtained', 'wcl6sn(ch3)4', 'catalyst', 'the', 'molecular', 'weight', 'product', 'however', 'lower', '167.', 'some', 'cyclooleﬁns', 'undergo', 'either', 'regular', 'cationic', 'polymerization', 'metathesis', 'one', 'depending', 'upon', 'catalyst', 'one', 'norbornene', 'derivatives', 'for', 'instance', '5-methylene-', '2-norbornene', 'polymerizes', 'cationic', 'mechanism', '11', 'combination', 'tungsten', 'hexachloride', 'tetraalkyltin', 'a', '14', 'combination', 'tungsten', 'halide', 'either', 'c2h5alcl2', 'mocl5', 'ticl4', 'acidic', 'catalysts', '166', 'yields', 'product', 'the', 'polymer', 'forms', 'repeat', 'units', '511', 'metathesis', 'polymerization', 'alicyclics', '303', 'ch2', 'ch2', 'n', 'on', 'hand', 'metathesis', 'type', 'polymerizations', 'norbornene', 'takes', 'place', 'wcl6–', '(c2h5)3al15', 'wcl6–(ch3)4sn', 'yield', '166:', 'n', 'the', 'product', 'poly13-cyclopentylenevinylene,', 'commercial', 'synthetic', 'specialty', 'rubber', 'trade', 'name', 'norsorex', 'reports', 'literature', 'show', 'may', 'one', 'mechanism', 'termination', '160', '165.', 'one', 'may', 'formation', 'cyclopropane', 'rings', 'this', 'typical', 'reaction', 'carbenes', '160.', 'another', 'one', 'reduction', 'transition', 'metal', 'formation', 'free', 'radicals', '160:', 'ch', 'wclx', 'chcl', 'wclx-1', 'still', 'another', 'way', 'may', 'hydrogen', 'migration', 'carbene', 'complex', '151:', 'ch2', 'ch', 'wclx', 'ch', 'ch2', 'wclx', 'the', 'chemistry', 'metathesis', 'polymerization', 'applied', 'preparation', 'unsaturated', 'polycarbonates', '168.this', 'case', 'acyclic', 'diene', 'metathesis', 'it', 'takes', 'place', 'lewis', 'acid', 'free-catalysts', 'employed', '169.', 'an', 'example', 'one', 'catalyst', 'mochc(ch3)2ph(n-26-', 'c6h3-i-pr2)occh3(cf3)22', 'one', 'interesting', 'point', 'process', 'unconjugated', 'dienes', 'polymerized', 'high', 'molecular', 'weight', 'linear', 'polymers', 'without', 'formations', 'cyclic', 'structures', 'the', 'ring-opening', 'catalysts', 'described', 'show', 'sensitivity', 'towards', 'oxygen', 'moisture', 'catalysts', 'however', 'based', 'ruthenium', 'osmium', 'often', 'referred', 'grubbs', 'catalysts', 'exhibit', 'good', 'stability', 'towards', 'oxygen', 'moisture', '170.', 'examples', 'catalysts', 'rucl3(hydrate)', 'oscl3(hydrate)', 'ruthenium', 'benzylidine', 'catalyst', 'like', '(cyclohexyl3p)2cl2', 'ru¼ch–ch¼cph', '(cyclohexyl3p)2ru¼chph', '170.', 'they', 'illustrated', 'follows', 'cl', 'cl', 'pcy3', 'ru', 'pcy3', 'ph', 'ph', 'cl', 'cl', 'pcy3', 'ru', 'pcy3', 'ph', 'h', 'these', 'materials', 'require', 'small', 'amount', 'solvent', 'activation', 'a', 'second', 'generation', 'grubbs', 'catalyst', 'higher', 'metathesis', 'activity', 'it', 'illustrated', 'follows', 'cl', 'l', 'ru', 'h', 'r', 'cl', 'pr3', 'in', 'ﬁrst', 'generation', 'l', 'pr3', 'shown', 'second', 'generation', 'l', 'n-heterocyclic', 'carbene', 'r', 'cyclohexyl', 'group', 'r0', 'phenyl', 'group', '304', '5', 'ring-opening', 'polymerizations', 'the', 'discrepancy', 'two', 'catalysts', 'elucidated', 'truhlar', 'et', 'al', '170', 'aid', 'computational', 'density', 'functional', 'method', 'named', 'mo6-l', 'they', 'found', 'benzylidine', 'ligand', 'catalysts', 'rotates', 'serves', 'toggle', 'switch', 'trigger', 'metathesis', 'reaction', 'the', 'rotation', 'precedes', 'dissociation', 'grubbs', '1', 'catalyst', 'occurs', 'synchronization', 'dissociation', 'grubbs', '2', 'catalyst', '170.', 'cl', 'l', 'ru', 'h', 'r', 'cl', 'pr3', '-pr3', 'rotation', 'l', 'ru', 'cl', 'cl', 'r', 'h', 'r', 'rotation', 'cl', 'l', 'ru', 'h', 'r', 'when', 'oleﬁn', 'substrate', 'coordinates', 'ruthenium', 'grubbs', '1', 'catalyst', 'must', 'overcome', 'electronic', 'effects', 'stemming', 'rotation', 'barrier', 'lower', 'grubbs', '2', 'following', 'examples', 'polymerization', 'reactions', 'carried', 'grubbs', 'catalysts', '170:', 'n', 'n', 'ch3', 'o', 'o', 'o', 'cl', 'r', 'o', 'n', 'o', 'n', 'o', 'ch3', 'och3', 'och3', 'n', 'och3', 'och3', 'another', 'example', 'using', 'different', 'ruthenium', 'catalyst', 'polymerization', 'cyclohexenyl', 'norbornene', 'form', 'high', 'molecular', 'weight', 'products', '171:', 'rucl3xh2o', 'alcohol', 'heat', 'c6h5cl', 'the', 'versatility', 'catalysts', 'illustrated', 'ring-opening', 'metathesis', 'polymerizations', 'norbornene', 'carried', 'liquid', 'carbon', 'dioxide', 'high', 'pressure', 'using', 'ru', '(h2o)6–(tos)2', 'the', 'product', 'reported', 'cis-ditactic', 'polynorbornene', '172.', 'it', 'noted', 'however', 'stereoselective', 'polymerizations', 'norbornene', 'conﬁned', 'catalysts', '511', 'metathesis', 'polymerization', 'alicyclics', '305', 'for', 'instance', 'polymerization', 'norbornene', 'tungsten', 'based', 'catalyst', 'combined', '(c2h5)3al', 'co-catalyst', 'ph', 'ph', 'cl', 'o', 'o', 'o', 'n', 'w', 'cl', 'reported', 'yielded', '(cid4)78(cid6)c', '92', 'cis', 'polymer', '173.', 'nevertheless', 'grubbs', 'catalysts', 'versatile', 'made', 'great', 'impact', 'polymer', 'chemistry', 'following', 'additional', 'examples', 'use', 'grubbs', 'catalysts', 'one', 'ruthenium', 'catalysts', 'based', 'rucl2(arene)', 'dimers', 'ligands', 'durene', 'p-cymene', 'they', 'formed', 'addition', 'tricyclohexylphosphine', 'activated', '(trimethylsilyl)diazomethane', '174.', 'these', 'catalysts', 'show', 'good', 'functional', 'compatibility', 'preparation', 'variety', 'polyoctenamers', 'epoxide', 'acid', 'ether', 'ester', 'acetal', 'bromine', 'functionalities236', 'the', 'following', 'illustration', 'serves', 'example', '174:', 'p', 'h', 'ru', 'h', 'ru', 'o', 'h', 'p', 'o', 'o', 'o', 'n', 'in', 'addition', 'shown', 'ring-opening', 'metathesis', 'polymerizations', 'exhibit', 'characteristics', 'living', 'polymerizations', 'thus', 'polymerization', 'cyclobutene', 'tungsten', 'catalyst', 'w(ch-t-c4h9)(nar)(o-t-c4h9)2', '(ar', '¼', '26-diisopropyllphenyl)', 'shown', 'ﬁt', 'category', 'living', 'polymerization', 'used', 'form', 'block', 'copolymers', '175', '176.', 'similarly', 'substituted', 'cyclobutanes', 'polymerized', 'living', 'manner', 'using', 'molybdenum', 'catalyst', 'mo(chc(ch3)2ph)(nar)(oc(ch3)2cf3)2', 'combination', 'pphme2', '177.', 'also', 'bicyclo340heptene', 'polymerization', 'found', 'living', 'one', 'ruthenium', 'catalyst', '(pph3)2cl2ru¼chch¼cph2', 'used', '178.', 'ar', '¼', '26-diisopropylphenyl', 'polymers', 'contain', 'pendant', 'carbazole', 'groups', 'exhibit', 'photoconductivity', '(see', 'chap', '10)', 'formation', 'block', 'copolymers', 'pendant', 'carbazole', 'groups', 'reported', 'via', 'living', 'ring-opening', 'metathesis', 'polymerization', 'using', 'ruthenium', 'catalyst', '179.', 'in', 'addition', 'appears', 'ﬁrst', 'example', 'homogeneous', 'living', 'polymerization', 'water', 'reported', 'recently', '180.', 'the', 'reaction', 'carried', 'presence', 'bronsted', 'acid', 'using', 'alkylidine', 'ruthenium', 'complexes', 'water-', 'soluble', 'monomers', 'polymerized', 'quickly', 'quantitatively', 'absence', 'surfactants', 'organic', 'solvents', 'these', 'polymerizations', 'found', 'living', 'however', 'bronsted', 'acid', 'absent', '180.', 'it', 'suggested', 'function', 'acid', 'eliminate', 'hydroxide', 'ions', 'enhance', 'catalyst', 'activity', 'protonating', 'phosphine', 'ligands', '2180.', 'yong', 'swager', '181', 'reported', 'ring-opening', 'metathesis', 'copolymerizations', 'calixarene', 'containing', 'monomers', 'cyclooctene', 'norbornene', 'yield', 'high', 'molecular', 'weight', 'transparent', 'elastic', 'polymers', 'pitet', 'hillmyer', '182', 'combined', 'metathesis', 'ring-opening', 'polymerization', 'cyclic', 'ester', 'ring-', 'opening', 'metathesis', 'polymerization', 'form', 'triblock', 'aaba', 'copolymers', 'cycloocatadiene', 'dl-lactide', '306', '5', 'ring-opening', 'polymerizations', 'aco', 'oac', 'o', 'o', 'o', 'o', '3', 'rop', 'h', 'o', '1', 'rop', '2', 'hydrolysis', 'ho', 'n', 'oh', 'o', 'o', 'o', 'o', 'o', 'h', '2n', 'the', 'product', 'terpolymer', 'soft', 'midblock', 'component', 'hard', 'blocks', 'end', 'as', 'result', 'polymer', 'strong', 'tough', 'material', 'ruthenium', 'catalysts', 'reactive', 'towards', 'oleﬁns', 'as', 'result', 'possible', 'introduce', 'functional', 'groups', 'monomer', 'prior', 'polymerizations', 'this', 'demonstrated', 'hilf', 'kilbinger', '183', 'they', 'demonstrated', 'small', 'ring', 'vinyl', 'lactones', 'carbonates', 'efﬁcient', 'quenchers', 'oleﬁn', 'metathesis', 'polymerization', 'the', 'slow', 'kinetics', 'reaction', 'overcome', 'excess', 'reagent', 'the', 'rapid', 'termination', 'polymerization', 'reaction', 'yields', 'highly', 'functionalized', 'polymers', 'narrow', 'molecular', 'weight', 'distribution', 'ph', 'o', 'o', 'cl', 'cl', 'pcy3', 'ru', 'pcy3', 'ph', 'pcy3', 'ru', 'pcy3', 'o', 'cl', 'cl', 'o', 'ph', 'pcy3', 'ru', 'pcy3', 'o', 'cl', 'cl', 'o', 'o', 'ph', 'o', 'o', 'cl', 'cl', 'pcy3', 'ru', 'pcy3', 'cl', 'cl', 'pcy3', 'ru', 'pcy3', 'o', 'ph', 'metathesis', 'type', 'catalysts', 'also', 'polymerize', 'substituted', 'acetylenes', 'this', 'discussed', 'chap', '10', 'a', 'class', 'oleﬁn', 'metathesis', 'catalysts', 'contains', 'phosphite', 'ligands', 'advantages', 'current', 'catalysts', 'challenging', 'reactions', 'ring-closing', 'metatheses', 'hindered', 'dienes', 'cazin', 'et', 'al', '184', 'modiﬁed', 'existing', 'ruthenium', 'indenylidene', 'metathesis', 'catalyst', 'triisopropyl', 'phosphite', 'groups', 'form', 'cis', 'trans', 'phosphite', 'complexes', 'n', 'n', 'p(or)3', 'ph', 'cl', 'ru', 'cl', '512', 'polymerization', 'cyclic', 'amines', '307', 'the', 'catalysts', 'call', 'cis-', 'trans-caz-1', 'promote', 'difﬁcult', 'tosylamine', 'ring-closing', '100', 'conversion', 'compared', '60', 'achieved', 'existing', 'catalysts', 'and', 'considerably', 'smaller', 'amount', 'caz-1', 'needed', 'promote', 'ring-closing', 'metathesis', 'hindered', 'dienes', 'required', 'current', 'catalysts', 'the', 'caz-1', 'catalysts', 'also', 'show', 'unusually', 'good', 'stability', 'longevity', 'reactions', '184.', 'wathier', 'stoddart', 'grinstaff', 'reported', 'using', 'grubs', 'catalyst', 'form', 'high', 'molecular', 'weight', 'polymers', 'poly(ethyl-5-norbomene-2-carboxylate)', 'poly(methyl-5-oxanorbomene-2-carboxylate)', 'carrying', 'ester', 'functions', 'the', 'preparations', 'illustrated', 'follows', '185:', 'o', 'o', 'r', 'n', 'o', 'r', 'o', 'the', 'authors', 'point', 'synthesizing', 'high', 'molecular', 'weigh', 'polymers', 'aid', 'grubbs’', 'catalyst', 'difﬁcult', 'small', 'changes', 'structure', 'monomer', '(ie,', 'oxa-norbornene', 'vs', 'norbornene)', 'lead', 'drastic', 'change', 'polymerization', 'outcomes', 'on', 'hand', 'polymeri-', 'zation', 'norbornene', 'grubbs’', 'catalyst', 'lead', 'high', 'molecular', 'weight', 'polymers', 'relatively', 'narrow', 'molecular', 'weight', 'distribution', '185.', '512', 'polymerization', 'cyclic', 'amines', 'the', 'cyclic', 'amines', 'imines', '(aziridines)', 'polymerize', 'acidic', 'catalysts', '186.', 'this', 'reaction', 'illustrated', 'follows', 'h2', 'n', 'a', 'h', 'a', 'n', 'nh2', 'h', 'n', 'h', 'a', 'h2', 'n', 'a', 'h', 'n', 'n', 'h2', 'a', 'n', 'h', 'a', 'n', 'nh2', 'n', 'h', 'n', 'h', 'n', 'h', 'a', 'n', 'nh2', 'nh2', 'nh2', 'n', 'h', '308', '5', 'ring-opening', 'polymerizations', 'h', 'n', 'n', 'h', 'a', 'n', 'nh2', 'n', 'n', 'a', 'n', 'h2', 'nh2', 'a', 'n', 'h2', 'nh2', 'the', 'high', 'degree', 'strain', 'three-membered', 'rings', 'causes', 'rapid', 'polymerizations', 'a', 'variety', 'cationic', 'species', 'act', 'efﬁcient', 'catalysts', 'reactions', 'the', 'propagating', 'species', 'iminium', 'ions', 'propagation', 'steps', 'result', 'nucleophilic', 'attacks', 'monomers', 'ions', 'shown', 'branches', 'form', 'due', 'reactions', 'secondary', 'amine', 'groups', 'iminium', 'centers', 'they', 'also', 'result', 'attacks', 'imine', 'end', 'groups', 'inactive', 'polymer', 'chains', 'iminium', 'centers', 'propagating', 'species', 'as', 'reaction', 'progresses', 'slows', 'h', 'n', 'n', 'ha', 'n', 'nh2', 'n', 'n', 'protons', 'become', 'equilibrated', 'various', 'amines', '185.', 'the', 'polymer', 'also', 'extensively', 'cyclized', 'due', 'intramolecular', 'nucleophilic', 'attacks', 'primary', 'secondary', 'amines', 'iminium', 'group', 'the', 'product', 'contains', 'cyclic', 'oligomers', 'polymer', 'molecules', 'large', 'size', 'rings', 'the', 'termination', 'mechanism', 'still', 'fully', 'explained', 'it', 'believed', 'may', 'take', 'place', 'proton', 'abstractions', 'iminium', 'ions', 'counterions', 'nitrogen', 'polymer', 'chains', 'nitrogens', 'monomer', 'units', 'it', 'also', 'suggested', '185', '186', 'backbiting', 'ring', 'expansion', 'terminate', 'reactions', 'such', 'ring', 'expansions', 'result', 'formations', 'relatively', 'unreactive', 'piperazine', 'end', 'groups', 'n', 'h', 'a', 'n', 'h', 'n', 'n', 'h', 'substitution', 'ethylene', 'imine', 'ring', 'hinders', 'polymerization', '185.', 'the', '23', '12', 'substituted', 'aziridines', 'fail', 'polymerize', 'only', 'low', 'molecular', 'weight', 'linear', 'cyclic', 'oligomers', 'form', '1', '2', 'substituted', 'ethylene', 'imines', 'in', 'polymerization', 'secondary', 'cyclic', 'amines', 'formation', 'nonstrained', 'ammonium', 'salt', 'actually', 'termination', 'reaction', 'if', 'rate', 'therefore', 'propagation', 'considerably', 'higher', 'rate', 'termination', 'formation', 'high', 'molecular', 'weight', 'material', 'possible', 'thus', 'ratio', 'kpkt', 'therefore', 'high', 'the', 'rate', 'polymerization', 'written', ']', 'concentration', 'growing', 'chains', 'concentration', 'monomer', 'pn', 'assuming', 'termination', 'ﬁrst-order', 'reaction', 'rate', 'termination', 'expressed', 'rp', '¼', '(cid4)dmdt', '¼', 'kpm½pn', 'þ(cid3)', 'rt', '¼', '(cid4)d½pn', 'þ(cid3)dt', '¼', 'kt½pn', 'þ(cid3)', '513', 'ring-opening', 'polymerizations', 'cyclic', 'sulﬁdes', '309', 'if', 'hand', 'termination', 'result', 'reactions', 'growing', 'chains', 'amino', 'functions', 'polymer', 'second-order', 'reaction', 'm0', 'original', 'monomer', 'concentration', 'rt', '¼', '(cid4)d½pn', 'þ(cid3)dt', '¼', 'kt½pn', 'þ(cid3)ðm0', '(cid4)', 'mþ', '513', 'ring-opening', 'polymerizations', 'cyclic', 'sulﬁdes', 'ring-opening', 'polymerizations', 'cyclic', 'sulﬁdes', 'carried', 'anionic', 'cationic', 'coordi-', 'nated', 'mechanisms', '187–189.', 'these', 'polymerizations', 'easier', 'carry', 'oxygen', 'analogs', 'sulfur–carbon', 'bond', 'polarizable', 'on', 'hand', 'due', 'larger', 'size', 'sulfur', 'atoms', 'rings', 'less', 'strained', 'oxygen', 'compounds', 'as', 'result', 'sulfur', 'analog', 'tetrahydrofuran', 'fails', 'polymerize', 'in', 'cationic', 'polymerizations', 'propagating', 'species', 'sulfonium', 'ions', '189', '190', 'anionic', 'ones', 'sulﬁde', 'anions', 'goethals', 'drigvers', 'proposed', 'following', 'cationic', 'mechanism', 'polymerization', 'dimethyl', 'thiethane', '189:', '1', 'the', 'initiation', 'mechanism', 'triethyl', 'ﬂuoroborate', 'consists', 'alkylation', 'monomer', 'molecule', 'formations', 'cyclic', 'sulfonium', 'ions', 'the', 'reaction', 'occurs', 'instantaneously', 'quantitatively', '(c2h5)3', 'o', 'bf4', 's', 'c2h5', 's', 'bf4', '(c2h5)2o', '2', 'the', 'propagation', 'reaction', 'probably', 'involves', 'nucleophilic', 'attacks', 'a-carbon', 'atom', 'cyclic', 'sulfonium', 'ions', 'sulfur', 'atoms', 'monomer', 'molecules', 's', 'bf4', 's', 's', 'bf4', 's', 'the', 'existence', 'sulfonium', 'ions', 'among', 'propagating', 'species', 'conﬁrmed', 'nmr', 'studies', '191.', '3', 'termination', 'presumed', 'occur', 'formations', 'unreactive', 'sulfonium', 'ions', 'two', 'mechanisms', 'formation', 'sulfonium', 'ions', 'possible', '(1)', 'approaches', 'catalyst’s', 'electron', 'accepting', 'sites', '(2)', 'abstraction', 'hydrides', 'methyl', 'cations', '190:', 's', 'ch3', 'r', 's', 'r', 's', 'r', 'ch4', 's', 's', 'r', 's', 'r', 'r', '310', '5', 'ring-opening', 'polymerizations', 'there', 'indications', '“living”', 'chain-growth', 'mechanism', 'boron', 'triﬂuoride', 'diethyl', 'etherate', 'initiated', 'polymerizations', 'propylene', 'sulﬁde', '192', 'conversions', '5–20.', 'in', 'early', 'stages', 'polymerization', 'molecular', 'weight', 'corresponds', 'calculated', 'typical', '“living”', 'polymers', 'this', 'believed', 'take', 'place', 'formations', 'stable', 'sulfonium', 'ions', 'r', 'r', 'o', 'bf3', 's', 'monomer', 'bf3', 's', 's', 's', 'bf3', 'higher', 'conversions', 'thiirane', 'polymerizations', 'however', 'proceed', 'chain', 'scission', 'transfer', 'mechanism', 'inﬂuence', 'bf3•(c2h5)2o', '192.', 'this', 'indicated', 'change', 'molecular', 'weight', 'distribution', 'bimodal', 'character', 'when', 'reaction', 'complete', 'marked', 'decrease', 'average', 'molecular', 'weight', 'polymer', 'when', 'thietane', 'polymerizes', 'triethyl-oxonium', 'tetraﬂuoroborate', 'initiation', 'methylene', 'chloride', 'reaction', 'terminates', 'limited', 'conversion', '193.', 'this', 'results', 'reactions', 'reactive', 'chain', 'ends', '(cyclic', 'sulfonium', 'salts)', 'sulfur', 'atoms', 'polymer', 'backbone', 'in', 'propylene', 'sulﬁde', 'polymerization', 'however', 'terminations', 'mainly', 'due', 'formations', '12-membered', 'ring', 'sulfonium', 'salts', 'intramolecular', 'reactions', '193.', 'when', 'polymerizations', 'cyclic', 'sulﬁdes', 'carried', 'anionic', 'initiators', 'many', 'side', 'reactions', 'occur', 'on', 'hand', 'common', 'anionic', 'initiators', 'like', 'koh', 'yield', 'optically', 'active', 'polymers', 'optically', 'active', 'propylene', 'sulﬁde', '194.', 'an', 'example', 'side', 'reaction', 'formation', '192', 'propylene', 'sodium', 'sulﬁde', 'sodium', 'naphthalene', 'initiated', 'polymerizations', 'such', 'reactions', 'rapid', 'even', '(cid4)78(cid6)c', 'a', 'similar', 'reaction', 'shown', 'take', 'place', 'ethyllithium', '195:', 's', 'c2h5li', 's', 'c2h5', 'li', 'other', 'side', 'reactions', 'occur', 'butyllithium-initiated', 'polymerizations', 'cleavages', 'polysulﬁdes', '192:', 's', 'lic4h9', 's', 'c4h9', 's', 'li', 's', 'c4h9', 'li', 's', 'high', 'molecular', 'weight', 'polymers', 'prepared', 'ethylene', 'sulﬁde', 'diethylzinc–water', 'catalyst', '196.', 'the', 'polymers', 'form', 'two', 'steps', 'initially', 'insoluble', 'crystalline', 'polymers', 'form', 'room', 'temperature', 'high', 'catalyst', 'monomer', 'ratio', 'these', 'product', 'polymers', 'contain', 'catalyst', 'act', 'seeds', 'polymerizations', 'though', 'ﬁnal', 'polymers', 'insoluble', 'molecular', 'weights', 'estimated', 'high', 'at', 'conversion', '20', 'molecular', 'weights', 'believed', '900000', '196.', 'when', 'diethylzinc', 'prereacted', 'optically', 'active', 'alcohols', 'optically', 'active', 'poly(propylene', 'sulﬁde)s', 'form', '197–199.', 'cadmium', 'salts', 'also', 'effective', 'catalysts', 'polymerization', 'thiiranes', 'the', 'polymers', 'substituted', 'thiiranes', 'high', 'stereoregularity', '514', 'copolymerization', 'cyclic', 'monomers', '311', '514', 'copolymerization', 'cyclic', 'monomers', 'many', 'copolymers', 'prepared', 'cyclic', 'monomers', 'these', 'form', 'ring-opening', 'copolymerizations', 'monomers', 'similar', 'functional', 'groups', 'well', 'different', 'ones', 'some', 'cyclic', 'monomers', 'also', 'copolymerize', 'linear', 'monomers', 'only', 'copolymers', 'cyclic', 'monomers', 'however', 'currently', 'used', 'industrially', 'the', 'composition', 'copolymers', 'depends', 'upon', 'reaction', 'conditions', 'counter', 'ions', 'solvents', 'reaction', 'temperatures', 'the', 'initiator', 'system', 'important', 'cyclic', 'monomers', 'different', 'functional', 'groups', 'copolymerized', 'also', 'different', 'propagating', 'centers', 'involved', 'propagation', 'process', 'copolymerizations', 'difﬁcult', 'achieve', 'prominent', 'among', 'copolymers', 'cyclic', 'ethers', 'interpolymers', 'oxiranes', 'tetrahydrofuran', 'thus', 'ethylene', 'oxide', 'copolymerizes', 'tetrahydrofuran', 'aid', 'boron', 'triﬂuoride–ethylene', 'glycol', 'catalytic', 'system', '200.', 'the', 'resultant', 'copolyether', 'diol', 'contains', 'virtually', 'unsaturation', 'another', 'example', 'copolymer', 'allyl', 'glycidyl', 'ether', 'tetrahydrofuran', 'formed', 'antimony', 'pentachloride', 'catalyst', '201:', 'o', 'o', 'o', 'o', 'x', 'o', 'o', 'in', 'addition', 'liquid', 'copolymers', 'form', '13-dioxolane', 'ethylene', 'oxide', 'boron', 'triﬂuoride', 'used', 'catalyst', '1.', 'also', 'rubbery', 'copolymer', 'forms', 'tetrahydrofuran', '33-diethoxycyclobutane', 'phosphorus', 'pentaﬂuoride', 'catalyst', '202.', 'a', '33-bis(chloromethyl)', 'oxacyclobutane', 'copolymerizes', 'tetrahydrofuran', 'boron', 'ﬂuoride', 'ferric', 'chloride', 'catalysis', 'the', 'product', 'also', 'rubbery', 'material', '1.', 'various', 'copolymers', 'reported', 'trioxane', 'dioxolane', 'glycidyl', 'ethers', '2', '3.', 'for', 'instance', 'copolymer', 'trioxane', 'dioxolane', 'forms', 'sncl4', 'bf3', 'hclo4', 'catalysts', 'the', 'products', 'reaction', 'differ', 'molecular', 'weights', 'molecular', 'weight', 'distributions', 'copolymerizations', 'trioxane', 'phenylglycidyl', 'ether', 'yield', 'random', 'copolymers', '203.', 'different', 'lactones', 'made', 'interpolymerize', '204.', 'the', 'true', 'different', 'lactams', '205–207.', 'the', 'products', 'copolyesters', 'copolyamides', 'respectively', 'more', 'interesting', 'copolymers', 'cyclic', 'monomers', 'different', 'chemical', 'types', 'for', 'instance', 'cyclic', 'phosphite', 'copolymerize', 'lactone', '150(cid6)c', 'presence', 'basic', 'catalysts', '208:', 'o', 'o', 'o', 'o', 'p', 'o', 'p', 'o', 'o', 'o', 'o', 'n', 'aziridine', 'copolymerizes', 'succinimide', 'form', 'crystalline', 'polyamide', 'melts', '300(cid6)c', '209:', '312', '5', 'ring-opening', 'polymerizations', 'when', 'place', 'succinimide', 'cyclic', 'carbonate', 'used', 'high', 'molecular', 'weight', 'polyurethane', 'forms', '210:', 'h', 'n', 'h', 'n', 'o', 'n', 'h', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'n', 'o', 'n', 'h', 'n', 'n', 'h', 'n', 'h', 'n', 'o', 'h', 'n', 'o', 'o', 'o', 'o', 'o', 'terpolymers', 'form', 'epoxides', 'anhydrides', 'tetrahydrofuran', 'oxetane', 'trialkylaluminum', 'catalyst', '211:', 'o', 'o', 'o', 'n', 'o', 'o', 'copolymerizations', 'caprolactone', 'caprolactam', 'various', 'ratios', 'take', 'place', 'lithium', 'tetraalkylaluminate', 'catalyst', '212.', 'the', 'products', 'mainly', 'random', 'copolymers', 'block', 'homopolymers', 'when', 'lactones', 'copolymerize', 'cyclic', 'ethers', 'b-propiolactone', 'tetrahydrofuran', 'early', 'steps', 'reaction', 'cyclic', 'ethers', 'polymerize', 'almost', 'exclusively', '213.', 'this', 'due', 'greater', 'basicity', 'ethers', 'when', 'concentration', 'cyclic', 'ethers', 'depleted', 'equilibrium', 'value', 'consumption', 'decreases', 'markedly', 'polymerizations', 'lactams', 'commence', 'the', 'products', 'block', 'copolymer', '213.', '515', 'spontaneous', 'alternating', 'zwitterion', 'copolymerizations', 'this', 'type', 'copolymerization', 'results', 'spontaneous', 'interactions', 'nucleophilic', 'electrophilic', 'monomers', '(mn', 'me', 'respectively)', 'without', 'additions', 'catalysts', 'zwitterions', 'form', 'process', 'subsequently', 'leads', 'formation', 'polymers', '214–226.', 'the', 'mechanism', 'step-growth', 'polymerization', 'it', 'illustrated', 'follows', 'mn', 'me', 'mn', 'me', 'mn', 'me', 'mn', 'me', 'mn', 'me', 'mn', 'me', 'mn', 'me', 'mn', 'me', 'mn', 'me', 'mn', 'me', 'mn', 'me', '2', 'etc', '515', 'spontaneous', 'alternating', 'zwitterion', 'copolymerizations', '313', 'repeated', 'additions', 'charged', 'species', 'resulting', 'zwitterionic', 'products', 'lead', 'high', 'polymers', 'mn', 'me', 'mn', 'me', 'n', 'the', 'initial', 'zwitterion', 'forms', 'upon', 'combination', 'nucleophilic', 'electrophilic', 'monomer', 'called', 'genetic', 'zwitterion', '214.', 'intramolecular', 'reactions', 'produce', '“macrocycles”', 'mn', 'me', 'mn', 'me', 'n', 'me', 'mn', 'n-1', 'the', 'contribution', 'cyclization', 'reaction', 'however', 'apparently', 'small', '214.', 'a', 'reaction', 'also', 'take', 'place', 'free', 'monomer', 'zwitterion', 'one', 'ionic', 'sites', 'mn', 'me', 'mn', 'me', 'n', 'mn', 'mn', 'me', 'mn', 'me', 'n', 'mn', 'me', 'mn', 'me', 'me', 'n', 'such', 'reactions', 'disturb', 'alternating', 'arrangements', 'units', '–mn–me–', 'products', 'the', 'reactivity', 'monomers', 'determines', 'whether', 'homopropagations', 'occur', 'well', 'alternating', 'propa-', 'gation', 'depends', 'upon', 'dipole–dipole', 'interactions', 'mn', 'me', 'monomers', 'preference', 'ion–', 'dipole', 'reactions', 'ion', 'centers', 'zwitterions', 'monomers', 'homopropagations', '214.', 'an', 'example', 'alternating', 'copolymerization', 'via', 'zwitterion', 'intermediates', 'copolymerization', '2-oxazoline', 'b-propiolactone', 'it', 'takes', 'place', 'solution', 'polar', 'solvent', 'like', 'dimethyl-', 'formamide', 'room', 'temperature', 'period', 'day', 'yield', 'quantitative', 'conversions', '215:', 'a', 'zwitterion', 'forms', 'ﬁrst', 'key', 'intermediate', 'polymerization', 'the', 'onium', 'ring', '2-', 'oxazoline', 'opened', 'nucleophilic', 'attack', 'carboxylate', 'anion', 'carbon', '214:', 'n', 'o', 'o', 'o', 'o', 'o', '2', 'n', 'o', 'n', 'o', 'n', 'h', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'n', 'h', 'o', 'o', 'in', 'reaction', 'number', 'copolymer', 'molecules', 'increases', 'ﬁrst', 'reaches', 'maximum', 'ﬁnally', 'decreases', 'conversion', 'becomes', 'high', '214–226.', 'when', 'concentration', '314', '5', 'ring-opening', 'polymerizations', 'monomers', 'high', 'formation', '“genetic”', 'zwitterions', 'favored', 'as', 'concentration', 'macro-zwitterions', 'becomes', 'high', 'monomer', 'concentration', 'decreases', 'macro', 'zwitterions', 'react', 'preferentially', 'when', 'stoichiometry', 'observed', 'b-propiolactone', 'molecules', 'predominate', 'reactions', 'mixture', 'carboxylate', 'end', 'groups', 'react', 'various', 'ways', 'they', 'react', 'cyclic', 'onium', 'sites', 'zwitterions', 'also', 'free', 'b-propiolactones', 'incorporate', '50', 'propiolactone', 'units', '214.', 'another', 'example', 'copolymerization', '2-oxazoline', 'acrylic', 'acid', 'the', 'reaction', 'carried', 'combining', 'two', 'equimolar', 'quantities', 'heating', 'reaction', 'mixture', '60(cid6)c', 'presence', 'free', 'radical', 'inhibitor', 'such', 'inhibitor', 'p-methoxy', 'phenol', 'the', 'reaction', 'mixture', 'becomes', 'viscous', 'alternating', 'copolymer', 'forms', '218:', 'n', 'o', 'cooh', 'o', 'n', 'o', 'n', 'h', 'o', 'this', 'copolymer', 'identical', 'one', 'obtained', 'reacting', '2-oxazolone', 'b-propiolactone', 'the', 'acrylic', 'acid', 'converted', 'repeat', 'unit', 'one', 'forms', 'ring-opening', 'b-propiolactone', 'shown', 'previous', 'example', 'the', 'suggested', 'reaction', 'mechanism', 'involves', 'nucleophilic', 'attack', 'oxazolone', 'acrylic', 'acid', 'followed', 'proton', 'migration', '214:', 'a', 'similar', 'proton', 'migration', 'takes', 'place', 'copolymerizations', 'acrylamide', 'cyclic', 'imino', 'ethers', 'the', 'proton', 'migration', 'part', 'propagation', 'process', '219.', 'other', 'examples', 'copolymer-', 'izations', 'nucleophilic', 'monomer', '2-phenyl-123-dioxaphospholane', 'electrophilic', 'monomers', '224', '225.', 'here', 'electrophilic', 'monomers', 'either', 'acrylic', 'acid', 'propiolactone', 'identical', 'products', 'obtained', 'reactions', '223:', 'o', 'o', 'p', 'cooh', 'o', 'o', 'p', 'cooh', 'o', 'o', 'p', 'o', 'o', 'o', 'o', 'p', 'coo', '515', 'spontaneous', 'alternating', 'zwitterion', 'copolymerizations', '315', 'o', 'o', 'p', 'o', 'o', 'the', 'opening', 'phosphonium', 'ring', 'requires', 'higher', 'temperatures', '(above', '120(cid6)c)', 'follows', 'pattern', 'arbusov', 'reaction', '224', '226.', 'examples', 'monomers', 'also', 'act', 'nucleophiles', 'reaction', 'p-formyl', 'benzoic', 'acid', '214,', 'acrylamide', '224,', 'ethylene', 'sulfonamide', 'all', 'three', 'react', 'manner', '224:', 'o', 's', 'nh', 'o', 'o', 'o', 'p', 'o', 's', 'o', 'nh2', 'o', 'o', 'p', 'o', 'p', 'o', 'o', 's', 'nh', 'o', 'n', 'it', 'reasonable', 'expect', 'compounds', 'act', 'one', 'time', 'mn', 'monomers', 'times', 'me', 'depending', 'upon', 'comonomer', 'this', 'case', 'salicylyl', 'phenyl', 'phosphonite', '224.', 'in', 'presence', 'bezoquinone', 'behaves', 'mn', 'monomer', 'produces', '11', 'alternating', 'copolymer', 'room', 'temperature', '224:', 'p', 'o', 'o', 'o', 'x', 'x', 'o', 'o', 'x', 'x', 'o', 'p', 'o', 'o', 'x', 'x', 'o', 'n', 'x', 'x', 'x', '¼', 'y', '¼', 'h', 'x', '¼', 'y', '¼', 'cl', 'x', '¼', 'y', '¼', 'ch3', 'x', '¼', 'cl', 'y', '¼', 'cn', 'the', 'reaction', 'called', 'redox', 'copolymerization', 'reaction', '224.', 'the', 'trivalent', 'phosphorus', 'monomer', 'oxidized', 'pentavalent', 'state', 'process', 'polymerization', 'quinone', 'structure', 'key', 'intermediate', 'reduced', 'hydroquinone', 'the', 'phosphonium-phenolate', 'zwitterion', '316', '5', 'ring-opening', 'polymerizations', 'p', 'o', 'o', 'o', 'o', 'o', 'nucleophilic', 'attack', 'phenoxide', 'anion', 'opens', 'phosphonium', 'ring', 'due', 'enhanced', 'electro-', 'philic', 'reactivity', 'mixed', 'anhydride', 'acid', 'structures', '224.', 'salicylyl', 'phenylphosphonite', 'however', 'combination', '2-methyl-2-oxazoline', 'behaves', 'me', 'monomer', '224.', 'terpolymerizations', 'mechanism', 'sequence', 'ordered', '111', 'components', 'also', 'take', 'place', 'the', 'following', 'example', '224:', 'o', 'o', 'p', 'cooch3', 'co2', 'o', 'p', 'o', 'o', 'o', 'o', 'och3', 'n', 'in', 'addition', '21', 'binary', 'copolymerizations', 'also', 'observed', 'following', 'example', 'binary', 'copolymerization', '226:', 'p', 'o', 'o', 'o', '2', 'o', 'c', 'h', 'o', 'c', 'o', 'c', 'h', 'o', 'p', 'o', 'c', 'o', 'n', 'h', '516', 'ring-opening', 'polymerizations', 'free', 'radical', 'mechanism', 'there', 'reports', 'literature', 'ring-opening', 'polymerizations', 'free', 'radical', 'mechanism', 'one', 'polymerization', 'substituted', 'vinyl', 'cyclopropanes', '227.', 'the', 'substituents', 'radical', 'stabilizing', 'structures', 'help', 'free', 'radical', 'ring-opening', 'polymerizations', 'cyclopropane', 'rings', 'this', 'illustrated', 'follows', '516', 'ring-opening', 'polymerizations', 'free', 'radical', 'mechanism', '317', 'n', 'n', 'r', 'n', 'n', 'r', 'r', 'n', 'n', 'n', 'n', 'n', 'a', 'high', 'molecular', 'weight', 'polymer', 'forms', 'in', 'place', 'nitrile', 'groups', 'ester', 'groups', 'utilized', 'well', 'the', 'polymerizations', 'vinyl', 'cylopropanes', 'proceed', 'cationic', 'coordination', 'mechanisms', 'exclusively', 'double', 'bonds', 'free', 'radical', 'polymerizations', 'substituted', 'vinyl', 'cyclopropanes', 'however', 'take', 'place', 'ring-opening', 'polymerizations', 'propane', 'rings', 'in', 'similar', 'manner', 'ring-opening', 'polymerizations', 'ﬁve-membered', 'acetals', 'helped', 'free-', 'radical', 'stabilizing', 'substituents', '228.', 'complete', 'ring-opening', 'polymerizations', 'take', 'place', 'phenyl', 'substituted', 'compounds', 'o', 'o', 'r', 'r', 'o', 'o', 'r', 'o', 'o', 'o', 'o', 'n', 'some', 'heterocyclic', 'monomers', 'like', 'acetals', 'also', 'polymerize', 'free-radical', 'mechanism', '229.', 'particularly', 'interesting', 'almost', 'quantitative', 'ring-opening', 'polymerization', 'seven-', 'membered', 'acetal', '2-methylene-13-dioxepane', '230:', 'o', 'o', 'r', 'o', 'r', 'o', 'ch2', 'r', 'o', 'o', 'o', 'o', 'the', 'product', 'almost', 'pure', 'poly(e-caprolactone)', 'n', '318', '5', 'ring-opening', 'polymerizations', 'cyclic', 'allylic', 'sulﬁdes', 'shown', 'polymerize', 'free-radical', 'ring-opening', 'mechanism', '231.', 'the', 'key', 'structural', 'unit', 'appears', 'responsible', 'facile', 'ring-opening', 'allylic', 'sulﬁde', 'fragment', 'in', 'carbon–sulfur', 'bond', 'cleaved', '231:', 's', 's', 'r', 'n', 's', 's', 's', 'r', 's', 's', 'etc', 'n', 'r', 's', 'n', 's', 's', 'it', 'also', 'reported', 'recently', 'controlled', 'free-radical', 'ring-opening', 'polymerization', 'chain', 'extension', '“living”', 'polymer', 'achieved', 'polymerization', '2-methylene-13-dioxepane', 'presence', '2266-tetramethyl-1-piperidinyloxy', 'free', 'radical', '(tempo)', '232', 'the', 'reaction', 'initiated', 'di-tert-butyl', 'peroxide', '125(cid6)c', 'at', 'high', 'concentrations', 'piperidinyloxy', 'radical', 'polydispersity', 'product', '12', '232', '517', 'thermodynamics', 'ring-opening', 'polymerization', 'the', 'stability', 'ring', 'structure', 'well', 'stability', 'resultant', 'linear', 'polymer', 'determines', 'polymerizability', 'cyclic', 'monomers', 'thermodynamic', 'factors', 'therefore', 'paramount', 'impor-', 'tance', 'ring-opening', 'polymerizations', '233.', 'actually', 'polymerization', 'many', 'bond-strained', 'ring', 'monomers', 'favored', 'thermodynamically', 'thus', 'instance', 'dh', 'three-membered', 'cycloalkanes', '(cid4)1130', 'kjmole', 'ds', '(cid4)691', 'jmole', '(cid6)c', 'it', 'shown', '(sawada)', 'three-', 'four-membered', 'ring', 'structures', 'change', 'enthalpy', 'major', 'factor', 'determining', 'df', 'change', 'free', 'energy', 'for', 'three-membered', 'cycloalkanes', 'df', '(cid4)920', 'kjmole', 'four-', 'membered', 'ones', '(cid4)900', 'kjmole', 'the', 'entropy', 'change', 'ds', 'major', 'factor', 'polymerization', 'ﬁve-membered', 'cyclic', 'monomers', 'the', 'six-membered', 'ring', 'monomers', 'relatively', 'strain', 'free', 'hard', 'polymerize', 'an', 'exception', 'trioxane', 'whose', 'dh', 'close', 'zero', 'on', 'hand', 'enthalpy', 'entropy', 'factors', 'contribute', 'equally', 'free', 'energy', 'change', 'larger', 'rings', 'this', 'means', 'increases', 'temperature', 'df', 'becomes', 'less', 'less', 'negative', 'certain', 'temperatures', 'large', 'cyclic', 'monomers', 'polymerize', 'the', 'transanular', 'strain', 'seven-', 'eight-membered', 'rings', 'contributes', 'polymerizability', 'presence', 'substituents', 'cyclic', 'monomers', 'negative', 'effect', 'thermodynamic', 'feasibility', 'polymerize', 'on', 'hand', 'thermodynamic', 'feasibility', 'alone', 'determine', 'whether', 'cyclic', 'monomer', 'polymerize', 'the', 'entropy', 'changes', 'show', 'much', 'dependence', 'angle', 'strain', 'they', 'susceptible', 'however', 'conﬁgurational', 'inﬂuences', 'sawaada', '233', 'writes', 'entropy', 'change', 'polymerization', 'function', 'probability', 'ring', 'closure', 'dsp', '¼', '(cid4)b', 'ln', 'p', '(cid4)', 'p', 'probability', 'ring', 'closure', 'b', 'constants', 'the', 'probability', 'ring', 'closure', 'chain', 'n', 'repeating', 'units', 'taken', 'function', 'probability', 'chain', 'ends', 'come', 'together', 'this', 'probability', 'usually', 'expressed', 'radius', 'gyrations', '(r2)', 'root', 'square', 'review', 'questions', '319', 'distance', 'end', 'end', 'the', 'entropy', 'change', 'three-membered', 'rings', 'would', 'large', 'negative', 'value', 'for', 'larger', 'rings', 'negative', 'value', 'would', 'less', 'end', 'ends', 'would', 'apart', 'statistical', 'mechanics', 'treatment', 'shown', 'entropy', 'change', 'ring', 'closure', '223:', 'dsr', '¼', 'r', 'lnfpv2xvsng', 'p', 'probability', 'ring', 'closure', 'fraction', 'chain', 'ends', 'come', 'together', 'close', 'form', 'ring', 'structures', 'v', 'total', 'volume', 'system', 'vs', 'volume', 'constrained', 'skeletal', 'atom', 'prior', 'bond', 'breaking', 'x', 'number', 'monomer', 'units', 'ring', 'n', 'avogadro’s', 'number', '1', 'are', 'mechanisms', 'ring-opening', 'polymerizations', 'cyclic', 'monomers', 'chain-growth', 'step-', 'growth', 'reactions', 'explain', '1', 'write', 'rate', 'expression', 'propagation', 'ring-opening', 'polymerizations', 'equilibrium', 'propagation', 'depropagation', '2', 'write', 'kinetic', 'expression', 'total', 'concentration', 'monomer', 'segments', 'incorporated', 'review', 'questions', 'section', '51', 'section', '52', 'polymer', 'section', '53', '1', 'oxiranes', 'polymerized', 'three', 'different', 'mechanisms', 'what', 'explain', '2', 'write', 'chemical', 'reactions', 'mechanism', 'polymerization', 'ethylene', 'oxide', 'aid', 'stannic', 'chloride', 'does', 'high', 'molecular', 'weight', 'polymer', 'form', 'if', 'explain', '3', 'write', 'chemical', 'reactions', 'mechanism', 'polymerization', 'propylene', 'oxide', 'boron', 'triﬂuoride–water', '4', 'describe', 'mechanism', 'write', 'chemical', 'reactions', 'ring-opening', 'polymerizations', 'oxiranes', 'potassium', 'hydroxide', 'in', 'polymerization', 'propylene', 'oxide', 'koh', 'type', 'tacticity', 'polymer', 'forms', 'explain', '5', 'describe', 'mechanism', 'write', 'chemical', 'equations', 'coordinated', 'anionic', 'polymerizations', 'propylene', 'oxide', 'ferric', 'chloride', 'diethylzinc–water', 'show', 'reaction', 'mechanism', '6', 'discuss', 'general', 'characteristics', 'steric', 'control', 'polymerizations', 'oxiranes', '7', 'explain', 'mechanism', 'postulated', 'tsuruta', 'steric', 'control', 'polymerizations', 'oxiranes', 'aid', 'organozinc', 'compounds', 'giving', 'structure', 'catalyst', 'mode', 'monomer', 'insertion', 'mode', 'ring', 'opening', '5', 'ring-opening', 'polymerizations', '1', 'describe', 'initiation', 'process', 'polymerizations', 'oxetanes', 'including', 'initiators', 'reaction', '2', 'describe', 'mechanism', 'propagation', 'polymerizations', 'oxetanes', '1', 'discuss', 'including', 'chemical', 'equations', 'initiation', 'reactions', 'tetrahydrofuran', 'polymerization', 'including', 'mechanism', 'various', 'initiators', '2', 'discuss', 'propagation', 'reaction', 'polymerization', 'tetrahydrofuran', '3', 'when', 'ionic', 'covalent', 'species', 'present', 'polymerization', 'tetrahydrofuran', 'explain', 'conditions', 'cause', 'formation', 'species', 'draw', 'structures', '4', 'describe', 'termination', 'reaction', 'tetrahydrofuran', 'polymerization', 'including', 'living', '1', 'how', 'rates', 'oxepane', 'polymerization', 'compare', 'oxetane', 'tetrahydrofuran', 'what', 'affects', 'rates', '1', 'how', 'cationic', 'polymerizations', 'cyclic', 'acetals', 'differ', 'cyclic', '2', 'what', 'initiators', 'effective', 'polymerizations', 'trioxane', 'discuss', 'polymerizations', 'differ-', '3', 'describe', 'typical', 'polymerization', 'conditions', 'trioxane', '4', 'explain', 'proposed', 'reaction', 'mechanisms', 'polymerization', 'trioxane', 'including', 'coordi-', 'nated', 'mechanism', 'polymerizations', 'molybdenum', 'acetylacetonate', 'illustrate', 'chemi-', 'cal', 'structures', '5', 'discuss', 'polymerization', 'dioxalane', 'showing', 'mechanism', 'initiation', 'propagation', '6', 'how', 'polymer', 'copolymer', 'form', 'side', 'side', 'boron', 'triﬂuoride', 'initiated', 'terminations', 'different', 'initiators', 'polymerizations', 'dioxepane', '7', 'what', 'type', 'structures', 'obtained', 'ring-opening', 'polymerizations', 'trioxocane', 'show', '320', 'section', '54', 'mechanism', 'section', '55', 'polymerization', 'section', '56', 'section', '57', 'ethers', 'ent', 'initiators', 'explain', 'section', '58', '1', 'describe', 'cationic', 'polymerization', 'lactones', 'showing', 'initiation', 'propagation', 'processes', '2', 'repeat', 'question', 'one', 'anionic', 'polymerization', 'review', 'questions', '321', '3', 'describe', 'coordination', 'polymerization', 'lactones', '4', 'what', 'instances', '“living”', 'polymerizations', 'cyclic', 'lactones', 'type', 'catalysts', 'yield', 'type', 'polymerization', 'describe', 'give', 'examples', 'section', '59', 'polymerization', 'inﬂuences', 'reaction', 'hydrides', '1', 'what', 'three', 'mechanisms', 'polymerization', 'lactams', '2', 'describe', 'catalysts', 'useful', 'cationic', 'polymerizations', 'lactams', 'mechanism', '3', 'show', 'amidine', 'salts', 'form', 'cationic', 'polymerizations', 'lactams', 'explain', '4', 'discuss', 'anionic', 'polymerization', 'lactams', 'compare', 'cationic', 'one', '5', 'what', 'meant', 'lactomolytic', 'propagation', 'explain', '6', 'describe', 'proposed', 'mechanism', 'polymerizations', 'lactams', 'dialkoxyaluminum', '7', 'describe', 'hydrolytic', 'polymerization', 'lactams', '8', 'compare', 'cationic', 'anionic', 'hydrolytic', 'polymerizations', 'lactams', 'writing', 'three', 'modes', 'polymerization', 'side', 'side', 'discuss', 'show', 'side', 'reactions', 'take', 'place', 'one', '1', 'discuss', 'polymerization', 'n-carboxy-a-amino', 'acid', 'anhydrides', '1', 'what', 'metathesis', 'polymerization', 'explain', 'mechanism', 'show', 'reaction', '2', 'describe', 'metathesis', 'polymerization', 'methyl', 'cyclobutene', 'showing', 'mechanisms', 'initiation', '3', 'describe', '“living”', 'metathesis', 'polymerization', 'what', 'types', 'catalysts', 'useful', 'section', '510', 'section', '511', 'disubstitued', 'oleﬁn', 'propagation', 'polymerizations', 'section', '512', 'section', '513', '1', 'describe', 'polymerization', 'aziridines', 'showing', 'initiation', 'propagation', 'processes', '1', 'explain', 'three', 'mechanisms', 'cyclic', 'sulﬁdes', 'polymerized', 'describe', '2', 'describe', 'initiation', 'propagation', 'reactions', 'cationic', 'polymerizations', 'cyclic', 'sulﬁdes', '322', '5', 'ring-opening', 'polymerizations', '3', 'describe', 'termination', 'reaction', 'cyclic', 'sulﬁdes', 'cationic', 'polymerizations', '4', 'what', 'type', 'side', 'reactions', 'occur', 'anionic', 'polymerizations', 'cyclic', 'sulﬁdes', '1', 'discuss', 'copolymerizations', 'cyclic', 'monomers', 'giving', 'several', 'examples', '1', 'how', 'spontaneous', 'zwitterion', 'copolymerization', 'occur', 'explain', '2', 'what', 'meant', 'genetic', 'zwitterion', '3', 'give', 'several', 'examples', 'zwitterion', 'copolymerization', 'section', '514', 'section', '515', 'section', '516', 'recommended', 'reading', 'references', '1', 'explain', 'ring-opening', 'polymerizations', 'free-radical', 'mechanism', 'giving', 'two', 'examples', 'kj', 'ivin', 'jc', 'mol', 'oleﬁn', 'metathesis', 'metathesis', 'polymerization', 'academic', 'press', 'san', 'diego', '1997', '1', 'm', 'hitota', 'h', 'fukuda', 'makromol', 'chem,', '188', '2259', '(1987)', 'see', 'alsof', 'afsahar-taroni', 'm', 'scheer', 'p', 'rempp', 'e', 'fanata', 'makromol', 'chem,', '1978', '179', '849', '2', 'j', 'furukawa', 't', 'saegusa', 'polymerization', 'aldehydes', 'oxides', 'interscience', 'wiley', 'new', 'york', '1963', 's', 'penczek', 'p', 'kubisa', '“cationic', 'ring', 'opening', 'polymerization', 'ethers”', 'chapt', '48', '“comprehensive', 'polymer', 'science”', 'vol', '3', '(gc', 'eastmond', 'a', 'ledwith', 's', 'russo', 'p', 'sigwalt', 'eds)', 'pergamon', 'press', 'london', '1989', '3', 'rd', 'colclough', 'g', 'gee', 'wce', 'higginson', 'jb', 'jackson', 'm', 'litt', 'j', 'polymer', 'sci,1959', '34', '171', '4', 'am', 'eastham', 'fortschr', 'hochpolymer', '-', 'forsch1961,', '2', '18', '5', 'dj', 'worsford', 'a', 'm', 'eastham', 'j', 'am', 'chem', 'soc,', '1957', '79', '897', '6', 'gt', 'merall', 'ga', 'latremouille', 'am', 'eastham', 'j', 'am', 'chem', 'soc,', '1960', '82', '120', '7', 's', 'searles', 'm', 'tamres', 'j', 'am', 'chem', 'soc,', '1951', '73', '3704', '8', 'e', 'j', 'vandenberg', 'j', 'am', 'chem', 'soc1961,', '83', '3538', 'j', 'polymer', 'sci', '1964', 'b2', '1085', 'j', 'polymer', 'sci,', '1969', 'a-', '1', '7', '525', '9', 'cc', 'price', 'r', 'spector', 'j', 'am', 'chem', 'soc,', '1966', '88', '4171', '10', 'tl', 'cairns', 'rm', 'joyce', 'jr,', 'us', 'patent', '2445912', '(1948)', '11', 'ej', 'vandenberg', 'j', 'polymer', 'sci,', '1960', '47', '489', '12', 'b', 'aydogan', 'g', 'e', 'gunbas', 'a', 'durmus', 'l', 'toppare', 'y', 'yagci', 'macromolecules', '2010', '43', '101', '13', 'e', 'roithner', 'monatsch', 'chem,', '15', '679', '(1894)', 'j', 'chem', 'soc,', '68', '(1)', '319', '(1895)', '14', 's', 'perry', 'h', 'hibbert', 'j', 'am', 'chem', 'soc,', '62', '2599', '(1940)', 's', 'perry', 'h', 'hibbert', 'can', 'j', 'res,', '1953', '8', '102', '15', 'j', 'raynaud', 'y', 'gnanou', 'd', 'taton', 'am', 'chem', 'soc', 'polymer', 'preprints', '2009', '50', '(2)', '216', 'references', '737', '323', '16', 'he', 'pruitt', 'j', 'b', 'baggett', 'uspatent', '27061181', '(1955)', 're', 'parker', 'ns', 'isaacs', 'chem', 'rev,', '1959', '59', '17', 'cc', 'price', 'm', 'osgan', 'j', 'am', 'chem', 'soc,', '1956', '78', '4787', '(1956)', 'j', 'polymer', 'sci,1959', '34', '153', 'cc', 'price', '18', 'r', 'nomura', 'h', 'hisada', 'a', 'ninagawa', 'h', 'matsuda', 'makromol', 'chem,', 'rapid', 'commun,', '1980', '1', '135', 'ibid,', 'dd', 'carmelite', 'j', 'am', 'chem', 'soc1966,', '88', '4039', 'makromol', 'chem,', 'rapid', 'commun,', '1980', '1', '705', '19', 'y', 'watanabe', 't', 'yasuda', 't', 'aida', 's', 'inoue', 'macromolecules1992', '25', '1396', '20', 'r', 'd', 'colclough', 'g', 'gee', 'ah', 'jagger', 'j', 'polymer', 'sci,', '1960', '48', '273', '21', 'g', 'gee', 'trans', 'j', 'plastics', 'inst1960,', '28', '89', '22', 'ceh', 'bawn', 'a', 'ledwith', 'quarterly', 'reviews1962', '16', '(4)', '361', '23', 'cc', 'price', 'chapt', '1', 'polyethers', 'ej', 'vandenberg', 'ed,', 'am', 'chem', 'soc', 'symposium', 'series', '6', '1975', '24', 'rd', 'colclough', 'k', 'wilkinson', 'j', 'polymer', 'sci,', '1964', 'c4', '311', '25', 'j', 'furukawa', 'polymer', '1962', '3', '487', '26', 't', 'tsuruta', 's', 'inoue', 'm', 'ishimori', 'n', 'yoshida', 'j', 'polymer', 'sci,', '1964', 'c4', '267', '27', 't', 'tsuruta', 's', 'inoue', 'n', 'yoshida', 'y', 'yokota', 'makromol', 'chem,', '1965', '81', '191', '(1965)', 't', 'tsuruta', 'j', 'polymer', 'sci1972,', 'd', '180', '28', 'a', 'kassamaly', 'm', 'sepulchre', 'n', 'spassky', 'polymer', 'bull,', '1988', '19', '119', '(1988)', 'a', 'le', 'borgne', 'n', 'spassky', 'cl', 'jun', 'a', 'momtaz', 'makromol', 'chem,', '1988', '189', '637', '29', 'cc', 'price', 'r', 'spector', 'j', 'am', 'chem', 'soc1965,', '87', '2069', '30', 'cc', 'priceand', 'al', 'tamola', 'j', 'polymer', 'sci,', '1967', 'a-15', '497', '31', 'cc', 'price', 'mk', 'akkapediti', 'bt', 'debona', 'bc', 'furie', 'j', 'am', 'chem', 'soc1972,', '94', '3964', '32', 't', 'saegusa', 'y', 'hashimoto', 's', 'matsumoto', 'macromolecules', '1971', '4', '1', '33', 'j', 'b', 'rose', 'j', 'chem', 'soc,', '1956', '542', 's', 'penczek', 'p', 'kubisa', '“cationic', 'ring-opening', 'polymerizations”', 'chapters', '48', '49', 'comprehensive', 'polymer', 'science', 'vol', '3', 'gc', 'eastmond', 'a', 'ledwith', 's', 'russo', 'p', 'sigwalt', 'eds,', 'pergamon', 'press', 'london', '1989', '34', 'cc', 'price', 'accounts', 'chemical', 'research', '7', '294', '(1974)', '35', 'p', 'dreyfuss', 'mp', 'dreyfuss', '“oxetane', 'polymers”', 'pp', '653-670', 'encyclopedia', 'of', 'polymer', 'science', 'engineering', 'vol', '10', '2nd', 'ed', '(hf', 'mark', 'nm', 'bikales', 'cg', 'overberger', 'g', 'menges', 'eds)', 'wiley-', 'interscience', 'new', 'york', '1987', '36', 't', 'tsuruta', 'j', 'polymer', 'sci,', 'polymer', 'symp,', '67', '73', '(1980)', 't', 'tsuruta', 'y', 'kawakami', 'anionic', 'ring-', 'opening', 'polymerization', 'stereospeciﬁcity', 'epoxides', 'episulﬁdes', 'lactones', 'chapt', '33', 'comprehensive', 'polymer', 'science', 'vol', '3', '(gc', 'eastmond', 'a', 'ledwith', 's', 'russo', 'p', 'sigwalt', 'eds)', 'pergamon', 'press', 'london', '1989', '37', 't', 'saegusa', 'h', 'fujii', 's', 'kabayashi', 'h', 'ando', 'r', 'kawase', 'macromolecules1973', '6', '26', '38', 'pe', 'black', 'dj', 'worsford', 'can', 'j', 'chem,', '1976', '54', '3326', '39', 'ej', 'goethals', 'adv', 'polymer', 'sci,', '1977', '23', '101', '40', 'ej', 'goethals', 'makromol', 'chem,', '1978', '179', '1681', '41', 'm', 'bucquoye', 'e', 'goethals', 'eur', 'polymer', 'j1978,', '14', '323', '42', 't', 'saegusa', 'chapt', '1', 'polymerization', 'heterocyclics', '(o', 'vogl', 'j', 'furukawa', 'eds)', 'dekker', 'new', 'york', '1973', '43', 'j', 'kops', 's', 'hvilsted', 'h', 'spauggaerd', 'macromolecules', '1980', '13', '1058', '44', 'm', 'bucquoye', 'e', 'goethals', 'eur', 'polymer', 'j,', '1978', '14', '323', '45', 'ej', 'vandenberg', 'j', 'polymer', 'sci1969,', 'a-17', '525', '46', 'e', 'j', 'vandenberg', 'a', 'e', 'robinson', 'chapt', '7', 'polyethers', '(e', 'j', 'vandenberg', 'ed)', 'amchemsoc', 'symposium', 'series', '6', '1975', '47', 'd', 'takeuchi', 't', 'aida', 'macromolecules', '1966', '29', '8096', '(1996)', '48', 'jv', 'crivello', 'am', 'chem', 'soc', 'polymer', 'preprints', '2006', '47(1)', '208', '49', 's', 'inoue', 't', 'aida', 'cyclic', 'ethers', 'chapt', '4', 'ring', 'opening', 'polymerizations', 'vol', '1', '(kj', 'ivin', 't', 'saegusa', 'eds)', 'elsevier', 'london', '1984', '50', 'k', 'hamann', 'angew', 'chem,', '1951', '63', '231', '(19)', 'h', 'meerwein', 'e', 'kroning', 'j', 'prakt', 'chem,1937', '2', '147', '25', 'h', 'meerwein', 'd', 'delfs', 'h', 'morschel', 'angew', 'chem,', '1960', '72', '927', '51', 'ceh', 'bawn', 'c', 'fitzsimmons', 'a', 'ledwith', 'proc', 'chem', 'soc,', '1964', '391', '52', 'a', 'ledwith', 'adv', 'chem', 'series', '1969', '91', '317', '53', 'i', 'kuntz', 'j', 'polymer', 'sci,', '1967', 'a-15', '193', '54', 'jv', 'crivello', 'jhw', 'lam', 'j', 'polymer', 'chem,', 'polymer', 'chem', 'ed,', '1981', '19', '539', '55', 't', 'saegusa', 'makromol', 'chem1974,', '175', '1199', '56', 'k', 'matyjaszewski', 'st', 'penczek', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1974', '12', '1905', '57', 'st', 'penszek', 'makromol', 'chem,', 'suppl,', '1979', '3', '17', '58', 'am', 'boyle', 'k', 'matyjaszewski', 'st', 'penczek', 'macromolecules', '1977', '10', '269', '59', 'k', 'matyjaszewski', 't', 'diem', 'st', 'penczek', 'makromol', 'chem1979,', '180', '1917', '60', 'p', 'dreyfuss', 'm', 'p', 'dreyfuss', 'adv', 'chem', 'series1969', '91', '335', '324', '5', 'ring-opening', 'polymerizations', '61', 'kobayashi', 'h', 'danda', 't', 'saegusa', 'macromolecules1974', '7', '415', '62', 'im', 'robinson', 'g', 'pruckmayr', 'macromolecules', '1979', '12', '1043', '63', 't', 'saegusa', 's', 'matsumoto', 'j', 'polymer', 'sci1968,', 'a-16', '459', '64', 't', 'saegusa', 's', 'matsumoto', 'macromolecules1968', '1', '442', '65', 'p', 'dreyfuss', 'j', 'macroml', 'sci-chem,', '1975', 'a7', '(7)', '1361', '66', 's', 'kabayashi', 'h', 'danda', 't', 'sayegusa', 'macromolecules', '1974', '7', '415', '67', 'g', 'pruckmayr', 'tk', 'wu', 'macromolecules', '1978', '11', '265', '68', 'ej', 'vandenberg', 'j', 'polymer', 'sci,', '1969', 'a-17', '525', '69', 's', 'penczek', 'p', 'kubisa', 'chapt', '5', 'ring', 'opening', 'polymerizations', '(t', 'saegusa', 'e', 'goethals', 'eds)', 'am', '70', 'k', 'wessermel', 'e', 'fischer', 'k', 'gutweiler', 'hd', 'hermann', 'h', 'chedron', 'ang', 'chem,', 'intern', 'edit1967,', '6', '(6)', 'chem', 'soc', 'symposium', 'series', '59', '1977', '526', '71', 'mb', 'price', 'fb', 'mcandrew', 'j', 'macromol', 'sci-chem,', '1967', 'a1', '(2)', '231', '72', 'm', 'iguchi', 'br', 'polymer', 'j,', '1973', '5', '195', '73', 'g', 'l', 'collins', 'r', 'k', 'greene', 'f', 'm', 'bernardinelli', 'w', 'v', 'garruto', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1979', '17', '667', '74', 't', 'miki', 't', 'higashimura', 's', 'okamura', 'j', 'polymer', 'sci1967,', 'a-15', '95', '75', 'cd', 'kennedy', 'wr', 'sorenson', 'gg', 'mcclafﬁn', 'am', 'chem', 'soc', 'polymer', 'preprints', '1966', '7', '667', '76', 'v', 'jaack', 'w', 'kern', 'makromol', 'chem1963,', '62', '1', '77', 'k', 'weissermel', 'e', 'fischer', 'k', 'gutweiler', 'kunststoffe1964', '54', '410', '78', 'm', 'ikeda', 'j', 'chem', 'soc', 'japan', 'ind', 'chem', 'sect', '1962', '65', '691', '(1962)', '(from', 'ref', '1,', 'p', '245)', '79', 'y', 'yamashita', 'm', 'okada', 't', 'suyama', 'makromol', 'chem,', '1968', '111', '277', '80', 'ba', 'rosenberg', 'ba', 'kamarov', 'ti', 'ponomareva', 'ns', 'enikolopyan', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '81', 's', 'penczek', 'p', 'kubisa', 'chapt', '5', 'ring', 'opening', 'polymerizations', '(t', 'saegusa', 'e', 'goethals', 'eds)', 'am', '1973', '11', '1', 'chem', 'soc', 'symposium', 'series', '1977', '59', '82', 'jw', 'hill', 'w', 'h', 'carothers', 'j', 'am', 'chem', 'soc,', '1935', '57', '925', '83', 'aa', 'strepikheev', 'a', 'v', 'volokhima', 'dokl', 'akad', 'nauk', 'sssr', '1954', '99', '407', '84', 'rc', 'schulz', 'k', 'albrecht', 'c', 'rentsch', 'q', 'v', 'tran', 'thi', 'chapt', '6', 'ring', 'opening', 'polymerization', '(t', 'saegusa', 'e', 'goethals', 'eds)', 'amchemsoc', 'symposium', 'series', '59', '1977', '85', 'd', 'weichert', 'j', 'polymer', 'sci,', '1967', 'c16', '2701', '86', 'k', 'satome', 'i', 'hodiza', 'makromol', 'chem1965,', '82', '41', '87', 'yn', 'sazanov', 'usp', 'khim,', '1968', '37', '1084', '88', 'eb', 'ludvig', 'dk', 'khomyakov', 'gs', 'sanina', 'j', 'polymer', 'sci,', 'symposia', '42', '1973', '289', 'e', 'b', 'ludvig', 'b', 'g', 'belenkaya', 'a', 'k', 'khomyakov', 'eur', 'polymer', 'j,', '1981', '17', '1097', '89', 'k', 'makiguchi', 't', 'satoh', 't', 'kakuchi', 'macromolecules', '2011', '44', '(7)', '1999', '90', 'd', 'bourissou', 'b', 'martin-vaca', 'a', 'dumitrescu', 'm', 'graullier', 'f', 'lacombe', 'macromolecules', '2005', '38', '9993', '91', 's', 'penszek', 'j', 'polymer', 'sci', 'part', 'a', 'polym', 'chem,', '2000', '381919', '92', 'l', 'zang', 'f', 'nederberg', 'rc', 'pratt', 'rm', 'waymouth', 'jl', 'hedrick', 'cw', 'wade', 'macromolecules', '2007', '40', '4154', '93', 'a', 'leborgne', 'd', 'grenier', 're', 'prud’homme', 'n', 'spassky', 'eur', 'polymer', 'j,', '1981', '17', '1103', '94', 'rh', 'young', 'm', 'matzner', 'la', 'pilato', 'chapt', '11', 'ring-opening', 'polymerizations', '(t', 'saegusa', 'e', 'goethals', 'eds)', 'am', 'chem', 'soc', 'symposium', 'series', '59', '1977', '95', 'gl', 'brode', 'jv', 'koleske', 'chapt', '7', 'polymerization', 'heterocyclics', '(d', 'vogl', 'j', 'furukawa', 'eds)', 'dekker', 'new', 'york', '1973', 'j', 'g', 'noltes', 'f', 'verbeek', 'h', 'g', 'j', 'overmars', 'j', 'boersma', 'j', 'organometal', 'chem,', '1970', '24', '257', '96', 'e', 'j', 'shin', 'w', 'jeong', 'd', 'a', 'culkin', 'j', 'l', 'hedrick', 'r', 'm', 'waymouth', 'am', 'chem', 'soc', 'polymer', 'preprints', '2009', '50', '(1)', '60', '97', 'ph', 'teyssie', 'jp', 'bioul', 'a', 'hamitou', 'j', 'heuschen', 'l', 'hocks', 'r', 'jerome', 't', 'ouhadi', 'chapt', '12', 'ring', 'opening', 'polymerizations', '(t', 'saegusa', 'e', 'goethals', 'eds)', 'am', 'chem', 'soc', 'symposiym', '59', 'acs,', 'washington', '1977', 'macromolecules1993', '26', '5530', '98', 'a', 'duda', 'z', 'florjanizyk', 'a', 'hofman', 's', 'stromkowski', 's', 'penzek', 'macromolecules', '1990', '23', '1640', '99', 't', 'endo', 't', 'aida', 'a', 'inoue', 'macromolecules', '1987', '20', '2982', '100', 'ra', 'gross', 'y', 'zang', 'g', 'conrad', 'rw', 'lenz', 'macromolecules', '1988', '21', '657', '101', 'hr', 'kricheldorf', 'i', 'kreiser-saunders', 'makromol', 'chem', '1990', '191', '1057', '102', 'h', 'yasuda', 'h', 'yamamoto', 'k', 'yokota', 's', 'miyake', 'a', 'nakamura', 'j', 'am', 'chem', 'soc1992', '114', '4908', '103', 'sj', 'mclain', 'ne', 'drysdale', 'e', 'polym', 'prepr1992', '174', '104', 'y', 'shen', 'z', 'shen', 'y', 'zhang', 'k', 'yao', 'macromolecules1996', '29', '8289', '105', 'wim', 'stevels', 'mjk', 'ankon', 'pj', 'dijkstra', 'j', 'feijen', 'macromolecules', '1996', '29', '8296', '106', 'wim', 'stevels', 'mjk', 'ankon', 'j', 'feijen', 'j', 'macromol', 'chem', 'phys1995', '146', '2417', '107', 'sj', 'mclain', 'tm', 'ford', 'ne', 'drysdale', 'am', 'chem', 'soc', 'polymer', 'preprints', '1992', '33', '(2)', '463', '(1992)', 'references', '325', '108', 'a', 'le', 'borgne', 'c', 'pluta', 'n', 'spassky', 'macromol', 'rapid', 'commun1995,', '15', '955', '109', 'm', 'yamashita', 'y', 'takemoto', 'e', 'ihara', 'h', 'yasuda', 'macromolecules', '1996', '29', '1798', '110', 'y', 'yao', 'q', 'shen', 'j', 'hu', 'gaofenzi', 'xuebao', '1997', '(6)', '672', '(from', 'chem', 'abstr1998', '128', '154413k16441sh)', '111', 'dr', 'wilson', 'rg', 'bearman', 'j', 'polymer', 'sci,', '1970', 'a-118', '2161', '112', 'rd', 'lundberg', 'jv', 'koleske', 'kb', 'wischmann', 'j', 'polymer', 'sci1969,', 'a-17', '2915', '113', 'j', 'sebena', '‘lactam', 'polymerization', '‘polymerization', 'heterocyclics', '(o', 'vogl', 'j', 'furukawa', 'eds)', 'dekker', 'new', 'york', '1973', '114', 'm', 'rothe', 'g', 'bertalan', 'chapt', '9', 'ring', 'opening', 'polymerizations', '(t', 'saegusa', 'e', 'goethals', 'eds)', 'am', 'chem', 'soc', 'symposium', 'series', '59', '1977', 'g', 'bertalan', 'i', 'rusznak', 'p', 'anna', 'm', 'boros-ivcz', 'g', 'marosi', 'polym', 'bull,', '198919', '539', 'g', 'bertalan', 'tt', 'nagy', 'p', 'valko', 'a', 'boros', 'm', 'boros-ivcz', 'p', 'anna', 'polym', 'bull,', '1988', '19', '547', '115', 'j', 'sebenda', 'chapt', '8', 'polymerization', 'heterocyclics', '(0', 'vogl', 'j', 'furukawa', 'eds)', 'dekker', 'new', 'york', '1973', 'j', 'sebenda', '“anionic', 'ring-opening', 'polymerizations', 'lactams”', 'chapt', '35', 'comprehensive', 'polymer', 'science', 'vol', '3', '(gc', 'eastmond', 'a', 'ledwith', 's', 'russo', 'p', 'sigwaldt', 'eds)', 'pergamon', 'press', 'london', '1989', '116', 'rb', 'homer', 'cd', 'johnson', 'p', '188', 'the', 'chemistry', 'amides', '(j', 'zabicky', 'ed)', 'wiley-interscience', 'new', 'york', '1970', '117', 'm', 'rothe', 'h', 'boenisch', 'd', 'essig', 'makromol', 'chem,', '1966', '91', '24', '118', 's', 'dubravszky', 'f', 'geleji', 'makromol', 'chem,', '1967', '110', '246', '119', 's', 'dubravszky', 'f', 'geleji', 'makromol', 'chem,', '1967', '143', '259', '120', 's', 'dubravszky', 'f', 'geleji', 'makromol', 'chem,', '1967', '105', '261', '121', 's', 'dubravszky', 'f', 'geleji', 'makromol', 'chem,', '1967', '113', '270', '122', 'm', 'rothe', 'g', 'reinisch', 'w', 'jaeger', 'i', 'schopov', 'makromol', 'chem1962,', '54', '183', '123', 'gm', 'burnett', 'aj', 'macarthur', 'jn', 'hay', 'eur', 'polymer', 'j,', '1967', '3', '321', '124', 'k', 'g', 'wyness', 'makromol', 'chem1960,', '38', '189', '125', 'g', 'bertalan', 'm', 'rothe', 'makromol', 'chem,', '1973', '172', '249', '126', 'm', 'rothe', 'j', 'mazanek', 'makromol', 'chem1971,', '145', '197', '127', 'm', 'pracejus', 'chem', 'ber,', '1959', '92', '988', '128', 't', 'makino', 's', 'inoue', 't', 'tsuruta', 'makromol', 'chem1970,', '131', '147', '129', 's', 'barzakay', 'm', 'levy', 'd', 'vofsi', 'j', 'polymer', 'sci1966,', 'a-14', '2211', '130', 'h', 'k', 'hall', 'jr,', 'j', 'am', 'chem', 'soc,', '1958', '80', '6404', '131', 'j', 'sebenda', 'j', 'polymer', 'sci1968,', 'c-23', '169', '132', 'e', 'sittler', 'j', 'sebenda', 'j', 'polymer', 'sci,', '1967', 'c-16', '67', '133', 'o', 'wichterle', 'makromol', 'chem,', '1960', '35', '174', '134', 'o', 'wichterle', 'j', 'sebenda', 'j', 'kralicek', 'fortschr', 'hochpolym', 'forsch,', '1961', '2', '578', '135', 'g', 'champetier', 'h', 'sekiguchi', 'j', 'polymer', 'sci,', '1960', '48', '309', '136', 'h', 'sekiguchi', 'j', 'polymer', 'sci,', '1960', '48', '309', '137', 'h', 'sekiguchi', 'b', 'cautin', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1973', '11', '1601', '138', 'tm', 'frunze', 'va', 'kotelnikov', 'tv', 'volkova', 'vv', 'kurashov', 'eur', 'polymer', 'j,', '1981', '17', '1079.', 'j', 'sebenda', 'v', 'kouril', 'eur', 'polymer', 'j,', '1971', '7', '1637', '139', 'h', 'tani', 't', 'kanomi', 'j', 'polymer', 'sci1966,', 'a-14', '301', '140', 'g', 'stea', 'gb', 'gechele', 'eur', 'polymer', 'j,', '1965', '1', '213', '141', 'cv', 'goebel', 'p', 'cefelin', 'j', 'stehlicek', 'j', 'sebenda', 'j', 'polymer', 'sci,', '1972', 'a-110', '1411', '142', 'j', 'roda', 'z', 'vortubcova', 'j', 'klalicek', 'j', 'stehlicek', 's', 'pokorny', 'makromol', 'chem1981,', '182', '2117', '143', 'n', 'mougin', 'ca', 'veith', 're', 'cohen', 'y', 'gnanou', 'macromolecules', '2004', '35', '1992', '144', 'ph', 'hermans', 'd', 'heikens', 'pf', 'van', 'velden', 'j', 'polymer', 'sci,', '1958', '30', '81', '145', 'vv', 'korshak', 'rv', 'kydryatsev', 'va', 'sergeev', 'lb', 'icikson', 'izv', 'akad', 'nauk', 'ssr', 'otd', 'khim', 'nauk', '1962', '1468', '146', 'h', 'yumoto', 'n', 'ogata', 'makromol', 'chem1957,', '25', '71', '147', 'm', 'rothe', 'h', 'boenisch', 'd', 'essig', 'makromol', 'chem,', '1966', '91', '24', '148', 'j', 'n', 'hay', 'j', 'polymer', 'chem,', 'polymer', 'letters1965', '5', '577', '149', 'ob', 'salamatina', 'dk', 'bonetskaya', 's', 'm', 'skuratov', 'ns', 'enikolopyan', 'vysokomol', 'soyed1969,', 'a-11', '158', '150', 'hk', 'reimschuessel', 'gj', 'dege', 'j', 'polymer', 'sci,', '1970', 'a-18', '3265', 'g', 'disilvestro', 'p', 'sozzani', 's', 'bruckner', 'l', 'malpezzi', 'c', 'guaita', 'makromol', 'chem,', '1987', '188', '2745', '151', 'dgh', 'ballard', 'ch', 'bamford', 'j', 'chem', 'soc,', '1956', '381', '152', 'm', 'szwarc', 'fortschr', 'hochpolymer', 'forsch,', '1965', '4', '1', '153', 'y', 'iwakura', 'k', 'uno', 'j', 'polymer', 'sci,', '1968', 'a-16', '2165', '154', 'hr', 'kricheldorf', 'makromol', 'chem,', '1977', '178', '1959', 'hr', 'kricheldorf', '“anionic', 'ring-opening', 'polymerizations', 'n-carboxyanhydrides”', 'chap', '36', 'comprehensive', 'polymer', 'science', 'vol', '3', '(gc', 'eastmond', 'a', 'ledwith', 's', 'russo', 'p', 'sigwalt', 'eds)', 'pergamon', 'press', 'london', '1989', '326', '5', 'ring-opening', 'polymerizations', '155', 't', 'makino', 's', 'inoue', 't', 'tsuruta', 'makromol', 'chem1970,', '131', '147', '156', 's', 'freireich', 'd', 'gertner', 'a', 'zilkha', 'eur', 'polymerj,', '1974', '10', '439', '157', 'n', 'calderon', 'ea', 'ofstead', 'jp', 'ward', 'wa', 'judy', 'kw', 'scott', 'j', 'am', 'chem', 'soc1968,', '90', '4133', '158', 'd', 'l', 'pickelt', 'n', 'politakos', 'a', 'avgeropoulos', 'j', 'm', 'messman', 'macromolecules', '2009', '42', '(20)', '7781', '159', 'r', 'alamo', 'j', 'guzman', 'jg', 'fatou', 'makromol', 'chem,', '1981', '182', '725', 'kw', 'scott', 'n', 'calderon', 'ea', 'ofstead', 'wa', 'judy', 'jp', 'ward', 'adv', 'chem', 'series1961', '91', '399', '160', 'ba', 'dolgoplosk', 'j', 'polymer', 'sci,', 'symposium', '1980', '67', '99', '161', 'jl', 'herrisson', 'y', 'charwin', 'makromol', 'chem,', '1970', '141', '161', '162', 'mt', 'macella', 'ma', 'busch', 'el', 'mueteris', 'j', 'am', 'chem', 'soc,', '1976', '98', '1283', '163', 'tj', 'katz', 'jm', 'mcginnis', 'j', 'am', 'chem', 'soc,', '1975', '97', '1952', '164', 'jm', 'mcginnis', 'tj', 'katz', 's', 'hurwitz', 'j', 'am', 'chem', 'soc1976,', '98', '606', '165', 'aj', 'amass', 'ta', 'mcgourtey', 'eur', 'polymer', 'j,', '1980', '16', '23', 'aj', 'amass', 'm', 'lotﬁpur', 'ja', 'zurimendi', 'bj', 'tighe', 'c', 'thompson', 'makromol', 'chem,', '1987', '188', '2121', '166', 'kj', 'ivin', 'ld', 'theodore', 'bsr', 'reddy', 'jj', 'rooney', 'makromol', 'chem,', 'rapid', 'commun,', '1980', '1', '467', 's', 'streck', '167', 'ct', 'thu', 't', 'bastelberger', 'h', 'hocker', 'makromol', 'chem,', 'rapid', 'commun1981,', '2', '7', '168', 'jg', 'nel', 'kb', 'wagner', 'jm', 'boncella', 'macromolecules1991', '24', '2649', 'm', 'lindmartk', 'nd', 'kb', 'wagner', 'chemtech', '1989', '19', '489', 'macromolecules1987', '20', '2949', '169', 'kb', 'wagner', 'jt', 'patton', 'macromolecules', '1993', '26', '249', '170', 'bm', 'bovak', 'rh', 'grubbs', 'j', 'am', 'chem', 'soc1988,', '110', '7542', 'p', 'schwab', 'rh', 'grubbs', 'jw', 'ziller', 'j', 'am', 'chem', 'soc1996,', '118', '100', 'd', 'lynn', 's', 'kanoka', 'rh', 'grubbs', 'j', 'am', 'chem', 'soc1996', '118', '784', 'truhral', 'et', 'al', 'organometallics', '2011', 'doi101021om200529m', '171', 'w', 'apichatachutapan', 'lj', 'mathias', 'am', 'chem', 'soc', 'polymer', 'preprints1996', '37', '(2)315', '172', 'jg', 'hamilton', 'jj', 'rooney', 'jm', 'desimone', 'c', 'mistele', 'macromolecules', '1998', '31', '4387', '173', 'y', 'nakayama', 'n', 'ikushima', 'a', 'nakamura', 'chem', 'lett,', '1997', '9', '861', '174', 'a', 'demonceau', 'aw', 'stumpf', 'e', 'salve', 'af', 'noels', 'macromolecules', '1997', '30', '3127', '175', 'z', 'wu', 'dr', 'wheeler', 'rh', 'grubbs', 'j', 'am', 'chem', 'soc,', '1992', '114', '146', '176', 'z', 'wu', 'rh', 'grubbs', 'macromolecules', '1994', '27', '6700', '177', 'z', 'wu', 'rh', 'grubbs', 'macromolecules', '1995', '28', '3502', '178', 'z', 'wu', 'ad', 'benedicto', 'rh', 'grubbs', 'macromolecules', '1993', '26', '4975', '179', 'j', 'gratt', 're', 'cohen', 'macromolecule', '1997', '30', '3137', '180', 'dm', 'lynn', 'b', 'mohr', 'rh', 'grubbs', 'j', 'am', 'chem', 'soc1998,', '120', '1627', '181', 'y', 'yong', 't', 'swager', 'macromolecules', '2007', 'web', 'report101021ma071304s0024-9297(07)0134-6', '182', 'lm', 'pitet', 'ma', 'hillmyer', 'macromolecules', '2009', '42', '3674', '183', 'shilf', 'fm', 'kilbinger', 'am', 'chem', 'soc', 'polymer', 'preprints', '2008', '49', '(1)', '531', '184', 'csj', 'cazin', 'et', 'al,', 'chem', 'commun2010', '46', '7115', '185', 'm', 'wathier', 'ss', 'stoddart', 'mw', 'grinstaff', 'am', 'chem', 'soc', 'polymer', 'preprints', '2007', '48', '(2)', '249', '186', 'r', 'a', 'patsiga', 'j', 'macromol', 'sci,', 'rev', 'macromol', 'chem,', '1967', 'c1', '(2)', '223', 'gd', 'jones', 'dc', 'mac', 'williams', 'na', 'baxtor', 'j', 'org', 'chem1965,', '30', '1994', 'gd', 'jones', 'chapt', '14', 'the', 'chemistry', 'cationic', 'polymerization', '(ph', 'plesch', 'ed)', 'pergamon', 'press', 'oxford', '1963', 'ej', 'goethals', '“cyclic', 'amines”', 'chap', '10', 'ring-opening', 'polymerizations', 'vol', '2', '(kj', 'ivin', 't', 'saegusa', 'eds)', 'elsevier', 'london', '1984', '187', 'p', 'dreyfuss', 'j', 'macromol', 'sci-chem,', '1973', '7', '1361', '188', 'm', 'morton', 'rf', 'kammereck', 'lj', 'fetters', 'macromolecules', '1971', '4', '11', 'brit', 'polymer', 'j,', '1971', '3', '120', '189', 'ej', 'goethals', 'w', 'drijvers', 'makromol', 'chem,', '1970', '136', '73', 't', 'aida', 'k', 'kawaguchi', 's', 'inoue', '190', 'j', 'k', 'stille', 'j', 'e', 'empren', 'the', 'chemistry', 'sulﬁdes', '(a', 'v', 'tobolsky', 'ed)', 'wiley-interscience', 'new', 'york', 'macromolecules', '1990', '23', '3887', '1968', '191', 'e', 'j', 'goethals', 'w', 'drijvers', 'makromol', 'chem1973,', '165', '329', '192', 'f', 'lautenschlaeger', 'polymerization', 'heterocyclics', '(o', 'vogel', 'j', 'furukawa', 'eds)', 'dekker', 'new', 'york', '1973', 'n', 'spassky', 'chapt', '14', 'ring-opening', 'polymerizations', '(t', 'saegusa', 'e', 'goethals', 'eds)', 'am', 'chem', 'soc', 'symposium', '59', 'acs,', 'washington', '1977', '193', 'e', 'j', 'goethals', 'w', 'drijvers', 'd', 'van', 'ooteghem', 'am', 'boyle', 'j', 'macromol', 'sci-chem1973,', '7', '1375', '194', 't', 'tsunetsugu', 'j', 'furukawa', 't', 'fueno', 'j', 'polymer', 'sci1971,', 'a-19', '3541', '195', 'm', 'morton', 'r', 'f', 'kammereck', 'j', 'am', 'chem', 'soc1970,', '92', '3217', '196', 'rh', 'gobran', 'r', 'larson', 'j', 'polymer', 'sci1970,', 'c-31', '77', '197', 'j', 'furukawa', 'n', 'kawabata', 'a', 'kato', 'j', 'polymer', 'sci,', 'polymer', 'letters1967', '5', '1073', '198', 's', 'inoue', 't', 'tsuruta', 'j', 'furukawa', 'makromol', 'chem1962,', '53', '215', '199', 'n', 'spassky', 'p', 'sigwalt', 'eur', 'polymer', 'j,', '1971', '7', '7', '200', 'w', 'j', 'murbach', 'a', 'adicoff', 'ind', 'eng', 'chem1960,', '52', '772', 'references', '327', '201', 'japan', 'patent', '10046', '(1960)', 'du', 'pont', 'co', '(from', 'ref', '1)', '202', 't', 'saegusa', 'h', 'imai', 'j', 'furukawa', 'makromol', 'chem1962,', '56', '55', '203', 'h', 'cherdron', 'polymerizaton', 'heterocyclics', '(o', 'vogl', 'j', 'furukawa', 'eds)', 'dekker', 'new', 'york', '1973', '204', 'k', 'tada', 'y', 'numata', 't', 'saegusa', 'j', 'furukawa', 'makromol', 'chem1964,', '77', '220', '205', 'sm', 'glickman', 'es', 'miller', 'us', 'patent', '3016367', '(1962)', '206', 'rm', 'hedrick', 'eh', 'motters', 'tm', 'butler', 'us', 'patent', '3120503', '(1964)', '207', 'hr', 'kricheldorf', 'w', 'e', 'hull', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1978', '16', '2253', '208', 'w', 'fish', 'w', 'hoffman', 'j', 'koskikallio', 'j', 'chem', 'ind,', '1956', '756', '209', 't', 'ragiua', 's', 'narisawa', 'k', 'manobe', 'm', 'kobata', 'j', 'polymer', 'sci,', '1966', 'a-14', '2081', '210', 'e', 'k', 'drecksel', 'us', 'patent', '424457', '(1954)', '211', 'h', 'l', 'hsieh', 'j', 'macromol', 'sci', '-chem1973,', '7', '1525', '212', 'm', 'l', 'hsieh', 'chapt', '10', 'ring', 'opening', 'polymerization', '(t', 'saegusa', 'e', 'goethals', 'eds)', 'acs', 'symposium', 'series', '1977', '59', '213', 'a', 'k', 'khomyakov', 'e', 'b', 'ludvig', 'n', 'n', 'shapetko', 'eur', 'polymer', 'j', '1981,', '17', '1089', '214', 't', 'saeguse', 'angew', 'chem,', 'intern', 'ed,', '1977', '16', '826', '215', 't', 'saegusa', 'h', 'ikeda', 'fujii', 'macromolecules', '1972', '5', '354', '216', 't', 'saegusa', 's', 'kobayashi', 'y', 'kimura', 'macromolecules', '1974', '7', '1', '217', 't', 'saegusa', 'h', 'ikeda', 's', 'hirayanagi', 'y', 'kimura', 's', 'kobayaski', 'macromolecules', '1975', '8', '259', '218', 't', 'saegusa', 's', 'kobayashi', 'y', 'kimura', 'macromolecules1974', '7', '139', '219', 't', 'saegusa', 's', 'kobayashi', 'y', 'kimura', 'macromolecules1975', '8', '374', '220', 't', 'saegusa', 'y', 'kimura', 's', 'kobayashi', 'macromolecules', '1977', '10', '239', '221', 't', 'saegusa', 's', 'kobayashi', 'j', 'furukawa', 'macromolecules', '1976', '9', '728', '222', 't', 'saegusa', 'y', 'kimura', 's', 'sawada', 's', 'kobayaski', 'macromolecules', '1974', '7', '956', '223', 't', 'saegusa', 'makromol', 'chem,', 'suppl,', '1979', '3', '157', '224', 't', 'saegusa', 'y', 'kimura', 'n', 'ishikawa', 's', 'kobayashi', 'macromolecules', '1976', '9', '724', '225', 't', 'saeguse', 's', 'kobayashi', 'j', 'furukawa', 'macromolecules', '1977', '10', '73', '226', 't', 'saeguse', 'makromol', 'chem,', 'suppl,', '1981', '4', '73', 't', 'aida', 'y', 'mackawa', 's', 'asano', 's', 'inoue', 'macromolecules', '1988', '21', '1195', 's', 'inoue', 't', 'aida', 'chemtech1994', '24', '(5)', '28', '227', 'i', 'cho', 'kd', 'ahn', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1979', '17', '3169', '228', 'i', 'cho', 'ms', 'gong', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1982', '20', '61', '229', 't', 'endo', 'wj', 'balley', 'j', 'polymer', 'sci,', 'polymer', 'letters1940', '14', '25', '230', 'wj', 'balley', 'z', 'ni', 'sr', 'wu', 'macromolecules1982', '1', '711', '231', 'ra', 'evans', 'e', 'rizzardo', 'macromolecule', '1996', '29', '6983', '232', 'y', 'wei', 'ej', 'connors', 'x', 'jia', 'c', 'wang', 'j', 'polymer', 'sci.', 'part', 'apolym', 'chem1998', '36', '761', '233', 'h', 'sawada', 'thermodynamics', 'polymerization', 'chapt7', 'dekker', 'new', 'york', '1976', 'chapter', '6', 'common', 'chain-growth', 'polymers', '61', 'polyethylene', 'related', 'polymers', 'polyethylene', 'produced', 'commercially', 'large', 'quantities', 'many', 'parts', 'world', 'the', 'monomer', 'synthesized', 'various', 'sources', 'today', 'however', 'ethylene', 'comes', 'petroleum', 'high', 'temperature', 'cracking', 'ethane', 'gasoline', 'fractions', 'other', 'potential', 'sources', 'probably', 'found', 'depending', 'upon', 'availability', 'raw', 'materials', 'two', 'main', 'types', 'polyethylene', 'manufactured', 'commercially', 'these', 'low', '(092–093', 'gcm3)', 'high', '(094–097', 'gcm3)', 'density', 'polymers', 'the', 'low-density', 'material', 'branched', 'high-', 'density', 'one', 'mostly', 'linear', 'much', 'crystalline', 'the', 'important', 'applications', 'low-density', 'polyethylene', 'ﬁlms', 'sheets', 'paper', 'wire', 'cable', 'coatings', 'injection', 'molding', 'the', 'high-density', 'material', 'ﬁnds', 'use', 'blow', 'molded', 'objects', 'injection', 'molding', '611', 'preparation', 'polyethylene', 'free-radical', 'mechanism', 'up', 'late', '1960s', 'low-density', 'polyethylene', 'produced', 'commercially', 'high-pressure', 'free-radical', 'polymerization', 'much', 'replaced', 'preparation', 'copolymers', 'ethylene', 'a-oleﬁns', 'coordination', 'polymerization', 'these', 'preparations', 'discussed', 'chapter', 'high-pressure', 'polymerizations', 'ethylene', 'however', 'might', 'still', 'practiced', 'places', 'therefore', 'discussed', 'the', 'reaction', 'requires', 'minimum', 'pressure', '500', 'atm', '1', 'proceed', 'the', 'branched', 'products', 'contain', 'long', 'short', 'branches', 'well', 'vinylidene', 'groups', 'with', 'increase', 'pressure', 'temperature', 'polymerization', 'decrease', 'degree', 'branching', 'amount', 'vinylidene', 'groups', '2', '3.', 'free-radical', 'commercial', 'polymerizations', 'conducted', '1000–3000', 'atm', 'pressure', '80–300(cid1)c', 'the', 'reaction', 'two', 'peculiar', 'characteristics', '(1)', 'high', 'exotherm', '(2)', 'critical', 'dependence', 'monomer', 'concentration', 'in', 'addition', 'high', 'pressures', 'oxygen', 'acts', 'initiator', 'at', '2000', 'atm', 'pressure', '165(cid1)c', 'temperature', 'however', 'maximum', 'safe', 'level', 'oxygen', '0075', 'ethylene', 'gas', 'reaction', 'mixture', 'any', 'amount', 'oxygen', 'beyond', 'level', 'cause', 'explosive', 'decompositions', 'history', 'polyethylene', 'manufacture', 'contains', 'stories', 'workers', 'killed', 'explosions', 'yet', 'oxygen', 'concentration', 'monomer', 'directly', 'proportional', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-96', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '329', '330', '6', 'common', 'chain-growth', 'polymers', 'percent', 'conversion', 'monomer', 'polymer', 'though', 'inversely', 'proportional', 'polymer’s', 'molecular', 'weight', 'this', 'limits', 'many', 'industrial', 'practices', 'conducting', 'reactions', '2000', 'atm', '200(cid1)c', 'these', 'reactions', 'done', 'therefore', '1000', '2000', 'atm', 'pressures', 'small', 'quantities', 'oxygen', 'limited', '02', 'ethylene', 'accurately', 'metered', '4', '5.', 'the', 'conversion', 'per', 'pass', 'continuous', 'reactors', 'usually', 'low', '15–20.', 'there', 'induction', 'period', 'varies', 'inversely', 'oxygen', 'concentration', 'power', '023', 'during', 'period', 'oxygen', 'consumed', 'autocatalytically', 'this', 'accompanied', 'signiﬁcant', 'decrease', 'pressure', 'a', 'high', 'concentration', 'ethylene', 'necessary', 'fast', 'rate', 'chain', 'growth', 'relative', 'rate', 'termination', 'also', 'high', 'temperatures', 'required', 'practical', 'rates', 'initiation', 'if', 'oxygen', 'completely', 'excluded', 'pressure', 'raised', '3500', '7750', 'atm', 'using', 'relatively', 'low', 'temperatures', '50–80(cid1)c', 'linear', 'polyethylene', 'forms', '6.', 'the', 'reactions', 'take', '20', 'h', 'various', 'solvents', 'used', 'like', 'benzene', 'isooctane', 'methyl', 'ethyl', 'alcohols', 'higher', 'ethyl', 'alcohol', 'concentrations', 'low', 'concentrations', 'initiator', 'result', 'higher', 'molecular', 'weights', 'the', 'products', 'range', '2000', 'wax-like', 'polymers', '4000000', 'nearly', 'intractable', 'materials', 'favorite', 'free-radical', 'initiators', 'reaction', 'benzoyl', 'peroxide', 'azobisisobutyroni-', 'trile', 'di-t-butylperoxydicarbonate', 'di-t-butyl', 'peroxide', 'dodecanoyl', 'peroxide', 'above', 'conditions', 'differ', 'however', 'typical', 'commercial', 'ones', 'high', 'pressures', 'long', 'reaction', 'times', 'practical', 'the', 'actual', 'commercial', 'conditions', 'vary', 'depending', 'upon', 'location', 'individual', 'technology', 'company', 'often', 'tubular', 'multiple-tray', 'autoclaves', 'used', '7.', 'good', 'reactor', 'design', 'must', 'permit', 'dissipation', 'heat', 'polymerization', '(800–1000', 'calg)', 'good', 'control', 'parameters', 'reaction', 'tubular', 'reactors', 'judged', 'advantage', 'stirred', 'autoclaves', 'offering', 'greater', 'surface-to-volume', 'ratios', 'better', 'control', 'residence', 'time', '7.', 'on', 'hand', 'stirred', 'autoclaves', 'offer', 'uniform', 'temperature', 'distribution', 'throughout', 'reactor', 'the', 'tubular', 'reactors', 'described', 'consisting', 'stainless', 'steel', 'tubes', '05', '1', 'internal', '2', 'external', 'diameters', 'the', 'residence', 'time', 'tubes', '3', '5', 'min', 'equipped', 'pistons', 'pressure', 'regulation', 'pressure', 'might', 'also', 'controlled', 'ﬂow', 'pulses', 'reactor', '8.', 'for', 'oxygen-initiated', 'reactions', 'optimum', 'conditions', '7', '003–01', 'oxygen', '190–210(cid1)c', '1500', 'atm', 'pressure', 'at', 'pressure', 'density', 'ethylene', '046', 'gcm3', 'this', 'compares', 'favorably', 'critical', 'density', 'ethylene', '022', 'gcm3', 'once', 'polymerization', 'initiated', 'liquid', 'monomer', 'acts', 'solvent', 'polymer', 'impurities', 'acetylene', 'hydrogen', 'cause', 'chain', 'transferring', 'must', 'carefully', 'removed', 'in', 'processes', 'hindered', 'phenols', 'added', 'small', 'quantities', '(between', '10', '1000', 'ppm)', 'this', 'effect', 'reducing', 'long-chain', 'branching', 'yields', 'ﬁlm', 'grade', 'resins', 'better', 'clarity', 'lower', 'haze', 'reduced', 'amount', 'microgels', 'also', 'diluents', 'used', 'practices', 'their', 'main', 'purpose', 'act', 'heat-exchanging', 'mediums', 'also', 'help', 'remove', 'polymer', 'reactor', 'such', 'diluents', 'water', 'benzene', 'ethyl', 'methyl', 'alcohols', 'sometimes', 'chain', 'transferring', 'agents', 'like', 'carbon', 'tetrachloride', 'ketones', 'aldehydes', 'cyclohexane', 'might', 'also', 'added', 'control', 'molecular', 'weight', 'the', 'ﬁnished', 'product', '(polymer–monomer', 'mixture)', 'conveyed', 'separator', 'almost', 'unreacted', 'ethylene', 'removed', 'high', 'pressure', '(3500–5000', 'psi)', 'recycled', 'the', 'polymer', 'extruded', 'palletized', 'ethylene', 'conversion', 'per', 'pass', 'limiting', 'factor', 'economics', 'a', 'tubular', 'reactor', 'illustrated', 'fig', '61', '61', 'polyethylene', 'related', 'polymers', '331', 'fig', '61', 'illustration', 'tubular', 'reactor', 'polyethylene', 'prepared', 'way', 'may', 'many', '20–30', 'short', 'branches', 'per', '10000', 'carbon', 'atoms', 'chain', '9', 'one', 'two', 'long-chain', 'branches', 'per', 'molecule', 'due', '“backbiting”', '10', '(explained', 'chap', '3)', 'ch2', 'the', 'reaction', 'results', 'predominantly', 'ethyl', 'butyl', 'branches', 'the', 'ratio', 'ethyl', 'butyl', 'groups', 'roughly', '21', '11', '12.', 'chain', 'transferring', 'tertiary', 'hydrogens', 'location', 'short', 'branches', 'causes', 'elimination', 'reactions', 'formation', 'vinylidene', 'groups', '13', '14.', 'this', 'mechanism', 'also', 'accounts', 'formation', 'low', 'molecular', 'weight', 'species', 'r', 'rh', 'c3h7', 'commercial', 'grades', 'low-density', 'polyethylene', 'vary', 'widely', 'number', 'short', 'long', 'branches', 'average', 'molecular', 'weights', 'molecular', 'weight', 'distributions', 'mwmn', '332', '6', 'common', 'chain-growth', 'polymers', '20', '50', 'commercial', 'low-density', 'materials', 'the', 'short', 'branches', 'control', 'degree', 'crystallinity', 'stiffness', 'polymer', 'density', 'they', 'also', 'inﬂuence', 'ﬂow', 'properties', 'molten', 'material', '612', 'preparation', 'polyethylene', 'coordination', 'mechanism', 'low-density', 'polyethylene', 'prepared', 'coordination', 'polymerization', 'copolymerization', 'ethylene', 'a-oleﬁns', 'this', 'discussed', 'section', 'copolymers', 'ethylene', 'finding', 'catalytic', 'systems', 'would', 'allow', 'formation', 'amorphous', 'low-density', 'polyethylene', 'monomer', 'alone', 'low-pressure', 'polymerization', 'however', 'economically', 'worthwhile', 'goal', 'to', 'end', 'consider-', 'able', 'research', 'carried', 'develop', 'catalytic', 'systems', 'particular', 'attention', 'given', 'metallocenes', 'single-site', 'catalysts', 'oleﬁn', 'polymerization', 'originally', 'metallocene', 'catalysts', 'typical', 'metal', 'complexes', 'two', 'cyclopentadienyl', 'substituted', 'cyclopentadienyl', 'groups', 'many', 'variations', 'developed', 'since', 'these', 'materials', 'used', 'combination', 'methyl', 'aluminoxane', 'potential', 'forming', 'polymers', 'high', 'precision', 'nevertheless', 'time', 'probably', 'still', 'safe', 'say', 'low-density', 'polyethylene', 'prepared', 'many', 'perhaps', 'processes', 'catalytic', 'systems', 'mentioned', 'book', 'this', 'material', 'manufactured', 'world', 'different', 'considerations', 'govern', 'decisions', 'processes', 'catalytic', 'systems', 'the', 'probably', 'true', 'high-density', 'polyethylene', 'new', 'catalysts', 'based', 'palladium', 'nickel', 'complexes', 'bulky', 'a-diimine', 'ligands', 'developed', '17–19.', 'they', 'yield', 'highly', 'branched', 'moderately', 'branched', 'polymers', 'ethylene', 'well', 'propylene', '1-hexene', 'the', 'polyoleﬁns', 'produced', 'catalysts', 'contain', 'considerable', 'amount', 'branches', 'along', 'backbone', 'randomly', 'distributed', 'throughout', 'molecules', 'in', 'catalysts', 'molecular', 'weight-limiting', 'b-hydrogen', 'elimination', 'process', 'common', 'palladium', 'nickel', 'catalysts', 'suppressed', 'use', 'bulky', 'a-diamine', 'ligands', '19.', 'this', 'allows', 'formation', 'high', 'molecular', 'weight', 'polymers', 'ethylene', 'a-oleﬁns', 'a', 'nickel-based', 'catalyst', 'illustrated', 'follows', 'i-pr', 'r', 'r', 'i-pr', 'i-pr', 'ni', 'i-pr', 'o(et)2', 'b', '3', 'it', 'claimed', 'branching', 'polyethylene', 'controlled', 'extent', 'product', 'even', 'branched', 'conventional', 'low-density', 'polyethylene', '(12—300', 'branches1000', 'atoms)', '18', '19.', 'the', 'cationic', 'ni-diimine', 'catalyst', 'shown', '(r', '¼', 'h', 'ch3)', 'methylaluminoxane', 'analog', 'found', 'polymerize', 'ethylene', 'toluene', 'room', 'temperature', 'rate', '110000', 'kgnih', 'this', 'comparable', 'metallocene', 'rates', 'the', 'pd-based', 'catalysts', 'less', 'active', 'ni', 'analogs', '19.', 'when', 'nickel', 'catalysts', 'used', 'extent', 'branching', 'function', 'temperature', 'ethylene', 'pressure', 'catalyst', 'structure', 'branching', 'increases', 'temperature', 'rises', 'at', 'higher', 'ethylene', '61', 'polyethylene', 'related', 'polymers', '333', 'pressure', 'less', 'branching', 'occurs', 'brookhart', 'et', 'al', 'illustrate', 'mechanism', 'polymerization', 'follows', '18:', 'r', 'r', '2', 'x', 'ar', 'm', 'x', 'ar', 'ethylene', 'activator', 'ar', 'r', 'm', 'ar', 'ar', 'm', 'ar', 'beta-hydride', 'elimination', 'r', 'ethylene', 'm', 'r', 'm', 'r', 'ethylene', 'chain', 'migration', 'via', 'insertion', 'ar', 'm', 'ar', 'migraatory', 'insertion', 'm', 'ar', 'migratory', 'insertion', 'r', 'ar', 'ar', 'ar', 'm', 'ar', 'ar', 'm', 'ar', 'ar', 'ar', 'polymer', 'r', 'additional', 'chain', 'insertion', 'migation', 'branched', 'polymer', 'x', '¼', 'ch3', 'br', 'm', '¼', 'pd', 'ni', 'r', '¼', '(ch2ch2)nch3', 'ar', '¼', '26-dialkylphenyl', 'a', 'patent', 'polymerization', 'process', 'oleﬁns', '(especially', 'ethylene', 'a-oleﬁns', 'cyclopentene', 'ﬂuorinated', 'oleﬁns)', 'describes', 'catalytic', 'systems', '20.', 'the', 'hindered', 'diimines', 'stabilize', 'alkyl', 'ni(ii)', 'pd(ii)', 'cationic', 'complexes', 'after', 'preparation', 'complexes', 'reduced', 'methylalu-', 'minoxane', 'activated', 'lewis', 'acids', 'capable', 'forming', 'non-coordinating', 'counterions', '20.', 'in', 'addition', 'preparation', 'catalysts', 'based', 'iron', 'cobalt', '21', 'also', 'reported', 'these', 'complexes', 'bulky', 'pyridine', 'bis-imine', 'ligands', 'iron', 'cobalt', 'also', 'activated', 'methylaluminoxane', 'n', 'nm', 'cl', 'n', 'cl', 'the', 'iron-based', 'catalysts', 'reported', 'considerably', 'active', 'cobalt', 'analogs', '21.', 'the', 'yield', 'linear', 'narrow', 'molecular', 'weight', 'distribution', 'polyethylene', 'per', 'gram', 'reported', 'high', '21.', '334', '6', 'common', 'chain-growth', 'polymers', 'baugh', 'et', 'al', '22', 'synthesized', 'characterized', 'series', 'nickel(ii)', 'iron(ii)', 'complexes', 'general', 'formula', 'lmx2', 'containing', 'bidentate', '(for', 'm', '¼', 'ni)', 'tridentate', '(for', 'm', '¼', 'fe)', 'heterocycle-', 'imine', 'ligands', 'activation', 'pre-catalysts', 'methyl', 'aluminoxane', 'yields', 'active', 'catalyst', 'systems', 'oligomerizationpolymerization', 'ethylene', 'compared', 'a-diimine', 'nickel', 'bis', '(imino)pyridine', 'iron', 'catalysts', 'metal', 'systems', 'provide', 'half', 'steric', 'protection', 'consequently', 'catalytic', 'activities', 'signiﬁcantly', 'lower', 'lower', 'activities', 'attributed', 'reduced', 'stability', 'active', 'species', 'polymerization', 'conditions', 'the', 'lower', 'molecular', 'weights', 'products', 'explained', 'result', 'increased', 'hydrogen', 'transfer', 'rates', 'variations', 'within', 'heterocyclic', 'components', 'ligand', 'showed', 'steric', 'electronic', 'factors', 'inﬂuence', 'polymerization', 'behavior', 'catalysts', 'hanaoka', 'oda', 'coworkers', 'report', '23', 'single-site', 'polymerization', 'catalysts', 'consid-', 'erable', 'interest', 'industrially', 'today', 'afford', 'highly', 'controllable', 'polymerization', 'performances', 'based', 'precise', 'design', 'catalyst', 'architecture', 'industrial', 'applications', 'among', 'point', 'constrained', 'geometry', 'catalyst', 'phenoxy-induced', 'complex', 'call', 'phenics–ti', 'used', 'together', 'methyl', 'aluminoxane', 'si', 'ti', 'cl', 'cl', 'n', 't-bu', 'concentrated', 'geometry', 'catalyst-ti', 'si', 'cl', 'cl', 'ti', 'o', 't-bu', 'phenics-ti', 'these', 'half-metallocene', 'catalysts', 'anionic', 'armed-pendant', 'well', 'developed', 'industrial', 'production', 'copolymers', 'ethylene', '1-oleﬁns', 'modiﬁcation', 'cyclopentadienyl', 'ring', 'system', 'mainly', 'tuned', 'ﬁnely', 'control', 'polymerization', 'behaviors', 'activity', 'molecular', 'weight', 'regiochemistry', 'in', 'general', 'minimizing', '21-insertion', 'essential', 'obtain', 'high', 'molecular', 'weight', 'polyoleﬁns', 'otherwise', 'facile', '6-elimination', 'occurs', 'leading', 'termination', 'chain', 'growth', 'thus', 'largely', 'open', 'coordination', 'sites', 'half-metallocene', 'catalyst', 'systems', 'possess', 'indispensable', 'problem', 'irregularity', 'propagation', 'through', 'tuning', 'bulkiness', 'substituents', 'bridged-silicon', 'unit', 'phenics–ti', 'claimed', 'demonstrated', '21-insertion', 'propylene', 'also', 'controlled', 'bridging', 'substituents', 'produce', 'high', 'molecular', 'weight', 'polypropylene', '23.', 'hong', 'coworkers', '24', 'concluded', 'generally', 'desirable', 'immobilize', 'single-site', 'metallocene', 'catalysts', 'suitable', 'carrier', 'obtain', 'ideal', 'product', 'morphology', 'ultrahigh', 'molecular', 'weight', 'polyethylenes', 'successfully', 'prepared', 'titanium', 'complexes', 'bearing', 'phenoxy-imine', 'chelate', 'ligands', 'n', 'o', 't-bu', 'ticl2', '2', '61', 'polyethylene', 'related', 'polymers', '335', 'table', '61', 'typical', 'conditions', 'preparation', 'high-density', 'polyethylene', 'approx', 'temperature', 'approx', 'pressure', 'usual', 'state', 'polymer', 'reaction', 'mixture', 'ziegler–natta', 'process', '75(cid1)c', '60', 'psi', 'suspension', 'cr2o3support', '140(cid1)c', '420', 'psi', 'suspension', 'moo3support', '234(cid1)c', '1000', 'psi', 'suspension', 'they', 'demonstrated', 'catalyst', 'immobilized', 'silica', 'the', 'product', 'yields', 'ultrahigh', 'molecular', 'weight', 'polyethylene', 'increased', 'polymerization', 'temperature', 'resulted', 'higher', 'activity', 'lower', 'molecular', 'weight', 'polyethylene', '613', 'commercial', 'high-density', 'polyethylene', 'properties', 'manufacture', 'high-density', 'polyethylene', '(094–097', 'gcm3)', 'produced', 'commercially', 'two', 'types', 'catalysts', '1', 'ziegler–natta', 'type', 'catalysts', '2', 'transition', 'metal', 'oxides', 'various', 'supports', 'the', 'two', 'catalytic', 'systems', 'used', 'different', 'conditions', 'both', 'types', 'undergone', 'evolution', 'earlier', 'development', 'the', 'original', 'practices', 'summarized', 'table', '61', 'the', 'ziegler', 'process', 'yields', 'polyethylene', 'low', '094cm3', 'density', 'process', 'modiﬁcations', 'result', 'products', 'density', '0965', 'gcm3', 'the', 'transition', 'metal', 'oxide', 'catalysts', 'support', 'hand', 'yield', 'products', 'density', 'range', '0960–0970', 'gcm3', 'the', 'original', 'development', 'ziegler', 'led', 'appears', 'almost', 'endless', 'number', 'patents', 'various', 'coordination-type', 'catalysts', 'processes', 'as', 'described', 'chap', '4', 'catalysts', 'vastly', 'improved', 'progress', 'made', 'toward', 'enhanced', 'efﬁciency', 'selectivity', 'the', 'amount', 'polymer', 'produced', 'per', 'gram', 'transition', 'metal', 'increased', 'manyfold', 'in', 'addition', 'new', 'catalysts', 'based', 'zirconium', 'compounds', 'complexed', 'methyl', 'aluminoxane', 'oligomers', '(sometimes', 'called', 'kominsky', 'catalysts)', 'developed', 'they', 'yield', 'high', 'quantities', 'polyethylene', 'per', 'gram', 'catalyst', 'for', 'instance', 'catalyst', 'bis(cyclopentadienyl)-zirconium', 'dichloride', 'combined', 'methylaluminoxane', 'claimed', 'yield', '5000', 'kg', 'linear', 'polyethylene', 'per', 'gram', 'zirconium', 'per', 'hour', '14.', 'an', 'important', 'factor', 'catalysts', 'activity', 'degree', 'oligomerization', 'aluminoxane', 'moiety', 'the', 'catalytic', 'effect', 'enhanced', 'increase', 'number', 'alternating', 'aluminum', 'oxygen', 'atoms', 'these', 'catalysts', 'long', 'storage', 'life', 'offer', 'high', 'activity', 'need', 'removed', 'product', 'amount', 'present', 'negligible', '14', '15.', 'this', 'makes', 'work-up', 'product', 'simple', 'the', 'continuous', 'solution', 'processes', 'usually', 'carried', '120', '160(cid1)c', '400–500', 'lbin2', 'pressure', 'the', 'diluents', 'may', 'cyclohexane', 'isooctane', 'in', 'one', 'zone', 'reactors', 'solid', 'catalyst', 'evenly', 'dispersed', 'throughout', 'reactor', 'in', 'two', 'zone', 'reactors', '(specially', 'constructed)', 'polymerizations', 'conducted', 'stirring', 'lower', 'zone', 'catalysts', 'present', 'concentrations', '02–06', 'diluent', 'puriﬁed', 'ethylene', 'fed', 'bottom', 'portions', 'reactors', 'the', 'polymers', 'form', 'carried', 'small', 'portions', 'catalyst', 'top', 'removed', 'to', 'compensate', 'loss', 'additional', 'catalysts', 'added', 'intermittently', 'upper', '“quiescent”', 'zones', 'in', 'suspension', 'slurry', 'polymerizations', 'various', 'suspending', 'agents', 'like', 'diesel', 'oil', 'lower', 'petro-', 'leum', 'fractions', 'heptane', 'toluene', 'mineral', 'oil', 'chlorobenzene', 'others', 'used', 'the', 'polymerization', 'temperatures', 'kept', '50', '75(cid1)c', 'slightly', 'elevated', 'pressures', 'like', '25', 'lbin2', 'if', 'batch', 'reactions', 'last', '1', '4', 'h', 'the', 'slurry', 'reactor', 'illustrated', 'fig', '62', 'polymerizations', 'catalyzed', 'transition', 'metal', 'oxides', 'support', 'described', 'variously', 'employing', 'solidliquid', 'suspensions', 'ﬁxed', 'beds', 'solidgas-phase', 'operations', 'it', 'appears', 'however', 'industrial', 'practices', 'mainly', 'conﬁned', 'use', 'solidliquid', 'suspension', 'processes', 'the', 'polymerization', 'carried', 'surface', 'catalyst', 'suspended', 'hydrocarbon', 'diluent', '336', '6', 'common', 'chain-growth', 'polymers', 'fig', '62', 'commercial', 'ﬂow', 'reactor', 'slurry', 'polymerization', 'ethylene', 'ziegler–natta', 'catalysts', 'illustrated', 'british', 'patent', '826', '523', 'in', 'continuous', 'slurry', 'processes', 'temperatures', 'kept', '90', '100(cid1)c', 'pressures', '400', '450', 'lbin2', 'the', 'catalyst', 'concentrations', 'range', '0004', '003', 'typical', 'diluents', 'n-pentane', 'n-hexane', 'individual', 'catalyst', 'particles', 'become', 'imbedded', 'polymer', 'granules', 'reaction', 'proceeds', 'the', 'granules', 'removed', 'slurry', 'containing', '20–40', 'solids', 'there', 'variations', 'individual', 'processes', 'in', 'procedures', 'temperature', 'kept', 'high', 'enough', 'keep', 'polymer', 'solution', 'in', 'others', 'kept', 'deliberately', 'low', 'maintain', 'polymer', 'slurry', 'the', 'products', 'separated', 'monomer', 'recycled', 'they', 'cooled', 'precipitated', '(if', 'solution)', 'collected', 'ﬁltration', 'centrifugation', 'various', 'reactors', 'developed', 'handle', 'different', 'slurry', 'polymerization', 'processes', 'the', 'slurry', 'maintained', 'suspension', 'ethylene', 'gas', 'the', 'gas', 'rises', 'top', 'maintains', 'agitation', 'polymer', 'particles', 'settle', 'bottom', 'collected', 'several', 'companies', 'adopted', 'loop', 'reactors', 'these', 'arranged', 'ﬂowing', 'reactants', 'diluents', 'continuously', 'pass', 'entrance', 'receiving', 'zone', 'the', 'heavier', 'particles', 'gravitate', 'ﬂowing', 'receiving', 'zone', 'lighter', 'diluents', 'reactants', 'recycled', 'to', 'accommodate', 'settling', 'area', 'must', 'large', 'enough', 'heavy', 'polymer', 'particles', 'collected', 'separated', 'in', 'addition', 'suspension', 'gas-phase', 'process', 'developed', 'no', 'diluent', 'used', 'polymeri-', 'zation', 'step', 'highly', 'puriﬁed', 'ethylene', 'gas', 'combined', 'continuously', 'dry-powdery', 'catalyst', 'fed', 'vertical', 'ﬂuidized', 'bed', 'reactor', 'the', 'reaction', 'carried', '270', 'psi', '85–100(cid1)c', 'the', 'circulating', 'ethylene', 'gas', 'ﬂuidizes', 'bed', 'growing', 'granular', 'polymer', 'serves', 'remove', 'heat', '15.', 'formed', 'polymer', 'particles', 'removed', 'intermittently', 'lower', 'sections', 'vertical', 'reactor', 'the', 'product', 'contains', '5', 'monomer', 'recovered', 'recycled', 'control', 'polymer', 'density', 'achieved', 'copolymerization', 'a-oleﬁns', 'molecular', 'weights', 'molecular', 'weight', 'distributions', 'controlled', 'catalyst', 'modiﬁcations', 'varying', 'operating', 'conditions', 'andor', 'use', 'chain', 'transferring', 'agents', '15,', 'hydrogen', '16.', 'this', 'illustrated', 'fig', '63', '61', 'polyethylene', 'related', 'polymers', '337', 'fig', '63', 'illustration', 'gas-phase', 'process', '(from', 'burdett', 'permission', 'american', 'chemical', 'society)', 'the', 'reactors', 'ﬂuidized', 'gas-phase', 'process', 'simple', 'design', 'there', 'mechanical', 'agitators', 'rely', 'upon', 'blowers', 'keep', 'bed', 'ﬂuidized', 'well', 'mixed', 'catalysts', 'cocatalysts', 'fed', 'directly', 'reactor', '25.', 'rieger', 'coworkers', '26', 'investigated', 'gas-phase', 'polymerization', 'ethylene', 'supported', 'a-diimine', 'nickel', 'catalysts', 'the', 'reaction', '25', '26', '14', 'dithiane', 'ligands', 'ni(acac)2', 'trityl', 'tetrakis(pentaﬂuorophenyl)borate', 'gave', 'corresponding', 'ni(ii)', 'complexes', 'high', 'yields', 'these', 'complexes', 'supported', 'silica', 'without', 'chemical', 'tether', 'used', 'catalysts', 'ethylene', 'polymerization', 'reactions', 'gas', 'phase', 'furthermore', 'ethylene', 'polymerized', 'unsup-', 'ported', '25-complexes', 'homogeneous', 'solution', 'comparison', 'the', 'inﬂuence', 'ligand', 'structure', 'hydrogen', 'temperature', 'polymerization', 'performance', 'investigated', 'the', 'supported', 'catalysts', 'showed', 'moderate', 'high', 'activities', 'produced', 'polyethylenes', 'ranging', 'high-density', 'polyethylene', 'linear', 'low-density', 'polyethylene', 'without', 'addition', 'a-oleﬁn', 'comonomer', 'b(c6f5)4', 'sio2', 'r', 'r', 'r', 'r', 'n', 'n', 'ni', 'r', 'r', 'o', 'o', 'linear', 'branched', 'pe', 'low', 'mw', 'linear', 'branched', 'pe', 'high', 'mw', '338', '6', 'common', 'chain-growth', 'polymers', 'table', '62', 'properties', 'commercial', 'polyethylene', 'properties', 'density', 'melting', 'point', 'amorphous', 'structure', 'free-radical', 'polymerization', '092–093', 'gcm3', '108–1107(cid1)c', '431', '20–30', 'ethyl', 'butyl', 'branches1000', 'carbons', 'ziegler–natta', 'type', 'catalysts', '094', 'gcm3', '129–131(cid1)c', '258', 'mainly', 'linear', '7', 'ethyl', 'branches', 'double', 'bonds', '06–21000', 'carbons', 'long', 'branches', 'types', 'bonds', '15', 'terminal', 'vinyl', '68', 'vinylidene', '17', 'internal', 'trans', 'oleﬁn', 'from', 'various', 'sources', 'literature', '1000', 'carbons', '01–11000', 'carbons', '43', 'terminal', '32', 'vinylidene', '25', 'internal', 'trans', 'oleﬁns', 'metal', 'oxides', 'support', '095–096', 'gcm3', '136(cid1)c', '258', 'almost', 'linear', 'up', '31000', 'carbons', '94', 'terminal', '1', 'vinylidene', '5', 'internal', 'trans', 'oleﬁns', 'the', 'weight', 'average', 'molecular', 'weights', 'commercial', 'low-', 'high-density', 'polyethylenes', 'range', '5000', '300000', 'very', 'low', 'molecular', 'weight', 'polyethylene', 'waxes', 'high', 'molecular', 'weight', 'materials', 'also', 'available', 'the', 'molecular', 'weight', 'distributions', 'high-density', 'polyethylene', 'vary', '4', '15', 'the', 'product', 'generally', 'fewer', 'three', 'branches', 'per', 'thousand', 'carbon', 'atoms', '9.', 'table', '62', 'summarizes', 'properties', 'various', 'polyethylenes', '614', 'materials', 'similar', 'polyethylene', 'materials', 'quite', 'similar', 'polyethylene', 'obtained', 'starting', 'materials', 'the', 'prominent', 'formation', 'polymethylene', 'similar', 'high', 'molecular', 'weight', 'parafﬁn', 'hydrocarbons', 'diazoalkanes', 'the', 'reaction', 'originally', 'carried', 'pechmann', '27', 'small', 'quantities', 'white', 'ﬂocculent', 'powder', 'formed', 'ether', 'solution', 'diazomethane', 'bamberger', 'tschirner', '28', 'showed', 'white', 'powder', 'polymethylene', '–(–ch2–)n–', 'melts', '128(cid1)c', 'the', 'synthesis', 'improved', 'since', 'introduction', 'various', 'catalysts', 'the', 'reaction', 'yield', 'highly', 'crystalline', 'polymers', 'melt', '1365(cid1)c', '29', 'molecular', 'weight', 'millions', '30.', 'among', 'catalysts', 'boron', 'compounds', 'efﬁcient', '30.', 'bawn', 'et', 'al', '31', 'postulated', 'mechanism', 'catalytic', 'action', 'it', 'consists', 'initial', 'coordination', 'monomer', 'initiator', 'bf3', 'this', 'followed', 'loss', 'nitrogen', 'shift', 'ﬂuorine', 'atom', 'boron', 'carbon', 'the', 'successive', 'additions', 'molecules', 'diazoalkane', 'follow', 'similar', 'path', 'shift', 'chain', 'fragment', 'electron-deﬁcient', 'carbon', 'n', 'ch2', 'n', 'bf3', 'n', 'n', 'f3b', 'ch2', 'n', 'n', 'f3b', 'ch2', 'n2', 'f3b', 'f', 'ch2', 'n2', 'f3b', 'f3b', 'ch2n2', 'f', 'f3b', 'n', 'n', 'f', 'n', 'ch2n2', 'f3b', 'f', 'f', 'n', '62', 'polypropylene', '339', 'the', 'resulting', 'macromolecules', 'still', 'reactive', 'toward', 'additional', 'diazoalkanes', 'the', 'step-growth', 'polymerization', 'reactions', 'also', 'yield', 'block', 'copolymers', 'f3b', 'n', 'f', 'r', 'formation', 'polymethylene', 'reaction', 'practical', 'commercial', 'utilization', 'colloidal', 'gold', 'ﬁne', 'copper', 'powder', 'also', 'catalyze', 'diazoalkane', 'polymerizations', 'the', 'reaction', 'appears', 'precede', 'formation', 'alkylidine', 'carbene', 'species', 'bound', 'surfaces', 'metals', '31–33.', 'the', 'initiations', 'completed', 'additions', 'diazoalkanes', 'bound', 'carbenes', 'followed', 'liberation', 'nitrogen', 'termination', 'may', 'take', 'place', 'chain', 'transfer', 'perhaps', 'monomer', 'solvent', '31–33.', 'many', 'different', 'diazoalkanes', 'lend', 'polymerization', 'reactions', 'polypentylidine', 'polyhexylidine', 'polyheptilidine', 'polyoctylidine', 'form', 'gold', 'complex', 'catalyst', 'aucl3-', 'pyridine', '34.', 'an', 'entirely', 'different', 'route', 'preparation', 'macroparafﬁns', 'high-pressure', 'reaction', 'hydrogen', 'carbon', 'monoxide', 'transition', 'metals', 'like', 'ﬁnely', 'divided', 'ruthenium', 'catalyze', 'reaction', 'at', 'pressures', '200', 'atm', 'temperatures', '140(cid1)c', 'polymethylene', 'molecular', 'weight', 'high', '100000', 'forms', '35:', 'h2', 'co', 'ru', 'n', '62', 'polypropylene', 'propylene', 'monomer', 'like', 'ethylene', 'obtained', 'petroleum', 'sources', 'free-radical', 'polymeri-', 'zations', 'propylene', 'a-oleﬁns', 'completely', 'controlled', 'chain', 'transferring', '36.', 'they', 'therefore', 'polymerized', 'coordination', 'polymerization', 'at', 'present', 'mainly', 'isotactic', 'polypropylene', 'used', 'large', 'commercial', 'quantities', 'also', 'utilization', 'atactic', 'polypropylene', 'well', 'syndiotactic', 'polypropylene', 'hand', 'still', 'mainly', 'laboratory', 'curiosity', 'the', 'polypropylene', 'originally', 'described', 'natta', 'contained', 'less', '50', 'isotactic', 'fractions', 'the', 'remainder', 'atactic', 'material', 'some', 'stereoblocks', 'composed', 'isotactic', 'atactic', 'polypropylenes', 'also', 'formed', 'this', 'type', 'product', 'forms', 'a-oleﬁns', 'polymerized', 'inert', 'hydrocarbons', 'catalysts', 'prepared', 'reducing', 'high', 'valence', 'metal', 'compounds', 'like', 'ticl4', 'organometallic', 'compounds', 'like', 'al(c2h5)', 'prepared', 'reducing', 'high', 'valence', 'metal', 'compounds', 'like', 'ticl4', 'organometallic', 'compounds', 'like', 'al(c2h5)3', 'later', 'heterogeneous', 'highly', 'crystalline', 'catalysts', 'based', 'transition', 'metals', '(valence', '3', 'less)', 'like', 'ticl2', 'ticl3', 'zrcl3', 'vcl3', 'developed', 'yielded', 'stereospeciﬁc', 'polypropylene', 'the', 'metal', 'halides', 'combined', 'selected', 'metal', 'alkyls', 'only', 'alkyls', 'picked', 'would', 'destroy', 'crystalline', 'lattice', 'transition', 'metal', 'salts', 'process', 'reaction', 'the', 'resultant', 'isotactic', 'material', 'catalysts', 'yielded', 'crystalline', 'polypropylenes', 'high', 'fractions', 'the', 'products', 'however', 'also', 'contained', 'low', 'molecular', 'weight', 'fractions', 'amorphous', 'stereoblock', 'materials', 'still', 'required', 'costly', 'puriﬁcation', 'separations', 'obtain', 'relatively', 'pure', 'isotactic', 'polypropylene', 'the', 'atactic', 'polymer', 'wax-like', 'substance', 'lacks', 'toughness', 'also', 'presence', 'amorphous', 'materials', 'low', 'molecular', 'weight', 'compounds', 'causes', 'tackiness', '340', '6', 'common', 'chain-growth', 'polymers', 'table', '63', 'polypropylenes', 'prepared', 'natta', '37', 'transition', 'metal', 'halide', 'ticl3', '(b)', 'ticl3', '(a', 'g', 'd)', 'ticl3', '(a', 'g', 'd)', 'ticl3', '(a', 'g', 'd)', 'ticl3', '(a', 'g', 'd)', 'ticl3', '(a', 'g', 'd)', 'vcl3', 'ticl2', 'metal', 'alkyl', 'halide', 'al(c2h5)3', 'al(c2h5)3', 'al(c2h5)2ci', 'be(c2h5)2', 'mg(c2h5)2', 'zn(c2h5)2', 'al(c2h5)3', 'al(c2h5)3', 'crystallinity', '40–50', '96–98', '96–98', '94–96', '78–85', '30–40', '73', '75', 'table', '64', 'effect', 'addition', 'lewis', 'bases', 'amount', 'crystalline', 'fraction', 'polypropylene', 'transition', 'metal', 'halides', 'ticl3', 'ticl3', 'ticl3', 'ticl3', 'ticl3', 'from', 'natta', 'et', 'al', '38', 'aluminum', 'alkyl', '2al(c2h5)br2', '2al(c2h5)cl2', '2al(c2h5)cl2', '2al(c2h5)br2', '2al(c2h5)cl2', 'crystallinity', '985', '95', '93', '99', '96', 'lewis', 'base', 'pyridine', 'n(c2h5)3', 'nh(c2h5)2', 'n(c4h9)4i(cid3)', 'n(c4h9)4br(cid3)', 'impedes', 'processing', 'table', '63', 'lists', 'catalysts', 'amounts', 'crystallinity', 'polymers', 'reported', 'natta', 'et', 'al', '37.', 'to', 'avoid', 'costly', 'puriﬁcation', 'isotactic', 'polypropylene', 'three-component', 'catalyst', 'systems', 'developed', 'some', 'original', 'ones', 'appear', 'reported', 'natta', 'found', 'addition', 'lewis', 'bases', 'enhances', 'quantity', 'crystalline', 'material', 'table', '64', 'shows', 'effects', 'addition', 'lewis', 'bases', 'amount', 'crystallinity', 'reported', 'natta', 'et', 'al', '38.', 'many', 'three-component', 'systems', 'developed', 'since', '39–43.', 'also', 'development', 'active', 'catalysts', '44', '45', 'eliminates', 'need', 'remove', 'ﬁnished', 'product', '15.', 'the', 'ﬁrst', 'improvement', 'catalyst', 'productivity', 'came', 'treating', 'ticl3', '(formed', 'ticl4', 'al(c2h5)cl2)', 'aliphatic', 'ethers', 'resulting', 'yields', '520', 'g', 'polymer', 'gram', 'ti', '46.', 'further', 'improvement', 'achieved', 'supporting', 'ticl3', 'mgcl2', 'producing', 'supported', 'catalyst', 'reacting', 'ticl4', 'mg(oc2h5)', 'magnesium', 'compounds', 'this', 'raised', 'productivity', '3000', 'g', 'polymer', 'every', 'gram', 'ti', '46.', 'the', 'products', 'however', 'contained', 'low', 'percentages', '(20–40)', 'addition', 'lewis', 'base', 'like', 'nnn0n0-tetramethyl', 'ethylenediamine', 'solid', 'component', 'ethyl', 'benzoate', 'solution', 'raised', 'isotactic', 'content', '93', 'productivity', '2500', 'g', 'polymer', 'per', 'gram', 'ti', '41.', 'claims', 'made', 'today', 'much', 'greater', 'catalyst', 'activity', 'it', 'reported', 'instance', 'catalyst', 'efﬁciencies', '40', 'kg', 'polymer', 'per', '1', 'g', 'ti', 'achieved', 'such', 'yields', 'require', 'proper', 'choice', 'catalysts', 'control', 'polymeri-', 'zation', 'conditions', 'the', 'isotactic', 'fractions', 'products', 'reported', 'range', '95', '97', '47–49.', 'isotactic', 'isomer', 'in', 'catalyst', 'system', 'ticl3mgcl2c6h5cooc2h5al(c2h5)3', 'high', 'activity', 'initially', 'attributed', 'higher', 'propagation', 'rates', 'rather', 'increase', 'concentration', 'active', 'sites', '50.', 'the', 'higher', 'activity', 'catalysts', 'however', 'shown', 'instead', 'due', 'higher', 'numbers', 'active', 'centers', 'slightly', 'higher', 'values', 'kp', '51.', 'subsequent', 'trends', 'modiﬁcations', 'supported', 'ziegler–natta', 'catalysts', 'consisted', 'using', 'sterically', 'hindered', 'amines', '52–54.', 'for', 'instance', '62', 'polypropylene', '341', '2266-tetramethylpiperidine', 'might', 'used', 'together', 'different', 'trialkylaluminum', 'compounds', 'modiﬁer-cocatalyst', 'systems', 'supported', 'catalysts', 'support', 'tix4', 'alr3', 'n', 'x', 'represents', 'halogen', 'other', 'analogous', 'amines', 'like', '124-trimethylpiperazine', '2345-tetraethylpiperidine', 'also', 'used', 'preparations', 'titanium', 'halide', 'catalysts', 'supported', 'mgcl2', 'the', 'amine', 'remains', 'built-in', 'modiﬁer', 'catalyst', 'system', '53.', 'subsequent', 'research', 'efforts', 'concentrated', 'soluble', 'catalytic', 'systems', 'like', 'di-z5-cyclopentadie-', 'nyldiphenyltitanium', 'tetrabenzylzirconium', 'complexed', 'methylaluminoxane', '(ch3)2al–', '–o–al(ch3)–n–al(ch3)2', 'such', 'catalysts', 'however', 'yield', 'products', 'contain', '85', 'isotactic', 'polypropylene', '55–61,', 'reactions', 'conducted', 'low', 'temperatures', '(cid3)45(cid1)c', 'lower', 'a', 'major', 'breakthrough', 'occurred', 'rigid', 'chiral', 'metallocene', 'initiators', 'developed', 'like', '11-ethylene-di-z5-indenylzirconium', 'dichloride', 'complexed', 'methylaluminoxane', 'in', 'place', 'zirconium', 'titanium', 'hafnium', 'analogs', 'also', 'used', 'these', 'catalysts', 'highly', 'isospeciﬁc', '62–64', 'used', 'low', 'temperatures', 'the', 'compounds', 'illustrated', 'chap', '4', 'typical', 'catalysts', 'consist', 'aluminum', 'transition', 'metal', 'ratios', '103', '1041', 'many', 'yield', '98–99', 'isotactic', 'fractions', 'polymer', 'in', 'addition', 'active', 'catalysts', 'yielding', 'large', 'quantities', 'polymer', 'per', 'gram', 'zirconium', 'it', 'also', 'reported', 'elastomeric', 'polypropylenes', 'formed', 'monomer', 'aid', 'metallocene', 'catalysts', '62–64.', 'because', 'rigid', 'chiral', 'metallocene', 'catalysts', 'produce', 'isotactic', 'polypropylene', 'achiral', 'ones', 'produce', 'atactic', 'form', 'waymouth', 'coates', '62', 'prepared', 'bridged', 'metallocene', 'catalyst', 'indenyl', 'ligands', 'rotate', 'metal–ligand', 'bond', 'axis', 'the', 'rotation', 'causes', 'catalyst', 'isomerize', 'chivalric', 'nonchiralic', 'geometries', 'isotactic', 'block', 'zr', 'zr', 'atactic', 'block', 'indenyl', 'ligands', 'however', 'found', 'rotate', 'faster', 'polymerization', 'reaction', 'this', 'prevents', 'formation', 'stereoregular', 'polymer', 'blocks', '62.', 'to', 'overcome', 'phenyl', 'substituents', 'added', 'ligands', 'slow', 'rotation', 'speed', 'monomer', 'insertion', 'yet', 'rotate', 'faster', 'time', 'required', 'formation', 'whole', 'polymeric', 'chain', 'the', 'product', 'catalytic', 'system', 'bis(2-phenylindenyl)zirconium', 'dichloride', 'plus', 'methylaluminoxane', 'found', 'yield', 'elastomeric', 'block', 'copolymers', 'isotactic', 'atactic', 'polypropylene', '62.', 'vincenzo', 'et', 'al', '63', 'reported', '13c', 'nmr', 'microstructural', 'analysis', 'polypropylene', 'samples', 'produced', 'two', 'representative', '“oscillating”', 'metallocene', 'catalysts', 'found', 'largely', 'different', 'steric', 'hindrance', 'the', 'original', 'mechanistic', 'proposal', '“oscillation”', 'two', 'enantio-', 'morphous', 'racemic-like', '(isotactic-selective)', 'meso-like', '(non-stereoselective)', 'conformation', 'according', 'cannot', 'explain', 'observed', 'polymer', 'conﬁguration', 'they', 'feel', 'isotactic-stereoblock', 'nature', 'polymers', 'obtained', 'catalyst', 'proves', 'unambiguously', 'active', 'cation', '“oscillates”', 'two', 'enantiomorphous', 'racemic-like', 'conformations', 'average', 'frequency', 'even', 'high', 'propene', 'concentration', 'slightly', 'lower', 'monomer', 'insertion', 'the', 'less', 'hindered', 'catalyst', 'gives', 'instead', 'largely', 'stereoirregular', '342', '6', 'common', 'chain-growth', 'polymers', 'polypropylene', 'logical', 'consequence', 'faster', 'ligand', 'rotation', 'however', 'depending', 'use', 'conditions', '(in', 'particular', 'nature', 'cocatalyst', 'polarity', 'solvent)', 'polymerization', 'products', 'may', 'also', 'contain', 'appreciable', 'amounts', 'fairly', 'isotactic', 'fraction', 'the', 'peculiar', 'microstructure', 'fraction', 'isotactic', 'blocks', 'relative', 'conﬁguration', 'spanned', 'short', 'atactic', 'ones', 'rules', 'possibility', 'latter', 'due', 'active', 'species', 'meso-like', 'conformation', 'point', 'rather', 'conformationally', '“locked”', 'racemic-like', 'species', 'restricted', 'ring', 'mobility', 'the', 'hypothesis', 'stereorigidity', 'induced', 'proximity', 'counter', 'anion', 'would', 'play', 'role', 'inter-annular', 'bridge', 'racemic-bis(indenyl)', 'ansa-metallocenes', 'tested', 'computer', 'modeling', 'found', 'viable', 'preparation', 'elastomeric', 'polypropylenes', 'also', 'reported', 'chien', 'et', 'al', '64.', 'two', 'metallocene', 'catalysts', 'different', 'stereospeciﬁcities', 'used', 'the', 'isospeciﬁc', 'catalyst', 'precursors', 'either', 'rac-ethylene', 'bis-(1-z5-indenyl)zirconium', 'dichloride', 'rac-dimethylsilylene', 'bis(1-z5-indenyl)zirco-', 'nium', 'dichloride', 'the', 'unspeciﬁc', 'one', 'ethylene', 'bis(9-z5-ﬂuorenyl)zirconium', 'dichloride', 'the', 'precursors', 'activated', 'triphenyl', 'carbenium', 'tetrakis(pentaﬂuorophenyl)borate', 'triisobutylaluminum', 'the', 'resultant', 'catalysts', 'exhibit', 'high', 'activity', 'yielding', 'products', 'range', 'tough', 'plastomers', 'weak', 'elastomers', '64.', '621', 'manufacturing', 'techniques', 'the', 'earliest', 'commercial', 'methods', 'used', 'slurry', 'polymerizations', 'liquid', 'hydrocarbon', 'diluents', 'like', 'hexane', 'heptane', 'these', 'diluents', 'carried', 'propylene', 'catalyst', 'small', 'amounts', 'hydrogen', 'fed', 'reaction', 'mixtures', 'control', 'molecular', 'weights', 'the', 'catalyst', 'system', 'consisted', 'deep', 'purple', 'violet-colored', 'ticl3', 'reacted', 'diethyl', 'aluminum', 'chloride', 'the', 'ticl3', 'often', 'prepared', 'reduction', 'ticl4', 'aluminum', 'powder', 'these', 'reactions', 'carried', 'stirred', 'autoclaves', 'temperatures', '90(cid1)c', 'pressures', 'sufﬁcient', 'maintain', 'liquid', 'phase', 'the', 'concentration', 'propylene', 'reaction', 'mixtures', 'ranged', '10', '20.', 'the', 'products', 'formed', 'discrete', 'particles', 'removed', '20–40', 'concentrations', 'solids', 'unreacted', 'monomer', 'withdrawn', 'product', 'mixtures', 'reused', 'the', 'catalysts', 'deactivated', 'dissolved', 'products', 'alcohol', 'containing', 'hcl', 'removed', 'steam', 'extraction', 'this', 'followed', 'extraction', 'amorphous', 'fractions', 'hot', 'liquid', 'hydrocarbons', 'later', 'bulk', 'polymerization', 'processes', 'developed', 'liquid', 'propylene', 'either', 'used', 'diluent', 'loop', 'reactor', 'permitted', 'boil', 'remove', 'heat', 'reaction', 'the', 'second', 'done', 'stirred', 'vessels', 'vapor', 'space', 'top', 'more', 'recently', 'gas-phase', 'polymerizations', 'propylene', 'introduced', 'the', 'technology', 'similar', 'gas-phase', 'technology', 'ethylene', 'polymerizations', '15', 'described', 'sect', '61', '622', 'syndiotactic', 'polypropylene', 'isotactic', 'polypropylene', 'received', 'attention', 'commercially', 'desirable', 'nevertheless', 'syndiotactic', 'polypropylene', 'though', 'less', 'crystalline', 'greater', 'clarity', 'elasticity', 'impact', 'resis-', 'tance', 'it', 'melts', 'however', 'lower', 'temperature', 'this', 'isomer', 'originally', 'prepared', 'heterogeneous', 'titanium-based', 'catalysts', 'soluble', 'vanadium-based', 'ones', 'the', 'heterogeneous', 'catalysts', 'gave', 'low', 'yields', 'syndiotactic', 'fractions', 'in', 'fact', 'original', 'samples', 'contained', 'percent', 'desired', 'material', 'almost', 'impurity', 'the', 'yield', 'syndiotactic', 'polypropylene', 'increased', 'decrease', 'polymerization', 'temperature', 'still', 'remained', 'low', '65.', 'highly', 'syndiotactic', 'polypropylene', 'prepared', 'natta', 'et', 'al', '38', 'homogeneous', 'catalysts', 'formed', 'vcl4', 'vanadium', 'tri-acetylacetonate', 'aluminum', 'dialkyl', 'halide', 'anisole', '(cid3)48', '(cid3)78(cid1)c', '63', 'polyisobutylene', '343', 'table', '65', 'comparison', 'isotactic', 'syndiotactic', 'polypropylenes', 'isomer', 'isotactic', 'crystal', 'structure', 'monoclinic', 'triclinic', 'hexagonal', 'orthorhombic', 'syndiotactic', '089–091', 'gcm3', '138', 'density', '25(cid1)c', '092–043', 'gcm3', '0943', 'gcm3', 'mp', '((cid1)c)', '171–186', 'typical', 'mw', '220–700', 'k', 'mn', '38–160', 'k', 'no', 'isotactic', 'fractions', 'formed', 'this', 'led', 'development', 'many', 'effective', 'soluble', 'catalysts', 'the', 'catalyst', 'components', 'conditions', 'preparation', 'quite', 'important', 'maintaining', 'control', 'syndiotactic', 'placement', 'for', 'effective', 'soluble', 'catalyst', 'ratio', 'air2x', 'vanadium', 'compound', 'maintained', '3', '10', '66.', 'the', 'organic', 'portion', 'organoaluminum', 'compound', 'either', 'methyl', 'ethyl', 'isobutyl', 'neopentyl', 'phenyl', 'methylstyryl', '9', '67.', 'in', 'addition', 'vcl4', 'vanadium', 'tri-acetylacetonate', '66,', 'various', 'vanadates', 'used', 'like', 'vo', '(or)xcl3(cid3)x,', 'x', '¼', '1', '2', '3', '65.', 'the', 'exact', 'nature', 'vanadium', 'compound', 'however', 'important', 'resultant', 'steric', 'arrangement', 'product', 'for', 'instance', 'vcl4', 'combined', 'al', '(c2h5)2f', 'forms', 'heterogeneous', 'catalyst', 'yields', 'isotactic', 'isomer', '65.', 'vanadium', 'tri-', 'acetylacetonate', 'hand', 'upon', 'reacting', 'al(c2h5)2f', 'forms', 'soluble', 'catalyst', 'yields', 'syndiotactic', 'isomer', '66.', 'addition', 'certain', 'electron', 'donors', 'increases', 'amount', 'syndiotactic', 'placement', 'these', 'anisole', 'furan', 'diethyl', 'ether', 'cycloheptanone', 'ethyl', 'acetate', 'thiophene', '67.', 'the', 'optimum', 'results', 'obtained', 'anisole', 'vanadium', 'ratio', '11', 'also', 'highest', 'amount', 'syndiotactic', 'polymer', 'obtained', 'soluble', 'catalysts', 'prepared', 'used', 'low', 'temperatures', 'even', 'low', 'temperatures', 'however', 'like', '(cid3)78(cid1)c', 'amount', 'syndiotacticity', 'obtained', 'speciﬁc', 'catalyst', 'decreases', 'time', '65', '66', '68.', 'this', 'indicates', 'deterioration', 'syndiotactic', 'placing', 'sites', 'on', 'hand', 'polymerization', 'propylene', 'soluble', 'vanadium', 'tri-acetylacetonate–al(c2c5)2cl', 'system', 'reported', '“living”', 'type', 'polymerization', '69.', 'the', 'product', 'narrow', 'molecular', 'weight', 'distribution', '(mwmn', '¼', '105–120)', 'a', 'kinetic', 'study', 'indicates', 'absence', 'chain', 'transferring', 'termination', 'temperatures', '(cid3)65(cid1)c', 'more', 'recent', 'catalysts', 'syndiotactic', 'polypropylene', 'complexes', 'like', 'i-propyl(cyclopen-', 'tadienyl-1-ﬂuorenyl)hafnium', 'dichloride', 'methyl', 'aluminoxane', '70.', 'another', 'similar', 'catalyst', 'i-propyl(z5-cyclopentadienyl-z3-ﬂuorenyl)zirconium', 'dichloride', 'methyl', 'aluminoxane', 'these', 'catalysts', 'yield', 'polymers', 'high', 'syndiotactic', 'material', '(the', 'zirconium-based', 'compound', 'yields', '86', 'racemic', 'pentads)', '70', '71.', 'commercial', 'production', 'syndiotactic', 'polypropylene', 'early', 'stages', 'what', 'catalytic', 'system', 'used', 'however', 'disclosed', 'time', 'some', 'properties', 'two', 'isomers', 'isotactic', 'syndiotactic', 'polypropylenes', 'compared', 'table', '65', 'the', 'molecular', 'weights', 'syndiotactic', 'polypropylenes', 'vary', 'number', 'average', 'molecular', 'weight', '25000–60000', 'depending', 'upon', 'reaction', 'conditions', '70.', 'also', 'isotactic', 'polypropylene', 'less', 'one', 'double', 'bond', 'per', '1000', 'carbon', 'atoms', '72.', 'a', 'typical', 'mwmn', '¼', '5–12', '63', 'polyisobutylene', 'the', 'original', 'commercial', 'methods', 'preparing', 'high', 'molecular', 'weight', 'polyisobutylene', 'cationic', 'polymerization', 'good', 'yields', 'reported', '1940', 'the', 'reaction', 'carried', '(cid3)40', '(cid3)80(cid1)c', 'diluent', 'bf3', 'catalysis', '72.', 'this', 'developed', 'current', 'commercial', 'practices', 'polymerizing', 'isobutylene', '(cid3)80', '(cid3)100(cid1)c', 'using', 'liquid', 'ethylene', 'methyl', 'chloride', 'diluent', '73', '74.', 'even', 'low', 'temperatures', 'reaction', 'quite', 'violent', 'methods', 'developed', 'therefore', 'dissipate', '344', '6', 'common', 'chain-growth', 'polymers', 'heat', 'in', 'one', 'called', '“ﬂash', 'polymerization', 'process”', 'catalyst', '(a', 'lewis', 'acid', 'like', 'bf3', 'alcl3', 'instance)', 'added', 'solution', 'cooled', 'isobutylene', 'solution', 'the', 'polymerization', 'takes', 'place', 'rapidly', 'complete', 'seconds', 'heat', 'reaction', 'removed', 'vaporization', 'diluent', 'such', 'reactions', 'however', 'difﬁcult', 'carry', 'conventional', 'batch', 'reactors', 'two', 'types', 'procedures', 'therefore', 'adopted', '75.', 'the', 'ﬁrst', 'one', 'built', 'around', 'moving', 'stainless', 'steel', 'belt', 'contained', 'inside', 'gas-tight', 'reactor', 'housing', 'isobutylene', 'liquid', 'ethylene', 'one', 'source', 'lewis', 'acid', 'ethylene', 'solution', '(01–03', 'based', 'monomer)', 'another', 'source', 'fed', 'continuously', 'onto', 'moving', 'belt', 'mixed', 'moved', 'the', 'movement', 'belt', 'adjusted', 'speed', 'polymerization', 'complete', 'polymer', 'arrives', 'end', 'travel', 'removed', 'scraper', 'processed', 'in', 'second', 'process', 'polymerization', 'carried', 'multiple', 'kneaders', 'mixers', 'these', 'arranged', 'series', 'descending', 'steps', 'here', 'reaction', 'mixture', 'carried', 'one', 'kneader', 'another', 'temperature', 'raised', 'station', 'completed', 'last', 'one', 'all', 'commercially', 'important', 'polyisobutylenes', 'linear', 'head', 'tail', 'polymers', 'tertiary', 'butyl', 'groups', 'one', 'end', 'chains', 'vinylidene', 'groups', 'n', 'the', 'differences', 'lie', 'molecular', 'weights', 'they', 'range', '2000', '20000', 'viscous', 'liquids', '100000', '400000', 'high', 'molecular', 'weight', 'elastomers', 'resemble', 'unmilled', 'crepe', 'rubber', 'the', 'polymers', 'degrade', 'readily', 'thermal', 'abuse', 'they', 'stabilized', 'effectively', 'however', 'adding', 'small', 'quantities', '(01–10)', 'stabilizers', 'aromatic', 'amines', 'phenols', 'sulfur', 'compounds', 'polyisobutylenes', 'soluble', 'many', 'hydrocarbons', 'resistant', 'attacks', 'many', 'chemicals', 'coordination', 'polymerizations', 'ziegler–natta', 'catalysts', 'yield', 'similar', 'polymers', 'range', 'viscous', 'liquids', 'rubbery', 'solids', 'at', '0(cid1)c', 'catalyst', '116', 'ti', 'al', 'molar', 'ratio', 'yields', 'polymer', 'molecular', 'weight', '5000–6000', '76.', 'the', 'molecular', 'weight', 'however', 'dependent', 'upon', 'reaction', 'time', 'this', 'contrasts', 'polymerizations', 'ethylene', 'propylene', '1-butene', 'catalysts', 'molecular', 'weights', 'products', 'independent', 'reaction', 'time', 'in', 'addition', 'questions', 'exact', 'molecular', 'structures', 'products', '76.', 'bochmann', 'coworkers', '77', 'carried', 'polymerizations', 'isobutylene', 'copolymerizations', 'isoprene', 'using', 'cationic', 'zirconocene', 'hydride', 'complexes', 'the', 'combination', 'cp2zrh', 'various', 'trityl', 'salts', 'weakly', 'coordinating', 'anions', 'gives', 'binuclear', 'cationic', 'hydrides', 'cp0', '4zr2h', '(m-h)2+x(cid3)', 'powerful', 'initiators', 'polymerization', 'isobutene', 'copolymeriza-', 'tion', 'isoprene', 'the', 'temperature', 'dependence', 'm', 'indicative', 'cationic', 'mechanism', 'the', 'highest', 'molecular', 'weights', 'obtained', 'scrupulously', 'dry', 'conditions', 'high', 'molecular', 'weight', 'polyisobutylene', 'fair', 'tensile', 'strength', 'suffers', 'disadvantage', 'considerable', 'cold', 'ﬂow', 'a', 'copolymer', 'isobutylene', 'isoprene', 'cross-linking', 'therefore', 'used', 'commercial', 'elastomer', 'called', '“butyl', 'rubber”', 'the', 'isoprene', 'present', 'copolymer', 'minor', 'proportions', '(14–45)', 'the', 'uncross-linked', 'material', 'similar', 'polyisobutylene', 'copolymers', 'isobutylene', 'dienes', 'also', 'called', 'butyl', 'rubbers', 'they', 'also', 'terpolymers', 'third', 'component', 'may', 'cyclopentadiene', 'improved', 'ozone', 'resistance', 'the', 'molecular', 'weights', 'copolymers', 'vary', 'inversely', 'quantities', 'isoprene', 'incorporated', 'polymerization', 'temperatures', 'amount', 'impurities', 'present', 'polymeriza-', 'tion', 'impurities', 'like', 'n-butene', 'water', 'act', 'chain', 'transferring', 'agents', '79.', 'to', 'maintain', 'uniform', 'molecular', 'weights', 'conversions', 'usually', 'kept', 'exceeding', '60.', '64', 'poly(a-oleﬁn)s', '64', 'poly(a-oleﬁn)s', '345', 'many', 'a-oleﬁns', 'polymerized', 'ziegler–natta', 'catalysts', 'yield', 'high', 'polymers', 'many', 'polymers', 'found', 'stereospeciﬁc', 'crystalline', 'polymerizations', 'a-oleﬁns', 'general', 'structure', 'ch2', 'ch', '(cid3)', 'ðch2þx', '(cid3)', 'r', 'x', '0–3', 'r', 'denotes', 'ch3', 'ch–(ch3)2', 'c(ch3)3', 'c6h5', 'catalyzed', 'vanadium', 'trichloridetriethyl', 'aluminum', '80.', 'the', 'conversions', 'fairly', 'high', 'though', 'higher', 'crystallinity', 'obtained', 'titanium-based', 'catalysts', '81.', 'addition', 'lewis', 'bases', '(c4h9)2o', '(c4h9)3n', '(c4h9)3p', 'catalyst', 'system', 'increases', 'crystallinity', '82.', '641', 'properties', 'poly(a-oleﬁn)s', 'many', 'poly(a-oleﬁn)s', 'reported', 'literature', 'used', 'commercially', 'various', 'reasons', 'table', '66', 'lists', 'oleﬁns', 'polymerized', 'ziegler–natta', 'catalysts', '72', '83.', '642', 'poly(butene-1)', 'isotactic', 'poly(butene-1)', 'produced', 'commercially', 'three-component', 'coordination-type', 'catalysts', 'it', 'manufactured', 'continuous', 'process', 'simultaneous', 'additions', 'reaction', 'vessel', 'monomer', 'solution', 'suspension', 'ticl2–alcl3', 'solution', 'diethyl', 'aluminum', 'chloride', '84.', 'the', 'efﬂuent', 'containing', 'suspension', 'product', 'continually', 'removed', 'reactor', 'molecular', 'weight', 'control', 'achieved', 'regulating', 'reaction', 'temperature', 'the', 'efﬂuent', 'contains', 'approximately', '5–8', 'atactic', 'polybutene', 'dissolved', 'liquid', 'carrier', 'the', 'suspended', 'isotactic', 'fractions', '(92–98)', 'isolated', 'catalyst', 'decomposition', 'removal', 'the', 'product', 'density', '092', 'gcm3', 'melts', '124–130(cid1)c', 'isotactic', 'polybutene', 'crystallizes', 'three', 'different', 'forms', 'when', 'cools', 'melt', 'originally', 'crystallizes', 'metastable', 'crystalline', 'one', 'after', 'several', 'days', 'however', 'transforms', 'different', 'form', 'noticeable', 'changes', 'melting', 'point', 'density', 'ﬂexural', 'modulus', 'yield', 'hardness', 'accompany', 'transformation', 'the', 'third', 'crystalline', 'form', 'results', 'crystallization', 'solution', 'the', 'polymer', 'exhibits', 'good', 'impact', 'tear', 'resistance', 'it', 'also', 'resistant', 'environmental', 'stress-cracking', '643', 'poly(4-methyl', 'pentene-1)', 'another', 'commercially', 'produced', 'polyoleﬁn', 'isotactic', 'poly(4-methyl', 'pentene-1)', 'the', 'polymer', 'carries', 'trade', 'name', 'tpx', 'this', 'material', 'known', 'high', 'transparency', 'good', 'electrical', 'properties', 'heat', 'resistance', 'poly(4-methyl', 'pentene-1)', 'density', '083', 'gcm3', 'this', 'polyoleﬁn', 'exhibits', 'poor', 'load-bearing', 'properties', 'susceptible', 'uv', 'degradation', 'it', 'also', 'poor', 'barrier', 'moisture', 'gases', 'scratches', 'readily', 'this', 'limits', 'use', 'many', 'applications', 'poly(4-methyl', 'pentene)', 'produced', 'process', 'equipment', 'polypropylene', 'a', 'post', 'ﬁnishing', 'de-ashing', 'step', 'however', 'required', 'in', 'addition', 'aseptic', 'conditions', 'maintained', 'manufacture', 'prevent', 'contamination', 'may', 'affect', 'clarity', '346', '6', 'common', 'chain-growth', 'polymers', 'table', '66', 'properties', 'poly(a-oleﬁn)s', 'monomer', '3', '9', '13', 'crystallinity', 'pendant', 'groups', '70', '100', 'state', 'crystalline', 'crystalline', 'crystalline', 'crystalline', 'rubber', 'amorphous', 'some', 'crystallinity', 'crystalline', 'hard', 'crystalline', 'hard', 'crystalline', 'hard', 'crystalline', 'hard', 'crystalline', 'hard', 'rubber', 'amorphous', 'crystalline', 'slightly', 'rubbery', 'crystalline', 'intractable', 'rubber', 'amorphous', 'mp', '((cid1)c)', '136', '165–168', '124–130', '75', '–', '45', '240–285', '200–240', '300–350', '350', '160', '–', '158', '360', '–', 'afrom', 'refs', '72', '83', 'a', 'number', 'similar', 'polyoleﬁns', 'pendant', 'side', 'groups', 'known', 'these', 'include', 'poly(3-methyl', 'butene-1)', 'poly(44-dimethyl', 'pentene-1)', 'poly(vinyl', 'cyclohexane)', 'due', 'increased', 'cohe-', 'sive', 'energy', 'ability', 'pack', 'tight', 'structures', 'effect', 'increasing', 'stiffness', 'pendant', 'groups', 'polymers', 'high', 'melting', 'point', 'this', 'seen', 'table', '69', 'many', 'polymers', 'however', 'tend', 'undergo', 'complex', 'morphological', 'changes', 'standing', 'this', 'result', 'ﬁssures', 'planes', 'weakness', 'structure', '65', 'copolymers', 'ethylene', 'propylene', '347', '65', 'copolymers', 'ethylene', 'propylene', 'many', 'monomers', 'copolymerized', 'ethylene', 'variety', 'polymerization', 'methods', 'when', 'ethylene', 'copolymerized', 'oleﬁns', 'resultant', 'hydrocarbon', 'polymers', 'reduced', 'regularity', 'lower', 'density', 'lower', 'softening', 'point', 'lower', 'brittle', 'point', 'copolymers', 'ethylene', 'propylene', 'commercially', 'important', 'family', 'materials', 'they', 'vary', 'elastomers', 'contain', '80', 'ethylene', '20', 'propylene', 'polypropylene', 'modiﬁed', 'small', 'amounts', 'ethylene', 'improve', 'impact', 'resistance', 'metallocene', 'catalysts', 'produce', 'random', 'alternating', 'copolymers', 'ethylene', 'propylene', '85.', 'at', 'present', 'appear', 'commercial', 'utilization', 'alternating', 'copolymers', 'they', 'reported', 'form', 'polymerizations', 'catalyzed', 'bridged', 'ﬂuorenyl', 'catalysts', '85.', '651', 'ethylene', 'propylene', 'elastomers', 'the', 'commercial', 'ethylene-propylene', 'rubbers', 'typically', 'range', 'propylene', 'content', '30', '60,', 'depending', 'upon', 'intended', 'use', 'such', 'copolymers', 'prepared', 'ziegler–natta', 'type', 'catalysts', 'soluble', 'catalysts', 'true', 'solution', 'processes', 'preferred', 'the', 'common', 'catalyst', 'systems', 'based', 'vcl4', 'vocl3', 'v(acac)3', 'vo(or)3', 'vocl(or)2', 'vocl2(or)', 'etc', 'various', 'organoaluminum', 'derivatives', 'the', 'products', 'predominantly', 'amorphous', 'polymerization', 'reactions', 'usually', 'carried', '40(cid1)c', 'solvents', 'like', 'chlorobenzene', 'pentane', 'the', 'resultant', 'random', 'copolymers', 'recovered', 'alcohol', 'precipitation', 'because', 'elastomers', 'almost', 'completely', 'saturated', 'cross-linking', 'difﬁcult', 'a', 'third', 'monomer', 'diene', 'included', 'preparation', 'rubbers', 'carry', 'trade', 'names', 'eptr', 'epdm', 'inclusion', 'third', 'monomers', 'presents', 'problems', 'copolymerization', 'reactions', 'for', 'instance', 'important', 'maintain', 'constant', 'feed', 'mixtures', 'monomers', 'obtain', 'constant', 'compositions', 'yet', 'two', 'three', 'monomers', 'gaseous', 'third', 'one', 'liquid', 'natta', '86', 'developed', 'technique', 'depends', 'upon', 'maintaining', 'violent', 'agitation', 'solvent', 'gaseous', 'monomers', 'bubbled', 'liquid', 'phase', 'this', 'referred', '“semi-ﬂow', 'technique”', 'the', 'process', 'allows', 'compositions', 'gaseous', 'liquid', 'phases', 'equilibrium', 'less', 'constant', '87.', 'other', 'techniques', 'evolved', 'since', 'all', 'designed', 'maintain', 'constant', 'polymeri-', 'zation', 'mixtures', 'therefore', 'the', 'vanadium-based', 'catalyst', 'systems', 'deteriorate', 'time', 'decrease', 'number', 'catalytic', 'centers', 'polymerizations', 'progress', 'the', 'rate', 'decay', 'affected', 'conditions', 'used', 'catalyst', 'preparation', 'compositions', 'catalysts', 'temperature', 'solvents', 'lewis', 'bases', 'it', 'also', 'affected', 'type', 'concentration', 'third', 'monomer', '88–90.', 'additions', 'chlorinated', 'compounds', 'deactivated', 'catalysts', 'however', 'help', 'restore', 'activity', '91', '92.', 'catalyst', 'decay', 'also', 'overcome', 'continually', 'feeding', 'catalyst', 'components', 'polymerization', 'medium', '93.', 'while', 'third', 'monomer', 'common', 'diene', 'like', 'isoprene', 'often', 'bridged', 'ring', 'structure', 'least', 'one', 'double', 'bond', 'ring', 'in', 'typical', 'terpolymer', 'rubbers', '60–40', 'ratios', 'ethylene', 'propylene', 'diene', 'components', 'usually', 'comprise', '3', 'total', 'some', 'specialty', 'rubbers', 'however', 'may', 'contain', '10', 'diene', 'even', 'reaction', 'conditions', 'always', 'chosen', '348', '6', 'common', 'chain-growth', 'polymers', 'obtain', '12', 'placement', 'diene', 'dienes', 'common', 'use', 'ethylidine', 'norbornene', 'methylene', 'norbornene', '14-hexadiene', 'dicyclopentadiene', 'cycloocatadiene', 'ethylidine', 'norbornene', 'methyle', 'norbornene', '14-hexadiene', 'dicyclopentadiene', 'cyclooctadiene', 'in', 'addition', 'patent', 'literature', 'describes', 'many', 'dienes', 'an', 'idealized', 'picture', 'segment', 'uncross-linked', 'gum', 'stock', 'might', 'shown', 'following', 'structure', '652', 'copolymers', 'ethylene', 'a-oleﬁns', 'ethylene', 'carbon', 'monoxide', 'many', 'copolymers', 'ethylene', 'a-oleﬁns', 'prepared', 'commercially', 'thus', 'ethylene', 'copolymerized', 'butene-1', 'comonomer', 'included', 'lower', 'regularity', 'density', 'polymer', 'many', 'copolymers', 'prepared', 'transition', 'metal', 'oxide', 'catalysts', 'support', 'the', 'comonomer', 'usually', 'present', 'approximately', '5', 'quantities', 'this', 'sufﬁcient', 'lower', 'crystallinity', 'markedly', 'improve', 'impact', 'strength', 'resistance', 'environmental', 'stress-', 'cracking', 'copolymers', 'ethylene', 'hexene-1', 'hexene-1', 'content', 'less', '5,', 'also', 'produced', 'reason', 'in', 'cases', 'monomers', 'homopolymerize', 'ziegler–natta', 'coordination', 'catalysts', 'also', 'copolymerize', '94.', 'in', 'addition', 'monomers', 'homopolymerize', 'may', 'still', 'copolymerize', 'form', 'alternating', 'copolymers', 'because', 'lifetime', 'growing', 'polymer', 'molecule', 'relatively', 'long', '(can', 'long', 'several', 'minutes)', 'block', 'copolymerization', 'possible', 'changes', 'monomer', 'feeds', 'also', 'nature', 'transition', 'metal', 'compound', 'inﬂuences', 'reactivity', 'ratios', 'monomers', 'copolymerizations', 'on', 'hand', 'nature', 'organometallic', 'compound', 'effect', '95.', 'it', 'also', 'appears', 'changes', 'reaction', 'temperature', '0', '75(cid1)c', 'effect', 'r', 'values', 'copolymers', 'formed', 'using', 'either', 'soluble', 'heterogeneous', 'ziegler–natta', 'one', 'problem', 'encountered', 'heterogeneous', 'catalysts', 'tendency', 'formed', 'polymers', 'coat', 'active', 'sites', 'this', 'forces', 'monomers', 'diffuse', 'sites', 'may', 'cause', 'starvation', 'active', 'monomer', 'diffuse', 'equal', 'rates', 'like', 'ethylene', 'propylene', 'ethylene', 'butene-1', 'manufactured', 'use', 'anionic', 'coordination', 'catalysts', 'enables', 'variations', 'many', 'different', 'block', 'copolymers', 'oleﬁns', '65', 'copolymers', 'ethylene', 'propylene', '349', 'molecular', 'structures', 'products', 'it', 'possible', 'vary', 'length', 'stereoregularity', 'blocks', 'this', 'accomplished', 'feeding', 'alternately', 'different', 'monomers', 'reactor', 'when', 'necessary', 'blocks', 'consist', 'pure', 'homopolymers', 'addition', 'reaction', 'allowed', 'subside', 'if', 'residual', 'monomer', 'remains', 'removed', '96.', 'this', 'requires', 'long', 'lifetime', 'growing', 'chains', 'insigniﬁcant', 'amount', 'termination', 'the', 'stability', 'anion', 'depends', 'upon', 'catalyst', 'system', 'one', 'technique', 'catalyst', 'preparation', 'form', 'ticl3', 'reducing', 'ticl4', 'diethyl', 'aluminum', 'chloride', 'followed', 'careful', 'washing', 'product', 'reduction', 'remove', 'by-product', 'al(c2h5)cl', 'other', 'reports', 'describe', 'using', 'a-form', 'ticl3', 'heat', 'treating', 'form', 'b', 'g-forms', 'yields', 'stereospeciﬁc', 'products', 'the', 'transition', 'metal', 'oxide', 'catalysts', 'support', 'cro3silica–alumina', '(phillips)', 'moo3al2o3', '(standard', 'oil)', 'used', 'copolymerize', 'minor', 'quantities', 'a-oleﬁns', 'ethylene', 'such', 'copolymerizations', 'introduce', 'short', 'pendant', 'groups', 'polymer', 'backbones', 'ethylene', 'oleﬁns', 'also', 'copolymerized', 'carbon', 'monoxide', 'form', 'polymers', 'aliphatic', 'ketones', 'using', 'transition', 'metal', 'catalysts', 'like', 'palladium(ii)', 'coupled', 'non-coordinating', 'anions', 'there', 'numerous', 'reports', 'catalysts', 'literature', 'one', 'example', 'compound', 'composed', 'bidentate', 'diarylphosphinopropane', 'ligand', 'two', 'acetonitrile', 'molecules', 'coordinating', 'pd2', 'coupled', 'bf3', 'counterions', 'this', 'compound', 'bis(acetonitrile)palladium(ii)-13-bis(diphenyl-', 'phosphino)propane-(tetraﬂuoborate)', 'illustrated', 'follows', '97:', '+', 'r3pd', '(bf4)2', 'n', 'n', 'p', 'p', 'ph', 'ph', 'r', 'ph', 'ph', 'when', 'tetraﬂuoroborate', 'replaced', 'perchlorate', 'compound', 'active', 'catalyst', '97.', 'one', 'copolymer', 'ethylene', 'carbon', 'monoxide', 'available', 'commercially', 'the', 'material', 'offered', 'trade', 'name', 'carilon', 'actually', 'terpolymer', 'contains', 'small', 'quantity', 'propylene', 'it', 'reported', '98', 'use', 'palladium', 'catalyst', 'permits', 'formation', 'perfectly', 'alternating', 'interpolymer08', 'the', 'product', 'reported', 'tough', 'chemical', 'resistant', 'material', 'hustad', 'et', 'al', '99', 'developed', 'technique', 'make', 'polydisperse', 'polyethylene', 'diblock', 'copolymers', '1-octene', 'distribution', 'block', 'lengths', 'when', 'melted', 'compressed', 'ﬁlms', 'distinct', 'polymeric', 'segments', 'self-assemble', 'layered', 'patterns', 'semi-crystalline', 'hard', 'amorphous', 'phases', 'because', 'phase', 'different', 'refractive', 'index', 'block', 'copolymer', 'shown', 'function', 'photonic', 'crystal', 'scatter', 'visible', 'light', 'x', '1-x', 'x', 'c6h13', '1-x', 'c6h13', '350', '6', 'common', 'chain-growth', 'polymers', 'nomura', 'coworkers', '100', 'studied', 'copolymerization', 'ethylene', 'various', 'pentenes', 'n', 'etc', 'n', '¼', '8', '12', 'the', 'polymerizations', 'carried', 'titanium', 'catalysts', 'illustrated', 'titanium', 'compound', 'combined', 'methyl', 'aluminoxane', 'ti', 'cl', 'cl', 'o', 'ti', 'cl', 'cl', 'n', 'their', 'results', 'show', 'monomer', 'reactivities', 'inﬂuenced', 'substituents', 'oleﬁns', 'also', 'nature', 'catalytically', 'active', 'species', 'derlin', 'kaminsky', '101', 'reported', 'copolymerizations', 'ethylene', 'propylene', 'sterically', 'hindered', 'monomer', '3-methyl-1-butene', 'using', 'titanium', 'zirconium', 'metallocenes', 'methyl', 'aluminoxane', 'cocatalyst', 'tritto', 'coworkers', '102', 'reported', 'complex', 'pd(k2-po-2-(2-meoc6h4)2pc6h4so3)', 'me(dmso)', 'investigated', 'single-component', 'catalyst', 'copolymerization', 'ethylene', 'norbornene', 'the', 'catalyst', 'illustrated', 'follows', 'o', 'p', 'pd', 'o', 's', 'o', 'o', 'o', 'o', 's', 'the', 'copolymers', 'obtained', 'good', 'yields', 'molar', 'masses', 'signiﬁcantly', 'higher', 'polyethylene', 'three', 'copolymers', 'formed', 'n', 'n', 'n', 'oac', 'oh', '65', 'copolymers', 'ethylene', 'propylene', '351', 'determination', 'microstructure', 'reactivity', 'ratios', 'revealed', 'strong', 'inherent', 'tendency', 'form', 'alternating', 'copolymers', 'the', 'catalyst', 'used', 'illustrated', 'follows', 'jordan', '103', 'described', 'copolymerization', 'ethylene', 'vinyl', 'ethers', 'vinyl', 'ﬂuoride', '(ortho-phospheno-arenesulfonate)pdme(pyridine)', 'the', 'reaction', 'ar', 'p', 'ar', 'pd', 'me', 'n', 's', 'o', 'o', 'o', 'or', 'ar', '2-ome-ph', 'or', 'n', 'or', '653', 'copolymers', 'propylene', 'dienes', 'although', 'presently', 'lacking', 'industrial', 'importance', 'alternating', 'copolymers', 'made', 'propyl-', 'ene', 'butadiene', '104', 'also', 'propylene', 'isoprene', '105.', 'copolymers', 'propylene', 'butadiene', 'form', 'vanadium-', 'titanium-based', 'catalysts', 'combined', 'aluminum', 'alkyls', 'the', 'catalysts', 'prepared', 'low', 'temperature', '((cid3)70(cid1)c)', 'also', 'found', 'presence', 'halogen', 'atoms', 'catalyst', 'essential', '75.', 'carbonyl', 'compounds', 'ketones', 'esters', 'others', 'effective', 'additives', 'a', 'reaction', 'mechanism', 'based', 'alternating', 'coordination', 'propylene', 'butadiene', 'transition', 'metal', 'proposed', 'furukawa', '104.', '654', 'copolymers', 'ethylene', 'vinyl', 'acetate', 'various', 'copolymers', 'ethylene', 'vinyl', 'acetate', 'prepared', 'free-radical', 'mechanism', 'emulsion', 'polymerizations', 'both', 'reactivity', 'ratios', 'close', '10', '106.', 'the', 'degree', 'branching', 'copolymers', 'strongly', 'temperature-dependent', '107.', 'these', 'materials', 'ﬁnd', 'wide', 'use', 'areas', 'paper', 'coatings', 'adhesives', 'in', 'addition', 'hydrolyzed', 'form', 'copolymers', 'ethylene', 'vinyl', 'alcohol', 'such', 'resins', 'available', 'commercially', 'various', 'ratios', 'polyethylene', 'poly(vinyl', 'alcohol)', 'range', '30', 'poly(vinyl', 'alcohol)', 'high', '70.', 'vinyl', 'acetate', 'residues', 'ethylene–vinyl', 'acetate', 'copolymers', 'reduce', 'regularity', 'polyethylene', 'this', 'reduces', 'crystallinity', 'polymer', 'materials', 'containing', '45', 'vinyl', 'acetate', 'elastomers', 'cross-linked', 'peroxides', '655', 'ionomers', 'another', 'group', 'commercial', 'copolymers', 'ethylene', 'formed', 'acrylic', 'methacrylic', 'acids', 'ethylene', 'major', 'component', 'the', 'copolymerizations', 'carried', 'high', 'pressures', 'these', 'materials', 'range', 'comonomer', 'content', '3', '20.', 'typical', 'values', '10.', 'a', 'large', 'proportion', 'carboxylic', 'acid', 'groups', '(40–50)', 'prereacted', 'metal', 'ions', 'like', 'sodium', '352', '6', 'common', 'chain-growth', 'polymers', 'zinc', 'the', 'copolymer', 'salts', 'called', 'ionomers', 'trade', 'name', 'like', 'syrlin', 'the', 'materials', 'tend', 'behave', 'similarly', 'cross-linked', 'polymers', 'ambient', 'temperature', 'stiff', 'tough', 'yet', 'processed', 'elevated', 'temperatures', 'aggregation', 'ionic', 'segments', 'different', 'polymeric', 'molecules', 'destroyed', 'the', 'material', 'becomes', 'mobile', 'cooling', 'aggregates', 'reform', 'ionomers', 'exhibit', 'good', 'low', 'temperature', 'ﬂexibility', 'they', 'tough', 'abrasion-resistant', 'resins', 'adhere', 'well', 'metal', 'surfaces', '66', 'homopolymers', 'conjugated', 'dienes', 'many', 'different', 'polymers', 'conjugated', 'dienes', 'prepared', 'commercially', 'variety', 'processes', 'depending', 'upon', 'need', 'they', 'formed', 'free-radical', 'ionic', 'coordinated', 'anionic', 'polymerizations', 'in', 'addition', 'various', 'molecular', 'weights', 'homopolymers', 'copolymers', 'ranging', 'thousand', 'liquid', 'polymers', 'high', 'molecular', 'weight', 'ones', 'synthetic', 'rubbers', 'market', '661', 'polybutadiene', '13-butadiene', 'simplest', 'conjugated', 'dienes', 'produced', 'commercially', 'thermal', 'cracking', 'petroleum', 'fractions', 'catalytic', 'dehydrogenation', 'butane', 'butene', 'polymerization', 'butadiene', 'potentially', 'lead', 'three', 'poly(12-butadiene)s', 'atactic', 'isotactic', 'syndiotactic', 'two', 'cis', 'trans', 'forms', 'poly(14-butadiene)', 'this', 'discussed', 'chaps', '3', '4', 'free-radical', 'polymerizations', '13-butadiene', 'usually', 'result', 'polymers', '78–82', '14-type', 'placement', '18–22', '12-adducts', 'the', 'ratio', '14', '12', 'adducts', 'independent', 'tempera-', 'ture', 'polymerization', 'moreover', 'ratio', 'obtained', 'polymerizations', 'carried', 'bulk', 'emulsion', 'the', 'ratio', 'trans-14', 'cis-14', 'tends', 'decrease', 'however', 'temperature', 'reaction', 'decreases', 'polybutadiene', 'polymers', 'formed', 'free-radical', 'mechanism', 'branched', 'residual', 'unsaturations', 'polymeric', 'chains', 'subjects', 'free-radical', 'attacks', 'ch2', 'ch', 'etc', 'ch2', '66', 'homopolymers', 'conjugated', 'dienes', '353', 'should', 'branching', 'become', 'excessive', 'inﬁnite', 'networks', 'form', 'the', 'products', 'become', 'cross-linked', 'insoluble', 'infusible', 'such', 'materials', 'called', 'popcorn', 'polymers', 'this', 'phenomenon', 'common', 'bulk', 'polymerizations', 'the', 'cross-linked', 'polymers', 'form', 'nodules', 'occupy', 'much', 'volume', 'monomers', 'formed', 'often', 'clog', 'polymerization', 'equipment', 'sometimes', 'even', 'rupturing', 'high', 'molecular', 'weight', 'homopolymers', '13-butadiene', 'formed', 'free-radical', 'mechanism', 'lack', 'type', 'elastomeric', 'properties', 'needed', 'commercial', 'rubbers', 'copolymers', 'butadi-', 'ene', 'however', 'styrene', 'acrylonitrile', 'useful', 'prepared', 'large', 'scale', 'this', 'discussed', 'another', 'section', '6611', 'liquid', 'polybutadiene', 'low', 'molecular', 'weight', 'liquid', 'homopolymers', '13', 'butadiene', 'also', 'liquid', 'copolymers', 'ﬁnd', 'industrial', 'uses', 'many', 'applications', 'these', 'materials', 'range', 'molecular', 'weights', '500', '5000', 'depending', 'upon', 'mode', 'polymerization', 'liquid', 'polybutadienes', 'formed', 'cationic', 'polymerizations', 'high', 'trans-14', 'content', 'such', 'materials', 'ﬁnd', 'applications', 'industrial', 'coatings', 'they', 'usually', 'prepared', 'lewis', 'acids', 'chlorinated', 'solvents', 'when', 'reactions', 'catalyzed', 'alcl3', '(cid3)78(cid1)c', 'two', 'types', 'polymers', 'form', '108.', 'one', 'soluble', 'insoluble', 'depending', 'upon', 'extent', 'conversion', 'alcl3', 'albr3', 'bf3–et2o', 'produce', 'polymers', 'ratios', 'trans-14', '12', 'adducts', 'these', 'range', '4', '5', 'polymerizations', 'carried', 'ethylene', 'chloride', '108', 'catalyzed', 'ticl4', 'yield', 'products', 'lower', 'ratios', 'trans-14', '12', 'adducts', 'the', 'ratios', 'two', 'placements', 'affected', 'solvents', 'they', 'also', 'affected', 'additions', 'complexing', 'agents', 'nitroethane', 'nitrobenzene', '108.', 'the', 'changes', 'however', 'small', 'hydroxyl-terminated', 'liquid', 'polybutadienes', 'prepared', 'reactions', 'diisocyanates', 'form', 'elastomeric', 'polyurethanes', '(see', 'chap', '6)', 'such', 'materials', 'prepared', 'anionic', 'polymerizations', '“living”', 'polymers', 'quenched', 'appropriate', 'molecular', 'weight', 'these', 'polybutadienes', 'also', 'formed', 'free-radical', 'mechanism', 'the', 'microstructures', 'two', 'products', 'differ', 'however', 'may', 'affect', 'properties', 'ﬁnished', 'products', 'to', 'form', 'hydroxyl-terminated', 'polymers', 'free-radical', 'mechanism', 'polymerization', 'reactions', 'may', 'initiated', 'hydroxyl', 'radicals', 'hydrogen', 'peroxide', 'a', 'new', 'approach', 'preparation', 'hydroxyl-terminated', 'liquid', 'polybutadiene', 'use', 'cyclic', 'monomer', '15-cyclooctadiene', 'ruthenium', 'metathesis', 'catalyst', '(see', 'chap', '5', 'grubbs', 'catalysts', 'section', 'metathesis', 'ring', 'opening', 'polymerization)', 'acetate', 'functionalized', 'chain', 'transfer', 'agent', '109', '110.', 'the', 'acetate-functionalized', 'chain', 'transfer', 'agent', 'cis-2-butene-14-diacetate', 'the', 'reaction', 'carried', 'without', 'solvent', 'proceeds', '50(cid1)c', '6', 'h', 'inert', 'gas', 'purge', '111.', 'the', 'acetate', 'protecting', 'groups', 'provide', 'compatibility', 'ruthenium', 'catalyst', 'subsequent', 'polymerization', 'acetate', 'groups', 'converted', 'hydroxyl', 'end', 'groups', 'aid', 'base', 'like', 'sodium', 'methoxide', 'liquid', 'polybutadienes', 'high', '12', 'placement', 'also', 'available', 'commercially', 'these', 'range', 'reactive', 'polymers', 'containing', 'approximately', '70', 'vinyl', 'groups', 'reactive', 'ones', 'containing', '90', '12', 'units', 'the', 'materials', 'formed', 'anionic', 'polymerization', 'either', 'sodium', 'naphthalene', 'sodium', 'dispersions', 'organolithium', 'initiators', 'polar', 'solvents', 'carboxyl', 'group', 'terminated', 'liquid', 'polybutadienes', 'predominantly', 'used', 'modiﬁers', 'epoxy', 'resins', '(chap', '7)', 'they', 'formed', 'anionic', 'mechanisms', 'solution', 'organolithium', 'catalysts', 'like', 'diphenylethanedilithium', 'butanedilithium', 'isoprenelithium', 'lithium', 'methylnaphtha-', 'lene', 'complexes', 'cyclohexane', 'choice', 'solvent', 'the', 'reaction', 'quenched', 'carbon', 'dioxide', 'introduce', 'terminal', 'carboxyl', 'groups', '354', '6', 'common', 'chain-growth', 'polymers', 'table', '67', 'microstructures', 'polybutadienes', 'prepared', 'coordination', 'catalysts', 'catalyst', 'tii4al(c2h5)3', 'tibr4al(c4h9)3', 'b-ticl3al(c2h5)3', 'ti(oc4h9)4al(2h5)3', 'ti(oc6h5)4r3al', '(p-cycloocatadiene)2nicf3co2h', 'bis(p-crotyl', 'nicl)', 'bis(p-crotyl', 'nii)', 'cocl2al(c2h5)2cl', 'cocl2al(c2h5)3', 'cocl2al(c2h5)3pyridine', 'co-stearatealr2cl', 'vcl3alr3', 'vocl3al(c2h5)3', 'vcl4alcl3', 'cr(c6h5cn)6al(c2h5)3', 'moo2(or)3al(c2h5)3', 'from', 'various', 'sources', 'literature', 'microstructure', '()', 'cis-14', '95', '88', '80', '–', '90–100', '100', '92', '–', '96–97', '94', '90–97', '98', '–', '–', '–', '100', '–', 'trans-14', '2', '3', '12', '–', '–', '–', '–', '94', '25', '3', '–', '1', '99', '97–9', '95', '–', '–', '12', '3', '9', '8', '99–100', '–', '–', '–', '–', '1–15', '3', '–', '1', '1', '2–3', '–', '–', '75', '6612', 'high', 'molecular', 'weight', 'polybutadiene', 'high', 'molecular', 'weight', 'polybutadiene', 'homopolymers', 'prepared', 'commercially', 'anionic', 'catalysts', 'coordination', 'ones', 'polybutadiene', 'formed', 'sodium', 'dispersions', 'prepared', 'industrially', 'former', 'ussr', 'perhaps', 'might', 'still', 'produced', 'area', 'today', 'this', 'sodium-', 'catalyzed', 'polybutadiene', 'contains', '65', '12-adducts', '112.', 'many', 'preparations', 'others', 'however', 'utilize', 'either', 'alkyllithium', 'ziegler–natta', 'type', 'catalysts', 'prepared', 'titanium', 'tetra', 'iodide', 'preferably', 'containing', 'cobalt', 'because', 'high', 'molecular', 'weight', 'polybutadiene', 'prepared', 'different', 'catalytic', 'systems', 'choice', 'catalyst', 'usually', 'governed', 'desired', 'microstructure', 'product', 'alﬁn', 'catalysts', 'yield', 'high', 'molecular', 'weight', 'polymers', 'large', 'amount', 'trans-14', 'structures', 'both', 'molecular', 'weight', 'microstructure', 'affected', 'signiﬁcantly', 'however', 'variations', 'alﬁn', 'catalyst', 'components', 'these', 'alkyl', 'groups', 'organometallic', 'compounds', 'alkoxide', 'portions', 'when', 'butadiene', 'polymerized', 'lithium', 'metal', 'alkyllithium', 'catalysts', 'inert', 'solvents', 'like', 'hexane', 'heptane', 'must', 'used', 'obtain', 'high', 'cis-14', 'placement', '(see', 'chap', '4)', 'based', '13c', 'nmr', 'spectra', '14-polybutadiene', 'formed', 'n-butyllithium', 'consists', 'blocks', 'cis-14', 'units', 'trans-14', 'units', 'separated', 'isolated', 'vinyl', 'structures', '113:', 'x', 'z', 'the', 'quantity', 'units', 'polybutadienes', 'approximately', '48–58', 'trans-14', '33–45', 'cis-14', '7–10', '12', 'units', '112.', 'there', 'little', 'effect', 'reaction', 'temperatures', 'upon', 'composition', 'as', 'described', 'chap', '3', 'however', 'addition', 'lewis', 'bases', 'profound', 'effect', 'reactions', 'tetrahydrofuran', 'solvent', 'result', '12', 'placement', 'high', '87.', 'the', 'microstructures', 'polybutadienes', 'prepared', 'ziegler–natta', 'catalysts', 'vary', 'catalyst', 'composition', 'it', 'possible', 'form', 'polymers', 'high', 'either', '12', 'placement', '14', 'units', 'the', 'catalysts', 'type', 'placement', 'summarized', 'table', '67', '66', 'homopolymers', 'conjugated', 'dienes', '355', 'butadiene', 'polymerized', 'chromium', 'oxide', 'catalyst', 'support', 'form', 'solid', 'homopolymers', 'the', 'products', 'however', 'tend', 'coat', 'catalyst', 'within', 'hours', 'start', 'reaction', 'interfere', 'polymerization', 'polybutadiene', 'also', 'prepared', 'presence', 'molybdenum', 'catalyst', 'promoted', 'calcium', 'hydride', 'the', 'product', 'contains', '80', '14', 'units', '20', '12', 'units', 'of', '14', 'units', '625', 'cis', '375', 'trans', '110.', 'cobalt', 'oxide', 'silica–alumina', 'presence', 'alkyl', 'aluminum', 'also', 'yields', 'high', 'cis-14', 'structure', 'polymers', 'an', '12', 'polybutadiene', 'prepared', 'n-butyllithium', 'modiﬁed', 'bis-piperidino', 'ethane', 'the', 'atactic', 'polymer', 'formed', 'hexane', '(cid3)5', '20(cid1)c', 'temperature', '111.', 'the', '100', '12', 'placement', 'postulated', 'proceed', 'according', 'following', 'scheme', '111.', 'first', 'complex', 'forms', 'base', 'butyllithium', 'the', 'complex', 'reacts', 'butadiene', 'form', 'new', 'complex', 'this', 'followed', 'insertion', 'butadiene', 'carbon–lithium', 'bond', 'n', 'n', 'li', 'n', 'n', 'li', 'n', 'n', 'li', 'annunziata', 'et', 'al', '114', 'reported', 'group', '4', 'metals', 'complexes', 'bearing', 'anilidomethylpyridine', 'ligands', 'prepared', 'after', 'activation', 'albu2h', 'methylalumoxane', 'catalysts', 'tested', '13-butadiene', 'a-oleﬁn', 'polymerization', 'the', 'zirconium', 'complexes', 'showed', 'higher', 'activity', 'titanium', 'analogous', 'polymerization', 'ethylene', 'resulted', 'cases', 'production', 'high', 'molecular', 'weight', 'linear', 'polyethylene', 'on', 'hand', 'propylene', 'polymerization', 'tests', 'provided', '356', '6', 'common', 'chain-growth', 'polymers', 'substantially', 'atactic', 'polypropylene', '13-butadiene', 'polymerizations', 'produced', 'cis-14', 'polybutadiene', 'use', 'zirconium', 'complexes', 'produced', 'polymers', 'content', 'cis-14', 'units', 'higher', '999', 'claimed', '662', 'polyisoprene', 'polyisoprenes', 'occur', 'nature', 'they', 'also', 'prepared', 'synthetically', 'most', 'commercial', 'processes', 'try', 'duplicate', 'naturally', 'occurring', 'material', '6621', 'natural', 'polyisoprenes', 'rubber', 'hydrocarbon', 'principle', 'component', 'raw', 'rubber', 'the', 'subject', 'discussed', 'greater', 'detail', 'chap', '7', 'natural', 'rubber', '97', 'cis-14', 'polyisoprene', 'it', 'obtained', 'tapping', 'bark', 'rubber', 'trees', '(hevea', 'brasiliensis)', 'collecting', 'exudates', 'latex', 'consisting', '32–35', 'rubber', 'a', 'similar', 'material', 'also', 'found', 'sap', 'many', 'plants', 'shrubs', 'the', 'structure', 'natural', 'rubber', 'investigated', '100', 'years', '1920', 'however', 'chemical', 'structure', 'elucidated', 'it', 'shown', 'linear', 'polymer', 'consisting', 'head', 'tail', 'links', 'isoprene', 'units', '98', 'bonded', '14', '6622', 'synthetic', 'polyisoprenes', 'in', 'following', 'natural', 'rubber', 'synthetic', 'efforts', 'devoted', 'obtaining', 'high', 'cis-14', 'polyiso-', 'prene', 'forming', 'synthetic', '“natural”', 'rubber', 'two', 'types', 'polymerizations', 'yield', 'products', 'approach', 'one', 'use', 'ziegler–natta', 'type', 'catalysts', 'anionic', 'polymerization', 'alkyllithium', 'compounds', 'hydrocarbon', 'solvents', 'one', 'commercial', 'process', 'instance', 'uses', 'reaction', 'products', 'ticl4', 'triisobutylaluminum', 'alti', 'ratio', '09–11', 'catalyst', 'diphenyl', 'ether', 'lewis', 'bases', 'sometimes', 'added', 'catalyst', 'modiﬁers', '113–116.', 'the', 'process', 'results', 'approximately', '95', 'cis-14', 'polyisoprene', 'product', 'typically', 'reactions', 'carried', 'continuous', 'basis', 'usually', 'hexane', 'take', '2–4', 'h', 'polymerizations', 'often', 'done', 'two', 'reaction', 'lines', 'consisting', 'four', 'kettles', 'arranged', 'series', 'the', 'heat', 'reaction', 'partially', 'absorbed', 'precooling', 'feed', 'streams', 'the', 'remaining', 'heat', 'absorbed', 'cooled', 'surfaces', 'when', 'stream', 'exits', 'conversion', '80.', 'addition', 'shortstop', 'solution', 'stabilizes', 'product', 'alkyllithium-initiated', 'polymerizations', 'isoprene', 'yield', 'polymers', '92–93', 'cis-14', 'content', 'one', 'industrial', 'process', 'uses', 'butyllithium', 'continuous', 'reaction', 'two', 'lines', 'consisting', 'four', 'reaction', 'kettles', 'the', 'heat', 'reaction', 'removed', 'vaporization', 'solvent', 'monomer', 'the', 'catalyst', 'solution', 'added', 'solvent', 'stream', 'intensively', 'mixed', 'isoprene', 'monomer', 'stream', 'fed', 'ﬁrst', 'reactor', 'after', 'stream', 'leaves', 'reactor', 'small', 'quantities', 'methanol', 'injected', 'stages', 'reaction', 'mixture', 'this', 'limits', 'molecular', 'weight', 'stopping', 'reaction', 'fresh', 'butyllithium', 'catalyst', 'added', 'next', 'stage', 'next', 'reactor', 'initiate', 'new', 'polymer', 'growth', '117–119.', 'as', 'described', 'chaps', '3', '4', 'monomer', 'placement', 'polyisoprene', 'chain', 'occur', 'potentially', 'nine', 'different', 'ways', 'these', 'three', 'tactic', 'forms', '12', 'adducts', 'two', '14', 'adducts', 'cis', 'trans', 'three', 'tactic', 'forms', '34-adducts', 'in', 'addition', 'possibility', 'head', 'head', 'tail', 'tail', 'insertion', 'though', 'common', 'addition', 'head', 'tail', 'table', '68', 'presents', 'various', 'microstructures', 'obtained', 'polymerizations', 'isoprene', 'different', 'catalysts', '66', 'homopolymers', 'conjugated', 'dienes', 'table', '68', 'polymerization', 'products', 'isoprene', 'mode', 'polymerization', 'free', 'radical', 'cationic', 'solvent', 'emulsion', 'water', '–', 'chloroform', '(30(cid1)c)', 'approximate', 'cis-14', '32', '37', '–', 'trans-14', '65', '51', '90', '12', '6', '4', '4', 'anionic', 'lithium', 'ethyllithium', 'butyllithium', 'sodium', 'ethylsodium', 'butylsodium', 'potassium', 'ethylpotassium', 'butylpotassium', 'rubidium', 'cesium', 'ethyllithium', 'ethylsodium', 'lithium', 'alﬁn', 'pentane', 'pentene', 'pentene', 'pentene', 'pentene', 'pentane', 'pentane', 'pentane', 'pentane', 'pentane', 'pentene', 'ethyl', 'ether', 'ethyl', 'ether', 'ethyl', 'ether', 'pentane', 'coordination', 'catalysis', 'a-ticl3alr3', 'vcl3al(c2h5)3', 'ticl4al(c2h5)3', 'ti', 'l4alr3', 'amine', 'cocl2alr3', 'pyridine', 'v(acetylacetonate)3alr3', 'ti(or)4al(c2h5)3', 'from', 'various', 'sources', 'literature', '0', '0', '0', '43', '42', '35', '52', '39', '41', '47', '51', '29', '14', '27', '52', '91', '99', '–', '94', '94', '93', '0', '6', '4', '0', '24', '20', '5', '4', '6', '0', '4', '27', '95–96', '100', '96', '90', '95', '357', '34', '7', '9', '6', '6', '6', '7', '51', '45', '54', '40', '31', '34', '39', '37', '60', '76', '63–65', '16', '–', '6', '7', '7', '8', '6', '6', '8', '8', '5', '10', '5–7', '5', 'cationic', 'polymerizations', 'isoprene', 'proceed', 'readily', 'butadiene', 'though', 'yield', 'low', 'molecular', 'weight', 'liquid', 'polymers', 'alcl3', 'stannic', 'chloride', 'used', 'chlorinated', 'solvents', 'temperatures', '0(cid1)c', 'without', 'chlorinated', 'solvents', 'however', 'polymerizations', 'isoprene', 'require', 'temperatures', '0(cid1)c', 'at', 'high', 'conversions', 'cationic', 'polymerizations', 'isoprene', 'result', 'formations', 'cross-linked', 'material', '120.', 'the', 'soluble', 'portions', 'polymers', 'high', 'trans-14', 'structures', 'alﬁn', 'catalysts', 'yield', 'polymers', 'higher', 'trans-14', 'structures', 'free-radical', 'emulsion', 'polymerizations', '121.', 'chromium', 'oxide', 'catalysts', 'support', 'polymerize', 'isoprene-like', 'butadiene', 'solid', 'polymers', 'here', 'however', 'polymerization', 'process', 'polymer', 'particles', 'cover', 'catalyst', 'completely', 'within', 'hours', 'start', 'reaction', 'retard', 'stop', 'polymer', 'formation', 'the', 'polymerization', 'conditions', 'used', 'butadiene', 'the', 'reactions', 'carried', 'ﬁxed', 'bed', 'catalysts', 'containing', '3', 'chromium', 'oxide', 'sio2–al2o3', 'conditions', '88(cid1)c', '42', 'kgcm2', 'pressure', 'charge', 'containing', '20', 'isoprene', '80', 'isobutane', '122.', 'the', 'mixed', 'molybdenum–alumina', 'catalyst', 'calcium', 'hydride', 'also', 'yields', 'polyisoprene', 'lithium', 'metal', 'dispersions', 'form', 'polymers', 'isoprene', 'high', 'cis-14', 'contents', 'shown', 'table', '68', 'these', 'polymers', 'form', 'hydrocarbon', 'solvents', 'this', 'done', 'industrially', 'products', 'called', 'coral', 'rubbers', 'they', 'contain', 'small', 'percentage', '34-structures', 'trans-14', '12', 'units', 'the', 'materials', 'strongly', 'resemble', 'hevea', 'rubber', '358', '6', 'common', 'chain-growth', 'polymers', 'use', 'ziegler–natta', 'catalysts', 'seen', 'table', '68', 'yield', 'almost', 'cis-14-polyisoprene', 'almost', 'trans-14-polyisoprene', 'the', 'microstructure', 'depends', 'upon', 'ratio', 'titanium', 'aluminum', 'ratios', 'tial', '051', '151', 'yield', 'cis', 'isomer', 'a', '11', 'ratio', 'optimum', 'ratios', 'tial', '151', '31', 'yield', 'trans', 'structures', '123.', 'the', 'titanium', 'aluminum', 'ratios', 'also', 'affect', 'yields', 'polymers', 'well', 'microstructures', 'there', 'also', 'inﬂuence', 'molecular', 'weight', 'product', '124.', 'variations', 'catalyst', 'compositions', 'however', 'affect', 'relative', 'amounts', '14', '34', '12', 'placements', 'only', 'cis', 'trans', 'arrangements', 'affected', 'in', 'addition', 'molecular', 'weights', 'polymers', 'microstructures', 'relatively', 'insensitive', 'catalyst', 'concentrations', 'the', 'temperatures', 'reactions', 'however', 'affect', 'rates', 'molecular', 'weights', 'microstructures', 'use', 'ziegler–natta', 'catalysts', 'seen', 'table', '68', 'yield', 'almost', 'cis-14-polyisoprene', 'almost', 'trans-14-polyisoprene', 'the', 'microstructure', 'depends', 'upon', 'ratio', 'titanium', 'aluminum', 'variations', 'catalyst', 'compositions', 'however', 'affect', 'relative', 'amounts', '14', '34', '12', 'placements', 'only', 'cis', 'trans', 'arrangements', 'affected', 'in', 'addition', 'molecular', 'weights', 'polymers', 'microstructures', 'relatively', 'insensitive', 'catalyst', 'concentrations', 'the', 'temperatures', 'reactions', 'however', 'affect', 'rates', 'molecular', 'weights', 'microstructures', '67', 'methyl', 'rubber', 'poly(23-dimethylbutadiene)', 'early', 'attempts', 'preparations', 'synthetic', 'rubbers', 'resulted', 'developments', 'elastomers', '23-dimethylbutadiene', 'the', 'material', 'called', '“methyl', 'rubber”', 'claimed', 'yield', 'better', 'elastomeric', 'properties', 'polybutadiene', 'methyl', 'rubber', 'produced', 'germany', 'world', 'war', 'i', 'monomer', 'prepared', 'acetone', 'the', 'polymerizations', 'carried', 'free-radical', 'mechanism', 'anionically', 'using', 'sodium', 'metal', 'dispersions', 'initiation', 'later', 'demonstrated', '23-dimethyl', 'polybutadiene', 'polymerized', 'high', 'cis-14', 'polymer', 'ziegler–natta', 'catalysts', '125', '126.', '68', 'chloroprene', 'rubber', 'poly(2-chloro-13-butadiene)', '2-chloro-13-butadiene', '(chloroprene)', 'originally', 'synthesized', '1930', 'the', 'material', 'polymerize', 'spontaneously', 'elastomer', 'good', 'resistance', 'oil', 'weathering', 'commercial', 'production', 'chloroprene', 'rubber', 'started', '1932', 'since', 'many', 'types', 'polymers', 'copolymers', 'developed', 'trivial', 'generic', 'name', 'neoprene', 'the', 'monomer', 'prepared', 'acetylene', 'nh4cl', 'ccl4', 'hcl', 'cl', 'it', 'also', 'formed', 'butadiene', 'chloroprene', 'polymerized', 'commercially', 'free-radical', 'emulsion', 'polymerization', 'the', 'reaction', 'carried', '40(cid1)c', '90', 'conversion', 'a', 'typical', 'recipe', 'emulsion', 'polymerization', 'follows', '127:', '68', 'chloroprene', 'rubber', 'poly(2-chloro-13-butadiene)', '359', 'material', 'water', 'chloroprene', 'rosin', 'naoh', 'k2s2o8', 'sulfur', 'methylene-bis-(na-naphthalenesulfonic', 'acid)', 'parts', '150', '100', '4', '(stabilizer)', '08', '(stabilizer)', '07', 'when', 'polymerization', 'reaches', '90', 'conversion', 'reaction', 'mixture', 'cooled', '20(cid1)c', 'tetraethylthiuram', 'disulﬁde', 'added', 'this', 'done', 'prevent', 'pendant', 'unsaturation', 'polychloroprene', 'backbones', 'cross-linking', 'forming', 'branches', 'an', 'unmodiﬁed', 'polymer', 'difﬁcult', 'process', 'even', '70', 'conversion', 'to', 'overcome', 'sulfur–tetraethylthiuram', 'modiﬁ-', 'cation', 'carried', 'when', 'product', 'treated', 'thiuram', 'exchange', 'reaction', 'takes', 'place', 'yield', 'stable', 'thiuram-modiﬁed', 'polymer', 'reduced', 'molecular', 'weight', 'it', 'believed', 'reaction', 'takes', 'place', 'cleavage', 'sulfur', 'links', 'formed', 'polymerization', 'presence', 'sulfur', 'formation', 'free', 'radicals', '128', 'cl', 'cl', 's', 's', 's', 's', 'n', 'cl', 's', 's', 'n', 's', 's', 's', 's', 'cl', 's', 'n(c2h5)2', 'n(c2h5)2', 's', 's', 'n(c2h5)2', 'cl', 's', 's', 'n', 's', 'n(c2h5)2', 's', 's', 'after', 'reaction', 'tetraethylthiuram', 'disulﬁde', 'completed', 'latex', 'acidiﬁed', 'acetic', 'acid', 'short', 'coagulation', 'the', 'rubber', 'recovered', 'low', 'temperature', '(about', '(cid3)15(cid1)c)', 'form', 'sheets', 'deposition', 'latex', 'cooled', 'rotating', 'drums', '127', '128.', 'the', 'polymer', 'formed', 'technique', 'consists', '85', 'trans-14', 'units', '10', 'cis-14', 'units', '15', '12', 'units', '10', '34', 'units', 'the', 'polymer', 'essentially', 'linear', 'molecular', 'weight', 'equal', 'approximately', '100000', 'the', 'sulfur-modiﬁed', 'polychloroprenes', 'sold', 'trade', 'name', 'neoprene-g', 'an', 'unmodiﬁed', 'version', 'prepared', 'mercaptan', 'chain', 'transferring', 'agents', '(neoprene', 'w)', 'polymer', 'molecular', 'weight', '200000', '128', '130.', 'table', '69', 'lists', 'structures', 'polychloroprenes', 'form', 'free-radical', 'polymerization', 'different', 'temperatures', 'chloroprene', 'polymerizes', 'cationic', 'polymerization', 'aid', 'lewis', 'acids', 'chlorinated', 'solvents', 'when', 'aluminum', 'chloride', 'used', 'mixture', 'ethyl', 'chloride–methylene', 'chloride', 'solvent', 'mixture', '(cid3)80(cid1)c', 'polymer', '50', '14', 'units', '131', '132.', 'if', 'polymerized', 'boron', 'triﬂuoride', 'product', 'consists', '50–70', '14-adducts', 'a', 'high', 'trans-14', 'poly(2-', 'chloro-13-butadiene)', 'forms', 'x-ray', 'radiation', 'polymerization', 'large', 'crystals', 'chloroprene', '(cid3)130', '(cid3)180(cid1)c', 'it', '978', 'trans-14', '130.', 'presumably', 'mechanism', 'polymerization', 'free', 'radical', '360', '6', 'common', 'chain-growth', 'polymers', 'table', '69', 'structures', 'polychloroprenes', 'formed', 'free-radical', 'polymerization', 'polymerization', 'temperature', '((cid1)c)', '(cid3)40', '(cid3)20', '(cid3)10', '10', '40', '40', '100', 'from', 'refs', '127–132', '14', 'cis', '5', '6', '7', '9', '10', '13', '13', 'trans', '94', '915', '–', '84', '86', '81', '889', '71', '12', '09', '07', '–', '11', '16', '09', '24', '34', '03', '05', '–', '10', '10', '03', '24', 'references', '112', '113', '112', '112', '112', '113', '112', '69', 'special', 'polymers', 'dienes', 'there', 'many', 'reports', 'literature', 'preparations', 'polymers', 'various', 'substituted', 'dienes', 'most', 'commercial', 'signiﬁcance', 'some', 'however', 'interesting', 'materials', 'an', 'example', 'polymer', '2-t-butyl-13-butadiene', 'formed', 'ticl4', 'either', 'alkylaluminum', 'aluminum', 'hydride', 'catalysts', '132.', 'the', 'polymer', 'crystalline', 'melts', '106(cid1)c', 'it', 'dissolved', 'common', 'solvents', 'based', 'x-ray', 'data', 'monomer', 'placement', 'high', 'cis-14', 'poly(carboxybutadiene)s', 'also', 'forms', 'coordination', 'catalysts', '133–135:', 'r', 'r', 'h', 'coor', 'h', 'coor', 'n', 'r', 'h', 'r', '¼', 'ch3', 'r0', '¼', 'ch3', 'c2h5', 'c4h9', 'c6h5', 'x-ray', 'crystallography', '133–135', 'showed', 'placement', 'trans-isotactic', 'based', 'mode', 'packing', 'chains', 'crystalline', 'regions', 'encumbrance', 'side', 'groups', 'relationship', 'main', 'chain', 'erythro', 'conﬁguration', 'assigned', '134.', 'the', 'polymers', 'therefore', 'trans-erythro-isotactic', 'ﬁve', 'different', 'polymerization', '13-pentadiene', 'potentially', 'result', 'insertions', 'monomers', 'these', '14-cis', '14-trans', '12-cis', '12-trans', '34', 'in', 'addition', 'potentially', '3-cis-14', '3', 'trans-14', 'structures', '(isotactic', 'syndiotactic', 'atactic)', 'formations', 'trans-14', 'isotactic', 'cis-14', 'isotactic', 'cis-14', 'syndiotactic', 'polymers', 'possible', 'ziegler–natta', 'catalysts', '136–138.', 'amorphous', 'polymers', 'also', 'form', 'predominantly', 'cis-14', 'trans-14', 'lack', 'tactic', 'order', 'stereospeciﬁcity', 'poly(13-pentadiene)', 'strongly', 'dependent', 'upon', 'solvent', 'used', 'polymerization', 'thus', 'cis-14', 'syndiotactic', 'polymers', 'form', 'aromatic', 'solvents', 'trans-12', 'aliphatic', 'ones', 'the', 'preparations', 'require', 'cobalt', 'halidealuminum', 'alkyl', 'dichloride(or', 'dialkyl', 'chloride)', 'catalysts', 'combinations', 'lewis', 'bases', 'to', 'form', 'trans-14', 'structure', 'catalyst', 'containing', 'aluminum', 'titanium', 'ratio', 'close', '5', 'must', 'used', '139.', '610', 'cyclopolymerization', 'conjugated', 'dienes', 'conjugated', 'dienes', 'like', 'isoprene', 'butadiene', 'chloroprene', 'cyclopolymerize', 'catalysts', 'consisting', 'aluminum', 'alkyls', 'like', 'ethylaluminum', 'dichloride', 'titanium', 'tetrachloride', '141.', 'the', 'ladder', 'polymers', 'form', 'contain', 'fused', 'cyclic', 'structures', 'the', 'products', 'prepared', 'hexane', 'generally', 'insoluble', 'powders', 'prepared', 'aromatic', 'solvents', 'soluble', 'even', '611', 'copolymers', 'dienes', '361', 'molecular', 'weights', 'high', '142.', 'a', 'high', 'ratio', 'transition', 'metal', 'halide', 'aluminum', 'alkyl', 'must', 'used', 'such', 'ratio', 'might', 'conceivably', 'mean', 'mechanism', 'polymerization', 'cationic', 'also', 'conventional', 'cationic', 'initiators', 'used', 'yield', 'similar', 'products', 'the', 'cyclization', 'occurs', 'propagation', 'unsaturation', 'products', 'vary', 'none', 'high', '80,', 'depending', 'upon', 'initiator', 'used', '142.', 'different', 'mechanisms', 'offered', 'explain', 'cyclization', '13', 'dienes', '142', '143.', 'the', 'cyclization', 'might', 'conceivably', 'occur', 'sequential', 'process', 'perhaps', 'attacks', 'propagating', 'carbon', 'cation', 'trans-14', 'double', 'bonds', 'r', 'r', 'r', 'represent', 'either', 'propagating', 'carbon', 'cation', 'initiating', 'species', 'the', 'extensive', 'cyclization', 'may', 'result', 'sequential', 'process', '142', '143.', 'cyclopolymerizations', 'typically', 'result', 'low', 'conversions', 'dormant', 'reaction', 'mixtures', 'when', 'additional', 'monomer', 'added', 'dormant', 'mixtures', 'reinitiate', 'polymerizations', 'proceed', 'limited', 'conversions', 'if', 'original', 'dormant', 'mixtures', 'allowed', 'stand', 'long', 'time', 'unreacted', 'monomers', 'slowly', 'consumed', '142.', 'polymerization', '23-dimethylbutadiene-13', 'ziegler–natta', 'catalysts', 'consisting', 'al(i-c4h9)3–ticl4', 'yields', 'cis-14-polydimethylbutadiene', 'described', 'earlier', 'this', 'however', 'takes', 'place', 'aluminum', 'alkyl', 'excess', 'if', 'hand', 'ratio', 'al', 'ti', '1', 'less', 'cyclic', 'polymer', 'forms', 'instead', 'the', 'product', 'reduced', 'unsaturation', 'trans-14', 'units', 'chain', '144.', 'a', 'complex', 'catalyst', 'consisting', 'al(i-c4h9)3–cocl2', 'yields', 'polymers', 'predomi-', 'nantly', 'cis-14', '20', '12', 'units', 'on', 'hand', 'acid', 'catalysts', 'like', 'al(c2h5)cl2', 'yield', 'cyclic', 'polymers', '143', '144.', 'a', 'polymer', 'formed', 'aid', 'x-ray', 'radiation', 'low', 'temperatures', 'also', 'contains', 'cyclic', 'units', 'trans-14', '145.', 'butadiene', 'isoprene', 'also', 'form', 'type', 'polymer', 'conditions', '145.', '611', 'copolymers', 'dienes', 'several', 'different', 'elastomers', 'copolymers', 'butadiene', 'produced', 'commercially', 'the', 'major', 'ones', 'copolymers', 'butadiene', 'styrene', 'butadiene', 'acrylonitrile', 'some', 'terpolymers', 'third', 'component', 'unsaturated', 'carboxylic', 'acid', 'also', 'manufactured', 'block', 'copolymers', 'isoprene', 'styrene', 'butadiene', 'styrene', 'important', 'commercial', 'elastomers', '6111', 'gr-s', 'rubber', 'copolymerization', 'butadiene', 'styrene', 'free-radical', 'mechanism', 'explored', 'thoroughly', '146.', 'the', 'original', 'efforts', 'started', 'world', 'war', 'i', 'germany', 'subsequent', 'work', '362', '6', 'common', 'chain-growth', 'polymers', 'table', '610', 'typical', 'recipes', 'preparation', 'butadiene–styrene', 'rubbers', 'emulsion', 'polymerization', '“hot”', 'process', '“cold”', 'process', 'material', 'butadiene', 'styrene', 'water', 'fatty', 'acid', 'soap', 'n-dodecyl', 'mercaptan', 't-dodecyl', 'mercaptan', 'potassium', 'persulfate', 'auxiliary', 'surfactant', 'potassium', 'chloride', 'p-menthane', 'hydroperoxide', 'ferrous', 'sulfate', 'ethylenediamine', 'tetraacetic', 'acid', 'sodium', 'salt', 'sodiumformaldehyde', 'sulfoxylate', 'from', 'ref', '127', 'patent', 'literature', 'parts', '75', '25', '180', '50', '05', '–', '03', '–', '–', '–', '–', '–', 'purpose', 'comonomer', 'comonomer', 'carrier', 'emulsiﬁer', 'chain', 'transferring', 'agent', '–', 'initiator', '–', '–', '–', '–', '–', 'parts', '72', '28', '180', '45', '–', '02', '–', '03', '03', '006', '001', '005', '005', 'purpose', 'comonomer', 'comonomer', 'carrier', 'emulsiﬁer', '–', 'chain', 'transferring', 'agent', '–', 'stabilizer', 'stabilizer', 'initiator', 'system', '1930s', 'followed', 'particularly', 'strong', 'impetus', 'united', 'states', 'world', 'war', 'ii', 'this', 'led', 'development', 'gr-s', 'rubber', 'united', 'states', 'buna-s', 'rubber', 'germany', 'after', 'world', 'war', 'ii', 'reﬁnements', 'introduced', 'preparatory', 'procedures', '“cold”', 'rubber', 'developed', 'industrially', 'copolymer', 'prepared', 'emulsion', 'copolymeriza-', 'tion', 'butadiene', 'styrene', 'low', 'temperatures', 'continuous', 'process', 'a', 'typical', 'product', 'random', 'distribution', 'copolymer', 'butadiene', 'content', 'ranging', '70', '75.', 'the', 'diene', 'monomer', 'placement', 'roughly', '18', 'cis-14', '65', 'trans-14', '17', '12', 'mn', 'copolymers', '100000', 'a', '“redox”', 'initiator', 'used', 'cold', 'process', '“hot”', 'one', 'also', '“hot”', 'process', 'carried', '50(cid1)c', '12', 'h', 'approximately', '72', 'conversion', 'the', '“cold”', 'process', 'also', 'carried', '12', 'h', '5(cid1)c', '60', 'conversion', 'the', 'two', 'recipes', 'preparation', 'gr-s', 'rubbers', 'shown', 'table', '610', 'comparison', '“hot”', '“cold”', 'processes', 'in', 'polymerizations', 'unreacted', 'monomer', 'removed', 'in', '“hot”', 'one', 'reaction', 'often', 'quenched', 'addition', 'hydroquinone', '“cold”', 'one', 'addition', 'n', 'n-diethyldithiocarbamate', 'after', 'monomers', 'steam', 'stripped', 'processes', 'antioxidant', 'like', 'n-phenyl-2-naphthylamine', 'added', 'the', 'latex', 'usually', 'coagulated', 'addition', 'sodium', 'chloride–sulfuric', 'acid', 'solution', 'the', '“cold”', 'process', 'yields', 'polymers', 'less', 'branching', '“hot”', 'one', 'slightly', 'higher', 'trans', 'cis', 'ratios', 'during', 'middle', '1960s', 'series', 'butadiene–styrene', 'isoprene–styrene', 'block-copolymer-', 'elastomers', 'developed', 'these', 'materials', 'possess', 'typical', 'rubber-like', 'properties', 'ambient', 'temperatures', 'act', 'like', 'thermoplastic', 'resins', 'elevated', 'ones', 'the', 'copolymers', 'vary', 'diblock', 'structures', 'styrene', 'butadiene', 'n', '611', 'copolymers', 'dienes', 'fig', '64', 'illustration', 'polystyrene', 'polybutadiene', 'domains', '363', 'triblock', 'ones', 'like', 'styrene–butadiene–styrene', 'n', 'a', 'typical', 'triblock', 'copolymer', 'may', 'consist', '150', 'styrene', 'units', 'end', 'macromol-', 'ecule', '1000', 'butadiene', 'units', 'center', 'the', 'special', 'physical', 'properties', 'block', 'copolymers', 'due', 'inherent', 'incompatibility', 'polystyrene', 'polybutadiene', 'polyisoprene', 'domains', 'blocks', 'within', 'bulk', 'material', 'the', 'polystyrene', 'domains', 'dispersed', 'continuous', 'matrixes', 'polydienes', 'major', 'components', 'at', 'ambient', 'temperature', 'tg', 'polystyrene', 'domains', 'rigid', 'immobilize', 'ends', 'polydiene', 'segments', 'in', 'effect', 'serve', 'ﬁller', 'particles', 'cross-links', 'above', 'tg', 'polystyrene', 'however', 'domains', 'easily', 'disrupted', 'material', 'processed', 'thermoplastic', 'polymer', 'the', 'separation', 'domains', 'illustrated', 'fig', '64', 'separations', 'aggregations', 'these', 'thermoplastic', 'elastomers', 'prepared', 'anionic', 'solution', 'polymerization', 'organome-', 'tallic', 'catalysts', 'a', 'typical', 'example', 'preparation', 'polymerization', '7525', 'mixture', 'butadienestyrene', 'presence', 'sec-butyllithium', 'hydrocarbon–ether', 'solvent', 'blend', 'at', 'reaction', 'conditions', 'butadiene', 'blocks', 'form', 'ﬁrst', 'butadiene', 'consumed', 'styrene', 'blocks', 'form', 'in', 'preparations', 'monomers', 'added', 'sequentially', 'taking', 'advantage', '“living”', 'nature', 'anionic', 'polymerizations', 'these', 'block', 'copolymers', 'narrow', 'molecular', 'weight', 'distributions', 'also', 'sizes', 'blocks', 'restricted', 'narrow', 'ranges', 'maintain', 'optimum', 'elastomeric', 'properties', '6112', 'gr-n', 'rubber', 'butadiene–acrylonitrile', 'rubbers', 'another', 'group', 'useful', 'synthetic', 'elastomers', 'these', 'copolymers', 'originally', 'developed', 'germany', 'found', 'superior', 'oil', 'resistance', 'butadiene–styrene', 'rubbers', 'commercially', 'materials', 'produced', 'free-radical', 'emulsion', '364', '6', 'common', 'chain-growth', 'polymers', 'polymerization', 'similarly', 'butadiene–styrene', 'copolymers', 'similarly', '“hot”', '“cold”', 'processes', 'employed', '“low”', '“medium”', '“high”', 'grades', 'solvent-resistant', 'copolymers', 'formed', 'depending', 'upon', 'amount', 'acrylonitrile', 'copolymer', 'range', '25', '40.', 'the', 'butadiene', 'placement', 'copolymers', 'approximately', '775', 'trans-14', '125', 'cis-14', '10', '12-units', 'also', 'polymers', 'formed', '“cold”', 'process', 'less', 'branched', 'narrower', 'molecular', 'weight', 'distribution', 'formed', '“hot”', 'process', 'an', 'interesting', 'alternating', 'copolymer', 'butadiene', 'acrylonitrile', 'developed', 'japan', '147.', 'the', 'copolymer', 'formed', 'coordination', 'catalysts', 'consisting', 'alr3', 'alcl3', 'vocl3', 'suspension', 'polymerization', 'process', 'the', 'product', '94', 'alternate', 'reported', 'good', 'mechanical', 'properties', 'good', 'oil', 'resistance', '612', 'polystyrene', 'polystyrene-like', 'polymers', 'styrene', 'produced', 'united', 'states', 'benzene', 'ethylene', 'friedel–craft', 'reaction', 'followed', 'dehydrogenation', 'alumina', '600(cid1)c', 'polystyrene', 'ﬁrst', 'prepared', '1839', 'though', 'material', 'confused', 'oxidation', 'product', 'styrene', 'monomer', '148.', 'today', 'polystyrene', 'produced', 'large', 'quantities', 'much', 'known', 'material', '6121', 'preparation', 'polystyrene', 'free-radical', 'mechanism', 'styrene', 'one', 'monomers', 'lends', 'polymerization', 'free-radical', 'cationic', 'anionic', 'coordination', 'mechanisms', 'this', 'due', 'several', 'reasons', 'one', 'resonance', 'stabilization', 'reactive', 'polystyryl', 'species', 'transition', 'state', 'lowers', 'activation', 'energy', 'propagation', 'reaction', 'another', 'low', 'polarity', 'monomer', 'this', 'facilitates', 'attack', 'free-radicals', 'differently', 'charged', 'ions', 'metal', 'complexes', 'in', 'addition', 'side', 'reactions', 'occur', 'ionic', 'polymerizations', 'monomers', 'functional', 'groups', 'possible', 'styrene', 'polymerizes', 'dark', 'free-radical', 'mechanism', 'slowly', 'presence', 'light', '149.', 'also', 'styrene', 'formed', 'dark', 'reported', 'greater', 'amount', 'syndiotactic', 'placement', '150.', 'the', 'amount', 'branching', 'polymer', 'prepared', 'free-radical', 'mechanism', 'increases', 'temperature', '136.', 'this', 'also', 'depends', 'upon', 'initiator', 'used', '151.', 'the', 'following', 'information', 'evolved', 'free-radical', 'polymerization', 'styrene', '1', 'styrene', 'polymerizes', 'thermally', '151–155.', 'this', 'discussed', 'chap', '3', '2', 'oxygen', 'retards', 'polymerizations', 'styrene', 'at', 'higher', 'temperatures', 'however', 'rate', 'accelerated', 'due', 'peroxide', 'formation', '156.', '3', 'the', 'rate', 'styrene', 'polymerizations', 'bulk', 'initially', 'low', 'conversions', 'ﬁrst', 'order', 'respect', 'monomer', 'concentrations', 'in', 'solution', 'however', 'second', 'order', 'respect', 'monomer', '157.', 'polystyrene', 'manufactured', 'free-radical', 'polymerization', 'atactic', 'isotactic', 'polystyrene', 'formed', 'ziegler–natta', 'catalysts', 'introduced', 'commercially', '1960s', 'failed', 'gain', 'acceptance', 'syndiotactic', 'polystyrene', 'produced', 'commercially', 'industrially', 'free-radical', 'styrene', 'polymerizations', 'carried', 'bulk', 'emulsion', 'solution', 'suspension', 'the', 'clear', 'plastic', 'generally', 'prepared', 'mass', 'polymerization', 'because', 'polysty-', 'rene', 'soluble', 'monomer', 'mass', 'polymerization', 'carried', 'completion', 'results', 'tremendous', 'increase', 'melt', 'viscosity', 'to', 'avoid', 'styrene', 'polymerized', 'bulk', 'agitated', 'kettle', 'reaction', 'carried', '30–40', 'conversion', 'after', 'viscous', 'syrup', 'transferred', 'another', 'type', 'reactor', 'completion', 'reaction', 'according', 'one', 'early', '612', 'polystyrene', 'polystyrene-like', 'polymers', '365', 'fig', '65', 'adiabatic', 'tower', 'mass', 'polymerization', 'styrene', 'german', 'patent', 'polymerization', 'completed', 'plate', 'frame', 'ﬁlter', 'press', '157.', 'water', 'circulating', 'press', 'removes', 'heat', 'reaction', 'solid', 'polymer', 'formed', 'inside', 'frames', 'this', 'process', 'still', 'used', 'places', '158.', 'another', 'approach', 'use', 'adiabatic', 'towers', 'styrene', 'ﬁrst', 'partially', 'polymerized', 'two', 'agitated', 'reaction', 'kettles', '80–100(cid1)c', 'the', 'syrup', 'solution', 'polymer', 'monomer', 'fed', 'continually', 'towers', 'top', 'the', 'temperatures', 'towers', 'gradually', 'increased', '100–110(cid1)c', 'top', '180–200(cid1)c', 'bottom', 'by', 'time', 'material', 'reaches', 'bottom', '3', 'h', 'polymerization', '92–98', 'complete', '159.', 'the', 'unreacted', 'monomer', 'removed', 'recycled', 'a', 'modiﬁcation', 'process', 'remove', 'monomer', 'vapor', 'top', 'tower', 'reuse', 'an', 'improvement', 'procedure', 'use', 'agitated', 'towers', '160.', 'to', 'avoid', 'channeling', 'inside', 'towers', 'better', 'heat', 'transfer', 'three', 'towers', 'arranged', 'series', 'they', 'equipped', 'slow', 'agitators', 'grids', 'pipes', 'cooling', 'heating', '160.', 'polymeric', 'melt', 'heated', '95', '225(cid1)c', 'reduce', 'viscosity', 'help', 'heat', 'transfer', 'a', 'solvent', 'like', 'ethyl', 'benzene', 'may', 'added', 'a', 'vacuum', 'devolatalizer', 'removes', 'monomer', 'solvent', 'product', '(fig', '65)', '6122', 'polystyrene', 'prepared', 'ionic', 'chain-growth', 'polymerization', 'much', 'research', 'devoted', 'cationic', 'anionic', 'polymerizations', 'an', 'investigation', 'cationic', 'polymerization', 'styrene', 'al(c2h5)2clrcl', '(r', '¼', 'alkyl', 'aryl)', 'catalystcocatalyst', 'system', 'reported', 'kennedy', '161', '162.', 'the', 'efﬁciency', '(polymerization', 'initiation)', 'determined', 'relative', 'stability', 'andor', 'concentration', 'initiating', 'carbocations', 'provided', 'cocatalyst', 'rcl', 'n-butyl', 'isopropyl', 'sec-butyl', 'chlorides', 'exhibit', 'low', 'cocatalytic', 'efﬁciencies', 'low', 'tendency', 'ion', 'formation', 'triphenylmethyl', 'chloride', 'also', 'poor', 'cocatalyst', 'triphenylmethyl', 'ion', 'forms', 'stable', 'propagating', 'styryl', 'ion', 'initiation', 'styrene', 'polymerizations', 'carbocations', 'well', 'established', '163.', '366', '6', 'common', 'chain-growth', 'polymers', 'anionic', 'polymerization', 'styrene', 'amyl', 'sodium', 'yields', 'isotactic', 'polymer', '164.', 'polymerizations', 'catalyzed', 'triphenylmethylpotassium', 'also', 'yield', 'stereospeciﬁc', 'polysty-', 'rene', '165.', 'the', 'true', 'organolithium', 'compounds', '166', '167.', 'in', 'butyllithium', 'initiated', 'polymerizations', 'styrene', 'benzene', 'termination', 'claimed', 'occur', 'association', 'propagating', 'anions', 'lithium', 'counterions', 'another', 'butyllithium', 'molecule', '168.', 'this', 'contradicted', 'claims', 'terminations', 'result', 'association', 'two', 'propagating', 'chains', '169.', 'alﬁn', 'catalysts', 'polymerize', 'styrene', 'yield', 'stereospeciﬁc', 'products', '170.', 'coordination', 'catalysts', 'based', 'aluminum', 'alkyls', 'titanium', 'halides', 'yield', 'isotactic', 'polystyrene', '171–174.', 'the', 'polymer', 'matches', 'isotactic', 'polystyrene', 'formed', 'amylsodium', 'it', 'composed', 'head-to-tail', 'sequences', 'main', 'chain', 'fold', 'helical', 'there', 'three', 'monomer', 'units', 'per', 'helical', 'fold', '171', '172.', 'the', 'catalyst', 'composition', 'however', 'strong', 'bearing', 'microstructure', 'resultant', 'polymer', '175', '176.', 'ishihara', 'et', 'al', 'reported', '1986', 'syndiotactic', 'polystyrene', 'prepared', 'aid', 'organic', 'inorganic', 'titanium', 'compounds', 'activated', 'methylaluminoxane', '177.', 'there', 'much', 'greater', 'incentive', 'commercialize', 'syndiotactic', 'polystyrene', 'isotactic', 'one', 'this', 'isotactic', 'polystyrene', 'crystallizes', 'slow', 'rate', 'that', 'makes', 'impractical', 'many', 'industrial', 'applications', 'syndiotactic', 'polystyrene', 'hand', 'crystallizes', 'fast', 'rate', 'melting', 'point', '275(cid1)c', 'compared', '240(cid1)c', 'isotactic', 'one', 'suitable', 'use', 'strong', 'structural', 'material', 'many', 'catalysts', 'investigated', 'preparation', 'syndiotactic', 'polystyrene', 'high', 'activity', 'claimed', 'half-sandwich', 'titanocenes', 'type', 'cpticl3', 'indticl3', 'substituted', 'indticl3', 'methylaluminoxane', 'cocatalyst', '178.', 'it', 'also', 'reported', 'bme(c6f5)3', 'borates', 'used', 'precursors', 'instead', 'methyl-aluminoxane', '178.', 'in', 'addition', 'disclosed', 'ﬂuorinated', 'catalysts', 'exhibit', 'high', 'activities', 'produce', 'polymers', 'higher', 'molecular', 'weight', 'polymers', '178.', 'the', 'mechanism', 'polymerization', 'investigated', 'different', 'groups', 'grassi', 'zambelli', 'claim', 'syndiotactic', 'styrene', 'polymerization', 'proceeds', 'secondary', 'insertion', 'styrene', 'ti–alkyl', '(or', 'growing', 'polymer', 'chain)', 'bond', 'in', 'half', 'titanocene', 'catalyst', 'polymer', 'chain', 'appears', 'z6-coordinated', 'metal', 'active', 'species', '179:', 'ti', 'r', 'n', '(r)bc6f5)3', '612', 'polystyrene', 'polystyrene-like', 'polymers', '367', 'maron', 'coworkers', '180', 'reported', 'theoretical', 'methods', 'used', 'investigate', 'syndiospeciﬁcity', 'styrene', 'polymerization', 'catalyzed', 'single-site', 'single-component', 'allyl', 'ansa-lanthanidocenes', '(s)', 'h', 'h', 'pol', 'ln', '(r)', 'si', 'two', 'limiting', 'chain', 'end', 'stereocontrol', 'mechanisms', 'studied', 'namely', 'migratory', 'insertion', 'site', 'epimerization', 'site', 'stereoconﬁguration', 'independent', 'backside', 'insertion', '“stationary”', 'polymer', 'chain', 'four', 'consecutive', 'insertions', 'styrene', 'computed', 'reveal', '(i)', 'backside', 'insertions', 'favorable', 'least', 'favorable', 'frontside', 'insertions', 'the', 'formation', 'syndiotactic', 'polymer', 'controlled', 'thermodynamics', 'moreover', 'odd', '(ﬁrst', 'third)', 'insertions', '21-down-si-type', 'kinetically', 'favored', '21-up-re-ones', 'this', 'control', 'conjunction', 'two', 'effects', 'minimization', 'styrene–styrene', 'styrene', '(phenyl', 'ring)–ﬂuorenyl', 'repulsions', 'the', 'steric', 'hindrance', 'polymer', 'chain', 'induces', 'fourth', 'insertion', 'exocyclic', 'coordination', 'ﬂuorenyl', 'ligand', 'compensated', 'z6', 'coordination', 'one', 'phenyl', 'ring', 'growing', 'chain', 'syndiotactic', 'polystyrene', 'available', 'commercially', 'trade', 'name', 'questra', 'this', 'material', 'produced', 'aid', 'metallocene', 'catalyst', 'sold', 'several', 'grades', '181.', 'there', 'small', 'interest', 'forming', 'isotactic', 'polystyrenes', 'vary', 'narrow', 'molecular', 'weight', 'distributions', 'limited', 'practical', 'applications', 'purely', 'academic', 'interests', 'several', 'preparations', 'virtually', 'monodisperse', 'polystyrenes', 'mwmn', '¼', '106', 'anionic', 'polymerizations', 'developed', 'the', 'materials', 'available', 'commercially', '181–186,', 'small', 'quantities', 'use', 'standards', 'gpc', '6123', 'polymers', 'substituted', 'styrenes', 'many', 'derivatives', 'styrene', 'readily', 'synthesized', 'some', 'commercially', 'available', 'one', 'a-methyl', 'styrene', 'it', 'formed', 'propylene', 'benzene', 'process', 'similar', 'styrene', 'preparation', 'alcl3', '368', '6', 'common', 'chain-growth', 'polymers', 'due', 'allylic', 'nature', 'a-methyl', 'styrene', 'cannot', 'polymerized', 'free-radical', 'mechanism', 'it', 'polymerizes', 'readily', 'however', 'ionic', 'one', 'resins', 'based', 'copolymers', 'a-methyl', 'styrene', 'available', 'commercially', 'other', 'styrene', 'derivatives', 'obtained', 'commercially', '1', 'alkyl', 'aryl', 'substituted', 'styrenes', 'r1', 'r2', 'alkyl', 'aryl', 'groups', '2', 'halogen', 'derivatives', 'x1', 'x2', '¼', 'f', 'cl', 'br', 'i', '3', 'polar-substituted', 'styrenes', 'r', 'r', 'x1', 'x2', 'r', 'r', '¼', 'cn', 'cho', 'cooh', 'ococh3', 'oh', 'och3', 'no2', 'nh2', 'so3h', 'vinyl', 'toluene', 'polymerizes', 'readily', 'free-radical', 'mechanism', '100(cid1)c', 'the', 'absolute', 'rate', 'temperature', 'greater', 'styrene', 'the', 'activation', 'energy', 'vinyl', 'toluene', 'polymerization', '17–19', 'kcal', 'per', 'mole', 'styrene', '21', 'kcal', 'mole', 'this', 'monomer', 'also', 'polymerized', 'ionic', 'coordination', 'mechanisms', 'earlier', 'attempts', 'polymerization', 'a-methyl', 'styrene', 'ziegler–natta', 'catalysts', 'successful', '187', '188.', 'later', 'however', 'shown', 'polymeri-', 'zation', 'take', 'place', 'ticl4al(c2h5)3', '(cid3)78(cid1)c', 'the', 'activity', 'catalyst', 'dp', 'depend', 'ratio', 'aluminum', 'titanium', 'nature', 'solvent', 'aging', 'catalyst', '189.', 'the', 'optimum', 'ratio', 'aluminum', 'alkyl', 'titanium', 'chloride', '10–12', 'mixing', 'aging', 'catalyst', 'must', 'done', 'room', 'temperature', 'valence', 'titanium', 'must', 'maintained', '3', '4', '189.', 'this', 'led', 'sakurada', 'suggest', 'reaction', 'actually', 'proceeds', 'via', 'cationic', 'rather', 'coordinated', 'anionic', 'mechanism', '189.', 'various', 'reports', 'literature', 'describe', 'cationic', 'polymerizations', 'a-methyl', 'styrene', 'lewis', 'acids', '190–192.', 'the', 'products', 'mostly', 'low', 'molecular', 'weight', 'polymers', 'containing', 'unsaturation', 'pendant', 'phenylindane', 'groups', 'a', 'high', 'molecular', 'weight', 'polymer', 'prepared', 'a-methyl', 'styrene', 'cationic', 'polymerization', '(cid3)90', '(cid3)130(cid1)c', 'alcl3', 'ethyl', 'chloride', 'carbon', 'disulﬁde', '193.', 'the', 'product', 'narrow', 'molecular', 'weight', 'distribution', 'some', 'tg', 'tm', 'values', 'polystyrene-like', 'materials', 'including', 'isotactic', 'polystyrene', 'presented', 'table', '611', '612', 'polystyrene', 'polystyrene-like', 'polymers', '369', 'table', '611', 'transition', 'temperatures', 'poly(phenylene', 'alkene)s', 'tg', '((cid1)c)', 'polymer', '80', 'references', '178', 'tm', '((cid1)c)', '(i)', '240', '(s)', '272', 'references', '178', '7381', '178', 'amorphous', '176', '178', '178', '178', '123', '178', '219', '310', '330', '240', '77', '92', '91', '84', '60', '80', '177', '290', '176', '177', '207–208', '176', '177', '2', '177', '176', '176', '176', '176', '(continued)', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', '370', 'table', '611', '(continued)', 'polymer', '6', 'common', 'chain-growth', 'polymers', 'tg', '((cid1)c)', '34–40', 'references', '177', 'tm', '((cid1)c)', '180', 'references', '177', 'n', 'n', 'n', '60–65', '177', '240', '177', '10', '177', '162–168', '177', 'from', 'refs', '196–198', '613', 'copolymers', 'styrene', 'many', 'copolymers', 'styrene', 'manufactured', 'large', 'commercial', 'scale', 'because', 'styrene', 'copolymerizes', 'readily', 'many', 'monomers', 'possible', 'obtain', 'wide', 'distribution', 'properties', 'random', 'copolymers', 'form', 'quite', 'readily', 'free-radical', 'mechanism', '185', '194.', 'some', 'also', 'formed', 'ionic', 'mechanism', 'in', 'addition', 'graft', 'block', 'copolymers', 'styrene', 'also', 'among', 'commercially', 'important', 'materials', 'most', 'comonomers', 'differ', 'styrene', 'polarity', 'reactivity', 'a', 'desired', 'copolymer', 'composition', 'achieved', 'however', 'utilization', 'copolymerization', 'parameters', 'based', 'kinetic', 'data', 'quantum-chemical', 'considerations', 'this', 'done', 'industrially', 'preparations', 'styrene–acry-', 'lonitrile', 'styrene–methyl', 'methacrylate', 'styrene–maleic', 'anhydride', 'copolymers', 'different', 'compositions', '6131', 'high-impact', 'polystyrene', 'for', 'many', 'applications', 'homopolymer', 'styrene', 'brittle', 'to', 'overcome', 'many', 'different', 'approaches', 'originally', 'tried', 'these', 'included', 'use', 'high', 'molecular', 'weight', 'polymers', 'use', 'plasticizers', 'ﬁllers', '(glass', 'ﬁber', 'wood', 'ﬂour', 'etc)', 'deliberate', 'orientation', 'polymeric', 'chains', 'copolymerization', 'addition', 'rubbery', 'substances', 'effect', 'plasticizers', 'severe', 'practical', 'use', 'use', 'high', 'molecular', 'weight', 'polymers', 'exhibits', 'marginal', 'improvement', 'use', 'ﬁllers', '613', 'copolymers', 'styrene', '371', 'though', 'beneﬁcial', 'mostly', 'conﬁned', 'united', 'states', 'orientation', 'limited', 'sheets', 'ﬁlaments', 'copolymerization', 'usually', 'lowers', 'softening', 'point', 'much', 'addition', 'rubbery', 'materials', 'however', 'improve', 'impact', 'resistance', 'polystyrene', 'this', 'done', 'therefore', 'extensively', 'the', 'common', 'rubbers', 'used', 'purpose', 'butadiene–styrene', 'copolymers', 'some', 'butadiene', 'homopolymers', 'also', 'used', 'lesser', 'extent', 'the', 'high-impact', 'polystyrene', 'presently', 'prepared', 'dissolving', 'rubber', 'styrene', 'monomer', 'polymerizing', 'styrene', 'this', 'polymerization', 'either', 'done', 'bulk', 'suspension', 'the', 'product', 'contains', 'styrene–butadiene', 'rubber', 'styrene', 'homopolymer', 'considerable', 'portion', 'styrene-graft', 'copolymer', 'forms', 'polystyrene', 'radicals', 'attack', 'rubber', 'molecules', 'the', 'product', 'enhanced', 'impact', 'resistance', 'past', 'practices', 'however', 'consisted', 'simply', 'blending', 'mixture', 'polystyrene', 'rubber', 'two-roll', 'mill', 'high', 'shear', 'internal', 'mixer', 'passing', 'extruder', 'the', 'impact', 'strength', 'product', 'moderately', 'better', 'unmodiﬁed', 'polymer', 'another', 'procedure', 'blend', 'polystyrene', 'emulsion', 'latex', 'styrene–butadiene', 'rubber', 'emulsion', 'latex', 'coagulate', 'two', 'together', 'the', 'product', 'also', 'marginally', 'better', 'impact', 'strength', 'styrene', 'homopolymer', 'this', 'practice', 'however', 'may', 'still', 'existence', 'places', 'in', 'high-impact', 'polystyrene', 'rubber', 'exists', 'discrete', 'droplets', 'less', '50', 'mm', 'diameter', 'in', 'effect', 'polymerization', 'serves', 'form', 'oil', 'oil', 'emulsion', '199', 'polystyrene', 'continuous', 'phase', 'rubber', 'dispersed', 'phase', 'the', 'graft', 'copolymer', 'forms', 'serves', '“emulsify”', 'heterogeneous', 'polymer', 'solution', '200.', 'commercial', 'high-impact', 'polystyrene', 'usually', 'contains', '5–20', 'styrene–butadiene', 'rubber', 'the', 'particle', 'size', 'ranges', '1', '10', 'mm', 'high-impact', 'polystyrene', 'may', 'much', 'seven', 'times', 'impact', 'strength', 'polystyrene', 'half', 'tensile', 'strength', 'lower', 'hardness', 'lower', 'softening', 'point', '6132', 'abs', 'resins', 'styrene–acrylonitrile', 'copolymers', 'produced', 'commercially', 'use', 'structural', 'plastics', 'the', 'typical', 'acrylonitrile', 'content', 'resins', '20', '30.', 'these', 'materials', 'better', 'solvent', 'oil', 'resistance', 'polystyrene', 'higher', 'softening', 'point', 'in', 'addition', 'exhibit', 'better', 'resistance', 'cracking', 'crazing', 'enhanced', 'impact', 'strength', 'although', 'acrylonitrile', 'copolymers', 'enhanced', 'properties', 'polystyrene', 'still', 'inadequate', 'many', 'applications', 'acrylonitrile–butadiene–styrene', 'polymers', 'known', 'abs', 'resins', 'therefore', 'developed', 'although', 'abs', 'resins', 'potentially', 'produced', 'variety', 'ways', 'two', 'main', 'processes', 'in', 'one', 'acrylonitrile–styrene', 'copolymer', 'blended', 'butadiene–acrylonitrile', 'rubber', 'in', 'one', 'interpolymers', 'formed', 'polybutadiene', 'styrene', 'acrylonitrile', 'in', 'ﬁrst', 'one', 'two', 'materials', 'blended', 'rubber', 'mill', 'internal', 'mixer', 'blending', 'two', 'materials', 'also', 'achieved', 'combining', 'emulsion', 'latexes', 'two', 'materials', 'together', 'coagulating', 'mixture', 'peroxide', 'must', 'added', 'blends', 'order', 'achieve', '-', 'cross-linking', 'elastomer', 'attain', 'optimum', 'properties', 'a', 'wide', 'range', 'blends', 'made', 'technique', 'various', 'properties', '201.', 'most', 'common', 'commercial', 'blends', 'abs', 'resins', 'may', 'contain', '70', 'parts', 'styrene–acrylonitrile', 'copolymer', '(7030)', '40', 'parts', 'butadiene–nitrile', 'rubber', '(6535)', 'in', 'second', 'process', 'styrene', 'acrylonitrile', 'copolymerized', 'presence', 'polybutadi-', 'ene', 'latex', 'the', 'product', 'mixture', 'butadiene', 'homopolymer', 'graft', 'copolymer', '372', '6', 'common', 'chain-growth', 'polymers', '6133', 'copolymers', 'styrene', 'maleic', 'anhydride', 'many', 'styrene–maleic', 'anhydride', 'copolymers', 'produced', 'commercially', 'special', 'uses', 'these', 'formed', 'free-radical', 'copolymerization', 'many', 'commercial', 'grades', 'partially', 'esteriﬁed', 'molecular', 'weights', 'polymers', 'may', 'range', '1500', '50000', 'depending', 'upon', 'source', 'the', 'melting', 'points', 'copolymers', 'vary', '110', '220(cid1)c', 'depending', 'upon', 'molecular', 'weight', 'degree', 'hydrolysis', 'esteriﬁcation', 'also', 'ratio', 'styrene', 'maleic', 'anhydride', 'only', 'small', 'percent', 'styrene', 'copolymers', 'reported', 'literature', 'achieved', 'industrial', 'impor-', 'tance', 'some', 'interesting', 'copolymers', 'styrene', 'reported', 'utilized', 'commercially', 'copolymers', 'various', 'unsaturated', 'nitriles', 'this', 'includes', 'vinylidene', 'cyanide', 'fumaronitrile', 'malononitrile', 'methacrylonitrile', 'acrylonitrile', 'cinnamonitrile', '292.', 'often', 'copolymerization', 'styrene', 'nitriles', 'yields', 'copolymers', 'higher', 'heat', 'distortion', 'temperature', 'higher', 'tensiles', 'better', 'craze', 'resistance', 'higher', 'percent', 'elongation', 'styrene', 'also', 'copolymerized', 'many', 'acrylic', 'methacrylic', 'esters', 'products', 'better', 'weathering', 'properties', 'often', 'form', 'copolymerization', 'acrylates', 'lowers', 'value', 'tg', '203.', '614', 'polymers', 'acrylic', 'methacrylic', 'esters', 'there', 'many', 'synthetic', 'procedures', 'preparing', 'acrylic', 'acid', 'esters', 'one', 'way', 'used', 'early', 'make', 'acrylic', 'esters', 'ethylene', 'oxide', 'o', 'hcn', 'ho', 'n', 'h2o', 'n', '35', '-', '40', 'oc', 'amine', 'catalyst', 'or', 'roh', 'h2so4', 'o', 'h2o', 'nh3', 'o', 'oh', 'another', 'route', 'oxidation', 'propylene', 'cobalt', 'molybdenum', 'catalyst', '400–500(cid1)c', 'o', 'o', 'o', 'o', 'roh', 'oh', 'o', 'or', 'many', 'industrial', 'preparations', 'start', 'acetylene', 'carbon', 'monoxide', 'alcohol', 'water', 'co', 'roh', 'ni(co)4', 'co', 'roh', 'ni', 'o', 'o', 'or', 'oh', '614', 'polymers', 'acrylic', 'methacrylic', 'esters', '373', 'one', 'route', 'a-methyl', 'acrylic', 'acid', '(or', 'methacrylic', 'acid)', 'esters', 'cyanohydrin', 'reaction', 'acetone', 'o', 'hcn', 'n', 'h2so4', 'n', 'ro', 'o', 'oh', 'roh', 'h2so4', 'h2o', 'ho', 'o', '6141', 'polymerizations', 'acrylic', 'methacrylic', 'esters', 'free-radical', 'bulk', 'polymerizations', 'acrylate', 'esters', 'exhibit', 'rapid', 'rate', 'accelerations', 'low', 'conversions', 'this', 'often', 'results', 'formation', 'high', 'molecular', 'weight', 'polymer', 'cross-linked', 'material', 'the', 'cross-linking', 'result', 'chain', 'transferring', 'abstractions', 'labile', 'tertiary', 'hydrogens', 'already', 'formed', '“dead”', 'polymeric', 'chains', '204.', 'eventually', 'termination', 'combination', 'branched', 'radicals', 'leads', 'cross-linked', 'structures', 'addition', 'chain', 'transferring', 'agents', 'like', 'mercaptans', '(that', 'reduces', 'length', 'primary', 'chains)', 'helps', 'prevent', 'gel', 'formation', 'there', 'labile', 'tertiary', 'hydrogens', 'methacrylic', 'esters', 'the', 'growing', 'methacrylate', 'radicals', 'still', 'capable', 'abstracting', 'hydrogens', 'a-methyl', 'groups', 'such', 'abstractions', 'however', 'require', 'energy', 'important', 'problem', 'polymerizations', 'methacrylic', 'esters', '205.', 'nevertheless', 'occasional', 'formation', 'cross-linked', 'poly(alkyl', 'methacrylate)s', 'occur', 'this', 'due', 'chain', 'transferring', 'alcohol', 'moiety', '206', '210.', 'the', 'termination', 'reaction', 'free-radical', 'polymerizations', 'esters', 'acrylic', 'methacrylic', 'acids', 'takes', 'place', 'recombination', 'disproportionation', '206', '207.', 'methyl', 'methacrylate', 'polymerizations', 'however', 'terminate', '25(cid1)c', 'predominantly', 'disproportionation', '205.', 'oxygen', 'inhibits', 'free-radical', 'polymerization', 'a-methyl', 'methacrylate', '208.', 'the', 'reaction', 'oxygen', 'results', 'formation', 'low', 'molecular', 'weight', 'polymeric', 'peroxides', 'subsequently', 'decom-', 'pose', 'formaldehyde', 'methyl', 'pyruvate', '210:', 'o', 'o', 'o2', 'o', 'n', 'o', 'o', 'ch3o', 'cho', 'o', 'o', 'o', 'oxygen', 'less', 'effective', 'inhibiting', 'polymerizations', 'acrylic', 'esters', 'it', 'reacts', '400', 'times', 'faster', 'methacrylic', 'radicals', 'acrylic', 'ones', 'nevertheless', 'even', 'small', 'quantities', 'oxygen', 'affect', 'polymerization', 'rates', 'acrylic', 'esters', '216.', 'this', 'includes', 'photopolymerizations', 'gaseous', 'ethyl', 'acrylate', 'affected', 'oxygen', 'moisture', '217.', 'acrylic', 'methacrylic', 'esters', 'polymerize', 'free-radical', 'mechanism', 'atactic', 'polymers', 'the', 'sizes', 'alcohol', 'portions', 'esters', 'determine', 'tg', 'values', 'resultant', 'polymers', 'they', 'also', 'determine', 'solubility', 'resultant', 'polymers', 'hydrocarbon', 'solvents', 'oils', '374', '6', 'common', 'chain-growth', 'polymers', 'solvents', 'inﬂuence', 'rate', 'free-radical', 'homopolymerization', 'acrylic', 'acid', 'copolymerization', 'monomers', 'hydrogen', 'bonding', 'solvents', 'slow', 'reaction', 'rates', '219.', 'due', 'electron', 'withdrawing', 'nature', 'ester', 'groups', 'acrylic', 'methacrylic', 'ester', 'polymerize', 'anionic', 'cationic', 'mechanisms', 'lithium', 'alkyls', 'effective', 'initiators', 'a-methyl', 'methacrylate', 'polymerization', 'yielding', 'stereospeciﬁc', 'polymers', '213.', 'isotactic', 'poly(methyl', 'methacrylate)', 'forms', 'hydrocarbon', 'solvents', '214.', 'block', 'copolymers', 'isotactic', 'syndiotactic', 'poly(methyl', 'methacry-', 'late)', 'form', 'solvents', 'medium', 'polarity', 'syndiotactic', 'polymers', 'form', 'polar', 'solvents', 'like', 'ethylene', 'glycol', 'dimethyl', 'ether', 'pyridine', 'this', 'solvent', 'inﬂuence', 'related', 'lewis', 'basicity', '215', 'following', 'order', 'tetrahydrofurantetrahydropyrandioxanediethyl', 'ether', 'furthermore', 'polymerizations', 'solvating', 'media', 'like', 'ethylene', 'glycol', 'dimethyl', 'ether', 'tetrahydro-', 'furan', 'pyridine', 'using', 'biphenylsodium', 'biphenyllithium', 'yield', 'virtually', 'monodisperse', 'syndiotactic', 'poly(methyl', 'methacrylate)', '216.', 'the', 'nature', 'counterion', 'anionic', 'polymerizations', 'methyl', 'methacrylate', 'liquid', 'ammonia', 'alkali', 'metal', 'amide', 'alkali', 'earth', 'metal', 'amide', 'catalysts', 'important', 'variable', '217.', 'lithium', 'calcium', 'amides', 'yield', 'high', 'molecular', 'weight', 'polymers', 'though', 'reactions', 'tend', 'slow', 'sodium', 'amide', 'hand', 'yields', 'rapid', 'polymerizations', 'low', 'molecular', 'weight', 'polymers', 'polymers', 'formed', 'sodium', 'amide', 'however', 'narrower', 'molecular', 'weight', 'distribution', 'obtained', 'lithium', 'calcium', 'amides', 'calcium', 'amide', 'also', 'yields', 'high', 'molecular', 'weight', 'polymers', 'ethyl', 'acrylate', 'methyl', 'methacrylate', 'monomers', 'aromatic', 'aliphatic', 'solvents', 'temperatures', '(cid3)8', '110(cid1)c', 'when', 'however', 'tetrahydrofuran', 'acetonitrile', 'used', 'solvents', 'much', 'lower', 'molecular', 'weight', 'products', 'form', '218.', 'products', 'anionic', 'polymerizations', 'methyl', 'methacrylate', 'catalyzed', 'grignard', 'reagents', '(rmgx)', 'vary', 'nature', 'r', 'x', 'groups', 'reaction', 'temperature', 'nature', 'solvent', '219–221.', 'secondary', 'alkyl', 'grignard', 'reagents', 'give', 'highest', 'yields', 'fastest', 'rates', 'reactions', 'isotacticity', 'products', 'increases', 'temperature', 'when', 'anion-radicals', 'alkali', 'metal', 'ketyls', 'benzophenone', 'initiate', 'polymerizations', 'methyl', 'methacrylate', 'amorphous', 'polymers', 'form', 'temperatures', '(cid3)78', '65(cid1)c', '222.', 'sodium', 'dispersions', 'hexane', 'yield', 'syndiotactic', 'poly(methyl', 'methacrylate)', '223.', 'a', '60–65', 'conversion', 'obtained', '24-h', 'period', 'reaction', 'temperature', '20–25(cid1)c', 'lithium', 'dispersions', '224,', 'butyllithium', '203,', 'grignard', 'reagents', '225', '226', 'yield', 'crystalline', 'isotactic', 'poly(t-butyl', 'acrylate)', 'the', 'reactions', 'take', 'place', 'bulk', 'hydrocarbon', 'solvents', 'isotactic', 'poly(isopropyl', 'acrylate)', 'forms', 'grignard', 'reagents', '226', '227.', 'coordination', 'polymerizations', 'methyl', 'methacrylate', 'diethyliron–bipyridyl', 'complex', 'nonpolar', 'solvents', 'like', 'benzene', 'toluene', 'yield', 'stereoblock', 'polymers', 'in', 'polar', 'solvents', 'however', 'like', 'dimethylformamide', 'acetonitrile', 'products', 'rich', 'isotactic', 'placement', '229.', 'there', 'many', 'reports', 'literature', 'polymerizations', 'acrylic', 'methacrylic', 'esters', 'ziegler–natta', 'catalysts', '230–233.', 'the', 'molecular', 'weights', 'products', 'microstructures', 'rates', 'polymerizations', 'depend', 'upon', 'metal', 'alkyl', 'transition', 'metal', 'salt', 'used', 'the', 'ratios', 'catalyst', 'components', 'also', 'important', '234', '235.', 'in', '1992', 'yasuda', 'et', 'al', '236', '237', 'reported', 'organolanthanide', 'complexes', 'type', 'cp2sm-r', '(where', 'cp', 'pentamethyl', 'cyclopentadienyl', 'r', 'either', 'alkyl', 'alkylaluminum', 'hydride)', 'initiate', 'highly', 'syndiotactic', 'living', 'polymerizations', 'methacrylates', 'it', 'also', 'reported', 'lanthanide', 'complexes', 'cp2yb(thf)1–3', 'cp2sm(thf)2', '(indenyl)2yb(thf)2', 'also', 'initiate', 'polymerizations', 'methylmethacrylate', '238.', 'although', 'low', 'initiator', 'efﬁciencies', '614', 'polymers', 'acrylic', 'methacrylic', 'esters', '375', 'observed', 'living', 'polymerizations', 'the', 'polymers', 'formed', 'dispersity', '11', 'high', 'syndiotactic', 'sequences', 'novak', 'boffa', 'studying', 'lanthanide', 'complexes', 'observed', 'unusual', 'facile', 'organometallic', 'electron', 'transfer', 'process', 'takes', 'place', 'generates', 'situ', 'bimetallic', 'lanthanide(iii)', 'initiators', 'polymerizations', 'methacrylates', '239.', 'they', 'concluded', 'methyl', 'methacrylate', 'polymerizations', 'initiated', 'cp2sm', 'complexes', 'occur', 'reductive', 'dimerizations', 'methyl', 'methacrylate', 'molecules', 'form', '“bisinitiators”', 'consists', 'two', 'samarium(iii)', 'enolates', 'joined', 'double', 'bond', 'terminally', '239.', 'their', 'conclusion', 'based', 'tendency', 'cp2sm', 'complexes', 'reductively', 'couple', 'unsaturated', 'molecules', 'co2me', 'o', '2', 'sm', '2', 'och3', 'cp', '2', 'sm(iii)', 'ch3o', 'sm', 'o', 'o', 'sm', 'och3', 'co2ch3', 'excess', 'ch3oh', 'co2ch3', 'n', 'ch3o2c', 'n', 'montei', 'coworkers', '240', 'reported', 'nickel', 'complexes', '(xo)nir(pph3)', '(x', '¼', 'n', 'p)', 'designed', 'polymerization', 'ethylene', 'effective', 'home-', 'copolymerization', 'butyl', 'acrylate', 'methyl', 'methacrylate', 'styrene', 'their', 'role', 'radical', 'initiators', 'demonstrated', 'calculation', 'copolymerization', 'reactivity', 'ratios', 'it', 'shown', 'efﬁciency', 'radical', 'initiation', 'improved', 'addition', 'pph3', 'nickel', 'complexes', 'well', 'increasing', 'temperature', 'the', 'dual', 'role', 'nickel', 'complex', 'radical', 'initiators', 'catalysts', 'exploited', 'succeed', 'copolymerization', 'ethylene', 'butyl', 'acrylate', 'methyl', 'methacrylate', 'multiblock', 'copolymers', 'containing', 'sequences', 'ethylene', 'polar', 'monomers', 'thus', 'prepared', '6142', 'acrylic', 'elastomers', 'polymers', 'lower', 'n-alkyl', 'acrylates', 'used', 'commercially', 'limited', 'extent', 'ethyl', 'butyl', 'acrylates', 'however', 'major', 'components', 'acrylic', 'elastomers', 'the', 'polymers', 'usually', 'formed', 'free-radical', 'emulsion', 'polymerization', 'because', 'acrylate', 'esters', 'sensitive', 'hydrolysis', 'basic', 'conditions', 'polymerizations', 'usually', 'conducted', 'neutral', 'acidic', 'ph', 'the', 'acrylic', 'rubbers', 'like', 'elastomers', 'must', 'cross-linked', 'vulcanized', 'obtain', 'optimum', 'properties', 'cross-linking', '376', '6', 'common', 'chain-growth', 'polymers', 'accomplished', 'reactions', 'peroxides', 'abstractions', 'tertiary', 'hydrogens', 'free', 'radicals', 'o', 'o', 'c2h5', 'r', 'o', 'o', 'c2h5', 'rh', '2', 'o', 'o', 'c2h5', 'o', 'o', 'c2h5', 'c2h5', 'o', 'o', 'another', 'way', 'cross-link', 'acrylic', 'elastomers', 'claisen', 'condensation', 'o', 'o', 'c2h5', 'o', 'o', 'c2h5', 'base', 'o', 'o', 'c2h5', 'o', 'the', 'illustrated', 'cross-linking', 'reactions', 'homopolymers', 'however', 'form', 'elastomers', 'poor', 'aging', 'properties', 'commercial', 'acrylic', 'rubbers', 'therefore', 'copolymers', 'ethyl', 'butyl', 'acrylate', 'small', 'quantities', 'comonomers', 'carry', 'special', 'functional', 'groups', 'cross-linking', 'such', 'comonomers', '2-chloroethylvinyl', 'ether', 'vinyl', 'chloroacetate', 'used', 'small', 'quantities', '(about', '5)', 'these', 'copolymers', 'cross-link', 'reactions', 'polyamines', '6143', 'thermoplastic', 'thermoset', 'acrylic', 'resins', 'among', 'methacrylic', 'ester', 'polymers', 'poly(methyl', 'methacrylate)', 'important', 'one', 'industrially', 'most', 'prepared', 'free-radical', 'polymerizations', 'monomer', 'great', 'deal', 'polymerizations', 'carried', 'bulk', 'typical', 'methods', 'preparation', 'clear', 'sheets', 'rods', 'consist', 'initial', 'partial', 'polymerizations', 'reaction', 'kettles', '90(cid1)c', 'peroxide', 'initiators', 'this', 'done', 'heating', 'stirring', '10', 'min', 'form', 'syrups', 'the', 'products', 'cooled', 'room', '614', 'polymers', 'acrylic', 'methacrylic', 'esters', '377', 'table', '612', 'typical', 'components', 'thermoset', 'acrylic', 'resins', 'monomers', 'contribute', 'rigidity', 'methyl', 'methacrylate', 'ethyl', 'methacrylate', 'styrene', 'vinyl', 'toluene', 'acrylonitrile', 'methacrylonitrile', 'flexibilizing', 'monomers', 'ethyl', 'acrylate', 'isopropyl', 'acrylate', 'butyl', 'acrylate', 'i-octyl', 'acrylate', 'decyl', 'acrylate', 'lauryl', 'methacrylate', 'monomers', 'used', 'cross-linking', 'acrylic', 'acid', 'methacrylic', 'acid', 'hydroxyethyl', 'acrylate', 'hydroxypropyl', 'acrylate', 'glycidyl', 'acrylate', 'glycidyl', 'methacrylate', 'acrylamide', 'aminoethyl', 'acrylate', 'temperature', 'various', 'additives', 'may', 'added', 'the', 'syrups', 'solutions', '20', 'polymer', 'dissolved', 'monomer', 'they', 'poured', 'casting', 'cells', 'polymerizations', 'completed', 'the', 'ﬁnal', 'polymers', 'high', 'molecular', 'weight', '1000000', 'poly(methyl', 'methacrylate)', 'intended', 'surface', 'coatings', 'prepared', 'solution', 'polymerization', 'the', 'molecular', 'weights', 'polymers', '90000', 'reaction', 'products', '40–60', 'solutions', 'often', 'used', 'directly', 'coatings', 'a', 'certain', 'amount', 'poly(methyl', 'methacrylate)', 'also', 'prepared', 'suspension', 'polymerization', 'the', 'molecular', 'weights', 'polymers', '60000', 'used', 'injection', 'molding', 'extrusion', 'thermosetting', 'acrylic', 'resins', 'used', 'widely', 'surface', 'coatings', 'both', 'acrylic', 'methacrylic', 'esters', 'utilized', 'term', 'applied', 'often', 'resins', 'terpolymers', 'even', 'tetra', 'polymers', 'monomer', 'chosen', 'special', 'function', '214.', 'one', 'selected', 'rigidity', 'surface', 'hardness', 'scratch', 'resistance', 'another', 'ability', 'ﬂexibilize', 'ﬁlm', 'third', 'one', 'cross-linking', 'in', 'addition', 'comonomers', 'necessarily', 'acrylic', 'methacrylic', 'esters', 'acids', 'for', 'instance', 'among', 'monomers', 'may', 'chosen', 'rigidity', 'may', 'methyl', 'methacrylate', 'on', 'hand', 'may', 'styrene', 'instead', 'vinyl', 'toluene', 'etc', 'the', 'true', 'components', 'table', '612', 'illustrates', 'common', 'components', 'found', 'thermoset', 'acrylic', 'resins', 'the', 'choice', 'cross-linking', 'reaction', 'may', 'depend', 'upon', 'desired', 'application', 'it', 'may', 'also', 'simply', 'depend', 'upon', 'price', 'particular', 'company', 'manufactures', 'resin', 'simply', 'overcome', 'patent', 'restrictions', 'some', 'common', 'cross-linking', 'reactions', 'illustrated', 'remaining', 'portion', 'section', 'if', 'functional', 'groups', 'carboxylic', 'acids', 'copolymer', 'terpolymer', 'cross-linking', 'accomplished', 'adding', 'diepoxide', 'o', 'o', 'cooh', 'cooh', 'o', 'o', 'o', 'o', 'r', 'oh', 'oh', '378', '6', 'common', 'chain-growth', 'polymers', 'other', 'reactions', 'also', 'cross-link', 'resins', 'pendant', 'carboxylic', 'acid', 'groups', 'for', 'instance', 'one', 'add', 'melamine', 'formaldehyde', 'condensate', 'n', 'o', '2', 'n', 'n', 'n', 'cooh', 'o', 'n', 'n', 'n', 'o', '2', '2', 'n', 'cooh', 'o', 'n', 'o', 'o', 'n', 'n', 'n', 'o', 'n', 'n', 'n', 'o', 'o', 'n', 'a', 'diisocyanate', 'phenolic', 'melamine–formaldehyde', 'resin', 'used', 'well', 'resins', 'pendant', 'hydroxyl', 'groups', 'also', 'cross-linked', 'materials', 'a', 'diisocyanate', 'effective', 'forming', 'urethane', 'linkages', 'when', 'pendant', 'groups', 'epoxides', 'like', 'glycidyl', 'esters', 'cross-linking', 'carried', 'dianhydrides', 'compounds', 'containing', 'two', 'carboxylic', 'acid', 'groups', '241.', 'aminoplast', 'resins', '(urea–formaldehyde', 'melamine–formaldehyde', 'similar', 'ones)', 'also', 'effective', '242.', 'pendant', 'amide', 'groups', 'terpolymers', 'containing', 'acrylamide', 'reacted', 'formaldehyde', 'form', 'methylol', 'groups', 'cross-linking', '243:', '615', 'acrylonitrile', 'methacrylonitrile', 'polymers', '379', 'o', 'n', 'nh2', 'o', 'h', 'h', 'n', 'o', 'n', 'h', 'ch2oh', 'the', 'product', 'reaction', 'thermoset', 'like', 'urea–formaldehyde', 'resin', '(see', 'chaps', '7', '9)', 'many', 'cross-linking', 'routes', 'described', 'patent', 'literature', 'many', 'different', 'functional', 'groups', 'available', '615', 'acrylonitrile', 'methacrylonitrile', 'polymers', 'polymers', 'acrylonitrile', 'used', 'synthetic', 'ﬁbers', 'elastomers', 'plastic', 'materials', 'the', 'monomer', 'formed', 'dehydration', 'ethylene', 'cyanohydrin', 'o', 'hcn', 'ho', 'n', '200-350', 'oc', 'catalyst', 'n', 'other', 'commercial', 'processes', 'exist', 'like', 'condensation', 'acetylene', 'hydrogen', 'cyanide', 'ammoxidation', 'propylene', 'nh3', '3', '2', 'o2', 'catalyst', 'n', 'acrylonitrile', 'polymerizes', 'readily', 'free-radical', 'mechanism', 'oxygen', 'acts', 'strong', 'inhibitor', 'when', 'polymerization', 'carried', 'bulk', 'reaction', 'autocatalytic', '242', '243.', 'in', 'solvents', 'like', 'dimethylformamide', 'however', 'rate', 'proportional', 'square', 'root', 'monomer', 'concentration', '242.', 'the', 'homopolymer', 'insoluble', 'monomer', 'many', 'solvents', 'acrylonitrile', 'polymerizes', 'also', 'anionic', 'mechanism', 'there', 'many', 'reports', 'literature', 'polymerizations', 'initiated', 'various', 'bases', 'these', 'alkali', 'metal', 'alkoxides', '246,', 'butyllithium', '247', '248,', 'metal', 'ketyls', '249', '250,', 'solutions', 'alkali', 'metals', 'ethers', '251', '252,', 'sodium', 'malonic', 'esters', '232,', 'others', 'the', 'propagation', 'reaction', 'quite', 'sensitive', 'termination', 'proton', 'donors', 'this', 'requires', 'use', 'aprotic', 'solvents', 'the', 'products', 'however', 'often', 'insoluble', 'solvents', 'in', 'addition', 'tendency', 'polymer', 'yellow', 'this', 'due', 'propagation', 'taking', 'place', '14', '34', 'insertion', 'addition', '12', 'placement', '253', '254:', 'n', '14', '34', 'n', 'n', 'n', '12', 'n', 'n', '380', '6', 'common', 'chain-growth', 'polymers', 'another', 'disadvantage', 'anionic', 'polymerization', 'acrylonitrile', 'formation', 'cyanoethylate', 'side', 'reaction', 'it', 'overcome', 'however', 'running', 'reaction', 'low', 'temperatures', 'an', 'example', 'polymerizations', 'initiated', 'kcn', '(cid3)50(cid1)c', 'dimethylformamide', '254,', 'butyllithium', 'toluene', '(cid3)78(cid1)c', '255.', 'both', 'polymerizations', 'yield', 'white', 'high', 'molecular', 'weight', 'products', 'free', 'cyanoethylation', 'it', 'suggested', 'terminations', 'anionic', 'polymerizations', 'acrylonitrile', 'proceed', 'proton', 'transfer', 'monomer', 'this', 'however', 'depends', 'upon', 'catalyst', 'concentrations', '256', '257.', 'at', 'low', 'concentrations', 'terminations', 'apparently', 'occur', 'cyclization', 'reaction', '257', 'instead', 'n', 'n', 'n', 'n', 'n', 'n', 'industrially', 'polyacrylonitrile', 'homopolymers', 'copolymers', 'prepared', 'mainly', 'free-radical', 'mechanism', 'the', 'reactions', 'often', 'conducted', 'low', 'temperatures', 'aqueous', 'systems', 'either', 'emulsions', 'suspensions', 'using', 'redox', 'initiation', 'colorless', 'high', 'molecular', 'weight', 'materials', 'form', 'bulk', 'polymerizations', 'difﬁcult', 'control', 'large', 'scale', 'over', 'half', 'polymer', 'prepared', 'industrially', 'use', 'textiles', 'most', 'copolymers', 'containing', '10', 'comonomer', 'the', 'comonomers', 'methyl', 'methacrylate', 'vinyl', 'acetate', '2-vinylpyridine', 'the', 'purpose', 'comonomers', 'make', 'ﬁbers', 'dyeable', 'polymerizations', 'solution', 'offer', 'advantage', 'direct', 'ﬁber', 'spinning', 'polyacrylonitrile', 'copolymers', 'also', 'used', 'barrier', 'resins', 'packaging', 'one', 'resin', 'contains', 'least', '70', 'acrylonitrile', 'often', 'methyl', 'acrylate', 'comonomer', 'the', 'material', 'poor', 'impact', 'resistance', 'one', 'industrial', 'process', 'copolymer', 'prepared', 'presence', '10', 'butadiene–acrylonitrile', 'rubber', 'emulsion', 'polymerization', 'the', 'product', 'contains', 'graft', 'copoly-', 'mer', 'polymer', 'blend', 'in', 'another', 'process', 'impact', 'resistance', 'copolymer', 'improved', 'biaxial', 'orientation', 'the', 'package', 'however', 'may', 'tendency', 'shrink', 'elevated', 'temperature', 'copolymer', 'crystallize', 'it', 'possible', 'form', 'clear', 'transparent', 'polyacrylonitrile', 'plastic', 'shapes', 'special', 'bulk', 'polymeri-', 'zation', 'technique', '258', '259.', 'the', 'reaction', 'initiated', 'p-toluenesulﬁnic', 'acid–hydrogen', 'peroxide', 'take', 'place', 'they', 'followed', 'spontaneous', 'initially', 'heterogeneous', 'polymerizations', 'transformations', 'high', 'conversion', 'homogeneous', 'transparent', 'polyacrylonitrile', 'plastics', '260.', 'a', 'major', 'condition', 'forming', 'transparent', 'solid', 'polymer', 'continuous', 'supply', 'monomer', 'ﬁll', 'gaps', 'formed', 'volume', 'contraction', 'polymerization', 'process', '261.', 'methacrylonitrile', 'ch2¼c(ch3)cn', 'also', 'prepared', 'several', 'routes', 'some', 'commercial', 'processes', 'based', 'acetone', 'cyanohydrin', 'intermediate', 'others', 'dehydrogenation', '(or', 'oxydehydrogenation)', 'isobutyronitrile', 'it', 'also', 'prepared', 'isobutylene', 'ammoxidation', 'nh3', 'o2', '3', '2', 'n', 'just', 'like', 'acrylonitrile', 'methacrylonitrile', 'polymerize', 'thermally', 'polymerizes', 'readily', 'presence', 'free-radical', 'initiators', 'unlike', 'polyacrylonitrile', 'polymethacrylonitrile', 'soluble', 'ketone', 'solvents', 'bulk', 'polymerizations', 'methacrylonitrile', 'disadvantage', 'long', 'reaction', 'time', 'the', 'rate', 'however', 'accelerates', 'temperature', 'the', 'polymer', 'soluble', 'monomer', 'ambient', 'conditions', '262.', '616', 'polyacrylamide', 'poly(acrylic', 'acid)', 'poly(methacrylic', 'acid)', '381', 'emulsion', 'polymerization', 'methacrylonitrile', 'convenient', 'way', 'form', 'high', 'molecular', 'weight', 'polymers', 'with', 'proper', 'choices', 'emulsiﬁers', 'rates', 'may', 'increased', 'increasing', 'numbers', 'particles', 'latexes', 'at', 'constant', 'degree', 'polymerization', 'methacrylonitrile', 'increases', 'rather', 'decreases', 'rate', 'polymerization', 'rises', '263.', 'initiation', 'rate', 'methacrylonitrile', 'polymerizes', 'readily', 'inert', 'solvents', 'the', 'polymer', 'depending', 'initiator', 'reaction', 'conditions', 'either', 'amorphous', 'crystalline', 'polymerizations', 'take', 'place', 'broad', 'range', 'temperatures', 'ambient', '(cid3)5(cid1)c', 'initiated', 'grignard', 'reagents', 'triphenyl', 'ethylsodium', 'sodium', 'liquid', 'ammonia', '264.', 'the', 'properties', 'polymers', 'essentially', 'polymers', 'formed', 'free-radical', 'mechanism', 'the', 'homopolymer', 'prepared', 'polymerization', 'liquid', 'ammonia', 'sodium', 'initiator', '(cid3)77(cid1)c', 'insoluble', 'acetone', 'soluble', 'dimethylformamide', '265.', 'when', 'formed', 'lithium', 'liquid', 'ammonia', '(cid3)75(cid1)c', 'molecular', 'weight', 'product', 'increases', 'monomer', 'concentration', 'decreases', 'initiator', 'concentration', 'if', 'however', 'potassium', 'initiates', 'reaction', 'rather', 'lithium', 'molecular', 'weight', 'independent', 'monomer', 'concentration', '266', '267.', 'polymethacrylonitrile', 'prepared', 'n-butyllithium', 'toluene', 'dioxane', 'crystalline', 'insoluble', 'solvents', 'like', 'acetone', '268.', 'when', 'polymerized', 'petroleum', 'ether', 'n-', 'butyllithium', 'methacrylonitrile', 'forms', 'living', 'polymer', '269.', 'highly', 'crystalline', 'polymethacry-', 'lonitrile', 'also', 'prepared', 'beryllium', 'magnesium', 'alkyls', 'toluene', 'wide', 'range', 'temperatures', '616', 'polyacrylamide', 'poly(acrylic', 'acid)', 'poly(methacrylic', 'acid)', 'commercially', 'acrylamide', 'formed', 'acrylonitrile', 'reaction', 'water', 'similarly', 'preferred', 'commercial', 'route', 'methacrylamide', 'methacrylonitrile', 'acrylamide', 'polymerizes', 'free-radical', 'mechanism', '270.', 'water', 'common', 'solvent', 'acrylamide', 'methacrylamide', 'polymerizations', 'polymers', 'precipitate', 'organic', 'solvents', 'crystalline', 'polyacrylamide', 'forms', 'metal', 'alkyls', 'hydrocarbon', 'solvents', 'anionic', 'mecha-', 'nism', '271.', 'the', 'product', 'insoluble', 'water', 'dimethylformamide', 'both', 'acrylic', 'methacrylic', 'acids', 'converted', 'anhydrides', 'acid', 'chlorides', 'the', 'acids', 'polymerize', 'aqueous', 'systems', 'free-radical', 'mechanism', 'polymerizations', 'monomers', 'nonpolar', 'solvents', 'like', 'benzene', 'result', 'precipitations', 'products', 'polymerizations', 'anhydrides', 'proceed', 'inter-', 'intramolecular', 'propagations', '272:', 'r', 'r', 'r', 'r', 'r', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'r', '¼', 'h', 'ch3', 'the', 'shown', 'cyclopolymerizations', 'produce', 'soluble', 'polymers', 'rather', 'gels', 'the', 'acid', 'chlorides', 'acrylic', 'methacrylic', 'acids', 'polymerize', 'free-radical', 'mechanism', 'dry', 'aromatic', 'aliphatic', 'solvents', 'molecular', 'weights', 'products', 'however', 'low', 'usually', '10000', '273', '274.', 'polyacrylic', 'polymethacrylic', 'acids', 'used', 'industrially', 'thickeners', 'cosmetics', 'ﬂocculating', 'agents', 'copolymerized', 'divinyl', 'benzene', 'ion-', 'exchange', 'resins', '382', '6', 'common', 'chain-growth', 'polymers', '617', 'halogen-bearing', 'polymers', 'the', 'volume', 'commercial', 'ﬂuorine-containing', 'polymers', 'large', 'compared', 'polymers', 'like', 'instance', 'poly(vinyl', 'chloride)', 'fluoropolymers', 'however', 'required', 'many', 'important', 'applications', 'the', 'main', 'monomers', 'tetraﬂuoroethylene', 'triﬂuorochloroethylene', 'vinyl', 'ﬂuoride', 'vinylidine', 'ﬂuoride', 'hexaﬂuoropropylene', '6171', 'polytetraﬂuoroethylene', 'this', 'monomer', 'prepared', 'chloroform', '275:', 'chcl3', '2hf', 'chcf2', '2hcl', '50', '-100', 'oc', '39', 'atm', 'pressure', '2', 'chcf2', '70', 'oc', 'f', 'f', 'f', 'f', 'hcl', 'tetraﬂuoroethylene', 'boils', '(cid3)763(cid1)c', 'it', 'product', 'pyrolytic', 'reaction', 'diﬂuorochloromethane', 'other', 'ﬂuorine', 'by-products', 'form', 'well', 'monomer', 'must', 'isolated', 'the', 'monomer', 'polymerizes', 'water', 'moderate', 'pressures', 'free-radical', 'mechanism', 'various', 'initiators', 'appear', 'effective', '276.', 'redox', 'initiation', 'preferred', 'the', 'polymerization', 'reaction', 'strongly', 'exothermic', 'water', 'helps', 'dissipate', 'high', 'heat', 'reaction', 'a', 'runaway', 'uncontrolled', 'polymer-', 'ization', 'lead', 'explosive', 'decomposition', 'monomer', 'carbon', 'carbon', 'tetraﬂuoride', '277:', 'f', 'f', 'f', 'f', 'c', 'cf4', 'polytetraﬂuoroethylene', 'linear', 'highly', 'crystalline', '278.', 'absence', 'terminal', 'cf2', 'cf--', 'groups', 'shows', 'polymerization', 'terminations', 'occur', 'disproportionation', 'probably', 'take', 'place', 'combination', '279.', 'the', 'molecular', 'weights', 'commercially', 'available', 'polymers', 'range', '39000', '9000000', 'polytetraﬂuoroethylene', 'inert', 'many', 'chemical', 'attacks', 'swollen', 'ﬂuorocarbon', 'oils', 'temperatures', '300(cid1)c', 'the', 'tm', 'polymer', '327(cid1)c', 'tg', '(cid3)100(cid1)c', 'the', 'physical', 'properties', 'polytetraﬂuoroethylene', 'depend', 'upon', 'crystallinity', 'molecular', 'weight', 'polymer', 'two', 'crystalline', 'forms', 'known', 'in', 'cases', 'chains', 'assume', 'helical', 'arrangements', 'ﬁt', 'crystalloids', 'one', 'arrangement', '15', 'cf2', 'groups', 'per', 'turn', '13', 'polytetraﬂuoroethylene', 'ﬂow', 'even', 'melting', 'point', 'this', 'attributed', 'restricted', 'rotation', 'around', 'c–c', 'bonds', 'high', 'molecular', 'weights', 'the', 'stiffness', 'solid', 'polymer', 'also', 'attributed', 'restricted', 'rotation', 'the', 'polymer', 'exhibits', 'high', 'thermal', 'stability', 'retains', 'physical', 'properties', 'wide', 'range', 'temperatures', 'the', 'loss', 'strength', 'occurs', 'crystalline', 'melting', 'point', 'it', 'possible', 'use', 'material', 'long', 'periods', '300(cid1)c', 'without', 'signiﬁcant', 'loss', 'strength', '617', 'halogen-bearing', 'polymers', '383', '6172', 'polychlorotriﬂuoroethylene', 'the', 'monomer', 'prepared', 'dechlorination', 'trichlorotriﬂuoroethane', 'zinc', 'dust', 'ethanol', 'cl', 'f', 'cl', 'f', 'f', 'cl', 'zn', 'c2h5oh', 'f', 'cl', 'f', 'f', 'zncl2', 'it', 'toxic', 'gas', 'boils', '(cid3)268(cid1)c', 'polymerization', 'chlorotriﬂuoroethylene', 'usually', 'carried', 'commercially', 'free-radical', 'suspension', 'polymerization', 'reaction', 'temperatures', 'kept', '0', '40(cid1)c', 'obtain', 'high', 'molecular', 'weight', 'product', 'a', 'redox', 'initiation', 'based', 'reactions', 'persulfate', 'bisulﬁte', 'ferrous', 'ions', 'often', 'used', 'commercial', 'polymers', 'range', 'molecular', 'weights', '50000', '500000', 'polychlorotriﬂuoroethylene', 'exhibits', 'greater', 'strength', 'hardness', 'creep', 'resistance', 'polytetraﬂuoroethylene', 'due', 'presence', 'chlorine', 'atoms', 'chains', 'however', 'packing', 'cannot', 'tight', 'polytetraﬂuoroethylene', 'melts', 'lower', 'temperature', 'the', 'melting', 'point', '214(cid1)c', 'the', 'degree', 'crystallinity', 'varies', '30', '85,', 'depending', 'upon', 'thermal', 'history', 'polymer', 'polytriﬂuorochloroethylene', 'soluble', 'certain', 'chloro', 'ﬂuoro', 'compounds', '100(cid1)c', 'it', 'ﬂows', 'melting', 'point', 'the', 'chemical', 'resistance', 'material', 'good', 'inferior', 'polytetraﬂuoroethylene', '6173', 'poly(vinylidine', 'ﬂuoride)', 'the', 'monomer', 'prepared', 'dehydrochlorination', '111-chlorodiﬂuoroethane', 'cl', 'f', 'f', 'f', 'h', 'hcl', 'dechlorination', '12-dichloro-11-diﬂuoroethane', '280:', 'cl', 'f', 'f', 'f', 'f', 'f', 'vinylidine', 'ﬂuoride', 'boils', '(cid3)84(cid1)c', 'the', 'monomer', 'polymerized', 'aqueous', 'systems', 'pressure', 'details', 'process', 'however', 'kept', 'trade', 'secrets', 'two', 'different', 'molecular', 'weight', 'materials', 'available', 'commercially', '300000', '6000000', 'poly(vinylidine', 'ﬂuoride)', 'crystalline', 'melts', '171(cid1)c', 'the', 'material', 'exhibits', 'fair', 'resistance', 'solvents', 'chemicals', 'inferior', 'polytetraﬂuoroethylene', 'polytriﬂuorochloroethylene', '6174', 'poly(vinyl', 'ﬂuoride)', 'vinyl', 'ﬂuoride', 'monomer', 'prepared', 'addition', 'hf', 'acetylene', 'the', 'monomer', 'gas', 'room', 'temperature', 'boils', '(cid3)722(cid1)c', 'commercially', 'vinyl', 'ﬂuoride', 'polymerized', 'aqueous', 'medium', 'using', 'either', 'redox', 'initiation', 'one', 'thermal', 'decomposition', 'peroxides', 'pressures', '384', '6', 'common', 'chain-growth', 'polymers', '1000', 'atm', 'may', 'used', 'radicals', 'generated', 'temperatures', '50', '100(cid1)c', 'yield', 'high', 'molecular', 'weight', 'polymers', 'poly(vinyl', 'ﬂuoride)', 'moderately', 'crystalline', 'the', 'crystal', 'melting', 'point', 'tm', 'approximately', '200(cid1)c', 'the', 'high', 'molecular', 'weight', 'polymers', 'dissolve', 'dimethylformamide', 'tetramethyl', 'urea', 'temperatures', '100(cid1)c', 'the', 'polymer', 'resistant', 'hydrolytic', 'attack', 'it', 'however', 'loose', 'hf', 'elevated', 'temperatures', '6175', 'copolymers', 'fluorooleﬁns', 'mary', 'different', 'copolymers', 'ﬂuorooleﬁns', 'possible', 'reported', 'literature', 'commer-', 'cial', 'use', 'ﬂuorooleﬁn', 'copolymers', 'however', 'restricted', 'mainly', 'elastomers', 'such', 'materials', 'offer', 'superior', 'solvent', 'resistance', 'good', 'thermal', 'stability', 'the', 'elastomers', 'important', 'industrially', 'vinylidine', 'ﬂuoride–chlorotriﬂuoroethylene', '260', 'vinylidine', 'ﬂuoride–hexaﬂuoropropylene', 'copolymers', '282.', 'these', 'copolymers', 'amor-', 'phous', 'due', 'irregularities', 'structures', 'range', 'properties', 'resinous', 'elastomeric', 'depending', 'upon', 'composition', '283.', 'those', 'contain', '50–70', 'mole', 'percent', 'vinylidine', 'ﬂuoride', 'elastomers', 'the', 'tg', 'ranges', '0', '(cid3)15(cid1)c', 'also', 'depending', 'upon', 'vinylidine', 'ﬂuoride', 'content', '284.', 'they', 'may', 'cross-linked', 'various', 'peroxides', 'polyamines', '284,', 'ionizing', 'radiation', 'the', 'cross-', 'linking', 'reactions', 'peroxides', 'take', 'place', 'hydrogen', 'abstraction', 'primary', 'radicals', 'r', 'n', 'f', 'f', 'f', 'f', '2', 'f', 'f', 'n', 'rh', 'f', 'f', 'n', 'f', 'f', 'copolymers', 'vinylidine', 'ﬂuoride', 'hexaﬂuoropropylene', 'prepared', 'aqueous', 'dispersions', 'using', 'persulfate', 'initiators', 'hexaﬂuoropropylene', 'homopolymerize', 'copolymerize', 'this', 'means', 'content', 'copolymer', 'cannot', 'exceed', '50.', 'preferred', 'compositions', 'appear', 'contain', '80', 'vinylidine', 'ﬂuoride', 'the', 'cross-linking', 'reactions', 'diamines', 'completely', 'understood', 'it', 'believed', 'reaction', 'takes', 'place', 'two', 'steps', '285', '286.', 'in', 'ﬁrst', 'one', 'dehydroﬂuorination', 'occurs', 'f', 'f', 'f', 'f', 'cf3', 'f', 'f', 'f', 'n', '-hf', 'cf3', 'f', 'f', 'f', 'f', 'f', 'f', 'n', 'the', 'elimination', 'catalyzed', 'basic', 'materials', 'these', 'may', 'form', 'mgo', 'often', 'included', 'reaction', 'medium', 'in', 'second', 'step', 'amine', 'groups', 'add', 'across', 'double', 'bonds', '617', 'halogen-bearing', 'polymers', '385', '2', 'cf3', 'f', 'f', 'f', 'f', 'f', 'f', 'n', 'h2n', 'nh2', 'r', 'cf3', 'f', 'f', 'f', 'hn', 'n', 'f', 'nh', 'cf3', 'r', 'f', 'f', 'f', 'f', 'n', 'free', 'diamines', 'used', 'cross-linking', 'reactive', 'cause', 'premature', 'gelation', 'it', 'common', 'practice', 'therefore', 'add', 'diamine', 'compounds', 'form', 'carbamates', 'like', 'ethylenediamine', 'carbamate', 'hexamethylene', 'diamine', 'carbamate', 'the', 'ﬂuoro', 'elastomers', 'exhibit', 'good', 'resistance', 'chemicals', 'maintain', 'useful', 'properties', '(cid3)50', '300(cid1)c', 'copolymers', 'tetraﬂuoroethylene', 'hexaﬂuoropropylene', 'truly', 'thermoplastic', 'polyperﬂuor-', 'ooleﬁns', 'fabricated', 'common', 'techniques', 'such', 'copolymers', 'soften', '285(cid1)c', 'continuous', 'use', 'temperature', '(cid3)260', '205(cid1)c', 'their', 'properties', 'similar', 'though', 'somewhat', 'inferior', 'polytetraﬂuoroethylene', '6176', 'miscellaneous', 'fluorine', 'containing', 'chain-growth', 'polymers', 'one', 'miscellaneous', 'ﬂuorooleﬁn', 'polymers', 'copolymer', 'triﬂuoronitrosomethane', 'tetraﬂuoroethylene', '287,', 'elastomer', 'it', 'formed', 'suspension', 'polymerization', 'one', 'procedure', 'carry', 'reaction', 'aqueous', 'solution', 'lithium', 'bromide', '(cid3)25(cid1)c', 'magnesium', 'carbonate', 'suspending', 'agent', 'no', 'initiator', 'added', 'reaction', 'takes', '20', 'h', 'because', 'reaction', 'inhibited', 'hydroquinone', 'accelerated', 'ultra-violet', 'light', 'believed', 'take', 'place', 'free-radical', 'mechanism', 'whether', 'chain-growth', 'polymerization', 'however', 'certain', 'a', '11', 'copolymer', 'always', 'formed', 'regardless', 'composition', 'monomer', 'feed', 'copolymerization', 'takes', 'place', 'low', 'temperatures', 'at', 'elevated', 'temperatures', 'however', 'cyclic', 'oxazetidines', 'form', 'instead', 'f', 'f', 'n', 'o', 'n', 'cf3', 'f', 'f', 'cf3', 'on', 'f2c', 'cf2', '386', '6', 'common', 'chain-growth', 'polymers', 'two', 'polyﬂuoroacrylates', 'manufactured', 'small', 'commercial', 'scale', 'special', 'uses', 'jet', 'engines', 'these', 'poly(11-dihyroperﬂuorobutyl', 'acrylate)', 'poly(3-perﬂuoromethoxy-11-dihydroperﬂuoropropyl', 'acrylate)', 'n', 'o', 'f', 'f', 'f', 'f', 'o', 'cf3', 'n', 'o', 'o', 'f', 'f', 'f', 'f', 'cf3', 'o', 'f', 'n', 'f', 'f', 'the', 'polymers', 'prepared', 'emulsion', 'polymerization', 'persulfate', 'initiators', 'although', 'many', 'ﬂuorine', 'containing', 'polymers', 'described', 'literature', 'possible', 'describe', 'they', 'utilized', 'commercially', 'large', 'scale', 'a', 'however', 'mentioned', 'examples', 'one', 'polyﬂuoroprene', '288:', 'the', 'polymer', 'formed', 'free-radical', 'mechanism', 'emulsion', 'polymerization', 'using', 'redox', 'initiation', 'all', 'three', 'possible', 'placements', 'monomer', 'occur', '267.', 'polyﬂuorostyrenes', 'described', 'many', 'publications', 'a', 'b-ﬂuorostyrene', 'formed', 'cationic', 'mechanism', '289.', 'the', 'material', 'softens', '240–260(cid1)c', 'an', 'abb-triﬂuorostyrene', 'polymerized', 'free-radical', 'mechanism', 'yield', 'amorphous', 'polymer', 'softens', '240(cid1)c', '290.', 'ring-substituted', 'styrenes', 'apparently', 'polymerize', 'similarly', 'styrene', 'isotactic', 'poly(o-ﬂuorostyrene)', 'melts', '265(cid1)c', 'it', 'forms', 'polymerization', 'ziegler–natta', 'catalysts', '291.', 'the', 'meta', 'analog', 'however', 'polymerized', 'conditions', 'yields', 'amorphous', 'material', '291.', '6177', 'poly(vinyl', 'chloride)', 'poly(vinyl', 'chloride)', 'used', 'industry', 'large', 'scale', 'many', 'applications', 'rigid', 'plastics', 'plastisols', 'surface', 'coatings', 'the', 'monomer', 'vinyl', 'chloride', 'prepared', 'acetylene', 'hcl', 'cl', '617', 'halogen-bearing', 'polymers', '387', 'the', 'reaction', 'exothermic', 'requires', 'cooling', 'maintain', 'temperature', '100', '108(cid1)c', 'the', 'monomer', 'also', 'prepared', 'ethylene', 'cl2', '30', '-', '50', 'oc', 'cl', 'cl', '500', 'oc', 'kaolin', 'cl', 'hcl', 'the', 'reaction', 'dehydrochlorination', 'carried', 'elevated', 'pressure', '3', 'atm', 'free-radical', 'polymerization', 'vinyl', 'chloride', 'studied', 'extensively', 'for', 'reactions', 'carried', 'bulk', 'following', 'observations', 'made', '292:', '1', 'the', 'polymer', 'insoluble', 'monomer', 'precipitates', 'polymerization', '2', 'the', 'polymerization', 'rate', 'accelerates', 'start', 'reaction', 'vinyl', 'chloride', 'relatively', 'unreactive', 'monomer', 'the', 'main', 'sites', 'initiation', 'occur', 'continuous', 'monomer', 'phase', '3', 'the', 'molecular', 'weight', 'product', 'depend', 'upon', 'conversion', 'depend', 'upon', 'concentration', 'initiator', '4', 'the', 'molecular', 'weight', 'polymer', 'increases', 'temperature', 'polymerization', 'decreases', 'the', 'maximum', 'relationship', 'however', '30(cid1)c', 'there', 'autoacceleration', 'bulk', 'polymerization', 'rate', 'vinyl', 'chloride', '293.', 'it', 'suggested', 'schindler', 'breitenbach', '294', 'acceleration', 'due', 'trapped', 'radicals', 'present', 'precipitated', 'polymer', 'swollen', 'monomer', 'molecules', 'this', 'inﬂuences', 'rate', 'termination', 'decreases', 'progressively', 'extent', 'reaction', 'propagation', 'rate', 'remains', 'constant', 'the', 'autocatalytic', 'effect', 'vinyl', 'chloride', 'bulk', 'polymerizations', 'however', 'depends', 'type', 'initiator', 'used', '295.', 'thus', '2', '20-azobisisobutyronitrile', 'initiates', 'polymerization', 'autocat-', 'alytic', 'effect', 'observed', '80', 'conversion', 'yet', 'benzoyl', 'peroxide', 'initiates', 'reaction', 'occurs', '20–30', 'conversion', 'when', 'vinyl', 'chloride', 'polymerized', 'solution', 'autoacceleration', 'also', 'major', 'feature', 'vinyl', 'chloride', 'free-radical', 'polymerization', 'chain', 'transferring', 'monomer', '296.', 'this', 'supported', 'experimental', 'evidence', '297', '298.', 'in', 'addition', 'growing', 'radical', 'chains', 'terminate', 'chain', 'transferring', '“dead”', 'polymer', 'molecules', 'the', 'propagations', 'proceed', 'polymer', 'backbone', '297.', 'such', 'new', 'growth', 'radicals', 'however', 'probably', 'short', 'lived', 'destroyed', 'transfer', 'monomer', '299.', 'the', '13c', 'nmr', 'spectroscopy', 'poly(vinyl', 'chloride)', 'reduced', 'tributyltin', 'hydride', 'original', 'polymer', 'contained', 'number', 'short', 'four-carbon', 'branches', '300.', 'showed', 'this', 'however', 'may', 'typical', 'poly(vinyl', 'chloride)', 'polymers', 'formed', 'free-radical', 'polymerization', 'it', 'conﬂicts', 'evidence', '13c', 'nmr', 'spectroscopy', 'chloromethyl', 'groups', 'principal', 'short', 'chain', 'branches', 'poly(vinyl', 'chloride)', '301', '302.', 'the', 'pendant', 'chloromethyl', 'groups', 'found', 'occur', 'frequency', '2–31000', 'carbons', 'the', 'formation', 'branches', 'seen', 'bovey', 'coworkers', 'depends', 'upon', 'head', 'head', 'additions', 'monomers', 'polymer', 'formation', 'such', 'additions', 'followed', '12', 'chlorine', 'shifts', 'subsequent', 'propagations', '301', '302.', 'evidence', 'still', 'studies', 'also', 'shows', 'head', 'head', 'placement', 'occurs', 'growth', 'reaction', '303.', 'it', 'suggested', 'may', '388', '6', 'common', 'chain-growth', 'polymers', 'essential', 'step', 'formation', 'branches', 'also', 'one', 'leading', 'formation', 'unsaturation', 'chain', 'ends', '303', '304:', 'cl', 'cl', 'cl', 'cl', 'ch2cl', 'ch2cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'h', 'poly(vinyl', 'chloride)', 'prepared', 'boron', 'alkyl', 'catalysts', 'low', 'temperatures', 'possesses', 'higher', 'amounts', 'syndiotactic', 'placement', 'essentially', 'free', 'branches', '305–307.', 'many', 'attempts', 'made', 'polymerize', 'vinyl', 'chloride', 'ionic', 'mechanisms', 'using', 'different', 'organometallic', 'compounds', 'combinations', 'metal', 'salts', '308–312.', 'attempts', 'also', 'made', 'polymerize', 'vinyl', 'chloride', 'ziegler–natta', 'catalysts', 'complexed', 'lewis', 'bases', 'to', 'date', 'however', 'established', 'unequivocally', 'vinyl', 'chloride', 'polymerize', 'ionic', 'mechanism', 'use', 'catalysts', 'yield', 'polymers', 'higher', 'crystallinity', 'these', 'reactions', 'however', 'carried', 'low', 'temperatures', 'greater', 'amount', 'syndiotactic', 'placement', 'occurs', 'free-radical', 'mechanism', '313.', 'vinyl', 'chloride', 'also', 'polymerized', 'alcl(c2h5)o2h5', 'vo(c3h7o2)', 'without', 'lewis', 'bases', '312.', 'here', 'however', 'evidence', 'indicates', 'free-radical', 'mechanism', 'on', 'hand', 'butyllithium–aluminum', 'alkyl-initiated', 'polymerizations', 'vinyl', 'chloride', 'unaffected', 'free-radical', 'inhibitors', '313.', 'also', 'molecular', 'weights', 'resultant', 'polymers', 'unaffected', 'additions', 'ccl4', 'acts', 'chain', 'transferring', 'agent', 'free-radical', 'polymerizations', 'this', 'suggests', 'ionic', 'mechanism', 'chain', 'growth', 'furthermore', 'reactivity', 'ratios', 'copolymeri-', 'zation', 'reactions', 'catalytic', 'system', 'differ', 'typical', 'free-radical', 'polymerizations', '313.', 'an', 'anionic', 'mechanism', 'also', 'postulated', 'polymerization', 'vinyl', 'chloride', 't-butylmagnesium', 'tetrahydrofuran', '314.', 'commercially', 'far', 'biggest', 'amount', 'poly(vinyl', 'chloride)', 'homopolymer', 'produced', 'suspension', 'polymerization', 'lesser', 'extent', 'emulsion', 'bulk', 'polymerization', 'very', 'little', 'polymer', 'formed', 'solution', 'polymerization', 'one', 'process', 'bulk', 'polymerization', 'vinyl', 'chloride', 'developed', 'france', 'initiator', 'monomer', 'heated', '60(cid1)c', 'approximately', '12', 'h', 'inside', 'rotating', 'drum', 'containing', 'stainless', 'steel', 'balls', 'typical', 'initiators', 'reaction', 'benzoyl', 'peroxide', 'azobisisobutyronitrile', 'the', 'speed', 'rotation', 'drum', 'controls', 'particle', 'size', 'ﬁnal', 'product', 'the', 'process', 'also', 'carried', 'two-reactor', 'arrangement', 'in', 'ﬁrst', 'one', 'approximately', '10', 'monomer', 'converted', 'the', 'material', 'transferred', 'second', 'reactor', 'polymerization', 'continued', 'reaches', '75–80', 'conversion', 'special', 'ribbon', 'blenders', 'present', 'second', 'reactor', 'control', 'operation', 'second', 'reactor', 'quite', 'critical', '315.', 'industrial', 'suspension', 'polymerizations', 'vinyl', 'chloride', 'often', 'carried', 'large', 'batch', 'reactors', 'stirred', 'jacketed', 'autoclaves', 'continuous', 'reactors', 'however', 'introduced', 'several', 'manufacturing', 'facilities', '315.', 'typical', 'recipes', 'call', '100', 'parts', 'vinyl', 'chloride', '180', 'parts', 'water', 'suspending', 'agent', 'like', 'maleic', 'acid–vinyl', 'acetate', 'copolymer', 'chain', 'transferring', 'agent', 'monomer', 'soluble', 'initiator', 'the', 'reaction', 'may', 'carried', '100', 'lbin2', 'pressure', '50(cid1)c', 'approximately', '15', 'h', 'as', 'monomer', 'consumed', 'pressure', 'drops', 'the', 'reaction', 'stopped', '617', 'halogen-bearing', 'polymers', '389', 'internal', 'pressure', '10', 'lbin2', 'remaining', 'monomer', '(about', '10)', 'drawn', 'recycled', 'the', 'product', 'discharged', 'emulsion', 'polymerizations', 'vinyl', 'chloride', 'usually', 'conducted', 'redox', 'initiation', 'such', 'reactions', 'rapid', 'carried', '20(cid1)c', '1–2', 'h', 'high', 'degree', 'conversion', 'commercial', 'poly(vinyl', 'chloride)s', 'range', 'molecular', 'weights', '40000', '80000', 'the', 'polymers', 'mostly', 'amorphous', 'small', 'amounts', 'crystallinity', '5.', 'the', 'crystalline', 'areas', 'syndiotactic', '317', '318.', 'poly(vinyl', 'chloride)', 'soluble', 'room', 'temperature', 'oxygen-containing', 'solvents', 'ketones', 'esters', 'ethers', 'others', 'it', 'also', 'soluble', 'chlorinated', 'solvents', 'the', 'polymer', 'however', 'soluble', 'aliphatic', 'aromatic', 'hydrocarbons', 'it', 'unaffected', 'acid', 'alkali', 'solutions', 'poor', 'heat', 'light', 'stability', 'poly(vinyl', 'chloride)', 'degrades', 'temperatures', '70(cid1)c', 'higher', 'exposed', 'sun', 'light', 'unless', 'stabilized', 'heating', 'changes', 'material', 'colorless', 'yellow', 'orange', 'brown', 'ﬁnally', 'black', 'many', 'compounds', 'tend', 'stabilize', 'poly(vinyl', 'chloride)', 'the', 'important', 'ones', 'include', 'lead', 'compounds', 'like', 'dibasic', 'lead', 'phthalate', 'lead', 'carbonate', 'also', 'effective', 'metal', 'salts', 'like', 'barium', 'calcium', 'zinc', 'octoates', 'stearates', 'laurates', 'organotin', 'compounds', 'like', 'dibutyl', 'tin', 'maleate', 'laurate', 'also', 'belong', 'list', 'epoxidized', 'drying', 'oils', 'effective', 'heat', 'stabilizers', 'particularly', 'coatings', 'based', 'poly(vinyl', 'chloride)', 'some', 'coating', 'materials', 'may', 'also', 'include', 'aminoplast', 'resins', 'like', 'benzoguanamine–formaldehyde', 'condensate', 'the', 'process', 'degradation', 'complex', 'it', 'involves', 'loss', 'hydrochloric', 'acid', 'the', 'reactions', 'free', 'radical', 'nature', 'though', 'ionic', 'reactions', 'appear', 'take', 'place', 'well', 'the', 'process', 'dehydro-', 'chlorination', 'results', 'formations', 'long', 'sequences', 'conjugated', 'double', 'bonds', 'it', 'commonly', 'believed', 'formation', 'conjugated', 'polyenes', 'chromophores', 'responsible', 'darkening', 'poly(vinyl', 'chloride)', 'in', 'addition', 'polymer', 'degrades', 'faster', 'open', 'air', 'inert', 'atmosphere', 'this', 'shows', 'oxidation', 'contributes', 'degradation', 'process', 'all', 'effective', 'stabilizers', 'hydrochloric', 'acid', 'scavengers', 'this', 'feature', 'alone', 'however', 'probably', 'account', 'stabilization', 'process', 'there', 'must', 'interaction', 'stabilizers', 'polymers', 'such', 'interaction', 'might', 'vary', 'depending', 'upon', 'particular', 'stabilizer', '61771', 'copolymers', 'vinyl', 'chloride', 'a', 'common', 'copolymer', 'vinyl', 'chloride', 'vinyl', 'acetate', 'copolymerization', 'vinyl', 'acetate', 'improves', 'stability', 'molding', 'characteristics', 'the', 'copolymers', 'also', 'used', 'ﬁbers', 'coatings', 'copolymers', 'intended', 'use', 'moldings', 'usually', 'prepared', 'suspension', 'polymeriza-', 'tion', 'those', 'intended', 'coating', 'purposes', 'prepared', 'solution', 'emulsion', 'suspension', 'polymerizations', 'the', 'copolymers', 'used', 'molding', 'typically', 'contain', '10', 'poly(vinyl', 'acetate)', 'copolymers', 'prepared', 'coating', 'purposes', 'contain', '10', '17', 'poly(vinyl', 'acetate)', 'for', 'coatings', 'third', 'comonomer', 'may', 'included', 'resins', 'this', 'third', 'component', 'may', 'instance', 'maleic', 'anhydride', 'small', 'quantities', 'like', '1,', 'improve', 'adhesion', 'surfaces', 'copolymers', 'vinyl', 'chloride', 'vinylidine', 'chloride', 'similar', 'properties', 'copolymers', 'vinyl', 'acetate', 'they', 'contain', '5', '12', 'poly(vinylidine', 'chloride)', 'intended', 'use', 'stabilized', 'calendaring', 'copolymers', 'containing', '60', 'vinyl', 'chloride', '40', 'acrylonitrile', 'used', 'ﬁbers', 'the', 'ﬁbers', 'spun', 'acetone', 'solution', 'they', 'nonﬂammable', 'good', 'chemical', 'resistance', '6178', 'poly(vinylidine', 'chloride)', 'vinylidine', 'chloride', 'homopolymers', 'form', 'readily', 'free-radical', 'polymerization', 'lack', 'sufﬁcient', 'thermal', 'stability', 'commercial', 'use', 'copolymers', 'however', 'small', 'amounts', 'comonomers', 'ﬁnd', 'many', 'applications', '390', '6', 'common', 'chain-growth', 'polymers', 'the', 'monomer', '112-trichloroethylene', 'vinylidine', 'chloride', 'prepared', 'dehydrochlorination', 'it', 'colorless', 'liquid', 'boils', '32(cid1)c', 'also', 'rather', 'hard', 'handle', 'polymerizes', 'standing', 'this', 'takes', 'place', 'upon', 'exposure', 'air', 'water', 'light', 'storage', 'inert', 'atmosphere', 'completely', 'prevent', 'polymer', 'formation', 'poly(vinylidine', 'chloride)', 'formed', 'bulk', 'solution', 'suspension', 'emulsion', 'polymerization', 'processes', 'the', 'products', 'highly', 'crystalline', 'regular', 'structures', 'melting', 'point', '220(cid1)c', 'the', 'structure', 'illustrated', 'follows', 'cl', 'cl', 'cl', '-hcl', '400', 'oc', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'this', 'regularity', 'structure', 'probably', 'due', 'little', 'chain', 'transferring', 'polymer', 'backbone', 'polymerization', 'such', 'regularity', 'structure', 'allows', 'close', 'packing', 'chains', 'result', 'effective', 'solvents', 'polymer', 'room', 'temperature', 'copolymerization', 'vinylidine', 'chloride', 'vinyl', 'chloride', 'reduces', 'regularity', 'structure', 'it', 'increases', 'ﬂexibility', 'allows', 'processing', 'polymer', 'reasonable', 'temperatures', 'due', 'extensive', 'crystallization', 'however', 'still', 'present', '8515', 'copolymers', 'vinylidine', 'chloride', 'vinyl', 'chloride', 'melt', '170(cid1)c', 'the', 'copolymerization', 'reactions', 'proceed', 'slower', 'rates', 'homopolymerizations', 'either', 'one', 'monomers', 'alone', 'higher', 'initiator', 'levels', 'temperatures', 'therefore', 'used', 'the', 'molecular', 'weights', 'products', 'range', '20000', '50000', 'these', 'materials', 'good', 'barriers', 'gases', 'moisture', 'this', 'makes', 'useful', 'ﬁlms', 'food', 'packaging', 'such', 'ﬁlms', 'formed', 'extrusion', 'biaxial', 'orientation', 'the', 'main', 'application', 'however', 'ﬁlaments', 'these', 'prepared', 'extrusion', 'drawing', 'the', 'tensile', 'strength', 'unoriented', 'material', '10000', 'lbin2', 'oriented', 'one', '30000', 'lbin2', 'vinylidine', 'chloride', 'also', 'copolymerized', 'acrylonitrile', 'this', 'copolymer', 'used', 'mainly', 'barrier', 'coating', 'paper', 'polyethylene', 'cellophane', 'it', 'advantage', 'heat', 'sealable', '618', 'poly(vinyl', 'acetate)', 'vinyl', 'acetate', 'monomer', 'prepared', 'reacting', 'acetylene', 'acetic', 'acid', 'o', 'oh', 'o', 'o', 'the', 'reaction', 'carried', 'liquid', 'vapor', 'phase', 'a', 'liquid', 'phase', 'reaction', 'requires', '75–80(cid1)c', 'temperatures', 'mercuric', 'sulfate', 'catalyst', 'the', 'acetylene', 'gas', 'bubbled', 'glacial', 'acetic', 'acid', 'acetic', 'anhydride', 'vapor', 'phase', 'reactions', 'carried', '210–250(cid1)c', '619', 'poly(vinyl', 'alcohol)', 'poly(vinyl', 'acetal)s', '391', 'typical', 'catalysts', 'cadmium', 'acetate', 'zinc', 'acetate', 'there', 'routes', 'vinyl', 'acetate', 'well', 'based', 'ethylene', 'commercially', 'poly(vinyl', 'acetate)', 'formed', 'bulk', 'solution', 'emulsion', 'suspension', 'polymerizations', 'free-radical', 'mechanism', 'in', 'polymerizations', 'chain', 'transferring', 'polymer', 'may', 'high', '30.', 'the', 'transfer', 'polymer', 'backbone', 'abstraction', 'tertiary', 'hydrogen', 'r', 'rh', 'it', 'also', 'take', 'place', 'methyl', 'proton', 'acetate', 'group', 'r', 'rh', 'o', 'n', 'o', 'n', 'o', 'o', 'n', 'o', 'o', 'n', 'o', 'o', 'the', 'polymer', 'head', 'tail', 'structure', 'highly', 'branched', 'it', 'quite', 'brittle', 'exhibits', 'cold', 'ﬂow', 'this', 'makes', 'useless', 'structural', 'plastic', 'it', 'however', 'quite', 'useful', 'coating', 'material', 'adhesive', 'wood', 'the', 'polymer', 'soluble', 'wide', 'range', 'solvents', 'swells', 'softens', 'upon', 'prolonged', 'immersion', 'water', 'at', 'higher', 'temperatures', 'extended', 'exposures', 'temperatures', '70(cid1)c', 'material', 'loses', 'acetic', 'acid', 'a', 'number', 'copolymers', 'known', 'vinyl', 'acetate', 'major', 'component', 'in', 'coatings', 'vinyl', 'acetate', 'often', 'used', 'copolymers', 'alkyl', 'acrylates', '(line', '2-ethylhexyl', 'acrylate)', 'esters', 'maleic', 'fumaric', 'acids', 'such', 'copolymers', 'typically', 'contain', '50–20', 'weight', 'comonomer', 'usually', 'formed', 'emulsion', 'polymerization', 'batch', 'processes', 'they', 'used', 'extensively', 'vehicles', 'emulsion', 'paints', 'shaver', 'coworkers', '319', 'investigated', 'mechanism', 'bis(imino)pyridine', 'ligand', 'framework', 'transition', 'metal', 'systems-mediated', 'polymerization', 'vinyl', 'acetate', 'initiation', 'using', 'azobisisobu-', 'tyronitrile', '120(cid1)c', 'results', 'excellent', 'control', 'poly(vinyl', 'acetate)', 'molecular', 'weights', 'polymer', 'dispersities', 'the', 'reaction', 'yields', 'vanadium-terminated', 'polymer', 'chains', 'readily', 'converted', 'proton-terminated', 'poly(vinyl', 'acetate)', 'poly(vinyl', 'alcohol)', 'irreversible', 'halogen', 'transfer', 'parent', 'complex', 'radical', 'derived', 'azobisisobutyronitrile', 'generates', 'active', 'species', '619', 'poly(vinyl', 'alcohol)', 'poly(vinyl', 'acetal)s', 'vinyl', 'alcohol', 'monomer', 'exist', 'keto', 'tautomer', 'much', 'stable', 'poly(vinyl', 'alcohol)', 'prepared', 'either', 'poly(vinyl', 'esters)', 'poly(vinyl', 'ethers)', 'commercially', 'however', 'prepared', 'exclusively', 'poly(vinyl', 'acetate)', 'the', 'preferred', 'procedure', '392', '6', 'common', 'chain-growth', 'polymers', 'transesteriﬁcation', 'reaction', 'using', 'methyl', 'ethyl', 'alcohols', 'alkaline', 'catalysts', 'yield', 'rapid', 'alcoholyses', 'a', 'typical', 'reaction', 'employs', '1', 'sodium', 'methoxide', 'carried', 'completion', '1', 'h', '60(cid1)c', 'the', 'product', 'contaminated', 'sodium', 'acetate', 'must', 'removed', 'the', 'reaction', 'transesteriﬁcation', 'illustrated', 'follows', 'ch3o', 'o', 'n', 'o', 'n', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'the', 'branches', 'poly(vinyl', 'acetate)', 'form', 'polymerization', 'result', 'chain', 'transfer-', 'ring', 'acetate', 'groups', 'cleave', 'transesteriﬁcation', 'as', 'result', 'poly(vinyl', 'alcohol)', 'lower', 'molecular', 'weight', 'parent', 'material', 'poly(vinyl', 'alcohol)', 'high', 'head', 'tail', 'structures', 'based', 'nmr', 'data', 'it', 'shows', 'presence', 'small', 'amount', 'adjacent', 'hydroxyl', 'groups', 'the', 'polymer', 'prepared', 'amorphous', 'poly(vinyl', 'acetate)', 'crystalline', 'relatively', 'small', 'size', 'hydroxyl', 'groups', 'permits', 'chains', 'line-up', 'crystalline', 'domains', 'synthesis', 'isotactic', 'poly(vinyl', 'alcohol)', 'reported', 'isotactic', 'poly(vinyl', 'ethers)', 'like', 'poly(benzyl', 'vinyl', 'ether)', 'poly(t-butyl', 'vinyl', 'ether)', 'poly', '(trimethylsilyl', 'vinyl', 'ether)', 'divinyl', 'compounds', 'poly(vinyl', 'alcohol)', 'water', 'soluble', 'the', 'hydroxyl', 'groups', 'attached', 'polymer', 'backbone', 'however', 'exert', 'signiﬁcant', 'effect', 'solubility', 'when', 'ester', 'groups', 'poly(vinyl', 'acetate)', 'cleaved', 'hydroxyl', 'content', '87–89,', 'polymer', 'soluble', 'cold', 'water', 'further', 'cleavage', 'ester', 'groups', 'results', 'reduction', 'solubility', 'products', 'require', 'heating', 'water', '85(cid1)c', 'dissolve', 'this', 'due', 'strong', 'hydrogen', 'bonding', 'also', 'causes', 'unplasticized', 'poly(vinyl', 'alcohol)', 'decompose', 'ﬂow', 'temperature', 'on', 'hand', 'due', 'hydrogen', 'bonding', 'polymer', 'tough', 'poly(vinyl', 'acetals)', 'prepared', 'reacting', 'poly(vinyl', 'alcohol)', 'aldehydes', 'reactions', 'poly', '(vinyl', 'alcohol)', 'ketones', 'yield', 'ketals', 'these', 'used', 'commercially', 'not', 'hydroxyl', 'groups', 'participate', 'formations', 'acetals', 'become', 'isolated', 'a', 'typical', 'poly(vinyl', 'acetal)', 'contains', 'acetal', 'groups', 'residual', 'hydroxyl', 'groups', 'residual', 'acetate', 'groups', 'incomplete', 'transesteriﬁcation', 'parent', 'polymer', 'poly(vinyl', 'acetal)s', 'formed', 'directly', 'poly(vinyl', 'acetate)', 'actually', 'done', 'commercially', 'preparations', 'poly(vinyl', 'formal)', 'a', 'typical', 'reaction', 'carried', 'presence', 'acetic', 'acid', 'formalin', 'sulfuric', 'acid', 'catalyst', '70(cid1)c', 'o', '70', 'oc', 'o', 'o', 'o', 'n', 'o', 'o', 'h2o', 'oh', 'o', 'o', 'poly(vinyl', 'butyral)', 'hand', 'prepared', 'poly(vinyl', 'alcohol)', 'butyraldehyde', 'sulfuric', 'acid', 'used', 'catalyst', 'commercially', 'poly(vinyl', 'formal)', 'poly(vinyl', 'butyral)', 'utilized', 'large', 'scale', 'coating', 'materials', 'review', 'questions', 'review', 'questions', 'section', '61', '393', '1', 'what', 'two', 'types', 'polyethylene', 'currently', 'manufactured', 'commercially', '2', 'describe', 'chemical', 'structure', 'low-density', 'polyethylene', 'produced', 'free-radical', 'mechanism', 'show', 'chemical', 'equations', 'groups', 'present', 'form', 'how', 'low-density', 'polyethylene', 'prepared', 'ionic', 'mechanism', '3', 'describe', 'conditions', 'procedure', 'commercial', 'preparation', 'polyethylene', 'free-radical', 'mechanism', 'role', 'oxygen', 'problems', 'associated', 'oxygen', '4', 'describe', 'tubular', 'reactor', 'preparation', 'polyethylene', '5', 'what', 'industrial', 'conditions', 'preparations', 'high-density', 'polyethylene', 'describe', 'continuous', 'solution', 'process', 'slurry', 'process', 'gas-phase', 'process', '6', 'show', 'chemical', 'reactions', 'polymethylene', 'forms', 'diazomethane', 'section', '62', 'homogeneous', '1', 'discuss', 'high', 'activity', 'catalysts', 'manufacturing', 'isotactic', 'polypropylene', 'heterogeneous', '2', 'what', 'current', 'techniques', 'polypropylene', 'manufacture', '3', 'how', 'syndiotactic', 'polypropylene', 'prepared', 'properties', '1', 'describe', 'two', 'industrial', 'processes', 'manufacturing', 'polybutylene', '1', 'draw', 'chemical', 'structure', 'isotactic', 'poly(butene-1)', 'how', 'prepared', 'used', '2', 'what', 'tpx', 'prepared', 'properties', 'section', '63', 'section', '64', 'section', '65', '1', 'discuss', 'copolymers', 'ethylene', 'propylene', 'how', 'prepared', 'what', 'catalysts', 'used', 'preparations', 'how', 'ethylene–propylene', 'rubbers', 'cross-linked', '2', 'what', 'copolymers', 'ethylene', 'higher', 'a-oleﬁns', 'prepared', '3', 'discuss', 'copolymers', 'ethylene', 'vinyl', 'acetate', 'how', 'prepared', 'used', '4', 'what', 'ionomers', 'describe', 'type', 'how', 'used', '5', 'describe', 'catalysts', 'used', 'preparations', 'aliphatic', 'ketones', 'copolymerization', 'ethylene', 'carbon', 'monoxide', '394', 'section', '66', '6', 'common', 'chain-growth', 'polymers', '1', 'discuss', 'polybutadiene', 'homopolymers', 'how', 'prepared', 'what', 'uses', '2', 'what', 'popcorn', 'polymers', 'what', 'causes', 'formation', '3', 'discuss', 'liquid', 'polybutadienes', 'how', 'prepared', 'used', '4', 'how', 'high', 'molecular', 'weight', 'polybutadienes', 'prepared', 'used', '5', 'discuss', 'polyisoprenes', 'what', 'natural', 'rubber', 'where', 'come', 'what', 'synthetic', 'polyisoprenes', 'how', 'prepared', '1', 'what', 'chloroprene', 'rubber', 'how', 'made', 'used', '1', 'what', 'poly(carboxybutadiene)s', '1', 'discuss', 'cyclopolymerization', 'conjugated', 'dienes', 'section', '67', '1', 'what', 'methyl', 'rubber', 'section', '68', 'section', '69', 'section', '610', 'section', '611', 'properties', 'section', '612', '1', 'what', 'sbr', 'rubber', 'explain', 'describe', 'preparation', 'properties', '2', 'what', 'block', 'copolymer', 'elastomers', 'how', 'prepared', 'gives', 'unique', '3', 'what', 'gr-n', 'rubber', 'explain', 'describe', 'preparation', 'properties', '1', 'how', 'atactic', 'syndiotactic', 'polystyrenes', 'prepared', 'commercially', 'describe', 'explain', '2', 'what', 'polymers', 'substituted', 'styrenes', 'available', 'commercially', 'how', 'prepared', '1', 'what', 'high-impact', 'polystyrene', 'prepared', '2', 'discuss', 'abs', 'resins', 'how', 'prepared', '395', '1', 'discuss', 'chemistry', 'free-radical', 'polymerization', 'acrylic', 'methacrylic', 'esters', '2', 'what', 'acrylic', 'elastomers', 'vulcanized', '3', 'how', 'poly(methyl', 'methacrylate)', 'prepared', 'commercially', 'plexiglas', 'form', 'sheets', 'rods', 'is', 'poly(methyl', 'methacrylate)', 'prepared', 'way', 'for', 'applications', '4', 'describe', 'thermosetting', 'acrylic', 'resins', 'used', 'industrial', 'coatings', 'how', 'prepared', 'how', 'cross-linked', '1', 'discuss', 'industrial', 'polymers', 'copolymers', 'acrylonitrile', 'methacrylonitrile', 'how', '1', 'describe', 'preparation', 'uses', 'polyacrylamide', 'poly(acrylic', 'acid)', 'polymethacrylic', 'acid', '1', 'how', 'polytetraﬂuoroethylene', 'prepared', 'properties', 'uses', '2', 'discuss', 'chemistry', 'polychlorotriﬂuoroethylene', 'poly(vinylidine', 'ﬂuoride)', 'poly(vinyl', '3', 'what', 'common', 'copolymers', 'ﬂuorooleﬁns', 'used', 'commercially', '4', 'discuss', 'chemistry', 'poly(vinyl', 'chloride)', 'poly(vinylidine', 'chloride)', '5', 'discuss', 'important', 'commercial', 'copolymers', 'vinyl', 'chloride', 'what', 'main', 'uses', '6', 'discuss', 'chemistry', 'poly(vinylidine', 'chloride)', 'review', 'questions', 'section', '613', 'section', '614', 'section', '615', 'prepared', 'used', 'section', '616', 'section', '617', 'ﬂuoride)', 'section', '618', 'section', '619', '1', 'discuss', 'preparation', 'properties', 'uses', 'poly(vinyl', 'acetate)', '1', 'how', 'poly(vinyl', 'alcohol)', 'prepared', 'used', 'converted', 'poly(vinyl', 'acetal)s', '396', 'references', '6', 'common', 'chain-growth', 'polymers', '1', 'ew', 'fawcett', 'or', 'gibson', 'mw', 'perrin', 'jg', 'paton', 'eg', 'williams', 'brit', 'pat', '571590', '(1937)', '2', 'ro', 'symcox', 'p', 'ehrlich', 'j', 'am', 'chem', 'soc,', '84', '531', '(1962)', '3', 'jc', 'woodbrey', 'p', 'ehrlich', 'j', 'am', 'chem', 'soc,', '85', '1580', '(1963)', '4', 'ro', 'gibson', '“the', 'discovery', 'polythene”', 'roy', 'inst', 'chem,', 'london', 'lectures', '1964', '1', '1', '5', 'ew', 'fawcett', 'et', 'al,', 'brit', 'pat', '471590', '(1937)', '6', 'ra', 'hines', 'wmd', 'bryant', 'aw', 'larchar', 'dc', 'pease', 'ind', 'eng', 'chem,', '1957', '49(7)', '1071', '7', 'alj', 'raum', 'chapt', '4', 'vinyl', 'allied', 'polymers', '(pd', 'ritchle', 'ed)', 'crc', 'press', 'cleveland', 'ohio', '1968', '8', 'monsanto', 'co,', 'us', 'pat', '2856395', '2852501', '9', 'g', 'downs', 'chapt', '5', 'vinyl', 'allied', 'polymers', '(pd', 'ritchle', 'ed)', 'crc', 'press', 'cleveland', 'ohio', '1968', '10', 'mj', 'roedel', 'j', 'am', 'chem', 'soc,', '1953', '75', '6110', '11', 'ah', 'willbourn', 'j', 'polymer', 'sci1959,', '34', '569', '12', 'de', 'dorman', 'ep', 'otacka', 'pa', 'bovey', 'macromolecules1972', '5', '574', '13', 'wg', 'oakes', 'rb', 'richards', 'j', 'chem', 'soc,', '1949', '2929', '14', 'kw', 'doak', '“low', 'density', 'polyethylene', '(high', 'pressure)”', 'encyclopedia', 'polymer', 'science', 'technology', 'vol', '6', '2nd', 'ed.', '(hf', 'mark', 'nm', 'bikales', 'cg', 'overberger', 'g', 'menges', 'eds)', 'wiley-', 'interscience', 'new', 'york', '1986', '15', 'k', 'weissermel', 'h', 'cherdron', 'j', 'berthold', 'b', 'diedrich', 'kd', 'keil', 'k', 'rust', 'h', 'strametz', 't', 'toth', 'j', 'polymer', 'sci,', 'symposium', '51', '187', '(1975)', '16', 'dm', 'rasmussen', 'chem', 'eng,', '1972', '104', '(sept', '18)', '17', 'w', 'keim', 'r', 'appel', 'a', 'storeck', 'c', 'kruger', 'r', 'goddard', 'anew', 'chem,', 'int', 'ed', 'engl,', '1981', '20', '(1)', '116', '18', 'ms', 'brookhart', 'lk', 'johnson', 'cm', 'killian', 'j', 'am', 'chem', 'soc,', '1995', '117', '6414', '19', 'lk', 'johnson', 'cm', 'killian', 'm', 'brookhart', 'j', 'am', 'chem', 'soc1995,', '117', '6441', '20', 'm', 'brookhart', 'et', 'al,', 'us', 'patent', '5', '866', '663', '(feb', '2', '1999)', 'chemtech1999', '29', '(2)', '39', '21', 'bl', 'small', 'm', 'brookhart', 'ama', 'bennett', 'j', 'am', 'chem', 'soc,', '120', '4049', '(1998)', 'ama', 'bennett', 'chemtech', '22', 'g', 'j', 'p;', 'baugh', 'simon', 'p', 'd;', 'hoarau', 'o,', 'gibeon', 'vernon', 'c;', 'wass', 'd', 'f;', 'white', 'a', 'j', 'p;', 'williams', 'd', 'j.', '1999', '29', '(7)', '24', 'inorganica', 'chimica', 'acta', '2003', '345', '279', '23', 't', 'senda', 'h', 'hanaoka', 't', 'hinoy', 'oda', 'h', 'tsurugi', 'k', 'mashima', 'macromolecules', '2009', '42', '(21)', '8006', '24', 'j', 'h', 'woo', 'y-s', 'ha', 'y-j', 'shin', 's', 'c', 'hong', 'amchem', 'soc', 'polymer', 'preprints', '2009', '50', '(1)196', '25', 'id', 'burdett', 'chemtech', '1992', '22', '616', '26', 'm', 'm', 'wagner', 'a', 'k', 'ott', 'b', 'rieger', 'macromolecules', '2010', '43', '3624', '27', 'h', 'pechmann', 'ber,', '1898', '31', '2643', '28', 'e', 'bamberger', 'f', 'tschirner', 'ber,', '1900', '33', '955', '29', 'gd', 'buckley', 'lh', 'cross', 'nh', 'roy', 'j', 'chem', 'soc,', '1950', '2714', '30', 'a', 'renfrew', 'p', 'morgan', 'polythene', 'wiley-interscience', 'new', 'york', 'ny,', '1957', 'h', 'meerwein', 'angew', 'chem,', '1948', 'a60', '78', '31', 'ceh', 'bawn', 'a', 'ledwith', 'p', 'matthies', 'j', 'polymer', 'sci,', '1959', '34', '93', '32', 'ceh', 'bawn', 'a', 'ledwith', 'chem', 'ind', '(london)', '1957', '1180', '33', 'gd', 'buckley', 'nh', 'ray', 'j', 'chem', 'soc,', '1952', '3701', '34', 'ag', 'nasini', 'g', 'saini', 't', 'trossarelli', 'j', 'polymer', 'sci,', '1960', '48', '435', 'pure', 'appl', 'chem,', '1962', '4', '255', '35', 'mg', 'krakonyak', 'ss', 'skorokhodov', 'vysokomol', 'soyed,', '1969', 'a11(4)', '797', 'h', 'pechler', 'b', 'firnhaber', 'dkroussis', 'a', 'dawallu', 'makromol', 'chem,', '1964', '70', '12', '36', 'ga', 'mortimer', 'lc', 'arnold', 'j', 'polymer', 'sci,', '1964', 'a2', '4247', '37', 'g', 'natta', 'p', 'pino', 'g', 'mazzanti', 'p', 'longi', 'gazz', 'chim', 'ital1957,', '87', '570', '(from', 'private', 'translation)', '38', 'g', 'natta', 'i', 'pasquon', 'a', 'zambelli', 'g', 'gatti', 'j', 'polymer', 'sci1961,', '51', '387', '39', 'avison', 'corp', 'us', 'patents', '3134642', '(may', '26', '1964)', '2980664', '(april', '18', '1961)', '3055878', '(sept25', '1962)', '3216987', '(nov', '9', '1596)', '3303179', '(feb', '7', '1967)', '3328375', '(june', '27', '1967)', '3362916', '(jan', '9', '1964)', '3313791', '(april', '11', '1967)', '3255167', '(june', '7', '1966)', 'german', 'patent', '1236789', '(march', '16', '1967)', '1234218', '(april', '6', '1967)', '40', 'eastman', 'kodak', 'co,', 'us', 'patents', '3232919', '(feb', '1', '1966)', '3058696', '(oct', '10', '1962)', '3081287', '(march', '12', '1963)', '2962487', '(nov', '29', '1960)', '3143537', '(july', '21', '1964)', '3004015', '(oct', '10', '1965)', '3149097', '(sept', '15', '1964)', '3201379', '(aug', '17', '1965)', '3213073', '(oct', '19', '1965)', '3230208', '(jan', '18', '1966)', '3149097', '(sept', '15', '1964)', '3186977', '(june', '1', '1965)', '3189590', '(june', '15', '1965)', '3072629', '(jan', '8', '1963)', '3178401', '(april', '13', '1965)', '3284427', '(nov', '8', '1966)', 'british', 'patents', '921039', '(march', '13', '1963)', '930633', '(july', '3', '1963)', '1007030', '(oct', '13', '1965)', '100348', '(aug', '4', '1965)', '1000720', '(aug', '11', '1965)', 'french', 'patents', '1315782', '(jan', '25', '1963)', '41', 'montecantini-edison', 'spa,', 'us', 'patents', '3112300', '(nov', '26', '1963)', '3259', '613', '(july', '5', '1966)', '3139418', '(june', '30', '1964)', '3141872', '(july', '21', '1964)', '3277069', '(oct', '4', '1969)', '3252954', '(may', '25', '1966)', 'brit', 'patents', 'references', '397', '1014944', '(jan', '10', '1964)', 'canad', 'patents', '649164', '(sept', '25', '1962)', 'german', 'patents', '1238667', '(april', '13', '1967)', '1214', '000', '(june', '6', '1961)', 'italian', 'patents', '(from', 'chem', 'abstr)', '646950', '(sept', '20', '1965)', '42', 'phillips', 'petroleum', 'co', 'us', 'patents', '3280092', '(oct', '18', '1966)', '3119798', '(jan', '28', '1964)', '3147241', '(sept', '1', '1964)', '3182049', '(may', '4', '1965)', '3210332', '(oct', '5', '1965)', '3317502', '(may', '2', '1967)', 'brit', 'patents', '1017988', '(jan', '26', '1966)', '1034155', '(june', '291966)', '1119033', '(march', '7', '19680)', 'belgian', 'patent', '695060', '(june', '9', '1967)', 'german', 'patent', '1266504', '(april', '18', '1968)', '43', 'shell', 'oil', 'co', 'us', 'patents', '3147238', '(sept', '1', '1964)', '3240773', '(march', '15', '1966)', '3282906', '(nov11966)', '3398130', '(aug', '20', '1968)', '3311603', '(march', '28', '1967)', '3394118', '(july', '23', '1968)', '3264277', '(aug', '2', '1966)', 'brit', 'patent', '1006919', '(oct', '6', '1965)', '44', 'u', 'giannini', 'a', 'casata', 'p', 'lorgi', 'r', 'mazzocchi', 'germ', 'pat', 'appl', '2346577', '(1974)', '45', 'hr', 'sailors', 'jp', 'hogam', 'j', 'macromol', 'sci-chem1981,', 'a15', '1377', '46', 'p', 'pino', 'r', 'mulhaupt', 'angew', 'chem,', 'int', 'ed,', 'engl,', '1980', '19', '857', '47', 'k', 'weissermel', 'h', 'cherdron', 'j', 'berthold', 'b', 'diedrich', 'kd', 'keil', 'k', 'rust', 'h', 'strimetz', 't', 'toth', 'j', 'polymer', '48', 's', 'sivaram', 'ind', 'eng', 'chem', 'prod', 'res', 'dev,', '1977', '16(2)', '121', '49', 'ee', 'vermel', 'va', 'zakharov', 'zk', 'bukatova', 'gp', 'shkurina', 'sg', 'echevskaya', 'em', 'moroz', 'sv', 'sudakova', 'sci,', 'sympos', '1975', '51', '187', 'vysokomol', 'soyed,', '1980', '22(1)', '22', '50', 'tw', 'campbell', 'ac', 'haven', 'jr,', 'j', 'appl', 'polymer', 'sci1959,', '1', '73', 'p', 'carradini', 'v', 'busico', 'g', 'guerra', '“monoalkene', 'polymerization', 'stereochemistry”', 'chap', '3', 'comprehensive', 'polymer', 'science', 'vol4', '(gc', 'eastmond', 'a', 'ledwith', 's', 'russo', 'p', 'sigwalt', 'eds)', 'pergamon', 'press', 'oxford', '1989', '51', 'u', 'giannini', 'makromol', 'chem,', 'suppl1981,', '5', '216', '52', 'kh', 'reichert', 'hu', 'moritz', 'j', 'appl', 'polymer', 'sci,', '1981', '36', '151', '53', 'j', 'lewis', 'pe', 'okieimen', 'gs', 'park', 'j', 'macromol', 'sci-chem,', '1982', 'a17', '1021', '54', 'fa', 'bovey', 'kb', 'abbas', 'fc', 'schilling', 'wh', 'starnes', 'jr,', 'macromolecules1975', '8', '437', '55', 'd', 'o’sullivan', 'chem', 'eng', 'news', 'p', '29', '(july', '4', '1983)', '56', 'd', 'cam', 'e', 'albizzati', 'p', 'cinquina', 'makromol', 'chem,', '1990', '191', '1641', '57', 'ja', 'ewen', 'j', 'am', 'chem', 'soc,', '1984', '106', '6355', '58', 'b', 'rieger', 'x', 'mu', 'dt', 'mallin', 'md', 'rausch', 'jcw', 'chien', 'macromolecules', '1990', '23', '3559', '59', 'hn', 'cheng', 'ja', 'ewen', 'makromol', 'chem,', '1989', '190', '823', '60', 'a', 'grassi', 'a', 'zambelli', 'l', 'resconi', 'e', 'albizzati', 'r', 'mazzocchi', 'macromolecules', '1988', '21', '617', '61', 'gg', 'arzonmanidis', 'nm', 'karayannis', 'chemtech', '1993', '23', '43', '62', 'rw', 'waymouth', 'gw', 'coates', 'science', '1995', '267', '222', '63', 'b,', 'vincenzo', 'c,', 'v', 'van', 'axel', 'a,', 'p', 'c,', 'roberta', 's', 'annalaura', 't', 'giovanni', 'm', 'vacatello', 'j', 'am', 'chem', 'soc,', '2003', '125(18)', '5451—5460', '64', 'jcw', 'chien', 'md', 'iwamoto', 'md', 'rausch', 'w', 'wedler', 'hh', 'winter', 'macromolecules1997', '30', '3447', '65', 'j', 'boor', 'jr', 'ea', 'youngman', 'j', 'polymer', 'sci1966,', 'a-14', '1861', '66', 'g', 'natta', 'i', 'pasquon', 'a', 'zambelli', 'j', 'am', 'chem', 'soc,', '1962', '84', '1488', '67', 'a', 'zambelli', 'g', 'natta', 'i', 'pasquon', 'j', 'polymer', 'sci,', '1963', 'c4411', '68', 'vw', 'buls', 'tl', 'higgins', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1973', '11', '925', 'y', 'doi', 's', 'ueki', 't', 'keii', 'macromolecules', '1979', '12(5)', '814', '69', 'ig', 'farbenindustri', 'a-g,', 'german', 'pat', '6342', '78', '(1936)', '70', 'jbrandrup', 'eh', 'immergut', 'polymer', 'handbook', 'wiley-interscience', 'new', 'york', '1989', '71', 't', 'asanuma', 'y', 'nishimori', 'm', 'ito', 'n', 'uchikawa', 't', 'shiomura', 'polym', 'bull1991,', '25', '567', '72', 'rm', 'thomas', 'wj', 'sparks', 'pk', 'frolich', 'm', 'otto', 'm', 'mueller-cuuradi', 'j', 'am', 'chem', 'soc1940,', '62', '276', '73', 'rb', 'cundall', 'the', 'chemistry', 'cationic', 'polymerization', '(ph', 'plesch', 'ed)', 'pergamon', 'press', 'macmillan', 'co,', '74', 'jp', 'kennedy', 'polymer', 'chemistry', 'synthetic', 'elastomers', 'part', '1', 'jp', 'kennedy', 'egm', 'tornquist', 'ed', 'new', 'york', '1963', 'wiley-interscience', 'new', 'york', '1968', '75', 'a', 'gandini', 'h', 'cheradame', '“cationic', 'polymerization”', 'encyclopedia', 'polymer', 'science', 'engineering', 'vol', '2', 'hf', 'mark', 'nm', 'bikales', 'cg', 'overberger', 'g', 'menges', 'eds,', 'wiley', 'interscience', 'new-york', '1985', '76', 'ng', 'gaylord', 'hf', 'mark', '“linear', 'stereoregular', 'addition', 'polymers”', 'interscience', 'ny,', '1959', '77', 's', 'garratt', 'ag', 'carr', 'g', 'langstein', 'm', 'bochmann', 'macromolecules', '2003', '36', '4276', '78', 'ja', 'ewen', 'rl', 'jones', 'a', 'razavi', 'jd', 'ferrara', 'j', 'am', 'chem', 'soc,', '1988', '110', '6255', '79', 'g', 'natta', 'g', 'mazzanti', 'a', 'valvassori', 'g', 'pajaro', 'chim', 'ind', '(milan)', '1957', '39', '733', '80', 'kr', 'dunham', 'j', 'vanderberghe', 'jwh', 'faber', 'le', 'contois', 'j', 'polymer', 'sci,', '1963', 'a-11', '751', '81', 'g', 'natta', 'p', 'carradini', 'iw', 'bassi', 'gazz', 'chim', 'ital1959,', '89', '784', '82', 'hw', 'coover', 'jr,', 'fb', 'joyner', 'j', 'polymer', 'sci,', '1965', 'a-13', '2407', '83', 'e', 'suzuki', 'm', 'tamura', 'y', 'doi', 't', 'keii', 'makromol', 'chem,', '1979', '180', '2235', '84', 'w', 'passmann', 'ind', 'eng', 'chem,', '1970', '62(5)', '48', '398', '6', 'common', 'chain-growth', 'polymers', '85', 'h', 'yasuda', 'e', 'ihara', 'm', 'morimoto', 'm', 'nodono', 's', 'yoshioka', 'm', 'furo', 'macromol', 'symp1995', '95', '203', '86', 'w', 'marconi', 's', 'cesea', 'g', 'della', 'foruna', 'chim', 'ind', '(milan)1964', '46', '1131', '87', 's', 'cesea', 's', 'arrighetti', 'w', 'marconi', 'chim', 'ind', '(milan)', '1968', '50', '171', '88', 's', 'cesea', 'g', 'bertolini', 'g', 'santi', 'pv', 'duranti', 'j', 'polymer', 'sci,', '1971', 'a-19', '1575', '89', 'ek', 'gladding', 'bs', 'fischer', 'jw', 'collette', 'ind', 'eng', 'chem,', 'prod', 'res', 'devel1962,', '1', '65', '90', 'ew', 'duck', 'w', 'cooper', '23-rd', 'iupac', 'congress', 'macromol', 'prepr,', '1971', '2', '722', '91', 'ek', 'easterbrook', 'tj', 'brett', 'fc', 'loveless', 'dn', 'mathews', '23-rd', 'iupac', 'congress', 'macromol', 'prepr1971,', '2', '712', '92', 'c', 'cozewith', 'g', 'ver', 'strate', 'macromolecules', '1971', '4', '482', '93', 'a', 'akimoto', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1972', '10', '3113', '94', 'ca', 'lukach', 'hm', 'spurlin', 'copolymerization', '(ge', 'ham', 'ed)', 'interscience', 'ny,', '1964', '95', 'g', 'natta', 'g', 'mazzanti', 'a', 'valvassori', 'g', 'sartori', 'a', 'barbagallo', 'j', 'polymer', 'sci1961,', '51', '411', '429', '96', 'g', 'bier', 'g', 'lehmann', 'copolymerization', 'ge', 'ham', 'ed,', 'interscience', 'ny,', '1964', '97', 'jg', 'hefner', 'bws', 'kolthammer', 'us', 'patent', '5554777', '(sept', '10', '1996)', '98', 'chemical', 'engineering', 'news', 'october', '5', '1998', 'p18', '99', 'pg', 'hustad', 'jd', 'weinhord', 'gr', 'marchnd', 'ei', 'garcia-maitin', 'pl', 'robers', 'macromolecules', 'doi101021', 'ma9002819', 'chemengnews', 'may', '4', '2009', '100', 'kenichi', 'kakinuki', 'michiya', 'fujiki', 'kotohiro', 'nomura', 'macromolecules', '2009', '10', '1921ma900576v', '101', 'derlin', 'kaminsky', 'macromolecules', 'asap', 'article', '101021web', 'release', 'date', 'may', '10', '2007', '102', 'a', 'ravasio', 'l', 'boggioni', 'i', 'tritto', 'macromolecules', '2011', '44', '4180', '103', 'rf', 'jordan', 'am', 'chem', 'soc', 'polymer', 'preprints', '2010', '51', '(2)', '372', '104', 'j', 'furukawa', 'angew', 'makromol', 'chem1972,', '23', '189', '105', 'j', 'furukawa', 's', 'tsuruki', 'jkiji', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed1973,', '11', '1819', '106', 'gbier', 'angew', 'chem,', '1961', '73', '186', '107', 'mj', 'wisotsky', 'ab', 'kober', 'ia', 'zlochower', 'j', 'appl', 'polymer', 'sci1971,', '15', '1737', '108', 'te', 'ferrington', 'av', 'tobolsky', 'j', 'polymer', 'sci,', '1958', '31', '25', '109', 'ma', 'hillmyer', 'st', 'nguyen', 'rh', 'grubbs', 'macromolecules', '1997', '30', '718', '110', 'dj', 'valenti', 'kb', 'wagner', 'macromolecules', '1998', '31', '2764', '111', 'am', 'bravakis', 'le', 'bailey', 'm', 'pigeon', 's', 'collins', 'macromolecules', '1998', '31', '1000', '112', 'fp', 'gintz', 'chapt', '11', 'vinyl', 'allied', 'polymers', 'pd', 'ritchie', 'ed,', 'vol', 'i', 'crc', 'press', 'cleveland', 'ohio', '1968', '113', 'vd', 'mochel', 'j.', 'polymer', 'sci,', '1972', 'a-110', '1009', '114', 'l', 'annunziata', 's', 'pragilola', 'dpappalardo', 'c', 'tedesco', 'c', 'pellecchia', 'macromolecules', '2011', '44(7)', '1934', '115', 'wm', 'saltman', 'et', 'al', 'j', 'am', 'chem', 'soc1958,', '80', '5615', '116', 'wm', 'saltman', 'j', 'polymer', 'sci1963,', 'a-11', '373', '117', 'rs', 'stearns', 'et', 'al', 'j', 'polymer', 'sci,', '1959', '41', '381', '118', 'av', 'tobolsky', 'j', 'polymer', 'sci1959,', '40', '73', '119', 'fw', 'stavely', 'et', 'al', 'ind', 'eng', 'chem,', '1956', '48(4)', '778', '120', 'ga', 'thomas', 'wh', 'carmody', 'j', 'am', 'chem', 'soc1933,', '55', '3854', '121', 'jc', 'd’ianni', 'kautschuk', 'gummi', 'kunststoffe', '1966', '19(3)', '138', 'jd', 'd’ianni', 'fj', 'naples', 'je', 'field', 'ind', 'eng', 'chem1950,', '41', '95', 's.', 'kostjuk', 's', 'ouardad', 'f', 'peruch', 'a', 'defﬁeux', 'c', 'absalon', 'j', 'e', 'puskas', 'f', 'ganachaud', 'macromolecules', '2011', '44', '1372', '122', 'je', 'field', 'm', 'feller', 'us', 'patent', '2728758', '(dec', '27', '1955)', '123', 'he', 'adams', 'rs', 'stearns', 'wa', 'smith', 'jl', 'binder', 'ind', 'eng', 'chem,', '1958', '50', '1507', '124', 's', 'minekawa', 'et', 'al', 'japanese', 'patent', '45-36519', '(nov', '20', '1970)', '125', 'tf', 'yen', 'j', 'polymer', 'sci,', '1959', '33', '535', '126', 'o', 'solomon', 'c', 'amrus', 'rev', 'chem', '(romania)', '1958', '9', '150', '127', 'm', 'gippin', 'us', 'patent', '3442878', '(may', '6', '1969)', '128', 'al', 'klebanski', 'j', 'polymer', 'sci1958,', '30', '363', 'rubber', 'chem', 'technol1959,', '32', '588', '129', 'jt', 'maynard', 'we', 'mochel', 'j', 'polymer', 'sci,', '1954', '13', '235', '251', '130', 'rr', 'garrett', 'cd', 'hargreaves', 'ii', 'dn', 'robinson', 'j', 'macromol', 'sci-chem1970', 'a4(8)', '1679', '131', 'cen', 'bawn', 'am', 'north', 'js', 'walker', 'polymer', '1964', '5', '419', '132', 'cen', 'bawn', 'dgt', 'cooper', 'am', 'north', 'polymer', '1966', '7(3)', '113', '133', 'w', 'marconi', 'a', 'mazzei', 's', 'cucinella', 'm', 'cesari', 'j', 'polymer', 'sci,', '1964', 'a-12', '426', '134', 'g', 'natta', 'm', 'farina', 'm', 'donati', 'm', 'peraldo', 'chim', 'ind', '(milan)', '1960', '42', '1363', '135', 'g', 'natta', 'm', 'farina', 'm', 'donati', 'makromol', 'chem,', '1961', '43', '251', '136', 'g', 'natta', 'p', 'corradini', 'p', 'canis', 'j', 'polymer', 'sci,', '1965', 'a-13', '11', 'g', 'natta', 'l', 'porri', 'p', 'corradini', 'g', 'zanini', 'f', 'ciampelli', 'j', 'polymer', 'sci1961,', '51', '463', 'g', 'natta', 'l', 'porri', 'g', 'stoppa', 'g', 'allegra', 'f', 'ciampelli', 'j', 'polymer', 'sci,', '1963', 'b1', '67', '137', 'g', 'natta', 'l', 'porri', 'a', 'carbonaro', 'f', 'ciampelli', 'g', 'allegro', 'makromol', 'chem,', '1962', '51', '229', 'references', '399', '138', 'g', 'natta', 'l', 'porri', 'a', 'carbonaro', 'f', 'stoppa', 'makromol', 'chem,', '1964', '77', '114', '139', 'g', 'natta', 'l', 'porri', 'acs', 'polymer', 'preprints', '5(2)', '1163', '(1964)', 'k', 'bujadoux', 'j', 'josefonvicz', 'j', 'neel', 'eur', 'polymer', 'j,', '1970', '6', '1233', '140', 'al', 'halasa', 'df', 'lohr', 'je', 'hall', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1981', '19', '1357', '141', 'ng', 'gaylord', 'i', 'kossler', 'm', 'stolka', 'j', 'vodehnal', 'j', 'polymer', 'sci1964,', 'a-12', '3969', '142', 'ng', 'gaylord', 'i', 'kossler', 'm', 'stolka', 'j', 'macromol', 'sci,-', 'chem,', '1968', 'a2(2)', '421', '143', 'k', 'hasegawa', 'r', 'asami', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed1978', '161449', 'k', 'hasegawa', 'r', 'asami', 't', 'higashimura', 'macromolecules', '1977', '10', '585592', '144', 'ii', 'yermakova', 'yen', 'kropacheva', 'a1', 'kol’tsov', 'ba', 'dolgoplosk', 'vysokomol', 'soyed,', '1969', 'a11(7)', '1639', '145', 'ng', 'gaylord', 'm', 'stolka', 'v', 'stepan', 'i', 'kossler', 'j', 'polymer', 'sci1968,', 'c(23)', '317', '146', 'er', 'moore', '(ed)', '“styrene', 'polymers”', 'encyclopedia', 'polymer', 'science', 'engineering', 'vol', '16', '2nd', 'ed,', '(hf', 'mark', 'nm', 'bikales', 'cg', 'overberger', 'g', 'menges', 'eds)', 'wiley-interscience', 'new', 'york', '1989', '147', 't', 'takamatsu', 'a', 'ohnishi', 't', 'nishikida', 'j', 'furukawa', 'rubber', 'age1973', '23', '148', 'e', 'simon', 'ann,', '1839', '31', '265', '149', 'h', 'stobbe', 'g', 'posnjack', 'ann1909,', '259', '150', 'gv', 'schulz', 'a', 'physik', 'chem', '(leipzig)', '1939', 'b44', '277', '151', 'sr', 'mayo', 'j', 'am', 'chem', 'soc1953', '75', '6133', '152', 'rr', 'hiatt', 'pd', 'bartlett', 'j', 'am', 'chem', 'soc,', '1959', '81', '1149', '153', 'j', 'kurze', 'dj', 'stein', 'p', 'simak', 'r', 'kaiser', 'angew', 'makroml', 'chem,', '1970', '12', '25', '154', 'fr', 'mayo', 'j', 'am', 'chem', 'soc1968,', '90', '1289', '155', 'j', 'wiesner', 'p', 'mehnert', 'makromol', 'chem,', '1973', '165', '1', '156', 'fa', 'bovey', 'im', 'kolthoff', 'chem', 'rev1948,', '42', '491', '157', 'o', 'roehm', 'german', 'pat', '656469', '(1938)', '158', 'n', 'platzer', 'ind', 'eng', 'chem1970,', '62(1)', '6', '159', 'igfarbenindustri', 'a', '-g,german', 'patent', '634278', '(1936)', '160', 'dow', 'chem', 'co', 'us', 'patent', '2727884', '(1955)', '161', 'jp', 'kennedy', 'j', 'macromol', 'sci-chem1969,', 'a3', '861', '162', 'jp', 'kennedy', 'j', 'macromol', 'sci-chem,', '1969', 'a3', '885', '163', 'fs', 'd’yachkovskii', 'ga', 'kazaryan', 'ns', 'yenikolopyan', 'vysokomol', 'soyed1969,', 'a11', '822', '164', 'aa', 'morton', 'e', 'grovenstein', 'jr,', 'j', 'am', 'chem', 'soc,', '1952', '74', '5435', '165', 'jlr', 'wllliams', 'tm', 'laasko', 'wj', 'dulmage', 'j', 'org', 'chem,', '1958', '23', '638', '1206', '166', 'd', 'baum', 'w', 'betz', 'w', 'kern', 'makromol', 'chem1960,', '42', '89', '167', 'rf', 'zern', 'nature', '1960', '187', '410', '168', 'kf', 'o’drlscoll', 'av', 'tobolsky', 'j', 'polymer', 'sci,', '195935', '259', '169', 'rcp', 'cubbon', 'd', 'margerison', 'proc', 'roy', 'soc', '(london)', '1962', 'a268', '260', '170', 'aa', 'morton', 'ld', 'taylor', 'j', 'polymer', 'sci1959,', '38', '7', '171', 'g', 'natta', 'makromol', 'chem,', '1955', '16', '77', '172', 'g', 'natta', 'j', 'polymer', 'sci,', '1955', '16', '143', '173', 'cg', 'overberger', 'j', 'polymer', 'sci,', '1959', '35', '381', '174', 'ac', 'shelyakov', 'dokl', 'akad', 'nauk', 'ussr', '1958', '122', '1076', '175', 'dy', 'yoon', 'pr.', 'sundararajan', 'pj', 'flory', 'macromolecules', '1975', '8', '776', '176', 'va', 'kargin', 'va', 'kabanov', 'ii', 'mardenko', 'polym', 'sci.', 'ussr', '1960', '1', '41', '177', 'n', 'ishihara', 't', 'seimiya', 'm', 'kuramoto', 'm', 'uoi', 'macromolecules', '1986', '19', '2464', '178', 'w', 'kaminsky', 's', 'lenk', 'v', 'scholz', 'hw', 'roesky', 'a', 'herzog', 'macromolecules', '1997', '30', '7647', '179', 'a', 'grassi', 'a', 'zambelli', 'am', 'chem', 'soc', 'polymer', 'preprints', '1996', '37', '(2)', '523', '180', 'l', 'perrin', 'e', 'kiriiiov', 'j', 'f', 'carpentier', 'l', 'maron', 'macromolecules', '2010', '43', '6330', '181', 'h', 'ohgi', 'tsato', 'macromolecules', '1999', '32', '2403', '182', 'f', 'wenger', 'sps', 'yen', 'makromol', 'chem,', '1961', '43', '1', '183', 'dj', 'worsford', 's', 'bywater', 'can', 'j', 'chem,', '1960', '38', '1881', '184', 'dp', 'wyman', 't', 'altares', 'jr,', 'makromol', 'chem,', '1964', '72', '68', 't', 'altares', 'jr,', 'dp', 'wyman', 'vr', 'allen', 'j', 'polymer', 'sci,', '1965', 'a3', '4131', '185', 'f', 'wenger', 'sps', 'yen', 'am', 'chem', 'soc', 'polymer', 'preprints', '1962', '3', '163', '186', 'sps', 'yen', 'makromol', 'chem,', '1965', '81', '152', '187', 't', 'altares', 'jr', 'el', 'clark', 'ind', 'eng', 'chem,', 'prod', 'res', 'dev', '1970', '9', '(2)', '168', '188', 'g', 'natta', 'f', 'danusso', 'd', 'sianesi', 'makromol', 'chem,', '1958', '28', '253', '189', 'g', 'natta', 'f', 'danusso', 'd', 'sianesi', 'makromol', 'chem,', '1959', '30', '238', '190', 'y', 'sakurada', 'j', 'polymer', 'sci,', '1963', 'a-11', '2407', '191', 'fs', 'dalton', 'rh', 'tomlinson', 'j', 'chem', 'soc,', '1953', '151', '192', 'do', 'jordan', 'ar', 'mathieson', 'j', 'chem', 'soc,', '1952', '2354', '400', '6', 'common', 'chain-growth', 'polymers', '193', 'dj', 'worsford', 's', 'bywater', 'j', 'am', 'chem', 'soc,', '1957', '9', '491', '194', 'a', 'hersberger', 'jc', 'reid', 'rg', 'heilliman', 'ind', 'eng', 'chem,', '1945', '37', '1073', '195', 'g', 'natta', 'spe', 'j,', '1959', '15', '373', '196', 'ja', 'price', 'mr', 'lytton', 'bg', 'ranby', 'j', 'polymer', 'sci', '1961', '51', '541', '197', 'rf', 'boyer', 'in', 'vol13', 'encyclopedia', 'polymer', 'science', 'technology', '(hf', 'mark', 'ng', 'gaylord', 'nmbikales', 'eds)', 'wiley-interscience', 'new', 'york', 'ny,', '1970', '198', 'fr', 'mayo', 'c', 'walling', 'chem', 'rev,', '1950', '46', '191', '199', 'rh', 'boundy', 'rf', 'boyer', '“styrene', 'its', 'polymers', 'copolymers', 'derivatives”', 'reinhold', 'new', 'york', '1952', '200', 'lj', 'young', 'chapt', '8', 'copolymerization', '(ge', 'ham', 'ed)', 'interscience', 'new', 'york', 'ny', '1964', '201', 'la', 'wood', 'j', 'polymer', 'sci,', '1958', '28', '319', '202', 'jg', 'fox', 's', 'gratch', 'ann', 'ny', 'acad', 'sci,', '1953', '57', '367', '203', 'rh', 'wiley', 'gm', 'brauer', 'j', 'polymer', 'sci,', '1948', '3', '455', '647;', 'ibid', '1949', '4', '351', '204', 'jg', 'bevington', 'hw', 'melville', 'rp', 'taylor', 'j', 'polymer', 'sci', '1954', '12', '449', '205', 'ge', 'molan', 'j', 'polymer', 'sci,', '1965', 'a-13', '126', '206', 'ge', 'molan', 'j', 'polymer', 'sci,', '1965', 'a-13', '4235', '207', 'ud', 'standt', 'j', 'klein', 'makrom', 'makromol', 'chem,', 'rapid', 'comm,', '1981', '2', '41', '208', 'ge', 'schildknecht', 'vinyl', 'related', 'polymers', 'wiley', 'new', 'york', '1952', '209', 'ge', 'barnes', 'rm', 'olofson', 'go', 'jones', 'j', 'am', 'chem', 'soc,', '1950', '72', '210', '210', 'hw', 'melville', 'proc', 'roy', 'soc', '(london)', '1938', 'a', '167', '99', '211', 'gd', 'dixon', 'j', 'polymer', 'sci', 'polymer', 'chem', 'ed,', '1974', '12', '1717', '212', 'h', 'berghmans', 'g', 'smets', 'makromol', 'chem,', '1960', '115', '187', '213', 'dl', 'glusker', 'e', 'stiles', 'yoncoskie', 'j', 'polymer', 'sci', '1961', '49', '197', '214', 't', 'fox', 'j', 'am', 'chem', 'soc,', '1958', '80', '1768', '215', 'we', 'goode', 'fh', 'owens', 'rp', 'fellman', 'wh', 'snuder', 'je', 'moore', 'j', 'polymer', 'chem,', '1960', '46', '317', '216', 'we', 'goode', 'wh', 'snyderand', 'rg', 'fettes', 'j', 'polymer', 'sci,', '42', '367', '(1960)', '217', 'fj', 'welch', 'us', 'patent', '3048572', '(aug', '7', '1962)', '218', 'h', 'nagai', 'j', 'appl', 'polymer', 'sci,', '1963', '7', '1697', '219', 's', 'smith', 'j', 'polymer', 'sci,', '1959', '38', '259', '220', 'wg', 'gall', 'ng', 'mccrum', 'j', 'polymer', 'sci,', '1961', '50', '489', '221', 'mr', 'miller', 'ge', 'rauhut', 'j', 'am', 'chem', 'soc,', '1958', '80', '4115', '222', 'mr', 'miller', 'ge', 'rauhut', 'j', 'polymer', 'sci,', '1959', '38', '63', '223', 'we', 'goode', 'fh', 'owens', 'wl', 'myers', 'j', 'polymer', 'sci,', '1960', '47', '75', '224', 'b', 'garrett', 'et', 'al', 'j', 'am', 'chem', 'soc,', '1959', '81', '1007', '225', 'we', 'goode', 'rp', 'fellman', 'fh', 'owens', 'voli', 'macromolecular', 'syntheses', '(cg', 'overberger', 'ed)', 'wiley', 'new', 'york', 'ny,', '1963', 'ny,', '1963', '226', 'cf', 'ryan', 'jj', 'gormley', 'voli', 'macromolecular', 'syntheses', '(cg', 'overberger', 'ed)', 'wiley', 'new', 'york', '227', 'a', 'yamamoto', 't', 'shimizu', 'i', 'ikeda', 'polymer', 'j,', '1979', '1', '171', '(information', 'obtained', 'a', 'yamamoto', 't', 'yamamoto', 'macromol', 'rev)', '1978', '13', '161', '228', 'h', 'abe', 'k', 'imai', 'm', 'matsumoto', 'j', 'polymer', 'sci,', '1965', 'b3', '1053', '229', 'a', 'yamamoto', 't', 'yamamoto', 'macromol', 'chem,', '1978', '13', '161', '230', 'n', 'yamazaki', 't', 'shu', 'kogyo', 'kagaku', 'zashi', '74', '2382', '(1971)', '(from', 'a', 'yamamoto', 't', 'yamamoto)', 'macromol', 'chem,', '1978', '13', '161', '231', 'ss', 'dixit', 'ab', 'deshpande', 'sl', 'kapur', 'j', 'polymer', 'sci', '1970', 'a-18', '1289', '232', 'a', 'ravve', 'jt', 'khamis', 'us', 'patents', '3306883', '(feb', '28', '1967)', '3323946', '(june', '6', '1967)', '233', 'rh', 'yokum', 'functional', 'monomers', 'their', 'preparations', 'polymerization', 'application', 'voli', 'ii', 'dekker', 'new', 'york', 'ny,', '1973', '234', 'a', 'ravve', 'jt', 'khamis', 'j', 'macromol', 'sci,-chem', '1967', 'a1(8)', '1423', '235', 'a', 'ravve', 'kh', 'brown', 'j', 'macromol', 'sci,-chem,', '1979', 'a13(2)', '285', '236', 'h', 'yasuda', 'h', 'yamamoto', 'k', 'yokota', 's', 'miyake', 'a', 'nakamura', 'j', 'am', 'chem', 'soc', '1992', '114', '4908', '237', 'h', 'yasuda', 'h', 'yamamoto', 'm', 'yamashita', 'k', 'yokota', 'a', 'nakamura', 's', 'miyake', 'y', 'kai', 'n', 'kanehisa', 'macromolecules', '1993', '26', '7134', '238', 'rm', 'waymouth', 'mk', 'leclerc', 'angew', 'chem,', '1998', '37', '922', '239', 'bm', 'novak', 's', 'boffa', 'am', 'chem', 'soc', 'polymer', 'preprints', '1997', '38', '(1)', '442', '240', 'a', 'leblanc', 'e', 'grau', 'j-p', 'broyer', 'c', 'boisson', 'r', 'spitz', 'v', 'montei', 'macromolecules', '2011', '44', '3293', '241', 'a', 'roig', 'je', 'figueruelo', 'e', 'liano', 'j', 'polymer', 'sci,', '1968', 'c(16)', '4141', '242', 'dp', 'kelly', 'gjh', 'melrose', 'dh', 'solomon', 'j', 'appl', 'polymer', 'sci,', '1963', '7', '1991', '243', 'ch', 'bamford', 'ad', 'jenkins', 'proc', 'royal', 'soc,', '(london)', 'ser', 'a', '1953', 'a216', '515', '244', 'wm', 'thomas', 'jj', 'pellon', 'j', 'polymer', 'sci,', '13', '329', '(1954)', 'references', '401', '245', 'a', 'zilkha', 'ba', 'feit', 'm', 'frankel', 'j', 'chem', 'soc,', '1959', '928', '246', 'm', 'frankel', 'a', 'ottolenghi', 'm', 'albeck', 'a', 'zilkha', 'j', 'chem', 'soc,', '1959', '3858', '247', 'a', 'ottolenghi', 'a', 'zilkha', 'j', 'polymer', 'sci,', '1963', 'a-11', '647', '248', 's', 'inoue', 't', 'tsuruta', 'j', 'furukawa', 'makromol', 'chem,', '1960', '42', '12', '249', 'a', 'zilkha', 'p', 'neta', 'm', 'frankel', 'j', 'chem', 'soc,', '1960', '3357', '250', 'fs', 'dainton', 'em', 'wiles', 'an', 'wright', 'j', 'polymer', 'sci,', '1960', '45', '111', '251', 'jl', 'down', 'j', 'lewis', 'b', 'moore', 'g', 'wilkinson', 'proc', 'chem', 'soc,', '1957', '209', '252', 'rb', 'cundall', 'dd', 'eley', 'r', 'worrall', 'j', 'polymer', 'sci,', '58', '869', '(1962)', '253', 'ns', 'wooding', 'wcs', 'higginson', 'j', 'chem', 'soc,', '1952', '774', '254', 'csh', 'chen', 'n', 'colthup', 'w', 'deichert', 'rl', 'webb', 'j', 'polymer', 'sci', '1960,', '45', '247', '255', 'ml', 'miller', 'j', 'polymer', 'sci,', '1962', '56', '203', '256', 'we', 'hanford', 'rm', 'joyce', 'j', 'am', 'chem', 'soc,', '1946', '68', '2082', '257', 'r', 'kiyama', 'j', 'osugi', 's', 'kusuhara', 'rev', 'phys', 'chem,', 'japan', '1957', '27', '22', '258', 'n', 'shavit', 'a', 'oplaka', 'm', 'levy', 'j', 'polymer', 'sci,', '1966', 'a-14', '2041', '259', 'n', 'shavit', 'm', 'konigsbuch', 'a', 'oplaka', 'us', 'patent', '3345350', '260', 'n', 'shavit', 'm', 'konigsbuch', 'j', 'polymer', 'sci,', '1967', 'c(16)', '43', '261', 's', 'amdur', 'n', 'shavit', 'j', 'polymer', 'sci', '1967', 'a-15', '1297', '262', 'w', 'kern', 'h', 'fernow', 'rubber', 'chem', 'technol,', '1945', '18', '267', '263', 'le', 'ball', 'jl', 'greene', 'vol', '15', 'encyclopedia', 'polymer', 'science', 'technology', '(hf', 'mark', 'ng', 'gaylord', 'nm', 'bikales', 'eds)', 'wiley-interscience', 'new', 'york', 'ny', '1971', '264', 'rg', 'beaman', 'j', 'am', 'chem', 'soc', '1948', '70', '3115', '265', 'oh', 'bullitt', 'jr,', 'us', 'patent', '2608555', '(1952)', '266', 'cg', 'overberger', 'em', 'pearce', 'n', 'mayers', 'j', 'polymer', 'sci,', '1948', '34', '109', '267', 'cg', 'overberger', 'h', 'yuki', 'n', 'urakawa', 'j', 'polymer', 'sci1960,', '45', '127', '268', 'wk', 'wilkinson', 'us', 'patent', '3087919', '(1963)', '269', 'ben-ami', 'feit', 'e', 'heller', 'a', 'zilkha', 'j', 'polymer', 'sci', '1966', 'a-14', '1151', '270', 'wm', 'thomas', 'dw', 'wang', 'acrylamide', 'polymers', 'encyclopedia', 'polymer', 'science', 'engineering', 'vol1', 'hf', 'mark', 'nm', 'bikalis', 'cg', 'overberger', 'g', 'menges', 'eds,', 'wiley-', 'interscience', 'new', 'york', '1985', '271', 'k', 'butler', 'pr', 'thomas', 'gj', 'tyler', 'j', 'polymer', 'sci', '1960', '48', '357', '272', 'jf', 'jones', 'j', 'polymer', 'sci,', '1958', '33', '15', '273', 's', 'boyer', 'a', 'rondeau', 'bull', 'soc', 'chim', 'france', '5', '25', '274', 'lar', 'hall', 'wj', 'belanger', 'w', 'kirk', 'jr,', 'ya', 'sundatrom', 'j', 'appl', 'polymer', 'sci,', '1959', '2', '246', '275', 'sv', 'gangal', '“tetraﬂuoroethylene', 'polymers”', 'encyclopedia', 'polymer', 'science', 'engineering', 'vol', '16', '(hf', 'mark', 'nm', 'bikalis', 'cg', 'overberger', 'g', 'menges', 'eds)', 'wiley-interscience', 'new', 'york', '1989', '276', 'r', 'kiyama', 'j', 'osugi', 's', 'kusuhara', 'rev', 'phys', 'chem,', 'japan', '1957', '2722', '277', 're', 'moynihan', 'j', 'am', 'chem', 'soc', '1957', '81', '1045', '278', 'klberry', 'jh', 'peterson', 'j', 'am', 'chem', 'soc', '1951', '73', '5195', '279', 'et', 'mcbee', 'hm', 'hill', 'gb', 'bachman', 'ind', 'eng', 'chem,', '1949', '41', '70', '280', 'le', 'robb', 'fj', 'honn', 'dr', 'wolf', 'rubber', 'age', '1957', '82', '286', '281', 'js', 'rugg', 'ac', 'stevenson', 'rubber', 'age', '1957', '82', '102', '282', 'l', 'mandelkern', 'gm', 'martin', 'fa', 'quinn', 'j', 'res', 'nat', 'bur', 'stand,', '1957', '58', '137', '283', 'cbgrifﬁns', 'jc', 'montermoso', 'rubber', 'age', '1959', '77', '559', '284', 'jf', 'smith', 'rubber', 'world', '1960', '142(3)', '102', '285', 'jfsmith', 'gt', 'perkins', 'j', 'appl', 'polymer', 'sci,', '1961', '5(16)', '460', '286', 'kl', 'paciorek', 'et', 'al,', 'j', 'polymer', 'sci,', '1960', '45', '405', '287', 'rj', 'orr', 'hl', 'williams', 'can', 'j', 'chem,', '1955', '33', '1328', '288', 'j', 'ulbricht', 'r', 'sourisseau', 'faserforsch', 'textiltech,', '1955', '12', '547', '289', 'f', 'bergmann', 'a', 'kalmus', 'e', 'breuer', 'j', 'am', 'chem', 'soc,', '1958', '20', '4540', '290', 'di', 'livingston', 'pm', 'kamath', 'rs', 'corley', 'j', 'polymer', 'sci,', '1956', '20', '485', '291', 'd', 'sianeai', 'p', 'carradini', 'j', 'polymer', 'sci,', '1959', '43', '531', '292', 'g', 'talanini', 'e', 'peggion', 'chapt', '5', 'vinyl', 'polymerization', '(g', 'ham', 'ed)', 'dekker', 'new', 'york', 'ny', '1967', 'j', 'lewis', 'pe', 'okieimen', 'gs', 'park', 'j', 'macromol', 'sci-chem,', '(1982', 'a17', '1021)', '293', 'wi', 'bengough', 'rgw', 'norrish', 'proc', 'roy', 'soc', '(london)', '1950', 'a', '200', '301', '294', 'a', 'schindler', 'jw', 'breitenbach', 'ric', 'sci', 'suppl,', '1955', '25', '34', '295', 'ej', 'alerman', 'wm', 'wagner', 'j', 'polymer', 'sci,', '1951', '9', '541', '296', 'k', 'nozaki', 'discuss', 'faraday', 'soc,', '1947', '2', '337', '297', 'kj', 'mead', 'rm', 'fuoss', 'j', 'am', 'chem', 'soc,', '1942', '64', '277', '298', 'ch', 'bamford', 'wg', 'barb', 'ad', 'jenkins', 'pf', 'onyon', 'the', 'kinetics', 'vinyl', 'polymerization', 'free-radical', 'mechanism', 'butterworth', 'london', '1954', '402', '6', 'common', 'chain-growth', 'polymers', '299', 't', 'hjertberg', 'e', 'sorvik', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1981', '19', '363', '300', 'a', 'caraculacu', 'ec', 'burniana', 'g', 'robila', 'j', 'polymer', 'sci', 'polymer', 'chem,', 'ed,', '1978', '16', '2741', '301', 'g', 'vancso', '0', 'egyed', 's', 'pekker', 'a', 'janossy', 'polymer', '1982', '23', '14', '302', 'kh', 'reichert', 'hu', 'moritz', 'j', 'appi', 'polymer', 'sci,', '1981', '36', '151', '303', 'l', 'petiaud', 'o', 'pham', 'makromol', 'chem,', '1977', '78', '741', '304', 'jwl', 'fordham', 'ph', 'burleigh', 'cl', 'sturm', 'j', 'polymer', 'sci,', '1959', '41', '73', '305', 'u', 'giannini', 's', 'cesca', 'chim', 'ind', '(milan)', '1962', '44', '371', '306', 'vg', 'gason-zade', 'vv', 'mazurek', 'vp', 'sklizkova', 'vysokomol', 'soedin,', '1968', 'a10(3)', '479', '307', 'a', 'gyot', 'pham-qung-tho', 'j', 'chim', 'phys,', '1966', '63', '742', '308', 'a', 'gyot', 'dl', 'trung', 'r', 'ribould', 'cr', 'acad', 'sci,', '(paris)', '266', '1968', 'c', '1139', '309', 'v', 'jisova', 'm', 'kolinsky', 'd', 'lim', 'j', 'polymer', 'sci,', '1970', 'a-18', '1525', '310', 'n', 'yamazaki', 'k', 'sasaki', 's', 'kambara', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1964', '2', '487', '311', 'a', 'guyot', 'pham', 'aung', 'tho', 'j', 'polymer', 'sci,', '1964', 'c4', '299', '312', 'lf', 'albright', 'chem', 'eng,', '85', '(july', '3', '1967)', '313', 'cs', 'fuller', 'chem', 'rev,', '1940', '26', '162', '314', 'kk', 'kikukawa', 's', 'nozakura', 's', 'murahashi', 'kobunshi', 'kagaku', '1964', '25', '19', '(from', 'chem', 'abstr)', '1968', '69', '1968x', '315', 'k', 'noro', 'chapt', '6', 'polyvinyl', 'alcohol', 'ca', 'finch', 'ed,', 'wiley', 'new', 'york', 'ny,', '1973', '316', 'ja', 'brydson', 'plastic', 'materials', 'van', 'norstrand', 'london', '1966', '317', 'cs', 'fuller', 'chem', 'rev,', '1940', '26', '162', '318', 'g', 'natta', 'p', 'garradini', 'j', 'polymer', 'sd,', '1956', '20262', '319', 'l', 'e', 'n', 'allan', 'e', 'd', 'cross', 't', 'w', 'francjs-pranger', 'm', 'e', 'hanhan', 'm', 'r', 'jones', 'j', 'k', 'pearson', 'm', 'r', 'perry', 't', 'storr', 'm', 'p', 'shaver', 'macromolecules', '2011', '44', '(11)', '4072', 'chapter', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '71', 'mechanism', 'kinetics', 'step-growth', 'polymerization', 'two', 'types', 'monomers', 'undergo', 'step-growth', 'polymerizations', '1', '4', '5.', 'both', 'polyfunctional', 'one', 'type', 'possesses', 'one', 'kind', 'functionality', 'an', 'example', 'adipic', 'acid', 'two', 'functional', 'groups', 'carboxylic', 'acid', 'groups', 'another', 'one', 'hexamethylene', 'diamine', 'two', 'amine', 'functional', 'groups', 'to', 'second', 'type', 'belong', 'monomers', 'functional', 'groups', 'needed', 'condensation', 'molecule', 'an', 'example', 'monomer', 'p-amino', 'benzoic', 'acid', 'amino', 'carboxylic', 'acid', 'groups', 'present', 'molecule', 'chain', 'growths', 'proceed', 'reactions', 'two', 'different', 'functional', 'groups', 'types', 'molecules', 'an', 'exception', 'formation', 'polyanhydrides', 'polymeric', 'chains', 'formed', 'two', 'carboxylic', 'acid', 'groups', 'reacting', 'splitting', 'water', 'there', 'exceptions', 'well', '711', 'reactions', 'functional', 'groups', 'kinetic', 'considerations', 'paramount', 'importance', 'understanding', 'mechanism', 'step-growth', 'polymerization', '1.', 'as', 'stated', 'chap', '1', 'chain-growth', 'polymerizations', 'take', 'place', 'discreet', 'steps', 'each', 'step', 'reaction', 'two', 'functional', 'groups', 'like', 'instance', 'polyesteriﬁcation', 'reaction', 'reaction', '–cooh', '–oh', 'the', 'increase', 'molecular', 'weight', 'slow', 'the', 'ﬁrst', 'step', 'condensation', 'two', 'monomers', 'form', 'dimer', 'o', 'r', 'o', 'oh', 'oh', 'ho', 'r', 'oh', 'o', 'r', 'o', 'ho', 'o', 'oh', 'r', 'h2o', 'a', 'dimer', 'react', 'next', 'another', 'monomer', 'form', 'trimer', 'o', 'r', 'o', 'o', 'r', 'o', 'o', 'r', 'o', 'oh', 'o', 'oh', 'r', 'ho', 'oh', 'o', 'o', 'r', 'oh', 'ho', 'r', 'o', 'o', 'h2o', '403', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-97', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '404', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'two', 'dimers', 'combine', 'form', 'tetramer', 'o', 'r', 'o', 'oh', 'o', 'oh', 'r', 'o', 'r', 'o', 'o', 'r', 'o', 'oh', 'r', 'r', 'oh', 'o', 'o', 'o', 'these', 'step', 'condensations', 'continue', 'slowly', 'molecular', 'weights', 'polymers', 'increasing', 'step', 'in', 'reactions', 'monomeric', 'species', 'disappear', 'early', 'reaction', 'mixtures', 'long', 'large', 'molecular', 'weight', 'species', 'develop', 'in', 'step-growth', 'polymerizations', 'weight', 'basis', 'less', '1', 'monomeric', 'species', 'remain', 'time', 'average', 'chain', 'length', 'attains', 'size', 'ten', 'combined', 'monomeric', 'units', '1', '3', '4', '6.', 'one', 'important', 'characteristic', 'step-growth', 'polymerizations', 'functional', 'group', 'one', 'molecule', 'capable', 'reacting', 'opposite', 'functional', 'group', 'molecule', 'thus', 'instance', 'reaction', 'polyesteriﬁcation', 'carboxylic', 'acid', 'group', 'one', 'molecule', 'regardless', 'size', 'react', 'hydroxy', 'another', 'one', 'this', 'true', 'step-growth', 'polymerizations', 'it', 'means', 'rates', 'step-growth', 'polymerizations', 'sums', 'rates', 'reactions', 'molecules', 'various', 'sizes', 'a', 'useful', 'assumption', 'applied', 'reactivities', 'functional', 'groups', 'remain', 'throughout', 'reaction', 'regardless', 'size', 'molecules', 'attached', 'this', 'allows', 'treating', 'step-growth', 'polymerizations', 'like', 'reactions', 'small', 'molecules', 'general', 'observations', 'would', 'suggest', 'slower', 'reactivity', 'functional', 'groups', 'attached', 'large', 'molecules', 'this', 'however', 'usually', 'due', 'lower', 'diffusion', 'rates', 'large', 'molecules', 'the', 'actual', 'reactivity', 'functional', 'groups', 'depends', 'upon', 'collision', 'frequencies', '(number', 'collisions', 'per', 'unit', 'time)', 'groups', 'upon', 'rate', 'diffusion', 'functional', 'groups', 'terminal', 'ends', 'large', 'molecules', 'greater', 'mobility', 'remaining', 'portions', 'molecules', 'whole', 'in', 'addition', 'reactivity', 'one', 'given', 'functional', 'group', 'bifunctional', 'molecule', 'altered', 'reaction', 'group', '(if', 'neighboring', 'group', 'effect)', 'this', 'implies', 'reactivities', 'functional', 'groups', 'altered', 'polymerization', 'the', 'kinetics', 'step-growth', 'polymerization', 'derived', 'polyesteriﬁcation', 'reaction', 'follows', 'course', 'acid-catalyzed', 'esteriﬁcations', '2.', '1', 'protonation', 'step', 'o', 'oh', 'ha', 'k1', 'k2', 'oh', 'oh', 'a', '2', 'reaction', 'protonated', 'carboxylic', 'acid', 'group', 'alcohol', 'oh', 'oh', 'k3', 'k4', 'oh', 'o', 'o', 'h2o', 'h', 'oh', 'oh', 'o', 'h', 'k5', 'the', 'polyesteriﬁcations', 'like', 'many', 'reactions', 'equilibrium', 'reactions', 'they', 'must', 'conducted', 'way', 'allows', 'equilibrium', 'shift', 'right', 'attain', 'high', 'molecular', 'weights', '71', 'mechanism', 'kinetics', 'step-growth', 'polymerization', '405', 'one', 'way', 'continual', 'removal', 'by-products', 'in', 'situations', 'reactions', 'take', 'place', 'nonequilibrium', 'conditions', 'k4', '712', 'kinetic', 'considerations', 'the', 'rate', 'polymerization', 'expressed', 'rate', 'disappearance', 'one', 'functional', 'groups', 'in', 'reactions', 'polyesteriﬁcation', 'rate', 'disappearance', 'carboxyl', 'groups', '(cid1)dco2h/dt', 'rp', '¼', '(cid1)d½co2h(cid3)', 'dt', '¼', 'k3½cðoh(cid4)þ2(cid3)½oh(cid3)', 'in', 'equation', 'co2h,', 'oh,', '½cðoh(cid4)þ2(cid3)', 'represent', 'carboxyl', 'hydroxy', 'protonated', 'carboxyl', 'groups', 'respectively', 'also', 'possible', 'write', 'equilibrium', 'expression', 'proton-', 'ation', 'reaction', 'acid', 'follows', 'k', '¼', '¼', 'k1', 'k2', '½cðoh(cid4)þ2(cid3)½a(cid5)(cid3)', '½cooh(cid3)½ha(cid3)', 'this', 'equation', 'combined', 'rate', 'expression', '(cid1)d½cooh(cid3)', 'dt', '¼', 'k1k3½cooh(cid3)½oh(cid3)½ha(cid3)', 'k2½a(cid5)(cid3)', '¼', 'k3k½cooh(cid3)½oh(cid3)½ha(cid3)', 'if', 'catalyst', 'present', 'dicarboxylic', 'acid', 'acts', 'catalyst', 'ha', 'replaced', 'cooh', 'expression', 'becomes', '(cid1)d½cooh(cid3)', 'dt', '¼', 'k½cooh(cid3)2½oh(cid3)', 'in', 'expression', 'k1', 'k2', 'k3', 'concentration', '½a(cid5)(cid3)', 'ions', 'replaced', 'experimentally', 'determined', 'rate', 'constant', 'k', 'in', 'step-growth', 'polymerization', 'reactions', 'concentrations', 'two', 'functional', 'groups', 'close', 'stoichiometric', 'this', 'allows', 'writing', 'rate', 'equation', 'follows', 'in', 'equation', 'm', 'represents', 'concentration', 'reacting', 'species', 'they', 'hydroxy', 'carboxylic', 'acid', 'groups', 'polyesteriﬁcation', 'reaction', 'amino', 'carboxylic', 'acid', 'groups', 'polyamidation', 'reaction', 'the', 'equation', 'also', 'written', 'follows', 'integrating', 'get', '(cid1)d½m(cid3)', 'dt', '¼', 'k½m(cid3)3', 'k', 'dt', '¼', '(cid1)d½m(cid3)½m(cid3)3', '2kt', '¼', '1½m(cid3)2', 'þ', 'constant', '406', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'the', 'constant', 'equation', 'equals', '1[mo2', 'mo', 'represents', 'initial', 'concentration', 'reactants', '(of', 'hydroxyl', 'carboxyl', 'groups', 'polyesteriﬁcation)', 'time', '¼', '0', 'at', 'start', 'polymerization', 'mo', 'molecules', 'present', 'after', 'progress', 'reaction', 'm', 'molecules', 'left', 'mo', '(cid1)', 'm', 'number', 'molecules', 'participated', 'formation', 'polymeric', 'chains', 'the', 'conversion', 'p', 'written', 'according', 'carothers', '6,', 'p', '¼', '½mo(cid3)', '(cid1)', '½m(cid3)', '½mo(cid3)', '½m(cid3)', '¼', '½mo(cid3)ð1', '(cid1)', 'pþ', 'dp', '¼', '1', '1', '(cid1)', 'p', 'dp', '¼', '½mo(cid3)', '½m(cid3)', '½m(cid3)', '¼', '½mo(cid3)ð1', '(cid1)', 'pþ', 'concentration', 'm', 'given', 'time', 'degree', 'polymerization', 'it', 'important', 'realize', 'equation', 'order', 'dp', '50', 'necessary', 'achieve', '98', 'conversion', '(p', 'must', 'equal', '098)', 'the', 'value', 'dp', 'given', 'time', 'equal', 'ratio', 'monomer', 'molecules', 'present', 'start', 'reaction', 'divided', 'number', 'molecules', 'still', 'present', 'particular', 'time', 'by', 'combining', 'expression', 'caruthers', 'equation', 'solving', 'm,', 'one', 'obtains', 'for', 'second-order', 'rate', 'expression', 'equation', 'written', 'replacing', '1ð1', '(cid1)', 'pþ', 'dp', 'one', 'obtains', '1f½mo(cid3)ð1', '(cid1)', 'pþg', '(cid1)', '1½mo(cid3)', '¼', 'kt', 'dp', '¼', '½mo(cid3)kt', 'þ', '1', 'using', 'equation', 'possible', 'calculate', 'rate', 'constant', '(if', 'known)', 'concentration', 'monomers', 'time', 'required', 'reach', 'desired', 'number', 'average', 'molecular', 'weight', 'when', 'catalyst', 'present', 'carboxylic', 'acid', 'assumes', 'role', 'catalyst', 'third-order', 'rate', 'expression', '(shown', 'above)', 'must', 'employed', 'by', 'integrating', 'third-order', 'rate', 'expression', 'one', 'obtains', '(cid1)', 'd½m(cid3)dt', '¼', 'k½m(cid3)3', '1½m(cid3)2', '(cid1)', '1½mo(cid3)2', '¼', '2kt', '71', 'mechanism', 'kinetics', 'step-growth', 'polymerization', '407', 'substituting', 'm', 'carothers', 'equation', 'rearranging', 'resultant', 'equation', 'one', 'obtains', 'also', 'written', '1', '½mo(cid3)2ð1', '(cid1)', 'pþ2', '(cid1)', '1', '½mo(cid3)2', '¼', '2kt', '1ð1', '(cid1)', 'pþ2', '¼', '2kt½mo(cid3)2', 'þ', '1', '2', 'dp', '¼', '2kt½mo(cid3)2', 'þ', '1', 'the', 'equation', 'shows', 'without', 'catalyst', 'molecular', 'weight', 'increases', 'gradually', 'it', 'deduced', 'discussion', 'high', 'stoichiometric', 'balance', 'essential', 'attaining', 'high', 'molecular', 'weight', 'this', 'means', 'presence', 'monofunctional', 'impurity', 'strong', 'limiting', 'effect', 'molecular', 'weight', 'product', 'the', 'impurity', 'blocks', 'one', 'end', 'chain', 'reacting', 'this', 'useful', 'however', 'required', 'limit', 'dp', 'product', 'for', 'instance', 'small', 'quantities', 'acetic', 'acid', 'sometimes', 'added', 'preparations', 'polyamides', 'limit', 'molecular', 'weight', 'a', 'a', 'type', 'no', 'in', 'polymerizations', 'monomers', 'functional', 'groups', 'molecule', 'like', 'a', '——', 'a', 'b', '——', 'b', '(ie,', 'diamine', 'diacid)', 'number', 'functional', 'groups', 'present', 'designated', 'no', 'b', 'represent', 'number', 'functional', 'groups', 'present', 'start', 'reaction', 'they', 'twice', 'number', 'a', '——', 'a', 'b', '——', 'b', 'molecules', 'present', 'if', 'number', 'no', 'b', 'slightly', 'larger', 'no', 'a', 'stoichiometric', 'imbalance', 'reaction', 'mixture', 'this', 'imbalance', 'designated', 'r', 'b', 'b', 'type', 'these', 'numbers', 'no', 'a', 'no', 'r', '¼', 'no', 'ano', 'b', 'a', 'þ', 'no', '(it', 'common', 'deﬁne', 'ratio', 'r', 'less', 'equal', 'unity', 'b', 'groups', 'bþ2', 'present', 'excess)', 'the', 'total', 'number', 'monomers', 'start', 'reaction', 'ðno', 'no', 'að1', 'þ', 'ð1rþþ2', 'the', 'extent', 'reaction', 'p', 'deﬁned', 'portion', 'functional', 'groups', 'a', 'reacted', 'given', 'time', 'the', 'portion', 'functional', 'groups', 'b', 'reacted', 'time', 'designated', 'rp', 'the', 'unreacted', 'portions', 'a', 'b', 'groups', 'designated', '1', '(cid1)', 'p', '1', '(cid1)', 'rp', 'respectively', 'the', 'total', 'number', 'unreacted', 'a', 'groups', 'reaction', 'mixture', 'would', 'no', 'að1', '(cid1)', 'pþ', 'this', 'reaction', 'mixture', 'also', 'contains', 'no', 'bð1', '(cid1)', 'rpþ', 'unreacted', 'b', 'groups', 'the', 'total', 'number', 'chain', 'ends', 'polymer', 'molecules', 'sum', 'unreacted', 'a', 'b', 'groups', 'because', 'polymer', 'molecule', '½no', 'að1', '(cid1)', 'pþ', 'þ', 'no', 'the', 'number', 'average', 'degree', 'polymerization', 'equal', 'total', 'number', 'a', '——', 'a', 'b', '——', 'b', 'molecules', 'present', 'start', 'reaction', 'divided', 'number', 'polymer', 'molecules', 'end', 'this', 'represented', 'follows', 'two', 'chain', 'ends', 'total', 'number', 'bð1', '(cid1)', 'rpþ(cid3)2', 'chain', 'ends', 'dpn', '¼', 'að1', 'þ', '1rþ2', 'no', 'að1', '(cid1)', 'pþ', 'þ', 'no', '½no', 'bð1', '(cid1)', 'rpþ(cid3)2', 'expression', 'reduced', '(since', 'r', '¼', 'no', 'ano', 'b)', '408', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'the', 'molecular', 'weight', 'product', 'controlled', 'precise', 'stoichiometry', 'polymeri-', 'zation', 'reaction', 'this', 'done', 'simply', 'quenching', 'reaction', 'mixture', 'speciﬁed', 'time', 'desired', 'molecular', 'weight', 'achieved', 'flory', 'derived', 'statistical', 'methods', 'relating', 'molecular', 'weight', 'distribution', 'degree', 'conversion', '1', '3.', 'in', 'polymerizations', 'reaction', 'step', 'links', 'two', 'monomer', 'molecules', 'together', 'this', 'means', 'number', 'mers', 'polymer', 'backbone', 'always', 'larger', 'one', 'number', 'kind', 'functional', 'groups', 'a', 'b', 'if', 'x', 'monomers', 'chain', 'number', 'functional', 'groups', 'reacted', 'x', '(cid1)', '1', 'the', 'functional', 'groups', 'unreacted', 'remain', 'ends', 'chains', 'if', 'designate', 'p', 'extent', 'reaction', 'degree', 'conversion', 'probability', 'x', '(cid1)', '1', 'a', 'b', 'reacted', 'px', '(cid1)', '1', 'probability', 'ﬁnding', 'unreacted', 'functional', 'group', 'p', '(cid1)', '1', 'the', 'probability', 'ﬁnding', 'polymer', 'molecule', 'contains', 'x', 'monomer', 'units', 'unreacted', 'functional', 'group', 'a', 'b', 'px(cid1)1ð1', '(cid1)', 'pþ', 'at', 'given', 'time', 'number', 'molecules', 'present', 'reaction', 'mixture', 'n', 'the', 'fraction', 'contains', 'x', 'units', 'designated', 'nx', 'deﬁned', 'the', 'carothers', 'equation', 'deﬁnes', 'nno', '¼', '1', '(cid1)', 'p', 'the', 'expression', 'nx', 'therefore', 'written', 'dp', '¼', '1', 'þ', 'r', '2', 'þ', 'r', '(cid1)', '2rp', 'p', '¼', 'ðno', '(cid1)', 'nþno', 'nx', '¼', 'npx(cid1)1ð1', '(cid1)', 'pþ', 'nx', '¼', 'noð1', '(cid1)', 'pþ2px(cid1)1', 'no', 'course', 'number', 'monomer', 'units', 'present', 'start', 'reaction', 'to', 'determine', 'molecular', 'weight', 'distribution', 'polymeric', 'species', 'form', 'given', 'degree', 'conversion', 'desirable', 'express', 'weight', 'average', 'number', 'average', 'molecular', 'weights', 'terms', 'like', 'p', 'by', 'deﬁning', 'mo', 'mass', 'repeating', 'unit', 'number', 'average', 'molecular', 'weight', 'weight', 'average', 'molecular', 'weight', 'mn', '¼', 'modp', '¼', 'moð11', '(cid1)', 'pþ', 'mw', '¼', 'wxxmo', 'x', 'wx', 'weight', 'fraction', 'molecules', 'containing', 'x', 'monomer', 'units', 'it', 'equal', 'xnxno', 'written', 'based', 'equation', 'nx', 'wx', '¼', 'xð1', '(cid1)', 'pþ2px(cid1)1', 'the', 'weight', 'average', 'molecular', 'weight', 'expressed', 'it', 'shown', 'summation', 'x2px(cid1)1', '¼', 'ð1', 'þ', 'pþð1', '(cid1)', 'pþ3', 'based', 'weight', 'average', 'molecular', 'weight', 'p', 'mw', '¼', 'moð1', '(cid1)', 'pþ2', 'x2px(cid1)1', 'x', 'mw', '¼', 'moð1', 'þ', 'pþð1', '(cid1)', 'pþ', '71', 'mechanism', 'kinetics', 'step-growth', 'polymerization', '409', 'molecular', 'weight', 'distribution', 'mwmn', '¼', '1', 'þ', 'p', 'it', 'interesting', 'equation', 'tells', 'us', 'high', 'conversion', 'p', 'approaches', '1', 'molecular', 'weight', 'distribution', 'approaches', '2', 'there', 'experimental', 'conﬁrmation', 'until', 'discussion', 'concerned', 'formation', 'linear', 'polymers', 'the', 'presence', 'however', 'monomers', 'two', 'functional', 'groups', 'results', 'formation', 'branched', 'structures', 'an', 'example', 'preparation', 'polyester', 'dicarboxylic', 'acid', 'glycol', 'reaction', 'mixture', 'also', 'contains', 'glycerol', 'chain', 'growth', 'polymerization', 'restricted', 'two', 'directions', 'products', 'much', 'complex', 'this', 'illustrated', 'trifunctional', 'molecule', 'condensing', 'difunctional', 'one', 'na', 'bbn', 'a', 'a', 'a', 'a', 'ba', 'b', 'a', 'b', 'ab', 'a', 'b', 'b', 'a', 'a', 'a', 'further', 'growth', 'course', 'possible', 'every', 'unreacted', 'functional', 'group', 'lead', 'gelation', 'the', 'onset', 'gelation', 'predicted', 'modiﬁed', 'form', 'carothers', 'equation', '1.', 'this', 'equation', 'includes', 'average', 'functionality', 'factor', 'averages', 'functionality', 'functional', 'groups', 'involved', 'an', 'example', 'reaction', 'mixture', 'difunctional', 'monomers', 'trifunctional', 'ones', 'added', 'branching', 'cross-linking', 'the', 'average', 'functionality', 'fave', 'may', '(2', '2', '2', '3)4', '¼', '225', 'the', 'carothers', 'equation', 'discussed', 'states', 'no', 'n', 'represent', 'quantities', 'monomer', 'molecules', 'present', 'initially', 'conversion', 'point', 'p', 'the', 'number', 'functional', 'groups', 'reacted', 'point', '2(no', '(cid1)', 'n)', 'in', 'modiﬁed', 'equation', 'number', 'molecules', 'present', 'initially', 'nofave', 'the', 'equation', 'becomes', 'non', 'replaced', 'dp', 'expression', 'becomes', 'it', 'generally', 'accepted', 'gelation', 'occurs', 'average', 'degree', 'polymerization', 'becomes', 'inﬁnite', 'at', 'point', 'second', 'term', 'equation', 'becomes', 'zero', 'when', 'occurs', 'conversion', 'term', 'becomes', 'pc', 'it', 'critical', 'reaction', 'conversion', 'point', 'gelation', 'however', 'less', 'likely', 'major', 'concern', 'polymerization', 'reactions', 'small', 'quantities', 'tri-', 'multifunctional', 'monomers', 'present', 'in', 'preparation', 'alkyds', 'instance', '(described', 'chapter)', 'glycerin', 'trifunctional', 'usually', 'present', 'p', '¼', 'ðno', '(cid1)', 'nþno', 'p', '¼', '2ðno', '(cid1)', 'nþ', 'nofave', '(cid1)p', '¼', '2fave', '(cid1)', '2dpfave', 'pc', '¼', '2fave', '410', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'if', 'amount', 'glycerin', 'small', 'product', 'branched', 'in', 'addition', 'might', 'one', 'branch', 'per', 'molecule', 'a', 'bb', 'ba', 'ab', 'ba', 'ab', 'ba', 'ab', 'ba', 'ab', 'ba', 'statistical', 'methods', 'developed', 'prediction', 'gelation', '7.', 'these', 'actually', 'predict', 'gelation', 'lower', 'level', 'carothers', 'equation', 'shown', 'as', 'example', 'use', 'reaction', 'three', 'monomers', 'a', 'b', 'c', 'we', 'assume', 'functionality', 'two', 'monomers', 'fa', 'fb', 'equal', 'two', 'fc', 'greater', 'two', 'the', 'critical', 'reaction', 'conversion', 'written', 'pc', '¼', '1½r', 'þ', 'rpðfc', '(cid1)', '2þ(cid3)12', '713', 'ring', 'formation', 'step-growth', 'polymerizations', 'step', 'growth', 'polymerizations', 'complicated', 'cyclization', 'reactions', 'accompany', 'formations', 'linear', 'polymers', 'such', 'ring', 'formations', 'occur', 'reactions', 'monomers', 'either', 'type', 'functional', 'groups', 'different', 'ones', 'following', 'illustrations', 'cyclization', 'reactions', 'r', 'h2n', 'cooh', 'o', 'h2o', 'b', 'a', 'a', 'b', 'b', 'o', 'h', 'h', 'r', 'n', 'r', 'x', 'h2n', 'cooh', 'x', 'nh', 'cooh', 'r', '2', 'ho', 'cooh', 'o', 'o', 'or', 'o', 'r', 'ho', 'x', 'cooh', 'x-1', 'o', 'o', 'similarly', 'dicarboxylic', 'acids', 'cyclize', 'anhydrides', 'whether', 'ring', 'formation', 'likely', 'take', 'place', 'depends', 'upon', 'size', 'ring', 'form', 'if', 'cyclization', 'results', 'rings', 'strained', 'bond', 'angles', 'repulsions', 'due', 'crowding', 'probability', 'formation', 'low', 'so', 'small', 'rings', 'less', 'ﬁve', 'members', 'form', 'readily', 'five-membered', 'rings', 'however', 'essentially', 'free', 'bond', 'angle', 'distortion', 'greater', '71', 'mechanism', 'kinetics', 'step-growth', 'polymerization', '411', 'chance', 'form', 'greater', 'ﬁve-membered', 'rings', 'planar', 'six-', 'seven-membered', 'rings', 'also', 'form', 'freely', 'though', 'quite', 'easily', 'ﬁve-membered', 'ones', 'six-membered', 'rings', 'favored', 'seven-membered', 'ones', 'rings', 'eight', 'twelve', 'members', 'relatively', 'strain-free', 'bond', 'angle', 'distortion', 'thermodynamically', 'unstable', 'this', 'substituents', '(hydrogens', 'others)', 'forced', 'positions', 'repulsion', 'due', 'crowding', 'also', 'little', 'ring', 'formation', 'eight', 'twelve', 'members', 'whether', 'cyclization', 'take', 'place', 'polymerization', 'also', 'depends', 'kinetic', 'feasibility', 'cyclize', 'this', 'feasibility', 'function', 'probability', 'functional', 'end', 'groups', 'molecule', 'approach', 'as', 'size', 'monomer', 'increases', 'size', 'potential', 'ring', 'an', 'increase', 'size', 'monomer', 'however', 'also', 'means', 'increase', 'number', 'different', 'conﬁgurations', 'monomer', 'molecule', 'assume', 'very', 'conﬁgurations', 'two', 'ends', 'become', 'adjacent', '4.', 'with', 'fewer', 'chances', 'end', 'groups', 'encountering', 'decreased', 'probability', 'ring', 'formation', 'from', 'practical', 'considerations', 'ring', 'formations', 'mainly', 'problem', 'ﬁve-', 'six-', 'seven-membered', 'rings', 'form', 'formation', 'large', 'rings', 'twelve', 'members', 'seldom', 'encountered', '5.', 'it', 'reported', 'least', 'one', 'instance', 'products', 'reaction', 'affected', 'isomer', 'used', '244.', 'thus', 'following', 'knoevenagel', 'condensation', 'polymer', 'form', 'ortho', 'isomer', 'reacted', 'h5c2', 'n', 'n', 'c2h5', 'h5c2', 'n', 'n', 'c2h5', 'cho', 'cho', 'cn', 'nc', 'nh', 'nh', 'o', 'o', 'ch', 'hc', 'cn', 'nc', 'o', 'nh', 'nh', 'n', 'o', 'by', 'contrast', 'meta', 'isomer', 'forms', 'ring', 'structures', 'instead', '242:', 'h5c2', 'n', 'n', 'c2h5', 'h5c2', 'n', 'n', 'c2h5', 'cho', 'cho', 'cn', 'nc', 'nh', 'nh', 'o', 'o', 'nc', 'cn', 'o', 'nh', 'o', 'nh', '412', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '714', 'techniques', 'polymer', 'preparation', 'many', 'step-growth', 'polymerizations', 'carried', 'mass', 'bulk', 'type', 'polymerization', 'this', 'widely', 'done', 'due', 'convenience', 'also', 'results', 'minimum', 'contamination', 'few', 'step-', 'growth', 'reactions', 'highly', 'exothermic', 'thermal', 'control', 'hard', 'maintain', 'because', 'equilibrium', 'considerations', 'important', 'reactions', 'usually', 'carried', 'way', 'allows', 'continuous', 'removal', 'by-product', 'occasionally', 'polymerizations', 'carried', 'dispersion', 'convenient', 'carriers', 'solution', 'polymerizations', 'sometimes', 'used', 'way', 'moderating', 'reactions', 'step-growth', 'polymerizations', 'also', 'carried', 'certain', 'monomers', 'low', 'temperature', 'technique', 'known', 'interfacial', 'polymerization', 'interfacial', 'polycondensation', '28.', 'the', 'reactions', '(applicable', 'fast', 'reactions)', 'conducted', 'interface', 'two', 'immiscible', 'liquids', 'usually', 'one', 'liquids', 'water', 'one', 'organic', 'solvent', 'an', 'example', 'may', 'schotten–baumann', 'polarization', 'reaction', 'in', 'interfacial', 'polymerization', 'diamine', 'would', 'aqueous', 'phase', 'diacid', 'chloride', 'organic', 'phase', 'the', 'strong', 'reactivity', 'acid', 'chloride', 'groups', 'amines', 'allows', 'reaction', 'carried', 'room', 'temperature', 'n', 'h2n', 'nh2', 'x', 'o', 'cl', 'o', 'cl', 'o', 'o', 'n', 'h', 'n', 'x', 'h', 'n', '2n', 'hcl', 'addition', 'base', 'aqueous', 'phase', 'removes', 'hydrochloric', 'acid', 'forms', 'catalyzes', 'reaction', 'the', 'choice', 'organic', 'solvent', 'important', 'appears', 'reaction', 'occurs', 'organic', 'side', 'interface', '28.', 'there', 'several', 'important', 'differences', 'interfacial', 'polymerizations', 'high', 'temperature', 'condensations', 'much', 'higher', 'molecular', 'weight', 'products', 'form', 'polymerizations', 'interface', 'this', 'probably', 'due', 'high', 'speed', 'reactions', 'diamines', 'diffuse', 'organic', 'phase', 'diacid', 'chloride', 'chain', 'ends', '28.', 'exact', 'stoichiometry', 'necessary', 'attain', 'high', 'molecular', 'weights', 'interfacial', 'polycondensation', 'the', 'opposite', 'true', 'high', 'temperature', 'polymerizations', 'interfacial', 'polycondensation', 'interesting', 'procedure', 'often', 'used', 'demonstrations', 'polymer', 'chemistry', 'courses', 'polyamides', 'prepared', 'rapidly', 'front', 'class', 'diacid', 'chlorides', 'diamines', 'the', 'products', 'removed', 'quickly', 'form', 'pulling', 'string', 'interface', '47.', 'polyesters', 'also', 'prepared', 'diacid', 'chlorides', 'bisphenols', 'on', 'hand', 'preparation', 'polyesters', 'glycols', 'diacid', 'chlorides', 'usually', 'unsuccessful', 'due', 'low', 'reactivity', 'dialcohols', 'the', 'diacid', 'chlorides', 'tend', 'undergo', 'hydrolysis', 'instead', 'commercially', 'proce-', 'dure', 'far', 'conﬁned', 'mainly', 'preparations', 'polycarbonates', '(discussed', 'chapter)', '72', 'polyesters', 'the', 'class', 'compounds', 'called', 'polyesters', 'consists', 'heterochain', 'macromolecular', 'compounds', 'possess', 'repeat', 'carboxylate', 'ester', 'groups', 'backbones', 'this', 'excludes', 'polymers', 'ester', 'groups', 'located', 'pendant', 'groups', 'like', 'acrylic', 'methacrylic', 'polymers', 'poly(vinyl', 'esters)', 'esters', 'cellulose', 'starch', 'what', 'remains', 'however', 'still', 'large', 'group', 'polymeric', 'materials', 'subdivided', 'saturated', 'unsaturated', 'polyesters', '721', 'linear', 'saturated', 'polyesters', 'the', 'saturated', 'polyesters', 'ﬁnd', 'commercial', 'applications', 'mostly', 'linear', 'except', 'specially', 'prepared', 'branched', 'polymers', 'used', 'preparation', 'polyurethanes', 'the', 'linear', 'polyesters', 'became', '72', 'polyesters', '413', 'commercially', 'important', 'materials', 'early', 'century', 'still', 'ﬁnd', 'many', 'uses', 'industry', 'the', 'earliest', 'studies', 'reported', 'condensations', 'ethylene', 'trimethylene', 'hexamethylene', 'decamethylene', 'glycols', 'malonic', 'succinic', 'adipic', 'sebacic', 'ortho', 'phthalic', 'acids', '6.', 'later', 'studies', 'showed', 'condensations', 'yield', 'high', 'molecular', 'weight', 'compounds', '44.', 'nevertheless', 'polyesters', 'exhibit', 'poor', 'hydrolytic', 'stability', 'generally', 'low', 'melting', 'subsequently', 'however', 'found', 'aromatic', 'dicarboxylic', 'acids', 'yield', 'polymers', 'high', 'melting', 'points', 'poly(ethylene', 'terephthalate)', 'melts', '265(cid6)c', 'important', 'commercial', 'material', 'physical', 'properties', 'linear', 'polyesters', 'follow', 'general', 'observation', 'relationships', 'physical', 'properties', 'chemical', 'structures', 'polymers', '(see', 'chap', '2)', 'aromatic', 'diacids', 'andor', 'glycols', 'aromatic', 'rings', 'structures', 'yield', 'polyesters', 'high', 'melting', 'points', 'aliphatic', 'ones', 'yield', 'low', 'melting', 'solids', 'viscous', 'liquids', 'in', 'addition', 'hydrogen', 'bonding', 'dipole', 'interactions', 'polarizations', 'stiff', 'interchain', 'bonds', 'molecular', 'symmetry', 'regularity', 'ability', 'polymeric', 'chains', 'undergo', 'close', 'packing', 'raise', 'melting', 'points', 'conversely', 'bulky', 'side', 'chains', 'ﬂexible', 'interchain', 'bonds', 'lower', 'melting', 'points', '7211', 'synthetic', 'methods', 'the', 'synthetic', 'methods', 'general', 'use', 'preparations', 'linear', 'polyesters', '15', 'summarized', 'follows', '1', 'dibasic', 'acids', 'reacted', 'glycols', 'stoichiometric', 'balance', 'strictly', 'maintained', 'when', 'low', 'boiling', 'glycols', 'used', 'however', 'often', 'added', 'slight', 'excess', 'excess', 'gradually', 'removed', 'vacuum', '6', 'sweeping', 'inert', 'gas', 'reaction', 'mixture', '6', '8.', 'this', 'procedure', 'useful', 'dicarboxylic', 'acids', 'otherwise', 'require', 'high', 'temperatures', 'strong', 'catalysts', 'running', 'reaction', 'high', 'temperatures', 'cause', 'glycols', 'condense', 'ethers', 'dicarboxylic', 'acids', 'decompose', '2', 'diesters', 'half', 'esters', 'dicarboxylic', 'acids', 'amine', 'salts', 'acids', 'reacted', 'glycols', '19:', 'o', 'o', 'n', 'r', 'o', 'r', 'o', 'r', 'r', 'n', 'ho', 'ho', 'o', 'o', 'o', 'r', 'o', 'r', 'n', 'roh', 'the', 'transesteriﬁcation', 'reaction', 'practical', 'use', 'high', 'melting', 'poorly', 'soluble', 'dicarboxylic', 'acids', 'in', 'addition', 'less', 'energy', 'needed', 'remove', 'alcohol', 'water', '3', 'hydroxy', 'acids', 'like', 'p-hydroxyethoxybezoic', 'acid', 'o-hydroxydecanoic', 'acid', 'capable', 'self-', 'condensation', 'form', 'polyesters', '10:', 'n', 'ho', '2', 'o', 'oh', 'o', '2', 'o', 'n', '414', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '4', 'although', 'practical', 'commercially', 'polyesters', 'prepared', 'laboratory', 'reacting', 'aliphatic', 'dibromides', 'silver', 'salts', 'dibasic', 'acids', '6:', 'o', 'o', 'r', 'br', 'br', 'ag', 'o', 'r', 'o', 'ag', 'o', 'o', 'r', 'o', 'r', 'o', 'n', '5', 'polyesters', 'also', 'form', 'reactions', 'dicarboxylic', 'acid', 'anhydrides', 'glycols', '10:', 'n', 'n', 'ho', 'r', 'oh', 'o', 'o', 'r', 'n', 'o', 'o', '6', 'glycol', 'carbonates', 'undergo', 'ester', 'interchange', 'reactions', 'dibasic', 'acids', '11:', 'n', 'o', 'r', 'o', 'o', 'n', 'oh', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', '2', 'r', 'n', 'n', 'h2co3', '7', 'ester', 'interchange', 'reactions', 'glycol', 'acetates', '12', '13', 'diphenol', 'diacetates', '12', 'dicarboxylic', 'acids', 'o', 'n', 'o', 'o', 'n', 'r', 'o', 'oh', 'oh', 'o', 'o', 'o', 'o', 'r', 'o', 'r', 'n', 'o', 'n', 'oh', '8', 'acid', 'chlorides', 'react', 'diphenols', 'form', 'polyesters', 'the', 'reaction', 'quite', 'efﬁcient', 'scavenger', 'hcl', 'added', 'aqueous', 'phase', 'such', 'scavengers', 'tertiary', 'amines', '15:', 'r', 'o', 'o', 'n', 'cl', 'cl', 'r', 'n', 'ho', 'oh', 'o', 'o', 'r', 'r', 'o', 'o', 'n', 'hcl', 'n', '72', 'polyesters', '415', 'when', 'place', 'dicarboxylic', 'acid', 'chlorides', 'phosgene', 'used', 'polycarbonates', 'form', 'ohn', 'r', 'ho', 'o', 'n', 'cl', 'cl', 'o', 'r', 'o', 'o', '2nhcl', 'n', '9', 'polyesters', 'also', 'form', 'ring', 'opening', 'polymerizations', 'lactones', 'this', 'discussed', 'chap', '5', 'many', 'modiﬁcations', 'above-described', 'reactions', 'known', 'for', 'instance', 'poly(propylene', 'phthalate)', 'prepared', 'phthalic', 'anhydride', 'propylene', 'oxide', '14.', 'the', 'reaction', 'catalyzed', 'tertiary', 'amines', 'probably', 'form', 'carboxylate', 'ion', 'intermediates', 'o', 'o', 'o', 'r3n', 'o', 'r3', 'o', 'o', 'n', 'o', 'o', 'o', 'r3', 'n', 'o', 'o', 'o', 'o', 'r3', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'this', 'produces', 'low', 'molecular', 'weight', 'polyester', 'a', 'modiﬁcation', 'reaction', 'dianhydride', 'glycol', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'rho', 'oh', 'o', 'o', 'ho', 'o', 'o', 'o', 'o', 'r', 'o', 'oh', 'ho', 'n', 'o', 'o', 'o', 'o', 'oh', 'o', 'o', 'r', 'in', 'esteriﬁcation', 'reactions', 'catalysts', 'increase', 'speed', 'condensations', 'such', 'catalysts', 'either', 'acids', 'bases', '416', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'beyond', 'many', 'polyester', 'syntheses', 'found', 'literature', 'common', 'use', 'for', 'instance', 'polyesters', 'form', 'additions', 'carboxylic', 'acids', 'divinyl', 'ethers', '29:', 'o', 'n', 'r', 'o', 'oh', 'oh', 'n', 'o', 'o', 'r', 'o', 'r', 'o', 'o', 'o', 'o', 'r', 'o', 'n', 'polyesters', 'also', 'form', 'tischenko', 'reaction', 'dialdehydes', '26', '27.', 'the', 'intramolecular', 'hydride', 'transfer', 'reaction', 'typically', 'catalyzed', 'bases', 'o', 'r', 'o', 'c4h9li', 'o', 'o', 'r', 'n', 'another', 'reported', 'procedure', 'consists', 'condensations', 'nitriles', 'glycols', 'the', 'resultant', 'poly', '(iminoether', 'hydroxide)s', 'hydrolyze', 'polyesters', '32.', 'r', 'n', 'nc', 'cn', 'n', 'ho', 'r', 'oh', 'hcl', 'h2o', 'carbon', 'suboxide', 'condenses', 'glycols', 'form', 'polyesters', '26:', 'o', 'n', 'o', 'r', 'n', 'ho', 'oh', 'nh2cl', 'nh2cl', 'r', 'o', 'o', 'o', 'r', 'r', 'r', 'o', 'o', 'n', 'o', 'n', 'o', 'o', 'r', 'o', 'o', 'n', 'o', 'o', 'n', 'also', 'alternating', 'free-radical', 'addition', 'copolymerization', 'cyclohexene', 'formic', 'acid', 'perhaps', 'via', 'charge', 'transfer', 'donor–acceptor', 'complexes', 'yields', 'polyesters', '30:', 'n', 'n', 'cooh', 'i2', 'rooh', 'cyclic', 'carbonates', 'oxalates', 'glycolates', 'polymerize', 'ring', 'opening', 'polymerizations', 'yield', 'polyesters', 'an', 'example', 'conversion', 'cyclic', 'carbonate', 'low', 'molecular', 'weight', 'polymer', '(about', '4000)', '39', '40:', '72', 'polyesters', '417', 'another', 'example', 'ring', 'opening', 'polymerization', 'oxalate', '39.', 'again', 'low', 'molecular', 'weight', 'polymers', 'result', 'n', 'o', 'o', 'o', 'k2co3', '150', 'oc', 'n', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'n', 'lactides', 'hand', 'subjected', 'ring', 'opening', 'polymerization', 'lewis', 'acids', 'yield', 'high', 'molecular', 'weight', 'polymers', '41:', 'also', 'a-hydroxycarboxylic', 'acids', 'polymerize', 'high', 'molecular', 'weight', 'polymers', 'formation', 'anhydrosulﬁdes', '42:', 'n', 'ho', 'o', 'oh', 'socl2', 'o', 'n', 'o', 's', 'o', 'o', 'o', 'n', 'o', 'in', 'addition', 'reported', '49', 'polyesteriﬁcation', 'reactions', 'possible', 'room', 'temperature', 'high', 'molecular', 'weight', 'polyesters', 'form', 'directly', 'carboxylic', 'acids', 'phenols', 'these', 'solution', 'polymerization', 'reactions', 'proceed', 'mild', 'conditions', 'near', 'neutral', 'ph', 'equimolar', 'mixtures', 'acids', 'alcohols', 'condense', 'reactions', 'driven', 'additions', 'water', 'across', 'carbodiimide', 'groups', 'substituted', 'ureas', 'form', 'by-products', 'rn', 'nr', 'rcooh', 'roh', 'o', 'rcoor', 'n', 'n', 'r', 'h', 'r', 'h', 'the', 'reaction', 'useful', 'preparations', 'isoregic', 'ordered', 'chains', 'translational', 'polar', 'symmetry', 'it', 'also', 'applied', 'polymerizations', 'functional', 'chiral', 'monomers', '7212', 'commercial', 'linear', 'saturated', 'polyesters', 'many', 'linear', 'aliphatic', 'polyesters', 'produced', 'commercially', 'they', 'relatively', 'low', 'molecular', 'weight', 'less', '10000', 'the', 'main', 'use', 'materials', 'plasticizers', 'poly(vinyl', 'chloride)', 'polymers', 'copolymers', 'such', 'polyesters', 'usually', 'formed', 'dicarboxylic', 'acids', 'glycols', 'often', 'monocarboxylic', 'acids', 'monohydroxy', 'compounds', 'added', 'towards', 'end', 'reaction', '418', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'small', 'quantities', 'control', 'molecular', 'weight', 'cap', 'reactive', 'end', 'groups', 'the', 'condensation', 'reactions', 'carried', '200–250(cid6)c', 'inert', 'atmosphere', 'to', 'obtain', 'molecular', 'weight', '1000', 'reactions', 'run', 'several', 'hours', 'for', 'higher', 'molecular', 'weights', 'however', 'glycols', 'added', 'excess', 'initial', 'products', 'heated', 'vacuum', '(about', '1', 'mmhg)', 'several', 'hours', 'among', 'high', 'molecular', 'weight', 'aliphatic-aromatic', 'polyesters', 'highest', 'commercial', 'volume', 'material', 'poly(ethylene', 'terephthalate)', 'most', 'prepared', 'dimethylene', 'terephthalate', 'ethylene', 'glycol', 'transesteriﬁcation', 'reaction', 'o', 'n', 'o', 'n', 'ho', 'o', 'o', 'oh', 'o', 'o', 'o', 'o', 'n', 'often', 'reaction', 'carried', 'two', 'steps', 'an', 'excess', 'two', 'moles', 'ethylene', 'glycol', 'used', 'ﬁrst', 'stage', 'reaction', 'carried', '150–210(cid6)c', 'form', 'bis(2-hydroxyethyl)terephthalate', 'small', 'amount', 'oligomer', 'methanol', 'removed', 'o', 'o', 'o', 'o', 'ho', 'oh', 'o', 'o', 'ho', 'o', 'o', '3', 'ho', 'oh', 'oh', 'ho', 'o', 'o', 'o', 'o', 'o', '4', 'in', 'second', 'stage', 'temperature', 'maintained', '270–285(cid6)c', 'reaction', 'carried', 'vacuum', '1', 'mmhg', 'the', 'bis(2-hydroxyethyl)terephthalate', 'undergoes', 'transesteri-', 'ﬁcation', 'reaction', 'excess', 'glycol', 'removed', 'o', 'o', '2', 'ho', 'oh', 'o', 'o', 'ho', 'o', 'o', 'o', 'o', 'o', 'o', 'ho', 'h', '4', 'o', 'o', 'oh', 'in', 'process', 'ethanol', 'forms', 'removed', 'various', 'metal', 'oxide', 'acetate', 'catalysts', 'employed', 'ﬁrst', 'stage', 'these', 'antimony', 'barium', 'calcium', 'cobalt', 'lead', 'manganese', 'titanium', 'zinc', 'oxides', 'acetates', 'carbonates', 'alcoholates', 'alkanoates', 'also', 'used', 'based', 'disclosures', 'patent', 'literature', 'appears', 'antimony', 'compounds', 'particularly', 'trioxide', 'dominate', 'ﬁeld', 'catalysts', 'second', 'stage', 'reaction', '16.', 'the', 'exact', 'mechanism', 'antimony', 'compounds', 'act', 'catalysts', 'syntheses', 'polyesters', 'still', 'investigated', 'it', 'shown', 'bis(2-hydroxyethyl)-terephthalate', 'competes', 'successfully', 'oligo-', 'mer', 'end', 'groups', 'sb2o3', 'complex', 'compound', 'metal', 'oxide', 'unreactive', 'polymerizations', '17.', 'in', 'addition', 'polymerizations', 'vacuum', 'increase', '72', 'polyesters', '419', 'metal', 'surfaces', 'antimony', 'oxide', '18.', 'it', 'suggested', 'reaction', 'antimony', 'trioxide', 'ethylene', 'glycol', 'results', 'formation', 'antimony', 'glycolates', 'ligand', 'number', '3', '19', '20:', 'o', 'o', 'sb', 'o', 'o', 'sb', 'o', 'o', 'ho', 'oh', 'o', 'o', 'sb', 'o', 'o', 'oh', 'sb', 'o', 'o', 'h', '3', 'a', 'study', 'antimony', 'glycolates', 'effective', 'catalysts', 'preparation', 'poly(ethylene', 'terephthal-', 'ate)', 'varying', 'number', 'hydroxyethoxy', 'ligands', 'rated', 'following', 'decreasing', 'order', 'effectiveness', '20:', 'this', 'led', 'suggestion', 'maerov', '20', 'key', 'step', 'condensation', 'may', 'involve', 'chemical', 'reaction', 'ties', 'hydroxyl', 'chain', 'end', 'catalyst', 'molecule', 'introduction', 'second', 'hydroxyethoxy', 'chain', 'end', 'followed', 'right', 'electronic', 'bond', 'shift', '5', '¼', '321', '(cid7)', '0', 'o', 'o', 'o', 'o', 'h', 'o', 'r', 'r', 'sb', 'o', 'the', 'role', 'antimony', 'establish', 'favorable', 'spatial', 'conﬁguration', 'transition', 'state', '20.', 'an', 'earlier', 'study', 'however', 'resulted', 'conclusion', 'antimony’s', 'activity', 'inversely', 'proportional', 'hydroxyl', 'group', 'concentration', '21.', 'based', 'model', 'reactions', 'preparation', 'poly(ethylene', 'terephthalate)', 'ester', 'interchange', 'optimum', 'molar', 'ratio', 'ethylene', 'glycol', 'dimethyl', 'terephthalate', '24', '1', 'this', 'ratio', 'allows', 'complete', 'removal', 'methanol', '22.', 'the', 'overall', 'polyesteriﬁcation', 'reaction', 'third', 'order', '22', '23.', 'in', 'addition', 'high', 'molecular', 'weight', 'polymerizations', 'poly(ethylene', 'terephthalate)', 'invariably', 'produce', 'cyclic', 'oligomers', 'by-products', '24', '25.', 'eight', 'different', 'cyclic', 'species', 'identiﬁed', 'one', 'commercial', 'polymer', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'the', 'important', 'side', 'reaction', 'however', 'formation', 'diethylene', 'glycol', 'it', 'becomes', 'included', 'polymer', 'di(oxyethylene)oxy', 'link', 'commercial', 'polymerizations', 'carried', 'molecular', 'weights', '20000', 'reached', 'use', 'ﬁbers', 'higher', 'ones', 'use', 'injection', 'moldings', '420', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'extrusions', 'these', 'materials', 'may', 'contain', '2–4', 'mole', 'percent', 'di(oxyethylene)oxy', 'units', '54.', 'presence', 'units', 'inﬂuences', 'degree', 'whiteness', 'polyesters', 'melting', 'temperature', 'a', 'process', 'developed', '54', 'reduce', 'presence', 'di(oxyethylene)oxy', 'units', 'poly(ethylene', 'terephthalate)', 'the', 'condensation', 'reactions', 'still', 'carried', 'two', 'steps', 'in', 'ﬁrst', 'one', 'period', 'precondensation', 'material', 'prepared', 'melt', 'described', 'previously', 'in', 'second', 'step', 'however', 'reaction', 'carried', 'melting', 'temperature', 'this', 'still', 'yields', 'high', 'molecular', 'weight', 'polymers', 'the', 'products', 'however', 'low', 'di(oxyethylene)oxy', 'linkages', 'there', 'continued', 'interest', 'developing', 'process', 'direct', 'esteriﬁcation', 'terephthalic', 'acid', 'ethylene', 'glycol', 'it', 'appear', 'however', 'currently', 'practiced', 'commercial', 'scale', 'us', 'in', 'japan', 'process', 'commercialized', 'terephthalic', 'acid', 'reacted', 'two', 'moles', 'ethylene', 'oxide', 'form', 'dihydroxy', 'ester', 'situ', 'starting', 'material', 'one', 'mole', 'ethylene', 'glycol', 'removed', 'vacuum', 'subsequent', 'condensation', 'process', 'also', 'reported', '25', 'polymer', 'prepared', 'direct', 'esteriﬁcation', 'room', 'temperature', 'presence', 'picryl', 'chloride', 'the', 'reaction', 'also', 'performed', '120(cid6)c', 'presence', 'diphenyl', 'chloro-phosphate', 'toluenesulfonyl', 'chloride', '25.', 'this', 'done', 'solution', 'pyridine', 'either', 'solvent', 'cosolvent', 'pyridine', 'acts', 'scavenger', 'hcl', 'by-product', 'reaction', 'perhaps', 'also', 'activator', '(by', 'converting', 'acid', 'reactive', 'ester', 'intermediate)', 'another', 'commercially', 'important', 'high', 'molecular', 'weight', 'polyester', 'poly(butylene', 'terephthalate)', 'also', 'called', 'poly(tetramethylene', 'terephthalate)', 'the', 'polymer', 'prepared', 'catalyzed', 'ester', 'inter-', 'change', 'dimethyl', 'terephthalate', '14-butane', 'diol', 'o', 'o', 'ho', 'n', 'n', 'o', 'o', 'oh', 'o', 'o', 'o', 'o', 'o', 'n', 'n', 'ch3oh', 'this', 'synthesis', 'also', 'carried', 'two', 'stages', 'in', 'stage', 'one', 'excess', 'diol', 'reacted', 'dimethyl', 'terephthalate', '(about', '131)', 'insure', 'complete', 'removal', 'methanol', 'zinc', 'acetate', 'favored', 'catalyst', 'reaction', 'a', 'prepolymer', 'mixture', 'bis(hydroxybutyl)terephthalate', 'higher', 'oligomers', 'forms', 'stage', 'two', 'conducted', 'vacuum', '1', 'mmhg', 'high', 'enough', 'temperature', '(usually', 'least', '60(cid6)c', 'melting', 'temperature', 'polyester)', 'remove', 'excess', 'diol', 'reach', 'high', 'molecular', 'weight', 'zinc', 'oxide', 'favored', 'catalyst', 'stage', 'prolonged', 'heating', 'reaction', 'mixture', 'excessive', 'temperatures', 'results', 'formation', 'large', 'proportions', 'tetrahydrofuran', 'this', 'objectionable', 'affects', 'properties', 'product', 'it', 'also', 'results', 'lower', 'molecular', 'weight', 'polyesters', 'a', 'polyester', 'terephthalic', 'acid', '12-dimethylol', 'cyclohexane', 'produced', 'mainly', 'use', 'ﬁbers', 'this', 'polymer', 'also', 'formed', 'dimethyl', 'terephthalate', 'diol', 'transesteriﬁcation', 'reaction', 'the', 'material', 'following', 'structure', 'o', 'o', 'o', 'o', 'n', 'the', 'polymer', 'stiffer', 'poly(ethylene', 'terephthalate)', 'higher', 'melting', 'a', 'polyester', 'manufactured', 'japan', 'methyl', 'ester', 'p-2-hydroxyethoxybenzoic', 'acid', '72', 'polyesters', 'transesteriﬁcation', 'n', 'ho', 'o', 'o', 'oh', '421', 'o', 'o', 'n', 'o', 'the', 'product', 'used', 'ﬁber', 'during', 'last', '8', '10', 'years', 'considerable', 'effort', 'introduce', 'new', 'potentially', 'high', 'volume', 'polyester', 'poly(trimethylene', 'terephthalate)', '24,', 'market', 'the', 'polyester', 'sold', 'commercial', 'trade', 'name', '“corterra”', 'ﬁber', 'trade', 'name', '“sorona”', 'polyesters', 'based', '26-disubstituted', 'naphthalene', 'gained', 'commercial', 'importance', 'recent', 'years', 'due', 'higher', 'tensile', 'strength', 'higher', 'modulus', 'good', 'mechanical', 'properties', 'these', 'polyesters', 'offer', 'additional', 'advantage', 'poly(ethylene', 'terephthalate)', 'poly(butylene', 'tere-', 'phthalate)', 'higher', 'ﬁrst', 'second', 'transition', 'temperatures', 'polyesters', 'based', 'naphthalene', 'typically', 'prepared', 'either', 'solution', 'polymerization', 'ester', 'interchange', 'melt', 'or', 'n', 'ho', 'ohx', 'o', 'o', 'n', 'roh', 'o', 'o', 'x', 'n', 'o', 'n', 'ro', 'x', '¼', '24', '7213', 'copolyesters', 'mixed', 'dicarboxylic', 'acids', 'usually', 'used', 'form', 'copolyesters', 'for', 'instance', 'terephthalic', 'isophthalic', 'acids', 'reacted', 'together', '14-dimethylol', 'cyclohexane', 'form', 'copolyester', 'the', 'product', 'amor-', 'phous', 'transparent', 'another', 'copolyester', 'manufactured', 'terephthalic', 'isophthalic', 'aliphatic', 'dicarboxylic', 'acid', 'like', 'adipic', 'either', '14', 'butanediol', '16-hexanediol', 'the', 'aliphatic', 'dicarboxylic', 'acid', 'used', 'minor', 'quantities', 'many', 'copolyesters', 'used', 'high-strength', 'adhesives', 'in', 'addition', 'thermoplastic', 'polyester', 'elastomers', 'these', 'produced', 'equilibrium', 'melt', 'transesteriﬁcation', 'dimethyl', 'terephthalate', '14-butanediol', 'poly(tetramethylene', 'ether)', 'glycol', '(molecular', 'weight', '1000)', 'because', 'equilibrium', 'conditions', 'exist', 'melt', 'products', 'random', 'copolymers', 'o', 'o', '2n', 'o', 'o', 'o', 'o', 'n', 'ho', 'oh4', 'n', 'ho', 'o', 'oh', 'x', 'o', 'o', 'o', 'o', '4', 'o', 'o', 'x', 'n', '422', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'wholly', 'aromatic', 'polyesters', 'produced', 'high', 'temperature', 'applications', 'the', 'materials', 'must', 'prepared', 'p-', 'also', 'good', 'abrasion', 'resistance', 'one', 'commercial', 'polyester', 'hydroxybenzoic', 'acid', 'another', 'one', 'copolyester', 'prepared', 'p-hydroxybenzoic', 'acid', 'pp0-biphenol', 'terephthalic', 'o', 'o', 'n', 'o', 'o', 'o', 'n', 'isophthalic', 'acids', 'o', 'o', 'o', 'the', 'shown', 'polyester', 'melt', 'decomposes', '550(cid6)c', 'turner', 'et', 'al', '38', 'reported', 'preparation', 'triptysene', 'containing', 'polyesters', 'these', 'polyesters', 'illustrated', 'follows', 'o', 'o', 'o', 'x', 'o', 'o', 'o', 'x', '¼', '24', '6', 'the', 'polyester', 'prepared', 'melt-phase', 'synthesis', 'transesteriﬁcation', 'reaction', 'higher', 'thermal', 'stability', 'claimed', 'polyesters', 'containing', 'triptysene', 'analogous', 'polyesters', 'without', 'the', 'properties', 'various', 'polyesters', 'summarized', 'wilfong', '15.', 'table', '71', 'presents', 'tm', 'tg', 'values', 'polyesters', 'based', 'information', 'wilfong', 'sources', 'literature', 'among', 'new', 'methods', 'forming', 'polyesters', 'preparation', 'completely', 'aromatic', 'polyesters', 'direct', 'condensation', 'hydroxyaromatic', 'acids', '(like', 'hydroxybenzoic)', 'aid', 'triphenylpho-', 'sphorous', 'compounds', 'dichlorophenylphosphine', '38:', 'ho', 'o', 'o', 'p', '3', 'n', 'o', 'o', 'n', 'oh', 'hydroxybenzoic', 'acid', '43.', 'hexachlorotriphosphatriazine', 'also', 'used', 'attain', 'direct', 'polycondensations', 'hoch2ch2oh', '265–284', '72', 'polyesters', 'table', '71', 'approximate', 'melting', 'points', 'polyesters', 'dicarboxylic', 'acid', 'hooccooh', 'hooccooh', 'hooch2ch2cooh', 'hooch2ch2cooh', 'hooc(ch2)4cooh', 'hooc(ch2)4cooh', 'hooc(ch2)8cooh', 'hooc(ch2)8cooh', 'glycol', 'hoch2ch2oh', 'hoch2ch2ch2oh', 'hoch2ch2oh', 'hoch2ch2ch2oh', 'hoch2ch2oh', 'hoch2ch2ch2oh', 'hoch2ch2oh', 'hoch2ch2ch2oh', 'hooc', 'oh', 'o', 'ho', 'oh', 'hooc', 'cooh', 'hooc', 'cooh', 'hoch2ch2oh', 'hooc', 'cooh', 'hooc', 'cooh', 'hooc', 'cooh', 'hoch2ch2oh', 'hoch2ch2oh', 'hoch2ch2oh', 'hoch2ch2oh', 'hoch2ch2oh', 'hoch2ch2oh', 'hoch2ch2oh', 'ooch', 's', 'ch2', '4', 'o', '2', '2', 'n', 'h', '2', 'hooc', 'hooc', 'cooh', 'cooh', 'cooh', 'hooc', 'tg', '((cid6)c)', '–', '–', '–', '–', '–', '–', '–', '–', '–', '69', '119', '–', '–', '–', '–', '–', '–', '–', '–', '423', 'tm', '((cid6)c)', '172', '89', '108', '52', '50', '46', '79–80', '58', '350', 'decomp', 'trans', '120', 'cis', '30', '188', '270', '355', '170', '200', '252', '273', '143', '(continued)', 'hoch2ch2oh', '113', '260', '424', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'table', '71', '(continued)', 'dicarboxylic', 'acid', 'hooc', 'cooh', 'glycol', 'hoch2ch2oh', 'tg', '((cid6)c)', 'tm', '((cid6)c)', '132', 'hoch2ch2ch2oh', '80', '226–232', 'hooc', 'cooh', 'hooc', 'cooh', 'hooc', 'cooh', 'hooc', 'cooh', 'hooc', 'o', '2', 'hooc', 'cooh', 'hooc', 'cooh', 'hooc', 'cooh', 'ho–(–ch2–)6–oh', 'ho–(–ch2–)8–oh', 'ho–(–ch2–)10–oh', 'hoch2ch2oh', 'ho', 'ho', 'ho', 'o', '2', 'ch2', '2', 's', '2', 'o', 'o', '–', '–', '–', '45', '25', '173', '154–161', '129–132', '130–138', '152', '283', '150', '348', '279', '330', '164', '315', 'hooc', 'cooh', 'ho', 'oh', 'from', 'various', 'sources', 'literature', '722', 'linear', 'unsaturated', 'polyesters', 'the', 'materials', 'group', 'linear', 'copolyesters', 'one', 'dicarboxylic', 'acids', 'aliphatic', 'unsaturated', 'diacid', 'the', 'unsaturation', 'introduced', 'polymer', 'backbone', 'purpose', 'subsequent', 'cross-linking', 'unsaturated', 'polyester', 'technology', 'developed', 'use', 'glass', 'ﬁber', 'laminates', 'thermosetting', 'molding', 'compositions', 'casting', 'resins', 'solventless', 'lacquers', 'propylene', 'glycol', 'often', 'used', 'diol', 'to', 'lesser', 'extent', 'also', 'used', 'glycols', 'like', 'diethylene', 'glycol', 'greater', 'ﬂexibility', 'neopentyl', 'glycol', 'somewhat', 'better', 'thermal', 'resistance', 'bisphenol', 'a', '(22', 'bis(4-hydroxyphenyl)', 'propane)', 'used', 'better', 'chemical', 'resistance', 'needed', 'use', 'mixed', 'diols', 'common', 'many', 'unsaturated', 'dicarboxylic', 'acids', 'used', 'maleic', '(as', 'anhydride)', 'fumaric', 'acids', 'common', 'chloro', 'maleic', 'chloro', 'fumaric', 'acids', 'also', 'employed', '72', 'polyesters', '425', 'the', 'saturated', 'dicarboxylic', 'acids', 'act', 'modiﬁers', 'while', 'aliphatic', 'dicarboxylic', 'acids', 'used', 'common', 'one', 'ortho', 'phthalic', 'acid', '(added', 'reaction', 'mixture', 'anhydride)', 'the', 'acid', 'improves', 'compatibility', 'styrene', 'polymerized', 'presence', 'polyester', 'form', 'hard', 'rigid', 'cross-linked', 'materials', 'other', 'modiﬁers', 'used', 'obtain', 'special', 'properties', 'when', 'ﬂexible', 'product', 'needed', 'adipic', 'sebacic', 'acids', 'may', 'used', 'instead', 'for', 'better', 'heat', 'resistance', 'endo-', 'methylene', 'tetrahydrophthalic', 'anhydride', '(nadic', 'anhydride)', 'may', 'utilized', 'flame', 'retardency', 'achieved', 'using', 'chlorinated', 'dicarboxylic', 'acids', 'like', 'tetrachlorophthalic', 'styrene', 'common', 'monomer', 'used', 'cross-linking', 'unsaturated', 'polyesters', 'when', 'special', 'properties', 'required', 'monomers', 'like', 'methyl', 'methacrylate', 'may', 'employed', 'sometimes', 'done', 'combination', 'styrene', 'diallyl', 'phthalate', 'triallyl', 'cyanurate', 'form', 'better', 'heat-resistant', 'products', 'an', 'example', 'typical', 'batch', 'preparation', 'polyester', 'one', '12', 'moles', 'propylene', 'glycol', '067', 'moles', 'maleic', 'anhydride', '033', 'moles', 'phthalic', 'anhydride', 'combined', 'propylene', 'glycol', 'used', 'excess', 'compensate', 'loss', 'reaction', 'the', 'condensation', '150–200(cid6)c', 'lasts', '6–16', 'h', 'constant', 'removal', 'water', 'by-product', 'an', 'aromatic', 'solvent', 'like', 'toluene', 'xylene', 'often', 'added', 'reaction', 'mixtures', 'facilitate', 'water', 'removal', 'azeotropic', 'distillation', 'esteriﬁcation', 'catalysts', 'like', 'toluene', 'sulfonic', 'acid', 'reduce', 'reaction', 'time', 'in', 'addition', 'reactions', 'blanketed', 'inert', 'gases', 'like', 'nitrogen', 'carbon', 'dioxide', 'prevent', 'discoloration', 'oxygen', 'high', 'temperatures', 'when', 'molecular', 'weights', '1000–2000', 'reached', 'products', 'cooled', '90(cid6)c', 'blended', 'vinyl', 'monomers', 'often', 'blends', 'mixtures', 'equal', 'weights', 'polyesters', 'monomers', 'the', 'structure', 'above-described', 'unsaturated', 'polyester', 'illustrated', 'follows', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'it', 'interesting', 'unsaturated', 'polyesters', 'actually', 'prepared', 'anionic', 'alternating', 'copolymerization', 'epoxide', 'bicyclic', 'bis(g-butyrolactone)', 'bearing', 'isopropenyl', 'group', '34.', 'the', 'reaction', 'illustrated', 'follows', 'o', 'o', 'o', 'o', 'r', 'o', 'pph3', 'o', 'o', 'o', 'o', 'o', 'n', 'r', 'if', 'reaction', 'carried', 'prolonged', 'period', 'time', 'insoluble', 'cross-linked', 'product', 'results', '723', 'network', 'polyesters', 'surface', 'coatings', 'the', 'original', 'polyesters', 'coatings', 'prepared', 'phthalic', 'anhydride', 'glycerol', 'referred', 'glyptals', 'glyptal', 'resins', '426', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'oh', 'ho', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'the', 'products', 'polyesteriﬁcations', 'brittle', 'materials', 'they', 'therefore', 'modiﬁed', 'oils', 'either', 'drying', 'nondrying', 'such', 'oil-modiﬁed', 'resins', 'bear', 'names', 'alkyds', 'while', 'glycerol', 'widely', 'used', 'polyhydroxy', 'compounds', '(polyols)', 'also', 'utilized', 'these', 'may', 'trimethylolpropane', 'pentaerythritol', 'sorbitol', 'others', 'phthalic', 'anhydride', 'usually', 'used', 'alkyd', 'preparations', 'other', 'dicarboxylic', 'compounds', 'however', 'may', 'also', 'included', 'modiﬁcation', 'properties', 'common', 'modiﬁers', 'might', 'isophthalic', 'adipic', 'sebacic', 'acids', 'maleic', 'anhydride', 'in', 'addition', 'many', 'acid', 'modiﬁers', 'described', 'patent', 'literature', 'the', 'oils', 'alkyd', 'resins', 'usually', 'vegetable', 'origin', 'they', 'classiﬁed', 'type', 'amount', 'residual', 'unsaturation', 'drying', 'semidrying', 'nondrying', 'oils', 'the', 'drying', 'oils', 'contain', 'residual', 'unsaturation', 'nondrying', 'ones', 'contain', 'mostly', 'saturated', 'fatty', 'acids', 'alkyds', 'also', 'classiﬁed', 'quantity', 'modifying', 'oil', 'contain', 'short', 'medium', 'long', 'oil', 'alkyds', 'short', 'oil', 'alkyds', 'contain', '30–50', 'oil', 'usually', 'baked', 'obtain', 'hard', 'dry', 'surface', 'medium', 'oil', '(50–65)', 'long', 'oil', '(65–75)', 'alkyds', 'air', 'dry', 'upon', 'addition', 'metal', 'dryers', 'there', 'two', 'main', 'methods', 'preparation', 'alkyd', 'resins', 'in', 'ﬁrst', 'one', 'called', 'fatty', 'acid', 'process', 'free', 'fatty', 'acid', 'coesteriﬁed', 'directly', 'dibasic', 'acid', 'polyol', '200–240(cid6)c', 'the', 'reaction', 'may', 'carried', 'without', 'solvent', 'ﬁrst', 'heating', 'inert', 'atmosphere', 'at', 'end', 'inert', 'gas', 'may', 'blown', 'resin', 'bottom', 'reaction', 'kettle', 'remove', 'water', 'unreacted', 'materials', 'as', 'modiﬁcation', 'small', 'quantity', 'solvent', 'may', 'used', 'remove', 'water', 'esteriﬁcation', 'continuously', 'azeotropic', 'distillation', 'aid', 'moisture', 'traps', 'in', 'second', 'method', 'known', 'alcoholysis', 'process', 'drying', 'oil', 'heated', 'glycerol', 'ﬁrst', 'stage', 'reaction', '240(cid6)c', 'this', 'usually', 'done', 'presence', 'transesteriﬁcation', 'basic', 'catalyst', 'form', 'monoglycerides', 'oocr', 'oh', 'oh', 'rcoo', 'oocr', 'ho', '2', 'oh', '3', 'ho', 'oocr', 'after', 'ﬁrst', 'stage', 'complete', 'phthalic', 'anhydride', 'without', 'another', 'dibasic', 'acid', 'added', 'copolyesteriﬁcation', 'carried', 'manner', 'ﬁrst', 'stage', 'reaction', 'the', 'conditions', 'reaction', 'carried', 'rate', 'temperature', 'raised', 'condensation', 'affect', 'molecular', 'weight', 'distribution', 'ﬁnal', 'product', 'in', 'addition', '72', 'polyesters', '427', 'monoglyceride', 'content', 'original', 'reaction', 'mixture', 'determines', 'microgel', 'content', 'alkyd', 'dynamic', 'properties', 'dried', 'ﬁlm', '31.', 'the', 'ﬁnished', 'alkyd', 'resin', 'illustrated', 'follows', 'r', 'o', 'o', 'r', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', '724', 'polycarbonates', 'a', 'special', 'group', 'polyesters', 'carbonic', 'acid', 'known', 'polycarbonates', 'the', 'ﬁrst', 'polycarbonates', 'prepared', 'early', '1898', 'einhorn', 'reacting', 'phosgene', 'hydroquinone', 'resorcinol', '33.', 'these', 'materials', 'lack', 'desirable', 'properties', 'remain', 'laboratory', 'curiosities', 'during', 'ﬁfties', 'however', 'new', 'polymers', 'developed', '440-dihydroxydiphenyl', 'alkanes', 'these', 'polycarbonates', 'high', 'melting', 'points', 'good', 'thermal', 'hydrolytic', 'stability', 'nevertheless', 'date', 'one', 'polycarbonate', 'achieved', 'signiﬁcant', 'commercial', 'importance', 'it', 'based', '220-', 'bis(4-hydroxyphenyl)propane', 'there', 'two', 'main', 'methods', 'preparing', 'polycarbonates', 'one', 'direct', 'reaction', 'phosgene', 'diphenol', 'one', 'ester', 'interchange', 'the', 'direct', 'phosgenation', 'form', 'schotten–baumann', 'reaction', 'carried', 'presence', 'base', 'n', 'ho', 'oh', 'cocl2', 'n', 'o', 'o', 'n', 'o', 'the', 'reaction', 'may', 'carried', 'presence', 'pyridine', 'acts', 'catalyst', 'hcl', 'scavenger', 'often', 'chlorinated', 'solvent', 'used', 'diluent', 'pyridine', 'phosgene', 'bubbled', 'solution', 'diphenol', '25–35(cid6)c', 'the', 'pyridine', 'hydrochloride', 'precipitates', 'washing', 'pyridine', 'solution', 'dilute', 'hcl', 'water', 'polymer', 'precipitated', 'nonsolvent', 'an', 'interfacial', 'polymerization', 'procedure', 'also', 'employed', 'direct', 'phosgenation', 'a', 'caustic', 'solution', 'diphenol', 'dispersed', 'organic', 'chlorinated', 'solvent', 'containing', 'small', 'quantities', 'tertiary', 'amine', 'phosgene', 'bubbled', 'reaction', 'mixture', '25(cid6)c', 'when', 'reaction', 'complete', 'organic', 'phase', 'contains', 'polymer', 'it', 'separated', 'product', 'isolated', 'the', 'ester', 'interchange', 'method', 'carried', 'diphenol', 'diphenyl', 'carbonate', 'n', 'ho', 'oh', 'n', 'o', 'o', 'o', 'o', 'oh', 'o', 'n', 'o', '2n', 'to', 'obtain', 'high', 'molecular', 'weights', 'method', 'almost', 'complete', 'removal', 'phenol', 'required', 'the', 'reaction', 'carried', 'typical', 'basic', 'catalysts', 'like', 'lithium', 'hydride', 'zinc', 'oxide', 'antimony', 'oxide', 'inert', 'atmosphere', 'initial', 'reaction', 'temperature', '150(cid6)c', 'it', 'raised', '1', 'h', '428', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'period', '210(cid6)c', 'pressure', 'reduced', '20', 'mmhg', 'reaction', 'mixture', 'heated', '300(cid6)c', '5–6', 'h', '1', 'mmhg', 'heating', 'stopped', 'desired', 'viscosity', 'reached', 'a', 'synthetic', 'route', 'polycarbonates', 'reported', 'uses', 'crown', 'ethers', 'crown', 'ethers', 'generally', 'form', 'stable', 'complexes', 'metal', 'cations', 'increasing', 'dissociation', 'ion', 'pairs', 'provide', 'highly', 'reactive', 'unsolvated', 'anions', 'this', 'led', 'direct', 'preparations', 'new', 'polycarbonates', 'ao-', 'dibromo', 'compounds', 'carbon', 'dioxide', 'potassium', 'carbonate', 'salts', 'diols', '36', 'presence', '18-crown-6-ether', 'n', 'br', 'br', '2n', 'co2', 'nk2co3', 'rb', 'o', 'o', 'o', 'br', 'ko', 'o', 'o', 'br', 'mm', '18', 'crown-6-ether', 'co2', 'ok', 'o', 'o', 'o', 'o', 'o', 'o', 'm', 'o', 'o', 'o', '18', 'crown-6-ether', 'br', 'n', 'nkbr', '2nbr', 'n', 'when', 'potassium', 'salt', 'diol', 'present', 'reaction', 'mixture', 'mixed', 'polyesters', 'form', '36:', 'in', 'addition', 'crown', 'ethers', 'cryptates', 'polyglyme', 'exhibit', 'similar', 'behavior', '37.', 'another', 'direct', 'route', 'polycarbonate', 'reported', 'okuyama', 'et', 'al', '36.', 'the', 'process', 'consists', 'oxidative', 'carbonylation', 'procedure', 'it', 'catalyzed', 'pd', 'complex', 'system', '110-di-tert-butyl-', '330-methylenediiimidazolin-220dylidenepalladium', 'dibromide', 'produces', 'highest', 'molecular', 'weight', 'polymer', '(mn', '¼', '9600', 'mw', '¼', '24000)', '80', 'yields', 'table', '72', 'presents', 'tg', 'tm', 'values', 'polycarbonates', 'picked', 'literature', '725', 'polyesters', 'lactones', 'polyesters', 'obtainable', 'ring', 'opening', 'polymerization', 'lactones', '(see', 'chap', '5)', 'produced', 'commercially', 'large', 'scale', 'judging', 'patent', 'literature', 'however', 'continuing', 'interest', 'materials', 'particularly', 'japan', 'because', 'fairly', 'good', 'hydrolytic', 'stability', 'polypivalyllactone', 'advanced', 'stages', 'commercial', 'development', 'o', 'o', 'n', '72', 'polyesters', '429', 'table', '72', 'melting', 'points', 'polycarbonates', 'structure', 'parent', 'diphenol', 'approximately', 'tg', '((cid6)c)', '145–150', 'approximately', 'tm', '((cid6)c)', '265–270', 'ho', 'ho', 'ho', 'ho', 'ho', 'ho', 'oh', 'oh', 'oh', 'oh', 'oh', 'oh', 'h', 'c3h7', 'c3h7', 'o', 'o', 'o', '3', 'ho', 'o', 'oh', 'ho', 'oh', 'from', 'various', 'literature', 'sources', '125–130', '185–200', '145–150', '170–180', '121', '210–230', '–', '–', '–', '180–210', '205–210', '250–260', '170', '240–250', '430', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'the', 'polymer', 'spun', 'elastic', 'yarn', 'ﬁne', 'denier', 'it', 'also', 'claimed', 'exhibit', 'good', 'mechanical', 'properties', 'molding', 'compares', 'favorably', 'commercial', 'polyesters', 'nylons', 'also', 'polycaprolactone', 'reported', 'used', 'medical', 'applications', 'biodegradable', 'surgical', 'sutures', 'postoperative', 'support', 'pins', 'splints', '39.', 'similar', 'uses', 'also', 'found', 'two', 'polyesters', 'poly(lactic', 'acid)', 'poly-(glycolic', 'acid)', '40.', 'the', 'two', 'polymers', 'form', 'cyclic', 'dimers', 'cationic', 'ring', 'opening', 'polymerizations', 'aid', 'lewis', 'acids', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', '73', 'polyamides', 'the', 'family', 'synthetic', 'polymeric', 'materials', 'amide', 'linkages', 'backbones', 'large', 'it', 'includes', 'synthetic', 'linear', 'aliphatic', 'polyamides', 'carry', 'generic', 'name', 'nylon', 'aromatic', 'polyamides', 'fatty', 'polyamides', 'used', 'adhesives', 'coatings', 'in', 'addition', 'synthetic', 'materials', 'also', 'large', 'family', 'naturally', 'occurring', 'polymers', 'a-amino', 'acids', 'called', 'proteins', 'the', 'latter', 'ones', 'discussed', 'chap', '8', 'rest', 'naturally', 'occurring', 'polymers', 'the', 'nylons', 'include', 'polyamides', 'produced', 'ring', 'opening', 'polymerizations', 'lactams', 'condensa-', 'tion', 'products', 'diamines', 'dicarboxylic', 'acids', '731', 'nylons', 'the', 'nylons', 'named', 'number', 'carbon', 'atoms', 'repeat', 'units', 'the', 'materials', 'formed', 'ring', 'opening', 'polymerizations', 'lactams', 'therefore', 'carry', 'one', 'number', 'names', 'like', 'instance', 'nylon', '6', 'formed', 'caprolactam', 'by', 'method', 'nomenclature', 'nylon', 'prepared', 'condensing', 'diamine', 'dicarboxylic', 'acid', 'like', 'example', 'hexamethylene', 'diamine', 'adipic', 'acid', 'called', 'nylon', '66', 'it', 'customary', 'ﬁrst', 'number', 'represent', 'number', 'carbons', 'diamine', 'second', 'number', 'represent', 'number', 'carbons', 'diacid', 'a', 'discussion', 'various', 'individual', 'nylons', 'follows', 'not', 'industrially', 'important', 'at', 'present', 'know', 'prepare', 'nylon', '1', 'anionic', 'polymerization', 'isocyanates', 'this', 'reaction', 'discussed', 'chap', '4', 'r', 'n', 'n', 'o', 'nacn', 'dmf', 'r', 'n', 'o', '73', 'polyamides', '431', 'potassium', 'sodium', 'cyanide', 'catalyze', 'reaction', 'it', 'carried', '(cid1)20', '(cid1)100(cid6)c', 'an', 'example', 'following', 'preparation', '47:', 'n', 'o', 'kcn', 'n', 'o', 'o', 'n', 'o', 'the', 'resultant', 'polymer', 'molecular', 'weight', 'one', 'million', 'when', 'methyl', 'group', 'replaced', 'butyl', 'product', 'tough', 'ﬁlm', 'former', 'depolymerizes', 'presence', 'catalysts', 'many', 'interesting', 'high', 'molecular', 'weight', 'polymers', 'various', 'substitutions', 'formed', 'reaction', 'low', 'temperature', 'polymerization', 'n-carboxy-a-amino', 'acid', 'anhydrides', 'results', 'formations', 'nylon', '2', 'this', 'reaction', 'also', 'discussed', 'chap', '5', 'these', 'polymers', 'mainly', 'interest', 'protein', 'chemists', 'model', 'studies', 'naturally', 'occurring', 'poly(a-amino', 'acids)', 'nylon', '3', 'synthesized', 'intramolecular', 'hydrogen', 'transfer', 'polymerization', 'acrylamide', '48', '(see', 'chap', '4)', 'n', 'o', 'b', 'nh2', 'h', 'n', 'n', 'o', 'the', 'ﬁbers', 'nylon', '3', 'reported', 'resemble', 'natural', 'silk', '50.', 'they', 'possess', 'high', 'water', 'absorbency', 'good', 'light', 'oxygen', 'stability', 'the', 'polymer', 'however', 'high', 'melting', 'melt', 'spinning', 'molding', 'extrusion', '46.', 'nylon', '3', 'ﬁbers', 'spun', 'however', 'special', 'solutions', 'containing', 'formic', 'acid', '52.', 'it', 'difﬁcult', 'synthesize', 'b-propiolactam', 'a', 'synthetic', 'route', 'however', 'found', 'substituted', 'propiolactams', 'like', 'b-butyrolactams', '50.', 'the', 'compounds', 'form', 'nucleophilic', 'additions', 'carbonylsulfamoyl', 'chloride', 'oleﬁns', '51:', 'clo2s', 'n', 'o', 'n', 'o', 'n', 'o', 'clo2s', 'h', 'the', 'shown', 'b-butyrolactam', 'polymerizes', 'readily', 'anionic', 'mechanism', 'yielding', 'high', 'molecular', 'weight', 'polymer', 'this', 'lactam', 'preparation', 'reaction', 'quite', 'general', 'it', 'carried', 'propene', '1-butene', '1-hexene', 'styrene', '50.', 'although', 'substituted', 'lactams', 'harder', 'polymerize', '53,', 'four-membered', 'lactams', 'exhibit', 'strong', 'tendency', 'toward', 'ring', 'opening', 'even', 'substituted', 'b-propiolactams', 'polymerize', 'well', '50.', 'the', 'rate', 'polymerization', 'however', 'tend', 'decrease', 'number', 'substituents', 'nylon', '4', 'polypyrrolidone', 'attractive', 'polymer', 'use', 'ﬁbers', 'the', 'original', 'syntheses', 'nylon', '4', '2-pyrrolidone', 'carried', 'alkaline', 'catalyzed', 'ring', 'opening', 'polymerizations', 'promoted', 'n-acylpyrrolidone', '49.', 'the', 'products', 'reactions', 'melt', '260', '265(cid6)c', '432', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'they', 'unstable', 'temperatures', 'cannot', 'melt', 'spun', 'fibers', 'however', 'prepared', 'dry', 'spinning', 'hydrocarbon', 'suspensions', '49.', 'later', 'found', 'anionic', 'ring', 'opening', 'polymerizations', '2-pyrrolidone', 'activated', 'co2', 'place', 'n-acyl', 'derivative', 'resultant', 'higher', 'molecular', 'weight', 'product', 'much', 'better', 'heat', 'resistance', '54.', 'this', '“new”', 'nylon', '4', 'reportedly', 'melt', 'spun', 'nylon', '5', 'poly(a-piperidone)', 'prepared', 'ring', 'opening', 'anionic', 'polymerization', 'valerolactam', '53.', 'the', 'reaction', 'requires', 'pure', 'monomer', 'yield', 'high', 'molecular', 'weight', 'polymer', '53:', 'n', 'h', 'o', 'n', 'one', 'route', 'valerolactam', 'cyclopentadiene', 'h', 'o', 'h2noh', 'bechmann', 'rearrangement', 'oh', 'n', '259', 'oc', 'h2so', 'o', 'h', 'n', 'valerolactam', 'also', 'polymerized', 'aid', 'coordination', 'catalysts', 'high', 'molecular', 'weight', 'polymer', 'using', 'alkali', 'metal-al(c2h5)3', 'catalysts', 'alkali', 'metal', 'alkyl-al(c2h5)3', 'catalysts', '55.', 'the', 'polymerizations', 'require', 'relatively', 'long', 'times', 'nylon', '6', 'obtained', 'via', 'ring', 'opening', 'polymerization', 'caprolactam', 'n', 'o', 'h', 'n', 'this', 'polymer', 'developed', 'years', 'important', 'commercial', 'material', 'as', 'result', 'many', 'preparatory', 'routes', 'developed', 'starting', 'material', 'polymerization', 'reaction', 'studied', 'thoroughly', 'the', 'common', 'starting', 'materials', 'preparations', 'caprolactam', 'phenol', 'cyclohexane', 'toluene', 'some', 'caprolactam', 'also', 'made', 'aniline', 'in', 'synthetic', 'processes', 'key', 'material', 'cyclohexanone', 'oxime', 'the', 'route', 'based', 'phenol', 'shown', 'follows', 'h', 'oh', 'o', 'oh', 'o', 'h2noh', 'noh', 'beckmann', 'rearrangement', '250', 'oc', 'h2so4', 'o', 'h', 'n', 'o', 'o', 'o', 'h', 'n', 'n', 'n', 'n', 'h', '73', 'polyamides', '433', 'a', 'by-product', 'reaction', 'ammonium', 'sulfate', 'to', 'avoid', 'necessity', 'disposing', 'ammonium', 'sulfate', 'many', 'caprolactam', 'producers', 'sought', 'routes', 'oxime', 'one', 'approach', 'form', 'directly', 'reacting', 'cyclohexanone', 'ammonia', 'hydrogen', 'peroxide', 'presence', 'tungstic', 'acid', 'catalyst', '56:', 'o', 'nh3', 'h3pw2o40', '8h20', 'noh', 'h2o', 'the', 'reaction', 'conducted', 'water', 'product', 'oxime', 'extracted', 'organic', 'solvent', 'another', 'process', 'based', 'photo-nitrosyl', 'chlorination', 'here', 'cyclohexane', 'converted', 'one', 'step', 'cyclohexanone', 'oxime', 'hydrochloride', '57:', 'hu', 'nocl', 'no', 'cl', 'cl', 'no', 'hcl', 'noh', 'hcl', 'another', 'process', 'uses', 'ketene', 'form', 'cyclohexene', 'acetate', '58:', 'o', 'o', 'o', 'o', 'hno3', 'h2o', 'o2n', 'h', 'cooh', 'among', 'recent', 'developments', 'one-step', 'synthesis', 'caprolactam', 'cyclohexanol', '59:', '2', 'oh', 'licl', 'ca(scn)2', '90-100', 'oc', 'o', 'n', 'h', 'there', 'processes', 'caprolactam', 'syntheses', 'well', 'however', 'thorough', 'discussion', 'subject', 'belongs', 'books', 'dedicated', 'subject', 'the', 'mechanism', 'reaction', 'ring', 'opening', 'polymerization', 'lactams', 'discussed', 'chap', '5', 'several', 'important', 'side', 'reactions', 'accompany', 'polymerization', 'one', 'formation', 'cyclic', 'oligomers', '58.', 'the', 'cyclic', 'oligomers', 'soluble', 'water', 'alcohol', 'mixtures', 'range', 'size', 'cyclic', 'dimers', 'cyclic', 'nonamers', '60–62.', 'formation', 'compounds', 'may', 'governed', 'equilibrium', '63.', 'the', 'polyamide', 'also', 'thermally', 'oxidize', 'upon', 'prolonged', 'exposure', 'heat', 'air', 'another', 'important', 'side', 'reaction', 'decarboxylation', 'occurs', 'high', 'temperatures', 'this', 'no2', 'o', 'o', 'h', 'n', 'o', '434', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'result', 'interaction', 'carboxyl', 'group', 'molecule', 'caprolactam', 'amide', 'group', '58:', 'cooh', '5', 'o', 'n', 'n', 'h', 'o', 'h', 'o', 'oh', 'o', '4', 'o', 'n', 'o', 'o', 'n', 'n', 'co2', 'polymerizations', 'caprolactam', 'conducted', 'inert', 'atmospheres', 'prevent', 'oxidative', 'decompositions', 'these', 'result', 'formations', 'carbon', 'monoxide', 'carbon', 'dioxide', 'acetaldehyde', 'formaldehyde', 'methanol', 'caprolactam', 'even', 'oxidize', 'air', 'temperatures', '70', '100(cid6)c', '64,', 'according', 'following', 'scheme', 'o', 'o', 'n', 'h', 'o', 'h', 'o', '-h2o', 'n', 'o', 'h', 'o', 'n', 'o', 'h', 'cooh', 'cooh', 'much', 'nylon', '6', 'used', 'producing', 'ﬁbers', 'polycaprolactam', 'prepared', 'water', 'catalyzed', 'polymerizations', 'best', 'suited', 'purpose', 'it', 'also', 'used', 'molding', 'though', 'anionically', 'polymerized', 'caprolactam', 'used', 'well', '65.', 'the', 'polymerizations', 'carried', 'batch', 'continuous', 'processes', 'often', 'tubular', 'ﬂow', 'reactors', 'employed', 'a', 'typical', 'polymerization', 'reaction', 'carried', 'follows', 'caprolactam', 'water', '(5–10', 'weight', 'monomer)', 'acetic', 'acid', '(about', '01)', 'fed', 'reactor', 'nitrogen', 'atmosphere', 'the', 'reaction', 'mixture', 'heated', '250(cid6)c', '12', 'h', 'internal', 'pressure', 'maintained', '15', 'atmospheres', 'venting', 'steam', 'the', 'product', 'polymerization', 'extruded', 'ribbon', 'quenched', 'chopped', 'chips', 'it', 'consists', '90', 'polymer', '10', 'low', 'molecular', 'weight', 'compounds', 'monomer', 'the', 'polymer', 'puriﬁed', 'either', 'water', 'leaching', '85(cid6)c', 'vacuum', 'extraction', 'undesirable', 'by-products', '180(cid6)c', 'castings', 'nylon', '6', 'commonly', 'formed', 'situ', 'molds', 'here', 'preparation', 'polymer', 'anionic', 'mechanism', 'preferred', 'the', 'catalyst', 'systems', 'consist', '01–10', 'mole', 'percent', 'acetyl', 'caprolactam', '015–05', 'mole', 'percent', 'sodium', 'caprolactam', 'the', 'reaction', 'temperature', 'kept', '140', '180(cid6)c', 'an', 'exotherm', 'raise', 'much', '50(cid6)c', 'polymerization', 'proceeds', 'nylon', '7', 'nylon', '9', 'part', 'process', 'developed', 'russia', 'form', 'polyamides', 'use', 'ﬁbers', 'the', 'process', 'starts', 'telomerization', 'ethylene', '66.', 'a', 'free-radical', 'polymerization', 'ethylene', 'conducted', 'presence', 'chlorine', 'compounds', 'act', 'chain-transferring', 'agents', 'the', 'reaction', '73', 'polyamides', '435', 'table', '73', 'compositions', 'telomers', 'x', 'fraction', '()', '1', '2', '3', '4', '4', 'from', 'nesmeyanov', 'reundlina', '66,', 'permission', 'tetrahedral', 'letters', 'elsevier', 'science', 'ltd', '5', '44', '28', '15', '8', 'carried', '120–200(cid6)c', 'temperature', '400–600', 'atmospheres', 'pressure', 'the', 'preferred', 'chain-', 'transferring', 'agents', 'reaction', 'ccl4', 'cocl2', '66:', 'r', 'r', 'ch2', 'ccl4', 'r', 'ch2cl', 'ccl3', 'ccl3', 'cl', 'cl', 'cl', 'ch2', 'x-1', 'ccl4', 'cl', 'cl', 'cl', 'cl', '4', 'the', 'resultant', 'chloro', 'alkanes', 'hydrolyzed', 'sulfuric', 'acid', 'after', 'hydrolyses', 'products', 'treated', 'ammonia', 'cl', 'cl', 'cl', 'o', 'ho', 'cl', 'x', 'cl', 'x', 'h2so4', 'h2o', 'o', 'ho', 'cl', 'x', 'nh3', 'o', 'o', 'nh3', 'x', 'the', 'amino', 'acid', 'condensed', 'lactam', 'subsequently', 'polymerized', 'table', '73', 'shows', 'composition', 'telomers', 'free-radical', 'polymerization', '66.', 'as', 'seen', 'table', 'economics', 'producing', 'nylon', '7', 'process', 'favorable', 'one', 'may', 'wish', 'an', 'advantage', 'however', 'producing', 'nylon', '7', 'polymer', 'contains', 'little', 'monomer', 'spun', 'without', 'washing', 'extraction', 'required', 'nylon', '6', 'an', 'early', 'synthesis', 'nylon', '8', 'used', 'cyclooctatetraene', 'formed', 'acetylene', 'converted', 'nylon', '8', 'follows', '4', 'h', 'o', 'o', 'weak', 'base', 'o', 'noh', 'beckmann', 'rearrangement', 'h2so4', 'o', 'n', 'h', 'etc', '436', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'the', 'acetylene', 'later', 'replaced', 'butadiene', 'economic', 'reasons', 'butadiene', 'cyclodimerized', 'hydrogenated', 'cyclooctane', 'oxime', 'prepared', 'directly', 'cyclooctane', 'photonitrosation', 'nocl', 'hν', 'noh', 'hcl', 'beckmann', 'rearrangement', '7', 'o', 'n', 'h', 'etc', 'nylon', '9', 'poly(o-pelargonamide)', 'produced', 'russia', 'together', 'nylon', '7', 'poly', '(aminoenanthic', 'acid)', 'described', 'in', 'us,', 'kohlhase', 'et', 'al', '67', 'developed', 'route', 'nylon', '9', 'via', 'ozonolysis', 'unsaturated', 'fatty', 'acids', 'like', 'obtained', 'soybean', 'oil', 'the', 'glycerol', 'fatty', 'acid', 'esters', 'oleic', 'linoleic', 'linolenic', 'acids', 'transesteriﬁed', 'methanol', 'form', 'methyl', 'esters', 'the', 'esters', 'cleaved', 'via', 'ozonolysis', 'yield', 'methylazelaldehyde', 'by-', 'products', 'removed', 'the', 'puriﬁed', 'product', 'reacted', 'ammonia', 'reduced', 'raney', 'nickel', 'yield', 'methyl', 'ester', 'amino', 'acid', 'h2n', 'och3', 'o', 'after', 'hydrolysis', 'puriﬁcation', 'free', 'amino', 'acid', 'converted', 'high', 'molecular', 'weight', 'polymers', '68.', 'to', 'date', 'nylon', '9', 'commercialized', 'us,', 'though', 'polymer', 'high', 'melting', 'point', '209(cid6)c', 'ﬂexible', 'nylon', '6', 'it', 'also', 'lower', 'water', 'absorption', 'nylon', '11', 'originally', 'synthesized', 'france', 'the', 'monomer', 'o-amino-undecanoic', 'acid', 'obtained', 'methyl', 'ricinoleate', 'comes', 'castor', 'oil', 'methyl', 'ricinoleate', 'ﬁrst', 'cleaved', 'thermally', 'heptaldehyde', 'methyl', 'undecylenate', 'oh', '5', 'och3', '∆', '7', 'o', 'c6h13', 'h', 'och3', '8', 'o', 'the', 'ester', 'hydrolyzed', 'converted', 'amino', 'acid', 'o', '8', 'och3', 'br', 'counter', 'markovnikov', 'addition', 'h2o', 'o', 'oh', '10', 'oh', 'h3n', 'hbr', 'peroxide', 'o', 'o', '10', 'nh3', 'o', 'o', '8', 'polycondensation', 'conducted', 'melt', 'nitrogen', '215(cid6)c', 'several', 'hours', 'the', 'polymer', 'transparent', 'natural', 'form', 'it', 'high-impact', 'resistance', 'low', 'moisture', 'absorption', 'good', 'low', 'temperature', 'ﬂexibility', 'it', 'also', 'manufactured', 'us', 'nylon', '12', 'produced', 'us,', 'japan', 'europe', 'original', 'development', 'coming', 'europe', 'all', 'current', 'manufacturing', 'processes', 'polyamide', 'formed', 'ring', 'opening', 'polymeriza-', 'tion', 'lauryl', 'lactam', 'based', 'cyclododecatriene', 'this', 'ring', 'compound', 'obtained', '73', 'polyamides', '437', 'table', '74', 'approximate', 'melting', 'points', 'polyamides', 'nylon', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', 'from', 'various', 'literature', 'sources', 'repeat', 'unit', '–(ch2)2–co–nh–', '–(ch2)3–co–nh–', '–(ch2)4–co–nh–', '–(ch2)5–co–nh–', '–(ch2)6–co–nh–', '–(ch2)7–co–nh–', '–(ch2)8–co–nh–', '–(ch2)9–co–nh–', '–(ch2)10–co–nh–', '–(ch2)11–co–nh–', '–(ch2)12–co–nh–', 'mp', '((cid6)c)', '320–330', '260–265', '260', '215–220', '225–230', '195', '197–200', '173', '185–187', '180', '173', 'trimerization', 'butadiene', 'using', 'ziegler–natta', 'type', 'catalysts', 'one', 'patent', 'reports', 'using', 'polyalkyl-', 'titanate', 'dialkylaluminum', 'monochloride', '69:', '3', 'h', 'the', 'cyclododecatriene', 'converted', 'lauryl', 'lactam', 'different', 'processes', 'one', 'consists', 'hydrogenation', 'cyclic', 'triene', 'followed', 'oxidation', 'cyclic', 'ketone', 'conversion', 'oxime', 'rearrangement', 'beckmann', 'reaction', 'lactam', 'h', 'o', 'o', 'etc', 'another', 'process', 'utilizes', 'photonitrosation', 'nocl', 'hν', 'oh', 'n', 'beckmann', 'rearrangement', 'h2so4', 'o', 'n', 'h', 'there', 'still', 'processes', 'lack', 'industrial', 'importance', 'nylon', '12', 'like', 'nylon', '11', 'exhibits', 'low', 'moisture', 'absorbency', 'good', 'dimensional', 'stability', 'good', 'ﬂexibility', 'low', 'temperatures', 'preparations', 'nylons', 'reported', 'time', 'time', 'literature', 'for', 'one', 'reason', 'another', 'however', 'developed', 'industrially', 'important', 'materials', 'thus', 'instance', 'time', 'known', 'nylon', '13', 'prepared', 'erucic', 'acid', 'found', 'crambe', 'rapeseed', 'oils', 'the', 'polymer', 'supposed', 'quite', 'similar', 'nylon', '11', 'though', 'lower', 'melting', 'the', 'melting', 'points', 'nylons', 'describe', 'summarized', 'table', '74', '438', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'nylon', '66', 'condensation', 'product', 'hexamethylene', 'diamine', 'adipic', 'acid', 'this', 'polyamide', 'originally', 'synthesized', '1935', 'ﬁrst', 'produced', 'commercially', '1938', 'it', 'still', 'one', 'major', 'commercial', 'nylons', 'produced', 'today', 'because', 'high', 'molecular', 'weight', 'required', 'polymers', 'possess', 'good', 'physical', 'properties', 'necessary', 'follow', 'exact', 'stoichiometry', 'reactants', 'condensation', 'to', 'achieve', 'practice', 'initially', 'form', '“nylon', 'salt”', 'prior', 'polymeriza-', 'tion', 'to', 'equimolar', 'quantities', 'adipic', 'acid', 'hexamethylene', 'diamine', 'combined', 'aqueous', 'environment', 'form', 'solutions', 'salt', 'the', 'end', 'point', 'controlled', 'electrochemically', 'an', 'alternate', 'procedure', 'combine', 'diacid', 'diamine', 'boiling', 'methanol', 'a', '11', 'adduct', 'precipitates', 'ﬁltered', 'dissolved', 'water', 'n', 'h2n', 'nh2', '6', 'n', 'hooc', 'cooh', '4', 'n', 'ooc', 'h3n', 'coo', '6', 'nh3', '4', 'a', '60–75', 'solution', 'salt', 'water', 'fed', 'reaction', 'kettle', 'in', 'typical', 'batch', 'desired', 'limit', 'molecular', 'weight', 'process', 'acetic', 'acid', 'may', 'also', 'added', '(10000–15000)', 'the', 'temperature', 'reaction', 'kettle', 'raised', '220(cid6)c', 'due', 'water', 'steam', 'reactor', 'internal', 'pressure', '20', 'atmospheres', 'develops', 'after', '1–2', 'h', 'temperature', 'raised', '270–280(cid6)c', 'some', 'steam', 'bled', 'maintain', 'internal', 'pressure', '20', 'atmospheres', 'the', 'temperature', 'maintained', 'bleeding', 'steam', 'continued', '2', 'h', 'during', 'period', 'internal', 'pressure', 'gradually', 'reduced', 'atmospheric', 'in', 'processes', 'vacuum', 'applied', 'point', 'reaction', 'kettle', 'high', 'molecular', 'weight', 'products', 'desired', 'when', 'reaction', 'complete', 'molten', 'polymer', 'ejected', 'kettle', 'applying', 'pressure', 'nitrogen', 'carbon', 'dioxide', 'in', 'one', 'continuous', 'process', 'desired', 'conditions', 'maintained', 'reaction', 'mixture', 'moves', 'various', 'zones', 'reactor', 'tubular', 'reactors', 'also', 'often', 'employed', 'continuous', 'polymerizations', 'nylon', '610', 'prepared', 'procedure', 'nylon', '66', 'salt', 'hexamethylene', 'diamine', 'sebacic', 'acid', 'nylon', '69', 'prepared', 'salt', 'hexamethylene', 'diamine', 'azelaic', 'acid', 'the', 'melting', 'points', 'various', 'nylons', 'formed', 'diamines', 'dicarboxylic', 'acids', 'one', 'commercial', 'polyamide', 'prepared', 'condensation', 'cycloaliphatic', 'diamine', 'twelve', 'carbon', 'dicarboxylic', 'acid', 'the', 'diamine', 'bis(p-aminocyclohexyl)methane', 'prepared', 'aniline', 'presented', 'table', '75', 'nh2', '2', 'o', 'h2n', 'nh2', 'h', 'ru', 'catalyst', 'h2n', 'nh2', '73', 'polyamides', '439', 'melting', 'point', '((cid6)c)', 'table', '75', 'melting', 'points', 'nylons', 'nylon', '46', '47', '49', '410', '510', '66', '69', '610', '612', '86', 'repeat', 'unit', '–nh–(ch2)4–nh–co–(ch2)4–co–', '–nh–(ch2)4–nh–co–(ch2)5–co–', '–nh–(ch2)4–nh–co–(ch2)7–co–', '–nh–(ch2)4–nh–co–(ch2)8–co–', '–nh–(ch2)5–nh–co–(ch2)8–co–', '–nh–(ch2)6–nh–co–(ch2)4–co–', '–nh–(ch2)6–nh–co–(ch2)7–co–', '–nh–(ch2)6–nh–co–(ch2)8–co–', '–nh–(ch2)6–nh–co–(ch2)10–co–', '–nh–(ch2)8–nh–co–(ch2)4–co–', '–nh–(ch2)8–nh–co–(ch2)8–co–', '–nh–(ch2)9–nh–co–(ch2)4–co–', '–nh–(ch2)9–nh–co–(ch2)8–co–', '–nh–(ch2)10–nh–co–(ch2)4–co–', '–nh–(ch2)10–nh–co–(ch2)8–co–', '–nh–(ch2)11–nh–co–(ch2)8–co–', '–nh–(ch2)12–nh–co–(ch2)4–co–', '–nh–(ch2)12–nh–co–(ch2)8–co–', '–nh–(ch2)13–nh–co–(ch2)11–co–', '96', '910', '106', '1010', '1110', '126', '1210', '1313', 'from', 'ref', '70', 'literature', 'sources', '810', 'salt', '204', '138', '175', '129', '183', '170', '153', '164', '125', '159', '142', '178', '153', '144', '157', 'polymer', '278', '233', '223', '236', '195', '250', '205', '209', '212', '235', '197', '204–205', '174–176', '230', '194', '168–169', '208–210', '171–173', '174', 'the', 'diamine', 'condensed', 'dodecanedioic', 'acid', 'obtained', 'cyclodode-', 'catriene', 'the', 'structure', 'polyamide', 'follows', 'h', 'n', 'h', 'n', '10', 'n', 'o', 'o', 'the', 'polymer', 'tm', '280–290(cid6)c', 'tg', '120(cid6)c', 'it', 'exhibits', 'lower', 'moisture', 'pick', 'nylons', '6', '66', 'increased', 'hardness', 'tensile', 'strength', 'though', 'lower', 'impact', 'strength', 'the', 'bulk', 'polymer', 'used', 'ﬁber', 'production', 'the', 'ﬁbers', 'trade', 'name', '“quiana”', 'claimed', 'exhibit', 'high', 'luster', 'silk', 'like', 'feel', 'a', 'number', 'copolyamides', 'manufactured', 'commercially', 'suit', 'various', 'needs', 'one', 'polyamide', 'formed', 'condensation', 'trimethylhexamethylene', 'diamine', 'terephthalic', 'acid', 'the', 'diamine', 'mixture', '244', '224', 'isomers', 'h2n', 'h2n', 'nh2', 'nh2', '440', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'the', 'mixture', 'two', 'isomers', 'synthesized', 'isophorone', 'according', 'following', 'scheme', 'o', 'o', 'ho', 'h', 'o', 'o', 'o', 'oh', 'ho', 'o', 'oh', 'o', 'the', 'mixture', 'isomers', 'trimethyl', 'adipic', 'acids', 'treated', 'ammonia', 'converted', 'amides', 'dehydrated', 'nitriles', 'reduced', 'amines', 'trimethyl', 'adipic', 'acid', 'isomers', 'nh3', 'n', 'h', 'h2n', 'n', 'n', 'n', 'nh2', 'h2n', 'nh2', 'this', 'polyamide', 'prepared', 'somewhat', 'differently', 'salts', 'diamine', 'isomers', 'terephthalic', 'acid', 'partially', 'polycondensed', 'reaction', 'completed', 'extrusion', '71,', 'melt', 'viscosity', 'polymer', 'high', 'the', 'product', 'amorphous', 'exhibits', 'greater', 'light', 'trasmittancy', 'it', 'melts', '200(cid6)c', 'sold', 'trade', 'name', 'trogamid', 't', 'many', 'commercial', 'nylon', 'copolymers', 'also', 'formed', 'melt', 'mixing', 'different', 'nylons', 'amide', 'interchange', 'reactions', 'occur', 'melt', 'conditions', 'at', 'ﬁrst', 'block', 'copolymers', 'form', 'prolonged', 'heating', 'stirring', 'results', 'formation', 'random', 'copolymers', 'nylon', 'copolymers', 'also', 'best', 'prepared', 'directly', 'mixed', 'monomers', 'nylon', 'polymers', 'generally', 'exhibit', 'high-impact', 'strength', 'toughness', 'good', 'ﬂexibility', 'abrasion', 'resistance', 'the', 'principal', 'structural', 'differences', 'many', 'nylons', 'length', 'aliphatic', 'segments', 'amide', 'linkages', 'as', 'result', 'differences', 'properties', 'depend', 'mainly', 'upon', 'amount', 'hydrogen', 'bonding', 'possible', 'functional', 'groups', 'amount', 'crystallinity', 'also', 'due', 'high', 'cohesive', 'energy', 'nylons', 'soluble', 'solvent', 'the', 'melt', 'viscosity', 'materials', 'however', 'generally', 'low', 'in', 'one', 'series', 'melting', 'points', 'polyamides', 'polymers', 'contain', 'even', 'numbers', 'methylene', 'groups', 'amide', 'linkages', 'fall', 'higher', 'curve', 'contain', 'odd', 'ones', '72.', 'this', 'due', 'crystalline', 'arrangement', 'polymeric', 'chains', '72', '73.', 'a', 'zigzag', 'planar', 'conﬁguration', 'polymers', 'even', 'number', 'methylene', 'linkages', 'allows', '50', 'functional', 'groups', 'form', 'hydrogen', 'bonds', 'this', 'conﬁguration', 'however', 'allows', 'polymers', 'odd', 'numbers', 'methylene', 'linkages', 'form', '100', 'hydrogen', 'bonding', '72', '73.', 'polyamides', 'like', 'nylon', '66', 'nylon', '610', 'arrange', 'pleated', 'sheets', 'crystallization', 'hydrogen', 'bonds', 'form', 'n–h', 'group', 'one', 'molecule', 'c¼o', 'moieties', 'neighboring', 'one', '73', 'polyamides', '732', 'fatty', 'polyamides', '441', 'the', 'fatty', 'polyamides', 'produced', 'reacting', 'di-', 'polyfunctional', 'amines', 'polybasic', 'acids', 'result', 'condensations', 'unsaturated', 'vegetable', 'oil', 'acids', 'the', 'commonly', 'used', 'amines', 'ethylene', 'diamine', 'diethylene', 'triamine', 'the', 'dicarboxylic', 'acids', 'synthesized', 'heating', 'mixtures', 'unsaturated', 'vegetable', 'fatty', 'acids', 'the', 'starting', 'materials', 'may', 'come', 'linseed', 'soybean', 'tung', '(also', 'called', 'china', 'wood)', 'oil', 'the', 'fatty', 'acids', 'heated', 'several', 'hours', '300(cid6)c', 'if', 'catalyst', 'used', 'heating', 'done', 'lower', 'temperature', 'after', 'condensation', 'volatile', 'fractions', 'removed', 'vacuum', 'the', 'residues', 'called', 'dimer', 'acids', 'condensed', 'amines', 'a', 'formation', 'one', 'dimer', 'acid', 'linoleic', 'acid', 'illustrated', 'follows', '5', 'cooh', '7', 'isomerized', 'linoleic', 'acid', '4', 'linoleic', 'acid', 'cooh', '7', '4', 'cooh', '7', '5', '5', 'h', 'n', '4', 'o', '7', 'n', 'h', 'hooc', 'o', 'n', 'h', '7', '7', 'h', 'n', 'the', 'polyamide', 'shown', 'dimer', 'acid', 'condensed', 'diamine', 'like', 'ethylene', 'diamine', 'illustrated', 'follows', 'two', 'types', 'fatty', 'polyamides', 'available', 'commercially', 'solid', 'liquid', 'the', 'solid', 'polymers', 'mostly', 'linear', 'condensation', 'products', 'diacids', 'diamines', 'range', 'molecular', 'weights', '2000', '15000', 'the', 'liquid', 'ones', 'highly', 'branched', 'low', 'molecular', 'weights', 'materials', 'produced', 'condensations', 'dimer', 'acids', 'triamines', 'even', 'higher', 'polyamines', '733', 'special', 'reactions', 'formation', 'polyamides', 'there', 'occasional', 'reports', 'literature', 'use', 'special', 'reactions', 'form', 'polyamides', 'one', 'synthesis', 'via', 'enamines', 'in', 'case', 'diisocyanates', 'condensed', 'cyclopentanone', 'enamines', 'morpholine', 'piperidine', '77:', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '442', 'n', 'o', 'n', 'n', 'o', 'n', 'n', 'o', 'o', 'n', 'o', 'o', 'n', 'h', 'n', 'h', 'n', 'the', 'molecular', 'weights', 'polymers', 'decrease', 'ring', 'sizes', 'ketone', 'components', 'increase', 'excess', 'diisocyanate', 'yields', 'branched', 'cross-linked', 'polymers', 'the', 'enamine', 'units', 'polymers', 'hydrolyzed', 'formic', 'acid', 'corresponding', 'ketones', '77.', 'a', 'similar', 'reaction', 'reported', 'aromatic', 'isocyanates', 'imidazoles', '14', 'tetramethylene-nn0-di', 'imidazoles', 'reacted', 'aromatic', 'polyisocyanates', 'form', 'thermoset', 'polyamides', '45:', 'n', 'n', 'n', 'n', '4', 'ar(nco)x', 'ar', 'n', 'h', 'n', 'o', 'n', 'n', 'h', 'n', 'ar', 'n', '4', 'o', 'another', 'route', 'polyamides', 'via', 'ritter', 'reaction', '78:', 'o', 'o', 'h2so4', 'hcooh', 'n', 'r', 'n', 'o', 'o', 'h', 'n', 'h', 'n', 'r', 'o', 'o', 'n', 'when', 'aromatic', 'di', 'nitriles', 'used', 'high', 'melting', 'polymers', 'good', 'thermal', 'stability', 'form', '78.', 'direct', 'polycondensation', 'various', 'dicarboxylic', 'acids', 'diamines', 'possible', '81', 'mild', 'conditions', 'using', 'catalytic', 'system', 'enol', 'phosphite', 'presence', 'imidazole', 'one', 'enol', 'phosphite', 'diethyl1-methyl-3-oxo-1-butenyl', 'phosphite', 'polymers', 'inherent', 'viscosities', '1–025', 'form', 'among', 'organic', 'bases', 'imidazole', 'effective', '84.', 'the', 'reaction', 'applicable', 'aliphatic', 'aromatic', 'dicarboxylic', 'acids', 'diamines', '73', 'polyamides', 'direct', 'polycondensation', 'diamine', 'dicarboxylic', 'acid', 'carried', 'supercritical', 'carbon', 'dioxide', 'preparation', 'nylon', '46', '84.', 'h2n', 'hooc', 'nh2', 'cooh', 'this', 'nylon', 'reported', 'melt', '210(cid6)c', 'h', 'n', 'o', 'n', 'h', '443', 'n', 'o', '734', 'aromatic', 'polyamides', 'in', 'section', 'discussed', 'wholly', 'aromatic', 'polyamides', 'also', 'mixed', 'polyamides', 'prepared', 'aromatic', 'diacids', 'aliphatic', 'diamines', 'vice', 'versa', 'one', 'material', 'already', 'described', 'sect', '732', 'another', 'one', 'called', 'nylon', '6t', 'formed', 'interfacial', 'polymerization', 'terephthaloyl', 'chloride', 'hexamethylene', 'diamine', 'the', 'polymer', 'good', 'heat', 'stability', 'strength', 'unaffected', 'heating', '185(cid6)c', '5', 'h', 'the', 'polyamide', 'melts', '370(cid6)c', 'when', 'hexamethylene', 'diamine', 'replaced', 'tetramethylene', 'diamine', 'melting', 'point', 'rises', '430(cid6)c', 'the', 'condensation', 'product', 'isophthalic', 'acid', 'tetramethylene', 'diamine', 'melts', '250(cid6)c', 'fully', 'aromatic', 'polyamides', 'form', 'reactions', 'aromatic', 'diacid', 'chlorides', 'aromatic', 'diamines', '79', '80.', 'an', 'example', 'formation', 'poly(m-phenylene', 'diamine', 'isophthalamide)', 'o', 'n', 'h', 'h', 'n', 'o', 'o', 'n', 'h', 'n', 'h', 'n', 'n', 'o', 'the', 'polymer', 'prepared', 'dimethylacetamide', 'isophthaloyl', 'chloride', 'm-phenylene', 'diamine', 'presence', 'acid', 'scavenger', 'room', 'temperature', 'o', 'o', 'cl', 'cl', 'h2n', 'nh2', 'polymer', '444', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'because', 'polymer', 'soluble', 'dimethylacetamide', 'containing', '5', 'licl', 'ﬁbers', 'spun', 'directly', 'solution', 'this', 'polyamide', 'melts', '371(cid6)c', 'degradation', 'it', 'ﬁre-resistant', 'the', 'ﬁbers', 'sold', 'trade', 'name', 'nomex', 'such', 'aromatic', 'polyamides', 'regular', 'structure', 'reported', 'possess', 'better', 'ﬂexibility', 'higher', 'temperature', 'resistance', '74', '75.', 'preparations', 'ordered', 'copolyamides', 'described', 'follows', 'nn-m-phenylene-bis(m-aminobenzamide)', 'formed', 'ﬁrst', 'reacted', 'isophthaloyl', 'chloride', 'interfacial', 'condensation', 'techniques', 'yield', 'product', 'melts', '410(cid6)c', 'o', 'o', 'o', 'o', 'h2n', 'nh2', 'cl', 'cl', 'n', 'h', 'h', 'n', 'n', 'h', 'o', 'o', 'h', 'n', 'n', 'h', 'n', 'h', 'o', 'n', 'o', 'the', 'condensation', 'carried', 'terephthaloyl', 'chloride', 'yields', 'polymer', 'melts', '450(cid6)c', 'preparations', 'many', 'wholly', 'aromatic', 'polyamides', 'aromatic', 'diacid', 'chlorides', 'aromatic', 'diamines', 'reported', 'literature', '79', '80.', 'in', 'addition', 'several', 'polymers', 'manufactured', 'fully', 'partially', 'substituted', '(para)', 'structures', 'they', 'carry', 'trade', 'name', 'ht4', 'as', 'described', 'previous', 'section', 'direct', 'polycondensation', 'various', 'nylon', 'salts', 'possible', 'mild', 'conditions', 'presence', 'polyphosphates', 'organic', 'bases', '84.', 'this', 'reaction', 'useful', 'forming', 'aromatic', 'polyamides', 'takes', 'place', 'also', 'presence', 'thionyl', 'chloride', '84:', 'o', 'o', 'cl', 'o', 'h2n', 'cl', 'o', 'nh2', 'socl', 'n', 'h', 'n', 'h', 'n', 'metal', 'salts', 'like', 'lithium', 'chloride', 'signiﬁcantly', 'enhance', 'reactions', 'carboxylic', 'acids', 'amines', 'promoted', 'triphenyl', 'phosphite', '82.', 'this', 'allows', 'direct', 'polycondensation', 'dicarboxylic', 'acids', 'diamines', 'self-condensation', 'p-aminobenzoic', 'acid', '82.', 'the', 'presence', 'solvent', 'markedly', 'enhances', 'reaction', 'best', 'results', 'obtained', 'n-methylpyrrolidone', 'high', 'molecular', 'weight', 'polyamides', 'form', 'mixed', 'solvents', 'like', 'pyridine', 'n-methylpyrrolidone', 'used', 'form', 'polyisophthalamides', '82.', 'this', 'combination', 'solvents', 'however', 'yields', 'low', 'molecular', 'weight', 'polyterephthalamides', 'on', 'hand', 'reaction', 'carried', 'presence', '73', 'polyamides', '445', 'polymeric', 'matrices', 'poly(ethylene', 'oxide)', 'poly(4-vinylpyridine)', 'high', 'molecular', 'weight', 'polyterephthalamides', 'form', '82:', 'ho', 'o', 'nh2', 'n', 'n', 'o', 'oh', 'nh2', 'o', 'o', 'p', 'n-methyl', 'pyrrolidone', 'licl', 'poly(n-vinylpyridine)', '3', 'h', 'n', 'n', 'oh', 'oh', 'o', 'p', '2', 'h', 'n', 'o', 'o', 'h', 'n', 'the', 'heck', 'reaction', 'extended', 'carbonylation', 'aromatic', 'dibromides', 'aromatic', 'diamines', 'presence', 'carbon', 'monoxide', '112.', 'high', 'molecular', 'weight', 'aromatic', 'polyamides', 'form', 'help', 'palladium', 'catalysis', 'h2n', 'o', 'nh2', 'br', 'br', 'h', 'n', 'o', '2co', 'pd', 'cat', '-hbr', 'n', 'o', 'the', 'polymerization', 'reaction', 'takes', 'place', 'homogeneous', 'dimethylacetamide', 'solution', 'cata-', 'lytic', 'amounts', 'pdcl2(pph3)2', 'hbr', 'scavenger', 'the', 'carbonylation', 'polycondensation', 'proceeds', 'rapidly', '115(cid6)c', 'almost', 'complete', '15', 'h', '112.', 'this', 'reaction', 'also', 'used', 'prepare', 'many', 'aromatic-aliphatic', 'polyamides', 'corresponding', 'aliphatic', 'diamines', 'aromatic', 'dibromides', 'palladium', 'relatively', 'high-priced', 'catalyst', 'would', 'preferable', 'lower-priced', 'nickel', 'catalyst', 'could', 'used', 'instead', 'all', 'attempts', 'however', 'form', 'polymers', 'nickel-catalyzed', 'carbon-', 'ylation', 'polycondensations', 'aromatic', 'diamines', 'aromatic', 'dibromides', 'failed', 'yield', 'high', 'molecular', 'weight', 'materials', '112.', 'trimethylsilyl-substituted', 'amines', 'undergo', 'variety', 'reactions', 'electrophiles', '113.', 'this', 'reaction', 'extended', 'recently', 'preparations', 'high', 'molecular', 'weight', 'aromatic', 'polyamides', 'low', 'temperature', 'solution', 'polycondensation', 'n-trimethylsilylated', 'aromatic', 'diamines', 'condensed', 'aromatic', 'diacid', 'chlorides', '114', '(cid1)10(cid6)c', 'amide', 'solvent', 'h', 'n', 'h', 'n', 'o', 'o', 's', 'ar', 'si', 'cl', 'ar', 'cl', '-(ch3)3sicl', 'h', 'n', 'h', 'n', 'ar', 'ar', 'n', 'o', 'o', '446', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'preparations', 'poly-p-phenyleneterephthalamide', 'polycondensations', 'n-silylated', 'diamine', 'proceed', 'rapidly', 'parent', 'diamine', '113.', 'in', 'addition', 'products', 'higher', 'molecular', 'weights', 'similar', 'commercial', 'materials', 'made', 'parent', 'diamine', 'sold', 'trade', 'name', 'kevlar', 'silylated', 'diamines', 'also', 'condense', 'diphenyl', 'esters', 'aromatic', 'dicarboxylic', 'acids', '114:', 'h', 'n', 'n', 'si', 'h', 'n', 'si', 'n', 'o', 'o', 'o', 'o', 'o', 'h', 'n', '-phosi(ch3)3', '100', 'oc', 'h', 'n', 'o', 'n', 'o', 'o', 'the', 'product', 'shown', 'aromatic', 'polyamide', 'reported', 'sufﬁciently', 'high', 'molecular', 'weight', 'useful', 'material', '113.', 'an', 'example', 'specialty', 'aromatic', 'polyamides', 'ﬂuorinated', 'polyamide', 'it', 'prepared', 'attempt', 'form', 'polymer', 'superior', 'heat', 'stability', 'resistance', 'hydrolytic', 'attacks', '76:', 'f', 'f', 'o', 'o', 'f', 'f', 'h', 'n', 'n', 'o', 'mitsuru', 'et', 'al', '113', 'reported', 'direct', 'synthesis', 'nomex', 'aromatic', 'polyamide', 'mentioned', 'solid', 'state', 'polycondensation', 'h2n', 'nh2', '∆', 'cooh', 'o', 'o', 'n', 'h', 'h', 'n', 'n', 'the', 'reactions', 'carried', 'melting', 'state', 'reactants', 'ﬁrst', '360(cid6)c', 'oligomerization', '260(cid6)c', '1', 'h', '74', 'aromatic', 'polyamide-imides', 'aromatic', 'polyester-imides', '447', '74', 'aromatic', 'polyamide-imides', 'aromatic', 'polyester-imides', 'the', 'aromatic', 'polyamide-imides', 'related', 'aromatic', 'polyamides', 'described', 'previous', 'section', 'aliphatic', 'materials', 'type', 'reported', 'originally', '1947', 'they', 'formed', 'reacting', 'tricarboxylic', 'acids', 'diamines', '83:', 'o', 'oh', 'o', 'o', 'n', 'ho', 'oh', 'r', 'n', 'h2n', 'nh2', 'o', 'oh', 'o', 'o', 'r', 'h2n', 'n', 'h', 'r', 'n', 'nh2', '∆', 'n', 'h', 'o', 'r', 'h2n', 'n', 'h', 'n', 'o', 'o', 'nh2', 'r', 'n', 'the', 'aliphatic', 'polyamide-imides', 'prepared', 'date', 'don’t', 'desirable', 'properties', 'when', 'aromatic', 'diacids', 'employed', 'however', 'products', 'exhibit', 'good', 'heat', 'stability', 'toughness', 'this', 'led', 'development', 'number', 'useful', 'materials', 'three', 'general', 'methods', 'employed', 'form', 'aromatic', 'polyamide-imides', '88.', 'the', 'ﬁrst', 'one', 'consists', 'initial', 'reaction', 'mole', 'diacid', 'chloride', 'two', 'moles', 'diamine', 'the', 'product', 'reacted', 'dianhydride', 'condensed', 'imide', 'nh2', 'nh2', 'cl', 'cl', 'o', 'o', '2', 'o', 'o', 'o', 'h', 'n', 'h', 'n', 'interfacial', 'h2n', 'polycondensation', 'o', 'o', 'h', 'n', 'nh2', 'h', 'n', 'o', 'o', 'o', 'h', 'n', 'h', 'n', 'h', 'n', 'o', 'o', 'h', 'n', 'o', 'o', 'o', 'o', 'o', 'hooc', 'h', 'n', 'o', 'cooh', 'n', 'o', 'o', 'o', 'n', 'o', 'h2o', 'in', 'second', 'method', 'dianhydride', 'prereacted', 'excess', 'diamine', 'the', 'product', 'reacted', 'diacid', 'chloride', 'interfacial', 'polymerization', 'technique', '448', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'in', 'third', 'method', 'anhydride', 'like', 'instance', 'trimellitic', 'anhydride', 'condensed', 'diamine', 'form', 'preliminary', 'condensate', 'an', 'acetylated', 'diamine', 'used', 'initial', 'condensation', '85:', 'o', 'o', 'r', 'n', 'h', 'n', 'h', 'hooc', '2', 'o', 'o', 'r', 'n', 'h', 'n', 'h', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', '125-325', 'oc', '2ch3cooh', 'this', 'followed', 'reaction', 'different', 'diamine', 'o', 'o', 'n', 'o', 'o', 'o', 'r', 'n', 'h', 'n', 'h', 'o', 'o', 'n', 'n', 'o', 'o', 'o', 'o', 'o', 'r', 'n', 'h', 'n', 'h', 'the', 'techniques', 'applied', 'preparations', 'polyester-imides', 'a', 'diester', 'formed', 'ﬁrst', 'trimellitic', 'anhydride', 'n', 'the', 'product', 'condensed', 'diamine', 'form', 'polyester-imide', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'nh2', 'nh2', 'o', 'n', 'o', 'o', 'o', 'o', 'o', 'n', 'n', 'o', '74', 'aromatic', 'polyamide-imides', 'aromatic', 'polyester-imides', '449', 'several', 'polyamide-imides', 'available', 'commercially', 'two', 'based', 'trimellitic', 'anhydride', 'methylene', 'dianiline', 'they', 'however', 'prepared', 'two', 'different', 'processes', 'in', 'ﬁrst', 'one', 'polymer', 'formed', 'anhydride', 'diamine', 'nh2', 'n', 'h', 'h', 'n', 'n', 'n', 'o', 'ho', 'hooc', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'h2n', 'o', 'n', 'h', 'cooh', 'n', 'o', 'o', 'n', 'o', 'o', 'in', 'second', 'one', 'trimellitic', 'anhydride', 'reacted', 'diisocyanate', 'n', 'o', 'n', 'n', 'o', 'n', 'co2', 'n', 'h', 'n', 'another', 'polyamide-imide', 'formed', 'reaction', 'trimellitic', 'anhydride', 'isophthalic', 'acid', 'diisocyanate', 'it', 'following', 'structure', 'h', 'n', 'o', 'o', 'n', 'h', 'n', 'h', 'o', 'o', 'n', 'o', 'n', 'most', 'polyamide-imides', 'heat-resistant', 'polyimides', 'discussed', 'next', 'section', 'they', 'however', 'easier', 'process', 'the', 'polyester-imides', 'might', 'considered', '“upgraded”', 'polyesters', 'though', 'properties', 'vary', 'depending', 'upon', 'chemical', 'structure', '86.', '450', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '75', 'polyimides', 'it', 'interesting', 'formation', 'linear', 'aromatic', 'polyimide', 'observed', 'early', '1908', 'polyimide', 'formed', 'heating', '4-aminophthalic', 'anhydride', 'n', 'h2n', 'o', 'o', 'o', 'n', 'o', 'o', 'n', 'formation', 'polymers', 'however', 'time', 'considered', 'undesirable', 'material', 'pursued', '87.', 'it', 'learned', 'since', 'imide', 'link', 'thermally', 'stable', 'amide', 'one', 'polyimides', 'useful', 'materials', 'many', 'polyimides', 'developed', 'since', 'aromatic', 'structures', 'polymeric', 'backbone', 'raise', 'melting', 'temperatures', 'yield', 'stiffer', 'tougher', 'materials', 'most', 'sought-after', 'polyimides', 'therefore', 'products', 'aromatic', 'tetra', 'acids', '(or', 'dianhydrides)', 'aromatic', 'diamines', 'many', 'commercial', 'preparations', 'aromatic', 'polyimides', 'include', 'preliminary', 'step', 'forming', 'polyamic', 'acids', 'ﬁrst', '88:', 'o', 'o', 'n', 'o', 'r', 'o', 'h2n', 'r', 'nh2', 'o', 'o', 'cast', 'ﬁlm', 'o', 'o', 'ho', 'r', 'o', 'o', 'oh', 'n', 'h', 'h', 'n', 'r', 'n', 'o', 'o', 'ho', 'r', 'o', 'o', 'oh', 'n', 'h', 'h', 'n', 'r', 'n', 'o', 'o', 'o', 'o', 'n', 'r', 'n', 'r', 'n', 'this', 'followed', 'imidation', 'often', 'polymer', 'applied', 'substrate', 'coating', 'the', 'polyamic', 'acids', 'usually', 'prepared', 'solution', 'suitable', 'solvents', 'nn-dimethyl-', 'formamide', 'dimethyl', 'sulfoxide', 'n-methyl-2-pyrrolidone', 'the', 'reactions', 'require', 'anhydrous', 'like', '50(cid6)c', '(or', 'lower)', 'some', 'however', 'need', 'high', 'conditions', 'relatively', 'low', 'temperatures', 'temperatures', 'high', '175(cid6)c', '88.', 'the', 'two', 'reagents', 'combined', 'solution', 'the', 'order', 'addition', 'reagent', 'purity', 'inﬂuence', 'molecular', 'weight', 'products', 'may', 'range', '13000', '55000', '88.', 'some', 'imidation', 'accompanies', 'ﬁrst', 'step', 'it', 'desirable', 'polyamic', 'acid', 'formation', 'degree', 'imidation', 'exceed', '50.', 'the', 'step', 'conversion', 'polyamic', 'acids', 'polyimides', 'take', 'place', '300(cid6)c', 'thin', 'ﬁlms', 'with', 'cyclizing', 'agents', 'however', 'take', 'place', 'much', 'lower', 'temperatures', '88.', 'lozano', 'et', 'al', 'reported', 'improved', 'method', 'forming', 'high', 'molecular', 'weight', 'polyimides', '88.', 'they', 'used', 'situ', 'sililation', 'diamines', 'carry', 'reaction', 'trimethylchlorosilane', 'added', 'diamine', 'solution', 'prior', 'addition', 'electrophilic', 'monomer', 'commercially', 'commonly', 'used', 'aromatic', 'dianhydrides', 'pyromellitic', 'dianhydride', 'benzophenone', 'tetracarboxylic', 'dianhydride', 'the', 'common', 'amines', 'industrial', 'practices', '75', 'polyimides', '451', 'meta-phenylene', 'diamine', 'methylene', 'dianiline', 'oxy-dianiline', 'a', 'typical', 'polyimide', 'preparation', 'shown', 'follows', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'h', 'n', 'oh', 'o', 'o', 'ho', 'o', 'n', 'h2n', 'o', '2', 'h', 'n', 'n', 'polyamic', 'acid', 'o', 'o', 'n', 'n', 'o', 'o', 'o', 'n', 'the', 'shown', 'polymer', 'melts', '600(cid6)c', 'heat', 'stable', '500(cid6)c', 'inert', 'atmosphere', 'it', 'sold', 'trade', 'name', 'kapton', 'a', 'different', 'polyether-imide', 'sells', 'trade', 'name', 'ultem', 'another', 'route', 'polyimides', 'reactions', 'diisocyanates', 'dianhydrides', '89', '90:', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'n', 'o', 'o', 'o', 'n', 'o', 'o', 'n', 'o', 'o', 'o', 'n', 'o', 'o', 'n', 'o', 'n', 'o', 'n', 'o', 'n', 'the', 'reaction', 'kept', '120', '200(cid6)c', '91.', 'in', 'place', 'isocyanates', 'also', 'possible', 'use', 'aldimines', 'ketimines', 'well', 'silylated', 'diamines', '452', 'n', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'r', 'n', 'r', 'r', 'r', 'n', 'n', 'r', 'n', 'si', 'si', 'r', 'n', 'n', 'si', 'si', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'n', 'r', 'n', 'o', 'o', 'o', 'o', 'o', 'n', 'r', 'n', 'o', 'o', 'n', 'o', 'o', 'n', 'r', 'n', 'r', '2n', 'o', 'si', 'si', 'when', 'silylated', 'diamines', 'used', 'trimethylsilyl', 'esters', 'polyamic', 'acids', 'formed', 'ﬁrst', 'desilylated', 'methanol', 'due', 'increased', 'solubility', 'silylated', 'aromatic', 'amines', 'initial', 'condensations', 'formations', 'polyamic', 'acid', 'trimethylsilyl', 'esters', 'done', 'various', 'solvents', 'yielding', 'high', 'molecular', 'weight', 'polymers', 'the', 'highest', 'molecular', 'weights', 'obtained', 'dimethyla-', 'cetamide', '50(cid6)c', 'other', 'solvents', 'like', 'tetrahydrofuran', 'chloroform', 'used', 'well', 'though', 'appear', 'yield', 'slightly', 'lower', 'molecular', 'weight', 'products', '91.', 'the', 'ﬁlms', 'silylated', 'precursors', 'polyamic', 'acid', 'convert', 'directly', 'heat', 'treatment', 'yellow', 'transparent', 'tough', 'ﬁlms', 'aromatic', 'polyimides', 'elimination', 'trimethylsilanol', '91.', 'other', 'preparations', 'polyimides', 'include', 'use', 'di-half', 'esters', 'tetracarboxylic', 'acids', '92:', 'ho', 'ho', 'o', 'n', 'o', 'o', 'o', 'oh', 'oh', 'r', 'n', 'h2n', 'nh2', 'o', 'o', 'n', 'o', 'o', 'n', 'r', 'n', '2n', 'h2o', '2n', 'roh', 'polyimides', 'also', 'prepared', 'reactions', 'diimides', 'dihalides', '93:', 'h', 'n', 'o', 'o', 'o', 'o', 'n', 'h', 'cl', 'cl', 'o', 'o', 'n', 'o', 'n', 'o', 'n', '75', 'polyimides', '453', 'reactions', 'bis', 'maleimides', 'compounds', 'containing', 'active', 'hydrogens', 'also', 'lead', 'formations', 'polyimides', '94:', 'n', 'n', 'n', 'n', 'n', 'n', 'o', 'o', 'n', 'r', 'n', 'o', 'o', 'o', 'o', 'n', 'r', 'n', 'o', 'o', 'n', 'also', 'unsaturated', 'diimides', 'reacted', 'sulfur', 'halides', 'form', 'polyimides', '95:', 'n', 'n', 'ar', 'n', 'o', 'o', 'o', 'o', 'sxcl2', 'o', 'o', 'o', 'o', 'n', 'ar', 'n', 'sx', 'cl', 'n', 'x', '1', '2', 'n', 'n', 'n', 'n', 'cl', 'diepoxides', 'add', 'pyromelitimide', 'presence', 'base', 'form', 'polyimides', 'pendant', 'hydroxyl', 'groups', '96:', 'n', 'h', 'n', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'n', 'o', 'n', 'h', 'o', 'o', 'n', 'o', 'o', 'o', 'oh', 'oh', 'o', 'n', 'tertiary', 'amines', 'quaternary', 'ammonium', 'halides', 'catalyze', 'reaction', 'acetylation', 'pendant', 'hydroxyl', 'groups', 'product', 'yields', 'polymers', 'soluble', 'solvents', 'like', 'dioxane', 'dimethylformamide', '454', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'pyromelitimide', 'also', 'add', 'double', 'bonds', 'form', 'polyimides', '97:', 'hn', 'n', 'o', 'o', 'o', 'o', 'n', 'n', 'h', 'n', 'base', 'o', 's', 'o', 'o', 's', 'o', 'n', 'o', 'o', 'n', 'o', 'o', 'hν', 'polyimides', 'also', 'form', 'photo', 'additions', 'aliphatic', 'aromatic', 'bis', 'maleimides', 'benzene', '98.', 'the', 'reactions', 'involve', '2', '2', 'cycloadditions', 'yield', 'homoannular', 'diene', 'intermediates', 'diels–alder', 'additions', 'follow', 'result', 'formations', 'polymers', 'n', 'r', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'n', 'r', 'o', 'o', 'n', 'n', 'r', 'n', 'o', 'o', 'o', 'o', 'head-to-head', 'tail-to-tail', 'structire', 'diels–alder', 'reactions', 'yield', 'polyimides', 'instance', 'following', '99:', 'n', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'n', 'o', 'n', '75', 'polyimides', '455', 'also', 'nn-bis(ethoxycarbonyl)', 'pyromelitimide', 'condenses', 'diamines', 'yield', 'polyimides', '100:', 'o', 'n', 'n', 'o', 'c2h5', 'c2h5', 'h2n', 'nh2', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'h', 'n', 'n', 'h', 'c2h5', 'o', 'o', 'o', 'o', 'n', 'o', '2n', 'c2h5', 'n', 'o', 'nh2', 'the', 'ﬁrst', 'step', 'preparation', 'takes', 'place', 'solution', 'after', 'casting', 'ﬁlm', 'second', 'step', 'takes', 'place', '240(cid6)c', 'vacuum', 'interesting', 'polyimides', 'also', 'form', 'reactions', '220660-', 'biphenyltetracarboxylic', 'acid', 'anhydride', '101', 'aromatic', 'diamines', 'like', '440-diaminodiphenyl', 'ether', 'o', 'o', 'o', 'o', 'c2h5', 'o', 'n', 'o', 'h', 'n', 'n', 'h', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'h2n', 'o', 'nh2', 'o', 'n', 'o', 'o', 'n', 'as', 'previous', 'cases', 'polyamic', 'acid', 'forms', 'ﬁrst', 'solution', 'approximately', '40(cid6)c', 'it', 'converted', 'polyimide', 'heating', 'acetic', 'anhydride', 'reﬂux', '18', 'h', 'heterocyclic', 'dianhydrides', 'like', 'pyrazinetetracarboxylic', 'dianhydride', 'also', 'react', 'diamines', 'form', 'polyimides', '102.', 'such', 'polymers', 'harder', 'form', 'however', 'two', 'types', 'thermoset', 'polyimides', 'currently', 'prepared', 'commercially', 'they', 'based', 'low', 'molecular', 'weight', 'bis', 'imides', 'bis', 'maleimides', 'bis-5-norbornene-23-dicarboximides', 'due', 'unsaturations', 'materials', 'cross-link', 'free-radical', 'mechanism', 'tight', 'networks', 'michael', 'type', 'additions', 'primary', 'secondary', 'amines', 'bis', 'maleimides', 'often', 'used', 'chain-extend', 'cross-linking', 'this', 'reduces', 'cross-linking', 'density', 'brittleness', '115.', 'the', 'materials', 'designated', 'term', 'pmr', 'polymerizable', 'monomeric', 'reactants', 'ueda', 'et', 'al', '102', 'synthesized', 'polyimide', '36-bis-(4-aminophenylenesulfanyl)pyridazine', 'nn', 's', 's', 'o', 'o', 'n', 's', 's', 's', 'n', 'o', 'o', 'n', 'the', 'polymer', 'high', 'refractive', 'index', 'high', 'transparency', 'aimed', 'optical', 'devices', '456', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '76', 'polyethers', 'polyethers', 'form', 'chain-growth', 'polymerizations', 'carbonyl', 'compounds', 'ring', 'opening', 'polymerizations', 'cyclic', 'ethers', 'acetals', 'discussed', 'chap', '6', 'in', 'section', 'discussed', 'poly(phenylene', 'oxide)s', 'phenoxy', 'resins', '761', 'poly(phenylene', 'oxide)s', 'these', 'polymers', 'known', 'good', 'thermal', 'stability', 'good', 'mechanical', 'properties', 'commercially', 'aromatic', 'polyethers', 'prepared', 'oxidative', 'coupling', 'phenols', '102.', 'to', 'obtain', 'linear', 'polymers', 'achieve', 'high', 'molecular', 'weights', '2', '6', 'positions', 'phenol', 'must', 'protected', 'substituents', 'this', 'causes', 'aromatic', 'rings', 'couple', '14', 'positions', 'when', '26-dimethylphenol', 'used', 'reaction', 'takes', 'place', 'room', 'temperature', 'oxygen', 'bubbled', 'solution', 'phenol', 'presence', 'amine-cuprous', 'salt', 'catalyst', 'n', 'oh', 'cucl-pyridine', 'o2', 'o', 'n', 'phenols', 'halogen', 'substituents', 'require', 'higher', 'temperatures', 'large', 'substituents', 'lead', 'carbon', 'carbon', 'coupling', 'instead', 'carbon', 'oxygen', 'the', 'active', 'catalyst', 'believed', 'basic', 'cupric', 'salt', 'forms', 'oxidation', 'cuprous', 'chloride', 'followed', 'complexation', 'two', 'molecules', 'amine', '102:', 'o', 'o', 'cl', 'cu', 'oh', 'nr3', 'nr3', 'this', 'step-growth', 'polymerization', 'involving', 'phenoxy', 'radicals', 'the', 'polymer', 'formation', 'illustrated', 'follows', 'cl', 'cu', 'oh', 'oh', 'cl', 'cu', 'o', 'oh', 'h2o', 'nr3', 'nr3', 'nr3', 'nr3', '76', 'polyethers', '457', 'dissociation', 'leads', 'aryloxy', 'radicals', 'two', 'new', 'radicals', 'couple', 'quinone', 'ketals', 'formed', 'initially', 'they', 'dissociate', 'yield', 'original', 'aryloxy', 'radicals', 'couple', '102:', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'formation', 'aryloxy', 'radicals', 'intermediates', 'established', 'esr', 'spectroscopy', 'studies', 'showed', 'presence', 'monomeric', 'polymeric', 'radicals', 'reaction', 'mixture', '103.', 'coupling', 'occurs', 'two', 'paths', 'one', 'rearrangements', 'one', 'redistribution', 'in', 'redistribution', 'process', 'two', 'aryloxy', 'radicals', 'couple', 'yield', 'unstable', 'quinone', 'ketal', 'shown', '102.', 'this', 'ketal', 'decomposes', 'rapidly', 'either', 'back', 'original', 'aryloxy', 'radicals', 'two', 'different', 'aryloxy', 'radicals', 'follows', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'quinnnone', 'ketal', 'o', 'o', 'o', 'o', 'oh', '458', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'the', 'redistribution', 'process', 'leads', 'production', 'polymers', 'low', 'molecular', 'weight', 'radicals', 'it', 'appears', 'process', 'unlikely', 'take', 'place', 'high', 'molecular', 'weight', 'radicals', 'many', 'steps', 'involved', 'production', 'monomer', 'radicals', 'quinone', 'ketals', 'intermediates', 'rearrangement', 'the', 'carbonyl', 'oxygen', 'ketal', 'within', 'bonding', 'distance', 'para', 'position', 'next', 'succeeding', 'benzene', 'ring', '104.', 'the', 'rearrangement', 'therefore', 'give', 'rise', 'new', 'ketal', 'second', 'ring', 'carries', 'carbonyl', 'oxygen', 'the', 'carbonyl', 'oxygen', 'ﬁnally', 'ends', 'terminal', 'unit', '104', 'reduced', 'oh', 'o', 'o', 'o', 'rearranegement', 'o', 'o', 'h', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'oh', 'the', 'quinone', 'rearranges', 'phenol', 'enolization', 'the', 'product', 'identical', 'one', 'obtained', 'direct', 'head', 'tail', 'coupling', 'two', 'aryloxy', 'radicals', '102.', 'it', 'important', 'realize', 'processes', 'redistribution', 'rearrangement', 'reactions', 'occur', 'within', 'polymer', 'molecule', 'at', 'point', 'rearrangement', 'may', 'dissociation', 'aryloxy', 'radicals', 'also', 'redistribution', 'occur', 'transfer', 'single', 'unit', 'rearrangement', 'followed', 'dissociation', 'allows', 'number', 'monomer', 'units', 'transferred', 'essentially', 'single', 'step', '26-diphenylphenol', '26-dimethylphenol', 'copolymerize', 'oxidative', 'coupling', '104.', 'if', 'diphenyl', 'derivative', 'polymerized', 'ﬁrst', 'subsequently', 'dimethyl', 'derivative', 'added', 'reaction', 'mixture', 'block', 'copolymers', 'form', 'if', 'however', 'order', 'reversed', 'phenols', 'polymerized', 'together', 'random', 'copolymer', 'results', '104.', 'poly(phenylene', 'oxide)s', 'also', 'formed', 'oxidative', 'displacement', 'bromides', '4-bromo-', '26-dimethylphenol', '102', '105.', 'compounds', 'like', 'potassium', 'ferricyanide', 'lead', 'oxide', 'silver', 'oxide', 'catalyze', 'reaction', 'n', 'br', 'oh', 'k2frcno6', 'koh', 'o', 'n', 'poly(phenylene', 'oxide)s', 'also', 'form', 'photodecomposition', 'benzene-14-diazooxides', '102:', 'n', 'n2', 'hν', 'o', 'o', 'n', 'n2', 'oxidative', 'coupling', 'process', 'used', 'commercially', 'although', 'poly(phenylene', 'oxide)', 'important', 'commercial', 'material', 'initially', 'processing', 'problem', 'material', 'ﬁrst', 'introduced', 'currently', 'large', 'portion', 'polymer', 'sold', 'blend', 'polystyrene', '(probably', 'high-impact)', 'make', 'attractive', 'economically', 'easier', 'process', 'the', 'ratios', 'poly(26-', 'dimethylphenylene', 'oxide)', 'polystyrene', 'range', 'approximately', '11', '12', 'materials', 'sold', 'trade', 'name', 'noryl', 'a', 'ﬁre', 'retardant', 'grade', 'containing', '5', 'additive', 'believed', 'triphenyl', 'phosphate', 'also', 'market', '77', 'polyacetals', 'polyketals', '762', 'phenoxy', 'polymers', '459', 'these', 'materials', 'part', 'technology', 'epoxy', 'resins', 'discussed', 'separate', 'section', 'chapter', 'the', 'polymers', 'bridge', 'gap', 'thermosetting', 'resins', 'thermoplastic', 'polymers', 'used', 'forms', 'commercially', '116.', 'an', 'idealized', 'picture', 'phenoxy', 'polymers', 'shown', 'follows', 'o', 'o', 'n', 'oh', 'the', 'polymer', 'forms', 'caustic', 'catalyzed', 'condensations', 'diphenols', 'epichlorohydrin', 'any', 'diphenol', 'combination', 'diphenols', 'undergo', 'reaction', 'in', 'commercial', 'practice', 'however', 'mainly', '44-isopropylidinediphenol', 'commonly', 'called', 'bisphenol', 'a', 'used', 'theoretically', 'phenoxy', 'resins', 'form', 'equimolar', 'reactions', 'epichlorohydrin', 'diphenol', 'there', 'however', 'number', 'side', 'reactions', 'accompany', 'condensation', 'to', 'get', 'around', 'obtain', 'high', 'molecular', 'weight', 'polymers', 'syntheses', 'carried', 'two', 'steps', 'in', 'ﬁrst', 'one', 'excess', 'epichlorohydrin', 'used', 'form', 'diepoxide', 'ho', 'oh', 'cl', '2', 'o', 'oh', 'o', 'o', 'in', 'second', 'step', 'equimolar', 'quantities', 'diepoxide', 'reacted', 'diphenol', 'o', 'oh', 'n', 'o', 'o', 'o', 'o', 'o', 'ho', 'n', 'oh', 'o', 'o', 'n', 'both', 'reactions', 'conducted', 'solution', 'methyl', 'ethyl', 'ketone', 'choice', 'solvent', 'the', 'commercial', 'resins', 'range', 'molecular', 'weights', '15000', '200000', '77', 'polyacetals', 'polyketals', 'polyacetals', 'polyketals', 'polyethers', 'form', '(1)', 'condensations', 'glycols', 'carbonyl', 'compounds', '(2)', 'exchange', 'reactions', 'acetals', 'ketals', '(3)', 'additions', 'diols', 'dialkenes', '109:', 'o', 'n', 'r', 'r', 'ho', 'n', 'r', 'oh', 'r', 'r', 'r', 'o', 'o', 'n', '460', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'an', 'acid-catalyzed', 'exchange', 'reaction', 'glycols', 'acetals', 'yields', 'polyacetals', 'follows', 'n', 'r', 'o', 'o', 'r', 'ho', 'n', 'r', 'oh', 'o', 'o', 'r', 'n', '2nroh', 'these', 'reactions', 'often', 'lead', 'cyclic', 'acetals', 'interfere', 'formation', 'high', 'molecular', 'weight', 'products', 'useful', 'polyacetals', 'however', 'formed', 'pentaerythritol', 'acetals', 'dialdehydes', 'ho', 'n', 'ho', 'oh', 'oh', 'ro', 'n', 'ro', 'o', 'o', 'r', 'or', 'or', 'h', 'h', 'r', 'n', 'o', 'n', 'o', 'o', 'o', 'acetals', 'generally', 'exhibit', 'poor', 'resistance', 'hydrolytic', 'attack', 'some', 'however', 'much', 'resistant', 'others', 'depending', 'upon', 'glycol', 'the', 'following', 'formal', 'reported', 'exhibit', 'good', 'hydrolytic', 'stability', 'acidic', 'basic', 'conditions', '109:', 'pentaerythritol', 'yields', 'spiro', 'polymers', 'reaction', 'the', 'products', 'offer', 'superior', 'thermal', 'resistance', '114:', 'ho', 'oh', 'n', 'ho', 'oh', 'on', 'o', 'o', 'o', 'o', 'o', 'n', 'the', 'addition', 'diols', 'dialkenes', 'illustrated', 'addition', 'glycol', 'divinyl', 'ether', 'n', 'o', 'o', 'r', 'n', 'ho', 'oh', 'r', 'h', 'r', 'o', 'o', 'o', 'r', 'n', '78', 'poly(p-xylylene)s', '461', 'commercially', 'large', 'volume', 'acetals', 'produced', 'polymerizations', 'formaldehyde', 'ring', 'opening', 'polymerizations', 'trioxane', 'these', 'reactions', 'discussed', 'chapters', 'three', 'four', 'two', 'materials', 'manufactured', 'country', 'one', 'homopolymer', 'formaldehyde', 'polyoxymethylene', 'it', 'sold', 'trade', 'name', 'delrin', 'the', 'material', 'end-capped', 'prevent', 'depolymerization', 'acetylating', 'terminal', 'hydroxyl', 'groups', 'the', 'one', 'copolymer', 'formaldehyde', 'small', 'quantities', 'comonomer', 'sold', 'trade', 'name', 'celcon', 'copolymerization', 'accomplishes', 'objective', 'end-capping', 'it', 'also', 'makes', 'resistant', 'attacks', 'bases', 'polyoxymethylene', 'highly', 'crystalline', 'this', 'due', 'easy', 'packing', 'simple', 'polar', 'chains', 'the', 'crystallinity', 'estimated', '60–77.', 'polyoxymethylene', 'strong', 'material', 'good', 'resistance', 'creep', 'fatigue', 'abrasion', '78', 'poly(p-xylylene)s', 'the', 'original', 'preparation', 'p-xylylene', 'carried', 'vacuum', 'pyrolysis', 'p-xylene', '900–950(cid6)c', 'the', 'intermediate', 'p-xylylene', 'polymerizes', 'spontaneously', 'upon', 'condensation', 'cooler', 'surfaces', '106:', 'n', '∆', 'vacuum', 'n', 'n', 'the', 'process', 'improved', 'using', 'di-p-xylylene', 'intermediate', '106.', 'this', 'dimer', 'converts', 'polymer', 'milder', 'conditions', 'quantitatively', 'both', 'methylene', 'bridges', 'cleave', 'form', 'p-xylylene', 'reactive', 'intermediate', '600', 'oc', 'vacuuma', '(', '1', 'mm', 'hg)', '2', 'ch2', 'n', 'ch2', 'the', 'molecular', 'weights', 'polymers', 'estimated', 'high', '500000', 'the', 'total', 'process', 'sometimes', 'called', 'transport', 'polymerization', 'poly(p-xylylene)', 'ﬁlms', 'produced', 'commer-', 'cially', 'tm', 'polymer', 'crystalline', '400(cid6)c', 'carries', 'trade', 'name', 'perylene', 'films', 'poly(p-xylylene)', 'fair', 'thermal', 'stability', 'brittle', 'exhibit', 'good', 'chemical', 'resistance', 'good', 'electrical', 'insulators', 'pyrolysis', 'xylene', 'steam', '950(cid6)c', 'yields', 'dimer', 'intermediate', 'the', 'yield', 'reported', '15', '88.', 'it', 'possible', 'form', 'substituted', 'poly(p-xylylene)s', 'starting', 'substituted', 'structures', 'among', 'compounds', 'reported', 'chlorinated', 'brominated', 'compounds', 'well', 'containing', 'alkyl', 'cyano', 'acetyl', 'carboxymethyl', 'derivatives', 'when', 'di-p-xylylenes', 'unsym-', 'metrically', 'substituted', 'two', 'homogeneous', 'polymers', 'form', 'pyrolysis', 'two', 'condense', 'spontaneous', 'polymerization', 'two', 'different', 'temperatures', '88.', 'n', 'n', 'cl', 'cl', 'n', '462', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'poly(p-xylylene)s', 'also', 'prepared', 'reactions', 'among', 'condensation', 'trimethyl(p-methylbenzyl)', 'ammonium', 'halide', 'presence', 'base', '47:', 'n', 'n', 'br', 'naoh', 'a', 'route', 'halogen-substituted', 'polymer', 'reaction', 'di', 'halo-p-xylylenes', 'caustic', 'it', 'results', '16', 'elimination', 'hcl', 'formation', 'chlorine-substituted', 'poly(p-xylylene)', '47:', 'cln', 'cl', 'naoh', 'n', 'with', 'excess', 'caustic', 'however', 'chlorine', 'removed', 'presumably', 'all-conjugated', 'poly', '(phenylene', 'vinylene)', 'polymer', 'forms', '47:', 'n', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'p-bis(trichloromethyl)', 'benzene', 'pyrolyzed', 'copper', 'gauze', '300–600(cid6)c', 'temperatures', 'form', 'aaa0a0-tetrabromo-p-xylylene', 'this', 'monomer', 'polymerizes', 'temperatures', '140(cid6)c', '107:', 'cl', 'cl', 'cl', 'n', 'cl', 'cl', 'cu', 'cl', 'cl', 'cl', 'cl', 'cucl2', 'cl', 'cl', 'cl', 'cl', 'n', 'p-xylylene', 'polymers', 'also', 'form', 'electrolytically', '88', 'dioxane-water', 'solution', 'mercury', 'lead', 'cathodes', 'carbon', 'rod', 'anodes', 'during', 'reaction', 'quinone', 'dimethide', 'intermediate', 'probably', 'forms', '88.', 'many', 'substituted', 'polymers', 'also', 'form', 'reaction', '88.', 'following', 'example', 'one', 'br', 'f', 'f', 'f', 'f', 'br', 'f', 'f', 'f', 'n', 'f', 'another', 'interesting', 'synthesis', 'equilibration', 'reaction', 'suitably', 'substituted', 'styrenes', 'p-xylylenes', 'spontaneously', 'polymerize', 'linear', 'polymers', '108.', 'a', 'suitable', 'substituent', 'p-cyanophenylmethyl', 'group', 'such', 'compounds', 'undergo', 'complete', 'ionization', 'rearrange', 'presence', 'strong', 'base', '79', 'sulfur-containing', 'polymers', '463', 'the', 'p-xylylene', 'intermediate', 'polymerizes', 'n', 'n', 'h', 'n', 'n', 'h', 'h', 'n', 'h', 'n', 'h', 'n', 'n', 'n', 'h', 'n', 'there', 'however', 'competing', 'side', 'reactions', 'dimerization', '108:', '79', 'sulfur-containing', 'polymers', 'industrially', 'important', 'sulfur-containing', 'polymers', 'polysulfones', 'polysulﬁdes', 'the', 'materials', 'differ', 'considerably', 'properties', 'use', '791', 'polysulfones', 'these', 'materials', 'important', 'group', 'engineering', 'plastics', 'aliphatic', 'polysulfones', 'ﬁrst', 'synthesized', 'end', 'nineteenth', 'century', '109.', 'that', 'synthesis', 'based', 'reactions', 'so2', 'oleﬁns', '464', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'r', 'so2', 'o', 's', 'o', 'n', 'r', 'aliphatic', 'sulfones', 'however', 'lack', 'good', 'thermal', 'stability', 'commercially', 'important', 'aromatic', 'sulfones', 'hand', 'many', 'desirable', 'physical', 'properties', 'they', 'clear', 'rigid', 'tough', 'materials', 'high', 'tg', 'several', 'aromatic', 'sulfones', 'prepared', 'commercially', 'the', 'original', 'preparation', 'aromatic', 'polysulfones', 'described', '1958', '110.', 'this', 'followed', 'investigations', 'many', 'different', 'structures', 'polysulfones', 'one', 'current', 'commercial', 'material', 'condensation', 'product', '220', 'bis(hydroxyphenyl)', 'propane', '440-bis(chlorophenyl)', 'sulfone', 'it', 'forms', 'williamson', 'synthesis', 'reactivity', 'halogens', 'enhanced', 'sulfone', 'groups', '47:', 'o', 's', 'o', 'o', 's', 'o', 'o', 'n', 'ona', 'cl', 'cl', 'n', 'nao', 'n', 'o', 'o', 'o', '2n', 'nacl', 'n', 'the', 'condensation', 'takes', 'place', '160(cid6)c', 'inert', 'atmosphere', 'suitable', 'solvents', 'like', 'chlorobenzene', 'commercially', 'polymers', 'available', 'molecular', 'weight', 'ranges', '20000', '40000', 'much', 'higher', 'molecular', 'weight', 'materials', 'however', 'form', 'readily', 'aromatic', 'polysulfones', 'also', 'form', 'friedel–craft', 'reaction', '111,', 'instance', 'lewis', 'acid', 'cl', 'o', 's', 'o', 'n', 'hcl', 'o', 's', 'o', 'n', 'lewis', 'acid', 'n', 'cl', 'o', 's', 'o', 'o', 's', 'o', 'o', 's', 'cl', 'n', 'o', 'o', 'o', 's', 'o', 'o', 'n', 'the', 'reaction', 'takes', 'place', '80–250(cid6)c', 'solution', 'melt', 'lewis', 'acid', 'catalysts', 'used', 'concentrations', '01–10', 'mole', 'percent', 'cross-linking', 'polysubstitution', 'appear', 'problem', 'some', 'chain', 'branching', 'however', 'occur', 'ortho', 'substitution', 'possible', '79', 'sulfur-containing', 'polymers', 'table', '76', 'relative', 'tg', 'values', 'aromatic', 'polysulfones', '121', 'polymer', 'repeat', 'unit', 'o', 'o', 'o', 'o', 's', 'o', 'o', 's', 'o', 'o', 's', 'o', 'o', 's', 'o', 'o', 'o', 's', 'o', 'o', 'o', 'o', 's', 'o', 'o', 's', 'o', 'o', 'o', 's', 'o', 'o', 'o', 's', 'o', 'o', 'o', 's', 'o', '465', 'tg', '((cid6)c)', '220', '180', '290', '280', '265', '230', 'it', 'estimated', 'however', 'ortho', 'substitution', 'amounts', '5–10.', 'removal', 'catalyst', 'reaction', 'tedious', 'table', '76', 'presents', 'tg', 'values', 'aromatic', 'polysulfones', '792', 'polythiols', 'polymercaptans', 'these', 'polymers', 'form', 'reactions', 'di', 'thiols', 'aldehydes', 'ketones', 'r', 'n', 'hs', 'sh', 'r', '(h)', 'n', 'r', 'o', 'r', '(h)', 's', 's', 'r', 'n', 'r', '466', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'this', 'reaction', 'used', 'commercially', 'prepare', 'aromatic', 'polymercaptan', 'p-bis', '(mecaptomethyl)benzene', 'n', 'hs', 'sh', 'n', 'o', 's', 's', 'n', 'the', 'polymer', 'melts', '150(cid6)c', 'spun', 'ﬁbers', '109.', 'another', 'polymercaptan', 'poly(phenylene', 'sulﬁde)', 'prepared', 'several', 'routes', 'this', 'polymer', 'forms', 'sodium', 'p-bromothiophenol', '250–305(cid6)c', '109.', 'it', 'good', 'thermal', 'resistance', 'melts', '287(cid6)c', 'br', 'sna', 's', 'n', 'nabr', 'it', 'also', 'forms', 'reaction', 'dichlorobenzene', 'sodium', 'sulﬁde', '110.', 'to', 'date', 'exact', 'mechanism', 'polymerization', 'carried', 'commercially', 'n-methylpyrrolidone', 'solution', 'fully', 'established', 'recent', 'evidence', 'indicates', 'solvent', 'ionic', 'step', 'growth', 'snar', 'mechanism', 'predominates', '110:', 'cl', 'cl', 'nas', 'solvent', 'na', 'nas', 'cl', 'cl', 's', 'na', 'cl', 'cl', 'cl', 'cl', 's', 'na2s', 'cl', 's', 'n', 'the', 'polymer', 'possesses', 'broad', 'high', 'melting', 'point', 'glass', 'transition', 'temperature', '(cid8)85(cid6)c', 'an', 'important', 'characteristic', 'poly(phenylene', 'sulﬁde)', 'ability', 'undergo', 'changes', 'upon', 'heating', '110.', 'this', 'change', 'complex', 'completely', 'understood', 'it', 'appears', 'involve', 'varying', 'degrees', 'oxidation', 'cross-linking', 'chain', 'scission', 'when', 'heated', '315', '415(cid6)c', 'polymer', 'melts', 'thickens', 'gels', 'eventually', 'solidiﬁes', 'dark', 'infusible', 'solid', 'this', 'curing', 'phenomenon', 'makes', 'polymer', 'useful', 'many', 'applications', 'range', 'coatings', '(powder', 'slurry)', 'molding', '(by', 'injection', 'compression', 'sintering)', 'poly(phenylene', 'sulﬁde)', 'also', 'becomes', 'highly', 'conductive', 'electrically', 'dopant', 'added', 'conducting', 'polymers', 'discussed', 'chap', '8', 'elastomers', 'based', 'poly(alkylene', 'sulﬁde)s', 'still', 'another', 'group', 'sulfur-containing', 'polymers', 'they', 'represented', 'general', 'structural', 'formula', '(cid1)½(cid1)r(cid1)sx(cid1)(cid3)n(cid1)', 'x', '¼', '2(cid1)4', 'the', 'widely', 'used', 'methods', 'preparation', 'based', 'reactions', 'sodium', 'polysulﬁdes', 'alkyl', 'dichlorides', '110:', 'ncl(cid1)r(cid1)cl', 'þ', 'na2sx', '(cid1)½(cid1)r(cid1)sx(cid1)', '(cid3)n(cid1)', 'þ', '2nnacl', 'these', 'reactions', 'usually', 'carried', 'dispersions', 'an', 'aqueous', 'sodium', 'polysulﬁde', 'containing', 'surfactant', 'like', 'alkyl', 'aryl', 'sulfonate', 'sodium', 'hydroxide', 'magnesium', 'chloride', 'heated', '80(cid6)c', '79', 'sulfur-containing', 'polymers', '467', 'the', 'magnesium', 'chloride', 'forms', 'magnesium', 'hydroxide', 'acts', 'nucleating', 'agent', 'bis(2-', 'chloroethyl)', 'formal', 'added', '2', 'h', 'stirring', 'external', 'cooling', 'exothermic', 'reaction', '90(cid6)c', 'after', 'addition', 'temperature', 'stirring', 'usually', 'maintained', 'additional', '2', 'h', 'complete', 'process', 'the', 'product', 'contains', 'distribution', 'polysulﬁde', 'groups', 'the', 'polysulﬁde', 'anions', 'interchange', 'continuously', 's', 's', 'r', 'rs', 'ss', 'r', 'ss', 's', 's', 'r', 's', 's', 's', 's', 'rs', 'ss', 'r', 'this', 'allows', 'building', 'molecular', 'weights', 'polymers', 'additions', 'excess', 'polysulﬁde', 'also', 'low', 'molecular', 'weight', 'fractions', 'washed', 'by', 'manipulations', 'molecular', 'weights', '500000', 'readily', 'achieved', 'it', 'interesting', 'particular', 'step-growth', 'polymerization', 'order', 'obtain', 'high', 'molecular', 'weights', 'strict', 'stoichiometry', 'required', 'one', 'components', 'deliberately', 'added', 'excess', 'several', 'different', 'grades', 'poly(alkyl', 'sulﬁde)s', 'available', 'commercially', 'one', 'form', 'hydroxyl-', 'terminated', 'formed', 'coagulating', 'formed', 'polymer', 'aqueous', 'dispersion', 'sulfuric', 'acid', 'the', 'terminal', 'halogens', 'hydrolyze', 'process', 'hydroxyl', 'groups', 'for', 'easier', 'processing', 'elastomers', 'usually', 'reacted', 'disulﬁde', 'like', 'benzothiazyl', 'disulﬁde', 'this', 'reduces', 'molecular', 'weight', 'chain', 'cleavage', 'r', 's', 's', 'r', 's', 's', 'n', 's', '2', 'n', 's', 'n', 's', 'r', 's', 's', 'the', 'elastomers', 'chain-extended', 'metal', 'oxides', 'like', 'zinc', 'oxide', 'couple', 'terminal', 'hydroxy', 'groups', 'the', 'thing', 'also', 'done', 'using', 'diisocyanates', 'another', 'group', 'consists', 'thiol-terminated', 'low', 'molecular', 'weight', 'polymers', 'they', 'form', 'heating', 'alkyl', 'sulﬁdes', 'aqueous', 'dispersion', 'sodium', 'bisulphite', 'sodium', 'sulﬁte', 'hour', '80(cid6)c', 'this', 'results', 'mercaptide', 'thiothiol', 'terminal', 'groups', 's', 's', 'r', 's', 's', 'r', 's', 's', 'r', 'sh', 'the', 'sulﬁte', 'ion', 'prevents', 'reversal', 'equilibrium', 'splitting', 'sulfur', 'thiothiol', 's', 's', 'r', 's', 'sh', 'so3', 's', 's', 'r', 's', 'sh', 'rs', 's', 's', 'r', 's', 's', 'r', 'sh', 's2o3', 'these', 'polymers', 'cross-link', 'oxidative', 'coupling', 'mercaptide', 'groups', 'sh', 'o', 'hs', 's', 's', 'h2o', 'reactions', 'metal', 'peroxides', 'like', 'lead', 'peroxide', 'they', 'also', 'react', 'epoxy', 'resins', '468', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'poly(alkylene', 'sulﬁde)s', 'exceptionally', 'oil-resistant', 'elastomers', 'they', 'also', 'exhibit', 'good', 'resis-', 'tance', 'solvents', 'weathering', 'on', 'hand', 'elastomers', 'lack', 'strength', 'synthetic', 'rubbers', 'possess', 'unpleasant', 'odor', 'aliphatic', 'polysulﬁdes', 'also', 'form', 'reactions', 'di', 'thiols', 'alkyl', 'dihalides', 'high', 'molecular', 'weight', 'polymers', 'however', 'hard', 'form', '109:', 'n', 'nas', 'r', 'sna', 'n', 'br', 'r', 'br', 's', 'r', 's', 'r', 'n', '2n', 'nabr', 'similar', 'polymers', 'form', 'addition', 'di', 'thiols', 'dioleﬁns', '109.', 'the', 'reaction', 'take', 'place', 'either', 'free-radical', 'mechanism', 'ionic', 'one', 'a', 'free-radical', 'mechanism', 'requires', 'presence', 'peroxides', 'achieve', 'counter', 'markownikoff', 'additions', 'across', 'double', 'bonds', 'n', 'hs', 'sh', 'n', 'peroxide', 's', 's', '6', '6', 'n', 'it', 'step-growth', 'polymerization', 'process', 'the', 'additions', 'double', 'bonds', 'involve', 'series', 'hydrogen', 'exchange', 'reactions', '108:', 'r', '2', 'hs', 'sh', 'i2', 'r', '2', 'hs', 's', '2hi', '2', 'r', 'hs', 'sh', 'hs', 'r', 's', 'r', 's', 'sh', '2', '2', 'r', 's', 'sh', 'hs', 'r', 's', 'etc', 'when', 'additions', 'take', 'place', 'ionic', 'mechanisms', 'catalyzed', 'either', 'acids', 'bases', 'these', 'additions', 'follow', 'markownikoff', 'rule', 'r', 's', 's', 'n', 'r', 'n', 'hs', 'sh', 'n', '710', 'polyurethanes', 'the', 'polyurethanes', '126', '132', 'sometimes', 'also', 'called', '“isocyanate', 'polymers”', 'they', 'characterized', 'urethane', 'linkage', 'other', 'functional', 'groups', 'however', 'may', 'present', 'polymer', 'well', 'the', 'urethane', 'linkages', 'produced', 'polymers', 'several', 'different', 'routes', 'among', 'common', 'ones', 'reactions', 'isocyanate', 'groups', 'compounds', '710', 'polyurethanes', 'bearing', 'hydroxyl', 'groups', 'such', 'compounds', 'may', 'glycols', 'dihydroxy-terminated', 'polyethers', 'polyesters', 'others', 'difunctional', 'reactants', 'produce', 'linear', 'polyurethanes', '469', 'o', 'n', 'h', 'n', 'h', 'n', 'r', 'o', 'r', 'o', 'o', 'r', 'oh', 'ho', 'o', 'n', 'n', 'o', 'r', 'a', 'special', 'method', 'preparation', 'polyurethane', 'polymers', 'reported', '245.', 'in', 'new', 'procedure', 'use', 'made', '“latent”', 'aminimide', 'monomers', 'stable', 'reactive', 'normal', 'storage', 'conditions', 'an', 'example', 'monomer', 'aminimide', 'like', '111-trimethylamine', '2-(4-', 'hydroxymethylbenzoyl)-imide', 'undergoes', 'self-polyaddition', '150(cid6)c', 'yield', 'polyure-', 'thane', '245:', 'ho', 'n', 'n', 'o', '∆', 'ho', 'nco', 'n', 'o', 'o', 'n', 'h', 'n', 'the', 'monomer', 'prepared', 'reacting', 'methyl', 'para-hydroxymethyl-benzoate', 'ill-', 'trimethylhydrazinium', 'p-toluene', 'sulfonate', 'catalyzed', 'sodium', 'methoxide', 'at', '170(cid6)c', '90', 'monomer', 'converts', 'polymer', '5', 'h', '245.', '7101', 'preparations', 'polyfunctional', 'isocyanates', 'polyfunctional', 'isocyanates', 'formed', 'many', 'ways', '122.', 'commercially', 'important', 'one', 'reactions', 'phosgene', 'amines', 'amine', 'salts', 'other', 'reactions', 'however', 'like', 'carbon', 'monoxide', 'nitro', 'compounds', 'also', 'utilized', 'additions', 'isocyanic', 'acid', 'oleﬁns', 'also', 'gaining', 'prominence', '7102', 'commercial', 'polyisocyanates', 'two', 'types', 'diisocyanates', 'employed', 'polymer', 'preparations', 'aromatic', 'aliphatic', 'ones', 'the', 'commonly', 'used', 'aromatic', 'diisocyanates', 'toluene', 'diisocyanate', '440-diphenylmethane', 'diiso-', 'cyanate', 'commercial', 'toluene', 'diisocyanate', 'often', 'comes', 'mixture', '24', '26', 'isomers', 'ratios', '8020', '6535', 'when', 'reaction', 'takes', 'place', 'room', 'temperature', 'four', 'position', '8–10', 'times', 'reactive', 'two', 'at', 'elevated', 'temperatures', 'however', 'difference', 'reactivity', 'decreases', '100(cid6)c', 'reactivity', 'isocyanate', 'groups', 'positions', 'approximately', 'equal', '470', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'phosgenation', 'aniline-formaldehyde', 'condensates', 'yields', 'isocyanates', 'functionality', 'averages', '26–28', 'the', 'structure', 'product', 'shown', 'follows', 'n', 'o', 'n', 'o', 'n', 'o', 'x', 'x', '2', '3', 'among', 'aromatic', 'diisocyanates', 'commercial', 'use', 'p-phenylene', 'diisocyanate', 'm-', 'phenylene', 'diisocyanate', '1-chloro-24-phenylene', 'diisocyanate', '330-dimethyl-440-bisphenylene', 'diisocyanate', '440-bis(2-methylisocyanophenyl)-methane', '440-bis(2-methoxyisocyanophenyl)', 'methane', 'the', 'common', 'aliphatic', 'diisocyanates', 'hexamethylene', 'diisocyanate', 'hydrogenated', '(h12)', '440-', 'dipenylmethane', 'diisocyanate', 'isophorone', 'diisocyanate', '224-trimethylhexamethylene', 'diisocyanate', '244-trimethylhexamethylene', 'diisocyanate', 'other', 'aliphatic', 'diisocyanates', 'commercially', 'used', 'lysine', 'diisocyanate', 'methylcyclohexyl', 'diisocyanate', 'isopropylidine', 'bis-(4-cyclohexyl', 'isocy-', 'anate)', 'tetramethylene', 'diisocyanate', 'many', 'additional', 'polyfunctional', 'isocyanates', 'described', 'literature', '7103', 'chemical', 'reactions', 'isocyanates', 'these', 'reactions', 'divided', 'two', 'categories', 'they', 'additions', 'compounds', 'active', 'hydrogens', 'self-condensation', 'the', 'reactions', 'well', 'described', 'organic', 'chemistry', 'textbooks', 'little', 'reason', 'describe', 'it', 'noteworthy', 'though', 'uncatalyzed', 'reactions', 'isocyanates', 'various', 'active', 'hydrogen', 'compounds', 'probably', 'broadly', 'similar', 'among', 'investigated', 'reactions', 'ones', 'alcohols', 'isocyanates', '123:', 'r', 'n', 'o', 'roh', 'koh', 'o', 'r', 'o', 'r', 'h', 'n', 'o', 'h', 'r', 'r', 'n', 'h', 'o', 'o', 'r', 'o', 'r', 'o', 'o', 'r', 'r', 'r', 'n', 'h', 'n', 'h', 'o', 'roh', 'electron', 'withdrawing', 'substituents', 'increase', 'positive', 'charge', 'isocyanate', 'carbon', 'move', 'negative', 'charge', 'away', 'site', 'reaction', 'o2n', 'n', 'o', 'n', 'o', 'o', 'n', 'o', 'this', 'makes', 'attack', 'electron', 'donor', 'easier', 'yields', 'faster', 'reaction', 'electron', 'donating', 'groups', 'therefore', 'opposite', 'effect', '710', 'polyurethanes', 'table', '77', 'relative', 'effects', 'catalysts', 'reactivity', 'phenylisocyanate', '126', '127', '132', '471', 'relative', 'rates', 'reactions', 'n-butyl', 'alcohol', 'catalyst', 'none', 'n-methyl', 'morpholine', 'triethyl', 'amine', 'triethylene', 'diamine', 'tributyltin', 'acetate', 'dibutyltin', 'diacetate', '10', '40', '86', '1200', '30000', '60000', '7104', 'the', 'effect', 'catalysts', 'catalysts', 'exert', 'strong', 'inﬂuence', 'rates', 'reactions', 'isocyanates', 'active', 'hydrogen', 'compounds', 'most', 'widely', 'used', 'ones', 'tertiary', 'amines', 'metal', 'salts', 'particularly', 'tin', 'compounds', 'the', 'mechanism', 'catalysis', 'tertiary', 'amines', 'believed', '124–126', 'proceed', 'according', 'following', 'scheme', 'r3n', 'catalyst', 'r', 'n', 'o', 'n', 'r3', 'o', 'o', 'r', 'r', 'n', 'h', 'r3', 'n', 'n', 'o', 'r', 'o', 'r', 'o', 'r', 'n', 'h', 'r3n', 'r3', 'r', 'n', 'n', 'hor', 'o', 'the', 'catalytic', 'activity', 'tertiary', 'amines', 'generally', 'parallels', 'base', 'strength', 'except', 'steric', 'hindrance', 'pronounced', 'tin', 'compounds', 'exert', 'much', 'stronger', 'catalytic', 'effects', 'reactions', 'tertiary', 'amines', 'this', 'illustrated', 'table', '77', 'the', 'mechanism', 'catalysis', 'metal', 'salts', 'believed', 'operate', 'follows', '128:', 'r', 'n', 'o', 'mx2', 'r', 'n', 'o', 'r', 'n', 'o', 'mx2', 'mx2', 'r', 'n', 'o', 'h', 'mx2', 'o', 'r', 'r', 'n', 'o', 'roh', 'mx2', 'or', 'mx2', 'o', 'r', 'n', 'h', 'or', 'mx2', 'o', 'r', 'h', 'or', 'mx2', 'o', 'r', 'n', 'h', '472', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'table', '78', 'tm', 'values', 'ﬁber-forming', 'polyurethanes', '126', '132', 'table', '79', 'the', 'effect', 'molar', 'ratios', 'molecular', 'weight', 'product', '134–136', 'diisocyanate', 'ocn(ch2)4nco', 'ocn(ch2)4nco', 'ocn(ch2)4nco', 'ocn(ch2)5nco', 'ocn(ch2)6nco', 'ocn(ch2)6nco', 'ocn(ch2)8nco', 'ocn(ch2)8nco', 'glycol', 'ho(ch2)4oh', 'ho(ch2)6oh', 'ho(ch2)10oh', 'ho(ch2)4oh', 'ho(ch2)4oh', 'ho(ch2)5oh', 'ho(ch2)4oh', 'ho(ch2)6oh', 'tm', '((cid6)c)', '190', '180', '170', '159', '183', '159', '160', '153', 'ratio', 'epichlorohydrin', 'phenol', '1001', '201', '141', '1331', '1251', '121', 'molecular', 'weight', '370', '451', '791', '802', '1133', '1420', '7105', 'polyurethane', 'fibers', 'originally', 'main', 'interest', 'polyurethanes', 'preparation', 'ﬁbers', 'they', 'tend', 'resemble', 'aliphatic', 'polyamides', 'though', 'ﬁbers', 'harder', 'dye', 'wiry', 'hard', 'handle', 'the', 'common', 'preparatory', 'procedure', 'add', 'continuously', 'diisocyanate', 'glycol', 'letting', 'temperature', 'rise', 'slowly', '200(cid6)c', 'the', 'reaction', 'exothermic', 'excess', 'heat', 'must', 'removed', 'an', 'inert', 'nitrogen', 'atmosphere', 'maintained', 'reaction', 'mixture', 'table', '78', 'lists', 'tm', 'values', 'aliphatic', 'polyurethanes', 'today', 'much', 'less', 'interest', 'polyurethane', 'ﬁbers', 'one', 'polyure-', 'thane', 'ﬁber', 'commercialized', 'sometime', '1960', '1965', 'unique', 'elastomeric', 'material', 'expandable', 'textiles', 'it', 'alternating', 'block', 'copolymer', '“soft”', '“hard”', 'segments', 'the', '“soft”', 'segments', 'form', 'hydroxyl-terminated', 'aliphatic', 'polyethers', 'polyesters', 'molecular', 'weight', '1000', '4000', 'they', 'linked', '“hard”', 'segments', 'urethane', 'linkages', 'the', '“hard”', 'segments', 'form', 'aromatic', 'diisocyanates', 'aliphatic', 'aromatic', 'diamines', 'hydrazine', 'hydrazine', 'derivatives', 'sometimes', 'used', 'chain', 'extenders', 'place', 'diamines', 'a', 'preparation', 'described', 'follows', 'macro', 'glycols', 'reacted', 'ﬁrst', 'excess', 'diisocyanate', 'ho', 'oh', 'ocn', 'r', 'excess', 'nco', 'r', 'ocn', 'o', 'o', 'nco', 'r', 'o', 'n', 'h', 'h', 'n', 'o', 'the', 'reaction', 'mixture', 'chain-extended', 'diamine', 'r', 'ocn', 'o', 'o', 'nco', 'nco', 'r', 'ocn', 'h2n', 'r', 'nh2', 'n', 'h', 'o', 'h', 'n', 'h', 'n', 'o', 'o', 'o', 'o', 'o', 'r', 'r', 'o', 'n', 'h', 'n', 'h', 'n', 'h', 'h', 'n', 'h', 'n', 'h', 'n', 'r', 'r', 'o', 'o', 'o', 'r', 'n', 'h', '710', 'polyurethanes', '473', 'the', 'prepolymers', 'spun', 'ﬁbers', 'subsequently', 'cross-linked', 'cross-linked', 'spun', 'obtain', 'resilient', 'elastomeric', 'ﬁbers', '7106', 'polyurethane', 'elastomers', 'solid', 'elastomers', 'divided', 'three', 'categories', 'namely', 'cast', 'millable', 'thermoplastic', 'ones', 'the', 'cast', 'elastomers', 'formed', 'casting', 'liquid', 'reaction', 'mixtures', 'low', 'molecular', 'weight', 'prepolymers', 'heated', 'molds', 'cross-link', 'convert', 'high', 'molecular', 'weight', 'materials', 'slightly', 'branched', 'polyesters', 'combined', 'ﬁrst', 'diisocyanates', 'like', 'toluene', 'diisocyanate', 'form', 'prepolymers', 'degassed', 'vacuum', 'elevated', 'temperature', '(about', '70(cid6)c)', 'the', 'reaction', 'mixtures', 'poured', 'molds', 'heated', 'several', 'hours', '110(cid6)c', 'form', 'solid', 'elastomers', 'these', 'elastomers', 'soft', 'resilient', 'lack', 'good', 'mechanical', 'strength', 'elastomers', 'better', 'mechanical', 'strength', 'form', 'linear', 'hydroxyl-terminated', 'polyesters', 'polyethers', 'these', 'macro', 'glycols', 'also', 'prereacted', 'ﬁrst', 'diisocyanates', 'similarly', 'procedure', 'used', 'expandable', 'ﬁbers', 'the', 'products', 'however', 'mixed', 'low', 'molecular', 'weight', 'glycols', 'diamines', 'heated', 'molds', '110(cid6)c', '24', 'h', 'slightly', 'less', 'stoichiometric', 'amounts', 'glycols', 'diamines', 'used', 'polymers', 'terminated', 'isocyanate', 'groups', 'these', 'terminal', 'isocyanate', 'groups', 'react', 'mold', 'urethane', 'hydrogens', 'form', 'allophanate', 'cross-links', 'trimerization', 'isocyanate', 'groups', 'might', 'also', 'take', 'place', 'though', 'usually', 'requires', 'catalyst', 'form', 'temperatures', '130(cid6)c', 'n', 'o', 'o', 'n', 'n', 'o', 'o', 'n', 'o', 'n', 'n', 'o', 'the', 'concentrations', 'allophanate', 'links', 'vary', 'time', 'cure', '129.', 'also', 'cross-', 'linking', 'reactions', 'conducted', 'inert', 'nitrogen', 'atmospheres', 'little', 'scission', 'cross-links', 'takes', 'place', 'network', 'structure', 'forms', 'cure', 'in', 'open', 'air', 'however', 'scissions', 'cross-links', 'extensive', '129', 'products', 'poorer', 'physical', 'properties', 'a', 'drawback', 'cast', 'elastomers', 'limited', 'shelf', 'life', 'need', 'store', 'absence', 'moisture', 'as', 'result', 'millable', 'elastomers', 'developed', 'these', 'produced', 'ﬁrst', 'forming', 'hydroxy-', 'terminated', 'linear', 'polyurethanes', 'reactions', 'linear', 'aliphatic', 'polyesters', 'polyethers', 'diisocyanates', 'the', 'prepolymers', 'rubbers', 'gums', 'compounded', 'rubber', 'mills', 'ingredients', 'cross-linked', 'cross-linking', 'accomplished', 'adding', 'either', 'diisocyanates', 'sulfur', 'peroxides', 'diisocyanates', 'dimers', 'dissociate', '150(cid6)c', 'often', 'used', 'n', 'n', 'o', 'o', 'n', 'o', 'n', 'o', '∆', '2', 'n', 'o', 'n', 'o', '474', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'unsaturated', 'prepolymers', 'cross-link', 'peroxides', 'sulfur', 'this', 'unsaturation', 'present', 'backbone', 'pendant', 'groups', 'vulcanization', 'cross-linking', 'elastomers', 'sulfur', 'peroxides', 'discussed', 'chap', '8', 'thermoplastic', 'elastomers', 'exhibit', 'physical', 'properties', 'similar', 'cast', 'millable', 'elastomers', 'ambient', 'temperatures', 'these', 'materials', 'however', 'cross-linked', 'ﬂow', 'elevated', 'temperatures', 'they', 'fabricated', 'like', 'thermoplastic', 'polymers', 'high', 'molecular', 'weight', 'hydroxyl-terminated', 'such', 'polymers', 'form', 'linear', 'hydroxyl-terminated', 'polyester', 'polyethers', 'condensed', 'diisocyanates', 'glycols', 'strict', 'stoichiometry', 'must', 'maintained', 'achieve', 'high', 'molecular', 'weights', 'a', 'structure', 'study', 'carried', 'model', 'compound', 'one', 'elastomer', '130', 'prepared', '440-', 'diphenylmethane', 'diisocyanate', 'butanediol', 'hard', 'segments', 'it', 'shown', 'chains', 'probably', 'linked', 'together', 'stacks', 'c¼o(cid9)(cid9)(cid9)h–n', 'hydrogen', 'bonds', 'urethane', 'groups', 'this', 'bonding', 'stabilizes', 'overall', 'structure', 'directions', 'perpendicular', 'chain', 'axis', 'such', 'arrangement', 'molecules', 'also', 'proposed', 'earlier', '131.', '7107', 'polyurethane', 'foams', 'these', 'foams', 'chemically', 'similar', 'polyurethane', 'materials', 'except', 'gas', 'evolutions', 'reactions', 'take', 'place', 'simultaneously', 'chain', 'lengthening', 'cross-linking', 'this', 'results', 'formation', 'cellular', 'structures', 'the', 'degree', 'cross-linking', 'determines', 'great', 'extent', 'rigidity', 'foam', 'in', 'addition', 'linear', 'slightly', 'branched', 'polymers', 'produce', 'ﬂexible', 'foams', 'highly', 'branched', 'polymers', 'form', 'rigid', 'ones', 'the', 'foaming', 'caused', 'liberation', 'co2', 'reactions', 'added', 'water', 'isocyanate', 'groups', 'n', 'o', 'h2o', '2', 'n', 'o', 'h2o', 'h', 'n', 'h', 'n', 'o', 'oh', 'o', 'h', 'n', 'o', 'o', 'n', 'h', 'h', 'co2', 'h', 'n', 'h', 'n', 'o', 'co2', 'in', 'addition', 'many', 'industrial', 'practices', 'additional', 'carbon', 'dioxide', 'freon', 'gas', '(a', 'major', 'source', 'environmental', 'pollution)', 'may', 'introduced', 'system', 'cures', 'for', 'rigid', 'foams', 'low', 'boiling', 'liquid', 'may', 'added', 'form', 'additional', 'bubbles', 'appropriate', 'catalysts', 'foam', 'stabilizers', 'surfactants', 'added', 'control', 'foam', 'formation', 'cell', 'size', 'cure', 'the', 'catalysts', 'either', 'tin', 'compounds', 'tertiary', 'amines', 'the', 'surfactants', 'necessary', 'control', 'cell', 'size', 'usually', 'based', 'siloxanes', '711', 'epoxy', 'resins', 'these', 'resins', 'comprise', 'general', 'class', 'cross-linkable', 'low', 'molecular', 'weight', 'materials', 'epirane', 'rings', 'main', 'functional', 'group', '134', '135.', 'it', 'include', 'polyethers', 'formed', 'ring', 'opening', 'polymerizations', 'ethylene', 'propylene', 'oxides', '711', 'epoxy', 'resins', '475', '7111', 'preparation', 'commercial', 'epoxy', 'resins', 'the', 'earliest', 'developed', 'commercial', 'epoxy', 'resins', 'diglycidyl', 'ethers', '440-isopropylidine-', 'diphenol', '(bisphenol', 'a)', 'formed', 'reacting', 'epichlorohydrin', 'diphenol', 'the', 'reaction', 'sequences', 'involve', 'formations', 'alkoxide', 'ions', 'followed', 'nucleophilic', 'additions', 'least', 'hindered', 'carbons', '134:', 'ho', 'oh', 'oh', 'ho', 'o', 'h2o', 'oh', 'o', 'o', 'cl', 'o', 'oh', 'o', 'o', 'this', 'followed', 'ring', 'closures', 'internal', 'displacement', 'chloride', 'ions', 'ho', 'o', 'cl', 'ho', 'o', 'o', 'o', 'cl', 'as', 'explained', 'discussion', 'phenoxy', 'resins', 'reaction', 'two', 'moles', 'epichlorohydrin', 'one', 'mole', 'diphenol', 'yields', 'higher', 'molecular', 'weight', 'by-products', '10', 'diether', 'o', 'o', 'x', '¼', '1–10', 'o', 'o', 'x', 'oh', 'o', 'o', 'to', 'obtain', 'high', 'yields', 'diether', 'quantities', 'epichlorohydrin', 'reaction', 'mixtures', 'must', 'doubled', 'tripled', '131.', 'this', 'result', 'yields', 'high', '70.', 'another', 'advantage', 'large', 'excess', 'epichlorohydrin', 'reaction', 'mixtures', 'serves', 'solvent', '134.', 'a', 'different', 'route', 'diglycidyl', 'ethers', 'via', 'friedel–craft', 'reactions', '135:', 'cl', 'o', 'ho', '01', '-', '02', 'zncl2', 'oh', 'cl', 'o', 'o', 'cl', 'oh', 'oh', '476', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'na2si03', '5h2o', 'o', 'o', 'o', 'o', 'the', 'higher', 'molecular', 'weight', 'resins', 'form', 'presence', 'caustic', 'result', 'reactions', 'glycidyl', 'ethers', 'phenoxy', 'anions', 'o', 'o', 'oh', 'o', 'o', 'the', 'number', 'repeat', 'units', 'resin', 'determined', 'molar', 'ratios', 'reactants', 'this', 'illustrated', 'table', '69', '7112', 'the', 'cross-linking', 'reactions', 'diglycidyl', 'ethers', 'bisphenol', 'a', 'cannot', 'cross-linked', 'heating', 'alone', 'chemical', 'cross-linking', 'agents', 'must', 'added', 'most', 'commonly', 'used', 'compounds', 'tertiary', 'amines', 'polyfunctional', 'amines', 'acid', 'anhydrides', 'lewis', 'acid', 'phenols', 'compounds', 'like', 'dicyandiamide', 'however', 'also', 'used', 'the', 'reactions', 'tertiary', 'amines', 'epoxy', 'groups', 'result', 'formations', 'quaternary', 'bases', 'o', 'r3n', 'r2n', 'o', 'the', 'product', 'reacts', 'hydroxyl', 'compounds', 'form', 'anions', 'o', 'r3n', 'ho', 'r', 'r3n', 'oh', 'o', 'r', 'the', 'anions', 'turn', 'initiate', 'polymerizations', 'epoxy', 'groups', 'r', 'o', 'o', 'r', 'o', 'o', 'o', 'o', 'r', 'o', 'r', 'o', 'o', 'o', 'o', 'r', 'o', 'o', 'o', 'o', 'etc', 'because', 'monomer', 'diepoxide', 'three-dimensional', 'lattice', 'results', 'a', 'similar', 'three-dimensional', 'product', 'forms', 'reactions', 'cross-linking', 'materials', 'like', 'boron', 'triﬂuoride-etherate', 'boron', '711', 'epoxy', 'resins', '477', 'trichloride-amine', 'complexes', 'imidazole', 'derivatives', 'all', 'compounds', 'initiate', 'polymerizations', 'epirane', 'ring', 'cross-linking', 'epoxy', 'resins', 'primary', 'secondary', 'amines', 'somewhat', 'different', 'react', 'nucleophilic', 'addition', 'epirane', 'ring', 'r', 'nh2', 'o', 'r', 'n', 'h', 'oh', 'o', 'nr', 'ho', 'oh', 'a', 'considerable', 'amount', 'evidence', 'suggests', 'reaction', 'accelerated', 'proton', 'donors', 'such', 'donors', 'may', 'water', 'phenols', 'alcohols', '136:', 'o', 'r', 'nh2', 'h2o', 'ho', 'h', 'h', 'o', 'nr', 'h', 'h', 'oh', 'oh', 'nr', 'h', 'r', 'oh', 'h2o', 'epoxy', 'resins', 'also', 'cured', 'aid', 'ionic', 'liquids', '136.', 'the', 'cross-linking', 'reaction', 'takes', 'place', 'elevated', 'temperatures', 'highly', 'idealized', 'pictures', 'reaction', 'products', 'bisphenol', 'a', 'diglycidyl', 'ethers', 'diamines', 'found', 'literature', '137.', 'in', 'actuality', 'products', 'probably', 'complex', 'they', 'certainly', 'complex', 'reactions', 'involve', 'higher', 'molecular', 'weight', 'epoxy', 'resins', 'many', 'different', 'aliphatic', 'aromatic', 'polyamines', 'available', 'cross-linking', 'epoxy', 'resins', 'some', 'ethylenediamine', 'diethylene', 'triamine', 'tetraethylene', 'pentamine', 'many', 'others', 'among', 'aromatic', 'polyamine', 'p-phenylene', 'diamine', 'm-phenylene', 'diamine', '440-diaminodiphenyl-methane', 'diaminodiphenylsulfone', 'triethylene', 'tetramine', 'a', 'special', 'cross-linking', 'agent', 'epoxy', 'resins', 'dicyanodiamide', 'also', 'referred', 'trade', 'name', 'cyanoguanidine', 'it', 'used', 'high', 'temperature', 'cures', 'believed', 'compound', 'condenses', 'epirane', 'structures', 'form', '2-aminooxazoline', 'derivatives', '137:', 'n', 'h', 'h', 'n', 'o', 'o', 'h2n', 'n', 'h', 'o', 'dicyanodiamide', 'n', 'n', 'o', 'n', 'h', '478', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'earlier', 'however', 'speculated', 'guanidyl', 'urea', 'forms', '140.', 'the', 'curing', 'mechanism', 'epoxy', 'resins', 'imidazole', 'investigated', 'the', 'following', 'mecha-', 'nism', 'proposed', '137:', 'n', 'n', 'h', 'c2h5', 'h', 'n', 'n', 'c2h5', 'o', 'o', 'h', 'n', 'n', 'c2h5', 'many', 'cyclic', 'acid', 'anhydrides', 'used', 'industrially', 'cross-linking', 'epoxy', 'resins', 'these', 'mono', 'polyanhydrides', 'following', 'typical', 'commercially', 'used', 'anhydrides', 'o', 'h', 'n', 'n', 'c2h5', 'oh', 'n', 'n', 'c2h5', 'n', 'n', 'oh', 'o', 'o', 'oh', 'c2h5', 'n', 'n', 'c2h5', 'o', 'o', 'o', 'o', 'o', 'o', 'c12h23', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'ch3', 'pyromellitic', 'dianhydride', 'phthalic', 'anhydride', 'oo', 'o', 'o', 'o', 'o', 'maleic', 'anhydride', 'o', 'cl', 'cl', 'cl', 'cl', 'o', 'o', 'methylbicyclo', '221hept-5-', 'ene-23-dicarboxylic', 'acid', 'anhydride', 'o', 'o', 'dodecylsuccinic', 'anhydride', 'cl', 'cl', 'chlorendic', 'anhydride', 'r', 'o', 'o', 'o', 'photoadduct', 'alkylbenzene', 'maleic', 'anhydride138', '711', 'epoxy', 'resins', '479', 'the', 'anhydrides', 'react', 'traces', 'moisture', 'pendant', 'hydroxyl', 'groups', 'ﬁrst', 'this', 'opens', 'anhydride', 'groups', 'frees', 'carboxylic', 'acids', 'reactions', 'epoxy', 'rings', '1', '2', 'o', 'o', 'o', 'o', 'o', 'o', 'h2o', 'oh', 'cooh', 'cooh', 'cooh', 'o', 'o', 'o', 'oh', 'o', 'o', 'o', 'o', 'when', 'epoxy', 'hydroxyl', 'groups', 'present', 'little', 'reaction', 'takes', 'place', 'even', 'elevated', 'temperatures', 'high', '200(cid6)c', 'proton', 'donors', 'however', 'catalyze', 'reaction', '139.', 'when', 'anhydrides', 'present', 'reaction', 'mixtures', 'diesters', 'form', 'result', 'termolecular', 'transition', 'states', 'kinetic', 'data', 'support', '136:', 'oh', 'o', 'tertiary', 'amines', 'effective', 'catalyzing', 'reactions', 'anhydrides', 'epoxies', 'ho', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'r3n', 'o', 'o', 'o', 'nr3', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'nr3', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'nr3', 'o', '480', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'reactions', 'presence', 'alcohols', 'take', 'somewhat', 'different', 'path', '142', '143:', 'r3n', 'roh', 'r3n', 'hor', 'r3n', 'hor', 'o', 'o', 'o', 'r3n', 'or', 'h', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'r3n', 'ho', 'or', 'o', 'or', 'r3n', 'ho', 'o', 'r0oh', 'represents', 'epoxide', 'molecules', 'hydroxyl', 'function', 'alcohols', 'system', 'another', 'similar', 'group', 'epoxy', 'resins', 'called', 'epoxy', 'novolacs', 'forms', 'reactions', 'epichlorohydrin', 'low', 'molecular', 'weight', 'phenolic', 'novolacs', '(phenolic', 'novolac', 'resins', 'discussed', 'next', 'section)', 'oh', 'cl', 'n', 'o', 'o', 'also', 'cl', 'n', 'oh', 'o', 'o', 'n', 'n', 'o', 'o', 'n', 'typically', '22–38', 'liquid', 'epoxy', 'novolacs', '3–7', 'solid', 'resins', 'epoxy', 'resins', 'also', 'prepared', 'commercially', 'bisphenol', 'f', 'blend', 'ortho', 'para', 'diphenol', 'methanes', 'o', 'o', 'o', 'o', 'epoxy', 'resins', 'formed', 'condensations', 'epichlorohydrin', 'resorcinol-based', 'phenolic', 'resins', 'also', 'formed', 'commercially', '711', 'epoxy', 'resins', '481', 'another', 'material', 'based', 'condensation', 'product', 'glyoxal', 'phenol', 'o', 'o', '4', 'ho', 'ho', 'oh', '4', 'o', 'cl', 'o', 'o', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'well', 'o', 'o', 'oh', 'o', 'o', 'h', 'h', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'several', 'nitrogen-containing', 'aromatic', 'epoxy', 'resins', 'also', 'commercialized', 'these', 'conden-', 'sation', 'products', 'aromatic', 'amines', 'epichlorohydrin', 'following', 'examples', '140:', 'h2n', 'oh', '3', 'cl', 'n', 'o', 'o', 'the', 'cross-linking', 'reactions', 'tetrafunctional', 'epoxy', 'resins', 'aromatic', 'primary', 'diamines', 'investigated', 'spectoscopy', '141.', 'uv-visible', 'ﬂuorescence', 'spectroscopies', 'materials', 'gelation', 'show', 'signiﬁcant', 'amounts', 'amines', 'ﬁnished', 'products', 'the', 'infrared', 'spectra', 'also', 'show', 'ether', 'formation', 'becomes', 'signiﬁcant', 'late', 'cure', 'in', 'addition', 'cure', 'especially', 'air', 'oxidations', 'degradations', 'occur', '141.', 'this', 'results', 'color', 'formation', '482', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '7113', 'cycloaliphatic', 'epoxides', 'many', 'cycloaliphatic', 'epoxies', 'products', 'oxidation', 'cyclic', 'oleﬁns', 'peracids', 'like', 'peracetic', 'these', 'materials', 'commercialized', 'years', 'use', 'active', 'diluents', 'though', 'later', 'withdrawn', 'one', 'commercial', 'group', 'resins', 'obtained', 'cyclopentadiene', '2', '2hcl', '2', 'cl', 'koh', 'o', 'o', 'ooh', 'o', 'o', 'o', 'bis(23-epoxycyclopentyl)', 'ether', 'another', 'group', 'cycloaliphatic', 'epoxy', 'resins', 'prepared', 'via', 'diels–alder', 'additions', 'followed', 'tischenko', 'reaction', 'completed', 'epoxidation', 'a', 'preparation', 'instance', 'may', 'start', 'butadiene', 'acrolein', '2', '2', 'diels-alder', 'o', 'o', 'o', 'o', 'tischenko', 'o', 'o', '2', 'o', 'ooh', 'o', 'o', 'examples', 'cycloaliphatic', 'epoxy', 'resins', 'o', 'o', 'o', 'o', '4', 'o', 'bis(34-epoxy-6-mehyl-cyclohexyl)adipate', 'o', 'o', 'o', 'vinyl', 'cyclohexyl', 'dioxide', 'o', 'o', 'o', 'o', '2-(34-epoxycyclohexyl)-55-', 'spiro(34-epoxy)cyclohexane-m-dioxane', 'o', 'o', 'dicyclopentadiene', 'dioxide', '712', 'phenol-formaldehyde', 'resins', '483', 'acid', 'anhydrides', 'effective', 'curing', 'agents', 'cycloaliphatic', 'epoxy', 'resins', 'amines', 'in', 'addition', 'amines', 'might', 'also', 'react', 'ester', 'groups', 'present', 'materials', 'form', 'undesirable', 'by-products', '712', 'phenol-formaldehyde', 'resins', 'the', 'phenolic', 'resins', 'condensation', 'products', 'phenol', 'formaldehyde', '144–146', '148.', 'these', 'materials', 'among', 'earliest', 'commercial', 'synthetic', 'plastics', 'two', 'different', 'methods', '144–146', 'used', 'prepare', 'in', 'ﬁrst', 'one', 'condensations', 'base', 'catalyzed', 'second', 'one', 'acid-catalyzed', 'the', 'products', 'formed', 'basic', 'catalysts', 'called', 'resols', 'acidic', 'ones', 'novolacs', 'phenolic', 'resins', 'used', 'widely', 'coatings', 'laminates', 'the', 'pure', 'resins', 'friable', 'use', 'structural', 'materials', 'they', 'become', 'useful', 'plastics', 'however', 'ﬁlled', 'various', 'ﬁllers', '7121', 'resols', 'these', 'thermosetting', 'resins', 'form', 'reactions', 'phenols', 'formaldehyde', 'water', 'presence', 'catalytic', 'amounts', 'bases', 'under', 'conditions', 'phenol', 'exists', 'resonance-stabilized', 'anion', 'o', 'o', 'o', 'o', 'the', 'addition', 'phenol', 'anion', 'formaldehyde', 'typical', 'nucleophilic', 'reaction', 'o', 'o', 'o', 'o', 'protein', 'transfer', 'o', 'oh', 'both', 'ortho', 'para', 'methylolphenols', 'form', 'reaction', 'phenol', 'reactive', 'monosubstituted', 'phenols', 'hard', 'isolate', 'reaction', 'mixture', 'di-', 'trisubstitution', 'occurs', 'rapidly', 'no', 'substitutions', 'ever', 'shown', 'take', 'place', 'meta', 'position', 'the', 'overall', 'reaction', 'follows', '484', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'oh', 'oh', 'ho', 'o', 'o', 'ph', '7', 'o', 'ph', '7', 'o', 'o', 'oh', 'o', 'o', 'oh', 'oh', 'o', 'oh', 'ho', 'oh', 'o', 'when', 'aqueous', 'reaction', 'phenol', 'formaldehyde', 'catalyzed', 'sodium', 'hydroxide', 'carried', '30(cid6)c', '5', 'h', 'products', '147:', 'the', 'remaining', '3', 'unreacted', 'phenol', 'as', 'reaction', 'continues', 'methylolphenols', 'condense', 'form', 'methylene', 'bridges', '246-trimethylolphenol', '24-dimethylolphenol', '26-dimethylolphenol', 'p-methylolphenol', 'o-methylolphenol', 'o', '2', 'oh', 'o', 'o', 'oh', 'o', 'o', '37', '24', '7', '17', '12', 'o', 'oh', 'oh', 'o', '712', 'phenol-formaldehyde', 'resins', '485', 'the', 'para', 'substituted', 'methylolphenols', 'course', 'react', 'manner', '2', 'o', 'o', 'oh', 'oh', 'o', 'formation', 'methylene', 'bridges', 'take', 'place', 'one', 'two', 'mechanisms', 'one', 'direct', 'sn2', 'displacement', 'o', 'o', 'o', 'o', 'oh', 'o', 'o', 'oh', 'the', 'one', 'addition', 'methylolated', 'phenols', 'molecules', 'quinone', 'methides', 'form', 'typical', 'reaction', 'conditions', 'particularly', 'temperatures', 'elevated', '144–146:', 'h2o', 'o', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'oh', 'o', 'o', 'o', 'oh', 'hoh', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '486', 'o', 'two', 'methylol-substituted', 'phenols', 'react', 'mechanism', 'o', 'o', 'oh', 'oh', 'oh', 'o', 'o', 'o', 'o', 'ho', 'ho', 'ho', 'o', 'h2o', 'the', 'shown', 'para', 'substituted', 'methylolphenols', 'as', 'reaction', 'continues', 'leads', 'formation', 'trinuclear', 'tetra', 'nuclear', 'phenolic', 'resins', 'a', 'typical', 'liquid', 'resole', 'quite', 'low', 'molecular', 'weight', 'it', 'may', 'contain', 'two', 'three', 'benzene', 'rings', 'carried', 'little', 'condensation', 'yields', 'solid', 'resole', 'the', 'ph', 'usually', 'adjusted', 'neutral', 'resoles', 'heated', 'cross-linking', 'under', 'neutral', 'slightly', 'acidic', 'conditions', 'methylol', 'groups', 'tend', 'form', 'dibenzyl', 'ethers', 'oh', 'o', 'oh', 'oh', 'ho', 'oh', 'oh', 'oh', '∆', 'oh', 'these', 'dibenzyl', 'ethers', 'unstable', 'higher', 'temperatures', '150(cid6)c', 'decompose', 'yield', 'methylene', 'bridges', 'formaldehyde', '147:', 'o', 'ho', 'oh', 'o', 'the', 'structure', 'typical', 'resole', 'contains', 'dimethylene', 'ether', 'methylene', 'bridges', 'well', 'methylol', 'groups', 'fusible', 'soluble', 'resols', 'called', 'a-stage', 'resins', 'further', 'reactions', 'cause', 'resins', 'pass', 'rubbery', 'stage', 'still', 'swollen', 'solvents', 'this', 'called', 'b-stage', 'the', 'ﬁnally', 'cross-linked', 'material', 'called', 'c-stage', 'resin', 'the', 'cross-linking', 'process', 'involves', 'complex', 'competing', 'reactions', 'each', 'may', 'inﬂuenced', 'reaction', 'conditions', 'when', 'cross-linking', '712', 'phenol-formaldehyde', 'resins', '487', 'resoles', 'take', 'place', 'neutral', 'slightly', 'acidic', 'conditions', 'methylene', 'ether', 'linkages', 'form', 'upon', 'heating', 'ethers', 'turn', 'split', 'formaldehyde', 'shown', 'the', 'dibenzyl', 'ethers', '144–146', '148', 'also', 'break', 'elevated', 'temperatures', 'form', 'quinone', 'methides', 'the', 'quinone', 'methides', 'undergo', 'variety', 'reactions', 'including', 'cycloadditions', 'methides', 'form', 'chroman', 'groups', '2', 'o', 'o', 'o', 'the', 'para', 'quinone', 'methides', 'couple', '2', 'o', 'ho', 'oh', '7122', 'novolacs', 'the', 'phenolic', 'resins', 'form', 'acid-catalyzed', 'condensations', 'phenols', 'formaldehyde', 'different', 'resols', 'at', 'ph', 'seven', 'protonation', 'carbonyl', 'group', 'formaldehyde', 'takes', 'place', 'ﬁrst', 'followed', 'electrophilic', 'aromatic', 'substitution', 'ortho', 'para', 'positions', 'phenol', 'the', 'initial', 'steps', 'reactions', 'also', 'take', 'place', 'water', 'here', 'however', 'molar', 'excess', 'phenol', '(1251)', 'must', 'used', 'reactions', 'equimolar', 'basis', 'acidic', 'conditions', 'form', 'cross-linked', 'resins', 'at', 'ratio', 'eight', 'moles', 'formaldehyde', 'ten', 'moles', 'phenol', 'novolacs', 'approximate', 'molecular', 'weight', '850', 'form', '148.', 'when', 'ratio', 'formaldehyde', 'phenol', '910', 'molecular', 'weight', 'approximately', '1000', 'reached', 'this', 'appears', 'near', 'limit', 'beyond', 'cross-linking', 'results', 'the', 'reaction', 'follows', 'o', 'h', 'oh', 'oh', '488', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'oh', 'oh', 'h', 'oh', 'oh', 'h', 'o', 'oh', 'oh', 'oh', 'oh', 'oh', 'oh', 'oh', 'h', 'because', 'reaction', 'taking', 'place', 'ph', '7', 'shown', 'p-', 'o-methylolphenols', 'transitory', 'present', 'small', 'concentrations', 'hydrogen', 'ions', 'convert', 'benzylic', 'carbocations', 'react', 'rapidly', 'free', 'phenol', 'this', 'illustrated', 'follows', 'oh', 'oh', 'h', 'oh', 'oh', 'h2o', 'oh2', 'ch2', 'oh', 'ho', 'ch2', 'ho', 'ho', 'oh', 'h', 'h', 'ho', 'the', 'ortho', 'substituted', 'methylolphenols', 'react', 'manner', 'further', 'metholylation', 'dihydroxy', 'diphenyl', 'methanes', 'takes', 'place', 'formaldehyde', 'used', 'methylol', 'groups', 'react', 'quickly', 'form', 'methylene', 'bridges', 'the', 'para', 'position', 'reactive', 'ortho', '144–146', 'ph', '3', 'the', 'opposite', 'true', 'however', 'ph', '5–6', 'ortho', 'position', 'reactive', 'typical', 'novolacs', 'formed', 'reactions', 'high', 'molecular', 'weight', 'contain', 'six', 'ten', 'benzene', 'rings', 'if', 'divalent', 'metal', 'salts', 'like', 'zinc', 'acetate', 'acetic', 'acid', 'used', 'catalyze', 'reaction', 'ortho', 'positions', '712', 'phenol-formaldehyde', 'resins', '489', 'become', 'considerably', 'reactive', '150.', 'as', 'result', 'ortho-methylene', 'bridges', 'predominate', 'products', 'zn', 'ho', 'oh', 'oh', 'zn', 'o', 'oh', 'zn', 'oh', 'h', 'o', 'zn', 'o', 'oh', 'h', 'zn', 'o', 'o', 'h2o', 'oh', 'oh', 'oh', 'zn', 'oh', 'oh', 'zn', 'oh', 'o', 'oh', 'oh', 'h', 'oh', 'oh', 'it', 'possible', 'form', 'high', 'molecular', 'weight', 'novolacs', 'carrying', 'reactions', 'alkyl', 'phenyl', 'ethers', 'formaldehyde', 'acetic', 'acid', 'presence', 'perchloric', 'acid', '149:', 'cn', 'o', 'cn', 'o', 'o', 'hclo4', 'naoch3', 'n', 'oh', 'n', 'novolacs', 'cross-linked', 'additions', 'formaldehyde', 'soluble', 'thermoplastic', 'materials', 'the', 'additional', 'formaldehyde', 'form', 'paraform', 'oligomer', 'formaldehyde', '490', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'decomposes', 'formaldehyde', 'upon', 'heating', 'it', 'also', 'come', 'hexamethylenetetramine', 'condensation', 'product', 'formaldehyde', 'ammonia', 'hexamethylenetetramine', 'decomposes', 'formaldehyde', 'ammonia', 'upon', 'heating', 'some', 'ammonia', 'picked', 'novolacs', 'result', 'benzylamine', 'bridges', 'product', '6', 'o', '4nh3', 'oh', 'h', 'n', 'n', 'n', 'n', 'n', 'oh', 'the', 'mechanism', 'reaction', 'discussed', 'next', 'section', '7123', 'ammonia-catalyzed', 'phenolic', 'resins', 'these', 'resins', 'differ', 'resols', 'benzylamine', 'bridges', 'present', 'structure', 'the', 'reactions', 'result', 'early', 'losses', 'water', 'allow', 'higher', 'molecular', 'weight', 'buildups', 'resins', 'gel', 'nitrogen-containing', 'resols', 'darker', 'color', 'regular', 'resols', 'the', 'di', 'benzylamine', 'bridges', 'form', 'follows', '145:', 'oh', 'oh', 'n', 'h', '2', 'oh', 'nh3', '(rnh2)', 'the', 'overall', 'mechanism', 'shown', 'special', 'case', 'mannich', 'reaction', '(r)', 'nh', 'o', '(r)', 'nh', 'oh', 'h', 'h', 'h', 'h', 'o', 'o', '(r)', 'h', 'oh', 'n', 'h', '(r)', 'h', 'n', 'h', 'h2o', 'o', '(r)', 'h', 'n', 'h', 'oh', '712', 'phenol-formaldehyde', 'resins', '491', 'o', 'h', '(r)', 'n', 'h', 'h', 'o', 'o', 'o', 'o', 'n', 'h', 'in', 'amine', 'ammonia-catalyzed', 'reactions', '145,', 'additions', 'condensations', 'occur', 'almost', 'simultaneously', 'methylol', 'groups', 'still', 'present', 'ﬁnished', 'resins', 'extent', '15–30', 'groups', 'per', '100', 'phenol', 'residues', 'the', 'structures', 'branched', 'degree', 'branching', 'depends', 'upon', 'amine', 'used', '7124', 'typical', 'commercial', 'preparations', 'the', 'resols', 'usually', 'prepared', 'typical', 'reaction', 'kettles', 'using', '15–20', 'moles', 'formaldehyde', 'per', 'mole', 'phenol', 'the', 'reactions', 'rapid', 'condensations', 'resoles', 'might', 'accomplished', '1', 'h', 'formaldehyde', 'often', 'added', 'form', 'formalin', 'the', 'quantity', 'added', 'caustic', 'ammonia', 'might', 'comprise', 'one', 'percent', 'phenol', 'reaction', 'mixture', 'these', 'reactions', 'carried', 'water', 'reﬂux', 'speciﬁed', 'time', 'the', 'ph', 'lowered', 'neutral', 'water', 'distilled', 'usually', 'reduced', 'pressure', 'the', 'progress', 'condensation', 'followed', 'measuring', 'melting', 'point', 'gel', 'time', '(time', 'required', 'material', 'become', 'thermoset', 'speciﬁed', 'temperature)', 'solubility', 'free', 'phenol', 'content', 'better', 'quality', 'novolacs', 'resoles', 'prepared', 'stainless', 'steel', 'resin', 'kettles', 'for', 'novolacs', 'typical', 'recipe', 'might', 'call', 'mole', 'phenol', '08', 'moles', 'formaldehyde', '(usually', 'added', 'formalin', '37', 'solution', 'water)', 'acid', 'catalysts', 'like', 'oxalic', 'hydrochloric', 'others', 'added', 'amounts', '1–2', 'weight', 'phenol', 'oxalic', 'acid', 'favored', 'hydrochloric', 'sulfuric', 'phosphoric', 'due', 'corrosion', 'problems', 'in', 'addition', 'vapors', 'hydrochloric', 'acid', 'tend', 'react', 'vapors', 'formal-', 'dehyde', 'form', 'carcinogenic', 'compound', '110-dichlorodimethyl', 'ether', 'the', 'reactions', 'conducted', 'reﬂux', 'temperature', 'water', '2–4', 'h', 'maleic', 'acid', 'sometimes', 'used', 'form', 'high', 'melting', 'novolacs', 'in', 'typical', 'preparations', 'novolacs', 'molten', 'phenol', 'usually', 'kept', '65(cid6)c', 'introduced', 'reaction', 'kettle', 'heated', '95(cid6)c', 'the', 'catalyst', 'added', 'this', 'followed', 'addition', 'formaldehyde', 'solution', 'kettle', 'stirring', 'rate', 'allows', 'gentle', 'reﬂux', 'after', 'addition', 'heating', 'stirring', 'continued', 'almost', 'formaldehyde', 'used', 'at', 'point', 'resins', 'separate', 'aqueous', 'phase', 'water', 'distilled', 'temperature', 'raised', 'process', '160(cid6)c', 'the', 'unreacted', 'phenol', 'removed', 'vacuum', 'distillation', 'the', 'end', 'reaction', 'may', 'determined', 'melting', 'point', 'product', 'melt', 'viscosity', 'cresols', 'also', 'often', 'used', 'preparations', 'phenolic', 'resins', 'these', 'may', 'individual', 'isomers', 'mixtures', 'three', 'cresilic', 'acids', 'mixtures', 'three', 'isomers', 'rich', 'm-cresol', 'low', 'o-cresol', 'preferred', 'xylenols', '(all', 'six', 'isomers)', 'also', 'common', 'use', 'form', 'alkali-resistant', 'grades', 'phenolic', 'resins', 'high', '35-xylenol', 'mixtures', 'preferred', 'also', 'resorcinol', 'forms', 'reactive', 'phenolic', 'resins', 'used', 'preparations', 'cold-setting', 'adhesives', 'higher', 'homologues', 'phenol', 'like', 'bisphenol', 'a', 'used', 'prepare', 'special', 'phenol-formaldehyde', 'condensates', '492', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '713', 'amino', 'polymers', 'currently', 'bulk', 'commercial', 'polymers', 'would', 'ﬁt', 'category', 'urea-formaldehyde', 'melamine-formaldehyde', 'resins', '151.', 'over', 'years', 'however', 'many', 'materials', 'might', 'ﬁt', 'group', 'prepared', 'adopted', 'use', 'various', 'reasons', '7131', 'urea-formaldehyde', 'resins', 'these', 'thermosetting', 'resins', 'ﬁnd', 'applications', 'coatings', 'adhesives', 'laminating', 'molding', 'compositions', 'the', 'materials', 'formed', 'water', 'ph', '7', 'start', 'reaction', 'methylol', 'derivatives', 'form', 'condense', 'rapidly', 'acidic', 'conditions', 'the', 'initial', 'step', 'urea', 'undergoes', 'nucleophilic', 'addition', 'formaldehyde', 'shown', 'follows', '2', 'h2n', 'nh2', '3', 'o', 'h2n', 'oh', 'in', 'past', 'believed', 'condensations', 'take', 'place', 'ph', '7', 'include', 'formations', 'cyclic', 'intermediates', 'this', 'however', 'never', 'demonstrated', '151.', 'nmr', 'spectra', 'urea-formaldehyde', 'resins', 'show', '152', 'condensations', 'acidic', 'conditions', 'proceed', 'via', 'formations', 'methylene', 'linkages', 'h2n', 'oh', 'h2n', 'h2n', 'n', 'h', 'n', 'h', 'oh', 'under', 'alkaline', 'conditions', 'hand', 'dimethylene', 'ether', 'groups', 'form', 'instead', '152:', 'h2n', 'oh', 'h2n', 'oh', 'o', 'o', 'h2n', 'o', 'nh2', 'n', 'h', 'n', 'h', 'in', 'addition', 'highly', 'condensed', 'water-soluble', 'resins', 'contain', 'hemiformal', 'groups', 'further', 'reactions', 'may', 'result', 'formation', 'polymeric', 'materials', '153.', 'this', 'especially', 'true', 'ratios', 'formaldehyde', 'urea', 'low', 'some', 'opinion', 'linear', 'oligomeric', 'o', 'n', 'h', 'oh', 'o', 'o', 'o', 'o', 'n', 'h', 'n', 'h', 'n', 'h', 'o', 'o', 'n', 'h', 'o', 'n', 'h', '713', 'amino', 'polymers', '493', 'condensates', 'form', 'instead', 'these', 'urea-formaldehyde', 'condensates', 'separate', 'colloidal', 'dispersions', 'stabilized', 'association', 'excess', 'formaldehyde', '153.', 'the', 'cross-linking', 'reaction', 'consists', 'agglomeration', 'colloidal', 'particles', 'accompanying', 'release', 'formaldehyde', 'this', 'opinion', 'supported', 'several', 'observations', '(1)', 'one', 'plots', 'logarithm', 'solution', 'viscosity', 'time', 'polymerization', 'plot', 'exhibits', 'sharp', 'break', 'also', 'plot', 'differs', 'similar', 'ones', 'phenol-formaldehyde', 'condensation', 'reactions', 'show', 'continuous', 'increases', 'viscosity', '(2)', 'scanning', 'electron', 'micrographs', 'fully', 'cured', 'resins', 'show', 'surface', 'characteristics', 'resemble', 'surfaces', 'coagulated', 'coalesced', 'colloidal', 'particles', 'high', 'molecular', 'weight', 'polymers', '(3)', 'x-ray', 'diffraction', 'patterns', 'laser', 'raman', 'spectra', 'cross-linked', 'resins', 'show', 'crystalline', 'areas', 'material', 'absence', 'water', 'similar', 'patterns', 'obtained', 'hydrogen-bonded', 'proteins', 'close', 'chain', 'packing', 'on', 'hand', 'ft-ir', 'studies', '154', 'show', 'methylene', 'ether', 'cross-links', 'present', 'cured', 'resin', 'there', 'also', 'indications', 'presence', 'cyclic', 'ether', 'units', 'the', 'information', 'also', 'suggests', 'ﬁnal', 'structure', 'urea-', 'formaldehyde', 'resin', 'may', 'function', 'feed', 'ratio', 'ph', 'formed', 'urea-formaldehyde', 'resins', 'surface', 'coatings', 'commonly', 'modiﬁed', 'solubility', 'organic', 'solvents', 'reacting', 'alcohols', 'form', 'ether', 'groups', 'usually', 'n-butyl', 'alcohol', 'used', 'the', 'reaction', 'carried', 'basic', 'conditions', 'acidiﬁcation', 'n2n', 'oh', 'c4h9oh', 'h2n', 'c4h9', 'o', 'h2o', 'o', 'n', 'h', 'o', 'n', 'h', 'after', 'etheriﬁcation', 'reaction', 'mixture', 'acidiﬁed', 'resin', 'reacted', 'acquire', 'desired', 'degree', 'condensation', 'a', 'typical', 'butylated', 'urea-formaldehyde', 'resin', 'contains', '05–10', 'moles', 'butyl', 'ether', 'groups', 'per', 'mole', 'urea', '7132', 'melamine-formaldehyde', 'resins', 'these', 'resins', 'quite', 'similar', 'urea-formaldehyde', 'condensates', 'probably', 'reason', 'ﬁnd', 'similar', 'applications', 'melamine', 'reacts', 'formaldehyde', 'slightly', 'alkaline', 'conditions', 'form', 'mixtures', 'various', 'methylolmelamines', '155:', 'nh2', 'n', 'n', 'oh', 'o', 'h2n', 'n', 'nh2', 'n', 'oh', 'h', 'n', 'n', 'h2n', 'n', 'oh', 'n', 'h', 'h', 'n', 'n', 'oh', 'n', 'h2n', 'n', 'nh2', 'h', 'n', 'n', 'h', 'n', 'oh', 'n', 'n', 'h', 'ho', 'n', 'oh', '494', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'further', 'heating', 'causes', 'condensation', 'resins', 'the', 'rate', 'resiniﬁcations', 'ph-dependent', 'melamine-formaldehyde', 'resins', 'also', 'etheriﬁed', 'solvent', 'solubility', 'methanol', 'often', 'used', 'hexamethyl', 'ether', 'hexamethylolmelamine', 'well', 'higher', 'homologues', 'available', 'com-', 'mercially', 'the', 'hexamethyl', 'ether', 'shown', 'follows', 'o', 'n', 'o', 'n', 'n', 'o', 'n', 'n', 'o', 'n', 'o', 'o', 'the', 'ethers', 'cleave', 'upon', 'acidiﬁcation', 'network', 'structures', 'form', 'for', 'methylolated', 'melamines', 'etheriﬁed', 'acidiﬁcation', 'necessary', 'heating', 'alone', 'often', 'adequate', 'network', 'formation', 'melamine-formaldehyde', 'resins', 'reputation', 'harder', 'moisture-', 'resistant', 'urea-formaldehyde', 'ones', '714', 'silicone', 'polymers', 'these', 'semiinorganic', 'materials', 'important', 'industrially', '156–160.', 'they', 'exhibit', 'good', 'thermal', 'stability', 'good', 'electrical', 'insulating', 'characteristics', 'water', 'repellency', 'act', 'release', 'coatings', 'materials', 'in', 'addition', 'properties', 'maintained', 'wide', 'range', 'temperatures', 'as', 'result', 'polymers', 'many', 'diverse', 'uses', '7141', 'polysiloxanes', 'the', 'silicon', 'atom', 'carbon', 'periodic', 'table', 'similar', 'electronic', 'arrangement', 'silicon', '1s2', '2s2', '2p6', '3s2', '3p2', 'the', 'larger', 'atomic', 'radius', 'however', 'makes', 'silicon–silicon', 'single', 'bond', 'much', 'less', 'energetic', 'because', 'silanes', '(sinh2n', '2)', 'much', 'less', 'stable', 'alkanes', 'the', 'opposite', 'however', 'true', 'silicon–oxygen', 'bonds', 'energetic', '(about', '22', 'kcalmole)', 'carbon', 'oxygen', 'bonds', 'polysiloxanes', 'therefore', 'recurring', 'si–o', 'linkages', 'backbones', 'the', 'starting', 'materials', 'prepared', 'hydrolyses', 'alkyl', 'arylsilicone', 'halides', '156–160.', 'organosilicone', 'halides', 'turn', 'made', 'commercially', 'heating', 'alkyl', 'aryl', 'halides', 'silicon', '250–289(cid6)c', 'copper', 'catalyzes', 'reaction', 'rcl', 'si', '250', '-', '280', 'oc', 'cu', 'sicl4', 'rsicl3', 'r2sicl2', 'r3sicl', '714', 'silicone', 'polymers', '495', 'the', 'materials', 'also', 'formed', 'grignard', 'reaction', 'rmgcl', 'rmgcl', 'sicl4', 'r2sicl2', 'rsicl3', 'r3sicl', 'mgcl2', 'mgcl2', '(ch3)3sicl', 'mgcl', '(ch3)3si', 'mgcl2', 'alkyl', 'silanes', 'also', 'prepared', 'additions', 'trichlorosilanes', 'ethylene', 'acetylene', 'trichlorosilane', 'reacts', 'aromatic', 'compounds', 'presence', 'boron', 'trichloride', 'hsicl3', 'hsicl3', 'hsicl3', '2', 'bcl3', 'sicl3', 'sicl3', 'sicl2', '2', 'the', 'siloxane', 'linkages', 'result', 'hydrolysis', 'halides', 'the', 'products', 'hydrolyses', 'silanols', 'unstable', 'condense', 'cl', 'cl', 'ho', 'oh', 'h2o', 'r', 'si', 'r', 'r', 'si', 'r', 'r', 'si', 'r', 'r', 'o', 'si', 'o', 'r', 'h2o', 'the', 'shown', 'reaction', 'one', 'possible', 'route', 'siloxane', 'polymers', 'as', 'general', 'method', 'however', 'approach', 'satisfactory', 'ring', 'formations', 'accompany', 'reactions', 'some', 'rings', 'form', 'hydrolyses', 'trichlorosilanes', 'structurally', 'complex', '161.', 'they', 'may', 'even', 'possess', 'three-dimensional', 'structures', '162', '163.', 'high', 'molecular', 'weight', 'polymers', 'however', 'form', 'readily', 'ring', 'opening', 'polymerizations', 'such', 'polymerizations', 'applied', 'simple', 'rings', 'form', 'dihalides', 'complex', 'ones', 'trihalides', 'ring', 'opening', 'polymerizations', 'carried', 'puriﬁed', '(by', 'distillation)', 'cyclic', 'intermediates', 'catalyzed', 'either', 'acids', 'bases', '164,', 'leading', 'linear', 'siloxane', 'polymers', 'o', 'si', 'si', 'o', 'o', 'si', 'h2so4', 'si', 'o', 'n', 'acid-catalyzed', 'polymerizations', 'yield', 'lower', 'molecular', 'weight', 'polymers', 'mostly', 'oils', 'the', 'molecular', 'weights', 'oils', 'controlled', 'additions', 'hexamethyldisiloxane', 'polymerization', 'reactions', 'when', 'catalyzed', 'bases', 'high', 'molecular', 'weight', 'elastic', 'polymers', 'form', 'si', 'o', 'n', 'o', 'si', 'si', 'o', 'o', 'si', 'naoh', '2', 'si', 'nao', 'o', 'na', '4n', 'co2', 'si', 'ho', 'o', 'h', '4n', '2na2co3', '496', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'another', 'method', 'forming', 'polydialkyldisiloxanes', 'reacting', 'difunctional', 'oligomers', 'cyclic', 'organisilicons', 'n', 'o', 'si', 'si', 'o', 'o', 'si', 'si', 'o', '(ch3)2cl2', 'cl', 'si', 'si', 'o', '4n', 'cl', 'si', 'cl', 'o', 'si', '4n', 'cl', '2h2o', 'si', 'si', 'ho', 'o', 'oh', '4n', 'molecular', 'weights', 'polydimethyl', 'siloxanes', 'reach', '700000', 'higher', 'within', 'range', 'molecular', 'weights', '4000', '25000', 'materials', 'ﬂuids', 'various', 'viscosities', 'most', 'common', 'commercial', 'liquid', 'polydimethyl', 'siloxanes', 'prepared', 'dimethyl', 'dichloro', 'siloxane', 'many', 'elastomers', 'also', 'based', 'dimethylsiloxane', 'special', 'polymers', 'prepared', 'substituents', '7142', 'silicone', 'elastomers', 'the', 'elastomers', 'high', 'molecular', 'weight', 'silicone', 'polymers', 'must', 'cross-linked', 'obtain', 'rubber-', 'like', 'properties', 'one', 'way', 'accomplished', 'hydrogen', 'abstraction', 'free-radicals', 'generated', 'decomposition', 'added', 'peroxides', '24-dichlorobenzoyl', 'peroxide', 'often', 'used', 'purpose', 'it', 'decomposed', '110', '150(cid6)c', 'the', 'reaction', 'shown', 'follows', '2', 'si', 'si', 'si', 'si', 'o', 'o', 'o', 'o', 'cl', '2', 'cl', 'o', 'o', 'cl', 'cl', 'cooh', 'si', 'si', 'si', 'si', 'o', 'o', 'o', 'o', 'si', 'si', 'si', 'si', 'o', 'o', 'o', 'o', 'si', 'si', 'si', 'si', 'o', 'o', 'o', 'o', 'ch2', 'ch2', 'si', 'si', 'si', 'si', 'o', 'o', 'o', 'o', '714', 'silicone', 'polymers', '497', 'the', 'accomplished', 'replacing', 'small', 'quantity', 'methyl', 'groups', 'vinyl', 'ones', 'this', 'done', 'including', 'small', 'amount', 'vinyl', 'methyldichlorosilane', 'monomer', 'mix', '(about', '01)', 'the', 'product', 'small', 'quantity', 'pendant', 'vinyl', 'groups', 'cross-', 'links', 'readily', 'free-radical', 'mechanism', 'when', 'portions', 'methyl', 'groups', 'polysiloxane', 'backbones', 'replaced', 'phenyl', 'structures', 'elastomers', 'exhibit', 'particularly', 'good', 'low', 'temperature', 'properties', '159.', 'room', 'temperature', 'cross-linkable', 'polysiloxane', 'elastomers', '(commonly', 'called', 'rtv-s)', 'prepared', 'two', 'techniques', 'in', 'ﬁrst', 'one', 'chloro', 'siloxanes', 'functionality', 'larger', 'two', 'added', 'hydroxyl-terminated', 'prepolymers', 'the', 'products', 'subsequently', 'cross-linked', 'second', 'addition', 'polyfunctional', 'compounds', 'like', 'tetraalkoxysilane', 'presence', 'tin', 'catalysts', 'like', 'stannous', 'octoate', 'cross-linking', 'occurs', 'room', 'temperature', 'this', 'reaction', 'may', 'vary', '10', 'min', '24', 'h', 'o', 'si', 'oh', 'ho', 'si', 'c2h5o', 'si', 'oc2h5', 'oc2h5', 'oc2h5', 'o', 'si', 'oh', 'ho', 'si', 'o', 'si', 'o', 'o', 'si', 'si', 'o', 'si', 'o', 'o', 'si', 'in', 'second', 'one', 'polysiloxanes', 'terminated', 'hydroxyl', 'groups', 'either', '(1)', 'acetylated', '(2)', 'converted', 'ketoximes', '(3)', 'etheriﬁed', 'the', 'cross-linking', 'activated', 'reaction', 'atmospheric', 'moisture', 'o', 'si', 'o', '(1)', 'ho', 'si', 'o', 'si', 'si', 'o', 'o', 'ch3cooh', '(2)', 'r', 'r', 'n', 'si', 'o', 'o', 'ho', 'si', 'o', 'si', 'si', 'o', 'hon', 'r', 'r', '(3)', 'si', 'o', 'or', 'ho', 'si', 'o', 'si', 'si', 'o', 'roh', 'the', 'acetate-capped', 'rtv-s', 'exhibit', 'good', 'adhesion', 'substrates', 'released', 'acetic', 'acid', 'corrosive', '498', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '7143', 'polysiloxane', 'coating', 'resins', 'these', 'materials', 'usually', 'prepared', 'two', 'stages', 'in', 'ﬁrst', 'one', 'low', 'molecular', 'weight', 'intermediates', 'formed', 'in', 'second', 'one', 'intermediates', 'reacted', 'resins', 'possessing', 'functional', 'groups', 'like', 'alkyds', 'polyesters', 'acrylic', 'resins', 'the', 'common', 'low', 'molecular', 'weight', 'intermediates', 'si', 'o', 'o', 'x', 'r', 'ho', 'si', 'o', 'r', 'si', 'o', 'o', 'si', 'r', 'o', 'si', 'o', 'o', 'si', 'r', 'r', 'o', 'o', 'si', 'r', 'oh', 'x', '¼', '3–6', '7144', 'fluorosilicone', 'elastomers', 'the', 'elastomers', 'based', 'polysiloxanes', 'contain', 'triﬂuoropropyl', 'methyl', 'siloxane', 'units', 'the', 'materials', 'used', 'sealants', 'elastomers', 'ﬂuids', 'aerospace', 'applications', 'the', 'monomers', 'prepared', 'according', 'following', 'scheme', 'cl', 'cl', 'cl', 'hf', 'f', 'f', 'f', 'the', 'dichloro', 'silane', 'converted', 'cyclic', 'trimer', 'f', 'f', 'f', 'h', 'cl', 'si', 'cl', 'f', 'f', 'f', 'cl', 'si', 'cl', 'o', 'si', 'si', 'o', 'o', 'si', 'f', 'f', 'f', 'f', 'f', 'f', 'ring', 'opening', 'polymerizations', 'basic', 'catalysts', 'convert', 'trimers', 'polymers', 'elevated', 'temperatures', 'some', 'vinyl', 'silane', 'usually', 'copolymerized', 'material', 'subsequent', 'cross-', 'linking', 'these', 'elastomers', 'reported', 'capable', 'maintaining', 'original', 'strengths', 'temperatures', 'high', '205(cid6)c', 'long', 'periods', 'time', '715', 'polysilanes', '499', '7145', 'polyarylsiloxanes', '(also', 'see', 'sect', '7174)', 'many', 'different', 'polyarylsiloxanes', 'reported', 'literature', 'only', 'general', 'industrial', 'use', 'present', 'though', 'many', 'exhibit', 'interesting', 'physical', 'properties', 'might', 'used', 'future', 'preparation', 'one', 'material', '164', 'starts', 'reaction', 'aniline', 'dichlorosilane', 'presence', 'hcl', 'scavenger', '2', 'nh2', 'cl2', 'si', 'r2', 'r2', '2hcl', 'n', 'si', '2', 'h', 'the', 'product', 'dianilinosilane', 'reacted', 'diphenols', 'like', 'hydroquinone', 'n', 'h', 'si', '2', 'ho', '∆', 'oh', 'si', 'o', 'o', '2', 'nh2', 'polymers', 'prepared', 'procedure', 'molecular', 'weights', '80000', '164.', 'it', 'also', 'possible', 'start', 'diphenoxysiloxane', 'catalyze', 'reaction', 'sodium', 'lithium', 'metals', 'reactions', 'cyclic', 'silazanes', 'arylene', 'disalanols', 'yield', 'polymers', 'molecular', 'weights', 'high', '900000', '164:', 'n', 'ho', 'si', 'si', 'oh', 'n', 'h', 'n', 'si', 'si', 'o', 'si', 'o', 'is', 'si', 'o', 'nch3nh2', '4', 'n', '715', 'polysilanes', 'polymers', 'silicon–silicon', 'single', 'bonds', 'backbone', 'known', 'time', 'it', 'within', 'last', '10–15', 'years', 'however', 'high', 'molecular', 'weight', 'materials', 'developed', '165.', 'behind', 'current', 'interest', 'materials', 'realization', 'various', 'potential', 'applications', 'these', 'ceramic', 'ﬁbers', '166,', 'microlithography', '165', '167,', 'photoconduction', '168,', 'nonlinear', 'optical', 'materials', '169.', '500', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'the', 'polymers', 'prepared', 'disubstitued', 'dichlorosilanes', 'reacting', 'alkali', 'metal', 'dispersions', 'reductive', 'coupling', 'process', 'the', 'polymerizations', 'appear', 'characteristics', 'chain-growth', 'rather', 'step-growth', 'reactions', '170:', 'rrsicl2', 'na', 'nacl', 'r', 'si', 'r', 'the', 'illustrated', 'reaction', 'sodium', 'dispersions', 'requires', 'greater', '80(cid6)c', 'temperatures', 'proceed', 'satisfactorily', 'when', 'mixtures', 'different', 'dialkylsubstituted', 'dichlorosilanes', 'reacted', 'manner', 'copolymers', 'form', '171.', 'recently', 'ambient', 'temperature', 'sonochemical', 'reductive', 'coupling', 'process', 'developed', '172.', 'the', 'reaction', 'carried', 'presence', 'ultrasound', 'results', 'relatively', 'high', '(mn', '¼', '50000–100000)', 'molecular', 'weight', 'materials', 'narrow', 'molecular', 'weight', 'distributions', 'in', 'addition', 'reported', '172', 'polymers', 'also', 'formed', 'anionic', 'ring', 'opening', 'polymeri-', 'zation', 'cyclotetrasilanes', 'yield', 'polymers', 'molecular', 'weights', '10000–100000', '716', 'phosphonitrile', 'polymers', 'these', 'polymers', 'also', 'called', 'polyphosphazenes', 'useful', 'materials', 'substituted', 'organic', 'compounds', '173.', 'they', 'prepared', 'hexachloro-cyclotriphosphazenes', 'ring', 'opening', 'polymerizations', 'cl2', 'p', 'n', 'n', 'p', 'n', 'p', 'n', 'cl2', 'cl2', 'n', 'p', 'n', 'cl', 'cl', 'cl', 'p', 'n', 'it', 'believed', 'mechanism', 'polymerization', 'involves', 'attack', 'electron-rich', 'nitrogen', 'one', 'cyclic', 'monomer', 'upon', 'another', 'one', 'at', 'ﬁrst', 'cation', 'forms', 'ionization', 'phosphorus', 'chloride', 'bond', '173:', 'cl2', 'p', 'n', 'n', 'p', 'n', 'p', 'cl2', 'cl2', 'ionization', 'n', 'p', 'cl2', 'cl', 'n', 'p', 'cl2', 'this', 'followed', 'attack', 'another', 'monomer', 'molecule', 'cl', 'p', 'n', 'cl', 'n', 'p', 'cl2', 'cl2', 'n', 'p', 'cl2', 'cl2', 'p', 'n', 'n', 'p', 'n', 'p', 'cl2', 'n', 'p', 'n', 'p', 'cl2', 'cl2', 'cl', 'p', 'n', 'n', 'p', 'cl2', 'n', 'p', '2', 'cl', 'cl2', 'n', 'p', 'cl2', 'n', 'p', '(3n2)', 'cl', 'cl2', 'cl', 'p', 'n', 'n', 'p', 'n', 'p', 'cl2', 'cl2', '716', 'phosphonitrile', 'polymers', '501', 'alkyl', 'aryl', 'alkoxy', 'aminocyclotriphosphazenes', 'fail', 'polymerize', 'this', 'believed', 'due', 'absence', 'easily', 'ionizable', 'halogen', 'phosphorus', 'bonds', 'at', 'time', 'materials', 'presumably', 'facilitate', 'ionization', 'phosphorus', 'halogen', 'bonds', 'water', 'carboxylic', 'acids', 'metals', 'accelerate', 'polymerization', '173.', 'the', 'all-inorganic', 'polymer', 'decomposes', 'readily', 'elevated', 'temperatures', 'sensitive', 'hydrolytic', 'attack', 'quantitative', 'replacements', 'halogen', 'groups', 'however', 'possible', 'alkoxy', 'aryloxy', 'alkyl', 'aryl', 'amino', 'groups', 'yield', 'much', 'stable', 'materials', 'the', 'replacements', 'achieved', 'reﬂuxing', 'inorganic', 'polymer', 'ether', 'solvent', 'several', 'hours', 'sodium', 'alkoxide', 'aryloxide', 'metal', 'alkyl', 'aryl', 'primary', 'secondary', 'amine', 'of', 'particular', 'interest', 'substitutions', 'ﬂuoroalkoxy', 'groups', 'like', 'following', 'cl', 'cl', 'pn', 'n', '2n', 'c3f7', 'h2o', 'na', 'thf', 'och2c3f7', 'pn', 'n', 'och2c3f7', 'mixed', 'ﬂuoroalkoxy', 'compounds', 'used', 'obtain', 'variety', 'properties', 'the', 'materials', 'ﬁnd', 'application', 'elastomers', 'exhibit', 'good', 'chemical', 'resistance', 'good', 'thermal', 'stability', 'in', 'addition', 'many', 'retain', 'useful', 'elastomeric', 'properties', 'low', 'temperature', 'as', 'result', 'among', 'applications', 'attractive', 'use', 'sealants', 'fuel', 'lines', 'arctic', 'environment', 'phosphonitrile', 'polymers', 'self-extinguishing', 'ﬁre', 'retardant', 'this', 'led', 'development', 'ﬂame-', 'heat-resistant', 'polyimide', 'composites', 'prepared', 'maleimide-substituted', 'phosphazenes', '231', '232.', 'the', 'maleimide', 'group', 'used', 'cross-linking', 'the', 'substituted', 'phosphazene', 'illustrated', 'follows', 'o', 'o', 'o', 'p', 'n', 'o', 'n', 'p', 'o', 'p', 'n', 'o', 'o', 'n', 'o', 'o', 'n', 'o', 'a', 'deferent', 'approach', 'synthesis', 'polyphosphazenes', 'reported', '179.', 'it', 'based', 'condensation', 'suitable', 'si–n–p', 'precursors', 'si', 'n', 'x', 'r', 'p', 'r', '∆', '-(cj3)3', 'six', 'n', 'p', 'r', 'r', 'rr0', '¼', 'alkyl', 'aryl', 'x', '¼', 'och2cf3', 'o–ph', 'in', 'preparation', 'desired', 'substituents', 'introduced', 'polymerization', 'the', 'resultant', 'polymers', '179', 'soluble', 'various', 'solvents', 'their', 'molecular', 'weight', 'distributions', 'vary', '14', '35', 'mw', '50000–150000', '502', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '717', 'high-performance', 'polymers', 'the', 'polymers', 'found', 'common', 'commercial', 'uses', 'ﬁbers', 'ﬁlms', 'structural', 'resins', 'fail', 'withstand', 'elevated', 'temperatures', '250(cid6)c', 'long', 'periods', 'time', 'decompose', 'there', 'need', 'however', 'various', 'technologies', 'materials', 'tolerate', 'temperatures', '300(cid6)c', 'reasonably', 'long', 'periods', 'many', 'materials', 'might', 'high', 'priced', 'common', 'commercial', 'use', 'they', 'ﬁnd', 'application', 'however', 'specialized', 'areas', 'include', 'space', 'aeronautic', 'military', 'technologies', 'higher', 'price', 'justiﬁed', 'greatly', 'enhanced', 'performance', 'in', 'developing', 'tough', 'heat-resistant', 'polymers', 'chemists', 'pursued', 'several', 'goals', '191.', 'these', '1', 'to', 'improve', 'heat', 'stability', 'available', 'polymers', 'introducing', 'structural', 'modiﬁcations', '2', 'to', 'develop', 'new', 'macromolecules', 'based', 'chemical', 'structures', 'capable', 'withstanding', 'high', 'temperatures', '3', 'because', 'many', 'inorganic', 'molecules', 'thermally', 'stable', 'organic', 'ones', 'develop', 'inorganic', 'inorganic–organic', 'polymeric', 'materials', 'improvement', 'thermal', 'stability', 'existing', 'polymers', 'based', 'following', 'considerations', '(1)', 'the', 'primary', 'bond', 'energy', 'atoms', 'polymeric', 'chain', 'greatest', 'source', 'thermal', 'stability', 'the', 'strength', 'bonds', 'therefore', 'imposes', 'upper', 'limit', 'vibrational', 'energy', 'molecule', 'may', 'withstand', 'without', 'bond', 'ruptures', 'in', 'cases', 'cyclic', 'repeating', 'units', 'ladder', 'polymers', 'rupture', 'one', 'bond', 'ring', 'may', 'lead', 'loss', 'molecular', 'weight', 'in', 'polymers', 'two', 'bonds', 'would', 'break', 'within', 'ring', 'chain', 'rupture', 'probability', 'low', 'this', 'means', 'ladder', 'polymers', 'exhibit', 'greater', 'heat', 'stability', 'single-stranded', 'chains', '183.', '(2)', 'secondary', 'bond', 'forces', 'cohesive', 'energies', 'contribute', 'additional', 'stability', 'molecule', '(3)', 'resonance', 'energy', 'aromatic', 'heterocyclic', 'structures', 'contributes', 'additional', 'amount', 'thermal', 'stability', 'bond', 'strength', '(4)', 'polymers', 'high', 'melting', 'softening', 'temperatures', 'generally', 'heat-resistant', '7171', 'fluorine', 'containing', 'aromatic', 'polymers', 'the', 'high', 'strength', 'carbon-ﬂuorine', 'bonds', 'shielding', 'effect', 'highly', 'electronegative', 'ﬂuorine', 'atoms', 'improve', 'heat', 'stability', 'one', 'material', 'synthesized', 'heating', 'perﬂuoroalkyl', 'amidines', 'melting', 'points', 'ammonia', 'evolves', 'triazine', 'ring', 'containing', 'polymers', 'form', '174:', 'n', 'n', '(cf2)', 'x', '(cf2)', 'x', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'the', 'thermal', 'stability', 'polymers', 'vacuum', 'equivalent', 'polytetraﬂuoroethylene', '174.', '717', 'high-performance', 'polymers', '503', 'among', 'ﬂuorine', 'containing', 'materials', 'reported', 'literature', 'group', 'polyesters', '175:', 'o', 'o', '1', '2', '3', 'o', 'o', 'o', 'f', '4', 'f', 'o', 'n', 'f', '4', 'f', 'o', 'n', 'o', 'o', 'o', 'o', '4', 'o', '4', 'n', 'f', 'f', 'f', 'f', 'f', 'f', 'n', 'i', 'i', 'cu', 'powder', 'f', 'f', 'n', 'f', 'f', 'f', 'f', 'f', 'f', 'n', 'i', 'ok', 'o', 'n', 'f', 'f', 'f', 'f', 'their', 'heat', 'stability', 'physical', 'properties', 'however', 'appear', 'superior', 'conventional', 'materials', 'the', 'ullmann', 'reaction', 'used', 'prepare', 'polyperﬂuorophenylene', '176.', 'the', 'products', 'however', 'low', 'molecular', 'weight', 'higher', 'molecular', 'weight', 'materials', 'like', 'poly(perﬂuorophenylene', 'ether)', 'form', 'potassium', 'pentaﬂuorophenoxide', '177,', 'mn', '12500', 'liu', 'et', 'al', 'reported', 'improved', 'process', 'synthesizing', 'ﬂuorinated', 'polymers', 'supercritical', 'carbon', 'dioxide', '177.', 'the', 'improvement', 'consisted', 'modifying', 'reaction', 'system', 'designing', 'using', 'sampling', 'tubes', 'low', 'molecular', 'weight', 'ﬂuorine', 'containing', 'polymers', 'form', 'perﬂuoroaromatic', 'compounds', 'loss', 'aromaticity', 'react', 'bis(ﬂuoroxy)-diﬂuoromethane', '180.', 'more', 'interesting', 'formation', 'poly(aaa0a0-tetraﬂuoro-p-xylylene)', 'polymerization', 'technique', 'closely', 'resembles', 'preparation', 'poly(p-xylylene)', 'vacuum', 'pyrolysis', 'dimer', '181:', 'f', 'f', 'ch2', 'ch2', 'f', 'f', 'f', 'f', 'vacuum', '600', '-', '700', 'oc', 'ch2', 'ch2', 'f', 'f', 'f', 'f', 'ch2', 'ch2', 'n', 'f', 'f', '504', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'the', 'resultant', 'polymers', 'maintain', 'useful', 'mechanical', 'properties', '3000', 'h', 'air', '250(cid6)c', 'fluorinated', 'epoxy', 'resins', 'another', 'group', 'materials', 'might', 'potentially', 'possess', 'improved', 'thermal', 'stability', 'this', 'however', 'demonstrated', 'it', 'shown', '190', 'though', 'materials', 'key', 'intermediates', 'organic', 'coating', 'plastics', 'require', 'special', 'properties', 'like', 'hydrophobicity', 'oleophobicity', 'light', 'stability', 'low', 'friction', '7172', 'polyphenylene', 'this', 'polymer', 'completely', 'aromatic', 'character', '182.', 'polymerization', 'benzene', 'polyphenylene', 'therefore', 'investigated', 'quite', 'thoroughly', '184', '185.', 'benzene', '186', 'aromatic', 'structures', '184', '185', 'polymerize', 'believed', 'radical-cationic', 'mechanism', 'in', 'type', 'polymerization', 'benzene', 'polymerizes', 'mild', 'conditions', 'presence', 'certain', 'lewis', 'acids', 'combined', 'oxidizing', 'agents', '186–188:', 'n', '3n', 'cucl2', 'alcl3', 'h', '3n', 'cucl2', '2n', 'hcl', 'h', 'n', 'the', 'yield', 'polymers', 'reaches', 'maximum', 'value', '(close', 'quantitative)', 'aluminum', 'chloride', 'cupric', 'chloride', 'molar', 'ratio', '21', '180.', 'solvents', 'concentrations', 'temperatures', 'affect', 'molecular', 'weights', 'products', '189.', 'other', 'lewis', 'acids', 'effective', 'benzene', 'polymerizations', 'mocl5', 'fecl3', 'moocl4', 'the', 'products', 'however', 'possess', 'greater', 'degrees', 'structural', 'irregularity', '183.', 'theoretical', 'considerations', 'indicate', '183', 'polymerization', 'benzene', 'rings', 'become', 'associated', 'stacked', 'end', 'end', 'arrangement', 'as', 'result', 'radical-cation', 'becomes', 'delocalized', 'entire', 'chain', 'c6h6', 'c6h6', 'c6h6', 'c6h6', 'c6h6', 'c6h6', 'o,', '(h', '-e)', 'n', 'n', 'the', 'bond', 'formation', 'shown', 'also', 'accompanied', 'simultaneous', 'depropagation', 'loss', 'benzene', 'molecules', 'chain', 'buildup', 'stops', 'radical-cation', 'terminal', 'phenyl', 'group', 'becomes', 'small', 'promote', 'association', 'the', 'polymer', 'stable', '500–600(cid6)c', 'oxidizes', 'slowly', 'it', 'however', 'quite', 'insoluble', 'high', 'melting', 'point', 'makes', 'difﬁcult', 'process', 'even', 'determine', 'molecular', 'weight', 'introduction', 'irregularity', 'polymeric', 'structure', 'copolymerizing', 'terphenyl', 'biphenyl', 'triphenylbenzene', 'benzene', 'results', 'formation', 'soluble', 'products', 'their', 'molecular', 'weights', 'however', 'low', '3000', '192.', 'they', 'melt', '300', '400(cid6)c', 'contain', 'phenyl', 'branches', 'fused', 'rings', 'the', 'copolymers', 'cross-linked', 'xylylene', 'glycol', 'benzene-135-trisulfonyl', 'chloride', 'soluble', 'alkyl', 'substituted', 'poly(p-phenylene)s', 'prepared', '193', 'coupling', 'process', 'using', 'palladium', 'catalysts', '717', 'high-performance', 'polymers', '505', 'r', 'br', 'b(oh)2', 'r', 'r', 'r', 'n', 'syntheses', 'also', 'include', 'formations', 'copolymers', 'aromatics', 'compounds', 'lack', 'substituents', 'the', 'polymers', 'hexyl', 'longer', 'side', 'chains', 'soluble', 'toluene', '193.', 'goodson', 'et', 'al', '246', 'reported', 'formation', 'soluble', 'derivatives', 'poly(p-phenylene)', 'high', 'molecular', 'weight', 'via', 'suzuki', 'coupling', 'reactions', 'catalyzed', 'palladium', '(0)', 'precursors', 'presence', 'either', 'triphenylphosphine', 'tri(o-tolyl)phosphine', 'use', 'triphenylphosphine', 'however', 'appar-', 'ently', 'resulted', 'incorporation', 'phosphine', '246.', 'formation', 'poly(benzoyl-14-phenylene)', 'aid', 'nickel-catalyzed', 'reaction', 'reported', '247:', 'o', 'cl', 'n', 'cl', 'nacl', 'p(c5h5)3nai', 'hcon(ch3)2', '70', 'oc', 'o', 'n', 'when', 'sodium', 'iodide', 'replaced', '220-bipyridine', 'reaction', 'proceeds', 'faster', 'the', 'polymers', 'prepared', 'bipyridine', 'exhibit', 'glass', 'transition', '217(cid6)c', 'loose', '3', 'weight', '500(cid6)c', 'either', 'air', 'nitrogen', '247.', 'carter', 'et', 'al', '247', 'used', 'organolithium-activated', 'nickel', 'catalysts', 'synthesize', 'polyarylates', 'br', 't-buli', 'diphenylphosphinopropane', 'nickel', 'bypyridine', 'br', 'br', 'thf', 'br', 'n', '7173', 'diels–alder', 'polymers', 'the', 'diels–alder', 'reaction', 'used', 'form', 'many', 'polymeric', 'materials', 'one', 'material', 'instance', 'forms', 'reaction', 'diethynylbenzene', 'cyclopentadienone', '194–196.', 'the', 'products', 'phenylated', 'polyphenylenes', 'reach', 'molecular', 'weights', 'mn', '40000', '506', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'o', 'o', 'x', '300', 'oc', 'o', 'o', 'x', '-', 'co', 'n', 'x', 'n', 'x', '¼', '12', 'these', 'polymers', 'amorphous', 'form', 'clear', 'ﬁlms', 'stable', 'air', '550(cid6)c', 'another', 'example', 'condensation', '2-vinyl-13-butadiene', 'p-benzo-quinone', 'the', 'product', 'ladder', 'structure', 'double-stranded', 'polymer', '197:', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', '717', 'high-performance', 'polymers', '507', 'this', 'double-stranded', 'polymer', 'shown', 'soluble', 'hexaﬂuoroisopropyl', 'alcohol', '198', 'infusible', 'when', 'however', 'rings', 'made', 'ﬂexible', 'condensing', '2-vinyl-13-', 'butadiene', 'large', 'ring', 'bis', 'fumarate', 'solubility', 'improves', '198.', 'some', 'stable', 'polymers', 'also', 'form', '13-dipole', 'addition', 'reactions', '190.', 'bis-sydnones', 'instance', 'condense', 'diacetylene', 'form', 'pyrazole', 'rings', 'polymer', 'backbone', 'the', 'reactions', 'presumably', 'proceed', 'diels–alder', 'intermediates', '199:', 'h', 'n', 'n', 'o', 'o', 'o', 'o', 'n', 'n', 'h', 'nc', 'cn', 'n', 'n', 'n', 'n', 'similarly', 'bis-sydnones', 'condense', 'quinone', '199:', 'h', 'n', 'n', 'o', 'o', 'o', 'o', 'n', 'n', 'h', 'o', 'o', 'n', 'o', 'o', 'n', 'n', 'n', 'n', 'n', 'the', 'resultant', 'polymers', 'high', 'molecular', 'weights', 'slightly', 'soluble', 'solvents', 'like', 'dimethylformamide', 'their', 'powders', 'decompose', 'near', '420(cid6)c', 'air', 'near', '500(cid6)c', 'nitrogen', 'atmosphere', '13-dipolar', 'additions', 'bisnitrileimines', '(generated', 'tetrazoles', 'dienes', 'di', 'nitriles)', 'result', 'formations', 'polypyrazoles', 'polytriazoles', '200.', 'some', 'examples', 'reactions', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'nc', 'cn', 'n', 'n', 'n', 'n', 'n', 'n', 'n', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '508', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'f', 'f', 'n', 'n', 'n', 'n', 'n', 'n', 'nn', 'n', 'n', 'by', 'condensation', 'high', 'molecular', 'weight', 'polymers', 'form', '200.', 'they', 'decompose', 'near', '500(cid6)c', 'air', 'nitrogen', 'atmosphere', 'cyclopentadienone', 'derivatives', 'condense', 'diels–alder', 'reaction', 'homopolycycloaddition', 'the', 'reactions', 'involve', 'series', 'steps', 'consisting', 'initial', 'cycloaddition', 'followed', 'loss', 'carbon', 'monoxide', 'expulsion', 'bridge', 'carbonyl', 'group', '201:', 'br', 'br', 'c', 'c', 'o', 'o', 'albr3', 'cs2', 'n-bromosuccinimide', 'ccl4', 'br', 'br', 'ki', 'acetone', 'o', 'o', '(c6h5ch2)2co', 'base', 'f', 'f', 'f', 'f', 'f', 'f', 'o', 'o', 'o', 'o', 'o', 'c', 'o', 'o', '717', 'high-performance', 'polymers', '509', '-co', 'o', 'self', 'condensation', 'o', 'n', '-h', 'o', 'n', 'the', 'products', 'ladder', 'polymers', 'varying', 'molecular', 'weights', 'they', 'loose', 'approximately', '30', '201', 'weight', '700(cid6)c', 'nitrogen', 'atmosphere', 'another', 'example', 'condensation', 'bis', 'dienophiles', 'dienes', '202:', 'n', 'o', 'o', 'o', 'o', 'n', 'o', '510', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'n', 'o', 'o', 'o', 'o', 'n', 'o', 'n', 'o', 'o', 'o', 'o', 'n', '∆', '-co', 'o', 'o', 'n', 'n', 'o', 'o', 'n', 'only', 'low', 'molecular', 'weight', 'polymers', 'form', 'however', 'together', 'insoluble', 'possibly', 'cross-', 'linked', 'material', 'bis', 'maleimides', 'also', 'condense', 'bisfulvene', '202.', 'the', 'products', 'condensations', 'depolymerize', 'reversibly', 'also', 'reactions', 'maleic', 'anhydride', 'dienes', 'like', 'bicycloheptene', 'bicyclohexene', 'dicyclopentenyl', 'ether', 'dicyclohexenyl', 'ether', 'yield', 'soluble', 'low', 'molecular', 'weight', 'polymers', '204:', 'o', 'o', 'o', '2', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'photo-cycloaddition', 'reactions', 'also', 'form', 'low', 'molecular', 'weight', 'polymers', 'heterocyclic', 'rings', 'backbone', 'the', 'reactions', 'photoinitiated', 'benzophenone', 'obtain', 'repeated', '2p', '2p', 'photo-', 'cycloadditions', 'bismaleimides', '205:', 'r', 'r', 'r', 'r', 'n', 'r', 'n', 'o', 'o', 'o', 'o', 'path', 'a', 'r', 'r', 'hν', 'path', 'b', 'r', 'r', 'o', 'o', 'n', 'r', 'n', 'r', 'o', 'r', 'o', 'o', 'n', 'r', 'n', 'n', 'r', 'n', 'o', 'r', 'r', 'o', 'r', 'r', 'o', 'o', 'n', 'o', 'o', 'path', 'c', 'r', 'r', 'insoluble', 'crosslinked', 'polymer', 'r', '¼', '(ch2)x', 'these', 'polymeric', 'materials', 'form', 'transparent', 'ﬂexible', 'ﬁlms', '205.', '717', 'high-performance', 'polymers', '511', 'diels–alder', 'reaction', 'also', 'used', 'modify', 'chemical', 'structures', 'aromatic', 'polyamides', 'this', 'improves', 'heat', 'stability', 'raises', 'tg', 'value', 'makes', 'rigid', 'higher', 'temperatures', '203.', '7174', 'silicon-containing', 'aromatic', 'polymers', 'these', 'materials', 'potential', 'stable', 'high', 'temperatures', 'one', 'typical', 'preparatory', 'procedure', 'consists', 'condensing', 'bisphenols', 'suitable', 'silicon', 'derivatives', '227:', 'r2si(or)2', 'r2sicl2', 'ho', 'ar', 'oh', 'r', 'r', 'si', 'o', 'ar', 'o', 'n', 'many', 'polymers', 'soluble', 'exhibit', 'good', 'heat', 'stability', 'an', 'example', 'one', 'material', 'prepared', 'procedure', 'shown', 'si', 'o', 'o', 'n', 'it', 'looses', '10', 'weight', '600(cid6)c', '191.', 'the', 'starting', 'materials', 'also', 'silanols', 'instance', 'synthesis', 'polycarbonates', '227.', 'preformed', 'bis-silanols', 'used', 'particular', 'example', 'si', 'si', 'n', 'ho', 'o', 'x', 'o', 'o', 'o', 'si', 'si', 'o', 'x', 'oh', 'o', 'n', 'cl', 'cl', 'n', 'si', 'si', 'o', 'x', 'o', 'o', 'o', 'si', 'si', 'o', 'x', 'n', 'x', '¼', '0', '1', 'rings', '228:', 'one', 'publication', 'describes', 'syntheses', 'copolyesters', 'copolyamides', 'contain', 'phenoxalin', '512', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'cl', 'o', 'cl', 'ho', '2', 'o', 'o', '2', 'o', 'o', 'o', 'o', 'o', 'cl', 'oh', 'cl', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'the', 'copolymers', 'show', 'little', 'degradation', 'temperatures', '400(cid6)c', 'in', 'addition', 'copolymers', 'high', 'phenoxailin', 'content', 'soluble', 'several', 'solvents', '228.', 'there', 'also', 'reports', 'literature', 'attempts', 'modify', 'polysiloxane', 'backbones', 'increase', 'heat', 'stability', 'these', 'modiﬁcations', 'consist', 'making', 'changes', 'electronic', 'character', 'si–o', 'bonds', 'prevent', 'rearrangements', 'high', 'temperatures', 'low', 'molecular', 'weight', 'cyclic', 'products', 'it', 'known', 'making', 'bonds', 'ionic', 'covalent', 'increases', 'heat', 'stability', '191.', 'a', 'metal', 'electropositive', 'therefore', 'used', 'form', 'metal-oxygen', 'linkages', 'form', 'polymers', 'ionic', 'character', 'such', 'metals', 'aluminum', 'titanium', 'tin', 'boron', 'the', 'results', 'however', 'far', 'disappointing', '7175', 'direct', 'condensation', 'polymers', 'many', 'polymers', 'enhanced', 'heat', 'stability', 'prepared', 'simply', 'direct', 'condensation', 'these', 'aromatic', 'polymers', 'often', 'contain', 'heterocyclic', 'unit', 'the', 'materials', 'high', 'melting', 'somewhat', 'infusible', 'usually', 'low', 'solubility', 'many', 'aromatic', 'polyimides', 'belong', 'polyimides', 'separate', 'class', 'polymers', 'discussed', 'earlier', 'section', 'many', 'common', 'commer-', 'cial', 'materials', 'on', 'hand', 'materials', 'described', 'section', 'might', 'considered', 'special', 'perhaps', 'points', 'still', 'high', 'priced', 'common', 'usage', '71751', 'polyimidazoles', 'polybenzthiazoles', 'related', 'polymers', 'many', 'polybenzimidazoles', 'prepared', 'direct', 'condensation', 'they', 'colored', 'polymers', 'mostly', 'melt', '400(cid6)c', 'one', 'material', 'formed', '330-diaminobenzidine', 'diphenyl', 'isophthalate', 'heating', 'two', 'together', '350–400(cid6)c', 'inert', 'atmosphere', '206:', 'h2n', 'n', 'h2n', 'nh2', 'nh2', 'n', 'o', '∆', 'o', 'o', 'n', 'n', 'h', 'o', 'n', 'n', 'n', 'h', '717', 'high-performance', 'polymers', '513', 'films', 'ﬁbers', 'material', 'exhibit', 'good', 'mechanical', 'properties', 'temperature', '300(cid6)c', 'above', 'temperature', 'however', 'degrades', 'rapidly', 'air', '206.', 'similar', 'polymers', 'polybenzoxazoles', '207', '208:', 'n', 'h2n', 'ho', 'nh2', 'oh', 'cl', 'o', 'n', 'o', 'n', 'o', 'n', 's', 'o', 'n', 'n', 'n', 'n', 'o', 'n', 's', 'n', 'n', 'the', 'true', 'polybenzthiazoles', '209:', 'n', 'h2n', 'hs', 'nh2', 'sh', 'n', 'o', 'o', 'polyoxidiazoles', 'also', 'belong', 'general', 'class', 'materials', '210:', 'n', 'h', 'n', 'n', 'n', 'n', 'n', 'n', 'h', 'cl', 'cl', 'polybenztriazoles', '211:', 'h', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'h', 'cl', 'n', 'c', 'n', 'c', 'cl', 'o', 'cl', 'n', 'o', 'o', 'o', 'n', 'n', 'hν', 'n', 'n', 'n', 'n', '514', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'all', 'materials', 'maintain', 'useful', 'properties', '300(cid6)c', 'air', 'formed', 'ﬁbers', 'some', 'polymeric', 'materials', 'completely', 'free', 'hydrogens', 'an', 'example', 'polysulfodiazole', '212,', 'polyimide', 'prepared', 'pyrazine-1245-tetracarboxylic', 'diaminothiazine', '213.', 'this', 'material', 'exhibits', 'particularly', 'good', 'heat', 'stability', '213:', 'acid', 'anhydride', 'h2n', 'nh2', 'n', 'n', 's', 'o', 'o', 'o', 'n', 'n', 's', 'n', 'n', 'h', 'n', 'ho', 'o', 'c', 'c', 'o', 'o', 'o', 'o', 'n', 'n', 'h', 'n', 'n', 'oh', 'o', 'c', 'c', 'o', '-2h2o', 'n', 'n', 'o', 's', 'n', 'o', 'n', 'n', 'o', 'n', 'o', 'n', 'films', 'polysulfodiazole', 'maintain', 'strength', 'stability', '592(cid6)c', 'preparations', 'several', 'similar', 'polyimides', 'reported', '219.', 'a', 'polyimide', 'however', 'prepared', 'diaminothiazole', 'pyromellitic', 'dianhydride', 'chars', '320(cid6)c', 'air', 'the', 'chemistry', 'preparations', 'principal', 'types', 'polyheteroarylenes', 'review', 'krongauz', '214.', 'a', 'series', 'thermally', 'stable', 'organic', 'solvent-soluble', 'polyimides', 'synthesized', 'reacting', '37-diaminophenothiazinium', 'chloride', '(thionine)', 'four', 'different', 'dianhydrides', '243.', 'these', 'polyimides', 'illustrated', 'follows', 'n', 'n', 's', 'cl', 'n', 'o', 'o', 'o', 'n', 'o', 'n', 'many', 'polymers', 'described', 'section', 'prepared', 'either', 'one', 'two', 'steps', 'in', 'one-step', 'process', 'polyphosphoric', 'acid', 'often', 'employed', 'solvent', 'it', 'proton', 'donor', 'promotes', 'condensation', 'acts', 'cyclodehydrating', 'agent', 'frequently', 'yielding', 'polymers', 'high', 'molecular', 'weights', 'shaping', 'resulting', 'polymers', 'however', 'problem', 'due', 'infusibility', 'insolubility', 'when', 'prepared', 'two', 'steps', 'polymer', 'shaped', 'ﬁlms', 'ﬁbers', 'ﬁrst', 'much', 'cyclization', 'takes', 'place', 'material', 'still', 'fusible', 'solvent-soluble', 'this', 'followed', 'heating', 'complete', 'process', 'cyclization', 'process', 'occurs', 'solid', 'polymers', 'become', 'increasingly', 'rigid', 'reaction', 'progresses', 'the', 'products', 'however', 'prepared', 'procedure', 'fully', 'cyclized', 'formed', 'one-step', 'process', 'phosphoric', 'acid', 'solution', '7176', 'oligomers', 'terminal', 'functional', 'groups', 'the', 'above-described', 'two-step', 'processes', 'yield', 'polymers', 'evolve', 'volatiles', 'upon', 'heating', 'condensations', 'continue', 'to', 'overcome', 'drawback', 'prepolymers', 'developed', 'undergo', 'addition-type', 'reactions', 'fairly', 'moderate', 'time-temperature', 'schedules', '215', '216.', '717', 'high-performance', 'polymers', '515', 'such', 'prepolymers', 'terminated', 'functional', 'groups', 'following', 'example', 'one', 'material', 'oligomer', 'polyquinoxaline', 'terminated', 'acetylene', 'groups', 'n', 'o', 'o', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'no', 'volatiles', 'detected', 'mass', 'spectrometry', '215', '216', 'thermo', 'gravimetric', 'analyses', 'cross-linking', 'reaction', 'the', 'thermo', 'oxidative', 'stability', 'resultant', 'polymers', 'least', 'equivalent', 'polyphenylquioxalines', 'terminated', 'acetylene', 'the', 'cross-linking', 'reaction', 'shown', 'model', 'compound', 'intramolecular', 'cyclization', '215–217:', 'it', 'necessary', 'acetylenic', 'groups', 'terminal', 'ends', 'prepolymers', 'they', 'also', 'located', 'pendant', 'structures', '216.', 'aromatic', 'polyamides', 'terminal', 'acetylenic', 'groups', '218', 'formed', '220-', 'diiododiphenyl-440-dicarbonyl', 'chloride', 'reacted', 'aromatic', 'diamines', 'the', 'phenylethynyl', 'groups', 'introduced', 'reacting', 'iodine', 'moieties', 'copper', 'phenyl', 'acetylide', 'thermal', 'treatment', 'converted', 'prepolymers', '9-phenyl', 'dibenzanthracene-based', 'rigid-rod', 'polymers', 'fail', 'melt', '500(cid6)c', 'high', 'molecular', 'weight', 'polyquinoxaline', 'polymers', 'prepared', '330440-tetraamino-', 'biphenyl', 'reacted', 'aromatic', 'bis(a-diketones)', 'andor', 'ethynyl-substituted', 'aromatic', 'bis', '(a-diketones)', '219.', 'the', 'polymers', 'contain', '0', '5', '10', '30', '100', 'pendant', 'groups', 'also', 'ethynyl-', 'substituted', 'diketones', 'synthesized', 'following', 'procedure', '2', 'br', 'o', '1', 'alcl3', 'cl', '2', 'hydrolysis', 'o', 'o', 'o', 'so2', 'br', '2', 'br', '2', 'pdcl2', 'o', 'o', '516', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'n', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'si', '2', 'p', '3', '2', 'k2co3', 'o', 'si', '2', 'h2n', 'h2n', 'n', 'n', '2', 'o', 'o', 'nh2', 'nh2', 'o', 'n', 'the', 'synthesis', 'completed', 'condensation', 'tetramine', 'above', 'shown', 'reactions', 'yield', 'polymers', 'high', 'tg', 'values', 'the', 'materials', 'however', 'exhibited', 'lowered', 'thermo', 'oxidative', 'stability', 'the', 'found', 'true', 'ethynyl', 'moieties', 'replaced', 'phenylethynyl', 'groups', '219.', 'other', 'functional', 'groups', 'investigated', '229', 'phenylethynyl', 'phenylbutadiynyl', 'phenylbutenyl', 'biphenylene', 'styryl', 'maleimide', 'nadimide', '(5-norbornene-23-dicarboximide)', 'cyanate', 'n-cyanourea', '230.', 'the', 'advantage', 'terminally', 'capped', 'prepolymers', 'melt', 'lower', 'temperatures', 'dissolved', 'different', 'solvent', 'heating', 'materials', 'converts', 'thermally', 'stable', 'polymeric', 'networks', 'a', 'recent', 'paper', 'reports', 'preparation', 'quinoline', 'oligomers', 'end-capped', '4-', 'acetylbenzocyclobutene', '6-acetyl-8-phenyl-12-dihydro-3', '4cyclobuta-12-bquinone', '(cbq)', '8-acetyl-6b10b-dihydrobenzo-jcyclobuta-12-a-acenaphthalene(bcban)', '230.', 'the', 'structures', 'two', 'cross-linking', 'groups', 'o', 'ph', 'o', 'n', 'cbq', 'bcban', 'it', 'reported', '230', 'oligomer', 'capped', 'bcban', 'yields', 'cured', 'ﬁlm', 'exhibits', 'good', 'ﬂexural', 'moduli', 'superior', 'heat', 'stability', 'air', '400(cid6)c', '717', 'high-performance', 'polymers', '7177', 'cardo', 'polymers', '517', 'these', 'special', 'group', '201', '208', '220', 'polymeric', 'materials', 'the', 'name', 'cardo', 'comes', 'latin', 'meaning', 'loop', '201', '208.', 'the', 'polymers', 'contain', 'cyclic', 'structures', 'may', 'perpendicular', 'aromatic', 'backbones', 'an', 'example', 'would', 'cardo', 'polybenzimidazole', 'n', 'n', 'h', 'o', 'n', 'n', 'h', 'n', 'o', 'o', 'n', 'o', 'another', 'example', 'cardo', 'polyester', '201', '208:', 'many', 'cardo', 'polymers', 'exhibit', 'improved', 'solubility', 'different', 'solvents', 'little', 'sacriﬁce', 'properties', 'a', 'survey', 'made', 'physical', 'properties', 'different', 'cardo', 'copolyimides', 'varying', 'microstructures', '220.', 'as', 'might', 'expected', 'copolyimides', 'increasing', 'aliphatic', 'frag-', 'ment', 'content', 'shown', 'exhibit', 'highest', 'impact', 'ﬂexural', 'strength', '7178', 'double-stranded', 'polymers', 'preparations', 'many', 'double-stranded', 'polymers', 'reported', 'for', 'instance', 'polypyrrolones', 'formed', '2356-tetraaminobenzene', 'various', 'dianhydrides', 'tetracarboxylic', 'acid', 'compounds', '213.', 'following', 'two', 'examples', '1', 'o', 'o', 'o', 'o', 'o', 'o', 'h2n', 'h2n', 'nh2', 'nh2', 'o', 'n', 'n', 'o', 'n', 'n', 'n', '518', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '2', 'ho', 'ho', 'o', 'o', 'o', 'oh', 'oh', 'o', 'h2n', 'h2n', 'different', 'ladder', 'polyquinoxalines', 'prepared', 'well', 'one', 'example', 'shown', '233:', 'o', 'o', 'o', 'o', 'h2n', 'h2n', 'nh2', 'nh2', 'o', 'n', 'n', 'n', 'o', 'n', 'n', 'nh2', 'nh2', 'n', 'n', 'n', 'n', 'n', 'some', 'ladder', 'polyquinoxalines', 'found', 'stable', 'air', '460(cid6)c', 'nitrogen', '683(cid6)c', 'not', 'attempts', 'formations', 'ladder', 'polymers', 'yielded', 'completely', 'cyclized', 'fused', 'ring', 'structures', 'for', 'instance', 'attempt', 'form', 'polymer', 'tetraaminonaphthalene', 'naphthalene', 'tetracarboxylic', 'acid', 'dianhydride', 'failed', 'yield', 'complete', 'cyclizations', '222.', 'an', 'interesting', 'polymer', 'containing', 'macrocyclic', 'rings', 'formed', 'pyromellitic', 'tetranitrile', 'condensation', 'dianilino', 'ether', '223:', 'nc', 'nc', 'cn', 'cn', 'h2n', 'o', 'nh2', '717', 'high-performance', 'polymers', '519', 'the', 'description', 'double-stranded', 'polymers', 'synthesized', 'reported', 'literature', 'beyond', 'scope', 'book', 'this', 'section', 'concluded', 'mentioning', 'two', 'materials', 'one', 'prepared', 'alkali', 'catalyzed', 'polymerization', 'phenyltrichlorosilane', '224:', 'cl', 'si', 'cl', 'cl', 'h2o', 'oh', 'o', 'si', 'o', 'o', 'sio', 'o', 'si', 'o', 'si', 'n', 'the', 'polymer', 'cis-syndiotactic', '225', 'cis-anti-cis', 'arrangement', 'phenyl', 'groups', 'it', 'stable', 'temperatures', '525(cid6)c', 'the', 'ones', 'soluble', 'ladder', 'conjugated', 'polymer', 'composed', 'perylene', 'diimides', 'thieno', '32', 'bthiophene', 'prepared', 'recently', 'synthesis', 'via', 'photo', 'cyclization', 'irradiation', 'sunlight', '224.', 'these', 'soluble', 'polymers', 'prepared', 'photonic', 'applications', 'illustrated', 'follows', '520', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'o', 'n', 'n', 'o', 'n', 'n', 'o', 'o', 's', 's', 'o', 'o', '7179', 'poly(arylene', 'ether)s', 'poly(arylene', 'ether', 'ketone)s', 'high-performance', 'polymeric', 'material', 'poly(arylene', 'ether)s', 'poly(arylene', 'ether', 'ketone)s', 'they', 'used', 'structural', 'resins', 'composite', 'fabrications', 'offer', 'attractive', 'combination', 'chemical', 'mechanical', 'physical', 'properties', 'commercial', 'poly(ether', 'ketone)s', 'also', 'poly(arylene', 'ether)s', 'formed', 'diacid', 'chloride', 'friedel–craft', 'reaction', '234.', 'o', 'n', 'n', 'cl', 'o', 'o', 'o', 'n', 'o', 'similar', 'polyketones', 'form', 'dicarboxylic', 'acids', 'rather', 'dicarboxylic', 'acid', 'chlorides', 'p2o5—methanesulfonic', 'acid', 'used', 'catalyst', '235.', 'all', 'materials', 'form', 'group', 'tough', 'high', 'melting', 'resins', 'some', 'current', 'preparations', 'poly(arylene', 'ether)s', 'carried', 'nucleophilic', 'displacements', 'activated', 'aromatic', 'dihalides', 'dinitro', 'groups', 'alkali', 'metal', 'bisphenates', 'the', 'reactions', 'take', 'place', 'polar', 'aprotic', 'solvents', 'the', 'glass', 'transition', 'temperatures', 'tensile', 'strengths', 'tensile', 'moduli', 'materials', 'tend', 'increase', 'heterocyclic', 'units', 'incorporated', 'backbones', 'poly(arylene', 'ether)s', 'containing', 'imide', '236,', 'phenylquinoxaline', '237–239,', 'imidazoles', '240,', 'pyrazoles', '241,', '134-oxadiazoles', '240,', 'benzoxazoles', '240,', 'benzimidazoles', '240', 'groups', 'prepared', 'h', 'n', 'n', 'h', 'cl', 'o', '718', 'dendrimers', 'polyrotaxanes', '521', 'the', 'preparation', 'polymeric', 'materials', 'illustrated', 'follows', '241:', 'ho', 'oh', 'x', 'y', 'x', 'n', 'n', 'r', 'o', 'o', 'y', 'n', 'n', 'n', 'r', 'r', '¼', 'h', 'ph', 'x', '¼', 'cl', 'f', 'y', '¼', 'carbonyl', 'sulfone', 'diketone', 'aryl', 'group', 'the', 'shown', 'medium', 'high', 'molecular', 'weight', 'polymers', 'exhibit', 'good', 'solubility', 'solvents', 'like', 'dimethyl', 'acetamide', 'good', 'thermal', 'stability', 'the', 'true', 'poly(arylene', 'ether)s', 'mentioned', 'as', 'result', 'high-performance', 'thermoplastics', 'potential', 'useful', 'low', 'cost', 'composite', 'fabrications', 'one', 'paper', '242', 'describes', 'preparation', 'poly(imidoaryl', 'ether', 'ketone)s', 'poly(imidoaryl', 'ether', 'sulfone)s', 'exhibit', 'particularly', 'good', 'heat', 'stability', 'the', 'synthesis', 'illustrated', 'follows', 'ho', 'oh', 'f', 'x', 'f', 'k2co3', 'solvent', '160-220', 'oc', 'o', 'o', 'n', 'r', 'o', 'o', 'n', 'r', 'o', 'x', 'n', 'r', '¼', 'ch3', 'ch3(ch2)11', 'phenyl', 'x', '¼', 'so2', 'co', 'isophthaloyl', 'based', 'thermogravimetric', 'analyses', 'phenyl', 'substituted', 'polymers', 'lose', '10', 'weight', 'air', 'nitrogen', '550(cid6)c', 'in', 'addition', 'polymers', 'high', 'glass', 'transition', 'temperatures', 'remain', 'soluble', 'common', 'solvents', 'like', 'chloroform', 'methylene', 'chloride', 'these', 'high', 'molecular', 'weights', 'polymers', 'cast', 'solution', 'give', 'tough', 'ﬂexible', 'ﬁlms', '242.', '718', 'dendrimers', 'polyrotaxanes', 'one', 'fascinating', 'things', 'polymer', 'chemistry', 'ability', 'chemists', 'develop', 'new', 'interesting', 'molecular', 'structures', 'this', 'result', 'formation', 'materials', 'unique', 'properties', 'among', 'materials', 'dendrimers', 'polyrotaxanes', '522', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '7181', 'dendrimers', 'hyperbranched', 'polymers', 'highly', 'branched', 'polymeric', 'materials', 'large', 'number', 'end', 'groups', 'offer', 'unique', 'physical', 'properties', 'dendrimers', '(described', 'chap', '1)', 'differ', 'linear', 'polymers', 'viscosity', 'thermal', 'behavior', 'a', 'variety', 'applications', 'forecast', 'highly', 'branched', 'globular', 'macromolecules', 'one', 'early', 'approach', 'dedrimer', 'synthesis', 'relied', 'upon', 'michael', 'reaction', 'using', 'repeated', 'sequential', 'additions', 'amine', 'ab-unsaturated', 'ester', '248:', 'nh3', 'h2nch2ch2nh2', 'o', 'o', 'h2nch2ch2nhcoch2ch2', 'n', 'ch2ch2conhch2ch2nh2', 'ch2ch2conhch2ch2nh2', 'repeat', 'h2n(ch2)2nc(ch)2', 'h2n(ch2)2nc(ch)2', 'n(ch2)2nc(ch2)2n', 'h', 'o', 'h', 'o', 'h', 'o', 'h2n(ch2)2nc(ch)2n', 'h', 'o', 'o', '(ch2)2cn(ch2)2nh2', 'h', 'o', '(ch2)2cn(ch2)2nh2', 'ho', '(ch2)2cn(ch2)2nh2', 'h', 'o', 'h', 'o', 'h2n(ch2)2nc(ch)2n', 'h', '(ch2)2cn(ch2)2nh2', 'in', 'ﬁrst', 'step', 'ammonia', 'added', 'excess', 'methyl', 'acrylate', 'the', 'product', 'reacted', 'excess', 'ethylenediamine', 'form', 'star', 'molecule', 'three', 'arms', 'the', 'product', 'second', 'step', 'reacted', 'excess', 'methyl', 'acrylate', 'this', 'followed', 'reaction', 'excess', 'ethylenediamine', 'form', 'molecule', 'six', 'arms', 'the', 'sequences', 'repeated', 'yield', 'star', 'polymer', 'such', 'star', 'polymers', 'sometimes', 'referred', 'starburst', 'polymers', 'a', 'recent', 'publication', 'described', 'preparation', 'dendrimers', 'functional', 'aliphatic', 'polyesters', 'based', '22-bis(hydroxymethyl)propionic', 'acid', '249.', 'nn0-dicyclohexylcarbodiimide', 'used', 'coupling', 'agent', 'double-stage', 'convergent', 'approach', 'reduced', 'number', 'synthetic', 'liquid', 'chromatographic', 'steps', 'required', 'preparations', 'puriﬁcations', 'the', 'hydroxyl', 'functional', 'dendrimers', 'subjected', 'variety', 'surface', 'modiﬁcations', 'reactions', 'different', 'acid', 'chlorides', '249.', 'another', 'publication', 'describes', 'formation', 'polyphenylene', 'dendrimers', 'formed', 'nanoparticles', '102', 'benzene', 'rings', '250.', 'in', 'preparation', 'a—b', 'type', 'monomer', '2345-tetrakis(4-', 'triisopropylsilylethynylphenyl)-cyclopenta-23-dienone', 'condensed', '4', '2cycloaddition', '330550-tetraethynyl-biphenyl', '718', 'dendrimers', 'polyrotaxanes', '523', '(i-pr)3si', 'si(i-pr)3', '(i-pr)3si', 'si(i-pr)3', 'r', 'r', 'r', 'r', 'o', 'r', 'r', 'r', 'r', 'r', 'r', 'the', 'condensation', 'reactions', 'carried', '180–200(cid6)c', 'aromatic', 'solvents', '250.', 'preparation', 'dendrimers', 'usually', 'involves', 'numerous', 'synthetic', 'steps', 'some', 'attempted', 'simplify', 'process', 'merely', 'forming', 'hyperbranched', 'polymer', 'the', 'products', 'less', 'perfect', 'structures', 'dendrimers', 'offer', 'advantage', 'synthesized', 'one', 'step', 'a', 'recent', 'publication', 'describes', 'preparation', 'hyperbranched', 'poly(e-caprolactone)s', '251.', 'living', 'ring', 'opening', 'polymerizations', 'carried', 'aid', 'aluminum', 'benzyl', 'oxide', 'initiator', 'the', 'resultant', 'polymers', 'functionalized', 'benzylidine-protected', '220bis(hydroxymethyl)propionic', 'acid', 'after', 'subsequent', 'deprotection', 'a-carboxylic-o-dihydroxyl', 'functional', 'ab2', 'macromonomers', 'formed', 'these', 'ab2', 'polyesters', 'condensed', 'hyperbranched', 'polymers', 'room-', 'temperature', 'esteriﬁcation', 'synthesis', 'using', 'dicyclohexylcarbodiimide', '4-(dimethylamino)', 'pyridinium', '4-toluenesulfonate', '251.', '7182', 'polyrotaxanes', 'preparation', 'polymeric', 'materials', 'consist', 'linear', 'structures', 'threaded', 'large', 'rings', '(see', 'chap', '1)', 'intrigued', 'many', 'the', 'result', 'various', 'publications', 'appeared', 'literature', 'describing', 'preparations', '252', '253.', 'in', 'many', 'instances', 'crown', 'ethers', 'used', '524', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', 'polyesters', 'polyamides', 'polyurethanes', 'polystyrene', 'threaded', '254.', 'one', 'approach', 'synthesis', 'carry', 'polymerization', 'reaction', 'presence', 'macrocycles', 'form', 'threading', 'one', 'step', '254.', 'another', 'approach', 'stir', 'solution', 'melt', 'aliphatic', 'crown', 'ether', 'linear', 'polymer', 'like', 'polyurethane', '255.', 'the', 'hydrogen', 'bonding', 'nh', 'groups', 'polymer', 'ether', 'oxygen', 'atoms', 'cyclic', 'species', 'described', 'driving', 'force', 'threading', 'the', 'products', 'preparations', 'described', 'pseudorotaxanes', 'o', '2', 'nh', 'o', 'n', 'h', 'the', 'degree', 'threading', 'found', 'increase', 'increasing', 'molar', 'feed', 'ratio', 'crown', 'ethers', 'polymeric', 'nh', 'groups', '255.', 'polymeric', 'rotaxanes', 'reviewed', 'recently', 'harada', 'et', 'al', '719', 'thermodynamics', 'step-growth', 'polymerization', 'the', 'number', 'average', 'degree', 'polymerization', '(cid1)pn', 'condensation', 'reaction', 'diacid', 'glycol', 'elimination', 'water', 'function', 'extent', 'reaction', 'we', 'therefore', 'write', 'in', 'reaction', 'equilibrium', 'constant', 'rate', 'constant', 'independent', 'molecular', 'size', 'it', 'therefore', 'possible', 'write', '(cid1)pn', '¼', '1ð1', '(cid1)', 'pþ', 'k', '¼', 'pnhð1', '(cid1)', 'pþ2', 'nh', 'concentration', 'water', 'formed', 'reaction', 'p', 'extent', 'reaction', 'by', 'token', 'average', 'degree', 'polymerization', 'written', '(cid1)pn', '¼', 'ð2knhþ½ð1', 'þ', '4knhþ12', '(cid1)', '1(cid3)', '(cid1)pn', '¼', 'ðknhþ12', 'knh', 'equal', '1', 'therefore', 'ln', '(cid1)pndð1tþ', '¼', '½d', 'ln', 'kdð1tþ', '(cid1)', 'ln', 'nhdð1tþ(cid3)', '1', '2', 'the', 'free', 'energy', 'change', 'step', 'growth', 'condensation', 'df', '¼', 'dh', '(cid1)', 'tds', '¼', 'rt', 'ln', 'k', 'by', 'plotting', 'ln', 'k', 'reciprocals', 'absolute', 'temperatures', 'one', 'obtains', 'roughly', 'linear', 'relationship', 'sawada', 'wrote', 'therefore', 'constant', 'dh', 'enthalpy', 'change', 'reaction', 'ln', '(cid1)pndð1tþ', '¼', 'adh', '525', 'review', 'questions', 'review', 'questions', 'section', '71', 'tetramers', 'etc', '1', 'describe', 'types', 'monomers', 'undergo', 'step-growth', 'polymerizations', '2', 'illustrate', 'step-growth', 'polymerization', 'formation', 'poly(butylene', 'adipate)', 'showing', 'dimers', '3', 'does', 'size', 'molecule', 'inﬂuence', 'reactivity', 'functional', 'group', 'explain', '4', 'how', 'would', 'express', 'rate', 'disappearance', 'one', 'functional', 'groups', '5', 'assuming', 'concentrations', 'two', 'reacting', 'functional', 'groups', 'stochiometrically', 'equal', 'write', 'rate', 'equation', '6', 'write', 'equation', 'degree', 'polymerization', 'step-growth', 'polymerization', '7', 'what', 'stoichiometric', 'imbalance', 'designated', '8', 'write', 'expression', 'molecular', 'weight', 'average', 'product', 'also', 'molecular', 'weight', 'distribution', '9', 'by', 'including', 'average', 'functionality', 'factor', 'write', 'equation', 'degree', 'conversion', '10', 'discuss', 'ring', 'formations', 'accompany', 'step-growth', 'polymerizations', '11', 'explain', 'step-growth', 'polymers', 'formed', 'melt', 'interfacial', 'polycondensation', 'why', 'isn’t', 'preparatory', 'technique', 'applicable', 'preparation', 'aliphatic', 'polyesters', 'yet', 'works', 'well', 'preparation', 'aliphatic', 'polyamides', '12', 'in', 'trying', 'form', 'polyester', 'g-hydroxybutyric', 'acid', 'percent', 'conversion', 'required', 'obtain', 'molecular', 'weight', '25000', 'section', '72', '1', 'write', 'chemical', 'equations', 'eight', 'common', 'synthetic', 'methods', 'preparing', 'polyesters', 'can', 'discuss', 'advantages', 'disadvantages', 'synthesis', '2', 'describe', 'giving', 'chemical', 'equations', 'catalysts', 'reaction', 'conditions', 'commercial', 'preparations', 'poly(ethylene', 'terephthalate)', '3', 'explain', 'including', 'chemical', 'structures', 'polyester', 'elastomers', 'formed', '4', 'what', 'linear', 'unsaturated', 'polyesters', 'explain', 'show', 'chemical', 'reactions', 'prepared', 'how', 'cross-linked', 'explain', 'help', 'chemical', 'reactions', '5', 'describe', 'network', 'polyesters', 'explain', 'prepared', 'two', 'different', 'techniques', '6', 'what', 'short', 'medium', 'long', 'oil', 'alkyds', '7', 'there', 'two', 'different', 'techniques', 'forming', 'polycarbonates', 'describe', '8', 'what', 'synthetic', 'route', 'polycarbonates', 'aid', 'crown', 'ethers', 'explain', 'help', 'chemical', 'equations', '9', 'what', 'commercial', 'uses', 'polycaprolactone', 'section', '73', '1', 'discuss', 'nylon', 'nomenclature', '2', 'discuss', 'chemistry', 'preparation', 'nylons', '1', '3', '4', '5', 'showing', 'equations', '3', 'discuss', 'common', 'synthetic', 'routes', 'caprolactam', '4', 'describe', 'conditions', 'preparation', 'nylon', '6', '526', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '5', 'describe', 'chemical', 'equations', 'preparations', 'nylon', '7', '9', 'what', 'shortcomings', 'process', 'used', 'russia', 'explain', '6', 'describe', 'chemical', 'equations', 'synthetic', 'route', 'nylons', '8', '12', '7', 'how', 'nylon', '11', 'produced', 'methyl', 'ricinoleate', 'show', 'steps', '8', 'describe', 'chemical', 'equations', 'commercial', 'synthesis', 'trogamid', 't', '9', 'what', 'fatty', 'polyamides', 'formed', '10', 'what', 'nylon', '6t', 'produced', '11', 'discuss', 'chemistry', 'preparation', 'fully', 'aromatic', 'polyamides', 'showing', 'chemical', 'structures', '12', 'explain', 'example', 'heck', 'reaction', 'extended', 'form', 'aromatic', 'polyamides', '1', 'discuss', 'chemistry', 'aromatic', 'polyamide-imides', 'aromatic', 'polyester-imides', 'give', 'examples', '1', 'what', 'common', 'commercial', 'procedures', 'preparations', 'aromatic', 'polyimides', 'describe', 'illustrations', '2', 'what', 'dianhydrides', 'commonly', 'used', 'commercially', 'preparations', 'polyimides', '3', 'illustrate', 'chemical', 'reactions', 'polyimides', 'formed', 'aromatic', 'diisocyanates', 'aromatic', 'dianhydrides', 'do', 'ketimines', 'nn0-bis(trimethylsilyl)', 'compounds', '4', 'how', 'polyimides', 'formed', 'reactions', 'diimides', 'dihalides', 'show', 'chemical', '5', 'show', 'reactions', 'formation', 'polyimides', 'reactions', 'sulfur', 'halides', 'unsaturated', 'diimides', 'diepoxides', 'diimides', '6', 'illustrate', 'explain', 'photochemical', 'reactions', 'bis', 'maleimides', 'benzene', 'form', '7', 'illustrate', 'polyimides', 'form', 'diels–alder', 'reactions', '1', 'discuss', 'illustrate', 'chemical', 'equations', 'formation', 'poly(phenylene', 'oxide)', 'oxida-', 'tive', 'coupling', 'reaction', '26-disubstituted', 'phenols', '2', 'what', 'commercial', 'material', 'called', 'noryl', 'explain', '3', 'what', 'phenoxy', 'resins', 'describe', 'prepared', 'explain', 'used', 'section', '74', 'section', '75', 'reactions', 'polyimides', 'section', '76', 'section', '77', '1', 'discuss', 'polyacetals', 'describe', 'polyacetals', 'available', 'commercially', 'today', 'review', 'questions', 'section', '78', 'section', '79', 'section', '710', 'section', '711', 'section', '712', '527', '1', 'describe', 'poly(p-xylylene)', 'originally', 'prepared', '2', 'discuss', 'transport', 'polymerization', 'explain', 'currently', 'practiced', 'commercially', '3', 'describe', 'routes', 'polyxylylenes', 'what', 'properties', 'polyxylylenes', '1', 'what', 'important', 'industrial', 'sulfur-containing', 'polymers', '2', 'show', 'synthetic', 'routes', 'aromatic', 'sulfones', 'prepared', '3', 'describe', 'preparation', 'poly(phenylene', 'sulﬁde)', 'properties', 'uses', '4', 'how', 'poly(alkylene', 'sulﬁde)', 'prepared', 'used', 'commercially', '1', 'illustrate', 'chemical', 'equations', 'routes', 'forming', 'polyisocyanates', '2', 'what', 'mechanisms', 'reactions', 'isocyanates', 'hydrogen', 'donors', 'how', 'reactions', 'catalyzed', 'discuss', 'mechanism', '3', 'discuss', 'polyurethane', 'ﬁbers', 'how', 'made', 'what', 'properties', '4', 'discuss', 'polyurethane', 'elastomers', 'including', 'properties', 'preparation', '5', 'discuss', 'polyurethane', 'foams', 'how', 'prepared', '1', 'discuss', 'chemistry', 'epoxy', 'resins', 'based', 'diglycidyl', 'ethers', 'bisphenol', 'a', 'preparations', 'cross-linking', 'reactions', 'amines', 'dianhydrides', 'dicyanodiamide', '2', 'what', 'epoxy', 'novolacs', 'new', 'epoxy', 'resins', 'containing', 'nitrogen', '3', 'discuss', 'chemistry', 'cycloaliphatic', 'epoxides', '1', 'discuss', 'chemistry', 'resoles', 'showing', 'chemical', 'reactions', 'form', 'what', 'quinone', 'methides', 'meant', 'stages', 'a', 'b', 'c', 'resins', '2', 'describe', 'chemistry', 'novolacs', 'formed', 'cross-linked', '3', 'explain', 'products', 'ammonia', 'amine-catalyzed', 'condensation', 'phenol', 'formaldehyde', '4', 'describe', 'typical', 'commercial', 'preparations', 'resols', 'novolacs', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '1', 'discuss', 'chemistry', 'urea-formaldehyde', 'resins', 'preparation', 'uses', '2', 'do', 'question', 'one', 'melamine-formaldehyde', 'resins', '1', 'how', 'silicon', 'compounds', 'differ', 'carbon', 'compounds', '2', 'how', 'starting', 'materials', 'prepared', 'silicone', 'resins', '3', 'how', 'high', 'molecular', 'weight', 'silicone', 'resins', 'formed', 'ring', 'opening', 'polymerization', 'what', 'products', 'acid', 'catalysis', 'basic', 'catalysis', '4', 'describe', 'silicone', 'elastomers', 'what', 'rtv-s', 'explain', 'show', 'way', 'cross-linked', '5', 'discuss', 'ﬂuorosilicone', 'elastomers', '6', 'discuss', 'polyarylsiloxanes', '1', 'why', 'interest', 'polysilanes', 'show', 'chemical', 'reactions', 'formed', '1', 'what', 'polyphosphazines', 'formed', 'used', '528', 'section', '713', 'section', '714', 'section', '715', 'section', '716', 'section', '717', 'materials', '1', 'what', 'chemical', 'options', 'available', 'improve', 'heat', 'stability', 'toughness', 'polymeric', '2', 'discuss', 'ﬂuorine', 'containing', 'aromatic', 'polymers', '3', 'discuss', 'chemistry', 'preparation', 'polyphenylene', '4', 'discuss', 'diels–alder', 'polymers', 'giving', 'least', 'four', 'examples', 'showing', 'structures', 'starting', 'materials', 'products', '5', 'discuss', 'silicon-containing', 'aromatic', 'polymers', '6', 'what', 'direct', 'condensation', 'polymers', 'how', 'polybenzimidazoles', 'polybezoxazoles', 'polybenzthiazoles', 'polyoxidiazoles', 'polybenzotriazoles', 'polysulfodiazoles', 'prepared', 'illustrate', 'chemical', 'equations', '7', 'discuss', 'chemistry', 'oligomers', 'terminated', 'functional', 'groups', 'form', 'thermoset', 'high', 'heat-resistant', 'materials', '8', 'what', 'cardo', 'polymers', 'advantages', 'prepared', '9', 'discuss', 'double-stranded', 'polymers', 'prepared', '10', 'discuss', 'chemistry', 'poly(arylene', 'ether)s', 'poly(arylene', 'ketone)s', '529', '1', 'describe', 'dendrimers', 'polyrotaxanes', 'references', 'section', '718', 'further', 'reading', 'references', 'me', 'rodgers', 'te', 'long', 'eds', 'synthetic', 'methods', 'step-growth', 'polymers', 'wiley', 'new', 'york', '2003', '1', 'pj', 'flory', 'principles', 'polymer', 'chemistry', 'chapters', '238', '9', 'cornell', 'university', 'press', 'ithaca', 'new', 'york', '1953', 'pem', 'allan', 'cr', 'patrick', 'kinetics', 'mechanism', 'polymerization', 'reactions', 'chap', '5', 'wiley', '-', 'interscience', 'new', 'york', '1974', '2', 'i', 'vansco-szmercsanyi', 'e', 'makay-bodi', 'eur', 'polym', 'j,', '1969', '5', '145', '155', '3', 'pj', 'flory', 'chem', 'rev,', '1946', '39', '137', '4', 'dh', 'solomon', '(ed)', 'step', '-', 'growth', 'polymerization', 'dekker', 'new', 'york', '5', 'g', 'odian', 'principles', 'polymerization', 'wiley', '3nd', 'ed,', 'new', 'york', '1991', '6', 'wh', 'carothers', 'j', 'am', 'chem', 'soc,', '1929', '51', '2548', 'wh', 'carothers', 'ja', 'arvin', 'j', 'am', 'chem', 'soc,', '1929', '51', '2560', 'wh', 'carothers', 'jw', 'hill', 'j', 'am', 'chem', 'soc,', '1932', '54', '1559', '1577', '7', 'wh', 'stockmayer', 'j', 'polymer', 'sci,', '1952', '9', '69', 'ibid', '1953', '11', '424', '8', 'e', 'heisenberg', 'aj', 'watzl', 'can', 'pat,', '570148', '(1959)', '9', 'jt', 'dickson', 'hpw', 'huggill', 'jc', 'welch', 'brit', 'pat,', '590451', '(1947)', '10', 'jr', 'whinﬁeld', 'nature', '1946', '158', '930', '11', 'jgn', 'drewitt', 'j', 'lincoln', 'us', 'pat,', '2799667', '(1957)', '12', 'jg', 'cook', 'brit', 'pat', '590417', '(1947)', '13', 'f', 'reeder', 'er', 'wallsgrove', 'brit', 'pat,', '651762', '(1951)', '14', 'rf', 'fischer', 'j', 'polymer', 'sci,', '1960', '44', '155', '15', 'rf', 'wilfong', 'j', 'polymer', 'sci,', '1961', '54', '385', '16', 'h', 'ludwig', 'ed,', 'polyester', 'fibers', 'chemistry', 'technology', 'wiley', '-', 'interscience', 'new', 'york', '1971', '17', 'rw', 'stevenson', 'hr', 'nettleton', 'j', 'polymer', 'sci', '1968', 'a-16', '889', '18', 'rw', 'stevenson', 'j', 'polymer', 'sci', '1969', 'a-17', '395', '19', 'dm', 'chay', 'cc', 'cumbo', 'mj', 'randolph', 'pc', 'yates', 'us', 'pat,', '3676477', '(july', '11', '1972)', '20', 'sb', 'maerov', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1979', '17', '4033', '21', 'sg', 'hovenkamp', 'j', 'polymer', 'sci,', '1971', 'a-19', '3617', '22', 'c', 'm', 'fontana', 'j', 'polymer', 'sci', '1968', 'a-16', '2343', 'k', 'ravindranath', 'ra', 'mashelkar', 'j', 'appl', 'polymer', 'sci,', '1981', '26', '3179', '23', 'sd', 'hamann', 'dh', 'solomon', 'j', 'swift', 'j', 'macromol', 'sci', '-chem', '1968', 'a2(1)', '153', 'i', 'goodman', 'bf', 'nesbitt', 'polymer', '1960', '1', '384', 'j', 'polymer', 'sci,', '48', '423', '(1960)lh', 'peebles', 'jr,', 'mw', 'huffman', 'ct', 'ablett', 'j', 'polymer', 'sci', '1969', 'a-17', '479', 'e', 'ito', 's', 'okamura', 'polymer', 'letters', '1969', '7', '483', '24', 'chem', 'eng', 'news', '2002', 'may', '20', 'p14', '25', 'h', 'tanaka', 'y', 'iwanaga', 'g', 'wu', 'k', 'sanui', 'n', 'ogata', 'polymer', 'j,', '1982', '14', '648', 'f', 'higashi', 'a', 'hoshio', 'y', 'yamada', 'm', 'ozawa', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1985', '23', '69', 'f', 'higashi', 'n', 'akiyama', 'i', 'takahashi', 't', 'koyama', 'j', 'polymer', 'sci', 'polymer', 'chem', 'ed,', '1984', '22', '1653', '26', 'vv', 'korshak', 'sv', 'rogozin', 'vi', 'volkov', 'vysokomol', 'soed,', '1959', '1', '804', '27', 'k', 'harashi', 'macomololecules', '1970', '3', '5', '28', 'pw', 'morgan', 'condensation', 'polymers', 'by', 'interfacial', 'solution', 'methods', 'wiley', '-', 'interscience', 'new', 'york', '1965pw', 'morgan', 'j', 'macrom', 'sci,', '1981', 'a-15', '683', 'f', 'millich', 'ce', 'carraher', 'jr,', 'eds', 'interfacial', 'sunthesis', 'dekker', 'new', 'york', '1975', '1977', '29', 'ng', 'gaylord', 'polyethers', 'part', 'i', 'wiley-interscience', 'new', 'york', '1963', '30', 'cg', 'gebelein', 'j', 'polymer', 'sci', '1972,', 'a-110', '1763', '31', 't', 'nagata', 'j', 'appl', 'polymer', 'sci,', '1969', '13', '2601', '32', 'en', 'zilberman', 'ae', 'kulikova', 'n', 'm', 'teplyakov', 'j', 'polymer', 'sci,', '1962', '56', '417', '530', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '33', 'wf', 'christopher', 'dw', 'fox', 'polycarbonates', 'reinhold', 'new', 'york', '1962', 'h', 'schnell', 'chemistry', 'physics', 'polycarbonates', 'wiley-interscience', 'new', 'york', '1964', 'h', 'vernaleken', 'interfacial', 'syntheses', 'efmillich', 'ce', 'carreher', 'jr', 'ed,', 'dekker', 'new', 'york', '1975', '34', 's', 'ohsawa', 'k', 'morino', 'a', 'sudo', 't', 'endo', 'macromolecules', '2011', '44', '(7)', '1814', '35', 'cj', 'pedersen', 'hk', 'frensdorff', 'angew', 'chem,', 'int', 'ed,', '1972', '11', '16', '36', 'k', 'saga', 'y', 'toshida', 's', 'hosoda', 's', 'ikeda', 'makromol', 'chem,', '1977', '178', '2747', 'k', 'saga', 's', 'hosoda', 's', 'ikeda', 'j', 'polymer', 'sci,', 'polymer', 'letters', '1977', '15', '611', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1979', '17', '517', 'k-i', 'okuyama', 'j-i', 'sugiyama', 'r', 'nagahata', 'm', 'asai', 'm', 'ueda', 'ktakeuchi', 'macromolecules', '2003', '36(19)', '6953', '37', 'r', 'rokicki', 'wkuran', 'j', 'kielkiewicz', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1982', '20', '967', '38', 'y', 'liu', 's', 'r', 'turner', 'g', 'wlikes', 'macronrolecuies', '2011', '44(11)', '4049', 'f', 'higashi', 'n', 'kokubo', 'm', 'goto', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1980', '18', '2879', '39', 'hj', 'sanders', 'chem', 'eng', 'news', '1985', '63(13)', '30', '40', 'ks', 'devi', 'p', 'vasudevan', 'j', 'macromol', 'sci,', 'rev', 'macromol', 'chem', 'phys,', '1985c25', '325', '41', 'ce', 'lowe', 'us', 'pat', '2688162', '(1954)', '42', 'cen', 'bawn', 'a', 'ledwith', 'chem', 'ind', '(london)', '1957', '1180', '43', 'f', 'higashi', 'k', 'kubota', 'm', 'sekizuka', 'm', 'higashi', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed', '1981', '19', '2681', '44', 'fw', 'billmaeyer', 'jr', 'ad', 'eckard', 'macromolecules', '1969', '2', '103', '(1969)', '45', 'uh', 'so', 'acs', 'polymer', 'preprints', '32(1)', '369', '(1991)', '46', 'n', 'ogata', 'bull', 'chem', 'soc', 'japan', '33', '906', '(1960)', '47', 'wr', 'sorensen', 'tw', 'campbell', '“preparative', 'methods', 'polymer', 'chemistry”', '2-nd', 'ed,', 'wiley', '-', 'interscience', 'new', 'york', '1968', 'gc', 'east', 's', 'hassell', 'j', 'chem', 'ed,', '60', '69', '(1983)', '48', 'ds', 'breslow', 'ga', 'hulsa', 'as', 'metlack', 'j', 'am', 'chem', 'soc,', '79', '3760', '49', 'js', 'moore', 'si', 'stupp', 'macromolecules', '1990', '23', '65', '50', 'h', 'bestian', 'angew', 'chem,', 'int', 'ed,', '1968', '7', '278', '51', 'r', 'graf', 'liebig', 'ann', 'chem,', '1963', '661', '111', '52', 'j', 'masamoto', 'k', 'sasaguri', 'c', 'ohizumi', 'k', 'yamaguchi', 'h', 'kabayashi', 'j', 'appl', 'polymer', 'sci,', '1970', '14', '667', '53', 'hk', 'hall', 'jr,', 'j', 'am', 'chem', 'soc,', '1958', '80', '6404', '54', 'sg', 'havenkamp', 'jp', 'munting', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '1970', '8', '679', 'm', 'drocher', 'j', 'appl', 'polymer', 'sci,', '1981', '36', '217', '55', 'h', 'tani', 't', 'konomi', 'j', 'polymer', 'sci', '1968,', 'a-16', '2295', '56', 's', 'tsuda', 'chm', 'econ', 'eng', 'rev', '1970,', '39', '57', 'e', 'muller', 'angew', 'chem,', '1959', '71', '229', '58', 'hk', 'reinschuessel', 'macromol', 'rev,', '1977', '12', '65', '59', 'brit', 'pat', '1391135', '(1975)', '(from', 'ref', '58)', '60', 'ph', 'hermans', 'recueil', 'trav', 'chim,', '1953', '72', '798', '61', 'i', 'rothe', 'm', 'rothe', 'chem', 'ber', '1955', '88', '284', '62', 'm', 'rothe', 'j', 'polymer', 'sci,', '1958', '30', '227', '63', 'm', 'rothe', 'makromol', 'chem,', '1960', '35', '183', '64', 'a', 'reiche', 'nad', 'w', 'schon', 'chem', 'ber,', '1967', '99', '3238', 'kunststoffe', '1967', '57', '49', '65', 'i', 'kohan', 'nylon', 'plastics', 'wiley', 'new', 'york', '1973', '66', 'an', 'nesmeyanov', 'rk', 'freundlina', 'tetrahedron', '196217', '65', '67', 'wl', 'kohlhase', 'eh', 'pryde', 'jc', 'cowan', 'j', 'am', 'oil', 'chem', 'soc,', '1970', '47', '183', '68', 'wr', 'miller', 'eh', 'pryde', 'ra', 'awl', 'wl', 'kohlhase', 'dj', 'moore', 'ind', 'eng', 'chem', 'prod', 'res', 'dev,', '1971', '10', '442', '69', 'us', 'pat', '3476820', '(1968)', '70', 'dd', 'coffman', 'gj', 'berchet', 'wr', 'peterson', 'ew', 'spangel', 'j', 'polymer', 'sci,', '2', '306', '(1947)', '71', 'us', 'pat', '#', '3145193', '3150113', '3150117', '(1964)', '3198771', '(1965)', '3294758', '(1966)', '72', 'r', 'hill', 'ee', 'walker', 'j', 'polymer', 'sci,', '3', '609', '(1948)', '73', 'cw', 'bunn', 'ev', 'garner', 'proc', 'roy', 'soc(london)', 'a189', '39', '(1947)', '74', 'j', 'preston', 'wb', 'black', 'j', 'polymer', 'sci,', 'polymer', 'letters', '3', '845', '(1965)', '75', 'j', 'preson', 'wb', 'black', 'j', 'polymer', 'sci,', 'c', '-', '23', '441', '(1968)', '76', 'bf', 'malichenko', 'vv', 'senkova', 'et', 'al', 'vysokomol', 'soyedin', 'ser', 'b', '14(6)', '423', '(1972)', '77', 'wh', 'daly', 'w', 'kern', 'makromol', 'chem,', '108', '1', '(1967)', 'jt', 'chapin', 'bk', 'onder', 'wj', 'farrissey', 'am', 'chemsoc', 'polymer', 'preprints', '21(2)', '130', '(180)', '78', 'n', 'ogata', 'k', 'sanui', 'm', 'harada', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '17', '2401', '(1979)', '79', 'yp', 'khanna', 'em', 'pearce', 'bd', 'forman', 'da', 'bini', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '19', '2799', '(1981)', '80', 'yp', 'khanna', 'em', 'pearce', 'js', 'smith', 'dt', 'burkitt', 'h', 'njuguna', 'dm', 'hinderlang', 'bd', 'forman', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '19', '2817', '(1981)', '81', 'm', 'ueda', 's', 'aoyama', 'm', 'konno', 'y', 'imai', 'makromol', 'chem,', '179', '2089', '(1978)', 'references', '531', '82', 'f', 'higashi', 'm', 'goto', 'h', 'kakinoki', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '18', '1711', '(1980)', 'n', 'yamazaki', 'm', 'matsumoto', 'f', 'higashi', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '13', '1373', '(1975)', 'f', 'higashi', 'y', 'tagushi', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '18', '2875', '(1980)', 'f', 'higashi', 'n', 'akiyama', 'si', 'ogata', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '21', '1025', '(1983)', '83', 'cj', 'frosch', 'us', 'pat', '2421024', '(1947)', '84', 'k', 'yaniguchi', 'g', 'ricicki', 'w', 'kawanobe', 's', 'nakahama', 'n', 'yamazaki', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '20', '118', '(1982)', 'z', 'zhang', 'k', 'huang', 'z', 'liu', 'macromolecules', '2011', '44', '(4)', '820', '85', 'df', 'loncrini', 'us', 'pat', '3182073', '(1965)', '86', 'jl', 'nieto', 'makromol', 'chem,', '183', '557', '(1982)', '87', 'mt', 'bogert', 'rr', 'renshaw', 'j', 'am', 'chem', 'soc,', '30', '1135', '(1908)', '88', 'h', 'lee', 'd', 'stoffey', 'k', 'neville', 'new', 'linear', 'polymers', 'mcgraw-hill', 'new', 'york', '1967', 'kl', 'mittal', '(ed)', 'polyimides', 'vol', '1', '2', 'plenum-press', 'new', 'york', '1984', 'mi', 'beesonov', 'mm', 'kotonvvkudriavtsev', 'la', 'laius', 'polyimides', 'thermally', 'stable', 'polymers', 'plenum-press', 'new', 'york', '1987', 'mw', 'ranney', 'polyimide', 'manufacture', 'noyes', 'data', 'corp', 'park', 'ridge', 'nj,', '1971', 'c', 'feger', 'mk', 'kojastech', 'je', 'mcgrath', '(ed-s)', 'polyimideschemistry', 'characterization', 'materials', 'elsevier', '1989', 'd', 'wilson', 'p', 'hergenrother', 'h', 'stenzenberger', '(eds)', 'polyimides', 'chapman', 'hall', '1990', 'd', 'm', 'munoz', 'm', 'celle', 'jg', 'de', 'la', 'campa,', 'j', 'de', 'abajo', 'a', 'e', 'lozano', 'macromolecules', '2009', '42', '(15)', '5892', '89', 'ra', 'meyers', 'j', 'polymer', 'sci,', 'a-17', '2757', '(1969)', 'wj', 'farissey', 'jr,', 'js', 'rose', 'ps', 'carleton', 'j', 'appl', 'polymer', 'sci,', '14', '1093', '(1970)', 'ps', 'carleton', 'wj', 'farissey', 'jr,', 'js', 'rose', 'j', 'appl', 'polymer', 'sci,', '16', '2983', '(1972)', '90', 'wm', 'alvino', 'le', 'edelman', 'j', 'appl', 'polymer', 'sci,', '19', '2961', '(1975)', 'nd', 'ghatge', 'up', 'mulik', 'j', 'polymer', '91', 'r', 'merten', 'angew', 'chem,', 'int', 'ed,', '10', '294', '(1971)', 'b', 'y', 'oishi', 'm', 'kakimoto', 'y', 'imai', 'macromolecules', '24', 'sci,', 'polymer', 'chem', 'ed,', '18', '1905', '(1980)', '3475', '(1991)', '92', 'vl', 'bell', 'j', 'polymer', 'sci,', 'polymer', 'letter', '5', '941', '(1966)', '93', 's', 'nishizaki', 'a', 'fukami', 'kogyo', 'kogaku', 'zasshi', '68', '383', '(1965)', 'chem', 'abstr,', '63', '3057', '(1965)', '94', 'aa', 'berlin', 'tv', 'zelenetskaya', 'rm', 'aseeva', 'zh', 'vseh', 'khim', 'obschest,', '15', '591', '(1966)', '95', 'ra', 'meyers', 'er', 'wilson', 'j', 'polymer', 'sci,', 'polymer', 'letters', '6', '531', '(1968)', '96', 'y', 'iwakura', 'f', 'hayano', 'j', 'polymer', 'sci,', 'a-17', '597', '(1969)', '97', 'm', 'russo', 'l', 'mortillaro', 'j', 'polymer', 'sci,', 'a-17', '3337', '(1969)', '98', 'y', 'musa', 'm', 'p', 'stevens', 'j', 'polymer', 'sci,', 'a-110', '319', '(1972)', '99', 'f', 'w', 'harris', 'j', 'k', 'stille', 'macromolecules', '1', '463', '(1968)', '100', 'y', 'imai', 'j', 'polymer', 'sci,', 'polymer', 'letters', '8', '555', '(1970)', '101', 'ok', 'goins', 'rl', 'van', 'deusen', 'j', 'polymr', 'sci,', 'polymer', 'letters', '6', '821', '(1968)', '102', 'as', 'hay', 'adv', 'polymer', 'sci,', '4', '496', '(1967)', 'ro', 'johnson', 'hs', 'burlhis', 'j', 'polymer', 'sci,', 'polymer', 'symp,', '70', '129', '(1983)', 'n-h', 'you', 'y', 'nakamura', 't', 'higashihara', 's', 'ando', 'm', 'ueda', 'am', 'chem', 'soc', 'polymer', 'preprints', '2009', '50', '(1)', '493', '103', 'wgb', 'huysmans', 'wa', 'waters', 'j', 'am', 'chem', 'soc,', '(b)', '1163', '(1967)', '104', 'gd', 'cooper', 'a', 'katchman', 'chapt', '43', '“addition', 'condensation', 'polymerization', 'processes”', 'acs', 'publication', 'advances', 'chemistry', 'series', '91', '1969', 'jg', 'bennet', 'jr', 'gd', 'cooper', 'macromolecules', '3', '101', '(1970)', '105', 'cj', 'kurian', 'cc', 'price', 'j', 'polymer', 'sci.', '49', '267', '(1961)', '106', 'la', 'errede', 'm', 'szwarc', 'quart', 'rev', '(london)', '12', '301', '(1958)', 'la', 'errede', 's', 'gregorian', 'j', 'polymer', 'sci,', '60', '21', '(1962)', 'la', 'errede', 'n', 'knoll', 'j', 'polymer', 'sci,', '60', '33', '(1962)', 'la', 'errede', 'brit', 'pat', '920515', '(1963)', '107', 'h', 'gilch', 'angew', 'chem,', 'intern', 'ed,', '4', '598', '(1965)', 'j', 'polymer', 'sci,', 'a-14', '43', '(1966)', '108', 'kluiber', 'j', 'org', 'chem,', '6', '2037', '(1965)', '109', 'rb', 'akin', 'acetal', 'resins', 'reinhold', 'new', 'york', '1962', 'ng', 'gaylord', 'ed,', 'polyethers', 'part', 'iii', 'wiley', '-', 'interscience', 'new', 'york', '1962', '110', 'rb', 'seymour', 'gs', 'kirshenbaum', '(eds)', 'high', 'performance', 'materials', 'their', 'origin', 'development', 'elsevier', 'new', 'york', '1986', 'dg', 'brady', 'j', 'appl', 'polymer', 'sci,', 'symposia', '36', '231', '1981', 'df', 'fahey', 'ce', 'ash', 'macromolecules', '24', '4242', '(1991)', '111', 'h', 'a', 'vogel', 'j', 'polymer', 'sci,', 'a-18', '2035', '(1970)', '112', 'm', 'yoneyama', 'm', 'kakimoto', 'y', 'imai', 'macromolecules', '21', '109', '(1988)', 'm', 'yoneyama', 'm', 'kakimoto', 'y', 'imai', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '27', '1985', '(1989)', 'm', 'yoneyama', 't', 'konishi', 'n', 'kakimoto', 'y', 'imai', 'makromol', 'chem,', 'rapid', 'commun,', '11', '381', '(1990)', 'y', 'imai', 'am', 'chem', 'soc', 'polymer', 'preprints', '32(1)', '331', '(1991)', '113', 'jf', 'klebe', 'adv', 'org', 'chem,', '8', '97', '(1972)', 'z', 'cenyao', 'h', 'tomoya', 'u', 'mitsuru', 'am', 'chem', 'soc', 'polymer', '114', 'y', 'oshi', 'm', 'kakimoto', 'y', 'imai', 'macromolecules', '20', '703', '(1987)', 'ibid,', '21', '547', '(1988)', 'y', 'imai', 'amchem', 'preprints', '2010', '51(2)', '506', 'soc', 'polymer', 'preprints', '32(1)', '397', '(1991)', '532', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '115', 'hd', 'stenzenberger', 'm', 'herzog', 'w', 'romer', 'r', 'scheiblich', 'nj', 'reeves', 'br', 'polym', 'j,', '15', '2', '(1983)', 'ik', 'varma', 's', 'sharma', 'eur', 'polym', 'j,', '20', '1101', '(1984)', '116', 'ca', 'may', '(ed)', '“epoxy', 'resins', 'chemistry', 'technology”', '2nd', 'ed,', 'dekker', 'new', 'york', '1988', 'b', 'sedlacek', 'j', 'kahovek', '(eds)', 'crosslinked', 'epoxies', 'de', 'gruyler', 'new', 'york', '1986', 'rs', 'bauer', '(ed)', '“epoxy', 'resin', 'chemistry”', '2', 'vols', 'am', 'chem', 'soc', 'symp', 'ser', '114', '221', 'amer', 'chem', 'so,', 'washington', 'dc,', '1979', '1983', '117', 'wj', 'bailey', 'aavolpe', 'j', 'polymer', 'sci,', 'a-18', '2109', '(1970)', '118', 'rn', 'johnson', 'ag', 'faruham', 'ra', 'glendenning', 'wf', 'hall', 'cn', 'merriam', 'j', 'polymer', 'sci,', 'a-15', '2375', '(1967)', '119', 'jb', 'rose', 'polymer', '15', '456', '(1974)', '120', 'te', 'attwood', 'ab', 'newton', 'jb', 'rose', 'brit', 'polym', 'j,', '4', '391', '(1972)', '121', 'ab', 'newton', 'jb', 'rose', 'polymer', '13', '465', '(1972)', '122', 's', 'ozaki', 'chem', 'rev,', '72', '457', '(1972)', '123', 'kc', 'frisch', 'lp', 'rumao', 'j', 'macromol', 'sci,', 'rev', 'marcomol', 'chem,', 'c-5', '103', '(1970)', '124', 'jw', 'baker', 'jb', 'haldsworth', 'j', 'chem', 'soc,', '1974', '713', '125', 'jw', 'baker', 'j', 'gaunt', 'j', 'chem', 'soc,', '1949', '919', '2427', '126', 'kn', 'edwards', '(ed)', 'urethane', 'chemistry', 'applications', 'am', 'chem', 'soc', 'symposium', 'ser', '172', 'am', 'chem', 'soc,', 'washington', 'dc,', '1981', '127', 'f', 'hostettler', 'ef', 'cox', 'ind', 'eng', 'chem52,', '609', '(1960)', '128', 'jw', 'britain', 'pg', 'gemeinhardt', 'j', 'appl', 'polymer', 'sci,', '4', '207', '(1960)', '129', 'm', 'furukawa', 't', 'yokoyama', 'makromol', 'chem,', '182', '2201', '(1981)', '130', 'j', 'blackwell', 'kh', 'gardner', 'polymer', '20', '13', '(1979)', '131', 'r', 'bonart', 'angew', 'makromol', 'chem,', '5859', '259', '(1977)', '132', 'bo', 'dombrow', '“polyurethanes”', 'reinhold', 'ny', '1957', '133', 'castan', 'us', 'pat', '2444333', '(1948)', '134', 'wg', 'potter', 'epoxy', 'resins', 'springer-verlag', 'new', 'york', '1970', 'ca', 'may', '(ed)', '“epoxy', 'resins', 'chemistry', 'technology”', '2-nd', 'ed,', 'dekker', 'new', 'york', '1988', '135', 'pf', 'bruins', 'epoxy', 'resin', 'technology', 'wiley-interscience', 'new', 'york', '1968', 'b', 'sedlacek', 'j', 'kahovek', '(eds)', 'crosslinked', 'epoxies', 'de', 'gruyler', 'new', 'york', '1986', 'rs', 'bauer', '(ed)', 'epoxy', 'resin', 'chemistry', '2', 'vols,', 'acs', 'sympos', 'series', 'am', 'chem', 'soc,', 'washington', 'dc,', '1979', '1983', '136', 'y', 'tanaka', 'h', 'kakuichi', 'j', 'appl', 'polymer', 'sci,', '7', '1063', '2951', '(1963)', 'm', 'a', 'm', 'rahmathullah', 'a', 'jeyarajasingam', 'b', 'merritt', 'm', 'van', 'landingham', 's', 'h', 'mcknight', 'g', 'r', 'palmese', 'macromolecules', '2009', '42', '(9)', '3219', '137', 'm', 'fischer', 'f', 'lohse', 'r', 'schmid', 'makromol', 'chem,', '181', '1251', '(1980)', 'f', 'riccardi', 'mm', 'joullied', 'wa', 'ramanchick', 'aa', 'giscavage', 'j', 'polymer', 'sci,', 'polymer', 'letters', '21', '127', '(1982)', '138', 'js', 'bradshaw', 'mp', 'stevens', 'j', 'appl', 'polymer', 'sci,', '10', '1809', '(1966)', '139', 'l', 'shechter', 'j', 'wynstra', 'ind', 'eng', 'chem,', '48', '86', '(1956)', '140', 'tf', 'saunders', 'mf', 'levy', 'jf', 'serino', 'j', 'polymer', 'sci,', 'a-15', '1609', '(1967)', '141', 'e', 'pyun', 'csp', 'sung', 'macromolecules', '24', '855', '(1991)', '142', 'm', 'dibenedetto', 'j', 'coat', 'tech,', '52', '65', '(1980)', '143', 'mk', 'autoon', 'jl', 'koenig,', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '19', '549', '(1981)', '144', 'df', 'gould', 'phenolic', 'resins', 'reinhold', 'new', 'york', '1959', '145', 'njl', 'megson', 'phenolic', 'resin', 'chemistry', 'academic', 'press', 'new', 'york', '1958', 'a', 'knop', 'la', 'plato', 'phenolic', 'resins', 'springer-verlag', 'new', 'york', '1979', '146', 'rw', 'martin', 'the', 'chemistry', 'phenolic', 'resins', 'wiley', '-', 'interscience', 'new', 'york', '1956', '147', 'a', 'zinke', 'st', 'pucher', 'monatsh', '79', '26', '(1948)', '148', 'ts', 'carswell', '“phenoplasts”', 'wiley', '-', 'interscience', 'new', 'york', '1947', '149', 'al', 'cupples', 'h', 'lee', 'dg', 'stoffey', 'chapt', '15', '“advances', 'chemistry', 'series”', '92', 'rf', 'gould', 'ed,', '150', 'a', 'ninagawa', 'h', 'matsuda', 'makromol', 'chem,', 'rapid', 'commun,', '2', '449', '(1981)', 'a', 'ninagawa', 'y', 'ohnishi', 'h', 'takeuchi', 'h', 'matsuda', 'macromol', 'chem,', 'rapid', 'commun,', '6', '793', '(1985)', '151', 'cp', 'vale', 'aminoplastics', 'cleaver', '-', 'hume', 'press', 'london', '1950', 'cp', 'vale', 'wgk', 'taylor', 'aminoplastics', 'am', 'chem', 'soc,', '1970', 'iliffe', 'london', '1964', '152', 'sm', 'kambanis', 'rc', 'vasishth', 'j', 'appl', 'polymer', 'sci,', '15', '1911', '(1971)', '153', 'ak', 'dunker', 'we', 'johns', 'r', 'rammon', 'wl', 'plagemann', 'j', 'adhesion', '17', '275', '(1985)', 'ak', 'dunker', 'we', 'johns', 'r', 'rammon', 'b', 'farmer', 'sj', 'johns', 'j', 'adhesion', '19', '153', '(1986)', '154', 'ss', 'jada', 'j', 'appl', 'polym', 'sci,', '35', '1573', '(1988)', '155', 'k', 'koeda', 'j', 'chem', 'soc', 'japan', 'pure', 'chem', 'soc,', '75', '571', '(1954)', '156', 'eg', 'rochow', '“an', 'introduction', 'chemistry', 'silicones”', '2nd', 'ed,wiley', '-', 'interscience', 'new', 'york', '1951', '157', 'rr', 'mcgregor', 'silicones', 'their', 'uses', 'mcgraw', '-', 'hill', 'new', 'york', '1954', 'references', '533', '158', 'rn', 'meals', 'fm', 'lewis', 'silicones', 'reinhold', 'new', 'york', '1959', '159', 'w', 'noll', 'chemistry', 'technology', 'silicones', 'academic', 'press', 'new', 'york', '1968', '160', 'gg', 'freeman', 'silicones', 'an', 'introduction', 'chemistry', 'applications', 'iliffe', 'london', '1962', '161', 'ka', 'andrianov', 'zhdanov', 'izvest', 'akad', 'nauk', 'sssr,', 'otdel', 'khim', 'nauk', '1033', '(1954)', '162', 'mmsprung', 'gguenther', 'j', 'am', 'chem', 'soc,', '77', '6045', '(1955)', 'ibid', '77', '3996', '(1955)', '163', 'ee', 'bostick', 'chept', '8', 'ring', 'opening', 'polymerizations', 'kc', 'frisch', 'sl', 'reegen', 'eds,', 'dekker', 'new', 'york', '164', 'ka', 'andrianov', 'methods', 'synthesis', 'organometalloid', 'polymersinternsymp', 'inorg', 'polymers', '1969', 'nottingham', 'england', '1961', '165', 'rd', 'miller', 'adv', 'chem', 'ser,', '224', '413', '(1990)', '166', 's', 'yajima', 'j', 'hayashiand', 'm', 'omori', 'chem', 'lett,', '1975', '931', '167', 'rd', 'miller', 'j', 'michi', 'chem', 'rev,', '89', '1359', '(1989)', '168', 'r', 'west', 'j', 'organomet', 'che,', '300', '327', '(1986)', '169', 'ma', 'abkowitz', 'm', 'stolka', 'rj', 'weagley', 'km', 'mcgrane', 'fe', 'knier', 'adv', 'chem', 'ser,', '224', '467', '(1990)', '170', 'k', 'matyjaszewski', 'am', 'chem', 'soc', 'polymer', 'preprints', '31(2)', '224', '(1990)', '171', 'ar', 'wolf', 'i', 'nozue', 'j', 'maxka', 'r', 'west', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '26', '701', '(1988)', '172', 'k', 'metyjaszewski', 'yl', 'chen', 'j', 'organomet', 'chem,', '340', '7', '(1988)', '173', 'hr', 'allcock', 'chem', 'rev,', '72', '315', '(1972)', 'hr', 'allcock', 'angew', 'chem', 'int', 'ed,', '16', '124', '(1977)', 'science', '193', '1214', '(1976)', 'polymer', '21', '673', '(1980)', 't', 'l', 'evans', 'hr', 'allcock', 'j', 'macromol', 'sci-', 'chem,', 'a16', '409', '(1981)', 'hr', 'allcock', 'j', 'polymer', 'sci,', 'polym', 'symp,', '70', '71', '(1983)', '174', 'hc', 'brown', 'j', 'polymer', 'sci,', '44', '9', '(1960)', '175', 'j', 'm', 'cox', 'ba', 'wright', 'ww', 'wright', 'j', 'appl', 'polymer', 'sci,', '8', '2935', '(1964)', '176', 'm', 'hellman', 'aj', 'bilbo', 'wj', 'plummer', 'j', 'am', 'chem', 'soc,', '77', '3650', '(1955)', '177', 'wj', 'plummer', 'la', 'well', '145th', 'acs', 'meeting', '1963', 'z-t', 'liu', 'j-g', 'chen', 'z-w', 'liu', 'j', 'lu', 'macromolecules', '2008', 'asap', 'article', '101021ma800193v', '178', 'ga', 'edwards', 'g', 'goldﬁnger', 'j', 'polymer', 'sci,', '36', '589', '(1955)', '179', 'rh', 'neilson', 'p', 'wisian-neilson', 'chem', 'rev,', '88', '541', '(1988)', 'rh', 'neilson', 'p', 'wisian-neilson', 'jj', 'meister', 'ak', 'roy', 'gl', 'hagnauer', 'macromolecules', '20', '910', '(1987)', 'rh', 'neilson', 'dj', 'jinkerson', 's', 'karthikeyan', 'r', 'samuel', 'ce', 'wood', 'am', 'chem', 'soc', 'polymer', 'preprints', '32(3)', '483', '(1991)', '180', 'ij', 'hotchkiss', 'r', 'tephens', 'jc', 'tatlow', 'j', 'fluorine', 'chem,', '10', '541', '(1977)', '181', 'sw', 'chow', 'we', 'loeb', 'ce', 'white', 'j', 'appl', 'polymer', 'chem,', '13', '2325', '(1969)', '182', 'cs', 'marvel', 'appl', 'polymer', 'symp,', 'no', '22', '47', '(1973)', '183', 'jg', 'speight', 'p', 'kovacic', 'fw', 'koch', 'j', 'macromol', 'sci,', '-', 'revs', 'chem,', 'c', '5', '295', '(1971)', 'sa', 'milosevich', 'k', 'saichek', 'l', 'hunchey', 'wb', 'england', 'p', 'kovacic', 'j', 'am', 'chem', 'soc,', '105', '1088', '(1983)', '184', 'mb', 'jones', 'p', 'kovacic', 'd', 'lanska', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '19', '89', '(1981)', 'mb', 'jones', 'pkovacic', 'rf', 'howe', 'j', 'polymer', 'sci.', 'polymer', 'chem', 'ed,', '19', '235', '(1981)', 'cfhsing', 'mb', 'jones', 'pkovacic', 'j', 'polymer', 'sci,polymer', 'chem', 'ed,', '19', '973', '(1981)', 'bs', 'lamb', 'p', 'kovacic', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '18', '2423', '(1980)', '185', 'i', 'khoury', 'p', 'kovacic', 'j', 'polymer', 'sci,', 'polymer', 'letters', '19', '395', '(1981)', '186', 'p', 'kovacic', 'c', 'wu', 'j', 'polymer', 'sci,', '47', '45', '(1960)', 'pkovacic', 'a', 'kyriakis', 'tetrahedron', 'letters', 'p467', '(1962)', 'p', 'kovacic', 'a', 'hyriakis', 'j', 'am', 'chem', 'soc,', '75', '454', '(1963)', '187', 'p', 'kovacic', 'rj', 'hopper', 'j', 'polymer', 'sci,', '4', '1445', '(1966)', 'p', 'kovacic', 'j', 'ozionick', 'j', 'org', 'chem,', '29', '100', '(1964)', '188', 'p', 'kovacic', 'lc', 'hsu', 'j', 'polymer', 'sci,', '4', '5', '(1966)', '189', 'gg', 'emgstrom', 'p', 'kovacic', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '15', '2453', '(1977)', '190', 'jr', 'grifﬁth', 'rf', 'brady', 'jr,', 'chemtech', '19(6)', '370', '(1989)', 'h', 's-w', 'hu', 'amd', 'jr', 'grifﬁth', 'am', 'chem', 'soc', 'polymer', 'preprints', '32(3)', '216', '(1991)', '191', 'pe', 'cassidy', 'thermally', 'stable', 'polymers', 'dekker', 'new', 'york', '1980', 'jp', 'critchy', 'gj', 'knight', 'ww', 'wright', 'heat', 'resistant', 'polymers', 'plenum', 'press', 'new', 'york', '1983', 'ah', 'frazer', 'high', 'temperature', 'resistant', 'polymers', 'wiley-interscience', 'new', 'york', '1968', '192', 'n', 'bilow', 'lj', 'miller', 'j', 'macromol', 'sci,', 'chem,', 'a1', '183', '(1967)', 'ibid', 'a3', '501', '(1969)', '193', 'm', 'rahahn', 'ad', 'schluter', 'g', 'wegner', 'makromol', 'chem,', '191', '1991', '(1990)', '194', 'h', 'mukamal', 'fw', 'harris', 'jk', 'stille', 'j', 'polymer', 'sci,', 'a-15', '2721', '(1967)', '195', 'jk', 'stille', 'gk', 'noren', 'j', 'polymer', 'sci,', 'polymer', 'letters', '7', '525', '(1969)', '196', 'cl', 'schilling', 'jr,', 'ja', 'reed', 'jk', 'stille', 'macromolecules', '2', '85', '(1969)', '197', 'wj', 'bailey', 'j', 'economy', 'me', 'hermes', 'j', 'org', 'chem,', '27', '3259', '(1962)', '198', 'wj', 'bailey', 'aa', 'volpe', 'j', 'polymer', 'sci,', 'a-18', '2109', '(1970)', '199', 'jk', 'stille', 'ma', 'bedford', 'j', 'polymer', 'sci,', 'a-16', '2331', '(1968)', 'jk', 'stille', 'j', 'macromol', 'sci,', 'a3', '1043', '(1969)', '200', 'jk', 'stille', 'ld', 'gotter', 'j', 'polymer', 'sci,', 'a-17', '2493', '(1969)', '534', '7', 'step-growth', 'polymerization', 'step-growth', 'polymers', '201', 'jk', 'stille', 'gk', 'noren', 'l', 'green', 'j', 'polymer', 'sci,', 'a-18', '2245', '(1970)', 'jk', 'stille', 'rm', 'harris', 'sm', 'podaki', 'macromolecules', '14', '486', '(1981)', '202', 'ea', 'kraiman', 'us', 'pat', '#', '2890206', '(1959)', '2890207', '(1959)', '3074915', '(1963)', '203', 'v', 'saukaran', 'cs', 'marvel', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '18', '1835', '(1980)', '204', 'k', 'meyersen', 'jyc', 'wang', 'j', 'polymer', 'sci,', 'a-15', '1845', '(1967)', '205', 'fc', 'de', 'schryver', 'wj', 'feast', 'g', 'smets', 'j', 'polymer', 'sci,', 'a-18', '1939', '(1970)', '206', 'h', 'vogel', 'cs', 'marvel', 'j', 'polymer', 'sci,', '50', '511', '(1961)', '207', 'cj', 'abshire', 'cs', 'marvel', 'makromol', 'chem,', '44–46', '388', '(1961)', '208', 'vv', 'korshak', 'es', 'krongauz', 'al', 'rusanov', 'j', 'polymer', 'sci,', 'c16', '2635', '(1967)', 'vv', 'korshak', 'sv', 'vinogradova', 'ys', 'vygodskii', 'rev', 'makromol', 'chem,', '12', '45', '(1974-1975)', '209', 'pm', 'hergenrother', 'w', 'wrasidlo', 'hh', 'levine', 'j', 'polymer', 'sci,', 'a-13', '1665', '(1965)', '210', 't', 'kubota', 'r', 'nakanishi', 'j', 'polymer', 'sci,', 'polymer', 'letters', '2', '655', '(1964)', '211', 'mr', 'lilyquist', 'jr', 'holsten', 'j', 'polymer', 'sci,', 'c19', '77', '(1967)', '212', 'ah', 'frazer', 'wp', 'fitzgerald', 'jr,', 'j', 'polymer', 'sci,', 'c19', '95', '(1967)', '213', 'ss', 'hersch', 'j', 'polymer', 'sci,', 'a-17', '15', '(1969)', '214', 'es', 'krongauz', 'uspekhi', 'khim,', '42', '857', '(1973)', '215', 'rf', 'kovar', 'gfl', 'ehlers', 'fe', 'arnold', 'am', 'chem', 'soc', 'polymer', 'preprints', '16(2)', '246', '(1975)', '216', 'fl', 'heldberg', 'fe', 'arnold', 'am', 'chem', 'soc', 'polymer', 'preprints', '16(1)', '677', '(1975)', '217', 'c', 'arnold', 'jr,', 'j', 'polymer', 'sci,', 'macromol', 'rev,', '14', '265', '(1979)', '218', 'v', 'saukaran', 'sc', 'lin', 'cs', 'marvel', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '18', '495', '(1980)', '219', 'pm', 'hergenrother', 'macromolecules', '14', '891', '898', '(1981)', '220', 'sv', 'vinogradova', 'js', 'visodskii', 'uspekhi', 'khim,', '42', '1225', '(1973)', 'vv', 'korshak', 'sv', 'alekseeva', 'iy', 'slonium', 'makromol', 'chem,', '184', '235', '(1983)', '221', 'gb', 'vaughanjc', 'rose', 'gp', 'brown', 'j', 'polymer', 'sci,', 'a-19', '1117', '(1971)', '222', 'fe', 'arnold', 'rl', 'van', 'deusen', 'j', 'polymer', 'sci,', 'polymer', 'letters', '6', '815', '(1968)', '223', 'rj', 'gaymans', 'ka', 'hodd', 'wa', 'holmes-walker', 'polymer', '12', '400', '(1971)', '224', 'ka', 'andrianov', 'metalorganic', 'polymers', 'wiley-interscience', 'new', 'york', '1965', 'z', 'yuan', 'y', 'xiao', 'y', 'yang', 'arid', 't', 'xiong', 'macromolecules', '2011', '44(7)', '1788', '225', 'jf', 'brown', 'j', 'polymer', 'scic1,', '83', '(1963)', '226', 'hr', 'allcock', 'chem', 'rev,', '72', '315', '(1972)', '227', 'h', 'rosenberg', 'tt', 'tsai', 'nk', 'ngo', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '20', '1', '(1982)', '228', 'h', 'kondo', 'm', 'sato', 'm', 'yokoyama', 'eu', 'polym', 'j,', '18', '679', '(1982)', '229', 'fw', 'harris', 'hj', 'spinelli', 'reactive', 'oligomers', 'acs', 'symp,', 'ser282', 'amer', 'chem', 'soc,', 'washington', 'dc,', '1985', '230', 'ta', 'upshaw', 'jk', 'stille', 'jp', 'droske', 'macromolecules', '24', '2143', '(1991)', '231', 'd', 'kumar', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '23', '1661', '(1985)', '232', 'd', 'kumar', 'gm', 'fohlen', 'j', 'a', 'parker', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '22', '927', '1141', '(1984)', '233', 'jk', 'stille', 'me', 'feeburger', 'j', 'polymer', 'sci,', 'polymer', 'letters', '5', '989', '(1967)', '234', 'k', 'niume', 'f', 'toda', 'k', 'uno', 'y', 'iwakara', 'j', 'polymer', 'sci,', 'polymer', 'letter', '15', '283', '(1977)', '235', 'm', 'ueda', 't', 'kano', 't', 'waragai', 'h', 'sugito', 'makromol', 'chem,', 'rapid', 'commun,', '6', '847', '(1985)', '236', 'dm', 'white', 't', 'takekoshi', 'fj', 'williams', 'hm', 'relees', 'pe', 'donohue', 'hj', 'klopfer', 'gr', 'toncks', 'js', 'manello', 'tomathews', 'rw', 'schlueny', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '19', '1635', '(1981)', '237', 'jl', 'hedrick', 'jw', 'labadie', 'macromolecules', '21', '1883', '(1988)', '238', 'jw', 'connell', 'pm', 'hergenrother', 'am', 'chem', 'soc', 'polymer', 'preprints', '29(1)', '172', '(1988)', '239', 'fw', 'harris', 'je', 'korleski', 'polym', 'matl', 'sci', 'eng', 'proc,', '61', '870', '(1989)', '240', 'jg', 'smith', 'jr,', 'jw', 'connell', 'pm', 'hergenrother', 'am', 'chem', 'soc', 'polymer', 'preprints', '32(3)', '193', '(1991)', '241', 'rg', 'bass', 'kr', 'srinivasan', 'am', 'chem', 'soc', 'polymer', 'preprints', '32(1)', '619', '(1991)', '242', 'm', 'strukelj', 'as', 'hay', 'macromolecules', '24', '6870', '(1991)', '243', 'hm', 'gajiwala', 'r', 'zand', 'macromolecules', '26', '5976', '(1993)', '244', 'ap', 'chaﬁn', 'et', 'al,', 'macromolecules', '30', '1515', '(1997)', '245', 's-d', 'lee', 'f', 'sanda', 't', 'endo', 'j', 'polymer', 'sci', 'polym', 'chemed,', '35', '1333', '(1997)', '246', 'pe', 'doodson', 'tl', 'wallow', 'bm', 'novak', 'macromolecules', '31', '2047', '(1998)', '247', 'y', 'wang', 'rp', 'quirck', 'macromolecules', '28', '3495', '(1995)', 's', 'b', 'jhaveri', 'j', 'j', 'peterson', 'k', 'r', 'carter', 'macromolecules', '2008', '41(23)', '8977', '248', 'da', 'tomalia', 'h', 'baker', 'j', 'dewald', 'm', 'hall', 'g', 'kallos', 's', 'martin', 'j', 'roeck', 'j', 'ryder', 'p', 'smith', 'polymer', 'j', '17', '117', '(1985)', 'ibid,', 'macromolecules', '19', '22466', '(1986)', '249', 'hihre', 'a', 'hult', 'jmj', 'frechet', 'i', 'gitsov', 'macromolecules', '31', '4061', '(1998)', '250', 'k', 'mullen', 'et', 'al,', 'chem', 'commun,', '10', '1139', '(1998)', '251', 'm', 'tollsas', 'b', 'atthoff', 'h', 'claesson', 'jl', 'hedrick', 'macromolecules', '31', '3439', '(1998)', 'references', '535', '252', 'hw', 'gibson', 'ms', 'bheda', 'pt', 'engen', 'prog', 'polym', 'sci,', '19', '843', '(1994)', '253', 'db', 'ambalino', 'jf', 'stoddart', 'chem', 'revs', '95', '2725', '(1995)', '254', 'hw', 'gibson', 's', 'liu', 'cgong', 'q', 'ji', 'e', 'josephs', 'macromolecules', '30', '3711', '(1997)', '255', 'c', 'gong', 'q', 'ji', 'c', 'subramaninam', 'hw', 'gibson', 'macromolecules', '31', '1814', '(1998)', '256', 'a', 'harada', 'a', 'hashidzume', 'h', 'yamaguchi', 'y', 'takashima', 'chemrev,', '2009', '109', '5974', 'chapter', '8', 'naturally', 'occurring', 'polymers', '81', 'naturally', 'occurring', 'polymers', 'there', 'many', 'naturally', 'occurring', 'polymeric', 'materials', 'many', 'quite', 'complex', 'it', 'possible', 'however', 'apply', 'arbitrary', 'classiﬁcation', 'divide', 'six', 'main', 'categories', 'these', '1', 'polysaccharides', 'this', 'category', 'includes', 'starch', 'cellulose', 'chitin', 'pectin', 'alginic', 'acid', 'natural', 'gums', 'others', '2', 'proteins', 'naturally', 'occurring', 'polyamides', 'found', 'animal', 'vegetable', 'sources', '3', 'polyisoprenes', 'natural', 'rubbers', 'similar', 'materials', 'isolated', 'saps', 'plants', '4', 'polynucleotides', 'include', 'deoxyribonucleic', 'acid', '(dnas)', 'ribonucleic', 'acids', '(rnas)', 'found', 'living', 'organisms', '5', 'lignin', 'polymeric', 'materials', 'coniferyl', 'alcohol', 'related', 'substances', '6', 'naturally', 'occurring', 'miscellaneous', 'polymers', 'like', 'instance', 'shellac', 'resin', 'secreted', 'lac', 'insect', 'this', 'complex', 'cross-linked', 'polyester', '91016-trihydroxy-exadecanoic', 'acid', '(aleuritic', 'acid)', 'the', 'structure', 'also', 'includes', 'unsaturated', 'long-chain', 'aliphatic', 'acids', 'together', 'compounds', '1.', '82', 'polysaccharides', 'fischer', '2', 'carried', 'original', 'investigations', 'monomeric', 'species', 'many', 'polysaccharides', 'last', 'century', 'he', 'able', 'demonstrate', 'conﬁgurational', 'relationships', 'within', 'monosaccharides', 'the', 'monomers', 'naturally', 'occurring', 'polymers', 'ﬁve-', 'six-carbon', 'sugars', 'there', 'considerable', 'variety', 'among', 'polysaccharides', 'polymers', 'generally', 'tend', 'polydisperse', 'depending', 'upon', 'source', '821', 'hemicelluloses', 'hydrolyses', 'hemicelluloses', 'yield', 'mixtures', 'glucose', 'glucuronic', 'acid', 'xylose', 'arabinose', 'galactose', 'galacturonic', 'acid', 'mannose', 'rhamnose', 'some', 'common', 'polymers', 'pentoses', 'also', 'known', 'pentosans', 'xylan', 'galactan', 'araban', 'others', 'pentosans', 'found', 'large', 'amounts', '(20–40)', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-98', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '537', '538', '8', 'naturally', 'occurring', 'polymers', 'cereal', 'straws', 'brans', 'large-scale', 'industrial', 'preparations', 'furfural', 'instance', 'based', 'materials', 'xylan', 'one', 'better-known', 'hemicelluloses', 'component', 'plant', 'cell', 'membranes', 'this', 'pentosan', 'occurs', 'association', 'cellulose', 'the', 'structure', 'xylan', 'shown', '14-polyxylose', '3:', 'h', 'o', 'ho', 'o', 'ho', 'o', 'oh', 'h', 'oh', 'o', 'o', 'another', 'hemicellulose', 'galactan', 'minor', 'component', 'coniferous', 'deciduous', 'woods', 'larch', 'wood', 'shown', 'contain', '8', 'polymer', '4.', 'araban', 'polyarabinose', 'found', 'plant', 'saps', 'all', 'pectins', 'also', 'belong', 'family', 'hemicelluloses', 'these', 'gelatinizing', 'substances', 'found', 'many', 'plants', 'particularly', 'fruit', 'juices', 'crude', 'pectins', 'contain', 'pentosans', 'galactosans', 'similar', 'materials', 'puriﬁed', 'pectins', 'yield', 'hydrolysis', 'galacturonic', 'acid', 'methanol', 'these', 'high', 'molecular', 'weight', 'polymers', 'believed', 'consists', 'good', 'extent', 'poly', '(galacturonic', 'acid)', 'partially', 'esteriﬁed', 'methyl', 'alcohol', 'in', 'addition', 'polymers', 'contain', 'galactose', 'arabinose', 'molecules', 'the', 'polymer', 'probably', 'linear', '5–10', '14-glucosidal', 'linkage', 'monomers', 'the', 'relative', 'amount', 'various', 'components', 'depends', 'upon', 'source', 'pectin', 'citrus', 'pectin', 'instance', 'rich', 'galacturonic', 'acid', 'poor', 'galactose', 'arabinose', 'plant', 'gums', 'mucilages', 'high', 'molecular', 'weight', 'polysaccharides', 'composed', 'hexoses', 'pentoses', 'they', 'also', 'contain', 'uronic', 'acid', 'units', 'among', 'gums', 'gum', 'arabic', 'gum', 'tragacanth', 'many', 'others', '822', 'starch', 'this', 'widely', 'distributed', 'substance', 'vegetable', 'kingdom', 'chief', 'reserve', 'carbohydrate', 'plants', 'starch', 'consists', 'single', 'repeat', 'units', 'd-glucose', 'linked', 'together', '1', '4', 'carbons', 'a-linkages', '(cis)', '10.', 'there', 'two', 'types', 'starch', 'molecules', 'amylose', 'amylopectin', 'the', 'ﬁrst', 'one', 'mainly', 'linear', 'polymer', 'its', 'molecular', 'weight', 'range', '30000', '1000000', 'though', 'mostly', '200000–300000', 'amylose', 'often', 'pictured', 'spiral', 'form', 'due', 'conformation', 'a-glucoside', 'bonds', 'ch2oh', 'o', 'o', 'ho', 'oh', 'ho', 'o', 'ho', 'ho', 'o', 'o', 'ho', 'o', 'ch2oh', 'ch2oh', '82', 'polysaccharides', '539', 'amylopectin', 'hand', 'branched', 'carbon', '6', 'ch2oh', 'o', 'ho', 'ch2oh', 'o', 'oh', 'o', 'ho', 'oh', 'o', 'o', 'ho', 'oh', 'o', 'n', 'the', 'ratios', 'amylopectin', 'amylose', 'many', 'natural', 'starches', '31', 'the', 'main', 'commercial', 'source', 'starch', 'country', 'corn', 'lesser', 'amounts', 'industrial', 'starch', 'obtained', 'potatoes', 'wheat', 'tapioca', '(not', 'necessarily', 'order)', 'the', 'extraction', 'starch', 'plant', 'material', 'done', 'grinding', 'plant', 'tissues', 'water', 'the', 'slurry', 'ﬁltered', 'obtain', 'suspension', 'starch', 'granules', 'these', 'granules', 'collected', 'aid', 'centrifuge', 'dried', 'when', 'water', 'suspension', 'starch', 'granules', 'heated', '60–80(cid1)c', 'granules', 'swell', 'rupture', 'this', 'results', 'formation', 'viscous', 'colloidal', 'dispersion', 'containing', 'dissolved', 'starch', 'molecules', 'cooling', 'dispersion', 'results', 'formation', 'gel', 'due', 'aggregation', 'amylose', 'molecules', 'it', 'essentially', 'crystallization', 'phenomenon', 'known', 'retrogration', 'by', 'comparison', 'amylopectin', 'molecules', 'cannot', 'associate', 'readily', 'due', 'branching', 'gel', 'conditions', 'starches', 'modiﬁed', 'chemically', 'various', 'ways', 'some', 'acetate', 'phosphate', 'esters', 'produced', 'commercially', 'well', 'hydroxyalkyl', 'tertiary', 'aminoalkyl', 'ethers', 'both', 'unmodiﬁed', 'modiﬁed', 'starches', 'used', 'principally', 'papermaking', 'paper', 'coating', 'paper', 'adhesives', 'textile', 'sizes', 'food', 'thickeners', 'there', 'many', 'reports', 'literature', 'graft', 'copolymers', 'starch', 'the', 'work', 'often', 'done', 'search', 'biodegradable', 'materials', 'packaging', 'agricultural', 'mulches', 'most', 'chemical', 'modiﬁcations', 'starch', 'parallel', 'cellulose', '823', 'cellulose', 'this', 'polysaccharide', 'found', 'widely', 'nature', 'it', 'major', 'constituent', 'plant', 'tissues', '(50–70,', 'depending', 'upon', 'wood)', 'ﬁbers', 'leaf', 'stalks', 'chemically', 'cellulose', '14-b-poly-anhydroglucose', '12', '(trans)', 'ch2oh', 'ho', 'o', 'o', 'oh', 'n', '540', '8', 'naturally', 'occurring', 'polymers', 'n', 'represents', 'several', 'thousand', 'units', 'hydrolysis', 'cellulose', 'yields', '95–96', 'd-glucose', 'this', 'establishes', 'structure', 'acetolysis', 'cellulose', 'however', 'yields', 'cellobiose', 'disaccharide', '4-o-b-d-glucopyranosyl-d-glucopyranose', 'ho', 'ho', 'oh', 'o', 'o', 'ho', 'ho', 'ch2oh', 'o', 'oh', 'ch2oh', 'the', 'structure', 'cellulose', 'therefore', 'ofﬁcially', 'based', 'cellobiose', 'units', 'careful', 'molecular', 'weight', 'measurements', 'many', '12', 'established', 'dp', 'cellulose', 'ranges', '2000', '6700', 'depending', 'upon', 'source', 'the', 'polymer', 'highly', 'crystalline', 'characterized', 'high', 'degree', 'intermolecular', 'intramolecular', 'hydrogen', 'bonding', 'this', 'prevents', 'thermo-', 'plastic', 'decomposes', 'upon', 'heating', 'without', 'melting', '8231', 'regenerated', 'cellulose', 'cellulose', 'used', 'many', 'forms', 'often', 'modiﬁed', 'chemically', 'render', 'soluble', 'organic', 'solvents', 'in', 'modiﬁcations', 'treated', 'manner', 'allows', 'forming', 'desired', 'shapes', 'like', 'ﬁlms', 'ﬁbers', 'followed', 'restoration', 'chemically', 'insoluble', 'form', 'the', 'material', 'called', 'regenerated', 'cellulose', 'several', 'processes', 'evolved', 'preparation', 'regenerated', 'cellulose', 'one', 'developed', 'far', 'back', '1884', 'converts', 'ﬁrst', 'nitrate', 'ester', 'the', 'nitrated', 'material', 'dissolved', 'mixture', 'ethyl', 'alcohol', 'diethyl', 'ether', 'extruded', 'ﬁbers', 'the', 'ﬁbers', 'denitrated', 'treatment', 'ammonium', 'hydrogen', 'sulﬁde', '40(cid1)c', 'the', 'product', 'called', 'chardonnet', 'silk', 'it', 'appears', 'process', 'longer', 'practiced', 'anywhere', 'in', 'another', 'process', 'cellulose', 'dissolved', 'ammoniacal', 'cupric', 'hydroxide', '(cu(nh3)4(oh)2)', 'the', 'solution', 'spun', 'ﬁber', 'dilute', 'sulfuric', 'acid', 'solution', 'regenerate', 'cellulose', 'the', 'product', 'called', 'cuprammonium', 'rayon', 'the', 'material', 'may', 'still', 'manufactured', 'limited', 'scale', 'the', 'third', 'probably', 'major', 'commercial', 'process', 'used', 'today', 'forms', 'material', 'known', 'viscose', 'rayon', 'the', 'regenerated', 'cellulose', 'prepared', 'sold', 'ﬁber', 'well', 'ﬁlm', 'known', 'cellophane', 'the', 'viscose', 'properly', 'referred', 'xanthate', 'process', 'consists', 'forming', 'cellulose', 'xanthate', 'reacting', 'alkali', 'cellulose', 'carbon', 'disulﬁde', 'oh', 'ho', 'o', 'ch2oh', 'o', 'naoh', 'ho', 'cs2', 'ho', 'oh', 'o', 'ch2ona', 'o', 'oh', 'o', 'o', 's', 's', 'na', 'ch2', 'o', 'in', 'typical', 'procedure', 'cellulose', 'steeped', 'approximately', '20', 'aqueous', 'sodium', 'hydroxide', 'solution', 'room', 'temperature', 'anywhere', '20', 'min', 'whole', 'hour', 'it', 'believed', 'treatment', 'results', 'formation', 'sodium', 'alcoholate', 'every', 'hydroxymethyl', 'group', 'the', 'resultant', 'material', 'pressed', 'remove', 'excess', 'liquid', 'shredded', 'aged', '2–3', 'days', 'the', 'aging', 'known', 'cause', 'molecular', 'weight', 'reduction', 'after', 'aging', 'alkali', 'cellulose', 'treated', 'carbon', 'disulﬁde', '2–4', 'h', 'form', 'cellulose', 'xanthate', 'the', 'amount', 'xanthate', 'groups', 'product', 'average', 'one', 'per', 'every', 'two', 'glucose', 'units', 'the', 'material', 'dissolved', 'dilute', 'sodium', 'hydroxide', 'solution', '82', 'polysaccharides', '541', 'aged', '2–5', 'days', 'during', 'aging', 'period', 'xanthate', 'groups', 'decompose', 'the', 'solution', 'spun', 'dilute', 'sulfuric', 'acid', 'regenerate', 'cellulose', 'form', 'ﬁbers', '10', '-', '15', 'h2so4', '10', '-', '20', 'na2so4', 'n', 'oh', 'ho', 'o', 'o', 'n', 'oh', 'oh', 'ho', 'o', 'o', 'o', 's', 's', 'na', 'the', 'rayon', 'ﬁbers', 'washed', 'bleached', 'submitted', 'various', 'treatments', 'like', 'dyeing', 'etc,', 'depending', 'upon', 'intended', 'use', 'when', 'cellulose', 'xanthate', 'extruded', 'narrow', 'slits', 'acid', 'baths', 'cellophane', 'ﬁlms', 'form', 'these', 'ﬁlms', 'usually', 'plasticized', 'washing', 'baths', 'containing', 'glycerin', '8232', 'derivatives', 'cellulose', 'many', 'derivatives', 'f', 'cellulose', 'synthesized', 'years', '12–14.', 'these', 'include', 'esters', 'organic', 'inorganic', 'acids', 'ethers', 'various', 'graft', 'copolymers', 'only', 'however', 'achieved', 'commercial', 'importance', 'one', 'earliest', 'commercial', 'esters', 'cellulose', 'cellulose', 'nitrate', 'it', 'originally', 'prepared', 'explosive', '(guncotton)', 'middle', 'nineteenth', 'century', 'later', 'medical', 'aid', '(collodion', 'covering', 'wounds)', 'later', 'ﬁlms', 'cellulose', 'nitrate', 'used', 'photography', 'called', 'celluloid', 'nitrocellulose', 'also', 'probably', 'ﬁrst', 'successful', 'commercial', 'plastic', 'used', 'form', 'many', 'articles', 'today', 'generally', 'displaced', 'materials', 'cellulose', 'nitrate', 'however', 'still', 'used', 'surface', 'ﬁnishes', 'though', 'gradually', 'displaced', 'cellulose', 'nitrated', 'mixtures', 'nitric', 'sulfuric', 'acids', 'the', 'type', 'acid', 'mixture', 'used', 'depends', 'intended', 'products', 'for', 'preparation', 'plastic', 'grade', 'materials', '25', 'nitric', 'acid', 'combined', '55', 'sulfuric', 'acid', '20', 'water', 'the', 'dried', 'cellulose', 'soaked', '20–60', 'min', '30–40(cid1)c', 'there', 'little', 'change', 'appearance', 'structure', 'cellulose', 'maintained', 'the', 'bulk', 'acid', 'removed', 'usually', 'spinning', 'centrifuge', 'remaining', 'acid', 'washed', 'copious', 'amounts', 'water', 'the', 'product', 'often', 'bleached', 'sodium', 'hypochlorite', 'washed', 'the', 'degree', 'nitration', 'controlled', 'reaction', 'conditions', 'particularly', 'amount', 'water', 'nitrating', 'bath', 'products', '19–20', 'nitrate', 'groups', 'per', 'glucose', 'unit', 'used', 'plastics', 'lacquers', 'some', 'materials', 'however', 'nitrate', 'content', 'high', '20–24', 'groups', 'per', 'glucose', 'used', 'lacquers', 'the', 'higher', 'nitrate', 'content', '24–28', 'groups', 'per', 'glucose', 'materials', 'intended', 'use', 'explosives', 'the', 'esteriﬁcation', 'reaction', 'illustrated', 'follows', 'oh', 'ho', 'o', 'o', 'n', 'ch2oh', 'hno3h2so4', 'ho', 'ono2', 'o', 'o', 'n', 'ch2ono2', '542', '8', 'naturally', 'occurring', 'polymers', 'the', 'molecular', 'weight', 'cellulose', 'nitrates', 'used', 'plastics', 'lacquers', 'usually', 'reduced', 'this', 'done', 'heating', 'slurry', 'polymer', 'water', '130–160(cid1)c', '30', 'min', 'pressure', 'cellulose', 'acetate', 'also', 'prepared', 'originally', 'nineteenth', 'century', 'commercial', 'develop-', 'ment', 'however', 'started', 'early', 'twentieth', 'century', 'in', '1920s', 'acetate', 'rayon', 'acetate', 'ﬁbers', 'developed', 'cellulose', 'acetate', 'became', 'important', 'molding', 'material', 'at', 'time', 'cellulose', 'lacquers', 'also', 'developed', 'today', 'however', 'many', 'materials', 'replaced', 'polymers', 'the', 'acetylation', 'reaction', 'cellulose', 'often', 'prepared', 'forming', 'solution', 'mixture', 'acetic', 'anhydride', 'sulfuric', 'acid', 'this', 'results', 'formation', 'triacetate', 'when', 'lower', 'degree', 'esteriﬁcation', 'desired', 'triacetate', 'partially', 'hydrolyzed', 'a', 'two-step', 'procedure', 'needed', 'possible', 'control', 'degree', 'esteriﬁcation', 'reaction', 'acetic', 'anhydride', 'sulfuric', 'acid', 'in', 'typical', 'process', 'dry', 'cellulose', 'pretreated', '300', 'parts', 'acetic', 'anhydride', '1', 'part', 'sulfuric', 'acid', '400', 'parts', 'methylene', 'chloride', 'the', 'reaction', 'mixture', 'agitated', 'tempera-', 'ture', 'maintained', '25–35(cid1)c', '5–8', 'h', 'by', 'end', 'period', 'cellulose', 'dissolved', 'cellulose', 'triacetate', 'formed', 'solution', 'partial', 'hydrolysis', 'accomplished', 'adding', 'methylene', 'chloride', 'solution', 'aqueous', 'acetic', 'acid', '(50)', 'the', 'solution', 'allowed', 'stand', 'reach', 'desired', 'degree', 'hydrolysis', 'this', 'usually', 'takes', '72', 'h', 'room', 'temperature', 'sulfuric', 'acid', 'still', 'present', 'acetylation', 'neutralized', 'addition', 'sodium', 'acetate', 'methylene', 'chloride', 'distilled', 'the', 'partially', 'hydrolyzed', 'cellulose', 'acetate', 'precipitated', 'addition', 'water', 'washed', 'thoroughly', 'the', 'washing', 'also', 'includes', '2-h', 'wash', 'dilute', 'sulfuric', 'acid', 'remove', 'hydrogen', 'sulfate', 'esters', 'cause', 'polymer', 'instability', 'the', 'process', 'illustrated', 'follows', 'ho', 'oh', 'o', 'o', 'ch2oh', '(ch3co2)o', 'n', 'h2so4', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'o', 'o', 'n', 'h', 'o', 'o', 'o', 'o', 'h2o', 'h', 'cellulose', 'triacetate', 'also', 'prepared', 'heterogeneous', 'process', 'presence', 'benzene', 'non-solvent', 'the', 'triacetate', 'forms', 'processes', 'hard', 'mold', 'used', 'ﬁlms', 'ﬁbers', 'the', 'diacetate', 'suited', 'plasticization', 'molding', '82', 'polysaccharides', '543', 'many', 'esters', 'cellulose', 'prepared', 'various', 'times', 'including', 'mixed', 'esters', 'various', 'cellulose', 'acetate-butyrates', 'manufactured', 'today', 'perhaps', 'best', 'known', 'mixed', 'esters', 'they', 'synthesized', 'manner', 'cellulose', 'acetate', 'mixed', 'anhydrides', 'used', 'esteriﬁcation', 'reactions', 'catalyzed', 'sulfuric', 'acid', 'the', 'products', 'slightly', 'hydrolyzed', 'the', 'butyric', 'groups', 'enhance', 'ﬂexibility', 'moisture', 'resistance', 'the', 'materials', 'reputation', 'tough', 'plastics', 'used', 'applications', 'tool', 'handles', 'lower', 'molecular', 'weight', 'grades', 'also', 'used', 'surface', 'ﬁnishes', 'several', 'cellulose', 'ethers', 'also', 'prepared', 'commercially', 'the', 'original', 'patents', 'preparation', 'cellulose', 'ethers', 'date', '1912', 'in', 'spite', 'cellulose', 'ethers', 'never', 'attained', 'industrial', 'importance', 'cellulose', 'esters', 'the', 'ethers', 'prepared', 'reacting', 'alkali', 'cellulose', 'alkyl', 'halide', 'epoxide', 'ho', 'naoh', 'ho', 'oh', 'o', 'o', 'oh', 'n', 'oh', 'o', 'o', 'c2h5cl', 'n', 'ho', 'oh', 'na', 'o', 'o', 'n', 'oc2h5', 'ho', 'oh', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'oh', 'oh', 'ho', 'o', 'ho', 'o', 'o', 'n', 'ona', 'o', 'ch2oh', 'typical', 'commercial', 'ethers', 'methyl', 'ethyl', 'hydroxyethyl', 'hydroxypropyl', 'carboxy-methyl', 'aminoethyl', 'benzyl', 'ethyl', 'cellulose', 'used', 'industrially', 'plastic', 'similarly', 'cellulose', 'acetate', 'the', 'water-soluble', 'ethers', 'like', 'methyl', 'carboxymethyl', 'hydroxyethyl', 'used', 'thickeners', 'foods', 'paper', 'manufacturing', 'cellulose', 'reacted', 'acrylonitrile', 'form', 'cyanoethylether', 'the', 'michael', 'condensation', 'takes', 'place', 'alkali', 'cellulose', 'ona', 'ho', 'o', 'o', 'n', 'ona', 'n', 'o', 'ho', 'n', 'o', 'o', 'n', 'o', 'n', '544', '8', 'naturally', 'occurring', 'polymers', 'cyanoethylated', 'cellulose', 'appear', 'used', 'commercially', 'quantity', 'very', 'stable', 'silyl', 'ethers', 'form', 'cellulose', 'treated', 'trimethyl', 'chloro-silane', 'bis', '(trimethylsilyl)-acetamide', '15:', 'oh', 'ho', 'o', 'o', 'ch2oh', 'n', 'si', 'cl', 'ho', 'si(ch3)3', 'n', 'si(ch3)3', 'o', 'o', 'o', 'si(ch3)3', 'o', 'n', 'o', 'si(ch3)3', 'some', 'interesting', 'approaches', 'cellulose', 'modiﬁcation', 'possible', 'via', 'formations', 'double', 'bonds', 'glucopyranosine', 'unit', '56', 'positions', '16.', 'this', 'accomplished', 'dehydro-halogenating', 'previously', 'formed', '6-iodocellulose', 'oh', 'ho', 'o', 'o', 'n', 'i', 'oh', 'ho', 'o', 'o', 'n', 'the', 'resultant', 'unsaturated', 'compound', 'converted', 'number', 'derivatives', 'examples', 'ho', 'oh', 'o', 'o', 'hccl3', 'ho', 'ho', 'oh', 'o', 'o', 'hsicl3', 'ho', 'n', 'n', 'oh', 'ccl3', 'o', 'o', 'oh', 'o', 'o', 'sicl3', 'n', 'n', 'other', 'compounds', 'added', 'across', 'double', 'bonds', 'carbon', 'tetrachloride', 'phosphorus', 'trichloride', 'methyl', 'alcohol', 'many', 'graft', 'copolymers', 'cellulose', 'reported', 'some', 'described', 'chap', '9', 'in', 'wood', 'cellulose', 'present', 'lignin', 'natural', 'phenolic', 'polymer', 'described', 'sect', '83', 'kobayshi', 'coworkers', '16', 'grafted', 'phenolic', 'resins', 'onto', 'cellulose', 'form', 'artiﬁcial', 'wood', 'polymer', '82', 'polysaccharides', 'ho', 'o', 'o', 'o', 'ho', 'o', 'n', 'oh', 'oh', 'r', 'fe-salen', 'h2o2', 'pyridine', 'hybrid', 'polymer', '545', 'o', 'r', 'n', 'the', 'reaction', 'catalyzed', 'complex', 'iron', 'nn0-ethylene', 'bis(sallicylidene', 'amine)', 'well', 'horseradish', 'peroxidase', 'enzyme', 'carry', 'oxidative', 'coupling', 'phenols', 'using', 'hydrogen', 'peroxide', 'oxidizing', 'agent', 'the', 'product', 'new', 'type', 'plastic', '824', 'miscellaneous', 'polysaccharides', 'other', 'polysaccharides', 'found', 'nature', 'include', 'alginic', 'acid', 'isolated', 'certain', 'brown', 'seaweeds', '17.', 'the', 'monomers', 'polymer', 'similar', 'cellulose', 'linked', 'trans', 'b', '14', 'positions', 'a', 'sulfate', 'group', 'bearing', 'polysaccharide', 'isolated', 'another', 'seaweed', 'red', 'color', 'this', 'polymer', 'called', 'carrageenan', 'it', 'consists', 'two', 'fractions', '17.', 'the', 'ﬁrst', 'one', 'galactose', 'units', 'linked', 'though', '1', '3', '1', '4', 'carbons', 'a', 'sulfate', 'group', 'found', 'carbon', '2', 'units', 'oh', 'o', 'o', 'ho', 'o', 'oh', 'so3', 'o', 'o', 'o', 'n', 'ho', 'ch2', 'oh', '546', '8', 'naturally', 'occurring', 'polymers', 'the', 'second', 'fraction', 'galactose', 'units', 'linked', '1', '4', 'others', 'ether', 'group', 'linking', 'carbons', '3', '6', 'ch2oh', 'o', 'oh', 'n', 'ho', 'n', 'o', 'o', 'oh', 'a', 'similar', 'polysaccharide', 'also', 'obtained', 'seaweeds', 'called', 'agar', 'it', 'similar', 'structure', 'less', 'sulfate', 'groups', 'per', 'chain', 'crab', 'shrimp', 'shell', 'wastes', 'abundant', 'source', 'chitin', 'nitrogen', 'atom-containing', 'polysaccharide', 'ch2oh', 'o', 'n', 'h', 'n', 'o', 'the', 'polymer', 'deacetylated', 'yield', 'amine', 'group', 'bearing', 'polysaccharide', '83', 'lignin', 'these', 'polymers', 'also', 'constituents', 'wood', '(about', '25–30)', '18', '19.', 'it', 'uncertain', 'molecular', 'weights', 'polymers', 'materials', 'quite', 'complex', 'structure', 'the', 'extraction', 'processes', 'lignin', 'result', 'considerable', 'loss', 'molecular', 'weights', 'the', 'structures', 'lignin', 'vary', 'depending', 'upon', 'source', 'generally', 'considered', 'polymers', 'coniferyl', 'alcohol', 'ho', 'oh', 'o', 'an', 'idealized', 'picture', 'lignin', 'formed', 'coniferyl', 'alcohol', 'published', 'freudenberg', '20.', 'it', 'appears', 'reasonable', 'however', 'lignins', 'might', 'perhaps', 'also', 'form', 'different', 'trees', 'compounds', 'coniferyl', 'alcohol', 'related', 'also', 'found', 'woods', 'also', 'appears', 'plausible', 'several', 'compounds', 'including', 'coniferyl', 'alcohol', 'actually', 'participate', 'together', 'lignin', 'formation', 'depending', 'upon', 'species', 'wood', 'figure', '81', 'shows', 'chemical', 'structures', 'compounds', 'many', 'attempts', 'made', 'convert', 'lignin', 'useful', 'material', 'coatings', 'adhesives', 'only', 'limited', 'success', 'however', 'achieved', 'a', 'reaction', 'product', 'formaldehyde', 'used', 'wood', 'adhesive', 'in', 'addition', 'lignin', 'obtained', 'wood', 'pulping', 'sulfate', 'process', '(as', 'sulfonate)', 'utilized', 'limited', 'extent', 'asphalt', 'extender', 'oil-well', 'drilling', 'mud', 'additive', '85', 'proteins', 'fig', '81', 'compounds', 'occur', 'various', 'woods', 'might', 'perhaps', 'participate', 'formation', 'lignin', 'o', 'oh', 'oh', 'o', 'o', 'o', '547', 'o', 'o', 'charidol', 'coniferyl', 'alcohol', 'safrole', 'isosafrole', 'oh', '84', 'polyisoprene', 'natural', 'rubber', 'polyisoprene', '21.', 'it', 'produced', 'commercially', 'sap', 'trees', 'called', 'hevea', 'brasilensis', 'sometimes', 'referred', 'hevea', 'rubber', 'these', 'trees', 'yield', 'latex', 'containing', 'approximately', '35', 'rubber', 'hydrocarbon', '5', 'nonrubber', 'solids', 'like', 'proteins', 'lipids', 'inorganic', 'salts', 'the', 'remaining', '60', 'latex', 'water', 'the', 'hydrocarbon', 'polymer', 'consists', '97', 'cis-14', 'units', '1', 'trans-14', 'units', '2', '34', 'units', 'head', 'tail', 'structure', 'molecular', 'weights', 'naturally', 'occurring', 'polyisoprene', 'range', '200000', '500000', 'a', 'verity', 'shrubs', 'small', 'plants', 'including', 'weeds', 'like', 'dandelion', 'milkweed', 'also', 'contain', 'polyisoprene', 'sap', 'the', 'guayule', 'shrub', 'grows', 'mexico', 'southern', 'united', 'states', 'good', 'potential', 'source', 'natural', 'rubber', 'work', 'going', 'various', 'places', 'cultivate', 'shrub', 'potential', 'rubber', 'production', 'an', 'almost', 'trans-14', 'polymer', 'called', 'gutta-percha', 'found', 'exudations', 'various', 'trees', 'genus', 'palaquium', 'sapotaceae', 'habit', 'the', 'molecular', 'weights', 'polymers', 'range', '42000', '100000', 'balata', 'chicle', 'also', 'mainly', 'trans-14-polyisoprenes', 'found', 'saps', 'plants', 'west', 'indies', 'mexico', 'south', 'america', 'chapter', '9', 'deals', 'various', 'reactions', 'polymers', 'including', 'natural', 'synthetic', 'rubber', 'that', 'includes', 'vulcanization', 'rubber', 'while', 'many', 'commercial', 'applications', 'cis', 'isomer', 'gutta-percha', 'utilization', 'limited', 'wire', 'coatings', 'impregnation', 'textile', 'belting', 'component', 'varnishes', 'its', 'use', 'limited', 'considerably', 'harder', 'natural', 'rubber', '85', 'proteins', 'these', 'materials', 'building', 'blocks', 'animal', 'tissues', '22', '26', '28', '31.', 'to', 'lesser', 'extent', 'also', 'found', 'vegetable', 'sources', 'because', 'major', 'constituents', 'animal', 'bodies', 'including', 'skins', 'hairs', 'blood', 'proteins', 'much', 'greater', 'interest', 'biochemists', 'nevertheless', 'proteins', 'important', 'commercial', 'materials', 'these', 'include', 'animal', 'glues', 'silk', 'wool', 'it', 'beyond', 'scope', 'book', 'however', 'render', 'thorough', 'discussion', 'proteins', 'for', 'reason', 'basic', 'principles', 'presented', 'proteins', 'naturally', 'occurring', 'polyamides', 'polymer', 'a-amino', 'acids', 'the', 'structure', 'illustrated', 'follows', 'r', 'n', 'h', 'n', 'o', '548', '8', 'naturally', 'occurring', 'polymers', 'because', 'r', 'represents', 'many', 'different', 'groups', 'many', 'different', 'combinations', 'a-amino', 'acids', 'possible', 'proteins', 'complex', 'molecules', 'the', 'arrangement', 'sequence', 'amino', 'acids', 'proteins', 'referred', 'primary', 'structure', 'the', 'amide', 'linkage', 'referred', 'biochemistry', 'peptide', 'linkage', 'peptide', 'group', 'a', 'dipeptide', 'compound', 'consisting', 'two', 'amino', 'acids', 'tripeptide', 'three', 'etc', 'polypeptide', 'refers', 'proteins', 'though', 'term', 'often', 'reserved', 'lower', 'molecular', 'weight', 'fractions', 'usually', 'less', '10000', 'many', 'proteins', 'monodisperse', 'this', 'distinguishes', 'many', 'naturally', 'occurring', 'polymers', 'polysaccharides', 'polydisperse', '851', 'a-amino', 'acids', 'twenty-ﬁve', 'known', 'naturally', 'occurring', 'amino', 'acids', 'isolated', 'various', 'proteins', 'hydrolysis', 'all', 'one', 'glycine', 'possess', 'asymmetric', 'carbon', 'table', '81', 'lists', 'naturally', 'occurring', 'amino', 'acids', 'gives', 'structures', '26', '28', '31.', 'among', 'shown', 'amino', 'acids', 'certain', 'number', 'known', 'essential', 'amino', 'acids', 'they', 'synthesized', 'human', 'bodies', 'must', 'ingested', 'human', 'metabolism', 'all', 'optically', 'active', 'amino', 'acids', '(that', 'means', 'except', 'glycine)', 'l', 'conﬁguration', 'in', 'addition', 'amino', 'acids', 'exist', 'zwitterions', '852', 'structures', 'chemistry', 'proteins', 'proteins', 'separated', 'two', 'major', 'groups', 'ﬁbrous', 'proteins', 'globular', 'proteins', 'depending', 'upon', 'shapes', 'the', 'ﬁbrous', 'proteins', 'long', 'molecules', 'function', 'structural', 'materials', 'animal', 'tissues', 'hydrogen', 'bonding', 'holds', 'water', 'insoluble', 'molecules', 'together', 'form', 'extended', 'coiled', 'chains', 'to', 'group', 'belong', 'collagen', 'protein', 'connecting', 'tissues', 'myosin', 'protein', 'muscles', 'keratin', 'protein', 'found', 'hair', 'nails', 'horns', 'feathers', 'ﬁbroin', 'protein', 'silk', 'ﬁbers', 'globular', 'proteins', 'held', 'strong', 'intramolecular', 'hydrogen', 'bonds', 'spherical', 'elliptical', 'forms', 'their', 'intermolecular', 'forces', 'weak', 'soluble', 'water', 'dilute', 'salt', 'solutions', 'to', 'group', 'proteins', 'belong', 'enzymes', 'many', 'hormones', 'egg', 'albumin', 'hemoglobin', 'some', 'proteins', 'also', 'contain', 'non-peptide', 'portion', 'attached', 'chemically', 'polyamide', 'chain', 'the', 'non-peptide', 'moieties', 'called', 'prosthetic', 'groups', 'proteins', 'groups', 'called', 'conjugated', 'proteins', 'examples', 'hemoglobin', 'myoglobin', 'consist', 'polypeptide', 'portions', 'iron–porphyrin', 'prosthetic', 'groups', 'attached', 'this', 'particular', 'prosthetic', 'group', 'called', 'heme', 'illustrated', 'fig', '82', 'there', 'also', 'number', 'proteins', 'associated', 'nucleic', 'acid', 'they', 'known', 'nucleoproteins', 'numerous', 'studies', 'protein', 'structures', 'shown', 'common', 'conformations', 'protein', 'chains', '(ﬁbrous)', 'either', 'a-helix', 'b-sheets', 'random', 'coils', '26.', 'the', 'steric', 'arrangement', 'conformations', 'proteins', 'referred', 'secondary', 'structure', 'composition', 'a-', 'amino', 'acids', 'polypeptide', 'chains', 'called', 'primary', 'structure', '26.', 'based', 'x-ray', 'crystallogra-', 'phy', 'data', 'pauling', 'et', 'al', '24', 'deduced', 'a-helix', 'type', 'conﬁguration', 'formed', 'accommodates', 'hydrogen', 'bonding', 'nitrogen', 'carbonyl', 'oxygen', '(see', 'fig', '83)', 'it', 'allows', 'space', 'bulky', 'substituents', 'amino', 'acids', 'stabilizes', 'structure', 'the', 'a-helix', 'probably', 'important', 'secondary', 'structure', 'proteins', '26.', 'the', 'two', 'a-helix', 'illustrations', 'pauling', 'et', 'al', '24.', 'the', 'one', 'left', 'shows', 'right-handed', 'helix', 'it', 'interesting', 'note', 'a-helix', 'conformation', 'may', '85', 'proteins', 'table', '81', 'naturally', 'occurring', 'amino', 'acids', '26', '28–31', 'name', 'glycine', 'structure', '549', 'optical', 'rotation', '()', 'alanine', 'valine', 'leucine', 'isoleucine', 'serine', 'threonine', 'cysteine', 'methionine', 'cystine', 'h2n', 'cooh', 'cooh', 'nh2', 'cooh', 'nh2', 'cooh', 'nh2', 'cooh', 'nh2', 'cooh', 'ho', 'nh2', 'nh2', 'oh', 'cooh', 'cooh', 'nh2', 'nh2', 'hs', 's', 'nh2', 'cooh', 'nh2', 'cooh', 'nh2', '()', '()', '((cid3))', '()', '((cid3))', '((cid3))', '((cid3))', '((cid3))', '((cid3))', '((cid3))', '(continued)', 'hooc', 's', 's', 'cooh', 'amino', 'acids', 'aromatic', 'groups', 'phenylalanine', '550', '8', 'naturally', 'occurring', 'polymers', 'table', '81', '(continued)', 'name', 'tyrosine', 'structure', 'optical', 'rotation', '((cid3))', 'ho', 'ho', 'i', 'i', 'cooh', 'cooh', 'nh2', 'nh2', 'ho', 'i', 'i', 'i', 'i', 'cooh', 'nh2', 'amino', 'acids', 'heterocyclic', 'structures', 'proline', 'diiodotyrosine', 'thyroxine', 'hydroxyproline', 'acidic', 'amino', 'acids', 'aspartic', 'acid', 'asparagine', 'glutamic', 'acid', 'glutamine', 'h', 'n', 'cooh', 'h', 'n', 'cooh', 'cooh', 'nh2', 'o', 'nh2', 'cooh', 'cooh', 'nh2', 'nh2', 'cooh', 'ho', 'hooc', 'h2n', 'hooc', 'o', 'h2n', '()', '()', '((cid3))', '((cid3))', '()', '((cid3))', '()', '(continued)', '85', 'proteins', 'table', '81', '(continued)', 'name', 'basic', 'amino', 'acids', 'lysine', 'hydroxylysine', 'arginine', 'tryptophane', 'histidine', 'fig', '82', 'prostate', 'group', 'heme', 'structure', 'optical', 'rotation', '551', 'nh2', 'h2n', 'cooh', '4', 'oh', 'nh2', 'h2n', 'hn', 'nh2', 'h', 'n', 'n', 'h', '2', 'h', 'n', '3', 'cooh', 'cooh', 'cooh', 'nh2', 'cooh', 'n', 'nh2', 'n', 'h', '()', '((cid3))', '()', '((cid3))', '((cid3))', 'ch2', 'ch', 'ch3', 'ch3', 'ch3', 'n', 'n', 'fe', 'n', 'n', 'ch', 'ch2', 'ch3', 'ho2cch2ch2', 'ch2ch2co2h', 'also', 'occur', 'water', 'solutions', 'this', 'due', 'van', 'der', 'waal', 'interactions', '25,', 'water', 'molecules', 'interfere', 'hydrogen', 'bonding', 'holds', 'helix', 'together', 'shown', 'fig', '83', 'not', 'proteins', 'however', 'form', 'helical', 'structures', 'if', 'substituent', 'groups', 'amino', 'acids', 'small', 'found', 'silk', 'ﬁbroin', 'polypeptide', 'chains', 'line', 'side', 'side', 'form', 'sheet-like', 'arrangements', 'the', 'chains', 'tend', 'contract', 'accommodate', 'hydrogen', 'bonding', 'form', 'pleated', 'sheets', 'this', 'called', 'b-arrangement', 'such', 'arrangement', 'parallel', 'antiparallel', 'the', 'identity', 'period', 'parallel', 'one', '65', 'a˚', 'anti-parallel', '70', 'a˚', 'the', 'secondary', 'structures', 'proteins', 'describe', 'completely', 'arrangement', 'macromolecules', 'there', 'may', 'instance', 'sections', 'may', 'exhibit', 'irregularity', 'or', 'sections', 'may', 'linked', 'chemically', 'sulfur–sulfur', 'bonds', 'cystine', 'groups', 'there', 'may', 'also', 'areas', '552', 'fig', '83', 'a-helix', 'structure', 'proteins', '8', 'naturally', 'occurring', 'polymers', 'folding', 'helix', 'allows', 'hydrogen', 'bonding', 'distant', 'sites', 'the', 'overall', 'three-dimensional', 'picture', 'protein', 'structure', 'referred', 'tertiary', 'structure', 'disruption', 'tertiary', 'structure', 'proteins', 'called', 'denaturation', 'when', 'protein', 'composed', 'single', 'peptide', 'chain', 'arrangement', 'called', 'quaternary', 'structure', 'this', 'associa-', 'tion', 'results', 'non-covalent', 'interactions', 'there', 'relationship', 'primary', 'structures', 'amino', 'acid', 'content', 'many', 'proteins', 'secondary', 'structures', '27.', 'the', 'helical', 'contents', 'inversely', 'proportional', 'amount', 'serine', 'threonine', 'valine', 'cysteine', 'proline', 'molecule', 'conversely', '28,', 'valine', 'isoleucine', 'serine', 'cysteine', 'threonine', 'non-helix-forming', 'amino', 'acids', 'proline', 'due', 'speciﬁc', 'conﬁguration', 'actually', 'disrupts', 'helical', 'structure', 'present', 'polypeptide', '29.', 'in', 'addition', 'proteins', 'composed', 'low', 'ratios', 'polar', 'nonpolar', 'amino', 'acids', 'tendency', 'aggregate', '30.', 'also', 'globular', 'protein', 'aqueous', 'environment', 'tend', 'form', 'shapes', 'nonpolar', 'groups', 'located', 'inside', 'structure', 'this', 'due', 'thermodynamic', 'nature', 'hydrophobic', 'side', 'chains', 'the', 'polar', 'ones', 'hand', 'tend', 'located', 'outside', 'toward', 'water', '31.', 'to', 'date', 'much', 'information', 'available', 'proteins', 'others', 'some', 'thoroughly', 'explored', 'proteins', 'mentioned', 'keratins', 'proteins', 'found', 'wool', 'hair', 'fur', 'skin', 'nails', 'horns', 'scales', 'feathers', 'etc', 'they', 'insoluble', 'peptide', 'chains', 'linked', 'disulﬁde', 'bonds', '32', '33.', 'many', 'keratins', 'contain', 'coils', 'a-helixes', '34–36.', 'some', 'keratins', 'however', 'found', 'consist', 'complicated', 'b-helical', 'structures', 'this', 'apparently', 'fully', 'explained', 'wool', 'keratin', 'shown', 'range', 'molecular', 'weight', '8000', '80000', '37.', 'the', 'extensibility', 'a-keratins', 'believed', 'due', 'helical', 'structures', 'the', 'extent', 'keratin', 'hardness', '(in', 'claws', 'horns', 'nails)', 'believed', 'due', 'amount', 'sulfur', 'links', 'silk', 'ﬁbers', 'obtained', 'secretion', 'silkworm', 'double', 'ﬁlaments', 'enclosed', 'coating', 'gum', '(sericin)', 'secreted', '40.', 'the', 'amino', 'acid', 'sequence', 'silk', 'protein', 'shown', '(glycine–serine–glycine–alanine–glycine–alanine)n', 'the', 'polypeptide', 'chains', 'bound', 'antiparallel', 'pleated', 'b-sheet', 'structures', 'hydrogen', 'bonding', '31', '39', '42.', 'the', 'structures', 'also', 'held', 'together', 'van', 'der', 'waal', 'forces', '31', '38.', '85', 'proteins', '553', 'the', 'protein', 'skin', 'extracellular', 'connective', 'tissues', 'animals', 'collagen', 'the', 'polymer', 'rigid', 'cross-linked', 'mild', 'hydrolysis', 'disrupts', 'rigid', 'secondary', 'valence', 'forces', 'produces', 'gelatin', '26.', 'the', 'fundamental', 'unit', 'collagen', 'exists', 'triple', 'helix', '41.', 'three', 'left-handed', 'helices', 'twist', 'together', 'form', 'right-handed', 'threefold', 'super', 'helix', '31.', 'collagen', 'composed', 'mainly', 'glycine', 'proline', 'hydroxyproline', 'some', 'amino', 'acids', 'also', 'present', 'minor', 'amounts', 'a', 'protein', 'similar', 'collagen', 'elastin', 'present', 'elastic', 'tissues', 'tendons', 'arteries', 'hydrolyses', 'elastin', 'rubber-like', 'properties', 'however', 'yield', 'gelatin', 'mildly', 'hydrolyzed', 'elastin', 'fractionated', 'two', 'proteins', '26.', 'among', 'studied', 'globular', 'proteins', 'myoglobin', 'hemoglobin', 'myoglobin', 'consists', 'single', 'chain', '153', 'amino', 'acid', 'residues', 'prosthetic', 'group', 'contains', 'iron', 'called', 'heme', 'myoglobin', 'polypeptides', 'eight', 'helical', 'segments', 'consist', 'right-handed', 'a-helices', 'interrupted', 'corners', 'non-helical', 'regions', 'the', 'overall', 'shape', 'resembles', 'pocket', 'heme', 'group', 'ﬁts', 'the', 'pocket', 'hydrophobic', 'two', 'side', 'groups', 'nonpolar', 'the', 'heme', 'group’s', 'two', 'carboxylic', 'acids', 'protrude', 'surface', 'contact', 'water', '43.', 'the', 'hemoglobin', 'similar', 'myoglobin', 'complex', '44.', 'there', 'four', 'heme', 'groups', 'enclosed', 'hemoglobin', 'structure', 'detailed', 'conformational', 'analysis', 'shown', 'hemoglobin', 'build', '2', '(cid4)', '2', 'myoglobin-like', 'subunits', 'a2', 'b2', '45', '46.', 'casein', 'present', 'several', 'animal', 'vegetable', 'sources', 'commercially', 'however', 'casein', 'primarily', 'obtained', 'milk', 'contains', '3', 'protein', 'the', 'polymer', 'isolated', 'either', 'acid', 'coagulation', 'help', 'enzymes', 'obtained', 'animal', 'stomachs', 'it', 'heteroge-', 'neous', 'the', 'molecular', 'weight', 'large', 'portion', 'bovine', 'casein', '75000', '100000', 'it', 'consists', 'two', 'components', 'b', 'casein', 'belongs', 'groups', 'proteins', 'identiﬁed', 'phosphoproteins', 'hydroxyl', 'residues', 'hydroxy', 'amino', 'acids', 'esteriﬁed', 'phosphoric', 'acid', 'one', 'group', 'proteins', 'far', 'fully', 'identiﬁed', 'glycoprotein', 'this', 'group', 'proteins', 'contains', 'prosthetic', 'group', 'either', 'carbohydrate', 'derivative', 'carbohydrate', 'glycoproteins', 'found', 'mucous', 'secretions', 'very', 'special', 'proteins', 'called', 'enzymes', 'these', 'biological', 'catalysts', 'their', 'primary', 'function', 'increase', 'rate', 'reactions', 'organisms', 'found', 'living', 'systems', 'many', 'enzymes', 'like', 'pepsin', 'trypsin', 'relatively', 'simple', 'proteins', 'others', 'conjugated', 'proteins', 'containing', 'prosthetic', 'groups', 'often', 'known', 'coenzymes', 'because', 'extreme', 'importance', 'biochemists', 'enzymes', 'actions', 'investigated', 'extensively', 'the', 'full', 'structures', 'several', 'enzymes', 'determined', 'one', 'enzyme', 'lysozyme', 'lysozyme', 'enzymes', 'occur', 'many', 'species', 'plants', 'animals', 'chemical', 'behavior', 'may', 'differ', 'the', 'enzyme', 'found', 'egg', 'white', 'peptide', 'chain', 'consisting', 'two', 'sections', 'approximately', 'equal', 'size', 'the', 'two', 'sections', 'separated', 'deep', 'cleft', 'this', 'enzyme', 'performs', 'function', 'binding', 'substrates', 'within', 'cleft', 'hydrogen', 'bonds', 'the', 'substrate', 'hydrolyzed', 'aid', 'glutamine', '(35th', 'amino', 'acid)', 'aspertine', '(52nd', 'amino', 'acid)', 'egg', 'lysozymes', 'primary', 'structure', 'contains', '129', 'amino', 'acid', 'residues', 'the', 'polymer', 'single', 'polypeptide', 'chain', 'cross-', 'linked', 'four', 'places', 'disulﬁde', 'bonds', '47.', 'in', 'addition', 'demonstrated', '72', 'secondary', 'structure', 'enzymic', 'protein', 'essential', 'protein’s', 'catalytic', 'activity', 'also', 'shown', 'structure', 'remains', 'intact', 'neat', 'organic', 'solvents', '72.', 'the', 'molecules', 'however', 'denatured', 'water–organic', 'solvent', 'mixtures', 'the', 'a-helix', 'lysozyme', 'instance', 'enzyme', 'crystalline', 'dissolved', 'neat', 'acetonitrile', '35', 'a-helix', 'pure', 'water', 'value', '23.', 'in', '6030', 'mixture', 'volume', 'acetonitrile', 'water', 'reduced', '13', '72.', 'some', 'uncertainty', 'transition', 'state', 'reaction', 'enzymes', 'like', 'b-phosphoglucomutase-catalyzed', 'transfer', 'phosphoryl', 'group', 'substrate', 'sugar', 'metabolism', 'resolved', 'recently', 'allan', 'dunaway', 'demonstrated', 'means', '19f', 'nuclear', 'magnetic', 'resonance', 'transition', 'state', 'involved', 'bipyramidal', 'oxyphosphorane', 'intermediate', '72.', '554', '8', 'naturally', 'occurring', 'polymers', '853', 'synthetic', 'methods', 'preparation', 'polypeptides', 'studies', 'protein', 'structures', 'functions', 'nature', 'mode', 'actions', 'case', 'enzymes', 'part', 'various', 'investigations', 'much', 'effort', 'also', 'put', 'syntheses', 'different', 'polypeptide', 'such', 'polymers', 'actually', 'formed', 'mixture', 'various', 'amino', 'acids', 'simply', 'heating', 'together', 'the', 'products', 'however', 'complex', 'polymeric', 'materials', 'random', 'distribution', 'amino', 'acids', 'resemble', 'naturally', 'occurring', 'materials', 'base', 'catalyzed', 'ring', 'opening', 'polymerization', 'reactions', 'n-carboxy-a-amino', 'acid', 'anhydrides', 'also', 'result', 'formations', 'polypeptides', 'n', 'h', 'r', 'r', 'o', 'base', 'o', 'o', 'o', 'h', 'n', 'r', 'r', 'co2', '(for', 'mechanism', 'reaction', 'see', 'chap', '5)', '48.', 'over', 'years', 'many', 'polypeptides', 'synthesized', 'reaction', 'these', 'however', 'homopolymers', 'individual', 'amino', 'acids', 'copolymerization', 'leads', 'block', 'copolymers', 'ability', 'form', 'random', 'copolymers', 'con-', 'trolled', 'sequences', 'amino', 'acids', 'would', 'match', 'naturally', 'occurring', 'proteins', 'appears', 'beyond', 'reach', 'reaction', '49', '50.', 'duplication', 'naturally', 'occurring', 'polypeptides', 'needed', 'however', 'understand', 'details', 'structures', 'lead', 'biological', 'activities', 'one', 'early', 'works', 'consisted', 'assembling', '23', 'amino', 'acids', 'form', 'synthetic', 'pig', 'corticotropin', '43.', 'the', 'molecules', 'built', 'stepwise', '43.', 'one', 'technique', 'used', 'syntheses', 'protect', 'terminal', 'amino', 'nitrogen', 'forming', 'protective', 'derivatives', 'subsequently', 'easily', 'cleaved', 'this', 'often', 'done', 'converting', 'amide', 'groups', 'p-toluenesulfonyl', 'chloride', 'used', 'manner', 'it', 'also', 'possible', 'form', 'imides', 'reactions', 'phthalic', 'anhydride', 'r', 'r', 'o', 'cl', 'o', 'h2n', 'cooh', 'o', 'o', 'o', 'h2n', 'cooh', 'h', 'n', 'o', 'cooh', 'o', 'r', 'r', 'o', 'o', 'n', 'cooh', 'the', 'condensations', 'carried', 'number', 'different', 'techniques', '26', '28.', 'a', 'shown', 'one', 'carry', 'amino', 'acyl', 'halide', 'reactions', 'h', 'n', 'o', 'cooh', 'socl2', 'pcl5', 'o', 'r', 'h', 'n', 'o', 'o', 'cl', 'o', 'r', '85', 'proteins', '555', 'r', 'h2n', 'cooh', 'h', 'n', 'o', 'o', 'r', 'o', 'r', 'cooh', 'n', 'h', 'the', 'schotten–baumann', 'reaction', 'used', 'many', 'peptide', 'syntheses', 'it', 'usually', 'carried', 'presence', 'common', 'bases', 'remove', 'halogen', 'acid', 'another', 'reaction', 'also', 'utilized', 'often', 'acid', 'azide', 'condensation', 'o', 'h', 'n', 'r', 'hno3', 'oh', 'h2n', 'nh2', 'r', 'o', 'h', 'n', 'r', 'o', 'n', 'h', 'h', 'r', 'o', 'r', 'h', 'n', 'h', 'n', 'n', 'h', 'n', 'h', 'r', 'o', 'h2n', 'coor', 'n3', 'or', 'a', 'unique', 'way', 'coupling', 'carboxylic', 'acids', 'amine', 'groups', 'using', 'nn0-dicyclohexyl', 'carbodiimide', 'method', 'this', 'illustrated', 'follows', 'h', 'n', 'cooh', 'o', 'n', 'n', 'h2n', 'c2h5', 'o', 'n', 'h', 'n', 'h', 'o', 'h', 'n', 'o', 'n', 'h', 'o', 'o', 'o', 'c2h5', 'o', 'n', 'h', 'n', 'h', 'in', 'addition', 'mentioned', 'rather', 'painstaking', 'techniques', 'polypeptide', 'syntheses', 'elegant', 'technique', 'developed', 'merriﬁeld', '53.', 'this', 'solid', 'phase', 'peptide', 'synthesis', 'automates', 'reaction', 'sequences', 'the', 'method', 'makes', 'use', 'insoluble', 'cross-linked', 'polymer', 'substrate', 'pendant', 'reactive', 'groups', 'attachment', 'peptide', 'chains', 'chloromethylated', 'polystyrene', 'microgels', 'often', 'used', '(see', 'chap', '9', 'discussions', 'use', 'chloromethylated', 'polystyrene', '556', '8', 'naturally', 'occurring', 'polymers', 'reactions', 'polymers)', 'the', 'chloromethyl', 'moieties', 'serve', 'initiating', 'sites', 'formation', 'polypeptides', 'o', 'r', 'o', 'n', 'h', 'o', 'o', 'ch2cl', 'h', 'r', 'o', 'o', 'o', 'o', 'n', 'h', 'r', 'o', 'h', 'n', 'h', 'o', 'a', 'new', 'amino', 'acid', 'protective', 'group', 'added', 'the', 'sequence', 'additions', 'controlled', 'repeated', 'many', 'times', 'build', 'desired', 'polypeptide', 'chain', 'unwanted', 'by-products', 'syntheses', 'washed', 'away', 'ﬁltered', 'new', 'step', 'this', 'method', 'lends', 'well', 'automation', 'automatic', 'peptide', 'synthesizers', 'available', 'commercially', 'one', 'automatic', 'peptide', 'synthesizer', 'employed', 'instance', 'synthesis', 'ribonuclease', 'enzyme', 'in', 'another', 'instance', 'enzyme', 'ferredoxin', 'consists', '55', 'amino', 'acid', 'residues', 'synthesized', '26.', 'most', 'recent', 'approaches', 'protein', 'syntheses', 'include', 'use', 'templates', 'spontaneous', 'self-', 'assembly', 'multiple', 'copies', 'derivatized', 'peptide', '70.', 'the', 'resultant', 'structure', 'however', 'conventional', 'linear-chain', 'protein', 'instead', 'oxime', 'bonds', 'formed', 'amino-oxyacetyl', 'groups', 'peptides', 'aldehyde', 'groups', 'template', 'the', 'method', 'claimed', 'many', 'potential', 'applications', 'bode', 'et', 'al', '70', 'reported', 'new', 'method', 'synthesis', 'peptide', 'oligomers', 'the', 'reaction', 'creates', 'aide', 'linkages', 'a-keto', 'carboxylic', 'acids', 'n-alkylhydroxylamines', 'o', 'r', 'oh', 'ho', 'r', 'o', 'n', 'h', '-h2o', '-', 'co2', 'r', 'o', 'r', 'n', 'h', '854', 'chemical', 'modiﬁcation', 'proteins', 'proteins', 'utilized', 'commercially', 'ancient', 'times', 'either', 'naturally', 'occurring', 'form', 'modiﬁed', 'manner', 'use', 'silk', 'wool', 'ﬁbers', 'instance', 'goes', 'back', 'long', 'time', 'many', 'animal', 'glues', 'prepared', 'bones', 'hides', 'cattle', 'sheep', 'also', 'around', 'long', 'time', 'today', 'glues', 'replaced', 'rapidly', 'synthetic', 'materials', 'those', 'still', '86', 'nucleic', 'acids', '557', 'manufactured', 'usually', 'formed', 'steaming', 'bones', 'hides', 'pressure', 'treating', 'hot', 'water', 'several', 'cycles', 'this', 'degrades', 'collagen', 'make', 'soluble', 'the', 'aqueous', 'solution', 'concentrated', 'vacuum', 'evaporation', 'water', 'the', 'material', 'gels', 'dried', 'pulverized', 'milder', 'hydrolysis', 'yields', 'gelatin', 'used', 'commercially', 'foods', 'casein', 'milk', 'protein', '(less', 'readily', 'available', 'casein', 'vegetable', 'sources', 'hardly', 'ever', 'used)', 'also', 'used', 'adhesives', 'here', 'synthetics', 'gradually', 'taking', 'at', 'one', 'time', 'used', 'produce', 'ﬁber', 'plastic', 'formed', 'cross-linking', 'formaldehyde', 'the', 'cross-linking', 'reaction', 'carried', 'immersing', 'proteins', 'formaldehyde', 'solution', '(4–5)', '55–65(cid1)c', 'long', 'periods', 'time', 'days', 'even', 'months', 'depending', 'upon', 'size', 'article', 'the', 'cross-', 'linking', 'reaction', 'involved', 'pendant', 'amino', 'groups', 'quite', 'similar', 'reactions', 'urea-', 'melamine-formaldehyde', 'resins', '(see', 'chap', '7)', 'some', 'condensation', 'formation', 'methylene', 'bridges', 'may', 'also', 'involve', 'amide', 'nitrogens', 'it', 'appear', 'likely', 'casein', 'ﬁbers', 'plastics', 'still', 'produced', 'anywhere', 'extensive', 'research', 'work', 'gone', 'modiﬁcation', 'proteins', 'commercial', 'applications', 'academic', 'reasons', 'thus', 'instance', 'frances', 'et', 'al', 'developed', 'new', 'reaction', 'introduces', 'single', 'reactive', 'ketones', 'aldehydes', 'n-terminal', 'groups', 'protein', 'proteins', 'mixed', 'pyridoxal', 'phosphate', '44.', 'the', 'researchers', 'also', 'developed', 'palladium-catalyzed', 'allylic', 'alkyl-', 'ation', 'attaches', 'long', 'lipid', 'tails', 'proteins', 'process', 'used', 'customize', 'solubility', 'enzymes', 'antibodies', 'viral', 'capsids', 'proteins', '86', 'nucleic', 'acids', 'these', 'protein-bound', 'polymers', 'essential', 'many', 'biological', 'processes', 'they', 'perform', 'functions', 'directing', 'syntheses', 'proteins', 'living', 'cells', 'constitute', 'chemical', 'basis', 'heredity', '56', '57.', 'the', 'polymers', 'polyphosphate', 'esters', 'sugars', 'contain', 'pendant', 'heterocyclic', 'amines', 'called', '“bases”', 'there', 'two', 'principle', 'types', 'nucleic', 'acids', 'two', 'different', 'sugars', 'one', 'd-2-deoxyribose', 'found', 'deoxyribonucleic', 'acid', '(dna)', 'o', 'p', 'o', 'n', 'sugar', 'o', 'base', 'oh', '3', '4', '2', '1', 'o', 'oh', 'ch2oh', '5', 'ho', 'oh', '3', '4', '2', '1', 'o', 'oh', 'ch2oh', '5', 'nh2', 'n', 'n', 'n', 'n', 'o', 'oh', 'oh', 'nh2', 'n', 'n', 'n', 'n', 'o', 'oh', 'oh', 'ho', 'oh2c', 'o', 'p', 'oh', '558', '8', 'naturally', 'occurring', 'polymers', 'the', 'one', 'd-ribose', 'found', 'ribonucleic', 'acids', '(rna)', 'the', 'sugars', 'furanose', 'form', 'they', 'linked', 'hydroxyl', 'groups', 'carbons', '3', '5', 'phosphate', 'esters', 'the', 'heterocyclic', 'amine', '“bases”', 'attached', 'carbon', '1', 'replacing', 'hydroxyl', 'group', 'a', 'sugar', 'molecule', 'base', 'attached', 'referred', 'nucleoside', 'a', 'nucleoside', 'esteriﬁed', 'phosphoric', 'acid', 'called', 'nucleotide', 'all', 'heterocyclic', 'amines', 'occur', 'nucleic', 'acids', '(dna', 'rna)', 'derivatives', 'either', 'pyrimidine', 'purine', 'these', 'nh2', 'nh2', 'n', 'o', 'n', 'h', 'n', 'o', 'n', 'h', 'h', 'ch3', 'n', 'ch3', 'o', 'n', 'h', 'o', 'cutosine', '5-methylcytosine', 'thymine', '86', 'nucleic', 'acids', '559', 'o', 'h', 'n', 'o', 'n', 'h', 'uracil', 'nh2', 'n', 'n', 'n', 'n', 'h', 'o', 'h', 'n', 'h2n', 'n', 'n', 'n', 'h', 'adenine', 'guanine', 'the', 'naming', 'nucleosides', 'depends', 'upon', 'sugars', 'thus', 'adenine', 'attached', 'ribose', 'called', 'adenosine', 'when', 'attached', 'deoxyribose', 'called', 'deoxyadenosine', 'hydrolysis', 'nucleoproteins', 'separates', 'acids', 'proteins', 'further', 'hydrolysis', 'yields', 'components', 'nucleic', 'acids', 'namely', 'sugars', 'bases', 'phosphoric', 'acid', 'the', 'nucleic', 'acids', 'differ', 'depending', 'upon', 'source', 'chain', 'lengths', 'sequences', 'distributions', 'bases', 'just', 'like', 'proteins', 'primary', 'structure', 'nucleic', 'acids', 'determined', 'partial', 'sequential', 'hydrolysis', '861', 'dna', 'rna', 'deoxynucleic', 'acids', 'isolated', 'types', 'living', 'cells', 'established', 'main', 'function', 'carry', 'genetic', 'information', '57.', 'these', 'high', 'molecular', 'weight', 'polymeric', 'materials', 'some', 'found', 'high', '100', 'million', 'analyses', 'dna', 'structures', 'show', 'numbers', 'adenine', 'bases', 'always', 'number', 'thymine', 'bases', 'also', 'numbers', 'guanine', 'bases', 'always', 'equal', 'numbers', 'cytosines', 'based', 'information', 'various', 'analyses', 'x-ray', 'investigation', 'structure', 'watson', 'crick', 'concluded', 'secondary', 'structure', 'dna', 'must', 'double', 'helix', '58.', 'two', 'separate', 'right-handed', 'helical', 'chains', 'wind', 'around', 'held', 'together', 'hydrogen', 'bonding', 'base', 'pairs', 'the', 'bases', 'paired', 'adenine', 'thymine', 'guanine', 'cytosine', 'o', 'h', 'h', 'thymine', 'sugar', 'n', 'n', 'h', 'n', 'n', 'n', 'n', 'n', 'adenine', 'sugar', 'o', '107', 'a', 'cyiosine', 'sugar', 'n', 'n', 'h', 'n', 'h', 'o', 'n', 'h', 'n', 'n', 'o', 'h', 'h', '107', 'a', 'n', 'n', 'guanine', 'sugar', 'the', 'base', 'pairs', 'extended', 'perpendicularly', 'toward', 'center', 'deoxyribose–phosphate', 'ester', 'chains', 'located', 'outside', 'helix', 'the', 'two', 'strands', 'antiparallel', 'one', 'turn', 'helix', 'corresponds', 'ten', 'nucleotide', 'pairs', '34', 'a˚', 'length', 'the', 'width', 'helix', '20', 'a˚', 'evidence', 'presented', 'dnas', 'native', 'forms', 'cyclic', '59', 'may', 'even', 'occur', 'two', 'interlocking', 'rings', 'while', 'known', 'dna', 'molecules', 'form', 'right-handed', 'helix', 'left-handed', 'helix', 'prepared', 'synthetically', 'laboratory', '52.', 'it', 'speculated', 'instances', 'nature', 'biological', 'function', '53.', 'these', 'dna', 'left-handed', 'helixes', 'may', 'exist', 'conformations', 'named', 'z-dnas', 'backbones', 'zigzag', 'molecule', '560', '8', 'naturally', 'occurring', 'polymers', 'there', 'less', 'information', 'secondary', 'structures', 'rnas', 'it', 'known', 'rna', 'molecules', 'lower', 'molecular', 'weight', 'dna', 'molecules', 'in', 'addition', 'known', 'three', 'main', 'types', 'rnas', 'living', 'cells', 'these', 'ribosomal', 'rna', '(r-rna)', 'transfer', 'rna', '(t-rna)', 'messenger', 'rna', '(m-rna)', 'the', 'molecular', 'weights', 'three', 'forms', 'average', '1000000', '25000', '500000', 'respectively', 'rna', 'molecules', 'help', 'hydrogen', 'bonding', 'take', 'three-dimensional', 'cloverleaf', 'structures', '54.', 'the', 'molecule’s', 'three-dimensional', 'shape', 'also', 'assumes', 'l-shape', 'cloverleaf', 'bent', '862', 'synthetic', 'methods', 'preparation', 'nucleic', 'acids', 'over', 'last', '20–30', 'years', 'methods', 'developed', 'synthesize', 'short', 'deoxyribonucleotide', 'chains', '57.', 'one', 'synthetic', 'procedure', 'illustrated', 'follows', 'c', 'o', '3', 'o', 'n', 'o', 'n', 'h', 'o', 'oh', 'n', 'n', 'o', 'nh2', 'n', 'o', 'oh', 'o', 'n', 'o', 'po', 'o', 'n', 'n', 'o', 'n', 'h', 'o', 'n', 'o', 'nh', 'o', 'n', 'n', 'n', 'o', 'p', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'ho', '1', 'oh', '2.h', 'o', 'n', 'h', 'o', 'n', 'o', 'nh', 'n', 'n', 'n', 'o', 'p', 'o', 'o', 'o', 'n', 'o', 'o', 'o', '87', 'polyalkanoates', '561', 'the', 'bulky', 'triphenylmethyl', 'moiety', 'functions', 'block', '50', 'hydroxyl', 'groups', 'removed', 'necessary', 'the', 'true', 'acetyl', 'portion', 'also', 'serves', 'block', '30', 'hydroxyl', 'position', 'the', 'product', 'used', 'expansion', 'chain', 'another', 'approach', 'syntheses', 'nucleic', 'acids', 'use', 'polymeric', 'supports', 'syntheses', 'polypeptides', 'the', 'preparation', 'protecting', 'groups', 'attachment', 'carbon', '5', 'deoxyribose', 'surface', 'cross-linked', 'polystyrene', 'illustrated', 'follows', '57:', 'o', 'croslinked', 'postyrene', 'surface', 'och3', 'och3', 'oh', 'cl', 'another', 'protective', 'group', 'used', 'dimethoxy', 'trityl', 'the', 'carbon', '3', 'deoxyribose', 'protected', 'phosphoramide', 'benzoyl', 'groups', 'used', 'protect', 'adenine', 'guanine', 'bases', 'lately', 'place', 'cross-linked', 'polystyrene', 'controlled', 'pore', 'glass', 'supports', 'become', 'popular', 'commercial', 'synthesizing', 'machines', 'available', 'today', 'polynucleotide', 'syntheses', 'these', 'similar', 'synthesizers', 'used', 'polypeptide', 'syntheses', '87', 'polyalkanoates', 'many', 'bacteria', '(b-hydroxyalkanoate)s', 'general', 'structure', 'potential', 'source', 'naturally', 'occurring', 'polyesters', 'mainly', 'poly', 'ho', 'ch', 'ch2', 'c', 'o', 'ch', 'ch2', 'ch2', 'cooh', 'r', 'o', 'r', 'o', 'c', 'o', 'ch', 'n', 'r', 'polymer', 'poly(b-hydroxybutyrate)', 'r', '¼', 'ch3', '69.', 'the', 'material', 'found', 'form', 'hard', 'crystalline', 'granules', 'many', 'bacterial', 'cells', 'the', 'common', 'one', 'poly(b-hydroxybutyrate)', 'discovered', 'back', '1920s', 'identiﬁed', 'polymer', 'd-((cid3))-b-hydroxybutyric', 'acid', 'in', 'native', 'state', 'polymer', 'may', 'reach', 'molecular', 'weights', '1000000', 'higher', 'it', 'forms', 'compact', 'right-handed', 'helix', 'twofold', 'screw', 'axis', 'repeat', 'unit', '596', 'a˚', 'because', 'poly', '(b-hydroxybutyrate)', 'stereoregular', 'highly', 'crystalline', 'the', 'substitution', 'b-position', 'makes', 'thermally', 'unstable', 'this', 'limits', 'use', 'plastics', 'it', 'found', 'however', 'oxygen', 'starvation', 'bacterial', 'cultures', 'results', 'formation', 'copolymer', 'b-hydroxybutyric', 'acid', 'b-hydroxyvaleric', 'acid', 'instead', 'homopolymer', 'further', 'investigations', 'showed', 'many', '11', 'different', 'b-hydroxy', 'acid', 'constituents', 'present', 'different', 'naturally', 'occurring', 'polyalkanoates', 'depending', 'upon', 'bacterial', 'source', 'conditions', 'growth', 'today', 'family', 'products', 'marketed', '562', '8', 'naturally', 'occurring', 'polymers', 'trade', 'name', 'biopol', 'available', 'commercially', 'range', 'properties', 'melting', 'point', 'toughness', 'ﬂexibility', 'others', 'the', 'melting', 'points', 'range', '80', '180(cid1)c', 'it', 'reported', 'pseudomonas', 'oleovorans', 'microorganism', 'forced', 'produce', 'thermo-', 'plastic', 'elastomer', 'growing', 'substrate', 'containing', 'sodium', 'octanoate', '71.', 'the', 'product', 'poly', '(b-hydroxy', 'octanoate)', 'it', 'contains', 'crystalline', 'regions', 'act', 'physical', 'cross-links', 'also', 'addition', 'poly(ethylene', 'glycol)', 'molecular', 'weight', '200', 'cultivation', 'media', 'alcaligines', 'eutrophus', 'polymer', 'formation', 'carbon', 'source', 'used', '4-hydroxybutyrate', 'affects', 'type', 'ester', 'produces', '73.', 'addition', 'glycol', 'amounts', '0', '2', 'weight', 'results', 'increased', 'amounts', '4-hydroxybutyrate', 'incorporation', '66', '86', 'mol%', 'produce', 'poly(3-dydroxyalkanoate)', 'when', 'however', 'amount', 'glycol', 'increased', '4,', 'amount', 'incorporation', '4-hydroxybutyrate', 'decreases', '64.', 'the', 'product', 'also', 'contains', 'diblock', 'copolymer', 'poly(3-hydroxyalkanoate)', 'poly(ethylene', 'glycol)', '73.', 'review', 'questions', 'section', '81', 'section', '82', 'plant', 'gums', '1', 'list', 'six', 'main', 'categories', 'naturally', 'occurring', 'polymers', '2', 'draw', 'chemical', 'structures', 'repeat', 'units', 'category', 'naturally', 'occurring', 'polymers', '1', 'describe', 'hemicellulose', 'this', 'include', 'explanation', 'xylan', 'galactan', 'araban', '2', 'in', 'discussing', 'starches', 'explain', 'amylose', 'amylopectin', 'explain', 'draw', 'structures', '3', 'discuss', 'cellulose', 'how', 'cellulose', 'differ', 'starch', '4', 'what', 'cellobiose', 'draw', 'structure', 'give', 'chemical', 'name', '5', 'what', 'regenerated', 'cellulose', 'explain', 'chardonnet', 'silk', 'cuprammonium', 'rayon', 'viscose', 'rayon', 'prepared', '6', 'discuss', 'chemistry', 'cellulose', 'nitrates', 'how', 'prepared', 'used', '7', 'discuss', 'chemistry', 'cellulose', 'acetate', 'how', 'prepared', 'used', 'describe', 'mixed', 'cellulose', 'esters', '8', 'discuss', 'chemistry', 'cellulose', 'ethers', '9', 'show', 'reaction', 'cellulose', 'acrylonitrile', 'form', 'new', 'derivatives', 'give', 'two', 'examples', '11', 'draw', 'structures', 'alginic', 'acid', 'chitin', '10', 'iodocellulose', 'dehydrohalogenated', 'form', 'double', 'bonds', 'polymer', 'this', 'used', 'section', '83', '1', 'discuss', 'chemistry', 'lignin', 'drawing', 'structure', 'coniferyl', 'alcohol', 'can', 'think', 'useful', 'product', 'lignin', '563', 'references', 'section', '84', 'section', '85', '1', 'describe', 'natural', 'rubber', 'how', 'obtained', 'what', 'chemical', 'structure', '2', 'what', 'gutta-percha', 'balata', 'chicle', 'explain', '1', 'explain', 'meant', 'polypeptides', '2', 'explain', 'difference', 'ﬁbrous', 'proteins', 'globular', 'proteins', '3', 'what', 'nucleoproteins', 'what', 'prosthetic', 'group', 'give', 'example', '4', 'explain', 'meant', 'secondary', 'structure', 'protein', 'a-helix', '5', 'what', 'meant', 'tertiary', 'structure', 'protein', '6', 'what', 'enzyme', 'how', 'function', '7', 'discuss', 'aid', 'chemical', 'equations', 'synthetic', 'routes', 'polypeptides', '8', 'discuss', 'chemical', 'modiﬁcations', 'proteins', 'commercial', 'purposes', 'section', '86', '1', 'what', 'basic', 'structure', 'unit', 'nucleic', 'acids', '2', 'how', 'sugars', 'differ', 'dna', 'rna', '3', 'draw', 'structures', 'nucleoside', 'nucleotide', '4', 'discuss', 'syntheses', 'nucleic', 'acids', 'recommended', 'reading', 'k', 'kamide', 'cellulose', 'cellulose', 'derivatives', 'elsevier', 'amsterdam', '2005', 'references', '1', 'e', 'hicks', 'shellac', 'it’s', 'origins', 'applications', 'chemical', 'publishing', 'co,', 'new', 'york', '1961', 's', 'maiti', 'mds', 'rahman', 'j,macromol', 'sci,', 'rev', 'macromol', 'chem', 'phys,', 'c26', '441', '(1986)', '2', 'e', 'fischer', 'ber,', '24', '18362683', '(1881)', '29', '1377', '(1896)', '3', 'wn', 'haworth', 'el', 'hirst', 'ha', 'hampton', 'j', 'chem', 'soc,', '1929', '1739', '4', 't', 'svedberg', 'j', 'phys', 'colloid', 'chem,', '51', '1', '(1947)', '5', 'kh', 'meyer', 'h', 'mark', 'der', 'aufbau', 'der', 'hochpolymeren', 'organischen', 'naturstoffe', 'akademie', 'verlaga-', 'gesellschaft', 'leipzig', '1930', '6', 's', 'morell', 'kp', 'link', 'j', 'biol', 'chem,', '100', '385', '(1933)', '7', 's', 'morell', 'l', 'bauer', 'kp', 'link', 'j', 'biol', 'chem,', '105', '1', '(1934)', '8', 'pa', 'levene', 'lc', 'kreider', 'j', 'biol', 'chem,', '129', '591', '(1937)', '9', 'pa', 'levene', 'gm', 'meyer', 'm', 'kuna', 'science', '89', '370', '(1939)', '10', 'rl', 'whistler', 'jn', 'bemiller', 'eds', 'industrial', 'gums', '2nd', 'ed,', 'academic', 'press', 'new', 'york', '1973', '11', 'rl', 'whistler', 'ef', 'paschall', 'eds', 'starch', 'chemistry', 'technology', 'vols', 'i', 'ii', 'academic', 'press', 'ny', '1965', '12', 'nm', 'bikales', 'l', 'segal', 'eds', 'cellulose', 'cellulose', 'derivatives', 'wiley-interscience', 'ny', '1971', 'a', 'hebeish', 'jt', 'guthrie', 'the', 'chemistry', 'technology', 'cellulose', 'derivatives', 'springer-verlag', 'new', 'york', '1981', '564', '8', 'naturally', 'occurring', 'polymers', '13', 'em', 'fetters', 'ed', 'chemical', 'reactions', 'polymers', 'wiley-interscience', 'ny,', '1964', '14', 'za', 'rogovin', 'vysokomol', 'soyed,', 'a13', '(2)', '437', '(1971)', '15', 'j', 'f', 'klebe', 'hl', 'finkbeiner', 'j', 'polymer', 'sci,', 'a-1', '7', '1947', '(1969)', '16', 'lg', 'nikologovskaya', 'ls', 'galbraikh', 'ys', 'kozlova', 'za', 'rogovin', 'vysokomol', 'soyed,', 'a', '12', '2762', '(1970)', 'h', 'tonami', 'h', 'uyama', 's', 'kobayshi', 'macromolecules', '2004', '37', '7901', '17', 's', 'maiti', 's', 'das', 'm', 'maiti', 'a', 'ray', 'polymer', 'applications', 'renewable', 'resource', 'materials', 'ce', 'carraher', 'jr', 'lh', 'sperling', '(eds)', 'plenum', 'press', 'new', 'york', '1983', '18', 'ea', 'mac', 'gregor', 'ct', 'greenwood', 'polymers', 'nature', 'wiley', 'new', 'york', '1980', '19', 'ia', 'pearl', 'the', 'chemistry', 'lignin', 'dekker', 'new', 'york', '1967', '20', 'k', 'freudenberg', 'science', '148', '595', '(1965)', '21', 'fr', 'eirich', 'ed,', 'science', 'technology', 'rubber', 'academic', 'press', 'new', 'york', '1978', 'ad', 'roberts', 'ed,', 'natural', 'rubber', 'science', 'technology', 'oxford', 'university', 'press', 'new', 'york', '1988', '22', 'm', 'joly', 'a', 'physico-chemical', 'approach', 'denaturation', 'proteins', 'academic', 'press', 'ney', 'york', '1965', '23', 'l', 'pauling', 'rb', 'corey', 'proc', 'natl', 'acad', 'sci,', '37', '729', '(1951)', 'ibid,', '39', '253', '(1953)', '24', 'l', 'pauling', 'rb', 'corey', 'hr', 'branson', 'proc', 'natl', 'acad', 'sci,', '37', '205', '(1951)', '25', 'am', 'liquori', 'acta', 'cryst,', '8', '375', '(1955)', '26', 's', 'seifter', 'pm', 'gallop', 'the', 'proteins', 'vol', '4', '2nd', 'ed,', 'h', 'neurath', 'ed,', 'academic', 'press', 'new', 'york', '1966', 'ag', 'walton', 'polypeptides', 'protein', 'structure', 'elsevier', 'new', 'york', '1981', '27', 'dr', 'davis', 'j', 'mol', 'biol,', '9', '605', '(1964)', '28', 'er', 'blout', 'polyamino', 'acids', 'polypeptides', 'proteins', 'ma', 'stahmann', '(ed)', 'univ', 'wisconsin', 'press', 'madison', '1962', 'london', '1963', '29', 'mf', 'perutz', 'jc', 'kendrew', 'hc', 'watson', 'j', 'mol', 'biol,', '13', '669', '(1965)', '30', 'rt', 'hatch', 'nature', '206', '777', '(1965)', '31', 're', 'dickerson', 'i', 'geis', 'the', 'structure', 'action', 'proteins', 'harper', 'row', 'new', 'york', '1969', '32', 'ch', 'bamford', 'a', 'elliott', 'fiber', 'structure', 'jws', 'hearle', 'rn', 'peters', '(eds)', 'butterworth', 'co', 'ltd,', '33', 'b', 'jergenson', 'optical', 'rotary', 'dispersion', 'proteins', 'other', 'macromolecules', 'springer', 'verlag', 'new', 'york', '1969', '34', 'fh', 'crick', 'acta', 'cryst,', '6', '689', '(1953)', '35', 'rdb', 'fraser', 'tp', 'mac', 'rae', 'nature', '195', '1167', '(1962)', '36', 'c', 'cohen', 'kc', 'holmes', 'j', 'mol', 'biol,', '6', '423', '(1963)', '37', 'wb', 'ward', 'h', 'p', 'lundgren', 'adv', 'protein', 'chem,', '9', '243', '(1954)', '38', 'rh', 'peters', 'textile', 'chemistry', 'vol', 'i', 'elsevier', 'publishing', 'co,', 'new', 'york', '1963', '39', 'f', 'lucas', 'jtb', 'shaw', 'sg', 'smith', 'biochem', 'j,', '66', '468', '(1957)', '40', 'rbd', 'fraser', 'tp', 'mac', 'rae', 'ge', 'rogers', 'nature', '193', '1052', '(1962)', '41', 'a', 'rich', 'fh', 'crick', 'j', 'mol', 'biol,', '3', '483', '(1961)', '42', 'rbd', 'fraser', 'tp', 'mac', 'rae', 'dad', 'parry', 'e', 'suzuki', 'polymer', '10', '(10)', '810', '(1969)', '43', 'jc', 'kendrew', 'science', '139', '1259', '(1963)', '44', 'mb', 'frances', 'et', 'al,', 'angew', 'chem', 'inter', 'ed', '2006', '45', '5307', 'j', 'am', 'chem', 'soc', '2006', '128', '1080', '45', 'g', 'buse', 'angew', 'chem,', 'intern', 'ed,', '10', '(10)', '663', '(1971)', '46', 'mf', 'peruz', 'g', 'will', 'atc', 'north', 'nature', '185', '416', '(1960)', '47', 'p', 'jolles', 'angew', 'chem,', 'intern', 'ed,', '8', '(4)', '227', '(1969)', '48', 'e', 'katalski', 'm', 'sela', 'advan', 'protein', 'chem,', '13', '243', '(1958)', '49', 'm', 'idelson', 'er', 'blout', 'j', 'am', 'chem', 'soc,', '80', '2387', '(1958)', '50', 'y', 'shalitin', 'e', 'katchalski', 'j', 'am', 'chem', 'soc,', '82', '1630', '(1960)', '51', 'k', 'hofmann', 'chem', 'eng', 'news', 'p145', '(aug', '7', '1967)', '52', 'r', 'schwyzer', 'p', 'sieber', 'helv', 'chim', 'acta', '49', '134', '(1966)', '53', 'rb', 'merriﬁeld', 'j', 'am', 'chem', 'soc,', '85', '2149', '(1963)', '54', 'chem', 'eng', 'news', 'p28', '(april', '22', '1968)', '55', 'b', 'gutte', 'rb', 'merriﬁeld', 'j', 'am', 'chem', 'soc,', '91', '501', '(1961)', '56', 'jn', 'davidson', 'the', 'biochemistry', 'nucleic', 'acids', '7th', 'ed,', 'academic', 'press', 'new', 'york', '1972', '57', 'nk', 'kochetkov', 'ei', 'budovskii', '(eds)', 'organic', 'chemistry', 'nucleic', 'acids', 'plenum', 'press', 'london', 'part', 'a', '1971', 'part', 'b', '1972', 'lb', 'townsend', 'rs', 'tipson', '(eds)', 'nucleic', 'acid', 'chemistry', 'wiley-interscience', 'new', 'york', '1986', '58', 'jd', 'watson', 'fh', 'crick', 'nature', '171', '734', '964', '(1953)', '59', 'g', 'felsenfeld', 'ht', 'miles', 'ann', 'rev', 'biochem,', '36', '407', '(1967)', '60', 'jl', 'fox', 'chem', 'end', 'news', 'p', '14', '(dec', '14', '1979)', '61', 'g', 'kolata', 'science', '214', '1108', '(1981)', '62', 'lb', 'townsend', 'rs', 'nripson', '(ed)', 'nucleic', 'acid', 'chemistry', 'wiley-interscience', 'new', 'york', '1986', '565', 'references', '435', '(1974)', '63', 'fl', 'suddath', 'gj', 'quigley', 'a', 'mcpherson', 'jl', 'sussman', 'ahj', 'wang', 'nc', 'seeman', 'a', 'rich', 'science', '185', '64', 'se', 'chang', 'd', 'ish-horowicz', 'j', 'mol', 'biol,', '84', '375', '(1974)', '65', 'kl', 'agarwal', 'a', 'yamazaki', 'pj', 'cashion', 'hg', 'khorana', 'angew', 'chem,', 'intern', 'ed,', '11', '(6)', '451', '(1972)', '66', 'jm', 'frechet', 'schurch', 'j', 'am', 'chem', 'soc,', '93', '492', '(1971)', '67', 'm', 'lemoigne', 'cr', 'acad', 'sci,', '180', '1539', '(1925)', 'bull', 'soc', 'chim', 'biol,', '8', '1291', '(1926)', '68', 'j', 'cornibert', 'rh', 'marchessault', 'h', 'benoit', 'g', 'weil', 'macromolecules', '3', '741', '(1970)', '69', 'rh', 'marchessault', 'amchem', 'soc', 'polymer', 'preprints', '29', '(1)', '584', '(1988)', '70', 'k', 'rose', 'j', 'am', 'chem', 'soc,', '116', '30', '(1994)', 'j', 'bode', 'et', 'al', 'j', 'am', 'chem', 'soc,', '2006', '128', '1452', '71', 'kg', 'gagnon', 'rw', 'lenz', 'rj', 'farris', 'rc', 'fuller', 'rubber', 'chem', 'tech,', '65', '761', '(1992)', '72', 'k', 'griebenow', 'am', 'klibanov', 'j', 'am', 'chem', 'soc,', '118', '11695', '(1996)', 'kn', 'allen', 'd', 'dunway', 'science', '2003', '299', '2067', '73', 'f', 'shi', 'ra', 'gross', 'dr', 'rutheford', 'macromolecules', '29', '10', '(1996)', 'chapter', '9', 'organic', 'reactions', 'polymers', '91', 'reactivity', 'macromolecules', 'in', 'consideration', 'various', 'chemical', 'reactions', 'macromolecules', 'reactivity', 'functional', 'groups', 'must', 'compared', 'small', 'molecules', 'the', 'comparisons', 'stimulated', 'many', 'investigations', 'led', 'conclusions', 'functional', 'groups', 'exhibit', 'equal', 'reactivity', 'large', 'small', 'molecules', 'conditions', 'identical', 'these', 'conclusions', 'supported', 'theoretical', 'evidence', '1', '2.', 'speciﬁcally', 'apply', 'following', 'situations', '1:', '1', 'reactions', 'take', 'place', 'homogeneous', 'ﬂuid', 'media', 'reactants', 'intermediates', 'end', 'products', 'fully', 'soluble', 'these', 'conditions', 'exist', 'start', 'end', 'reactions', '2', 'all', 'elementary', 'steps', 'involve', 'individual', 'functional', 'groups', 'the', 'reacting', 'species', '3', 'the', 'steric', 'factors', 'low', 'molecular', 'weight', 'compounds', 'selected', 'comparison', 'must', 'similar', 'small', 'mobile', 'large', 'molecules', 'the', 'illustrated', 'examples', 'for', 'instance', 'rates', 'photochemical', 'cis–trans', 'isomerization', 'azobenzene', 'residues', 'backbones', 'ﬂexible', 'polymeric', 'chains', 'analogous', 'small', 'molecules', '3.', 'another', 'example', 'activation', 'energy', 'cis–trans', 'isomerization', 'azo-aromatic', 'polymers', 'it', 'low', 'molecular', 'weight', 'analogs', '4.', 'a', 'third', 'example', 'experiment', 'comparing', 'conformational', 'transitions', 'eximers', 'large', 'small', 'molecules', 'a', 'sandwich', 'complex', 'forms', 'excited', 'aromatic', 'chromophore', 'ar', 'similar', 'chromo-', 'phore', 'ground', 'state', 'irradiated', 'light', 'appropriate', 'wavelength', 'the', 'conformation', 'required', 'excimer', 'correspond', 'prohibitive', 'energy', 'requirement', 'unexcited', 'molecule', 'all', 'conformational', 'transitions', 'must', 'take', 'place', 'lifetime', 'excited', 'state', 'chromophore', 'type', '5:', 'ar', 'ar', 'ar', 'ar', 'x', 'x', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-99', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '567', '568', '9', 'organic', 'reactions', 'polymers', 'the', 'ratio', 'ﬂuorescence', 'intensity', 'excimer', 'normal', 'molecule', 'measure', 'probability', 'conformational', 'transition', 'takes', 'place', 'excited', 'lifetime', 'a', 'polyamide', 'small', 'proportion', 'following', 'units', 'used', 'comparison', 'n', 'h', 'n', 'o', 'n', 'h', 'emission', 'spectra', 'dilute', 'solutions', 'polyamide', 'low', 'molecular', 'weight', 'analog', 'measured', 'range', 'temperatures', 'they', 'showed', 'activation', 'energies', 'conformational', 'transitions', 'required', 'excimer', 'formations', 'essentially', 'materials', '5.', 'in', 'addition', 'bimolecular', 'activation-controlled', 'reactions', 'independent', 'degree', 'polymerization', '6.', 'simple', 'sn2', 'reactions', 'reactive', 'groups', 'attached', 'chain', 'ends', 'mono-', 'disperse', 'macromolecules', 'wide', 'range', 'molecular', 'weights', 'independent', 'dp', '7', '8', 'range', '20–2000', '7.', 'this', 'shown', 'three', 'different', 'reactions', 'in', 'ﬁrst', 'one', 'reactivities', 'chlorine-terminated', 'low', 'high', 'molecular', 'weight', 'polystyrenes', 'towards', 'polystyryllithium', 'equal', 'benzene', 'cyclohexane', 'solvents', 'cl', 'li', 'in', 'second', 'one', 'reactivity', 'primary', 'amine-terminated', 'polyoxyethylenes', 'sulfonyl', 'chloride-ended', 'polyoxyethylenes', 'chloroform', 'also', 'o', 'nh2', 'o', 'so2', 'adduct', 'so2cl', 'in', 'third', 'one', 'chain', 'length', 'dependence', 'propagation', 'rates', 'measured', 'polymerizations', 'methyl', 'methacrylate', 'in', 'range', 'dps', '130', '14200', 'propagation', 'constant', 'shown', 'independent', 'chain', 'length', '8.', 'on', 'hand', 'unequal', 'reactivity', 'observed', '1', 'in', 'bimolecular', 'reactions', 'diffusion-controlled', '2', 'when', 'neighboring', 'group', 'participations', 'become', 'signiﬁcant', '3', 'when', 'properties', 'polymers', 'solution', 'altered', 'gelation', '4', 'when', 'tacticities', 'polymers', 'affect', 'neighboring', 'group', 'interactions', '5', 'when', 'heterogeneous', 'conditions', 'affect', 'accessibility', 'reactive', 'sites', 'there', 'special', 'situations', 'occur', 'for', 'instance', 'electrostatic', 'charges', 'carried', 'polymers', 'may', 'extend', 'long', 'distances', 'solutions', 'may', 'manifest', 'reactions', '91', 'reactivity', 'macromolecules', '569', 'charged', 'reagents', 'sometimes', 'chain', 'ﬂexibility', 'folding', 'cause', 'functional', 'groups', 'come', 'together', 'interact', 'though', 'may', 'located', 'well', 'apart', 'polymer', 'backbone', 'polymer', 'solutions', 'type', 'comparable', 'dispersions', 'individual', 'droplets', 'concentrated', 'solutions', 'some', 'statements', 'may', 'require', 'additional', 'clariﬁcation', 'an', 'elaboration', 'therefore', 'follows', '911', 'diffusion-controlled', 'reactions', 'reactions', 'bimolecular', 'affected', 'viscosity', 'medium', '9.', 'the', 'translational', 'motions', 'ﬂexible', 'polymeric', 'chains', 'accompanied', 'concomitant', 'segmental', 'rearrangements', 'whether', 'applies', 'particular', 'reaction', 'however', 'hard', 'tell', 'for', 'instance', 'two', 'dynamic', 'processes', 'affect', 'reactions', 'like', 'termination', 'rates', 'chain-growth', 'polymerizations', 'if', 'termination', 'processes', 'controlled', 'translational', 'motion', 'rates', 'reactions', 'might', 'expected', 'vary', 'translational', 'diffusion', 'coefﬁcients', 'polymers', 'termination', 'reactions', 'however', 'controlled', 'diffusions', 'entire', 'molecules', 'segmental', 'diffusions', 'within', 'coiled', 'chains', '10.', 'the', 'reactive', 'ends', 'assume', 'positions', 'exposed', 'mutual', 'interaction', 'affected', 'viscosity', 'medium', '912', 'paired', 'group', 'neighboring', 'group', 'effects', 'when', 'random', 'irreversible', 'intramolecular', 'reactions', 'occur', 'polymeric', 'backbones', 'functional', 'groups', 'adjacent', 'expected', 'react', 'there', 'however', 'upper', 'limit', 'conversions', 'this', 'upper', 'limit', 'due', 'statistical', 'probability', 'functional', 'groups', 'bound', 'become', 'isolated', 'the', 'limit', 'conversion', 'calculated', '865', '11.', 'theoretically', 'quantitative', 'conversions', 'possible', 'reversible', 'reactions', 'paired', 'functional', 'groups', 'macromolecular', 'backbones', 'the', 'ability', 'however', 'isolated', 'reactive', 'groups', 'ﬁnd', 'pair', 'depends', 'either', 'upon', 'particularly', 'high', 'driving', 'forces', 'upon', 'time', 'required', 'accomplish', 'complete', 'conversions', '12.', 'for', 'reactions', 'initiated', 'randomly', 'different', 'sites', 'probability', 'high', 'two', 'groups', 'terminal', 'units', 'eventually', 'meet', 'react', 'since', 'reactions', 'reversible', 'least', 'theory', 'high', 'conversions', 'possible', 'neighboring', 'group', 'participation', 'usually', 'deduced', 'three', 'types', 'evidence', '1', 'if', 'reactions', 'occur', 'rapidly', 'rate-determining', 'step', 'expected', 'considerations', '2', 'if', 'stereochemistry', 'reactions', 'suggests', 'neighboring', 'group', 'involvement', '3', 'if', 'molecular', 'rearrangements', 'occur', 'groups', 'remain', 'bonded', 'reaction', 'centers', 'break', 'away', 'atoms', 'originally', 'attached', 'substrates', 'there', 'many', 'examples', 'literature', 'describe', 'neighboring', 'group', 'effects', 'reactions', 'polymers', 'one', 'example', 'hydrolysis', 'poly(p-nitrophenyl', 'methacrylate-co-acrylic', 'acid)', 'the', 'high', 'reaction', 'rate', 'neutral', 'ph', 'due', 'attacks', 'carboxylic', 'moieties', 'upon', 'neighboring', 'carbonyl', 'carbons', '13–15.', 'decomposition', 'rates', 't-butyl', 'acrylate-styrene', 'copolymers', '16', 'serve', 'another', 'example', 'experimental', 'data', 'show', 'pronounced', 'acceleration', 'samples', 'this', 'interpreted', 'terms', 'intra', 'intermolecular', 'interactions', 'esters', 'carboxylic', 'groups', 'it', 'follows', 'suggestion', 'cherkezyan', 'litmanovich', '16', '17', 'instantaneous', 'reactivity', 'group', 'depends', 'microenvironment', 'that', 'includes', '(for', 'reactions', 'polymer', 'molten', 'condition)', 'two', 'nearest', 'units', 'chain', '(internal', 'neighbors)', 'two', 'units', 'belonging', 'two', 'different', 'chains', '(external', 'neighbors)', '570', '9', 'organic', 'reactions', 'polymers', 'another', 'example', 'neighboring', 'group', 'effect', 'behavior', 'polyacrylamides', 'hydrolyses', 'there', 'two', 'distinct', 'successive', 'rates', '18.', 'after', 'conversions', '40–50', 'reached', 'reactions', 'slow', 'this', 'due', 'accumulations', 'negative', 'electrostatic', 'charges', 'polymeric', 'backbones', '18.', 'in', 'alkaline', 'media', 'increasing', 'negative', 'charges', 'along', 'chains', 'exert', 'electrostatic', 'repulsions', 'toward', 'hydroxyl', 'ions', 'this', 'results', 'rate', 'decreases', '913', 'effect', 'molecular', 'size', 'an', 'example', 'effect', 'dp', 'rates', 'alkaline', 'hydrolyses', 'poly(vinyl', 'acetate)s', 'rapid', 'increases', 'rates', 'seen', '19', 'large', 'small', 'molecules', 'reactions', 'progress', 'solvents', 'good', 'products', 'like', 'acetone–water', 'mixtures', 'used', 'reactions', 'these', 'however', 'poor', 'solvents', 'staring', 'materials', 'high', 'dp', 'low', 'molecular', 'weight', 'molecules', 'soluble', 'this', 'means', 'start', 'reaction', 'large', 'molecules', 'coiled', 'reactive', 'sites', 'readily', 'available', 'as', 'reactions', 'progress', 'chains', 'unravel', 'sites', 'became', 'accessible', 'accompanying', 'increases', 'reaction', 'rates', 'because', 'small', 'molecules', 'soluble', 'reactive', 'sites', 'accessible', 'start', 'reactions', 'rates', 'constant', 'there', 'many', 'report', 'literature', 'effects', 'chain', 'conformation', '19–25.', 'one', 'example', 'radical', 'bromination', 'poly(methyl', 'styrene)', '20', 'n-bromosuccinimide-benzoyl', 'peroxide', 'br2-k2co3-light', '13c', 'nmr', 'spectroscopy', 'shows', 'differences', 'reactivities', 'methyl', 'groups', '3', '4', 'positions', 'benzene', 'rings', 'isotactic', 'atactic', 'polystyrenes', '20.', 'the', 'differences', 'reactivities', 'poly(vinyl', 'alcohol)s', 'isotactic', '(meso)', 'syndiotactic', '(dl-diol)', 'portions', 'polymers', 'cis', 'trans', 'acetals', '26–28', 'another', 'example', 'in', 'extending', 'model', 'compounds', 'reactions', 'stereo', 'isomers', 'pentane-24-diol', 'heptane-', '146-triol', 'formaldehyde', 'take', 'place', 'much', 'faster', 'meso', 'dl-diol', 'portions', '26–28.', 'even', 'important', 'steric', 'effects', 'imposed', 'restricted', 'rotations', 'for', 'instance', 'quaternizations', 'chloromethylated', 'polyether', 'sulfones', 'exhibit', 'decreasing', 'rates', 'high', 'degrees', 'substitution', 'this', 'attributed', 'restricted', 'rotations', 'polymeric', 'chains', 'phenomenon', 'observed', 'ﬂexible', 'chloromethylated', 'polystyrene', 'identical', 'conditions', '23', '24.', '914', 'effects', 'changes', 'solubility', 'changes', 'solubility', 'occur', 'courses', 'various', 'reactions', 'such', 'changes', 'observed', 'instance', 'chlorination', 'polyethylene', 'aromatic', 'chlorinated', 'solvents', '29.', 'there', 'increase', 'solubility', '30', 'conversion', 'reached', 'after', 'solubility', 'decreases', 'reaches', 'minimum', '50–60', 'chlorine', 'content', 'following', 'increases', 'this', 'however', 'typical', 'many', 'reactions', 'polymers', 'solutions', 'more', 'common', 'starting', 'material', 'soluble', 'product', 'opposite', 'true', 'higher', 'conversions', 'usually', 'expected', 'polymers', 'solvated', 'chains', 'fully', 'extended', 'in', 'situations', 'reagents', 'ready', 'access', 'reactive', 'sites', '29.', 'if', 'products', 'insoluble', 'reaction', 'medium', 'tend', 'precipitate', 'reaction', 'progresses', 'sites', 'become', 'increasingly', 'less', 'accessible', 'this', 'result', 'low', 'conversions', 'premature', 'terminations', 'if', 'opposite', 'true', 'product', 'soluble', '91', 'reactivity', 'macromolecules', '571', 'starting', 'material', 'homogeneous', 'limited', 'reactions', 'controlled', 'when', 'starting', 'material', 'incompatible', 'product', 'mutual', 'precipitation', 'coiling', 'chains', 'take', 'place', 'this', 'also', 'result', 'limited', 'reactions', 'in', 'addition', 'minor', 'differences', 'constitutions', 'two', 'polymers', 'cause', 'incompatibility', 'for', 'instance', 'among', 'methacrylate', 'polymers', 'incompatibilities', 'benzene', 'solutions', 'result', 'differences', 'amount', 'branching', 'alkyl', 'groups', '29.', 'problems', 'solvent', 'incompatibility', 'sometimes', 'overcome', 'using', 'mixtures', 'solvents', 'those', 'good', 'starting', 'materials', 'combined', 'good', 'products', 'with', 'careful', 'experimentation', 'may', 'possible', 'develop', 'mixture', 'solvents', 'keep', 'components', 'solution', '30.', 'in', 'instances', 'however', 'insolubility', 'products', 'might', 'advantage', 'this', 'case', 'alcoholysis', 'reactions', 'poly(vinyl', 'acetate)', 'polymer', 'precipitates', 'reactions', 'absorbs', 'catalyst', 'the', 'phenomenon', 'permits', 'complete', 'alcoholysis', 'particularly', 'higher', 'molecular', 'weight', 'species', 'precipitate', 'ﬁrst', 'secondary', 'reactions', 'like', 'cross-linking', 'gelation', 'result', 'precipitations', 'solution', 'the', 'extent', 'reactions', 'however', 'necessarily', 'limited', 'diffusions', 'low', 'molecular', 'weight', 'species', 'still', 'possible', 'isolation', 'useful', 'products', 'however', 'often', 'becomes', 'difﬁcult', '915', 'effects', 'crystallinity', 'crystallinity', 'affect', 'reactivity', 'reactions', 'carried', 'polymers', 'solid', 'state', 'heterogeneous', 'conditions', 'the', 'differences', 'accessibility', 'reactive', 'sites', 'vary', 'amount', 'crystallinity', 'cellulose', 'instance', 'often', 'reacted', 'solid', 'state', 'degree', 'crystallinity', 'expressed', 'terms', 'reactivity', 'various', 'reagents', '31.', 'the', 'progress', 'reaction', 'sometimes', 'monitored', 'loss', 'crystallinity', 'what', 'signiﬁcant', 'however', 'greater', 'accessibility', 'amorphous', 'regions', 'results', 'reaction', 'products', 'special', 'properties', 'an', 'example', 'heterogeneous', 'homogeneous', 'chlorination', 'polyethylene', 'two', 'different', 'products', 'obtained', '32.', 'the', 'material', 'heterogeneous', 'chlorination', 'much', 'less', 'randomly', 'substituted', 'remains', 'crystalline', 'chlorine', 'content', '55.', 'the', 'products', 'homogeneous', 'reactions', 'hand', 'amorphous', '35', 'substitution', '916', 'reactions', 'that', 'favor', 'large', 'molecules', 'hydrophobic', 'interactions', 'play', 'important', 'roles', 'many', 'polymeric', 'reactions', 'they', 'instance', 'signiﬁcant', 'hydrolyses', 'low', 'molecular', 'weight', 'esters', 'catalyzed', 'polymeric', 'sulfonic', 'acid', 'reagents', 'like', 'poly(styrene', 'sulfonic', 'acid)', 'in', 'reactions', 'hydrogen', 'ions', 'located', 'close', 'macromolecules', '19.', 'the', 'hydrolytic', 'cations', 'located', 'regions', 'macromolecules', 'bulk', 'solution', 'the', 'rates', 'reactions', 'high', 'low', 'molecular', 'weight', 'catalysts', 'hand', 'like', 'hcl', 'hydrogen', 'ions', 'distributed', 'evenly', 'throughout', 'reaction', 'medium', 'as', 'result', 'rates', 'lower', 'adsorption', 'ester', 'groups', 'polymeric', 'sites', 'accompanied', 'increase', 'apparent', 'rate', 'constant', 'compared', 'reactions', 'hcl', '572', '9', 'organic', 'reactions', 'polymers', 'examples', 'hydrolyses', 'methyl', 'butyl', 'acetates', '19.', 'another', 'example', 'formation', 'eximers', 'exiplexes', 'polyesters', 'methacrylate', 'polymers', 'always', 'favor', 'large', 'molecules', 'small', 'ones', '33.', 'proton', 'transfer', 'reaction', 'poly(vinyl', 'quinoline)', '34', 'serve', 'third', 'example', 'the', 'emission', 'excitation', 'absorption', 'spectra', 'polymer', 'mixture', 'dioxane', 'water', 'compared', '2-methylquinoline', 'the', 'emissions', 'coming', 'protonated', 'heterocy-', 'clic', 'rings', 'polymer', 'occur', 'sooner', 'low', 'molecular', 'weight', 'compound', '34.', '92', 'addition', 'reactions', 'polymers', 'double', 'bonds', 'backbones', 'pendant', 'groups', 'undergo', 'numerous', 'addition', 'reactions', 'some', 'discussed', 'section', '921', 'halogenation', 'hydro', 'chlorination', 'natural', 'rubber', 'often', 'accompanied', 'cyclization', '35', '36:', 'h', 'cl', 'h', 'cl', 'trans-14', '12-polybutadiene', 'hydro', 'halogenated', 'mild', 'conditions', 'gaseous', 'hcl', '51.', 'the', 'true', 'copolymers', 'butadiene', 'piperylene', 'also', 'isotactic', 'trans-14-', 'piperylene', 'the', 'addition', 'hcl', 'asymmetric', 'double', 'bond', 'trans', 'polypiperylene', 'occurs', 'stereoselective', 'way', 'judging', '13c', 'nmr', '51', 'spectra', 'polysilanes', 'alkene', 'substituents', 'add', 'hcl', 'hbr', 'presence', 'lewis', 'acids', '58.', 'the', 'products', 'corresponding', 'chlorine', 'bromine', 'containing', 'polymers', 'little', 'degradation', 'polysilane', 'backbone', 'r', 'si', 'si', 'hx', 'lewis', 'acid', 'r', 'si', 'si', 'x', '92', 'addition', 'reactions', '573', 'chlorinations', 'rubber', 'however', 'fairly', 'complex', 'several', 'reactions', 'occur', 'simultaneously', 'these', 'appear', '(2)', 'substitutions', '(3)', 'cyclizations', '(4)', 'cross-linkings', 'as', 'result', 'additions', 'halogens', 'double', 'bonds', 'minor', 'portion', 'overall', 'reaction', 'scheme', '37', '38.', 'in', 'ccl4', 'following', 'steps', 'known', 'occur', '(1)', 'additions', 'double', 'bond', 'cl2', 'substitution', 'cl', 'cl2', 'substitution', 'cl', 'hcl', 'hcl', 'cl', 'cl', 'cl', 'cycliz', 'addition', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'halogenation', 'reactions', 'unsaturated', 'polymers', 'follow', 'two', 'simultaneous', 'paths', 'ionic', 'free', 'radical', 'ionic', 'mechanisms', 'give', 'soluble', 'products', 'chlorination', 'reactions', 'polybutadiene', '42.', 'the', 'free-radical', 'mechanisms', 'hand', 'cause', 'cross-linking', 'isomerization', 'addition', 'products', 'if', 'free-radical', 'reactions', 'suppressed', 'soluble', 'materials', 'form', 'natural', 'rubber', 'chlorinated', 'benzene', 'however', 'addition', 'much', '30', 'weight', 'chlorine', 'without', 'cyclization', '39', '40.', 'also', 'chlorination', 'polyalkenamers', 'cis', 'trans', 'yields', 'soluble', 'polymers', 'x-rays', 'show', 'products', 'partly', 'crystalline', '43', '44.', 'the', 'crystalline', 'segments', 'obtained', '14-trans', 'polyisoprene', 'diisotactic', 'poly(erythro', 'dichlorobutamer)s', 'obtained', '14-cis', 'isomer', 'diisotactic', 'poly(threo', '12-dichlorobutamer)s', '45.', 'additive', 'type', 'chlorination', 'natural', 'rubber', 'also', 'carried', 'phenyl', 'iododichloride', 'sulfuryl', 'chloride', '39', '40.', 'traces', 'peroxides', 'must', 'present', 'initiate', 'reactions', 'this', 'suggests', 'free-radical', 'mechanism', 'some', 'cyclization', 'accompanies', 'reaction', 'well', '40.', 'in', 'ccl4', 'ﬁrst', '25', 'chlorine', 'atoms', 'add', 'per', '100', 'isoprene', 'units', '23', 'double', 'bonds', 'disappear', 'small', 'quantity', 'hcl', 'forms', 'subsequent', '105', 'chlorinations', 'however', 'cause', 'loss', '53', 'double', 'bonds', 'rubber', 'brominated', '30(cid1)c', 'if', 'traces', 'alcohol', 'present', 'reaction', 'appears', 'go', 'entirely', 'addition', '39', '40.', 'without', 'alcohol', 'substitutions', 'take', 'place', 'rapidly', 'simultaneously', 'additions', 'double', 'bonds', '41.', 'exomethylene', 'groups', 'intramolecular', 'cyclic', 'structures', 'form', 'process', 'slow', 'additions', 'bromine', 'vinylidene', 'double', 'bonds', 'result', 'formations', 'tri', 'bromides', '–c5h7br3', 'also', 'cis', 'trans', 'isomers', 'polyisoprene', '41', 'brominate', 'differently', 'substitution', 'reactions', 'take', 'place', 'brominations', 'n-bromosuccinimide', 'they', 'accompanied', 'cyclizations', '39.', 'brominations', 'polybutadienes', 'n-bromosuccinimide', 'yield', 'a-brominated', 'polybutadienes', '46', '47', 'may', 'also', 'contain', 'butane', 'diylidene', 'units', 'the', 'products', 'act', 'typical', 'alkyl', 'halides', 'undergo', 'grignard-wurz', 'reactions', '574', '9', 'organic', 'reactions', 'polymers', 'n', 'br', 'o', 'o', 'br', 'mgbr', 'the', 'bromination', 'reaction', 'accompanied', 'shifts', 'double', 'bonds', 'coupled', 'sites', 'substitution', 'several', 'different', 'substituents', 'form', 'the', 'polymers', 'may', 'contain', 'pentane', 'diylidene', 'hexane', 'diylidene', 'heptane', 'diylidene', 'units', '46', '47.', 'by', 'contrast', 'chlorination', 'polybutadiene', 'benzene', 'straightforward', 'addition', 'reaction', 'halogens', 'double', 'bonds', '48', '49:', 'cl2', 'n', 'cl', 'cl', 'n', 'cl', 'n', 'cl', 'very', 'little', 'hcl', 'liberated', 'double', 'bonds', 'consumed', 'when', 'ccl4', 'used', 'place', 'benzene', 'substitutions', 'occur', 'latter', 'stages', 'reactions', 'if', 'cross-linking', 'occurs', 'time', 'substitutions', 'may', 'extensive', 'the', 'cross-linking', 'reactions', 'believed', 'involve', 'carbocationic', 'intermediates', 'polybutadiene', 'halogenated', 'readily', 'tetrahydrofuran', 'iodine', 'chloride', 'bromine', '49.', 'the', 'products', 'glassy', 'polymers', 'these', 'products', 'dehalogenate', 'reactions', 'organolithium', 'compounds', 'like', 'n-butyllithium', 'sec-butyllithium', 'polystyryllithium', 'tetrahydrofuran', 'solution', 'dehalogenation', 'poly(iodo-chlorinated', 'butadiene)', 'n-butyllithium', 'yields', 'product', 'different', 'cistrans', 'ratios', 'also', 'accompanied', 'partial', 'cross-linking', 'the', 'reactions', 'may', 'involve', '49', 'halogen-metal', 'exchanges', 'followed', 'intra-', 'intermolecular', 'elimination', 'lithium', 'halide', 'in', 'brominations', 'polybutadienes', 'couplings', 'eliminations', 'take', 'place', 'both', 'iodo-chlorinated', 'brominated', 'polybutadienes', 'form', 'graft', 'copolymers', 'reacted', 'polystyryllithium', 'tetrahydrofuran', '50.', 'gel', 'formation', 'however', 'accompanies', 'grafting', 'reaction', '922', 'hydrogenation', 'atactic', '14-polybutadiene', 'syndiotactic', '12-polybutadiene', 'hydrogenated', '100(cid1)c', '50', 'bar', 'pressure', 'hydrogen', 'soluble', 'catalyst', '[(ph)3-p3rhcl.', 'complete', 'saturation', 'double', 'bonds', 'results', '52.', 'butadiene', 'acrylonitrile', 'copolymers', 'also', 'hydrogenated', 'quantitatively', '92', 'addition', 'reactions', '575', 'rhodium', 'catalyst', 'mild', 'conditions', '53.', 'the', 'kinetics', 'consistent', 'mechanism', 'active', 'rh-catalyst', 'interacts', 'unsaturation', 'polymer', 'rate-determining', 'step', 'the', 'nitrile', 'group', 'however', 'appears', 'also', 'interact', 'catalyst', 'inhibit', 'rates', '53.', 'hydrogenation', 'carbon-to-nitrogen', 'double', 'bonds', 'polymer', 'backbones', 'pendant', 'groups', 'carried', 'lithium', 'borohydride', '86:', 'n', 'lialh4', 'thf', 'reflux', 'n', 'n', 'n', 'n', 'n', 'it', 'reported', '485', 'syndiotactic', 'polystyrene', 'hydrogenated', 'nisio2', 'pdbaso4', 'catalysts', 'the', 'ni', 'catalyst', 'yields', 'complete', 'hydrogenenation', 'low', 'molecular', 'weight', 'polymer', 'used', 'hydrogenation', 'high', 'molecular', 'weight', 'polystyrene', 'however', 'incomplete', 'on', 'hand', 'pd', 'catalyst', 'yields', 'completely', 'hydrogenating', 'material', 'the', 'hydrogenated', 'syndiotactic', 'polystyrene', 'crystalline', 'material', 'good', 'heat', 'resistance', '485.', '923', 'addition', 'carbenes', 'polyisoprenes', 'polybutadienes', 'also', 'modiﬁed', 'reactions', 'carbenes', 'dichlorocarbene', 'adds', 'natural', 'rubber', 'dissolved', 'chloroform', 'phase', 'transfer', 'reaction', 'aqueous', 'naoh', '54.', 'a', 'phase', 'transfer', 'reagent', 'must', 'used', 'aqueous', 'naoh', 'solid', 'sodium', 'hydroxide', 'used', 'without', 'phase', 'transfer', 'reagent', 'there', 'evidence', 'cis–trans', 'isomerization', 'distribution', 'substituents', 'random', '54.', 'diﬂuorocarbene', 'generated', 'mild', 'neutral', 'conditions', 'adds', '14-cis-', '14-trans-', 'polybutadienes', 'give', 'materials', 'containing', 'cyclopropane', 'groups', '55.', 'the', 'addition', 'takes', 'place', 'randomly', 'give', 'atactic', 'stereo', 'sequence', 'distributions', '55:', 'f', 'f', 'n', 'cf2', 'n', 'fluorocarbene', 'formed', 'phenyl(ﬂuorodichloromethyl)mercury', 'thermolysis', 'situ', 'also', 'adds', '14', 'cis-', 'trans-polybutadienes', 'the', 'carbene', 'add', 'various', 'levels', '57.', 'the', 'addition', '576', '9', 'organic', 'reactions', 'polymers', 'stereospeciﬁc', 'preserves', 'alkene', 'geometry', 'parent', 'polybutadiene', 'also', 'addition', 'random', 'showing', 'reactivity', 'double', 'bonds', 'independent', 'sequence', 'environment', '57.', 'dichlorocarbene', 'generated', 'situ', 'organomercury', 'precursor', 'phenyl(bromo', 'dichloromethyl)mercury', '(seyferth', 'reagent)', 'adds', 'polybutadiene', 'similar', 'manner', '56.', 'the', 'reactions', 'take', 'place', 'homogeneous', 'conditions', 'they', 'carried', '1', 'solutions', 'polymer', 'benzene', 'using', '10–20', 'mol', 'excess', 'reagent', '924', 'electrophilic', 'additions', 'aldehydes', 'these', 'additions', 'double', 'bonds', 'like', 'prins', 'reaction', 'carried', 'natural', 'synthetic', 'rubbers', '59', '60.', 'they', 'take', 'place', 'rapidly', 'presence', 'acid', 'catalysts', 'aqueous', 'formaldehyde', '61,', 'paraform', 'ccl4', '62,', 'used', 'the', 'catalysts', 'inorganic', 'acids', 'anhydrous', 'lewis', 'acids', 'like', 'boron', 'triﬂuoride', 'acetic', 'acid', 'solution', '63:', 'oh', 'oh', 'oh', 'cl', 'the', 'reaction', 'takes', 'different', 'path', 'absence', 'catalyst', '62:', 'cl', 'o', 'o', 'oh', 'oh', 'the', 'products', 'prins', 'reaction', 'rubbers', 'thermoplastic', 'polymers', 'possess', 'fair', 'resistance', 'acids', 'bases', 'free', 'hydroxyl', 'groups', 'products', 'available', 'cross-linking', 'diisocyanates', '64', 'means', 'the', 'prins', 'reaction', 'carried', 'directly', 'rubber', 'latexes', '65.', 'it', 'also', 'possible', 'mill', 'rubber', 'together', 'formaldehyde', 'heat', 'resultant', 'mixture', 'presence', 'anhydrous', 'metal', 'chlorides', '64', 'get', 'similar', 'results', '66.', 'higher', 'aldehydes', 'also', 'react', 'natural', 'rubber', '67.', 'the', 'reaction', 'works', 'best', 'puriﬁed', 'rubber', 'additions', 'take', 'place', 'without', 'catalyst', '180(cid1)c', 'presence', 'alcl3–nacl', '120(cid1)c', 'these', 'reactions', 'carried', 'solid', 'phase', 'milling', 'rubber', 'aldehyde', 'like', 'glyoxal', '68.', 'heating', 'pressure', 'vessel', '175(cid1)c', 'required', 'complete', 'reaction', 'infra-red', 'spectra', 'products', 'reactions', 'solution', 'show', 'presence', 'ether', 'carbonyl', 'hydroxyl', 'groups', '69.', 'two', 'types', 'additions', 'appear', 'take', 'place', '69:', '92', 'addition', 'reactions', '577', '1', 'o', 'o', 'o', 'cho', 'ho', 'cho', '2', 'o', 'o', 'oh', 'o', 'products', 'reactions', 'rubber', 'glyoxal', 'strong', 'tendency', 'become', 'spontaneously', 'insoluble', 'this', 'probably', 'due', 'presence', 'residual', 'aldehyde', 'groups', 'treatment', 'product', '24-dinitrophenyl-hydrazine', 'eliminates', 'spontaneous', 'gelation', 'chloral', 'adds', 'polyisoprene', 'similarly', 'the', 'reaction', 'catalyzed', 'lewis', 'acids', '70.', 'both', 'alcl3', 'bf3', 'efﬁcient', 'catalysts', 'less', 'cross-linking', 'encountered', 'aluminum', 'chloride', 'infra-red', 'spectra', 'products', 'shows', 'presence', 'hydroxyl', 'groups', 'chlorine', 'atoms', 'vinylidene', 'unsaturation', '70.', '925', 'polar', 'additions', 'a', 'number', 'polar', 'additions', 'unsaturated', 'polymers', 'known', 'these', 'include', 'michael', 'addition', 'hydroboration', '13-dipolar', 'additions', 'ene', 'reaction', 'ritter', 'reaction', 'diels–alder', 'additions', 'others', '9251', 'michael', 'addition', 'among', 'polar', 'additions', 'unsaturated', 'polymers', 'reactions', 'amines', 'ammonia', 'unsaturated', 'polyesters', 'form', 'michael', 'condensation', 'thus', 'instance', 'additions', 'poly(16-hexanediol', 'maleate)', 'poly(16-hexanediol', 'fumarate)', '71', 'show', 'difference', 'reactivity', 'two', 'isomers', 'the', 'maleate', 'polyester', 'reacts', 'ammonia', 'yield', 'cross-linked', 'product', 'room', 'temperature', 'stoichiometric', 'quantities', 'excess', 'ammonia', 'alcohol', 'used', 'at', 'reaction', 'conditions', 'fumarate', 'isomer', 'adds', 'percent', 'ammonia', '71.', 'in', '11', 'mixture', 'chloroform', 'ethanol', 'however', 'approximately', 'half', 'fumarate', 'double', 'bonds', 'react', 'also', 'maleate', 'polyester', 'reacts', 'differently', 'piperidine', 'cyclohexylamine', 'in', 'butyl', 'alcohol', '60(cid1)c', 'polymer', 'initially', 'isomerizes', 'precipitates', 'after', 'isomerization', 'complete', 'temperature', 'raised', '80(cid1)c', 'polymer', 'redissolves', 'an', 'exothermic', 'reaction', 'follows', 'michael', 'type', 'adducts', 'form', '71.', '578', '9', 'organic', 'reactions', 'polymers', '9252', 'hydroboration', 'polymers', 'copolymers', 'butadiene', 'isoprene', 'styrene', 'react', 'diborane', '72.', 'a', 'suitable', 'solvent', 'reaction', 'tetrahydrofuran', 'subsequent', 'hydrolyses', 'result', 'introductions', 'hydroxyl', 'groups', 'polymer', 'backbones', 'the', 'reactions', 'diborane', 'rapid', 'some', 'side', 'reactions', 'however', 'also', 'occur', '72.', '9253', '13-dipolar', 'additions', 'cyclic', 'structures', 'form', 'polymer', 'backbones', '13-dipolar', 'additions', 'carbon', 'carbon', 'carbon', 'nitrogen', 'double', 'bonds', '73.', 'because', 'many', '13-dipoles', 'heteroatoms', 'additions', 'lead', 'formations', 'ﬁve-membered', 'heterocyclic', 'rings', 'an', 'example', 'addition', 'nitrilimine', 'unsaturated', 'polyesters', '73:', 'r', 'r', 'n', 'n', 'r', 'r', 'n', 'n', 'also', 'iodine', 'isocyanate', 'adds', 'polyisoprene', 'the', 'product', 'converted', 'methyliodo-', 'carbamate', 'iodourea', 'derivatives', '74:', 'i', 'n', 'o', 'ch3oh', 'o', 'n', 'i', '∆', 'h', 'n', 'o', 'o', 'o', 'o', 'n', 'i', 'h', '92', 'addition', 'reactions', '579', 'iodine', 'isocyanate', 'additions', 'result', 'approximately', '40', 'yields', 'the', 'products', 'undergo', 'typical', 'reactions', 'isocyanate', 'group', '74,', 'instance', 'well', 'n', 'i', 'o', 'n', 'i', 'o', 'hx', 'naoh', 'hx', 'nh2', 'nh', 'nh3', 'h2n', 'n', 'o', 'i', 'i', 'h', 'o', 'n', 'nh2', 'hx', 'halogen', 'acid', 'the', 'products', 'exhibit', 'enhanced', 'heat', 'stability', '74.', 'dipolar', 'cycloadditions', 'take', 'place', 'nitrones', 'nitrile', 'oxides', 'add', 'butadiene', 'rubber', 'some', 'products', 'contain', 'isoxazolidine', 'rings', '75:', 'r', 'o', 'n', 'oh', 'h', 'h2o', 'r', 'n', 'o', 'polybutadiene', 'r', 'o', 'n', 'n', 'cl', 'oh', '(c2h5)3n', '-hcl', 'polybutadiene', 'cn', 'o', 'o', 'n', '9', 'organic', 'reactions', 'polymers', '580', 'the', 'product', 'compound', '75.', 'the', 'modiﬁcation', 'butadiene', 'rubber', 'carried', 'extent', '31', 'mol%', 'higher', 'tensile', 'modulus', 'values', 'greater', 'strength', 'parent', 'a', 'ﬁnal', 'example', '13', 'dipolar', 'addition', 'pendant', 'azide', 'groups', '87.', 'the', 'reaction', 'takes', 'place', 'phenyl', 'vinyl', 'sulfoxide', 'dimethylformamide', 'forty-eight', 'hours', '110(cid1)c', 'required', 'azide', 'groups', 'become', 'undetectable', 'infra-red', 'spectroscopy', 'the', 'product', 'precipitates', 'addition', 'ether', '87:', 'o', 's', 'n3', 's', 'o', 'h', 'n', 'n', 'n', '9254', 'the', 'ene', 'reaction', 'the', 'polymers', 'conjugated', 'dienes', 'also', 'modiﬁed', 'via', 'ene', 'reactions', '76,', 'instance', 'x¼y', 'o¼n–', '–n¼n–', 'c¼s', 'c¼o', 'orc¼c.', 'an', 'example', 'addition', 'triazolidones', '76:', 'hx', 'y', 'r', 'n', 'o', 'o', 'n', 'n', 'h', 'x', 'y', 'o', 'o', 'n', 'n', 'n', 'r', 'this', 'results', 'formation', 'pendant', 'urazole', 'groups', 'the', 'exact', 'structure', 'products', 'however', 'fully', 'established', 'the', 'tensile', 'strength', 'polymers', 'improves', 'considerably', 'accompanied', 'dramatic', 'loss', 'molecular', 'weight', '76.', 'nevertheless', 'ene', 'reagents', 'like', 'c-nitroso', 'activated', 'azo', 'compounds', 'efﬁcient', 'adding', 'rubber', 'they', 'add', 'minutes', 'temperatures', '100', '140(cid1)c', 'in', 'case', 'azo', 'compound', 'addition', 'greater', '90.', '92', 'addition', 'reactions', '581', 'substituted', 'aryl', 'sulfonyl', 'azides', 'decompose', 'elevated', 'temperatures', 'nitrenes', 'add', 'natural', 'rubber', 'ar', 'n', 'n', 'n', '∆', 'o', 's', 'o', 'ar', 'n', 'n', 'o', 's', 'o', 'n', 'o', 's', 'ar', 'n', 'o', 'n', 'ar', 'o', 's', 'o', 'novel', 'types', 'polyamines', 'cationic', 'polyelectrolytes', 'form', 'polymers', 'conjugated', 'dienes', '77', 'reactions', 'carbon', 'monoxide', 'amines', 'water', '150(cid1)c', '1000–1500', 'psi', 'pressure', 'the', 'reaction', 'illustrated', 'follows', '77:', 'co', 'h', 'n', 'r', 'r', 'h2o', 'r', 'n', 'r', '9255', 'the', 'ritter', 'reaction', 'this', 'reaction', 'carried', 'natural', 'rubber', 'synthetic', 'polyisoprenes', '78:', 'h', 'rcn', 'n', 'r', 'h', 'o', 'the', 'carbon', 'cation', 'apparently', 'reacts', 'nucleophile', 'present', 'when', 'reaction', 'carried', 'dichloroacetic', 'acid', 'chlorine', 'atoms', 'detected', 'product', '78.', '582', '9', 'organic', 'reactions', 'polymers', '9256', 'diels–alder', 'condensations', 'crotonic', 'acid', 'esters', 'cellulose', 'undergo', 'addition', 'reaction', 'cycloaliphatic', 'amines', 'like', 'morpholine', 'piperidine', 'aliphatic', 'primary', 'amines', '79.', 'unsaturated', 'polymers', 'also', 'undergo', 'diels–alder', 'reactions', 'one', 'example', 'reaction', 'hexachlorocyclopentadiene', 'polycyclopentadiene', '80:', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'the', 'addition', 'takes', 'place', 'inert', 'atmosphere', '140–150(cid1)c', 'over', '90', 'mol%', 'conversion', 'achieved', '6', 'h', 'diels–alder', 'condensations', 'fumaric', 'maleic', 'acids', 'polyesters', 'various', 'dienes', '81', 'serve', 'another', 'example', 'these', 'reactions', 'require', '20', 'h', 'room', 'temperature', 'diels–alder', 'condensations', 'also', 'carried', 'polymers', '135', 'hexatriene', '135-heptatriene', '246-octatriene', '82.', 'sulfonate-substituted', 'maleic', 'anhydride', 'adds', 'low', 'functionality', 'hydrocarbon', 'elastomers', 'like', 'epdm', 'presumably', 'via', 'alder-ene', 'type', 'reaction', '83:', 'so3', 'so3h', 'polymer', 'polymer', 'so3h', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'thiols', 'add', 'diene', 'rubbers', 'free-radical', 'mechanism', '84.', 'thus', 'antioxidants', 'like', '4-(mercaptoacetamido)-diphenylamine', 'add', '–sh', 'groups', 'double', 'bonds', 'cis-polyisoprene', 'polybutadiene', 'presence', 'free-radical', 'initiators', '84.', 'thiol', 'compounds', 'also', 'add', 'photochemically', 'polymers', 'containing', 'double', 'bonds', 'for', 'instance', 'unsaturation', 'introduced', 'polyepichlorohydrin', 'partial', 'elimination', 'reaction', 'the', 'product', 'reacts', 'mercaptans', 'aided', 'photosensitizer', '(like', 'benzophenone)', 'ultraviolet', 'light', '85:', 'o', 'ch2cl', 'hnbp', 'hsr', 'o', 'o', 'ch2cl', 'o', 'o', 'ch2sr', 'ch2cl', 'o', '92', 'addition', 'reactions', '583', '9257', 'epoxidation', 'reactions', 'these', 'addition', 'reactions', 'unsaturated', 'polymers', 'like', 'liquid', 'polybutadiene', 'developed', 'preparations', 'useful', 'commercial', 'materials', '88–94.', 'patent', 'literature', 'describes', 'procedures', 'use', 'hydrogen', 'peroxide', 'presence', 'organic', 'acids', 'heavy', 'metal', 'salts', 'reaction', 'conditions', 'place', 'limitation', 'molecular', 'weights', 'polymers', 'easier', 'handle', 'lower', 'viscosity', 'solutions', 'a', 'modiﬁcation', 'procedures', 'use', 'peracetic', 'acid', 'place', 'hydrogen', 'peroxide', '95–97.', 'the', 'efﬁcient', 'methods', 'rely', 'upon', 'formations', 'organic', 'peracids', 'situ', 'cationic', 'exchange', 'resins', 'acting', 'catalysts', '98.', 'this', 'illustrated', 'follows', 'hooh', 'ch3cooh', 'h', 'o', 'ooh', 'o', 'ooh', 'h2o', 'o', 'ch3cooh', 'the', 'reaction', 'accompanied', 'formations', 'by-products', 'o', 'r-cooh', 'o', 'r', 'o', 'oh', 'h2o', 'polybutadienes', 'high', '14-structures', 'tend', 'epoxidize', 'readily', 'yield', 'less', 'viscous', 'products', '100', '101.', 'the', 'epoxidation', 'reaction', 'also', 'carried', 'poly(14-cyclopentadiene)', '99:', 'o', 'ooh', 'o', 'perbenzoic', 'acid', 'effective', 'reagent', 'chloroform', 'methylene', 'chloride', 'solutions', '0–20(cid1)c', 'the', 'conversions', 'high', 'yielding', 'brittle', 'materials', 'soluble', 'many', 'solvents', 'the', 'products', 'cast', 'transparent', 'ﬁlms', '99.', 'monoperphthalic', 'p-nitroperbenzoic', 'acids', 'also', 'efﬁcient', 'epoxidizing', 'agents', 'they', 'however', 'cause', 'cross-linking', 'case', 'epoxidation', 'polycyclopentadiene', '99.', 'the', 'products', 'react', 'like', 'typical', 'epoxy', 'compounds', '99:', 'ho', 'o', 'ho', 'o', 'ho', 'cl', 'hcl', '584', '9', 'organic', 'reactions', 'polymers', 'some', 'reactions', 'epoxy', 'groups', '99:', 'ho', 'cl', 'ho', 'cl', 'o', 'ho', 'o', 'o', 'ch2cl', 'clch2cooh', 'hoh', 'piperidine', 'ho', 'oh', 'ho', 'n', '93', 'rearrangement', 'reactions', 'there', 'different', 'types', 'polymeric', 'rearrangements', 'one', 'isomerizations', 'polymers', 'double', 'bonds', 'others', 'intramolecular', 'cyclizations', 'polymeric', 'backbone', '931', 'isomerization', 'reactions', 'the', 'isomerization', 'cis-polybutadiene', 'carried', 'aid', 'ultraviolet', 'light', 'gamma', 'radiation', 'when', 'light', 'used', 'free-radical', 'reaction', 'requires', 'photoinitiators', '102', '103.', 'the', 'mechanism', 'involves', 'freely', 'rotating', 'transitory', 'free-radicals', 'polymer', 'backbone', 'these', 'form', 'additions', 'photoinitiator', 'fragments', 'double', 'bonds', 'the', 'adducts', 'break', 'releasing', 'attached', 'initiator', 'fragments', 'reestablishing', 'double', 'bonds', 'the', 'new', 'conﬁgurations', 'trans', 'thermodynamically', 'stable', '102', '103.', 'this', 'shown', 'follows', 'x', 'cis', 'x', 'x', 'x', '93', 'rearrangement', 'reactions', '585', 'x•', 'represents', 'free-radical', 'fragment', 'photoinitiator', 'with', 'gamma', 'radiation', 'need', 'additives', '104', '105.', 'here', 'mechanism', 'isomerization', 'believed', 'involve', 'direct', 'excitation', 'p-electrons', 'double', 'bonds', 'anti-', 'bonding', 'orbitals', 'free', 'geometric', 'interconversions', 'possible', 'in', 'benzene', 'solutions', 'energy', 'transfers', 'take', 'place', 'excited', 'benzene', 'molecules', 'polymer', 'double', 'bonds', 'when', '14-polyisoprene', 'ﬁlms', 'irradiated', 'light', '122,', 'cis–trans', 'isomerizations', 'occur', 'in', 'process', 'quantities', '14-unsaturations', 'decrease', 'also', 'vinyl', 'vinylidene', 'double', 'bonds', 'cyclopropyl', 'groups', 'form', 'this', 'illustrated', 'follows', 'hn', 'hydrogen', 'shift', 'isomerization', '586', '9', 'organic', 'reactions', 'polymers', 'polybutadiene', 'also', 'isomerize', 'cis–trans', 'lose', 'unsaturation', 'irradiated', 'solid', 'state', 'this', 'must', 'done', 'vacuum', 'ultraviolet', 'light', '1236', '2537', 'a˚', '106', '107.', 'both', 'free-radical', 'ionic', 'mechanisms', 'suspected', 'operate', 'simultaneously', '932', 'cyclizations', 'intramolecular', 'rearrangements', 'cyclization', 'reactions', 'natural', 'rubber', 'polymers', 'conjugated', 'dienes', 'known', 'long', 'time', 'the', 'reactions', 'occur', 'presence', 'lewis', 'strong', 'protonic', 'acids', 'they', 'result', 'loss', 'elastomeric', 'properties', 'unsaturation', 'carbon', 'cations', 'form', 'intermediate', 'step', 'subsequent', 'formation', 'polycyclic', 'structures', '108', '109:', 'h', 'h', 'in', 'similar', 'manner', 'polymers', 'pendant', 'unsaturation', 'undergo', 'cyclization', 'reactions', 'benzene', 'presence', 'bf3', 'pocl3', 'yielding', 'ladder', 'structures', 'the', 'exact', 'nature', 'initiation', 'process', 'clear', 'water', 'may', 'needed', 'initiation', 'step', '110', '111:', '93', 'rearrangement', 'reactions', '587', 'the', 'reactions', 'result', 'formations', 'six-membered', 'monocyclic', 'fused', 'polycyclic', 'units', 'these', 'reactions', 'carbon', 'cations', 'also', 'lead', 'molecular', 'rearrangements', 'like', '12', 'shifts', 'protons', 'resulting', 'formations', 'ﬁve-membered', 'rings', 'spiro', 'cyclopentane', 'repeat', 'units', '111.', 'cyclization', 'reactions', 'polyisoprene', 'catalyzed', 'ticl4', '112', 'sulfuric', 'acid', '113–115', '117.', 'the', 'products', 'appear', 'similar', 'infra-red', 'spectra', '110', 'minor', 'differences', 'also', 'small', 'number', 'fused', 'rings', 'product', 'polyacrylonitrile', 'converts', 'red', 'solid', 'heated', '200(cid1)c', '116.', 'only', 'small', 'amount', 'volatile', 'material', 'given', 'further', 'heating', 'red', 'residue', '350(cid1)c', 'higher', 'converts', 'black', 'brittle', 'material', 'this', 'black', 'material', 'ladder', 'structure', 'n', 'n', 'n', 'n', 'n', 'n', 'heating', 'polyacrylonitrile', 'presence', 'oxygen', 'yields', 'quinone-type', 'structures', 'd', 'o', 'o', 'n', 'h', 'n', 'h', 'n', 'h', 'further', 'heating', 'polymer', 'high', 'temperatures', 'excess', '2000(cid1)c', 'results', 'formation', 'graphite-like', 'structures', '116.', 'all', 'almost', 'nitrogen', 'lost', 'probably', 'hcn', 'n2', 'migration', 'double', 'bonds', 'well-established', 'phenomenon', 'polymers', 'conjugated', 'dienes', 'it', 'occurs', 'instance', 'vulcanization', 'rubber', '(discussed', 'later', 'section)', 'it', 'also', 'occurs', 'upon', 'simply', 'heating', 'polymers', '117–119.', 'thus', 'double', 'bonds', 'shift', 'natural', 'rubber', 'heated', 'temperatures', '150(cid1)c', '120:', 'hydro', 'chlorination', 'rubber', 'solution', 'causes', 'different', 'kind', 'double', 'bond', 'shirt', '121:', 'd', 'o2', 'o', 'o', 'hcl', 'hcl', 'cl', '588', '9', 'organic', 'reactions', 'polymers', 'in', 'polymers', 'pendant', 'double', 'bonds', 'facile', 'cycloadditions', 'occur', 'upon', 'heating', '123–125:', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'many', 'polymeric', 'structures', 'rearrange', 'proper', 'conditions', 'for', 'instance', 'poly(440-', 'diphenylpropane', 'isophthalate)', 'rearranges', 'upon', 'irradiation', 'uv', 'light', 'structure', 'containing', 'o-hydroxybenzophenones', '126:', 'o', 'o', 'hn', 'o', 'o', 'o', 'oh', 'o', 'o', 'the', 'mechanism', 'believed', 'photo-induced', 'fries', 'rearrangement', 'it', 'may', 'similar', 'one', 'catalyzed', 'lewis', 'acids', 'another', 'example', 'spontaneous', 'rearrangement', 'solid', 'state', 'poly(a-phenylethylisonitrile)', '128:', 'n', 'n', 'based', 'studies', 'molecular', 'models', 'concluded', '128', 'substituents', 'force', 'progres-', 'sive', 'one-handed', 'twisting', 'helix', 'shape', 'molecules', 'this', 'occurs', 'extent', 'vicinal', 'imino', 'double', 'bonds', 'conjunction', 'as', 'result', 'benzylic', 'hydrogen', 'atom', 'becomes', 'localized', 'electrons', 'imino', 'groups', 'it', 'proceeds', 'constant', 'screw', 'direction', 'around', 'axis', 'helix', 'such', 'steric', 'conﬁnement', 'causes', 'tautomeric', 'rearrangements', 'shown', 'results', 'relaxation', 'compression', 'an', 'additional', 'driving', 'force', 'gain', 'conjugation', 'imino', 'double', 'bonds', 'aromatic', 'rings', 'substituent', 'intramolecular', 'rearrangements', 'polymers', 'ketone', 'groups', 'subjects', 'several', 'studies', 'the', 'schmidt', 'beckmann', 'rearrangements', 'carried', 'copolymers', 'ethylene', 'carbon', 'monoxide', '129:', 'hn3', 'h2so4', 'o', 'x', 'nh2oh', 'oh', 'n', 'x', 'h', 'n', 'x', 'h', 'n', 'x', 'o', 'o', '93', 'rearrangement', 'reactions', '589', 'the', 'starting', 'materials', 'low', 'medium', 'molecular', 'weight', 'copolymers', 'infra-red', 'spectra', 'products', 'schmidt', 'reaction', 'show', 'high', 'degree', 'conversion', '129.', 'the', 'yields', 'oximes', 'subsequent', 'beckmann', 'rearrangements', 'also', 'high', 'a', 'schmidt', 'reaction', 'poly(p-acetyl', 'styrene)', 'yields', 'product', 'containing', 'acetyl', 'amino', 'styrene', 'groups', 'high', 'yields', 'products', 'surprisingly', 'pure', '130.', 'the', 'solvent', 'acetic', 'acid', 'converted', 'material', 'precipitates', 'sticky', 'mass', 'hn3', 'h2so4', 'h', 'o', 'o', 'n', 'h', 'nh2', 'another', 'rearrangement', 'reaction', 'isomerization', 'unsaturated', 'polyesters', 'upon', 'heating', 'thus', 'instance', 'maleate', 'polyesters', 'rearrange', 'fumarate', 'analogs', 'elevated', 'temperatures', '131:', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'it', 'reported', 'polymers', 'cis-stilbene', 'moieties', 'backbone', 'undergo', 'photocylization', 'almost', 'quantitative', '473:', 'uv', 'light', 'nbu4f', '∆', 'hcl', 'ho', '590', '9', 'organic', 'reactions', 'polymers', '94', 'substitution', 'reactions', 'many', 'substitution', 'reactions', 'carried', 'polymers', 'order', 'replace', 'atoms', 'backbones', 'pendant', 'groups', 'atoms', 'groups', 'atoms', 'these', 'reactions', 'differ', 'much', 'small', 'molecules', '941', 'substitution', 'reactions', 'saturated', 'polymeric', 'hydrocarbons', 'it', 'often', 'desirable', 'replace', 'hydrogens', 'halogens', 'fluorination', 'polyethylene', 'carried', 'dark', 'simply', 'exposing', 'polymer', 'either', 'sheet', 'powder', 'form', 'ﬂuorine', 'gas', 'the', 'reaction', 'exothermic', 'best', 'dilute', 'gas', 'nitrogen', '91', 'allow', 'gradual', 'introduction', 'ﬂuorine', 'avoid', 'destruction', 'polymer', '132', '133.', 'in', 'manner', 'however', 'surface', 'layers', 'ﬂuorinated', 'substitutions', 'occur', 'molecular', 'layers', 'deep', 'in', 'surface', 'ﬂuorination', 'vacuum', 'ultraviolet', 'glow', 'discharge', 'plasma', 'photon', 'component', 'plasma', 'enhances', 'reactivity', 'polymer', 'surfaces', 'toward', 'ﬂuorine', 'gas', '134:', 'gas', 'phase', 'f2', 'þ', 'hnð210(cid3)360', 'nmþ', '2f(cid4)', 'solid', 'surfacech2', '−ch2', 'hν', '(160', 'nm)', 'ch2', 'ch', 'e', 'e', 'ch2', 'ch', 'the', 'surface', 'free-radicals', 'also', 'cause', 'elimination', 'hydrogen', 'radicals', 'formation', 'double', 'bonds', 'whether', 'free-radicals', 'unsaturation', 'units', 'reactive', 'toward', 'ﬂuorine', 'a', 'ﬁlm', '3', 'mil', 'thick', 'completely', 'ﬂuorinated', '100-mesh', 'phosphor', 'bronze', 'gauze', 'reaction', 'allowed', 'proceed', 'several', 'days', '135.', 'fluorination', 'also', 'carried', 'mercuric', 'cupric', 'ﬂuorides', 'hydroﬂuoric', 'acid', 'the', 'reaction', 'must', 'carried', '110(cid1)c', '50', 'h', 'as', 'much', '20', 'ﬂuorine', 'introduced', '136.', 'in', 'direct', 'ﬂuorination', 'powdered', 'high-density', 'polyethylene', 'gas', 'diluted', 'helium', 'nitrogen', 'accompanying', 'exotherm', 'causes', 'partial', 'fusion', 'in', 'addition', 'destruction', 'crystalline', 'regions', '137.', 'on', 'hand', 'ﬂuorination', 'single', 'crystals', 'polyethylene', 'result', 'ﬂuorine', 'atoms', 'placed', 'carbon', 'skeleton', 'without', 'disruption', 'crystal', 'structure', 'the', 'extent', 'cross-linking', 'however', 'hard', 'assess', '138.', 'the', 'reaction', 'characteristics', 'free-radical', 'mechanism', '139:', 'f2', '2f(cid4)', 'rh', 'þ', 'f(cid4)', 'r(cid4)', 'þ', 'hf', 'r(cid4)', 'þ', 'f2', 'rf', 'þ', 'f(cid4)', 'chlorinations', 'polyethylene', 'carried', 'dark', 'presence', 'light', 'the', 'two', 'reactions', 'however', 'different', 'though', 'take', 'place', 'free-radical', 'mechanism', 'when', 'carried', 'dark', '100(cid1)c', 'higher', 'catalyst', 'needed', 'probably', 'residual', 'peroxides', 'oxidation', 'starting', 'material', 'oxygen', 'must', 'excluded', 'inhibits', 'reaction', 'degrades', 'product', '140.', 'the', 'reaction', 'catalyzed', 'traces', 'ticl4', '141.', 'such', 'trace', 'quantities', '94', 'substitution', 'reactions', '591', 'even', 'residual', 'titanium', 'halide', 'ziegler-natta', 'catalyst', 'left', 'polymer', 'polymerization', 'reaction', 'when', 'carried', '50(cid1)c', 'chlorobenzene', '–chcl–', 'groups', 'form', '142:', 'cl2', 'this', 'slows', 'chlorination', 'adjacent', 'groups', 'trace', 'amounts', 'oxygen', 'catalyze', 'chlorinations', 'presence', 'visible', 'light', '140.', 'the', 'reaction', 'ultraviolet', 'light', 'accompanied', 'cross-linking', 'the', 'photochemical', 'process', 'illustrated', 'follows', 'cl', 'cl', 'hcl', 'cl', '1', 'cl2', 'hν', '2cl', '2', 'cl', '3', 'cl2', 'etc', 'chlorination', 'polyethylene', 'result', 'varying', 'amounts', 'hydrogen', 'atoms', 'replaced', 'chlorine', 'it', 'possible', 'form', 'product', 'contains', '70', 'weight', 'chlorine', 'the', 'amount', 'chlorination', 'affects', 'properties', 'product', 'at', 'low', 'levels', 'substitution', 'material', 'still', 'resembles', 'parent', 'compound', 'when', 'however', 'level', 'chlorine', 'reaches', '30–40,', 'material', 'becomes', 'elastomer', 'at', 'levels', 'exceeding', '40,', 'polymer', 'stiffens', 'becomes', 'hard', 'commercial', 'chlorinations', 'polyethylene', 'usually', 'conducted', 'high-density', '(d', '096)', 'linear', 'polymers', 'the', 'molecular', 'weights', 'starting', 'materials', 'vary', 'high', 'molecular', 'weight', 'polymers', 'form', 'tough', 'elastomers', 'low', 'molecular', 'weight', 'materials', 'however', 'allow', 'easier', 'processing', 'products', 'the', 'reactions', 'carried', 'carbon', 'tetrachloride', 'methylene', 'dichloride', 'chloroform', 'reﬂux', 'temperatures', 'solvents', 'pressures', 'atmospheric', 'overcome', 'poor', 'solubility', 'the', 'solubility', 'improves', 'degree', 'chlorination', 'industrially', 'two', 'different', 'procedures', 'used', 'in', 'ﬁrst', 'one', 'reactions', 'conducted', '95–130(cid1)c', 'when', 'chlorinations', 'reach', 'level', '15,', 'polymers', 'become', 'soluble', 'temperatures', 'lowered', 'considerably', '143.', 'in', 'second', 'one', 'reactions', 'conducted', 'polymers', 'suspended', 'solvent', 'when', 'chlorine', 'content', 'reaches', '40', 'polymers', 'become', 'soluble', 'chlorinations', 'continued', 'solution', 'by', 'continuing', 'reaction', 'chlorine', 'content', '60', 'reached', 'the', 'products', 'two', 'processes', 'differ', 'the', 'ﬁrst', 'one', 'yields', 'homogeneous', 'product', 'chlorine', 'atoms', 'distributed', 'uniformly', 'throughout', 'molecules', 'chlorination', 'suspension', 'hand', 'yields', 'heterogeneous', 'materials', 'segments', 'polymeric', 'molecules', 'chlorinated', 'some', 'commercial', 'chlorinations', 'conducted', 'water', 'suspensions', 'these', 'reactions', 'carried', '65(cid1)c', 'approximately', '40', 'levels', 'chlorine', 'achieved', 'the', 'temperatures', 'raised', '75(cid1)c', 'drive', 'conversions', 'in', 'procedures', 'agglomerations', 'particles', 'problem', 'to', 'overcome', 'water', 'usually', 'saturated', 'hcl', 'cacl2', '144.', 'problems', 'agglomeration', 'also', 'encountered', 'suspension', 'chlorinations', 'solvents', 'like', 'ccl4', 'infra-red', 'spectra', 'chlorinated', 'polyethylenes', 'show', 'presence', 'various', 'forms', 'substitutions', 'there', '–chcl–chcl–', 'well', '–ccl2–', 'groups', 'present', 'materials', '145', '146.', 'surface', 'photo', 'chlorination', 'polyoleﬁn', 'ﬁlms', '146', 'considerably', 'improves', 'barrier', 'properties', 'ﬁlms', 'permeations', 'gases', 'chlorinations', 'polypropylene', 'usually', 'result', 'severe', 'degradations', 'polymer', 'when', 'ticl4', 'chlorination', 'catalyst', 'presumably', 'less', 'degradation', 'occurs', '140.', 'studies', 'bromination', 'polypropylene', '(atactic)', 'show', 'reaction', 'carried', 'dark', 'ccl4', '60(cid1)c', 'substitution', 'reactions', 'proceed', 'rate', '05/h', '147.', '592', '9', 'organic', 'reactions', 'polymers', 'chlorosulfonation', 'polyethylene', 'commercial', 'process', 'the', 'reaction', 'resembles', 'chlorination', 'step', 'hydrogen', 'abstraction', 'chlorine', 'radicals', 'it', 'catalyzed', 'pyridine', '148–150', 'pictured', 'follows', '147:', 'cl', 'so2', 'hcl', 'so2cl', 'so2', 'cl', 'cl', 'cl', 'cl2', 'cl2', 'the', 'amount', 'so2', 'vs', 'chlorine', 'reaction', 'mixture', 'affects', 'resultant', 'ratios', 'chlorosul-', 'fonation', 'vs', 'chlorination', 'polymer', 'these', 'ratios', 'amounts', 'conversion', 'vary', 'temperature', '149.', 'commercially', 'produced', 'chlorosulfonated', 'polyethylene', 'contains', 'approximately', '26–29', 'chlorine', '12–17', 'sulfur', '151.', 'the', 'material', 'elastomer', 'remains', 'ﬂexible', '(cid3)50(cid1)c', 'it', 'commonly', 'cross-linked', '(vulcanized)', 'sulfonyl', 'chloride', 'groups', 'heating', 'either', 'solid', 'solution', '150(cid1)c', 'results', 'loss', 'so2', 'hcl', 'if', 'material', 'heated', '2', 'h', '175(cid1)c', 'so2cl', 'groups', 'removed', 'in', 'gamma', 'radiation-induced', 'chlorosulfonations', 'sulfoxidations', 'polyethylene', 'powders', '153', 'room', 'temperature', 'ratios', 'so2cl', 'groups', 'cl', 'groups', 'decrease', 'increases', 'radiation', 'polypropylene', 'chlorosulfonated', 'extent', 'containing', '6', 'chlorine', '14', 'sulfur', 'without', 'embrittlement', 'the', 'reaction', 'done', 'ccl4', '55(cid1)c', 'there', 'apparently', 'less', 'degrada-', 'tion', 'direct', 'chlorination', 'reaction', '148', '152.', '942', 'substitution', 'reactions', 'halogen-bearing', 'polymers', 'procedures', 'commercial', 'chlorinations', 'poly(vinyl', 'chloride)', 'vary', 'low', 'temperature', 'chlorinations', 'done', 'aqueous', 'dispersions', 'polymers', 'reacted', 'chlorine', 'gas', 'presence', 'swelling', 'agents', 'like', 'chloroform', 'these', 'light', 'catalyzed', 'reactions', 'usually', 'carried', '50(cid1)c', 'they', 'result', 'substitutions', 'methylene', 'hydrogens', '158', '159.', 'cl', 'cl', 'hν', 'cl2', 'hcl', 'cl', 'some', 'breakdown', 'polymers', 'accompanies', 'reactions', 'suspension', '154.', 'irregularity', 'structures', 'release', 'hcl', 'signiﬁcantly', 'contribute', 'it', 'appears', 'degradations', 'initiated', 'hcl', 'released', 'result', 'chlorination', 'in', 'process', 'double', 'bonds', 'form', 'immediately', 'upon', 'formation', 'become', 'saturated', 'chlorines', 'a', 'study', 'conducted', 'solution', 'chlorination', 'poly(vinyl', 'chloride)', 'presence', 'free', 'radicals', '155', 'generated', 'azobisisobutyronitrile', 'the', 'reaction', 'appears', 'proceed', 'two', 'stages', 'the', 'ﬁrst', 'one', 'takes', 'place', '60', 'ch2', 'groups', 'reacted', 'after', 'second', 'stage', 'original', '–chcl', 'groups', 'attacked', 'some', 'unreacted', '–ch2–', 'groups', 'however', 'remain', '155.', '94', 'substitution', 'reactions', '593', 'photo', 'chlorination', 'achieve', '90', 'conversion', '156', '157.', 'typical', 'commercially', 'chlorinated', 'poly', '(vinyl', 'chloride)', 'however', 'chlorine', 'content', '66–67.', 'the', 'material', 'contains', 'methylene', 'groups', 'effect', 'copolymer', 'vinyl', 'chloride', '12-dichloroethylene', 'chlorination', 'raises', 'softening', 'temperature', 'poly(vinyl', 'chloride)', 'the', 'products', 'exhibit', 'poorer', 'heat', 'stability', 'parent', 'material', 'higher', 'melt', 'viscosity', 'high-temperature', 'chlorinations', 'chlorinated', 'solvents', '100(cid1)c', 'result', 'extensive', 'substitutions', 'methylenic', 'hydrogens', 'well', 'the', 'reactions', 'however', 'accompanied', 'extensive', 'chain', 'scissions', 'the', 'products', 'soluble', 'solvents', 'like', 'acetone', 'methylene', 'chloride', 'low', 'softening', 'points', 'low', 'impact', 'strength', 'poor', 'color', 'stability', 'chlorination', 'poly(vinylidene', 'chloride)', 'carried', 'sulfuryl', 'chloride', 'using', 'azobisiso-', 'butyronitrile', '155.', 'the', 'reaction', 'appears', 'proceed', 'three', 'steps', '1', 'formation', 'so2cl(cid129)', 'free', 'radicals', 'abstraction', '2', 'formation', 'free', 'radicals', 'polymeric', 'backbones', 'cl', 'cl', 'cl', 'cl', 'so2cl', 'cl', 'cl', 'cl', 'cl', '3', 'transfer', 'reactions', 'active', 'sites', 'cl', 'cl', 'cl', 'cl', 'so2cl2', 'so2cl', 'cl', 'cl', 'cl', 'cl', 'cl', 'chlorination', 'poly(vinyl', 'ﬂuoride)', 'yields', 'product', '40–50', 'chlorine', 'content', '158.', 'fluorination', 'poly(vinylidene', 'ﬂuoride)', 'reported', '160.', 'when', 'mixtures', 'ﬂuorine', 'nitrogen', 'gases', 'used', 'reactions', 'limited', 'amount', 'diffusion', 'ﬂuorine', 'polymer', 'network', 'x-ray', 'photoelectron', 'spectroscopy', 'shows', 'presence', '–cf2–', '–chf–', '–ch2–', 'groups', 'product', '161.', 'many', 'attempts', 'modiﬁcations', 'poly(vinyl', 'chloride)', 'reported', 'literature', 'often', 'reactions', 'based', 'expectations', 'polymers', 'react', 'like', 'typical', 'alkyl', 'halides', 'unfortunately', 'place', 'nucleophilic', 'substitutions', 'polymers', 'often', 'undergo', 'rapid', 'sequential', 'eliminations', 'hcl', 'along', 'chains', 'nevertheless', 'many', 'substitution', 'reactions', 'still', 'possible', 'successfully', 'carried', 'one', 'example', 'replacement', '43', 'chlorine', 'atoms', 'azide', 'groups', '162:', 'cl', 'cl', 'cl', 'n3', 'nan3', 'the', 'reaction', 'carried', '60(cid1)c', '10', 'h', 'polymer', 'cross-link', 'it', 'proceeds', 'faster', 'dmf', 'solvents', 'once', 'substituted', 'polymer', 'becomes', 'photosensitive', '594', '9', 'organic', 'reactions', 'polymers', 'cross-links', 'irradiated', 'ultraviolet', 'light', 'the', 'presence', 'azide', 'groups', 'allows', 'modiﬁcations', 'one', 'modiﬁcation', 'reaction', 'isocyanate', '162:', 'cl', 'n3', 'cl', 'n', 'n', 'n', 'o', 'by', 'similar', 'reaction', 'phosphinimine', 'groups', 'formed', 'polymer', 'backbone', '162:', 'cl', 'n3', 'cl', 'n3', 'p', '3', 'mgbr', 'some', 'modiﬁcations', 'azide', 'group', 'containing', 'polymers', '163', '164:', '3', 'cl', 'n', 'p', 'cl', 'n', 'n', 'n', 'h', 'cl', 'n3', 'lialh4', 'cl', 'n', 'n', 'n', 'cl', 'nh2', 'note', 'the', 'illustration', 'reactions', 'lialh4', 'shown', 'implies', 'chloride', 'atoms', 'remain', 'intact', 'polymer', 'backbones', 'it', 'appears', 'likely', 'however', 'might', 'get', 'removed', 'double', 'bonds', 'might', 'form', 'instead', 'backbone', '94', 'substitution', 'reactions', '595', 'poly(vinyl', 'chloride)', 'reacts', 'various', 'thiols', 'ethylene', 'diamine', 'produce', 'monosulﬁde', 'derivatives', '165:', 'rsh', 'hn2', 'cl', 'cl', 'nh2', 'cl', 'sr', 'when', 'sulfur', 'added', 'reaction', 'mixture', 'disulﬁdes', 'form', 'instead', '165:', 'rsh', 's', 'h2n', 'nh2', 'cl', 'cl', 'cl', 's', 'r', 's', 'the', 'thiolate', 'ions', 'serve', 'strong', 'nucleophiles', 'also', 'weak', 'bases', '166.', 'when', 'poly(vinyl', 'chloride)', 'suspended', 'water', 'presence', 'swelling', 'agents', 'phase', 'transfer', 'catalysts', 'reacts', 'mercaptans', 'like', '3-n-(2-pyridyl)carbamoyl-propylthiol', 'hs', 'cl', 'o', 'n', 'n', 'h', 'cl', 's', 'o', '3', 'n', 'h', 'n', 'acetoxylation', 'poly(vinyl', 'chloride)', 'carried', 'homogeneous', 'conditions', '167.', 'crown', 'ethers', 'like', '18-crown-6', 'solubilize', 'potassium', 'acetate', 'mixtures', 'benzene', 'tetrahydrofuran', 'methyl', 'alcohol', 'generate', 'unsolvated', 'strongly', 'nucleophilic', '“naked”', 'acetate', 'anions', 'these', 'react', 'readily', 'polymer', 'mild', 'conditions', '167.', 'substitutions', 'chlorine', 'atoms', 'polymeric', 'backbones', 'anionic', 'species', 'take', 'place', 'sn2', 'mechanism', 'the', 'reactions', 'also', 'proceed', 'sn1', 'mechanism', 'that', 'however', 'requires', 'formations', 'cationic', 'centers', 'backbones', 'rate-determining', 'step', 'substitutions', 'competition', 'elimination', 'reactions', 'it', 'conceivable', 'anionic', 'species', 'may', '(depending', 'upon', 'basicity)', 'also', 'facilitate', 'elimination', 'reactions', 'without', 'undergoing', 'substitutions', '167.', 'reactions', 'poly(vinyl', 'chloride)', 'aromatic', 'amines', 'amino', 'alcohols', 'aliphatic', 'amines', 'dmf', 'solution', 'result', 'substitutions', 'eliminations', '168.', 'reactions', 'aniline', 'yield', 'following', 'structure', '168:', 'nh2', 'n', 'cl', 'nh2', 'nh2', '596', '9', 'organic', 'reactions', 'polymers', 'carbanionic', 'reaction', 'sequences', 'used', 'introduce', 'various', 'photosensitive', 'groups', '169.', 'the', 'process', 'consists', 'creating', 'carbanionic', 'centers', 'backbone', 'reacting', 'various', 'halogenated', 'derivatives', 'li', 'cl', 'li', 'cl', 'cl', 'na', 'na', 'cl', 'na', 'cl', 'na', 'cl', 'o', '94', 'substitution', 'reactions', '597', 'cl', 'o', 'cl', 'o', 'a', 'redox', 'cyclopentadienyl', 'iron', 'moiety', 'also', 'introduced', 'poly(vinyl', 'chloride)', 'backbone', 'similar', 'technique', '170.', 'many', 'attempts', 'reported', 'replacing', 'halogens', 'poly(vinyl', 'chloride)', 'poly(vinyl', 'bromide)', 'poly(vinyl', 'iodide)', 'alkali', 'metal', 'hydrogen', 'for', 'instance', 'effort', 'form', 'poly(vinyl', 'lithium)', 'polymers', 'reacted', 'organolithium', 'compounds', 'metallic', 'lithium', 'the', 'reactions', 'alkyllithium', 'however', 'resulted', 'substitutions', 'alkyl', 'groups', 'similarly', 'reactions', 'shown', 'previously', '171:', 'c4h9li', 'br', 'libr', 'c4h9', 'reactions', 'metallic', 'lithium', 'lead', 'formations', 'polyenes', '171.', 'on', 'hand', 'poly(vinyl', 'chloride)', 'reacted', 'metal', 'hydrides', 'like', 'lithium', 'aluminum', 'hydride', 'mixture', 'tetrahydrofuran', 'decalin', '100(cid1)c', 'macroalkanes', 'form', '172:', 'lialh4', 'cl', 'licl', 'replacement', 'chlorine', 'nn-dialkyl', 'dithiocarbamate', 'reported', 'occur', '50–60(cid1)c', 'dmf', 'solvent', '173:', 's', 'r2', 'na', 'n', 's', 'cl', 's', 's', 'n', 'r2', 'the', 'reaction', 'catalyzed', 'ethylene', 'diamine', '174.', '943', 'substitution', 'reactions', 'polymers', 'aromatic', 'rings', 'there', 'interesting', 'reports', 'literature', 'reactions', 'carried', 'backbones', 'polystyrenes', 'there', 'also', 'many', 'reports', 'literature', 'aromatic', 'substitution', 'reactions', 'polystyrene', 'only', 'however', 'industrial', 'practice', '598', '9', 'organic', 'reactions', 'polymers', '9431', 'reactions', 'polystyrene', 'photo', 'chlorination', 'polystyrene', 'involves', 'replacement', 'hydrogens', 'b', 'positions', '175', '176.', 'it', 'believed', 'past', 'chlorine', 'atoms', 'react', 'preferentially', 'a-position', 'chlorine', 'content', 'product', 'reaches', '20', 'weight', 'after', 'thought', 'chlorines', 'introduced', 'position', 'later', 'however', 'contradicted', '177.', 'in', 'fact', 'polystyrene', 'photo-chlorinated', 'carbon', 'tetrachloride', 'low', 'temperatures', 'like', '13(cid1)c', 'substituted', 'equally', 'positions', 'at', 'higher', 'temperatures', 'like', '78(cid1)c', 'substitutions', 'b-position', 'actually', 'predominate', '177.', 'chlorinations', 'poly(p-methyl', 'styrene)', 'somewhat', 'selective', 'pendant', 'methyl', 'groups', 'result', 'di-', 'tri-substitutions', 'p-methyl', 'position', 'only', 'small', 'amounts', 'chlorine', 'introduced', 'polymer', 'backbones', '178.', 'substitutions', 'backbones', 'however', 'possible', 'use', 'so2cl2', 'chlorinating', 'agent', 'in', 'case', 'half', 'chlorines', 'still', 'replace', 'methyl', 'hydrogens', 'half', 'replace', 'hydrogens', 'backbone', 'free', 'radical', 'additions', 'maleic', 'anhydride', 'polystyrene', 'backbones', 'carried', 'help', 'either', 'peroxides', 'ultraviolet', 'light', '164.', 'approximately', '2', 'anhydride', 'introduced', 'if', 'however', 'additions', 'carried', 'a-brominated', 'polystyrene', 'anhydride', 'content', 'polymer', 'raised', '15', '164:', 'o', 'o', 'o', 'o', 'o', 'o', 'hν', 'note', 'extra', 'hydrogen', 'shown', 'maleic', 'anhydride', 'moiety', 'product', 'presumably', 'comes', 'chain', 'transferring', 'bromination', 'polystyrene', 'n-bromosuccinimide', 'benzoyl', 'peroxide', 'ccl4', 'room', 'temperature', 'achieve', '61', 'conversion', '4', 'h', 'considerable', 'degradation', 'however', 'accompanies', 'reaction', '180.', '9432', 'chloromethylation', 'reactions', 'chloromethylation', 'reactions', 'aromatic', 'rings', 'polystyrene', 'styrene', 'copolymers', 'carried', 'extensively', 'chlorodimethyl', 'ether', '(a', 'carcinogen)', 'good', 'solvent', 'polymers', 'it', 'therefore', 'commonly', 'employed', 'reagent', '181', '182.', 'laboratory', 'preparations', 'carried', 'mixtures', 'carbon', 'disulﬁde', 'ether', 'using', 'zinc', 'chloride', 'catalyst', 'a', '9-h', 'reaction', 'room', 'temperature', 'yields', '10', 'substitution', '183.', 'the', 'chloromethylation', 'process', '184', 'occurs', 'two', 'steps', 'benzyl', 'methyl', 'ether', 'forms', 'intermediate', 'cross-linking', 'reaction', 'aromatic', 'nuclei', 'formed', 'ch2cl', 'group', 'occurs', 'side', 'reaction', 'there', 'strong', 'indications', 'chloromethylation', 'takes', 'place', 'one', 'position', 'ring', '185.', 'the', 'true', 'bromomethylation', '185.', 'stannic', 'chloride', 'effective', 'catalyst', 'friedel-craft', 'reaction', '186.', 'iodomethylation', 'also', 'carried', 'manner', 'similar', 'results', '179.', 'when', 'reactions', 'carried', 'cross-linked', 'styrene', 'copolymers', 'chlorodimethyl', 'ether', 'stannic', 'chloride', 'catalyst', 'accompanied', 'strong', 'morphological', 'changes', '187.', 'if', 'reactions', 'carried', 'low', 'levels', 'chloromethylating', 'agents', 'catalysts', 'occur', 'less', 'homogeneously', 'larger', 'levels', '94', 'substitution', 'reactions', '599', 'either', 'however', 'result', 'greater', 'levels', 'chloromethylation', 'also', 'higher', 'degrees', 'secondary', 'cross-linkings', 'uneven', 'distributions', 'chloromethyl', 'groups', '188.', 'another', 'technique', 'chloromethylating', 'polystyrene', 'react', 'methylal', 'thionyl', 'chloride', 'presence', 'zinc', 'chloride', '189:', 'o', 'o', 'socl2', 'zncl2', '(ch3)2so3', 'ch2cl', 'chloromethyl-substituted', 'polystyrenes', 'also', 'prepared', 'poly(p-methyl', 'styrene)s', 'treating', 'aqueous', 'sodium', 'hypochlorite', 'presence', 'phase', 'transfer', 'catalysts', 'like', 'benzyltriethylammonium', 'chloride', '190.', 'the', 'conversions', 'methyl', 'chloromethyl', 'groups', 'high', '20', 'without', 'detectable', 'morphological', 'changes', '187.', 'if', 'reactions', 'carried', 'low', 'levels', 'chloromethylating', 'agents', 'catalysts', 'occur', 'less', 'homogeneously', 'larger', 'levels', 'either', 'however', 'result', 'greater', 'levels', 'chloromethylation', 'also', 'higher', 'degrees', 'secondary', 'cross-linkings', 'uneven', 'distributions', 'chloromethyl', 'groups', '188.', 'another', 'technique', 'chloromethylating', 'polystyrene', 'react', 'methylal', 'thionyl', 'chloride', 'presence', 'zinc', 'chloride', '189.', 'techniques', 'chloromethylating', 'polyarylether', 'sulfones', 'polyphenylene', 'oxide', 'phenolic', 'resins', 'model', 'compounds', 'described', 'recently', '191.', 'when', 'subsequent', 'products', 'converted', 'quater-', 'nary', 'amines', 'decrease', 'quaternization', 'rate', 'increase', 'degree', 'substitution', 'this', 'may', 'due', 'steric', 'effects', 'imposed', 'restricted', 'rotation', 'polymeric', 'chains', '191.', 'this', 'phenomenon', 'observed', 'quaternization', 'poly(chloromethyl', 'styrene)', 'the', 'chloromethylation', 'reaction', 'polysulfone', 'chloromethyl', 'ether', 'catalyzed', 'stannic', 'chloride', 'illustrated', 'follows', 'o', 's', 'o', 'o', 's', 'o', 'o', 'o', 'ch2cl', 'o', 'o', 'ch2cl', 'o', 's', 'o', 'o', 's', 'o', 'vinyl', 'benzyl', 'chloride', 'monomer', 'available', 'commercially', 'it', 'possible', 'therefore', 'simply', 'prepare', 'chloromethylated', 'polystyrene', 'copolymers', 'monomer', 'without', 'chloromethylation', 'reactions', '9433', 'reactions', 'halomethylated', 'polymers', 'many', 'known', 'reactions', 'halomethyl', 'groups', 'polymers', 'possible', 'one', 'instance', 'convert', 'poly(chloromethyl', 'styrene)', 'poly(hydroxymethyl', 'styrene)', '183.', 'also', 'iodomethylated', 'polystyrene', 'treated', 'triethyl', 'phosphite', 'order', 'carry', 'arbuzov', 'reaction', '192:', '600', '9', 'organic', 'reactions', 'polymers', 'p(oc2h5)3', '-c2h5i', 'ch2i', 'p', 'i', 'o', 'p', '(oc2h5)3', '(oc2h5)2', 'chloromethylated', 'polystyrene', 'also', 'converted', 'phosphonium', 'salt', 'use', 'wittig', 'reaction', '193:', 'p', '3', 'ch2cl', 'rcho', 'ch2cl2', '50', 'aqnaoh', 'r', 'p', '3', 'the', 'product', 'reaction', 'chloromethylated', 'polystyrene', 'triphenylphosphine', 'also', 'convert', 'nucleophiles', '194.', 'in', 'addition', 'use', 'phase', 'transfer', 'catalyst', 'converts', 'soluble', 'chloromethylated', 'polystyrenes', 'phosphine', 'oxides', 'reactions', 'dioctylphosphine', 'serve', 'example', '195.', 'sometimes', 'phase', 'transfer', 'reactions', 'easier', 'carry', 'conventional', 'ones', 'this', 'case', 'witting', 'reaction', 'both', 'linear', 'cross-linked', 'chloromethylated', 'polystyrenes', 'react', 'smoothly', 'triphenylphosphine', 'give', 'derivatives', 'react', 'various', 'aldehydes', '196', '197.', 'phase', 'transfer', 'catalysts', 'also', 'used', 'carrying', 'nucleophilic', 'substitutions', 'aid', 'sulﬁdes', 'like', 'tetrahydrothiophine', '198:', 'nan3', 'h2o', 'ch2cl', 's', 's', 'n3', 's', 'n3', '94', 'substitution', 'reactions', '601', 'when', 'chloromethylated', 'cross-linked', 'polystyrene', 'reacted', 'potassium', 'superoxide', 'yield', 'depends', 'upon', 'type', 'solvent', 'used', 'in', 'dimethylsulfoxide', 'presence', '18-crown-6', 'ether', 'conversion', 'hydroxymethyl', 'groups', '45.', 'in', 'benzene', 'however', '25.', 'high', 'conversions', 'obtained', 'catalyzing', 'reaction', 'tetrabutylammonium', 'iodide', 'mixture', 'solvents', 'this', 'results', '85', 'conversions', 'hydroxymethyl', 'groups', 'rest', 'become', 'iodide', 'groups', '199.', 'quaternary', 'salts', 'effective', 'crown', 'ethers', 'reactions', 'salts', 'oxygen-anions', 'carboxylate', 'phenolate', '200.', 'on', 'hand', 'like', 'dicyclohexyl-18-crown-6', 'exhibit', 'higher', 'catalytic', 'activity', 'quaternary', 'salts', 'reactions', 'salts', 'sulfur', 'anions', 'also', 'catalytic', 'activity', 'phase', 'transfer', 'catalysts', 'toward', 'nitrogen', 'anions', 'intermediate', 'toward', 'sulfur', 'anions', 'solid–liquid', 'two-phase', 'systems', 'generally', 'give', 'higher', 'degrees', 'conversion', 'liquid–liquid', 'systems', 'when', 'however', 'lipophilic', 'phase', 'transfer', 'catalysts', 'used', 'lipophilic', 'reagents', 'high', 'degrees', 'substitutions', 'achieved', 'liquid–liquid', 'two-phase', 'systems', '200.', 'toward', 'oxygen', 'lipophilic', 'crown', 'ethers', 'conversion', 'chloromethylated', 'styrene', 'anionic', 'exchange', 'resins', 'done', 'commercially', 'amination', 'reactions', 'form', 'quaternary', 'ammonium', 'groups', '201', '202.', 'this', 'reaction', 'illustrated', 'follows', '203:', 'n(ch3)3', 'ch2cl', 'hn(ch3)2', 'cl', 'n', 'clch2ch2oh', 'nhcl', 'n', 'cl', 'oh', 'the', 'kinetics', 'amination', 'chloromethylated', 'polystyrene', 'monohydroxy', 'dialkyl', 'tertiary', 'amines', 'shows', 'reactions', 'proceed', 'two', 'steps', 'two', 'different', 'rates', 'the', 'rate', 'changes', 'take', '602', '9', 'organic', 'reactions', 'polymers', 'place', 'conversions', '45–50', '205.', 'these', 'rates', 'favorably', 'inﬂuenced', 'increases', 'dielectric', 'constants', 'solvents', '204.', 'two', 'different', 'rate', 'constants', 'also', 'exist', 'reactions', '3-', 'alkylaminopropionitrile', '9434', 'friedel-craft', 'alkylation', 'reactions', 'friedel-craft', 'acylations', 'polystyrene', 'carried', 'cs2', 'ccl4', 'reﬂux', 'temperatures', 'solvents', 'the', 'yields', 'high', '209:', 'o', 'cl', 'catalyst', 'o', 'chloromethylated', 'copolymers', 'styrene', 'divinyl', 'benzene', 'undergo', 'friedel-craft', 'type', 'reactions', 'condensations', '1-phenyl-3-methylpyrazolone', '1-phenyl-23-dimethyl-', 'pyrazolone-5', 'presence', 'either', 'zncl2', 'bf3', 'sncl4', '208:', 'o', 'n', 'n', 'ch2cl', 'o', 'n', 'n', 'polystyrene', 'also', 'cross-link', 'friedel-craft', 'reaction', '210:', 'clch2', 'ch2cl', 'sncl4', '94', 'substitution', 'reactions', '603', 'the', 'condensation', 'takes', 'place', 'dichloroethane', 'stannic', 'chloride', 'catalyst', '50(cid1)c', '210.', 'the', 'maximum', 'reaction', 'rate', 'varies', 'initial', 'concentration', '14-dimethyl-25-dichloro-', 'methylbenzene', 'shown', 'initial', 'concentration', 'sncl4', 'cross-linked', 'polystyrene', 'particles', 'beads', 'also', 'form', 'friedel-craft', 'suspension', 'cross-linking', 'polystyrene', '14-dichloromethyl-25', 'dimethyl-benzene', '211.', 'the', 'polymer', 'dissolved', 'nitrobenzene', 'two-phase', 'reaction', 'occurs', '70', 'weight', 'aqueous', 'suspension', 'zncl2', 'poly(vinyl', 'alcohol)', 'used', 'suspending', 'agent', '9435', 'sulfonation', 'reactions', 'sulfonation', 'reactions', 'polystyrene', 'copolymers', 'divinyl', 'benzene', 'carried', 'commercially', 'prepare', 'ion', 'exchange', 'resins', 'partial', 'sulfonations', 'polystyrenes', 'achieved', 'presence', 'ethers', 'when', '50', 'aromatic', 'rings', 'sulfonated', 'polymers', 'become', 'water-soluble', 'at', 'lesser', 'amounts', 'sulfonation', '25–50,', 'polymers', 'solvent-soluble', '212', '213.', 'when', 'polystyrene', 'sulfonated', 'chlorinated', 'hydrocarbons', 'complex', 'dioxane-so3', 'polymer', 'precipitates', 'solution', 'low', 'concentrations', '214', '215.', 'complexes', 'ketones', 'so3', 'also', 'used', 'sulfonate', 'polystyrene', 'halogenated', 'solvents', '216.', 'the', 'ratio', 'sulfonation', 'favorable', 'poly(vinyl', 'toluene)', 'polystyrene', 'conditions', '217.', 'also', 'sulfur', 'dioxide', 'swells', 'polystyrene', 'the', 'polymer', 'sulfonated', 'medium', 'sulfur', 'trioxide', 'chlorosulfonic', 'acid', '218.', 'polystyrene', 'sulfonated', 'cs2', 'aluminum', 'chloride', 'catalyst', 'water-insoluble', 'free', 'acid', 'form', '219.', '9436', 'nitration', 'reduction', 'diazotization', 'nitration', 'polystyrene', 'originally', 'carried', 'long', 'time', 'ago', '220.', 'a', 'nitrating', 'mixture', 'nitric', 'sulfuric', 'acids', 'dissolves', 'polymer', 'nitro', 'derivative', 'forms', '50(cid1)c', 'within', '3', 'h', '221:', 'hno3', 'h2so4', 'no2', 'no2', 'the', 'reaction', 'accompanied', 'loss', 'molecular', 'weight', 'nitration', 'isotactic', 'polystyrene', 'yields', 'crystalline', 'product', '(about', '16', 'no2ring)', 'parent', 'compound', '222.', 'here', 'however', 'loss', 'molecular', 'weight', 'accompanies', 'reaction', '223.', 'polystyrene', 'nitrated', 'mild', 'conditions', 'using', 'acetyl', 'nitrate', 'the', 'product', 'contains', 'approximately', '06', 'nitro', 'groups', 'per', 'benzene', 'ring', '224.', 'the', 'nitro', 'groups', 'polynitrostyrene', 'reduced', 'phenyl', 'hydrazine', 'acts', 'hydrogen', 'donor', '225:', '604', '9', 'organic', 'reactions', 'polymers', 'nh2', 'n', 'h', '3', 'no2', '3', '3n2', 'h2o', 'nh2', 'polyaminostyrene', 'undergo', 'typical', 'reactions', 'aromatic', 'amines', 'diazotization', '226.', 'the', 'diazonium', 'salt', 'decomposes', 'ferrous', 'ions', 'yield', 'polymeric', 'free-radicals', 'hno2', 'h', 'fe', 'nh2', 'n2', 'cl', '9437', 'metalation', 'reactions', 'functional', 'polystyrene', 'derivatives', 'starting', 'materials', 'reactions', 'many', 'multistep', 'syntheses', 'an', 'example', 'metalation', 'polystyrenes', 'use', 'intermediates', '227:', 'c4h9li', 'o', 'r', 'r', 'i', 'li', 'r', 'r', 'oh', 'some', 'reactions', 'lithiated', 'polystyrene', '227:', 'rcn', 'pcl', '2', 'r', 'o', 'li', 'p', '3', 'when', 'polystyrenelithium', 'aminated', 'reagent', 'prepared', 'methoxyamine', 'methyllithium', 'two', 'reaction', 'mechanisms', 'possible', 'one', 'may', 'proceed', 'via', 'nitrene', 'intermediates', 'one', 'via', 'electrophilic', 'nitrenium', 'ions', '228.', 'many', 'reactions', 'polystyrenelithium', 'found', 'literature', '229–232.', 'sodium', 'metalated', 'polystyrene', 'reacts', 'similar', 'manner', '228:', '94', 'substitution', 'reactions', '605', '2', 'na', 'cl', 'si', 'cl', 'cl', 'na', 'si', 'cl', 'polyiodostyrene', 'good', 'starting', 'material', 'many', 'reactions', 'some', '233', '234:', '2', 'o', 'oh', 'i', 'o', 'o', 'i', 'o', 'o', 'cl2', 'k2co3', 'icl2', 'io', '606', '9', 'organic', 'reactions', 'polymers', 'poly(26-dimethyl-14-phenylene', 'oxide)', 'brominated', 'n-bromo-succinimide', '235.', 'the', 'product', 'also', 'subsequently', 'used', 'reactions', '235,', 'instance', 'phosphoryla-', 'tion', 'triethyl', 'phosphite', 'o', 'nbs', 'ccl4', 'ch2br', 'o', '(oc2h5)2', 'p', 'o', 'o', '944', 'reactions', 'acrylic', 'methacrylic', 'related', 'polymers', 'the', 'functional', 'groups', 'polymers', 'acrylic', 'methacrylic', 'esters', 'undergo', 'typical', 'reactions', 'groups', 'there', 'therefore', 'numerous', 'reports', 'literature', 'reactions', '9441', 'reduction', 'ester', 'groups', 'perhaps', 'reported', 'reactions', 'polymers', 'reductions', 'functional', 'groups', 'among', 'reaction', 'lithium', 'aluminum', 'hydride', 'reduce', 'ester', 'groups', 'the', 'success', 'however', 'depends', 'upon', 'reaction', 'medium', 'poly(methyl', 'methacrylate)', 'reduced', 'poly(methallyl', 'alcohol)', 'ether', 'solvents', '236:', 'the', 'results', 'however', 'inconclusive', 'combustion', 'analyses', 'fail', 'match', 'theoretical', 'composition', 'poly(methallyl', 'alcohol)', 'it', 'impossible', 'tell', 'extent', 'reduction', 'takes', 'place', '236.', 'inconclusive', 'results', 'also', 'obtained', 'similar', 'reductions', 'poly(methyl', 'acrylate)', 'mixtures', 'tetrahydrofuran', 'benzene', 'the', 'product', 'reduction', 'acetylated', 'acetic', 'anhydride', 'pyridine', '237', 'follows', 'lialh4', 'o', 'o', 'lialh4', 'o', 'o', 'n', 'o', 'o', 'o', 'oh', 'oh', 'oh', 'o', 'o', '94', 'substitution', 'reactions', '607', 'hydrolysis', 'water', 'product', 'acetylation', 'followed', 'treatment', 'hot', 'm-cresol', 'subsequent', 'extraction', 'hydrochloric', 'acid', 'remove', 'suspended', 'inorganic', 'matter', '237,', 'yields', 'material', 'still', 'soluble', 'pyridine', 'm-cresol', 'somewhat', 'similar', 'results', 'obtained', 'reductions', 'high', 'molecular', 'weight', 'poly(methyl', 'acrylate)', 'lithium', 'aluminum', 'hydride', '238', 'tetrahydrofuran', 'the', 'reaction', 'yields', 'product', 'soluble', 'mixtures', 'hydrochloric', 'acid', 'either', 'methyl', 'alcohol', 'dioxane', 'tetrahydrofuran', 'the', 'problem', 'apparently', 'due', 'residual', 'aluminum', 'hard', 'remove', '239.', 'if', 'however', 'reduction', 'carried', 'n-methylmorpholine', 'solution', 'followed', 'addition', 'potassium', 'tartrate', 'pure', 'product', 'isolated', '240.', 'n-methylmorpholine', 'good', 'solvent', 'reductions', 'various', 'macromolecules', 'metal', 'hydrides', '236.', 'in', 'addition', 'solvent', 'permits', 'use', 'strong', 'naoh', 'solutions', 'hydrolyze', 'addition', 'complexes', 'form', 'other', 'polymers', 'reduced', 'bearing', 'nitrile', 'amide', 'imide', 'lactam', 'oxime', 'pendant', 'groups', 'reduction', 'polymethacrylonitrile', 'however', 'yields', 'product', '70', 'primary', 'amine', 'groups', '241.', 'complete', 'reductions', 'pendant', 'carbonyl', 'groups', 'lialh4', 'solvents', 'n-', 'methylmorpholine', 'however', 'reported', 'thus', 'copolymer', 'methyl', 'vinyl', 'ketone', 'styrene', 'fully', 'reduced', 'tetrahydrofuran', '242.', 'reductions', 'metal', 'hydrides', 'often', 'preliminary', 'steps', 'additional', 'reactions', 'for', 'instance', 'product', 'lialh4', 'reduction', 'syndiotactic', 'poly(methyl', 'methacrylate)', 'reacted', 'succinic', 'anhydride', 'converted', 'amide', '243:', 'o', 'o', 'o', 'lialh4', 'o', 'o', 'oh', 'rnh2', 'o', '2', 'oh', 'o', 'o', 'o', '2', 'nhr', 'o', 'o', 'substituted', 'succinic', 'anhydride', 'used', 'well', 'when', 'poly(dimethyl', 'itaconate)', 'reduced', 'lialh4', 'thf', 'product', 'contains', 'ash', '93', 'functional', 'groups', 'reduced', '239:', 'o', 'o', 'o', 'o', 'lialh4', 'oh', 'oh', '608', '9', 'organic', 'reactions', 'polymers', '9442', 'nucleophilic', 'electrophilic', 'substitutions', 'many', 'conversions', 'functional', 'groups', 'acrylic', 'methacrylic', 'resins', 'reported', 'one', 'conversion', 'methyl', 'acrylate', 'hydrazide', 'direct', 'reaction', 'hydrazine', '244:', 'h2n', 'nh2', 'h2o', 'o', 'o', 'o', 'n', 'h', 'nh2', 'the', 'reaction', 'requires', '101', 'ratio', 'hydrazine', 'ester', 'groups', 'in', 'laboratory', 'carried', 'steam', 'bath', 'period', '2–3', 'h', 'approximately', '60–80', 'ester', 'groups', 'convert', '244.', 'the', 'hydrazides', 'form', 'various', 'hydrazones', 'reactions', 'aldehydes', 'ketones', 'r', 'r', 'o', 'rcho', 'o', 'n', 'h', 'n', 'r', 'r', '(or', 'h)', 'n', 'h', 'nh2', 'o', 'o', 'the', 'hydrazide', 'also', 'converted', 'azide', '244:', 'nano2', 'h', 'o', 'n3', 'n', 'h', 'nh2', 'a', 'hoffman', 'reaction', 'azide', 'yields', 'cross-linked', 'polymer', '244.', 'syndiotactic', 'poly(methyl', 'methacrylate)', 'converts', 'hydrazide', 'similar', 'manner', '245.', 'nucleophilic', 'substitution', 'reactions', 'carried', 'poly(methyl', 'methacrylate)', 'hetero-', 'cyclic', 'organolithium', 'reagents', '246.', 'the', 'reactions', 'conducted', 'homogeneous', 'solutions', 'tetrahydrofuran', 'benzene', 'combined', 'hexamethyl-phosphoramide', 'copolymers', 'form', 'tautomeric', 'keto-b-heterocyclic', 'structures', 'following', 'heterocyclic', 'reagents', 'useful', '246:', '2-picolinyllithium', '(44-dimethyl-2-oxazole-2-yl)methyllithium', 'quinaldinyllithium', '2-thia-', 'zole-2-yl-methyllithium', 'in', 'attempts', 'carry', 'arndt-eister', 'reactions', 'poly(methacryloyl', 'chloride)', 'polymer', 'reacted', 'diazomethane', 'various', 'molar', 'ratios', 'different', 'temperatures', '247.', 'initially', 'acid', 'chloride', 'groups', 'react', 'diazomethane', 'expected', 'the', 'products', 'however', 'undergo', 'subsequent', 'reactions', 'neighboring', 'acid', 'chloride', 'groups', 'form', 'cyclic', 'structures', '247:', '94', 'substitution', 'reactions', '609', 'o', 'o', 'cl', 'cl', 'ch2n2', 'o', 'o', 'chn2', 'cl', 'o', 'o', 'n2', 'when', 'curtius', 'lossen', 'rearrangement', 'reactions', 'attempted', 'poly(acryloyl', 'chloride)', '248,', 'products', 'fairly', 'regular', 'polyampholytes', 'somewhat', 'similar', 'results', 'obtained', 'hoffman', 'reaction', 'polyacrylamide', '279.', 'a', 'schmidt', 'reaction', 'poly(acrylic', 'acid)', 'also', 'yields', 'mixed', 'results', '250.', 'when', 'run', 'acetic', 'acid', 'intermolecular', 'reactions', 'appear', 'predominate', 'intramolecular', 'ones', 'also', 'products', 'formed', 'acetic', 'acid', 'higher', 'nitrogen', 'content', 'formed', 'dioxane', '250.', 'the', 'nmr', 'spectra', 'show', 'presence', 'acid', 'anhydride', 'groups', 'this', 'additional', 'effect', 'lowering', 'yield', 'diels-alder', 'reactions', 'carried', 'poly(furfuryl', 'methacrylate)', 'dienophiles', 'like', 'maleic', 'anhydride', 'maleimide', '252.', 'dilute', 'solutions', '(10)', 'polymers', 'benzene', 'used', 'requiring', '30', 'molar', 'excess', 'dienophiles', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', '610', 'complete', 'the', 'additions', 'take', 'place', 'room', 'temperature', 'reactions', 'take', '7', '30', 'days', '252', 'polyacrylonitrile', 'reacts', 'hydroxylamine', 'product', 'metalated', 'elements', 'group', 'iv', '(sn', 'ge', 'si)', 'this', 'convenient', 'route', 'formation', 'polymers', 'pendant', 'organometallic', 'groups', '253:', '9', 'organic', 'reactions', 'polymers', 'h2noh', 'n', 'clsnr3', 'hon', 'nh2', 'n', 'o', 'nh2', 'snr3', 'the', 'yields', 'high', 'reactions', 'carried', 'dilute', 'solutions', 'polyacrylonitrile', 'dimethylformamide', '75(cid1)c', 'the', 'solutions', 'must', 'contain', '15', 'mol', 'hydroxylamine', 'hydrochlo-', 'ride', 'sodium', 'carbonate', 'per', 'mole', 'acrylonitrile', 'groups', '253.', 'a', 'ritter', 'reaction', 'carried', 'atactic', 'polyacrylonitrile', 'n-hydroxymethylamides', 'acetic', 'benzoic', 'benzene-sulfonic', 'acids', '254.', 'when', 'reactions', 'carried', 'n-', 'hydroxymethylimides', 'succinic', 'phthalic', 'acids', 'tetramethylene', 'sulfone', 'stronger', 'tendency', 'toward', 'cross-linking', 'copolymers', 'methacroyl', 'chloride', 'undergo', 'arbuzov', 'rearrangement', 'reactions', 'triethyl', 'phosphite', 'dimethylformamide', 'dioxane', 'benzene', '75(cid1)c', '255.', 'the', 'conversions', 'high', 'ranging', '96', '98.', 'the', 'aldehyde', 'groups', 'polyacrolein', 'reduced', 'meerwein-ponndorf', 'reaction', 'there', 'limit', 'however', 'amount', 'alcoholate', 'used', 'concentrations', 'free', 'aldehyde', 'groups', 'starting', 'material', '256.', 'also', 'ester', 'condensations', 'take', 'place', '(tischenko', 'reaction)', 'time', 'reductions', 'occur', '256.', '945', 'substitution', 'reactions', 'poly(vinyl', 'alcohol)', 'there', 'many', 'practical', 'uses', 'products', 'reactions', 'poly(vinyl', 'alcohol)', 'among', 'commercial', 'preparations', 'poly(vinyl', 'acetal)s', 'formed', 'condensations', 'aldehydes', 'two', 'materials', 'currently', 'marketed', 'poly(vinyl', 'formal)', 'poly(vinyl', 'butyral)', 'the', 'ﬁrst', 'one', 'formed', 'partially', 'hydrolyzed', 'poly(vinyl', 'acetate)', 'dissolved', 'aqueous', 'acetic', 'acid', 'excess', 'formaldehyde', 'the', 'mixture', 'heated', 'sulfuric', 'acid', 'added', 'reaction', 'allowed', 'proceed', '70–90(cid1)c', '6', 'h', 'sulfuric', 'acid', 'neutralized', 'formal', 'precipitates', '94', 'substitution', 'reactions', '611', 'two', 'different', 'industrial', 'processes', 'used', 'preparations', 'butyral', 'in', 'acetate', 'free', 'poly(vinyl', 'alcohol)', 'used', 'in', 'ﬁrst', 'one', '10', 'solutions', 'starting', 'material', 'treated', 'butyraldehyde', 'sulfuric', 'acid', 'the', 'mixtures', 'heated', '90(cid1)c', '1½', 'h', 'products', 'precipitate', 'they', 'neutralized', 'washed', 'dried', 'in', 'second', 'one', 'poly(vinyl', 'alcohol)', 'suspended', 'ethanolethyl', 'acetate', 'butyraldehyde', 'together', 'strong', 'mineral', 'acid', 'added', 'the', 'solutions', 'neutralized', 'the', 'butyrals', 'separate', 'they', 'neutralized', 'resin', 'washed', 'dried', 'if', 'poly(vinyl', 'alcohol)', 'ﬁlms', 'reacted', 'formaldehyde', 'water', 'containing', 'salt', 'acid', 'catalyst', '(heterogeneous', 'formalization)', 'cross-linking', 'occurs', 'the', 'number', 'cross-links', 'increases', 'decreasing', 'acid', 'concentration', 'ﬁxed', 'amounts', 'formaldehyde', 'salt', '257.', 'direct', 'reactions', 'poly(vinyl', 'alcohol)', 'aldehydes', 'kornblum', 'reaction', 'result', 'formations', 'acetals', 'also', 'contain', 'residual', 'hydroxyl', 'group', 'often', 'acetate', 'groups', 'the', 'acetate', 'groups', 'incomplete', 'hydrolysis', 'parent', 'poly(vinyl', 'acetate)', 'used', 'form', 'poly(vinyl', 'alcohol)', 'reactions', 'poly(vinyl', 'alcohol)', 'ketones', 'yield', 'similar', 'ketals', 'at', 'present', 'ketals', 'offered', 'commercially', 'alkyl', 'etheriﬁcation', 'poly(vinyl', 'alcohol)', 'occurs', 'polymer', 'combined', 'n-alkyl', 'halides', 'dimethylsulfoxide', 'combined', 'pyridine', '258', '259.', 'it', 'suggested', 'alkyl', 'halides', 'convert', 'aldehydes', 'acids', 'act', 'intermediates', 'dimethylsulfoxide-', 'pyridine', 'solution', '258', '259:', 'rch2br', 'oh', 'n', 'o', 's', 'hbr', 'o', 'r', 'many', 'modiﬁcations', 'poly(vinyl', 'alcohol)', 'carried', 'form', 'photosensitive', 'materials', 'thus', 'unsaturation', 'introduced', 'pendant', 'groups', 'photocross-linking', 'one', 'example', 'condensation', 'pyridinium', 'quinolinium', 'salts', '260:', 'oh', 'oh', 'n', 'o', 'o', 'n', '9', 'organic', 'reactions', 'polymers', '612', 'subject)', 'the', 'material', 'cyclodimerizes', 'exposure', 'light', '261', '(see', 'chap', '10', 'additional', 'discussion', 'schotten-baumann', 'esteriﬁcations', 'poly(vinyl', 'alcohol)', 'used', 'extensively', 'preparations', 'various', 'derivatives', 'the', 'reactions', 'appear', 'proceed', 'well', 'acid', 'chlorides', 'employed', 'two-', 'phase', 'systems', '262.', 'the', 'polymers', 'dissolved', 'water', 'solutions', 'blended', '111', 'equal', 'volume', 'naoh', 'solutions', 'cyclohexanone', 'they', 'mixed', 'thoroughly', 'solutions', 'acid', 'chlorides', 'mixtures', 'cyclohexanone', 'toluene', 'the', 'reaction', 'mixture', 'stirred', 'vigorously', '90', 'min', '(cid3)5', '5(cid1)c', 'obtain', 'desired', 'product', 'metalation', 'poly(vinyl', 'alcohol)', 'used', 'form', 'ether', 'derivatives', '263:', 'o', 's', 'na', 'o', 's', 'oh', 'ona', 'br', 'x', 'nabr', 'ona', 'o', 'x', 'the', 'ritter', 'reaction', 'poly(vinyl', 'alcohol)', 'yields', 'soluble', 'products', 'only', 'hydroxyl', 'groups', 'however', 'converted', 'amide', 'structures', '264:', 'rcn', 'h2so4', 'oh', 'oh', 'h', 'n', 'r', 'oh', 'o', 'also', 'possible', 'neighboring', 'group', 'interactions', 'may', 'lead', 'cyclizations', 'formations', '13-oxazines', '264.', '946', 'miscellaneous', 'exchange', 'reactions', 'many', 'miscellaneous', 'exchange', 'reactions', 'reported', 'literature', 'a', 'presented', 'one', 'reaction', 'reduction', 'pendant', 'carbonyl', 'groups', 'poly(vinyl', 'methyl', 'ketone)', 'metal', 'hydrides', '242', '265:', 'lialh4', 'kbh4', 'o', 'r', 'ho', 'r', '94', 'substitution', 'reactions', 'carbonyls', '266:', 'another', 'one', 'introduction', 'unsaturation', 'pendant', 'groups', 'wittig', 'reaction', 'pendant', 'o', 'r', 'r', 'p', '3', 'r', 'r', '613', 'o', 'p', '3', 'the', 'reaction', 'also', 'carried', 'copolymer', 'ethylene', 'carbon', 'monoxide', '266.', 'polycaprolactam', 'treated', 'either', 'so2cl2', 'pocl3', 'pcl5', '70(cid1)c', 'introduce', 'ionic', 'chlorine', 'groups', '267.', 'the', 'main', 'product', 'poly(aa-dichloro-caprolactam)', 'o', 'h', 'n', '5', 'so2cl2', 'o', 'cl', 'n', '5', 'cl', 'also', 'nylon', '66', 'reacted', 'triﬂuoroacetic', 'anhydride', 'triﬂuoroacetyl', 'nylon', 'forms', '268:', 'o', 'o', 'o', 'o', '4', 'n', 'h', '6', 'n', 'h', 'cf3', 'cf3', 'o', 'o', 'o', 'syndiotactic', 'poly(2-methallyl', 'hydrogen', 'phthalate)', 'prepared', 'amidated', 'according', 'following', 'scheme', '269:', '4', 'n', 'n', '6', 'o', 'o', 'cf3', 'cf3', 'o', 'o', 'o', 'lialh4', 'o', 'o', 'rnh2', 'o', 'o', 'o', 'o', 'h', 'n', 'r', 'o', 'ho', 'o', 'o', 'poly(pyridine', 'ether', 'sulfone)s', 'nitrobenzene', '70.', 'the', 'reaction', 'illustrated', 'follows', '614', '9', 'organic', 'reactions', 'polymers', 'n', 'o', 'n', 'o', 'o', 'o', '(ch3)2so2', 'o', 's', 'o', 'o', 's', 'o', 'o', 'o', 'o', 'o', 'a', '20', 'mol', 'excess', 'alkylating', 'reagent', 'required', 'reaction', 'must', 'conducted', '6', 'h', '80(cid1)c', 'dichloroketene', 'generated', 'ultrasound-promoted', 'dechlorination', 'reaction', 'thrichloroacetyl', 'chloride', 'zinc', 'adds', 'carbon–carbon', 'double', 'bonds', 'poly(methyl-1-phenyl-1-silane-cis-', 'pent-3-ene)', '271.', 'this', 'illustrated', 'follows', 'ph', 'si', 'cl', 'o', 'ether', 'ph', 'si', 'o', '95', 'cross-linking', 'reactions', 'polymers', 'these', 'reactions', 'quite', 'numerous', 'utilized', 'long', 'time', 'they', 'also', 'include', 'thermosetting', 'processes', 'polymers', 'many', 'discussed', 'previous', 'chapters', '951', 'vulcanization', 'elastomers', 'cross-linking', 'natural', 'rubber', 'discovered', 'goodyear', 'back', '1839', 'sulfur', 'original', 'cross-linking', 'agent', 'still', 'utilized', 'today', 'many', 'processes', 'early', 'studies', 'demonstrated', 'cross-links', 'mainly', 'polysulﬁdes', '4', 's(8)', '140', 'oc', '10', 'hours', 's', '(4-6)', 's', '(4-6)', '95', 'cross-linking', 'reactions', 'polymers', '615', 'the', 'reactions', 'take', 'place', 'temperatures', 'industrially', 'carried', '50', '75(cid1)c', 'at', 'lower', 'temperatures', 'however', 'process', 'may', 'take', 'days', 'complete', 'at', 'temperatures', '135–155(cid1)c', 'approximately', '8', 'sulfur', '(by', 'weight', 'rubber)', 'reacts', '272.', 'also', 'sulfur', 'dissolves', 'unvulcanized', 'rubber', 'even', 'room', 'temperature', 'the', 'overall', 'mechanism', 'reaction', 'still', 'studied', 'most', 'evidence', 'points', 'ionic', 'mechanism', 'sulfonium', 'ion', 'intermediate', '272.', 'it', 'shown', '273', 'straightforward', 'reaction', 'sulfur', 'rubber', 'insufﬁcient', 'somehow', '40', '100', 'atoms', 'sulfur', 'must', 'combined', 'order', 'obtain', 'one', 'cross-link', 'out', '40–100', 'atoms', '6–10', 'actually', 'engaged', 'formation', 'cross-links', 'the', 'rest', 'atoms', 'form', 'cyclic', 'sulﬁde', 'units', 'become', 'spread', 'along', 'main', 'chain', '273.', 'to', 'improve', 'efﬁciency', 'vulcanization', 'reaction', 'various', 'accelerators', 'developed', 'among', 'zinc', 'oxide', 'combined', 'fatty', 'acids', 'andor', 'amines', 'zinc', 'oxide', 'forms', 'zinc', 'mercaptides', 'like', '(xs)2znl2', 'x', 'electron', 'withdrawing', 'substituent', 'l', 'ligand', 'carboxyl', 'amine', 'group', 'the', 'function', 'ligand', 'render', 'complex', 'soluble', 'the', 'mercaptide', 'complexes', 'believed', 'react', 'additional', 'sulfur', 'form', 'zinc', 'perthiomercaptides', 'the', 'accelerators', 'commonly', 'used', 'derivatives', '2-mercaptobenzothiazole', 'they', 'effective', 'used', 'combinations', 'metal', 'oxides', 'fatty', 'acids', '(referred', 'activators)', 'the', 'favorite', 'activators', 'zinc', 'oxide', 'combined', 'stearic', 'acid', 'the', 'combinations', 'permit', 'rapid', 'vulcanizations', 'take', 'minutes', 'compared', 'hours', 'sulfur', 'used', 'alone', 'in', 'process', 'vulcaniza-', 'tion', '220-dithiobisbenzthiazole', 'forms', 'initially', 'reacts', 'sulfur', 'form', 'polysulﬁdes', '273:', 'n', 's', 'c', 's', 's', 'c', 'n', 's', 'sx', 'n', 's', 'c', 's', 'sx', 's', 'c', 'n', 's', 'the', 'products', 'reactions', 'sulfur', 'turn', 'react', 'natural', 'synthetic', 'rubber', 'allylic', 'hydrogen', 'this', 'concerted', 'reaction', 'results', 'formation', 'sulfur', 'containing', 'adducts', 'polymers', 's', 's', 's', 'n', 's', 'n', 's', 's', 's', 'n', 's', 'sh', 'n', 's', 'crosslinking', 's', 'x', 'once', 'cross-links', 'formed', 'transformations', 'take', 'place', 'some', 'consist', 'reactions', 'shorten', 'polysulﬁde', 'links', 's', 'x', 's', '2', 's', '616', '9', 'organic', 'reactions', 'polymers', 'also', 'cross-links', 'lost', 'elimination', 'reactions', 's', 'x', 's', 'h', 'x', 'in', 'addition', 'cyclic', 'sulfur', 'compounds', 'form', 'process', '273:', 'sx', 's', 'sx-y', '952', 'cross-linking', 'polymers', 'aid', 'peroxides', 'the', 'reaction', 'utilized', 'many', 'polymers', 'polyoleﬁns', 'polymers', 'dienes', 'others', 'the', 'reactions', 'natural', 'rubber', 'illustrated', 'o', 'o', '2', '2', 'o', 'o', '2ch3', 'o', 'hydrogen', 'abstraction', 'oh', '(', 'ch4)', 'follows', '274:', '(', 'ch3', ')', '2', '96', 'graft', 'copolymers', '617', 'the', 'extent', 'cross-linking', 'shown', 'clear', 'it', 'known', 'however', 'cleavage', 'reactions', 'followed', 'free-radical', 'recombinations', 'take', 'place', '274:', 'ch2', 'polymeric', 'chains', 'bearing', 'free', 'radicals', 'combine', 'give', 'branched', 'structures', 'additions', 'chains', 'freeradical', 'double', 'bonds', 'result', 'formations', 'cross-links', '274.', '953', 'miscellaneous', 'cross-linking', 'reactions', 'polymers', 'many', 'miscellaneous', 'cross-linkings', 'polymeric', 'materials', 'reported', 'literature', 'for', 'instance', 'poly(acryloyl', 'chloride)', 'cross-linked', 'diamines', '275:', 'r', 'h2n', 'nh2', '2', 'cl', 'o', 'o', 'o', 'n', 'h', 'r', 'n', 'h', 'in', 'similar', 'manner', 'polymers', 'pendant', 'chlorosulfonate', 'groups', 'cross-link', 'reacted', 'diamines', 'glycols', '275.', '96', 'graft', 'copolymers', 'this', 'important', 'part', 'polymer', 'syntheses', 'used', 'many', 'industrial', 'processes', 'in', '1967', 'battaerd', 'tregear', '282', 'published', 'book', 'subject', 'contains', '1000', 'references', 'journal', 'publications', '1200', 'references', 'patents', 'in', 'addition', 'several', 'monographs', 'many', 'review', 'papers', '283.', 'the', 'synthetic', 'methods', 'developed', 'date', 'range', 'using', 'free', 'radical', 'attacks', 'polymeric', 'backbones', 'highly', 'reﬁned', 'ionic', 'reactions', 'there', 'examples', 'ionic', 'reactions', 'attach', 'side-chains', 'well-designated', 'locations', '961', 'free-radical', 'grafting', 'chain-transferring', 'process', 'this', 'technique', 'probably', 'one', 'simplest', 'ways', 'form', 'graft', 'copolymers', 'it', 'consists', 'carrying', 'free-radical', 'polymerizations', 'monomers', 'presence', 'polymers', 'preformed', 'different', '618', '9', 'organic', 'reactions', 'polymers', 'monomers', 'a', 'prerequisite', 'synthesis', 'active', 'sites', 'must', 'form', 'polymeric', 'backbones', 'course', 'reactions', 'ideally', 'occurs', 'steps', 'initiations', 'consist', 'attacks', 'initiating', 'radicals', 'backbones', 'initiation', 'i', 'i', '2', 'i', 'i', 'a', 'a', 'a', 'i', 'h', 'a', 'a', 'a', 'propagations', 'precede', 'backbone', 'sites', 'propagation', 'a', 'a', 'a', 'nm', '(m)n-1m', 'a', 'a', 'a', 'terminations', 'take', 'place', 'many', 'ways', 'of', 'course', 'termination', 'combination', 'lead', 'cross-linked', 'insoluble', 'polymers', 'undesirable', 'an', 'ideal', 'termination', 'takes', 'place', 'chain', 'transferring', 'another', 'site', 'polymer', 'backbone', 'initiate', 'another', 'chain', 'growth', 'the', 'however', 'ideal', 'picture', 'in', 'reality', 'efﬁciency', 'grafting', 'technique', 'depends', 'upon', 'following', '1', 'competitions', 'different', 'materials', 'present', 'reaction', 'mixture', 'monomer', 'solvent', 'polymer', 'backbone', 'radical', 'species', 'this', 'includes', 'competition', 'chains', 'growth', 'chain', 'transferring', 'species', 'present', '2', 'competition', 'terminating', 'processes', 'disproportionation', 'chain', 'transferring', 'the', 'conditions', 'vary', 'considerably', 'possible', 'carry', 'reactions', 'bulk', 'solution', 'emulsion', 'when', 'reaction', 'take', 'place', 'emulsion', 'success', 'depends', 'greatly', 'experimen-', 'tal', 'techniques', 'the', 'rate', 'diffusion', 'factor', 'anything', 'affects', 'rate', 'must', 'considered', 'because', 'grafting', 'efﬁciency', 'depends', 'upon', 'chain', 'transferring', 'backbone', 'knowledge', 'chain-', 'transferring', 'constants', 'help', 'predict', 'outcome', 'reactions', 'sometimes', 'information', 'chain-transferring', 'constants', 'available', 'literature', 'it', 'may', 'however', 'possible', 'obtain', 'information', 'reactions', 'low', 'molecular', 'weight', 'compounds', 'similar', 'structures', '284–286.', 'one', 'assumes', 'equal', 'reactivity', 'toward', 'attacking', 'radicals', 'the', 'validity', 'assump-', 'tion', 'demonstrated', 'oligomers', '281–283.', 'the', 'reactivity', 'initiating', 'radicals', 'toward', 'backbones', 'vary', 'also', 'vary', 'efﬁciency', 'grafting', 'benzoyl', 'peroxide-initiated', 'polymerizations', 'methyl', 'methacrylate', 'monomer', 'instance', 'presence', 'polystyrene', '284', 'yield', 'appreciable', 'quantities', 'graft', 'copolymers', 'very', 'little', 'graft', 'copolymers', 'however', 'form', 'di-t-butyl', 'peroxide', 'initiates', 'reactions', 'azobisisobutyronitrile', 'also', 'fails', 'yield', 'appreciable', 'quantities', 'graft', 'copolymers', 'this', 'due', 'inefﬁcient', 'chain', 'transferring', 'polymer', 'backbones', 't-butoxy', 'isobutyronitrile', 'radicals', 'a', 'better', 'approach', 'work', 'chung', 'coworkers', '288', 'grafted', 'maleic', 'anhydride', 'polypropylene', 'use', 'boraneo2', 'initiator', 'this', 'initiator', 'claimed', 'form', 'situ', 'monooxidized', 'adduct', '(r–o', 'o–br2)', 'these', 'adducts', 'carry', 'hydrogen', 'abstractions', 'polypropylene', 'chains', 'ambient', 'temperature', 'this', 'results', 'formation', 'stable', 'tertiary', 'polymer', 'radicals', 'react', 'maleic', 'anhydride', 'form', 'graft', 'copolymers', 'not', 'chain', 'transferring', 'backbones', 'results', 'formations', 'graft', 'copolymers', 'an', 'example', 'polymerization', 'vinyl', 'acetate', 'presence', 'poly(methyl', 'methacrylate)', 'no', 'graft', 'copolymers', 'form', 'independent', 'reactivity', 'initiators', '285.', 'in', 'fact', 'grafts', 'poly(vinyl', 'acetate)', 'poly(methyl', 'methacrylate)', 'polystyrene', 'cannot', 'prepared', 'technique', '286.', 'grafting', 'efﬁciency', 'may', 'increase', 'temperature', '287.', 'this', 'could', 'due', 'higher', 'activation', 'energy', 'transfer', 'reaction', 'propagation', 'reaction', '288.', 'meaningful', 'effects', '96', 'graft', 'copolymers', '619', 'temperature', 'however', 'always', 'observed', 'in', 'grafting', 'polystyrene', 'poly(butyl', 'acrylate)', 'emulsion', 'instance', 'noticeable', 'difference', '60', '90(cid1)c', 'technique', '289.', 'the', 'presence', 'sites', 'high', 'transfer', 'constants', 'polymeric', 'backbone', 'enhances', 'efﬁciency', 'grafting', 'such', 'sites', 'introduced', 'deliberately', 'these', 'instance', 'mercaptan', 'groups', '290', 'formed', 'reacting', 'h2s', 'polymer', 'containing', 'epoxy', 'groups', '2hs', 'o', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'ho', 'hs', 'o', 'o', 'o', 'o', 'o', 'o', 'sh', 'o', 'hs', 'free-radical', 'polymerizations', 'acrylic', 'methacrylic', 'esters', 'presence', 'backbones', 'result', 'high', 'yield', 'graft', 'copolymers', '291', '292.', 'another', 'example', 'formation', 'mercaptan', 'groups', 'cellulose', 'order', 'form', 'graft', 'copolymers', '293:', 's', 'o', 'ho', 'oh', 'o', 'o', 'h2s', 'o', 'ho', 'oh', 'o', 'ho', 'o', 'sh', 'pendant', 'nitro', 'groups', 'also', 'effective', 'chain', 'transfer', 'grafting', 'reactions', 'thus', 'graft', 'copolymers', 'polystyrene', 'cellulose', 'acetate', 'p-nitrobenzoate', '294', 'poly(vinyl', 'p-nitrobenzoate)', '295', 'form', 'readily', 'nitro', 'groups', 'appear', 'effective', 'formations', 'graft', 'copolymers', 'radical', 'mechanism', 'double', 'bonds', 'located', 'pendant', 'groups', '294.', '962', 'free-radical', 'grafting', 'reactions', 'polymers', 'double', 'bonds', 'carbon', 'carbon', 'double', 'bonds', 'either', 'backbone', 'pendant', 'groups', 'potential', 'sites', 'free-radical', 'attacks', 'in', 'addition', 'double', 'bonds', 'hydrogen', 'atoms', 'neighboring', 'carbons', 'allylic', 'potential', 'sites', 'chain', 'transferring', 'because', 'rubbers', 'natural', 'synthetic', 'possess', 'unsaturations', 'used', 'extensively', 'backbones', 'various', 'grafting', 'reactions', 'whether', 'reactivity', 'initiating', 'radical', 'important', 'determining', 'grafting', 'efﬁciency', 'completely', 'established', 'graft', 'copolymers', 'poly(methyl', 'methacrylate)', 'gutta-percha', 'however', 'form', 'good', 'yields', 'initiator', 'benzoyl', 'peroxide', '296', '297.', 'yet', 'azobisisobutyronitrile', 'used', 'mixture', 'homopolymers', 'forms', 'work', '14c-labeled', 'initiators', 'shows', 'primary', 'radicals', 'react', 'addition', 'double', 'bonds', 'transfer', 'methylene', 'group', '298.', '620', '9', 'organic', 'reactions', 'polymers', 'grafting', 'reactions', 'polybutadiene', 'however', 'proceed', 'via', 'chain', 'transferring', 'growing', 'chain', 'radical', 'backbone', '299.', 'nevertheless', 'strong', 'evidence', 'also', 'shows', 'initiator', 'radicals', 'interact', 'directly', 'polymeric', 'backbones', '299', '300.', 'when', 'graft', 'copolymers', 'polystyrene', 'natural', 'rubber', 'form', 'chain', 'length', 'attached', 'branches', 'equals', 'chain', 'lengths', 'unattached', 'polystyrene', 'homopolymer', 'forms', 'simulta-', 'neously', '301.', 'this', 'led', 'following', 'conclusions', '303:', '1', 'as', 'concentration', 'rubber', 'increases', 'length', 'grafted', 'branches', 'diminishes', '2', 'when', 'concentration', 'initiator', 'increases', 'length', 'branches', 'diminishes', '3', 'when', 'concentration', 'monomers', 'increases', 'length', 'branches', 'also', 'increases', '4', 'when', 'polymerization', 'time', 'increases', 'length', 'branches', 'remains', 'number', 'remains', 'number', 'branches', 'increases', 'number', 'diminishes', 'number', 'increases', '963', 'preparation', 'graft', 'copolymers', 'aid', 'macromonomers', 'the', 'chain-transferring', 'methods', 'preparing', 'graft', 'copolymers', 'suffer', 'disadvantages', 'low', 'efﬁciency', 'contamination', 'homopolymers', 'the', 'efﬁciency', 'forming', 'graft', 'copolymers', 'how-', 'ever', 'increases', 'use', 'macromonomers', 'a', 'macromonomer', 'macromolecular', 'monomer', 'oligomer', 'polymer', 'polymerizable', 'end', 'group', 'when', 'macromonomers', 'copolymerized', 'monomers', 'comb-shaped', 'polymers', 'form', '302', '303.', 'the', 'copolymerizations', 'free', 'radical', 'ionic', 'mechanism', 'some', 'examples', 'macromonomers', 'presented', 'table', '91', 'a', 'preparation', 'graft', 'copolymers', 'aid', 'macromonomers', 'illustrated', 'follows', '304:', 'o', 'o', 'o', 'o', 'r', 'h', 's', 'r', 'x', 'n', 'r', 'y', 'initiator', 'r', 'n', 'y', 'ho', 'oh', 'o', 'o', 'o', 'r', 's', 'o', 'r', 'x', 'h', 'many', 'variations', 'technique', 'possible', '96', 'graft', 'copolymers', 'table', '91', 'some', 'macromonomers', 'reported', 'literature', 'macromonomer', 'o', 'o', 'c4h9', 'n', 'oh', 'o', 'o', 'o', 'o', 's', 'n', 'cooch3', 'o', 'o', 'o', 'h', 's', 'n', 'cooch3', 'o', 'o', 'n', 'n', 'h', 'o', 'n', 'h', 'cl', 'n', 'o', 'o4', 'n', 'o', 'o', 'o4', 'n', '621', 'references', '320', '321', '322', '323', '324', '325', '326', '(continued)', '622', 'table', '91', '(continued)', 'macromonomer', 'o', '9', 'organic', 'reactions', 'polymers', 'references', 'o', 'n', '2', 'o', 'o', 'h', 'n', 'o', 'si', 'si', 'o', 'cl', 'n', 'o', 'o', 'o', '3', 'si', 'si', 'r', 'n', 'o', 'n', 'o', '327', '328', '329', '330', '331', '964', 'initiations', 'polymerizations', 'backbone', 'polymers', 'high', 'degrees', 'grafting', 'free-radical', 'mechanism', 'attained', 'polymerizations', 'initiated', 'backbones', 'polymer', 'one', 'way', 'done', 'form', 'peroxides', 'backbone', 'structures', 'decompositions', 'peroxides', 'yield', 'initiating', 'radicals', 'half', 'attached', 'backbones', 'an', 'example', 'preparation', 'graft', 'copolymers', 'polystyrene', '305', '306:', 'catalyst', 'o2', 'o', 'oh', '96', 'graft', 'copolymers', '623', 'thermal', 'cleavage', 'peroxides', 'leads', 'macromolecules', 'free-radicals', 'sites', 'hydroxyl', 'radicals', 'also', 'form', 'initiate', 'formations', 'homopolymers', 'decompositions', 'peroxides', 'redox', 'mechanisms', 'however', 'increase', 'yields', 'graft', 'copolymers', 'stop', 'formations', 'hydroxy', 'radicals', '303:', 'fe', 'fe', 'oh', 'o', 'oh', 'o', 'some', 'homopolymers', 'still', 'form', '307.', 'air', 'oxidation', 'polypropylene', 'result', 'formation', 'hydroperoxide', 'units', 'sites', 'tertiary', 'hydrogens', '383.', 'the', 'polymer', 'also', 'oxidized', 'dissolved', 'cumene', 'contains', 'cumene', 'hydroperoxide', '70–80(cid1)c', 'a', 'product', 'containing', '08', 'oxygen', 'weight', 'hydroperoxide', '308', 'formed', 'subsequently', 'reacted', 'form', 'graft', 'copolymers', 'various', 'monomers', '309–311', 'used', 'vinyl', 'acetate', '2-vinyl', 'pyrrolidone', 'many', 'hydroperoxidations', 'polymers', 'reported', 'literature', 'the', 'materials', 'used', 'formations', 'graft', 'copolymers', 'one', 'example', 'hydroperoxidation', 'poly(ethylene', 'terephthalate)', 'poly(e-caproamide)', 'the', 'products', 'yield', 'graft', 'copolymers', 'various', 'acrylic', 'methacrylic', 'esters', 'acrylic', 'methacrylic', 'acids', '312–314.', 'ozone', 'treatment', 'polymers', 'also', 'cause', 'hydroperoxidation', 'labile', 'hydrogens', 'it', 'however', 'also', 'cause', 'extensive', 'degradation', 'backbone', 'polymers', 'attacks', 'ozone', 'double', 'bonds', 'backbones', 'convert', 'unstable', 'ozonides', 'starch', 'ozonized', 'form', 'graft', 'copolymers', '315', '316.', 'the', 'true', 'cellulose', '317,', 'poly(vinyl', 'chloride)', '318,', 'polyethylene', '319.', 'hydroperoxides', 'form', 'without', 'noticeable', 'degradation', 'this', 'allows', 'subsequent', 'preparations', 'graft', 'copolymers', 'in', 'similar', 'manner', 'possible', 'start', 'copolymers', 'acryloyl', 'methacryloyl', 'chloride', 'react', 'hydroperoxides', '332.', 'this', 'illustrated', 'follows', 'ooh', 'o', 'o', 'o', 'cl', 'o', 'o', 'o', 'o', 'o', 'the', 'decomposition', 'pendant', 'peroxide', 'presence', 'vinyl', 'monomers', 'yields', 'mixtures', 'graft', 'copolymers', 'homopolymers', 'preparation', 'subsequent', 'decomposition', 'polymers', 'diazonium', 'salts', 'also', 'used', 'form', 'graft-copolymers', 'an', 'example', 'nitrated', 'polystyrene', 'reduced', 'amine', 'derivative', 'diazotized', '333.', 'the', 'decomposition', 'diazonium', 'salt', 'results', 'formation', 'radicals', '624', '9', 'organic', 'reactions', 'polymers', 'hno2', 'no2', 'fe', 'nh', 'hno3', 'n2cl', 'the', 'radical', 'sites', 'capable', 'initiating', 'polymerizations', 'monomers', 'a', 'similar', 'approach', 'taken', 'cellulose', '334.', 'mercerized', 'cotton', 'sodium', 'salt', 'carboxymethyl', 'cellulose', 'react', 'p-aminophenacyl', 'chloride', 'oh', 'o', 'o', 'o', 'n', 'cl', 'nh2', 'ho', 'o', 'na', 'oh', 'o', 'o', 'o', 'n', 'ho', 'o', 'nh2', 'the', 'material', 'converted', 'diazonium', 'salts', 'decomposed', 'ferrous', 'ions', 'presence', 'vinyl', 'monomers', 'form', 'graft', 'copolymers', 'acrylonitrile', 'forms', 'graft', 'copolymer', 'readily', 'without', 'formation', 'homopolymers', 'styrene', 'vinyl', 'acetate', 'however', 'a', 'modiﬁcation', 'technique', 'conduct', 'diazotization', 'reaction', 'presence', 'emulsiﬁers', '335.', 'the', 'amounts', 'graft', 'copolymers', 'form', 'acrylic', 'methacrylic', 'monomers', 'n-vinylpyrrolidone', 'depend', 'upon', 'nature', 'ph', 'emulsiﬁers', 'reaction', 'time', 'temperature', 'ceric', 'ions', 'form', 'graft', 'copolymers', 'various', 'macromolecules', 'redox', 'mechanism', 'the', 'reactions', 'illustrated', 'follows', 'r', 'oh', 'ce', 'r', 'oh', 'ceric', 'complex', 'r', 'oh', '(or', 'r', 'o', ')', 'ce', 'h', 'the', 'almost', 'exclusive', 'formation', 'free', 'radicals', 'polymeric', 'backbones', 'results', 'formations', 'many', 'products', 'close', 'free', 'homopolymers', '346', '350.', 'the', 'reactions', '96', 'graft', 'copolymers', '625', 'widely', 'used', 'formations', 'graft', 'copolymers', 'poly(vinyl', 'alcohol)', 'particularly', 'cellulose', 'starch', 'the', 'grafting', 'reaction', 'fails', 'however', 'attempted', 'polysaccharides', 'lack', 'free', 'hydroxyl', 'groups', 'second', 'third', 'carbons', 'this', 'led', 'speculation', '351', 'bond', 'carbons', 'cleaves', 'in', 'process', 'free', 'radicals', 'presumably', 'form', 'second', 'carbons', 'aldehyde', 'structures', 'third', 'carbons', 'glucose', 'units', 'this', 'point', 'view', 'however', 'generally', 'accepted', 'instead', 'proposed', 'likely', 'positions', 'attacks', 'ceric', 'ions', 'c1', 'carbons', 'glucoses', 'end', 'polysaccharide', 'chains', '352.', 'this', 'supported', 'observation', 'oxidation', 'cellulose', 'important', 'prerequisite', 'formation', 'graft', 'copolymers', '353.', 'graft', 'copolymerizations', 'redox', 'mechanism', 'also', 'take', 'place', 'aid', 'ions', 'this', 'includes', 'grafting', 'cellulose', 'backbones', 'ferrous', 'ions', 'hydrogen', 'peroxide', '354.', 'redox', 'grafting', 'reactions', 'also', 'take', 'place', 'nylon', 'polyester', 'for', 'instance', 'graft', 'copolymers', 'methyl', 'methacrylate', 'nylon', '6', 'prepared', 'manganic', 'cobaltic', 'ferric', 'ions', '355.', 'another', 'example', 'grafting', 'poly(glycidyl', 'methacrylate)', 'poly(ethylene', 'terephthalate)', 'ﬁbers', 'aid', 'ferrous', 'ion–hydrogen', 'peroxide', 'the', 'reaction', 'depends', 'concentration', 'monomer', 'hydrogen', 'peroxide', 'time', 'temperature', '356.', '965', 'photochemical', 'syntheses', 'graft', 'copolymers', 'photo', 'labile', 'groups', 'polymers', 'serve', 'sites', 'photoinitiated', 'graft', 'copolymerizations', 'for', 'instance', 'polymers', 'copolymers', 'vinyl', 'ketone', 'decompose', 'ultraviolet', 'light', 'presence', 'acrylonitrile', 'methyl', 'methacrylate', 'vinyl', 'acetate', 'graft', 'copolymers', 'form', '357:', 'o', 'o', 'o', 'o', 'o', 'ch3', 'the', 'free', 'radicals', 'unattached', 'backbone', 'polymers', 'like', 'methyl', 'radical', 'shown', 'also', 'initiate', 'polymerizations', 'considerable', 'amounts', 'homopolymers', 'form', 'well', 'in', 'instances', 'graft', 'copolymers', 'form', 'result', 'chain', 'transferring', 'takes', 'place', 'photodecomposition', 'photo', 'labile', 'materials', 'an', 'example', 'formation', 'graft', 'copolymers', 'polyacrylamide', 'natural', 'rubber', 'poly(vinyl', 'pyrrolidone)', 'dextrin', 'aid', 'benzophenone', 'ultraviolet', 'light', '358.', 'the', 'free', 'radicals', 'photodecomposition', 'benzophenone', 'react', 'polymers', 'chain', 'transferring', 'the', 'growth', 'acrylamide', 'subsequently', 'initiated', 'polymer', 'backbones', 'photo', 'tendering', 'dyes', 'used', 'manner', 'cellulose', '359.', 'thus', 'anthraquinone', 'dyes', 'adsorbed', 'cellulose', 'upon', 'irradiation', 'proton', 'abstractions', 'take', 'place', 'creating', 'initiating', 'radicals', 'backbone', 'polysaccharide', '626', '9', 'organic', 'reactions', 'polymers', 'o2s', 'o2s', 'o2s', 'o', 'o', 'o', 'o', 'oh', 'o', 'so2', 'hν', 'o2s', 'so2', 'o', 'o', 'so2', 'ho', 'oh', 'o', 'o', 'oh', 'oh', 'o', 'oh', 'so2', 'ho', 'monomer', 'o', 'graft', 'copolymer', 'it', 'believed', 'dye', 'mono', 'radicals', 'disproportionate', 'hydroquinones', 'quinones', 'transfer', 'reactions', 'solvent', 'lead', 'formations', 'homopolymers', 'this', 'gives', 'high', 'yields', 'graft', 'copolymers', 'methyl', 'methacrylate', 'cellulose', 'the', 'true', 'acrylonitrile', '360.', 'on', 'hand', 'small', 'quantities', 'graft', 'copolymers', 'form', 'styrene', 'vinyl', 'acetate', 'monomers', '360.', 'it', 'also', 'possible', 'form', 'graft', 'copolymers', 'surface', 'ﬁbers', 'coating', 'photoinitiators', 'like', 'benzophenone', 'together', 'monomer', 'irradiating', 'ultravio-', 'let', 'light', '414.', 'similar', 'action', 'anthraquinone', 'dyes', 'shown', 'benzophenone', 'excited', 'triplet', 'state', 'mainly', 'abstracts', 'hydrogens', 'forms', 'radicals', 'surface', '415.', '966', 'graft', 'copolymer', 'formation', 'aid', 'high-energy', 'radiation', 'high-energy', 'radiation', 'sources', 'include', 'gamma', 'rays', 'radioactive', 'elements', 'electron', 'beams', 'accelerators', 'gamma', 'rays', 'nuclear', 'reactors', 'the', 'energy', 'radiated', 'sources', 'sufﬁ-', 'ciently', 'high', 'rupturing', 'covalent', 'bonds', 'this', 'results', 'formations', 'free', 'radicals', 'several', 'different', 'methods', 'used', 'form', 'graft', 'copolymers', 'one', 'irradiation', 'open', 'air', 'the', 'free', 'radicals', 'form', 'scavenge', 'oxygen', 'form', 'peroxides', 'hydroperoxides', 'polymeric', 'chains', 'these', 'subsequently', 'decomposed', 'presence', 'monomers', 'form', 'graft', 'copolymers', 'when', 'decomposed', 'thermally', '361,', 'hydroperoxides', 'yield', 'much', 'greater', 'quantities', 'homopolymers', 'peroxides', 'when', 'however', 'decompositions', 'done', 'room', 'temperature', 'redox', 'mechanisms', 'formations', 'homopolymers', 'reduced', '361', '362.', 'another', 'method', 'irradiation', 'vacuum', 'this', 'results', 'formations', 'trapped', 'radicals', 'polymer', 'backbones', 'after', 'irradiation', 'polymers', 'heated', 'absence', 'oxygen', 'presence', 'vinyl', 'monomers', 'the', 'best', 'results', '96', 'graft', 'copolymers', '627', 'obtained', 'irradiations', 'done', 'low', 'temperatures', 'tg', 'polymers', 'high', 'degree', 'crystallinity', 'also', 'beneﬁcial', 'mobility', 'chains', 'results', 'loss', 'trapped', 'radicals', 'when', 'monomers', 'added', 'however', 'heat', 'must', 'applied', 'result', 'loss', 'radicals', 'the', 'third', 'method', 'mutual', 'irradiation', 'inert', 'atmosphere', 'polymers', 'monomers', 'together', 'the', 'polymers', 'either', 'swelled', 'dissolved', 'monomers', 'the', 'relative', 'sensitivities', 'two', 'species', 'monomer', 'polymer', 'radiation', 'important', 'factors', 'third', 'procedure', 'efﬁciency', 'grafting', 'depends', 'upon', 'formations', 'free', 'radicals', 'polymer', 'backbones', 'if', 'small', 'number', 'free', 'radicals', 'form', 'irradiations', 'produce', 'mainly', 'homopolymers', 'also', 'polymers', 'tend', 'degrade', 'irradiation', 'block', 'copolymers', 'form', 'instead', 'presence', 'solvents', 'chain-transferring', 'agents', 'tends', 'lower', 'amount', 'grafting', '363.', 'it', 'also', 'shown', '409', 'energetic', 'heavy', 'ions', 'also', 'produce', 'graft', 'copolymers', 'the', 'results', 'appear', 'similar', 'obtained', 'electron', 'beams', 'also', 'many', 'papers', 'reported', 'use', 'plasma', 'surface', 'modiﬁcation', 'ﬁlms', 'the', 'process', 'result', 'formation', 'graft', 'copolymers', 'accompanied', 'introduction', 'monomer', 'monomers', 'one', 'example', 'use', 'argon', 'plasma', 'graft', 'polyacrylamide', 'polyaniline', 'ﬁlms', '410.', 'the', 'near', 'ultraviolet', 'light', 'plasma', 'induces', 'reaction', 'other', 'monomers', 'grafted', 'reaction', '4-styrenesulfonic', 'acid', 'acrylic', 'acid', '410.', '967', 'preparation', 'graft', 'copolymers', 'ionic', 'chain-growth', 'step-growth', 'polymerization', 'reactions', 'both', 'anionic', 'cationic', 'mechanisms', 'used', 'form', 'graft', 'copolymers', 'a', 'typical', 'example', 'graft', 'copolymer', 'formation', 'anionic', 'mechanism', 'grafting', 'polyacrylonitrile', 'polystyrene', '364:', '2', 'na', 'nacl', 'na', 'cl', 'na', 'n', 'n', 'another', 'example', 'ionic', 'graft', 'copolymerization', 'reaction', 'carried', 'pendant', 'oleﬁnic', 'groups', 'using', 'ziegler-natta', 'catalysts', 'coordinated', 'anionic', 'type', 'polymerization', '365.', 'the', 'proce-', 'dure', 'consists', 'two', 'steps', 'in', 'ﬁrst', 'one', 'diethylaluminum', 'hydride', 'added', 'across', 'double', 'bonds', 'the', 'product', 'subsequently', 'treated', 'transition', 'metal', 'halide', 'this', 'yields', 'active', 'catalyst', 'polymerizations', 'a-oleﬁns', 'by', 'method', 'polyethylene', 'polypropylene', 'grafted', 'butadiene', 'styrene', 'copolymers', 'propylene', 'monomer', 'polymerization', 'results', 'formations', 'isotactic', 'polymeric', 'branches', '628', '9', 'organic', 'reactions', 'polymers', 'al(c2h5)2h', 'al', 'c2h5', 'c2h5', 'ticl4(tic3vocl3)', 'n', 'o', 'another', 'example', 'formation', 'graft', 'copolymers', 'formaldehyde', 'starch', 'dextrin', 'poly', '(vinyl', 'alcohol)', '366', '367.', 'this', 'procedure', 'also', 'carried', 'two', 'steps', 'potassium', 'naphthalene', 'ﬁrst', 'reacted', 'backbone', 'polymer', 'dimethylsulfoxide', 'the', 'formaldehyde', 'introduced', 'gaseous', 'form', 'alkoxide', 'solution', 'a', 'similar', 'reaction', 'used', 'form', 'graft', 'copolymers', 'poly(ethylene', 'oxide)', 'cellulose', 'acetate', '391.', 'poly(ethylene', 'oxide)', 'also', 'grafted', 'starch', 'for', 'instance', 'preformed', 'polymer', '392', 'terminated', 'chloroformate', 'end', 'groups', 'used', 'potassium', 'starch', 'alkoxide', 'o', 'starch', 'o', 'k', 'cl', 'o', 'o', 'starch', 'o', 'o', 'o', 'n', 'o', 'o', 'cl', 'o', 'n', 'o', 'cl', 'the', 'products', 'water-soluble', 'the', 'efﬁciency', 'coupling', 'process', 'however', 'decreases', 'increase', 'dp', 'poly(ethylene', 'oxide)', 'lithiated', 'polystyrene', 'reacts', 'readily', 'halogen-bearing', 'polymers', 'like', 'polychlorotriﬂuor-', 'oethylene', '411.', 'this', 'utilized', 'formation', 'graft', 'copolymers', 'the', 'reactions', 'conducted', 'solutions', 'well', 'preparations', 'surface', 'grafts', 'ﬁlms', '411.', 'an', 'example', 'cationic', 'grafting', 'reaction', 'formation', 'graft', 'copolymers', 'polyisobutylene', 'polystyrene', 'backbones', '393.', 'polystyrene', 'chloromethylated', 'reacted', 'aluminum', 'bromide', 'carbon', 'disulﬁde', 'solution', 'this', 'followed', 'introduction', 'isobutylene', 'cl', 'o', 'cl', '96', 'graft', 'copolymers', '629', 'etc', 'ch2', 'albr3cl', 'albr3cl', 'the', 'however', 'yields', '5–18', 'graft', 'copolymer', 'even', '(cid3)60(cid1)c', 'it', 'possible', 'considerable', 'amounts', 'cross-linking', 'occur', 'reaction', 'conditions', 'may', 'perhaps', 'reason', 'low', 'yield', '393.', 'another', 'example', 'grafting', 'cellulose', 'bf3', 'adsorbed', 'surface', 'polymer', 'it', 'reacts', 'hydroxy', 'groups', 'yields', 'reactive', 'sites', 'cationic', 'polymerization', 'a-methyl', 'styrene', 'isobutylene', '402.', 'these', 'reactions', 'carried', '(cid3)80(cid1)c', 'cationic', 'graft', 'copolymerizations', 'trioxane', 'carried', 'help', 'reactive', 'c–o–c', 'links', 'number', 'polymers', 'like', 'poly(vinyl', 'acetate)', 'poly(ethylene', 'terephthalate)', 'poly(vinyl', 'butyral)', '403.', 'many', 'graft', 'copolymers', 'also', 'formed', 'ring', 'opening', 'polymerizations', '404.', 'the', 'reactions', 'active', 'hydrogens', 'pendant', 'groups', 'like', 'hydroxyl', 'carboxyl', 'amine', 'amide', 'thiol', 'others', 'initiate', 'ring', 'opening', 'polymerizations', 'an', 'example', 'preparation', 'graft', 'copolymers', 'ethylene', 'oxide', 'styrene', '405.', 'copolymers', 'styrene', '1–2', 'hydrolyzed', 'vinyl', 'acetate', '(vinyl', 'alcohol', 'hydrolysis)', 'initiate', 'polymerizations', 'ethylene', 'oxide', 'graft', 'copolymers', 'form', 'recently', 'solutions', 'polysilanes', 'treated', 'controlled', 'amounts', 'triﬂic', 'acid', '(cf3so2oh)', 'ch2cl2', 'afterwards', 'tetrahydrofuran', 'this', 'yielded', 'graft', 'copolymer', 'poly(tetramethylene', 'oxide)', 'polysilane', 'backbones', '412.', 'an', 'interesting', 'series', 'papers', 'published', 'kennedy', 'coworkers', 'use', 'alkylaluminum', 'compounds', 'initiators', 'graft', 'copolymerizations', '366', '367.', 'allylic', 'chlorines', 'form', 'active', 'carbon', 'cations', 'presence', 'initiator', 'this', 'also', 'true', 'macromolecular', 'carbon', 'cation', 'sources', '402.', 'as', 'result', 'high', 'grafting', 'efﬁciency', 'achieved', 'many', 'different', 'polymerizations', 'using', 'macromolecular', 'cationogens', 'alkylaluminum', 'compounds', 'in', 'instances', 'formation', 'graft', 'copolymers', 'greater', '90.', 'the', 'grafting', 'reaction', 'illustrated', 'follows', '367.', 'n', 'o', 'al(c2h5)cl', 'cl', 'n', 'o', 'n', '630', '9', 'organic', 'reactions', 'polymers', 'the', 'temperatures', 'reactions', 'nature', 'aluminum', 'compounds', 'important', 'synthetic', 'variables', '367.', 'on', 'hand', 'many', 'graft', 'copolymerizations', 'cationic', 'mechanism', 'suffer', 'low', 'grafting', 'efﬁciencies', 'they', 'also', 'often', 'accompanied', 'large', 'formations', 'homopolymers', 'use', 'however', 'living', 'cationic', 'processes', 'appears', 'overcome', 'drawback', 'an', 'illustration', 'another', 'preparation', 'graft', 'copolymer', 'polyisobutylene', 'polystyrene', 'backbone', '413:', 'bcl3', 'ch2cl', 'isobutylene', 'o', 'o', 'bcl3', 'o', 'o', 'x', '968', 'miscellaneous', 'graft', 'copolymerizations', 'in', 'rather', 'interesting', 'reaction', 'ethylene', 'oxide', 'graft-copolymerized', 'nylon', '66', '406.', 'formation', 'graft', 'copolymer', 'greatly', 'enhances', 'ﬂexibility', 'material', 'high', 'melting', 'point', 'nylon', 'still', 'maintained', 'thus', 'nylon', '66', 'contains', 'much', '50', 'weight', 'grafted', 'poly(ethylene', 'oxide)', 'still', 'melts', '221(cid1)c', 'apparent', 'tg', '(cid3)40(cid1)c', 'it', 'also', 'maintains', 'ﬂexibility', 'useful', 'properties', 'wide', 'range', 'temperatures', '406:', 'o', 'o', 'n', '6', '4', 'n', 'h', 'o', 'o', 'h', 'an', 'entirely', 'different', 'procedure', 'used', 'form', 'graft', 'copolymers', 'step-growth', 'polymeri-', 'zation', '347.', 'formaldehyde', 'condensed', 'either', 'phenol', 'p-cresol', 'p-nonyl', 'phenol', 'resin', 'attached', 'either', 'nylon', '6', 'nylon', '66', 'nylon', '610', 'nylon', '11', 'backbones', 'initially', 'formaldehyde', '97', 'block', 'copolymers', '631', 'prereacted', 'excess', 'phenol', 'presence', 'nylon', 'without', 'catalyst', 'temperatures', 'high', 'enough', 'cause', 'condensation', 'this', 'followed', 'addition', 'toluenesulfonic', 'acid', 'lower', 'temperatures', 'at', 'point', 'free', 'formaldehyde', 'longer', 'present', 'reaction', 'mixture', 'cause', 'gelation', 'novolac', 'molecules', 'attach', 'nylon', 'backbones', 'the', 'excess', 'phenol', 'washed', 'away', 'leaving', 'pure', 'graft', 'copolymers', '347.', 'yagci', 'coworkers', 'reported', 'special', 'preparation', 'perfectly', 'alternating', 'poly(p-phenylene)', 'amphilitic', 'graft', 'copolymers', 'combination', 'controlled', 'free-radical', 'polymerization', 'suzuki', 'coupling', 'process', '345.', '97', 'block', 'copolymers', 'these', 'polymers', 'consist', 'two', 'strands', 'different', 'polymers', 'attached', 'there', 'appear', 'general', 'stipulation', 'minimum', 'size', 'block', 'there', 'appear', 'however', 'general', 'agreement', 'sequence', 'larger', 'units', 'in', 'describing', 'block', 'copolymer', 'helpful', 'following', 'structural', 'parameters', 'available', 'characterize', 'material', '1', 'copolymer', 'sequence', 'distribution', 'well', 'length', 'number', 'blocks', '2', 'the', 'chemical', 'nature', 'blocks', '3', 'the', 'average', 'molecular', 'weight', 'molecular', 'weight', 'distribution', 'blocks', 'copolymer', 'block', 'copolymers', 'particularly', 'a—b—a', 'type', 'exhibit', 'properties', 'quite', 'different', 'random', 'copolymers', 'even', 'mixtures', 'homopolymers', 'the', 'physical', 'behavior', 'block', 'copolymers', 'related', 'solid', 'state', 'morphology', 'phase', 'separation', 'occurs', 'often', 'copolymers', 'this', 'result', 'dispersed', 'phases', 'consisting', 'one', 'block', 'dispersed', 'continuous', 'matrix', 'second', 'block', 'such', 'dispersed', 'phases', 'hard', 'domains', 'either', 'crystalline', 'glassy', 'matrices', 'soft', 'rubber-like', 'an', 'interesting', 'example', 'block', 'copolymers', 'work', 'de', 'ruijter', 'et', 'al', '348,', 'prepared', 'series', 'block', 'copolymers', 'contain', 'rigid', 'liquid', 'crystal', 'forming', 'blocks', 'poly(p-phenylene', 'terephthalamide)', 'ﬂexible', 'blocks', 'hexamethylene', 'adipamide', 'the', 'polymers', 'prepared', 'one-pot', 'procedure', 'low-temperature', 'polycondensation', 'reaction', 'n-methyl-2-pyrrolidone', '971', 'block', 'copolyesters', 'two', 'polyester', 'homopolymers', 'react', 'form', 'block', 'copolymers', 'molten', 'state', 'temperatures', 'high', 'enough', 'ester', 'interchange', '414.', 'as', 'reaction', 'mixtures', 'stirred', 'heated', 'interchanges', 'initially', 'involve', 'large', 'segments', 'with', 'time', 'however', 'smaller', 'smaller', 'segments', 'form', 'transesteriﬁcations', 'continue', 'to', 'prevent', 'eventual', 'formation', 'random', 'copolymers', 'reactions', 'limited', 'time', 'ester', 'interchange', 'retarded', 'particularly', 'esteriﬁcation', 'catalysts', 'like', 'zinc', 'calcium', 'acetate', 'present', 'addition', 'phosphorous', 'acid', 'triphenyl', 'phosphite', '415.', 'this', 'improves', 'chances', 'forming', 'block', 'copolymers', 'the', 'procedure', 'applied', 'preparation', 'block', 'copolymers', 'poly(ethylene', 'terephthalate)', 'poly(ethylene', 'maleate)', 'poly(ethylene', 'citraconate)', 'poly(ethylene', 'itaconate)', '416.', 'with', 'ester', 'interchange', 'catalysts', 'like', 'titanium', 'alkoxides', 'complexes', 'melt', 'randomization', 'may', 'inhibited', 'adding', 'arsenic', 'pentoxide', 'deactivates', '417.', '632', '9', 'organic', 'reactions', 'polymers', 'block', 'copolyesters', 'also', 'form', 'reactions', 'hydroxy', 'acid', 'chloride-terminated', 'prepolymers', '419.', 'this', 'take', 'place', 'melt', 'solution', 'solvents', 'chlorobenzene', 'o-dichlorobenzene', '418.', 'for', 'relatively', 'inactive', 'acid', 'chlorides', 'like', 'terephthaloyl', 'chloride', 'high', 'reaction', 'temperatures', 'required', 'phosgene', 'also', 'reacts', 'hydroxy-terminated', 'polyesters', 'form', 'block', 'copolymers', 'the', 'reactions', 'must', 'carried', 'inert', 'solvents', 'block', 'copolyethers', 'also', 'form', 'readily', 'ester', 'interchange', 'reactions', 'low', 'molecular', 'weight', 'diesters', '348:', 'o', 'o', '(n1)', 'o', 'o', 'n', 'ho', 'h', 'o', 'oh', '4', 'oh', 'n', 'o', 'o', 'o', 'o', 'o', '2', 'o', 'o', 'n', 'o', '4', '(2n', '2)', 'ch3oh', 'acetates', 'tin', 'lead', 'manganese', 'antimony', 'zinc', 'well', 'esters', 'orthotitanates', 'catalyze', 'reactions', '421.', 'optimum', 'temperatures', 'reactions', '230', '260(cid1)c', '003–1', 'mm', 'hg', 'pressure', '421.', 'block', 'copolymers', 'also', 'form', 'ring', 'opening', 'polymerizations', 'lactones', 'carboxyl-terminated', 'macromolecular', 'initiators', 'used', '422:', 'na', 'co2', 'o', 'na', 'o', 'o', 'o', 'o', 'o', 'x', 'o', 'o', '972', 'block', 'copolyamides', 'simple', 'melt', 'blending', 'reactions', 'also', 'applied', 'preparations', 'block', 'copolyamides', 'similarly', 'process', 'polyesters', 'with', 'time', 'total', 'equilibrium', 'conditions', 'also', 'gradually', 'achieved', 'melt', '423.', 'interfacial', 'polycondensation', 'also', 'useful', 'preparation', 'block', 'copolymers', 'when', 'mixed', 'diacid', 'chlorides', 'andor', 'mixed', 'diamines', 'reacted', 'active', 'diacid', 'chlorides', 'andor', 'diamines', 'react', 'preferentially', 'blocks', 'form', 'in', 'addition', 'possible', 'carry', 'growth', 'one', 'segments', 'ﬁrst', 'fairly', 'large', 'size', 'follow', 'addition', 'comonomers', '424.', '97', 'block', 'copolymers', '633', '973', 'polyurethane-polyamide', 'block', 'copolymers', 'these', 'block', 'copolymers', 'formed', 'many', 'ways', 'one', 'technique', 'prereact', 'diamine', 'diacid', 'chloride', 'the', 'polyamide', 'forms', 'treated', 'bischloroformate', 'attach', 'polyure-', 'thane', 'blocks', '425.', 'the', 'process', 'reversed', 'polyurethane', 'formed', 'ﬁrst', 'attached', 'polyamide', 'blocks', '425.', '974', 'polyamide-polyester', 'block', 'copolymers', 'block', 'copolymers', 'consisting', 'polyamide', 'polyester', 'blocks', 'form', 'melt', 'blending', '426.', 'the', 'reactions', 'probably', 'involve', 'aminolyses', 'terminal', 'ester', 'groups', 'polyesters', 'terminal', 'amine', 'groups', 'polyamides', 'ester', 'interchange', 'catalysts', 'accelerate', 'reaction', '427.', 'block', 'polyester-polyamides', 'also', 'form', 'initiation', 'ring', 'opening', 'polymerizations', 'lactones', 'terminal', 'amine', 'groups', 'polyamides', '428:', 'nh2', 'n', 'o', 'o', 'o', 'n', 'h', 'o', 'oh', 'n', 'if', 'polyamide', 'terminal', 'amine', 'groups', 'ends', 'triblock', 'copolymers', 'form', '975', 'polyurethane', 'ionomers', 'these', 'materials', 'reviewed', 'special', 'class', 'block', 'copolymers', '433.', 'they', 'linear', 'polyaddition', 'products', 'diisocyanates', 'containing', 'nonrandom', 'distributions', 'ionic', 'centers', 'the', 'preparations', 'similar', 'polyurethane', 'elastomers', 'described', 'chap', '6', 'one', 'example', 'material', 'prepared', 'high', 'molecular', 'weight', 'polyester', 'free', 'ionic', 'centers', 'terminated', 'isocyanate', 'groups', 'end', 'the', 'prepolymer', 'coupled', 'n-methylamino-220-diethanol', 'form', 'segmented', 'polymer', 'n', 'n', 'similar', 'products', 'form', 'isocyanate-terminated', 'polyethers', 'this', 'material', 'cross-linked', 'difunctional', 'quaternizing', 'agents', '14-bis(chloromethyl)', 'benzene', '434:', 'cl', 'n', 'ch2', 'ch2', 'n', 'cl', '634', '9', 'organic', 'reactions', 'polymers', 'the', 'products', 'cationic', 'ionomers', 'anionic', 'ionomers', 'form', 'similarly', 'coupling', 'chains', 'bifunctional', 'anionic', '“chain', 'lengtheners”', '435:', 'o', 'n', 'n', 'o', 'h2n', 'nh2', 'o', 'o', 'o', 'n', 'h', 'o', 'o', 'n', 'h', 'n', 'h', 'o', 'n', 'h', 'n', 'because', 'interactions', 'chains', 'polyurethane', 'ionomers', 'similar', 'properties', 'cross-linked', 'elastomers', 'in', 'solution', 'strongly', 'associated', '976', 'block', 'copolymers', 'poly(a-oleﬁn)s', 'these', 'block', 'copolymers', 'form', 'readily', 'appropriate', 'ziegler-natta', 'catalysts', 'used', '436.', 'this', 'discussed', 'chaps', '4', '6', 'in', 'addition', 'special', 'technique', 'preparations', 'block', 'copolymers', 'at', 'outset', 'reaction', 'one', 'monomer', 'used', 'feed', 'a', 'typical', 'catalyst', 'might', 'a-ticl3(c2h5)3al', 'after', 'ﬁrst', 'monomer', 'bubbled', 'short', 'period', 'perhaps', '5', 'min', 'addition', 'stopped', 'unreacted', 'monomer', 'removed', 'evacuation', 'ﬂushing', 'the', 'second', 'monomer', 'introduced', 'may', 'also', 'bubbled', 'period', 'the', 'addition', 'second', 'monomer', 'may', 'stopped', 'second', 'monomer', 'evacuated', 'whole', 'thing', 'might', 'repeated', 'if', 'equal', 'length', 'block', 'desired', 'addition', 'times', 'monomer', 'may', 'varied', 'adjust', 'different', 'rates', 'polymerization', '436.', 'kuhlman', 'klosin', '436', 'reported', 'forming', 'multiblock', 'copolymers', 'polyethylene', 'tuning', 'block', 'composition', 'catalyst', 'selection', 'chain', 'shuttling', 'fast', 'exchange', 'growing', 'polymer', 'chains', 'among', 'catalyst', 'centers', 'mediated', 'chain', 'shuttling', 'agent', '(csa)', 'enabled', 'production', 'ethylene-based', 'oleﬁn', 'block', 'copolymers', 'diethyl', 'zinc', 'used', 'chain', 'shufﬂing', 'agent', 'thus', 'copolymerization', 'ethylene', 'a-oleﬁn', 'certain', 'catalyst', 'pairs', 'pretense', 'csa', 'produces', 'blocks', 'alternately', 'highly', 'lightly', 'branched', 'chain', 'shutling', 'polymerizations', 'bet', 'conducted', 'continuous', 'reactors', 'a', 'catalyst', 'system', 'derived', 'titanocene', 'complex', 'methyl-aluminoxane', '(see', 'chap', '5)', 'used', 'polymerize', 'propylene', 'depending', 'polymerization', 'conditions', 'produce', 'block', 'copolymer', 'crystalline', 'amorphous', 'elastomeric', 'polypropylene', '440:', 'ti', 'site', '1', 'site', '2', 'ti', '↓', 'isotactic', 'sequences', '↓', 'atactic', 'sequences', '97', 'block', 'copolymers', '635', 'in', 'process', 'formation', 'crystalline', 'domains', 'involves', 'consecutive', 'insertions', 'one', 'lateral', 'coordination', 'sites', 'catalyst', 'give', 'rise', 'isotactic', 'sequences', 'whereas', 'consecutive', 'insertions', 'site', '(2)', 'give', 'rise', 'atactic', 'amorphous', 'sequences', 'interconversion', 'two', 'states', 'must', 'occur', 'within', 'lifetime', 'given', 'polymer', 'chain', 'order', 'generate', 'physically', 'cross-linked', 'network', 'believed', 'occur', 'via', 'occasional', 'isomerizations', 'polymer', 'chains', '(ie,', 'interconversion', 'metal', 'center)', 'preparation', 'oscillating', 'catalyst', 'yields', 'elastomeric', 'polypropylene', 'also', 'reported', 'others', '441.', '977', 'simultaneous', 'use', 'free', 'radical', 'ionic', 'chain-growth', 'polymerizations', 'this', 'technique', 'allows', 'formation', 'many', 'different', 'types', 'block', 'copolymers', '437.', 'lithium', 'metal', 'used', 'initiate', 'polymerizations', 'solvents', 'varying', 'polarity', 'monomers', 'like', 'styrene', 'a-', 'methylstyrene', 'methyl', 'methacrylate', 'butyl', 'methacrylate', '2-vinylpyridine', '4-vinyl', 'pyridine', 'acryloni-', 'trile', 'methyl', 'acrylate', 'used', 'the', 'mechanism', 'initiation', 'depends', 'upon', 'formation', 'ion-', 'radicals', 'reactions', 'lithium', 'double', 'bonds', 'ch2', 'li', 'li', 'propagation', 'reactions', 'proceed', 'active', 'sites', 'radical', 'carbanion', 'when', 'two', 'different', 'monomers', 'present', 'free-radical', 'propagation', 'favors', 'formation', 'copolymers', 'propagation', 'end', 'favors', 'formation', 'homopolymers', 'there', 'tendency', 'therefore', 'form', 'ab—b', 'type', 'block', 'copolymers', 'several', 'publications', 'appeared', 'recently', 'describe', 'use', 'controlled“living”', 'radical', 'polymerizations', 'form', 'block', 'copolymers', 'thus', 'jerome', 'et', 'al', '435', 'described', 'formation', 'block', 'copolymers', 'using', 'initiator', 'capable', 'initiating', 'simultaneously', 'dual', 'living', 'polymerizations', 'n', 'o', 'ho', 'o', 'o', 'n', 'h', 'o', 'o', 'o', 'n', 'o', 'n', '636', '9', 'organic', 'reactions', 'polymers', 'in', 'similar', 'manner', 'yoshida', 'osagawa', '436', 'synthesized', 'poly(e-caprolactone)', '2266-', 'tetramethylpiperdine-1-oxyl', '(tempo)', 'one', 'end', 'anionic', 'polymerization', 'caprolactone', 'using', 'aluminum', 'tri(4-oxy-tempo)', 'initiator', 'the', 'tempo-supported', 'polycaprolactone', 'behaved', 'poly-', 'meric', 'counter', 'radical', 'controlled“living”', 'radical', 'polymerization', 'styrene', 'form', 'block', 'copolymers', '436.', 'also', 'kotani', 'et', 'al', '437', 'reported', 'using', 'controlled“living”', 'atom', 'transfer', 'radical', 'polymerization', '(atrp)', 'form', 'block', 'copolymers', 'ethyl', 'n-butyl', 'methacrylates', 'a', 'ternary', 'initiating', 'system', 'consists', 'carbon', 'tetrachloride', 'tris(triphenyl-phosphine)ruthenium', 'dichloride', 'rucl2(pph3)3,', 'aluminum', 'compounds', 'produced', 'aba', 'triblock', 'copolymers', '437.', 'huang', 'coworkers', '437', 'reported', 'preparation', 'series', 'well-deﬁned', 'amphiphilic', 'block', 'copolymers', 'containing', 'conjugated', 'poly(ﬂuorene)', '(pf)', 'block', 'coil', 'like', 'poly(2-(dimethylamino)', 'ethyl', 'methacrylate)', '(pdmaema)', 'the', 'block', 'copolymers', 'synthesized', 'atrp', 'the', 'reactions', 'initiated', '2-bromoisobutyrate', 'end-capped', 'macroinitiator', 'using', 'cucl', '11471010-hexamethyltriethylenetetramine', 'catalyst', 'matron', 'grubbs', 'formed', 'block', 'copolymers', 'combining', 'ring', 'opening', 'metathesis', 'polymeriza-', 'tion', 'atrp', '437.', 'use', 'made', 'fast', 'initiating', 'ruthenium', 'metathesis', 'catalyst', 'form', 'three', 'functionized', 'atrp', 'different', 'monotelechelic', 'poly(oxa)norbornenes', 'the', 'polymerizations', 'styrene', 'tert-butyl', 'acrylate', 'followed', 'ends', 'coca', 'et', 'al', '438', 'showed', 'general', 'method', 'transforming', 'living', 'ring', 'opening', 'metathesis', 'polymerization', 'controlled“living”', 'atom', 'transfer', 'polymerizations', 'form', 'block', 'copolymers', 'ring', 'opening', 'polymerizations', 'norbornene', 'dicyclopentadiene', 'followed', 'witting-like', 'reactions', 'p-(bromomethyl)benzaldehyde', 'form', 'efﬁcient', '(atp)', 'macroninitiators', 'formation', 'block', 'copolymers', 'styrene', '478:', 'n', 'o', 'mo', 'n', 'o', 'o', 'mo', 'n', 'o', 'n', 'other', 'cationic', 'ring', 'opening', 'polymerizations', 'also', 'transformed', 'atrp', 'yield', 'block', 'copolymers', '439.', 'thus', 'formation', 'block', 'copolymers', 'initiated', 'poly(tetramethylene', 'glycol)', 'containing', 'one', 'bromopropionyl', 'end', 'group', 'these', 'used', 'form', 'block', 'copolymers', 'atp', 'polymerization', 'styrene', 'methyl', 'methacrylate', 'methyl', 'acrylate', 'ho', 'o', 'n', 'br', 'o', 'o', 'cubrnbipy', 'toluene100°c', 'block', 'copolymer', '97', 'block', 'copolymers', '637', '978', 'preparation', 'block', 'copolymers', 'homogeneous', 'ionic', 'copolymerization', 'formation', 'block', 'copolymers', 'method', 'depends', 'upon', 'ability', 'form', '“living”', 'chain', 'ends', 'among', 'anionic', 'systems', 'following', 'polymerizations', 'ﬁt', 'requirement', '1', 'polymerizations', 'nonpolar', 'monomers', 'alkali', 'metal-aromatic', 'electron', 'transfer', 'initiators', '2', 'polymerizations', 'nonpolar', 'monomers', 'organolithium', 'compounds', 'hydrocarbon', 'solvents', '399.', '3', 'acrylonitrile', 'polymerizations', 'dimethyl', 'formamide', 'initiated', 'sodium', 'triethylthioisopropox-', 'ethers', '398.', 'yaluminate', '(cid3)78(cid1)c', '340.', '4', 'copolymerizations', 'hexaﬂuoroacetone', 'cyclic', 'oxides', 'initiated', 'csf', '341.', '5', 'polymerization', 'alkyl', 'isocyanates', 'initiated', 'organoalkali', 'species', 'hydrocarbons', '(cid3)78(cid1)c', '342.', 'among', 'cationic', '“living”', 'polymerizations', 'used', 'block', 'copolymer', 'formation', '1', 'polymerizations', 'isobutylene', '361', 'andor', 'vinyl', 'ethers', '363', 'appropriate', 'catalysts', 'this', 'includes', 'formation', 'block', 'copolymers', 'two', 'types', 'monomers', '365.', '2', 'polymerizations', 'tetrahydrofuran', 'aid', 'chlorobenzene', 'diazonium', 'hexaﬂuorophosphate', '343,', 'triphenylmethyl', 'hexachloroantimonate', '344,', 'phosphorus', 'pentaﬂuoride', '445.', '3', 'polymerization', 'p-methyl', 'styrene', 'n-vinylcarbazole', 'indene', 'appropriate', 'catalysts', 'the', 'preparations', 'anionic', 'mechanism', 'a—b—a', 'type', 'block', 'copolymers', 'styrene', 'butadiene', 'carried', 'styrene', 'polymerized', 'ﬁrst', 'use', 'alkyl', 'lithium', 'initiators', 'hydrocarbon', 'solvents', 'usually', 'good', 'choice', 'one', 'seeks', 'form', 'greatest', 'amount', 'cis-14', 'microstructure', '346.', 'this', 'discussed', 'chap', '4', 'it', 'difﬁcult', 'however', 'form', 'block', 'copolymers', 'methyl', 'methacrylate', 'styrene', '“living”', 'methyl', 'methacrylate', 'polymers', 'fail', 'initiate', 'polymerizations', 'styrene', '347.', 'the', 'poly(methyl', 'methacrylate)', 'anions', 'may', 'sufﬁciently', 'basic', 'initiate', 'styrene', 'polymerizations', '345.', 'a', '“living”', 'cationic', 'polymerization', 'tetrahydrofuran', 'using', 'bh3', 'initiator', 'presence', 'epichlorohydrin', '33-bis(chloromethyl)oxacyclo-butane', '348,', 'results', 'formation', 'block', 'copolymers', 'two', 'types', 'form', 'one', 'a—b', 'type', 'it', 'consists', 'polytetrahydrofuran', 'blocks', 'attached', 'blocks', 'poly(33-bis(chloromethyl)oxacyclo-butane)', 'the', 'one', 'a—ab—b', 'type', '348.', 'the', 'preparation', 'well-deﬁned', 'sequential', 'copolymers', 'anionic', 'mechanism', 'explored', 'utilized', 'commercially', 'time', 'initially', 'cationic', 'methods', 'received', 'less', 'attention', 'demonstrated', 'kennedy', '424', 'large', 'variety', 'block', 'copolymers', 'formed', 'the', 'key', 'kennedy’s', 'early', 'work', 'tight', 'control', 'polymerization', 'reaction', 'the', 'initiation', 'propagation', 'events', 'must', 'fundamentally', 'similar', 'although', 'identical', '424:', 'ion', 'generation', 'cationization', 'propagation', 'rx', 'þ', 'mexn', 'ð', 'r(cid5)mex(cid6)', 'nþ1', 'r', 'r', 'r', 'in', 'scheme', 'chain', 'transfer', 'monomer', 'must', 'absent', 'termination', 'well', 'deﬁned', '9', 'organic', 'reactions', 'polymers', '638', 'termination', 'r', 'mexn1', 'r', 'x', 'mexn', 'x', 'halogen', 'me', 'metal', 'different', 'polymerizations', 'this', 'allows', 'formation', 'macromolecules', 'terminal', 'halogens', 'they', 'used', 'initiate', 'new', 'three', 'methods', 'developed', 'overcome', 'transfer', 'monomer', '424.', 'these', '(1)', 'use', 'inifers', '(2)', 'use', 'proton', 'traps', '(3)', 'establishing', 'conditions', 'rate', 'termination', 'much', 'faster', 'rate', 'transfer', 'monomer', 'the', 'ﬁrst', 'one', 'inifer', 'method', 'particularly', 'useful', 'formation', 'block', 'copolymers', 'it', 'allows', 'preparation', 'head', 'end', '(a', 'o)', 'functionalized', 'telechelic', 'polymers', 'bifunctional', 'initiators', 'transfer', 'agents', '(inifers)', 'used', 'the', 'following', 'illustrates', 'concept', '424:', 'ion', 'generation', 'xrx', 'þ', 'mexn', 'ð', 'xr(cid5)', 'þ', 'mex(cid6)', 'nþ1', 'cationization', 'propagation', 'xr', 'xr', 'xr', 'n', 'n-1', 'chain', 'transfer', 'inifers', 'xr', 'xrx', 'n-1', 'xr', 'x', 'xr', 'n-1', 'xr', 'n-1', 'r', 'ex', 'n-1', '97', 'block', 'copolymers', '639', 'in', 'scheme', 'inifer', 'xrx', 'usually', 'organic', 'dihalide', 'if', 'chain', 'transferring', 'inifer', 'faster', 'chain', 'transferring', 'monomer', 'polymer', 'end', 'groups', 'become', 'exclusively', 'terminated', 'halogens', 'it', 'also', 'possible', 'carry', '“living”', 'cationic', 'polymerization', 'isobutylene', 'initiated', 'difunctional', 'initiator', '435.', 'this', 'results', 'formation', 'bifunctional', '“living”', 'segments', 'polyisobutylene', 'soft', 'rubbery', 'upon', 'completions', 'polymerization', 'another', 'mono-', 'mer', 'one', 'yields', 'stiff', 'segments', 'high', 'tg', 'like', 'indene', 'introduced', 'living', 'charge', 'polymerization', 'second', 'monomer', 'initiated', 'ends', 'formed', 'polyisobutylene', 'when', 'reaction', 'complete', 'polymerization', 'quenched', 'preparations', 'variety', 'triblock', 'star', 'block', 'polymers', 'described', '435.', 'a', 'technique', 'developed', 'introducing', 'cationic', 'anionic', 'transformation', '438.', 'a', '“living”', 'carbocationic', 'polymerization', 'isobutylene', 'carried', 'ﬁrst', 'after', 'complete', 'ends', 'chains', 'quantitatively', 'transformed', 'polymerization-active', 'anions', 'the', 'additional', 'blocks', 'built', 'anionic', 'polymerization', 'a', 'triblock', 'polymer', 'poly(methyl', 'methacrylate)-', 'polyisobutylene-poly(methyl', 'methacrylate)', 'thus', 'formed', 'the', 'transformation', 'takes', 'several', 'steps', 'in', 'ﬁrst', 'one', 'compound', 'like', 'toluene', 'friedel-craft', 'alkylated', 'ao-di-tert-chloropolyi-', 'sobutylene', 'the', 'ditolylpolyisobutylene', 'forms', 'lithiated', 'step', 'two', 'form', 'ao-di-', 'benzyllithium', 'polyisobutylene', 'it', 'reacted', '11-diphenylethylene', 'give', 'corresponding', 'dianion', 'after', 'cooling', '(cid3)78(cid1)c', 'dilution', 'methyl', 'methacrylate', 'monomer', 'introduced', 'second', 'polymerization', '438', 'step', '3', 'formation', 'block', 'copolymers', 'polymers', 'functional', 'end', 'groups', 'used', 'many', 'ways', 'in', 'anionic', 'polymerization', 'various', 'technique', 'developed', 'terminating', 'chain', 'growth', 'reactive', 'end', 'groups', 'these', 'end', 'groups', 'allow', 'subsequent', 'formations', 'many', 'different', 'block', 'copolymers', 'one', 'active', 'terminal', 'group', 'toluene', 'diisocyanate', '439.', 'the', 'isocyanate', 'group', 'located', 'ortho', 'methyl', 'group', 'considerably', 'less', 'reactive', 'toward', 'lithium', 'species', 'due', 'steric', 'hindrance', '438.', 'the', 'unreacted', 'isocyanate', 'group', 'used', 'attachment', 'various', 'polymers', 'terminated', 'hydroxy', 'carboxy', 'amine', 'groups', 'other', 'functional', 'compounds', 'used', 'reactions', 'alkyl', 'aryl', 'halides', 'succinic', 'anhydride', 'n-bromophthalimide', '448,', 'chlorosilanes', '449.', 'because', 'block', 'copolymers', 'often', 'offer', 'properties', 'unattainable', 'simple', 'blends', 'random', 'copolymers', '364,', 'many', 'efforts', 'made', 'combine', 'dissimilar', 'materials', 'like', 'hydrophilic', 'hydrophobic', 'hard', 'soft', 'segments', 'shown', 'earlier', 'one', 'paper', '432', 'describes', 'formation', 'block', 'copolymers', 'containing', 'helical', 'polyisocyanide', 'elastomeric', 'polybutadiene', 'compound', '(z3-c3h5)-ni(oc(o)cf3)2', 'used', 'carry', '“living”', 'polymerization', 'butadiene', 'followed', 'polymerization', 'tert-butyl', 'isocyanide', 'helical', 'polymer', '979', 'special', 'reactions', 'preparation', 'block', 'copolymers', 'a', 'special', 'case', 'use', 'witting', 'reaction', 'poly(p-phenylene', 'pentadienylene)', '415', 'prepared', 'reaction', 'ﬁrst', 'this', 'utilized', 'preparation', 'block', 'copolymer', '456', 'according', 'following', 'scheme', '640', '9', 'organic', 'reactions', 'polymers', 'br', 'p', '3', 'p', 'n2', 'br', '3', 'nbuli', 'thf', 'o', 'h', 'p', 'n', 'polymer', '1', 'p', '3', 'o', '3', 'h', 'polymer', '1', 'n', 'h', 'n', 'n', 'x', 'a', 'preparation', 'block', 'copolymers', '“living”', 'ring-opening', 'oleﬁn', 'metathesis', 'polymerization', 'reported', '417.', 'initially', 'norbornene', 'exo-dicyclopentadiene', 'polymerized', 'bis(z5-cyclopen-', 'tadienyl)titanacyl', 'butane', 'the', 'resulting', 'living', 'polymers', 'reacted', 'terephthaldehyde', 'form', 'polymers', 'terminal', 'aldehyde', 'groups', 'the', 'aldehyde', 'groups', 'turn', 'initiate', 'polymerizations', 't-butyldimethylsilyl', 'vinyl', 'ether', 'aldol-group', 'transfer', 'polymerizations', '418.', 'following', 'illustration', 'process', 'o', 'h', 'o', 'si', 'si', 'o', 'h', 'o', 'si', 'n-1', 'o', 'n1', 'n1', 'subsequently', 'terminal', 'aldehyde', 'group', 'reduced', 'nabh4', 'silyl', 'groups', 'cleaved', 'treatment', 'tetrabutylammonium', 'ﬂuoride', 'produce', 'hydrophobic-hydrophilic', 'a—b', 'diblock', 'copolymers', '97', 'block', 'copolymers', '641', 'living', 'metathesis', 'type', 'polymerization', 'also', 'employed', 'form', 'block', 'copolymers', 'norbornene', 'derivative', 'bimetallic', 'ruthenium', 'catalysts', '(pr3)2cl2ru(¼ch-p-c6h4ch¼)', 'rucl2(pr3)2', '434.', 'this', 'illustrated', 'follows', '434:', 'cl', 'cl', 'pcy3', 'h', 'ru', 'c', 'pcy3', 'o', '2n', 'pcy3', 'c', 'ru', 'cl', 'cl', 'pcy3', 'h', 'cl', 'cl', 'pcy3', 'ru', 'pcy3', 'h', 'c', 'n', 'h', 'c', 'o', 'o', 'o', 'o', 'o', 'n', 'o', 'ch3', 'pcy3', 'ru', 'n', 'pcy3', 'cl', 'cl', 'o', 'n', 'ch3', 'r', 'r', '2m', 'o', 'n', 'ch3', 'ch2cl2', 'rt', '30', 'min', 'h', 'c', 'n', 'o', 'o', 'n', 'ch3', 'h', 'c', 'o', 'n', 'o', 'o', 'n', 'ch3', 'r', 'r', 'oac', '1romp', '2', 'hydrolysis', 'o', 'o', 'o', '3', 'rop', 'oh', 'o', 'n', 'o', 'o', 'o', 'h', '2n', 'o', 'r', 'r', 'aco', 'ho', 'h', 'o', 'o', 'o', 'pitet', 'hillmyer', '436', 'formed', 'triblock', 'copolymers', '15-cycloocatadiene', 'dl-lactide', 'combining', 'ring', 'opening', 'metathesis', 'polymerization', 'cyclic', 'ester', 'ring', 'opening', 'polymerization', 'proto', 'et', 'al', '434', 'living', 'isoselective', 'coordination', 'polymerization', 'styrene', 'form', 'isoselective', 'block', 'copolymers', 'this', 'accomplished', 'sequential', 'monomer', 'addition', '642', '9', 'organic', 'reactions', 'polymers', '9710', 'miscellaneous', 'block', 'copolymers', 'polyamide-polyether', 'block', 'copolymers', 'formed', 'variety', 'techniques', 'one', 'consists', 'initial', 'preparation', 'amine-terminated', 'polyethers', 'this', 'done', 'reacting', 'hydroxy-', 'terminated', 'polyethers', 'acrylonitrile', 'reducing', 'nitrile', 'groups', 'amines', '429:', 'oh', 'ch2', 'ch', 'c', 'n', 'o', 'ch2', 'ch2', 'c', 'n', 'h2', 'catalyst', 'copolymers', 'o', 'ch2', 'ch2', 'ch2', 'nh2', 'the', 'products', 'reduction', 'condense', 'carboxylic', 'acid-terminated', 'polyamides', 'form', 'block', 'another', 'one', 'form', 'polyethers', 'terminal', 'chloride', 'groups', '430.', 'hydroxy-group-', 'terminated', 'polyethers', 'instance', 'converted', 'halogen-terminated', 'polyethers', 'the', 'products', 'react', 'ammonia', 'amine-terminated', 'polymer', 'react', 'carboxylic', 'acid-', 'terminated', 'polyamides', '430.', 'a', 'british', 'patent', 'describes', 'preparations', 'block', 'copolymers', 'two', 'steps', 'in', 'ﬁrst', 'one', 'two', 'different', 'salts', 'hexamethylene', 'diamine', 'formed', 'one', 'carboxylic', 'acid-terminated', 'polyoxyethylene', 'one', 'adipic', 'acid', '(nylon', '66', 'salt)', 'in', 'second', 'step', 'two', 'salts', 'reacted', 'melt', 'caprolactam', 'used', 'place', 'second', 'salt', '431.', 'also', 'japanese', 'paper', 'describes', 'formations', 'block', 'copolymers', 'reacting', 'polyoxyethylene', 'melt', 'condensation', 'reactions', 'caprolactam', 'presence', 'dicarboxylic', 'acids', '432.', 'polyamide-6', '(nylon', '6)', 'form', 'block', 'copolymers', 'rubber', '419', 'poly(dimethyl-', 'siloxane)', '420.', 'in', 'latter', 'case', 'polysiloxane', 'forms', 'ﬁrst', '“living”', 'polymerization', 'terminated', 'acylated', 'caprolactam', 'the', 'caprolactam', 'portion', 'molecule', 'polymerized', 'aid', 'lithium', 'caprolactamate', 'r', 'o', 'o', 'o', 'si', 'si', 'x', 'si', 'o', 'n', 'o', 'o', 'o', 'li', 'n', 'h', 'n', 'z', 'r', 'o', 'o', 'o', 'si', 'si', 'x', 'si', 'h', 'n', 'o', 'o', 'o', '5', 'n', 'z', 'this', 'diblock', 'copolymer', 'melt', 'annealed', 'ca', '250(cid1)c', 'it', 'exhibits', 'superior', 'mechanical', 'properties', 'nylon', '6', 'homopolymer', '420.', '98', 'processes', 'polymer', 'degradation', '643', '9711', 'mechanochemical', 'techniques', 'formation', 'block', 'copolymers', 'these', 'techniques', 'rely', 'upon', 'high', 'shear', 'cause', 'bond', 'scissions', 'ruptured', 'bonds', 'result', 'formations', 'free', 'radical', 'ionic', 'species', '413.', 'when', 'application', 'shear', 'done', 'presence', 'monomers', 'block', 'copolymers', 'form', 'this', 'approach', 'exploited', 'fairly', 'extensively', 'such', 'cleavages', 'macromolecules', 'take', 'place', 'cold', 'mastication', 'milling', 'extrusion', 'polymers', 'visco-elastic', 'state', 'both', 'homolytic', 'heterolytic', 'scissions', 'possible', 'the', 'ﬁrst', 'one', 'yields', 'free', 'radical', 'second', 'one', 'ionic', 'species', 'heterolytic', 'scissions', 'require', 'energy', 'written', 'completely', 'unlikely', '413.', 'early', 'work', 'done', 'natural', 'rubber', '413.', 'it', 'swells', 'exposed', 'many', 'monomers', 'forms', 'visco-elastic', 'mass', 'when', 'swollen', 'mass', 'subjected', 'shear', 'mechanical', 'scission', 'resultant', 'radicals', 'initiate', 'polymerizations', 'the', 'mastication', 'reaction', 'shown', 'accompanied', 'formation', 'homopolymers', '413.', 'later', 'technique', 'applied', 'many', 'different', 'polymers', 'many', 'different', 'monomers', '414.', '98', 'processes', 'polymer', 'degradation', 'there', 'many', 'causes', 'degradation', 'polymers', 'the', 'chief', 'ones', 'among', 'heat', 'oxidation', 'light', 'ionic', 'radiation', 'hydrolysis', 'mechanical', 'shear', 'the', 'effect', 'discoloration', 'loss', 'molecular', 'weight', 'cross-linking', 'cyclization', 'the', 'loss', 'molecular', 'weight', 'severe', 'polymer', 'degraded', 'mixture', 'monomers', 'oligomers', 'the', 'effects', 'blackening', 'charring', 'loss', 'useful', 'properties', 'on', 'hand', 'mechanical', 'shearing', 'often', 'applied', 'polymers', 'like', 'rubbers', 'deliberately', 'reduce', 'molecular', 'weight', 'commer-', 'cial', 'processing', 'in', 'environment', 'synthetic', 'polymers', 'generally', 'degrade', 'due', 'man-made', 'environmental', 'pollutants', 'atmosphere', 'like', 'carbon', 'monoxide', 'sulfur', 'dioxide', 'nitrogen', 'oxide', 'oxidizing', 'smog', 'rich', 'ozone', 'molecular', 'weight', 'loss', 'occurs', 'breaking', 'primary', 'valence', 'bonds', 'such', 'chain', 'scissions', 'may', 'occur', 'random', 'points', 'along', 'polymer', 'backbone', 'may', 'take', 'place', 'terminal', 'ends', 'polymer', 'monomer', 'units', 'released', 'successively', 'this', 'last', 'effect', 'compared', 'unzipping', 'the', 'response', 'particular', 'polymeric', 'material', 'speciﬁc', 'causes', 'degradation', 'depends', 'upon', 'chemical', 'structure', 'this', 'chapter', 'presents', 'degradation', 'processes', 'typical', 'commercial', 'polymers', 'the', 'materials', 'selected', 'received', 'major', 'attention', 'literature', '981', 'thermal', 'degradation', 'common', 'chain-growth', 'polymers', 'thermal', 'degradation', 'polymers', 'conveniently', 'studied', 'pyrolytic', 'methods', 'the', 'polymer', 'litera-', 'ture', 'contains', 'many', 'reports', 'studies', 'conducted', 'various', 'temperatures', 'inert', 'atmospheres', 'air', 'vacuum', 'the', 'volatile', 'products', 'usually', 'monitored', 'accompanying', 'measurements', 'weight', 'loss', 'per', 'unit', 'time', 'the', 'reaction', 'rates', 'thus', 'measured', '1', 'loss', 'molecular', 'weight', 'function', 'temperature', 'extent', 'degradation', '2', 'the', 'quantity', 'composition', 'volatile', 'nonvolatile', 'products', 'degradation', '3', 'the', 'activation', 'energy', 'degradation', 'process', 'a', 'general', 'scheme', 'thermal', 'degradations', 'chain-growth', 'polymers', 'free-radical', 'reactions', 'written', 'follows', '644', '9', 'organic', 'reactions', 'polymers', 'random', 'type', 'chain', 'scission', 'chain-end', 'type', 'scission', '∆', '∆', 'in', 'thermal', 'chain', 'depolymerization', 'degradation', 'also', 'random', 'rupture', 'occurs', 'various', 'points', 'along', 'chain', 'the', 'products', 'various', 'size', 'fragments', 'polymer', 'usually', 'larger', 'individual', 'monomers', 'both', 'reactions', 'shown', 'take', 'place', 'simultaneously', 'polymer', 'chain', 'one', 'might', 'take', 'place', 'exclusively', 'also', 'chain', 'depropagation', 'may', 'necessarily', 'initiated', 'terminal', 'ends', 'macromolecules', 'though', 'depolymer-', 'izations', 'common', 'they', 'may', 'instead', 'start', 'points', 'imperfection', 'chain', 'structures', 'these', 'imperfections', 'might', 'consist', 'incorporated', 'initiator', 'fragments', 'peroxides', 'ether', 'links', 'they', 'might', 'formed', 'result', 'oxygen', 'molecules', 'present', 'chain', 'growths', 'weak', 'points', 'polymer', 'backbones', 'also', 'locations', 'tertiary', 'hydrogens', 'each', 'individual', 'polymer', 'depolymerize', 'speciﬁc', 'rate', 'degradation', 'products', 'peculiar', 'particular', 'chemical', 'structure', 'polymer', '446–452.', 'for', 'instance', 'poly(methyl', 'methacrylate)', 'converted', 'almost', 'quantitatively', 'back', 'monomers', 'the', 'depolymerization', 'occurs', 'terminal', 'end', 'unzipping', 'fashion', 'overall', 'molecular', 'weight', 'decreasing', 'slowly', 'proportion', 'amount', 'volatilized', 'polyethylene', 'hand', 'undergoes', 'scission', 'longer', 'oleﬁn', 'fragments', 'very', 'little', 'monomer', 'released', 'at', 'time', 'molecular', 'weight', 'tends', 'decrease', 'rapidly', 'small', 'amount', 'volatilization', 'takes', 'place', 'the', 'two', 'polymers', 'good', 'examples', 'extreme', 'behavior', 'chain-growth', 'polymers', 'most', 'chain-growth', 'polymers', 'however', 'fall', 'two', 'the', 'depolymerization', 'reaction', 'chain-growth', 'polymers', 'generally', 'occurs', 'free-radical', 'mechanism', 'reactions', 'similar', 'if', 'depropagation', 'major', 'portion', 'degradation', 'process', 'molecular', 'weight', 'reduction', 'proportional', 'quantity', 'monomer', 'forms', 'if', 'hand', 'chain', 'transferring', 'major', 'portion', 'degradation', 'process', 'rapid', 'loss', 'molecular', 'weight', 'little', 'accompanying', 'monomer', 'accumulation', 'reaction', 'products', 'large', 'segments', 'chains', 'the', 'rate', 'depolymeriza-', 'tion', 'exhibits', 'maximum', 'when', 'depropagation', 'takes', 'place', 'elevated', 'temperature', 'rate', 'equal', 'propagation', 'free-radical', 'polymerization', 'temperature', 'reaction', 'ceiling', 'temperature', '(see', 'chap', '3)', 'termination', 'take', 'place', 'disproportionation', 'secondary', 'reactions', 'however', 'may', 'occur', 'degradation', 'process', 'depending', 'upon', 'chemical', 'structure', 'polymer', 'such', 'side', 'reactions', 'instance', 'successive', 'eliminations', 'hydrochloric', 'acid', 'poly(vinyl', 'chrolide)', 'acetic', 'acid', 'poly(vinyl', 'acetate)', '982', 'thermal', 'degradation', 'polyoleﬁns', 'polymers', 'conjugated', 'dienes', 'one', 'early', 'study', 'thermal', 'degradation', 'polyethylene', 'carried', 'low', 'molecular', 'weight', 'polymers', '453.', 'later', 'work', 'repeated', 'high-density', 'polyethylene', '454.', 'the', 'volatile', 'products', 'identiﬁed', 'gas', 'chromatography', 'the', 'biggest', 'portion', 'volatiles', 'found', 'propylene', '98', 'processes', 'polymer', 'degradation', '645', 'the', 'remaining', 'materials', 'methane', 'ethane', 'propane', 'number', 'unsaturated', 'saturated', 'higher', 'hydrocarbons', 'the', 'quantity', 'ethylene', 'increased', 'slightly', 'increase', 'temperature', 'pyrolysis', 'formation', 'propylene', 'occur', 'two', 'ways', 'the', 'ﬁrst', 'one', 'intramolecular', 'radical', 'transfer', 'second', 'carbon', 'followed', 'decomposition', 'reaction', 'this', 'however', 'believed', 'contribute', 'signiﬁcantly', 'quantity', 'propylene', 'obtained', 'based', 'theoretical', 'consideration', '454.', 'the', 'second', 'one', 'occur', 'result', 'scissions', 'c–c', 'bonds', 'located', 'positions', 'b', 'terminal', 'double', 'bonds', 'such', 'double', 'bonds', 'probably', 'form', 'large', 'quantities', 'decomposition', 'process', 'thought', 'main', 'source', 'propylene', '454:', 'it', 'observed', 'isotactic', 'polypropylene', 'decomposes', 'thermally', 'mechanism', 'varies', 'different', 'temperatures', 'conditions', '455.', 'thus', '340(cid1)c', 'major', 'volatile', 'product', 'propane', '380(cid1)c', 'n-pentane', '420(cid1)c', 'propylene', 'the', 'propane', 'believed', 'originate', 'weak', 'spot', 'polymeric', 'chain', 'formation', 'n-pentane', 'involves', 'radical', 'abstraction', 'six-membered', 'ring', 'formation', 'backbiting', 'process', 'propylene', 'may', 'come', 'free-radical', 'depolymerization', 'process', 'cyclic', 'six-membered', 'ring', 'formation', 'involving', 'terminal', 'double', 'bond', '455.', 'the', 'thermal', 'degradation', 'diene', 'polymers', 'subject', 'several', 'studies', '456', '457.', 'the', 'scheme', 'polyisoprene', 'polybutadiene', 'degradation', 'postulated', 'part', 'golub', 'garguila', '458', '460.', 'it', 'based', 'infra-red', 'spectra', 'nmr', 'studies', 'products', '(h)', '(h)', '399', 'oc', '(h)', '(h)', '(h)', '(h)', '(h)', '(h)', '(h)', '(h)', '(h)', 'cyclized', 'rubber', '646', '9', 'organic', 'reactions', 'polymers', 'in', 'addition', 'spectroscopic', 'evidence', 'shows', 'cis–trans', 'isomerizations', 'well', 'cyclizations', 'occur', 'process', 'thermal', 'degradation', '14-polyisoprene', 'it', 'interesting', 'cis–trans', 'isomerizations', 'observed', 'temperatures', 'low', '200(cid1)c', '460.', '983', 'thermal', 'degradation', 'polystyrene', 'polystyrene-like', 'polymers', 'thermal', 'degradation', 'atactic', 'polystyrene', 'results', 'formation', 'volatile', 'products', 'contain', 'much', '42', 'monomer', 'small', 'quantities', 'toluene', 'ethylbenzene', 'methylstyrene', 'the', 'rest', 'volatile', 'material', 'made', 'dimers', 'trimers', 'tetramers', 'no', 'large', 'fractions', 'isolated', 'this', 'suggests', 'main', 'mechanism', 'decomposition', 'depropagation', 'the', 'rate', 'however', 'exhibits', 'maximum', 'rapid', 'decline', 'molecular', 'weight', 'this', 'indicates', 'scission', 'chains', 'some', 'evidence', 'presented', 'rapid', 'decrease', 'molecular', 'weight', 'result', 'scission', 'weak', 'points', 'polymer', 'independent', 'free-radical', 'depropagation', 'reaction', '461–465.', 'some', 'weak', 'spots', 'believed', 'occasional', '“head-to-head”', 'placement', 'monomers', '19.', 'it', 'suggested', 'weak', 'spots', 'may', 'also', 'structures', 'form', 'result', 'monomer', 'additions', 'aromatic', 'ring', 'chain', 'growth', '(see', 'chap', '6)', '466.', 'the', 'following', 'scheme', 'chain', 'scission', 'therefore', 'proposed', '466:', '∆', 'h', '∆', 'h', 'the', 'identity', 'weak', 'spots', 'however', 'established', 'certainty', '467.', 'the', 'evidence', 'indicate', '“weak', 'points”', 'head-to-head', 'bonds', 'branch', 'points', 'unsaturated', 'structure', '467.', 'the', 'rate', 'conversion', 'polystyrene', 'volatiles', 'measured', 'numerous', 'studies', 'in', 'contrast', 'polystyrene', 'poly(a-methyl', 'styrene)', 'yields', 'vacuum', 'pyrolyses', 'temperatures', '200', '500(cid1)c', '95–100', 'monomer', 'by', 'comparison', 'polystyrene', 'yields', '406', '457.', 'the', 'difference', 'attributed', 'fact', 'hydrogen', 'transfer', 'completely', 'blocked', 'sites', 'chain', 'scission', 'methyl', 'groups', 'a-positions', '457.', 'as', 'result', 'terminal', 'free-', 'radicals', 'unzip', 'monomers', 'dimers', '98', 'processes', 'polymer', 'degradation', '647', 'polystyrenes', 'substituted', 'benzene', 'ring', 'like', 'poly(vinyl', 'toluene)', 'behave', 'similarly', 'polystyrene', 'pyrolyzed', '457.', 'also', 'poly(m-methylstyrene)', '350(cid1)c', 'yields', '444', 'monomer', 'compared', 'polystyrene', 'yields', '406', 'monomer', 'conditions', 'the', 'rate', 'however', 'polystyrene', 'temperature', '024', 'mol%min', 'poly(m-methylstyrene)', '09', '457.', '984', 'thermal', 'degradation', 'methacrylic', 'acrylic', 'polymers', 'the', 'thermal', 'degradation', 'polymers', 'acrylic', 'methacrylic', 'alkyl', 'esters', 'process', 'depo-', 'lymerization', 'monomers', 'temperatures', '250(cid1)c', 'provided', 'alkyl', 'group', 'small', 'less', 'butyl', '468.', 'poly(t-butyl', 'methacrylate)', 'yields', 'quantitatively', 'isobutene', 'instead', 'it', 'shown', 'thermal', 'depolymerization', 'monomers', 'probably', 'common', 'poly(methacrylate', 'ester)s', 'as', 'size', 'alkyl', 'group', 'increases', 'however', 'particularly', 'within', 'secondary', 'tertiary', 'structures', 'increased', 'tendency', 'alkyl', 'group', 'also', 'decompose', 'this', 'decomposition', 'interferes', 'depolymerization', 'process', 'thermal', 'stability', 'poly(methyl', 'methacrylate)', 'appears', 'vary', 'molecular', 'weight', 'for', 'instance', 'sample', 'polymer', 'molecular', 'weight', '150000', 'heated', 'vacuum', '30', 'min', '318(cid1)c', 'yields', '743', 'volatiles', 'by', 'comparison', 'sample', 'polymer', 'molecular', 'weight', '5100000', 'heated', '30', 'min', '319(cid1)c', 'yielded', '352', 'volatiles', '456.', 'the', 'thermal', 'stability', 'copolymers', 'long-chained', 'diol', 'dimethacrylates', 'investigated', '583.', 'these', 'copolymers', 'included', '14-butane-', '15-pentane-', '16-hexane-', '18-octane-', '110-decane-', '112-dodecanediol', 'dimethacrylates', 'respectively', 'well', '22-bis4-(2-hydroxy-3-methacryloyl-', 'oxypropoxy)-phenylpropane', 'triethylene', 'glycol', 'dimethacrylate', 'the', 'polymers', 'found', 'thermally', 'stable', '(cid7)250(cid1)c', 'shown', 'initial', 'decomposition', 'temperature', 'degradation', 'proﬁles', 'a', 'quantitative', 'investigation', 'thermal', 'degradation', 'poly(ethyl', 'acrylate)', 'poly(n-propyl', 'acrylate)', 'poly(isopropyl', 'acrylate)', 'poly(n-butyl', 'acrylate)', 'poly(2-ethylhexyl', 'acrylate)', 'demonstrated', 'principle', 'volatile', 'products', 'carbon', 'dioxide', 'oleﬁn', 'alcohol', 'corresponding', 'alkyl', 'group', '469', '470.', 'the', 'following', 'mechanism', 'degradation', 'proposed', '469', '470:', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'co2', 'r', 'r', 'r', 'rr', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'more', 'recently', 'another', 'study', 'carried', 'thermal', 'decomposition', 'homopolymers', 'ethyl', 'methacrylate', 'n-butyl', 'methacrylate', '2-hydroxyethyl', 'methacrylate', 'well', 'copolymers', '471.', 'the', 'copolymers', 'hydroxyethyl', 'methacrylate', 'ethyl', 'methacrylate', 'butyl', 'methacrylate', 'found', 'degrade', 'unzipping', 'yield', 'monomers', 'similarly', 'poly(methyl', 'methacrylate)', 'in', 'addition', 'competition', 'unzipping', 'cross-linking', 'binary', 'copolymers', 'hydroxyethyl', 'methacrylate', 'ethyl', 'methacrylate', 'n-butyl', 'methacrylate', 'thermal', 'degradation', 'nitrile', 'polymers', 'particularly', 'acrylonitrile', 'studied', 'detail', '472–478.', 'it', 'shown', 'two', 'paths', 'degradation', 'depending', 'upon', 'temperature', '(see', 'also', 'chap', '8)', 'one', 'reaction', 'takes', 'place', 'low', 'temperature', '100', '200(cid1)c', '648', '9', 'organic', 'reactions', 'polymers', 'one', 'occurs', '240(cid1)c', 'at', 'low', 'temperature', 'polymers', 'develop', 'dark', 'color', 'there', 'however', 'little', 'evolution', 'volatiles', 'at', 'high', 'temperature', 'hand', 'evolutions', 'volatiles', 'thermally', 'stable', 'residues', 'develop', 'the', 'low', 'temperature', 'darkening', 'process', 'polyacrylonitrile', 'shown', 'intramolecular', 'cyclization', 'polymerization', 'cyanide', 'groups', '475', '476.', 'the', 'overall', 'reaction', 'illustrated', 'follows', 'c', 'c', 'c', 'c', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'the', 'shown', 'ladder', 'structure', 'actually', 'vary', 'idealized', 'picture', 'nevertheless', 'formation', 'fused', 'tetrahydropyridine', 'rings', 'demonstrated', 'ir', 'data', '475', '476.', 'at', 'time', 'many', 'irregularities', 'shown', 'structure', 'also', 'demonstrated', 'longer', 'isotactic', 'sequences', 'polymeric', 'structures', 'longer', 'sequences', 'ladder', 'structures', 'form', '479.', 'high', 'temperature', 'degradation', 'polyacrylonitrile', 'leads', 'formation', 'oligomers', 'the', 'general', 'form', 'oligomerized', 'material', 'shown', 'follows', '480:', 'nc', 'cn', 'n', 'nh', 'n', 'x', 'cn', 'x', '¼', '0–2', 'n', '¼', '0–5', 'the', 'structure', 'occurs', 'result', 'initiation', 'termination', 'process', 'cyclization', 'frequent', 'intervals', 'along', 'chain', 'the', 'growth', 'ladder', 'structure', 'however', 'terminates', 'due', 'hydrogen', 'transfer', 'this', 'process', 'associated', 'atactic', 'sequences', 'polymer', 'chain', 'cn', 'h', 'n', 'n', 'nc', 'nc', 'cn', 'cn', 'cn', 'n', 'cn', 'n', 'nh', 'n', 'nh', 'polymethacrylonitirle', 'develops', 'color', 'upon', 'heating', 'result', 'linking', 'adjacent', 'carbons', 'nitrogen', 'atoms', 'intramolecular', 'cyclization', 'reactions', 'similar', 'acrylonitrile', 'this', 'reaction', 'initiated', 'primarily', 'impurities', 'often', 'present', 'polymer', 'end', 'chains', 'various', 'locations', 'backbones', 'due', 'ring', 'formation', 'amount', 'monomer', 'obtained', 'polymer', '200(cid1)c', 'approximately', '50.', 'if', 'however', 'polymer', 'prepared', 'highly', 'puriﬁed', 'monomer', 'yield', 'monomer', 'upon', 'thermal', 'degradation', '300(cid1)c', '100.', 'also', 'yellow', 'color', 'develop', 'polymer', 'temperatures', '120–220(cid1)c', '457.', 'poly(vinylidine', 'cyanide)', 'structure', 'would', 'suggest', 'formation', 'double', 'ladder', 'polymer', 'upon', 'pyrolysis', 'it', 'shown', 'however', 'early', 'study', 'happen', 'instead', 'polymer', 'mostly', 'depolymerizes', '481', '482.', '98', 'processes', 'polymer', 'degradation', '649', '985', 'thermal', 'degradation', 'chlorocarbon', 'fluorocarbon', 'polymers', 'the', 'thermal', 'instability', 'poly(vinyl', 'chloride)', 'cause', 'concern', 'commercially', 'therefore', 'stimulated', 'extensive', 'investigations', 'the', 'overall', 'process', 'degradation', 'complex', 'still', 'completely', 'resolved', 'some', 'questions', 'remain', '1', 'is', 'overall', 'process', 'degradation', 'ionic', 'free', 'radical', 'nature', '2', 'what', 'exact', 'mechanism', 'initiation', 'it', 'easilly', 'observable', 'degradation', 'accompanied', 'evolution', 'hcl', 'blackening', 'polymer', 'also', 'elevated', 'temperatures', 'poly(vinyl', 'chloride)', 'gives', 'hydrogen', 'chloride', 'dehydrochlorination', 'also', 'accompanied', 'rapid', 'depolymerization', '483.', 'the', 'rate', 'decomposition', 'decreases', 'increasing', 'molecular', 'weight', 'highest', 'oxygen', 'lowest', 'helium', 'atmosphere', 'it', 'claimed', 'hcl', 'exerts', 'catalytic', 'effect', 'upon', 'rate', 'decomposition', '483.', 'this', 'contrasts', 'evidence', 'presented', 'earlier', 'hcl', 'accelerate', 'thermal', 'degradation', '484.', 'the', 'earlier', 'evidence', 'obtained', 'autocatalytic', 'decmposition', 'ﬁlms', 'based', 'eveidence', 'hypothesized', 'chloride', 'ion', 'acts', 'base', '(supporting', 'ionic', 'mechanism', 'decomposition)', '484:', 'hcl', '2', 'hcl', 'cl', 'cl', 'h', 'h', 'cl', 'h', 'cl2', 'cl', 'n', 'cl', 'hcl', 'cl', 'hcl', 'cl', 'n', 'in', 'recent', 'study', 'hcl', 'also', 'found', 'essential', 'initiation', 'unzipping', 'process', 'may', 'may', 'essential', 'depolymerization', 'reaction', 'process', 'started', '485.', 'the', 'shown', 'ionic', 'mechanism', 'disputed', 'study', 'esr', 'signals', 'recorded', 'thermal', 'decomposition', 'polymer', 'elevated', 'temperatures', '486.', 'this', 'strongly', 'supports', 'free-radical', 'mechanism', 'a', 'free-radical', 'mechanism', 'proposed', 'earlier', 'bamford', 'feuton', '487.', 'this', 'mechanism', 'illustrates', 'formation', 'hydrochloric', 'acid', 'based', 'rupture', 'carbon–chlorine', 'bonds', 'initiation', 'cl', 'cl', '∆', 'cl', 'cl', 'hcl', 'cl', '650', 'propagation', '9', 'organic', 'reactions', 'polymers', 'cl', 'hcl', 'cl', 'cl', 'cl', 'subsequent', 'interaction', 'macromolecular', 'radicals', 'leads', 'cross-linking', 'many', 'additional', 'investigations', 'demonstrated', 'thermal', 'decompositions', 'vinyl', 'polymers', 'pendant', 'electroneg-', 'ative', 'groups', 'designate', 'x', 'result', 'elimination', 'hx', 'formation', 'macromo-', 'lecular', 'residues', 'polyene', 'structures', '488.', 'in', 'addition', 'higher', 'temperatures', 'polyene', 'sequences', 'form', 'rearrange', 'large', 'numbers', 'aromatic', 'hydrocarbons', '489–492.', 'formation', 'polyenes', 'common', 'poly(vinyl', 'chloride)', 'poly(vinyl', 'bromide)', 'poly(vinyl', 'alcohol)', 'poly', '(vinyl', 'acetate)', '493.', 'the', 'polyene', 'structures', 'partly', 'decompose', 'formation', 'due', 'bond', 'stress', 'molecular', 'reorganization', 'processes', 'the', 'stresses', 'result', 'formation', 'conjugated', 'structures', 'along', 'polymeric', 'chains', 'when', 'chains', 'fracture', 'remaining', 'portions', 'polyene', 'sequences', 'form', 'aromatic', 'compounds', '47.', 'it', 'postulated', 'process', 'initially', 'involves', 'reactions', 'enone', 'groups', 'present', 'poly(vinyl', 'chloride)', 'anomalies', 'chlorine', 'units', 'neighboring', 'polymeric', 'chains', '494', '495:', 'o', 'h', 'cl', 'o', 'o', 'the', 'ﬁrst', 'step', 'formation', 'dihydropyran', 'ring', 'this', 'followed', 'retro-diels-alder', 'splitting', 'leads', 'regeneration', 'ab-unsaturated', 'ketone', 'formation', 'double', 'bond', 'poly', '(vinyl', 'chloride)', 'molecule', 'the', 'double', 'bond', 'initiates', 'subsequent', 'elimination', 'hydrogen', 'chloride', 'molecule', '495.', 'an', 'intramolecular', 'initiation', 'process', 'explains', 'constant', 'rate', 'dehydrochlorination', 'also', 'proposed', '496:', 'cl', 'h', 'cl', 'h', 'cl', 'hcl', 'it', 'pointed', 'poly(vinyl', 'chloride)', 'sense', 'worst', 'enemy', 'structural', 'defects', 'known', 'contribute', 'appreciably', 'thermal', 'instability', 'they', 'formed', 'polymer', 'molecules', 'process', 'polymerization', 'routes', 'involving', 'hydrogen', 'abstraction', 'polymer', 'backbone', '584.', '98', 'processes', 'polymer', 'degradation', '651', 'gupper', 'et', 'al', 'used', 'micro', 'raman', 'spectroscopy', 'study', 'thermal', 'degradation', 'poly(vinyl', 'chloride)', 'containing', 'various', 'additives', '586.', 'they', 'observed', 'linear', 'increase', 'conjugated', 'sequences', 'process', 'dehydrochlorination', 'carty', 'coworkers', 'investigated', 'thermal', 'decomposition', 'chlorinated', 'poly(vinyl', 'chloride)', '496.', 'the', 'thermal', 'decomposition', 'pure', 'chlorinated', 'poly(vinyl', 'chloride)', '(without', 'stabilizer', 'lubricant)', 'studied', 'dynamic', 'thermogravimetric', 'analysis', 'heating', 'rates', '5', '100(cid1)cmin', 'atmospheres', 'nitrogen', 'air', 'oxygen', 'in', 'case', 'two-step', 'decomposition', 'observed', 'similar', 'poly(vinyl', 'chloride)', 'dehydrochlorination', 'followed', 'pyrolysisoxidation', 'carbonaceous', 'residue', 'the', 'rate', 'dehydrochlorination', 'dependent', 'atmosphere', 'occur-', 'ring', 'slightly', 'slower', 'nitrogen', 'air', 'slightly', 'quickly', 'oxygen', 'air', 'the', 'decomposition', 'residual', 'char', 'clearly', 'dependent', 'conditions', 'formed', 'under', 'dynamic', 'conditions', 'chars', 'formed', 'high', 'heating', 'rates', 'appeared', 'resistant', 'oxidative', 'degra-', 'dation', 'formed', 'slowly', 'however', 'chars', 'formed', 'heating', 'different', 'rates', 'held', '500(cid1)c', 'char', 'formed', 'slowest', 'heating', 'rate', 'slowest', 'oxidized', 'the', 'uptake', 'oxygen', 'char', 'appears', 'rate', 'limiting', 'at', 'low', 'heating', 'rates', 'char', 'oxide', 'similar', 'air', 'oxygen', 'as', 'heating', 'rate', 'raised', 'rate', 'mass', 'loss', 'char', 'air', 'becomes', 'progressively', 'closer', 'nitrogen', '100(cid1)cmm', 'almost', 'identical', 'the', 'rates', 'thermal', 'decompositions', 'poly(vinylidine', 'chloride)s', 'shown', 'depend', 'upon', 'method', 'polymers', 'prepared', '497.', 'those', 'formed', 'pure', 'monomers', 'mass', 'polymerization', 'stable', 'polymers', 'prepared', 'emulsion', 'polymerization', 'hand', 'degrade', 'fastest', 'the', 'mechanism', 'degradation', 'poly(vinylidine', 'chloride)', 'proposed', 'follows', '498–500:', '1', 'hydrochloric', 'acid', 'eliminated', 'chain', 'reaction', '2', 'conjugated', 'sequences', 'condense', 'form', 'cross-linked', 'structures', 'some', 'support', 'mechanism', 'came', 'stepwise', 'heating', 'studies', 'poly(vinylidine', 'chloride)', '501.', 'although', 'polytetraﬂuoroethylene', 'reputation', 'quite', 'stable', 'thermally', 'degrade', 'elevated', 'temperatures', 'the', 'polymer', 'upon', 'pyrolysis', 'yields', 'almost', '100', 'monomer', 'the', 'mechanism', 'believed', 'free-radical', 'unzipping', 'chains', 'entire', 'chain', 'consumed', 'this', 'illustrated', 'follows', '457.', 'initially', 'chain', 'ruptures', 'followed', 'formation', 'monomers', 'f', 'f', 'ff', 'f', 'f', 'f', 'f', 'f', 'f', 'ff', 'f', 'f', 'ff', 'f', 'f', 'f', 'f', 'f', 'f', 'ff', '2', 'f', 'f', 'f', 'f', 'ff', 'f', 'f', 'polychlorotriﬂuoroethylene', 'less', 'stable', 'thermally', 'polytetraﬂuoro-ethylene', 'it', 'yields', 'much', '860', 'volatiles', '30', 'h', '3318(cid1)c', '11.', 'these', 'volatiles', 'contain', 'large', 'amounts', 'monomer', 'a', 'mechanism', 'resembles', 'postulated', 'degradation', 'mechanism', 'polytetraﬂuoroethylene', 'proposed', '457.', '652', '9', 'organic', 'reactions', 'polymers', 'unlike', 'polytetraﬂuoroethylene', 'polyvinyl', 'ﬂuoride', 'poly(vinylidine', 'ﬂuoride)', 'polytriﬂuor-', 'oethylene', 'yield', 'primarily', 'heating', 'hf', '457.', 'among', 'three', 'poly(vinylidine', 'ﬂuoride)', 'yields', 'larger', 'amounts', 'hf', 'two', 'polymers', 'accompanying', 'formation', 'double', 'bonds', '986', 'thermal', 'degradation', 'poly(vinyl', 'acetate)', 'thermal', 'decomposition', 'poly(vinyl', 'acetate)', 'results', 'loss', 'acetic', 'acid', 'the', 'reaction', 'typical', 'thermal', 'cleavages', 'esters', 'it', 'facilitated', 'formation', 'pseudo', 'six-membered', 'rings', 'result', 'interactions', 'b-hydrogens', 'alcohol', 'residues', 'carboxylic', 'groups', 'o', 'o', 'h', 'o', 'o', 'when', 'double', 'bonds', 'form', 'adjacent', 'methylene', 'groups', 'become', 'activated', 'the', 'loss', 'acetic', 'acid', 'main', 'product', 'temperatures', '200–250(cid1)c', 'beyond', 'temperatures', 'aromatic', 'pyrolytic', 'compounds', 'form', 'studies', 'thermal', 'degradation', 'copolymers', 'vinyl', 'chloride', 'vinyl', 'acetate', 'showed', 'copolymers', 'thermally', 'less', 'stable', 'homopolymers', '502', '503.', 'the', 'ratio', 'hydrochloric', 'acid', 'acetic', 'acid', 'volatilize', 'remains', 'constant', 'degradation', 'this', 'indicates', 'neither', 'evolved', 'preferentially', 'reaction', 'begins', '502', '503.', 'it', 'interesting', 'note', 'degradation', 'studies', 'copolymer', 'vinyl', 'chloride', 'styrene', 'also', 'demonstrated', 'copoly-', 'mer', 'less', 'stable', 'homopolymers', '504.', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', 'the', 'thermal', 'decomposition', 'step-growth', 'polymers', 'cannot', 'take', 'place', 'chain', 'reaction', 'like', 'chain-growth', 'polymers', 'as', 'result', 'materials', 'degrade', 'random', 'fashion', 'rupturing', 'weakest', 'bonds', 'ﬁrst', '991', 'thermal', 'degradation', 'polyoxides', 'polyoxymethylene', 'depolymerizes', 'formaldehyde', '220(cid1)c', 'this', 'found', 'ﬁrst-order', 'reaction', 'rate', 'varying', '042', '58/min', 'depending', 'upon', 'conditions', 'polymer', 'preparation', 'molecular', 'weight', 'polymer', '457.', 'poly(ethylene', 'oxide)', 'decomposes', 'upon', 'heating', 'lower', 'temperatures', 'polyethylene', 'among', 'volatile', 'products', 'found', 'formaldehyde', 'ethanol', 'ethylene', 'oxide', 'carbon', 'dioxide', 'water', 'poly(propylene', 'oxide)', 'also', 'less', 'heat', 'stable', 'polypropylene', 'isotactic', 'poly(propylene', 'oxide)', 'somewhat', 'stable', 'atactic', 'one', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '653', '992', 'thermal', 'degradation', 'polyesters', 'poly(ethylene', 'terephthalate)', 'decomposes', 'upon', 'heating', 'series', 'different', 'reactions', 'these', 'run', 'either', 'concurrently', 'consecutively', 'the', 'result', 'complex', 'mixture', 'volatile', 'nonvolatile', 'products', 'it', 'found', 'poly(ethylene', 'terephthalate)', 'maintained', 'molten', 'condition', 'inert', 'atmosphere', '282–323(cid1)c', 'slowly', 'converts', 'mixture', 'gaseous', 'low', 'molecular', 'weight', 'fragments', '581.', 'the', 'major', 'products', 'pyrolysis', 'poly(ethylene', 'terephthalate)', 'carbon', 'dioxide', 'acetaldehyde', 'terephthalic', 'acid', 'in', 'addition', 'detected', 'trace', 'amounts', 'anhydrides', 'benzoic', 'acid', 'p-acetylbenzoic', 'acid', 'acetophenone', 'vinyl', 'benzoate', 'water', 'methane', 'ethylene', 'acetylene', 'ketones', '505.', 'the', 'following', 'mechanism', 'degradation', 'postulated', '505:', 'o', 'o', 'o', 'o', 'o', 'o', '∆', 'o', 'o', 'o', 'o', 'the', 'vinyl', 'end', 'groups', 'form', 'cleavage', 'ester', 'groups', 'decompose', 'number', 'ways', 'oh', 'co2', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'the', 'thermal', 'degradation', 'poly(butylene', 'terephthalate)', 'examined', 'aid', 'laser', 'microprobe', 'mass', 'spectrometry', '506.', 'a', 'complex', 'multistage', 'decomposition', 'mechanism', 'observed', 'involves', 'two', 'reaction', 'paths', 'the', 'initial', 'degradation', 'takes', 'place', 'ionic', 'mechanism', 'this', 'results', 'evolution', 'tetrahydrofuran', 'this', 'followed', 'concerted', 'ester', 'pyrolyses', 'reactions', 'involve', 'intermediate', 'cyclic', 'transition', 'states', 'result', 'formation', '13-butadiene', 'simulta-', 'neous', 'decarboxylations', 'occur', 'decomposition', 'paths', 'the', 'latter', 'stages', 'decomposition', '654', '9', 'organic', 'reactions', 'polymers', 'characterized', 'evolutions', 'carbon', 'monoxide', 'various', 'aromatic', 'compounds', 'like', 'toluene', 'benzoic', 'acid', 'terephthalic', 'acid', 'the', 'ﬁrst', 'step', 'shown', 'follows', '506:', 'the', 'subsequent', 'decomposition', 'shown', 'actually', 'take', 'place', 'lower', 'temperatures', 'o', 'o', '∆', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'h', 'o', 'o', 'h', 'o', 'o', 'there', 'indications', 'moisture', 'among', 'decomposition', 'products', 'this', 'may', 'imply', 'acid', 'hydrolysis', 'plays', 'part', 'tetrahydrofuran', 'formation', 'o', 'o', 'h', 'h2o', 'oh2', 'o', 'o', '-h', 'o', 'oh', 'at', 'higher', 'temperatures', 'reaction', 'path', 'involves', 'elimination', '13-butadiene', '506', '507;', 'o', 'o', 'o', 'oh', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '655', 'in', 'oxidations', 'hydrocarbons', 'oxygen', 'believed', 'act', 'diradical', 'ground', 'state', 'this', 'would', 'explain', 'radical', 'combination', 'reactions', 'subsequent', 'hydrogen', 'abstraction', 'reaction', 'r(cid4)þ(cid4)o(cid3)(cid3)o(cid4)', 'r(cid3)(cid3)o(cid3)(cid3)o(cid4)', 'r', '(cid3)', 'o(cid3)o(cid4)', 'þ', 'r0h', 'ro', '(cid3)', 'oh', 'þ', 'r0(cid4)', 'the', 'rate', 'formation', 'peroxy', 'radical', 'much', 'higher', 'rate', 'hydrogen', 'abstraction', '521.', 'the', 'overall', 'rate', 'oxidation', 'polymeric', 'materials', 'atmospheric', 'oxygen', 'strongly', 'affected', 'light', 'heat', 'oxygen', 'concentration', 'moisture', 'presence', 'traces', 'impurities', 'the', 'impurities', 'however', 'act', 'either', 'catalysts', 'inhibitors', 'oxidation', 'the', 'degradation', 'poly-a-esters', 'studied', 'poly(isopropylidine', 'carboxylate)', '518', 'range', 'temperatures', '200', '800(cid1)c', 'among', 'decomposition', 'products', 'found', 'tetramethyl', 'glacolide', 'acetone', 'carbon', 'monoxide', 'lesser', 'extent', 'methacrylic', 'acid', 'the', 'primary', 'decomposition', 'product', 'appears', 'tetramethylene', 'glacolide', 'becomes', 'intermediate', 'upon', 'pyrolysis', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', '∆', 'also', 'o', 'o', '∆', 'o', 'o', 'o', 'o', 'o', 'o', 'co', 'decomposition', 'polycarbonates', 'studied', 'poly220-propane-bis-(4-phenyl)', 'carbonate', '519.', 'it', 'concluded', 'rearrangement', 'mechanism', 'main', 'intermediate', 'material', 'subsequent', 'formation', 'carbon', 'dioxide', 'volatile', 'phenolic', 'compounds', '519:', 'o', 'oh', 'o', 'o', 'o', 'o', '∆', 'o', 'o', 'h2o', '656', '9', 'organic', 'reactions', 'polymers', 'o', 'o', '2', 'oh', 'o', 'o', 'o', 'o', 'ho', 'oh', '993', 'thermal', 'degradation', 'polyamides', 'the', 'thermal', 'degradation', 'polyamides', 'starts', 'free-radical', 'cleavage', 'nitrogen–carbon', 'bonds', '520.', 'degradation', 'nylon', '6', 'illustrated', 'follows', 'when', 'nylon', '6', 'heated', '100', 'h', '305(cid1)c', 'half', 'nitrogen', 'escapes', 'polymer', 'small', 'amount', 'carbon', 'dioxide', 'forms', '457.', 'this', 'postulated', 'kamerbeek', 'et', 'al', 'result', 'reactions', 'two', 'terminal', 'amine', 'groups', '579:', 'o', 'o', 'n', 'h', 'o', 'nh2', '6', 'h2n', '6', 'n', 'h', 'o', 'n', 'h', '6', 'n', 'h', '6', 'n', 'h', 'nh3', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '657', 'in', 'decomposition', 'nylon', '66', 'additional', 'tendency', 'ring', 'closure', 'adipic', 'acid', '509:', 'h', 'n', 'o', '4', 'oh', 'o', '∆', 'h', 'n', 'o', 'η', 'ο', '2', 'nh2', 'co2', 'o', 'o', 'h', 'n', 'o', 'h2n', 'o', 'the', 'cyclization', 'reaction', 'also', 'occur', 'follows', '509', 'h', 'n', '∆', '4', 'o', 'o', 'h', 'n', 'h', 'n', 'perhaps', 'bimolecular', 'reaction', '509:', '2', 'o', 'o', 'h', 'n', 'co2', 'o', 'n', 'o', 'at', 'higher', 'temperatures', 'carbamoyl', 'ketone', 'might', 'convert', 'schiff', 'base', 'monomolecu-', 'lar', 'process', '509:', 'the', 'route', 'schiff', 'base', 'may', 'also', 'elimination', 'isocyanate', '509:', 'cn', 'h', 'o', 'o', 'co2', 'n', 'n', 'cn', 'h', 'o', 'o', 'co', 'n', 'rather', 'thermal', 'degradation', 'nylon', '6', 'possible', 'depolymerize', 'polyamide', 'kamimura', 'yamamoto', 'reported', 'possible', 'depolymerize', 'nylon', 'back', 'caprolactam', '510:', '658', '9', 'organic', 'reactions', 'polymers', 'o', 'nh', 'o', 'n', 'n', 'h', 'ionic', 'liquid', 'catalyst', '300', 'oc', 'the', 'best', 'yield', 'caprolactam', '86', 'obtained', 'n-methyl', 'n-propyl', 'piperidinium', 'bis', '(triﬂuoromethyl-sufonyl)imide', 'solvent', 'nn-dimethylpyridine', 'catalyst', 'in', 'past', 'believed', 'high', 'temperatures', 'nylons', 'degrade', 'faster', 'rate', 'elevated', 'humidity', 'this', 'assumption', 'however', 'shown', 'erroneous', 'bernstein', 'et', 'al', '457.', '994', 'thermal', 'degradation', 'epoxy', 'resins', 'the', 'thermal', 'degradations', 'bisphenol', 'a-based', 'epoxy', 'resins', 'cross-linked', 'diaminodiphe-', 'nylmethane', 'phthalic', 'anhydride', 'studied', 'aid', 'pyrolysis', 'radiochemical', 'gas', 'chromatography', 'technique', '511.', 'the', 'products', 'degradation', 'depend', 'upon', 'temperature', 'at', '400(cid1)c', 'amine', 'cross-linked', 'resins', 'yield', 'hydrogen', 'methane', 'water', 'the', 'resins', 'cross-linked', 'phthalic', 'anhydride', 'yield', 'hydrogen', 'methane', 'carbon', 'dioxide', 'water', 'the', 'number', 'degradation', 'products', 'increases', 'temperature', 'it', 'concluded', 'important', 'nonscission', 'reactions', 'resins', 'competing', 'dehydration', 'dehydrogenation', 'reactions', '511:', 'oh', 'r', 'r', 'h2o', 'a', 'nonscission', 'reaction', 'yields', 'methane', 'o', 'h2', 'r', 'r', 'h2o', 'the', 'scission', 'reactions', 'take', 'place', 'various', 'weak', 'spots', 'the', 'breakdown', 'bisphenol', 'a', 'segments', 'produces', 'phenol', '511:', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '659', 'o', '511:', '∆', 'o', 'oh', 'othre', 'fragments', 'the', 'main', 'differences', 'observed', 'amine-cured', 'anhydride-cured', 'resins', '1', 'the', 'amine', 'cross-linked', 'resin', 'generated', 'water', 'hydrogen', 'contained', '–ch2–choh–ch2–', 'groups', 'large', 'quantities', '2', 'scissions', 'anhydride-reacted', 'resins', 'tend', 'regenerate', 'anhydride', 'release', 'co', 'co2', '3', 'the', 'aliphatic', 'segments', 'amine', 'cross-linked', 'resins', 'yield', 'acetaldehyde', 'acetone', 'the', 'reverse', 'true', 'anhydride-cured', 'resins', 'this', 'thought', 'due', 'preferential', 'rupture', 'carbon–nitrogen', 'bonds', '511:', 'o', 'n', '2h', 'o', 'o', 'o', 'o', 'nh', 'o', '2h', 'h', 'o', 'the', 'anhydride-cured', 'compositions', 'may', 'tend', 'break', 'symmetrically', 'instead', 'o', 'o', 'o', 'oh', 'ho', 'o', 'o', '995', 'thermal', 'degradation', 'polyimides', 'polyoxidiazoles', 'polyquinoxalines', 'the', 'technique', 'mentioned', 'previous', 'section', 'pyrolysis', 'radiochemical', 'gas', 'chromatogra-', 'phy', 'also', 'applied', 'study', 'thermal', 'degradation', 'aromatic', 'polyimides', '512.', 'aromatic', 'polyimides', 'stable', 'thermally', 'previously', 'discussed', 'polymers', 'require', 'higher', 'temperature', 'decomposition', 'based', 'experimental', 'results', 'following', 'mechanism', 'proposed', '512.', 'initially', 'hydrogen', 'radicals', 'form', 'they', 'come', 'various', 'places', 'polymer', 'backbone', 'pyrolytic', 'decomposition', 'also', 'decomposition', 'amic', 'acid', 'may', 'yield', 'water', '(see', 'chap', '6)', '660', '9', 'organic', 'reactions', 'polymers', 'x', 'h2o', 'cooh', 'cooh', 'h2nx', 'x', 'co', 'n', 'x', 'n', 'o', 'o', 'h', 'o', 'n', 'h', 'h', 'co', 'h2n', 'cn', 'h2n', 'x', 'co', 'h2n', 'x', 'cn', 'another', 'study', 'thermal', 'degradation', 'aromatic', 'polyimides', 'led', 'similar', 'conclusions', '513.', 'at', 'lower', 'temperatures', 'degradation', 'polyimides', 'proceeds', 'hydrolytic', 'mechanism', 'greatly', 'inﬂuenced', 'presence', 'uncyclized', 'units', 'at', 'higher', 'temperatures', 'degradation', 'mainly', 'decarbonylation', 'imide', 'ring', 'breaking', 'polymer', 'chain', '“weak”', 'bonds', 'the', 'thermal', 'degradation', 'polyoxidiazoles', 'shown', 'proceed', 'mainly', 'heterocyclic', 'rings', 'apparently', 'weak', 'spots', '514', '515:', 'o', 'o', 'o2', 'co2', 'n', 'n', 'n', 'n', 'n2', 'h2n', 'nh2', 'nc', 'o', 'n', 'h', 'cn', 'o', 'n', 'h', 'the', 'process', 'aromatic', 'polyquinoxalines', 'decompose', 'random', 'at', '500–600(cid1)c', 'involves', 'opening', 'heterocyclic', 'ring', 'loss', 'fragment', '516:', 'n', 'n', 'n', 'n', '∆', 'at', 'still', 'higher', 'temperatures', '640', '690(cid1)c', 'dehydrogenation', 'nitrogen', 'residues', 'takes', 'place', 'c20h10n3', 'hcn', '(cn)2', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '661', '996', 'thermal', 'degradation', 'aromatic', 'polysulfones', 'in', 'aromatic', 'polysulfones', 'weakest', 'link', 'found', 'carbon–sulfur', 'bond', '517.', 'these', 'polymers', 'processed', '300(cid1)c', 'at', 'temperature', 'thermodegradative', 'processes', 'well', 'thermo-oxidative', 'ones', 'occur', 'the', 'rupture', 'c–s', 'bonds', 'produces', 'molecules', 'so2', 'fragments', 'polymer', 'molecules', 'terminal', 'phenyl', 'radicals', '517:', '∆', 'so2', 'the', 'phenyl', 'radicals', 'abstract', 'hydrogens', 'methyl', 'groups', 'present', 'polysulfone', 'prepared', 'bisphenol', 'a', 'o', 's', 'o', 'ch2', '997', 'thermal', 'degradation', 'polyethers', 'the', 'aromatic', 'polyethers', 'subject', 'rupture', 'ether', 'link', 'around', '300(cid1)c', '518.', 'products', 'ruptures', 'somewhat', 'similar', 'obtainable', 'decomposition', 'polysulfones', 'shown', 'the', 'thermal', 'degradation', 'polyoxymethylene', 'found', 'initiated', 'chain', 'ends', 'there', 'appear', 'three', 'possible', 'sites', 'initiation', 'homolytic', 'bond', 'cleavages', '519:', '998', 'thermal', 'degradation', 'cellulosic', 'materials', 'the', 'degradation', 'cellulose', 'triacetate', 'vacuum', 'analyzed', 'aid', 'chromatography', 'mass', 'spectrometry', 'infra-red', 'nmr', 'spectroscopy', '517.', 'the', 'mechanism', 'degradation', 'proposed', 'scotney', 'consist', 'primarily', 'deacetylation', 'polymer', 'chain', 'scission', 'chain', '14', 'glycosidic', 'linkage', 'pyranose', 'rings', '517.', 'the', 'type', 'products', 'formed', 'depends', 'upon', 'degradation', 'temperature', 'at', 'temperatures', '250(cid1)c', '350(cid1)c', 'unstable', 'intermediates', 'form', 'continued', 'heating', 'results', 'loss', 'carbon', 'monoxide', 'carbon', 'dioxide', 'acetic', 'acid', 'eventually', 'end', 'formation', 'tar', 'heating', '350(cid1)c', 'causes', 'condensation', 'radical', 'intermediates', 'cross-linked', 'aromatic', 'hydroaromatic', 'ring', 'systems', 'this', 'illustrated', 'follows', '662', '9', 'organic', 'reactions', 'polymers', 'oac', 'oac', 'o', 'o', 'o', 'ch2oac', 'ch2oac', 'o', 'o', 'oac', 'oac', 'oac', 'oac', 'o', 'o', 'ch2oac', 'ch2oac', 'o', 'o', 'oac', 'oac', '250', 'oc', 'hoac', 'ch2oac', 'o', 'o', 'o', 'o', 'oac', 'o', 'o', 'ch2oac', 'o', 'o', 'oac', 'glycosidic', 'scission', '250', 'oc', 'ch2oac', 'o', 'o', 'o', 'o', 'oac', 'tar', 'ch2oac', 'o', 'ooac', 'o', 'oac', 'o', 'o', 'the', 'mechanism', 'thermal', 'degradation', 'cellulose', 'appears', 'include', 'two', 'reactions', 'the', 'ﬁrst', 'reaction', 'consists', 'dehydration', 'second', 'one', 'scission', 'c–o', 'bonds', 'chains', 'rings', 'within', 'rings', '457.', 'this', 'give', 'rise', 'levoglucosan', '457:', 'ch2oh', 'oh', 'o', 'o', 'oh', 'o', 'oh', 'ch2oh', 'o', 'oh', 'o', 'o', 'oh', 'oh', 'oh', 'o', 'o', 'oh', 'o', 'oh', 'ch2oh', 'o', '999', 'hydrolytic', 'degradation', 'polymers', 'elevated', 'temperatures', 'hydrolytic', 'degradation', 'signiﬁcant', 'polymers', 'chain', 'links', 'react', 'water', 'polyesters', 'a', 'recent', 'study', 'reports', 'depolymerization', 'poly(ethylene', 'terephthalate)', 'processing', 'material', 'dried', 'thoroughly', 'prior', 'melting', '530.', 'the', 'hydrolytic', 'depolymeri-', 'zation', 'poly(ethylene', 'terephthalate)', 'carried', 'stirred', 'batch', 'reactor', '235', '250', '265(cid1)c', 'polymer', 'melting', 'point', 'autogenous', 'pressure', 'the', 'solid', 'products', 'mainly', 'composed', 'terephthalic', 'acid', 'the', 'liquid', 'products', 'mainly', 'composed', 'ethylene', 'glycol', 'small', 'amount', 'dimer', 'moreover', 'autocatalytic', 'mechanism', 'detected', 'that', 'indicates', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '663', 'hydrolytic', 'depolymerization', 'pet', 'catalyzed', 'carboxylic', 'acid', 'groups', 'produced', 'reaction', 'the', 'dependence', 'rate', 'constant', 'reaction', 'temperature', 'correlated', 'arrhenius', 'equation', '530.', '9910', 'oxidative', 'degradation', 'polymers', 'this', 'section', 'deals', 'oxidative', 'reactions', 'polymers', 'room', 'temperature', 'also', 'thermo-oxidative', 'degradation', 'elevated', 'temperatures', 'although', 'type', 'degradation', 'resembles', 'photo-oxidative', 'degradation', 'purpose', 'maintaining', 'clarity', 'latter', 'discussed', 'separately', 'next', 'section', 'the', 'simplest', 'oxidative', 'reactions', 'occur', 'hydrocarbon', 'polymers', 'this', 'free-radical', 'process', 'gener-', 'ally', 'follows', 'path', 'oxidation', 'low', 'molecular', 'weight', 'compounds', 'the', 'difference', 'however', 'propagation', 'reaction', 'in', 'oxidation', 'low-molecular', 'weight', 'hydrocarbons', 'step', 'chain', 'propagation', 'results', 'transfer', 'active', 'center', 'molecule', 'another', 'in', 'polymers', 'however', 'probability', 'transfer', 'low', 'instead', 'oxidation', 'propagates', 'along', 'polymer', 'backbone', '520.', '9911', 'oxidation', 'chain-growth', 'polymers', 'polymers', 'lack', 'double', 'bonds', 'like', 'polyethylene', 'considered', 'high', 'molecular', 'weight', 'parafﬁn', 'they', 'slow', 'oxidize', 'absence', 'uv', 'light', 'much', 'like', 'low', 'molecular', 'weight', 'hydrocarbons', 'on', 'hand', 'polymeric', 'materials', 'double', 'bonds', 'oxidize', 'rapidly', 'neverthe-', 'less', 'polymers', 'like', 'polyethylene', 'may', 'oxidize', 'rapidly', 'well', 'contaminated', 'metallic', 'ions', 'ions', 'catalyze', 'decomposition', 'peroxides', 'the', 'chemical', 'structure', 'polyoleﬁns', 'determines', 'susceptibility', 'oxidative', 'degradation', 'linear', 'polyethylene', 'absence', 'additives', 'resistant', 'oxidation', 'polypropylene', 'oxidizes', 'rather', 'readily', 'due', 'presence', 'labile', 'tertiary', 'hydrogens', 'it', 'demonstrated', 'instance', 'molecular', 'weigh', 'polypropylene', 'sheets', '138(cid1)c', 'oven', 'drop', '250000', 'approximately', '10000', '3', 'h', '522.', 'the', 'process', 'oxidation', 'shown', 'take', 'place', 'according', 'following', 'scheme', '522:', 'o2', 'o2', 'o', 'o', 'propagation', 'termination', 'ooh', 'o', 'oo', '664', '9', 'organic', 'reactions', 'polymers', 'ch2', 'o2', 'oo', 'o', 'o2', 'o', 'o', 'ooh', 'o', 'o', 'further', 'oxidative', 'degradation', 'fragments', 'leads', 'formation', 'carboxylic', 'acid', 'ester', 'g-lactone', '522.', 'it', 'also', 'found', 'main', 'oxidation', 'products', 'polyethylene', 'acid', 'ketone', 'on', 'hand', 'polypropylene', 'yields', 'upon', 'oxidation', 'approximately', 'equal', 'quantities', 'acid', 'ketone', 'aldehyde', 'ester', 'g-lactone', '522.', 'in', 'order', 'polymer', 'molecule', 'attacked', 'oxygen', 'must', 'come', 'contact', 'this', 'means', 'oxygen', 'must', 'able', 'permeate', 'material', 'otherwise', 'oxidation', 'occur', 'surface', 'it', 'shown', 'oxidation', 'occurs', 'readily', 'amorphous', 'regions', 'polymers', 'permeation', 'oxygen', 'hindered', 'chains', 'packed', 'tightly', 'together', 'crystallites', 'that', 'true', 'course', 'temperatures', 'tm', 'polymer', 'among', 'chain-growth', 'polymer', 'oxidation', 'polystyrene', 'investigated', 'thoroughly', 'it', 'found', 'rate', 'oxygen', 'absorption', 'number', 'chain', 'scissions', 'remain', 'constant', 'high', 'degree', 'reaction', 'there', 'evidence', 'cross-linking', 'circumstances', '523.', 'during', 'degradation', 'process', 'carbonyl', 'groups', 'accumulate', 'polymer', '524.', 'among', 'degradation', 'products', 'identiﬁed', 'benzaldehyde', '525', 'number', 'ketones', '526.', 'the', 'primary', 'oxidation', 'chain', 'scission', 'process', 'polystyrene', 'room', 'temperature', 'follows', '527:', 'o', 'o', 'o', 'o', 'hoo', 'reaction', 'also', 'proceed', 'manner', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '665', 'o', 'o', 'o', 'oh', 'o', 'o', 'oo', 'continuation', 'process', 'reduces', 'polymer', 'small', 'fragments', 'include', 'benzaldehyde', 'methyl', 'phenyl', 'ketone', '527.', 'when', 'solid', 'polystyrene', 'subjected', 'ozone', 'attack', 'carbonyl', 'peroxide', 'carboxylic', 'acid', 'groups', 'form', 'surface', 'polymer', '528.', 'the', 'reaction', 'rate', 'proportional', 'concentration', 'ozone', 'surface', 'sample', 'as', 'result', 'ozone', 'action', 'intramolecular', 'cross-', 'linking', 'takes', 'place', 'the', 'reaction', 'mechanism', 'ozone', 'attack', 'polystyrene', 'shown', 'follows', '528:', 'oo', 'o3', 'ho', 'ooh', 'o', 'o3', 'o', 'o', 'o', '666', '9', 'organic', 'reactions', 'polymers', 'in', 'poly(vinyl', 'toluene)', 'initial', 'oxidation', 'steps', 'consist', 'formations', 'radicals', 'tertiary', 'carbon', 'atoms', 'polystyrene', 'the', 'radicals', 'subsequently', 'form', 'peroxides', 'decompose', 'ketones', 'aldehydes', '9912', 'oxidation', 'step-growth', 'polymers', 'oxidation', 'step-growth', 'polymers', 'follows', 'paths', 'similar', 'oxidation', 'reactions', 'organic', 'molecules', 'thus', 'oxidation', 'poly(ethylene', 'terephthalate)', 'shown', 'proceed', 'follows', '529:', 'path1', 'o', 'o', 'o', 'o', '2', 'o', 'ooh', 'o', 'o', 'o', 'rh', 'r', 'o2', 'o', 'o', 'o', 'o', 'o', 'o', 'oh', 'o', 'o', 'o', 'o', 'o', 'oh', 'o', 'o', 'path2', 'o', 'o', 'o', 'o', '2', 'o', 'o2', 'o', 'o', 'o', 'o', 'oh', 'r', 'o', 'o', 'rh', 'o', 'o', 'during', 'thermal', 'oxidation', 'polyamides', 'n-vicinal', 'methylene', 'group', 'preferred', 'side', 'attack', '531.', 'the', 'reactivity', 'depends', 'apparently', 'conformation', 'amide', 'group', 'interaction', 'takes', 'place', 'p-electronic', 'system', 'carbonamide', 'group', 'c–h', 'bonds', 'in', 'polycaprolactam', 'exists', 'statistical', 'distribution', 'gauche', 'trans-conformations', 'the', 'n-vicinal', 'methylene', 'groups', 'trans-conformations', 'amide', 'groups', 'much', 'reactive', 'gauche', '531.', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '667', 'when', 'oxidation', 'polyamide', 'occurs', 'attack', 'no2', 'ozone', 'amide', 'groups', 'subject', 'attack', '532:', 'o', 'o', 'n', 'h', 'n', 'h', 'no2', 'o3', 'hno2', 'o', 'o', 'n', 'n', 'o', 'o', 'oh', 'o', 'n', 'o', 'ho2', 'this', 'appears', 'take', 'place', 'hydrogen-bonded', 'amide', 'groups', 'well', 'the', 'reaction', 'however', 'inhibited', 'benzaldehyde', 'benzoic', 'acid', 'when', 'degradation', 'take', 'place', 'occurs', 'random', '532.', 'thermo-oxidative', 'degradation', 'polyoxyglycols', 'reported', 'occur', 'typically', 'free-radical', 'mechanism', 'formation', 'hydroperoxides', 'the', 'decomposition', 'peroxides', 'leads', 'formation', 'acids', 'carbonyl', 'compounds', '533.', 'a', 'study', 'thermal', 'oxidation', 'oligomers', '(molecular', 'weight', 'approximately', '1900)', 'polyoxypropylene', 'glycols', 'showed', '70–120(cid1)c', 'end', 'hydroxyl', 'groups', 'responsible', 'degradation', 'chain', 'ruptures', 'occur', 'decomposition', 'hydroperoxides', '534:', 'two', 'types', 'cleavage', 'possible', 'one', 'take', 'place', 'carbon', 'carbon', 'single', 'bonds', 'follows', 'o', 'oh', 'o', 'o', 'o', 'o', 'o', 'o', 'oh', 'o', 'o', 'h2c', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'the', 'type', 'cleavage', 'occur', 'oxygen–carbon', 'bonds', 'both', 'long', 'short', 'molecular', 'chains', 'undergo', 'degradation', 'identical', 'manner', 'the', 'secondary', 'radicals', 'form', 'result', 'decomposition', 'alkoxy', 'radical', 'accelerate', 'oxidation', 'process', 'it', 'transforms', 'chain', 'process', 'accompanying', 'formation', 'various', 'compounds', 'the', 'products', 'alcohols', 'aldehydes', 'ketones', 'the', 'aldehydes', 'easily', 'oxidized', 'peracids', 'degrade', 'radicals', '668', '9', 'organic', 'reactions', 'polymers', 'thermal–oxidative', 'degradation', 'polysulfones', 'studied', '280(cid1)c', '535.', 'the', 'oxidation', 'radical', 'chain', 'reaction', 'initiation', 'consisting', 'hydrogen', 'abstraction', 'methyl', 'group', 'o', 's', 'o', 'o2', 'ch2', 'o', 's', 'o', 'hoo', 'the', 'process', 'continues', 'formation', 'alkyl', 'peroxide', 'radicals', 'isomerization', 'breakdown', 'formation', 'oxygen-containing', 'compounds', 'like', 'aldehydes', 'ketones', 'in', 'oxidative', 'pyrolysis', 'poly-p-xylene', 'oxygen', 'attacks', 'methylene', 'groups', 'ﬁrst', 'form', 'hydroperoxides', '536.', 'this', 'followed', 'chain', 'cleavage', 'oh', 'o', 'o', 'o2', 'h2c', 'o', 'oh', 'co', '-h', 'o', 'o2', 'o', 'oo', 'h', 'o', 'ooh', 'co2', 'h2o', '9913', 'photo-degradation', 'macromolecules', 'the', 'quantum', 'energies', 'associated', 'sunlight', 'violet', 'near', 'ultraviolet', 'portions', 'spectrum', 'magnitude', 'sufﬁcient', 'rupture', 'chemical', 'bonds', 'present', 'polymers', '537.', 'the', 'mechanism', 'photo-degradation', 'however', 'complex', 'would', 'visualized', 'simple', 'bond', 'rupture', 'photo-absorptions', 'complicated', 'various', 'factors', 'including', 'crystallinity', 'causes', 'scattering', 'light', 'in', 'addition', 'demonstrated', 'little', 'light', 'absorbed', 'wavelength', '2800', 'a˚', 'yet', 'wavelength', 'represents', 'lower', 'limit', 'sunlight', 'reaching', 'earth', 'on', 'hand', 'presence', 'impurities', 'act', 'photosensitizes', 'markedly', 'accelerate', 'degradation', 'process', 'polymers', 'current', 'commercial', 'use', 'large', 'scale', 'fall', 'roughly', 'three', 'categories', 'depending', 'upon', 'ability', 'withstand', 'photo-degradation', '538:', '1', 'polymers', 'resistant', 'sunlight', 'attacks', 'outdoors', 'like', 'polyethylene', 'poly(methyl', 'methacrylate)', '2', 'moderately', 'stable', 'polymers', 'like', 'poly(ethylene', 'terephthalate)', 'polycarbonate', '3', 'polymers', 'unstable', 'sunlight', 'require', 'ultraviolet', 'light', 'stabilizer', 'like', 'poly(vinyl', 'chloride)', 'polypropylene', 'nylons', 'rubbers', 'cellulose', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '669', 'in', 'addition', 'structural', 'instability', 'polymer', 'molecules', 'ultraviolet', 'light', 'degradation', 'may', 'also', 'accelerated', 'chromophores', 'form', 'oxidation', 'preparation', 'processing', 'these', 'compounds', 'act', 'excited', 'donors', 'transfer', 'energy', 'polymers', 'may', 'act', 'acceptors', 'an', 'intramolecular', 'energy', 'transfer', 'may', 'actually', 'occur', 'within', 'polymer', 'molecule', 'this', 'take', 'place', 'excited', 'chromophore', 'present', 'one', 'segment', 'chain', 'acceptor', 'another', 'segment', 'also', 'chromophores', 'like', 'carbonyl', 'groups', 'undergo', 'norrish', 'type', 'i', 'norrish', 'type', 'ii', 'reaction', 'several', 'studies', 'carried', 'photolysis', 'polyethylene', '539', '540.', 'the', 'primary', 'process', 'photo-degradation', 'however', 'still', 'elucidated', 'direct', 'irradiation', 'pure', 'low-density', 'polyeth-', 'ylene', 'ultraviolet', 'light', 'mercury', 'lamp', 'results', 'formation', 'free-radicals', '•', 'ch2–ch•–ch2', '539', '540.', 'identiﬁed', 'esr', 'they', 'alkyl', 'radicals', 'type', 'ch2–ch2', 'the', 'true', 'absorption', 'bands', 'polyethylene', 'however', 'located', 'wavelengths', 'shorter', '200', 'nm', 'it', 'difﬁcult', 'therefore', 'accept', 'radiation', 'lamp', 'caused', 'chain', 'scissions', 'output', 'lamp', 'much', 'longer', 'wavelength', 'it', 'suspected', 'therefore', 'degradation', 'result', 'photo-oxidation', 'this', 'discussed', 'next', 'section', 'a', 'similar', 'photo-degradation', 'process', 'believed', 'take', 'place', 'polypropylene', '541.', 'the', 'formation', 'free-radicals', 'ascribed', 'presence', 'oxidized', 'molecules', 'form', 'processing', 'the', 'oxidation', 'products', 'carbonyl', 'compounds', 'hydroperoxides', '542.', 'the', 'photolysis', 'carbonyl', 'derivatives', 'follows', '541:', 'o', 'hν', 'ch2', 'co', 'ch2', 'and', 'photolysis', 'chains', 'carrying', 'hydroperoxide', 'group', 'shown', 'manner', '541:', 'hν', 'cooh', 'hν', 'o', 'oh', 'photo-degradation', '12-polybutadiene', 'studied', 'ﬁlm', '543.', 'among', 'degradation', 'products', 'found', 'hydrogen', 'compounds', 'methyl', 'groups', 'vinyl', 'groups', 'cross-linked', 'material', 'the', 'following', 'mechanism', 'photo-initiation', 'degradation', 'reaction', 'proposed', '543:', 'ch2', 'o', 'hν', '210', 'nm', 'ch3', 'h', 'ch2', '670', '9', 'organic', 'reactions', 'polymers', 'the', 'mechanism', 'photo-degradation', 'poly(p-methylstyrene)', 'studied', 'several', 'investigators', '544–547.', 'a', 'gas', 'evolution', 'observed', 'irradiation', 'ultraviolet', 'light', 'this', 'gas', 'contains', 'hydrogen', 'major', 'portion', 'methane', 'ethane', 'minor', 'portions', 'there', 'also', 'traces', 'styrene', 'p-methylstyrene', 'toluene', '101.', 'the', 'gas', 'evolution', 'accompanied', 'cross-linking', 'the', 'start', 'process', 'pictured', 'follows', '547:', 'hν', 'ch2', 'ch2', 'ch2', 'the', 'products', 'react', 'yield', 'hydrogen', 'cross-linked', 'material', 'monomer', 'cyclohexadiene', 'photolysis', 'poly(p-isopropyl', 'styrene)', 'ultraviolet', 'light', '254', 'nm', 'vacuum', '10(cid3)6', 'mbar', 'room', 'temperature', 'yields', 'hydrogen', 'main', 'product', 'small', 'quantity', 'methane', 'ethane', 'trace', 'propane', '548.', 'poly(methyl', 'methacrylate)', 'depolymerizes', 'elevated', 'temperature', 'inﬂuence', 'ultravio-', 'let', 'light', '2597', 'nm', '549.', 'irradiation', 'polyacrylonitrile', 'however', 'leads', 'chain', 'scission', 'acrylonitrile', 'units', 'the', 'difference', 'thermal', 'ultraviolet', 'light', 'degradation', 'polyacryloni-', 'trile', 'principally', 'different', 'sites', 'initiation', 'fact', 'reaction', 'occurs', '160(cid1)c', 'presence', 'light', '280(cid1)c', 'darkness', '550.', 'when', 'copolymer', 'methyl', 'methacrylate', 'n-butyl', 'acrylate', 'irradiated', 'vacuum', 'elevated', 'temperature', 'like', '165(cid1)c', 'light', '2537', 'nm', 'gaseous', 'products', 'minor', '551.', 'these', 'hydrogen', 'carbon', 'monoxide', 'methane', 'the', 'liquid', 'products', 'predominantly', 'methyl', 'methacry-', 'late', 'n-butyl', 'acrylate', 'n-butyl', 'alcohol', 'n-butyraldehyde', 'the', 'degradation', 'explained', 'grassie', 'coworkers', '551', 'terms', 'radical', 'process', 'initiated', 'scission', 'pendant', 'acrylate', 'units', 'the', 'propagation', 'actually', 'combination', 'depropagation', 'chains', 'intra-', 'intermolec-', 'ular', 'chain', 'transfer', 'process', 'the', 'relative', 'importance', 'phase', 'depropagation', 'reaction', 'depends', 'upon', 'composition', 'copolymer', '551.', 'poly(vinyl', 'acetate)', 'shown', 'undergo', 'simultaneous', 'cross-linking', 'chain', 'scission', 'irradiated', 'vacuum', 'ultraviolet', 'light', '552.', 'the', 'following', 'mechanism', 'proposed', 'route', 'chain', 'cleavage', '552:', 'o', 'o', 'o', 'o', 'o', 'oh', 'o', 'o', 'o', 'o', 'oh', 'o', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '671', 'also', 'intramolecular', 'hydrogen', 'abstraction', 'assumed', 'occur', 'seven-membered', 'ring', 'transition', 'state', 'this', 'provide', 'route', 'chain', 'scission', 'oac', 'o', 'h', 'o', 'o', 'o', 'h', 'o', 'o', 'o', 'o', 'in', 'subsequent', 'studies', 'photo-degradation', 'poly(vinyl', 'acetate)', 'poly(vinyl', 'propionate)', 'vacuum', 'found', 'acids', 'aldehydes', 'carbon', 'dioxide', 'principal', 'products', '553.', 'two', 'mechanisms', 'one', 'involving', 'hydrogen', 'abstractions', 'postulated', 'one', 'takes', 'place', 'abstraction', 'acyl', 'radicals', 'formed', 'norrish', 'type', 'i', 'cleavages', 'intramolecular', 'hydrogen', 'abstraction', 'excited', 'carbonyl', 'groups', 'followed', 'fragmentation', '553:', 'hν', 'hν', 'o', 'o', 'r', 'o', 'o', 'r', '∗', 'h', 'o', 'o', 'r', 'o', 'o', 'r', 'o', 'r', 'oh', 'r', 'o', 'o', 'oh', 'a', 'second', 'perhaps', 'viable', 'mechanism', 'pictured', 'production', 'aldehydes', 'involving', 'hydrogen', 'abstraction', 'follows', '553:', 'hν', 'o', 'o', 'r', 'r', 'o', 'o', 'o', 'o', 'h', 'o', 'r', 'o', 'r', 'o', 'o', 'o', 'r', 'o', 'r', 'o', 'o', 'o', 'o', '672', '9', 'organic', 'reactions', 'polymers', 'the', 'main', 'chain', 'scissions', 'envisioned', 'involving', 'fragmentation', 'polymer', 'alkoxy', 'radicals', 'form', '553:', 'o', 'o', 'r', 'o', 'o', 'o', 'r', 'ch2', 'o', 'o', 'o', 'r', 'o', 'o', 'r', 'when', 'cast', 'poly(vinyl', 'chloride)', 'ﬁlms', 'irradiated', 'ultraviolet', 'light', 'various', 'intensities', 'temperatures', 'nitrogen', 'atmosphere', 'dehydrochlorination', 'reaction', 'takes', 'place', '554.', 'this', 'reaction', 'occurs', 'two', 'parts', 'during', 'ﬁrst', 'hour', 'reaction', 'dependent', 'intensity', 'light', 'temperature', 'becomes', 'independent', 'two', 'parameters', 'when', 'poly(vinyl', 'chloride)', 'ﬁlms', 'irradiated', 'presence', 'benzophenone', 'initiation', 'result', 'hydrogen', 'abstraction', 'polymer', 'excited', 'triplet', 'aromatic', 'ketone', '555:', 'this', 'followed', 'degradation', 'takes', 'place', 'chain', 'mechanism', 'o', 't1', '2', 'cl', 'oh', 'cl', 'cl', '2', 'cl', 'in', 'process', 'chlorine', 'radicals', 'abstract', 'hydrogens', 'form', 'hcl', 'the', 'quantum', 'yield', 'hydrogen', 'chloride', 'reaction', 'fhcl', '¼', '011', '556.', 'this', 'indicates', '1', 'every', '100', 'photons', 'absorbed', 'causes', 'dehydrochlorination', 'followed', 'formation', 'polyenes', 'it', 'led', 'conclusion', 'photochemical', 'process', 'must', 'take', 'place', 'polyene', 'sequences', 'hcl', '556', '557.', 'when', 'poly(phenyl', 'vinyl', 'ketone)', 'irradiated', 'ultraviolet', 'light', '365', 'nm', 'degrades', 'exhibits', 'loss', 'molecular', 'weight', '558.', 'no', 'benzaldehyde', 'found', 'among', 'products', 'that', 'excludes', 'norrish', 'type', 'i', 'reaction', 'indicates', 'polymer', 'cleaves', 'norrish', 'type', 'ii', 'mechanism', '558.', 'day', 'wiles', 'pictured', 'primary', 'steps', 'photochemical', 'decomposition', 'poly', '(ethylene', 'terephthalate)', 'follows', '559:', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '673', 'o', 'o', 'hν', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'ch2', 'the', 'mechanism', 'formation', 'carboxylic', 'acid', 'groups', 'terminal', 'ends', 'degradation', 'products', 'explained', 'norrish', 'type', 'ii', 'reaction', '559:', 'o', 'o', '∗', 'in', 'addition', 'photo-fries', 'rearrangements', 'occur', 'various', 'aromatic', 'polyester', 'ﬁlms', 'upon', 'ultraviolet', 'light', 'irradiation', '560', '561.', 'also', 'photo-fries', 'rearrangements', 'believed', 'occur', 'polycarbonate', 'resins', 'well', '562.', 'this', 'illustrated', 'follows', 'o', 'o', 'hν', 'o', 'o', 'oh', 'o', 'o', 'o', 'o', 'o', 'h', 'o', 'o', 'hν', 'o', 'o', 'o', 'o', 'o', 'o', '674', '9', 'organic', 'reactions', 'polymers', 'the', 'true', 'aromatic', 'polyamides', 'photo-fries', 'rearrangements', 'done', 'according', 'following', 'mechanism', '563:', 'o', '∗', 'hν', 'n', 'h', 'o', 'n', 'h', 'h', 'n', 'o', 'n', 'h', 'o', 'o', 'n', 'h', 'o', 'n', 'h', 'o', 'n', 'h', 'nh2', 'o', 'h', 'n', 'o', 'photolytic', 'decomposition', 'polycarbonate', 'ﬁlms', 'shown', 'produce', 'products', 'also', 'consistent', 'photo-fries', 'reaction', 'these', 'salicylic', 'acid', 'bisphenol', 'type', 'species', '580.', 'this', 'agreement', 'earlier', 'studies', 'showed', 'variety', 'processes', 'including', 'rearrangements', '(photo-fries)', 'photo-oxidation', 'occur', 'bisphenol', 'a-based', 'polycarbonate', 'photolyzed', '581', '582.', '9914', 'photo-oxidative', 'degradations', 'polymers', 'singlet', 'oxygen', 'lowest', 'excited', 'state', 'molecular', 'oxygen', '1dg', 'o2', 'known', 'cause', 'rapid', 'degradation', 'polymers', 'singlet', 'oxygen', 'forms', 'several', 'processes', 'including', 'one', 'two', 'mechanisms', '(1)', 'photosensitization', 'impurity', '(2)', 'energy', 'transfer', 'excited', 'triplet', 'state', 'chromophore', 'polyoleﬁns', 'unprotected', 'weathering', 'react', 'readily', 'oxygen', 'presence', 'sunlight', 'even', 'room', 'temperature', 'this', 'leads', 'loss', 'molecular', 'weight', 'often', 'embrittlement', 'the', 'mechanism', 'photo-oxidation', 'polyethylene', 'demonstrated', 'involve', 'carbonyl', 'groups', 'developed', 'polymer', 'via', 'oxidation', 'processing', '564', '565.', 'these', 'carbonyl', 'groups', 'act', 'chromophores', 'triplet', 'states', '(n', 'p)', 'quenched', 'molecular', 'oxygen', 'ground', 'triplet', 'state', 'electronically', 'excited', 'singlet', 'oxygen', 'molecules', 'form', 'result', '564.', 'c', '¼', 'o3ðn', 'p(cid8)þ', 'þ', 'o2ð3sg', '(cid3)þ', 'c', '¼', 'o(s0þþ1o2', 'the', 'norrish', 'type', 'ii', 'reaction', 'takes', 'place', 'cleavage', 'carbon', 'carbon', 'bonds', 'formation', 'oleﬁns', 'subsequently', 'formation', 'hydroperoxides', '99', 'thermal', 'degradation', 'common', 'step-growth', 'polymers', '675', 'o', 'hν', 'o', 'h', 'o', 'o', 'oh', 'the', 'hydroperoxides', 'apparently', 'photo', 'inductive', 'effect', 'overall', 'oxidation', 'polyethylene', '566.', 'photolysis', 'hydroperoxides', 'however', 'initiates', 'new', 'vinylidine', 'oxidation', 'it', 'shown', 'kinetics', 'photo-oxidation', 'polyethylene', 'characterized', 'superpo-', 'sition', 'two', 'phenomena', 'the', 'ﬁrst', 'corresponds', 'exponential', 'increase', 'concentration', 'carbonyl', 'groups', 'time', 'observed', 'kinetics', 'controlled', 'diffusion', 'oxygen', 'the', 'second', 'one', 'controlled', 'diffusion', 'corresponds', 'linear', 'increase', 'carbonyl', 'concentration', 'time', 'takes', 'place', 'degraded', 'samples', 'this', 'explained', 'terms', 'chain', 'rupture', 'amorphous', 'regions', 'polymer', 'allowing', 'free', 'access', 'oxygen', '567.', 'commercial', 'polypropylene', 'photo-degrade', 'rapidly', 'this', 'due', 'presence', 'chromophore', 'groups', 'form', 'polymerization', 'in', 'addition', 'chromophores', 'may', 'also', 'form', 'oxidation', 'singlet', 'oxygen', 'extrusion', 'air', 'at', 'temperatures', '250–300(cid1)c', 'light', '300', 'nm', 'initiate', 'oxidation', 'atmospheric', 'oxygen', '568.', 'the', 'hydroperoxide', 'groups', 'form', 'stable', 'decompose', 'alkoxy', 'radicals', 'form', 'ketones', 'backbones', 'ends', 'chains', 'these', 'ketones', 'turn', 'undergo', 'norrish', 'type', 'i', 'type', 'ii', 'reactions', 'irradiated', 'ultraviolet', 'light', '568:', 'o', 'hν', '90', 'oc', '10', 'o', 'ch2', 'oh', 'acetone', '676', '9', 'organic', 'reactions', 'polymers', 'photo-oxidation', 'poly(vinyl', 'chloride)', 'appears', 'greatly', 'enhanced', 'imperfections', 'polymer', 'structure', 'form', 'polymerization', 'processing', 'formation', 'peroxide', 'carbonyl', 'groups', 'difﬁcult', 'prevent', 'the', 'initiation', 'photo-oxidation', 'probably', 'results', 'free-radical', 'forms', 'groups', 'decompose', 'upon', 'irradiation', '568.', 'in', 'addition', 'double', 'bonds', 'form', 'loss', 'hcl', 'sensitize', 'formation', 'free', 'radicals', 'these', 'radicals', 'oxidized', 'also', 'promote', 'cross-linking', 'results', 'gels', 'photo-oxidation', 'cis-polybutadiene', 'also', 'shown', 'involve', 'singlet', 'oxygen', '570.', 'attacks', 'singlet', 'oxygen', 'double', 'bonds', 'formation', 'allylic', 'hydroperoxides', 'shifts', 'double', 'bonds', 'according', 'ene', 'reaction', 'conﬁrmed', 'several', 'studies', '571.', 'on', 'hand', 'study', 'model', 'compounds', 'photo-oxidation', 'polyisoprene', 'failed', 'show', 'formation', 'endo', 'peroxides', '14-cycloaddition', '572.', 'aromatic', 'polyurethanes', 'exhibit', 'strong', 'tendency', 'yellow', 'sunlight', 'this', 'may', 'accounted', 'two', 'mechanisms', 'based', 'formation', 'different', 'photoproducts', 'the', 'ﬁrst', 'one', 'found', 'photo-fries', 'reaction', 'caused', 'short', 'wave', 'length', 'light', '572.', 'the', 'second', 'one', 'caused', 'longer', 'wave', 'length', 'light', 'formation', 'quinonoid', 'structures', '573:', 'o', 'o', 'n', 'h', 'o', 'o', 'n', 'h', 'o', 'o', 'n', 'o', 'r', 'o', 'n', 'h', 'hνo2', 'o', 'r', 'n', 'o', 'o', 'r', 'n', 'o', 'photo-oxidation', 'nylon', '6', 'also', 'investigated', '574', 'the', 'subject', 'photo-oxidation', 'silicones', 'seldom', 'raised', 'polymers', 'known', 'stability', 'oxidation', 'nevertheless', 'investigation', 'photo-oxidative', 'degradation', 'materials', 'showed', 'materials', 'affected', 'combined', 'attacks', 'oxygen', 'ultraviolet', 'irradiation', '585.', 'the', 'groups', 'affected', 'functional', 'groups', 'fragile', 'backbones', 'thus', 'silicon', 'hydride', 'group', 'found', 'readily', 'photo-oxidized', 'perhaps', 'due', 'low', 'bond', 'energy', 'the', 'reaction', 'illustrated', 'follows', '585:', 'si', 'h', 'si', 'o2', 'si', 'oo', 'sih', 'si', 'ooh', 'the', 'main', 'process', 'occurs', 'silicones', 'contain', 'vinyl', 'groups', 'photo-scission', 'also', 'vinyl', 'groups', 'photo-oxidized', 'silanols', 'carboxylic', 'acids', 'esters', 'part', 'silanols', 'condensed', 'high', 'molecular', 'weight', 'silicones', '585.', 'review', 'questions', '677', '9915', 'degradation', 'polymeric', 'materials', 'ionizing', 'radiation', 'ionizing', 'radiation', 'absorbed', 'selectively', 'like', 'ultraviolet', 'instead', 'absorption', 'gamma', 'rays', 'function', 'electron', 'density', 'path', 'radiation', 'after', 'random', 'ray', 'strikes', 'atom', 'however', 'energy', 'transfer', 'occurs', 'free-radicals', 'may', 'form', 'subsequent', 'reactions', 'like', 'oxidation', 'chain', 'reactions', 'may', 'proceed', 'manner', 'uv', 'light', 'irradiation', 'in', 'addition', 'creating', 'free-radicals', 'ionizing', 'radiation', 'also', 'gives', 'rise', 'ions', 'electrons', 'excited', 'molecules', 'medium', 'this', 'followed', 'reactions', 'like', 'chain', 'scission', 'formation', 'unsaturation', 'cross-', 'linking', 'accompanied', 'formation', 'volatile', 'products', 'due', 'high', 'energy', 'ionizing', 'radiation', 'irradiation', 'dose', 'sufﬁciently', 'high', 'polymers', 'degrade', 'form', 'low', 'molecular', 'weight', 'fragments', 'the', 'difference', 'rates', 'degradation', 'amounts', 'cross-linking', 'among', 'vinyl', 'polymers', 'tend', 'unzip', 'yield', 'large', 'quantities', 'monomers', 'thermal', 'degradation', 'also', 'likely', 'loose', 'molecular', 'weight', 'rapidly', 'upon', 'irradiation', 'on', 'hand', 'vinyl', 'polymers', 'yield', 'low', 'quantities', 'monomers', 'pyrolysis', 'tend', 'cross-link', 'instead', 'also', 'observed', 'polymeric', 'materials', 'exhibit', 'high', 'heats', 'polymerization', 'formation', 'tend', 'cross-link', 'tend', 'degrade', '575.', 'it', 'reported', 'back', '1952', 'polyethylene', 'could', 'cross-linked', 'irradiation', 'controlled', 'manner', '577.', 'the', 'reaction', 'accompanied', 'evolution', 'considerable', 'quantities', 'volatile', 'gases', 'consist', 'mainly', 'hydrogen', 'the', 'portion', 'hydrogen', 'released', 'however', 'lower', 'amount', 'volatile', 'hydrocarbons', 'greater', 'polyethylene', 'branched', '578.', 'there', 'appears', 'inverse', 'relationship', 'amount', 'branching', 'quantity', 'hydrogen', 'gas', 'released', '578.', 'polypropylene', 'reported', 'cross-link', 'irradiation', '579.', 'at', 'time', 'large', 'evolution', 'gas', 'if', 'irradiation', 'carried', 'air', 'marked', 'degradation', 'even', 'fairly', 'low', 'doses', 'when', 'polyisobutylene', 'irradiated', 'molecular', 'weight', 'decreases', 'rapidly', 'viscous', 'low', 'molecular', 'weight', 'liquid', 'remains', 'at', 'time', 'mixture', 'hydrogen', 'methane', '(95', 'total', 'gas)', 'released', '582.', 'generally', 'aromatic', 'structures', 'stable', 'ionizing', 'radiation', 'aliphatic', 'ones', 'accordingly', 'polystyrene', 'resistant', 'polyethylene', '575.', 'in', 'addition', 'cross-linking', 'predominates', 'chain', 'scission', 'the', 'radiolysis', 'poly(methyl', 'methacrylate)', 'results', 'rapid', 'loss', 'molecular', 'weight', 'this', 'degradation', 'increases', 'intensity', 'radiation', '579.', 'the', 'volatile', 'products', 'form', 'hydrogen', 'carbon', 'dioxide', 'carbon', 'monoxide', 'methane', 'propane', 'methyl', 'methacrylate', 'monomer', 'this', 'varies', 'temperature', 'type', 'ionizing', 'radiation', 'polymer', 'exposed', 'polytetraﬂuoroethylene', 'extremely', 'sensitive', 'radiation', 'exhibits', 'marked', 'damage', 'mechanical', 'properties', 'it', 'shown', 'ﬂuoride', 'ions', 'evolve', 'polymer', 'irradiation', 'also', 'long', 'periods', 'afterward', '134.', 'the', 'molecular', 'weight', 'polymer', 'drops', 'steadily', 'dose', 'many', 'double', 'bonds', 'form', 'polymer', '580.', 'review', 'questions', 'section', '91', '1', 'what', 'necessary', 'conditions', 'fair', 'comparison', 'reactivity', 'functional', 'groups', 'macromolecules', 'small', 'molecules', '2', 'when', 'unequal', 'reactivity', 'observed', 'large', 'small', 'molecules', '678', '9', 'organic', 'reactions', 'polymers', '3', 'discuss', 'effect', 'diffusion-controlled', 'reactions', 'reactivity', 'macromolecules', '4', 'discuss', 'paired', 'group', 'neighboring', 'group', 'effects', 'inﬂuence', 'random', 'irreversible', 'reactions', '5', 'discuss', 'reactions', 'favor', 'large', 'molecules', 'section', '92', '1', 'explain', 'showing', 'chemical', 'equations', 'hydrochlorination', 'natural', 'rubber', 'often', 'accompanied', 'cyclization', '2', 'discuss', 'chlorination', 'natural', 'rubber', 'how', 'natural', 'rubber', 'polybutadiene', 'brominated', '3', 'discuss', 'aid', 'chemical', 'equations', 'hydrogenation', '14-polybutadiene', '4', 'how', 'polyisoprene', 'polybutadiene', 'modiﬁed', 'additions', 'carbenes', 'explain', 'discuss', 'showing', 'structures', 'starting', 'materials', 'products', '5', 'illustrate', 'prins', 'reactions', 'rubber', 'formaldehyde', 'glyoxal', 'how', 'reaction', 'carried', 'solid', 'phase', '6', 'discuss', 'polar', 'additions', 'unsaturated', 'polymers', 'give', 'examples', 'include', 'ene', 'reaction', '7', 'how', 'ritter', 'reactions', 'carried', 'isoprenes', '8', 'how', 'diels-alder', 'reactions', 'carried', 'unsaturated', 'polymers', 'use', 'chemical', '9', 'explain', 'polybutadiene', 'isoprene', 'epoxidized', 'giving', 'reagents', 'showing', 'equations', 'illustrate', 'by-products', 'section', '93', 'light', 'section', '94', '1', 'show', 'explain', 'cis-polybutadiene', 'rearranges', 'trans', 'result', 'irradiation', 'gamma', 'rays', 'ultraviolet', 'light', 'also', 'show', 'happens', 'polyisoprene', 'irradiated', 'ultraviolet', 'light', '2', 'discuss', 'aid', 'chemical', 'equations', 'cyclization', 'reactions', 'rubber', '3', 'how', 'polyacrylonitrile', 'ﬁbers', 'converted', 'graphite-like', 'ﬁbers', 'explain', 'showing', 'steps', 'thermal', 'cyclization', 'polyacrylonitrile', '4', 'discuss', 'migration', 'double', 'bonds', 'polymers', '5', 'how', 'poly(440-diphenylpropane', 'isophthalate)', 'rearrange', 'upon', 'irradiation', 'ultraviolet', '6', 'give', 'examples', 'schmidt', 'beckmann', 'rearrangements', '1', 'discuss', 'help', 'chemical', 'equations', 'ﬂuorination', 'reactions', 'polyethylene', '2', 'discuss', 'chlorination', 'polyethylene', 'polypropylene', '3', 'how', 'chlorosulfonation', 'polyethylene', 'carried', 'industrially', 'explain', 'write', 'chemical', 'equations', 'reactions', 'how', 'product', 'used', 'review', 'questions', '679', '4', 'discuss', 'chlorination', 'poly(vinyl', 'chloride)', '5', 'discuss', 'chlorination', 'poly(vinylidine', 'chloride)', 'poly(vinyl', 'ﬂuoride)', '6', 'describe', 'reactions', 'sodium', 'azide', 'poly(vinyl', 'chloride)', 'subsequent', 'reactions', 'azide', 'group', 'use', 'chemical', 'equations', '7', 'describe', 'reactions', 'poly(vinyl', 'chloride)', 'sulfur', 'compounds', '8', 'what', 'products', 'reactions', 'poly(vinyl', 'chloride)', 'aniline', 'show', 'explain', '9', 'explain', 'carbanionic', 'centers', 'formed', 'backbones', 'poly(vinyl', 'chloride)', 'molecules', 'show', 'subsequent', 'reactions', 'take', 'place', '10', 'what', 'happens', 'poly(vinyl', 'chloride)', 'treated', 'organolithium', 'compounds', 'explain', 'help', 'chemical', 'equations', '11', 'discuss', 'photo', 'chlorination', 'polystyrene', '12', 'discuss', 'chloromethylation', 'reactions', 'polystyrene', 'copolymers', 'showing', 'chemical', '13', 'discuss', 'various', 'reactions', 'chloromethylated', 'polystyrene', 'help', 'chemical', 'equations', '14', 'show', 'examples', 'friedel-craft', 'reactions', 'polystyrene', 'chloromethylated', 'polystyrene', '15', 'discuss', 'sulfonation', 'polystyrene', '16', 'discuss', 'nitration', 'polystyrene', 'reduction', 'nitropolystyrene', 'subsequent', 'diazotization', 'reactions', 'reaction', '17', 'discuss', 'matalation', 'polystyrene', 'subsequent', 'reactions', '18', 'discuss', 'reduction', 'reactions', 'polymers', 'methacrylic', 'acrylic', 'esters', 'metal', 'hydrides', 'what', 'solvents', 'give', 'optimum', 'conditions', '19', 'discuss', 'nucleophilic', 'substitution', 'reactions', 'poly(methyl', 'methacrylate)', 'arndt-eister', 'reactions', 'poly(methacryloyl', 'chloride)', 'curtius', 'lossen', 'rearrangements', 'poly', '(acryloyl', 'chloride)', '20', 'show', 'diels-alder', 'reaction', 'poly(furfuryl', 'methacrylate)', 'maleic', 'anhydride', '21', 'what', 'industrial', 'processes', 'preparation', 'poly(vinyl', 'butyral)', 'describe', 'process', 'showing', 'chemical', 'reactions', '22', 'how', 'schotten-baumann', 'esteriﬁcations', 'poly(vinyl', 'alcohol)', 'carried', 'explain', 'section', '95', '1', 'discuss', 'vulcanization', 'reactions', 'rubbers', 'show', 'chemical', 'reactions', '2', 'discuss', 'ionizing', 'radiation', 'cross-linking', 'polymers', '3', 'show', 'reaction', 'photocross-linking', 'poly(vinyl', 'cinnamate)', 'explain', 'mechanism', '4', 'show', 'reactions', 'photocross-linking', 'polymers', 'bearing', 'azide', 'groups', '5', 'draw', 'examples', 'polymers', 'pendant', 'benzoin', 'furoin', 'groups', '6', 'explain', 'mechanism', 'photocross-linking', '2p', '2p', 'type', 'dimerization', 'give', 'examples', '7', 'discuss', 'typical', 'ultraviolet', 'light-curable', 'coating', 'compositions', 'free', 'radical', 'cationic', 'mechanisms', '8', 'why', 'photoinitiators', 'preferred', 'photosensitizers', 'uv-curable', 'materials', '9', 'discuss', 'help', 'chemical', 'equations', 'mechanism', 'photoreduction', 'aromatic', 'ketone', 'photoinitiators', 'hydrogen', 'donors', 'electron', 'donors', '10', 'discuss', 'photoinitiators', 'photocross-linking', 'cationic', 'mechanisms', 'showing', 'reactions', 'products', '11', 'discuss', 'cross-linking', 'reactions', 'electron', 'beams', '9', 'organic', 'reactions', 'polymers', '1', 'what', 'meant', 'microporous', 'macroreticular', 'supports', '2', 'discuss', 'applications', 'polymeric', 'reagents', 'show', 'examples', '3', 'how', 'enzymes', 'immobilized', 'discuss', 'showing', 'chemical', 'reactions', '4', 'discuss', 'nonenzymatic', 'catalysts', '1', 'what', 'mechanism', 'free-radical', 'graft', 'copolymer', 'formation', 'chain-transferring', 'technique', 'explain', 'examples', 'discuss', 'advantages', 'disadvantages', '2', 'discuss', 'free-radical', 'grafting', 'reactions', 'polymers', 'double', 'bonds', 'give', 'examples', 'show', '3', 'how', 'macromonomers', 'used', 'form', 'graft', 'copolymers', 'give', 'several', 'examples', '4', 'how', 'polymerizations', 'initiated', 'backbones', 'polymers', 'explain', 'give', '5', 'discuss', 'photochemical', 'preparation', 'graft', 'copolymers', '6', 'how', 'high-energy', 'radiation', 'used', 'form', 'graft', 'copolymers', '7', 'discuss', 'formations', 'graft', 'copolymers', 'ionic', 'chain-growth', 'step-growth', 'polymerizations', '1', 'discuss', 'advantages', 'block', 'copolymers', 'polymer', 'blends', '2', 'how', 'block', 'copolyesters', 'formed', 'explain', 'chemical', 'equations', '3', 'how', 'block', 'copolyamides', 'formed', 'explain', 'question', '2', '4', 'how', 'polyurethane-polyamide', 'polyamide-polyester', 'block', 'copolymers', 'formed', 'explain', 'show', 'chemical', 'reactions', '5', 'what', 'polyurethane', 'ionomers', 'how', 'prepared', '6', 'describe', 'technique', 'preparation', 'block', 'copolymers', 'poly(a-oleﬁns)', '7', 'how', 'block', 'copolymers', 'form', 'simultaneous', 'free', 'radical', 'ionic', 'chain-growth', 'polymerizations', 'explain', 'give', 'examples', '8', 'discuss', 'formations', 'block', 'copolymers', 'homogeneous', 'ionic', 'copolymerization', '9', 'how', 'block', 'copolymers', 'form', 'mechanochemical', 'techniques', '680', 'section', '96', 'section', '97', 'reactions', 'examples', 'section', '98', 'section', '99', 'section', '910', '1', 'what', 'meant', 'polymer', 'degradation', 'various', 'causes', '1', 'what', 'common', 'technique', 'studying', 'thermal', 'degradation', 'polymers', '2', 'explain', 'two', 'thermal', 'degradation', 'processes', 'chain-growth', 'polymers', 'review', 'questions', '681', '3', 'describe', 'thermal', 'degradation', 'polyoleﬁns', '4', 'describe', 'thermal', 'degradation', 'polybutadienes', '5', 'what', 'thermal', 'degradation', 'process', 'polystyrene', 'polystyrene-like', 'polymers', '6', 'how', 'methacrylic', 'acrylic', 'esters', 'degrade', 'thermally', '7', 'what', 'happens', 'polyacrylonitrile', 'heated', 'describe', '8', 'discuss', 'thermal', 'degradation', 'poly(vinyl', 'chloride)', '9', 'discuss', 'thermal', 'degradation', 'ﬂuorocarbon', 'polymers', '10', 'discuss', 'thermal', 'degradation', 'poly(vinyl', 'acetate)', '1', 'can', 'step-growth', 'polymers', 'degrade', 'manner', 'chain-growth', 'polymers', '2', 'how', 'polyoxides', 'degrade', 'thermally', '3', 'discuss', 'thermal', 'degradation', 'polyesters', '4', 'discuss', 'thermal', 'degradation', 'polyamides', '5', 'discuss', 'thermal', 'degradation', 'epoxy', 'resins', '6', 'how', 'polyimides', 'polyquinoxalines', 'degrade', 'thermally', '7', 'discuss', 'thermal', 'degradation', 'polysulfones', '8', 'how', 'cellulosic', 'materials', 'degrade', 'thermally', 'section', '911', 'section', '912', 'section', '913', 'section', '914', '1', 'how', 'hydrolytic', 'degradation', 'polymers', 'take', 'place', '1', 'describe', 'oxidative', 'degradation', 'hydrocarbon', 'polymers', '2', 'describe', 'oxidative', 'degradation', 'polystyrene', '3', 'describe', 'oxidative', 'degradation', 'poly(ethylene', 'terephthalate)', '4', 'how', 'polyamides', 'degrade', 'oxidatively', '1', 'what', 'meant', 'photo-degradation', 'polymers', '2', 'describe', 'photolysis', 'polyethylene', '3', 'describe', 'photo-degradation', 'polybutadiene', 'poly(p-methylstyrene)', '4', 'how', 'poly(methyl', 'methacrylate)', 'degrade', 'elevated', 'temperature', 'inﬂuence', 'ultraviolet', 'light', '5', 'describe', 'photo-degradation', 'poly(vinyl', 'acetate)', 'poly(vinyl', 'chloride)', '6', 'what', 'mechanism', 'photo-degradation', 'aromatic', 'polyesters', '9', 'organic', 'reactions', 'polymers', '1', 'what', 'causes', 'photo-oxidative', 'degradation', 'take', 'place', 'polyoleﬁns', '2', 'how', 'commercial', 'polypropylene', 'degrade', 'photo-oxidatively', '1', 'if', 'select', 'polymeric', 'materials', 'use', 'near', 'nuclear', 'reactor', 'types', 'materials', 'would', 'select', 'explain', '682', 'section', '915', 'section', '916', 'references', '1', 'pj', 'flory', 'principles', 'polymer', 'chemistry', 'cornell', 'university', 'press', 'ithaca', 'new', 'york', '1953', 'chaptii', '2', 't', 'alfrey', 'jr,', 'chemical', 'reactions', 'polymers', '(em', 'fettersed),', 'wiley-interscience', 'new', 'york', '1964', 'capt', 'ia', '3', 'dtl', 'chen', 'h', 'morawetz', 'macromolecules', '9', '463', '(1976)', '4', 's', 'malkin', 'e', 'fisher', 'j', 'phys', 'chem,', '66', '2482', '(1962)', '5', 'h', 'morawetz', 'j', 'macromol', 'sci-chem,', 'a13', '311', '(1979)', '6', 'm', 'kamachi', 'makromol', 'chem,', 'suppl,', '14', '17', '(1985)', '7', 'a', 'okamoto', 'y', 'shimanuki', 'i', 'mita', 'eur', 'polym', 'j,', '18', '545', '(1982)', 'ibid', '19', '341', '(1983)', '8', 'kf', 'o’driscoll', 'hk', 'mahabadi', 'j', 'polymer', 'sci,', 'chem', 'ed,', '11', '869', '(1976)', '9', 's', 'benson', 'am', 'north', 'j', 'am', 'chem', 'soc,', '84', '935', '(1962)', '10', 'm', 'kamachi', 'adv', 'polym', 'sci,', '38', '56', '(1981)', '11', 'pj', 'flory', 'j', 'am', 'chem', 'soc,', '61', '1518', '(1939)', 'thk', 'barron', 'ea', 'boucher', 'trans', 'faraday', 'soc,', '65', '(12)', '3301', '(1969)', '12', 'cs', 'marvel', 'cl', 'levesque', 'j', 'am', 'chem', 'soc,', '61', '3234', '(1939)', '13', 'h', 'morawetz', 'pe', 'zimmerling', 'j', 'phys', 'chem,', '58', '753', '(1954)', '14', 'h', 'morawetz', 'e', 'gaetjens', 'j', 'polym', 'sci,', '32', '526', '(1958)', '15', 'pe', 'zimmerling', 'ew', 'westhead', 'jr,', 'h', 'morawetz', 'biochem', 'biophys', 'acta', '25', '376', '(1957)', '16', 'vo', 'cherkezyan', 'ad', 'litmanovich', 'eur', 'polym', 'j,', '21', '623', '(1985)', '17', 'ad', 'litmanovich', 'eur', 'polym', 'j,', '16', '269', '(1980)', '18', 's', 'sawant', 'h', 'morawetz', 'macromolecules', '17', '2427', '(1984)', '19', 'i', 'sakurada', 'off', 'j', 'iupac', '16', '(2-3)', '263', '(1968)', '20', 'd', 'pini', 'r', 'settambolo', 'a', 'raffaelli', 'p', 'salvadori', 'macromolecules', '20', '58', '(1987)', '21', 'e', 'gaetjens', 'h', 'morawetz', 'j', 'am', 'chem', 'soc,', '83', '1738', '(1961)', '22', 'n', 'plate', 'vysokomol', 'soyed,', 'a10', '(12)', '2650', '(1968)', '23', 'wh', 'daly', 'j', 'macromol', 'sci-chem,', 'a22', '713', '(1985)', '24', 'ar', 'mathieson', 'jv', 'mclaren', 'j', 'polymer', 'sci,', 'a3', '2555', '(1965)', '25', 'jc', 'layte', 'm', 'mandel', 'j', 'polymer', 'sci,', 'a2', '1879', '(1964)', '26', 'f', 'fleming', 'gr', 'marshal', 'tetrahedron', 'lett,', '2403', '(1970)', '27', 'a', 'patchornik', 'ma', 'kraus', 'j', 'am', 'chem', 'soc,', '92', '7587', '(1970)', '28', 'ebondi', 'm', 'fidkin', 'a', 'patchornik', 'israel', 'j', 'chem,', '6', '22', '(1968)', '29', 'o', 'fuchs', 'angew', 'chem,', 'intern', 'ed,', '7', '(5)', '394', '(1968)', '30', 'm', 'tsuda', 'makromol', 'chem,', '72', '174', '183', '(1964)', '31', 'dm', 'jones', 'adv', 'carbohydrate', 'chem,', '19', '229', '(1964)', '32', 'em', 'fetters', 'crystalline', 'oleﬁn', 'polymers', 'part', 'ii', '(ra', 'raff', 'kw', 'doaks', 'eds)', 'interscience', 'new', 'york', '1964', '33', 's', 'tazuke', 'makromol', 'chem,', 'suppl,', '14', '145', '(1985)', '34', 'mr', 'gomez-anton', 'jg', 'rodriguez', 'if', 'pierola', 'macromolecules', '19', '2932', '(1986)', '35', 'ad', 'roberts', '(ed)', 'natural', 'rubber', 'science', 'technology', 'oxford', 'university', 'press', 'new', 'york', '1988', '36', 'h', 'staudinger', 'h', 'staidinger', 'j', 'prakt', 'chem,', '162', '148', '(1943)', 'rubber', 'chem', 'technology', '17', '15', '(1944)', '37', 'cs', 'ramakrishnan', 'd', 'raghunath', 'jb', 'ponde', 'trans', 'inst', 'rubber', 'ind,', '29', '190', '(1953)', 'ibid,', '30', '129', '(1954)', '38', 'm', 'troussier', 'rubber', 'chem', 'technol,', '29', '302', '(1956)', '39', 'cf', 'broomﬁeld', 'j', 'chem', 'soc,', '1944', '114', 'rubber', 'chem', 'technol,', '17', '759', '(1944)', 'references', '683', '40', 'cs', 'ramakrishnan', 'd', 'raghunath', 'jb', 'ponde', 'trans', 'instit', 'rubber', 'ind,', '30', '129', '(1954)', 'rubber', 'chem', 'technol,', '28', '598', '(1955)', '41', 'iatutorskii', 'lv', 'sokolova', 'ba', 'dogadkin', 'vysokomol', 'soyed,', 'a13', '(4)', '952', '(1971)', '42', 'j', 'royo', 'l', 'gonzalez', 'l', 'ibarra', 'm', 'barbero', 'makromol', 'chem,', '168', '41', '(1973)', '43', 'g', 'dall’asta', 'p', 'meneghini', 'u', 'gennaro', 'makromol', 'chem', '154', '279', '(1972)', '44', 'g', 'dall’asta', 'p', 'meneghini', 'iw', 'bassi', 'u', 'gennaro', 'makromol', 'chem,', '165', '83', '(1973)', '45', 'iw', 'bassi', 'r', 'scordamaglia', 'makromol', 'chem,', '166', '283', '(1973)', '46', 'f', 'stelzer', 'k', 'hummel', 'c', 'graimann', 'j', 'hobisch', 'mg', 'martl', 'makromol', 'chem,', '188', '1795', '(1987)', '47', 'k', 'hummel', 'j', 'mol', 'cat,', '28', '381', '(1987)', 'k', 'hummel', 'pure', 'appl', 'chem,', '54', '351', '(1982)', '48', 'yg', 'garbachev', 'ka', 'garbatova', 'on', 'belyatskaya', 've', 'gul', 'vysokomol', 'soed,', '7', '1645', '(1965)', '49', 'pj', 'canterino', 'ind', 'eng', 'chem,', '49', '712', '(1957)', '50', 'e', 'ceausescu', 'r', 'bordeianu', 'e', 'buzdugan', 'i', 'cerchez', 'p', 'ghioca', 'r', 'stancu', 'j', 'macromol', 'sci', '-', 'chem,', 'a22', '803', '(1985)', '51', 'm', 'bruzzone', 'a', 'carbonaro', 'j', 'polymer', 'sci,', 'chem', 'ed,', '23', '139', '(1985)', '52', 'y', 'doi', 'a', 'yano', 'k', 'soga', 'dr', 'burﬁeld', 'macromolecules', '19', '2409', '(1986)', '53', 'na', 'mohammadi', 'gl', 'rempel', 'macromolecules', '20', '2362', '(1987)', '54', 'h', 'bradbury', 'mc', 'semale', 'perera', 'brit', 'polymer', 'j,', '18', '127', '(1986)', 'a', 'konietzny', 'u', 'bietham', 'angew', 'makromol', 'chem,', '74', '61', '(1978)', '55', 's', 'siddiqui', 're', 'cais', 'macromolecules', '19', '595', '(1986)', '56', 's', 'siddiqui', 're', 'cais', 'macromolecules', '19', '998', '(1986)', '57', 'recais', 's', 'siddiqui', 'macromolecules', '20', '1004', '(1987)', '58', 'h', 'stuger', 'r', 'west', 'macromolecules', '18', '2349', '(1985)', '59', 'hj', 'prins', 'chem', 'weekblad', '17', '932', '(1917)', 'ibid,', '16', '1510', '(1919)', '60', 'f', 'kirchhoff', 'chem', 'ztg,', '47', '513', '(1923)', '61', 'j', 'mcgavack', 'us', 'patent', '1640363', '(aug', '30', '1927)', '62', 'df', 'twiss', 'fa', 'jones', 'brit', 'patent', '348', '303', '(march', '24', '1930)', '63', 'df', 'twiss', 'fa', 'jones', 'brit', 'patent', '523', '734', '(july', '22', '1940)', '64', 'gh', 'latham', 'us', 'patent', '2417424', '(march', '18', '1947)', '65', 's', 'hirano', 'r', 'oda', 'j', 'soc', 'chem', 'ind', '(japan)', '47', '833', '(1944)', 'chem', 'abstr,', '42', '7982', '(1948)', '66', 'df', 'twiss', 'fa', 'jones', 'us', 'patent', '1915808', '(june', '27', '1933)', '67', 'c', 'harris', 'ber,', '37', '2708', '(1904)', '68', 'c', 'pinazzi', 'r', 'pautrat', 'compt', 'rend,', '254', '1997', '(1962)', 'chem', 'abstr,', '56', '15647', '(1962)', '69', 'c', 'pinazzi', 'r', 'pautrat', 'rev', 'gen', 'coutchuc', '39', '799', '(1962)', '70', 'c', 'pinazzi', 'r', 'pautrat', 'r', 'cheritat', 'compt', 'rend,', '256', '2390', '2607', '(1963)', '71', 'sk', 'lee', 'am', 'chem', 'soc', 'polymer', 'preprints', '8', '700', '(1967)', '72', 'y', 'minoura', 'h', 'ikeda', 'j', 'appl', 'polymer', 'sci,', '15', '2219', '(1971)', '73', 'g', 'caraculacu', 'i', 'zugravescu', 'j', 'polymer', 'sci,', 'polymer', 'letters', '6', '451', '(1968)', '74', 'cg', 'gubelein', 'j', 'macromol', 'sci', '-', 'chem,', 'a5', '433', '(1971)', '75', 'k', 'toda', 'y', 'numata', 't', 'katsumura', 'j', 'appl', 'polymer', 'sci,', '15', '2219', '(1971)', '76', 'kw', 'leong', 'gb', 'butler', 'j', 'macromol', 'sci', '-', 'chem,', 'a14287', '(1980)', 'csl', 'baker', 'd', 'bernard', 'am', 'chem', 'soc,', 'polymer', 'preprints', '26', '(2)', '29', '(1985)', '77', 'da', 'bansleben', 'f', 'jachimowicz', 'am', 'chem', 'soc,', 'polymer', 'preprints', '26', '(1)', '106', '(1985)', '78', 'm', 'anavi', 'a', 'zilkha', 'eu', 'polymer', 'j,', '5', '21', '(1969)', '79', 'jw', 'mench', 'b', 'fulkerson', 'ind', 'end', 'chem,', '7', '2', '(1968)', '80', 'jp', 'kennedy', 'ra', 'smith', 'j', 'polymer', 'sci,', 'chem', 'ed,', '18', '1523', '(1980)', '81', 'b', 'ivan', 'jp', 'kennedy', 'vsc', 'chang', 'j', 'polymer', 'sci,', 'chem', 'ed.', '18', '3177', '(1980)', '82', 'jp', 'kennedy', 'vsc', 'chang', 'wp', 'francik', 'j', 'polymer', 'sci,', 'chem', 'ed,', '20', '2809', '(1982)', '83', 'wa', 'thaler', 'sj', 'brois', 'fw', 'ferrara', 'macromolecules', '20', '254', '(1987)', '84', 'e', 'ceausescu', 's', 'bittman', 'v', 'fieroiu', 'e', 'badea', 'e', 'gruber', 'a', 'ciupitoiu', 'v', 'apostol', 'j', 'macroml', 'sci-chem,', 'a22', '(5-7)', '525', '(1985)', '85', 't', 'nishikubo', 't', 'shimokawa', 'h', 'arita', 'makromol', 'chem,', '188', '2105', '(1987)', '86', 't', 'katayama', 'hk', 'hill', 'jr,', 'macromolecules', '20', '1451', '(1987)', '87', 'hl', 'cohen', 'j', 'polymer', 'sci,', 'chem', 'ed,', '22', '2293', '(1984)', '88', 'gj', 'carlson', 'jr', 'skinner', 'cw', 'smith', 'ch', 'wilcoxen', 'jr,us', 'patent', '2870171', '(may', '6', '1958)', '89', 'cw', 'smith', 'gp', 'payne', 'us', 'patent', '2786854', '(march', '26', '1957)', '90', 'cm', 'gable', 'us', 'patent', '2870171', '(jan', '20', '1959)', '91', 'wc', 'smith', 'rec', 'trav', 'chim,', '29', '686', '(1930)', 'chem', 'abstr,', '24', '4261', '(1930)', '92', 'fp', 'greenspan', 're', 'light', 'jr,', 'us', 'patent', '2829130', '(april', '1958)', '93', 's', 'tocker', 'us', 'patent', '3043818', '(july', '1962)', '684', '9', 'organic', 'reactions', 'polymers', '94', 'rw', 'rees', 'us', 'patent', '3050507', '(august', '1962)', '95', 'h', 'lee', 'k', 'neville', 'handbook', 'epoxy', 'resins', 'mc', 'grow-hill', 'new', 'york', '1967', '96', 'wg', 'potter', 'epoxy', 'resins', 'springer-verlag', 'new', 'york', '1970', '97', 'ca', 'may', '(ed)', 'epoxy', 'resins', 'chemistry', 'technology', '2nd', 'ed,', 'dekker', 'new', 'york', '1988', '98', 'af', 'chadwick', 'do', 'barlow', 'aa', 'd’adieco', 'jw', 'wallace', 'j', 'am', 'oil', 'chemists', 'soc,', '35', '355', '(1958)', '99', 'k', 'meyersen', 'jyc', 'wang', 'j', 'polymer', 'sci,', 'a5', '725', '(1967)', '100', 'hs', 'makowski', 'm', 'lynn', 'dh', 'rotenberg', 'j', 'macromol', 'sci-chem,', 'a4', '1563', '(1980)', '101', 'd', 'zuchowska', 'polymer', '21', '514', '(1980)', '102', 'ma', 'golub', 'j', 'am', 'chem', 'soc,', '80', '1794', '(1958)', '103', 'ma', 'golub', 'j', 'am', 'chem', 'soc,', '81', '54', '(1959)', '104', 'ma', 'golub', 'j', 'am', 'chem', 'soc,', '82', '5093', '(1960)', '105', 'ma', 'golub', 'j', 'phys', 'chem,', '66', '1202', '(1962)', '106', 'ma', 'golub', 'cl', 'stephens', 'j', 'polymer', 'sci,', 'c-6', '(16)', '765', '(1967)', '107', 'rw', 'lenz', 'k', 'ohata', 'j', 'funt', 'j', 'polymer', 'sci-chem', 'ed,', '11', '2273', '(1973)', '108', 'ma', 'golub', 'j', 'hiller', 'can', 'j', 'chem,', '41', '937', '(1963)', '109', 'ma', 'golub', 'j', 'hiller', 'tetrahedron', 'letters', '2137', '(1963)', '110', 'rj', 'angelo', 'ml', 'wallach', 'rm', 'ikeda', 'am', 'chem', 'soc', 'polymer', 'preprints', '8', '(1)', '221', '(1967)', '111', 'ma', 'golub', 'j', 'heller', 'j', 'polymer', 'sci,', 'polymer', 'letters', '4', '469', '(1966)', '112', 'ng', 'gaylord', 'i', 'kossler', 'm', 'stolka', 'j', 'vodehnal', 'j', 'am', 'chem', 'soc,', '85', '641', '(1963)', '113', 'ng', 'gaylord', 'i', 'kossler', 'm', 'stolka', 'j', 'vodehnal', 'j', 'polymer', 'sci,', 'a-12', '3969', '(1964)', '114', 'm', 'stolka', 'j', 'vodehnal', 'i', 'kossler', 'j', 'polymer', 'sci,', 'a-12', '3987', '(1964)', '115', 'i', 'kossler', 'j', 'vodehnal', 'm', 'stolka', 'j', 'polymer', 'sci,', 'a-13', '2081', '(1965)', '116', 'p', 'ehrburger', 'jb', 'donnet', 'high', 'technology', 'rubbers', 'part', 'a', '(m', 'lewin', 'j', 'preston', 'eds)', 'dekker', 'new', 'york', '1985', 'g', 'henrici-olive’', 's', 'olive’', 'adv', 'polym', 'sci,', '51', '1', '(1983)', 'e', 'fitzer', 'angew', 'chem,', 'intern', 'ed,', '19', '375', '(1980)', '117', 'em', 'bevilacqua', 'rubber', 'chem', 'technol,', '30', '667', '(1957)', '118', 'em', 'bevilacqua', 'j', 'am', 'chem', 'soc,', '79', '2915', '(1957)', '119', 'em', 'bevilacqua', 'science', '126', '396', '(1957)', '120', 'em', 'bevilacqua', 'j', 'am', 'chem', 'soc,', '81', '5071', '(1959)', '121', 'g', 'salmon', 'chr', 'van', 'der', 'schee', 'j', 'polymer', 'sci,', '14', '287', '(1954)', '122', 'ma', 'golub', 'cl', 'stephens', 'j', 'polymer', 'sci,', 'a-16', '763', '(1968)', '123', 'ma', 'golub', 'j', 'polymer', 'sci,', 'polymer', 'letters', '12', '615', '(1974)', 'ibid,', '12', '295', '(1974)', '124', 'ma', 'golub', 'jl', 'rosenberg', 'j', 'polymer', 'sci,', 'chem', 'ed,', '18', '2548', '(1980)', '125', 'ma', 'golub', 'j', 'polymer', 'sci,', 'chem', 'ed,', '19', '1073', '(1981)', '126', 'rw', 'lenz', 'k', 'ohata', 'j', 'funt', 'j', 'polymer', 'sci,', 'chem', 'ed11,', '2273', '(1973)', '127', 'sb', 'maerov', 'j', 'polymer', 'sci,', 'a-13', '487', '(1963)', '128', 'f', 'millich', 'r', 'sinclair', 'chem', 'eng', 'news', 'p30', '(june', '26', '1967)', '129', 'rh', 'michel', 'wa', 'murphy', 'j', 'polymer', 'sci,', '55', '741', '(1961)', '130', 's', 'van', 'paeschen', 'makromol', 'chem,', '63', '123', '(1963)', '131', 'ja', 'banchette', 'jd', 'cotman', 'jr,', 'j', 'org', 'chem,', '23', '117', '(1958)', '132', 'h', 'morawetz', 'b', 'vogel', 'j', 'am', 'chem', 'soc,', '91', '563', '(1969)', '133', 'h', 'morawetz', 'g', 'gordimer', 'j', 'am', 'chem', 'soc,', '92', '7532', '(1970)', '134', 'ga', 'gordon', 're', 'cohen', 'rfbaddour', 'macromolecules', '18', '98', '(1985)', '135', 'aj', 'rudge', 'brit', 'patent', '710523', '(june', '16', '1954)', '136', 'i', 'vogt', 'w', 'krings', 'german', 'patent', '1086891', '(aug', '11', '1960)', '137', 'm', 'okada', 'k', 'makuuchi', 'kagyo', 'kagaku', 'zashi', '73', '1211', '(1970)', '(from', 'private', 'english', 'translation)', '138', 'h', 'schonhorn', 'pk', 'gallagher', 'jp', 'luongo', 'fj', 'padden', 'jr,', 'macromolecules', '3', '800', '(1970)', '139', 'wt', 'miller', 'al', 'dittman', 'j', 'am', 'chem', 'soc,', '78', '2793', '(1956)', '140', 'jr', 'myles', 'pj', 'garner', 'us', 'patent', '2422919', '(june', '24', '1947)', '141', 'wn', 'baxter', 'us', 'patent', '2849431', '(aug', '26', '1958)', '142', 'lb', 'krentsel', 'ad', 'litmanovich', 'iv', 'patsukhova', 'va', 'agasandyan', 'vysokomol', 'soyed,', 'a13', '2489', '(1971)', '143', 'm', 'goren', 'polim', 'vehomerim', 'plast,', '1', '8', '(1971)', '144', 'rs', 'taylor', 'us', 'patent', '2592763', '(april', '20', '1952)', '145', 'hw', 'thompson', 'p', 'torkington', 'trns', 'faraday', 'soc,', '41', '254', '(1945)', '146', 'k', 'nambu', 'j', 'appl', 'polymer', 'sci,', '4', '69', '(1960)', '147', 't', 'nakagawa', 's', 'yamada', 'j', 'appl', 'polymer', 'sci,', '16', '1997', '(1972)', '148', 'gd', 'jones', 'chemical', 'reactions', 'polymers', '(em', 'fetters', 'ed)wiley-', 'interscience', 'new', 'york', '1964', '149', 'brit', 'patent', '811848', '(april', '15', '1959)', 'chem', 'abstr,', '53', '23100f', '(1951)', '150', 'h', 'noeske', 'o', 'roeleau', 'us', 'patent', '2', '889259', '(june', '2', '1959)', 'references', '685', '151', 'a', 'mcalevy', 'us', 'patent', '2405971', '(august', '20', '1946)', '152', 're', 'brooks', 'ed', 'straom', 'a', 'mcalevy', 'rubber', 'world', '127', '791', '(1953)', '153', 'g', 'natta', 'g', 'mazzanti', 'm', 'buzzoni', 'ital', 'patent', '563508', '(april', '24', '1956)', '154', 'k', 'konishi', 'k', 'yamaguchi', 'm', 'takahisha', 'j', 'appl', 'polymer', 'sci,', '15', '257', '(1971)', '155', 'p', 'bertican', 'jj', 'bejat', 'g', 'vallet', 'j', 'chim', 'phys', 'physicochim', 'biol,', '67', '164', '(1979)', '156', 'rt', 'sikorski', 'e', 'czerwinska', 'polymer', '25', '1371', '(1984)', '157', 'rt', 'sikorski', 'e', 'czerwinska', 'eu', 'polymer', 'j,', '22', '179', '(1986)', '158', 'c', 'decker', 'j', 'polymer', 'sci,', 'polymer', 'letters', '25', '5', '(1987)', '159', 'g', 'svegliado', 'fz', 'grandi', 'j', 'appl', 'polymer', 'sci,', '13', '1113', '(1969)', '160', 'm', 'kolinsky', 'd', 'doskocilova', 'b', 'schneider', 'j', 'stokr', 'j', 'polymer', 'sci,', 'a-19', '791', '(1971)', '161', 'backskai', 'lp', 'lindeman', 'jw', 'adams', 'j', 'polymer', 'sci,', 'a-19', '991', '(1971)', '162', 'gg', 'scherer', 'p', 'pﬂuger', 'h', 'braun', 'j', 'klein', 'h', 'weddecke', 'makromol', 'chem,', 'rapid', 'commun,', '5', '611', '(1984)', '163', 'm', 'takeishi', 'm', 'okawara', 'j', 'polymer', 'sci,', 'polymer', 'letters', '7', '201', '(1969)', '164', 'm', 'takeishi', 'm', 'okawara', 'j', 'polymer', 'sci,', 'polymer', 'letters', '8', '829', '(1970)', '165', 'm', 'okawara', 't', 'endo', 'y', 'kurusu', 'progress', 'polymer', 'sci,', 'japan', '(k', 'imahori', 'ed)', 'vol', '4', 'p', '105', '1972', '166', 'k', 'mori', 'y', 'nakamura', 'j', 'polymer', 'sci,', 'polymer', 'letters', '9', '547', '(1971)', '167', 'g', 'levin', 'makromol', 'chem,', 'rapid', 'commun,', '5', '513', '(1984)', '168', 'mk', 'naqvi', 'p', 'josph', 'polymer', 'communication', '27', '8', '(1986)', '169', 'z', 'wolkober', 'i', 'varga', 'j', 'polymer', 'sci,', 'c', '(16)', '3059', '(1967)', '170', 'h', 'calvayrac', 'j', 'gole', 'bull', 'soc', 'chim', 'france', '1986', '1076', '171', 'e', 'roman', 'g', 'valenzuela', 'l', 'gargallo', 'd', 'radic', 'j', 'polymer', 'sci,', '21', '2057', '(1983)', '172', 'd', 'braun', 'e', 'seeling', 'makromol', 'chem,', '86', '124', '(1965)', '173', 'w', 'hahn', 'w', 'muller', 'makroml', 'chem,', '16', '71', '(1955)', '174', 'm', 'okawara', 'am', 'chemsoc', 'coatings', 'plastics', 'chemistry', 'preprints', '40', '39', '(1979)', '175', 'y', 'nakamura', 'k', 'mori', 'm', 'kaneda', 'nippon', 'kagaku', 'kaishi', '1620', '(1976)', 'ref', '164', '176', 'rk', 'jenkings', 'nr', 'byrd', 'jl', 'lister', 'j', 'appl', 'polymer', 'sci,', '12', '2059', '(1968)', '177', 'ra', 'haldon', 'jn', 'hay', 'j', 'polymer', 'sci,', 'a-15', '2295', '(1967)', '178', 'jc', 'bevington', 'l', 'rotti', 'eu', 'polym', 'j,', '8', '1105', '(1972)', '179', 'r', 'tarascon', 'm', 'hartney', 'mj', 'bowden', 'am', 'chem', 'soc', 'polymer', 'preprints', '25', '(1)', '289', '(1984)', '180', 't', 'saegusa', 'r', 'oda', 'bull', 'inst', 'chem', 'research', 'kyoto', 'univ33,', '126', '(1955)', 'chem', 'abstr,', '50', '181', 'f', 'helfferich', 'ion', 'exchange', 'mcgraw-hill', 'ny,', '1962', 'ga', 'olah', 'ws', 'tolgyesi', 'friedel-craft', 'related', 'reactions', '(ga', 'olah', 'ed)', 'wiley-interscience', 'new', 'york', '1964', 'we', 'wright', 'eg', 'topikar', 'sa', 'svejda', 'macromolecules', '24', '5879', '(1991)', '182', 'g', 'pozniak', 'w', 'trochimszuk', 'j', 'appl', 'polymer', 'sci,', '27', '1833', '(1982)', '183', 'ch', 'bamford', 'h', 'linsay', 'polymer', '14', '330', '(1973)', '184', 'r', 'hauptman', 'f', 'wolf', 'd', 'warnecke', 'plaste', 'kaut,', '18', '330', '(1971)', '185', 'ee', 'ergojin', 'sp', 'raﬁkov', 'ba', 'muhidtina', 'izv', 'akad', 'nauk', 'kaz', 'ssr,', '19', '(16)', '49', '(1969)', '186', 'gz', 'esipov', 'la', 'derevyanki', 'vp', 'markidin', 'rg', 'rakhuba', 'kl', 'poplavskii', 'vysokomol', 'soyed,', 'b12', '1357', '(1956)', '274', '(1970)', '187', 's', 'belfer', 'r', 'glozman', 'a', 'deshe', 'a', 'warshawsky', 'j', 'appl', 'polymer', 'sci,', '25', '2241', '(1980)', '188', 'sr', 'raﬁkov', 'gn', 'chelnokova', 'gm', 'dzhilkibayeva', 'vysokomol', 'soyed,', 'b10', '329', '(1968)', '189', 'l', 'galeazzi', 'german', 'pat', '2455946', '(june', '1975)', '190', 'wh', 'daly', 'j', 'macromol', 'sci,', 'chem,', 'a22', '713', '(1985)', '191', 's', 'mohanraj', 'wt', 'ford', 'macromolecules', '19', '2470', '(1986)', '192', 'sr', 'raﬁkov', 'gm', 'dzhilkibayeva', 'gn', 'chelkova', 'gb', 'shaltuper', 'vysokomol', 'soyed,', 'a12', '(7)', '1608', '(1970)', '193', 'p', 'hodge', 'j', 'waterhouse', 'polymer', '22', '1153', '(1981)', '194', 'm', 'hassanein', 'a', 'akelah', 'am', 'chem', 'soc', 'polymer', 'preprints', '26', '(1)', '88', '(1985)', '195', 'td', 'n’guyen', 'jc', 'gautier', 's', 'boileau', 'am', 'chem', 'soc', 'polymer', 'preprints', '23', '143', '(1982)', '196', 'p', 'hodge', 'bj', 'hunt', 'e', 'khoshdel', 'j', 'waterhouse', 'am', 'chem', 'soc', 'polymer', 'preprints', '23', '147', '(1982)', '197', 'p', 'hodge', 'bj', 'hunt', 'j', 'waterhouse', 'a', 'wightman', 'polymer', 'communications', '24', '70', '(1983)', '198', 'm', 'takeishi', 'n', 'umeta', 'makromol', 'chem,', 'rapid', 'commun,', '3', '875', '(1982)', '199', 'bn', 'kolarz', 'a', 'rapak', 'makromol', 'chem,', '185', '2511', '(1984)', '200', 't', 'iizawa', 't', 'nishikubo', 'y', 'masuda', 'm', 'okawara', 'macromolecules', '17', '992', '(1984)', '201', 'gd', 'jones', 'sj', 'goetz', 'j', 'polymer', 'sci,', '25', '201', '(1957)', '202', 'gd', 'jones', 'ind', 'eng', 'chem,', '44', '2686', '(1952)', '203', 'f', 'marinola', 'g', 'naumann', 'angew', 'makromolekulare', 'chem,', '45185', '(1968)', '204', 's', 'dragan', 'i', 'petrariu', 'm', 'dima', 'j', 'polymer', 'sci,', 'chem', 'ed,', '10', '3077', '(1986)', '205', 's', 'dragan', 'v', 'barboiu', 'd', 'csergo', 'i', 'petrariu', 'm', 'dima', 'makromol', 'chem,', '187', '(1986)', '686', '9', 'organic', 'reactions', 'polymers', '206', 're', 'kesting', 'synthetic', 'polymeric', 'membranes', 'mcgraw-hill', 'new', 'york', '1971', '207', 'wchen', 'r', 'mesrobian', 'd', 'ballantine', 'j', 'polymer', 'sci,', '23', '903', '(1957)', '208', 'gv', 'kharitonov', 'lp', 'belikova', 'vysokomol', 'soyed,', 'a11', '(11)', '2424', '(1969)', '209', 'wo', 'kenyon', 'gp', 'waugh', 'j', 'polymer', 'sci,', '32', '83', '(1958)', 'cc', 'unruh', 'j', 'appl', 'polymer', 'sci,', '2', '358', '(1959)', '210', 'wr', 'bussing', 'na', 'peppas', 'polymer', '24', '209', '(1983)', '211', 'dg', 'barar', 'kp', 'staller', 'na', 'peppas', 'j', 'polymer', 'sci,', 'chem', 'ed,', '21', '1013', '(1983)', '212', 'r', 'singer', 'a', 'demagistri', 'c', 'miller', 'makromol', 'chem,', '1819', '139', '(1956)', '213', 'r', 'singer', 'a', 'demagistri', 'j', 'chim', 'phys,', '47', '704', '(1950)', '214', 'm', 'baer', 'us', 'patent', '2533211', '(dec', '12', '1950)', '215', 'r', 'singer', 'us', 'patent', '2604', '456', '(july', '22', '1952)', '216', 'b', 'blaser', 'm', 'rugenstein', 't', 'tischbirek', 'us', 'patent', '2764576', '(sept', '251956)', '217', 'hh', 'roth', 'ind', 'eng', 'chem,', '46', '2435', '(1954)', '218', 'j', 'eichhorn', 'us', 'patent', '2877213', '(mar', '10', '1959)', '219', 'cfh', 'aleen', 'lm', 'minsk', 'us', 'patent', '2735841', '(feb', '21', '1956)', '220', 'j', 'blyth', 'aw', 'hofman', 'ann', 'chem,', '53', '316', '(1984)', '221', 'h', 'zenftman', 'j', 'chem', 'soc,', '1950', '820', '222', 'as', 'matlack', 'ds', 'breslow', 'j', 'polymer', 'sci,', '45', '265', '(1960)', '223', 'ja', 'blanchette', 'jd', 'cotman', 'jr,', 'j', 'org', 'chem,', '23', '117', '(1958)', '224', 'f', 'cramer', 'h', 'helbig', 'h', 'hettler', 'kh', 'scheit', 'h', 'seliger', 'angew', 'chem,', '78', '64', '(1966)', 'ibid,', 'intern', 'edit,', '5', '601', '(1966)', '225', 'b', 'philipp', 'g', 'reinisch', 'g', 'raﬂer', 'plaste', 'und', 'kautschuk', '3', '190', '(1972)', '226', 'wkern', 'rc', 'schulz', 'd', 'braun', 'chemiker', 'ztg/chem', 'apparatur', '84', '(12)', '385', '(1960)', '227', 'g', 'greber', 'j', 'toelle', 'makromol', 'chem,', '52', '208', '(1962)', '228', 'rp', 'quirk', 'pl', 'chemg', 'macromolecules', '19', '1291', '(1986)', '229', 'd', 'braun', 'makromol', 'chem,', '30', '85', '(1959)', '230', 'd', 'braun', 'makromol', 'chem,', '33', '181', '(1959)', '231', 'd', 'braun', 'h', 'daimond', 'g', 'becker', 'makromol', 'chem,', '62', '183', '(1963)', '232', 'd', 'braun', 'chimia', '14', '24', '(1960)', '233', 'y', 'yamada', 'm', 'okawara', 'makromol', 'chem,', '152', '153', '(1972)', '234', 'y', 'yamada', 'm', 'okawara', 'makromol', 'chem,', '152', '163', '(1972)', '235', 'k', 'chander', 'rc', 'anand', 'ik', 'varma', 'j', 'macromol', 'sci', '-', 'chem,', 'a20', '697', '(1983)', '236', 'j', 'petit', 'b', 'houel', 'comp', 'rend,', '246', '1427', '(1958)', '237', 'b', 'houel', 'comp', 'rend,', '246', '2488', '(1958)', '238', 'rc', 'schulz', 'p', 'elzer', 'w', 'kern', 'chimia', '13', '237', '(1959)', '239', 'cs', 'marvel', 'jh', 'shpherd', 'j', 'org', 'chem,', '24', '599', '(1961)', '240', 'hl', 'cohen', 'dg', 'borden', 'lm', 'minst', 'j', 'org', 'chem,', '26', '1274', '(1959)', '241', 'w', 'frey', 'e', 'klesper', 'eur', 'polym', 'j,', '22', '735', '(1986)', '242', 'hl', 'cohen', 'lm', 'minsk', 'j', 'org', 'chem,', '24', '1404', '(1959)', '243', 'ja', 'banchette', 'jd', 'cotman', 'j', 'org', 'chem,', '23', '1117', '(1958)', '244', 'w', 'kern', 'rc', 'schulz', 'angew', 'chem,', '69', '153', '(1957)', '245', 'r', 'vartan-boghossian', 'b', 'dederichs', 'e', 'klesper', 'eur', 'polym', 'j,', '22', '23', '(1986)', '246', 'r', 'roussel', 'mo', 'de', 'guerrero', 'jc', 'gallin', 'macromolecules', '19', '291', '(1986)', '247', 's', 'rondou', 'g', 'smets', 'mc', 'de', 'wilde-delvaux', 'j', 'polymer', 'sci,', '24', '261', '(1957)', '248', 'm', 'vrancken', 'g', 'smets', 'j', 'polymer', 'sci,', '14', '521', '(1954)', '249', 'm', 'mullier', 'g', 'smets', 'j', 'polymer', 'sci,', '23', '915', '(1957)', '250', 'a', 'ravve', 'j', 'polymer', 'sci,', 'a-16', '2889', '(1968)', '251', 'g', 'van', 'paeschen', 'makromol', 'chem,', '63', '123', '(1963)', '252', 'h', 'boudevska', 'izv', 'bulgar', 'acad', 'sci,', '3', '303', '(1970)', '253', 'ce', 'carraher', 'jr,', 'ls', 'wang', 'makromol', 'chem,', '152', '43', '(1972)', '254', 'v', 'janout', 'p', 'cafelin', 'makromol', 'chem,', '182', '2989', '(1981)', '255', 'ns', 'bondareva', 'en', 'rastovskii', 'zh', 'prikl', 'khim', '(leningrad)', '43', '(1)', '215', '(1970)', '256', 'iv', 'andreyeva', 'mm', 'kotin', 'lya', 'modreskay', 'yei', 'pokrovskii', 'gv', 'lyubimova', 'vysokomol', 'soyed,', 'a14', '(7)', '1565', '(1972)', '257', 'k', 'ogasawara', 'n', 'yamagami', 's', 'matuzawa', 'angew', 'makroml', 'chem,', '25', '15', '(1972)', '258', 'k', 'imai', 't', 'shiomi', 't', 'tsuchida', 'c', 'watanabe', 's', 'nishioka', 'j', 'polymer', 'sci,', 'a-121', '305', '(1983)', '259', 'k', 'imai', 't', 'shiomi', 'y', 'tezuka', 'm', 'miya', 'j', 'polymer', 'sci,', 'chem', 'ed,', '22', '841', '(1984)', '260', 'k', 'ichimura', 'j', 'polymer', 'sci,', 'chem', 'ed,', 'a-120', '1411', '(1982)', '261', 'k', 'ichimura', 's', 'watanabe', 'j', 'polymer', 'sci,', 'chem', 'ed,', 'a-120', '1419', '(1982)', '262', 'm', 'tsuda', 'h', 'tanaka', 'h', 'tagami', 'f', 'hori', 'makromol', 'chem,', '167', '183', '(1973)', 'references', '687', '263', 'p', 'gramain', 'j', 'le', 'moigue', 'eur', 'polymer', 'j,', '8', '703', '(1972)', '264', 'm', 'anavi', 'a', 'zilkha', 'eur', 'polymer', 'j,', '5', '21', '(1969)', '265', 'l', 'merle-aubry', 'y', 'merle', 'e', 'selegny', 'makromol', 'chem,', '172', '115', '(1973)', '266', 'lx', 'mallavarapu', 'a', 'ravve', 'j', 'polymer', 'sci,', 'a3', '593', '(1965)', '267', 'g', 'reinisch', 'dietrich', 'eur', 'polymer', 'j,', '6', '1269', '(1970)', '268', 'ej', 'gunster', 'rs', 'schulz', 'makromol', 'chem,', '180', '1891', '(1979)', '269', 'j', 'klein', 'e', 'klesper', 'makromol', 'chem,', 'rapid', 'commun,', '5', '701', '(1984)', '270', 'hr', 'kricheldorf', 'p', 'jahnke', 'n', 'scharnag', 'macromolecules', '25', '1382', '(1992)', '271', 'hsj', 'lee', 'wp', 'weber', 'macromolecules', '24', '4749', '(1991)', '272', 'hl', 'fisher', 'chemistry', 'natural', 'synthetic', 'rubber', 'reinhold', 'new', 'york', '1957', '273', 'pw', 'allen', 'dbarnard', 'b', 'saville', 'chem', 'brit,', '6', '382', '(1970)', '274', 'cg', 'moore', 'j', 'scanlan', 'j', 'polymer', 'sci,', '43', '23', '(1960)', '275', 'ng', 'gaylord', 'fs', 'ang', 'chemical', 'reactions', 'polymers', '(em', 'fetters', 'ed)', 'wiley-interscience', 'new', 'york', '1964', '276', 'sl', 'regen', 'j', 'org', 'chem,', '1977', '42', '875', '277', 'h', 'komeili-zadeh', 'hj-m', 'don', 'j', 'metzger', 'j', 'organ', 'chem,', '43', '156', '(1978)', '278', 's', 'colonna', 'r', 'fornasier', 'u', 'pfeiffer', 'j', 'chem', 'soc,', 'perkin', 'trans,', '1', '8', '(1978)', '279', 'e', 'chiellini', 'r', 'solaro', 'j', 'chem', 'soc,', 'chem', 'commun,', '1977', '231', '280', 'a', 'mastagli', 'a', 'froch', 'g', 'durr', 'cr', 'acad', 'sci,', 'paris', '235', '1402', '(1952)', '281', 'ec', 'blossey', 'dc', 'neckers', 'c', 'douglas', 'al', 'thayer', 'ap', 'schaap', 'j', 'am', 'chem', 'soc,', '95', '582', '(1973)', '282', 'haj', 'battaerd', 'gw', 'tregear', 'graft', 'copolymers', 'wiley-interscience', 'new', 'york', '1967', '283', 'p', 'dreyfus', 'rp', 'quirk', '“graft', 'copolymers”', 'encyclopedia', 'polymer', 'science', 'engineering', 'vol', '7', '2nd', 'ed,', '(hf', 'mark', 'nm', 'bikales', 'cg', 'overberger', 'g', 'menges', 'eds)', 'wiley-interscience', 'new', 'york', '1986', '284', 's', 'basu', 'jn', 'sen', 'sr', 'palit', 'proc', 'roy', 'soc', '(london)', 'a202', '485', '(1950)', '285', 'ra', 'gregg', 'fr', 'mayo', 'j', 'am', 'chem', 'soc,', '75', '3530', '(1953)', '286', 'd', 'lim', 'o', 'wichterle', 'j', 'polymer', 'sci,', '5', '606', '(1961)', '287', 'm', 'morton', 'i', 'purma', 'j', 'am', 'chem', 'soc,', '80', '5596', '(1958)', '288', 's', 'okamura', 'k', 'katagiri', 'makromol', 'chem,', '28', '177', '(1958)', 'zm', 'wang', 'h', 'hong', 'tc', 'chung', 'macromolecules', '2005', '38', '8966', '289', 'gv', 'schulz', 'g', 'henrici', 's', 'olive’', 'j', 'polymer', 'sci,', '17', '45', '(1955)', '290', 'g', 'smets', 'j', 'roovers', 'w', 'van', 'humbeek', 'j', 'appl', 'polymer', 'sci,', '5', '149', '(1961)', '291', 'g', 'smets', 'm', 'claesen', 'j', 'polymer', 'sci,', '8', '289', '(1952)', '292', 'ra', 'hays', 'j', 'polymer', 'sci,', '11', '531', '(1953)', '293', 'ra', 'gregg', 'fr', 'mayo', 'j', 'am', 'chem', 'soc,', '75', '3530', '(1953)', '294', 'a', 'gsperowicz', 'm', 'kolendowicz', 't', 'skowronski', 'polymer', '23', '839', '(1982)', '295', 'ms', 'gluckman', 'mj', 'kampf', 'jl', 'o’brien', 'tg', 'fox', 'rk', 'graham', 'j', 'polymer', 'sci,', '37', '411', '(1959)', '296', 'tg', 'fox', 'ms', 'guckman', 'fb', 'gornick', 'rk', 'graham', 'g', 'gratch', 'j', 'polymer', 'sci,', '37', '397', '(1959)', '297', 'dk', 'ray', 'chandhuri', 'jj', 'hermans', 'j', 'polymer', 'sci,', '48', '159', '(1960)', '298', 's', 'nakamura', 'e', 'yoshikawa', 'k', 'matsuzaki', 'j', 'appl', 'polymer', 'sci,', '25', '1833', '(1988)', '299', 's', 'nakamura', 'h', 'sato', 'k', 'matsuzaki', 'j', 'polymer', 'sci,', 'polymer', 'letters', '11', '221', '(1973)', '300', 'pw', 'allen', 'fm', 'merrett', 'j', 'polymer', 'sci,', '22', '193', '(1956)', '301', 'pw', 'allen', 'g', 'ayrey', 'cg', 'moore', 'j', 'scanlan', 'j', 'polymer', 'sci,', '36', '55', '(1959)', '302', 'rj', 'ceresa', '(ed)', 'block', 'graft', 'copolymers', 'wiley', 'interscience', 'new', 'york', '1973', '303', 'bd', 'gesner', 'rubber', 'chem', 'technol,', '38', '655', '(1965)', '304', 'pg', 'ghosh', 'pk', 'sengupta', 'j', 'appl', 'polymer', 'sci,', '11', '1603', '(1967)', '305', 're', 'wetton', 'jd', 'moore', 'be', 'fox', 'makromol', 'chem132,', '135', '(1970)', '306', 'lv', 'valentine', 'cb', 'chapman', 'ric', 'sci', 'suppl,', '25', '278', '(1955)', '307', 'gn', 'richards', 'j', 'appl', 'polymer', 'sci,', '5', '(17)', '553', '(1961)', '308', 'gi', 'simionescu', 's', 'dumitriu', 'j', 'polymer', 'sci,', 'c', '(37)', '187', '(1972)', '309', 'g', 'mino', 's', 'kaizerman', 'j', 'polymer', 'sci,', '31', '242', '(1958)', '310', 'rm', 'livshits', 'vp', 'alachev', 'mv', 'prokofeva', 'za', 'rogovin', 'vysokomol', 'soyed,', '6', '(4)', '655', '(1964)', '311', 'y', 'iwakura', 't', 'kurosaki', 'y', 'imai', 'j', 'polymer', 'sci,', 'a-1', '3', '1185', '(1965)', 'yiwakura', 'y', 'imai', 'k', 'yagi', 'j', 'polymer', 'sci,', 'a-1', '6', '801', '(1968)', 'ibid,', 'a-1', '6', '1625', '(1968)', 'y', 'imai', 'e', 'masuhara', 'y', 'iwakura', 'j', 'polymer', 'sci,', 'polymer', 'letters', '8', '75', '(1970)', '312', 'y', 'ogiwara', 'y', 'ogiwara', 'h', 'kubota', 'j', 'polymer', 'sci,', 'a-16', '1489', '(1968)', '313', 'aa', 'gulina', 'rm', 'livshits', 'za', 'rogovin', 'vysokomol', 'soyed,', '7', '(9)', '1693', '(1965)', '314', 's', 'lenka', 'pl', 'nayak', 'j', 'appl', 'polymer', 'sci,', '27', '1959', '(1962)', '315', 'a', 'hebeish', 's', 'shalaby', 'a', 'waly', 'a', 'bayazeed', 'j', 'polymer', 'sci,', '28', '303', '(1983)', '316', 'je', 'guillet', 'rgw', 'norrish', 'nature', '173', '625', '(1954)', 'proc', 'roy', 'soc', '(london)', 'a', '233', '172', '(1956)', '688', '9', 'organic', 'reactions', 'polymers', '317', 'gc', 'menon', 'sl', 'kapur', 'j', 'appl', 'polymer', 'sci,', '3', '54', '(1960)', '318', 'n', 'geacintov', 'v', 'stannett', 'ew', 'abrhamson', 'makromol', 'chem,', '36', '52', '(1959)', 'a', 'chapiro', 'j', 'polymer', 'sci,', '29', '321', '(1958)', 'ibid,', '34', '439', '(1959)', '319', 'g', 'smets', 'w', 'de', 'winter', 'g', 'delzenne', 'j', 'polymer', 'sci,', '55', '767', '(1961)', '320', 'a', 'chapiro', 'j', 'polymer', 'sci,', '48', '109', '(1960)', '321', 'y', 'hachihama', 's', 'takamura', 'technol', 'report', 'osaka', 'univ,', '11', '(485)', '431', '(1961)', 'chem', 'abstr,', '57', '6179', '(1962)', '322', 'aj', 'restaino', 'wn', 'reed', 'j', 'polymer', 'sci,', '36', '499', '(1959)', '323', 'a', 'noshay', 'je', 'mcgrath', 'block', 'copolymers', 'academic', 'press', 'new', 'york', '1977', '324', 'g', 'gruber', 'g', 'egle', 'makromol', 'chem,', '53', '206', '(1962)', '325', 'g', 'sasson', 'a', 'zilkha', 'eur', 'polymer', 'j,', '5', '315', '(1969)', '326', 'r', 'waak', 'ma', 'doran', 'polymer', '2', '365', '(1961)', '327', 'r', 'milkovich', 'am', 'chem', 'soc', 'polymer', 'preprints', '21', '40', '(march', '1980)', '328', 'r', 'milkovich', 'mt', 'chiang', 'us', 'patent', '3786116', '(1974)', '329', 'm', 'w', 'thompson', 'fa', 'waite', 'brit', 'patent', '1096912', '(1967)', '330', 'bw', 'jackson', 'us', 'patent', '3689593', '(1972)', '331', 'jp', 'kennedy', 'vsp', 'chang', 'ra', 'smith', 'b', 'ivan', 'polym', 'bull,', '1', '575', '(1978)', '332', 'jp', 'kennedy', 'preprints', '5th', 'intern', 'symposium', 'cationic', 'other', 'ionic', 'polymerizations', '6', '(1980)', '333', 'r', 'asami', 'm', 'takaki', 'k', 'kita', 'e', 'asakura', 'plym', 'bull,', '3', '83', '(1980)', '334', 'js', 'vrgas', 'jg', 'zillox', 'p', 'rempp', 'e', 'frauta', 'polymer', 'bull,', '3', '83', '(1980)', '335', 'p', 'rempp', 'polymer', 'prepr', 'jpn,', '29', '1397', '(1980)', '336', 'y', 'tanizaki', 'k', 'minagawa', 's', 'takase', 'k', 'watanabe', 'abstracts', 'acs-csj', 'chemical', 'congressinde', '136', '(1979)', '337', 'g', 'gerber', 'e', 'reese', 'makromol', 'chem,', '55', '96', '(1962)', '338', 'y', 'yamashita', 'j', 'appl', 'polymer', 'sci,', 'applied', 'polymer', 'symposia', '36', '193', '(1981)', '339', 'rb', 'mesrobian', 'h', 'mark', 'textile', 'j,', '23', '294', '(1953)', '340', 'dj', 'metz', 'rb', 'mesrobian', 'j', 'polymer', 'sci,', 'j16', '345', '(1955)', '341', 'jd', 'matlack', 'sn', 'chinai', 'ra', 'guzzi', 'dw', 'levi', 'j', 'polymer', 'sci,', '49', '533', '(1961)', '342', 'g', 'natta', 'e', 'beati', 'f', 'severini', 'j', 'polymer', 'sci,', '34', '685', '(1959)', '343', 'd', 'makulova', 'd', 'berek', 'sb', 'prac', 'chem', 'fak', 'sloven', 'vysokej', 'skily', 'tech,', '141', '(1961)', 'chem', 'abstr,', '57', '8729', '(1962)', '344', 'jj', 'wu', 'za', 'rogovin', 'aa', 'konkin', 'khim', 'volokna', '6', '11', '(1962)', 'chem', 'abstr,', '58', '11509', '(1963)', '345', 'elsahkulubey', 'yy', 'durmaz', 'al', 'demirel', 'y', 'yagci', 'macromolecules', '2010', '43', '2732', '346', 'a', 'bar-ilan', 'a', 'zilkha', 'eur', 'polymer', 'j,', '6', '403', '(1970)', '347', 'a', 'ravve', 'cw', 'fitko', 'j', 'polymer', 'sci,', 'a-14', '2533', '(1966)', '348', 'c', 'de', 'ruijter', 'w', 'f', 'jager', 'j', 'groenewold', 's', 'j', 'picken', 'macromolecules', '2006', '39', '3824', '349', 'schollenberger', 'polymer', 'technology', 'interscience', 'new', 'york', '1969', '350', 'kc', 'gupta', 'k', 'khandekar', 'am', 'chem', 'soc', 'polymer', 'preprints', '2005', '46', '(1)', '654', '351', 'g', 'ezra', 'a', 'zilkha', 'j', 'appl', 'polymer', 'sci,', '13', '1493', '(1969)', '352', 'g', 'kockelbergh', 'g', 'smets', 'j', 'polymer', 'sci,', '33', '227', '(1958)', '353', 'k', 'nakamura', 'sk', 'kuchi', 'bull', 'chem', 'soc', 'japan', '41', '1977', '(1968)', '354', 'm', 'tsuda', 'j', 'polymer', 'sci,', '7', '259', '(1969)', '355', 'hg', 'heine', 'hj', 'resenkranz', 'h', 'rudolph', 'angew', 'chem,', 'int', 'editengl,', '11', '974', '(1974)', '356', 'cj', 'groeneboom', 'hj', 'hageman', 't', 'overeem', 'ajm', 'weber', 'makromol', 'chem,', '183', '281', '(1982)', '357', 'j', 'hutchison', 'mc', 'lambert', 'a', 'ledwith', 'polymer', '14', '250', '(1973)', '358', 'a', 'ledwith', 'pure', 'appl', 'chem,', '49', '431', '(1977)', '359', 'wa', 'pryor', 'free', 'radicals', 'mcgraw-hill', 'new', 'york', '1966', '360', 'mj', 'gibian', 'tetrahedron', 'letters', '5331', '(1967)', '361', 'g', 'rausing', 's', 'sunner', 'tappi', '45', '(1)', '203a', '(1962)', '362', 'v', 'jacks', 'w', 'kern', 'makromol', 'chem,', '83', '71', '(1965)', '363', 'p', 'weiss', 'j', 'f', 'gerecht', 'ij', 'krems', 'j', 'polymer', 'sci,', '35', '343', '(1959)', '364', 'k', 'ito', 'y', 'tomi', 's', 'kawaguchi', 'macromolecules', '25', '1534', '(1992)', '365', 'sg', 'cohen', 'hs', 'haas', 'h', 'slotnick', 'j', 'polymer', 'sci,', '11', '913', '(1953)', 'hs', 'haas', 'sg', 'cohen', 'ac', 'oglesby', 'er', 'karlin', 'j', 'polymer', 'sci,', '15', '427', '(1955)', 'st', 'raﬁkov', 'gn', 'chelnokova', 'iv', 'zhuraleva', 'pn', 'gribkova', 'j', 'polymer', 'sci,', '53', '75', '(1961)', '366', 'jp', 'kennedy', 'ed,', 'j', 'appl', 'polymer', 'sci,', 'applied', 'polymer', 'symposia', '30', '(1977)', 'jp', 'kennedy', 'us', 'patent', '3349065', '(oct', '24', '1967)', 'jp', 'kennedy', 'polymer', 'chemistry', 'synthetic', 'elastomers', '(', 'jp', 'kennedy', 'e', 'tornquist', 'ed)wiley-interscience', 'new', 'york', '1968', 'jp', 'kennedy', 'jk', 'gillham', 'adv', 'polymer', 'sci,', '10', '1', '(1972)', '367', 'jp', 'kennedy', 'm', 'nakao', 'j', 'appl', 'polymer', 'sci,', 'appl', 'polymer', 'symposia', '30', '73', '(1977)', 'references', '689', '368', 'nbetz', 'a', 'le', 'moel', 'jp', 'duraud', 'c', 'darnez', 'macromolecules', '25', '213', '(1992)', '369', 'et', 'kang', 'kg', 'neoh', 'kl', 'tan', 'y', 'uyama', 'n', 'morikawa', 'y', 'ikada', 'macromolecules', '25', '1959', '(1992)', '370', 'bu', 'kolb', 'pa', 'patton', 'tj', 'mccarthy', 'am', 'chem', 'soc,', 'polymer', 'preprints', '28', '(2)', '248', '(1987)', '371', 'k', 'matyjaszewski', 'ygnanou', 'l', 'leibler', 'eds', '‘macromolecular', 'engineering', 'wiley', 'new', 'york', '2007', '372', 'j', 'hirkach', 'k', 'ruehl', 'k', 'matyjaszewski', 'am', 'chem', 'soc,', 'polymer', 'preprints', '29', '(2)', '112', '(1988)', '373', 'y', 'jiang', 'jmj', 'frechet', 'am', 'chem', 'soc,', 'polymer', 'preprints', '30', '(1)', '127', '(1989)', '374', 'jf', 'kenney', 'polymer', 'eng', 'sci,', '8', '(3)', '216', '(1968)', 'whcharch', 'jc', 'shivers', 'textile', 'research', 'j,', '29', '536', '(1959)', '375', 'imperial', 'chemical', 'industries', 'ltd', 'brit', 'patent', '802921', '(nov', '23', '1955)', '376', 'a', 'noshay', 'je', 'mcgrath', 'block', 'copolymers', 'overview', 'critical', 'survey', 'academic', 'press', 'new', 'york', '1977', '377', 'dh', 'solomon', 'chapter', '1', 'step', 'growth', 'poymerization', '(dh', 'solomon', 'ed)', 'dekker', 'new', 'york', '1972', '378', 'ja', 'brydson', 'plastic', 'materials', '4th', 'ed,', 'butterworth', 'scientiﬁc', 'london', '1982', '379', 'general', 'electric', 'co,', 'brit', 'patent', '984522', '(june', '1', '1960)', '380', 'schollenberger', 'polymer', 'technology', '(pf', 'bruins', 'ed)', 'interscience', 'new', 'york', '1969', '381', 'r', 'ukielski', 'h', 'wojcikiewicz', 'intern', 'polymer', 'science', 'technology', '3', '(11)', 't65', '(1976)', 'gk', 'hoexchele', 'wk', 'witsiepe', 'angew', 'makromol', 'chem,', '29-30', '267', '(1973)', '382', 'y', 'yamashita', 't', 'hane', 'j', 'polymer', 'sci,', 'polym', 'chem', 'ed11,', '425', '(1973)', 'y', 'yamashita', 'y', 'murase', 'k', 'ito', 'j', 'polymer', 'sci,', 'polym', 'chem', 'ed,', '11', '435', '(1973)', '383', 'vv', 'korshak', 'tm', 'frunze', 'synthetic', 'hetero-chain', 'polyamides', 's', 'manson', 'jerusalem', '1987', '384', 'pw', 'morgan', 'j', 'polymer', 'sci,', 'c4', '1075', '(1964)', 'dj', 'lyman', 'sl', 'yung', 'j', 'polymer', 'sci,', '40', '407', '(1959)', '385', 'dj', 'lyman', 'sl', 'yung', 'j', 'polymer', 'sci,', '40', '407', '(1959)', '386', 'vv', 'korshak', 'sv', 'vinogradova', 'mm', 'teplyakov', 'rd', 'fedorova', 'gsh', 'papara', 'vysokomol', 'soyed,', '8', '(12)', '2155', '(1966)', 'vvkorshak', 'svvinogradova', 'mmteplyakov', 'admaksomov', 'izv', 'vysok', 'uchebn', 'zaved', 'khim', 'khim', 'technol,', '10', '(6)', '688', '(1967)', '387', 'vv', 'korshak', 'sv', 'vinogradova', 'mm', 'teplyakov', 'vysokomol', 'soyed,', '7', '(8)', '1406', '(1965)', '388', 'y', 'goto', 's', 'miwa', 'chem', 'high', 'polymers', 'japan', '25', '595', '(1968)', '389', 'kl', 'mittal', 'polyamides', 'syntheses', 'characterizations', 'applications', 'vols', '1', '2', 'plenum', 'new', 'york', '1984', '390', 'imperial', 'chemical', 'industries', 'brit', 'patent', '807666', '(jan', '4', '1957)', '391', 'imperial', 'chemical', 'industries', 'brit', 'patent', '1148068', '(aug', '9', '1965)', '392', 'm', 'tsuruta', 'f', 'kabayashi', 'k', 'matsuyata', 'kogyo', 'kagaku', 'zasshi', '62', '1084', '1087', '(1959)', 'chem', 'abstr,', '57', '13977', '(1962)', '393', 'd', 'dieterich', 'w', 'keberle', 'h', 'witt', 'angew', 'chem,', 'intern', 'ed,', '9', '(1)', '40', '(1970)', '394', 'd', 'dieterich', 'o', 'bayer', 'j', 'peter', 'e', 'muller', 'ger', 'patent', '1156977', '(june', '5', '1962)', '395', 'd', 'dieterich', 'o', 'bayer', 'brit', 'patent', '1078202', 'w', 'keberle', 'd', 'dieterich', 'brit', 'patent', '1078688', '396', 'ca', 'lukach', 'hm', 'spurlin', 'copolymerization', '(ge', 'ham', 'ed)', 'interscience', 'new', 'york', '1964', 'hf', 'mark', 'sm', 'atlaspetrol', 'reﬁner', '39', '149', '(1960)', 'g', 'natta', 'j', 'polymer', 'sci,', '34', '21', '531', '(1959)', '397', 'av', 'tobolsky', 'kf', 'o’driscoll', 'das', '1', '114', '323', 'june', '24', '1959', '398', 'm', 'szwarc', 'j', 'smid', 'progress', 'reaction', 'kinetics', 'vol', 'ii', 'p', '219', 'pegamon', 'press', 'oxford', '1964', 'm', 'szwarc', 'nature', '178', '1168', '(1956)', '399', 's', 'bywater', 'fortsch-hochpolym', 'forsch,', '4', '66', '(1965)', '400', 'r', 'chiang', 'jh', 'rhodes', 'ra', 'evans', 'j', 'polymer', 'sci,', 'a-14', '3089', '(1966)', '401', 'nl', 'madison', 'amer', 'chem', 'spc', 'polymer', 'preprints', '7', '1099', '(1966)', '402', 'hyu', 'aj', 'bur', 'lj', 'fetters', 'j', 'chem', 'phys,', '44', '2568', '(1966)', '403', 'mp', 'dreyfuss', 'p', 'dreyfuss', 'polymer', '6', '93', '(1965)', '404', 'ceh', 'bawn', 'rm', 'bell', 'a', 'ledwith', 'polymer', '6', '95', '(1965)', '405', 'd', 'sims', 'makromol', 'chem,', '64', '151', '(1963)', '406', 's', 'bywater', 'macromolecular', 'chemistry', 'buterworths', 'longond', '1962', 'm', 'morton', 'anionic', 'polymerization', 'principles', 'practice', 'academic', 'press', 'new', 'york', '1983', '407', 'rk', 'graham', 'dl', 'dunkelburger', 'we', 'goode', 'j', 'am', 'chem', 'soc,', '82', '400', '(1960)', '408', 't', 'saegusa', 's', 'matsumoto', 'y', 'hashimoto', 'macromolecules', '3', '377', '(1970)', '409', 'lj', 'fetters', 'j', 'polymer', 'sci,', 'c', '(26)', '1', '(1969)', '410', 'jp', 'kennedy', 'makromol', 'chem,', 'suppl,', '3', '1', '(1979)', '411', 'rj', 'ceresa', 'block', 'graft', 'copolymers', 'butterworths', 'washington', 'dc,', '1962', '412', 'h', 'staudinger', 'ber,', '57', '1203', '(1924)', 'kautchuk', '5', '128', '(1929)', '413', 'dj', 'angier', 'wf', 'watson', 'j', 'polymer', 'sci,', '20', '235', '(1956)', '414', 'rj', 'ceresa', 'polymer', '1', '477', '(1960)', '415', 'b', 'gordon', 'iii', 'lf', 'hancock', 'amchemsoc', 'polymer', 'preprints', '26', '(1)', '98', '(1985)', '416', 'pj', 'hans', 'b', 'gordon', 'iii', 'amchemsoc', 'polymer', 'preprints', '28', '(2)', '310', '(1987)', '417', 'w', 'risse', 'rh', 'grubbs', 'amchemsoc', 'polymer', 'preprints', '30', '193', '(1989)', '690', '9', 'organic', 'reactions', 'polymers', '418', 'dy', 'sogah', 'ow', 'webster', 'macromolecules', '19', '1775', '(1986)', '419', 'd', 'patit', 'r', 'jerome', 'ph', 'teyssie', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '17', '2903', '(1979)', 'wt', 'allen', 'de', 'eaves', 'angew', 'makromol', 'chem,', '5859', '321', '(1977)', 'bh', 'wondraczek', 'jp', 'kennedy', 'j', 'polymer', 'sci,', 'polymer', 'chem', 'ed,', '20', '173', '(1982)', '420', 'ca', 'veith', 're', 'cohen', 'amchemsoc', 'polymer', 'preprints', '31', '(1)', '42', '(1990)', '421', 'jp', 'kennedy', 'b', 'keszler', 'y', 'tsunogae', 's', 'midha', 'am', 'chem', 'soc', 'polymer', 'preprints', '32', '(1)', '310', '(1991)', '422', 'tj', 'deming', 'bm', 'novak', 'macromolecules', '24', '5478', '(1991)', '423', 't', 'higashimura', 's', 'aoshima', 'm', 'sawamoto', 'makromol', 'chem', 'macromol', 'symp,', '1314', '457', '(1988)', '424', 'jp', 'kennedy', 'jl', 'price', 'polym', 'mater', 'sci,', '1991', '64', 'jp', 'kennedy', 'jl', 'price', 'k', 'koshimura', 'macromolecules', '24', '6567', '(1991)', '425', 't', 'pernecker', 'jp', 'kennedy', 'b', 'ivan', 'macromolecules', '25', '1642', '(1992)', '426', 'mg', 'kanatzidis', 'chem', 'eng', 'news', 'p', '38', '(dec', '3', '1990)', 'ly', 'chiang', 'pm', 'chaikin', 'do', 'cowan(eds)', 'advanced', 'organic', 'solid', 'state', 'materials', 'mat', 'res', 'soc', 'symp', 'proc,', '173', '(1990)', 'jr', 'reynolds', 'chemtech', '18', '440', '(1988)', '427', 'x-f', 'sun', 'sb', 'clough', 's', 'subrmanyam', 'a', 'blumstein', 'sk', 'tipathy', 'am', 'chem', 'soc', 'polymer', 'preprints', '33', '(1)', '576', '(1992)', '428', 's', 'subramanyam', 'a', 'blumstein', 'macromolecules', '24', '2668', '(1991)', '429', 'rd', 'mccullough', 'rd', 'lowe', 'am', 'chem', 'soc', 'polymer', 'preprints', '33', '(1)', '195', '(1992)', '430', 'w', 'rutsch', 'g', 'berner', 'r', 'kirchmayr', 'r', 'husler', 'g', 'rist', 'n', 'buhler', 'organic', 'coatings-science', 'technol-', 'ogy', '(gd', 'parﬁtt', 'av', 'patsis', 'eds)', 'dekker', 'new', 'york', 'vol', '8', '1986', '431', 'k', 'dietliker', 'm', 'rembold', 'g', 'rist', 'w', 'rutsch', 'f', 'sitek', 'rad-cure', 'europe', '87', 'proceedings', '3rd', 'conference', 'association', 'finishing', 'processes', 'sme', 'dearborn', 'mi', '1987', '432', 'g', 'rist', 'a', 'borer', 'k', 'dietliker', 'v', 'desobry', 'jp', 'fouassier', 'd', 'ruhlmenn', 'macromolecules', '25', '4182', '(1992)', '433', 'bj', 'sun', 'gc', 'bazan', 'am', 'chem', 'soc', 'polymer', 'preprints', '36', '(1)', '253', '(1995)', '434', 'm', 'weck', 'p', 'schwab', 'rh', 'grubbs', 'macromolecules', '29', '1789', '(1996)', 'a', 'proto', 'a', 'avagliano', 'd', 'saviello', 'r', 'ricciardi', 'c', 'capacchione', 'micrornolecules', '2010', '43', '5919', '435', 'r', 'jerome', 'jl', 'hedrick', 'cj', 'hawker', 'angew', 'chem', 'int', 'ed', 'engl', '37', '1274', '(1998)', 'cj', 'hawker', 'jl', 'hedrick', 'ee', 'malmstrom', 'm', 'tollsas', 'd', 'mecerreyes', 'g', 'moineau', 'ph', 'dubois', 'r', 'jerome', 'macromolecules', '31', '213', '(1998)', '436', 'e', 'yoshida', 'y', 'osagawa', 'macromolecules', '31', '1446', '(1998)', 'rlkuhlman', 'j', 'klosin', 'macromolecules', '2010', '45', '7903', 'lm', 'pitet', 'ma', 'hillmyer', 'macromolecules', '2009', '42', '3674', '437', 'y', 'kotani', 'm', 'kato', 'm', 'kamigaito', 'm', 'sawamoto', 'macromolecules', '29', '6979', '(1996)', 's', 'lu', 'q-l', 'fan', 's-j', 'chua', 'w', 'huang', 'macromolecules', '2003', '36', '304', 'jb', 'matson', 'rh', 'grubbs', 'macromolecules', '2008', 'asap', 'article101021ma800980p', '438', 's', 'coca', 'h-j', 'paik', 'k', 'matyjaszewski', 'macromolecules', '30', '6513', '(1997)', '439', 'a', 'kajiwara', 'k', 'matyjaszewski', 'macromolecules', '31', '3489', '(1998)', '440', 'wj', 'gauthier', 'jf', 'corrigan', 'nj', 'taylor', 's', 'collins', 'macromolecules', '28', '3771', '(1995)', '441', 'rw', 'waymouth', 'gw', 'coates', 'science', '267', '222', '(1995)', '442', 'jvohlidal', 'j', 'sedlacek', 'm', 'zigon', 'kovine', 'zlitine', 'tehnol', '31(6)', '581', '(1997)', 'chem', 'abstr129', '637', '(1998)', '443', 'y', 'misumi', 't', 'masuda', 'macromolecules', '31', '7572', '(1998)', '444', 'w', 'liu', 'j', 'kumar', 's', 'trypathy', 'kj', 'senecal', 'l', 'samuelson', 'j', 'am', 'chem', 'soc,', '121', '71', '(1999)', '445', 'h', 'nakatani', 'k-h', 'nitta', 't', 'uozumi', 'k', 'soga', 'polymer', '40', '1623', '(1999)', '446', 'la', 'wall', 'sl', 'madorsky', 'dw', 'brown', 's', 'straus', 'r', 'simka', 'j', 'am', 'chem', 'soc,', '76', '3430', '(1954)', '447', 'la', 'wall', 's', 'straus', 'j', 'polymer', 'sci,', '1960', '44', '313', '448', 's', 'madorsky', 'j', 'polymer', 'sci,', '1952', '9', '133', '449', 's', 'madorsky', 'j', 'polymer', 'sci,', '1953', '11', '491', '450', 's', 'madorsky', 's', 'straus', 'd', 'thompson', 'l', 'williamson', 'j', 'polymer', 'sci,', '1949', '4', '639', '451', 's', 'madorsky', 's', 'straus', 'j', 'polymer', 'sci,', '1959', '36', '183', '452', 's', 'madorsky', 's', 'straus', 'j', 'res', 'natn', 'bur', 'st,', '1948', '40', '417', '453', 'wg', 'oaks', 'rb', 'richards', 'jchem.', 'soc', '(london)', '1949', '2929', '454', 'y', 'tsuchiya', 'k', 'sumi', 'j', 'polymer', 'sci,', '1968', 'a-16', '415', '455', 'chem', 'eng', 'news', 'p', '41', 'sept', '28', '1964', '456', 's', 'madorsky', 'thermal', 'degradation', 'organic', 'polymers', 'interscience', 'new', 'york', '1964', '457', 'em', 'bavilacqua', 'thermal', 'stability', 'polymers', 'vol', '1,', 'rt', 'conley', '(ed)', 'dekker', 'new', 'york', '1970', 'r', 'bernstein', 'dr', 'derzon', 'kt', 'gillen', 'am', 'chem', 'soc', 'polymer', 'preprints', '2007', '48(1)', '611', '458', 'ma', 'golub', 'rj', 'gargiulo', 'polymer', 'letters', '1972', '10', '41', '459', 'm', 'galin-vacherot', 'eur', 'polymer', 'j,', '1971', '7', '1455', '460', 'gg', 'cameron', 'n', 'grassie', 'polymer', '19961', '2', '367', 'references', '691', '461', 'hhg', 'jellinek', 'j', 'polymer', 'sci,', '1948', '3', '850', '462', 'hhg', 'jellinek', 'j', 'polymer', 'sci,', '1949', '4', '1', '463', 'hhg', 'jellinek', 'j', 'polymer', 'sci,', '1949', '4', '13', '464', 'n', 'grassie', 'ww', 'kerr', 'trans', 'faraday', 'soc.', '1959', '66', '1050', '465', 'gg', 'cameron', 'n', 'grassie', 'makromol', 'chem,', '1962', '53', '72', '466', 'gg', 'cameron', 'gp', 'kerr', 'eur', 'polymer', 'j,', '1970', '6', '423', '467', 'jwc', 'crawford', 'j', 'soc', 'chem', 'ind,', '1949', '68', '201', '468', 'n', 'grassie', 'jg', 'speakman', 'j', 'polymer', 'sci,', '1971', 'a-19', '419', '469', 'n', 'grassie', 'jg', 'spaakman', 'tj', 'davis', 'j', 'polymer', 'sci,', '1971', 'a-19', '931', '470', 's', 'choudhary', 'k', 'lederer', 'eur', 'polymer', 'j,', '1982', '18', '1021', '471', 'wj', 'burlant', 'jl', 'parsons', 'j', 'polymer', 'sci,', '1956', '22', '156', '472', 'j', 'schulz', 'h', 'bayzer', 'w', 'stubchen', 'j', 'schurz', 'makromol', 'chem', '1957', '23', '152', 'ibid,', '1959', '29', '156', '473', 'sl', 'ma472orsky', 'amd', 's', 'strais', 'j', 'res', 'natl', 'bur', 'std.', '1958', '61', '77', '474', 'n', 'grassie', 'jn', 'hay', 'j', 'polymer', 'sci,', '1962', '56', '189', '475', 'n', 'grassie', 'jnhay', 'makromol', 'chem,', '1963', '64', '82', '476', 'jn', 'hay', 'j', 'polymer', 'sci,', '1968', 'a-16', '2127', '477', 'n', 'grassie', 'r', 'mcguchan', 'eur', 'polymer', 'j,', '1972', '8', '243', '478', 'na', 'kubasova', 'din', 'suan', 'din', 'ma', 'heiderich', 'mv', 'shishkina', 'vysokomol', 'soyed,', '1979', 'a13', '(1)', '162', '479', 'n', 'grassie', 'r', 'mcguchan', 'eur', 'polymer', 'j,', '1970', '6', '1277', 'ibid1971,', '7', '1091', 'ibid1971,', '71357', '480', 'h', 'gilbert', 'sj', 'averill', 'fe', 'miller', 'rf', 'schmidt', 'fd', 'stewart', 'hl', 'trumbull', 'j', 'am', 'chem', 'soc,', '1954', '76', '1074', '481', 'n', 'grassie', 'r', 'mcguchan', 'eur', 'polymer', 'j,', '1972', '8', '243', '482', 'p', 'bataille', 'bt', 'van', 'j', 'polymer', 'sci,', '1972', 'a-110', '1097', '483', 's', 'van', 'der', 'ven', 'wf', 'de', 'wit', 'angew', 'makromol', 'chem,', '1969', '8', '143', '484', 'jd', 'danforth', 'j', 'spiegel', 'j', 'bloom', 'j', 'macromol', 'sci', '–', 'chem,', '1982', 'a17', '(7)', '1107', '485', 'sa', 'liebman', 'dh', 'ahlstrom', 'ej', 'quinn', 'ag', 'geigley', 'jt', 'melluskey', 'j', 'polymer', 'sci,', '1971', 'a-19', '1921', '486', 'i', 'mita', 'aspects', 'degradation', 'stabilization', 'polymers', 'hh', 'jellinek', '(ed)', 'elsevier', 'amsterdam', '1978', '487', 'h', 'bamford', 'df', 'feuton', 'polymer', '1969', '10', '63', '488', 'epchang', 'rsalovey', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1974', '12', '2927', '489', 'do', 'hummel', 'hj', 'dussel', 'k', 'rubenacker', 'makromol', 'chem,', '1971', '145', '267', '490', 'rm', 'lum487', 'mm', 'o’mara', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1970', 'a-18', '1887', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1977', '15', '489', '491', 'a', 'ballistreri', 's', 'foti', 'g', 'montaudo', 'e', 'scamporrino', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1980', '18', '1147', '492', 'j', 'svetly', 'r', 'lucas', 'j', 'michalcova', 'm', 'kolinsky', 'makromol', 'chem,rapid', 'commun,', '1980', '1', '247', '493', 'j', 'svetly', 'r', 'lucas', 's', 'pokorny', 'm', 'kolinsky', 'makromol', 'chem,rapid', 'commun,', '1981', '2', '149', '494', 'l', 'valko', 'p', 'simon', 'vkello', 'makromol', 'chem,', '1982', '183', '3057', '495', 'rd', 'bohme', 'ra', 'wessling', 'j', 'appl', 'polymer', 'sci,', '1972', '16', '1761', '496', 'rf', 'boyer', 'j', 'phys', 'coll', 'chem,', '1947', '51', '80', 'e', 'elakesh', 'tr', 'hull', 'd', 'price', 'pcarty', 'j', 'vinyl', 'additive', '497', 'la', 'matheson', 'rf', 'boyer', 'ind', 'eng', 'chem,', '1942', '44', '867', '498', 'cb', 'havens', 'nbs', 'cicrular', '525', '1953', '107', '499', 'fh', 'winslow', 'wo', 'baker', 'ya', 'yager', 'proceeding', 'first', 'second', 'carbon', 'conferences', 'univ', '2003', '9(3)', '116', 'buffalo', 'press', '1956', '500', 'n', 'grassie', 'if', 'mclaren', 'ic', 'mcneill', 'eur', 'polymer', 'j,', '1970', '6', '679', '501', 'n', 'grassie', 'if', 'mclaren', 'ic', 'mcneill', 'eur', 'polymer', 'j,', '1970', '6', '865', '502', 'ra', 'haldeon', 'j', 'polymer', 'sci,', '1968', 'a-16', '951', '503', 'ep', 'goodings', 'soc', 'chem', 'ind', '(london)', 'monograph', '1961', '13', '211', '504', 'rm', 'lum', 'j', 'polymer', 'sci,', '1979', '17', '203', '505', 'v', 'passalaqua', 'f', 'pilati', 'v', 'zamoni', 'b', 'fortunato', 'p', 'manaresi', 'polymer', '1976', '17', '1044', '506', 'gj', 'sutton', 'bj', 'tighe', 'roberts', 'j', 'polymer', 'sci,chem', 'ed,', '1973', '11', '1079', '507', 'a', 'davis', 'jh', 'golden', 'j', 'chem', 'soc,', 'b', '(london)', '1968', '45', '508', 'f', 'wiloth', 'makromol', 'chem', '1971', '144', '283', '509', 'dp', 'bishop', 'da', 'smith', 'j', 'appl', 'polymer', 'sci,', '1970', '14', '205', '510', 'a', 'kamimura', 's', 'yamamoto', 'org', 'lett,', '2007', '9', '2533', '511', 'yep', 'krasnov', 'vp', 'aksenova', 'sn', 'kharkov', 'sa', 'baranova', 'vysokomol', 'soyed,', '1970', 'a12', '(4)', '873', '512', 'vv', 'rode', 'yem', 'bondarenko', 'vv', 'korshak', 'yes', 'krongauz', 'al', 'rusanov', 'dokl', 'ansssr', '1966', '171', '355', '513', 'vv', 'rode', 'yem', 'bondarenko', 'vysokomol', 'soyed1967,', 'a9(12)', '218', 'w', 'wrasidlo', 'j', 'polymer', 'sci,', '1970', 'a-18', '1107', '514', 'a', 'scotney', 'eur', 'polymer', 'j', '1972', '8', '175', 'ibid,', '1972', '8', '185', '515', 'ii', 'levantovskaya', 'gv', 'dralyuk', 'oa', 'mochalova', 'ia', 'yurkova', 'ms', 'akutin', 'bv', 'kovarkaya', 'vysokomol', 'soyed', '1971', 'a13', '(1)', '8', '692', '9', 'organic', 'reactions', 'polymers', '516', 'vv', 'kopylov', 'an', 'pravednilkov', 'visokomol', 'soyed,', '1969', 'a11', '849', '517', 'n', 'grassie', 'rs', 'roche', 'makromol', 'chem', '1968', '112', '16', '518', 'yu', 'sclyapnikov', 'ta', 'bagaenskay', 'sg', 'kiryushkin', 'tv', 'monakhova', 'eur', 'polymer', 'j', '1979', '15', '737', '519', 'wl', 'hawkins', 'fh', 'winslow', 'chemical', 'reactions', 'polymers', 'em', 'fetters', '(ed)', 'interscience', 'ny1964', '520', 'jh', 'adams', 'j', 'polymer', 'sci,', '1970', 'a-18', '1077', '521', 'vm', 'yurev', 'aa', 'pravednikov', 'ss', 'medvedev', 'doklakad', 'nauk', 'sssr,', '1959', '124', '335', '522', 'bg', 'achhammer', 'mj', 'reiney', 'fw', 'reinhart', 'j', 'resnat', 'bur', 'standards', '1951', '47', '116', '523', 'bg', 'achhammer', 'mj', 'reiney', 'fw', 'reinhart', 'j', 'polymer', 'sci,', '1952', '8', '555', '524', 'a', 'votinova', 'p', 'kobenko', 'f', 'marei', 'zh', 'ﬁz', 'khim,', '1952', '16', '106', '525', 'c', 'crouzet', 'j', 'marchal', 'j', 'appl', 'polymer', 'sci,', 'appl', 'symposia', '197935', '151', '526', 'sd', 'rasumovskii', 'on', 'karpukhin', 'aa', 'kefeli', 'tv', 'pokholok', 'gye', 'zaikov', 'vysokomol', 'soyed,', '1971', 'a13', '782', '527', 'lx', 'buxbaum', 'angew', 'chem', 'intern', 'ed,', '1968', '7', '182', '528', 'kc-yu,', 'w,b-zu', 'c', 'w-hsun', 'lnd', 'eng', 'chem', 'res', '1998', '37(4)', '1228', '529', 'g', 'valk', 'g', 'heidemann', 's', 'dugal', 'h', 'krussmann', 'angew', 'makromol', 'chem,', '1970', '10', '135', '530', 'hhg', 'jellinek', 'ak', 'chandhuri', 'j', 'polymer', 'sci,', '1972', 'a-110', '1773', '88', 'synthetic', 'lubricants', 'liquids', 'rs', 'gunderson', 'av', 'khart', '(eds)', 'chemistry', 'publishing', 'house', 'new', 'york', '1965', '531', 'pa', 'okunev', 'og', 'tarakanov', 'vysokomol', 'soyed', '1968', 'a10', '(1)', '173', '532', 'ii.', 'levantovskaya', 'gv', 'dralyuk', 'oa', 'mochalova', 'ia', 'yurkova', 'ms', 'akutin', 'bvkovarkaya', 'vysokomol', 'soyed,', '1971', 'a13', '(1)', '8', '533', 'hhg', 'jellinek', 'sh', 'ronel', 'j', 'polymer', 'sci,', '1971', 'a-19', '2605', '534', 'fitton', 'n', 'howard', 'gr', 'williamson', 'brit', 'polymer', 'j1970,', '2', '217', '535', 'b', 'ranby', 'jf', 'rabek', 'j', 'appl', 'polymer', 'sci,', 'appl', 'symposia', '1979', '35', '243', '536', 'y', 'hama', 'k', 'hosono', 'y', 'furui', 'k', 'shinohara', 'j', 'polymer', 'sci', '1969', 'b7', '839', '537', 't', 'takeshita', 'k', 'tsuji', 't', 'keiki', 'j', 'polymer', 'sci,', '1972', 'a-19', '1411', '538', 'k', 'tsuji', 't', 'seiki', 'j', 'polymer', 'sci,', 'letters', '1972', '10', '139', '539', 'dj', 'carlson', 'dm', 'wiles', 'macromolecules', '1969', '2', '587', '597', '540', 't', 'kagiya', 'k', 'takemoto', 'j', 'macromol', 'sci,', '1976', '5', '795', '541', 't', 'takashita', 'k', 'tsuji', 't', 'keiki', 'j', 'polymer', 'sci1972,', 'a-110', '2315', '542', 'im', 'rozman', 'izv', 'akad', 'nauk', 'sssr1958,', '22', '60', '543', 'g', 'oldham', 'ar', 'ware', 'sa', 'blackwell', 'nucl', 'energy', '1967', '20', '158', '544', 'na', 'weir', 'j', 'appl', 'polymer', 'sci1973,', '17', '401', '545', 'na', 'weir', 'th', 'milkie', 'eur', 'polymer', 'j,', '1980', '141', '546', 'n', 'grassie', 'bjd', 'torrance', 'j', 'polymer', 'sci,', '1968', 'a-16', '3303', '3315', '547', 'n', 'grassie', 'e', 'farish', 'eur', 'polymer', 'j,', '1967', '3', '627', '548', 'n', 'grassie', 'rh', 'jenkins', 'eur', 'polymer', 'j1973,', '9', '697', '549', 'c', 'david', 'm', 'borsu', 'g', 'gleuskens', 'eur', 'polymer', 'j,', '1970', '6', '959', '550', 'kj', 'buchanan', 'wj', 'mcgill', 'eur', 'polymer', 'j,', '1980', '16', '319', '551', 'wh', 'gibb', 'jr', 'maccallum', 'eur', 'polymer', 'j,', '1972', '8', '1223', '552', 'ed', 'owens', 'rj', 'bailey', 'j', 'polymer', 'sci,', '1972', 'a-110', '113', '553', 'c', 'decker', 'm', 'balander', 'eur', 'polymer', 'j,', '1982', '18', '1085', '554', 'ed', 'owens', 'ji', 'williams', 'j', 'polymer', 'sci,chem', 'ed,', '1974', '12', '1933', '555', 'c', 'david', 'w', 'demarteau', 'g', 'gueskens', 'polymer', '1967', '497', '556', 'm', 'day', 'dm', 'wiles', 'j', 'polymer', 'sciletters,', '1971', '9', '665', '557', 'sb', 'maerov', 'j', 'polymer', 'sci1965,', 'a-13', '487', '558', 'sm', 'cohen', 'rh', 'young', 'ah', 'markhart', 'j', 'polymer', 'sci,', '1971', 'a-19', '3263', '559', 'jf', 'rabek', 'g', 'ramme', 'g', 'cauback', 'b', 'ranby', 'vt', 'kagiya', 'eur', 'polymer', 'j,', '1979', '15', '339', '560', 'dj', 'carlsson', 'rd', 'parnell', 'dm', 'wiles', 'j', 'polymer', 'sciletters,', '1973', '11', '149', '561', 'o', 'cicchetti', 'adv', 'polymer', 'sci,', '1970', '7', '70', '562', 'ml', 'kaplan', 'po', 'keller', 'j', 'polymer', 'sciletters,', '1971', '9', '565', '563', 'jm', 'ginhac', 'jl', 'gardette', 'r', 'arnaud', 'j', 'lemaire', 'makromol', 'chem,', '1981', '182', '1017', '564', 'a', 'huvet', 'j', 'phillipe', 'j', 'verdu', 'eur', 'polymer', 'j1978,', '14', '709', '565', 'b', 'ranby', 'jf', 'rabek', 'j', 'appl', 'polymer', 'sci,', 'appl', 'symposia', '1979', '35', '243', '566', 'jf', 'rabek', 'g', 'cauback', 'b', 'ranby', 'j', 'appl', 'polymer', 'sci,', 'appl', 'symposia', '1979', '35', '299', '567', 'ml', 'kaplan', 'pg', 'keller', 'j', 'polymer', 'sci,', '1970', 'a-18', '3163', '568', 'jf', 'rabek', 'g', 'ramme', 'g', 'cauback', 'b', 'ranby', 'vt', 'kagiya', 'eur', 'polymer', 'j1979,', '15', '339', '569', 'j', 'chaineaux', 'c', 'tanielian', 'j', 'appl', 'polymer', 'sci,', 'appl', 'symposia', '1979', '35', '337', '570', 'jl', 'gardette', 'j', 'lemaire', 'makromol', 'chem,', '1982', '183', '2415', 'references', '693', '571', 'cs', 'schollenberger', 'fd', 'stewart', 'advances', 'urethane', 'science', 'technology', 'kc', 'frisch', 'sl', 'reegen', '(eds)', 'technology', 'publishing', 'co,', 'inc', 'westport', 'conn,', '1976', '572', 'l', 'reich', 'ss', 'stivala', 'elements', 'polymer', 'degradation', 'mcgraw-hill', 'book', 'co,', 'new', 'york', '1971', '573', 'a', 'chapiro', 'irradiation', 'polymers', 'rf', 'gould', '(ed)', 'advances', 'chemistry', 'series', '66', 'am', 'chem', 'soc,', 'washington', 'dc,', '1967', '574', 's', 'carroccio', 'c', 'puglisi', 'g', 'montaudo', 'macromolecules', '2003', '36', '7499', '575', 'f', 'harlen', 'w', 'simpson', 'fb', 'waddington', 'jd', 'waldon', 'ac', 'baskett', 'j', 'polymer', 'sci,', '1955', '18', '589', '576', 'aa', 'miller', 'ej', 'lawton', 'js', 'balwit', 'j', 'polymer', 'sci1954,', '14', '503', '577', 'a', 'chapiro', 'radiation', 'chemistry', 'poymeric', 'systems', 'interscience-wiley', 'new', 'york', '1962', '578', 'ep', 'goodings', 'thermal', 'degradation', 'polymers', 'soc', 'chem', 'ind,', 'monogr', '1961', 'no', '13', '211', '579', 'g', 'kamerbeek', 'h', 'kroes', 'w', 'grolle', 'thermal', 'degradation', 'polymers', 'soc', 'chem', 'ind,', 'monogr', '1961', 'no', '13', '357', '580', 'k', 'pielichowski', 'd', 'bogdal', 'j', 'pielichowski', 'a', 'boron', 'thermochim', 'acta', '1997', '307(2)', '155', '581', 'wh', 'starnes', 'jr,', 'h', 'chung', 'bj', 'wojciechowski', 'de', 'skillcorn', 'gm', 'benedikt', 'am', 'chem', 'soc', 'polymer', 'preprints', '1993(2)', '34', '114', '582', 'j', 'lacoste', 'y', 'israeli', 'am', 'chem', 'soc', 'polymer', 'preprints', '1993(2)', '34', '127', '583', 'cr', 'hoyle', 'ib', 'rufus', 'h', 'shah', 'am', 'chem', 'soc', 'polymer', 'preprints', '1993(2)', '34', '131', '584', 'a', 'rivaton', 'd', 'sallet', 'j', 'lemaire', 'j', 'polymer', 'degrad', 'stab', '1986', '141', '585', 'a', 'factor', 'wv', 'ligon', 'rj', 'may', 'macromolecules1987', '20', '2461', '586', 'a', 'gupper', 'p', 'wilhem', 'm', 'schiller', 'polymers', 'polymer', 'composites', '2003', '11(2)', '123', 'chapter', '10', 'polymeric', 'materials', 'special', 'applications', '101', 'polymer', 'supports', 'reagents', 'catalysts', 'drug', 'release', 'supports', 'materials', 'used', 'immobilization', 'various', 'reagents', 'catalysts', 'drugs', 'release', 'many', 'specially', 'prepared', 'macromolecules', 'reagents', 'catalysts', 'support', 'ﬁnd', 'applications', 'organic', 'syntheses', 'biochemical', 'reactions', 'special', 'separations', 'analyses', 'they', 'also', 'ﬁnd', 'uses', 'medicine', 'drug', 'release', 'etc', 'an', 'advantage', 'immobilized', 'polymeric', 'reagents', 'chemical', 'reactions', 'separated', 'often', 'easily', 'ﬁltration', 'products', 'reaction', 'cross-linked', 'polymeric', 'reagents', 'additional', 'advantage', 'several', 'different', 'polymeric', 'reagents', 'used', 'simultaneously', 'without', 'functional', 'groups', 'accessible', 'interaction', 'reactions', 'compounds', 'solution', 'require', 'high', 'dilutions', 'immobi-', 'lization', 'however', 'may', 'permit', 'reactions', 'carried', 'relatively', 'high', 'concentrations', 'immobilization', 'also', 'useful', 'syntheses', 'consist', 'many', 'steps', 'undesired', 'by-products', 'step', 'simply', 'washed', 'away', 'this', 'avoids', 'lengthy', 'isolation', 'puriﬁca-', 'tion', 'procedures', '2.', 'also', 'immobilizing', 'polymer', 'macromolecule', 'may', 'provide', 'microen-', 'vironmental', 'effects', 'attached', 'species', 'reactions', 'these', 'may', 'include', 'special', 'electronic', 'steric', 'conditions', 'could', 'different', 'existing', 'bulk', 'solution', 'the', 'chemical', 'uses', 'polymeric', 'reagents', 'originally', 'classiﬁed', 'patchornik', 'according', 'general', 'type', 'reaction', '1.', 'these', 'follows', 'complex', 'mixtures', '1', 'polymer-attached', 'reagents', 'used', 'special', 'separations', 'selectively', 'bind', 'one', 'species', 'a', 'bcde', 'a', 'b', 'cde', 'filter', 'recovery', 'a', 'b', 'a', 'b', 'the', 'polymer-bound', 'compound', 'separated', 'mixture', 'released', '2', 'the', 'polymer', 'catalyst', 'attached', 'such', 'catalysts', 'enzymes', 'inorganic', 'compounds', 'organometallic', 'compounds', 'a', 'substrate', 'a', 'product', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-910', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '695', '696', '10', 'polymeric', 'materials', 'special', 'applications', '3', 'the', 'polymeric', 'reagent', 'also', 'used', 'transfer', 'agents', 'low', 'molecular', 'weight', 'reactants', 'transfer', 'functional', 'moiety', 'aid', 'polymeric', 'agent', 'this', 'leaves', 'products', 'pure', 'form', 'ﬁltration', 'solvent', 'removal', '4', 'the', 'polymers', 'used', 'carriers', 'blocking', 'groups', 'syntheses', '1:', 'a', 'a', 'b', 'c', 'b', 'a', 'b', 'c', 'a', 'a', 'b', 'c', '1011', 'support', 'materials', 'many', 'different', 'support', 'materials', 'developed', '3', '4', 'since', 'original', 'use', 'merriﬁeld', 'polystyrene-based', 'support', 'material', 'polypeptide', 'synthesis', '5.', 'the', 'work', 'merriﬁeld', 'described', 'chap', '8', '(see', 'section', 'proteins)', 'beads', 'copolymers', 'styrene', 'divinyl', 'benzene', 'available', 'commercially', 'widely', 'used', 'supports', 'many', 'reactions', 'many', 'polymeric', 'materials', 'also', 'used', 'these', 'various', 'type', 'copolymers', 'styrene', 'polymers', 'the', 'list', 'includes', 'cellulose', 'starch', 'polyalkanes', 'polyamides', 'poly(glycidyl', 'methac-', 'rylate)', 'polyisobutylene', 'polynorbornene', 'polyacrylamide', 'others', 'in', 'instances', 'even', 'glass', 'used', 'the', 'prominent', 'support', 'materials', 'presented', '10111', 'materials', 'based', 'polystyrene', 'cross-linked', 'polystyrene', '(copolymer', 'divinyl', 'benzene)', 'original', 'support', 'material', 'used', 'merriﬁeld', 'polypeptide', 'syntheses', 'the', 'material', 'actually', 'terpolymer', 'styrene', 'chloromethyl', 'styrene', 'divinyl', 'benzene', 'x', 'z', 'x', 'cl', 'copolymers', 'styrene', 'divinyl', 'benzene', 'supports', 'since', 'received', 'many', 'uses', 'undergone', 'many', 'chemical', 'modiﬁcations', 'various', 'reactions', 'reagents', 'catalysts', 'the', 'material', 'functionalized', 'many', 'ways', 'thus', 'nitrated', 'chloromethylated', 'sulfonated', 'lithiated', 'carboxylated', 'acylated', 'the', 'greatest', 'use', 'made', 'chloromethylated', 'lithiated', 'derivatives', 'these', 'two', 'derivatives', 'react', 'nucleophilic', 'electrophilic', 'reagents', '101', 'polymer', 'supports', 'reagents', 'catalysts', 'drug', 'release', '697', 'respectively', 'resulting', 'wide', 'range', 'functionalized', 'polymers', 'for', 'various', 'modiﬁcation', 'reactions', 'polystyrene', 'see', 'chap', '9', 'two', 'types', 'cross-linked', 'polystyrene', 'often', 'favored', 'one', 'polymer', 'cross-linked', '1–2', 'divinyl', 'benzene', 'the', 'material', 'though', 'fairly', 'strong', 'mechanically', 'swells', 'expands', 'signiﬁcantly', 'volume', 'dispersed', 'proper', 'solvents', 'it', 'called', 'micro', 'porous', 'a', 'copolymer', '20', 'divinyl', 'benzene', 'second', 'type', 'it', 'prepared', 'presence', 'large', 'quantities', 'diluents', 'retain', 'products', 'expanded', 'form', 'as', 'result', 'structures', 'macro', 'porous', 'macro', 'reticular', 'the', 'advantages', 'micro', 'porous', 'macro', 'porous', 'structures', 'faster', 'reactions', 'less', 'fragility', 'easier', 'handling', 'although', 'macro', 'reticular', 'supports', 'less', 'often', 'used', 'advantage', 'useful', 'almost', 'solvent', 'as', 'variation', 'process', 'fre´chet', 'coworkers', 'introduced', 'reactive', 'ﬁltration', '6.', 'in', 'place', 'beads', 'used', 'discs', 'cross-linked', 'polystyrene', 'cut', 'rod', 'material', 'on', 'surface', 'discs', 'grafted', '22-dimethylazlactone', 'these', 'discs', 'used', 'ﬁltration', 'efﬁciently', 'scavenge', 'excess', 'amines', 'reaction', 'mixture', 'subsequently', 'frechet', 'coworkers', 'also', 'used', 'discs', 'ﬂow', 'acylation', 'reaction', '7.', 'to', 'improve', 'access', 'functional', 'groups', 'lee', 'coworkers', 'developed', 'process', 'placing', 'functional', 'groups', 'outer', 'surface', 'polystyrene', 'beads', '7.', 'this', 'done', 'surface', 'aminomethylation', 'preformed', 'cross-linked', 'commercially', 'available', 'polystyrene', 'beads', 'such', 'bead', 'usually', 'range', 'size', '100', '400', 'mesh', '(in)', 'diameter', 'functionalized', 'surface', 'reactions', 'surface', 'grafts', 'this', 'yielded', 'materials', 'lower', 'number', 'functional', 'groups', 'majority', 'accessible', 'lee', 'coworkers', 'utilized', 'beads', 'solid', 'peptide', 'synthesis', '7.', 'lee', 'coworkers', 'also', 'prepared', 'beads', 'n-heterocyclic', 'carbene', 'ligands', 'located', 'outer', 'shell', 'in', 'preparation', 'used', 'beads', 'formed', 'suspension', 'polymerization', '8', '9.', 'fre´chet', 'coworkers', '7', 'reported', 'variation', 'strategy', 'attaching', 'functional', 'groups', 'cross-linked', 'polystyrene', 'interior', 'soluble', 'star', 'polymer', 'matrix', 'in', 'process', 'catalytic', 'groups', 'core-conﬁned', 'use', 'low', 'molecular', 'weight', 'macroinitiators', 'form', 'surface', 'ﬁnal', 'polymer', 'presumably', 'allows', 'using', 'simultaneous', 'incompatible', 'reagents', 'like', 'acids', 'bases', 'physically', 'isolated', 'one', 'another', 'lu', 'toy', '10', 'illustrated', 'similar', 'approach', 'showing', 'core-functionalized', 'star', 'polymers', 'prepared', 'form', 'sulfonic', 'acid-functionalized', 'core', 'material', 'n', 'o', 'n', '1', 'styrene', 'divinylbenzene', '125', 'oc', '2', 'koh', '3', 'h', 'core', 'functionalized', 'star', 'polymer', 'o', 'o', 's', 'o', 'there', 'drawback', 'however', 'using', 'polymer', 'described', 'many', 'functional', 'groups', 'end', 'imbedded', 'inside', 'resin', 'access', 'requires', 'type', 'solvents', 'thoroughly', 'swell', 'resin', 'many', 'polar', 'solvents', 'however', 'fail', 'swell', 'cross-linked', 'polystyrene', 'adequately', 'yet', 'may', 'required', 'speciﬁc', 'reactions', 'that', 'led', 'modiﬁcations', 'use', '698', '10', 'polymeric', 'materials', 'special', 'applications', 'polar', 'cross-linking', 'materials', 'like', 'oligomers', 'glycols', 'one', 'example', 'cross-linked', 'polystyrene-', 'poly(ethylene', 'glycol)', 'composite', 'known', 'tentagel', '10.', 'it', 'illustrated', 'follows', 'x', 'z', 'x', 'z', 'ho', 'o', 'h', 'n', 'x', 'cl', 'x', 'o', 'oh', 'n', 'another', 'example', 'work', 'bradley', 'coworkers', 'incorporated', 'short', 'oligomer', '(polyethyl-', 'ene', 'glycol)', 'groups', 'backbone', 'cross-linked', 'polystyrene', '11.', 'the', 'oligomer', 'poly', '(ethylene', 'glycol)', 'preparation', 'also', 'acts', 'spacer', 'separate', 'polystyrene', 'backbone', 'locations', 'reactions', 'the', 'material', 'used', 'efﬁciently', 'solid', 'phase', 'peptide', 'synthesis', 'toy', 'et', 'al', '12', 'demonstrated', 'replacing', 'divinyl', 'benzene', 'ﬂexible', 'compounds', 'usually', 'increases', 'mechanical', 'stability', 'allows', 'absorb', 'solvent', 'in', 'addition', 'cross-linked', 'compounds', 'contain', 'oligomers', 'materials', 'ethylene', 'glycol', 'compatibility', 'polar', 'solvents', 'increases', 'subsequently', 'janda', 'coworkers', 'used', 'polytetrahydrofuran', 'sus-', 'pension', 'polymerization', 'styrene', 'prepare', 'general', 'solid', 'support', 'resins', 'organic', 'syntheses', '13.', 'susspension', 'polymerization', 'o', 'o', 'n', 'r', 'r', 'o', 'x', 'z', 'n', '13595', 'o', 'z', 'the', 'material', 'became', 'known', 'janda', 'gel', '13.', 'the', 'preparation', 'gel', 'n', 'equal', 'one', 'illustrated', '14.', 'an', 'interesting', 'application', 'cross-linked', 'styrene', 'resins', 'similar', 'janda', 'gel', 'developed', 'kobayashi', 'coworkers', 'used', 'materials', 'encapsulate', 'metal', 'catalysts', '15.', 'the', 'technique', '101', 'polymer', 'supports', 'reagents', 'catalysts', 'drug', 'release', '699', 'used', 'ﬁrst', 'entrap', 'metal', 'catalysts', 'coacervation', 'linear', 'polystyrene', 'functionalized', 'epoxide', 'groups', 'oligo', 'poly(ethylene', 'glycol)', 'the', 'polymer', 'becomes', 'cross-linked', 'upon', 'heating', 'reaction', 'epoxide', 'groups', 'polyethylene', 'glycol', 'entrapping', 'metal', 'o', 'o', '4', 'h', 'thf', 'pd(pph3)4', '120', 'oc', '6', '90', '4', 'o', 'o', 'the', 'solid', 'catalysts', 'used', 'successfully', 'catalyze', 'various', 'reactions', 'like', 'suzuki–miyura', 'reaction', '15,', 'aminocarbonylation', 'amidation', 'heck', 'reactions', '16.', 'polymers', 'containing', 'scandium', 'triﬂate', 'ruthenium', 'platinum', 'gold', 'prepared', 'perform', 'mukayama', 'aldol', 'alcohol', 'sulﬁde', 'oxidation', 'hydrogenation', 'indole', 'syntheses', '17.', 'incorporation', 'metals', 'like', 'palladium', 'improved', 'incorporating', 'phosphine', 'ligands', 'polymer', 'one', 'ligand', 'illustrated', 'follows', '19:', 'hoveyda', 'schrock', 'coworkers', '19', 'reported', 'using', 'chiral', 'cross-linking', 'compounds', 'immobilized', 'heterogeneous', 'polystyrene', 'resins', 'the', 'chiral', 'moiety', 'used', 'ligand', 'asymmetric', 'catalyses', 'in', 'one', 'application', 'used', 'material', 'prepare', 'recyclable', 'chiral', 'molybdenum', 'oleﬁn', 'metathesis', 'catalyst', 'used', 'enantioselective', 'ring', 'opening', 'ring-', 'closing', 'metathesis', 'reactions', 'the', 'material', 'illustrated', 'follows', 'p', 'o', 'o', 'o', 'o', '700', '10', 'polymeric', 'materials', 'special', 'applications', 'the', 'products', 'isolated', 'possessed', 'slightly', 'lower', 'enantiomeric', 'excess', 'obtained', 'corresponding', 'small', 'molecule', 'catalyst', 'in', 'similar', 'manner', 'sellner', 'et', 'al', '20', 'prepared', 'variety', 'polystyrene', 'beads', 'embedded', 'aaaa-tertiaryl-l3-dioxolane-45-dlmethanol', 'groups', 'they', 'subsequently', 'used', 'form', 'titanium', 'lewis', 'acid', 'addition', 'bu2zn', 'immobilized', 'catalyst', 'recyclable', 'chiral', 'benzaldehyde', 'they', 'also', 'reported', 'preparations', 'various', 'cross-linkers', 'containing', 'tertiary', 'aryl', 'groups', 'polymerizable', 'carbon-carbon', 'double', 'bonds', '20.', 'itsuno', 'et', 'al', '21', 'synthesized', 'cross-linked', 'polymer', 'support', 'chiral', '12-diamine', 'enantioselective', 'ruthenium', 'transfer', 'hydrogenation', 'catalysis', 'aromatic', 'ketones', 'a', 'variation', 'process', 'functionalize', 'polystyrene', 'incorporating', 'groups', 'like', 'hydroxy', 'acetoxy', 'nitrile', 'onto', 'backbone', 'polystyrene', 'copolymerization', 'monomers', '22.', 'following', 'two', 'examples', 'o', 'o', 'oh', 'ho', 'ch2cn', 'so', 'o', 'o', 'similar', 'work', 'frechet', 'grafted', 'functional', 'groups', 'heterogeneous', 'polystyrene', '(see', 'above)', 'hodges', 'used', 'living', 'free-radical', 'polymerization', 'prepare', 'referred', 'rasta', 'resin', '23.', 'the', 'merriﬁeld', 'resin', 'ﬁrst', 'functionalized', 'reduced', 'tempo', 'form', 'resin', 'core', 'with', 'help', 'rhodamine', 'dye', 'conﬁrmed', 'grafts', 'associated', 'functional', 'groups', 'located', '101', 'polymer', 'supports', 'reagents', 'catalysts', 'drug', 'release', '701', 'exterior', 'resin', 'beads', '2.', 'the', 'product', 'used', 'free-radical', 'polymerization', 'various', 'styrene', 'monomers', '4.', 'it', 'also', 'shown', 'resins', 'could', 'prepared', 'efﬁciently', 'using', 'microwave-initiated', 'polymerization', '22.', 'fontaine', 'extended', 'concept', 'using', 'atom', 'transfer', 'radical', 'polymerization', 'initiators', 'attach', 'poly(2-vinyl-44-dimethyl-5-oxazollne)', 'form', 'grafts', 'efﬁcient', 'nucleophilic', 'scavenging', '25', '26.', 'preparation', 'rasta', 'resin', 'illustrated', 'follows', 'cl', 'merrifield', 'resin', 'n', 'o', 'na', 'si', 'h', 'n', 'o', 'n', 'o', 'n', 'si', 'h', 'nco', 'n', 'o', 'n', 'nco', '10112', 'support', 'materials', 'other', 'than', 'polystyrene', 'barett', 'et', 'al', '27', 'well', 'hanson', 'coworkers', '28', 'used', 'low', 'molecular', 'weight', 'polyisobutylene', 'form', 'polyisobutylene-supported', 'catalysts', 'p', 'n', '3', 'h', 'n', 'o', 'n', 'pd', 't', 'fa', 's', 's', 'och3', 'och3', 'an', 'advantage', 'using', 'polyisobutylene', 'solubility', 'variety', 'nonpolar', 'solvents', 'ambient', 'temperatures', 'also', 'terminal', 'double', 'bond', 'polyisobutylene', 'readily', 'transformed', 'various', 'useful', 'groups', 'attachment', 'catalysts', 'reagents', 'thus', 'polyisobutylene', 'used', 'catalyst', 'support', 'anchoring', 'sulfur-cation-sulfur', 'pd(ii)', 'shown', 'these', 'catalysts', 'used', 'perform', 'allylic', 'substitution', 'heck', 'sonogashira', 'reactions', '26.', 'polyisobutylene', 'also', 'used', 'support', 'chiral', 'bisoxazoline', 'ligands', 'copper-catalyzed', 'cyclopropanation', 'reactions', '27.', 'other', 'uses', '702', '10', 'polymeric', 'materials', 'special', 'applications', 'included', 'catalysis', 'self-separating', 'atom', 'transfer', 'radical', 'polymerization', '28,', 'rcm', 'catalysis', '29,', 'catalysis', 'chromium-based', 'polycarbonate', 'polymerization', '30.', 'it', 'important', 'note', 'polyisobutylene-supported', 'catalysts', 'exhibited', 'close', 'similar', 'activity', 'unanchored', 'counterparts', 'in', 'addition', 'readily', 'recovered', 'reused', 'without', 'appreciable', 'loss', 'activity', 'na´jera', 'coworkers', '34', 'used', 'commercial', 'poly(styrene-alt-maleic', 'anhydride)', 'copolymer', 'support', 'material', 'the', 'copolymer', 'ﬁrst', 'reacted', 'hydroxylamine', 'converted', 'polymer-supported', 'n-hydroxy', 'succinimide', 'the', 'product', 'used', 'peptide-coupling', 'reactions', 'low', 'levels', 'racemization', 'reported', 'addition', 'hexane', 'allowed', 'removal', 'polymeric', 'material', 'ﬁltration', 'o', 'o', 'o', 'n', 'nh2oh', 'o', 'oh', 'n', 'o', 'n', 'the', 'polymeric', 'imide', 'could', 'reacted', 'primary', 'amines', 'ammonia', 'form', 'ammonium', 'salts', 'subsequent', 'reactions', 'carboxylic', 'acid', 'presence', 'coupling', 'reagent', 'it', 'could', 'converted', 'amides', 'functionalized', 'uranium', 'salt', 'use', 'polymer-supported', 'peptide', 'coupling', 'in', 'addition', 'anhydride', 'also', 'reacted', 'di(2-pyrldyl)methylamine', 'formed', 'recoverable', 'palladium', 'catalyst', 'cross-coupling', 'reactions', 'could', 'take', 'place', 'water', 'o', 'o', 'o', 'n', 'nh2', 'n', 'n', 'n', 'o', 'o', 'n', 'n', 'n', 'cl', 'pd', 'cl', 'na2pdcl4', 'n', 'o', 'o', 'n', 'n', 'n', '101', 'polymer', 'supports', 'reagents', 'catalysts', 'drug', 'release', '703', 'acrylic', 'polymers', 'support', 'materials', 'various', 'acrylic', 'polymers', 'copolymers', 'also', 'utilized', 'support', 'materials', 'for', 'instance', 'copolymer', 'ﬂuoroacrylate', 'monomers', 'prepared', 'bergbreiter', 'coworkers', 'form', 'support', 'material', 'selectively', 'soluble', 'ﬂuorinated', 'solvents', '28–32.', 'o', '10', 'o', '1', 'o', 'o', 'o', 'o', 'n', 'f', 'f', '7', 'f', 'f', 'f', '10', '1', 'n', 'o', 'o', 'o', 'o', 'n', 'f', 'f', '7', 'f', 'f', 'f', 'pph3', 'nh', '3', 'o', 'o', '10', '1', 'n', 'o', 'o', 'f', 'f', '7', 'cf3', 'rhcl', '3', 'the', 'resulting', 'polymer', 'converted', 'rhodium', 'hydrogenation', 'catalyst', 'o', 'n', 'o', 'o', 'o', '10', '1', 'n', 'o', 'o', 'f', 'f', '7', 'cf3', 'pph3', 'h', 'n', '3', 'o', 'o', '10', '1', 'n', 'o', 'o', 'f', 'f', '7', 'cf3', 'h2n(ch2)3pph3', 'clrh(ch2ch2)2i2', 'numerous', 'papers', 'also', 'published', 'use', 'polyacrylamide', 'copolymers', 'support', 'materials', 'following', 'example', '33–36.', 'o', '17', 'n', 'h', '10', 'o', '1', 'n', 'h', 'h', 'n', '5', 'o', 's', 'och3', 'pd', 'oac', 's', 'och3', 'bergbreiter', 'reported', 'using', 'material', 'mechanistic', 'studies', 'heck', 'reaction', '33.', '704', '10', 'polymeric', 'materials', 'special', 'applications', 'polyether-based', 'supports', 'tentagel', 'described', 'wide', 'use', 'today', 'solid', 'phase', 'syntheses', 'polyethylene', 'glycol', 'also', 'attached', 'various', 'polymers', 'form', 'support', 'resins', 'for', 'instance', 'fre´chet', 'coworkers', '37', 'used', 'cross-linked', 'methacrylate', 'esters', 'ethylene', 'glycol', 'oligomers', 'suspension', 'polymerization', 'synthesize', 'hydroxyl', 'group', 'functionalized', 'beads', 'these', 'beads', 'swell', 'well', 'variety', 'polar', 'solvents', 'another', 'example', 'grubbs', 'attaching', 'ruthenium', 'metathesis', 'catalyst', 'polyethylene', '38:', 'o', 'o', 'o', 'n', 'n', 'n', 'mes', 'mes', 'cl', 'cl', 'ru', 'o', '1012', 'special', 'gels', 'drug', 'release', 'metyjaszewski', 'et', 'al', 'reported', '39', 'preparation', 'nanostructured', 'hybrid', 'hydrogels', 'tissue', 'engi-', 'neering', 'drug', 'delivery', 'applications', 'they', 'incorporate', 'poly(oligo', '(ethylene', 'oxide)', 'monomethyl', 'ether', 'methacrylate)', 'nanogels', 'size', '100–200', 'nm', 'larger', '3d', 'matrix', 'hyaluronic', 'acid', 'gel', 'the', 'macroscopic', 'hydrogel', 'acts', '3d', 'matrix', 'regulator', 'drug', 'release', 'in', 'work', 'polymerizable', 'poly(oligo', '(ethylene', 'oxide)', 'monomethyl', 'ether', 'methacrylate)', 'nanogels', 'cova-', 'lently', 'incorporated', '3d', 'network', 'via', 'photo-polymerization', 'process', 'the', 'introduction', 'disulﬁde', 'moieties', 'polymerizable', 'groups', 'resulted', 'releasable', 'nanogels', 'cross-linked', '3d', 'network', 'reducing', 'environment', 'tan', 'fu', 'coworkers', '40', 'reported', 'preparation', 'ph-sensitive', 'biodegradable', 'polyurethane', 'drug', 'release', 'o', 'h', 'o', '2', 'o', 'h', 'n', 'h', 'n', 'o', 'n', 'o', 'oh', 'o', 'x', 'polyol', 'a', 'series', 'ph-sensitive', 'polyurethanes', 'synthesized', 'using', 'ph-sensitive', 'macrodiol', 'shown', 'tripeptide', 'chain', 'extender', 'the', 'obtained', 'polyurethane', 'number', 'average', 'molecular', 'weights', '4500–6200', 'the', 'material', 'cleaves', 'acid', 'media', '(ph', '4–6)', 'degrades', 'enzymatic', 'solution', 'mccormick', 'coworkers', '41', 'reported', 'synthesis', 'temperature-responsive', 'triblock', 'copolymer', 'consisting', 'a-methoxypropyl', '(ethylene', 'oxide)-b-poly(n-(3-aminopropyl)metha-', 'cryiamide)-b-poly(n-isopropylacrylamide)', 'at', 'room', 'temperature', 'polymer', 'hydrophilic', '101', 'polymer', 'supports', 'reagents', 'catalysts', 'drug', 'release', '705', 'exists', 'individual', 'molecule', 'aqueous', 'solution', 'increasing', 'solution', 'temperature', 'lower', 'critical', 'solution', 'temperature', 'leads', 'self-assembly', 'micelles', 'the', 'papma', 'portions', 'cross-', 'linked', 'terephthaldicarboxaldehyde', 'ph', '90', 'generate', 'shell', 'cross-linked', 'micelles', 'cleavable', 'imine', 'linkages', 'the', 'release', 'model', 'hydrophobic', 'drug', 'prednisolone', '21-acetate', 'studied', 'speciﬁc', 'ph', 'temperature', 'conditions', 'the', 'drug', 'released', 'ph', '60', 'hydrolytic', 'cleavage', 'imine', 'cross-links', 'within', 'swollen', 'micelles', 'occurred', '1013', 'utilization', 'support', 'materials', 'following', 'examples', 'prominent', 'utilizations', 'support', 'materials', '10131', 'immobilized', 'catalysts', 'it', 'may', 'accurate', 'refer', 'many', 'catalysts', 'attached', 'polymers', 'such', 'catalysts', 'inorganic', 'compounds', 'like', 'instance', 'lewis', 'acids', 'attached', 'organic', 'polymers', 'they', 'also', 'organic', 'biochemical', 'catalysts', 'perhaps', 'biggest', 'group', 'among', 'attached', 'catalysts', 'immobilized', 'enzymes', 'they', 'used', 'industrial', 'processes', 'well', 'research', 'laboratories', 'immobilization', 'often', 'improves', 'stability', 'rare', 'cases', 'activity', 'broader', 'range', 'ph', 'temperatures', 'another', 'advantage', 'elimination', 'enzyme', 'contamination', 'waste', 'streams', 'on', 'hand', 'immobilized', 'enzymes', 'often', 'less', 'active', 'immobilization', 'immobilized', 'enzymes', 'several', 'major', 'techniques', 'enzyme', 'immobilization', 'used', 'one', 'important', 'one', 'covalent', 'bonding', 'enzyme', 'support', 'material', 'such', 'attachment', 'usually', 'consists', 'reacting', 'functional', 'group', 'enzyme', 'active', 'enzymatic', 'process', 'functional', 'group', 'another', 'polymer', 'carrier', 'hydrophilic', 'groups', 'preferred', 'reactions', 'enzymes', 'aqueous', 'media', 'an', 'immobilization', 'enzyme', 'cellulose', 'azide', 'groups', '42', 'attached', 'serve', 'illustration', 'carboxymethyl', 'cellulose', 'starting', 'material', '1', 'esterification', '2', 'h2n', 'nh2', 'o', 'nano3', 'ho', 'oh', 'o', 'o', 'o', 'cooh', 'oh', 'o', 'n3', 'ho', 'o', 'o', 'o', 'enzyme', 'oh', 'o', 'h', 'n', 'oh', 'o', 'h', 'n', 'ho', 'o', 'o', 'ho', 'o', 'o', 'nh2', 'o', 'enzyme', '706', '10', 'polymeric', 'materials', 'special', 'applications', 'in', 'techniques', 'protein', 'may', 'bound', 'copolymer', 'maleic', 'anhydride', 'anhydride', 'groups', 'react', 'available', 'amine', 'groups', 'enzyme', '42.', 'other', 'techniques', 'may', 'utilize', 'cyanuric', 'chloride', 'attached', 'polysaccharides', 'immobilization', '42:', 'cl', 'o', 'n', 'n', 'n', 'n', 'enzyme', 'o', 'n', 'n', 'n', 'r', 'cl', 'n', 'r', 'enzyme', 'nh', 'n', 'r', 'oh', 'cl', 'polyaminostyrene', 'diazotized', 'treated', 'thiophosgene', 'used', 'enzyme', 'immobilization', '43:', 'n', 'n', 'n', 'n', 'h', 'enzyme', 'n', 'enzyme', 'n', 'n', 'hno2', 's', 'ccl2', 'nh2', 'n', 's', 'h', 'enzyme', 'n', 'n', 'h', 'n', 'enzyme', 'n', 'h', 's', 'because', 'epoxy', 'groups', 'carrier', 'molecules', 'capable', 'reacting', 'amine', 'carboxylic', 'acid', 'groups', 'enzymes', 'used', 'enzyme', 'immobilization', 'a', 'variation', 'technique', 'react', 'vinyl', 'monomer', 'contains', 'epoxy', 'groups', 'like', 'glycidyl', 'methacrylate', 'glycidyl', 'acrylate', 'enzyme', 'ﬁrst', 'the', 'product', 'polymerized', 'copolymerized', 'vinyl', 'portion', '44.', 'as', 'stated', 'earlier', 'many', 'cases', 'immobilization', 'enzymes', 'accompanied', 'loss', 'activity', 'in', 'instance', 'loss', 'activity', 'severe', 'a', 'special', 'technique', 'however', 'developed', '45,', 'enzyme', 'called', 'protease', 'immobilized', 'polymer', 'amino', 'glucose', 'units', 'form', 'covalently', 'bonded', 'carbohydrate-protein', 'conjugates', 'in', 'aqueous', 'solution', 'conjugated', 'enzymes', 'show', 'catalytic', 'activity', 'native', 'enzymes', 'at', 'elevated', 'temperatures', 'however', 'exhibit', 'enhanced', 'stability', 'in', 'addition', 'capable', 'catalyzing', 'reactions', 'organic', 'solvents', 'denature', 'inactivate', 'native', 'enzymes', '45.', '101', 'polymer', 'supports', 'reagents', 'catalysts', 'drug', 'release', '707', 'hiroshi', 'et', 'al', 'reported', '46', 'high-performance', 'immobilized', 'lipase', 'catalyst', 'polyester', 'synthesis', 'a', 'porous', 'polypropylene', 'found', 'good', 'support', 'immobilization', 'candida', 'antarctica', 'lipase', '(enzyme)', 'the', 'immobilized', 'lipase', 'polypropylene', 'efﬁciently', 'catalyzes', 'ring', 'opening', 'polymerization', '15-pentadecanolide', 'polycondensation', 'divinyl', 'sebacate', '18-octanediol', '10132', 'immobilized', 'nonenzymatic', 'catalysts', 'there', 'also', 'many', 'uses', 'special', 'nonenzymatic', 'polymeric', 'catalysts', 'for', 'instance', 'polymer-bound', 'crown', 'ethers', 'cryptates', 'channel', 'compounds', 'immobilized', 'act', 'polymeric', 'phase-', 'transfer', 'catalysts', 'the', 'catalytic', 'activity', 'based', 'selective', 'complex', 'formation', 'an', 'example', 'use', 'catalyze', 'polystyrene-attached', 'oxygen', 'heterocycles', '18-crown-6', 'cryptand222', 'replacements', 'bromine', 'n-octyl', 'bromide', 'iodine', 'cyanide', 'group', '47:', '9', 'n', 'c2h5', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'c8h17i', '(cn)', 'n', 'c8h17br', 'ki', '(kcn)', '9', 'n', 'c2h5', 'o', 'o', 'o', 'o', 'o', 'o', 'a', '95', 'yield', 'achieved', 'the', 'catalytic', 'activity', 'result', 'complexation', 'cations', 'results', 'increased', 'nucleophilicity', 'anions', 'interactions', 'ions', 'ion', 'pairs', 'vinyl', 'polymers', 'crown', 'ethers', 'shown', 'considerably', 'efﬁcient', 'interactions', 'unattached', 'crown', 'ethers', '48.', 'also', 'studies', 'diazo-471316-tetraoxacyclooctadecane', 'bound', 'polyacrylamide', 'gel', 'show', 'enhancement', 'cationic', 'complexation', 'compared', 'ligands', 'bound', 'polymers', '49.', 'on', 'hand', 'polymer-bound', 'crown', 'ethers', 'offer', 'advantage', 'unbound', 'ligands', 'koening–knorr', 'reaction', '50.', 'the', 'catalytic', 'properties', 'solute-binding', 'capabilities', 'pendant', 'crown', 'ethers', 'glyme', 'ligands', 'apparently', 'depend', 'spacing', 'ligands', 'they', 'also', 'depend', 'upon', 'structure', 'length', 'chains', 'connecting', 'ligand', 'bound', 'ions', 'solvent', '51.', 'in', 'low', 'polarity', 'solvents', 'ligands', 'activate', 'anionic', 'reactants', 'modiﬁcation', 'ion', 'pair', 'structures', 'ion', 'exchange', 'resins', 'used', 'long', 'time', 'catalyze', 'reactions', 'this', 'mentioned', 'instance', 'chap', '7', 'section', 'epoxy', 'resins', 'basic', 'ion', 'exchange', 'resins', 'also', 'used', 'condensations', 'furfural', 'aliphatic', 'aldehydes', '52.', 'a', 'still', 'different', 'kind', 'polymeric', 'catalyst', 'one', 'pendant', 'photosensitizers', 'attached', 'to', 'effective', 'sensitizer', 'portion', 'must', 'absorb', 'light', 'undergo', 'transition', 'singlet', 'longer', 'lived', 'triplet', 'state', '(see', 'sect', '104)', 'it', 'must', 'without', 'emitting', 'radiation', 'activate', 'substrate', 'molecule', 'return', 'ground', 'state', 'some', 'dyes', 'function', 'manner', 'an', 'example', 'rose', 'bengal', 'when', 'attached', 'cross-linked', 'polystyrene', '53,', 'used', 'produce', 'singlet', 'oxygen', '708', '10', 'polymeric', 'materials', 'special', 'applications', 'the', 'excited', 'oxygen', 'turn', 'hydroperoxidizes', 'oleﬁns', 'the', 'structure', 'polymer-bound', 'rose', 'bengal', 'illustrated', 'follows', 'i', 'o', 'o', 'o', 'cl', 'cl', 'cl', 'cl', 'o', 'i', 'i', 'polymer', 'bound', 'rose', 'bebgal', 'i', 'ona', 'the', 'photosensitized', 'hydroperoxidation', 'reaction', 'oleﬁns', '53', 'shown', 'follows', 'hν', 'o', 'o', 'p', 'rose', 'bengal', 'o', 'oh', 'metathesis', 'catalysts', 'including', 'grubbs', 'ruthenium', 'catalyst', '(see', 'chap', '5)', 'hard', 'separate', 'reaction', 'products', 'chemists', 'therefore', 'sought', 'ways', 'overcome', 'problem', 'immobilizing', 'catalyst', 'various', 'supports', 'buchmeister', 'recently', 'reviewed', 'various', 'polymer-supported', 'metathesis', 'catalysts', '54.', 'a', 'reported', 'early', 'grubbs', 'described', 'phosphine-derivatized', 'polystyrene-', 'supported', 'catalyst', '55', 'ibual', 'cl2', 'oc', 'co', 'w', 'co', 'similarly', 'grubbs', 'type', 'ruthenium', 'catalyst', 'phosphine-derivatized', 'polystyrene', '56', 'illustrated', 'follows', 'cl', 'cl', 'pr2', 'n', 'pr2', 'n', 'ru', 'this', 'catalyst', 'however', 'turned', 'considerably', 'less', 'active', 'ring', 'opening', 'metathesis', 'polymerization', 'norborn-2-ene', 'metathesis', 'polymerization', 'cis-2-pentene', 'better', 'results', 'obtained', 'ruthenium-based', 'metathesis', 'type', 'catalyst', 'immobilized', 'via', 'n-heterocy-', 'clic', 'carbene', '57.', 'additional', 'improvements', 'performance', 'obtained', 'catalyst', 'immobilized', 'monolithic', 'silica', 'rods', '101', 'polymer', 'supports', 'reagents', 'catalysts', 'drug', 'release', '709', 'n', 'o', 'o', 'n', 'n', 'r', 'r', 'cl', 'cl', 'ru', 'pcy3', '10133', 'immobilized', 'reagents', 'an', 'example', 'reagent', 'use', 'immobilized', 'triphenyl', 'methyl', 'lithium', 'transfer', 'charge', 'form', 'new', 'carbanion', 'li', 'li', 'n', 'the', 'carbanions', 'turn', 'react', 'ester', 'groups', 'yielded', 'desired', 'product', '710', '10', 'polymeric', 'materials', 'special', 'applications', 'the', 'yield', 'reaction', 'found', '90.', 'this', 'compares', 'favorably', '27', 'yield', 'obtainable', 'without', 'polymeric', 'supports', '58.', 'another', 'example', 'benzoylation', 'd-butyrolactone', 'when', 'acylation', 'reaction', 'carried', 'ester', 'solution', 'ester', 'enolate', 'must', 'completely', 'formed', 'acylating', 'agent', 'introduced', 'otherwise', 'acylating', 'agent', 'reacts', 'base', 'instead', 'during', 'period', 'however', 'ester', 'enolate', 'undergo', 'self-condensation', 'using', 'polymer-attached', 'reagents', 'how-', 'ever', 'benzoylation', 'd-butyrolactone', 'similar', 'compounds', 'achieved', '95', 'yields', '59', 'o', 'o', 'li', 'o', 'o', 'o', 'o', 'li', 'li', 'o', 'o', 'li', 'o', 'o', 'o', 'o', 'by', 'comparison', 'reaction', 'performed', 'solution', 'without', 'support', 'yield', '31', '59.', 'patchornik', 'termed', 'use', 'supports', 'manner', '“wolf', 'lamb', 'chemistry”', '58.', 'it', 'appears', 'unlikely', 'organic', 'reactions', 'beneﬁt', 'use', 'polymeric', 'reagents', 'catalysts', 'nevertheless', 'reagents', 'catalyst', 'appear', 'promising', 'many', 'applications', 'the', 'fact', 'polymer-bound', 'reagents', 'catalyst', 'expensive', 'however', 'may', 'preclude', 'wider', 'industrial', 'utilization', 'except', 'special', 'cases', '102', 'electricity-conducting', 'polymers', 'most', 'polymeric', 'materials', 'usually', 'know', 'insulators', 'yet', 'conducting', 'polymers', 'ﬁrst', 'prepared', '1862', 'letherby', 'formed', 'polyaniline', '60', 'electro', 'polymerization', 'it', 'past', '25–30', 'years', 'however', 'organic', 'polymers', 'capable', 'conducting', 'electric', 'current', '102', 'electricity-conducting', 'polymers', '711', 'created', 'considerable', 'interest', '61.', 'many', 'polymeric', 'materials', 'synthesized', 'since', 'at', 'present', 'completely', 'understood', 'mechanism', 'electric', 'current', 'passes', 'we', 'know', 'however', 'conductive', 'polymers', 'intrinsically', 'conducting', 'polymers', 'according', 'inzelt', '62,', 'similar', 'one', 'respect', 'they', 'consist', 'extended', 'p-conjugated', 'systems', 'namely', 'alternating', 'single', 'double', 'bonds', 'along', 'chain', 'one', 'early', 'known', 'conductive', 'polymers', 'polysulfurnitride', '(sn)x', 'inorganic', 'material', 'tends', 'explosive', 'becomes', 'superconducting', '03', 'k', '63.', 'since', 'many', 'conductive', 'polymers', 'evolved', 'the', 'investigated', 'ones', 'appear', 'polyacetylene', 'polyaniline', 'polypyrrole', 'polythiophene', 'poly(phenylene', 'sulﬁde)', 'poly(phenylene', 'vinylene)', 'derivatives', 'many', 'derivatives', 'materials', 'similar', 'ones', 'also', 'reported', 'the', 'mechanism', 'electro', 'polymerization', 'used', 'form', 'many', 'conductive', 'polymers', 'also', 'still', 'fully', 'understood', 'according', 'diaz', 'et', 'al', '64,', 'process', 'involves', 'sequence', 'coupling', 'steps', 'step', 'activated', 'two', 'species', 'the', 'polymer-', 'forming', 'process', 'requires', 'two', 'electron', 'per', 'step', 'it', 'also', 'includes', 'partial', 'oxidation', 'polymer', 'thus', 'polymer', 'formation', 'polymer', 'oxidation', 'occur', 'simultaneously', 'diaz', 'proposed', 'chain', 'propagation', 'process', 'polymer', 'formation', 'although', 'still', 'accepted', 'many', 'mechanism', 'questioned', 'heinze', 'et', 'al', '65', 'suggest', 'process', 'polymer', 'formation', 'consists', 'oligomer', 'formation', 'oxidation', 'followed', 's-coupling', 'chains', 'all', 'known', 'conducting', 'polymers', 'backbones', 'contiguous', 'sp2-hybridized', 'carbon', 'centers', 'in', 'centers', 'one', 'valence', 'electron', 'resides', 'pz', 'orbital', 'it', 'orthogonal', 'three', 'sigma', 'bonds', 'when', 'material', 'oxidized', 'reduced', 'removes', 'delocalized', 'electrons', 'the', 'electrons', 'obtain', 'high', 'mobility', 'as', 'result', 'conjugated', 'p-orbitals', 'form', 'one-dimensional', 'electronic', 'band', 'the', 'electrons', 'within', 'band', 'become', 'mobile', 'partially', 'emptied', 'depending', 'upon', 'chemical', 'structure', 'polymer', 'also', 'self-oxidizing', 'reducing', 'structural', 'disorder', 'polymer', 'molecules', 'interferes', 'electron', 'mobility', 'thus', 'instance', 'polyacetylene', 'exhibits', 'conductivity', '01–10', 'kscm', 'stretch-orienting', 'polya-', 'cetylene', 'process', 'aligns', 'chains', 'removes', 'much', 'disorder', 'increases', 'conductivity', '80', 'kscm', '66.', 'polyacetylene', 'shaped', 'silvery', 'looking', 'ﬁlm', 'the', 'polymer', 'thermodynamically', 'stable', 'trans', 'form', 'converts', 'cis', 'trans', 'heated', '150(cid1)c', 'partial', 'oxidation', 'ﬁlm', 'iodine', 'materials', 'transforms', 'increases', 'conductivity', '109-fold', 'the', 'process', 'transforming', 'polymer', 'conductive', 'form', 'chemical', 'oxidation', 'reduction', 'called', 'doping', 'two', 'types', 'polyacetylene', 'doping', 'possible', 'ðchþn', 'þ', '12i2', 'ðchþ(cid3)', 'n', 'ði3þ(cid4)', '033', 'oxidative', 'doping', '(p', '-', 'typeþ', 'ðchþn', 'þ', 'xna', 'ðna(cid3)þx½ðchþ(cid4)', 'n', '(cid5)', 'reductive', 'doping', '(n', '-', 'typeþ', 'the', 'doping', 'process', 'reversed', 'conductive', 'polymers', 'undoped', 'applying', 'electrical', 'potential', 'it', 'causes', 'dopant', 'ions', 'diffuse', 'structure', 'improvements', 'preparations', 'polyacetylene', 'came', 'several', 'developments', 'one', 'use', 'metathesis', 'polymerization', 'cyclooctatetraene', 'catalyzed', 'titanium', 'alkylidine', 'complex', 'the', 'product', 'improved', 'conductivity', 'intractable', 'unstable', 'by', 'attaching', 'substituents', 'possible', 'form', 'soluble', 'stable', 'materials', 'deposited', 'solution', 'various', 'substrates', 'substitution', 'however', 'lowers', 'conductivity', 'this', 'attributed', 'though', 'still', '712', '10', 'polymeric', 'materials', 'special', 'applications', 'steric', 'factors', 'introduced', 'substituents', 'force', 'double', 'bonds', 'polymeric', 'chains', 'twist', 'coplanarity', '67.', 'a', 'family', 'substituted', 'polyacetylenes', 'prepared', '68', 'actually', 'formed', 'ethynylpyridines', 'polymerization', 'reaction', 'takes', 'place', 'spontaneously', 'quaternization', 'process', 'r', 'x', 'n', 'x', 'n', 'r', 'n', 'x', 'r', 'x', 'bromine', 'iodine', 'like', 'substituted', 'polyacetylenes', 'materials', 'fairly', 'stable', 'air', 'soluble', 'polar', 'solvents', 'also', 'water', 'the', 'conductivity', 'polymers', 'improved', 'previously', 'reported', 'substituted', 'polyacetylenes', 'within', 'range', 'semiconductors', 'preparation', 'highly', 'conductive', 'polyacetylene', 'achieved', 'ziegler–natta', 'catalyst', 'used', 'shirakawa', 'aged', 'silicone', 'oil', '150(cid1)c', 'it', 'believed', 'reaction', 'results', 'formation', 'polymers', 'less', 'defects', 'structures', 'the', 'conductivity', 'materials', 'doped', 'actually', 'approaches', 'copper', '62.', 'considerable', 'progress', 'achieved', 'development', 'catalyst', 'systems', 'living', 'polymeri-', 'zation', 'various', 'substituted', 'acetylenes', 'last', '10', '15', 'years', '69.', 'nowadays', 'available', 'single-component', 'catalysts', 'based', 'stable', 'carbene', 'complexes', 'multicomponent', 'catalysts', 'based', 'moocl4', 'wocl4', 'operating', 'metathesis', 'mode', 'well', 'rh(diene)', 'complexes', 'operating', 'ziegler–natta', 'mode', 'for', 'instance', 'living', 'polymerization', 'phenyl', 'acetylene', 'synthesis', 'end-', 'functionalized', 'poly(phenyl', 'acetylene)', 'using', 'rh-based', 'catalyst', 'systems', 'rhci(nbd)2ar2c¼c', '(ph)lipph3', '(ar', '¼', 'ph', '4-me2-nc6h4)', 'reported', '70.', 'also', 'use', '(triphenyl', 'vinyl)lithium', 'functional', 'groups', 'dimethylamino', 'groups', 'results', 'formation', 'end-functionalized', 'poly(phenyl', 'acetylene)s', 'quantitatively', 'contained', 'functional', 'groups', 'initiating', 'chain', 'end', 'the', 'polymerization', 'phenyl', 'acetylene', 'catalyst', 'prepared', 'rhcl(norbornadiene)2', 'ph2c¼c(ph)li', 'pph3', 'proceeds', 'smoothly', 'benzene', 'give', 'quantitatively', 'yellow', 'polymer', 'number', 'average', 'molecular', 'weight', '5400', 'polydispersity', 'ratio', '114', 'the', 'reaction', 'illustrated', 'follows', 'rhcl', '(nbd)2', 'ph2cc', '(ph)', 'li', 'pph3', 'pph3', 'nhc', 'cph', 'rh', 'ph', 'ph', 'ph', 'pph3', 'ph', 'ph', 'n', 'ph', 'rh', 'ph', 'masuda', 'colleagues', 'reported', '71', 'synthesized', 'poly(anthryacetylenes)-bearing', 'oligo', 'oxyethylene', 'units', 'using', 'transition', 'metal', 'catalyst', 'wc16', '30', '34', 'yields', 'the', 'polymers', 'black', 'solids', 'these', 'polymers', 'soluble', 'chloroform', 'tetrahydrofuran', 'acetone', 'etc,', 'insoluble', 'alcohols', 'aliphatic', 'hydrocarbons', 'etc', 'the', 'uv–vs', 'spectra', 'polymers', 'showed', 'absorption', 'maxima', 'band', 'edges', 'around', '570', '750', 'nm', 'respectively', 'indicating', 'polymer', 'chains', 'possess', 'highly', 'extended', 'conjugation', 'these', 'polymers', 'exhibited', 'blue', 'emission', '102', 'electricity-conducting', 'polymers', '713', '(emission', 'maxima', '470', 'nm)', 'upon', 'photo-excitation', '380', 'nm', 'one', 'polymers', '(n', '¼', '4)', 'showed', 'fairly', 'large', 'ionic', 'conductivity', '(41', '(cid6)', '10', 'scm)', '80(cid1)c', 'upon', 'doping', 'li(cf3so2)2n', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'in', '1979', 'demonstrated', 'polypyrrole', 'formed', 'ﬁlm', 'electrochemical', 'oxidative', 'polymerization', 'pyrrole', 'monomer', 'acetonitrile', 'the', 'polymers', 'form', 'surface', 'electrode', 'peeled', 'ﬂexible', 'shiny', 'blue-black', 'ﬁlms', 'subsequently', '1982', 'shown', 'thiophene', 'also', 'electro', 'polymerized', 'oxidatively', 'anode', 'the', 'method', 'allows', 'control', 'oxidative', 'potential', 'polymerization', 'yielding', 'doped', 'ﬁlms', 'optimized', 'polymer', 'properties', 'both', 'polypyrrole', 'polythiophene', 'differ', 'polyacetylene', 'form', 'polymerization', 'doped', 'form', 'stable', 'air', 'they', 'however', 'less', 'conductive', 'doped', 'polyacetylene', 'the', 'exact', 'structures', 'polypyrrole', 'polythiophene', 'prepared', 'way', 'still', 'fully', 'established', 'the', 'process', 'oxidative', 'polymeriza-', 'tion', 'involves', 'reactive', 'cation', 'radical', 'intermediates', 'much', 'coupling', 'heterocyclic', 'rings', 'together', '2', '5', 'positions', 'x-ray', 'photoelectron', 'spectroscopy', 'shows', 'polypyrrole', 'formed', 'manner', '30', 'linkages', '2', '5', 'positions', 'they', 'might', '2', '3', 'positions', 'this', 'introduces', '“defects”', 'hypothetically', 'ideal', 'chain', 'reduces', 'conjugation', 'length', 'conductivity', 'the', 'ﬂexible', 'ﬁlms', 'polypyrrole', 'form', 'upon', 'electrochemical', 'oxidation', 'stable', 'air', 'water', 'may', 'also', 'heated', '200(cid1)c', 'without', 'much', 'change', 'electrical', 'properties', 'the', 'oxidative', 'polymerization', 'pyrrole', 'illustrated', 'follows', 'oxidation', 'r4ncio4', 'n', 'h', 'n', 'h', 'n', '(cio4)033', '2h', '2', 'e', 'a', 'regioselective', 'synthesis', 'highly', 'conductive', 'poly(3-alkylthiophene)s', 'reported', '72.', 'following', 'synthetic', 'procedure', 'used', 'r', 'ni', 'catallyst', 'brmg', 'br', 's', 'r', 's', 's', 'r', 'n', 's', 'r', 'the', 'iodine', 'doped', 'unoriented', 'poly(3-dodecyl', 'thiophene)s', 'exhibits', 'average', 'conductivity', '600', 's', 'cm', 'maximum', 'conductivity', '1000', 'scm', '72.', '714', '10', 'polymeric', 'materials', 'special', 'applications', 'more', 'recently', 'mccullough', 'coworkers', '73', 'reported', 'using', 'grignard', 'reaction', 'synthesize', 'poly(3-dodecyl', 'thiophene)', 'br', 'br', 's', 'brh', '1rmgcl', '2', 'ni(dppp)cl2', 'c12h25', 'c12h25', 'brh', 'n', 'c12h25', 'the', 'molecular', 'weights', 'products', 'found', 'range', '10000', '50000', 'earlier', 'mccullough', 'et', 'al', '74', 'reported', 'developed', 'end', 'group', 'functionalization', 'method', 'enables', 'synthesis', 'many', 'well-deﬁned', 'block', 'copolymers', 'form', 'nano', 'wires', 'high', 'electrical', 'conductivity', 'they', 'claim', 'discovered', 'nickel-initiated', 'regioregular', 'poly-', 'merization', 'alkyl-thiophenes', 'proceeds', 'chain', 'growth', 'mechanism', 'they', 'also', 'observed', 'degree', 'polymerization', 'poly(alkyl-thiophenes)', 'increases', 'conversion', 'predicted', 'molar', 'ratio', 'monomer', 'nickel', 'initiator', 'on', 'basis', 'experimental', 'results', 'concluded', 'nickel-initiated', 'cross-coupling', 'polymerization', 'essentially', 'living', 'system', 'low', 'polydispersities', 'irvin', 'et', 'al', '75', 'reported', 'forming', 'poly(34-diﬂuoro', 'thiophene)', 'anodic', 'polymerization', 's', 'si', 'si', 's', 'n', 'f', 'f', 'f', 'f', 'because', 'electron-poor', 'nature', 'monomer', 'electro', 'deposition', 'could', 'accomplished', 'strong', 'lewis', 'acid', 'bf3eto', 'mendez', 'weder', 'improved', 'considerably', 'conductivity', 'poly(34-diethylene', 'dioxy-', 'thiophene)', 'cross-linking', '76.', 'the', 'polymers', 'oxidatively', 'synthesized', 'presence', 'four', 'different', 'oxidative', 'cross-linkers', 's', 's', 's', 's', 's', 's', 's', 's', 'o', 'c8h17', 's', 'o', 'c8h17', 'the', 'cross-linking', 'compounds', 'incorporated', 'polymer', 'amounts', '05–2.', 'as', 'result', 'maximum', 'case', 'conductivity', 'increased', 'much', '36.', '102', 'electricity-conducting', 'polymers', '715', 'polyaniline', 'ﬁrst', 'prepared', 'beginning', 'last', 'century', 'several', 'oxidation', 'states', 'known', 'the', 'conductivity', 'color', 'material', 'vary', 'progressively', 'oxidation', 'only', 'one', 'form', 'however', 'known', 'emeraldine', 'salt', 'truly', 'conducting', 'the', 'material', 'prepared', 'readily', 'electrochemical', 'chemical', 'oxidation', 'aniline', 'aqueous', 'acid', 'media', 'common', 'oxidants', 'ammonium', 'peroxydisulfate', 'used', 'flexible', 'emeraldine', 'ﬁlms', 'cast', 'solutions', 'n-methylpyrrolidone', 'made', 'conductive', 'protonic', 'doping', 'this', 'done', 'dipping', 'ﬁlms', 'acid', 'exposing', 'acid', 'vapors', 'the', 'process', 'results', 'protonation', 'imine', 'nitrogen', 'atoms', 'n', 'h', 'n', 'n', 'h', 'insulator', 'n', 'h', 'n', 'n', 'h', 'emerald', 'bas', 'insulator', 'h', 'n', 'h', 'n', 'h', 'n', 'h', 'n', 'h', 'n', 'n', 'n', 'h', 'n', 'n', 'h', 'the', 'conductivity', 'emeraldine', 'salt', 'increases', 'decrease', 'ph', 'acid', 'used', 'dope', 'in', 'respect', 'polyaniline', 'emeraldine', 'form', 'differs', 'conductive', 'polymers', 'require', 'partial', 'oxidation', 'reduction', 'doping', 'protonation', 'imine', 'nitrogens', 'sufﬁcient', 'make', 'conductive', 'material', 'a', 'different', 'method', 'synthesizing', 'polyaniline', 'reported', '77.', 'it', 'uses', 'enzyme', 'horseradish', 'peroxidase', 'presence', 'hydrogen', 'peroxide', 'polymerize', 'aniline', 'to', 'prevent', 'reactions', 'ortho', 'positions', 'phenyl', 'rings', 'yield', 'insoluble', 'branched', 'materials', 'polyelectrolyte', 'template', 'like', 'sulfonated', 'polystyrene', 'used', 'the', 'polyelectrolyte', 'aligns', 'monomers', 'dopes', 'polyaniline', 'conducting', 'form', 'forms', 'irreversible', 'complex', 'polyaniline', 'keep', 'water-soluble', '77.', 'the', 'conductivity', 'complex', 'increases', 'increasing', 'polyaniline', 'sulfonated', 'polystyrene', 'molar', 'ratios', 'conductivities', '0005', 'scm', 'obtained', 'pure', 'complex', 'increase', '015', 'scm', 'additional', 'doping', 'exposure', 'hcl', 'vapor', '77.', 'lee', 'reported', '78', 'new', 'synthesis', 'polyaniline', 'aniline', 'hydrochloride', 'monomer', 'acts', 'surfactant', 'the', 'organic', 'phase', 'serves', 'diffuse', 'water', 'away', 'water-insoluble', 'oligomers', 'as', 'result', 'chains', 'grow', 'monomer', 'radicals', 'meet', 'active', 'polymer', 'chain', 'ends', 'organic', 'aqueous', 'interfaces', 'this', 'results', 'formation', 'polymer', 'high', 'structural', 'integrity', 'this', 'enables', 'electrons', 'holes', 'travel', 'perpendicular', 'direction', '716', '10', 'polymeric', 'materials', 'special', 'applications', 'han', 'et', 'al', '79', 'reported', 'preparation', 'highly', 'conductive', 'thermally', 'stable', 'self', 'doping', 'propylthiosulfonated', 'polyanilines', 'the', 'polymer', 'illustrated', 'follows', 'h', 'n', 'h', 's', 'so3', 'n', 'h', 'o', 'o', 's', 's', 'o', 'h', 'n', 's', 'so3', 'h', 'the', 'material', 'found', 'better', 'conductor', 'sulfonated', 'polyaniline', 'thermally', 'stable', 'poly(phenyl', 'vinylene)', 'synthesized', 'several', 'routes', 'one', 'step', 'growth', 'polymer-', 'ization', '80', 'witting', 'reaction', 'pph3', 'n', 'n', 'n', 'cho', 'ohc', 'there', 'several', 'routes', 'poly(phenyl', 'vinylene)', 'one', 'metathesis', 'ring', 'opening', 'polymerization', 'pph3', 'si', 'n', 'shrock', 'catalyst', '∆', 'osi(ch3)2c4h5', 'n', 'n', 'ho', 'si', 'n', 'the', 'polymer', 'doped', 'iodine', 'acids', 'ferric', 'chloride', 'alkoxy-substituted', 'poly(phenyl', 'vinylene)', 'easier', 'oxidize', 'exhibits', 'higher', 'conductivity', '103', 'photonic', 'polymers', '717', 'natori', 'et', 'al', 'reported', '81', 'synthesis', 'copolymer', 'consists', 'p-phenylene', 'p-phenylene', 'vynelene', 'styryl', 'amine', 'n', 'n', 'orientation', 'ﬁlms', 'yields', 'large', 'increases', 'conductivity', 'thus', 'ﬁlms', 'doped', 'oriented', 'poly', '(phenylene', 'vinylene)', '81', 'strength', 'high-performance', 'polymers', 'also', 'conductivities', 'measure', 'high', '104', 'scm', 'this', 'approximately', '1000', 'times', 'greater', 'unoriented', 'ﬁlms', 'another', 'interesting', 'material', 'consists', 'doped', 'forms', 'covalently', 'linked', 'siloxane-', 'phthalocyanin', '(pc)', 'complexes', 'si(pc)on', 'in', 'polymers', 'planar', 'phthalocyanin', 'units', 'apparently', 'stacked', 'face-to-face', 'form', 'columns', 'due', 'silicon-oxygen-silicon', 'bonds', 'the', 'polymers', 'appear', 'intrinsically', 'metallic', 'systems', 'the', 'principal', 'pathways', 'conductivity', 'perpendicular', 'phthalocyanin', 'planes', 'the', 'extended', 'p–p', 'systems', 'form', 'result', 'face', 'face', 'pathway', 'electron', '67.', 'there', 'numerous', 'applications', 'conducting', 'polymeric', 'materials', 'these', 'applications', 'include', 'use', 'electronics', 'organic', 'solar', 'cells', 'convert', 'light', 'electricity', 'others', '103', 'photonic', 'polymers', 'the', 'term', 'photonic', 'polymer', 'describes', 'wide', 'variety', 'polymeric', 'materials', 'respond', 'light', 'one', 'manner', 'another', 'these', 'rearrangement', 'upon', 'irradiation', 'light', 'certain', 'wave-', 'length', 'conductivity', 'electricity', 'upon', 'irradiation', 'dimerization', 'others', 'what', 'common', 'required', 'irradiation', 'light', 'often', 'always', 'speciﬁc', 'wavelength', 'undergo', 'change', 'to', 'understand', 'mechanism', 'reactions', 'ﬁrst', 'necessary', 'look', 'nature', 'light', 'reacts', 'matter', '1031', 'the', 'nature', 'light', 'photochemical', 'reactions', 'involve', 'interactions', 'light', 'matter', 'they', 'either', 'induce', 'chemical', 'changes', 'induce', 'luminescence', 'conversions', 'electronic', 'forms', 'energy', 'heat', 'for', 'practical', 'purposes', 'photochemical', 'reactions', 'discussed', 'limited', 'take', 'place', 'presence', 'light', 'ranges', 'ultra-violet', 'infra-red', 'light', 'independent', 'source', 'travels', 'vacuum', 'velocity', 'equal', '29979', '(cid6)', '1010', 'cms', 'but', 'true', 'vacuum', 'within', 'medium', 'light', 'moves', 'velocity', 'e', 'dielectric', 'constant', 'medium', 'magnetic', 'permeability', 'c0', '¼', 'cðemþ12', '718', '10', 'polymeric', 'materials', 'special', 'applications', 'fig', '101', 'the', 'instantaneous', 'electric', 'ey', 'magnetic', 'hx', 'ﬁeld', 'strength', 'vectors', 'light', 'wave', 'function', 'position', 'along', 'axis', 'propagation', '(from', 'calvert', 'pitts', '82)', 'our', 'concept', 'light', 'also', 'consists', 'packets', 'energy', 'travel', 'waves', 'in', 'packet', 'range', 'energies', 'these', 'energies', 'cannot', 'represented', 'one', 'wavelength', 'rather', 'whole', 'spectrum', 'wavelengths', 'the', 'energy', 'particular', 'wavelength', 'wave-packet', 'discrete', 'unit', 'quantum', 'electromagnetic', 'radiation', 'described', 'terms', 'transverse', 'plane', 'wave', 'involving', 'associated', 'electric', 'magnetic', 'ﬁelds', 'experimental', 'data', 'suggest', 'electric', 'vector', 'e', 'magnetic', 'vector', 'h', 'describe', 'respective', 'ﬁeld', 'strengths', 'aligned', 'planes', 'right', 'angles', 'one', 'another', 'planes', 'perpendicular', 'direction', 'propagation', 'wave', 'this', 'illustrated', 'calvin', 'pitts', '82,', 'shown', 'fig', '101', 'a', 'convenient', 'model', 'variation', 'ﬁeld', 'strength', 'function', 'time', 'k', 'distance', 'x', 'along', 'axis', 'propagation', 'one', 'described', 'cartesian', 'coordinates', 'sinusoidal', 'functions', 'following', 'equations', 'ey', '¼', 'a', 'sin', '2pðxl', '(cid7)', 'vtþ', 'hz', '¼', 'ðemþ12a', 'sin', '2pðxl', '(cid7)', 'vtþ', 'in', 'equations', 'ey', 'electric', 'ﬁeld', 'strength', 'vector', 'lying', 'xy', 'plane', 'increasing', 'along', 'y-axis', 'hz', 'magnetic', 'ﬁeld', 'strength', 'vector', 'lying', 'xz-plane', 'increasing', 'along', 'z-', 'axis', 'a', 'amplitude', 'electric', 'vector', '(the', 'intensity', 'wave', 'proportional', 'a2)', 'e', 'dielectric', 'constant', 'magnetic', 'permeability', 'medium', 'light', 'wave', 'transported', 'in', 'vacuum', 'e', 'approximately', 'unity', 'air', 'the', 'length', 'wave', 'distance', 'adjacent', 'maxima', 'vectors', 'measured', 'instant', 'along', 'direction', 'wave', 'propagation', '(the', 'x', 'axis)', 'l', 'v', 'frequency', 'number', 'complete', 'cycles', 'vector', 'position', 'change', 'per', 'second', 'the', 'relationship', 'l', 'v', 'c', 'velocity', 'radiation', 'the', 'frequency', 'v', 'independent', 'medium', 'radiation', 'travels', 'wavelength', 'l', 'velocity', 'c', 'hand', 'depend', 'e', 'medium', 'ordinary', 'light', 'polarized', 'it', 'consists', 'many', 'electromagnetic', 'vectors', 'undulating', 'ﬁxed', 'though', 'randomly', 'oriented', 'respect', 'planes', 'when', 'light', 'polarized', 'plane', 'believed', 'waves', 'electric', 'vectors', 'oriented', 'direction', 'when', 'light', 'polarized', 'elliptically', 'believed', 'two', 'plane', 'waves', 'equal', 'wavelength', 'frequency', 'identical', 'directions', 'propagation', 'electric', 'vectors', 'perpendicular', 'one', 'another', 'phase', 'shown', 'fig', '101', 'the', 'degree', 'polarization', 'light', 'p', 'usually', 'expressed', 'equation', 'cv', '¼', 'l', 'p', '¼', 'ijj', '(cid7)', 'i', 'ijj', 'þ', 'i', '103', 'photonic', 'polymers', '719', 'i|', 'i⊥', 'represent', 'intensities', 'light', 'transmitted', 'analyzer', 'parallel', 'perpendicular', 'plane', 'maximum', 'polarization', 'ht', 'incident', 'light', 'the', 'above-described', 'model', 'incomplete', 'even', 'called', 'naive', 'mathematically', 'however', 'successfully', 'account', 'many', 'observations', 'concerning', 'light', 'theory', 'used', 'successfully', 'describe', 'many', 'phenomena', 'associated', 'optics', '1032', 'quantum-mechanical', 'description', 'light', 'in', 'order', 'better', 'understand', 'emission', 'absorption', 'light', 'molecules', 'necessary', 'look', 'quantum-mechanical', 'concept', 'nature', 'light', 'in', 'concept', 'light', 'considered', 'beam', 'photons', 'whose', 'energies', 'quantized', 'detailed', 'description', 'quantum', 'mechanics', 'spectroscopy', 'beyond', 'scope', 'book', 'here', 'presented', 'major', 'conclusions', 'necessary', 'better', 'understanding', 'light', 'in', 'quantum', 'theory', 'radiation', '93,', 'electromagnetic', 'radiation', 'considered', 'beams', 'particles', 'as', 'stated', 'called', 'photons', 'each', 'photon', 'energy', 'e', 'deﬁned', 'planck', 'relationship', 'e', '¼', 'hu', '¼', 'hcl', 'h', 'proportionality', 'constant', 'called', 'planck’s', 'constant', 'it', 'equal', '66256', '(cid6)', '10─27', 'erg', 'quantum', 'the', 'quantum', 'theory', 'explained', 'many', 'phenomena', 'associated', 'interactions', 'light', 'matter', 'one', 'photoelectric', 'effect', 'this', 'effect', 'discovered', 'early', 'hertz', 'he', 'found', 'short', 'wavelength', 'radiation', 'incident', 'metal', 'surface', 'caused', 'instantaneous', 'emission', 'photoelectrons', 'the', 'classical', 'electromagnetic', 'wave', 'theory', 'fails', 'explain', 'phenomena', 'einstein', '1905', 'derived', 'equation', 'based', 'phenomenon', 'quantum', 'light', 'energy', 'hu', 'absorbed', 'metal', 'electron', 'emitted', 'this', 'energy', 'relationship', 'described', 'following', 'equation', 'o0', 'minimum', 'energy', 'necessary', 'remove', 'electron', 'mcu2', 'kinetic', 'energy', 'electron', 'emitted', 'mcu2', '¼', 'hu', '(cid7)', 'o0', '1033', 'interaction', 'light', 'organic', 'molecules', 'if', 'monochromatic', 'light', 'passes', 'uniform', 'thickness', 'absorbing', 'homogeneous', 'medium', 'absorbing', 'centers', 'acting', 'independently', 'energy', 'light', 'absorbed', 'follows', 'lambert-bouguer', 'law', 'according', 'law', 'physics', 'light', 'absorbed', 'independent', 'intensity', 'incident', 'light', 'intensity', 'radiation', 'reduced', 'fraction', 'proportional', 'thickness', 'absorbing', 'system', 'in', 'addition', 'beer’s', 'law', 'states', 'absorption', 'proportional', 'number', 'absorption', 'centers', 'the', 'two', 'laws', 'usually', 'combined', 'expressed', 'follows', 'didl', '¼', 'kci', 'i', 'intensity', 'radiation', 'l', 'length', 'optical', 'path', 'absorbing', 'medium', 'c', 'concentration', 'absorbing', 'centers', 'k', 'proportionality', 'constant', 'while', 'know', 'exceptions', 'lambert-bouguer', 'law', 'deviations', 'beer’s', 'law', 'due', 'partial', 'ionization', 'molecular', 'association', 'complexation', 'ﬂuorescence', 'portions', 'organic', 'molecules', 'whole', 'molecules', 'p', 'bonds', 'absorb', 'light', 'radiation', 'provided', 'right', 'wavelength', 'particular', 'groupings', 'arrangements', 'atoms', 'molecules', '720', '10', 'polymeric', 'materials', 'special', 'applications', 'fig', '102', 'the', 'orbitals', 'formaldehyde', '(from', 'jaffe', 'orchin)', 'give', 'rise', 'characteristic', 'absorption', 'bands', 'such', 'groups', 'atoms', 'usually', 'containing', 'p', 'bonds', 'referred', 'chromophores', 'examples', 'molecules', 'p', 'bonds', 'compounds', 'contain', 'carbonyl', 'nitro', 'groups', 'aromatic', 'rings', 'a', 'molecule', 'serves', 'example', 'carbonyl', 'arrangement', 'one', 'often', 'referred', 'molecule', 'formaldehyde', 'in', 'molecule', 'carbon', 'atom', 'linked', 'two', 'hydrogens', 'one', 'oxygen', 'bonds', 'the', 'hybrid', 'sp2', 'orbitals', 'bond', 'one', 'electron', 'carbon', 'one', 'oxygen', 'sp', 'orbital', 'in', 'addition', 'two', 'unbonded', 'n', 'electrons', 'oxygen', 'point', 'away', 'carbon', 'atom', 'the', 'orbitals', 'formaldehyde', 'simplest', 'carbonyl', 'compounds', 'illustrated', 'orchin', 'jaffe', '88,', 'shown', 'fig', '102', 'as', 'described', 'molecule', 'p', 'bonded', 'skeleton', 'shown', 'the', 'carbon', 'atom', 'attached', 'two', 'hydrogen', 'atoms', 'single', 'bond', 'oxygen', 'atom', 'double', 'bond', 'this', 'bonding', 'carbon', 'two', 'hydrogens', 'one', 'oxygen', 'atoms', 'means', 'sp2', 'hybrid', 'orbitals', 'the', 'orbitals', 'approximately', '120(cid1)', 'angles', 'in', 'ground', 'state', 'molecules', 'pair', 'electrons', 'form', 'bond', 'paired', 'opposite', 'antiparallel', 'spin', 'in', 'state', 'formaldehyde', 'molecule', 'planar', 'the', 'pauli', 'exclusion', 'principle', '83', 'states', 'two', 'electrons', 'identical', 'quantum', 'numbers', 'that', 'means', 'two', 'electrons', 'orbital', 'three', 'quantum', 'numbers', 'fourth', 'quantum', 'number', 'spin', 'quantum', 'number', 'must', 'different', 'the', 'total', 'spin', 'quantum', 'number', 'molecule', 'designated', 'letter', 'j', 'sum', 'spins', 'individual', 'electrons', 'letter', 's', 'the', 'spin', 'quantum', 'number', 'molecule', 'j', 'equal', '2s', '1', 'this', 'arrangement', 'electrons', 'p', 'orbital', 'generate', 'p', 'bonding', 'p', 'antibonding', 'orbitals', 'absorption', 'light', 'energy', 'chromophore', 'molecule', 'results', 'formation', 'excited', 'state', 'electronic', 'transition', 'ground', 'state', 'excited', 'state', 'such', 'light', 'may', 'ultraviolet', 'visible', 'region', 'electromagnetic', 'spectrum', 'range', '200', 'mm', 'approximately', '780', 'mm', 'promotion', 'electrons', 'bonding', 'orbitals', 'excited', 'states', 'requires', 'large', 'amount', 'energy', 'rupture', 'bonds', 'process', 'on', 'hand', 'electronic', 'transition', 'promote', 'one', 'n', 'electrons', 'oxygen', 'atom', 'formaldehyde', 'antibonding', 'nonbonding', 'orbital', 'n', 'p', 'level', 'requires', 'least', 'amount', 'energy', 'the', 'name', 'antibonding', 'one', 'might', 'deduce', 'type', 'orbital', 'electrons', 'make', 'contribution', 'binding', 'energy', 'molecule', 'in', 'formaldehyde', 'n', 'p', 'transition', 'excited', 'state', 'gives', 'rise', 'absorption', 'band', '(at', '270', 'mm)', 'this', 'relatively', 'weak', 'band', 'suggests', 'transition', 'forbidden', 'one', '(forbidden', 'mean', 'never', 'occurs', 'rather', 'highly', 'improbable)', 'it', 'referred', 'symmetry', 'forbidden', 'transition', 'the', 'reason', 'forbidden', 'crudely', 'justiﬁed', 'fact', 'p', 'xz', 'plane', '(see', 'fig', '102)', 'the', 'n', 'electrons', 'p', 'orbital', 'xz', 'plane', 'perpendicular', 'p', 'orbital', 'because', 'spaces', 'two', 'orbitals', 'overlap', 'poorly', 'likelihood', 'electronic', 'transition', 'one', 'quite', 'low', 'as', 'stated', 'ground', '(normal)', 'state', 'molecules', 'two', 'electrons', 'paired', 'the', 'pairing', 'means', 'electrons', 'opposite', 'anti-parallel', 'spins', 'after', 'absorbing', 'light', 'energy', 'singlet', 'excited', '721', '103', 'photonic', 'polymers', 'fig', '103', 'illustration', 'singlet', 'triplet', 'states', '(from', 'ravve', '91)', 'fig', '104', 'illustration', 'energy', 'transitions', '(from', 'ravve', '91)', 'state', 'two', 'electrons', 'maintain', 'anti-parallel', 'spins', 'the', 'n', 'p', 'excitation', 'however', 'lead', 'two', 'excited', 'state', 'singlet', '(s1)', 'triplet', '(t1)', 'one', 'absorption', 'band', '(at', '250', 'mm)', 'intersystem', 'crossing', 'triplet', 'state', 'singlet', 'results', 'reversal', 'spin', 'one', 'electrons', 'accompanying', 'loss', 'vibrational', 'energy', 'this', 'illustrated', 'fig', '103', 'the', 'intersystem', 'crossing', 'singlet', 'triplet', 'states', 'occur', 'high', 'efﬁciency', 'certain', 'kinds', 'molecules', 'particularly', 'aromatic', 'carbonyl-containing', 'compounds', 'electron–electron', 'repulsion', 'triplet', 'state', 'minimized', 'electrons', 'farther', 'apart', 'space', 'energy', 'lower', 'state', 'corresponding', 'excited', 'singlet', 'one', 'solvents', 'exert', 'high', 'inﬂuence', 'n', 'p', 'transitions', 'while', 'intersystem', 'crossing', 'forbidden', 'transition', '(see', 'above)', 'actually', 'occur', 'high', 'frequency', 'aromatic', 'carbonyl', 'compounds', 'the', 'chemical', 'mechanism', 'photo-excitation', 'organic', 'molecules', 'fully', 'described', 'various', 'books', 'photochemistry', '82', '84', '85', '87.', 'it', 'therefore', 'discussed', 'brieﬂy', 'the', 'transitions', 'illustrated', 'simpliﬁed', 'energy', 'diagram', 'shows', 'excited', 'singlet', 'state', 'various', 'paths', 'subsequent', 'return', 'ground', 'state', 'fig', '104', 'the', 'energy', 'diagram', '(fig', '104)', 'represents', 'energy', 'states', 'molecule', 'possesses', 'n', 'p', 'electrons', 's1', 's2', 'singlet', 'excited', 'states', 't1', 't2', 'excited', 'triplet', 'states', 'solid', 'lines', 'represent', 'electronic', 'transitions', 'they', 'accompanied', 'absorption', 'emissions', 'photons', 'radiationless', 'transitions', 'represented', 'doted', 'lines', 'the', 'diagram', 'shows', 'lowest', 'singlet', 'state', 's1', 'electrons', 'spin-paired', 'lowest', 'triplet', 'state', 't1', 'electrons', 'spin-unpaired', 'the', 'electron', 'excited', 'light', 'particular', 'wavelength', 'upper', 'singlet', 'level', 's2', 'relaxation', 'follows', 'via', 'internal', 'conversion', 'process', 's', 'level', 'the', 'excess', 'energy', 'dissipated', 'vibrational', 'interactions', 'giving', 'rise', 'evolution', 'heat', 'at', 's1', 'level', 'three', 'possible', '722', '10', 'polymeric', 'materials', 'special', 'applications', 'ways', 'excited', 'state', 'becomes', 'deactivated', 'the', 'return', 'ground', 'state', 'triplet', 'one', 'requires', 'inversion', 'spin', 'in', 'fig', '104', 'a0', 'represent', 'energies', 'light', 'absorbed', 'b', 'h', 'i', 'energies', 'internal', 'conversion', 'c', 'represents', 'return', 'ground', 'state', 'way', 'ﬂuorescence', 'return', 'way', 'phosphorescence', 'the', 'franck', 'condon', 'principle', 'states', 'electronic', 'transition', 'various', 'nuclei', 'molecule', 'change', 'position', 'moment', '90.', 'what', 'means', 'electronic', 'transitions', 'much', 'rapid', '(l0─15', 's)', 'nuclear', 'motions', '(10─12', 's)', 'immediately', 'transitions', 'nuclei', 'nearly', 'relative', 'positions', 'velocities', 'transitions', 'the', 'energy', 'various', 'bonding', 'antibonding', 'orbitals', 'increases', 'molecules', 'following', 'order', 'pn', 'p(cid8)', 's(cid8)', 'in', 'molecules', 'heteroatoms', 'oxygen', 'nitrogen', 'however', 'highest', 'ﬁlled', 'orbitals', 'ground', 'state', 'generally', 'nonbonding', 'essentially', 'atomic', 'n', 'orbitals', 'this', 'instance', 'case', 'ketones', 'aldehydes', 'these', 'molecules', 'possess', 'electrons', 'associated', 'oxygen', 'involved', 'bonding', 'molecules', 'the', 'n', 'electrons', 'formaldehyde', 'illustrated', 'follows', 'h', 'h', 'o', 'n-electrons', 'as', 'explained', 'triplet', 'state', 'spin', 'excited', 'electron', 'becomes', 'reversed', 'this', 'results', 'electrons', 'spin', 'from', 'purely', 'theoretical', 'approach', 'electronic', 'conﬁguration', 'allowed', 'due', 'fact', 'excited', 'electron', 'cannot', 'take', 'original', 'position', 'ground', 'state', 'assumes', 'original', 'spin', 'triplet', 'state', 'relative', 'long-lived', 'for', 'instance', 'benzophenone', '77(cid1)c', 'lifetime', '47', '(cid6)', '10─3', 'orchin', 'jaffe', 'wrote', '88', 'triplet', 'state', 'lifetime', '10─3', 'by', 'comparison', 'lifetime', 'singlet', 'state', '10─8', '10─7', 'also', 'triplet', 'state', 'molecule', 'behaves', 'free-radical', 'reactive', 'the', 'carbon', 'atom', 'higher', 'electron', 'density', 'excited', 'state', 'ground', 'state', 'this', 'results', 'higher', 'localized', 'site', 'photochemical', 'activity', 'orbital', 'oxygen', 'because', 'carbonyl', 'oxygen', 'excited', 'state', 'electron-deﬁcient', 'reacts', 'similarly', 'electrophilic', 'alkoxy', 'radical', 'it', 'instance', 'react', 'another', 'molecule', 'abstracting', 'hydrogen', 'at', 'higher', 'frequencies', '(shorter', 'wavelength)', 'light', 'light', 'energy', 'sufﬁciently', 'high', 'p', 'p', 'transitions', 'also', 'take', 'place', 'all', 'aromatic', 'compounds', 'conjugated', 'diene', 'structures', 'possess', 'delocalized', 'p', 'systems', 'because', 'n', 'electrons', 'transitions', 'systems', 'p', 'p.', 'in', 'general', 'excited', 'states', 'molecule', 'polar', 'ground', 'states', 'polar', 'solvents', 'therefore', 'tend', 'stabilize', 'excited', 'state', 'ground', 'state', 'as', 'shown', 'fig', '104', 'triplet', 'state', 'lower', 'energy', 'corresponding', 'singlet', 'state', 'this', 'due', 'fact', 'electron–electron', 'repulsion', 'minimized', 'share', 'other’s', 'orbitals', 'according', 'pauli', 'exclusion', 'principle', 'thus', 'less', 'energy', 'required', 'triplet', 'state', 'the', 'chemical', 'reactivity', 'organic', 'molecules', 'determined', 'principally', 'electron', 'distribution', 'molecule', 'when', 'electron', 'distribution', 'changes', 'due', 'absorption', 'light', 'subsequent', 'transitions', 'photochemical', 'reactions', 'take', 'place', 'molecule', 'electronically', 'excited', 'state', 'the', 'phenomenon', 'light', 'absorption', 'formation', 'excited', 'states', 'subsequent', 'reactions', 'obey', 'four', 'laws', 'organic', 'photochemistry', 'outlined', 'turro', '87:', '1', 'photochemical', 'changes', 'take', 'place', 'result', 'light', 'absorbed', 'molecules', '2', 'only', 'one', 'molecule', 'activated', 'one', 'photon', 'one', 'quantum', 'light', '3', 'each', 'quantum', 'photon', 'absorbed', 'molecule', 'given', 'probability', 'populating', 'either', 'singlet', 'state', 'lowest', 'triplet', 'state', '4', 'in', 'solution', 'lowest', 'excited', 'singlet', 'triplet', 'states', 'starting', 'points', 'photochemical', 'process', '103', 'photonic', 'polymers', '723', 'the', 'relationship', 'amount', 'light', 'number', 'photons', 'absorbed', 'number', 'molecules', 'result', 'undergo', 'reaction', 'deﬁned', 'quantum', 'yield', 'it', 'deﬁned', 'number', 'molecules', 'involved', 'particular', 'reaction', 'divided', 'number', 'quanta', 'absorbed', 'process', '1', '3.', 'another', 'fundamental', 'law', 'photochemistry', 'formulated', 'grotthus', 'draaper', '82', '83.', 'it', 'states', 'light', 'absorbed', 'molecule', 'effective', 'producing', 'photochemical', 'changes', 'molecule', 'there', 'also', 'fundamental', 'law', 'photochemistry', 'states', 'absorption', 'light', 'molecule', 'one-quantum', 'process', 'sum', 'primary', 'processes', 'quantum', 'yield', 'must', 'unity', '82', '83.', 'also', 'law', 'conservation', 'energy', 'requires', 'sum', 'primary', 'quantum', 'yields', 'processes', 'equal', 'unity', 'mathematically', 'expressed', 'ftotal', '¼', 'fi', '¼', '1', 'xi', 'f', 'quantum', 'yield', 'the', 'quantum', 'yield', 'photochemical', 'reactions', 'important', 'sheds', 'light', 'mechanisms', 'reactions', 'the', 'number', 'molecules', 'involved', 'particular', 'photoreaction', 'established', 'analytical', 'kinetic', 'process', 'number', 'quanta', 'absorbed', 'measured', 'aid', 'actinometer', 'the', 'quantum', 'yield', 'also', 'expressed', 'general', 'kinetic', 'terms', '1:', 'fi', '¼', '’es’rpi', 'fi', '¼', 'es’j', '’i', 'rpj', 'xi', 'the', 'equations', 'signify', 'quantum', 'yield', 'particular', 'photo', 'process', 'product', 'two', 'three', 'distinct', 'probabilities', 'these', '’es', 'probability', 'excited', 'state', 'undergo', 'primary', 'photoreaction', 'necessary', 'process', 'the', 'probability', 'metastable', 'ground', 'state', 'intermediate', 'proceed', 'stable', 'products', 'pi', 'probability', 'excited', 'state', 'undergo', 'primary', 'photoreaction', 'necessary', 'process', '’r', 'the', 'concept', 'matter', 'acquire', 'energy', 'discrete', 'units', '(quanta)', 'introduced', '1900', 'max', 'planck', '83.', 'the', 'corollary', 'quantization', 'energy', 'matter', 'must', 'quantized', 'ie,', 'constructed', 'discrete', 'levels', 'different', 'potential', 'energies', 'occupying', 'particular', 'levels', 'electrons', 'obviously', 'possess', 'energy', 'level', 'occupy', 'in', 'molecule', 'intramolecular', 'motions', 'electrons', 'associated', 'molecular', 'electronic', 'levels', 'must', 'taken', 'account', 'there', 'addition', 'electronic', 'levels', 'modes', 'vibration', 'rotation', 'also', 'quantized', 'in', 'words', 'absorption', 'photon', 'light', 'molecule', 'reaction', 'must', 'promote', 'transitions', 'quantum', 'states', 'this', 'requires', 'two', 'conditions', 'these', '(1)', 'molecular', 'state', 'energy', 'em', 'must', 'state', 'n', 'higher', 'energy', 'en', 'hu', '¼', 'en', '(cid7)', 'em', '(2)', 'must', 'speciﬁc', 'interaction', 'radiation', 'light-absorbing', 'portion', 'molecule', 'results', 'change', 'dipole', 'moment', 'molecule', 'transition', 'if', 'designate', 'wave', 'functions', 'states', 'n', 'cm', 'cn', 'respectively', 'transition', 'moment', 'integral', 'may', 'equal', 'zero', 'p', 'electric', 'dipole', 'operator', 'it', 'form', 'p', '¼', 'e', 'ri', 'vector', 'corresponds', 'dipole', 'moment', 'operator', 'electron', 'ri', 'e', 'electronic', 'charge', 'p', 'the', 'increase', 'energy', 'molecule', 'result', 'absorbing', 'quantum', 'radiation', 'expressed', 'relationship', '85:', 'rmn', '¼', 'ðcmpcnþ', 'de', '¼', 'hcl', '724', '10', 'polymeric', 'materials', 'special', 'applications', 'l', 'wavelength', 'interacting', 'radiation', 'all', 'reactions', 'photochemical', 'nature', 'involve', 'electronically', 'excited', 'states', 'one', 'time', 'each', 'one', 'states', 'deﬁnite', 'energy', 'lifetime', 'structure', 'the', 'property', 'state', 'may', 'differ', 'one', 'another', 'excited', 'states', 'different', 'chemical', 'entities', 'ground', 'state', 'behave', 'differently', 'the', 'return', 'ground', 'state', 'excited', 'state', 'shown', 'fig', '104', 'take', 'place', 'one', 'three', 'processes', '85:', '1', 'the', 'molecule', 'returns', 'directly', 'ground', 'state', 'this', 'process', 'accompanied', 'emission', 'light', 'different', 'wavelength', 'form', 'ﬂuorescence', '2', 'an', 'intersystem', 'conversion', 'process', 'takes', 'place', 't1', 'state', 'electron', 'reverses', 'spin', 'the', 'slower', 'decay', 'excitation', 'triplet', 'state', 'ground', 'state', 'accompanied', 'emission', 'phosphorescence', '3', 'the', 'molecule', 'uses', 'energy', 'excitation', 'undergo', 'chemical', 'reaction', 'as', 'explained', 'triplet', 'state', 'spin', 'excited', 'electron', 'becomes', 'reversed', 'this', 'results', 'electrons', 'spin', 'from', 'purely', 'theoretical', 'approach', 'electronic', 'conﬁguration', 'allowed', 'due', 'fact', 'excited', 'electron', 'cannot', 'take', 'original', 'position', 'ground', 'state', 'assumes', 'original', 'spin', 'triplet', 'state', 'relative', 'long-lived', 'for', 'instance', 'benzophenone', '77(cid1)c', 'lifetime', '47', '(cid6)', '10(cid7)3', 'orchin', 'jaffe', 'wrote', '88', 'triplet', 'state', 'lifetime', '10(cid7)3', 'by', 'comparison', 'lifetime', 'singlet', 'state', '10(cid7)8', '10(cid7)7', 'also', 'triplet', 'state', 'molecule', 'behaves', 'free-radical', 'reactive', 'the', 'carbon', 'atom', 'higher', 'electron', 'density', 'excited', 'state', 'ground', 'state', 'this', 'results', 'higher', 'localized', 'site', 'photochemical', 'activity', 'n', 'orbital', 'oxygen', 'because', 'carbonyl', 'oxygen', 'excited', 'state', 'electron-deﬁcient', 'reacts', 'similarly', 'electrophilic', 'alkoxy', 'radical', 'it', 'instance', 'react', 'another', 'molecule', 'abstracting', 'hydrogen', 'at', 'higher', 'frequencies', '(shorter', 'wavelength)', 'light', 'light', 'energy', 'sufﬁciently', 'high', 'p', 'p', 'transitions', 'also', 'take', 'place', 'all', 'aromatic', 'compounds', 'conjugated', 'diene', 'structures', 'possess', 'delocalized', 'p', 'systems', 'because', 'n', 'electrons', 'transitions', 'systems', 'p', 'p.', 'in', 'general', 'excited', 'states', 'molecule', 'polar', 'ground', 'states', 'polar', 'solvents', 'therefore', 'tend', 'stabilize', 'excited', 'state', 'ground', 'state', 'as', 'shown', 'fig', '104', 'triplet', 'state', 'lower', 'energy', 'corresponding', 'singlet', 'state', 'this', 'due', 'fact', 'electron–electron', 'repulsion', 'minimized', 'share', 'other’s', 'orbitals', 'stated', 'pauli', 'exclusion', 'principle', 'thus', 'less', 'energy', 'required', 'triplet', 'state', 'this', 'dissipation', 'excitation', 'energy', 'also', 'illustrated', 'follows', 'a0', 'light', 'a', 'a0', 'fluorescence', 'a0', 'heat', 'chemical', 'reaction', 'ao', 'represents', 'organic', 'molecule', 'a', 'represents', 'molecule', 'excited', 'state', 'in', 'process', 'energy', 'dissipation', 'singlet', 'return', 'ground', 'states', 'light', 'emission', 'ﬂuorescence', 'different', 'wavelength', 'light', 'absorbed', 'excitation', 'this', 'energy', 'lost', 'process', 'electron', 'returning', 'lowest', 'excited', 'state', 'ground', 'state', 'the', 'energy', 'however', 'may', 'also', 'depending', 'upon', 'structure', 'molecule', 'dissipated', 'form', 'heat', 'shown', 'and', 'also', 'third', 'form', 'energy', 'dissipation', 'occur', 'molecule', 'undergoes', 'chemical', 'reaction', 'depending', 'molecular', 'structure', 'chemical', 'reactions', 'rearrangement', 'isomerization', 'dimerization', '(or', 'coupling)', 'fragmentation', 'attack', 'another', '90–92', 'molecule', 'some', 'examples', 'reactions', 'hν', '103', 'photonic', 'polymers', '725', 'hν', 'hν', 'uv', 'light', '2', 'hν', 'cho', 'n', 'o', 'r', 'light', 'dark', 'no2', 'no2', 'o', 'r', 'many', 'examples', 'found', 'literature', 'most', 'familiar', 'isomerization', 'reaction', 'trans-stilbene', 'cis-stilbene', 'shown', 'it', 'observed', 'quantum', 'yield', 'stilbene', 'cis-', 'trans', 'isomerization', 'decreased', 'increase', 'viscosity', 'medium', '89.', 'in', 'addition', 'also', 'found', 'polymeric', 'matrix', 'photo-isomerization', 'inhibited', 'provided', 'occurs', 'glass', 'transition', 'temperature', 'polymer', 'an', 'example', 'fragmentation', 'molecule', 'decomposition', 'disulﬁdes', 'upon', 'irradiation', 'ultraviolet', 'light', 'appropriate', 'wavelength', 's', 's', '2', 's', 'hν', 'the', 'reaction', 'takes', 'place', 'peroxides', 'ketones', 'aldehydes', 'cleave', 'mechanism', 'norrish', 'reaction', '726', '10', 'polymeric', 'materials', 'special', 'applications', '1034', 'energy', 'transfer', 'process', 'photosensitizers', 'used', 'many', 'photonic', 'applications', 'to', 'understand', 'function', 'necessary', 'understand', 'energy', 'transfer', 'process', 'the', 'term', 'energy', 'transfer', '92', '93', 'refers', 'speciﬁcally', 'one-step', 'radiationless', 'transfer', 'electronic', 'excitation', 'donor', 'molecule', 'another', 'qualiﬁed', 'acceptor', 'molecule', 'one', 'chromophore', 'another', 'one', 'this', 'excludes', 'referred', 'trivial', 'energy', 'transfers', 'result', 'donor', 'emitting', 'light', 'subsequently', 'absorbed', 'acceptor', 'based', 'energy', 'spin', 'conservation', 'laws', 'two', 'priori', 'requirements', 'efﬁcient', 'energy', 'transfer', '(1)', 'process', 'must', 'thermoneutral', 'exothermic', 'occur', 'highest', 'efﬁciency', 'activation', 'energies', 'low', 'due', 'short', 'lifetimes', 'electronically', 'excited', 'state', '(2)', 'net', 'spin', 'changes', 'occur', 'if', 'donor', 'molecule', 'triplet', 'state', 'time', 'energy', 'transfer', 'process', 'acceptor', 'molecule', 'also', 'promoted', 'triplet', 'state', 'transfer', 'singlet', 'singlet', 'energy', 'possible', 'occurs', 'less', 'frequently', 'shorter', 'lifetimes', 'singlet', 'states', '92', '93.', 'energy', 'transfer', 'thus', 'process', 'excitation', 'energy', 'passes', 'one', 'photo-excited', 'molecule', 'often', 'referred', 'sensitizer', 'case', 'designated', 's,', 'another', 'adjacent', 'molecule', 'ground', 'state', 'often', 'referred', 'quencher', 'case', 'designated', 'q', 'the', 'quencher', 'must', 'thermodynamically', 'accessible', 'excited', 'state', 'one', 'whose', 'energy', 'lower', 's.', 'a', 'donor', 'molecule', 'must', 'possess', 'sufﬁciently', 'long', 'lifetime', 'efﬁcient', 'sensitizer', 'the', 'reaction', 'energy', 'transfer', 'illustrated', 'follows', 's(cid8)', 'þ', 'q', 's', 'þ', 'q(cid8)', 's(cid8)', 's0', 'þ', 'hn', 'designates', 'excited', 'state', 'in', 'process', 'energy', 'transfer', 's', 'returns', '(or', 'relaxes)', 'ground', 'state', 's', 'energy', 'transfer', 'categorized', 'involving', 'singlet', '(paired', 'electron', 'spins)', 'triplet', '(unpaired', 'electron', 'spins)', 'states', 'symmetry', 'rules', 'explained', 'require', 'singlet', 's', 'produce', 'singlet', 'q', 'triplet', 's3', 'produce', 'triplet', 'q3.', 'the', 'quenching', 'reaction', 'excited', 'state', 'expressed', 'equation', 'stern–volmer', 'the', 'reaction', 'shown', 'based', 'quenching', 'reaction', 'accompanied', 'release', 'heat', 's(cid8)', 'þ', 'q', 'q(cid8)', 'þ', 's0', 'ðk2þ', 's(cid8)', 's0', 'þ', 'heat', 'ðk3þ', 'the', 'equation', 'written', 'follows', 'f0fq', '¼', 'fk1', 'þ', 'k2½q(cid5)', 'þ', 'k3gðk1', 'þ', 'k3þ', 'in', 'experimental', 'studies', 'energy', 'transfer', 'convenient', 'express', 'experimental', 'results', 'another', 'form', 'stern–volmer', 'equation', 'follows', 'f0fq', '¼', '1', 'þ', 'kqt½q(cid5)', 'f0', 'quantum', 'yield', 'particular', 'process', 'absence', 'quenching', 'molecule', 'fq', 'quantum', 'yield', 'quenched', 'process', 'kq', 'q', 'concentration', 'quenching', 'molecules', 'bimolecular', 'rate', 'constant', 'quenching', 'process', 'lifetime', 'state', 'absence', 'quenching', 'molecules', 'it', 'equal', '1(k1', 'k3)', '103', 'photonic', 'polymers', '727', 'two', 'processes', 'proposed', 'explain', 'mechanism', 'energy', 'transfer', 'in', 'ﬁrst', 'one', 'energy', 'transfers', 'result', 'interactions', 'dipole', 'ﬁelds', 'excited', 'donors', 'ground', 'state', 'acceptor', 'molecules', '(long-range', 'forster', '(dipole–dipole))', '86', '90.', 'this', 'referred', 'resonance', 'transfer', 'mechanism', 'such', 'transfer', 'rapid', 'extinction', 'coefﬁcients', 'absorption', 'donor', 'acceptor-excited', 'states', 'involved', 'process', 'large', '(104–105', 'maximum)', 'when', 'dipolar', 'interactions', 'large', 'resonance', 'transfers', 'possible', 'distances', '50–100', 'a˚', 'close', 'proximities', 'donors', 'acceptors', 'however', 'required', 'weakly', 'absorbing', 'molecules', 'in', 'second', 'mechanism', '90', '(short-range', 'dexter', '(exchange))', 'excited', 'donor', 'acceptor', 'close', 'proximity', '(up', '(cid9)15', 'a˚', ')', 'electronic', 'clouds', 'overlap', 'slightly', 'in', 'region', 'overlap', 'location', 'excited', 'electron', 'indistinguishable', 'it', 'may', 'one', 'instant', 'either', 'donor', 'acceptor', 'molecule', 'should', 'pair', 'separate', 'excited', 'electron', 'acceptor', 'molecule', 'energy', 'transfer', 'achieved', 'mechanism', 'electron', 'transfer', 'discussed', 'next', 'section', 'both', 'absorption', 'emission', 'processes', 'may', 'intramolecular', 'localized', 'single', 'molecule', 'on', 'hand', 'also', 'involve', 'whole', 'crystals', 'may', 'act', 'absorbers', 'emitters', 'such', 'energy', 'transfers', 'manifest', 'different', 'ways', 'include', 'sensitized', 'ﬂuorescence', 'phosphorescence', 'concentration', 'depolarization', 'ﬂuorescence', 'photo-conduction', 'formation', 'triplet', 'acceptor', 'molecules', 'intermolecular', 'energy', 'transfer', 'electronic', 'vibrational', 'take', 'place', 'solid', 'liquid', 'gaseous', 'phases', 'in', 'addition', 'sensitized', 'excitation', 'q', 's', 'take', 'place', 'within', 'time', 'molecule', 's', 'remains', 'excited', 'state', 'in', 'summary', 'theoretical', 'empirical', 'considerations', 'suggest', 'two', 'modes', 'transfer', 'described', '1', 'only', 'two', 'molecules', 'close', 'proximity', 'centers', 'separated', 'sum', 'molecular', 'radii', 'transfer', 'take', 'place', '2', 'when', 'two', 'molecules', 'distances', 'exceed', 'collision', 'diameters', 'resonance', 'transfer', 'long', 'range', 'electronic', 'excitation', 'takes', 'place', 'though', 'coulombic', 'interactions', 'the', 'transfers', 'take', 'place', 'mechanism', '1', 'limited', 'diffusion', 'molecules', 'solution', 'affected', 'viscosity', 'medium', 'transfers', 'mechanism', '2', 'hand', 'much', 'less', 'sensitive', 'viscosity', 'medium', 'it', 'shown', 'foster', '86', 'rate', 'constant', 'resonance-energy', 'transfer', '(mechanism', '1)', 'function', 'distance', 'rate', 'constant', 'ðs(cid8)', 'q(cid8)þ', '¼', '1ðrorþ6ts', 'ts', 'actual', 'mean', 'lifetime', 's,', 'r', 'separation', 'centers', 's', 'q', 'r', 'critical', 'separation', 'donor', 'molecules', 'acceptor', 'molecule', 'the', 'efﬁciency', 'energy', 'transfer', 'expressed', 'turro', 'et', 'al', '94', 'follows', 'fet', '¼', 'ket½s(cid8)(cid5)½q(cid5)fket½s(cid8)(cid5)½q(cid5)', 'þ', 'kd½s(cid8)(cid5)g', 'the', 'transfer', 'long-range', 'excitation', 'mechanism', '2', 'form', 'singlet–singlet', 'transfer', 'triplet–singlet', 'transfer', 'triplet–triplet', 'transfer', 'due', 'fact', 'lifetime', 'triplet', 'state', 'molecule', 'longer', 'singlet', 'one', 'probable', 'one', 'participate', 'energy', 'transfer', 'molecules', 'undergo', 'intersystem', 'crossing', 'high', 'efﬁciency', 'like', 'benzophenone', 'efﬁcient', 'triplet', 'sensitizers', 'such', 'molecules', 'must', 'possess', 'high', 'energy', 'triplet', 'state', 'lifetime', 'least', '10(cid7)4', 'the', 'two', 'types', 'intermolecular', 'energy', 'transfers', 'expressed', 'follows', 'forster(dipole-dipole)', 'long', '-', 'range', 'ksqðrþ', '¼', 'k0', 'sðrsq', '0', 'rþ6', '728', '10', 'polymeric', 'materials', 'special', 'applications', 'dexter(exchange)', 'short', '-', 'range', 'ksqðrþ', '¼', 'k0', 'sq', 'expð(cid7)arþ', 'the', 'nomenclature', 'developed', 'connection', 'energy', 'charge', 'transfer', 'processes', 'follows', 'eximer', 'transient', 'dimer', 'formed', 'combination', 'excited', '(usually', 'aromatic)', 'molecule', 'second', 'similar', '(usually', 'unexcited)', 'molecule', 'such', 'dimer', 'bonds', 'excited', 'state', 'promptly', 'dissociates', 'losing', 'excitation', 'energy', 'the', 'term', 'exiplex', 'explained', 'birks', '92,', 'describes', 'complex', 'two', 'molecules', 'one', 'donor', 'one', 'acceptor', 'subsequently', 'dissociate', 'deactivation', 'process', 'one', 'components', 'exiplex', 'either', 'donor', 'acceptor', 'excited', 'state', 'counterpart', 'acceptor', 'donor', 'ground', 'state', 'an', 'eximer', 'special', 'case', 'two', 'constituent', 'molecules', 'identical', 'while', 'numerous', 'charge-transfer', 'complexes', 'form', 'certain', 'molecules', 'ground', 'state', 'number', 'compounds', 'form', 'charge–transfer', 'complexes', 'either', 'donor', 'acceptor', 'excited', 'state', 'formation', 'eximers', 'observed', 'number', 'aromatic', 'polymers', 'polystyrene', 'poly(vinyl', 'naphthalenes)', 'poly(vinyl', 'toluene)', 'others', '93.', 'an', 'exterplex', 'composed', 'three', 'molecules', 'often', 'takes', 'important', 'role', 'photophysical', 'photochemical', 'processes', 'polymers', 'pendant', 'aromatic', 'chromophores', 'dimeric', 'compounds', 'often', 'show', 'efﬁcient', 'exterplex', 'formation', 'due', 'high', 'local', 'chromophore', 'concentration', 'structure', 'it', 'observed', 'exiplex', 'emission', 'spectra', 'chromophore', 'usually', 'broad', 'structureless', 'red-shifted', 'corresponding', 'monomer', 'ﬂuorescence', 'the', 'extend', 'shift', 'function', 'distance', 'two', 'components', 'complex', 'it', 'also', 'strongly', 'affected', 'polarity', 'media', 'martic', 'et', 'al', '95', 'obtained', 'emission', 'spectra', 'exiplexes', 'anthracene', 'nn', 'dimethyl-p-toluidine', 'toluene', 'polystyrene', 'while', 'maximum', 'band', 'emission', 'spectra', 'toluene', '30(cid1)c', '616', 'nm', 'polystyrene', 'shifted', '400', 'nm', 'the', 'exiplex', 'emission', 'spectra', 'copolymer', 'styrene', '4-nn-diaminostyrene', '480', 'nm', 'the', 'maxima', 'emission', 'spectra', 'temperature-dependent', 'the', 'maxima', 'shifts', 'toluene', 'solution', 'shorter', 'wavelength', 'polystyrene', 'opposite', 'shifts', 'longer', 'wavelength', 'increase', 'temperature', 'the', 'maxima', 'approaches', 'common', 'value', 'glass', 'transition', 'temperature', 'polystyrene', 'similar', 'results', 'reported', 'farid', 'et', 'al', '96', 'studied', 'formation', 'exiplexes', '4-(1-pyrenyl)butyrate', 'different', 'solvents', 'polymers', 'chemical', 'physical', 'changes', 'take', 'place', 'molecules', 'absorb', 'energy', 'reach', 'excited', 'state', 'this', 'particularly', 'true', 'carbonyl', 'compounds', 'there', 'change', 'already', 'stated', 'dipole', 'moments', 'molecules', 'this', 'due', 'fact', 'dipole', 'moments', 'depend', 'upon', 'distribution', 'electrons', 'in', 'carbonyl', 'compounds', 'change', 'particularly', 'large', 'also', 'geometry', 'molecule', 'changes', 'ground', 'excited', 'states', 'in', 'addition', 'chemical', 'properties', 'molecules', 'change', 'thus', 'phenol', 'instance', 'weak', 'acid', 'excited', 'state', 'strong', 'acid', 'this', 'attributed', 'p', 'p', 'transition', 'one', 'pair', 'p', 'electrons', 'promoted', 'antibonding', 'orbital', 'by', 'reason', 'acid', 'strength', 'benzoic', 'acid', 'less', 'excited', 'state', 'charge', 'case', 'transferred', 'carbonyl', 'group', 'the', 'excited', 'states', 'phenol', 'benzoic', 'acid', 'illustrated', 'follows', '93:', 'oh', 'oh', 'light', '103', 'photonic', 'polymers', '729', 'o', 'oh', 'c', 'light', 'o', 'oh', 'c', '1035', 'the', 'electron', 'transfer', 'process', 'this', 'process', 'particularly', 'important', 'photonic', 'applications', 'solar', 'cells', 'simple', 'migration', 'energy', 'thermodynamically', 'neutral', 'process', 'it', 'allows', 'excitation', 'energy', 'deposited', 'site', 'solid', 'concentrated', 'solution', 'move', 'another', 'position', 'transferring', 'excitation', 'energy', 'absence', 'intermediate', 'quencher', 'electron', 'transfer', 'however', 'process', 'electron', 'passed', 'electron-rich', 'donor', 'electron-deﬁcient', 'acceptor', '95.', 'this', 'reaction', 'substan-', 'tially', 'accelerated', 'donor', 'acceptor', 'excited', 'electron', 'transfer', 'excited', 'state', 'donor', 'molecule', 'd', 'ground', 'state', 'acceptor', 'a', 'generates', 'radical', 'cation', 'd•', 'radical', 'anion', 'a(cid7)•', 'the', 'resulting', 'radical', 'ion', 'pair', 'exists', 'charge-separated', 'pair', 'ions', 'd(cid8)', 'þ', 'a', 'dþ(cid10)', 'þ', 'a(cid7)(cid10)', 'the', 'oxidized', 'reduced', 'species', 'usually', 'highly', 'energetic', 'storing', 'substantial', 'fraction', 'energy', 'absorbed', 'photon', 'the', 'charge', 'separation', 'occurs', 'photo-induced', 'electron', 'transfer', 'provides', 'way', 'convert', 'excitation', 'energy', 'excited', 'molecule', 'chemical', 'potential', 'form', 'radical', 'ion', 'pair', 'electron', 'migration', 'also', 'movement', 'electron', 'either', 'neutral', 'electron', 'donor', 'oxidized', 'one', 'ðd', 'þ', 'd(cid10)', 'd(cid10)', 'þ', 'dþ', 'reduced', 'acceptor', 'neutral', 'one', 'ða(cid10)', 'þ', 'a', 'a', 'þ', 'a(cid10)þ', '93.', 'these', 'thermoneutral', 'processes', 'called', 'hole', 'electron', 'migrations', 'respectively', 'permit', 'spatial', 'charge', 'separation', 'excited', 'donor', 'd,', 'reduced', 'acceptor', 'a•', 'this', 'separation', 'beyond', 'one', 'initially', 'produced', 'ion', 'pair', 'photo-induced', 'electron', 'transfer', 'after', 'absorption', 'light', 'sensitizer', 'energy', 'migrations', 'energy', 'transfer', 'move', 'excited', 'state', 'site', 'excitation', 'energy', 'converted', 'radical', 'ion', 'pair', 'photo-', 'induced', 'electron', 'transfer', 'a', 'kinetic', 'competition', 'takes', 'place', 'rates', 'several', 'possible', 'next', 'steps', 'these', 'steps', 'chemical', 'reaction', 'radical', 'ions', 'charge', 'migrations', 'sequential', 'electron', 'hole', 'transfers', 'actually', 'nonproductive', 'charge', 'recombination', 'called', 'back-electron', 'transfer', 'the', 'back-electron', 'transfer', 'regenerates', 'ground', 'states', 'donor', 'acceptor', '1036', 'the', 'charge', 'transfer', 'processes', 'polymers', 'charge', 'transfer', 'polymers', 'either', 'electronic', '(transfer', 'electrons', 'positive', 'charges', 'alone)', 'ionic', '(transfer', 'protons', 'larger', 'charged', 'species)', 'electronic', 'conduction', 'also', 'two', 'types', 'one', 'type', 'conduction', 'due', 'diffusion', 'electrons', 'localized', 'particular', 'molecule', '(this', 'usually', 'found', 'liquids', 'gases)', 'the', 'type', 'conduction', 'due', 'positive', 'negative', 'charges', 'localized', 'particular', 'part', 'molecules', 'such', 'charges', 'exchanges', 'like', 'polymeric', 'molecules', '(or', 'segments', 'single', 'polymeric', 'molecules)', '730', '10', 'polymeric', 'materials', 'special', 'applications', 'this', 'occur', 'without', 'net', 'energy', 'loss', '(resonant', 'charge', 'transfer)', 'it', 'shown', 'experimentally', 'electrical', 'conductivity', 'many', 'polymeric', 'materials', 'subjected', 'short', 'irradiation', 'pulses', 'consists', 'initially', '“prompt”', 'component', 'that', 'means', 'rapid', 'transfer', 'considerable', 'amount', 'charge', 'takes', 'place', 'comparatively', 'short', 'distance', '((cid9)100', 'a˚', ')', 'the', 'movement', 'charge', 'terminated', 'result', 'trapping', '“shallow”', 'traps', '90', '93', '97.', 'this', 'followed', '“delayed”', 'component', 'temperature-dependent', 'probably', 'indicates', 'thermally', 'activated', 'charge-hopping', 'process', 'shallow', 'traps', 'this', 'continues', 'terminated', '(after', '(cid9)1', 'mm)', 'trapping', 'deep', 'traps', 'recombination', '90', '93', '97.', 'there', 'major', 'difference', 'eximers', 'polymers', 'small', 'molecules', 'the', 'difference', 'least', 'polymers', 'large', 'part', 'excitation', 'excimer', 'site', 'appears', 'result', 'singlet', 'energy', 'migration', '93.', 'also', 'polymeric', 'materials', 'number', 'identical', 'chromophores', 'either', 'backbone', 'pendant', 'groups', 'photons', 'absorbed', 'excited', 'states', 'cannot', 'considered', 'localized', 'in', 'simple', 'cases', 'rigid', 'lattices', 'excitations', 'distributed', 'entire', 'volume', 'material', 'wave-like', 'linear', 'combination', 'local', 'excitations', '87', '90', '91', '93.', 'they', 'referred', 'tight-binding', 'excitations', '90', '91', '93.', 'as', 'one', 'might', 'expect', 'excimer', 'formations', 'polymers', 'depend', 'upon', 'properties', 'chromophores', 'upon', 'location', 'tacticity', 'conformation', 'distance', 'polymeric', 'chain', '90.', 'chromophores', 'greatly', 'affect', 'formation', 'eximers', 'also', 'possible', 'distinguish', 'two', 'different', 'types', 'energy', 'transfers', 'polymeric', 'materials', 'the', 'transfer', 'excitation', 'take', 'place', 'either', 'large', 'molecules', 'small', 'ones', 'thus', 'instance', 'polymer', 'transfer', 'excitation', 'energy', 'localized', 'chromophore', 'one', 'polymeric', 'chain', 'another', 'an', 'example', 'transfer', 'small', 'molecule', 'energy', 'transfer', 'polymer', 'like', 'polystyrene', 'scintillator', 'molecule', 'like', '14-bis2-(5-phenyloxazolyl)benzene', 'shown', '95:', 'in', 'addition', 'polymer', 'o', 'o', 'more', 'transfer', 'also', 'take', 'place', 'one', 'group', 'atoms', 'chromophore', 'located', 'polymeric', 'chain', 'one', 'section', 'molecule', 'intramolecularly', 'another', 'one', 'located', 'another', 'section', 'polymer', 'thus', 'copolymers', 'monomers', 'two', 'different', 'chromophores', 'groups', 'energy', 'absorbed', 'one', 'group', 'chromophores', 'transferred', 'chromophores', 'group', 'this', 'take', 'place', 'either', 'foster', 'exchange', 'mechanism', 'the', 'possibility', 'energy', 'transfer', 'one', 'chromophore', 'adjacent', 'different', 'chromophore', 'polymeric', 'chains', 'depends', 'large', 'extent', 'upon', 'lifetime', 'excitation', 'alternative', 'modes', 'deactivation', 'for', 'reason', 'readily', 'observed', 'form', 'energy', 'migration', 'one', 'occurs', 'mechanism', 'triplet', '88', '90', '93', '97.', 'intermolecular', 'energy', 'transfer', 'one', 'polymeric', 'material', 'another', 'molecules', 'solution', 'melt', 'also', 'take', 'place', '17.', 'this', 'demonstrated', 'intramolecular', 'excimer', 'exiplex', 'formation', 'solutions', 'polyesters', 'containing', 'naphthalene', 'carbazole', 'moieties', 'chemical', 'structures', '98.', 'in', 'general', 'migrations', 'energy', 'polymers', 'somewhat', 'complex', 'chain', 'folding', 'conformations', 'additional', 'factors', 'enter', 'picture', 'the', 'separation', 'interacting', 'units', 'affected', 'composition', 'polymer', 'geometry', 'polymeric', 'chains', 'ﬂexibility', 'backbones', '99.', 'there', 'two', 'limiting', 'cases', 'effects', 'polymer', 'folding', 'energy', 'transfer', 'efﬁciency', 'folding', 'polymer', 'excitation', 'conformation', 'sensitizers', 'held', 'within', 'hydrophobic', 'pocket', 'improves', 'efﬁciency', 'energy', 'migration', 'this', 'takes', 'place', 'large', 'number', 'intramolecular', 'hops', 'bond', 'interactions', 'intervene', 'sensitizer', 'ultimate', 'trap', '93.', 'if', 'polymers', 'ﬂexible', 'however', 'also', 'bend', 'photo-excitation', '103', 'photonic', 'polymers', '731', 'bring', 'otherwise', 'distant', 'chromophores', 'close', 'enough', 'energy', 'hop', 'one', 'skipping', 'intervening', 'units', 'thereby', 'considerably', 'shortening', 'effective', 'migration', 'distance', 'along', 'individual', 'polymer', 'chain', '93.', 'for', 'ﬂexible', 'polymers', 'solvents', 'promote', 'folding', 'motion', 'take', 'place', 'even', 'faster', 'excited', 'state', 'decay', '99.', 'intramolecular', 'singlet', 'energy', 'migration', 'also', 'proceed', 'via', 'electronic', 'coupling', 'bonds', 'form', 'polymer', 'backbone', 'in', 'random', 'walk', 'excitation', 'energy', 'migrates', 'without', 'directional', 'control', 'moving', 'back', 'forth', 'along', 'chain', 'across', 'space', 'through-space', 'interactions', 'pendant', 'chromophores', 'also', 'common', 'polymers', 'large', 'numbers', 'absorbing', 'units', '18.', 'one', 'also', 'include', 'movement', 'excitation', 'across', 'folds', 'loops', 'form', 'polymeric', 'chains', 'such', 'folds', 'result', 'packing', 'crystalline', 'domains', 'simply', 'temporary', 'collisions', 'in', 'principle', 'excitation', 'localized', 'ﬁnite', 'time', '(however', 'small)', 'particular', 'chromophore', 'transferred', 'another', 'one', 'chain', 'guellet', '94', 'deﬁnes', 'intramolecular', 'energy', 'migration', 'process', 'involves', 'one', 'exchange', 'excitation', 'energy', 'spectroscopically', 'identical', 'chromophores', 'attached', 'covalent', 'bonds', 'polymeric', 'chain', '12.', 'he', 'terms', '“energy', 'transfer”', 'single', 'step', 'migration', 'two', 'chromophores', 'one', 'involves', 'several', 'chromophores', '“energy', 'migration”', '93.', 'the', 'polymers', 'multiple', 'sensitizers', 'offer', 'several', 'routes', 'energy', 'migration', 'this', 'illustrated', 'follows', '99:', 'route', 'a', 'route', 'b', 'a', 'common', 'arrangement', 'photosensitive', 'groups', 'aligned', 'outside', 'spiral', 'arrangement', 'polymeric', 'chain', 'close', 'enough', 'proximity', 'energy', 'transfer', 'also', 'mentioned', 'earlier', 'folding', 'polymer', 'excitation', 'conformation', 'sensitizers', 'held', 'within', 'hydrophobic', 'pocket', 'improves', 'efﬁciency', 'energy', 'migration', 'large', 'number', 'intramolecular', 'hops', 'efﬁciency', 'energy', 'migration', 'also', 'helped', 'through-bond', 'interactions', 'intervene', 'sensitizer', 'ultimate', 'trap', '99.', 'also', 'mentioned', 'ﬂexible', 'polymer', 'frameworks', 'bend', 'polymeric', 'chains', 'manner', 'bring', 'otherwise', 'distant', 'chromophores', 'close', 'enough', 'together', 'excitation', 'energy', 'hop', 'one', 'another', 'in', 'case', 'energy', 'migration', 'skip', 'intervening', 'units', 'thereby', 'considerably', 'shorten', 'effective', 'migration', 'distance', 'along', 'single', 'polymer', 'chain', 'as', 'stated', 'ﬂexible', 'polymers', 'solvents', 'promote', 'folding', 'motion', 'even', 'faster', 'excited-', 'state', 'decay', '99.', 'intermolecular', 'energy', 'migration', 'also', 'occur', 'two', 'different', 'polymeric', 'molecules', 'thus', 'instance', 'turro', 'et', 'al', '95', 'investigated', 'inter-', 'intramolecular', 'energy', 'transfer', 'poly(styrene', 'sulfonate)', 'they', 'found', 'excimer', 'formation', 'adjacent', 'phenyl', 'groups', 'dominant', 'reaction', 'along', 'single', 'chain', 'two', 'different', 'chains', '95.', 'at', 'low', 'densities', 'excited', 'states', 'singlet', 'energy', 'transfer', 'sensitizer', 'nearest', 'quencher', '(perhaps', 'another', 'chain)', 'dominates', 'whereas', 'high', 'excited', 'state', 'densities', 'energy', 'migration', 'takes', 'place', 'series', 'donors', '99.', 'guellet', 'quotes', 'webber', 'reported', 'used', 'following', 'equation', '(that', 'called', 'crude', 'useful)', 'obtain', 'rough', 'estimates', 'energy', 'migration', 'diffusion', 'rate', 'along', 'polymer', 'backbone', '94:', 'kq', '¼', '4pn0ðdq', 'þ', 'kgþpr1', '000', '732', '10', 'polymeric', 'materials', 'special', 'applications', 'dq', 'normal', 'diffusion', 'constant', 'quencher', 'kg', 'energy', 'migration', 'diffusion', 'rate', 'along', 'polymer', 'in', 'aromatic', 'vinyl', 'polymers', 'excimer', 'emission', 'occur', 'initial', 'excitation', 'aromatic', 'chromophore', 'this', 'followed', 'intramolecular', 'singlet', 'energy', 'migration', 'either', 'along', 'polymer', 'chain', 'intermolecularly', 'along', 'chromophores', 'here', 'different', 'chains', 'polymer', 'bulk', 'form', 'chains', 'close', 'proximity', 'the', 'process', 'generally', 'continues', 'excitation', 'trapped', 'chain', 'conformation', 'suitable', 'excimer', 'formation', 'such', 'chain', 'conformation', 'referred', 'eximer-forming', 'site', 'if', 'polymer', 'solution', 'viscosity', 'low', 'interconversion', 'chain', 'conformations', 'proceeds', 'fairly', 'rapidly', 'in', 'cases', 'lifetimes', 'particular', 'conformation', 'limited', 'collision', 'processes', 'well', 'magnitude', 'rotational', 'barriers', 'respect', 'thermal', 'energy', '93.', 'in', 'solid', 'state', 'however', 'rotational', 'freedom', 'polymeric', 'chain', 'considerably', 'reduced', 'large-scale', 'conformational', 'changes', 'unlikely', 'there', 'still', 'possibility', 'however', 'adjacent', 'chromophores', 'marginal', 'eximer-forming', 'site', '94.', '1037', 'the', 'antenna', 'effect', 'polymers', 'it', 'originally', 'observed', 'schneider', 'springer', '101', 'efﬁcient', 'ﬂuorescence', 'occurs', 'small', 'amounts', 'acenaphthalene', 'copolymerized', 'styrene', 'fox', 'et', 'al', '100', 'observed', 'effect', 'copolymer', 'styrene', 'small', 'amount', 'vinyl', 'naphthalene', 'the', 'emission', 'naphthalene', 'ﬂuorescence', 'much', 'higher', 'solution', 'mixture', 'two', 'homopolymers', 'it', 'suggested', 'groups', 'phenomenon', 'due', 'energy', 'migration', 'styrene', 'sequences', 'naphthalene', 'moieties', 'guellet', 'coworkers', 'carried', 'quantitative', 'studies', 'phenomenon', 'various', 'polymers', 'contained', 'naphthalene', 'phenanthrene', 'donors', 'anthracene', 'trap', '94.', 'this', 'effect', 'similar', 'one', 'observed', 'ordered', 'chlorophyll', 'regions', 'green', 'plant', 'chloroplasts', '(antenna', 'chlorophyll', 'pigments)', 'it', 'therefore', 'named', 'antenna', 'effect', 'guellet', '94', 'demonstrated', 'effect', 'entirely', 'due', 'energy', 'migration', 'among', 'chromophores', 'form', 'antenna', 'rather', 'combination', 'migration', 'direct', 'forster', 'energy', 'transfer', 'trap', '94.', 'it', 'concluded', 'energy', 'migration', 'transfer', 'systems', 'primarily', 'due', 'long-range', 'forster', 'transfer', 'dipole-dipole', 'mechanism', '(discussed', 'earlier)', 'in', 'absence', 'trap', 'polymer', 'energy', 'migrate', 'along', 'backbone', 'polymer', 'chain', 'deactivated', 'processes', 'in', 'presence', 'singlet', 'energy', 'trap', 'lifetime', 'excitation', 'reduced', 'length', 'energy', 'migration', 'reduced', 'the', 'difference', 'form', 'energy', 'transfer', 'one', 'observed', 'solid', 'aromatic', 'polymers', 'photon', 'energy', 'collected', 'within', 'single', 'polymer', 'molecule', 'energy', 'transfer', 'intramolecular', 'the', 'antenna', 'effect', 'permits', 'collection', 'photon', 'energy', 'entire', 'region', 'space', '(the', 'hydrodynamic', 'volume', 'polymer)', 'transmitting', 'traps', 'located', 'polymer', 'chain', 'the', 'efﬁciency', 'relatively', 'independent', 'concentration', 'efﬁcient', 'even', 'dilute', 'solutions', '93.', '104', 'photosensitizers', 'as', 'explained', 'sect', '103', 'photosensitizers', 'molecules', 'absorb', 'energy', 'light', 'act', 'donors', 'transferring', 'energy', 'acceptor', 'molecules', 'the', 'molecules', 'receive', 'energy', 'may', 'turn', 'undergo', 'various', 'reactions', 'polymerizations', 'isomerizations', 'couplings', 'others', 'many', 'different', 'molecules', 'act', 'photosensitizers', 'useful', 'ones', 'various', 'aromatic', 'compounds', 'in', 'table', '101', 'listed', 'common', 'photosensitizes', 'appeared', 'various', 'publications', '104', 'photosensitizers', '733', 'table', '101', 'some', 'often', 'used', 'photosensitizers', 'photosensitizer', 'benzophenone', 'chemical', 'structure', 'o', 'acetophenone', 'o', 'fluorenone', 'o', 'triphenylene', 'anthraquinone', 'triphenylamine', 'phenanthrene', 'benzil', 'o', 'o', 'n', 'o', 'o', 'f', '10', '099', '095', '093', '088', '088', '076', '087', '092', '(continued)', '734', '10', 'polymeric', 'materials', 'special', 'applications', 'table', '101', '(continued)', 'photosensitizer', 'pyrene', 'chemical', 'structure', 'f', '040', '040', 'naphthalene', 'durene', 'anthracene', 'ch3', 'ch3', 'ch3', 'ch3', 'from', 'various', 'literature', 'sources', 'f', 'represents', 'quantum', 'yield', 'triplets', 'literature', 'the', 'process', 'photosensitization', 'energy', 'transfer', 'involves', 'formation', 'charge', 'transfer', 'complexes', 'a', 'good', 'photosensitizer', 'therefore', 'molecule', 'readily', 'absorbs', 'light', 'energy', 'also', 'one', 'readily', 'transfers', 'another', 'molecule', 'some', 'compounds', 'capable', 'forming', 'transfer', 'complexes', 'ground', 'state', 'many', 'form', 'exiplexes', 'excited', 'state', 'others', 'form', 'complexes', 'compound', 'ground', 'state', 'another', 'one', 'excited', 'state', 'called', 'excimers', 'excited', 'dimers', 'the', 'difference', 'excited', 'state', 'dimer', 'exiplex', 'dimers', 'possess', 'binding', 'energy', 'ground', 'state', 'exiplexes', 'lack', 'binding', 'energy', 'ground', 'state', 'this', 'described', 'sect', '1031', 'the', 'emission', 'spectra', 'two', 'molecules', 'capable', 'forming', 'exciplexes', 'depend', 'upon', 'distances', 'two', 'molecules', 'an', 'equation', 'excited', 'state', 'wave', 'function', 'one-to-one', 'exciplex', 'forms', 'donor', 'molecule', 'd', 'acceptor', 'molecule', 'a', 'written', 'guellet', 'follows', '94:', 'ce', '¼', 'ac1ðdþa(cid7)þ', 'þ', 'bc2ðd(cid7)aþþ', 'þ', 'gc3ðd(cid8)aþ', 'þ', 'dc4ðda(cid8)þ', 'the', 'ﬁrst', 'two', 'terms', 'right', 'side', 'equation', 'correspond', 'charge', 'resonance', 'states', 'last', 'two', 'excitation', 'resonance', 'states', 'thus', 'photosensitizer', 'explained', 'sect', '103', 'act', 'two', 'ways', 'energy', 'transfer', 'electron', 'transfer', 'to', 'exact', 'one', 'may', 'feel', 'true', 'photosensitizer', 'one', 'acts', 'energy', 'transfer', 'alone', 'this', 'however', 'always', 'case', 'also', 'event', 'electron', 'transfer', 'process', 'lead', 'photo-induced', 'decomposition', 'via', 'electron', 'transfer', '102.', 'the', 'rate', 'absorption', 'light', 'sensitizer', 'corresponds', 'excitation', 'ground', 'state', 'excited', 'singlet', 'expressed', '94:', 'iabs', '¼', 'd½s0(cid5)dt', '(cid7)', 'd½s1(cid5)dt', '105', 'photocross-linkable', 'polymers', '735', 'the', 'measurement', 'ﬂuorescence', 'phosphorescence', 'spectra', 'photosensitizers', 'important', 'providing', 'information', 'energy', 'excited', 'states', 'it', 'also', 'allows', 'identiﬁcation', 'phenomena', 'the', 'process', 'energy', 'transfer', 'requires', 'excited', 'donor', 'diffuse', 'proximity', 'acceptor', 'within', 'time', 'period', 'excited', 'lifetime', 'this', 'subject', 'viscosity', 'medium', 'efﬁciently', 'collision', 'process', 'range', 'r', 'collisions', 'occur', 'the', 'observed', 'rate', 'constant', 'energy', 'transfer', 'ket', 'governed', 'molecular', 'rate', 'constant', 'kdiff', 'diffusion-controlled', 'reaction', 'this', 'deﬁned', 'debye', 'equation', 'kdiff', '¼', '8rt3', '000(cid2)', 'ket', '¼', 'akdiff', 'probability', 'energy', 'transfer', 'r', 'universal', 'gas', 'constant', 't', 'temperature', 'kelvin', '(cid2)', 'viscosity', 'medium', 'poise', 'the', 'schmoluchowski', '94', 'equation', 'deﬁnes', 'diffusion', 'constant', 'terms', 'diffusion', 'coefﬁcient', 'sensitizer', 'acceptor', 'kdiff', '¼', '4p1', '000ðrs', 'þ', 'raþðds', 'þ', 'daþna2þf1', 'þ', '½rs', 'þ', 'raðt0ðds', 'þ', 'daþ2þ05(cid5)g', 'ds', 'da', 'diffusion', 'coefﬁcients', 'sensitizer', 'acceptor', 'rs', 'ra', 'molecular', 'radii', 'sensitizer', 'acceptor', 'na', 'avogadro', 'number', 't0', 'lifetime', 'excited', 'state', 'sensitizer', '105', 'photocross-linkable', 'polymers', 'some', 'photocross-linking', 'polymers', 'traced', 'back', 'ancient', 'days', 'pitch', 'photocross-', 'linked', 'decorative', 'purposes', '102.', 'in', 'modern', 'times', 'wide', 'varieties', 'photocross-linkable', 'polymers', 'developed', 'the', 'early', 'practice', 'photo', 'imaging', 'relied', 'mainly', 'upon', 'photo-', 'dimerization', 'reactions', 'these', 'reactions', 'common', 'photo-induced', 'reactions', 'organic', 'chemistry', 'namely', 'intermolecular', 'cyclization', 'this', 'reaction', 'cyclization', 'takes', 'place', 'two', 'reactive', 'species', 'one', 'electronically', 'excited', 'actually', 'predicted', 'woodward–hoffmann', 'rule', '104.', 'in', 'contrast', 'reactions', 'thermally', 'excited', 'ground', 'states', 'molecules', 'proceed', 'different', 'pathways', 'many', 'polymers', 'synthesized', 'possess', 'pendant', 'groups', 'capable', 'photo-', 'cyclization', 'intermolecularly', 'used', 'photo-imaging', 'technology', 'photocross-linking', 'tech-', 'nology', 'today', 'however', 'also', 'uses', 'coupling', 'reactions', 'radicals', 'chain', 'growth', 'polymerizations', 'result', 'photocross-linking', 'ionic', 'reactions', 'the', 'light-induced', 'polymerizations', 'multi-', 'functional', 'monomers', 'transform', 'liquid', 'resins', 'solid', 'polymers', 'almost', 'instantly', 'selectively', 'illuminated', 'areas', 'versatile', 'processes', 'they', 'however', 'achieved', 'variety', 'ways', 'thus', 'instance', 'decker', 'discussed', 'work', 'less', 'explored', 'areas', 'photo-curing', 'namely', 'laser-induced', 'ultra-fast', 'polymerization', 'uv', 'curing', 'binary', 'polymer', 'systems', '105.', 'by', 'using', 'highly', 'sensitive', 'acrylate', 'photoresists', 'relief', 'images', 'micronic', 'size', 'obtained', 'fast', 'scanning', 'focused', 'laser', 'beam', 'also', 'polymer', 'networks', 'different', 'architectures', 'obtained', 'uv', 'irradiation', 'various', 'monomer', 'blends', 'eg,', 'acrylate-epoxide', 'acrylate-vinyl', 'ethers', 'acrylate-polyene', 'vinyl', 'ether-maleate', 'thiol-polyene', '104.', 'this', 'mean', 'however', 'photocross-linking', 'polymers', 'unimportant', 'technologically', 'scientiﬁcally', 'the', 'fact', 'considerable', 'research', 'still', 'continues', 'ﬁeld', 'direct', 'indication', 'the', 'light', 'cross-linkable', 'reaction', 'like', 'cross-linking', 'reactions', 'results', 'gelation', 'extent', 'gelation', 'important', 'process', 'this', 'extent', 'tied', 'quantity', 'functional', 'groups', 'reaction', 'mixture', 'carothers', 'equations', 'relate', 'critical', 'extent', 'gelation', 'pc', 'gel', 'point', 'functionality', 'reactants', '736', '10', 'polymeric', 'materials', 'special', 'applications', 'pc', '¼', '2fave', 'fave', 'functionality', 'this', 'equation', 'however', 'written', 'reaction', 'mixtures', 'contain', 'two', 'different', 'functional', 'groups', 'stoichiometric', 'proportions', 'all', 'functional', 'groups', 'however', 'various', 'reasons', 'might', 'participate', 'photocross-linking', 'reactions', 'even', 'though', 'reactions', 'may', 'involve', 'identical', 'groups', 'equation', 'probably', 'would', 'apply', 'a', 'statistical', 'approach', 'developed', 'flory', '107', '108', 'stockmayer', '109', 'derive', 'expression', 'predicting', 'extent', 'reaction', 'gel', 'point', 'it', 'expressed', 'pc', '¼', '1½rðfwa', '(cid7)', '1þðfwb', '(cid7)', '1þ(cid5)05', 'fwa', 'fwb', 'weight', 'average', 'functionalities', 'a', 'b', 'functional', 'groups', 'here', 'functionalities', 'deﬁned', 'fwa', '¼', 'f', '2', 'ainai', 'x', 'x', 'fainai', 'fwb', '¼', 'f', '2', 'binbi', 'fbinbi', 'x', 'x', 'when', 'cross-linking', 'however', 'takes', 'place', 'cyclization', 'dimerization', 'pendant', 'groups', 'appears', 'gelation', 'probably', 'treated', 'instead', 'case', 'multiple', 'dimerizations', '1051', 'polymers', 'that', 'photocross-link', 'formation', 'cyclobutane', 'rings', 'many', 'photocross-linkable', 'polymers', 'photo-imaging', 'use', 'today', 'react', '2p', '2p', 'type', 'dimerization', 'accompanying', 'formation', 'cyclobutane', 'rings', '108', '109.', 'the', 'formation', 'cyclobutane', 'ring', 'simply', 'shown', 'photocross-linking', 'reaction', 'poly(naphthyl', 'vinyl', 'acrylate)', 'polymer', 'also', 'undergoes', 'type', 'dimerization', '110:', 'o', '2', 'o', 'uv', 'lighy', 'o', 'o', 'o', 'o', 'r', 'r', 'n', 'the', 'naphthalenes', 'become', 'bonded', 'cyclobutane', 'ring', '12', '13', 'positions', 'many', 'polymers', 'functional', 'groups', 'also', 'photocross-link', '2p', '2p', 'addition', 'following', 'illustra-', 'tion', 'groups', '111', '112:', 's', 'o', 'r', 'o', 'banzothiophene', 'oxide', 'coumarin', 'dibenzazepene', '105', 'photocross-linkable', 'polymers', '737', 'diphenylcyclobatane', 'y', 'n', 'r', 'stilbazole', 'o', 'nh', 'o', 'alkyne', 'maleimide', 'stilbene', 's', 'n', 'n', 'ch3', 'nh', 'o', 'nh', 'o', 'n', 'h', 'o', 'o', 'n', 'h', '123-thiadiazole', 'thiamine', 'uracil', 'ho2c', 'pendant', 'groups', 'anthracene', 'moieties', 'however', 'believed', 'cross-link', '4p', '4p', 'type', 'cycloaddition', '114:', 'many', 'photo-dimerizations', 'functional', 'groups', 'like', 'ones', 'shown', 'require', 'presence', 'photosensitizers', 'these', 'compounds', 'may', 'selectively', 'excited', 'triplet', 'state', 'sensitizers', 'right', 'type', 'energy', 'levels', 'for', 'efﬁcient', 'energy', 'transfer', 'occur', 'triplet', 'energy', 'donor', '738', '10', 'polymeric', 'materials', 'special', 'applications', 'approximately', '3', 'kcalmole', 'greater', 'acceptor', '113', '114.', 'this', 'type', 'sensitization', 'believed', 'diffusion', 'controlled', '115.', 'trecker', '116', 'lists', 'steps', 'sensitized', 'dimerization', 'reaction', 'follows', 'excitation', 's', 'ðsensitizerþ', 'þ', 'hn', 's(cid8)', 'energy', 'loss', 's(cid8)', '3', 's', 'sensitization', 's(cid8)', '3', 'þ', 'd', '(cid10)s', '(cid7)', 'd(cid10)', 'dimerization', '(cid10)s', '(cid7)', '(cid7)d(cid10)', 'þ', 'd', 's', 'þ', '(cid10)d', '(cid7)', 'd(cid10)', 'dimer', '(cid10)d', '(cid7)', 'd(cid10)', 'dimer(sþ', 'the', 'quantum', 'yield', 'expression', 'scheme', 'dimerization', 'shown', 'follows', '115:', '1f', '¼', '1fa', 'þ', 'k3fak4ðdþ', 'f', 'represents', 'quantum', 'yield', 'dimer', 'formation', 'f', 'efﬁciency', 'sensitizer', 'intersystem', 'crossing', 'd', 'initial', 'concentration', 'reactive', 'groups', 'k’s', 'speciﬁc', 'rate', 'constants', 'reactions', 'shown', 'scheme', '10511', 'polymers', 'pendant', 'cinnamoyl', 'functional', 'groups', 'minsk', 'et', 'al', '117', 'may', 'ﬁrst', 'synthesize', 'photocross-linkable', 'polymer', 'namely', 'poly', '(vinyl', 'cinnamate)', 'the', 'photochemistry', 'compound', 'similar', 'photo-cyclization', 'cinnamic', 'acid', 'discussed', 'bellow', 'section', 'it', 'interesting', 'reaction', 'cyclization', 'cinnamic', 'acid', 'take', 'place', 'even', 'solid', 'crystalline', 'stage', 'this', 'illustrates', 'reaction', 'requires', 'little', 'molecular', 'motion', 'similar', 'reactions', 'occur', 'polymeric', 'materials', 'functionalized', 'cinnamate', 'groups', 'the', 'photocross-linking', 'poly(vinyl', 'cinnamate)', 'illustrated', '2', 'n', 'o', 'uv', 'light', 'o', 'n', 'o', 'o', 'o', 'o', 'n', 'much', 'earlier', 'well', 'minsk', '1895', 'bertram', 'kursten', '118', 'recognized', 'solid', 'cinnamic', 'acid', 'undergoes', 'chemical', 'change', 'exposed', 'light', 'following', 'ruber', '119', 'established', 'change', 'dimerization', 'acid', 'form', 'cyclobutane', 'derivative', 'this', 'dimerization', 'results', 'formation', 'truxillic', 'truxinic', 'acids', '2', 'o', '2', 'o', 'h', 'hν', 'o', 'o', 'h', '105', 'photocross-linkable', 'polymers', '739', 'cooh', 'hooc', 'cooh', 'cooh', 'truxillic', 'acid', 'truxinic', 'acid', 'schmidt', 'coworkers', '120', '121', 'studied', 'reaction', 'mechanism', 'came', 'following', 'conclusions', '1', 'photo-dimerization', 'cinnamic', 'acid', 'esters', 'controlled', 'crystal', 'lattice', '2', 'dimerizations', 'possible', 'oleﬁnic', 'double', 'bonds', 'two', 'neighboring', 'molecules', 'crystals', '41', 'a˚', 'less', 'apart', '3', 'dimerizations', 'possible', 'double', 'bonds', '47', 'a˚', 'apart', 'the', 'dimerization', 'reaction', 'takes', 'place', 'upon', 'irradiation', 'light', 'wavelength', 'longer', '300', 'nm', 'it', 'demonstrated', 'subsequently', '122,', 'however', 'poly(vinyl', 'cinnamate)', 'adducts', 'dissociate', 'upon', 'irradiation', 'light', '254', 'nm', 'photo-dimerization', 'formation', 'cyclobutane', 'groups', 'demonstrated', 'enhanced', 'result', 'sensitized', 'irradiation', '123.', 'when', 'dealing', 'poly(vinyl', 'cinnamate)', 'reasonable', 'assume', 'degree', 'order', 'relationship', 'one', 'cinnamic', 'group', 'another', 'much', 'lower', 'found', 'crystal', 'lattice', 'cinnamic', 'acid', 'on', 'hand', 'higher', 'solutions', 'cinnamic', 'acid', 'groups', 'far', 'enough', 'apart', 'little', 'photo-dimerization', 'takes', 'place', 'photocross-linking', 'poly(vinyl', 'cinnamate)', 'include', 'following', 'reactions', '124:', '1', 'truxinic', 'acid', 'type', 'dimerization', 'irradiated', 'poly(vinyl', 'cinnamate)', 'occur', 'intramolecu-', 'larly', 'it', 'shown', 'follows', 'o', 'o', 'o', 'o', 'this', 'accompanied', 'formation', 'folded', 'parallel', 'chains', '2', 'truxillic', 'acid', 'type', 'intermolecular', 'dimerization', 'irradiated', 'poly(vinyl', 'cinnamate)', 'hand', 'illustrated', 'follows', '740', '10', 'polymeric', 'materials', 'special', 'applications', 'o', 'o', 'o', 'o', 'o', 'o', 'dimerization', 'type', 'would', 'accompanied', 'formation', 'folded', 'chains', 'reactions', 'formations', 'folded', 'parallel', 'chains', 'similar', 'exception', 'reacting', 'cinnamic', 'groups', 'apart', 'folded', 'chains', 'come', 'together', 'virtue', 'chain', 'folding', 'chains', 'located', 'parallel', 'right', 'distance', 'also', 'conceivably', 'yield', 'truxinic', 'acid', 'type', 'dimerization', 'this', 'would', 'similar', 'arrangements', 'crystal', 'lattices', 'formation', 'truxillic', 'dimer', 'like', 'reaction', '2', 'shown', 'requires', 'favorable', 'folding', 'two', 'different', 'chains', 'also', 'accompanying', 'possibility', 'double', 'bonds', 'may', 'simply', 'polymer-', 'ize', 'chain', 'propagating', 'reaction', '125.', 'this', 'observed', 'cinnamate', 'esters', '125.', 'attempts', 'made', 'determine', 'reaction', 'products', 'photocross-linked', 'poly(vinyl', 'cinnamate)', 'ﬁrst', 'hydrolyzing', 'isolating', 'identifying', 'acids', 'the', 'results', 'showed', 'a-', 'truxillic', 'acid', 'form', 'formation', 'b-truxillic', 'acid', 'however', 'demonstrated', 'in', 'addition', 'among', 'reaction', 'products', 'also', 'large', 'quantity', 'unreacted', 'cinnamic', 'acid', 'this', 'indicates', 'small', 'portion', 'double', 'bonds', 'participate', 'reaction', 'also', 'forgotten', 'small', 'changes', 'unsaturation', 'polymer', 'molecules', 'result', 'cross-linking', 'profound', 'effect', 'solubility', 'studies', '126', 'electronic', 'structures', 'photo-isomerization', 'photo-dimerization', 'cinnamic', 'acid', 'showed', 'phosphorescence', 'cinnamic', 'groups', 'occurs', '20000cm', 'also', 'demonstrated', 'photosensitizers', 'present', 'critical', 'distance', 'donor', 'sensitizer', 'acceptor', 'molecules', '(cinnamic', 'acid)', '10', 'a˚', '126.', 'although', 'details', 'incipient', 'photocross-linking', 'poly(vinyl', 'cinnamate)', 'date', 'still', 'fully', 'worked', 'accept', 'three', 'mechanisms', 'take', 'place', 'these', 'dimerizations', 'truxillic', 'truxinic', 'acid', 'type', 'structures', 'polymerizations', 'double', 'bonds', 'the', 'excited', 'states', 'molecules', 'produced', 'direct', 'irradiation', 'also', 'intersystem', 'crossing', 'appropriate', 'photosensi-', 'tizer', '126.', 'several', 'criterions', 'derived', 'proper', 'selection', 'sensitizers', '127', '128.', 'these', '1', 'the', 'triplet', 'state', 'must', 'energy', 'level', 'close', '50–55', 'kcalmole', 'cinnamate', 'moiety', '2', 'the', 'quantum', 'yield', 'ratio', 'phosphorescence', 'ﬂuorescence', 'higher', 'unity', '3', 'the', 'mean', 'lifetime', 'triplet', 'state', 'must', 'greater', '001', 'the', 'photosensitizing', 'activity', '(characterized', 'triplet', 'state)', 'derivatives', 'cinnamic', 'acid', 'beyond', 'phosphorescence', 'cinnamate', 'group', '(at', '20000cm', 'stated', 'above)', 'an', 'energy', 'transfer', 'diagram', 'poly', '(vinyl', 'cinnamate)', 'photosensitization', 'sensitizer', 'like', '2-nitroﬂuorene', 'published', '128.', 'the', 'rate', 'dimerization', 'obeys', 'ﬁrst-order', 'kinetics', 'in', 'addition', 'polymers', 'consisting', 'ﬂexible', 'segments', 'exhibit', 'higher', 'rates', 'photo-dimerization', 'composed', 'rigid', 'segments', '105', 'photocross-linkable', 'polymers', 'table', '102', 'effectiveness', 'sensitizers', 'relative', 'speed', 'cross-linking', 'poly(vinyl', 'cinnamate)', 'relative', 'speed', 'sensitizer', '1', '(none)', '3', 'naphthalene', '7', 'benzanthrone', '14', 'phenanthrene', '18', 'crysene', '20', 'benzophenone', '31', 'anthrone', '5-nitroacenaphthene', '84', 'from', 'ref', '131', 'sources', 'sensitizer', '4-nitroaniline', '3-nitroﬂuorene', '4-nitromethylaniline', '4-nitrobiphenyl', 'picramide', '4-nitro-26-dichlorodimethylaniline', 'michler’s', 'ketone', 'n-acyl-4-nitro-1-nathylamine', '100', '113', '137', '200', '400', '561', '640', '1100', '741', 'relative', 'speed', 'it', 'also', 'demonstrated', 'greater', 'photosensitivity', 'obtained', 'separating', 'cinnamic', 'group', 'polymer', 'backbone', 'introducing', '–ch2–ch2–o–', 'spacers', 'follows', '129', '130:', 'o', 'o', 'n', '2', 'o', 'in', 'addition', 'tsuda', 'oikawa', 'carried', 'molecular', 'orbital', 'calculations', 'electronic', 'structures', 'excited', 'states', 'poly(vinyl', 'cinnamate)', '131', '132.', 'they', 'based', 'calculations', 'take', 'place', 'according', 'reaction', 'intermolecular', 'concerted', 'cycloaddition', 'woodward–hoffmann’s', 'rule', 'this', 'means', 'cyclobutane', 'ring', 'formation', 'takes', 'place', 'nodal', 'plane', 'exists', 'central', 'double', 'bond', 'lowest', 'unoccupied', 'mo(llumo)', 'highest', 'occupied', 'mo', '(homo)', 'ground', 'state', 'cinnamoyloxy', 'group', 'this', 'within', 'picture', 'huckel', 'mo', 'extended', 'huckel', 'mo', 'theory', 'the', 'conclusion', 'concerted', 'cycloadditions', 'occur', 'favorably', 'lowest', 'triplet', 'state', 't1', 'second', 'excited', 'singlet', 'state', 's2', '132.', 'the', 'effectiveness', 'photosensitizers', 'accelerating', 'cross-linking', 'reaction', 'poly(vinyl', 'cinnamate)', 'illustrated', 'table', '102', 'some', '4', 'decades', 'original', 'development', 'poly(vinyl', 'cinnamate)', 'useful', 'photocross-', 'linkable', 'polymer', 'novel', 'optical', 'property', 'polymer', 'observed', 'when', 'material', 'irradiated', 'linearly', 'polarized', 'light', 'exhibits', 'polarization', 'holography', '133', '134.', 'the', 'exposure', 'thin', 'ﬁlms', 'poly(vinyl', 'cinnamate)', 'linearly', 'polarized', 'ultraviolet', 'light', 'causes', 'uniaxial', 'reorienta-', 'tion', 'liquid', 'crystal', 'layers', '135–142.', 'poly(vinyl', 'cinnamate)', 'derivatives', 'ability', 'align', 'thin', 'ﬁlms', 'liquid', 'crystal', 'moieties', 'direction', 'perpendicular', 'polarization', 'axis', 'linearly', 'polarized', 'ultraviolet', 'light', '143–145.', 'schadt', 'et', 'al', '136', 'suggested', 'surface-', 'settled', 'homogeneous', 'alignment', 'nematic', 'liquid', 'crystals', 'results', 'photo-dimerizations', 'cinnamate', 'moieties', 'formation', 'cyclobutane', 'rings', '(as', 'shown', 'earlier)', 'azimuthally', 'oriented', 'order', 'this', 'feels', 'determines', 'direction', 'liquid', 'crystal', 'alignment', '140.', 'ichimura', 'et', 'al', 'photo-induced', '141', 'suggested', 'different', 'photo-alignment', 'process', 'they', 'claimed', '742', '10', 'polymeric', 'materials', 'special', 'applications', 'homogeneous', 'liquid', 'crystalline', 'alignment', 'caused', 'polarization', 'photo-chromophores', 'uppermost', 'surfaces', 'substrates', 'due', 'repeated', 'az', 'photo-isomerizations', 'similarly', 'azobenzenes', '140–149.', 'this', 'also', 'shown', 'take', 'place', 'stilbenes', '140.', 'in', 'addition', 'demonstrated', '149', 'photo-isomerization', 'photo-dimerization', 'contribute', 'liquid', 'crystalline', 'alignment', 'photo-regulation', 'polymethacrylate', 'o-cinnamate', 'side', 'chains', 'displays', 'preferential', 'formation', 'z-isomer', 'dimerization', 'hand', 'takes', 'place', 'favorably', 'polymers', 'including', 'poly(vinyl', 'cinnamate)', '150.', 'the', 'liquid', 'crystals', 'alignment', 'ﬁlms', 'prepared', 'materials', 'cinnamate', 'group', 'irradiating', 'ﬁlms', 'linearly', 'polarized', 'uv', 'light', 'quite', 'uniform', 'all', 'aggregate', 'structures', 'lamellar', 'crystals', 'produced', 'photocross-linking', 'reaction', 'found', 'square', 'shape', '151.', 'this', 'application', 'ﬂat', 'panel', 'liquid', 'crystal', 'displays', 'it', 'led', 'others', 'synthesize', 'polymeric', 'materials', 'could', 'useful', 'photo-alignment', 'lee', 'et', 'al', '152', 'synthesized', 'soluble', 'photo-reactive', 'polyimide', 'cinnamate', 'chromophore', 'side', 'groups', 'the', 'polymer', 'poly(330-bis(cinnamoyloxy)-', '440-biphenylene', 'hexaﬂuoroisopropylidene', 'diphthalimide)', 'reasonably', 'high', 'molecular', 'weight', 'forms', 'good', 'quality', 'ﬁlms', 'conventional', 'solution', 'spin-casting', 'drying', 'cf3', 'cf3', 'o', 'o', 'n', 'n', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'the', 'polymer', 'thermally', 'stable', '340(cid1)c', 'positively', 'birefringent', 'the', 'photochemical', 'reactions', 'polymer', 'solution', 'ﬁlms', 'well', 'molecular', 'orientations', 'induced', 'exposure', 'linearly', 'polarized', 'ultraviolet', 'light', 'as', 'one', 'might', 'expect', 'cinnamate', 'chromophores', 'undergo', 'photo-isomerization', 'photo-dimerization', 'also', 'exposure', 'uv', 'light', 'induces', 'anisotropic', 'orientations', 'polymer', 'main', 'chains', 'cinnamate', 'side', 'groups', 'ﬁlms', 'the', 'irradiated', 'ﬁlms', 'homogeneously', 'align', 'nematic', 'liquid', 'crystal', 'molecules', 'along', 'direction', 'angle', '107(cid1)', 'respect', 'polarization', 'linearly', 'polarized', 'ultraviolet', 'light', 'this', 'coincides', 'orientation', 'direction', 'polyimide', 'chains', 'thus', 'liquid', 'crystal', 'alignment', 'process', 'principally', 'governed', 'irradiated', 'polyimide', 'ﬁlms', 'polymer', 'main', 'chains', 'unreacted', 'cinnamate', 'side', 'groups', '151.', 'nagata', 'hizakae', '153', 'reported', 'preparation', 'series', 'photocross-linkable', 'biodegradable', 'polymers', 'condensation', 'dichlorides', '440-(adipoyldioxy)dicinnamic', 'acid', 'alkane', 'diols', 'various', 'methylene', 'lengths', 'they', 'also', 'used', 'various', 'poly(ethylene', 'glycols)', 'molecular', 'weights', 'ranging', '200', '8300', 'among', 'interesting', 'polymers', 'cinnamate', 'functional', 'groups', 'high', 'polymeric', 'phosphazenes', 'bear', 'cinnamate', 'groups', '154.', 'a', 'typical', 'polymer', 'synthesis', 'follows', '105', 'photocross-linkable', 'polymers', '743', 'och2cf3', 'n', 'n', 'p', 'cl', 'n', 'n', 'cl', 'p', 'cl', 'n', 'och2cf3', 'n', 'p', 'o', 'br', 'och2cf3', 'n', 'n', 'p', 'o', 'oh', 'och2cf3', 'n', 'n', 'p', 'o', 'o', 'o', '1052', 'polymers', 'functional', 'chalcone', 'groups', 'pendant', 'chalcone', 'groups', 'polymers', 'behave', 'similarly', 'pendant', 'cinnamate', 'groups', 'thus', 'photocross-linkable', 'polymer', 'formed', 'instance', 'poly(vinyl', 'alcohol)', 'reaction', '40-substituted-4-caboxychalcone', 'homogeneous', 'dimethyl', 'formamide', 'solution', 'using', '246-', 'trinitrochlorobenzene', 'condensing', 'agent', '155.', 'n', 'o', 'o', 'r', 'o', 'the', 'photosensitivity', 'polymer', 'range', '1–5', 'mjcm2', 'according', 'watanabe', 'et', 'al', '155.', 'when', 'r', 'group', 'shown', 'p-br', 'm-no2', '(ch3)2n', 'cross-linking', 'via', 'formation', 'biradicals', 'derived', 'double', 'bonds', 'cinnamoyl', 'groups', 'abstraction', 'protons', 'neighboring', 'methyne', 'methylene', 'groups', 'this', 'reaction', 'dimerization', 'illustrated', 'follows', '156:', '744', '10', 'polymeric', 'materials', 'special', 'applications', 'n', '2', 'o', 'o', 'o', 'n', 'n', 'formation', 'polymers', 'reaction', 'free-radical', 'polymerizable', 'methacryloyl', 'groups', 'photocross-linkable', 'pendant', 'chalcone', 'units', 'also', 'reported', '157.', 'the', 'photocross-linking', 'reaction', 'takes', 'place', 'thin', 'ﬁlms', 'solutions', 'upon', 'irradiation', 'high', 'pressure', 'mercury', 'lamps', 'the', 'reaction', 'monitored', 'changes', 'uv', 'absorption', 'spectrum', 'the', 'rates', 'photocross-', 'linking', 'solutions', 'reported', 'faster', 'ﬁlms', 'photocross-linking', 'presence', 'triplet', 'photosensitizer', 'shows', 'signiﬁcant', 'changes', 'rate', 'disappearance', 'double', 'bonds', 'this', 'particular', 'polymer', 'pendant', 'chalcone', 'unit', 'found', 'photocross-link', 'high', 'rate', 'even', 'absence', 'photosensitizer', 'interest', 'negative', 'photoresist', '158.', '1053', 'polymers', 'functional', 'groups', 'similar', 'cinnamates', 'as', 'stated', 'earlier', 'photocross-linking', 'reactions', 'poly(vinyl', 'cinnamate)', 'reversibility', 'cross-linking', 'depends', 'upon', 'irradiation', 'light', 'proper', 'wavelength', 'in', 'addition', 'intramolecular', 'cyclizations', 'also', 'take', 'place', 'because', 'reactions', 'found', 'extensive', 'commercial', 'applications', 'number', 'polymers', 'resemble', 'poly(vinyl', 'cinnamate)', 'developed', 'following', 'two', 'examples', 'functional', 'groups', '159:', 'o', 'o', 'o', 'o', 'o', 'c', 'o', 'polyester', '105', 'photocross-linkable', 'polymers', '745', 'o', 'o', 'o', 'o', 'polycarbonate', 'o', 'o', 'o', '1054', 'polymers', 'pendant', 'furan', 'groups', 'tsuda', '160', 'attached', '2-furylacrolein', 'polymeric', 'backbone', 'following', 'reaction', 'n', 'o', 'oh', 'oh', 'h', 'o', 'o', 'n', 'o', 'o', 'h', 'in', 'presence', 'photosensitizers', 'polymer', 'found', 'tsuda', 'cross-link', 'considerably', 'faster', 'rate', 'poly(vinyl', 'cinnamate)', '160.', 'he', 'showed', 'cross-linking', 'reaction', 'also', 'results', 'formation', 'cyclobutane', 'derivatives', 'the', 'cross-linking', 'illustrated', '3-furfurylacrylic', 'ester', 'o', 'light', 'o', 'o', 'o', 'or', 'o', 'or', 'o', 'or', 'light', 'o', 'o', 'ro', 'o', 'or', 'o', 'tajima', 'arai', 'takeuchi', '161–163', 'studied', 'effects', 'singlet', 'oxygen', 'photocross-linking', 'poly(furfuryl', 'methacrylate)', 'the', 'singlet', 'oxygen', 'generated', 'fullerene', 'c60', 'the', 'reactions', 'carried', '1122-tetra-chloroethane', 'solutions', 'contained', 'polymer', 'fullerene', 'c60', 'gelation', 'occurs', 'exposed', 'visible', 'light', 'presence', 'photosensitizers', 'these', 'solutions', 'gel', 'several', 'hours', 'subsequently', 'solidify', 'completely', '163.', 'it', 'interesting', 'note', 'fullerines', 'also', 'effective', 'causing', 'sensitization', 'poly(furfuryl', 'methacrylate)', 'effective', 'causing', 'cross-linking', 'upon', 'uv', 'irradiation', '164.', 'this', 'attributed', 'long-lasting', 'ability', 'sensitize', 'oxygen', 'even', 'undergo', '746', '10', 'polymeric', 'materials', 'special', 'applications', 'oxidation', 'the', 'sensitivity', 'poly(furfuryl', 'methacrylate)', 'increases', 'linearly', 'concentration', 'c60', 'increased', '5', '(cid6)', '10(cid7)5', 'i', '(cid6)', '10(cid7)3', 'moll', 'saturation', 'sensitivity', 'high', 'c60', 'concentration', 'regions', 'observed', 'due', 'deﬁciency', 'oxygen', 'molecules', 'resist', 'ﬁlms', 'it', 'concluded', 'dissolution', 'rate', 'oxygen', 'atmosphere', 'resist', 'ﬁlm', 'lower', 'consumption', 'rate', '164.', 'preparation', 'photocross-linkable', 'furan-containing', 'polyimides', 'also', 'reported', '165.', 'it', 'also', 'found', 'polymer', 'cross-links', 'aid', 'singlet', 'oxygen', 'formations', 'ﬁne', 'pattern', 'images', 'formed', 'this', 'taken', 'clear', 'evidence', 'successful', 'photolithography', 'photo-curable', 'system', 'uses', 'c60', 'photosensitizer', '165.', '1055', 'polymers', 'pendant', 'maleimide', 'groups', 'preparation', 'photocross-linkable', 'co-', 'terpolymers', 'n-isopropylacrylamide', '2-(dimethy-', 'lmaleimido)', 'n-ethylacrylamide', 'photosensitive', 'component', '3-acryloylaminopropionic', 'acid', 'n-(2-(dimethylamino)ethyl)-acrylamide', 'ionizable', 'comonomers', 'reported', '166.', 'here', 'cross-linking', 'takes', 'place', 'formation', 'cyclobutane', 'moieties', 'x', 'o', 'nh', 'hn', 'o', 'o', 'uv', 'light', 'x', 'o', 'nh', 'hn', '2', 'o', 'n', 'o', 'o', 'n', 'o', 'o', 'n', 'o', 'o', 'nh', 'o', 'hn', 'x', '1056', 'polymers', 'pendant', 'abietate', 'dibenzazepine', 'groups', 'poly(vinylbenzyl', 'abietate)', 'ﬁlm', 'state', 'cross-linkable', 'via', 'photo-dimerization', 'conjugated', 'carbon-carbon', 'double', 'bonds', 'abietic', 'acid', 'moieties', '166.', 'what', 'photo-dimerization', 'product', 'looks', 'like', 'clear', 'formation', 'cyclobutane', 'rings', 'photo-dimerization', 'steroids', 'represented', 'follows', '166:', '105', 'photocross-linkable', 'polymers', '747', 'or', 'o', 'o', 'o', 'o', 'also', 'judging', 'photoreaction', 'cholestra-35-diene', '166,', 'one', 'might', 'expect', 'internal', 'cyclization', 'n', 'n', 'uv', 'light', 'o', 'o', 'o', 'o', 'the', 'cyclodimerization', 'reaction', 'dibenzazepine', 'reported', 'hyde', 'et', 'al', '167.', 'when', 'attached', 'polymer', 'cross-linking', 'reaction', 'visualized', 'follows', 'n', '2', 'o', 'o', 'o', 'n', 'n', 'the', 'cross-linking', 'result', 'either', 'direct', 'absorption', 'light', 'presence', 'triple', 'sensitizers', 'like', 'michler’s', 'ketone', '168.', 'the', '2p', '2p', 'addition', 'leads', 'formation', 'cyclobutane', 'cross-link', '748', '10', 'polymeric', 'materials', 'special', 'applications', 'coumarin', 'photo-dimerization', 'known', 'reaction', '169:', '2', 'hν', 'o', 'o', 'o', 'o', 'o', 'o', 'this', 'utilized', 'tian', 'et', 'al', '168', 'prepare', 'new', 'class', 'liquid', 'crystal', 'homopolymers', 'poly', 'll-4-(3-ethoxycarbonyl-coumarin-7-oxy)-carbonyl-phenyloxy-undecyl', 'methacrylate', 'containing', 'coumarin', 'moiety', 'photocross-linkable', 'unit', 'the', 'preparations', 'included', 'polymers', 'various', 'chain', 'lengths', 'also', 'liquid', 'crystalline-coil', 'diblock', 'liquid', 'crystalline-coil-liquid', 'crystalline', 'triblock', 'copolymers', 'polystyrene', 'coil', 'segment', 'formed', 'the', 'polymers', 'reported', 'synthesized', 'aid', 'atom', 'transfer', 'radical', 'polymerization', 'the', 'dimerization', 'coumarin', 'moieties', 'takes', 'place', 'upon', 'irradiation', 'light', 'l', '300', 'nm', 'yield', 'cross-linked', 'network', 'structures', 'lee', 'et', 'al', '169', 'reported', 'preparation', 'new', 'soluble', 'intrinsically', 'photosensitive', 'poly(amide-co-imide)s', 'containing', 'p-phenylenediacryloyl', 'moiety', 'the', 'copolymers', 'formed', 'p-phenylenediacryloyl', 'chloride', 'aromatic', 'dianhydrides', 'two', 'equivalents', 'aromatic', 'diamines', 'the', 'products', 'subsequently', 'imidized', 'reactions', 'poly(amide-co-amic', 'acid)', 'acetic', 'anhydride', 'pyridine', 'the', 'polymers', 'stable', '350(cid1)c', 'showed', 'good', 'solubility', 'polar', 'aprotic', 'solvents', 'became', 'insoluble', 'irradiation', 'due', 'photo-', 'dimerization', 'phenylenediacryloyl', 'moiety', 'the', 'photo-reactivity', 'increases', 'irradiation', 'temperature', '169.', '1057', 'polymers', 'that', 'cross-link', 'dimerization', 'nitrenes', 'other', 'combinations', 'free-radicals', 'form', 'covalent', 'bonds', 'the', 'aromatic', 'azide', 'groups', 'photo-decompose', 'nitrenes', 'irradiated', 'uv', 'light', 'the', 'nitrenes', 'form', 'possess', 'two', 'unpaired', 'electrons', 'similarly', 'carbenes', 'dimerize', 'readily', 'azo', 'groups', 'nr', 'n', 'n', 'uv', 'light', 'nr', 'n2', 'nr2', 'r', 'nn', 'r', 'this', 'reaction', 'utilized', 'photocross-linking', '1058', 'polymers', 'pendant', 'azide', 'groups', 'both', 'azide', 'sulfonyl', 'azide', 'groups', 'photo-reactive', 'decompose', 'active', 'nitrene', 'groups', 'nitrogen', 'upon', 'irradiation', 'the', 'dissociation', 'azide', 'moiety', 'follows', 'almost', 'every', 'transition', '105', 'photocross-linkable', 'polymers', '749', 'excited', 'n', 'p', 'state', 'high', 'vibrational', 'level', 'ground', 'state', '170', '171.', 'introductions', 'pendant', 'azide', 'sufonyl', 'azide', 'groups', 'polymeric', 'structures', 'possible', 'variety', 'ways', 'many', 'publications', 'describe', 'different', 'approaches', 'thus', 'tsuda', 'yamaoka', '81', 'introduced', 'azide', 'groups', 'phenolic', 'novolac', 'resin', 'following', 'scheme', 'oh', 'no2', 'oh', 'n3', '1', 'h2nnh2', 'pdc', '2', 'hno3', 'portion', 'novolac', 'oh', 'oh', 'tsunoda', 'yamaoka', '173', '174', 'also', 'condensed', 'formyl-1-naphthyl', 'azide', 'poly(vinyl', 'alcohol)', 'all', 'steps', 'synthesis', 'disclosed', 'the', 'preparation', 'illustrated', 'follows', 'nh2', 'n3', 'oh', 'o', 'h', 'o', 'h', 'ch2', 'o', 'o', 'n3', 'preparations', 'azide', 'derivatives', 'styrene-maleic', 'anhydride', 'copolymers', 'cellulose', 'gelatin', 'attaching', 'aromatic', 'azide', 'compounds', 'described', 'literature', '175.', 'most', 'resultant', 'polymers', 'cross-link', 'rapidly', 'exposed', 'light', '260', 'mm', 'wavelength', 'also', 'much', '90', 'hydroxy', 'groups', 'poly(vinyl', 'alcohol)', 'esteriﬁed', 'p-azido-benzoyl', 'chloride', 'these', 'reactions', 'must', 'carried', 'mixtures', 'chloroform', 'aqueous', 'sodium', 'hydroxide', '175.', 'earlier', 'merrill', 'unruh', '175', 'described', 'formation', 'derivatives', 'poly(vinyl', 'alcohol)', 'attachment', 'aromatic', 'azide', 'groups', 'most', 'azide', 'polymers', 'photocross-link', 'faster', 'rate', 'poly(vinyl', 'cinnamate)', 'exposed', 'light', '260', 'mm', 'in', 'addition', 'responded', 'well', 'photosensitization', 'also', 'observed', '50', '4-isomer', 'azidophthalate', 'shows', 'greater', 'speed', 'increase', '3-isomer', 'in', 'general', 'poly(vinyl', 'alcohol)', 'derivatives', 'reported', 'exhibit', 'higher', 'cross-linking', 'speeds', 'azide', 'functionalized', 'polymers', '176.', 'the', 'reaction', 'product', 'p-azidobenzoyl', 'chloride', 'polyvinyl', 'alcohol', 'investigated', 'tsuda', 'coworkers', '179.', 'in', 'polymer', 'studied', '90', 'hydroxy', 'groups', 'esteriﬁed', 'the', 'photocross-linking', 'reaction', 'followed', 'observing', 'changes', 'ultraviolet', 'infra-red', 'absorption', 'spectra', 'it', 'shown', 'simple', 'photochemical', 'reaction', 'occurs', 'stochiometrically', 'upon', '750', '10', 'polymeric', 'materials', 'special', 'applications', 'irradiation', 'also', 'absorption', 'band', 'observed', '1500cm', 'infra-red', 'region', 'irradiated', 'cross-linked', 'polymer', 'this', 'band', 'due', 'n¼n', 'stretching', 'vibration', 'azo', 'group', 'based', 'tsuda', 'concluded', 'cross-linking', 'reaction', 'takes', 'place', 'dimerization', 'expected', '179.', 'there', 'reports', 'literature', 'photosensitive', 'azide', 'polymers', 'formed', 'polymers', 'substituted', 'isocianate', 'groups', '176', '177.', 'the', 'preparation', 'properties', 'many', 'polymers', 'containing', 'pendant', 'aryl', 'azide', 'groups', 'described', 'delzenne', 'laridon', '178.', 'the', 'polymers', 'prepared', 'interfacial', 'polycondensation', 'azido-substituted', 'acid', 'chlorides', 'diols', 'diamines', 'also', 'one', 'experiment', 'cinnamate', 'moiety', 'combined', 'azide', 'group', 'together', 'one', 'pendant', 'functional', 'structure', '178.', 'it', 'found', 'however', 'addition', 'azide', 'groups', 'cinnamate', 'side', 'chain', 'increase', 'reactivity', 'a', 'marked', 'wavelength', 'dependence', 'observed', 'p', 'p', 'n', 'p', 'transitions', 'occurs', 'functional', 'groups', '179.', '1059', 'polymers', 'designed', 'cross-link', 'upon', 'irradiation', 'laser', 'beams', 'to', 'accommodate', 'needs', 'photolithography', 'polymers', 'developed', 'cross-link', 'upon', 'irradiation', 'ar-f', 'excimer', 'laser', 'source', '193', 'nm', '180.', 'to', 'end', 'synthesized', 'cyclooleﬁn-maleic', 'anhydride-alternating', 'copolymers', 'this', 'material', 'formed', 'free-radical', 'copolymerization', 'norbornene', 'maleic', 'anhydride', '(a)', 'thf', 'abin', 'o', 'o', 'o', '63', 'oc', 'n', 'o', 'o', 'o', '(b)', 'o', 'o', 'o', 'o', 'oh', 'n', 'o', 'o', 'o', 'o', 'ho', 'to', 'render', 'polymers', 'soluble', 'aqueous', 'base', 'acrylic', 'acid', 'terpolymerization', 'also', 'carried', 'shown', '(b)', '106', 'photo-responsive', 'polymers', 'photo-responsive', 'polymers', 'materials', 'able', 'respond', 'light', 'irradiation', 'undergoing', 'reversible', 'changes', 'chemical', 'structures', 'andor', 'physical', 'properties', 'also', 'photochromism', 'refers', 'photo-induced', 'reversible', 'transformations', 'chemical', 'compounds', 'two', '106', 'photo-responsive', 'polymers', '751', 'electronic', 'states', 'characterized', 'different', 'absorption', 'spectra', '181.', 'there', 'many', 'ways', 'photo-responsiveness', 'polymers', 'manifest', 'one', 'might', 'observe', 'changes', 'viscosity', 'polymeric', 'solutions', 'contraction', 'polymeric', 'chains', 'sol–gel', 'transitions', 'electrical', 'conduc-', 'tivity', 'even', 'color', 'changes', 'result', 'irradiation', 'light', 'appropriate', 'wavelength', 'another', 'interesting', 'manifestation', 'photo-responsiveness', 'special', 'polymers', 'change', 'permeability', 'gases', 'ﬁlms', 'these', 'changes', 'utilized', 'many', 'ways', 'thus', 'instance', 'structural', 'changes', 'due', 'isomerization', 'employed', 'align', 'liquid', 'crystals', 'photo-', 'conductivity', 'utilized', 'xerography', 'over', 'last', '2', '3', 'decades', 'photo-responsive', 'materials', 'grown', 'practical', 'scientiﬁc', 'importance', 'materials', 'useful', 'many', 'applications', '1061', 'polymers', 'harvesting', 'sun’s', 'energy', 'the', 'goal', 'harvesting', 'light', 'energy', 'led', 'research', 'polymeric', 'materials', 'could', 'potentially', 'mimic', 'photosynthesis', 'harvest', 'sun’s', 'heat', 'in', 'materials', 'choice', 'chromophores', 'critical', 'variable', 'the', 'location', 'chromophores', 'polymeric', 'chains', 'tacticity', 'polymers', 'also', 'important', 'weber', 'pointed', 'instance', 'among', 'number', 'chromophores', 'attached', 'polymeric', 'chains', 'naphthalene', 'carbazole', 'form', 'stable', 'eximers', 'phenanthrene', 'diphenyl', 'anthracene', '182.', 'at', 'present', 'many', 'polymeric', 'materials', 'utilized', 'vast', 'areas', 'nonsilver-based', 'imaging', 'information', 'storage', 'remote', 'sensing', 'electroresponsive', 'materials', 'displays', 'others', 'fox', 'cozzens', 'conclude', 'however', 'none', 'compare', 'efﬁciency', 'naturally', 'occurring', 'photon-harvesting', 'polymers', 'photosynthesis', '183.', '10611', 'polymers', 'norbornadiene', 'moieties', 'one', 'approach', 'harvesting', 'light', 'energy', 'utilize', 'pendant', 'groups', 'reversibly', 'absorb', 'light', 'energy', 'rearrange', 'release', 'absorbed', 'energy', 'heat', 'rearrangement', 'back', 'original', 'structure', 'to', 'end', 'research', 'going', 'various', 'laboratories', 'develop', 'systematically', 'derivatized', 'polymer', 'arrays', 'collect', 'convert', 'light', 'energy', 'among', 'photo-rearrangements', 'norbornadiene', 'quadricyclane', 'back', 'considerable', 'interest', 'photo-energy', 'stored', 'strain', 'energy', '(about', '96', 'kjmol)', 'quadricyclane', 'molecule', 'later', 'recovered', '184', 'ultraviolet', 'light', '310', 'nm', '250', 'nm', 'this', 'photo-isomerization', 'reaction', 'referred', 'valence', 'isomerization', 'it', 'reaction', 'electron', 'reshufﬂing', 'occurs', 'nuclei', 'move', 'make', 'break', 'new', 'p', 'bonds', 'a', 'number', 'polymers', 'therefore', 'prepared', 'norbornandiene', 'moieties', 'either', 'backbone', 'pendant', 'groups', 'among', 'polyesters', 'synthesized', 'donor-acceptor', 'norbornadiene', 'residues', 'main', 'chain', '184', 'polyaddition', '5-(4-methoxyphenyl)-l4677-pentamethyl-25-', 'norbornadiene-23-dicarboxylic', 'acid', '56-bis(4-methoxyphenyl)-77-', 'dimethyl-25-norbornadiene-2', '752', '10', 'polymeric', 'materials', 'special', 'applications', '3-dicarboxylic', 'acid', 'bis(epoxide)s', 'this', 'preparation', 'accompanying', 'photo-rearrangements', 'illustrated', 'follows', 'r"', 'r', 'cooh', 'r', 'r"', 'cooh', 'r', 'r"', 'o', 'r', 'r"', 'r"', 'o', 'o', 'o', 'o', 'r', 'r"', 'o', 'r"', 'o', 'o', 'o', 'o', 'oh', 'n', 'oh', 'r', 'r"', 'o', 'r"', 'o', 'o', 'o', 'o', 'o', 'ho', 'n', 'ho', 'the', 'photo-rearrangements', 'norbornadiene', 'residues', 'resulting', 'polyesters', 'observed', 'proceed', 'smoothly', 'quadricyclane', 'groups', 'also', 'found', 'norbornadiene', 'residues', 'polyesters', 'show', 'resistance', 'fatigue', 'repeated', 'cycles', 'interconversions', '184.', 'the', 'goal', 'utilize', 'photochemical', 'valence', 'isomerization', 'norbornadiene', 'quadricyclane', 'solar', 'energy', 'collection', 'storage', 'reported', 'others', '185.', 'nagai', 'et', 'al', 'synthesized', 'ﬁve', 'different', 'polymers', 'triﬂuoromethyl-substituted', 'norbornadiene', 'moieties', 'side', 'chains', 'main', 'chain', '185:', 'n', 'o', 's', 'o', 'cf3', 'cf3', 'cf3', 'oh', 'cf3', 'n', 'o', 'o', 'o', 'o', 's', 'cf3', 'cf3', '106', 'photo-responsive', 'polymers', '753', 'n', 'o', 'o', 'o', 'cf3', 'cf3', 's', 'cf3', 's', 'o', 'n', 'h', 'o', 's', 's', 'o', 's', 'o', 'o', '(ch2)6', 'n', 'h', 'n', 'n', 'all', 'polymers', 'exhibit', 'large', 'absorption', 'bands', 'visible', 'region', 'spectra', 'norbornadiene', 'moieties', 'polymers', 'isomerize', 'rapidly', 'in', 'addition', 'norbornadiene', 'moieties', 'also', 'exhibit', 'efﬁcient', 'fatigue', 'resistance', '185.', 'kawatsuki', 'et', 'al', '186', 'synthesized', 'styrene', 'polymers', 'pendant', 'norbornadiene', 'groups', 'attached', 'via', 'amide', 'linkages', 'r', 'o', 'n', 'o', 'n', 'h', 'r', 'methoxy', 'ring-substituted', 'aniline', 'group', 'attached', 'nitrogen', 'another', 'position', 'these', 'pendant', 'groups', 'also', 'undergo', 'reversible', 'conversions', 'quadricycline', 'units', 'polymer', 'ﬁlms', 'irradiated', 'ultraviolet', 'light', 'two', 'different', 'wavelengths', 'the', 'materials', 'exhibit', 'high', 'photosensitivity', 'well', 'large', 'red-shift', 'absorption', 'spectrum', 'upon', 'irradiation', 'sampei', 'et', 'al', '187', 'reported', 'polyaddition', '25-norbornadiene-23-dicarboxylic', 'acid', 'diglycidyl', 'ester', 'adipoyl', 'chloride', 'gave', 'polyester', 'containing', 'norbornadiene', 'residues', 'rearrangement', 'polymer', 'backbone', 'pendant', 'groups', 'when', 'photochemical', 'norbornadiene', 'residues', 'took', 'place', 'polymer', 'ﬁlms', 'rate', 'photochemical', 'reactions', 'polymer', 'backbones', 'higher', 'side', 'chains', '187.', 'kawashima', 'et', 'al', '188', 'reported', 'preparation', 'donor-acceptor', 'type', 'norbornadiene', 'carboxylic', 'acids', 'compounds', 'carbamoyl', 'groups', 'dipropylcarbamoyl', 'methylphenylcarbamoyl', 'propylcarbamoyl', 'phenyl-carbamoyl', 'benzyl', 'esters', 'also', 'prepared', 'addition', 'groups', 'polystyrenes', 'formed', 'polymers', 'pendant', 'donor-acceptor', 'type', 'norbornadiene', 'some', 'formed', '100', 'substitution', 'it', 'found', 'polymers', 'containing', 'phenylcarbamoyl', 'groups', 'exhibit', 'especially', 'high', 'photo-reactivity', 'in', 'addition', 'rate', 'photochemical', 'reaction', 'ﬁlms', 'norbornadiene', 'polymers', 'increases', 'efﬁciently', 'addition', '440-bis(diethyl-amino)benzo-', 'phenone', 'photosensitizer', 'as', 'result', 'norbornadiene', 'groups', 'polymers', 'isomerize', '754', '10', 'polymeric', 'materials', 'special', 'applications', 'quadricycline', 'groups', 'little', '20', 'the', 'stored', 'thermal', 'energy', 'irradiated', 'polymers', 'found', '32–52', 'kjmol', '10612', 'copolymers', 'methacrylates', 'cholesteric', 'monomers', 'groups', 'like', '6-oxo-16-dihydropyrimidin-3-ium-4-oleates', 'light-sensitive', 'undergo', 'intramolecu-', 'lar', 'photo-cyclization', 'bis-b-lactams', 'irradiated', 'uv', 'light', '320', '490', 'nm', '189.', 'n', 'n', 'hν', 'r', 'o', 'r', 'r"', 'r"', 'o', 'r', 'r"', 'r', 'n', 'n', 'o', 'r"', 'o', 'this', 'utilized', 'form', 'copolymers', 'methacrylates', 'colesteric', 'monomers', '190.', 'an', 'example', 'copolymers', 'rearrangement', 'shown', '190:', 'n', 'o', 'c', 'o', 'o', 'uv', 'light', 'n', 'o', 'c', 'o', '(ch2)4', '(ch2)4', '(ch2)4', '(ch2)4', 'co', 'o', 'co', 'o', 'o', 'o', 's', 'o', 's', 'n', 'n', 'o', '(ch2)3', 'o', 'n', 'o', 'n', '(ch2)3', 'o', 'c2h5ch3', 'c2h5ch3', 'ch3', 'c', 'h', '(ch2)3', 'ch3', 'c', 'h', '(ch2)3', 'it', 'found', 'rate', 'photo-rearrangement', 'affected', 'length', 'alkyl', 'side', 'group', '5', 'position', 'pyrimidinium', 'oleate', 'embedding', 'chromophore', 'shown', '190.', 'in', 'photosynthesis', 'antenna', 'pigments', 'harvest', 'energy', 'transfer', 'reaction', 'center', 'redox', 'reactions', '191.', 'different', 'antenna', 'chromophores', 'surround', 'reaction', 'center', 'arranged', 'morphologically', 'order', 'energy', 'gradient', '192–194.', 'such', 'arrangement', 'allows', 'energy', 'sequentially', 'transferred', 'efﬁciently', 'funneled', 'reaction', 'centers', 'small', 'distances', 'direction', 'decreasing', 'band', 'gaps', 'considerable', 'research', 'carried', 'means', 'develop', 'sequential', 'multistep', 'energy', 'transfer', 'systems', 'this', 'done', 'mimic', 'natural', 'light-', 'harvesting', 'process', '192–194,', 'also', 'possible', 'applications', 'use', 'optoelectronic', 'biological', 'systems', '195.', 'it', 'speculated', 'conjugated', 'polymeric', 'backbone', 'well-designed', 'interruptions', 'conjugation', 'insulating', 'spacers', 'might', 'allow', 'tuning', 'emission', 'properties', 'providing', 'alternate', 'model', '196.', '106', 'photo-responsive', 'polymers', '755', 'based', 'concept', 'krebs', 'coworkers', '197', 'reported', 'synthesis', 'light-harvesting', 'material', 'consists', 'three', 'structural', 'domains', 'two', 'conjugated', 'homopolymers', 'linked', 'zinc', 'porphyrin', 'a', 'b', 'n', 'a', 'n', 'a', 'b', 'n', 'n', 'ze', 'n', 'n', 'sections', 'a', 'b', 'block', 'copolymer', 'illustrated', 'the', 'polymer', 'constant', 'ratio', 'zinc', 'porphyrin', 'incorporated', 'monomer', 'units', 'regardless', 'molecular', 'weight', 'the', 'ratio', 'zinc', 'porphyrin', 'polymer', 'blocks', 'varied', 'material', 'varying', 'size', 'blocks', 'a', 'studies', 'energy', 'transfer', 'polymer', 'zinc', 'porphyrin', 'showed', 'actually', 'little', 'energy', 'transfer', 'material', 'solution', 'on', 'hand', 'quantitative', 'energy', 'transfer', 'solid', 'state', 'also', 'observed', 'light-harvesting', 'properties', 'three-domain', 'structures', 'depend', 'chain', 'lengths', 'conjugated', 'polymers', 'cheng', 'luh', 'reported', 'also', 'trying', 'develop', 'polymers', 'would', 'mimic', 'natural', 'photosynthesis', 'synthetic', 'polymers', '198.', 'they', 'point', 'silylene', 'moieties', 'used', 'extensively', 'insulating', 'spacers', '198.', 'in', 'general', 'silylene', 'spacer', 'contains', 'one', 'silicon', 'atom', 'conjugative', 'interactions', 'p', 'systems', 'silicon', 'moiety', 'observed', '199.', 'they', 'believe', 'therefore', 'introduction', 'energy', 'gradient', 'three', 'well-designed', 'chromophores', 'silylene-spaced', 'polymeric', 'chain', 'may', 'lead', 'sequential', 'energy', 'transfer', 'to', 'achieve', 'goal', 'carried', 'preparations', 'regioregular', 'silylene-spaced', 'copolymers', 'composed', 'energy', 'gradients', 'three', 'different', 'chromophores', 'one', 'polymers', 'prepared', 'way', 'illustrated', 'follows', 'si', 'si', 'si', 'd2', 'd2', 'si', 'a', 'n', 'their', 'synthesis', 'utilized', 'rhodium-catalyzed', 'hydrosilylation', 'bis-vinylsilanes', 'bis-alkynes', 'the', 'ratio', 'three', 'chromophores', 'polymer', '121', 'corresponding', 'd1', 'd2', 'a', 'chromophores', 'respectively', 'cheng', 'luh', 'found', 'upon', 'excitation', 'donor', 'chromophore', 'd1', 'emission', 'acceptor', 'a', 'observed', '199.', '1062', 'photo-isomerization', 'polymeric', 'materials', 'among', 'photo-rearrangements', 'cis-trans', 'isomerization', 'reaction', 'useful', 'one', 'a', 'well-known', 'example', 'cis-stilbene', 'trans', 'isomer', 'this', 'reaction', 'described', '756', '10', 'polymeric', 'materials', 'special', 'applications', 'many', 'books', '200.', 'the', 'isomerization', 'reaction', 'takes', 'place', 'many', 'oleﬁns', 'excited', 'singlet', 'triplet', 'states', 'perpendicular', 'instead', 'planar', 'geometry', 'this', 'means', 'excited', 'state', 'cis-trans', 'isomerism', 'disappears', 'upon', 'return', 'ground', 'state', 's0', 'possible', 'either', 'isomer', 'form', 'the', 'return', 'however', 'usually', 'takes', 'place', 'stable', 'form', 'generally', 'photo-isomerization', 'chromophores', 'dilute', 'solutions', 'ﬁrst-order', 'reaction', '10621', 'photo-isomerization', 'oleﬁnic', 'group', 'an', 'example', 'oleﬁnic', 'groups', 'rearrangement', 'work', 'onciu', 'et', 'al', '201', 'formed', 'three', 'bis', '(trimellitimide)s', 'condensing', 'three', 'aromatic', 'diamines', 'trimellitic', 'anhydride', 'this', 'followed', 'preparation', 'two', 'series', 'photo-reactive', 'copoly(amide-imide)s', 'direct', 'polyconden-', 'bis(trimellitimide)s', '14-phenylenediacrilic', 'acid', 'either', '440-diphenyl-', 'sation', 'methanediisocyanate', 'one', 'case', '16-diisocyanatohexane', 'another', 'case', 'respectfully', 'all', 'copoly(amide-imide)s', 'found', 'soluble', 'polar', 'aprotic', 'solvents', 'yield', 'transpar-', 'ent', 'ﬂexible', 'tough', 'ﬁlms', '201.', 'when', 'polymers', 'irradiated', 'solution', 'p-phenylene-', 'diacryloyl', 'units', 'undergo', 'trans–cis', 'photo-isomerization', '(2', '2)', 'photo-cycloadditions', '201.', 'the', 'fully', 'aromatic', 'polyamides', 'also', 'undergo', 'photo-fries', 'rearrangement', 'the', 'photo-fries', 'reaction', 'however', 'completely', 'suppressed', 'polymers', 'contain', 'aliphatic', 'amide', 'moiety', '22.', 'the', 'processes', 'also', 'observed', 'polymer', 'ﬁlms', '201.', 'polymers', 'prepared', 'condensation', '440-diacetylstiblene', 'bis(ketomethylene)monomer', '440-diamino-330-dibenzoylstilbene', 'bi(amino', 'ketone)', 'exhibit', 'photo-viscosity', 'effects', 'dilute', 'solutions', 'due', 'cis-trans', 'isomerization', '202.', 'the', 'preparation', 'polymers', 'photo-', 'rearrangements', 'illustrated', 'follows', 'h2n', 'nh2', 'o', 'o', 'n', 'n', 'n', 'n', 'o', 'o', 'n', 'n', 'uv', 'light', 'the', 'changes', 'viscosity', 'vary', '2', '23', 'result', 'irradiation', '106', 'photo-responsive', 'polymers', '757', '10622', 'photo-isomerization', 'azo', 'group', 'azobenzene', 'well-known', 'photo-responsive', 'chromophore', 'photo-induced', 'thermal', 'geometric', 'isomerizations', 'extensively', 'explored', '200.', 'azobenzene', 'derivatives', 'assume', 'trans', 'cis', 'conformations', 'respect', 'azo', 'linkage', 'azo', 'p', 'p', 'excitation', 'azo', 'n', 'p', 'excitation', 'trigger', 'trans-to-cis', 'cis-to-trans', 'isomerizations', '203–205.', 'the', 'azo', 'linkage', 'normally', 'exists', 'stable', 'trans', 'form', 'also', 'trans', 'isomer', 'azobenzene', 'exhibits', 'intense', 'absorption', 'around', '320', 'nm', 'due', 'p', 'p', 'transition', 'cis', 'isomer', 'shows', 'weak', 'absorption', 'n', 'p', 'transition', 'around', '430', 'nm', '206.', 'reversible', 'isomerizations', 'cis', 'trans', 'structures', 'due', 'transitions', 'photo-isomerization', 'proceed', 'almost', 'quantitatively', '207.', 'by', 'comparison', 'thermal', 'isomerizations', 'cis', 'trans', 'conﬁgurations', 'take', 'place', 'due', 'low', 'activation', 'energy', 'cis-to-trans', 'process', 'isomerizations', 'azo', 'chromophore', 'compounds', 'often', 'accompanied', 'drastic', 'changes', 'number', 'properties', 'instance', 'changes', 'dipole', 'moments', '208.', 'the', 'isomerization', 'back', 'trans', 'conﬁguration', 'readily', 'carried', 'either', 'thermally', 'visible', 'light', 'irradiation', 'changes', 'molecular', 'structure', 'cis-trans', 'isomerization', 'polymers', 'induce', 'contraction', 'expansion', 'polymeric', 'chains', 'microscopic', 'macroscopic', 'scale', 'this', 'demonstrated', 'polymers', 'azo', 'linkages', 'exposure', 'dark', 'light', 'result', 'contraction', 'much', '05', '209:', 'n', 'n', 'dark', 'light', 'n', 'n', 'one', 'example', 'copolymer', 'styrene', 'azobenzene', 'structures', 'attached', 'comono-', 'mer', 'benzene', 'portion', 'amide', 'linkages', '210:', 'n', 'n', 'heat', 'visible', 'light', 'nh', 'λ', '380', 'nm', 'nh', 'o', 'o', 'n', 'n', 'n', 'n', 'after', '10', 'min', 'irradiation', 'ultraviolet', 'light', 'photo-stationary', 'state', 'reached', 'consisting', '79', 'cis', 'isomer', 'back', 'isomerization', 'trans', 'sample', 'slow', 'dark(less', '10', '60', 'min)', 'much', 'faster', 'exposed', 'visible', 'light', '210.', '758', '10', 'polymeric', 'materials', 'special', 'applications', 'a', 'japanese', 'patent', '211', 'describes', 'preparation', 'isophthalic', 'type', 'polyesters', 'include', 'monomers', 'pendant', 'azo', 'groups', 'o', 'o', 'x', 'x', 'o', 'o', 'n', 'n', 'n', 'y', 'hydrogen', 'lower', 'alkyl', 'group', '¼', '1–3', 'n', '¼', '2–18', 'polyesters', 'based', 'monomers', 'claimed', 'useful', 'optical', 'recording', 'media', 'holograms', 'recording', 'low', 'light', 'absorption', 'without', 'loss', 'wide', 'range', 'working', 'wavelengths', 'izumi', 'coworkers', 'carried', 'similar', 'preparations', 'conjugated', 'polymers', 'azobenzenes', 'main', 'chain', '212', '213.', 'application', 'various', 'palladium-catalyzed', 'coupling', 'methods', 'suzuki', 'coupling', 'heck', 'reactions', 'allowed', 'formation', 'poly(p-phenylene)-', 'poly(phenyl', 'vinylene)-based', 'polymers', 'r', 'r', 'r', 'n', 'n', 'r', 'r', 'n', 'n', 'r', 'r', 'n', 'r', 'n', 'r', 'r', 'n', 'n', 'r', 'r', 'n', 'r', 'r', 'r0', 'r00', '¼', 'h', 'n-c6h13', 'dynamic', 'volumes', 'polymers', '213.', 'these', 'isomerization', 'processes', 'also', 'accompanied', 'changes', 'three-dimensional', 'hydro-', '106', 'photo-responsive', 'polymers', '759', '1063', 'changes', 'viscosity', 'solubility', 'polymeric', 'solutions', 'changes', 'viscosity', 'polymeric', 'solutions', 'associated', 'photo-induced', 'conformational', 'changes', 'macromolecules', 'observed', 'lovrien', '214.', 'he', 'reported', 'solutions', 'copolymer', 'methacrylic', 'acid', 'n-(220-dimethoxyazobenzene)acrylamide', 'exhibit', 'increase', 'speciﬁc', 'viscosity', 'irradiated', 'uv', 'light', 'he', 'also', 'observed', 'decrease', 'viscosity', 'poly', '(methacrylic', 'acid)', 'chrysophenine', 'solution', 'water', 'inﬂuence', 'uv', 'light', '214.', 'this', 'followed', 'various', 'reports', 'photo-viscosity', 'effects', 'solutions', 'azobenzene-based', 'polymers', 'matejka', 'dusek', '215', 'studied', 'copolymer', 'styrene', 'maleic', 'anhydride', 'azobenzene', 'side', 'chains', 'uv', 'light', 'irradiation', 'solution', 'polymer', '14-dioxane', 'causes', 'decrease', 'speciﬁc', 'viscosity', '24', '30', 'tetrahydrofuran', '1', '8.', 'they', 'also', 'observed', 'decrease', 'viscosity', 'reversible', 'the', 'magnitude', 'effect', 'found', 'related', 'quantity', 'azobenzene', 'linkages', 'present', 'polymer', 'hallensleben', 'menzel', '218', 'found', 'irradiation', 'poly(5-(4-phenylazobenzyl)-l-glutamate)', '14-dioxane', 'solution', 'uv', 'light', '(l', '470', 'nm)', 'decreases', 'viscosity', '9.', 'here', 'change', 'viscosity', 'accompanied', 'trans', 'cis', 'isomerization', 'estimated', '23.', 'with', 'additional', 'irradiation', '360', 'nm', 'uv', 'light', 'viscosity', 'decreases', 'additional', '9', 'isomerization', 'cis', 'reaches', '89.', 'irie', 'et', 'al', '217', 'synthesized', 'number', 'polyamides', 'azobenzene', 'groups', 'backbone', 'all', 'polymers', 'exhibit', 'photo-viscosity', 'effects', 'in', 'solutions', 'nn0-dimethylacetamide', '60', 'reduc-', 'tion', 'speciﬁc', 'viscosity', 'achieved', 'uv', 'light', 'irradiation', '(410', 'l', '350', 'nm)', 'the', 'initial', 'viscosity', 'regained', 'storage', 'dark', 'room', 'temperature', '30', 'h', 'changes', 'viscosity', 'solutions', 'dimethylsulfoxide', 'range', 'polyureas', 'azobenzene', 'groups', 'polymer', 'backbone', 'reported', '216.', 'the', 'irradiations', 'carried', '35(cid1)c', 'uv', 'light', '410', '350', 'nm', 'it', 'observed', 'intrinsic', 'viscosity', '40', 'lower', 'uv', 'irradiation', 'dark', 'also', 'toluene', 'solutions', 'polydimethylsiloxane', 'azobenzene', 'residues', 'shown', 'exhibit', '20', 'lower', 'viscosity', 'uv', 'light', 'irradiation', 'dark', '219.', 'this', 'effect', 'attributed', 'conformational', 'contraction', 'polymer', 'chains', 'due', 'dipole–dipole', 'interaction', 'neighboring', 'chromophores', '220.', 'this', 'conformational', 'change', 'might', 'possibly', 'illustrated', 'follows', 'n', 'n', 'n', 'n', 'n', 'uv', 'light', '760', '10', 'polymeric', 'materials', 'special', 'applications', 'n', 'n', 'n', 'n', 'also', 'copolymers', 'polystyrene', '4-(methacryloyl-amino)azobenzene', 'containing', '22–65', 'latter', 'irradiated', 'cyclohexane', 'solution', '15', 'ﬂashes', '347', 'nm', 'light', 'the', 'polymeric', 'chains', 'contract', '221.', 'this', 'occurs', 'high', 'rate', 'per', 'second', 'consequence', 'isomerization', 'at', 'later', 'stage', 'several', 'hundred', 'seconds', 'ﬂash', 'evidence', 'polymer', 'aggregation', 'precipitation', '221.', 'in', 'addition', 'azobenzene', 'residues', 'introduced', 'main', 'chain', 'poly(dimethylsiloxane)', 'reversible', 'solution', 'viscosity', 'changes', 'obtained', 'irradiation', 'ultraviolet', 'light', '221.', 'isomerization', 'cis', 'trans', 'back', 'azo', 'groups', 'however', 'mechanism', 'affect', 'photo-viscosity', 'change', 'polymeric', 'solutions', 'thus', 'reversible', 'solution', 'viscosity', 'changes', 'also', 'observed', '222', 'solutions', 'poly(dimethylacrylamide)', 'pendant', 'triphenylmethane', 'leucohydroxide', 'methanol', 'this', 'illustrated', 'follows', 'x', 'o', 'o', 'nh2', 'nh', 'x', 'o', 'o', 'nh2', 'nh', 'light', 'heat', 'n', 'c', 'oh', 'n', '2', 'oh', 'c', '2', 'light', 'irradiation', 'copolymer', 'contains', 'so', 'shown', 'viscosity', 'changes', 'due', 'positive', 'charges', 'form', 'pendant', 'groups', 'the', 'solubility', 'copolymer', 'styrene', 'cyclohexane', 'found', 'change', 'reversibly', 'upon', 'ultraviolet', '(2', 'mol)', 'spirobenzopyran', 'among', 'pendant', 'groups', '223.', 'this', 'believed', 'due', 'photo-isomerization', 'pendant', 'spirobenzopyran', 'groups', 'polar', 'merocyanine', 'form', 'resultant', 'decrease', 'polymer–solvent', 'interaction', 'subsequent', 'precipitation', 'higher', 'molecular', 'weight', 'fractions', 'polymer', 'a', 'copolymer', 'high', 'content', 'spirobenzopyran', 'groups', '(123', 'mol)', 'performs', 'negative', 'photo-resist', 'high', 'contrast', '223.', 'small', 'amounts', '1064', 'application', 'optical', 'data', 'storage', 'due', 'possible', 'utilization', 'photo-induced', 'orientation', 'polymeric', 'ﬁlms', 'optical', 'data', 'storage', 'phenomenon', 'quadratic', 'nonlinear', 'optical', 'effects', 'extensively', 'investigated', 'last', 'years', 'it', 'reported', 'instance', 'study', 'photo-isomerization', 'polymeric', 'environment', '106', 'photo-responsive', 'polymers', '761', 'series', 'polymers', 'containing', 'azo', 'dyes', 'large', 'differences', 'second', 'order', 'transition', 'temperature', 'compared', '225.', 'particular', 'emphasis', 'placed', 'relationship', 'photo-isomerization', 'tg', 'polymers', 'molecular', 'structure', 'as', 'result', 'shown', 'light-induced', 'nonpolar', 'orientation', 'high', 'tg', 'polyimides', '(tg', '350(cid1)c)', 'take', 'place', 'even', 'room', 'temperature', 'the', 'polymers', 'used', 'one', 'studies', 'illustrated', 'follows', '225:', 'r', 'r', 'o', 'o', 'n', 'o', 'o', 'n', 'n', 'o', 'o', 'o2n', 'n', 'n', 'n', 'n', 'o', 'n', 'o', 'n', 'n', 'n', 'n', 'no2', 'r', '¼', '(cf3)2–c', 'one', 'polymer', '–coo–(ch2)2–coo–', 'another', '762', '10', 'polymeric', 'materials', 'special', 'applications', 'from', 'behavior', 'mean', 'absorbance', 'concluded', 'azo', 'chromophores', 'revert', 'trans', 'form', 'completion', 'thermal', 'back', 'reaction', 'the', 'observed', 'increase', 'dichroic', 'ratio', 'ﬁrst', '25', 'h', 'believed', 'due', 'thermal', 'back', 'isomerization', 'due', 'relaxation', 'induced', 'orientation', '225.', 'heating', 'polymers', '170(cid1)c', '1', 'h', 'fails', 'erase', 'green', 'light-induced', 'dichroism', 'samples', 'this', 'dichroism', 'however', 'completely', 'erased', 'heating', 'samples', 'tg', '10', 'min', 'irradiation', 'ﬁlms', 'incident', 'light', 'gives', 'holograms', '226.', '1065', 'liquid', 'crystalline', 'alignment', 'the', 'relationship', 'glass', 'transitions', 'mobility', 'isomerization', 'conﬁned', 'polymer', 'systems', 'aroused', 'much', 'interest', 'it', 'inﬂuenced', 'need', 'alignment', 'liquid', 'crystalline', 'ﬂat', 'panel', 'displays', 'displays', 'ﬁlms', 'polyimides', 'widely', 'used', 'the', 'surfaces', 'usually', 'treated', 'produce', 'uniform', 'alignment', 'liquid', 'crystals', 'suitable', '“pretilt”', 'angles', 'the', 'treatments', 'consisted', 'rubbing', 'process', 'velvet', 'fabrics', 'search', 'new', 'methods', 'however', 'led', 'develop-', 'ment', 'molecular', 'structures', 'undergo', 'alignment', 'upon', 'irradiation', 'linearly', 'polarized', 'uv', 'light', '222', '223.', 'polymer-stabilized', 'liquid', 'crystals', 'low-molar-mass', 'liquid', 'crystal', 'their', 'bulk', 'align-', 'ment', 'texture', 'stabilized', 'polymer', 'network', 'such', 'polymer', 'network', 'usually', 'low', 'concentration', '222', '223.', 'several', 'types', 'polarized-light-induced', 'liquid', 'crystalline', 'aligning', 'molecules', 'reported', 'literature', '223.', 'one', 'photo-alignment', 'material', 'poly(vinyl', 'cinnamate)', 'the', 'polymer', 'copolymers', 'reported', 'ability', 'align', 'thin', 'ﬁlms', 'direction', 'perpendicular', 'axis', 'linearly', 'polarized', 'ultraviolet', 'light', 'this', 'photo-alignment', 'mechanism', 'fully', 'elucidated', 'present', 'a', 'drawback', 'using', 'poly(vinyl', 'cinnamate)', 'copolymers', 'low', 'glass', 'transition', 'temperature', 'as', 'result', 'remain', 'mobile', 'treatment', 'chain', 'orientation', 'other', 'materials', 'higher', 'tg', 'therefore', 'needed', 'among', 'promising', 'ones', 'polyimides', 'they', 'form', 'liquid', 'crystal', 'alignment', 'layers', 'ﬂat', 'panel', 'displays', 'possess', 'good', 'optical', 'transparencies', 'adhesion', 'heat', 'resistance', 'dimensional', 'stability', 'good', 'insulators', 'there', 'various', 'reports', 'literature', 'preparations', 'soluble', 'photo-reactive', 'polyimides', 'cinnamate', 'chromophore', 'side', 'groups', 'thus', 'reported', 'lee', 'et', 'al', '224', 'prepared', 'photo-reactive', 'polyimide', 'cinnamate', 'chromophores', 'side', 'groups', 'f3c', 'cf3', 'o', 'o', 'n', 'n', 'o', 'o', 'o', 'n', 'o', 'o', 'o', '106', 'photo-responsive', 'polymers', '763', 'this', 'polyimide', 'claimed', 'thermally', 'stable', '340(cid1)c', 'glass', 'transition', 'temperature', '181(cid1)c', 'also', 'demonstrated', 'cinnamate', 'chromophores', 'upon', 'irradiation', 'linearly', 'polarized', 'ultraviolet', 'light', 'undergo', 'photo-isomerization', 'dimerization', 'in', 'addition', 'light', 'exposure', 'induces', 'anisotropic', 'orientation', 'polymer', 'main', 'chains', 'cinnamate', 'side', 'groups', 'ﬁlm', 'the', 'irradiated', 'ﬁlms', 'align', 'homogeneously', 'nematic', 'liquid', 'crystal', 'molecules', 'along', 'one', 'direction', 'angle', '107(cid1)', 'respect', 'polarization', 'the', 'liquid', 'crystalline', 'alignment', 'found', 'thermally', 'stable', '200(cid1)c', 'it', 'also', 'reported', '225', 'photo-reactivity', 'side-chain', 'liquid', 'crystalline', 'polymers', 'align', 'liquid', 'crystals', 'parallel', 'mode', 'perpendicularly', 'depending', 'degree', 'photoreac-', 'tion', 'polymers', 'presumably', 'particular', 'polymer', 'multiphoto-align', 'liquid', 'crystal', 'pattern', 'without', 'change', 'direction', 'linearly', 'polarized', 'uv', 'light', 'the', 'chemical', 'structure', 'aligning', 'polymer', 'depicted', 'follows', 'o', 'p', 'o', 'o', '6', 'o', 'n', 'o', 'n', '¼', '2', '6', 'another', 'approach', 'liquid', 'crystalline', 'alignment', 'it', 'concluded', '226,', 'therefore', 'liquid', 'crystals', 'align', 'parallel', 'perpendicular', 'incident', 'e', 'direction', 'photocross-linked', 'polymer', 'ﬁlm', 'changing', 'degree', 'reacted', 'cinnamoyl', 'group', 'that', 'controlled', 'irradiation', 'time', 'a', 'bias-tilt', 'angle', 'liquid', 'crystals', 'director', 'substrate', 'also', 'realized', 'controlling', 'irradiation', 'angle', 'light', '226.', 'based', 'photo-isomerization', 'azo', 'compounds', 'polymeric', 'materials', 'part', 'polymer', 'structure', '227.', 'in', 'recent', 'years', 'investigation', 'use', 'azobenzene-containing', 'polymers', 'liquid', 'crystalline', 'alignment', 'became', 'quite', 'thorough', 'potential', 'application', 'holographic', 'storage', 'well', 'optical', 'photonic', 'use', '228–230.', 'the', 'photo-alignment', 'liquid', 'crystalline', 'polymers', 'containing', 'azobenzene', 'groups', 'advantage', 'local', 'variation', 'orientation', 'order', 'due', 'pixel-wise', 'irradiation', 'this', 'process', 'reported', 'take', 'place', 'via', 'angular-dependent', 'excitation', 'series', 'cis-trans', 'photo-', 'isomerization', 'cycles', 'rotational', 'diffusion', 'within', 'steady', 'state', 'photoreaction', 'this', 'results', 'photochromic', 'side', 'group', 'becoming', 'oriented', 'perpendicularly', 'electric', 'ﬁeld', 'vector', 'incident', 'light', 'establishing', 'oblate', 'order', 'ﬁlms', 'thus', 'studies', 'exposure', 'ﬁlms', 'liquid', 'crystalline', 'polymers', 'azobenzene', 'side', 'chains', 'linearly', 'polarized', 'light', '436', 'nm', '231', 'show', 'successive', 'occurrences', 'uniaxial', 'in-plane', 'orientations', 'followed', 'out-of-plane', 'orientations', 'azobenzenes', '231.', 'two', 'kinds', 'orientation', 'modes', 'observed', 'these', 'possibly', 'extreme', 'cases', 'linearly', 'polarized', 'light', 'electric', 'vector', 'parallel', 'xz-plane', 'comes', 'along', 'z-direction', 'one', 'uniaxial', 'in-plane', 'orientation', 'azobenzene', 'dipole', 'moment', 'parallel', 'x', 'axis', 'x-', 'y-direction', 'out-of-plane', '(homeotropic)', 'one', 'toward', 'z-direction', 'marked', 'dependence', 'photo-', 'orientation', 'processes', 'ﬁlm', 'temperatures', 'observed', 'in-plane', 'orientation', 'generated', 'glassy', 'state', 'photo-orientation', 'higher', 'temperatures', 'slightly', 'transition', 'temperature', 'smectic', 'nematic', 'phases', 'gives', 'rise', 'distinct', 'transformations', 'in-plane', 'orientation', 'early', 'state', 'successive', 'out-of-plane', 'reorientations', '232.', 'these', 'orientations', 'illustrated', 'follows', '764', '10', 'polymeric', 'materials', 'special', 'applications', 'n', 'n', 'n', 'n', 'n', 'n', 'plane', 'plane', 'n', 'n', 'also', 'reported', '233,', 'irradiation', 'liquid', 'crystalline', 'polyester', 'azobenzene', 'side', 'chains', 'light', 'argon', 'laser', '(514', '488', '351', 'nm)', 'results', 'orientation', '4-cyano-40-', 'alkoxyazobenzene', 'side', 'groups', 'the', 'polyester', 'characterized', 'smectic', 'nematic', 'phases', 'shows', 'strong', 'tendency', 'form', 'j-aggregates', 'the', 'orientation', 'process', 'cooperative', 'whereas', 'light-induced', 'orientation', 'photochromic', 'moiety', 'causes', 'ordering', 'alkylene', 'spacers', 'even', 'main-chain', 'segments', 'direction', 'it', 'concluded', '226', 'probable', 'mechanism', 'two-step', 'process', 'angular-selective', 'transformation', 'bulky', 'trans-isomers', 'rod-like', 'cis-isomeric', 'formed', 'red', 'light', 'the', 'aligned', 'cis-azobenzene', 'side', 'groups', 'become', 'strongly', 'j-aggregated', 'very', 'high', 'values', 'dichroism', '08', 'birefringence', '03', 'generated', 'result', 'combination', 'photo-induced', 'orientation', 'process', 'thermotropic', 'self-organization', 'takes', 'place', 'simultaneously', 'irradiation', 'conditions', 'the', 'process', 'results', 'uniaxial', 'prolate', 'order', 'ﬁlm', 'whereas', 'conventional', 'photo-', 'orientation', 'leads', 'biaxial', 'oblate', 'order', '93.', 'thus', 'direction', 'photo-induced', 'orientation', 'type', 'three-dimensional', 'orientation', 'order', 'controlled', 'wavelength', 'irradiating', 'linearly', 'polarized', 'light', 'this', 'also', 'done', 'ﬁlm', 'smetic', 'polyester', '4-cyano-40-alkoxyazobenzene', 'side', 'group', '233.', 'zhao', 'coworkers', '234', '235', 'reported', 'azobenzene', 'polymer', 'network', 'also', 'optically', 'align', 'ferroelectric', 'liquid', 'crystals', 'this', 'done', 'dissolving', 'two', 'chiral', 'dimethacrylate', 'one', 'chiral', 'diacrylate', 'monomers', 'containing', 'azobenzene', 'groups', 'commercial', 'ferroelectric', 'liquid', 'crystal', 'host', 'the', 'monomers', 'illustrated', 'follows', 'n', 'o', 'o', 'o', 'n', 'n', 'o', 'o', 'o', 'n', 'o', 'o', 'o', 'n', 'n', 'o', 'o', 'o', 'methacrylate', 'acrylate', '106', 'photo-responsive', 'polymers', '765', 'o', 'o', 'o', 'n', 'n', 'o', 'o', 'o', 'o', 'the', 'monomers', 'thermally', 'polymerized', 'simultaneously', 'irradiated', 'linearly', 'polarized', 'light', 'two', 'monomers', 'able', 'induce', 'bulk', 'alignment', 'liquid', 'crystals', 'direction', 'perpendicular', 'polarized', 'light', 'monomer', '1', 'effective', 'concentrations', 'low', '1.', 'it', 'also', 'concluded', 'experimental', 'evidence', 'photo-induced', 'bulk', 'alignment', 'ferroelectric', 'liquid', 'crystals', 'may', 'take', 'place', 'mechanism', 'different', 'one', 'takes', 'place', 'achiral', 'azopolymers', 'in', 'addition', 'zhao', 'coworkers', '236', 'reported', 'photo-induced', 'alignment', 'ferroelectric', 'liquid', 'crystals', 'using', 'azobenzene', 'polymer', 'networks', 'polyethers', 'polyepoxides', 'bulk', 'alignment', 'achieved', 'polymerizing', 'several', 'divinyl', 'ethers', 'diepoxide', 'monomers', 'bearing', 'azobenzene', 'moiety', 'here', 'thermal', 'polymerizations', 'conducted', 'solution', 'within', 'ferroelectric', 'liquid', 'crystals', 'exposing', 'reaction', 'mixture', 'linearly', 'polarized', 'irradiation', 'the', 'monomers', 'shown', 'follows', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'n', 'n', 'n', 'n', 'cn', 'o', 'o', 'o', 'o', 'o', 'n', 'n', 'o', 'o', 'n', 'n', 'o', 'cn', 'o', 'o', 'polymerization', 'monomers', 'achieved', 'cationic', 'mechanism', 'the', 'monomers', 'also', 'found', 'capable', 'inducing', 'stabilizing', 'bulk', 'alignment', 'liquid', 'crystals', 'zhao', 'coworkers', '234–237', 'concluded', 'however', 'mechanism', 'action', 'might', 'different', 'one', 'obtained', '766', '10', 'polymeric', 'materials', 'special', 'applications', 'chiral', 'azobenzene', 'polymethacrylates', 'instead', 'results', 'suggest', 'mechanism', 'might', 'based', 'formation', 'anisotropic', 'azobenzene', 'polyether', 'polyepoxide', 'network', 'in', 'still', 'another', 'subsequent', 'work', 'zhao', 'coworkers', '237', 'used', 'block', 'copolymers', 'composed', 'polystyrene', 'liquid', 'crystalline', 'azobenzene-containing', 'polymethacrylate', 'copolymers', 'model', 'system', 'o', 'o', 'n', 'o', 'o(ch2)6', 'o', 'n', 'n', 'och3', 'investigate', 'conﬁnement', 'effects', 'photo-alignment', 'photochemical', 'phase', 'transition', 'thermochromic', 'behavior', 'azobenzene', 'polymer', 'the', 'study', 'showed', '237', 'conﬁned', 'microphase-separated', 'domains', 'diblock', 'copolymers', 'azobenzene', 'polymer', 'behaves', 'differently', 'homopolymer', 'free', 'conﬁnement', 'the', 'conﬁnement', 'effects', 'manifested', '(1)', 'decreased', 'photo-induced', 'thermally', 'enhanced', 'orientation', 'azobenzene', 'transformation', 'liquid', 'mesogenic', 'groups', 'different', 'aggregation', 'states', 'crystalline', 'phase', 'isotropic', 'state', 'triggered', 'trans–cis', 'photo-isomerization', 'slower', 'recovery', 'liquid', 'crystalline', 'phase', 'thermally', 'induced', 'cis-trans', 'back', 'isomerization', '(3)', 'severely', 'reduced', 'even', 'suppressed', 'changes', 'aggregation', 'states', 'azobenzene', 'groups', 'heating', 'origin', 'thermochromic', 'property', 'the', 'common', 'cause', 'conﬁne-', 'ment', 'effects', 'restriction', 'imposed', 'conﬁning', 'geometry', 'either', 'order-disorder', 'disorder-order', 'reorganization', 'process', 'involving', 'motion', 'rearrangement', 'azobenzene', 'groups', '237.', '(2)', 'slower', 'zettsu', 'seki', 'reported', '238', 'preparation', 'group', 'azobenzene-containing', 'polymers', 'used', 'photo-induced', 'surface', 'relief', 'formations', 'these', 'soft', 'liquid', 'crystalline', 'azobenzene-', 'containing', 'copolymers', 'acrylate', 'methacrylate', 'monomers', 'bearing', 'oligo(ethylene', 'oxide)', 'chains', 'the', 'copolymers', 'display', 'smectic', 'liquid', 'crystal', 'phase', 'room', 'temperature', 'after', 'preexposure', 'ultraviolet', 'light', 'thin', 'ﬁlms', 'liquid', 'crystalline', 'polymers', 'show', 'highly', 'sensitive', 'photo-induced', 'material', 'transfer', 'generate', 'surface', 'relief', 'structures', 'the', 'typical', 'exposure', 'dose', 'required', 'full', 'polymer', 'migration', 'low', '50', 'mjcm2', 'o', 'o', '10', 'n', 'n', 'ch3', '5', 'o', 'ox', 'o', '1-x', 'o', 'r', 'n', 'the', 'inscribed', 'surface', 'relief', 'structures', 'rapidly', 'fully', 'erased', 'either', 'irradiation', 'incoherent', 'nonpolarized', 'ultraviolet', 'light', 'heating', 'close', 'clear', 'point', 'soft', 'liquid', 'crystalline', 'polymers', 'it', 'also', 'possible', 'chemically', 'cross-link', 'polymers', 'mixed', 'vapors', 'hydrogen', 'chloride', 'formaldehyde', 'surface', 'relief', 'inscription', 'this', 'results', 'drastic', 'improvement', 'shape', 'stability', 'maintaining', 'structure', 'high', 'temperatures', '250(cid1)c', 'after', 'cross-linking', 'trans-', 'to-cis', 'photo-isomerization', 'readily', 'proceeds', 'without', 'modiﬁcation', 'surface', 'morphology', 'therefore', 'applied', 'photo-switchable', 'alignment', 'nematic', 'liquid', 'crystals', '238.', '107', 'photo-conducting', 'polymers', '767', '107', 'photo-conducting', 'polymers', 'unless', 'polymers', 'contain', 'long', 'sequences', 'double', 'bonds', 'fairly', 'good', 'insulators', 'particularly', 'dark', 'nevertheless', 'number', 'common', 'polymers', 'show', 'measurable', 'increase', 'conductivity', 'irradiated', 'light', 'when', 'polymeric', 'materials', 'like', 'poly(vinyl', 'ﬂuoride)', 'poly(vinyl', 'acetate)', 'poly(vinyl', 'alcohol)', 'poly(n-vinyl', 'carbazole)', 'exposed', 'light', 'develop', 'charged', 'species', 'the', 'species', 'migrate', 'electric', 'ﬁeld', 'thus', 'conduct', 'electricity', 'when', 'poly(n-vinyl', 'carbazole)', 'doped', 'photosensitizers', 'compounds', 'form', 'charge-transfer', 'complexes', 'photosensitivity', 'increased', 'even', 'extended', 'visible', 'region', 'spectrum', 'since', 'discovery', '1957', 'poly(n-vinyl', 'carbazole)', 'photoconductive', 'properties', 'increasing', 'interest', 'synthesis', 'study', 'polymeric', 'materials', 'similar', 'properties', 'allow', 'various', 'photonic', 'applications', 'related', 'polymers', 'presently', 'utilized', 'photocopiers', 'laser', 'printers', 'electro-photographic', 'printing', 'plates', 'photoconductive', 'polymers', 'p-type', '(hole-transporting)', 'n-type', '(electron-transporting)', 'bipolar', '(capable', 'transporting', 'holes', 'electrons)', 'to', 'date', 'photoconductive', 'charge-', 'transporting', 'polymers', 'used', 'commercially', 'p-type', 'poly(vinyl', 'carbazole)', 'vinyl', 'derivatives', 'polynuclear', 'aromatic', 'polymers', 'poly', '(2-vinyl', 'carbazole)', 'poly(vinyl', 'pyrene)', 'high', 'photoconductive', 'efﬁciencies', 'these', 'materials', 'may', 'take', 'helical', 'conformation', 'successive', 'aromatic', 'side', 'chains', 'arranged', 'parallel', 'stack', 'in', 'arrangement', 'transfer', 'electrons', 'facilitated', 'also', 'believed', 'primary', 'mechanism', 'poly(vinylcarbazole)', 'charge', 'carrier', 'generation', 'due', 'excitation', 'carbazole', 'rings', 'ﬁrst', 'excited', 'singlet', 'state', 'this', 'polymer', 'absorbs', 'ultraviolet', 'light', '360-nm', 'region', 'forms', 'exciton', 'ionizes', 'electric', 'ﬁeld', 'the', 'excited', 'state', 'conductive', 'species', 'the', 'addition', 'equivalent', 'amount', 'electron', 'acceptor', 'like', '247-', 'trinitroﬂuorenone', 'shifts', 'absorption', 'polymer', 'visible', 'region', 'virtue', 'formation', 'charge', 'transfer', 'states', 'the', 'material', 'becomes', 'conductive', '550', 'nm', 'this', 'associated', 'electron-', 'positive', 'hole', 'pair', 'migrate', 'solid', 'polymeric', 'material', 'upon', 'dissociation', 'pair', 'charged', 'species', 'electron', 'positively', 'charged', 'hole', 'electron', 'becomes', 'conductive', 'state', 'to', 'achieve', 'additional', 'energy', 'required', 'result', 'singlet–singlet', 'interaction', '239,', 'singlet–triplet', 'interaction', '240,', 'singlet–photon', 'interaction', '239,', 'triplet–photon', 'interaction', '239,', 'two–photon', 'interaction', '240.', 'kepler', 'carried', 'ﬂuorescence', 'quenching', 'studies', 'concluded', 'migration', 'exciton', 'probable', 'energy', 'transfer', 'mechanism', 'poly(vinyl', 'carbazole)', '241.', 'he', 'furthermore', 'suggested', 'exiton', 'visit', '1000', 'monomer', 'units', 'lifetime', '241.', 'this', 'distance', '200', 'a˚', 'kang', 'coworkers', '242', 'also', 'explored', 'steady', 'state', 'pulsed', 'photo-conductivities', '4–8', 'mm', 'thick', 'ﬁlms', 'trans-polyphenylacetylene', 'also', 'trans-polyphenylacetylene', 'ﬁlms', 'doped', 'inorganic', 'organic', 'electron', 'acceptors', 'particularly', 'iodine', '23-dichloro-56-dicyano-p-', 'modulated', 'shallow', 'electron', 'traps', 'undoped', 'polymer', 'trapping', 'charge-transfer', 'complex', 'doped', 'polymer', '242.', 'guellet', '94', 'states', 'photo-conductivity', 'equal', 'current', 'density', 'j', 'divided', 'applied', 'ﬁeld', 'strength', 'e', 'j', 'apertureunit', 'electrode', 'area', 'this', 'related', 'number', 'negative-charge', 'carriers', '(usually', 'electrons)', 'per', 'unit', 'volume', 'p', 'number', 'positive', 'charge', 'carriers', '(or', 'positive', 'holes)', 'per', 'unit', '94', '¼', 'je', '¼', 'nemn', 'þ', 'pemp', '768', '10', 'polymeric', 'materials', 'special', 'applications', 'e', 'charge', 'electron', 'mn', 'mp', 'mobilities', 'negative', 'positive', 'carriers', 'respectively', 'photo-conductivity', 'mobility', 'charge', 'carrying', 'species', 'deter-', 'mined', 'relationship', '94:', 'thickness', 'ﬁlm', 'v', 'applied', 'voltage', 'carrier', 'drift', 'time', 'the', 'photo-effect', 'evaluated', 'terms', 'effective', 'gain', 'g', 'it', 'represents', 'number', 'generated', 'carriers', 'reaching', 'external', 'circuit', 'per', 'unit', 'time', 'compared', 'number', 'photons', 'absorbed', 'time', '94:', '¼', 'd2vt', 'g', '¼', 'jpei0ð1', '(cid7)', 'tþa', 'jp', 'photocurrent', 'e', 'electric', 'charge', 'i0', 'number', 'incident', 'photons', 'per', 'cm2s', 't', 'optical', 'transmittance', 'ﬁlm', 'a', 'area', 'sample', 'illuminated', '1071', 'photoconductive', 'polymers', 'based', 'carbazole', 'as', 'stated', 'primary', 'mechanism', 'charge-carrier', 'generation', 'poly(vinyl', 'carbazole)', 'appears', 'due', 'excitation', 'carbazole', 'rings', 'excited', 'singlet', 'states', '112.', 'while', 'singlet', 'excited', 'state', 'conductive', 'species', 'conductivity', 'believed', 'result', 'associated', 'electron-positive', 'hole', 'pair', 'migrating', 'solid', 'polymeric', 'material', 'dissociation', 'electron', 'pair', 'produces', 'separate', 'electron', 'positive', 'hole', 'way', 'electron', 'ends', 'conducting', 'state', '94.', 'this', 'requires', 'acquisition', 'energy', 'one', 'way', 'accomplished', 'exciton–surface', 'interaction', '243.', 'regensburger', 'published', 'absorption', 'spec-', 'trum', 'ﬂuorescence', 'spectrum', 'photocurrent', 'spectrum', '76', 'mm', 'ﬁlms', 'poly(n-vinyl', 'carbazole)', '244.', 'the', 'shape', 'response', 'photoconductor', 'wavelength', 'light', 'ﬂash', 'close', 'shape', 'absorption', 'spectra', 'bauser', 'klopffer', 'explain', 'result', 'interaction', 'singlet', 'excitons', 'trapped', 'holes', '245.', 'lyoo', 'used', 'low-temperature', 'initiator', '220-azobis(24-dimethyl-valeronitrile)', 'polymerize', 'n-', 'vinyl', 'carbazole', 'heterogeneous', 'solution', 'mixture', 'methyl', 't-butyl', 'alcohols', '247.', 'the', 'polymer', 'formed', 'mn', 'molecular', 'weights', '3', '(cid6)', '106', 'the', 'author', 'emphasized', 'method', 'provides', 'ultrahigh-molecular-weight', 'polymer', 'conversions', 'greater', '80.', 'n', 'n', 'the', 'optical', 'transparency', 'poly(vinyl', 'carbazole)', 'ﬁlms', 'produced', 'room', 'temperature', 'process', 'appears', 'quite', 'high', 'although', 'transparency', 'decreases', 'high', 'conversions', 'in', 'ﬁlm', 'form', 'material', 'useful', 'photoconductors', 'charge-transfer', 'complexes', 'electroluminescent', 'devices', 'the', 'higher', 'polymer', 'molecular', 'weight', 'typically', 'enhances', 'ﬁlm', 'mechanical', 'properties', '247.', '107', 'photo-conducting', 'polymers', '769', 'horhold', 'rathe', '248', 'reported', 'prepared', 'poly(9-methylcarbazole-36-diyl-', '12dipenylvinylene)', 'the', 'polymer', '(mn', '¼', '10000)', 'formed', 'dehalogenating', 'polycondensation', '36-bis(aa-dichlorobenzyl)-109-methylcarbazole', 'chromiun(ii)', 'acetate', 'this', 'polymer', 'found', 'also', 'highly', 'photoconductive', 'its', 'dark', 'conductivity', 'increases', 'doping', 'arsenic', 'pentaﬂuoride', '248.', 'photoconductive', 'polymers', 'copolymers', 'also', 'synthesized', 'haque', 'et', 'al', '249', '2-(9-', 'carbazolyl)-1-propenyl', 'vinyl', 'ether', 'the', 'polymers', 'copolymers', 'form', 'cationic', 'polymerization', 'mechanism', 'using', 'boron', 'triﬂuoride', 'ethylalumionum', 'dichloride', 'catalysts', '249.', 'charge', 'transfer', 'complexes', 'also', 'form', 'poly(vinyl', 'carbazole)', 'acts', 'donor', 'poly', '2-(methacroyloxy)ethyl-457-trinitro-9-oxo-2-ﬂuorene-carboxylate', 'acts', 'acceptor', '250:', 'o', '3', 'n', 'o', 'o', '2', 'n', 'o', 'o', 'o2n', 'o2n', 'the', 'required', 'mole', 'ratio', 'components', 'complex', '11', '250.', 'charge', 'transfer', 'complexation', 'occurs', 'similar', 'manner', 'poly(2-carbazolylethyl', 'acrylate)', 'molecularly', 'doped', '247-trinitroﬂuorene', 'quantum', 'efﬁciency', 'hole', 'propagation', 'copolymer', '005–10', 'molar', 'ratio', 'trinitroﬂuorene', 'carbazole', 'chromophores', 'higher', 'corresponding', 'trinitroﬂuorene', 'ethyltrinitroﬂuorene', 'doped', 'homopolymer', 'poly(2-', 'carbazolylethyl', 'acrylate)', '251.', 'kim', 'webber', 'studied', 'delayed', 'emission', 'spectra', 'poly(vinyl', 'carbazole)', 'doped', 'dimethylterephthalate', 'pyrene', '252.', 'on', 'basis', 'results', 'concluded', 'room', 'temperature', 'dimethylterephthalate', 'completely', 'quench', 'triplet', 'excitation', 'state', 'poly', '(vinyl', 'carbazole)', 'they', 'also', 'concluded', 'phosphorescent', 'states', 'poly(vinyl', 'carbazole)-', 'dimethylterephthalate', 'similar', 'implying', 'signiﬁcant', 'charge-transfer', 'character', 'former', 'in', '1985', 'polymeric', 'triphenylmethane', 'dyes', 'based', 'condensation', 'polymers', 'polyesters', 'polyurethanes', 'prepared', '252:', '770', '10', 'polymeric', 'materials', 'special', 'applications', 'o', 'o', 'n', 'o', 'cl', 'n', 'o', 'o', 'o', 'n', 'these', 'dyes', 'shown', 'sensitize', 'photo-conductivity', 'poly(n-vinyl', 'carbazole)', 'polyacetylene', 'derivatives', 'exhibit', 'unique', 'characteristics', 'semiconductivity', 'high', 'gas', 'per-', 'meability', 'helix', 'inversion', 'nonlinear', 'optical', 'properties', '253.', 'attempts', 'made', 'therefore', 'incorporate', 'carbazole', 'polyacetylene', 'hope', 'attaining', 'enhance', 'properties', '253.', 'n', 'n', 'n', 'n', 'n', 'it', 'found', '122', 'current', 'conducted', 'polymer', 'irradiation', '40–50', 'times', 'higher', 'dark', 'on', 'hand', 'electron', 'mobility', 'di-t-butylcarbazolepo-', 'lyacetylene', '(shown', 'above)', 'lower', 'poly(vinyl', 'carbazole)', 'this', 'attributed', 'bulkiness', 'butyl', 'groups', '254.', 'siloxanes', 'pendant', 'carbazole', 'groups', 'synthesized', 'strohriegl', '254', 'following', 'technique', 'x', 'n', 'si', 'o', 'si', 'n', 'o', 'x2', 'the', 'material', 'however', 'turn', 'photoconductive', 'the', 'photo-conductivity', 'copolymers', 'also', 'investigated', 'copolyacrylates', 'pendant', 'donor', 'acceptor', 'chromophores', '107', 'photo-conducting', 'polymers', '771', '2-n-457-trinitroﬂuorenone-2-ethyl', 'acrylate', 'exhibit', 'photo-generation', 'properties', '254.', 'these', 'copolymers', 'n', 'x', 'o', 'n', 'o', 'x', 'o', 'o2n', 'no2', 'no2', 'o', 'n', 'o', 'o', 'o', 'o', 'o', 'n', 'ch3', 'no2', 'no2', 'prepared', 'free-radical', 'copolymerization', 'appropriate', 'monomers', 'photo-conductivity', 'visible', 'obtained', 'charge', 'transfer', 'complexation', 'similar', 'work', 'done', 'earlier', 'natansohn', '255,', 'copolymerized', 'n-methyl', '13-', 'hydroxymethyl', 'carbazolyl', 'acrylate', 'acryloyl-30-hyroxypropyl-35-dinitrobenzoate', 'illumination', 'copolymer', 'samples', 'induces', 'certain', 'degree', 'ionization', 'accompanied', 'proton', 'transfer', 'there', 'permanent', 'increase', 'quantity', 'radicals', 'generated', 'light', 'another', 'number', 'radicals', 'apparently', 'forms', 'illumination', 'disappears', 'dark', '254.', 'it', 'reported', '256', 'polymer', 'formed', 'condensation', 'n-(3-isopentyl)-36-diformyl-', 'carbazole', '4-14-bis(-aminoethyl)aminophenyl-azonitrobenzene', 'yields', 'new', 'polyazomethine', 'carbazole-azo', 'polymer', 'the', 'product', 'soluble', 'organic', 'solvents', 'the', 'polymer', 'possesses', 'carbazole', 'moieties', 'azo', 'type', 'nonlinear', 'optical', 'chromophores', 'backbone', 'it', 'shows', 'high', 'photo-', 'conductivity', 'nonlinear', 'optical', 'properties', '1072', 'photo-conducting', 'polymers', 'that', 'are', 'not', 'based', 'carbazole', 'when', 'spiropyran', 'incorporated', 'plasticized', 'poly(vinyl', 'chloride)', 'membranes', 'placed', 'two', 'identical', 'nacl', 'solutions', 'irradiated', 'ultraviolet', 'light', 'long', 'periods', 'potential', 'decreases', '257.', 'this', 'shown', 'ryba', 'petranek', 'result', 'spiran', 'ring', 'opening', '257:', '772', '10', 'polymeric', 'materials', 'special', 'applications', 'n', 'r', 'no2', 'uv', 'light', 'visible', 'light', 'no2', 'n', 'r', 'o', 'when', 'irradiation', 'interrupted', 'membrane', 'irradiated', 'visible', 'light', 'potential', 'returns', 'original', 'value', 'ring', 'closes', 'back', 'spiran', 'structure', '125.', 'the', 'addition', 'electron', 'donors', 'like', 'dialkyl', 'aniline', 'kapton', 'polyimide', 'ﬁlm', 'o', 'n', 'o', 'o', 'n', 'o', 'o', 'n', 'results', 'enhancement', 'photocurrent', 'much', 'ﬁve', 'orders', 'magnitude', 'compared', 'virgin', 'material', '258.', 'freulich', 'explains', 'mechanism', 'enhancement', 'result', 'radiation', 'absorption', 'charge-transfer', 'complex', 'formed', 'added', 'electron', 'donor', 'imide', 'portion', 'polymer', 'backbone', 'excitations', 'followed', 'rapid', 'complete', 'electron', 'transfer', 'donor', 'pyromelitimide', 'yield', 'radical', 'anion', 'polymer', 'radical', 'cation', 'donor', '258.', 'these', 'species', 'undergo', 'rapid', 'back-electron', 'transfer', 'in', 'words', 'dialkyl', 'aniline', 'donates', 'one', 'unpaired', 'electrons', 'typical', 'photo-reduction', 'reaction', 'carbonyl', 'group', 'the', 'reaction', 'reversible', 'photo-conduction', 'mechanism', 'ion', 'radical', 'returning', 'ground', 'state', 'studies', 'thermochromic', 'solvatochromic', 'photoconductive', 'properties', '3-', '4-', '6-', '9-', 'poly(butoxycarbonylinethyl', 'urethane)-diacetylenes', 'result', 'changes', 'rod', '(red', 'blue)', 'coil', '(yellow)', 'conformations', 'polymer', 'backbone', '259.', 'photo-excitations', 'solutions', 'polymers', 'rod', 'state', 'result', 'large', 'transient', 'photo-conductivity', 'small', 'conductivity', 'signals', 'observed', 'coil', 'state', 'the', 'thermochromic', 'shift', 'occurs', 'going', 'rod', 'state', 'room', 'temperature', 'coil', 'state', '65(cid1)c', 'accompanied', 'decrease', 'photo-', 'conductivity', 'the', 'large', 'conductivity', 'signal', 'rod', 'state', 'attributed', 'formation', 'mobile', 'charge', 'carriers', 'possibly', 'via', 'interchain', 'charge', 'transfer', 'within', 'aggregates', 'the', 'decay', 'photo-', 'conductivity', 'nonexponential', 'extends', 'microseconds', 'wong', 'et', 'al', '260', 'reported', 'prepared', 'soluble', 'rigid-rod', 'organometallic', 'polymer', 'containing', 'electron-donating', 'electron-withdrawing', 'trans--pt-(tibutylphosphine)2-acetylene-r-acetylene-n', '(where', 'r', '¼', 'bithiazol-ediyl)', 'groups', 'the', 'polymer', 'formed', 'cuprous', 'iodide-catalyzed', 'dehydroha-', 'logenation', 'reaction', 'the', 'electron-donating', 'electron-withdrawing', 'properties', 'thiazole', 'ring', 'confer', 'solubility', 'polymer', 'this', 'polyacetylene', 'luminescent', 'singlet', 'emission', 'peak', '539', 'nm', 'photo-conducting', 'the', 'glass', 'transition', 'temperature', 'polymer', '215(cid1)', 'shows', 'relatively', 'good', 'thermal', 'stability', 'the', 'p-conjugation', 'ligands', 'extends', 'metal', 'core', 'absorption', 'peaks', 'show', 'signiﬁcant', 'red-shift', '17–26', 'nm', 'compared', 'bithienyl', 'counterparts', 'due', 'presence', 'electron-withdrawing', 'imine', 'nitrogen', 'atoms', '260.', 'kimura', 'et', 'al', '259', 'reported', 'applying', 'organic', 'photochromic', 'compounds', 'photochemical', 'switching', 'metal-ion', 'complexation', 'ionic', 'conduction', 'combining', 'photochromism', 'metal-ion', 'binding', 'property', 'crown', 'ether', 'derivatives', 'they', 'synthesized', 'vinyl', 'polymers', 'incorporating', 'crowned', 'spirobenzopyran', 'moiety', 'side', 'chain', '107', 'photo-conducting', 'polymers', '773', 'o', 'o', 'n', 'o', 'o', '1-x', 'o', 'och3', 'n', 'o', 'no2', 'ch3', 'ch3', 'the', 'crowned', 'spiropyran', 'electrically', 'neutral', 'form', 'bind', 'alkali', 'metal', 'ion', 'crown', 'ether', 'moiety', 'at', 'time', 'spirobenzopyran', 'portion', 'isomerizes', 'corresponding', 'merocyanine', 'form', 'photo-chemically', 'the', 'zwitterionic', 'merocyanine', 'form', 'crowned', 'spirobenzopyran', 'moiety', 'brings', 'signiﬁcant', 'change', 'metal-ion', 'binding', 'ability', 'this', 'prompted', 'authors', 'apply', 'compound', 'photo-responsive', 'ion-conductive', 'materials', 'they', 'observed', 'ion-', 'conductivity', 'increased', 'ultraviolet', 'light', 'decreased', 'visible', 'light', '259.', 'chan', 'coworkers', '269', 'prepared', 'polystyrenes', 'poly(methyl', 'methacrylate)s', 'contain', 'metal', 'complex', 'cores', 'o', 'o', 'n', 'm', 'n', 'o', 'm', 'o', 'h', 'h', 'oc', 're', 'cl', 'oc', 'co', 'h', 'h', 'o', '2pf6', 'o', 'm', 'ru', 'n', 'n', '774', '10', 'polymeric', 'materials', 'special', 'applications', 'when', 'polymers', 'doped', 'hole-transporting', 'triphenylamine', 'enhancement', 'photo-', 'conductivity', 'visible', 'region', 'observed', 'this', 'suggests', 'metal', 'complexes', 'serve', 'photosensitizers', 'instead', 'charge', 'carriers', '260.', 'chan', 'et', 'al', '260', 'observed', 'electric', 'ﬁeld-dependent', 'charge', 'separation', 'process', 'polymers', 'it', 'described', 'well', 'onsager’s', 'theory', 'charge', 'germinate', 'recombination', 'this', 'theory', 'assumes', 'fraction', 'absorbed', 'photons', 'produce', 'bound', 'thermalized', 'electron-hole', 'pairs', 'either', 'recombine', 'dissociate', 'combined', 'effects', 'coulombic', 'attraction', 'electric', 'ﬁeld', 'the', 'photo-generation', 'efﬁciency', 'given', 'product', 'quantum', 'yield', 'thermalized', 'pair', 'formation', 'pair', 'dissociation', 'probability', 'fðr0', 'eþ', '¼', 'f0', '1', '(cid7)', 'eer0', 'kt', '(cid2)', '(cid3)', '(cid7)1', '1', 'ig', 'e2', '4pe0erktr0', 'eer0', 'kt', 'ig', '(cid3)', '(cid2)', '(cid3)', 'g¼0', 'x', '(cid2)', 'ig', 'recursive', 'formula', 'given', 'igþ1ðxþ', '¼', 'igðxþ', '(cid7)', 'xgþ1', 'expð(cid7)xþðg', 'þ', '1þ', 'i0ðxþ', '¼', '1', '(cid7)', 'expð(cid7)xþ', 'f', 'primary', 'yield', 'thermalized', 'bound', 'pairs', 'r0', 'initial', 'thermalization', 'separation', 'bound', 'charges', 'er', 'relative', 'permeability', 'e', 'applied', 'electric', 'ﬁeld', 'strength', 'jenekhe', 'de', 'paor', '261', 'reported', 'exciplex', 'formation', 'photoelectron', 'transfer', 'several', 'n-type', '(electron', 'accepting)', 'p-conjugated', 'rigid-rod', 'polymers', 'donor', 'triarylamine', 'molecules', 'in', 'particular', 'reported', 'investigation', 'n-type', 'conjugated', 'polymer', 'poly(benzimido', 'azobenzo-', 'phenanthroline', 'ladder)', '132.', 'no', 'evidence', 'observed', 'ground', 'state', 'charge', 'transfer', 'strong', 'interactions', 'conjugated', 'polymer', 'pairs', 'transient', 'absorption', 'spectra', 'blend', 'thin', 'ﬁlms', '420–730', 'nm', 'region', 'obtained', 'various', 'time', 'delays', 'following', 'photo-', 'excitation', '532', 'nm', 'dramatically', 'enhanced', 'photo-induced', 'bleaching', '430–480', 'nm', 'region', 'observed', 'jenekhe', 'de', 'paor', 'propose', 'observed', 'enhanced', 'photo-bleaching', 'blends', 'consequence', 'photo-induced', 'electron', 'transfer', '261.', 'the', 'electron', 'transfer', 'illustrated', 'follows', 'n', 's', 's', 'n', 'o', 'n', 'n', 'n', 'n', 'o', 'n', 'uv', 'light', 'o', 'n', 'n', 'n', 'n', 's', 's', 'n', 'n', 'n', 'n', 'n', 'o', '108', 'polymer-based', 'solar', 'cells', '775', 'molecular', 'materials', '2-(2-hydroxyphenyl)benzoxazole', '2-(2-hydroxyphenyl)-', 'benzothiazole', 'contain', 'intramolecular', 'hydrogen', 'bonds', 'known', 'undergo', 'excited', 'state', '(charge', 'transfer)', 'intramolecular', 'proton', 'transfer', 'upon', 'photo-excitation', '108', 'polymer-based', 'solar', 'cells', 'polymer-based', 'solar', 'cells', 'also', 'known', 'organic', 'photovoltaic', 'cells', 'around', 'since', '1990s', 'but', 'performance', 'efﬁciency', 'end', '2010', 'convert', 'light', 'electricity', 'much', 'research', 'reached', 'approximately', '8.', 'this', 'good', 'enough', 'compete', 'inorganic', 'solar', 'cells', 'like', 'based', 'cadmium', 'teluride', 'convert', '10–15', 'light', 'electricity', 'this', 'single', 'digit', 'value', 'organic', 'solar', 'cells', 'pales', 'even', 'compared', 'highly', 'specialized', 'high-priced', 'state-of-the-art', 'inorganic', 'devices', 'conversion', 'efﬁciencies', 'topping', '40.', 'the', 'promise', 'low-cost', 'organic', 'solar', 'cells', 'however', 'encouraged', 'intense', 'research', 'many', 'laboratories', 'efforts', 'improve', 'efﬁciency', 'such', 'research', 'usually', 'focuses', 'solution-', 'processable', 'organic', 'polymers', 'converted', 'semiconductors', '262.', 'the', 'polymers', 'used', 'regarded', 'intrinsic', 'wide', 'band', 'gap', 'semiconductors', 'band', 'gaps', '14', 'ev', 'this', 'compared', 'insulators', 'band', 'gaps', '3', 'ev', 'doping', 'ﬁlm', 'forming', 'materials', 'done', 'introduce', 'extrinsic', 'charge', 'carriers', 'convert', 'organic', 'semiconductors', 'such', 'charge', 'carriers', 'explained', 'sect', '102', 'positive', 'p-type', 'negative', 'n-type', 'originally', 'donor-acceptor', 'bilayer', 'device', 'two', 'ﬁlms', 'used', 'n–p', 'junction', 'solar', 'cells', 'thus', 'fabricated', 'sandwich', 'structures', 'an', 'example', 'would', 'one', 'transparent', 'substrate', 'ﬁrst', 'coated', 'conductor', 'like', 'indium-tin', 'oxide', 'a', 'conducting', 'polymer', 'like', 'poly', '(ethylene', 'dioxythiphene)', 'doped', 'polystyrene-sulfonic', 'acid', 'would', 'applied', 'aqueous', 'solution', 'the', 'indium-tin', 'oxide', 'acts', 'electrode', 'hole', 'injection', 'extraction', 'the', 'polymer', 'covered', 'conductor', 'aluminum', 'foil', 'the', 'doped', 'polymer', 'illustrated', 'follows', 'o', 'o', 'n', 's', 'n', 'semi', 'conducting', 'polymer', 'doping', 'material', 'so3h', 'the', 'construction', 'above-described', 'solar', 'cell', 'illustrated', 'follows', 'poly(ethylene', 'dioxythiophene)-polystyrene', 'sulfonic', 'acid', 'indium', 'tin', 'oxide', 'electrode', 'aluminum', 'foil', 'transparen', 'layer', 'the', 'donor', 'material', 'containing', 'chromophore', 'absorbs', 'light', 'energy', 'generates', 'excitons', 'excitons', 'high-energy', 'couples', 'energetic', 'electrons', 'bound', 'positively', 'charged', '776', '10', 'polymeric', 'materials', 'special', 'applications', 'electron', 'vacancies', 'holes', 'to', 'produce', 'electric', 'current', 'electron-hole', 'pairs', 'must', 'migrate', 'interface', 'electron', 'donor', 'electron', 'acceptor', 'materials', 'upon', 'reaching', 'interface', 'electron-hole', 'pairs', 'splits', 'separate', 'mobile', 'charges', 'the', 'charges', 'diffuse', 'respective', 'electrodes', 'the', 'electrons', 'transported', 'electron-accepting', 'material', 'cathode', 'holes', 'hole-accepting', 'material', '(electron', 'donor)', 'anode', 'to', 'put', 'words', 'coulomb-correlated', 'electron-hole', 'pair', 'excitons', 'diffuse', 'donor-acceptor', 'interface', 'exciton', 'dissociation', 'occurs', 'via', 'electron-transfer', 'process', 'n-type', 'layer', 'with', 'aid', 'internal', 'electric', 'ﬁeld', 'n-type', 'layer', 'carriers', 'electrons', 'opposite', 'direction', 'the', 'electric', 'ﬁeld', 'turn', 'generates', 'photocurrent', 'photo', 'voltage', 'such', 'devices', 'known', 'planar', 'heterojunction', 'cells', 'such', 'arrangement', 'however', 'efﬁcient', 'excitons', 'decay', 'back', 'ground', 'level', 'diffuse', 'n-type', 'layer', 'to', 'overcome', 'difﬁculty', 'concept', 'bulk', 'heterojunction', 'introduced', '263.', 'by', 'blending', 'donor', 'acceptor', 'materials', 'together', 'interpenetrating', 'bicontinuous', 'network', 'junctions', 'large', 'donor-acceptor', 'interfacial', 'areas', 'achieved', 'this', 'results', 'enhanced', 'quantum', 'efﬁciency', 'charge', 'separation', 'efﬁcient', 'charge', 'collection', 'gaudiana', '264', 'likened', 'morphology', 'bulk', 'heterojunction', 'active', 'layer', 'sponge', 'the', 'solid', 'part', 'represents', 'nano-sized', 'interconnected', 'bits', 'acceptors', 'the', 'polymer', 'represented', 'holes', 'intimately', 'connected', 'holes', 'throughout', 'sponge', 'never', 'far', 'solid', 'region', 'blending', 'phases', 'scale', 'effect', 'distributes', 'small', 'regions', 'interface', 'throughout', 'photoactive', 'layer', 'as', 'result', 'excitons', 'need', 'diffuse', 'short', 'distance', 'quickly', 'reaching', 'donor–acceptor', 'interface', 'dissociate', 'separate', 'charges', 'an', 'advancement', 'efﬁciency', 'polymeric', 'solar', 'cells', '3', '5,', 'came', '2009', 'observed', 'promising', 'efﬁcient', 'charge', 'transfer', 'materials', 'prepared', 'combinations', 'poly', '(alkyl-thiophenes)', 'donors', '1-(3-methoxycarbonyl)propyl-1phenyl-66-methanofullere', 'acceptors', '265.', 'mild', 'heating', 'disperses', 'acceptor', 'molecules', 'among', 'donor', 'molecules', 'this', 'led', 'exploration', 'many', 'combinations', 'various', 'polymers', 'different', 'derivatives', 'fullerine', 'various', 'chromophores', 'in', 'attempt', 'lower', 'highest', 'occupied', 'molecular', 'orbitals', '(hmo)', 'polymer', 'stronger', 'electron-withdrawing', 'groups', 'new', 'polymers', 'developed', 'the', 'results', 'summarized', 'review', '265.', '108', 'polymer-based', 'solar', 'cells', '777', 'as', 'result', 'research', 'several', 'research', 'groups', 'reported', 'attaining', '5', 'efﬁciencies', 'combinations', '1-(3-methoxycarbonyl)propyl-1phenyl-66-', 'methanofullere', 'numerous', 'conducting', 'polymers', 'including', 'copolymers', 'containing', 'ﬂuorene', 'car-', 'bazole', 'cyclopentadithiophene', 'investigated', 'thiophene)', 'poly(3-hexyl', 'the', 'efﬁciency', 'organic', 'solar', 'cells', 'usually', 'deﬁned', '(cid2)', '¼', 'ðjscvocffþpin', 'pin', 'input', 'energy', 'solar', 'radiation', 'the', 'output', 'short', 'circuit', 'current', 'density', 'jsc', 'voc', 'open', 'circuit', 'voltage', 'ff', 'ﬁll', 'factor', 'much', 'research', 'effort', 'date', 'based', 'attempts', 'increase', 'voc', 'this', 'based', 'empirical', 'correlation', 'magnitude', 'open', 'circuit', 'voltage', 'difference', 'energies', 'hmo', 'donor', 'lumo', 'acceptor', 'it', 'expected', 'lowering', 'hmo', 'donor', 'voc', 'increased', '266.', 'to', 'achieve', 'goal', 'two', 'research', 'teams', 'headed', 'yu', 'yang', 'tested', 'series', 'copolymers', 'prepared', 'reacting', 'benzodithiophene', 'derivative', 'various', 'thienothiophenes', 'the', 'aim', 'lower', 'polymers’', 'hmo', 'attaching', 'successively', 'stronger', 'electron-withdrawing', 'groups', 'polymer', 'backbone', 'the', 'result', 'replacing', 'alkoxy', 'group', 'adjacent', 'carbonyl', 'group', 'alkyl', 'chain', 'position', 'group', 'lowered', 'homo', 'level', 'roughly', '01', 'ev', 'they', 'lowered', 'level', 'another', '01', 'ev', 'adding', 'ﬂuorine', 'atom', 'solar', 'cells', 'prepared', 'polymer', 'found', '68', 'efﬁcient', '267.', 'subsequently', 'yu', 'et', 'al', 'reported', 'slightly', 'improved', 'conversion', 'efﬁciency', '7', '268.', 'it', 'interesting', 'time', 'publication', 'review', 'reported', 'chem', 'eng', 'news', '269', 'heeger', 'gong', 'developed', 'broad', 'spectrum', 'donor', 'acceptor', 'combination', 'detect', 'photons', 'throughout', 'whole', 'light', 'spectrum', 'ultra-violet', 'infra-red', 'the', 'combination', 'two', 'materials', 'illustrated', 'follows', 'n', 'n', 's', 's', 's', 's', 's', 'h21c10', 'c10h21', '778', '10', 'polymeric', 'materials', 'special', 'applications', 'according', 'subsequent', 'write', 'chem', 'eng', 'news', '270,', 'private', 'laboratory', 'called', '“solamer”', 'claimed', 'without', 'disclosing', 'details', 'achieved', 'efﬁciency', '813.', 'they', 'also', 'stated', 'hope', 'achieve', 'efﬁciency', '10', 'end', '2011', 'syntheses', 'various', 'polymers', 'solar', 'cells', 'also', 'reviewed', 'cheng', 'et', 'al', '271', 'point', 'need', 'develop', 'better', 'p-type', 'materials', 'good', 'ﬁlm', 'properties', 'act', 'efﬁcient', 'chromophores', 'good', 'hole', 'mobility', 'suitable', 'molecular', 'orbitals', 'levels', 'they', 'also', 'point', 'magnitude', 'band', 'gap', 'energy', 'positions', 'homo', 'lumo', 'energy', 'levels', 'important', 'characteristics', 'determining', 'optical', 'electrical', 'properties', 'given', 'conjugated', 'polymer', 'these', 'course', 'turn', 'greatly', 'inﬂuence', 'ultimate', 'photovoltaic', 'performance', 'conversion', 'light', 'energy', 'electrical', 'energy', 'the', 'wavelength', 'maximum', 'photon', 'ﬂux', 'density', 'solar', 'spectrum', 'located', 'approximately', '700', 'nm', 'corresponds', 'low', 'energy', '177', 'ev', 'the', 'absorption', 'spectrum', 'conjugated', 'polymer', 'cover', 'red', 'near-infra-red', 'ranges', 'match', 'greater', 'part', 'terrestrial', 'solar', 'spectrum', 'absorb', 'maximum', 'photon', 'ﬂux', 'thus', 'highly', 'desirable', 'develop', 'conjugated', 'polymers', 'broader', 'absorptions', 'narrowing', 'optical', 'band', 'gap', 'at', 'time', 'materials', 'must', 'efﬁciently', 'absorb', 'light', 'the', 'overall', 'high', 'extinction', 'coefﬁcients', 'polymers', 'also', 'critical', 'importance', 'following', 'presented', 'published', 'results', 'recent', 'current', 'research', 'illustrate', 'bulk', 'effort', 'ﬁeld', 'the', 'examples', 'chosen', 'random', 'implication', 'best', 'ones', 'published', 'date', 'li', 'coworkers', '272', 'reported', 'synthesis', 'photovoltaic', 'properties', 'three', 'donor-acceptor', 'copolymers', 'containing', 'bithiazole', 'acceptor', 'one', 'illustrated', 'follows', 'h17c8', 'c8h17', 'n', 's', 'n', 's', 's', 'n', 's', 'n', 'the', 'two', 'copolymers', 'similar', 'the', 'copolymer', 'yielded', 'best', 'results', 'carbazole', 'replaced', 'following', 'molecule', '108', 'polymer-based', 'solar', 'cells', '779', 's', 's', 's', 'the', 'results', 'indicate', 'donor', 'units', 'carbazole', 'inﬂuence', 'band', 'gaps', 'electronic', 'energy', 'levels', 'photovoltaic', 'levels', 'the', 'hole', 'mobility', 'measured', '307', '(cid6)', '10(cid7)4', 'cm2v', 'honda', 'et', 'al', '273', 'reported', 'injecting', 'photosensitizer', 'dye', 'bulk', 'heterojunction', 'solar', 'cell', 'based', 'regioregular', 'poly(3-hexylthiophene)', '1-(3-methoxycarbonyl)propyl-1phenyl-66-', 'methanofullere', 'the', 'dye', 'photosensitizer', 'illustrated', 'follows', 'n', 'r', 'si', 'r', 'n', 'n', 'n', 'n', 'n', 'r', 'o', 'si', 'n', 'n', 'they', 'reported', 'injection', 'dye', 'resulted', 'increase', 'photocurrent', 'hiorns', 'et', 'al', '274', 'reported', 'preparation', 'block', 'copolymer', 'incorporated', 'fullerene', 'molecules', 'backbone', 'polymer', 'they', 'observed', 'band', 'gap', '23', '22', 'ev', 'block', 'copolymer', 'li', 'coworkers', '275', 'reported', 'syntheses', 'four', 'alternating', 'copolymers', 'carbazole', 'triphenylamine', 'conjugated', 'side', 'chain', 'acceptor', 'groups', '780', '10', 'polymeric', 'materials', 'special', 'applications', 'h17c8', 'c8h17', 'n', 'n', 's', 'o', 'n', 'o', 'n', 'o', 'the', 'four', 'copolymers', 'synthesized', 'contained', 'different', 'acceptor', 'end', 'groups', 'aldehyde', 'monocyano', 'dicyano', '13-diethyl-2-thiobarbituric', 'acid', 'through', 'changing', 'acceptor', 'groups', 'electronic', 'properties', 'energy', 'levels', 'copolymers', 'effectively', 'tuned', 'their', 'results', 'indicate', 'effective', 'approach', 'tuning', 'bandgaps', 'conjugated', 'polymers', 'the', 'polymers', 'used', 'donors', 'polymer', 'solar', 'cells', 'they', 'reported', 'however', 'conversion', 'efﬁciency', '276.', 'you', 'coworkers', '276', 'reported', 'syntheses', 'two', 'low-band', 'gap', 'polymers', 'based', 'benzo(12-', 'b45-b)dithiophene', 's', 's', 's', 'r1', 'r1', 's', 'r2', 'n', 'n', 's', 'r2', 'r1', '¼', '3-butylnonyl', 'r2', '¼', 'nonyl', 'send', 'polymer', 'r1', '¼', '3-hexylundecyl', 'r2', '¼', 'hydrogen', 'both', 'polymers', 'reported', 'performed', 'well', 'preliminary', 'bulk', 'heterojunction', 'solar', 'cells', 'reaching', 'power', 'conversion', 'efﬁciency', 'greater', '4.', 'jenekhe', 'watson', 'coworkers', '277', 'reported', 'synthesizing', 'three', 'new', 'donor-acceptor', 'conjugated', 'polymers', 'incorporating', 'thieno34-cpyrrole-46-dione', 'acceptor', 'dialkoxybithiophene', 'cyclopentadithiophene', 'donor', 'units', 'the', 'thieno34-cpyrrole-46-dione', 'acceptor', 'containing', 'materials', 'studied', 'bulk', 'heterojunction', 'solar', 'cells', 'organic', 'ﬁeld-effect', 'transistors', 'the', 'polymers', '108', 'polymer-based', 'solar', 'cells', '781', 'optical', 'band', 'gaps', '150–170', 'ev', 'the', 'highly', 'electron-rich', 'character', 'dialkoxybithiophene', 'polymers', 'however', 'destabilizes', 'hmo', 'signiﬁcantly', 'affects', 'photovoltaic', 'efﬁciency', 'power', 'conversion', 'efﬁciencies', '15.', 'on', 'hand', 'cyclopentadithiophene', 'copolymers', 'achieved', 'better', 'power', 'conversation', 'efﬁciency', 'greater', '3.', 'sharma', 'coworkers', '278', 'reported', 'synthesis', 'two', 'low-band', 'gap', 'copolymers', 'one', 'consists', 'alternating', 'dihexyloxyphenylene', 'a-4-(diphenylamino)phenyl', 'methylene-4-nitrobenzene', 'aceto-', 'nitrile', 'the', 'one', 'consists', 'alternating', 'dihexyloxyphenylene', 'aa0-(14-phenylene)', 'dimethylidynebis(-4-nitrobenzene', 'acetonitrile)', 'c6h13', 'no2', 'h13c6', 'n', 'n', 'nc', 'c6h13', 'nc', 'no2', 'polymer', '1', 'h13c6', 'cn', 'o2n', 'polymer', '2', 'they', 'reported', 'copolymers', 'showed', 'broad', 'absorption', 'curves', 'long-wavelength', 'absorption', 'maximum', 'around', '620', 'nm', 'optical', 'band', '168', '164', 'ev', 'polymers', 'both', 'polymers', 'studied', 'photovoltaic', 'response', 'bulk', 'heterojunction', 'solar', 'cells', 'they', 'observed', 'overall', 'power', 'conversion', 'efﬁciency', '315', '260', 'cast', 'polymers', 'further', 'improvement', 'led', '406', '335', 'devices', 'based', 'thermally', 'annealed', 'materials', 'wei', 'coworkers', '279', 'used', 'stille', 'polycondensation', 'prepare', 'series', 'low-band', 'gap', 'copolymers', 'conjugating', 'electron-accepting', 'pyrido34-bpyrazine', 'moieties', 'electron', 'rich', 'benzo12-b34-bdi', 'thiophene', 'cyclopentadithiophene', 'units', 'all', 'resulting', 'polymers', 'exhibited', 'excellent', 'thermal', 'stability', 'sufﬁcient', 'energy', 'offsets', 'efﬁcient', 'charge', 'transfer', 'dissociation', 'the', 'band', 'gaps', 'polymers', 'could', 'tuned', 'range', '146–160', 'ev', 'using', 'two', 'different', 'donors', 'different', 'electron-donating', 'abilities', 'the', 'three-component', 'copolymers', 'incorporating', 'thiophene', 'bithiophene', 'segments', 'respectively', 'absorbed', 'broadly', 'covering', 'solar', 'spectrum', '350', '800', 'nm', 'the', 'best', 'device', 'performance', 'resulted', 'power', 'conversion', 'efﬁciency', '315.the', 'polymeric', 'materials', 'illustrated', 'follows', 'n', 'n', 'n', 'donor', 's', 'donor', 'n', 'oc8h17', 'h17c8', 'c8h17', 's', 'c8h17o', 'oc8h17', 's', 's', 's', 'oc8h17', 'donor', 'donor', 'liu', 'et', 'al', '280', 'synthesis', 'evaluation', 'n-conjugated', 'copolymers', 'based', 'soluble', 'electro', 'active', 'benzo12-b45-bdifuran', 'chromophore', 'the', 'comonomer', 'units', 'consisted', 'thiophene', 'benzo', 'c[125', 'thiadiazole9-phenylcarbazole', 'these', 'copolymers', 'cover', 'broad', 'absorption', 'ranges', '782', '10', 'polymeric', 'materials', 'special', 'applications', '250', '700', 'nm', 'narrow', 'optical', 'band', 'gaps', '171–201', 'ev', 'the', 'band', 'gaps', 'molecular', 'electronic', 'energy', 'levels', 'tuned', 'copolymerizing', 'benzo12-b45-bdifuran', 'core', 'different', 'n-conjugated', 'electron-donating', 'withdrawing', 'units', 'different', 'ratios', 'ar', 's', 'ar', 'x', 'n', 'n', 's', 'o', 'cn', 'n', 'nc', 'o', 'n', 'n', 'bulk', 'heterojunction', 'solar', 'cell', 'devices', 'fabricated', 'liu', 'coworkers', 'using', 'copolymers', 'electron', 'donor', '(660-phenyl-c61-butyric', 'acid', 'methyl', 'ester)', 'electron', 'acceptor', 'the', 'preliminary', 'research', 'revealed', 'power', 'conversion', 'efﬁciencies', '017–059', 'am', '15', 'illumination', '(100', 'mwcm2)', 'thompson', 'coworkers', '281', 'point', 'despite', 'correlation', 'absorption', 'jsc', 'polymers', 'used', 'currently', 'high-performing', 'solar', 'cells', 'limited', 'absorption', 'breadths', 'rely', 'largely', 'band', 'gaps', 'as', 'result', 'heavy', 'reliance', 'fullerenes', '(especially', 'pc61bm', '1-(3-methoxycarbonyl)propyl-1phenyl-66-methanofullere)', 'absorb', 'photons', 'short', 'wavelength', 'range', 'considerable', 'absorption', 'losses', 'longer', 'wavelengths', 'although', 'fullerenes', 'absorb', 'short', 'wavelength', 'region', 'thus', 'complementary', 'many', 'polymers', 'point', 'evidence', '60', 'excitons', 'formed', 'pc61bm', 'phase', 'decay', 'harvested', 'contribute', 'jsc', '282.', 'this', 'research', 'group', 'mixed', 'graphene', 'oxide', 'acts', 'surfactant', 'fullerenes', 'c60', 'single-walled', 'carbon', 'nanotubes', 'water', 'coated', 'glass', 'slide', 'solution', 'heated', 'reduce', 'graphene', 'oxide', 'graphene', 'they', 'claimed', 'chips', 'using', 'photovoltaic', 'layer', 'much', 'efﬁcient', 'converting', 'light', 'electricity', 'organic', 'devices', 'developed', 'covalent', 'chemistry', 'simultaneous', 'research', 'polymeric', 'materials', 'research', 'also', 'carried', 'improving', 'construction', 'solar', 'cells', 'one', 'strategy', 'stack', 'two', 'light-absorbing', 'materials', 'tandem', 'cell', 'harvest', 'greater', 'fraction', 'solar', 'spectrum', 'by', 'inserting', 'gold', 'nanoparticles', 'two', 'layers', 'yang', 'yang', 'coworkers', 'coupled', 'two', 'tandem', 'cells', 'demonstrated', 'considerable', 'boost', 'inefﬁciency', 'light', 'conversion', 'electricity', '283.', 'review', 'questions', 'section', '101', 'section', '10111', '1', 'how', 'support', 'materials', 'utilized', 'discuss', '1', 'describe', 'merriﬁeld', 'resin', '2', 'what', 'two', 'types', 'cross-linked', 'polystyrenes', 'used', 'support', 'review', 'questions', '783', '3', 'what', 'tentagel', 'describe', 'illustrate', '4', 'what', 'jenda', 'gel', 'explain', 'illustrate', '5', 'describe', 'general', 'use', 'cross-linked', 'styrene', 'derivatives', 'support', 'sections', '10112–1014', '1', 'illustrate', 'isobutylene-based', 'support', 'material', '2', 'describe', 'acrylic', 'material', 'used', 'support', '3', 'describe', 'polyether-based', 'support', 'resin', '4', 'describe', 'gel', 'drug', 'release', 'section', '1013', '1', 'describe', 'immobilized', 'enzymes', '2', 'describe', 'nonenzyme', 'immobilized', 'catalysts', '3', 'describe', 'immobilized', 'reagents', 'section', '102', '1', 'discuss', 'polyacetylene', '2', 'discuss', 'polypyrrole', '3', 'discuss', 'polythiophene', 'derivatives', '4', 'discuss', 'polyaniline', '5', 'discuss', 'poly(phenyl', 'vinylene)', 'sections', '103–106', '1', 'explain', 'meant', 'photonic', 'polymer', '2', 'discuss', 'nature', 'light', '3', 'discuss', 'interaction', 'light', 'organic', 'molecules', '4', 'what', 'energy', 'transfer', 'process', '5', 'what', 'electron', 'transfer', 'process', '6', 'describe', 'charge', 'transfer', 'process', 'polymers', '7', 'describe', 'antenna', 'effect', 'polymers', '8', 'what', 'photosensitizer', 'give', 'examples', '9', 'discuss', 'photocross-linking', 'polymers', 'give', 'examples', '10', 'describe', 'polymers', 'designed', 'harvest', 'sun’s', 'energy', '11', 'discuss', 'photo-isomerization', 'polymeric', 'materials', '12', 'how', 'photo-isomerization', 'utilized', 'liquid', 'crystalline', 'alignment', '10', 'polymeric', 'materials', 'special', 'applications', '1', 'explain', 'photo-conductivity', 'polymers', 'give', 'examples', 'based', 'carbazole', 'based', 'carbazole', '1', 'describe', 'planar', 'heterojunction', 'solar', 'cell', '2', 'describe', 'bulk', 'heterojunction', 'solar', 'cell', '3', 'what', 'equation', 'deﬁnes', 'efﬁciency', 'organic', 'solar', 'cells', '784', 'section', '107', 'section', '108', 'references', '1', 'ma', 'kraus', 'a', 'patchornik', 'j', 'polymer', 'sci,', 'macromol', 'rev15,', '55', '(1980)', '2', 'ma', 'kraus', 'a', 'patchornik', 'israel', 'j', 'chemistry', '9', '269', '(1971)', '3', 'wg', 'herkstroeter', 's', 'farid', 'j', 'photochem,', '35', '71', '(1986)', '4', 'ra', 'gregg', 'fr', 'mayo', 'j', 'am', 'chem', 'soc,', '75', '3530', '(1953)', '5', 'rb', 'merriﬁeld', 'j', 'am', 'chem', 'soc,', '1963', '85', '2149', '6', 'tripp', 'j', 'a,', 'stein', 'j', 'a,', 'svec', 'f,', 'fre´chet', 'j', 'm', '3', 'org', 'lett', '2000', '2', '195', '7', 'tripp', 'j', 'a,', 'svec', 'f,', 'frechet', 'j', 'm,', 'j', 'comb', 'chem', '2001', '3', '604', '8', 'lee', 't-k,', 'ryoko', 's-j,', 'byun', 'j-w,', 'lee', 's-m,', 'lee', 'y-s', 'j', 'comb', 'chem', '2005', '7', '170', '9', 'kim', 'j-h,', 'jun', 'b-h,', 'byun', 'j-w,', 'lee', 'y-s', 'tetrahedron', 'lett', '2004', '45', '5827', '10', 'lu', 'toy', 'chem', 'rev,', '(web)', 'jan', '7', '2009', '11', 'alesso', 's', 'm,', 'yu', 'z,', 'pears', 'd,', 'worthington', 'p', 'a,', 'luke', 'r', 'w', 'a,', 'bradley', 'm,', 'tetrahedron', '2003', '59', '7163', '12', 'toy', 'p', 'h,', 'reger', 't', 's,', 'janda', 'k', 'd', 'aidrichimica', 'acta', '2000', '3387', '13', 'toy', 'p', 'h,', 'janda', 'kd.', 'tetrahedron', 'lett', '1999', '40', '6329', 'toy', 'p', 'h,', 'reger', 't', 's,', 'garibay', 'p,', 'garno', 'j', 'c,', 'malikayil', 'j', 'a,', 'liu', 'g', '-y,', 'janda', 'k', 'd,', 'j', 'comb', 'chem', '2001', '3117', '14', 'choi', 'mkw,', 'toy', 'ph', 'tetrohedron', '2004', '60', '2903', '15', 'aidyama', 'r,', 'aed', 'kobayashi', 's', 'j', 'am', 'chain', 'soc', '2003', '125', '3412', 'okamoto', 'k,', 'aklyama', 'it', 'kobayashl', 's', '16', 'okamoto', 'k,', 'aklyama', 'r,', 'kobayashi', 's', 'org', 'lett', '2004', '61987', 'hagio', 'h,', 'sugiura', 'm,', 'kobayashi', 's,', 'j', 'org', 'chem.', '2004', '69', '2871', 'org', 'lett', '2006', '8', '375', '17', 'okamoto', 'k,', 'aklyama', 'r,', 'yoshida', 'h,', 'yoshida', 't,', 'kobayash', 's,', 'j', 'am', 'chern', 'soc,', '2005', '127', '25', 'aklyama', 'r,', 'sagae', 't,', 'sugiura', 'm,', 'kobayashi', 's,', 'j', 'organomet', 'chem', '2004689', '3806', '18', 'takeuchi', 'm,', 'aklyama', 'r,', 'kobayashi', 's,', 'j', 'am', 'chem', 'soc,', '2005127', '13096', 'miyamura', 'h,', 'aklyama', 'r,', 'ishida', 't,', 'matsubara', 'r,', 'takeuchl', 'm,', 'kobayashi', 's,', 'tetrahedron', '2005', '61', '12177', 'matsumoto', 't,', 'ueno', 'm,', 'kobayashi', 'j,', 'miyamura', 'h,', 'mori', 'y,', 'kobayashl', 's,', 'adv.', 'synth', 'catal', '2007', '349', '531', 'miyazakl', 'y,', 'hagio', 'h,', 'kobayashi', 's,', 'org', 'biomol', 'chem', '2006', '4', '2529miyamura', 'h,', 'matsubara', 'r,', 'miyazakl', 'y,', 'kobayashl', 's,', 'angew', 'chem,', 'tnt', 'ed', '2007', '46', '4151', 'miyazaki', 'y,', 'kobayashl', 's,j', 'comb', 'chem,', '2008', '10', '355', '19', 'hullzsch', 'k', 'c,', 'jernelius', 'j', 'a,', 'hoveyda', 'a', 'h,', 'schrock', 'r', 'r,', 'angew', 'chem,', 'inted', '2002', '41', '589', '20', 'sellner', 'h,', 'rheiner', 'p', 'b,', 'seebach', 'd,', 'helv', 'chim', 'acta', '2002', '85352', '21', 'itsuno', 's,', 'chiba', 'm,', 'takahashi', 'm,', 'arakawa', 'v,', 'haraguchi', 'n', 'j', 'organomet', 'chem2007,', '692', '487', '22', 'kwong', 'c', 'k-w,', 'huang', 'r,', 'zhang', 'm,', 'shi', 'm,', 'toy', 'p', 'h', 'chem', 'eur', 'j', '2007', '13', '2369', '23', 'hodges', 'j', 'c,', 'harikrishnan', 'l', 's,', 'ault-justus', 's', 'j', 'comb', 'chem,', '2000', '2', '80', '24', 'mcalpine', 's', 'r,', 'lindsley', 'c', 'w,', 'hodges', 'j', 'c,', 'leonard', 'd', 'm,', 'flizen', 'g', 'f', 'jcombchem', '2001', '3', '125', '25', 'wisnoski', '0', 'd,', 'leister', 'w', 'h,', 'strauss', 'k', 'a,', 'zhao', 'z,', 'llndstey', 'c', 'w,', 'tetrohedron', 'lett', '2003', '44', '4321', 'pawluczykj.', 'm,', 'mcclain', 'r', 't,', 'denicola', 'c,', 'mulheam', 'j,', 'jr,', 'rudd', 'd', 'j,', 'lindsley', 'c', 'w', 'tetrahedron', 'lett.', '2007', '48', '1497', '26', 'foumier', 'd,', 'pascual', 's,', 'montembault', 'v,', 'haddleton', 'd', 'm,', 'fontaine', 'l,', 'j', 'combchem', '2006', '8', '522', '27', 'barrett', 'a', 'g', 'm,', 'hopkins', 'b', 't,', 'kobberling', 'j', 'chem', 'rev,', '2002', '102', '3301', 'references', '785', '28', 'hamad', 'a', 'm,', 'zhang', 'm,', 'vedantham', 'p,', 'mukherjee', 's,', 'herpet', 'r', 'h,', 'flynn', 'd', 'l,', 'hanson', 'p', 'r,', 'aldrichim', 'acta', '200538', '3', '29', 'bergbreiter', 'd', 'e,', 'li', 'j', 'chem', 'commun.', '2004', '42', '30', 'li', 'j,', 'sung', 's,', 'tian', 'j,', 'bergbreiter', 'd', 'e', 'tetrahedron', '2005', '6112081', '31', 'bergbreiter', 'd', 'e,', 'tian', 'j', 'tetrahedron', 'lett,', '2007', '48', '4499', '32', 'bergbreiter', 'd', 'e,', 'hamilton', 'p', 'n,', 'koshti', 'n', 'm,', 'j', 'am', 'chem', 'soc', '2007', '129', '10666', '33', 'hongfa', 'c,', 'tian', 'j,', 'bazzi', 'h', 's,', 'bergbreiter', 'd', 'e', 'org', 'lett,', '2007', '9', '3259', '34', 'chinchilla', 'r,', 'dodsworth', 'd', 'j,', 'na´jera', 'c,', 'soriano', 'j', 'm,', 'tetrahedron', 'lett.', '2001', '42', '4487', '35', 'bergbreiter', 'd', 'e,', 'franchina', 'j', 'g,', 'case', 'b', 'l,', 'org', 'lett', '2000', '2', '393', '36', 'bergbreiter', 'd', 'e', 'chem', 'rev,', '2002', '102', '3345', 'bergbreiter', 'd', 'e,', 'sung', 'sd..', 'adv', 'synth', 'catal', '2006', '34', '1352', '37', 'kita', 'r,', 'svec', 'f,', 'fre´chet', 'j', 'm', '1,', 'j', 'comb', 'chem,', '2001', '3564', '38', 'hong', 's', 'h;', 'grubbs', 'r', 'h', 'j', 'am', 'chem', 'soc', '2006', '128', '3508', '39', 'metyjaszewski', 'et', 'al,', 'am', 'chem', 'soc', 'polymer', 'preprints', '2008', '(2)', '203', '40', 'uzhou', 'lvu', 'm', 'ding', 'j', 'li', 'h', 'tan', 'z', 'wang', 'qfu', 'macromolecules', '2011', '44', '857', '41', 'x', 'xu', 'j', 'd', 'flores', 'cl', 'mccormick', 'macromolecules', '2011', '44', '1327', '42', 'hd', 'orth', 'w', 'brummer', 'angew', 'chem,', 'intern', 'edit,', '11', '(4)', '249', '(1972)', '43', 'g', 'manecke', 'w', 'stock', 'angew', 'chem,', 'intern', 'edit,', '17', '657', '(1978)', '44', 'l', 'd’anguiro', 'g', 'mazzola', 'p', 'cremonesi', 'b', 'focher', 'angew', 'makromol', 'chem,', '72', '31', '(1978)', '45', 'md', 'bednarski', 'mr', 'callstrom', 'j', 'am', 'chem', 'soc,', '114', '378', '(1992)', '46', 'u', 'hiroshi', 'k', 'mai', 't', 'takashi', 'kobayashi', 'shiro', 'polymer', 'journal', '2002', '34(12)', '970–972', '47', 'n', 'cinouini', 's', 'colojna', 'h', 'molinari', 'p', 'rundo', 'j', 'polymer', 'sci,chem', 'commun,', '1976', '394', '48', 's', 'kopolow', 'z', 'machacek', 'u', 'takaki', 'j', 'smid', 'j', 'macromol', 'sci-chem,', '1973', 'a7', '(5)', '1015', '49', 'p', 'kutchukov', 'a', 'ricard', 'c', 'quivron', 'eur', 'polymer', 'j,', '1980', '16', '753', '50', 'j', 'capillon', 'a', 'richard', 'c', 'quivron', 'am', 'chem', 'soc', 'polymer', 'preprints', '1980', '23', '(1)', '168', '51', 'j', 'smid', 'am', 'chem', 'soc', 'polymer', 'preprints', '1982', '23', '(1)', '168', '52', 'a', 'mastagli', 'a', 'froch', 'g', 'durr', 'cr', 'acad', 'sci,', 'paris', '1952', '235', '1402', '53', 'ec', 'blossey', 'dc', 'neckers', 'c', 'douglas', 'al', 'thayer', 'ap', 'schaapj', 'am', 'chem', 'soc,', '1973', '95', '5820', '54', 'mr', 'buchmeister', 'chem', 'rev,', '2008', 'asap', '1021cr', 'article', '800207n', '55', 'grubbs', 'r', 'h;', 'swetnick', 's;', 'su', 's', 'c-h', 'j', 'mo.', 'catal', '1977', '3', '11', '56', 'nguyen', 's;', 'grubbs', 'r', 'h', 'j', 'organomet', 'chem', '1995', '497', '195', '57', 'schurer', 's', 'c;', 'gessler', 's;', 'buscbmann', 'n;', 'blechert', 's', 'angew', 'chem', '2000', '112', '4062', '58', 'a', 'patchornik', 'bj', 'cohen', 'ma', 'kraus', 'j', 'am', 'chem', 'soc,', '103', '7620', '(1981)', '59', 'th', 'mough', 'ii', 'science', '217', '719', '(1982)', '60', 'hj', 'letherby', 'j', 'chem', 'soc,', '1862', '15', '161', '61', 'mg', 'kanatzidis', 'chem', 'eng', 'news', 'p', '38', '(dec', '3', '1990)', 'ly', 'chiang', 'pm', 'chaikin', 'do', 'cowan(eds)', 'advanced', 'organic', 'solid', 'state', 'materials', 'mat', 'res', 'soc', 'symp', 'proc,', '173', '(1990)', 'jr', 'reynolds', 'chemtech', '18', '440', '(1988)', '62', 'g', 'inzelt', '“conducting', 'polymers', 'new', 'era', 'electricity”', 'springer', 'new', 'york', '2008', '63', 'mg', 'kanatzidis', 'chemand', 'eng', 'news', 'p', '38', '(dec', '3', '1990)', 'ly', 'chiang', 'pm', 'chaikin', 'do', 'cowan(eds)', 'advances', 'organic', 'solid', 'state', 'materials', 'mat', 'res', 'soc', 'symp', 'proc,', '1990', '173', 'jr', 'reynolds', 'chemtech', '1988', '18', '440', '64', 'diaz', 'a', 'f,', 'caslillo', 'j', 'i,', 'logan', 'j', 'a,', 'lee', 'w', 'v', 'j', 'electroanal', 'chem', '1981', '129', '115', 'genies', 'e', 'm,', 'bidan', '6', 'diaz', 'a', 'f', 'j', 'electroanal', 'chem', '1983', '149', '101', '65', 'j', 'heinze', 'ba', 'fontana-urbe', 's', 'ludwigs', 'chem', 'rev.', '2010', '110(8)', '4724', '66', 'a', 'handbook', 'conducting', 'polymers', 'vol12', 'edited', 'ta', 'skotheim', 'rl', 'elsenbaumer', 'jr', 'reynolds', 'marcel', 'dekker', 'inc,', 'new', 'york', '1998.', '“semiconducting', 'polymers”', 'edited', 'g', 'hadziioannou', 'pfv', 'hutten', 'wiley-vch', 'weinheim', '2007', '67', 'x-f', 'sun', 'sb', 'clough', 's', 'subrmanyam', 'a', 'blumstein', 'sk', 'tipathy', 'am', 'chem', 'soc', 'polymer', 'preprints', '1992', '33', '(1)', '576', '68', 's', 'subramanyam', 'a', 'blumstein', 'macromolecules', '1991', '24', '2668', '69', 'rw', 'waymouth', 'gw', 'coates', 'science', '1995', '267', '222', '70', 'y', 'misumi', 't', 'masuda', 'macromolecules', '1998', '31', '7572', '71', 's', 'm', 'a', 'karim', 'r', 'nomura', 'f', 'sanda', 's', 'seki', 'm', 'watanabe', 't', 'masuda', 'macromolecules', '2003', '36', '4786', '72', 'rd', 'mccullough', 'rd', 'lowe', 'am', 'chem', 'soc', 'polymer', 'preprints', '1992', '33', '(1)', '195', '73', 'ae', 'juvier', 'g', 'sueve', 'rd', 'mccullough', 'am', 'chem', 'soc', 'polymer', 'preprints', '2008', '49', '(1)', '600', '74', 'e', 'e', 'sheina', 'j', 'liu', 'mc', 'iovu', 'd', 'w', 'laird', 'rd.', 'mccullough', 'macromolecules', '2004', '373526', 'liu', 'j;', 'sheina', 'e;', 'kowalewski', 't;', 'mccullough', 'r', 'd,angew', 'chem', '2002', '41', '329', '786', '10', 'polymeric', 'materials', 'special', 'applications', '75', 'dj', 'irvin', 'dl', 'witker', 'ap', 'closson', 'jd', 'stenger-smith', 'ja', 'irvin,', 'am', 'chem', 'soc,', 'polymer', 'preprints', '2008', '49', '(1)', '552', '76', 'jd', 'mendez', 'c', 'weder', 'am', 'chem', 'soc,', 'polymer', 'preprints,', '2008', '49(1)', '499', '77', 'w', 'liu', 'j', 'kumar', 's', 'trypathy', 'kj', 'senecal', 'l', 'samuelson', 'j', 'am', 'chem', 'soc,', '1999', '121', '71', '78', 's-h', 'lee', 'nature', '2006', '441', '65', '79', 'c-c', 'han', 'c-h', 'lu', 's-p', 'hong', 'k-f', 'yang', 'macromolecules', '2003', '36', '7908', '80', 'ta', 'skotheim', '“handbook', 'conducting', 'polymers”', 'ii', 'ed,', 'crc', 'press', 'new', 'york', '1997', '81', 'i', 'natori', 's', 'natori', 'k', 'tsuchiya', 'k', 'ogino', 'macromolecules', '2011', '44', '256', 'a', 'ledwith', 'polymer', '14', '(1973)', '82', 'jg', 'calvert', 'jn', 'pitts', '“photochemistry”', 'wiley', 'new', 'york', '1967', '83', 'fk', 'richtmeyer', 'eh', 'kennard', '“introduction', 'modem', 'physics”', 'mcgraw-hill', 'new', 'york', '1947', '84', 'cowan', 'drisko', '“elements', 'organic', 'photochemistry”', 'plenum', 'new', 'york', '1978', '85', 'rohatgi-mukherjee', '“fundamentals', 'photochemistry”', 'wiley', 'new', 'york', '1978', '86', 't', 'foster', 'disc', 'faraday', 'sec,', '1959', '27', '1', 'radiation', 'research', 'supplement', '1960', '2', '326', 'se', 'webber', 'chem', 'rev,', '1990', '901460-1482', '87', 'nj', 'turro', '“molecular', 'photochemistry”', 'benjamin', 'new', 'york', '2003', '88', 'm', 'orchin', 'hh', 'jaffe', '“the', 'importance', 'antibonding', 'orbitals”', 'houton', 'miffm', 'co,', 'boston', '1967', '89', 'd', 'gegiou', 'ka', 'muszkat', 'e', 'fischer', 'i', 'am', 'chem', 'soc,', '1968', '90', '12', '90', 'r', 'srinivasan', 'j', 'am', 'chem', 'soc,', '1964', '86', '3318', 'ke', 'wilzbach', 'l', 'kaplan', 'j', 'am', 'chem', 'soc,', '1964', '86', '3157', 'd', 'schulte-frohlinde', 'ann,', '1958', '615', '114', '91', 'a', 'ravve', '“light', 'associated', 'reactions', 'synthetic', 'polymers”', 'springer', 'new', 'york', '2006', '92', 'as', 'davydov', '“theory', 'molecular', 'excitations”', 'mcgraw', 'hill', 'new', 'york1962', 'rsh', 'liu', 'gs', 'hammond', 'j', 'am', 'chem', 'sec,', '1964', '86', '1892', '93', 'jb', 'birks', '“photophysics', 'aromatic', 'molecules”', 'wiley-interscience', 'london', '1970', '94', 'j', 'guellet', '“polymer', 'photophysics', 'photochemistry”', 'cambridge', 'university', 'press', 'cambridge', '1985', '95', 'nj', 'turro', 'jc', 'dalton', 'ds', 'weiss', '“organic', 'photochemistry”', 'ol', 'chapman', 'ed,', 'dekker', 'new', 'york', '1969', '96', 'pa', 'martic', 'rc', 'daly', 'jlr', 'williams', 'sy', 'farid', 'jpolymer', 'sci,', 'polym', 'lett', 'ed,', '1977', '15', '295', '97', 's', 'farid', 'pa', 'martec', 'drthompson', 'dp', 'specht', 'se', 'hartman', 'jlr', 'williams', 'pure', 'appl', 'chem,', '1979', '51', '241', '98', 'tj', 'meyer', '“photoinduced', 'electron', 'energy', 'transfer', 'soluble', 'polymers”', 'coord', 'chem', 'rev,', '1991', '111', '47', 'wg', 'herkstroeter', 'chapter', '1', 'creation', 'detection', 'excited', 'state', 'aa', 'lamola', 'ed,', 'dekker', 'new', 'york', '1971', '99', 's', 'tazuke', 'y', 'matsuyama', 'macromolecules', '1975', '8', '20', 'ibid,', '1977', '10', '215', '100', 'ma', 'fox', 'we', 'jones', 'jr,', 'dm', 'watkins', 'chem', 'eng', 'news', '1993', '(march', '15)', '18', '101', 'f', 'schneider', 'j', 'springer', 'makromol', 'chem', '1971', '146', '181', '102', 'la', 'gatechair', 'a', 'tiefenthaler', 'chapter', '3', '“radiation', 'curing', 'polymeric', 'materials”', 'ch', 'hoyle', 'jf', 'kinstle', 'ed,acs', 'symposium', 'series', '417', 'washington', 'dc', '1990', '103', 'f', 'gersheim', '“history', 'photography”', 'oxford', 'press', 'london', '1966', '104', 'wi', 'le', 'noble', 'highlights', 'organic', 'chemistry', 'chapter', '14', 'dekker', 'inc', 'new', 'york', '1974', '105', 'c', 'decker', 'polymer', 'international', '2002', '51(11)', '1141-1150', '106', 'm', 'wen', 'le', 'scriven', 'av', 'mccormick', 'macromolecules', '2003', '36(11)', '4140', 'ibid,', '4159', '107', 'pj', 'flory', 'j', 'am', 'chem', 'soc,', '1941', '63', '3083', '30913096', '108', 'pj', 'flory', '“principles', 'polymer', 'chemistry”', 'cornell', 'university', 'press', 'ithaca', 'ny,', '1953', '109', 'wh', 'stockmayer', 'j', 'polym', 'sci,', '1952', '9', '69', 'ibid,', '1953', '11', '424', '110', 'rc', 'dolby', 'rh', 'engebrecht', 'canadian', 'pat', '1106544', '(1981)', 'british', 'patent', '438960', '(nov', '26', '1935)', 'us', 'patent', '1965710', '(july', '10', '1934)', 'french', 'patent', '1351542', '(feb', '7', '1964)', 'us', 'patent', '3066117', '(nov', '27', '1962)', 'brit', 'patent', '822861', '(nov', '4', '1959)', 'us', 'patent', '2948706', '(aug', '9', '1960)', 'us', 'patent', '2738745', '(dec', '27', '1955)', '111', 'wg', 'herkstroeter', 's', 'farid', 'j', 'photochem,', '35', '71', '(1986)', '112', 'm', 'kato', 'j', 'polymer', 'sci1969,', 'b-7', '605', '113', 'mj', 'farrall', 'm', 'alexis', 'm', 'trecarton', 'polymer', '1983', '24', '114;', 'ao', 'patil', 'dd', 'deshpnde', 'ss', 'war', 'polymer', '1981', '22', '434', 'o', 'zimmer', 'h', 'meier', 'j', 'chem', 'soc', 'chem', 'commun,', '1982', '481', 'mp', 'stevens', 'a', 'd', 'jenkins', 'j', 'polym', 'sci,', 'chem', 'ed1979,', '17', '3675', 'm', 'kata', 'mhasegawa', 't', 'ichijo', 'j', 'polym', 'sci,', '1970', 'b-8', '263', 'ge', 'green', 'bp', 'stark', 'sa', 'zahir', 'j', 'macromol', 'sci.', 'rev', 'macromol', 'chemc21', '1991', '187', 'g', 'oster', 'n', 'yand', 'chem', 'rev.1968', '68', '125', '114', 'f', 'wilkinson', 'j', 'phys', 'chem,', '1961', '66', '2569', '115', 'hl', 'backstrom', 'k', 'sandros', 'acta', 'chem,', 'scand,', '1962', '16', '958', '116', 'dj', 'trecker', '“organic', 'photochemistry”', 'vol', '2', 'ol', 'chapman', 'ed,', 'dekker', 'new', 'york', '1969', '117', 'lm', 'minsk', 'jg', 'smith', 'van', 'devsen', 'jf', 'wright', 'j', 'appl', 'polym', 'sci,', '1959', '2', '302', '118', 'bertram', 'kursten', 'j', 'prakt', 'chem,', '1895', '51', '323', 'references', '787', '119', 'cn', 'ruber', 'ber,', '1902', '35', '2415', '120', 'gmj', 'schmidt', 'j', 'chem', 'sec,', '1964', '2014', '121', 'j', 'bergman', 'gmj', 'schmidt', 'fi', 'sonntag', 'j', 'chem', 'sec,', '1964', '2021', '122', 'md', 'cohen', 'gmj', 'schmidt', 'fi', 'sonntag', 'j', 'chem', 'sec,', '1964', '2000', '123', 'h', 'tanaka', 'e', 'otomegawa', 'j', 'polymer', 'sci,', 'chem', 'ed', '1974', '12', '1125', '124', 'hg', 'curme', 'cc', 'natale', 'dj', 'kelley', 'j', 'phys', 'chem,', '1967', '71', '767', '125', 'fi', 'sonntag', 'r', 'srinivasan', 'spe', 'conference', 'photopolymers', 'nov', '6-7', '1967', 'ellenville', 'ny', '126', 'c', 'libermann', 'm', 'zsuffa', 'ber,', '1911', '44', '841', '127', 'a', 'delzenne', 'ind', 'chim', 'belg', '1974', '39', '249', '128', 'k', 'nakamura', 's', 'kikuchi', 'bull', 'chem', 'sec,', 'japan', '1967', '40', '1027', '129', 'wm', 'moreau', 'am', 'chem', 'sec', 'preprints', '1969', '10(1)', '362', '130', 'k', 'nakamura', 's', 'kikuchi', 'bull', 'chem', 'sec,', 'japan', '1968', '411977', '131', 'm', 'tsuda', 'j', 'polymer', 'sci1969,', '7', '259', 'k', 'nakamura', 'sk', 'kuchi', 'bull', 'chem', 'soc', 'japan', '1968', '41', '1977', '132', 'm', 'tsuda', 's', 'oikawa', 'chapt', '29', 'ultraviolet', 'light', 'induced', 'reaction', 'polymers', 'am', 'chem', 'soc', '133', 'edkvasnikov', 'vm', 'kozenkov', 'va', 'barachevsky', 'zh', 'nauchn', 'prikl', 'fotogr', 'kinematogr', '1979', '24', '222', 'symposium', 'series', '25', 'washington', '197687', 'chem', 'abstr', '1979', '91', '99887h', '134', 'ba', 'barachevsky', 'spie', '1991', '1559', '184', '135', 'schadt', 'm;', 'seiberle', 'h;', 'schuster', 'a;', 'kelly', 's', 'm', 'jpn', 'j', 'appl', 'phys1995', '34', 'l764', '136', 'mschadt', 'h', 'seiberle', 'a', 'schuster', 'sm', 'kelly', 'jpn', 'j', 'appl', 'phys', '1995', '34', '3240', '137', 'ag', 'dyadyusha', 'ty', 'marusii', 'ya', 'reznikov', 'ai', 'khizhnyak', 'vy', 'reshetnyak', 'jetp', 'lett', '1992', '56', '17', '138', 'ty', 'marusii', 'ya', 'koznikov', 'mol', 'mater', '1993', '3', '161', '139', 'ag', 'dyadyusha', 'a', 'khizhnyak,', 'ty', 'marusii', 'vy', 'reshetnyak', 'ya', 'reznikov', 'ws', 'park', 'jpn', 'j', 'appl', 'phys', '1995', '34', 'l', '1000', '140', 'y', 'akita', 'h', 'akiyama', 'k', 'kudo', 'y', 'hayashi', 'k', 'ichimura', 'j', 'photopolym', 'sci', 'technol', '1995', '8', '75', '141', 'y', 'ichimura', 't', 'satoh', 's', 'kobayashi', 'j', 'photopolym', 'sci', 'technol', '19958', '257', '142', 'h', 'tomita', 'k', 'kudo', 'k', 'ichimura', 'liq', 'cryst', '1996', '20', '171', '143', 'm', 'schadt', 'k', 'schmitt', 'v', 'kozinkov', 'v', 'chigrinov', 'jpnj', 'appl', 'phys', '1992', '7', '2155', '144', 'k', 'ichimura', 'y', 'akita', 'h', 'akiyama', 'k', 'kudo', 'y', 'hayashi', 'macromolecules', '1997', '30', '903–911', '145', 'y', 'kawanishi', 't', 'tamaki', 'm', 'sakuragi', 't', 'seki', 'k', 'ichimura', 'mol', 'cryst', 'liq', 'cryst', '1992', '218', '153', '146', 'k', 'ichimura', 'y', 'hayashi', 'n', 'ishizuki', 'chem', 'lett', '1992', '1063', '147', 'k', 'ichimura', 'y', 'hayashi', 'h', 'akiyama', 't', 'ikeda', 'n', 'ishizuki', 'applphys', 'lett', '1993', '63', '449', '148', 'k', 'ichimura', 'h', 'akiyama', 'n', 'ishizuki', 'y', 'iawanishi', 'makromol', 'chem,', 'rapid', 'commun', '1993', '14', '8', '13', '149', 'h', 'akiyama', 'k', 'kudo', 'k', 'ichimura', 'macromol', 'chemrapid', 'commun', '1995', '16', '35', '150', 'm', 'schadt', 'k', 'schmitt', 'v', 'kozinkov', 'v', 'chigrinov', 'jpnj', 'appl', 'phys', '1992', '7', '2155', '151', 'c-h', 'zhang', 'z-h', 'yang', 'm-xding', 'liquid', 'crystals', '2003', '30(1)', '65–69', '152', 'sw', 'lee', 'si', 'kim', 'b', 'lee', 'w', 'choi', 'b', 'chae', 'sb', 'kim', 'm', 'ree', 'macromolecules', '2003', '36', '6527–6536', '153', 'm', 'nagata', 's', 'hizakae', 'macromolecular', 'bioscience', '2003', '3(8)', '412', '154', 'hr', 'allcoci', 'cg', 'cameron', 'macromolecules', '1994', '27', '3125', '3131', '155', 's', 'watanabe', 's', 'harashima', 'n', 'tsukada', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1986', '24', '1227', '156', 'ks', 'lyalikov', 'gl', 'gaena', 'n', 'a', 'evlashina', 'tr', 'leningrad', 'inst', 'kinoinzh', '197016', '42', '157', 'k', 'subramanian', 'v', 'krishnasamy', 's', 'nanjundan', 'av', 'kami', 'reddy.', 'ear', 'polym', ']', '2000', '36(11)', '2343–2350', '158', 'dh', 'choi', 'sj', 'oh', 'hb', 'clia', 'jy', 'lee', 'eur', 'polym', 'j', '2001', '37(10)', '1951-1959', '159', 'jlr', 'williams', 'spe', 'photopolymers', 'meeting', 'nov', '6-7', 'ellenville', 'ny1967', '160', 'm', 'tsuda', 'j', 'polymer', 'sci,', 'a-17', '259', '(1969)', '161', 'y', 'tajima', 'h', 'arai', 'k', 'takeuchi', 'kagaku', 'kogaku', 'ronbunshu', '1999', '25(6)', '873-877', '(private', 'translation)', '162', 'h', 'arai', 'y', 'tajima', 'k', 'takeuchi', 'j', 'photopolym', 'sci', 'technol', '1999', '12(1)', '121–124', '163', 'y', 'tajima', 'y', 'shigemitsu', 'h', 'arai', 'w', 'he', 'e', 'takeuchi', 'ktakeuchi', 'j', 'photopolym', 'sci', 'technol', '1999', '12(1)', '164', 'm', 'e', 'harmon', 'd', 'kuckling', 'c', 'w', 'frank', 'macromolecules', '200336162–172', '165', 'w-s', 'kim', 'h-s', 'jang', 'k-h', 'hong', 'k-h', 'seo', 'macromol', 'rapid', 'commun', '2001', '22(11)', '825-828', '166', 'j', 'march', '“advanced', 'organic', 'chemistry”', 'wiley', 'new', 'york', '1985', 'p', '216', '167', 'p', 'hyde', 'lj', 'kricka', 'a', 'ledwith', 'j', 'polymer', 'sci,lett', 'ed,', '1973', '11', '415', '168', 'y', 'tian', 'x', 'kong', 'y', 'nagase', 't', 'iyoda', 'journal', 'polymer', 'science', 'part', 'a', 'polymer', 'chemistry', '2003', '41(14)', '125-128', '2197-2206', '169', 'm-h', 'lee', 'ys', 'chung', 'j', 'kim', 'polymeric', 'materials', 'science', 'engineering', '2001', '84', '613', '170', 'a', 'reiser', 'hm', 'wagner', 'r', 'marley', 'g', 'bowes', 'trans', 'faraday', 'sec,', '63', '2403', '(1967)', '171', 'a', 'reiser', 'r', 'marley', 'trans', 'faraday', 'sec,', '1968', '64', '1806', '172', 'a', 'reiser', 'fw', 'willets', 'gc', 'terry', 'v', 'williams', 'r', 'marley', 'trans', 'faraday', 'soc,', '1969', '65', '3265', '788', '10', 'polymeric', 'materials', 'special', 'applications', '173', 't', 'tsunoda', 't', 'yamaoka', 'spe', 'conference', 'photopolymers', 'nov', '6-7', '1967', 'ellenville', 'ny', '174', 't', 'tsunoda', 't', 'yamaoka', 'g', 'nagarnatsu', 'm', 'hirohashi', 'spe', 'conference', 'photopolymers', 'oct', '15–16', '1970', 'ellenville', 'ny', '175', 'sh', 'merrill', 'cc', 'unruh', 'j', 'appl', 'polymer', 'sci,', '7', '273', '(1963)', '176', 'h', 'holtschmidt', 'g', 'oertel', 'ang', 'makromol', 'chem,', '1969', '9', '1', '177', 'q', 'lin', 'jp', 'gao', 'zy', 'wang', 'polymer', '2003', '44(19)', '5527–5531', '178', 'ga', 'delzenne', 'u', 'laridon', 'j', 'polymer', 'sci,', '1969', 'c-22', '1', '149', '179', 'a', 'yabe', 'm', 'tsuda', 'k', 'honda', 'h', 'tanaka', 'j', 'polymer', 'sci1972,', 'a-', '110', '2376', '180', 'z', 'roz', 's', 'maaref,', 's-s', 'sun', 'am', 'chem', 'soc', 'polymer', 'preprints', '2003', '44(1)86', '181', 'jc', 'craneand', 'rj', 'guglielmetti', 'organic', 'photochromic', 'thermochromic', 'compounds', 'plenum', 'press', 'new', 'york', '1999', '182', 'se', 'webber', 'chem', 'rev,', '1990', '90', '1469', '183', 'rb', 'fox', 'rf', 'cozzens', 'macromolecules', '1969', '2', '181', '184', 'a', 'ikeda', 'a', 'kameyama', 't', 'nishikubo', 't', 'nagai', 'macromolecules', '200134', '2728–2734', '185', 't', 'nagai', 'm', 'shiimada', 'y', 'ono', 't', 'nishikubo', 'macromolecules', '2003', '36', '1786', '186', 'n', 'kawatsuki', 'h', 'ono', 'h', 'takatsuka', 't', 'yamanioto', 'o', 'sangen', 'macromolecules', '199730', '6680', '187', 'm', 'sampei', 'k', 'hiramatu', 'a', 'kameyama', 't', 'nishikubo', 'kobunshi', 'ron-bunshu', '2000', '57(9)', '569–576', '188', 'n', 'kawashima', 'a', 'kameyama', 't', 'nishikubo', 't', 'nagai', 'reactive', 'functional', 'polymers', '2003', '55(1)', '75–88', '189', 'h', 'ritter', 'r', 'sperber', 'cm', 'weisshuhn', 'macromol', 'chem', 'phys,', '1994', '195', '3823', 't', 'deutschmann', '190', 'a', 'theis', 'b', 'menges', 's', 'mittler', 'm', 'mierawa', 't', 'pakula', 'h', 'ritter', 'macromolecules', '2003', '36', '7520', 'a', 'theis', 't', 'ritter', 'chem', 'phys,', '2000', '201', '1200', 'h', 'ritter', 'macromolecules', '2003', '36', '7552', '191', 'w', 'kuhlbrandt', 'dnwang', 'd', 'n', 'nature', '1991', '350', '130', 'w', 'kuhlbrandt', 'dn', 'wang', 'yfujiyoshi', 'nature', '1994', '367', '614', 'gmcdermott', 'sm', 'prince', 'aa', 'freer', 'am', 'hawthornthwaite-lawless', 'mz', 'papiz', 'rj', 'cogdell', 'nw', 'isaacs', 'nature', '1995', '374', '517', 'w', 'k€uhlbrandt', 'nature', '1995', '374', '497', '192', 'gr', 'fleming', 'gd', 'scholes', 'nature', '2004', '431', '256', 's', 'jang', 'md', 'newton', 'rj', 'silbey', 'jphys', 'rev', 'lett', '2004', '92', '218301', '193', 'c', 'devadoss', 'p', 'bharathi', 'js', 'moore', 'j', 'am', 'chem', 'soc1996', '118', '9635', 'jm', 'serin', 'dw', 'brousmiche', 'j', 'm', 'fre´chet', 'j', 'chem', 'commun', '2002', '2605', 't', 'weil', 'e', 'reuther', 'k', 'mullen', 'angew', 'chem,', 'int', 'ed', '2002', '41', '1900', 'i', 'yamazaki', 'ntamai', 'tyamazaki', 'a', 'murakami', 'm', 'mimuro', 'yj', 'fujita', 'phys', 'chem', '1988', '92', '5035', 'mn', 'berberan-santos', 'jmg', 'martinho', 'j', 'phys', 'chem', '1990', '94', '5847', '194', 's', 'kawahara', 'tuchimaru', 'murata', 'chem', 'commun1999', '563', 'ak', 'tong', 's', 'jockusch', 'z', 'li', 'h-rzhu', 'dl', 'akins', 'hj', 'turro', 'j', 'ju', 'j', 'am', 'chem', 'soc', '2001123', '12923', '195', 'rs', 'deshpande', 'v', 'bulovic', 'sr', 'forrest', 'appi', 'phys', 'lett', '1999', '75', '888;', 'k-y', 'peng', 's-a', 'chen', 'ws', 'fann', 'j', 'am', 'chem', 'soc', '2001', '123', '11388', 'j', 'kim', 'dt', 'mcquade', 'a', 'rose', 'z', 'zhu', 'tm', 'swager', 'j', 'am', 'chem', 'soc', '2001', '123', '11488', '196', 'pl', 'burn', 'ab', 'holmes', 'a', 'kraft', 'ddc', 'bradley', 'ar', 'brown', 'rh', 'friend', 'rw', 'gymer', 'nature', '1992', '356', '47', 'z', 'yang', 'i', 'sokolik', 'fe', 'karasz', 'macromolecules', '1993', '26', '1188', '197', 'kt', 'nielsen', 'h', 'spangard', 'fc', 'krebs', 'macromolecules', '2005', '38', '1180', '198', 'y-j', 'cheng', 't-y', 'luh', 'macromolecules', '2005', '38', '456–4568', '199', 'm-c', 'fang', 'a', 'watanabe', 'm', 'matsuda', 'macromolecules', '1996', '29', '6807', '200', 'j', 'march', '“advanced', 'organic', 'chemistry”', '3-ed,', 'wiley', 'new', 'york', '1985', '201', 'm', 'onciu', 'c', 'chirlac', 's', 'vlad', 'g', 'stoica', 'g', 'grigoriu', 'd', 'daniel', 'rev', 'roum', 'chim', '1998', '43(7)', '641-648', '202', 't', 'nishijima', 't', 'nagamura', 't', 'matsuo', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1981', '19', '65', '203', 's', 'mayer', 'r', 'znetel', 'macromol', 'chem', 'phys,', '1998', '199', '1675', '204', 'r', 'mruk', 'r', 'zentel', 'macromolecules', '2002', '35', '185', '205', 'o', 'pieroni', 'a', 'fissi', 'n', 'angelini', 'f', 'lenci', 'acc', 'chem', 'res,', '2001', '34', '9', '206', 'zimmerman', 'g;', 'chow', 'l-y;', 'paik', 'u-j', 'j', 'am', 'chem', 'soc', '1958', '80', '3528–3531', '207', 'kumar', 'g', 's;', 'neckers', 'd', 'c', 'chem', 'rev', '1989', '89', '1915-1925', '208', 'delang', 'j', 'j;', 'robertson', 'j', 'm;', 'woodward', '1', 'proc', 'r', 'soc', 'london', 'ser', 'a', '1939', '171', '398', '209', 'f', 'agolini', 'fp', 'gay', 'macromolecules', '1970', '3', '249', '210', 'c', 'gilles', 'i', 'faysal', 'v', 'm', 'rotello', 'macromolecules', '2000', '33', '9173', '211', 'y', 'nishikata', 'j', 'minabe', 'k', 'kono', 'k', 'baba', 'kazuo', 'jpn', 'kokai', 'tokkyo', 'koho', 'jp', '2000', '109719', '(april', '18', '2000)', '212', 'a', 'izumi', 'r', 'nomura', 't', 'masuda', 'macromolecules', '2001', '34', '4342–4347', '213', 'alzumi', 'm', 'teraguchi', 'r', 'nomura', 't', 'masuda', 'j', 'polym', 'sci,', 'polym', 'chem', '2000', '38', '1057-1063.', 'a', 'izumi', 'm', 'teraguchi', 'r', 'nomura', 't', 'masuda', 'macromolecules', '200033', '5347–5352', '214', 'r', 'lovrien', 'proc', 'natl', 'acad', 'sci', 'usa,', '1967', '57', '236', '215', 'l', 'matejka', 'k', 'dusek', 'makromol', 'chem,', '1981', '182', '3223', '216', 'ml', 'hallensleben', 'h', 'menzel', 'brit', 'polymer', 'j', '1990', '23', '199', 'references', '789', '217', 'm', 'irie', 'y', 'hirano', 's', 'hashimoto', 'k', 'hayashi', 'macromolecules', '1981', '14', '1912', 'm', 'moniruzzaman', 'cj', 'sabey', '218', 'gs', 'kamar', 'p', 'depra', 'dc', 'neckers', 'macromolecules', '1984', '17', '1912', 'gs', 'kamar', 'p', 'depra', 'k', 'zhang', 'gf', 'fernando', 'macromolecules', '2004', '37', '2572', 'dc', 'neckers', 'macromolecules', '1984', '17', '2463', '219', 'm', 'irie', 't', 'sizuki', 'makromol', 'chem', 'rapid', 'commun', '1987', '8', '607', 'm', 'irie', 'm', 'hsoda', 'makromol', 'chem', 'rapid', 'commun', '1985', '6', '533', 'p', 'strohriegl', 'makromol', 'chem,', 'rapid', 'commun,', '1986', '7', '771', 'rn', 'thomas', 'j', 'polymer', 'sci,', 'chem', 'ed,', '32', '2727', '(1994)', '220', 'o', 'ryba', 'j', 'petranek', 'makromol', 'chem,', 'rapid', 'commun,', '1988', '9', '125', '221', 'a', 'natansohn', 'p', 'rochon', 'j', 'gosselin', 's', 'xie', 'macromolecules', '1992', '25', '2268', '222', 'm', 'irie', 'r', 'iga', 'macromolecules', '1986', '19', '2480', '223', 's', 'mitra', 'am', 'chem', 'soc', 'polymer', 'preprints', '26', '(1)', '78', '(1985)', '224', 'ctlee', 'jr,', 'k', 'a', 'smith', 'ta', 'hatton', 'macromolecules', '200437', '5397', '225', 'm', 'han', 's', 'morino', 'k', 'ichimura', 'macromolecules', '2000', '33', '6360–6371', '226', 'i', 'zebger', 'm', 'rutloh', 'u', 'hoffmann', 'j', 'stumpe', 'hw', 'siesler', 's', 'hvilsted', 'macromolecules', '2003', '36', '9373i', 'tomatsu', 'a', 'hashidzume', 'a', 'harada', 'macromolecules', '2005', '38', '5223–5227', '227', 'tj', 'ikeda', 'j', 'mater', 'chem', '2003', '13', '2037', '228', 'y', 'zhao', 'y', 'chenard', 'n', 'paiement', 'macromolecules', '2000', '33', '5891', '229', 'y', 'zhao', 'y', 'chenard', 'macromolecules', '2000', '33', '5891', '230', 'm', 'han', 's', 'morino', 'k', 'ichimura', 'macromolecules', '2000', '33', '6360–6371', '231', 'r', 'rosenhauer', 'th', 'fischer', 'j', 'stumpe', 'r', 'gimenez', 'm', 'pinole', 'j', 'l', 'serrano', 'a', 'vinuales', 'd', 'broer', 'macromolecules', '200538', '2213-2222', '232', 'y', 'wu', 'j', 'mamiya', 'a', 'kanazawa', 't', 'shiono', 't', 'ikeda', 'q', 'zhang', 'macromolecules', '1999328829–8835', '233', 'i', 'zebger', 'm', 'rutloh', 'u', 'hoffmann', 'j', 'stumpe', 'hw', 'siesler', 's', 'hvilsted', 'macromolecules', '2003', '36', '9373', '234', 'y', 'zhao', 'n', 'paiement', 'adv', 'mat,', '2001', '13', '1891', '235', 'y', 'zhao', 'n', 'paiement', 's', 'sevigny', 's', 'leclair', 's', 'motallebi', 'm', 'gigueri', 'm', 'bouchard', 'spsi', 'proc,', '2003', '5003', '150', '236', 's', 'sevigny', 'l', 'bouchard', 's', 'motallebi', 'y', 'zhao', 'macromolecules', '2000', '33', '9033', '237', 'x', 'tong', 'l', 'cui', 'y', 'zhao', 'macromolecules', '2004', '37', '3101', '238', 'n', 'zettsu', 't', 'seki', 'macromolecules', '200437', '8692-8698', '239', 'sa', 'asher', 'pl', 'flaugh', 'g', 'washinger', 'g', 'spectroscopy', '1986', '1', '26', 'rj', 'carisonand', 'sa', 'asher', 'appl', 'spectrosc', '1984', '38', '297', 'creese', 'sa', 'asher', 'j', 'colloid', 'interface', 'sci', '2002', '248', '41', '240', 'sa', 'asher', 'j', 'holtz', 'l', 'liu', 'z', 'wu', 'j', 'am', 'chem', 'soc,', '1994', '116', '4997', '241', 'm', 'kamenjicki', 's', 'a', 'asher', 'macromolecules', '200437', '8293-8296', '242', 'i', 'zebger', 'm', 'rutloh', 'u', 'hoffmann', 'j', 'stumpe', 'hw', 'siesler', 'shvilsted', 'macromolecules', '2003', '36', '9373', '243', 'rg', 'kepler', 'phys', 'rev', 'lett,', '1967', '18', '951', 'p', 'holtzman', 'r', 'morris', 'rc', 'jarnagin', 'm', 'silver', 'phys', 'rev', 'lett,', '1967', '19', '506', '244', 'et', 'kang', 'p', 'ehrlich', 'ap', 'bhatt', 'wa', 'anderson', 'macromolecules', '17', '1020', '(1984)', '245', 'f', 'goodwin', 'ie', 'lyons', 'organic', 'semiconductors', 'wiley', 'new', 'york', '(1967)', '246', 'h', 'bauser', 'w', 'kloepfer', 'chem', 'phys', 'lett,', '1970', '7', '137', '247', 'w', 's', 'lyoo', 'j', 'polym', 'sci', 'a', 'polym', 'chem', '200139539–54555', '248', 'hh', 'horhold', 'h', 'rathe', 'makromol', 'chem,', '1987', '188', '2083', '249', 'sa', 'haque', 't', 'uryu', 'h', 'ohkawa', 'makromol', 'chem,', '1987', '188', '2521', '250', 'ch', 'hu', 'r', 'oshima', 'm', 'seno', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1988', '26', '1239', 'a', 'natansohn', 'j', 'polymer', 'sci,', '251', 't', 'masuda', 'acetylenic', 'polymers', 'polymeric', 'materials', 'encyclopedia', 'js', 'salamone', 'ed,', 'crc', 'press', 'new', 'york', 'chem', 'ed,', '1984', '22', '3161', '1996', '252', 'n', 'kim', 'se', 'weber', 'macromolecules', '1985', '18', '741', '253', 'f', 'sanda', 't', 'nakai', 'n', 'kobayashi', 't', 'masuda', 'macromolecules', '2004', '37', '2703', '254', 'p', 'strohriegl', 'makromol', 'chem,', 'rapid', 'commun,', '1986', '7', '771', '255', 'a', 'natansohn', 'j', 'polymer', 'sci,', 'chem', 'ed,', '1984', '22', '3161', '256', 'sc', 'suh', 'sc', 'shim', 'synth', 'met', '2000', '114(1)', '91-95', '257', 'o', 'ryba', 'j', 'petranek', 'makromol', 'chem,', 'rapid', 'commun1988,', '9', '125', '258', 'scfreilich', 'macromolecules', '1987', '20', '973', '259', 'k', 'kimura', 'h', 'sakamoto', 'rm', 'uda', 'macromolecules', '2004', '37', '1871', '260', 'sh', 'chan', 'lsm', 'lam', 'cw', 'tse', 'kyk', 'man', 'wt', 'wong', 'ab', 'djurisic', 'wk', 'chan', 'macromolecules', '261', 'sa', 'jenekhe', 'lr', 'de', 'paor', 'am', 'chem', 'soc', 'polymer', 'preprints', '1966', '37', '(1)', '94', '262', 'hoppe', 'h;', 'sariciftci', 'n', 's', 'j', 'mater', 'chem', '2004', '19', '1924', '263', 'scharber', 'm;', 'miihlbacher', 'd;', 'koppe', 'm;', 'denk', 'p;', 'waldauf', 'c;', 'heeger', 'a;', 'brabec', 'cadv.', 'mater', '2006', '18', '2003', '36', '5482', '789–794', '790', '10', 'polymeric', 'materials', 'special', 'applications', '789–794', '2010', '43', '5706', '50(1)', '487', '43', '6033', '264', 'chem', 'eng', 'news', 'august', '23', '2010', 'p', '12', '265', 'advanced', 'materials', '2009', '21', '19', '266', 'scharber', 'm;', 'miihlbacher', 'd;', 'koppe', 'm;', 'denk', 'p;', 'waldauf', 'c;', 'heeger', 'a;', 'brabec', 'cadv.', 'mater', '2006', '18', '267', 'yu', 'et', 'al', 'nat', 'photonics', '2009', '3', '649', '268', 'yu', 'et', 'al', 'acc', 'chem', 'res.', 'doi1021ar1000296', '269', 'chem', 'eng', 'news', 'aug', '17', '200910', '270', 'chem', 'eng', 'news', 'aug', '24', '2010', '271', 'y-j', 'cheng', 's-h', 'yang', 'ch-t', 'hsu', 'chem', 'rev,', '2009', '109', '272', 'm', 'zhang', 'h', 'fan', 'x', 'guo', 'y', 'he,', 'z', 'zhang', 'j', 'min', 'j', 'zhang', 'g', 'zhao', 'x', 'zhang', 'y', 'li', 'macromolecules', '273', 's', 'honda', 's', 'yokoya', 't', 'nogami', 'h', 'ohkita', 'h', 'benten', 's', 'ito', 'am', 'chem', 'soc', 'polymer', 'preprints', '2009', '274', 'r', 'c', 'hiorns', 'e', 'cloutet', 'eibarboure', 'a', 'khoukh', 'h', 'bejbouji', 'l', 'vlgnau', 'h', 'cramail', 'macromolecules', '2010', '275', 'z-g', 'zhang', 'y-l', 'liu', 'y', 'yan', 'khou', 'b', 'peng', 'g', 'zliao', 'm', 'zhang', 'xguo', 'e-t', 'kang', 'y', 'li', 'macromolecules', '2010', '43', '9376', '276', 'sc', 'price', 'ac', 'stuart', 'w', 'you', 'macromolecules', '2010', '43', '4609', '277', 'x', 'guo', 'h', 'xin', 'f', 's', 'kim', 'adtliyanage', 's', 'a', 'jenekhe', 'm', 'd', 'watson', 'macromolecules', '2011', '44', '269', '278', 'j', 'a', 'mlkroyannidis', 'a', 'n', 'kabanekis', 'p', 'balraju', 'g', 'd', 'sharma', 'macromolecules', '2010', '43', '5544', '279', 'm-c', 'yuan', 'm-y', 'chiu', 'c-m', 'chiang', 'k-h', 'wei', 'macromolecules', '2010', '43', '6270', '280', 'h', 'li', 'p', 'jiang', 'c', 'yi', 'c', 'li', 's-xliu', 's', 'tan', 'b', 'zhao', 'j', 'braun', 'w', 'meier', 't', 'wandlowski', 'sdecurtins', 'macromolecules', '2010', '43', '8058', '281', 'b', 'burkhart', 'p', 'p', 'khlyabich', 'tc', 'canak', 't', 'w', 'lajoie', 'b', 'c', 'thompson', 'macromolecules', '2011', '44', '1242', '282', 'walker', 'b,', 'tamayo', 'a', 'b,', 'dang', 'x-d,', 'zalar', 'p', 'seo', 'j', 'h;', 'garcia', 'a;', 'tantiwiwat', 'm;', 'nguyen', 't-q,', 'adv', 'funct', 'mater', '2009', '19', '1', '283', 'yang', 'yang', 'acs', 'nano', '2011', 'doi', '101021nn202144b', 'index', 'a', 'aba', 'block', 'copolymers', '305', '631', 'ab', 'block', 'copolymers', '305', 'abs', 'resins', '371', 'acetal', 'polymers', '180', '231', '273', 'acetylene', 'polymerization', '330', '358', '712', 'acrolein', 'polymerization', '226', 'acrylamide', 'polymerization', '87', '113', '139', '197', '198', '242', '381', '431', 'acrylic', 'elastomers', '375–376', 'acrylic', 'polymers', '647–648', '703', 'acrylic', 'polymers', 'support', 'materials', '696–704', 'acrylonitrile', 'methacrylonitrile', 'polymers', '379–381', 'activation', 'energy', 'viscous', 'ﬂow', '29–30', 'activators', 'lactam', 'polymerization', '285', 'addition', 'carbenes', '575–576', 'mode', '173', '207', 'polymer', 'deﬁnition', '3–7', 'reactions', 'polymers', '556', '568', '570', '574', 'adenine', '559', '561', 'adjacent-reentry', 'model', '37', 'agar', '546', 'alcoholysis', 'process', 'preparation', 'alkyds', '426', 'alﬁn', 'catalysts', 'initiation', '189–190', 'alginic', 'acid', '537', '545', 'alkyd', 'resins', '426', 'allylic', 'polymerization', '102–103', 'allylic', 'resins', '368', '557', '701', 'alpha', 'helix', '548', '552', '553', 'alternating', 'copolymer', 'deﬁnition', '3–7', 'a-amino', 'acids', '430', '431', '547', '548', 'amino', 'polymers', '492–494', 'ammonia', 'catalyzed', 'phenolic', 'resins', '490–491', 'amorphous', 'crystalline', 'arrangements', 'thermoplastic', 'polymers', '17', 'lactams', '290–296', 'lactones', '280–281', 'oleﬁns', '182', 'oxiranes', '255', 'antenna', 'effect', 'polymers', '732', 'araban', 'see', 'hemicelluloses', 'arbuzov', 'reaction', '599', 'aromatic', 'polyamide-imides', '447–449', 'aromatic', 'polyamides', '443–447', '515', '674', '756', 'aromatic', 'polyester-imides', '447–449', 'a-stage', 'phenolic', 'resin', '487–490', 'asymmetry', 'structural', 'recovery', '22', 'atactic', 'conformation', '11', '265', '341', '342', '570', 'atom', 'transfer', 'radical', 'polymerization', '121', '636', '702', '748', 'autoacceleration', '89–90', '387', 'average', 'degree', 'polymerization', '41', '71', '95', '137', '152–154', '254', '407', '409', '524', 'aziridine', 'polymerization', '307', '308', 'azonitrile', 'initiators', '73', 'decomposition', 'rates', '73', 'b', 'backbiting', '94', '177', '199', '268', '273', '275', '308', '331', '645', 'backbone', 'polymer', '4', '8', '46', '60', '61', '95', '310', '390–392', '408', '424', '507', '569', '584', '594', '618', '628', '643', '650', '663', '731', '741', '753', '759', '772', '777', 'balata', '547', 'bechmann', 'rearrangement', '432', 'benzoyl', 'peroxide', '74', '75', '78', '81', '122', '330', '387', '388', '570', '598', '618', '619', 'bernoulli', 'model', 'stereochemistry', '234', '236', '237', '239', 'bimetallic', 'mechanism', '203', 'bimolecular', 'initiating', 'systems', '76–79', 'bingham-newtonian', 'liquid', '28', '29', 'block', 'copolymers', 'amorphous', 'state', '21–34', 'amylose', 'amylopectin', '538', '539', 'anionic', 'chain', 'growth', 'polymerization', 'oleﬁns', '182–201', 'anionic', 'initiations', 'chain-growth', 'polymerization', '182–191', 'anionic', 'polymerization', 'aldehydes', '221', '223–226', 'copolyamides', '632', 'copolyesters', '631–632', 'deﬁnition', '3–7', 'preparation', 'homogeneous', 'ionic', 'polymerization', '637–639', 'poly(a-oleﬁns)', '634–635', 'a', 'ravve', 'principles', 'polymer', 'chemistry', 'doi', '101007978-1-4614-2212-9', '1st', '2nd', 'editions', 'kluwer', 'academicplenum', 'publishers', '1995', '2000', '3rd', 'edition', 'springer', 'sciencebusiness', 'media', 'llc', '2012', '791', '792', 'index', 'block', 'copolymers', '(cont)', 'simultaneous', 'use', 'free', 'radical', 'ionic', 'chain', 'growth', 'polymerization', '635–636', 'special', 'reactions', '639–641', 'boron', 'metal', 'alkyl', 'initiators', 'free-radical', 'polymerizations', '79', 'branched', 'polymer', 'deﬁnition', '3–7', 'bromination', 'rubber', '573', 'b-stage', 'phenolic', 'resin', '486', 'bulk', 'polymerization', '84', '89', '90', '133', '342', '353', '373', '380', '387', '388', 'buna', 'rubber', 'see', 'gr-s', 'butyl', 'rubber', '344', 'c', 'cage', 'effect', '80', 'canal', 'complexes', '113', '114', '142', 'caoutchouc', 'see', 'naturally', 'occurring', 'polyisoprenes', 'capillary', 'viscometer', '33', '54', 'capture', 'free-radical', 'monomer', '81', 'carbonyl', 'compounds', 'polymerization', '225', '456', 'carboxymethyl', 'cellulose', '624', '705', 'cardo', 'polymers', '517', 'carothers', 'equation', '407–410', '735', 'carrageenan', '545', 'casein', '134', '296', '553', '557', '571', '583', '728', 'catalyst', 'site', 'control', '206', 'cation', 'generator', '162', 'cationic', 'chain-growth', 'polymerization', '159', 'cationic', 'initiators', '221', '223', '255', '273', '278', '361', 'cationic', 'isomerization', 'polymerization', '174–177', '220–221', 'cationic', 'polymerization', 'aldehydes', '162', '165', '221–223', 'lactams', '284–297', 'lactones', '278–284', '415', 'oleﬁns', '79', '156', '157', '159', '166', '167', '173', '627', 'oxiranes', '264–266', 'ceiling', 'temperature', '88–89', '132', '644', 'celcon', '461', 'celluloid', '541', 'cellulose', 'acetate', '2', '542–543', '619', 'acetate-butyrate', '543', 'esters', '543', 'ethers', '543–545', 'nitrate', '1', '2', '23', '541', 'chain-growth', 'polymerization', '7', '69–139', '151–240', '365–367', '385', 'chain-growth', 'polymers', 'deﬁnition', '7', '664', 'chain', 'transfer', 'agents', '353', 'chain', 'transfer', 'constant', '104', 'chardonnet', 'silk', '540', 'charge', 'transfer', 'complexes', 'free-radical', 'polymerization', '108', '164', 'ionic', 'initiation', '164–166', 'charge', 'transfer', 'process', '728–732', 'chemical', 'modiﬁcation', 'proteins', '556–557', 'chemistry', 'ionic', 'chain', 'polymerization', '151', 'chemistry', 'ring-opening', 'polymerizations', '253', 'chicle', '547', 'chlorination', 'poly(vinyl', 'chloride)', '592', '593', '651', 'poly(vinyl', 'ﬂuoride)', '593', 'poly(vinylidine', 'chloride)', '389–390', 'polyethylene', '590–591', 'polypropylene', '591–592', 'polystyrene', '(see', 'halogenation', 'reactions', 'polystyrene)', 'rubber', '572', '573', '587', 'chloromethylation', 'polymers', 'aromatic', 'chloroprene', 'rubber', 'poly(2-chloro-13-butadiene)', 'rings', '597–606', '358–360', 'chlorosulfonation', 'polyethylene', '592', 'cis-trans', 'isomerization', '755–757', 'clathrate', 'polymerization', 'see', 'canal', 'complexes', 'cleavage', 'peroxides', '74', 'cobalt', 'mediated', 'polymerization', '116–117', 'coinitiator', '152', '156', '157', '162', '167', 'collagen', '548', '553', '557', 'colligative', 'properties', '51', 'collodion', '541', 'colman-fox', 'model', '234', 'combinations', 'atp', 'raft', '128–129', 'click', 'chemistry', 'atp', '128–129', 'comb-shaped', 'polymer', '620', 'commercial', 'linear', 'saturated', 'polyesters', '417–420', 'commercial', 'polyisocyanates', '469–470', 'commercial', 'preparations', 'phenolic', 'resins', '491', 'complex', 'coordination', 'catalysts', '227', 'condensation', 'polymer', 'deﬁnition', '5', 'conductive', 'polymers', '711', '715', 'cone', 'plate', 'rheometer', '32', '33', 'conﬁgurational', 'statistics', 'chain-growth', 'propagation', 'mechanism', '234–239', 'conformation', 'see', 'steric', 'arrangement', 'macromolecules', 'conjugated', 'proteins', '548', '553', 'constitutional', 'repeat', 'unit', '10', 'contact', 'ion', 'pair', 'see', 'intimate', 'ion', 'pair', 'controlled', '“living”', 'polymerization', '92', '114–131', '179', '269', '280', '635', '636', 'coordination', 'polymerization', 'lactones', '281–283', 'oleﬁns', '201–221', 'oxiranes', '255–266', 'copolyamides', '311', '439', '444', '511', '632', 'copolyesters', '45', '311', '421–424', '511', '631–632', 'copolymerization', 'cyclic', 'monomers', '311–312', 'equation', '96–101', 'free-radical', 'mechanism', '361', 'ionic', 'mechanism', '228–231', 'step-growth', 'mechanism', '6', 'copolymers', 'dienes', '362', 'ethylene', 'propylene', '230', '344', 'index', '793', 'ethylene', 'a-oleﬁns', 'carbon', 'monoxide', '348–351', 'ethylene', 'vinyl', 'acetate', '351', 'ﬂuorooleﬁns', '384–385', 'propylene', 'dienes', '351', 'styrene', '364', '372', '569', '598', '627', 'vinyl', 'chloride', '8', '389', '623', '652', 'counter', 'ion', '152', '154', '156', '160', '163', '168–173', '176–179', '191–196', '221', '222', '228', '229', '238', '253', '270', '272', '273', '294', '308', '311', '333', '349', '366', 'crosslinking', 'polymers', 'electron', 'beams', 'peroxides', '616–617', 'crosslinking', 'reactions', 'epoxy', 'resins', '476–481', 'polymers', '574', '598', '614–617', 'rubbers', '614', 'cryoscopy', '1', '54', '55', 'crystalline', 'melting', 'point', '(tm)', '40–42', '48', '368–370', '382', '384', '422–424', '428', '429', '439', '461', '472', 'crystalline', 'state', '34–43', 'crystallinity', 'polymers', '48', '340', 'crystallites', '36–401', '664', 'crystallization', 'melt', '34–36', 'solution', '36–38', 'c-stage', 'phenolic', 'resin', 'cuprammonium', 'rayon', '540', 'cyanoguanidine', '477', 'cyclization', 'reactions', 'intramolecular', 'd', 'dacron', 'degenerative', 'chain', 'transferring', '93', 'degradation', 'ionizing', 'radiation', '677', 'degradation', 'polymeric', 'materials', 'photo-oxidation', '674–676', 'degree', 'polymerization', '(dp)', 'anionic', 'polymerization', '153', 'cationic', 'polymerization', '153', 'free-radical', 'polymerization', '102', '103', 'delrin', '461', 'denaturation', 'proteins', '552', 'dendrimers', 'rotaxanes', 'hyperbranched', 'polymers', '522–523', 'deoxyribonucleic', 'acid', '(dna)', '537', '557–560', 'depolymerization', '88', '257', '461', '644', '645', '647', '649', '662', '663', 'derivatives', 'cellulose', '541–545', 'diallyl', 'phthalate', 'see', 'allylic', 'polymerization', 'diazotization', 'poly(amino', 'styrene)', '603–606', '706', 'dicyanodiamide', '477', 'diels-alder', 'polymers', '505–511', 'diels-alder', 'reactions', '454', '582', '609', 'diene', 'polymerization', '192', '210', '303', '347', '362', '645', '712', 'diffusion', 'controlled', 'reactions', '121', '569', 'diffusion', 'controlled', 'termination', '121', 'diisotactic', 'polymer', '207', '573', 'dilatent', 'ﬂow', '30', 'dimer', 'acids', '441', 'dipolar', 'additions', 'unsaturated', 'polymers', '577–578', 'direct', 'condensation', 'polymers', '512–514', 'dispersion', 'polymerization', '134', 'disproportionation', '70', '71', '80', '89', '93', '108', '119', '151', '373', '382', '618', '644', 'di', 'syndiotactic', 'polymer', '12', '343', '367', 'dna', 'see', 'deoxyribonucleic', 'acid', '(dna)', 'donor-acceptor', 'complexes', 'copolymerization', '107–111', 'double', 'stranded', 'polymers', '517–520', 'dp', 'see', 'degree', 'polymerization', '(dp)', 'drawn', 'ﬁbrilar', 'morphology', '39', 'dyad', '226', '234–236', '238', '239', 'dynamic', 'equilibrium', 'method', 'osmometry', '55', 'e', 'ebullioscopy', '54', '55', 'effect', 'catalysts', 'reaction', 'isocyanates', '471–472', 'effect', 'environment', 'polymers', '643', '760', 'effect', 'lewis', 'bases', 'coordination', 'polymerization', '219', 'effect', 'molecular', 'size', 'reactions', 'polymers', '570', 'effect', 'reaction', 'medium', 'free-radical', '87–88', 'effects', 'chain', 'conformation', 'reactions', 'effects', 'changes', 'solubility', 'reactions', 'polymers', '570–571', 'effects', 'chemical', 'structure', 'physical', 'properties', 'organic', 'polymers', '750', 'effects', 'crystallinity', 'reactions', 'polymers', '571', 'effects', 'dipole', 'interactions', '17–18', 'elasticity', '24–28', '31', '63', '342', 'elastomer', '18', '19', '24–28', '41', '44', '47', '63', '342', '344', '347–348', '351', '358', '361', '363', '371', '375–376', '379', '384', '385', '394', '395', '421', '466–468', '473–474', '496–498', '501', '525', '527', '528', '562', '582', '591', '592', '614–616', '633', '634', 'electricity', 'conducting', 'polymers', '710–717', 'electroinitiation', 'anionic', 'chain-growth', 'polymerization', '182', 'cationic', 'chain-growth', 'polymerization', '166', 'electrolytic', 'polymerization', 'deﬁnition', '166', 'electron', 'transfer', 'process', '375', '729', '776', 'electrophilic', 'additions', 'aldehydes', 'rubber', '576–577', 'emeraldine', 'salt', '715', 'emulsion', 'polymerization', '135', '136', '138', '139', '351', '357', '358', '362', '375', '380', '381', '386', '389–391', '651', 'enantiomorphic', 'site', 'control', '206', '238', 'end', 'group', 'anaylsis', '180', 'ene', 'reactions', 'polybutadiene', 'polyisoprene', '676', 'energy', 'dipole', 'interaction', '18', 'energy', 'transfer', 'process', '726–729', 'entropy', 'elasticity', '25', 'rearrangements', 'polymers', '586–589', 'polymers', '570', 'cycloaliphatic', 'epoxides', '482–483', 'cyclopolymerization', 'conjugated', 'dienes', '360–361', 'cytosine', '559', '794', 'index', 'enzymes', '2', '217', '545', '548', '553', '554', '556', '557', '695', '705–707', '715', 'episulﬁde', 'polymerization', 'epitaxial', 'crystallinity', '38', 'epoxidation', 'unsaturated', 'polymers', '583', 'epoxy', 'novolacs', '480', 'epoxy', 'resins', 'crosslinking', 'reactions', '476–481', 'preparation', '475–476', 'equation', 'work', 'done', 'stretching', 'elastomer', '25–27', 'equivalence', 'postulate', '12', '35', 'erythrodiisotactic', 'conformation', '207', 'erythrodisyndiotactic', 'conformation', 'ethyl', 'cellulose', '134', '543', 'ethylene', 'oxide', 'polymerization', '253', '259–261', '319', '629', 'ethylene-propylene', 'elastomers', '347–348', 'ethylene-vinyl', 'acetate', 'copolymers', '351', 'exiplex', 'see', 'charge', 'transfer', 'complexes', 'f', 'fatty', 'acid', 'process', '426', 'fatty', 'polyamides', '430', '441', 'fibrilar', 'morphology', '39', 'fibroin', '548', '551', 'fibrous', 'proteins', '548', 'first-order', 'markov', 'model', '236', '238', 'first-order', 'termination', 'reaction', '93', 'first-order', 'transition', 'temperature', '41', 'flash', 'polymerization', '344', 'flory-huggins', 'theory', 'polymer', 'solutions', '50', 'fluorination', 'polyethylene', '590', 'fluorine', 'containing', 'aromatic', 'polymers', '502–504', 'fluoropolymers', '382', 'fluorosilicone', 'elastomers', '498', 'folded-chain', 'lamella', '37', 'formaldehyde', 'polymerization', '6', '221–223', '461', 'formation', 'graft', 'copolymers', 'free-radical', 'chain', 'transferring', '617–619', 'formation', 'graft', 'copolymers', 'aid', 'high-energy', 'radiation', '626–627', 'formation', 'initiating', 'free-radicals', '72–80', 'fox-flory', 'equation', '23', 'free', 'ion', '154', '166', '173', '174', '201', '294', 'free-radical', 'chain', 'polymerization', '69–143', '151', 'free-radical', 'initiators', '125', '330', '380', '582', 'friedel-craft', 'alkylation', 'reactions', 'polymers', '602–603', 'friedel-craft', 'crosslinking', '464', '598', '603', 'fringed', 'micelle', 'model', '36', 'g', 'galactan', '537', '538', 'gas-phase', 'oleﬁn', 'polymerization', 'process', '336', '337', 'gelation', '6', '29', '385', '409', '410', '481', '568', '571', '577', '631', '735', '736', '745', 'gel', 'permeation', 'chromatography', '54', '57', '58', 'genetic', 'zwitterions', '314', 'glass', 'transition', 'glassy', 'state', '21–24', 'glass', 'transition', 'temperature', '22–24', '30', '48', '213', '466', '520', '521', '725', '728', '763', '772', 'globular', 'proteins', '548', '553', 'glyptals', '425', 'graft', 'copolymers', 'aid', 'ionizing', 'radiation', '166', 'aid', 'macromonomers', '620–622', 'deﬁnition', '3–7', 'ionic', 'chain-growth', 'step-growth', 'polymerizations', '627–630', 'preparation', '620–622', 'grafting', 'reactions', 'polymers', 'double', 'bonds', '619–620', 'gr-n', 'rubber', '363–364', 'group-transfer', 'polymerization', '231–234', 'gr-s', 'rubber', '361–363', 'grubbs', 'catalysts', '303', '305', '353', 'guanine', '559', '561', 'guayule', 'rubber', '547', 'guncotton', 'see', 'cellulose', 'nitrate', 'gutta-percha', 'rubber', '547', '619', 'h', 'halogenation', 'polymers', '572–574', 'halogenation', 'reactions', 'polystyrene', '573', 'halogen', 'bearing', 'polymers', '382–389', '592–597', '628', 'hdpe', 'see', 'high', 'density', 'polyethylene', '(hdpe)', 'head', 'head', 'placement', '265', '387', '646', 'head', 'tail', 'placement', '86', 'heme', '548', '551', '553', 'hemicelluloses', '537–538', 'hemoglobin', '548', '553', 'heterogeneous', 'ziegler-natta', 'catalysts', '202–207', 'heterolytic', 'cleavage', 'peroxides', '75', 'heterotactic', 'conformation', 'see', 'atactic', 'conformation', 'high', 'density', 'polyethylene', '(hdpe)', '42', '332', '335–338', '590', '644', 'higher', 'poly(a-oleﬁn)s', '201', '220', 'high', 'impact', 'polystyrene', '370–371', 'high', 'molecular', 'weight', 'polybutadiene', '354–356', 'high', 'performance', 'polymers', '502–521', 'homogeneous', 'ziegler-natta', 'catalysts', '207–209', '348', 'homolytic', 'cleavage', 'peroxides', '74', 'hookian', 'solid', '28', 'hydroboration', 'polymers', '577', '578', 'hydrodynamic', 'volume', '49', '57', '732', 'hydrogenation', 'polymers', '574–575', '699', '700', '703', 'hydrogen', 'bonding', '17', '18', '20', '48', '84', '132', '374', '392', '413', '440', '524', '540', '548', '551', '552', '559', 'hydrogen', 'transfer', 'polymerization', '197–198', '431', 'hydrolytic', 'degradation', 'polymers', '662–663', 'hydrolytic', 'polymerization', 'lactams', '296–297', 'hydroperoxides', '73', '623', '626', '667–669', '674–676', 'hytrel', 'copolyester', 'elastomer', '421', 'index', '795', 'i', 'imine', 'polymerization', '307', '308', 'immobilized', 'catalysts', '705–707', 'immobilized', 'enzymes', '705–707', 'immobilized', 'reagents', '709–710', 'immortal', 'polymerization', '261', 'induced', 'decomposition', 'peroxides', '72', '78', 'induced', 'dipole', '17', '18', 'induction', 'forces', 'polymers', '18–21', 'induction', 'period', '103', '106', '119', '184', '233', '274', '288', '291', '292', '330', 'inherent', 'viscosity', '53', '65', 'inhibition', 'retardation', '103–106', '141', 'inhibitors', '103–106', '141', '151', '314', '379', '388', '655', 'inifer', '115', '638', '639', 'initiation', 'anionic', 'chain-growth', 'polymerization', '182–191', 'cationic', 'chain-growth', 'polymerization', '155–167', 'free-radical', 'chain-growth', 'polymerization', '69–143', 'graft', 'copolymerization', 'polymeric', 'backbones', '622–625', 'lactams', 'polymerization', '284–297', 'lactones', 'polymerization', '278–284', 'polymerization', 'oxetanes', '266–269', 'polymerization', 'oxiranes', '255–266', 'polymerization', 'tetrahydrofuran', '269–273', 'polymerization', 'radioactive', 'sources', 'electron', 'beams', '80', 'interfacial', 'polycondensation', '412', '632', '750', 'interfacial', 'polymerization', '412', '427', '443', '447', 'intermolecular', 'forces', '18', '548', 'interpolymer', '96', '101', '108', '311', '349', '371', 'intimate', 'ion', 'pair', '167', '192', 'intra-intermolecular', 'polymerization', '175', 'intra-intermolecular', 'reaction', '91', '92', '175', '574', '670', 'intramolecular', 'cycloaddition', 'see', 'intra-intermolecular', 'polymerization', 'intramolecular', 'rearrangements', 'polymers', '588', 'intrinsic', 'viscosity', '53', '54', '759', 'ionic', 'chain-growth', 'polymerization', '7', '151–243', '253', '365–367', 'ion', 'pair', '152', '154', '158', '167', '168', '170–174', '178', '181', '183', '185', '190–192', '194', '195', '199', '201', '209', '218', '229', '272', '279', '293', '294', '428', '707', '729', 'isomerization', 'polymerization', 'cationic', '174–175', 'coordination', '220–221', 'isomerization', 'reactions', 'polymers', '584–586', 'isotactic', 'conformation', '35', '171', 'isotactic', 'dyad', '226', 'isotactic', 'triad', '114', 'k', 'keratin', '548', '552', 'ketones', 'polymerization', '228', 'kevlar', '46', '446', 'kinetic', 'chain', 'length', '70', '71', '89', '137', '139', 'kinetics', 'controlledliving', 'polymerization', '130–131', 'coordination', 'polymerization', 'oleﬁns', '201–221', 'free-radical', 'polymerizations', '69–71', '130–131', 'ionic', 'chain-growth', 'polymerization', '152–154', 'ring-opening', 'polymerizations', '253–255', 'step-growth', 'polymerizations', '403–412', 'l', 'lactams', 'syntheses', 'see', 'nylons', 'lactomolytic', 'propagation', '293', 'ladder', 'polymer', '3', '10', '13', '360', '502', '509', '518', '648', 'lamella', '37–39', '47', '114', '742', 'lauryl', 'lactam', '436', '437', 'lewis', 'acids', 'cationic', 'initiations', 'chain-growth', 'polymerization', '156–160', 'lexan', 'light', 'scattering', 'measurement', '56', 'lignin', '2', '537', '544', '546–547', '562', '762', '763', 'linear', 'polymer', 'deﬁnition', '3–7', 'linear', 'saturated', 'polyesters', '412–424', 'linear', 'unsaturated', 'polyesters', '424–425', 'liquid', 'crystal', 'polymers', '43–47', 'liquid', 'polybutadiene', '353–354', '583', 'living', 'anionic', 'polymerizations', '198', '200', '229', 'living', 'cationic', 'polymerizations', '115', '178–180', 'livingcontrolled', 'free-radical', 'polymerization', '114–131', 'low', 'density', 'polyethylene', '52', '218', '329', '331', '332', '337', 'lyotropic', 'liquid', 'crystal', '46', 'm', 'macromers', 'see', 'macromonomers', 'macromolecules', 'reactivity', '567–572', 'macromonomers', '117', '523', '620–622', 'manufacturing', 'techniques', 'polyethylene', '329', '335–338', 'manufacturing', 'techniques', 'polyisobutylene', '343–344', 'manufacturing', 'techniques', 'polypropylene', '345', 'manufacturing', 'techniques', 'polystyrene', '364', 'mark-houwink-sakurada', 'equation', '54', 'markov', 'statistical', 'model', 'ion', 'stereochemistry', '234', '236', 'mass', 'polymerization', '132', '134', '364', '365', '651', 'materials', 'similar', 'polyethylene', '338–339', 'mechanism', 'kinetics', 'chain', 'growth', 'polymerization', '403', 'mechanochemical', 'techniques', 'formation', 'block', 'copolymers', '643', 'melamine-formaldehyde', 'resins', '378', '492–494', '557', 'membrane', 'osmometry', '55', 'merriﬁeld', 'solid-phase', 'peptide', 'synthesis', '555', 'mesomorphic', 'state', 'liquid', 'crystal', 'polymers', '43–47', 'mesophases', '39', '40', '43–46', 'metal', 'alkyls', 'initiations', 'cationic', 'polymerization', '162–163', 'metalation', 'reactions', 'polymers', '604–606', 'metalloceneborate', 'complexes', '162', 'metallocene', 'catalysts', '213', '214', '216', '334', '341', '347', '367', 'metalloporphyrin', 'initiators', '261', '796', 'index', 'metathesis', 'polymerization', '301–307', '640', '641', '708', '711', 'methods', 'measuring', 'molecular', 'weights', '53–60', 'methyl', 'acrylate', 'synthesis', '522', 'methyl', 'methacrylate', 'synthesis', '8', '129', 'methyl', 'rubber', 'poly(23-dimethylbutadiene)', '358', 'miscellaneous', 'nylons', 'block', 'copolymers', '642', 'crosslinking', 'reactions', '617', 'exchange', 'reactions', '612–614', 'ﬂuorine', 'containing', 'chain-growth', 'polymers', '385–386', 'graft', 'copolymerizations', '630–631', 'polymers', 'dienes', '360', 'polysaccharides', '545–546', 'modulus', 'elasticity', '27', '31', 'molecular', 'cohesion', 'linear', 'polymers', '18', '19', 'molecular', 'weight', 'averages', '51–53', 'molecular', 'weights', 'molecular', 'weight', 'determinations', '51–60', 'monodisperse', 'deﬁnition', '53', 'monomer', 'deﬁnition', '88', '97', 'placement', '86', '92', '192', '196', '209', '228', '234', '356', '360', '362', 'reactivity', '84', '97', '98', '101', '114', '167', '228', '229', 'monometallic', 'mechanism', '203', '204', 'morphology', '19', '20', '34', '37–39', '138', '334', '631', '766', '776', 'myoglobin', '548', '553', 'n', 'naturally', 'occurring', 'polyisoprenes', '356', '547', 'naturally', 'occurring', 'polymers', '430', '537–562', 'nature', 'light', '717–719', 'negative', 'coefﬁcient', 'expansion', '63', 'nematic', 'liquid', 'crystals', '741', '766', 'neoprene', 'rubber', 'see', 'chloroprene', 'rubber', 'network', 'polyesters', 'surface', 'coatings', '425–427', 'network', 'structure', '3', '473', '494', '748', 'newtonian', 'liquid', '28–31', '33', 'nitration', 'polystyrene', 'subsequent', 'reactions', 'polynitrostyrene', '603', 'nitrile', 'rubber', 'see', 'gr-n', 'rubber', 'nitroxide', 'mediated', 'radical', 'polymerization', '121–125', 'nomenclature', 'chain-growth', 'polymers', '7–11', 'nomenclature', 'step-growth', 'polymers', '11', 'nonadjacent', 'reentry', 'model', '37', 'norrish-smith', 'effect', 'see', 'autoacceleration', 'noryl', '458', 'novolacs', '480', '483', '487–491', '631', '749', 'nucleic', 'acids', '2', '557–561', 'nucleophilic', 'substitution', 'reactions', 'acrylic', 'methacrylic', 'polymers', '606–610', 'nucleoproteins', '548', '559', 'nucleoside', '558', '559', 'nucleotide', '558', '559', 'number', 'average', 'degree', 'polymerization', '41', '71', '95', '152–154', '407', '524', '1', '430', '2', '431', '3', '431', '4', '431–432', '5', '432', '6', '9', '11', '430', '432–436', '625', '630', '642', '656', '657', '676', '7', '434–436', '8', '435–436', '9', '434', '436', '10', '438–440', '630', '11', '18', '20', '436', '437', '630', '12', '436–437', '13', '437', 'deﬁnition', '7', 'o', 'occlusion', '(heterogeneous)', 'polymerization', '172–173', '380', 'oil', 'length', '426', 'oleﬁn', 'metathesis', '301', '306', '640', '699', 'oligomer', 'deﬁnition', '7', 'oligomers', 'terminal', 'functional', 'groups', '514–516', 'one', 'electron', 'transfer', 'initiation', 'anionic', 'chain-growth', 'polymerization', '182–191', 'one', 'electron', 'transposition', 'initiation', 'cationic', 'chain-growth', 'polymerization', '154', 'optical', 'activity', 'polymers', '60–61', 'organic', 'reactions', 'polymers', '567–682', 'orientation', 'polymers', '760', 'osmometry', '55', '56', 'oxidation', 'chain-growth', 'polymers', '663–666', 'oxidation', 'step-growth', 'polymers', '666–668', 'oxidative', 'coupling', '456', '458', '467', '545', 'oxidative', 'degradation', 'polymers', '663', 'oxycarbon', 'cation', '273', 'p', 'paired', 'group', 'neighboring', 'group', 'effects', 'reaction', 'polymers', '569–570', 'pectins', '537', '538', 'penultimate', 'unit', '86', '170', '195', '196', '206', '208', '224', '234', '264', '265', '293', 'pepsin', '553', 'permanent', 'dipoles', '18', 'peroxides', '69', '72–82', '122', '125', '126', '162', '318', '330', '351', '353', '364', '371', '373', '376', '380', '383', '384', '387', '388', '433', '436', '467', '468', '473', '474', '496', '545', '570', '573', '583', '590', '598', '616–619', '622', '623', '625', '626', '644', '663', '665–668', '676', '715', '725', 'perylene', '461', '519', 'phase-transfer', 'polymerization', 'see', 'interfacial', 'polymerization', 'phenol-formaldehyde', 'resins', '483–491', 'phenolic', 'polymers', '544', 'phenoxy', 'polymers', '459', 'phillips', 'catalysts', '220', '349', 'phosphazene', 'polymers', '501', '742', 'index', '797', 'phosphonitrilic', 'polymers', '500–501', 'photochemical', 'reactions', '717', '722', '723', '742', '749', '753', 'photochemical', 'syntheses', 'graft-copolymers', '625–626', 'photo-conducting', 'polymers', '767–775', 'photocrosslinkable', 'polymers', '735–750', 'photo-degradation', 'macromolecules', '668–674', 'photoinitiation', 'cationic', 'polymerization', '269', 'free-radical', 'polymerization', '75', '80', '584', '585', '625', 'photo-isomerization', 'polymeric', 'materials', '755–758', 'photonic', 'polymers', '717–732', 'photo-oxidative', 'degradation', '663', '674–676', 'photo-responsive', 'polymers', '750–766', 'photosensitizers', '259', '582', '707', '726', '732–735', '737', '740', '741', '744–746', '753', '767', '774', '779', 'planar', 'network', 'deﬁnition', '3', 'plant', 'gums', '538', 'plexiglas', 'see', 'poly(methyl', 'methacrylate)', 'polar', 'addition', 'reaction', 'polymers', '577–584', 'polar', 'additions', 'polymeric', 'materials', 'poly(4-methyl', 'pentene)', '345', 'poly(acrylic', 'acid)', '134', '381', '609', 'poly(butene-1)', '345', 'poly(butylene', 'terephthalate)', '420', '653', 'poly(ethylene', 'naphthanate)', 'poly(ethylene', 'terephthalate)', '9', '11', '20', '413', '418–421', '623', '625', '629', '631', '653', '662', '666', '668', 'poly(methyl', 'methacrylate)', '8', '22', '23', '87', '97', '114', '232', '374', '376', '377', '606–608', '618', '619', '637', '639', '647', '670', '677', '773', 'poly(p-xylene)', '668', 'poly(phenylene', 'sulﬁde)', '466', '711', 'poly(phenylene', 'vinylene)', '711', 'poly(vinyl', 'alcohol)', '19', '134', '351', '391–392', '570', '603', '610–612', '625', '650', '743', '749', '767', 'poly(vinyl', 'ﬂuoride)', '23', '383–384', '593', '767', 'poly(vinylidene', 'chloride)', '42', '593', 'poly(vinylidene', 'ﬂuoride)', '593', 'polyacetals', 'polyketals', '459–461', 'polyacetylene', '277', '711', '713', '770', '772', 'polyacrylamide', '381', '570', '609', '625', '627', '696', '703', '707', 'polyacrylonitrile', '4', '37', '42', '380', '587', '610', '627', '648', '670', 'polyalkanoates', '561–562', 'polyamide-imides', '447–449', 'polyamide-polyester', 'block', 'copolymers', '633', 'polyaniline', '627', '710', '711', '715–716', 'polyarylsiloxanes', '499', 'polyarylsulfones', 'poly(arylene', 'ether)s', 'polybenzimidazoles', '512', '517', 'polybenzoxazoles', '513', 'polybenzthiazoles', '512–514', 'polybutadiene', '7', '8', '19', '86', '352–356', '358', '363', '371', '572–576', '579', '582–584', '586', '620', '639', '645', '676', 'high', 'molecular', 'weight', '354–356', 'liquid', '353–354', '583', 'polycaprolactam', 'see', 'nylons', '8', 'polycarbonates', '303', '412', '415', '427–429', '511', '655', '668', '673', '674', '702', '745', 'polychlorotriﬂuoroethylene', '42', '383', '651', 'polydisperse', 'deﬁnition', '53', 'polyester-imides', '447–449', 'polyesters', '5', '11', '17', '20', '43', '228', '283', '409', '412–430', '447–449', '469', '472–474', '498', '503', '517', '522–524', '537', '561', '572', '577', '578', '582', '589', '625', '631–633', '653–656', '662', '673', '707', '730', '744', '751–753', '758', '764', '769', 'polyesters', 'lactones', '428–430', 'polyether', 'based', 'supports', '704', 'polyether', 'imides', '451', 'polyethers', '253', '269', '451', '456–459', '469', '472–474', '570', '633', '642', '661', '704', '765', '766', 'polyethylene', 'preparation', '329–335', 'related', 'polymers', '329–339', 'polyethynylpyridine', 'polyimides', '449–455', '501', '512', '514', '659–660', '742', '746', '761–763', '772', 'polyisobutylene', '8', '19', '177', '180', '343–344', '628', '630', '639', '677', '696', '701', '702', 'polyisoprene', 'natural', 'synthetic', '547', 'polymer(s)', 'acrylic', 'methacrylic', 'esters', '372–379', 'based', 'solar', 'cells', '775–782', 'deﬁnition', '3–7', 'designed', 'crosslink', 'upon', 'irradiation', 'laser', 'beams', '750', 'functional', 'chalcone', 'groups', '743–744', 'functional', 'furan', 'groups', '745–746', 'functional', 'groups', 'similar', 'cinnamate', '744–745', 'harvesting', 'sun0s', 'energy', '751–755', 'pendant', 'abietate', 'dibenzazepine', 'groups', 'pendant', 'azide', 'groups', '748–750', 'pendant', 'cinnamoyl', 'functional', 'groups', '738–743', 'pendant', 'maleimide', 'groups', '746', 'substituted', 'styrenes', '367–370', 'supports', 'reagents', 'catalysts', 'drug', 'release', '746–748', '695–710', 'crosslink', 'dimerization', 'nitrenes', 'combination', 'free-radicals', 'covalent', 'bonds', '748', 'photocrosslink', 'formation', 'cyclobutane', 'rings', '736–473', 'polymercaptans', '465–468', 'polymeric', 'catalysts', '707', 'polymeric', 'materials', 'special', 'applications', '695–784', 'polymeric', 'reagents', '695', '696', '710', 'polymerization', 'acrylic', 'methacrylic', 'esters', '194', '372–375', '619', 'aldehydes', '221', '223', '224', '243', 'charge-transfer', 'complexes', '108', '109', '111', '113', '142', '164', '166', '769', 'complexes', 'lewis', 'acids', '111–113', 'cyclic', 'acetals', '273–278', 'cyclic', 'amines', '283', '307–309', 'cyclic', 'sulﬁdes', '309–310', 'dialdehydes', '416', '798', 'polymerization', '(cont)', 'dioxolane', '255', '273', '276–278', 'dioxopane', '277–278', 'ketones', 'isocyanataes', '228', 'lactones', '11', '278–284', '306', '312', '415', '428', '632', '633', 'monomers', 'multiple', 'double', 'bonds', '90–92', 'n-carboxy-a-amino', 'acid', 'anhydrides', '297–301', '431', '554', 'oxepanes', '273', 'oxetanes', '255', '266–269', '273', 'oxiranes', '255–266', '281', 'precipitation', '120', '221', '299', '347', '381', 'solution', '133', '134', '240', '363', '377', '388', '412', '421', 'strain', 'relief', '175–176', 'strain', 'relief', 'cationic', 'isomerization', '175–176', 'tetrahydrofuran', '269–273', '637', 'trioxane', '275', 'unsaturated', 'aldehydes', '226–227', 'polymer', 'preparation', 'techniques', 'free-radical', 'mechanism', '132–139', 'free-radical', 'polymerization', '133', 'polymethacrylonitrile', '380', '381', '607', 'polymethylene', '10', '338', '339', 'polyoxymethylene', '36', '221', '222', '274', '461', '652', '661', 'polyphenylene', '43', '504–505', '522', '599', 'polyphosphazenes', '500', '501', 'polypropylene', '20', '36', '42', '203', '206', '207', '213', '214', '334', '339–343', '345', '347', '356', '591', '592', '618', '623', '627', '634', '635', '645', '652', '663', '664', '668', '669', '675', '677', '707', 'polypyrrole', '711', '713', 'poly(vinyl', 'acetate)', 'reactions', '390–391', '571', '652', 'poly(vinyl', 'chloride)', 'reactions', '386–389', '595', 'poly(a-oleﬁn)s', '345–346', '634–635', 'poly(alkylene', 'sulﬁde)s', '466', '468', 'poly(arylene', 'ether', 'ketone)s', '520–521', 'poly(ether', 'ketone)s', '520', 'poly(phenylene', 'oxide)s', '456–458', 'poly(vinyl', 'acetal)s', '391–392', '610', 'polysaccharides', '2', '537–546', '548', '625', '706', 'polysilanes', '499–500', '572', '629', 'polysiloxane', 'coating', 'resins', '498', 'polysiloxanes', '494–498', '512', '642', 'polystyrene', '3–5', '7', '8', '11', '19', '22', '23', '34', '40–42', '58', '59', '85', '122', '125', '134', '139', '191', '230', '363–371', '458', '524', '555', '561', '568', '570', '575', '597–604', '618–620', '622', '623', '627', '628', '630', '646–647', '664–666', '677', '696–704', '707', '708', '715', '728', '730', '748', '753', '760', '766', '773', '775', 'polystyrene', 'polystyrene', 'like', 'polymers', '364–370', '646–647', 'polysulﬁdes', '310', '463', '466–468', '614', '615', 'polysulfodiazoles', '514', 'polysulfones', '463–465', '599', '661', '668', 'polytetraﬂuoroethylene', '42', '382', '383', '385', '502', '651', '652', '677', 'polythiophene', '61', '711', '713', 'polyurethanes', 'elastomers', '473–474', '633', 'index', 'ﬁbers', '472–473', 'foams', '474', 'ionomers', '633–634', 'polyamide', 'block', 'copolymers', '633', 'polyester', 'block', 'copolymers', '633', 'popcorn', 'polymerization', '353', 'precipitation', 'polymerization', 'see', 'solution', 'polymerization', 'primary', 'structure', 'proteins', '548', '552', '559', 'processes', 'polymer', 'degradation', '643–652', 'promoter', 'peroxide', 'decomposition', '77', 'propagation', 'anionic', 'chain-growth', 'polymerization', '191–198', 'anionic', 'coordination', 'polymerization', '281', 'cationic', 'chain-growth', 'polymerization', '152', 'free-radical', 'chain', 'growth', 'polymerization', '69', '84–92', '151', 'reaction', 'polymerization', 'oxetanes', '268–269', 'reaction', 'polymerization', 'oxiranes', '258', '266', 'reaction', 'polymerization', 'tetrahydrofuran', 'properties', 'poly(a-oleﬁn)s', '345', 'prosthetic', 'group', '548', '553', 'proteins', '134', '430', '431', '483', '493', '537', '547–557', '559', '271–272', '696', '706', 'pseudo-anionic', 'polymerization', '185', 'pseudo-cationic', 'polymerization', '173–174', 'q', 'q', 'e', 'scheme', '99–100', 'quantum', 'mechanical', 'description', 'light', '719', 'quantum', 'yield', '672', '723', '725', '726', '734', '738', '740', '774', 'quasiliving', 'polymerization', '178', '179', 'quaternary', 'structure', 'proteins', '552', 'quiana', '439', 'quinone', 'methides', '485', '487', 'r', 'radiation', 'initiated', 'polymerizations', '166', 'radical', 'lifetime', '71', '87', '114', 'reactivity', '70', '98–100', 'scavenger', '(see', 'inhibitors', 'retarders)', 'radical-anion', '186', '188', '191', 'radical-cation', '78', '504', 'radical-chain', 'polymerization', 'see', 'free-radical', 'chain', 'polymerization', 'radius', 'gyration', '48–50', '318', 'random', 'copolymer', 'deﬁnition', '4', 'rate', 'disappearance', 'functional', 'groups', 'step-growth', 'polymerization', '405', 'rayon', '540–542', 'reactions', 'acrylic', 'methacrylic', 'related', 'polymers', '606', 'halomethylated', 'polymers', '599–602', 'isocyanates', '442', '470–471', '579', '594', 'index', '799', 'polystyrene', '598', '603', '604', 'favor', 'large', 'molecules', '571–572', 'reactivity', 'macromolecules', '567–572', 'reactivity', 'ratios', '97–99', '101', '121', '228', '229', '233', '348', 'second', 'order', 'markov', 'model', '237', 'second', 'order', 'transition', 'temperature', '22', 'semiladder', 'polymer', 'deﬁnition', '3', 'shear', '351', '375', '388', 'rearrangement', 'reactions', 'polymers', '458', '584–589', '609', 'redox', 'copolymerization', 'reactions', '315', 'redox', 'grafting', 'reactions', '624', '625', 'redox', 'initiation', '76', '380', '382', '383', '389', 'reduced', 'transition', 'metal', 'oxide', 'catalysts', 'support', 'creep', 'compliance', '31', 'deﬁnition', '28', 'rate', '29', '30', '32', '33', 'stress', '28–33', 'stress', 'relaxation', 'model', 'stress', 'relaxation', 'modulus', '31', 'thinning', '29', '30', '219–220', 'reduced', 'viscosity', '53', 'reduction', 'ester', 'groups', 'acrylic', 'methacrylic', 'polymers', '606–607', 'poly(nitrostyrene)', '603', 'regenerated', 'cellulose', '540–541', 'relative', 'viscosity', '53', 'repeat', 'unit', '3', '5', '9–12', '23', '40–42', '92', '189', '262', '302', '314', '430', '437', '439', '465', '476', '538', '561', '587', 'resoles', '486', '487', '491', 'retarders', '103–106', 'retractive', 'force', 'stretched', 'elastomer', '25', 'retrogration', 'starch', '539', 'reversible', 'addition-fragmentation', 'chain', 'transfer', 'polymerization', '126–129', 'rheology', 'viscoelasticity', '27–34', 'ribonucleic', 'acids', '(rna)', '537', '558–560', 'ring', 'forming', 'step-growth', 'polymerizations', '410–411', 'ring', 'forming', 'polymerization', 'free-radical', 'chain', 'growth', 'polymerization', '91–92', 'ring', 'opening', 'metathesis', 'polymerization', 'alicyclics', '304', '305', 'ring', 'opening', 'polymerization', 'aziridines', '307', '308', '311', 'cyclic', 'acetals', '273–278', 'cyclic', 'sulﬁdes', '309–310', 'dioxepanes', '317', '318', 'dioxolanes', '273', '276–278', '311', 'free-radical', 'mechanism', '317', 'lactams', '253', '284–297', '311', '312', '430', '433', 'lactones', '253', '278–284', '306', '311', '312', '415', 'n-carboxy-a-amino', 'acid', 'anhydrides', '297–301', '554', 'oxepanes', '273', 'oxetanes', '266–269', '273', '312', 'oxiranes', '255–266', '271', '281', '311', 'phosphonitrilic', 'chloride', '500', 'siloxanes', '495', '498', 'tetrahydrofuran', '269–273', '309', 'trioxane', '222', '274–276', '278', '318', '461', '629', 'rna', 'see', 'ribonucleic', 'acids', 'rotaxanes', '7', '524', 'rtv', '497', 's', 'sbr', 'rubber', 'see', 'gr-s', 'rubber', 'secondary', 'bond', 'forces', '17', '18', '25', '132', '502', 'secondary', 'structure', 'proteins', '551', 'shellac', '537', 'shish-kebab', 'morphology', '38–39', 'silicon', 'containing', 'aromatic', 'polymers', '511–512', 'silicone', 'elastomers', '496–497', 'silicone', 'oil', '712', 'silicone', 'polymers', '494–499', 'silk', '439', '547', '556', 'size', 'exclusion', 'chromatography', '54', '57', '174', '230', 'smetic', 'liquid', 'crystals', '47', 'smith-ewart', 'kinetics', '137', 'solution', 'polymerization', '133–134', '363', '377', '388', '412', '421', 'solutions', 'polymers', '48–51', '53', 'solvent', 'effects', 'copolymerization', '100–102', 'solvent-separated', 'ion', 'pair', '167', '201', 'space', 'network', 'deﬁnition', '3', 'special', 'catalysts', 'lactone', 'polymerization', '283–284', 'special', 'gels', 'drug', 'release', '704–705', 'special', 'polymers', 'dienes', '360', 'special', 'reactions', 'formation', 'polyamides', '441–443', 'preparation', 'block', 'copolymers', '639–641', 'special', 'type', 'living/controlled', 'polymerizations', 'speciﬁc', 'viscosity', '53', '759', 'spherulites', '38', '39', '41', 'spherulitic', 'growth', '38–43', 'spontaneous', 'alternating', 'zwitterion', 'copolymerizations', '129–130', '312–316', 'spontaneous', 'terminations', '152', '178', 'spontaneous', 'zwitterion', 'polymerization', '312', 'starch', '1', '2', '134', '412', '537–539', '623', '625', '628', '696', 'star-shaped', 'polymer', 'deﬁnition', '3', 'static', 'capillary', 'osmometer', '55', 'statistics', 'propagation', '234–239', 'steady', 'state', 'assumption', '70', '153', 'step-growth', 'polymers', 'deﬁnition', '5', 'stereoregular', 'polymer', 'deﬁnition', '170', 'stereoselective', 'polymerization', '304', 'steric', 'arrangement', 'macromolecules', '11–13', 'steric', 'arrangement', 'polymers', '11', '12', 'steric', 'control', 'anionic', 'chain-growth', 'polymerization', '192–194', 'cationic', 'chain-growth', 'polymerization', '169–173', 'coordination', 'polymerization', '211–218', 'free-radical', 'chain-growth', 'polymerization', '113–114', '800', 'index', 'steric', 'control', '(cont)', 'polymerization', 'conjugated', 'dienes', '209–211', 'polymerization', 'oxiranes', '264–266', 'steric', 'polar', 'resonance', 'effects', 'free-radical', 'polymerization', '84–87', 'structure', 'chemistry', 'proteins', '548–553', 'structure', 'property', 'relations', 'organic', 'polymers', '17–21', 'styrene-butadiene', 'block', 'copolymers', '636', '637', 'styrene-butadiene', 'rubber', '371', 'styrene-maleic', 'anhydride', 'copolymers', '370', '372', '749', 'substitution', 'reactions', 'poly(vinyl', 'alcohol)', '610–612', 'sulfonation', 'polystyrene', '603', 'sulfur', 'containing', 'polymers', '463–468', 'support', 'materials', 'based', 'polystyrene', '696–701', 'polystyrene', '701–704', 'suspension', 'polymerization', '134', '138', '364', '377', '383', '385', '388', '697', '704', 'switchboard', 'model', '37', 'syndiotactic', 'conformation', '11', 'syndiotactic', 'dyad', '239', 'syndiotactic', 'polypropylene', '207', '213', '339', '342–343', 'synthetic', 'methods', 'preparation', 'nucleic', 'acids', '560–561', 'polypeptides', '554–556', 'synthetic', 'polyisoprene', '356–358', '581', 't', 'tacticity', '11', '23', '87', '192', '195', '207', '235', '730', '751', 'tdi', 'see', 'toluene', 'diisocyanate', '(tdi)', 'techniques', 'polymer', 'preparations', 'free-radical', 'chain-growth', 'polymerizations', '132', 'step-growth', 'polymerization', '412', 'teﬂon', 'see', 'polytetraﬂuoroethylene', 'telechelic', 'polymer', 'deﬁnition', '7', 'telomer', 'deﬁnition', '7', 'telomerization', 'reaction', '94', 'template', 'polymerization', '114', 'tempo', 'see', '2266-tetramethylpiperydinyl-1-oxyl', '(tempo)', 'tensile', 'strength', '13', '26', '53', '344', '371', '421', '439', '520', '580', 'termination', 'reactions', 'anionic', 'polymerization', '198–200', 'cationic', 'polymerization', '177–178', 'coordination', 'polymerization', '219', 'free-radical', 'polymerization', '92', 'terpolymerization', '101–102', '111', '316', 'terylene', 'see', 'poly(ethylene', 'terephthalate)', '2266-tetramethylpiperydinyl-1-oxyl', '(tempo)', '122–124', '318', '636', '700', 'tg', 'see', 'glass', 'transition', 'temperature', 'thermal', 'degradation', 'azo', 'compounds', 'peroxides', '72–75', 'cellulosic', 'materials', '661–662', 'chlorocarbon', 'ﬂuorocarbon', 'polymers', '649–652', 'common', 'chain-growth', 'polymers', '643–644', 'common', 'step-growth', 'polymers', '652', 'epoxy', 'resins', '658–659', 'methacrylic', 'acrylic', 'polymers', '647–648', 'poly(vinyl', 'acetate)', '652', 'polyamides', '656–660', 'polyesters', '653–656', 'polyethers', '661', 'polyoleﬁns', 'polymers', 'conjugated', 'dienes', '644–646', 'polyoxides', '652', 'polyoxidiazoles', '659–660', 'polyquinoxaline', '659–660', 'polystyrene', 'polystyrene-like', 'polymers', '646–647', 'step-growth', 'polymers', '652', 'thermal', 'polymerization', '106–107', '112', '765', 'thermodynamics', 'anionic', 'polymerization', '201', 'cationic', 'polymerization', '181–182', 'crystallization', '40–41', 'elasticity', '25–27', 'equilibrium', 'polymerization', '240', 'free-radical', 'polymerization', '131–132', 'polymer', 'solutions', '50–51', 'ring', 'opening', 'polymerization', '318–319', 'step-growth', 'polymerization', '524', 'thermoplastic', 'thermoset', 'acrylic', 'resins', '376–379', 'thermoplastic', 'elastomer', '366', '474', 'thermoplastic', 'polymer', 'deﬁnition', '363', 'thermosetting', 'polymer', 'deﬁnition', '6', 'thermotropic', 'liquid', 'crystal', '45', 'thikol', 'rubber', 'thixotropic', '29', '30', 'threodiisotactic', 'conformation', '207', 'threodisyndiotactic', 'conformation', 'tight', 'ion', 'pair', 'see', 'ion', 'pair', 'tm', 'see', 'crystalline', 'melting', 'point', '(tm)', 'toluene', 'diisocyanate', '(tdi)', '469', '473', '639', 'torlon', 'see', 'polyamide-imides', 'trammsdorff', 'effect', '89', 'transanular', 'polymerization', '92', '175', 'transfer', 'see', 'chain', 'transfer', 'transport', 'polymerization', '461', 'tritactic', 'conformation', '11', 'trogamid', 't', '440', 'trypsin', '553', 'two', 'electron', 'transposition', 'initiation', 'reactions', '155–156', 'tybrene', 'see', 'abs', 'resins', 'u', 'ultrapek', 'see', 'poly(ether', 'ketone)s', 'ultrathene', 'see', 'ethylene-vinyl', 'acetate', 'copolymers', 'universal', 'calibration', 'method', '58', 'unsaturated', 'polyesters', '412', '424–425', '578', '589', 'uracil', '559', '737', 'index', '801', 'urea-formaldehyde', 'resins', '492–493', 'uscolite', 'see', 'abs', 'resins', 'v', 'viscoelasticity', '28', '34', 'viscose', 'rayon', '540', 'viscosity', 'average', 'molecular', 'weight', '29', '53', 'vulcanization', 'elastomers', '614–616', 'x', 'xanthate', 'process', '540', 'xylan', '537', '538', 'z', 'zero-order', 'markov', 'model', 'zielger-natta', 'catalysts', 'heterogeneous', '202–207', 'homogeneous', '207–209', 'w', 'weight', 'average', 'molecular', 'weight', '30', '52', '53', '56', '218', '338', '408', 'zimm', 'plot', '56', 'zwitterion', 'polymerization', 'see', 'spontaneous', 'zwitterion', 'polymerization']
pract_list2 = []
pract_list2.append(test_ravve_list)
pract_list.append(test_schaller_list)
bigger_list = test_ravve_list + test_schaller_list
print("ravve len ", len(pract_list2))
print("schaller len ", len(pract_list))
test_big = pract_list + pract_list2
print(type(bigger_list))
print(len(bigger_list))

ravve len  1
schaller len  1
<class 'list'>
216192


In [25]:
word2vec_book_model(test_big, window = 3)
#weird so putting each book into a spot on the list, like brings the vocab from 89 to 11000

INFO - 07:32:13: collecting all words and their counts
INFO - 07:32:13: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 07:32:14: collected 169119 word types from a corpus of 216192 words (unigram + bigrams) and 2 sentences
INFO - 07:32:14: using 169119 counts as vocab in Phrases<0 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 07:32:14: source_vocab length 169119
INFO - 07:32:15: Phraser built with 143 phrasegrams
INFO - 07:32:15: collecting all words and their counts
INFO - 07:32:15: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 07:32:16: collected 25435 word types from a corpus of 204481 raw words and 2 sentences
INFO - 07:32:16: Loading a fresh vocabulary
INFO - 07:32:16: effective_min_count=2 retains 11707 unique words (46% of original 25435, drops 13728)
INFO - 07:32:16: effective_min_count=2 leaves 190753 word corpus (93% of original 204481, drops 13728)
INFO - 07:32:16: deleting the raw counts dictionary of 25

Time to build vocab: 0.01 mins


INFO - 07:32:16: training model with 6 workers on 11707 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=0 window=3
INFO - 07:32:17: worker thread finished; awaiting finish of 5 more threads
INFO - 07:32:17: worker thread finished; awaiting finish of 4 more threads
INFO - 07:32:17: worker thread finished; awaiting finish of 3 more threads
INFO - 07:32:17: worker thread finished; awaiting finish of 2 more threads
INFO - 07:32:17: worker thread finished; awaiting finish of 1 more threads
INFO - 07:32:17: worker thread finished; awaiting finish of 0 more threads
INFO - 07:32:17: EPOCH - 1 : training on 204481 raw words (16469 effective words) took 0.5s, 33323 effective words/s
INFO - 07:32:17: worker thread finished; awaiting finish of 5 more threads
INFO - 07:32:17: worker thread finished; awaiting finish of 4 more threads
INFO - 07:32:17: worker thread finished; awaiting finish of 3 more threads
INFO - 07:32:17: worker thread finished; awaiting finish of 2 more threads

Time to train the model: 0.26 mins


INFO - 07:32:32: precomputing L2-norms of word weight vectors


the size of the vocabulary is:  11707
[('capillary', 0.21297767758369446), ('compound', 0.20629289746284485), ('(high', 0.19976487755775452), ('2081', 0.19321009516716003), ('0', 0.19296225905418396), ('dispersing', 0.1917995810508728), ('useful', 0.19144238531589508), ('sponsler', 0.18872597813606262), ('411', 0.18822352588176727), ('hiorns', 0.18742133677005768)]


C:\Users\bowri\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


cosine similarity between polymer and plastic:  0.015703153
size of the corpus_count: 2
